In [ ]:
import csv
from playwright.sync_api import sync_playwright

# 중간 데이터 저장용 리스트
character_info = []
skill_info = []
passive_info = []
buff_debuff_info = []
coin_effect_info = []

# 캐릭터 정보 크롤링
def crawl_character_info(base_url):
    with sync_playwright() as p:
        browser = p.chromium.launch()
        page = browser.new_page()
        page.goto(base_url)

        identities = page.query_selector_all("selector_for_identities")  # 캐릭터 목록 가져오기
        for identity in identities:
            character = {
                "Character_ID": int(identity.query_selector("selector_for_id").inner_text()),
                "Name": identity.query_selector("selector_for_name").inner_text(),
                "Release_Date": identity.query_selector("selector_for_release_date").inner_text(),
                "Atk_Type_HIT": identity.query_selector("selector_for_hit_attack").inner_text(),
                "Atk_Type_PENETRATE": identity.query_selector("selector_for_penetrate_attack").inner_text(),
                "Atk_Type_SLASH": identity.query_selector("selector_for_slash_attack").inner_text(),
                "Stagger_Threshold_One": identity.query_selector("selector_for_stagger_1").inner_text(),
                "Stagger_Threshold_Two": identity.query_selector("selector_for_stagger_2").inner_text(),
                "Stagger_Threshold_Three": identity.query_selector("selector_for_stagger_3").inner_text()
            }
            character_info.append(character)

        browser.close()

# 스킬 정보 크롤링
def crawl_skill_info():
    with sync_playwright() as p:
        browser = p.chromium.launch()
        page = browser.new_page()

        # 각 캐릭터의 스킬 정보 크롤링
        for character in character_info:
            page.goto(f"https://limbus.kusoge.xyz/{character['Character_ID']}")  # 캐릭터 페이지

            skills = page.query_selector_all("selector_for_skills")  # 스킬 정보 가져오기
            for skill in skills:
                skill_info.append({
                    "Skill_ID": int(skill.query_selector("selector_for_skill_id").inner_text()),
                    "Character_ID": character["Character_ID"],
                    "Sin_Affinity": skill.query_selector("selector_for_sin_affinity").inner_text(),
                    "Skill_Name": skill.query_selector("selector_for_skill_name").inner_text(),
                    "Skill_Type": skill.query_selector("selector_for_skill_type").inner_text(),
                    "Skill_Power": int(skill.query_selector("selector_for_skill_power").inner_text()),
                    "Coin_Count": int(skill.query_selector("selector_for_coin_count").inner_text())
                })

        browser.close()

# 패시브 정보 크롤링
def crawl_passive_info():
    with sync_playwright() as p:
        browser = p.chromium.launch()
        page = browser.new_page()

        for character in character_info:
            page.goto(f"https://limbus.kusoge.xyz/{character['Character_ID']}")  # 캐릭터 페이지
            passives = page.query_selector_all("selector_for_passive")  # 패시브 정보 가져오기
            for passive in passives:
                passive_info.append({
                    "Passive_ID": int(passive.query_selector("selector_for_passive_id").inner_text()),
                    "Character_ID": character["Character_ID"],
                    "Passive_Type": passive.query_selector("selector_for_passive_type").inner_text(),
                    "Effect": passive.query_selector("selector_for_effect").inner_text()
                })

        browser.close()

# 버프/디버프 정보 크롤링
def crawl_buff_debuff_info():
    with sync_playwright() as p:
        browser = p.chromium.launch()
        page = browser.new_page()

        for character in character_info:
            page.goto(f"https://limbus.kusoge.xyz/{character['Character_ID']}")  # 캐릭터 페이지
            effects = page.query_selector_all("selector_for_effects")  # 버프/디버프 정보 가져오기
            for effect in effects:
                buff_debuff_info.append({
                    "Buff_Debuff_ID": int(effect.query_selector("selector_for_buff_debuff_id").inner_text()),
                    "Name": effect.query_selector("selector_for_buff_debuff_name").inner_text(),
                    "Effect_Description": effect.query_selector("selector_for_buff_debuff_description").inner_text()
                })

        browser.close()

# 동전 효과 크롤링
def crawl_coin_effect():
    with sync_playwright() as p:
        browser = p.chromium.launch()
        page = browser.new_page()

        for skill in skill_info:
            page.goto(f"https://limbus.kusoge.xyz/skill/{skill['Skill_ID']}")  # 스킬 페이지
            coin_effects = page.query_selector_all("selector_for_coin_effects")  # 동전 효과 가져오기
            for coin_effect in coin_effects:
                coin_effect_info.append({
                    "Skill_ID": skill["Skill_ID"],
                    "Coin_Number": int(coin_effect.query_selector("selector_for_coin_number").inner_text()),
                    "Coin_Effect": coin_effect.query_selector("selector_for_coin_effect_description").inner_text()
                })

        browser.close()

# CSV로 저장하는 함수
def save_to_csv(filename, data, fieldnames):
    with open(filename, 'w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(data)

# 메인 함수
def main():
    base_url = "https://limbus.kusoge.xyz/identity"
    crawl_character_info(base_url)
    crawl_skill_info()
    crawl_passive_info()
    crawl_buff_debuff_info()
    crawl_coin_effect()

    # 각 테이블을 CSV로 저장
    save_to_csv('character_info.csv', character_info, ['Character_ID', 'Name', 'Release_Date', 'Atk_Type_HIT', 'Atk_Type_PENETRATE', 'Atk_Type_SLASH', 'Stagger_Threshold_One', 'Stagger_Threshold_Two', 'Stagger_Threshold_Three'])
    save_to_csv('skill_info.csv', skill_info, ['Skill_ID', 'Character_ID', 'Sin_Affinity', 'Skill_Name', 'Skill_Type', 'Skill_Power', 'Coin_Count'])
    save_to_csv('passive_info.csv', passive_info, ['Passive_ID', 'Character_ID', 'Passive_Type', 'Effect'])
    save_to_csv('buff_debuff_info.csv', buff_debuff_info, ['Buff_Debuff_ID', 'Name', 'Effect_Description'])
    save_to_csv('coin_effect_info.csv', coin_effect_info, ['Skill_ID', 'Coin_Number', 'Coin_Effect'])

# 실행
main()

In [ ]:
import csv
from playwright.sync_api import sync_playwright

# 1. 외부 변수 선언 (캐릭터, 스킬, 동전 효과 등 데이터를 저장할 변수들)
character_info_list = []
skill_info_list = []
coin_effect_list = []

# 2. 크롤링 함수 정의
def crawl_character_data(page, character_url):
    # 크롤링하여 데이터를 변수에 저장
    page.goto(character_url)
    
    character_info = {
        "Character_ID": 1,  # 예시로 1로 설정
        "Name": page.query_selector("h1.character-name").inner_text(),
        "Release_Date": page.query_selector(".release-date").inner_text(),
        "Atk_Type_HIT": page.query_selector(".atk-type-hit").inner_text(),
        "Atk_Type_PENETRATE": page.query_selector(".atk-type-penetrate").inner_text(),
        "Atk_Type_SLASH": page.query_selector(".atk-type-slash").inner_text(),
        "Stagger_Threshold_One": page.query_selector(".stagger-threshold-one").inner_text(),
        "Stagger_Threshold_Two": page.query_selector(".stagger-threshold-two").inner_text(),
        "Stagger_Threshold_Three": page.query_selector(".stagger-threshold-three").inner_text()
    }
    
    # 외부 변수에 캐릭터 정보 저장
    character_info_list.append(character_info)

    # 스킬 정보 크롤링 (예시)
    skill_info = {
        "Skill_ID": 1,  # 예시로 1로 설정
        "Character_ID": 1,
        "Sin_Affinity": page.query_selector(".sin-affinity").inner_text(),
        "Skill_Name": page.query_selector(".skill-name").inner_text(),
        "Skill_Type": page.query_selector(".skill-type").inner_text(),
        "Skill_Power": int(page.query_selector(".skill-power").inner_text()),
        "Coin_Count": int(page.query_selector(".coin-count").inner_text())
    }
    
    # 외부 변수에 스킬 정보 저장
    skill_info_list.append(skill_info)

    # 동전 효과 크롤링 (예시)
    coin_effect = {
        "Skill_ID": 1,
        "Coin_Number": 1,  # 예시로 1로 설정
        "Coin_Effect": page.query_selector(".coin-effect").inner_text()
    }

    # 외부 변수에 동전 효과 저장
    coin_effect_list.append(coin_effect)

# 3. 크롤링 진행 (웹페이지 크롤링)
with sync_playwright() as p:
    browser = p.chromium.launch()
    page = browser.new_page()

    # 예시 URL로 캐릭터 정보 크롤링
    crawl_character_data(page, "https://limbus.kusoge.xyz/identity/character1")

    browser.close()

# 4. 크롤링이 끝난 후 CSV에 저장
# 캐릭터 정보 저장
with open('Character_Info.csv', mode='w', newline='') as file:
    fieldnames = ["Character_ID", "Name", "Release_Date", "Atk_Type_HIT", "Atk_Type_PENETRATE", "Atk_Type_SLASH", "Stagger_Threshold_One", "Stagger_Threshold_Two", "Stagger_Threshold_Three"]
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    for character_info in character_info_list:
        writer.writerow(character_info)

# 스킬 정보 저장
with open('Skill_Info.csv', mode='w', newline='') as file:
    fieldnames = ["Skill_ID", "Character_ID", "Sin_Affinity", "Skill_Name", "Skill_Type", "Skill_Power", "Coin_Count"]
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    for skill_info in skill_info_list:
        writer.writerow(skill_info)

# 동전 효과 저장
with open('Coin_Effect.csv', mode='w', newline='') as file:
    fieldnames = ["Skill_ID", "Coin_Number", "Coin_Effect"]
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    for coin_effect in coin_effect_list:
        writer.writerow(coin_effect)


In [ ]:
import csv
from playwright.async_api import async_playwright

# 중간 데이터 저장용 리스트
character_info = []
skill_info = []
passive_info = []
buff_debuff_info = []
coin_effect_info = []

async def get_element_attribute(element, selector, attribute):
    """ 지정된 요소에서 특정 속성 값을 추출하는 함수 """
    try:
        target_element = await element.query_selector(selector)
        if target_element:
            return await target_element.get_attribute(attribute)
    except Exception as e:
        print(f"Error getting element attribute: {e}")
    return None

async def handle_request(route, request):
    # 이미지, 스타일시트, 폰트, 스크립트 차단
    if request.resource_type in ["image", "font"]:
        #print(f"차단된 리소스: {request.url}")
        await route.abort()
    else:
        await route.continue_()

# 캐릭터 정보 크롤링
def crawl_character_info(base_url):
    with sync_playwright() as p:
        browser = p.chromium.launch()
        page = browser.new_page()
        await page.route("**/*", handle_request)
        page.goto(base_url)
        
        identities = page.query_selector_all("selector_for_identities")  # 캐릭터 목록 가져오기
        for identity in identities:
            character = {
                "Character_ID": int(identity.query_selector("selector_for_id").inner_text()),
                "Name": identity.query_selector("selector_for_name").inner_text(),
                "Release_Date": identity.query_selector("selector_for_release_date").inner_text(),
                "Atk_Type_HIT": identity.query_selector("selector_for_hit_attack").inner_text(),
                "Atk_Type_PENETRATE": identity.query_selector("selector_for_penetrate_attack").inner_text(),
                "Atk_Type_SLASH": identity.query_selector("selector_for_slash_attack").inner_text(),
                "Stagger_Threshold_One": identity.query_selector("selector_for_stagger_1").inner_text(),
                "Stagger_Threshold_Two": identity.query_selector("selector_for_stagger_2").inner_text(),
                "Stagger_Threshold_Three": identity.query_selector("selector_for_stagger_3").inner_text()
            }
            character_info.append(character)

        browser.close()

async def extract_rarity(row):
    """ 희귀도 추출 함수 """
    try:
        print("Attempting to locate rarity...")

        # <a> 태그의 부모 <td>로 이동
        parent_td = await row.query_selector('a[href^="/identity/"]')
        if parent_td:
            # 부모 td에서 형제로 이동
            parent = await parent_td.evaluate_handle('(node) => node.parentElement')

            # 첫 번째 형제 td로 이동 (희귀도 이미지가 있는 <td>)
            sibling = await parent.evaluate_handle('node => node.nextElementSibling')

            if sibling:
                # 희귀도 이미지 찾기
                rarity_img_src = await get_element_attribute(sibling, 'img[src^="https://img.kusoge.xyz/limbus/img/ui/rarity_"]', 'src')
                if rarity_img_src:
                    rarity_level = rarity_img_src.split('_')[-1].split('.')[0]  # rarity_1.png -> 1
                    print(f"Extracted rarity level: {rarity_level}")
                    return rarity_level

        print("Rarity image not found.")
    except Exception as e:
        print(f"Error extracting rarity: {e}")
    
    return "Unknown"

resource_mapping = {
    "AZURE": "Gloom",
    "VIOLET": "Envy",
    "AMBER": "Sloth",
    "SHAMROCK": "Gluttony",
    "CRIMSON": "Wrath",
    "SCARLET": "Lust"
}

def convert_resource(resource_name):
    """리소스 이름을 변환하는 함수"""
    return resource_mapping.get(resource_name, resource_name)  # 매핑이 없으면 원래 값을 그대로 반환

async def extract_atk_and_resource(row, skill_number):
    """ 공격 타입(ATK Type)과 자원(Attribute) 추출 함수 - 스킬 번호에 따라 처리 """
    try:
        print(f"Attempting to locate skill{skill_number} type and resource...")

        # <a> 태그의 부모 <td>로 이동
        parent_td = await row.query_selector('a[href^="/identity/"]')
        if parent_td:
            # 부모 td에서 형제로 이동
            parent = await parent_td.evaluate_handle('(node) => node.parentElement')

            # 해당 스킬 번호에 맞는 형제로 이동
            sibling = parent
            for _ in range(skill_number + 1):  # 스킬 번호에 맞게 이동
                sibling = await sibling.evaluate_handle('node => node.nextElementSibling')

            atk_type, resource = "Unknown", "Unknown"

            if sibling:
                # 공격 타입 이미지 (atk_type) 찾기
                atk_type_img_src = await get_element_attribute(sibling, 'img[src^="https://img.kusoge.xyz/limbus/img/ui/atk_type_"]', 'src')
                if atk_type_img_src:
                    atk_type = atk_type_img_src.split('_')[-1].split('.')[0]  # atk_type_SLASH.png -> SLASH
                    print(f"Extracted attack type: {atk_type}")

                # 자원 이미지 (resource) 찾기
                resource_img_src = await get_element_attribute(sibling, 'div.q-py-xs img[src^="https://img.kusoge.xyz/limbus/img/ui/attr_"]', 'src')
                if resource_img_src:
                    resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                    print(f"Extracted resource: {resource}")
                    
                    # 리소스를 변환
                    resource = convert_resource(resource)
                    print(f"Converted resource: {resource}")

            return {f'skill{skill_number}_atk_type': atk_type, f'skill{skill_number}_resource': resource}
        
    except Exception as e:
        print(f"Error extracting skill{skill_number} type and resource: {e}")
    
    return {f'skill{skill_number}_atk_type': "Unknown", f'skill{skill_number}_resource': "Unknown"}

async def extract_defense_and_resource(row):
    """ 방어 타입(Def Type)과 자원(Attribute) 추출 함수 """
    try:
        print("Attempting to locate defense type and resource...")

        # <a> 태그의 부모 <td>로 이동
        parent_td = await row.query_selector('a[href^="/identity/"]')
        if parent_td:
            # 부모 td에서 형제로 이동
            parent = await parent_td.evaluate_handle('(node) => node.parentElement')

            # 다섯 번째 형제 td로 이동 (방어 관련 정보가 있는 <td>)
            sibling = parent
            for _ in range(5):  # 방어 정보는 5번째 형제에 있음
                sibling = await sibling.evaluate_handle('node => node.nextElementSibling')

            def_type, def_resource = "Unknown", "Unknown"

            if sibling:
                # 방어 타입 이미지 (def_type) 찾기
                def_type_img_src = await get_element_attribute(sibling, 'img[src^="https://img.kusoge.xyz/limbus/img/ui/def_type_"]', 'src')
                if def_type_img_src:
                    def_type = def_type_img_src.split('_')[-1].split('.')[0]  # def_type_GUARD.png -> GUARD
                    print(f"Extracted defense type: {def_type}")

                # 자원 이미지 (resource) 찾기
                resource_img_src = await get_element_attribute(sibling, 'div.q-py-xs img[src^="https://img.kusoge.xyz/limbus/img/ui/attr_"]', 'src')
                if resource_img_src:
                    def_resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                    print(f"Extracted defense resource: {def_resource}")
                    
                    # 방어 리소스를 변환
                    def_resource = convert_resource(def_resource)
                    print(f"Converted defense resource: {def_resource}")

            return {'def_type': def_type, 'def_resource': def_resource}
        
    except Exception as e:
        print(f"Error extracting defense type and resource: {e}")
    
    return {'def_type': "Unknown", 'def_resource': "Unknown"}


async def scrape_character_name_and_url(row):
    """ 캐릭터 이름과 URL 추출 함수 """
    try:
        # 캐릭터 이름 추출
        character_name = await row.query_selector('a[href^="/identity/"]')
        name = await character_name.inner_text() if character_name else "Unknown"

        # 캐릭터 URL 추출
        character_url = await character_name.get_attribute('href') if character_name else ""
        full_url = f"https://limbus.kusoge.xyz{character_url}" if character_url else ""

        return {'name': name.strip(), 'url': full_url}

    except Exception as e:
        print(f"Error extracting character name and URL: {e}")
        return None

async def handle_request_interception(route, request):
    """ 이미지 요청을 차단하는 함수 """
    if "image" in request.resource_type:
        print(f"Blocking image request: {url}")
        await route.abort()  # 이미지 요청 차단
    else:
        await route.continue_()  # 다른 요청은 그대로 진행

async def scrape_character_list(page):

    # 이미지 요청 차단
    page.on("route", handle_request_interception)
    
    """ 캐릭터 리스트에서 정보 추출 함수 """
    await page.goto("https://limbus.kusoge.xyz/identity")
    
    character_rows = await page.query_selector_all('tr')
    if len(character_rows) > 1:
        first_row = character_rows[1]
        
        # 캐릭터 이름과 URL을 추출
        character_info = await scrape_character_name_and_url(first_row)
        if character_info:
            character_info['rarity'] = await extract_rarity(first_row)  # 희귀도 추출
            for i in range(1, 4):  # 스킬1~3 정보 추출
                skill_and_resource = await extract_atk_and_resource(first_row, i)
                character_info.update(skill_and_resource)  # 스킬 정보 추가
            defense_and_resource = await extract_defense_and_resource(first_row)  # 방어 정보 추출
            character_info.update(defense_and_resource)  # 방어 정보 추가
            return character_info
    return None

# 메인 함수
def main():
    base_url = "https://limbus.kusoge.xyz/identity"
    crawl_character_info(base_url)
    #crawl_skill_info()
    #crawl_passive_info()
    #crawl_buff_debuff_info()
    #crawl_coin_effect()

    # 각 테이블을 CSV로 저장
    #save_to_csv('character_info.csv', character_info, ['Character_ID', 'Name', 'Release_Date', 'Atk_Type_HIT', 'Atk_Type_PENETRATE', 'Atk_Type_SLASH', 'Stagger_Threshold_One', 'Stagger_Threshold_Two', 'Stagger_Threshold_Three'])
    #save_to_csv('skill_info.csv', skill_info, ['Skill_ID', 'Character_ID', 'Sin_Affinity', 'Skill_Name', 'Skill_Type', 'Skill_Power', 'Coin_Count'])
    #save_to_csv('passive_info.csv', passive_info, ['Passive_ID', 'Character_ID', 'Passive_Type', 'Effect'])
    #save_to_csv('buff_debuff_info.csv', buff_debuff_info, ['Buff_Debuff_ID', 'Name', 'Effect_Description'])
    #save_to_csv('coin_effect_info.csv', coin_effect_info, ['Skill_ID', 'Coin_Number', 'Coin_Effect'])

# 실행
await main()
'''
async def main():
    """ 메인 크롤링 함수 """
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.route("**/*", handle_request)
        character_info = await scrape_character_list(page)
        if character_info:
            print("Character Info:")
            print(f"Name: {character_info['name']}")
            print(f"URL: {character_info['url']}")
            print(f"Rarity: {character_info['rarity']} 단계")
            print(f"Skill1 Attack Type: {character_info['skill1_atk_type']}")
            print(f"Skill1 Resource: {character_info['skill1_resource']}")
            print(f"Skill2 Attack Type: {character_info['skill2_atk_type']}")
            print(f"Skill2 Resource: {character_info['skill2_resource']}")
            print(f"Skill3 Attack Type: {character_info['skill3_atk_type']}")
            print(f"Skill3 Resource: {character_info['skill3_resource']}")
            print(f"Defense Type: {character_info['def_type']}")
            print(f"Defense Resource: {character_info['def_resource']}")
        else:
            print("No characters found.")
        
        await browser.close()

# Jupyter 환경에서 실행
await main()
'''

In [ ]:
def method_a():
    # 메소드 A에서 변수 a 생성
    a = {"A": "value_from_A"}
    return a

def method_b():
    # 메소드 B에서 변수 b 생성
    b = {"B": "value_from_B"}
    return b

def method_c():
    # 메소드 C에서 변수 c 생성
    c = {"C": "value_from_C"}
    return c

def merge_methods():
    # 결과를 한 곳에서 병합
    result = {}
    result.update(method_a())  # method_a의 결과 병합
    result.update(method_b())  # method_b의 결과 병합
    result.update(method_c())  # method_c의 결과 병합
    return result

# 최종 결과
final_result = merge_methods()
print(final_result)


result.update({ "Skill_ID": 1, "Character_ID": 1, "Sin_Affinity": page.query_selector(".sin-affinity").inner_text(), "Skill_Name": page.query_selector(".skill-name").inner_text()})
result.update({“Skill_Type": page.query_selector(".skill-type").inner_text(), "Skill_Power": int(page.query_selector(".skill-power").inner_text()), "Coin_Count": int(page.query_selector(".coin-count").inner_text()) }

In [ ]:
import csv
from playwright.sync_api import sync_playwright

# Identity 객체 정의
class Identity:
    def __init__(self, identity_id, name, rarity):
        self.identity_id = identity_id
        self.name = name
        self.rarity = rarity
        self.skills = []  # Skill 객체 리스트
        self.level_stats = []  # LevelStat 객체 리스트
        self.attack_types = {}
        self.stagger_thresholds = {}
        self.coin_effect = []  # CoinEffect 객체 리스트
        self.buff_debuff_info = []  # BuffDebuffInfo 객체 리스트
        self.passive_info = []  # PassiveInfo 객체 리스트

    def update_attack_types(self, hit, penetrate, slash):
        self.attack_types = {"HIT": hit, "PENETRATE": penetrate, "SLASH": slash}

    def update_stagger_thresholds(self, one, two, three):
        self.stagger_thresholds = {"One": one, "Two": two, "Three": three}

    def add_coin_effect(self, effect_type, value):
        self.coin_effect.append({"Effect_Type": effect_type, "Value": value})

    def add_buff_debuff_info(self, effect_type, target, value):
        self.buff_debuff_info.append({"Effect_Type": effect_type, "Target": target, "Value": value})

    def add_passive_info(self, passive_name, description):
        self.passive_info.append({"Passive_Name": passive_name, "Description": description})

# LevelStat 객체 정의
class LevelStat:
    def __init__(self, level, hp, speed, defense):
        self.level = level
        self.hp = hp
        self.speed = speed
        self.defense = defense

# Skill 객체 정의
class Skill:
    def __init__(self, skill_id, skill_type, sin_affinity, name, power, coin_count):
        self.skill_id = skill_id
        self.skill_type = skill_type
        self.sin_affinity = sin_affinity
        self.name = name
        self.power = power
        self.coin_count = coin_count

# CoinEffect 객체 정의
class CoinEffect:
    def __init__(self, uptie, coin_number, coin_effect):
        self.uptie = uptie
        self.coin_number = coin_number
        self.coin_effect = coin_effect

# BuffDebuffInfo 객체 정의
class BuffDebuffInfo:
    def __init__(self, name, effect_description, dispellable):
        self.name = name
        self.effect_description = effect_description
        self.dispellable = dispellable

# PassiveInfo 객체 정의
class PassiveInfo:
    def __init__(self, passive_type, passive_coindition, passive_effect):
        self.passive_type = passive_type
        self.passive_coindition = passive_coindition
        self.passive_effect = passive_effect

# Identity 크롤링 함수
def crawl_identity_page(page):
    identity_elements = page.query_selector_all(".identity-item")
    identities = []

    for element in identity_elements:
        identity_id = element.get_attribute("data-id")
        name = element.query_selector(".identity-name").inner_text()
        rarity = int(element.query_selector(".identity-rarity").inner_text())

        # Identity 객체 생성 및 추가
        identity = Identity(identity_id, name, rarity)

        # Skill 정보 저장
        skills = element.query_selector_all(".skill-item")
        for skill in skills:
            skill_id = skill.get_attribute("data-skill-id")
            skill_type = skill.query_selector(".skill-type").inner_text()
            sin_affinity = skill.query_selector(".sin-affinity").inner_text()
            identity.skills.append(Skill(skill_id, skill_type, sin_affinity))

        identities.append(identity)

    return identities

# 상세 페이지 크롤링 함수
def crawl_identity_details(page, identity):
    page.goto(f"https://limbus.kusoge.xyz/identity/{identity.identity_id}")

    # Attack Types 업데이트
    atk_hit = page.query_selector(".atk-hit").inner_text()
    atk_penetrate = page.query_selector(".atk-penetrate").inner_text()
    atk_slash = page.query_selector(".atk-slash").inner_text()
    identity.update_attack_types(atk_hit, atk_penetrate, atk_slash)

    # Stagger Thresholds 업데이트
    stagger_one = page.query_selector(".stagger-one").inner_text()
    stagger_two = page.query_selector(".stagger-two").inner_text()
    stagger_three = page.query_selector(".stagger-three").inner_text()
    identity.update_stagger_thresholds(stagger_one, stagger_two, stagger_three)

    # Level Stats 저장
    level_stats_elements = page.query_selector_all(".level-stats")
    for level_stat in level_stats_elements:
        level = int(level_stat.query_selector(".level").inner_text())
        hp = int(level_stat.query_selector(".hp").inner_text())
        speed = int(level_stat.query_selector(".speed").inner_text())
        defense = int(level_stat.query_selector(".defense").inner_text())
        identity.level_stats.append(LevelStat(level, hp, speed, defense))

    # Coin Effect, Buff/Debuff, Passive 정보 추가 (가상의 예시로 추가)
    identity.add_coin_effect("Increase Attack", 10)
    identity.add_buff_debuff_info("Buff", "Ally", 20)
    identity.add_passive_info("Resilience", "Increases defense by 15%")

# CSV 저장 함수
def save_to_csv(identities):
    # Identity_Info 저장
    with open("Identity_Info.csv", mode="w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(["Identity_ID", "Name", "Rarity", "Atk_HIT", "Atk_PENETRATE", "Atk_SLASH", "Stagger_One", "Stagger_Two", "Stagger_Three"])
        for identity in identities:
            writer.writerow([
                identity.identity_id, identity.name, identity.rarity,
                identity.attack_types.get("HIT"), identity.attack_types.get("PENETRATE"), identity.attack_types.get("SLASH"),
                identity.stagger_thresholds.get("One"), identity.stagger_thresholds.get("Two"), identity.stagger_thresholds.get("Three")
            ])

    # Skill_Info 저장
    with open("Skill_Info.csv", mode="w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(["Skill_ID", "Identity_ID", "Skill_Type", "Sin_Affinity"])
        for identity in identities:
            for skill in identity.skills:
                writer.writerow([skill.skill_id, identity.identity_id, skill.skill_type, skill.sin_affinity])

    # Level_Based_Stats 저장
    with open("Level_Based_Stats.csv", mode="w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(["Identity_ID", "Level", "HP", "Speed", "Defense"])
        for identity in identities:
            for level_stat in identity.level_stats:
                writer.writerow([identity.identity_id, level_stat.level, level_stat.hp, level_stat.speed, level_stat.defense])

    # Coin_Effect 저장
    with open("Coin_Effect.csv", mode="w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(["Identity_ID", "Effect_Type", "Value"])
        for identity in identities:
            for effect in identity.coin_effect:
                writer.writerow([identity.identity_id, effect["Effect_Type"], effect["Value"]])

    # Buff_Debuff_Info 저장
    with open("Buff_Debuff_Info.csv", mode="w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(["Identity_ID", "Effect_Type", "Target", "Value"])
        for identity in identities:
            for info in identity.buff_debuff_info:
                writer.writerow([identity.identity_id, info["Effect_Type"], info["Target"], info["Value"]])

    # Passive_Info 저장
    with open("Passive_Info.csv", mode="w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(["Identity_ID", "Passive_Name", "Description"])
        for identity in identities:
            for passive in identity.passive_info:
                writer.writerow([identity.identity_id, passive["Passive_Name"], passive["Description"]])

# Playwright 크롤러
def main():
    identities = []  # Identity 객체 리스트

    with sync_playwright() as p:
        browser = p.chromium.launch(headless=True)
        page = browser.new_page()

        # Step 1: /identity 페이지 크롤링
        identities = crawl_identity_page(page)

        # Step 2: /identity/{Identity_ID} 상세 페이지 크롤링
        for identity in identities:
            crawl_identity_details(page, identity)

        browser.close()

    # Step 3: 객체 데이터를 CSV로 저장
    save_to_csv(identities)

# 실행
await main()

In [ ]:
from playwright.async_api import async_playwright

async def handle_request(route, request):
    # 이미지, 스타일시트, 폰트, 스크립트 차단
    if request.resource_type in ["image", "stylesheet", "font"]:
        #print(f"차단된 리소스: {request.url}")
        await route.abort()
    else:
        await route.continue_()
        
async def get_element_attribute(element, selector, attribute):
    """ 지정된 요소에서 특정 속성 값을 추출하는 함수 """
    try:
        target_element = await element.query_selector(selector)
        if target_element:
            return await target_element.get_attribute(attribute)
    except Exception as e:
        print(f"Error getting element attribute: {e}")
    return None

async def extract_rarity(row):
    """ 희귀도 추출 함수 """
    try:
        print("Attempting to locate rarity...")

        # <a> 태그의 부모 <td>로 이동
        parent_td = await row.query_selector('a[href^="/identity/"]')
        if parent_td:
            # 부모 td에서 형제로 이동
            parent = await parent_td.evaluate_handle('(node) => node.parentElement')

            # 첫 번째 형제 td로 이동 (희귀도 이미지가 있는 <td>)
            sibling = await parent.evaluate_handle('node => node.nextElementSibling')

            if sibling:
                # 희귀도 이미지 찾기
                rarity_img_src = await get_element_attribute(sibling, 'img[src^="https://img.kusoge.xyz/limbus/img/ui/rarity_"]', 'src')
                if rarity_img_src:
                    rarity_level = rarity_img_src.split('_')[-1].split('.')[0]  # rarity_1.png -> 1
                    print(f"Extracted rarity level: {rarity_level}")
                    return rarity_level

        print("Rarity image not found.")
    except Exception as e:
        print(f"Error extracting rarity: {e}")
    
    return "Unknown"

resource_mapping = {
    "AZURE": "Gloom",
    "VIOLET": "Envy",
    "AMBER": "Sloth",
    "SHAMROCK": "Gluttony",
    "CRIMSON": "Wrath",
    "SCARLET": "Lust"
}

def convert_resource(resource_name):
    """리소스 이름을 변환하는 함수"""
    return resource_mapping.get(resource_name, resource_name)  # 매핑이 없으면 원래 값을 그대로 반환

async def extract_atk_and_resource(row, skill_number):
    """ 공격 타입(ATK Type)과 자원(Attribute) 추출 함수 - 스킬 번호에 따라 처리 """
    try:
        print(f"Attempting to locate skill{skill_number} type and resource...")

        # <a> 태그의 부모 <td>로 이동
        parent_td = await row.query_selector('a[href^="/identity/"]')
        if parent_td:
            # 부모 td에서 형제로 이동
            parent = await parent_td.evaluate_handle('(node) => node.parentElement')

            # 해당 스킬 번호에 맞는 형제로 이동
            sibling = parent
            for _ in range(skill_number + 1):  # 스킬 번호에 맞게 이동
                sibling = await sibling.evaluate_handle('node => node.nextElementSibling')

            atk_type, resource = "Unknown", "Unknown"

            if sibling:
                # 공격 타입 이미지 (atk_type) 찾기
                atk_type_img_src = await get_element_attribute(sibling, 'img[src^="https://img.kusoge.xyz/limbus/img/ui/atk_type_"]', 'src')
                if atk_type_img_src:
                    atk_type = atk_type_img_src.split('_')[-1].split('.')[0]  # atk_type_SLASH.png -> SLASH
                    print(f"Extracted attack type: {atk_type}")

                # 자원 이미지 (resource) 찾기
                resource_img_src = await get_element_attribute(sibling, 'div.q-py-xs img[src^="https://img.kusoge.xyz/limbus/img/ui/attr_"]', 'src')
                if resource_img_src:
                    resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                    print(f"Extracted resource: {resource}")
                    
                    # 리소스를 변환
                    resource = convert_resource(resource)
                    print(f"Converted resource: {resource}")

            return {f'skill{skill_number}_atk_type': atk_type, f'skill{skill_number}_resource': resource}
        
    except Exception as e:
        print(f"Error extracting skill{skill_number} type and resource: {e}")
    
    return {f'skill{skill_number}_atk_type': "Unknown", f'skill{skill_number}_resource': "Unknown"}

async def extract_defense_and_resource(row):
    """ 방어 타입(Def Type)과 자원(Attribute) 추출 함수 """
    try:
        print("Attempting to locate defense type and resource...")

        # <a> 태그의 부모 <td>로 이동
        parent_td = await row.query_selector('a[href^="/identity/"]')
        if parent_td:
            # 부모 td에서 형제로 이동
            parent = await parent_td.evaluate_handle('(node) => node.parentElement')

            # 다섯 번째 형제 td로 이동 (방어 관련 정보가 있는 <td>)
            sibling = parent
            for _ in range(5):  # 방어 정보는 5번째 형제에 있음
                sibling = await sibling.evaluate_handle('node => node.nextElementSibling')

            def_type, def_resource = "Unknown", "Unknown"

            if sibling:
                # 방어 타입 이미지 (def_type) 찾기
                def_type_img_src = await get_element_attribute(sibling, 'img[src^="https://img.kusoge.xyz/limbus/img/ui/def_type_"]', 'src')
                if def_type_img_src:
                    def_type = def_type_img_src.split('_')[-1].split('.')[0]  # def_type_GUARD.png -> GUARD
                    print(f"Extracted defense type: {def_type}")

                # 자원 이미지 (resource) 찾기
                resource_img_src = await get_element_attribute(sibling, 'div.q-py-xs img[src^="https://img.kusoge.xyz/limbus/img/ui/attr_"]', 'src')
                if resource_img_src:
                    def_resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                    print(f"Extracted defense resource: {def_resource}")
                    
                    # 방어 리소스를 변환
                    def_resource = convert_resource(def_resource)
                    print(f"Converted defense resource: {def_resource}")

            return {'def_type': def_type, 'def_resource': def_resource}
        
    except Exception as e:
        print(f"Error extracting defense type and resource: {e}")
    
    return {'def_type': "Unknown", 'def_resource': "Unknown"}


async def scrape_character_name_and_url(row):
    """ 캐릭터 이름과 URL 추출 함수 """
    try:
        # 캐릭터 이름 추출
        character_name = await row.query_selector('a[href^="/identity/"]')
        name = await character_name.inner_text() if character_name else "Unknown"

        # 캐릭터 URL 추출
        character_url = await character_name.get_attribute('href') if character_name else ""
        full_url = f"https://limbus.kusoge.xyz{character_url}" if character_url else ""

        return {'name': name.strip(), 'url': full_url}

    except Exception as e:
        print(f"Error extracting character name and URL: {e}")
        return None

async def handle_request_interception(route, request):
    """ 이미지 요청을 차단하는 함수 """
    if "image" in request.resource_type:
        print(f"Blocking image request: {url}")
        await route.abort()  # 이미지 요청 차단
    else:
        await route.continue_()  # 다른 요청은 그대로 진행

async def scrape_character_list(page):

    # 이미지 요청 차단
    page.on("route", handle_request_interception)
    
    """ 캐릭터 리스트에서 정보 추출 함수 """
    await page.goto("https://limbus.kusoge.xyz/identity")
    
    character_rows = await page.query_selector_all('tr')
    if len(character_rows) > 1:
        first_row = character_rows[1]
        
        # 캐릭터 이름과 URL을 추출
        character_info = await scrape_character_name_and_url(first_row)
        if character_info:
            character_info['rarity'] = await extract_rarity(first_row)  # 희귀도 추출
            for i in range(1, 4):  # 스킬1~3 정보 추출
                skill_and_resource = await extract_atk_and_resource(first_row, i)
                character_info.update(skill_and_resource)  # 스킬 정보 추가
            defense_and_resource = await extract_defense_and_resource(first_row)  # 방어 정보 추출
            character_info.update(defense_and_resource)  # 방어 정보 추가
            return character_info
    return None

async def main():
    """ 메인 크롤링 함수 """
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.route("**/*", handle_request)
        character_info = await scrape_character_list(page)
        if character_info:
            print("Character Info:")
            print(f"Name: {character_info['name']}")
            print(f"URL: {character_info['url']}")
            print(f"Rarity: {character_info['rarity']} 단계")
            print(f"Skill1 Attack Type: {character_info['skill1_atk_type']}")
            print(f"Skill1 Resource: {character_info['skill1_resource']}")
            print(f"Skill2 Attack Type: {character_info['skill2_atk_type']}")
            print(f"Skill2 Resource: {character_info['skill2_resource']}")
            print(f"Skill3 Attack Type: {character_info['skill3_atk_type']}")
            print(f"Skill3 Resource: {character_info['skill3_resource']}")
            print(f"Defense Type: {character_info['def_type']}")
            print(f"Defense Resource: {character_info['def_resource']}")
        else:
            print("No characters found.")
        
        await browser.close()

# Jupyter 환경에서 실행
await main()

In [ ]:
import csv
from playwright.sync_api import sync_playwright

# Identity 객체 정의
class Identity:
    def __init__(self, identity_id, name, rarity):
        self.identity_id = identity_id
        self.name = name
        self.rarity = rarity
        self.skills = []  # Skill 객체 리스트
        self.level_stats = []  # LevelStat 객체 리스트
        self.attack_types = {}
        self.stagger_thresholds = {}
        self.coin_effect = []  # CoinEffect 객체 리스트
        self.buff_debuff_info = []  # BuffDebuffInfo 객체 리스트
        self.passive_info = []  # PassiveInfo 객체 리스트

    def update_attack_types(self, hit, penetrate, slash):
        self.attack_types = {"HIT": hit, "PENETRATE": penetrate, "SLASH": slash}

    def update_stagger_thresholds(self, one, two, three):
        self.stagger_thresholds = {"One": one, "Two": two, "Three": three}

    def add_coin_effect(self, uptie, coin_number, coin_effect):
        self.coin_effect.append(CoinEffect(uptie, coin_number, coin_effect))

    def add_buff_debuff_info(self, name, effect_description, dispellable):
        self.buff_debuff_info.append(BuffDebuffInfo(name, effect_description, dispellable))

    def add_passive_info(self, passive_type, passive_condition, passive_effect):
        self.passive_info.append(PassiveInfo(passive_type, passive_condition, passive_effect))

# LevelStat 객체 정의
class LevelStat:
    def __init__(self, level, hp, speed, defense):
        self.level = level
        self.hp = hp
        self.speed = speed
        self.defense = defense

# Skill 객체 정의
class Skill:
    def __init__(self, skill_id, skill_type, sin_affinity, name, power, coin_count):
        self.skill_id = skill_id
        self.skill_type = skill_type
        self.sin_affinity = sin_affinity
        self.name = name
        self.power = power
        self.coin_count = coin_count

# CoinEffect 객체 정의
class CoinEffect:
    def __init__(self, uptie, coin_number, coin_effect):
        self.uptie = uptie
        self.coin_number = coin_number
        self.coin_effect = coin_effect

# BuffDebuffInfo 객체 정의
class BuffDebuffInfo:
    def __init__(self, name, effect_description, dispellable):
        self.name = name
        self.effect_description = effect_description
        self.dispellable = dispellable

# PassiveInfo 객체 정의
class PassiveInfo:
    def __init__(self, passive_type, passive_condition, passive_effect):
        self.passive_type = passive_type
        self.passive_condition = passive_condition
        self.passive_effect = passive_effect

# Identity 크롤링 함수
def crawl_identity_page(page):
    identity_elements = page.query_selector_all(".identity-item")
    identities = []

    for element in identity_elements:
        identity_id = element.get_attribute("data-id")
        name = element.query_selector(".identity-name").inner_text()
        rarity = int(element.query_selector(".identity-rarity").inner_text())

        # Identity 객체 생성 및 추가
        identity = Identity(identity_id, name, rarity)

        # Skill 정보 저장
        skills = element.query_selector_all(".skill-item")
        for skill in skills:
            skill_id = skill.get_attribute("data-skill-id")
            skill_type = skill.query_selector(".skill-type").inner_text()
            sin_affinity = skill.query_selector(".sin-affinity").inner_text()
            name = skill.query_selector(".skill-name").inner_text()
            power = int(skill.query_selector(".skill-power").inner_text())
            coin_count = int(skill.query_selector(".skill-coin-count").inner_text())
            identity.skills.append(Skill(skill_id, skill_type, sin_affinity, name, power, coin_count))

        identities.append(identity)

    return identities

# 상세 페이지 크롤링 함수
def crawl_identity_details(page, identity):
    page.goto(f"https://limbus.kusoge.xyz/identity/{identity.identity_id}")

    # Attack Types 업데이트
    atk_hit = page.query_selector(".atk-hit").inner_text()
    atk_penetrate = page.query_selector(".atk-penetrate").inner_text()
    atk_slash = page.query_selector(".atk-slash").inner_text()
    identity.update_attack_types(atk_hit, atk_penetrate, atk_slash)

    # Stagger Thresholds 업데이트
    stagger_one = page.query_selector(".stagger-one").inner_text()
    stagger_two = page.query_selector(".stagger-two").inner_text()
    stagger_three = page.query_selector(".stagger-three").inner_text()
    identity.update_stagger_thresholds(stagger_one, stagger_two, stagger_three)

    # Level Stats 저장
    level_stats_elements = page.query_selector_all(".level-stats")
    for level_stat in level_stats_elements:
        level = int(level_stat.query_selector(".level").inner_text())
        hp = int(level_stat.query_selector(".hp").inner_text())
        speed = int(level_stat.query_selector(".speed").inner_text())
        defense = int(level_stat.query_selector(".defense").inner_text())
        identity.level_stats.append(LevelStat(level, hp, speed, defense))

    # Coin Effect, Buff/Debuff, Passive 정보 추가 (가상의 예시로 추가)
    identity.add_coin_effect("Uptie_1", 5, "Increase Attack by 10%")
    identity.add_buff_debuff_info("Buff", "Increase Speed by 10%", True)
    identity.add_passive_info("Resilience", "On Low HP", "Increase Defense by 15%")

# CSV 저장 함수
def save_to_csv(identities):
    # Identity_Info 저장
    with open("Identity_Info.csv", mode="w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(["Identity_ID", "Name", "Rarity", "Atk_HIT", "Atk_PENETRATE", "Atk_SLASH", "Stagger_One", "Stagger_Two", "Stagger_Three"])
        for identity in identities:
            writer.writerow([
                identity.identity_id, identity.name, identity.rarity,
                identity.attack_types.get("HIT"), identity.attack_types.get("PENETRATE"), identity.attack_types.get("SLASH"),
                identity.stagger_thresholds.get("One"), identity.stagger_thresholds.get("Two"), identity.stagger_thresholds.get("Three")
            ])

    # Skill_Info 저장
    with open("Skill_Info.csv", mode="w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(["Skill_ID", "Identity_ID", "Skill_Type", "Sin_Affinity", "Name", "Power", "Coin_Count"])
        for identity in identities:
            for skill in identity.skills:
                writer.writerow([skill.skill_id, identity.identity_id, skill.skill_type, skill.sin_affinity, skill.name, skill.power, skill.coin_count])

    # Level_Based_Stats 저장
    with open("Level_Based_Stats.csv", mode="w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(["Identity_ID", "Level", "HP", "Speed", "Defense"])
        for identity in identities:
            for level_stat in identity.level_stats:
                writer.writerow([identity.identity_id, level_stat.level, level_stat.hp, level_stat.speed, level_stat.defense])

    # Coin_Effect 저장
    with open("Coin_Effect.csv", mode="w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(["Identity_ID", "Uptie", "Coin_Number", "Coin_Effect"])
        for identity in identities:
            for effect in identity.coin_effect:
                writer.writerow([identity.identity_id, effect.uptie, effect.coin_number, effect.coin_effect])

    # Buff_Debuff_Info 저장
    with open("Buff_Debuff_Info.csv", mode="w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(["Identity_ID", "Name", "Effect_Description", "Dispellable"])
        for identity in identities:
            for info in identity.buff_debuff_info:
                writer.writerow([identity.identity_id, info.name, info.effect_description, info.dispellable])

    # Passive_Info 저장
    with open("Passive_Info.csv", mode="w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(["Identity_ID", "Passive_Type", "Passive_Condition", "Passive_Effect"])
        for identity in identities:
            for passive in identity.passive_info:
                writer.writerow([identity.identity_id, passive.passive_type, passive.passive_condition, passive.passive_effect])

# Playwright 크롤러
def main():
    identities = []  # Identity 객체 리스트

    with sync_playwright() as p:
        browser = p.chromium.launch(headless=True)
        page = browser.new_page()

        # Step 1: /identity 페이지 크롤링
        identities = crawl_identity_page(page)

        # Step 2: /identity/{Identity_ID} 상세 페이지 크롤링
        for identity in identities:
            crawl_identity_details(page, identity)

        browser.close()

    # Step 3: 객체 데이터를 CSV로 저장
    save_to_csv(identities)

await main()

In [ ]:
import asyncio
from playwright.async_api import async_playwright
import csv
import os

class CharacterScraper:
    def __init__(self, base_url, output_dir):
        self.base_url = base_url
        self.output_dir = output_dir
        self.data = []

        # Ensure output directory exists
        os.makedirs(self.output_dir, exist_ok=True)

    async def scrape(self):
        async with async_playwright() as p:
            browser = await p.chromium.launch(headless=True)
            page = await browser.new_page()
            await page.goto(self.base_url)

            # Scrape identities
            identities = await self.scrape_identities(page)
            print(f"Found {len(identities)} identities.")

            # Scrape data for each identity
            for identity in identities:
                print(f"Scraping data for {identity['name']}...")
                await self.scrape_identity(page, identity)

            await browser.close()

    async def scrape_identities(self, page):
        identities = []
        try:
            identity_elements = await page.query_selector_all('a[href^="/identity/"]')
            for element in identity_elements:
                href = await element.get_attribute('href')
                name = await element.text_content()
                if href and name:
                    identities.append({
                        "name": name.strip(),
                        "url": self.base_url + href
                    })
        except Exception as e:
            print(f"Error scraping identities: {e}")
        return identities

    async def scrape_identity(self, page, identity):
        try:
            await page.goto(identity['url'])
            name = identity['name']
            stats = await self.scrape_stats(page)
            skills = await self.scrape_skills(page)
            self.data.append({
                "name": name,
                "stats": stats,
                "skills": skills
            })
        except Exception as e:
            print(f"Error scraping identity {identity['name']}: {e}")

    async def scrape_stats(self, page):
        stats = {}
        try:
            rows = await page.query_selector_all('table.stats-table tr')
            for row in rows:
                cells = await row.query_selector_all('td')
                if len(cells) >= 2:
                    stat_name = (await cells[0].text_content()).strip()
                    stat_value = (await cells[1].text_content()).strip()
                    stats[stat_name] = stat_value
        except Exception as e:
            print(f"Error scraping stats: {e}")
        return stats

    async def scrape_skills(self, page):
        skills = []
        try:
            skill_rows = await page.query_selector_all('table.skills-table tr')
            for row in skill_rows:
                skill = {}
                cells = await row.query_selector_all('td')
                if cells:
                    skill["name"] = (await cells[0].text_content()).strip()
                    skill["description"] = (await cells[1].text_content()).strip()
                    skills.append(skill)
        except Exception as e:
            print(f"Error scraping skills: {e}")
        return skills


if __name__ == "__main__":
    # Define base URL and output directory
    BASE_URL = "https://limbus.kusoge.xyz"
    OUTPUT_DIR = "./output"

    # Initialize and run scraper
    scraper = CharacterScraper(BASE_URL, OUTPUT_DIR)
    asyncio.run(scraper.scrape())

In [ ]:
from playwright.async_api import async_playwright

async def handle_request(route, request):
    # 이미지, 스타일시트, 폰트, 스크립트 차단
    if request.resource_type in ["image", "stylesheet", "font"]:
        #print(f"차단된 리소스: {request.url}")
        await route.abort()
    else:
        await route.continue_()
        
async def get_element_attribute(element, selector, attribute):
    """ 지정된 요소에서 특정 속성 값을 추출하는 함수 """
    try:
        target_element = await element.query_selector(selector)
        if target_element:
            return await target_element.get_attribute(attribute)
    except Exception as e:
        print(f"Error getting element attribute: {e}")
    return None

async def extract_rarity(row):
    """ 희귀도 추출 함수 """
    try:
        print("Attempting to locate rarity...")

        # <a> 태그의 부모 <td>로 이동
        parent_td = await row.query_selector('a[href^="/identity/"]')
        if parent_td:
            # 부모 td에서 형제로 이동
            parent = await parent_td.evaluate_handle('(node) => node.parentElement')

            # 첫 번째 형제 td로 이동 (희귀도 이미지가 있는 <td>)
            sibling = await parent.evaluate_handle('node => node.nextElementSibling')

            if sibling:
                # 희귀도 이미지 찾기
                rarity_img_src = await get_element_attribute(sibling, 'img[src^="https://img.kusoge.xyz/limbus/img/ui/rarity_"]', 'src')
                if rarity_img_src:
                    rarity_level = rarity_img_src.split('_')[-1].split('.')[0]  # rarity_1.png -> 1
                    print(f"Extracted rarity level: {rarity_level}")
                    return rarity_level

        print("Rarity image not found.")
    except Exception as e:
        print(f"Error extracting rarity: {e}")
    
    return "Unknown"

resource_mapping = {
    "AZURE": "Gloom",
    "VIOLET": "Envy",
    "AMBER": "Sloth",
    "SHAMROCK": "Gluttony",
    "CRIMSON": "Wrath",
    "SCARLET": "Lust"
}

def convert_resource(resource_name):
    """리소스 이름을 변환하는 함수"""
    return resource_mapping.get(resource_name, resource_name)  # 매핑이 없으면 원래 값을 그대로 반환

async def extract_atk_and_resource(row, skill_number):
    """ 공격 타입(ATK Type)과 자원(Attribute) 추출 함수 - 스킬 번호에 따라 처리 """
    try:
        print(f"Attempting to locate skill{skill_number} type and resource...")

        # <a> 태그의 부모 <td>로 이동
        parent_td = await row.query_selector('a[href^="/identity/"]')
        if parent_td:
            # 부모 td에서 형제로 이동
            parent = await parent_td.evaluate_handle('(node) => node.parentElement')

            # 해당 스킬 번호에 맞는 형제로 이동
            sibling = parent
            for _ in range(skill_number + 1):  # 스킬 번호에 맞게 이동
                sibling = await sibling.evaluate_handle('node => node.nextElementSibling')

            atk_type, resource = "Unknown", "Unknown"

            if sibling:
                # 공격 타입 이미지 (atk_type) 찾기
                atk_type_img_src = await get_element_attribute(sibling, 'img[src^="https://img.kusoge.xyz/limbus/img/ui/atk_type_"]', 'src')
                if atk_type_img_src:
                    atk_type = atk_type_img_src.split('_')[-1].split('.')[0]  # atk_type_SLASH.png -> SLASH
                    print(f"Extracted attack type: {atk_type}")

                # 자원 이미지 (resource) 찾기
                resource_img_src = await get_element_attribute(sibling, 'div.q-py-xs img[src^="https://img.kusoge.xyz/limbus/img/ui/attr_"]', 'src')
                if resource_img_src:
                    resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                    print(f"Extracted resource: {resource}")
                    
                    # 리소스를 변환
                    resource = convert_resource(resource)
                    print(f"Converted resource: {resource}")

            return {f'skill{skill_number}_atk_type': atk_type, f'skill{skill_number}_resource': resource}
        
    except Exception as e:
        print(f"Error extracting skill{skill_number} type and resource: {e}")
    
    return {f'skill{skill_number}_atk_type': "Unknown", f'skill{skill_number}_resource': "Unknown"}

async def extract_defense_and_resource(row):
    """ 방어 타입(Def Type)과 자원(Attribute) 추출 함수 """
    try:
        print("Attempting to locate defense type and resource...")

        # <a> 태그의 부모 <td>로 이동
        parent_td = await row.query_selector('a[href^="/identity/"]')
        if parent_td:
            # 부모 td에서 형제로 이동
            parent = await parent_td.evaluate_handle('(node) => node.parentElement')

            # 다섯 번째 형제 td로 이동 (방어 관련 정보가 있는 <td>)
            sibling = parent
            for _ in range(5):  # 방어 정보는 5번째 형제에 있음
                sibling = await sibling.evaluate_handle('node => node.nextElementSibling')

            def_type, def_resource = "Unknown", "Unknown"

            if sibling:
                # 방어 타입 이미지 (def_type) 찾기
                def_type_img_src = await get_element_attribute(sibling, 'img[src^="https://img.kusoge.xyz/limbus/img/ui/def_type_"]', 'src')
                if def_type_img_src:
                    def_type = def_type_img_src.split('_')[-1].split('.')[0]  # def_type_GUARD.png -> GUARD
                    print(f"Extracted defense type: {def_type}")

                # 자원 이미지 (resource) 찾기
                resource_img_src = await get_element_attribute(sibling, 'div.q-py-xs img[src^="https://img.kusoge.xyz/limbus/img/ui/attr_"]', 'src')
                if resource_img_src:
                    def_resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                    print(f"Extracted defense resource: {def_resource}")
                    
                    # 방어 리소스를 변환
                    def_resource = convert_resource(def_resource)
                    print(f"Converted defense resource: {def_resource}")

            return {'def_type': def_type, 'def_resource': def_resource}
        
    except Exception as e:
        print(f"Error extracting defense type and resource: {e}")
    
    return {'def_type': "Unknown", 'def_resource': "Unknown"}


async def scrape_character_name_and_url(row):
    """ 캐릭터 이름과 URL 추출 함수 """
    try:
        # 캐릭터 이름 추출
        character_name = await row.query_selector('a[href^="/identity/"]')
        name = await character_name.inner_text() if character_name else "Unknown"

        # 캐릭터 URL 추출
        character_url = await character_name.get_attribute('href') if character_name else ""
        full_url = f"https://limbus.kusoge.xyz{character_url}" if character_url else ""

        return {'name': name.strip(), 'url': full_url}

    except Exception as e:
        print(f"Error extracting character name and URL: {e}")
        return None

async def handle_request_interception(route, request):
    """ 이미지 요청을 차단하는 함수 """
    if "image" in request.resource_type:
        print(f"Blocking image request: {url}")
        await route.abort()  # 이미지 요청 차단
    else:
        await route.continue_()  # 다른 요청은 그대로 진행

async def scrape_character_list(page):

    # 이미지 요청 차단
    page.on("route", handle_request_interception)
    
    """ 캐릭터 리스트에서 정보 추출 함수 """
    await page.goto("https://limbus.kusoge.xyz/identity")
    
    character_rows = await page.query_selector_all('tr')
    if len(character_rows) > 1:
        first_row = character_rows[1]
        
        # 캐릭터 이름과 URL을 추출
        character_info = await scrape_character_name_and_url(first_row)
        if character_info:
            character_info['rarity'] = await extract_rarity(first_row)  # 희귀도 추출
            for i in range(1, 4):  # 스킬1~3 정보 추출
                skill_and_resource = await extract_atk_and_resource(first_row, i)
                character_info.update(skill_and_resource)  # 스킬 정보 추가
            defense_and_resource = await extract_defense_and_resource(first_row)  # 방어 정보 추출
            character_info.update(defense_and_resource)  # 방어 정보 추가
            return character_info
    return None

async def main():
    """ 메인 크롤링 함수 """
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.route("**/*", handle_request)
        character_info = await scrape_character_list(page)
        if character_info:
            print("Character Info:")
            print(f"Name: {character_info['name']}")
            print(f"URL: {character_info['url']}")
            print(f"Rarity: {character_info['rarity']} 단계")
            print(f"Skill1 Attack Type: {character_info['skill1_atk_type']}")
            print(f"Skill1 Resource: {character_info['skill1_resource']}")
            print(f"Skill2 Attack Type: {character_info['skill2_atk_type']}")
            print(f"Skill2 Resource: {character_info['skill2_resource']}")
            print(f"Skill3 Attack Type: {character_info['skill3_atk_type']}")
            print(f"Skill3 Resource: {character_info['skill3_resource']}")
            print(f"Defense Type: {character_info['def_type']}")
            print(f"Defense Resource: {character_info['def_resource']}")
        else:
            print("No characters found.")
        
        await browser.close()

# Jupyter 환경에서 실행
await main()

In [ ]:
import csv
from playwright.sync_api import sync_playwright

class Identity:
    def __init__(self, identity_id, name, rarity):
        self.identity_id = identity_id
        self.name = name
        self.rarity = rarity
        self.skills = []  # Skill 객체 리스트
        self.level_stats = []  # LevelStat 객체 리스트
        self.attack_types = {}
        self.stagger_thresholds = {}
        self.coin_effect = []  # CoinEffect 객체 리스트
        self.buff_debuff_info = []  # BuffDebuffInfo 객체 리스트
        self.passive_info = []  # PassiveInfo 객체 리스트

    def update_attack_types(self, hit, penetrate, slash):
        self.attack_types = {"HIT": hit, "PENETRATE": penetrate, "SLASH": slash}

    def update_stagger_thresholds(self, one, two, three):
        self.stagger_thresholds = {"One": one, "Two": two, "Three": three}

    def add_coin_effect(self, uptie, coin_number, coin_effect):
        self.coin_effect.append(CoinEffect(uptie, coin_number, coin_effect))

    def add_buff_debuff_info(self, name, effect_description, dispellable):
        self.buff_debuff_info.append(BuffDebuffInfo(name, effect_description, dispellable))

    def add_passive_info(self, passive_type, passive_condition, passive_effect):
        self.passive_info.append(PassiveInfo(passive_type, passive_condition, passive_effect))

# LevelStat 객체 정의
class LevelStat:
    def __init__(self, level, hp, speed, defense):
        self.level = level
        self.hp = hp
        self.speed = speed
        self.defense = defense

# Skill 객체 정의
class Skill:
    def __init__(self, skill_id, skill_type, sin_affinity, name, power, coin_count):
        self.skill_id = skill_id
        self.skill_type = skill_type
        self.sin_affinity = sin_affinity
        self.name = name
        self.power = power
        self.coin_count = coin_count

# CoinEffect 객체 정의
class CoinEffect:
    def __init__(self, uptie, coin_number, coin_effect):
        self.uptie = uptie
        self.coin_number = coin_number
        self.coin_effect = coin_effect

# BuffDebuffInfo 객체 정의
class BuffDebuffInfo:
    def __init__(self, name, effect_description, dispellable):
        self.name = name
        self.effect_description = effect_description
        self.dispellable = dispellable

# PassiveInfo 객체 정의
class PassiveInfo:
    def __init__(self, passive_type, passive_condition, passive_effect):
        self.passive_type = passive_type
        self.passive_condition = passive_condition
        self.passive_effect = passive_effect

# 크롤링 객체
class Scraper:
    def __init__(self, identities):
        self.container = container  # DataContainer 객체와 연결

    def scrape(self):
        """데이터를 크롤링하여 DataContainer에 추가"""
        # 여기서는 예제를 위해 임의의 데이터를 생성
        scraped_data = [
            ["Item1", "Value1", "Detail1"],
            ["Item2", "Value2", "Detail2"],
            ["Item3", "Value3", "Detail3"],
        ]
        for data in scraped_data:
            self.container.add_data(data)  # DataContainer에 데이터 추가
        print("Scraping complete. Data added to container.")

# 메인 프로세스
def main():
    identities = [] # Identity 객체 리스트

    with sync_playwright() as p:
        brwoser = p.chromium.launch(headless=True)
        page = browser.new_page()

        # Step 1 :/identity 페이지 크롤링
        identities = crawl_identity_page(page)

        # Setp 2: /identity/{Identity_ID} 상세 페이지 크롤링
        for identity in identities:
            crawl_identity_details(page, identity)

        brwoser.close()

    #Step 3: 객체 데이터를 CSV로 저장
    #save_to_csv(identities)

await main()
'''
await main()

    # Scraper 객체 생성 (DataContainer 위임)
    scraper = Scraper(container=data_container)

    # 데이터 크롤링
    scraper.scrape()

    # 크롤링된 데이터를 CSV로 저장
    data_container.save_to_csv("output.csv")

# 실행
if __name__ == "__main__":
    main()
'''

In [ ]:
import csv
from playwright.sync_api import sync_playwright

class Identity:
    def __init__(self, identity_id, name, rarity):
        self.identity_id = identity_id
        self.name = name
        self.rarity = rarity
        self.skills = []  # Skill 객체 리스트
        self.level_stats = []  # LevelStat 객체 리스트
        self.attack_types = {}
        self.stagger_thresholds = {}
        self.coin_effect = []  # CoinEffect 객체 리스트
        self.buff_debuff_info = []  # BuffDebuffInfo 객체 리스트
        self.passive_info = []  # PassiveInfo 객체 리스트

    def update_attack_types(self, hit, penetrate, slash):
        self.attack_types = {"HIT": hit, "PENETRATE": penetrate, "SLASH": slash}

    def update_stagger_thresholds(self, one, two, three):
        self.stagger_thresholds = {"One": one, "Two": two, "Three": three}

    def add_coin_effect(self, uptie, coin_number, coin_effect):
        self.coin_effect.append(CoinEffect(uptie, coin_number, coin_effect))

    def add_buff_debuff_info(self, name, effect_description, dispellable):
        self.buff_debuff_info.append(BuffDebuffInfo(name, effect_description, dispellable))

    def add_passive_info(self, passive_type, passive_condition, passive_effect):
        self.passive_info.append(PassiveInfo(passive_type, passive_condition, passive_effect))

# LevelStat 객체 정의
class LevelStat:
    def __init__(self, level, hp, speed, defense):
        self.level = level
        self.hp = hp
        self.speed = speed
        self.defense = defense

# Skill 객체 정의
class Skill:
    def __init__(self, skill_id, skill_type, sin_affinity, name, power, coin_count):
        self.skill_id = skill_id
        self.skill_type = skill_type
        self.sin_affinity = sin_affinity
        self.name = name
        self.power = power
        self.coin_count = coin_count

# CoinEffect 객체 정의
class CoinEffect:
    def __init__(self, uptie, coin_number, coin_effect):
        self.uptie = uptie
        self.coin_number = coin_number
        self.coin_effect = coin_effect

# BuffDebuffInfo 객체 정의
class BuffDebuffInfo:
    def __init__(self, name, effect_description, dispellable):
        self.name = name
        self.effect_description = effect_description
        self.dispellable = dispellable

# PassiveInfo 객체 정의
class PassiveInfo:
    def __init__(self, passive_type, passive_condition, passive_effect):
        self.passive_type = passive_type
        self.passive_condition = passive_condition
        self.passive_effect = passive_effect

class DataContainer:
    def __init__(self):
        self.identities = [] # Identity 객체 리스트

    def add_identity(self, identity):
        self.identities.append(identity)

    def save_to_csv(self, file_name):
        #데이터를 CSV파일로 저장하는 로직

class Scraper:
    def __init__(self, container):
        self.container = container

    async def handle_request(route, request):
    # 이미지, 스타일시트, 폰트, 스크립트 차단
    if request.resource_type in ["image", "font"]:
        #print(f"차단된 리소스: {request.url}")
        await route.abort()
    else:
        await route.continue_()
        
    def scrape(self):
        with sync_playwright() as p:
            browser = p.chromium.lauch(headless=True)
            page = browser.new_page()
            await page.route("**/*", handle_request)

            # Step 1 :/identity 페이지 크롤링
            identities = self.crawl_identity_page(page

            # Step 2: /identity/{identity_ID} 상세 페이지 크롤링
            for identity in identities

In [ ]:
class Scraper:
    def __init__(self, container):
        self.container = container

    def scrape(self):
        with sync_playwright() as p:
            browser = p.chromium.launch(headless=True)
            page = browser.new_page()

            # Step 1: /identity 페이지 크롤링 (아이디만 크롤링)
            self.crawl_identity_page(page)

            # Step 2: /identity/{identity_id} 상세 페이지 크롤링 (세부 정보 추가)
            self.crawl_identity_details_for_all(page)

            browser.close()

    def crawl_identity_page(self, page):
        identity_ids = self.get_crawled_identity_ids(page)  # ID만 크롤링
        for identity_id in identity_ids:
            identity = Identity(identity_id)  # ID를 기반으로 빈 Identity 객체 생성
            self.container.add_identity(identity)  # 컨테이너에 추가

    def crawl_identity_details_for_all(self, page):
        # 모든 아이덴티티의 세부 정보 크롤링
        for identity in self.container.identities:
            identity.update_details(page)  # 각 아이덴티티의 세부 정보를 직접 갱신

    def get_crawled_identity_ids(self, page):
        return [1, 2]  # 예시로 ID만 반환


class Identity:
    def __init__(self, identity_id):
        self.identity_id = identity_id
        self.name = None
        self.skills = []
        self.details = None

    def update_details(self, page):
        # /identity/{identity_id} 상세 페이지에서 세부 정보 크롤링
        self.name = self.get_name_from_page(page)
        self.skills = self.get_skills_from_page(page)
        self.details = self.get_details_from_page(page)

    def get_name_from_page(self, page):
        return "Example Name"  # 실제 페이지에서 크롤링한 값으로 대체

    def get_skills_from_page(self, page):
        return ["Skill 1", "Skill 2"]  # 실제 페이지에서 크롤링한 값으로 대체

    def get_details_from_page(self, page):
        return "Detailed information"  # 실제 페이지에서 크롤링한 값으로 대체


class DataContainer:
    def __init__(self):
        self.identities = []

    def add_identity(self, identity):
        self.identities.append(identity)


def main():
    data_container = DataContainer()
    scraper = Scraper(container=data_container)
    scraper.scrape()


if __name__ == "__main__":
    main()


In [11]:
import csv
from playwright.sync_api import sync_playwright

class Scraper:
    def __init__(self, container):
        self.container = container

    async def handle_request(route, request):
        # 이미지, 폰트 차단
        if request.resource_type in ["image", "font"]:
            await route.abort()
        else:
            await route.continue_()
            
    async def scrape(self):
        async with sync_playwright() as p:
            browser = p.chromium.launch(headless=True)
            page = browser.new_page()
            await page.route("**/*", handle-request)

            # Step 1: /identity 페이지 크롤링 (아이디만 크롤링)
            self.crawl_identity_page(page)

            # Step 2: /identity/{identity_id} 상세 페이지 크롤링(세부 정보 추가)
            self.crawl_identity_details_for_all(page)

            browser.close()

    async def crawl_identity_page(self, page):
        identity_ids = self.get_crawled_identity_ids(page) # identity_ids 크롤링
        for identity_id in identity_ids:
            identity = Identity(identity_id) #ID를 기반으로 빈 Identity 객체 생성
            self.container.add_identity(identity) # 컨테이너에 추가

    async def crawl_identity_details_for_all(self, page):
        # 아이덴티티의 세부 정보 크롤링
        for identity in self.container.identities:
            identity.update_details(page) # 각 아이덴티니의 세부 정보를 직접 갱신

    async def get_crawled_identity_ids(self, page):
        return [1, 2] # 예시로 1, 2 반환

class Identity:
    def __init__(self, identity_id, name, rarity):
        self.identity_id = identity_id
        self.name = name
        self.rarity = rarity
        self.skills = []  # Skill 객체 리스트
        self.level_stats = []  # LevelStat 객체 리스트
        self.attack_types = {}
        self.stagger_thresholds = {}
        self.coin_effect = []  # CoinEffect 객체 리스트
        self.buff_debuff_info = []  # BuffDebuffInfo 객체 리스트
        self.passive_info = []  # PassiveInfo 객체 리스트

    def update_attack_types(self, hit, penetrate, slash):
        self.attack_types = {"HIT": hit, "PENETRATE": penetrate, "SLASH": slash}

    def update_stagger_thresholds(self, one, two, three):
        self.stagger_thresholds = {"One": one, "Two": two, "Three": three}

    def add_coin_effect(self, uptie, coin_number, coin_effect):
        self.coin_effect.append(CoinEffect(uptie, coin_number, coin_effect))

    def add_buff_debuff_info(self, name, effect_description, dispellable):
        self.buff_debuff_info.append(BuffDebuffInfo(name, effect_description, dispellable))

    def add_passive_info(self, passive_type, passive_condition, passive_effect):
        self.passive_info.append(PassiveInfo(passive_type, passive_condition, passive_effect))

# LevelStat 객체 정의
class LevelStat:
    def __init__(self, level, hp, speed, defense):
        self.level = level
        self.hp = hp
        self.speed = speed
        self.defense = defense

# Skill 객체 정의
class Skill:
    def __init__(self, skill_id, skill_type, sin_affinity, name, power, coin_count):
        self.skill_id = skill_id
        self.skill_type = skill_type
        self.sin_affinity = sin_affinity
        self.name = name
        self.power = power
        self.coin_count = coin_count

# CoinEffect 객체 정의
class CoinEffect:
    def __init__(self, uptie, coin_number, coin_effect):
        self.uptie = uptie
        self.coin_number = coin_number
        self.coin_effect = coin_effect

# BuffDebuffInfo 객체 정의
class BuffDebuffInfo:
    def __init__(self, name, effect_description, dispellable):
        self.name = name
        self.effect_description = effect_description
        self.dispellable = dispellable

# PassiveInfo 객체 정의
class PassiveInfo:
    def __init__(self, passive_type, passive_condition, passive_effect):
        self.passive_type = passive_type
        self.passive_condition = passive_condition
        self.passive_effect = passive_effect

class DataContainer:
    def __init__(self):
        self.identities = []

    def add_identity(self, identity):
        self.identities.append(identity)

async def main():
    data_container = DataContainer()
    scraper = Scraper(container=data_container)
    scraper.scrape()

await main()

/var/folders/27/r1qdd08x0fx8nvdhy823ntqw0000gn/T/ipykernel_45825/739058672.py:120: RuntimeWarning: coroutine 'Scraper.scrape' was never awaited
  scraper.scrape()


In [46]:
import csv
from playwright.async_api import async_playwright

resource_mapping = {
    "AZURE", "Gloom",
    "VIOLET", "Envy",
    "AMBER", "Sloth",
    "SHAMROCK", "Gluttony",
    "CRIMSON", "Wrath",
    "SCARLET", "Lust"
}

def convert_resource(resource_name):
    """리소스 이름을 변환하는 함수"""
    return resource_mapping.get(resource_name, resource_name) # 매핑이 없으면 원래 값을 그대로 반환

class Scraper:
    def __init__(self, container, base_url):
        self.container = container
        self.base_url = base_url

    async def handle_request(self, route, request):
        # 이미지, 폰트 차단
        if request.resource_type in ["image", "font"]:
            await route.abort()
        else:
            await route.continue_()
            
    async def scrape(self):
        async with async_playwright() as p:
            browser = await p.chromium.launch(headless=False)
            page = await browser.new_page()
            await page.route("**/*", self.handle_request)

            # Step 1: /identity 페이지 크롤링 (아이디만 크롤링)
            await self.crawl_identity_page(page)

            # Step 2: /identity/{identity_id} 상세 페이지 크롤링(세부 정보 추가)
            await self.crawl_identity_details_for_all(page)

            await browser.close()

    async def crawl_identity_page(self, page):
        await page.goto(self.base_url+"/identity")
        #await self.get_crawled_identity_page(page)  # identity_ids 크롤링하여 self.container에 Identity 객체를 저장
        await self.scrape_identity_name_and_identity_id(row)
        await self.extract_skill_and_resource(page)

    async def crawl_identity_details_for_all(self, page):
        # 아이덴티티의 세부 정보 크롤링
        for identity in self.container.identities:
            identity.update_details(page)  # 각 아이덴티니의 세부 정보를 직접 갱신

    async def scrape_identity_name_and_identity_id(row):
        """ 인격 번호와 인격 이름을 추출함"""
        try:
            # 인격 이름 추출
            identity_cell = await row.query_selector('a[href^="/identity/"')
            identity_name = await identity_cell.inner_text() if identity_cell else "Unknown"
            # identity_id 추출
            identity_href = await identity_cell.get_attribute('href') if identity_cell else ""
            identity_id f"{identity_href}".split('/identity/')[-1]
            identity = Identity(identity_id)
            self.container.add_identity(identity)
        except Exception as e:
            print(f"Error extracting identity name and identity id: {e}")


    async def get_crawled_identity_page(self, page):
        """캐릭터 리스트에서 정보 추출"""
        # 인격 링크 정보를 가져오기
        identity_links = await page.query_selector_all('a[href^="/identity/"]')
        #identity_ids = []
        for link in identity_links:
            # 인격 주소를 추출
            href = await link.get_attribute('href')
            if href:
                # 인격 주소에서 /identity/와 {identity_id}를 분리하여 identity_id 변수에 할당한다.
                identity_id = f"{href}".split('/identity/')[-1]
                identity = Identity(identity_id)
                self.container.add_identity(identity)
    '''
    async def extract_identity_page(self, page):
        """캐릭터 열에서 스킬 정보를 추출"""
        identity_rows = await page.query_selector_all('tr')
    '''
        
        

class Identity:
    def __init__(self, identity_id, name=None, rarity=None):
        self.identity_id = identity_id
        self.name = name
        self.rarity = rarity
        self.skills = []  # Skill 객체 리스트
        self.level_stats = []  # LevelStat 객체 리스트
        self.attack_types = {}
        self.stagger_thresholds = {}
        self.coin_effect = []  # CoinEffect 객체 리스트
        self.buff_debuff_info = []  # BuffDebuffInfo 객체 리스트
        self.passive_info = []  # PassiveInfo 객체 리스트

    def update_attack_types(self, hit, penetrate, slash):
        self.attack_types = {"HIT": hit, "PENETRATE": penetrate, "SLASH": slash}

    def update_stagger_thresholds(self, one, two, three):
        self.stagger_thresholds = {"One": one, "Two": two, "Three": three}

    def add_coin_effect(self, uptie, coin_number, coin_effect):
        self.coin_effect.append(CoinEffect(uptie, coin_number, coin_effect))

    def add_buff_debuff_info(self, name, effect_description, dispellable):
        self.buff_debuff_info.append(BuffDebuffInfo(name, effect_description, dispellable))

    def add_passive_info(self, passive_type, passive_condition, passive_effect):
        self.passive_info.append(PassiveInfo(passive_type, passive_condition, passive_effect))

    def update_details(self, page):
        # 세부 정보 업데이트 로직 (가정)
        pass

# LevelStat 객체 정의
class LevelStat:
    def __init__(self, level, hp, speed, defense):
        self.level = level
        self.hp = hp
        self.speed = speed
        self.defense = defense

# Skill 객체 정의
class Skill:
    def __init__(self, skill_id, skill_type, sin_affinity):
        self.skill_id = skill_id
        self.skill_type = skill_type
        self.sin_affinity = sin_affinity
        self.name = None
        self.power = None
        self.coin_count = None
    def update_details(self, name, power, coin_count):
        self.name = name
        self.power = power
        self.coin_count = coin_count

# CoinEffect 객체 정의
class CoinEffect:
    def __init__(self, uptie, coin_number, coin_effect):
        self.uptie = uptie
        self.coin_number = coin_number
        self.coin_effect = coin_effect

# BuffDebuffInfo 객체 정의
class BuffDebuffInfo:
    def __init__(self, name, effect_description, dispellable):
        self.name = name
        self.effect_description = effect_description
        self.dispellable = dispellable

# PassiveInfo 객체 정의
class PassiveInfo:
    def __init__(self, passive_type, passive_condition, passive_effect):
        self.passive_type = passive_type
        self.passive_condition = passive_condition
        self.passive_effect = passive_effect

class DataContainer:
    def __init__(self):
        self.identities = []

    def add_identity(self, identity):
        self.identities.append(identity)

    def get_identity(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity
        return None

async def main():
    data_container = DataContainer()
    scraper = Scraper(container=data_container,base_url="https://limbus.kusoge.xyz")
    await scraper.scrape()

await main()

SyntaxError: unterminated string literal (detected at line 59) (1345683316.py, line 59)

In [70]:
import csv
from playwright.async_api import async_playwright

resource_mapping = {
    "AZURE", "Gloom",
    "VIOLET", "Envy",
    "AMBER", "Sloth",
    "SHAMROCK", "Gluttony",
    "CRIMSON", "Wrath",
    "SCARLET", "Lust"
}

def convert_resource(resource_name):
    """리소스 이름을 변환하는 함수"""
    return resource_mapping.get(resource_name, resource_name) # 매핑이 없으면 원래 값을 그대로 반환

class Scraper:
    def __init__(self, container, base_url):
        self.container = container
        self.base_url = base_url

    async def handle_request(self, route, request):
        # 이미지, 폰트 차단
        if request.resource_type in ["image", "font"]:
            await route.abort()
        else:
            await route.continue_()
            
    async def scrape(self):
        async with async_playwright() as p:
            browser = await p.chromium.launch(headless=False)
            page = await browser.new_page()
            await page.route("**/*", self.handle_request)

            # Step 1: /identity 페이지 크롤링 (아이디만 크롤링)
            await self.crawl_identity_page(page)

            # Step 2: /identity/{identity_id} 상세 페이지 크롤링(세부 정보 추가)
            await self.crawl_identity_details_for_all(page)

            await browser.close()

    async def crawl_identity_page(self, page):
        await page.goto(self.base_url+"/identity")
        await self.extract_identity_page(page)

    async def crawl_identity_details_for_all(self, page):
        # 아이덴티티의 세부 정보 크롤링
        for identity in self.container.identities:
            identity.update_details(page)  # 각 아이덴티니의 세부 정보를 직접 갱신

    async def get_identity_id_from_row(self, row):
        """주어지 테이블 행(row)에서 identity_id를 추출하는 함수"""
        # 행(row0 안에 a 태그 시작하는 identity 링크 셀 선택
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            # href 속성에 ID 추출
            identity_url = await identity_cell.get_attribute('href')
            # URL에 ID 부분만 분리
            identity_id = identity_url.split('/identity/')[-1]
            return identity_id
        return "Unknown"

    async def get_identity_name_from_row(self, row):
        """주어지 테이블 행(row)에서 identity_name를 추출하는 함수"""
        # 행(row0 안에 a 태그 시작하는 identity 링크 셀 선택
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            # href 속성에 identity_name을 추출
            identity_name = (await identity_cell.inner_text()).replace("\n", " ").strip()
            return identity_name
        return "Unknown"

    async def scrape_identity_name_and_identity_id(self, row):
        """ 인격 번호와 인격 이름을 추출함"""
        try:
            # identity_id 추출
            identity_id = await self.get_identity_name_from_row(row)
            # 인격 이름 추출
            identity_name = await self.get_identity_name_from_row(row)
            identity = Identity(identity_id)
            self.container.add_identity(identity)
            self.container.update_identity_name(identity_id, identity_name)
        except Exception as e:
            print(f"Error extracting identity name and identity id: {e}")

    async def get_element_attribute(self, element, selector, attribute):
        """ 지정된 요소에서 특정 속성 값을 추출하는 함수 """
        try:
            target_element = await element.query_selector(selector)
            if target_element:
                return await target_element.get_attribute(attribute)
        except Exception as e:
            print(f"Error getting element attribute: {e}")
        return None
    
    async def extract_rarity(self, row):
        """ 희귀도 추출 함수 """
        try:
            #print("Attempting to locate rarity...")
            # <a> 태그의 부 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
                # 첫 번째 형제 td로 이동 (희귀 이미지 있는 <td>)
                sibling = await parent.evaluate_handle('node => node.nextElementSibling')
                if sibling:
                    # 희귀 이미 찾기
                    rarity_img_src = await self.get_element_attribute(sibling, 'img[src^="https://img.kusoge.xyz/limbus/img/ui/rarity_"]', 'src')
                    if rarity_img_src:
                        rarity_level = rarity_img_src.split('_')[-1].split('.')[0] # rarity_1.png -> 1
                        #print(f"Extracted rarity level: {rarity_level}")
                        identity_id = await self.get_identity_name_from_row(row)
                        self.container.update_identity_rarity(identity_id, rarity_level)
                    else:
                        print("Rarity image not found in the next sibling TD.")
        except Exception as e: 
            print(f"Error extracting rarity: {e}")

        return "Unknown"

    async def extract_identity_page(self, page):
        """캐릭터 열에서 스킬 정보를 추출"""
        identity_rows = await page.query_selector_all('tr')
        if len(identity_rows) > 1:
            for identity_row in identity_rows:
                await self.scrape_identity_name_and_identity_id(identity_row)
                await self.extract_rarity(identity_row)
                
class Identity:
    def __init__(self, identity_id, name=None, rarity=None):
        self.identity_id = identity_id
        self.name = name
        self.rarity = rarity
        self.skills = []  # Skill 객체 리스트
        self.level_stats = []  # LevelStat 객체 리스트
        self.attack_types = {}
        self.stagger_thresholds = {}
        self.coin_effect = []  # CoinEffect 객체 리스트
        self.buff_debuff_info = []  # BuffDebuffInfo 객체 리스트
        self.passive_info = []  # PassiveInfo 객체 리스트

    def update_name(self, name):
        self.rarity = name

    def update_rarity(self, rarity):
        self.rarity = rarity

    def update_attack_types(self, hit, penetrate, slash):
        self.attack_types = {"HIT": hit, "PENETRATE": penetrate, "SLASH": slash}

    def update_stagger_thresholds(self, one, two, three):
        self.stagger_thresholds = {"One": one, "Two": two, "Three": three}

    def add_coin_effect(self, uptie, coin_number, coin_effect):
        self.coin_effect.append(CoinEffect(uptie, coin_number, coin_effect))

    def add_buff_debuff_info(self, name, effect_description, dispellable):
        self.buff_debuff_info.append(BuffDebuffInfo(name, effect_description, dispellable))

    def add_passive_info(self, passive_type, passive_condition, passive_effect):
        self.passive_info.append(PassiveInfo(passive_type, passive_condition, passive_effect))

    def update_details(self, page):
        # 세부 정보 업데이트 로직 (가정)
        pass

# LevelStat 객체 정의
class LevelStat:
    def __init__(self, level, hp, speed, defense):
        self.level = level
        self.hp = hp
        self.speed = speed
        self.defense = defense

# Skill 객체 정의
class Skill:
    def __init__(self, skill_id, skill_type, sin_affinity):
        self.skill_id = skill_id
        self.skill_type = skill_type
        self.sin_affinity = sin_affinity
        self.name = None
        self.power = None
        self.coin_count = None
    def update_details(self, name, power, coin_count):
        self.name = name
        self.power = power
        self.coin_count = coin_count

# CoinEffect 객체 정의
class CoinEffect:
    def __init__(self, uptie, coin_number, coin_effect):
        self.uptie = uptie
        self.coin_number = coin_number
        self.coin_effect = coin_effect

# BuffDebuffInfo 객체 정의
class BuffDebuffInfo:
    def __init__(self, name, effect_description, dispellable):
        self.name = name
        self.effect_description = effect_description
        self.dispellable = dispellable

# PassiveInfo 객체 정의
class PassiveInfo:
    def __init__(self, passive_type, passive_condition, passive_effect):
        self.passive_type = passive_type
        self.passive_condition = passive_condition
        self.passive_effect = passive_effect

class DataContainer:
    def __init__(self):
        self.identities = []

    def update_identity_name(self, identity_id, identity_name):
        """특정 identity_id의 name 값을 업데이트"""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_name(identity_name)
                return f"Identity {identity_id}의 rarity가 {identity_name}로 업데이트되었습니다."
        return f"Identity {identity_id}를 찾을 수 없습니다."

    def update_identity_rarity(self, identity_id, rarity):
        """특정 identity_id의 rarity 값을 업데이트"""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_rarity(rarity)
                return f"Identity {identity_id}의 rarity가 {rarity}로 업데이트되었습니다."
        return f"Identity {identity_id}를 찾을 수 없습니다."

    def add_identity(self, identity):
        self.identities.append(identity)

    def get_identity(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity
        return None

async def main():
    data_container = DataContainer()
    scraper = Scraper(container=data_container,base_url="https://limbus.kusoge.xyz")
    await scraper.scrape()

await main()

In [18]:
import os
import csv
from playwright.async_api import async_playwright
from tabulate import tabulate

# Sin Color 이름 변환을 위한 매핑
resource_mapping = {
    "AZURE": "Gloom",
    "VIOLET": "Envy",
    "AMBER": "Sloth",
    "SHAMROCK": "Gluttony",
    "CRIMSON": "Wrath",
    "SCARLET": "Lust"
}

def convert_resource(resource_name):
    """Sin Color 이름을 변환하는 함수. 매핑되지 않은 경우 원래 이름을 반환."""
    return resource_mapping.get(resource_name, resource_name)

class Scraper:
    """웹 페이지 크롤링을 위한 클래스"""
    def __init__(self, container, base_url):
        self.container = container
        self.base_url = base_url

    async def handle_request(self, route, request):
        """이미지와 폰트 리소스 로드를 차단."""
        if request.resource_type in ["image", "font"]:
            await route.abort()
        else:
            await route.continue_()
            
    async def scrape(self):
        """크롤링 메인 함수"""
        async with async_playwright() as p:
            browser = await p.chromium.launch(headless=False)
            page = await browser.new_page()
            await page.route("**/*", self.handle_request)

            # Step 1: /identity 페이지에서 기본 정보 크롤링
            await self.crawl_identity_page(page)

            # Step 2: /identity/{identity_id} 상세 페이지 크롤링
            await self.crawl_identity_details_for_all(page)

            await browser.close()

    async def crawl_identity_page(self, page):
        """/identity 페이지로 이동하여 인격 기본 정보를 추출."""
        await page.goto(self.base_url + "/identity")
        await self.extract_identity_page(page)

    async def crawl_identity_details_for_all(self, page):
        """모든 인격의 세부 정보를 크롤링."""
        for identity in self.container.identities:
            identity.update_details(page)

    async def get_identity_id_from_row(self, row):
        """테이블 행(row)에서 인격 ID를 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_url = await identity_cell.get_attribute('href')
            identity_id = identity_url.split('/identity/')[-1]
            return identity_id
        return "Unknown"

    async def get_identity_name_from_row(self, row):
        """테이블 행(row)에서 인격 이름을 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_name = (await identity_cell.inner_text()).replace("\n", " ").strip()
            return identity_name
        return "Unknown"

    async def scrape_identity_name_and_identity_id(self, row):
        """인격 번호와 이름을 추출하여 DataContainer에 저장."""
        try:
            identity_id = await self.get_identity_id_from_row(row)
            identity_name = await self.get_identity_name_from_row(row)
            identity = Identity(identity_id)
            self.container.add_identity(identity)
            self.container.update_identity_name(identity_id, identity_name)
        except Exception as e:
            print(f"Error extracting identity name and identity id: {e}")

    async def get_element_attribute(self, element, selector, attribute):
        """요소에서 지정된 속성(attribute)을 추출."""
        try:
            target_element = await element.query_selector(selector)
            if target_element:
                return await target_element.get_attribute(attribute)
        except Exception as e:
            print(f"Error getting element attribute: {e}")
        return None
    
    async def extract_rarity(self, row):
        """테이블 행(row)에서 희귀도를 추출하여 업데이트."""
        try:
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
                sibling = await parent.evaluate_handle('node => node.nextElementSibling')
                if sibling:
                    rarity_img_src = await self.get_element_attribute(sibling, 'img[src^="https://img.kusoge.xyz/limbus/img/ui/rarity_"]', 'src')
                    if rarity_img_src:
                        rarity_level = rarity_img_src.split('_')[-1].split('.')[0]  # rarity_1.png -> 1
                        identity_id = await self.get_identity_id_from_row(row)
                        self.container.update_identity_rarity(identity_id, rarity_level)
                    else:
                        print("Rarity image not found in the next sibling TD.")
        except Exception as e: 
            print(f"Error extracting rarity: {e}")

        return "Unknown"

    async def extract_atk_and_resource(self, row, skill_number):
        """ 공격 타입(ATK Type)과 자원(Attribute) 추출 함수 - 스킬 번호에 따라 처리 """
        try:
            #print(f"Attempting to locate skill{skill_number} type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 해당 스킬 번호에 맞는 형제로 이동
                sibling = parent
                for _ in range(skill_number + 1):  # 스킬 번호에 맞게 이동
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                atk_type, resource = "Unknown", "Unknown"
        
                if sibling:
                    # 공격 타입 이미지 (atk_type) 찾기
                    atk_type_img_src = await self.get_element_attribute(sibling, 'img[src^="https://img.kusoge.xyz/limbus/img/ui/atk_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if atk_type_img_src:
                        atk_type = atk_type_img_src.split('_')[-1].split('.')[0]  # atk_type_SLASH.png -> SLASH
                        #print(f"Extracted attack type: {atk_type}")
                        self.container.update_skill(identity_id, skill_number, atk_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, 'div.q-py-xs img[src^="https://img.kusoge.xyz/limbus/img/ui/attr_"]', 'src')
                    if resource_img_src:
                        resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted resource: {resource}")
                        
                        # 리소스를 변환
                        resource = convert_resource(resource)
                        #print(f"Converted resource: {resource}")
                        self.container.update_sin_affinity(identity_id, skill_number, resource)
                        
        except Exception as e:
            print(f"Error extracting skill{skill_number} type and resource: {e}")

    async def extract_defense_and_resource(self, row):
        """ 방어 타입(Def Type)과 자원(Attribute) 추출 함수 """
        try:
            #print("Attempting to locate defense type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 다섯 번째 형제 td로 이동 (방어 관련 정보가 있는 <td>)
                sibling = parent
                for _ in range(5):  # 방어 정보는 5번째 형제에 있음
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                def_type, def_resource = "Unknown", "Unknown"
        
                if sibling:
                    # 방어 타입 이미지 (def_type) 찾기
                    def_type_img_src = await self.get_element_attribute(sibling, 'img[src^="https://img.kusoge.xyz/limbus/img/ui/def_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if def_type_img_src:
                        def_type = def_type_img_src.split('_')[-1].split('.')[0]  # def_type_GUARD.png -> GUARD
                        #print(f"Extracted defense type: {def_type}")
                        self.container.update_skill(identity_id, 'defense', def_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, 'div.q-py-xs img[src^="https://img.kusoge.xyz/limbus/img/ui/attr_"]', 'src')
                    if resource_img_src:
                        def_resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted defense resource: {def_resource}")
                        
                        # 방어 리소스를 변환
                        def_resource = convert_resource(def_resource)
                        #print(f"Converted defense resource: {def_resource}")
                        self.container.update_sin_affinity(identity_id, 'defense', def_resource)
 
        except Exception as e:
            print(f"Error extracting defense type and resource: {e}")

    async def extract_identity_page(self, page):
        """/identity 페이지의 모든 테이블 행에서 인격 정보를 추출."""
        identity_rows = await page.query_selector_all('tr')
        if len(identity_rows) > 1:
            for idx, identity_row in enumerate(identity_rows):
                if idx == 0:
                    continue # 0번째요소이름을 보여주는 열은 건너뜀
                await self.scrape_identity_name_and_identity_id(identity_row)
                await self.extract_rarity(identity_row)
                for i in range(1, 4):  # 스킬1~3 정보 추출
                    await self.extract_atk_and_resource(identity_row, i)
                await self.extract_defense_and_resource(identity_row)
            #self.container.display_identities()
            self.container.export_identities_data("identities.csv")

class Identity:
    """인격 정보를 저장하는 클래스"""
    def __init__(self, identity_id, name=None, rarity=None):
        self.identity_id = identity_id
        self.name = name
        self.rarity = rarity
        # 선언만 하는 부분 
        self.skill_1_type = None 
        self.skill_1_sin_affinity = None 
        self.skill_2_type = None 
        self.skill_2_sin_affinity = None 
        self.skill_3_type = None 
        self.skill_3_sin_affinity = None 
        self.defense_type = None 
        self.defense_sin_affinity = None
        self.skills = []  # Skill 객체 리스트
        self.level_stats = []  # LevelStat 객체 리스트
        self.attack_types = {}
        self.stagger_thresholds = {}
        self.coin_effect = []  # CoinEffect 객체 리스트
        self.buff_debuff_info = []  # BuffDebuffInfo 객체 리스트
        self.passive_info = []  # PassiveInfo 객체 리스트

    def update_name(self, name):
        self.name = name

    def update_rarity(self, rarity):
        self.rarity = rarity

    def update_details(self, page):
        # 세부 정보 업데이트 로직 (가정)
        pass

class DataContainer:
    """크롤링 데이터를 관리하는 클래스"""
    def __init__(self):
        self.identities = []

    def update_identity_name(self, identity_id, identity_name):
        """특정 ID의 이름을 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_name(identity_name)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_identity_rarity(self, identity_id, rarity):
        """특정 ID의 희귀도를 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_rarity(rarity)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_skill(self, identity_id, skill_number, new_type):
        """
        특정 identity의 skill_type을 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_type: 새 skill_type
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_type = new_type
                elif skill_number == 2:
                    identity.skill_2_type = new_type
                elif skill_number == 3:
                    identity.skill_3_type = new_type
                elif skill_number == "defense":
                    identity.defense_type = new_type
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def update_sin_affinity(self, identity_id, skill_number, new_affinity):
        """
        특정 identity의 sin_affinity를 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_affinity: 새 sin_affinity
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_sin_affinity = new_affinity
                elif skill_number == 2:
                    identity.skill_2_sin_affinity = new_affinity
                elif skill_number == 3:
                    identity.skill_3_sin_affinity = new_affinity
                elif skill_number == "defense":
                    identity.defense_sin_affinity = new_affinity
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def add_identity(self, identity):
        """새로운 인격 데이터를 추가."""
        self.identities.append(identity)
        
    def display_identities(self):
        """Identity, ID, Name, Rarity 정보를 출력"""
        if not self.identities:
            print("No identities found.")
        else:
            # 데이터를 리스트로 변환
            table = []
            for idx, identity in enumerate(self.identities, 1):
                table.append([
                    idx, identity.identity_id, identity.name, identity.rarity,
                    identity.skill_1_type, identity.skill_1_sin_affinity,
                    identity.skill_2_type, identity.skill_2_sin_affinity,
                    identity.skill_3_type, identity.skill_3_sin_affinity,
                    identity.defense_type, identity.defense_sin_affinity
                ])
            
            # 테이블 헤더 정의
            headers = [
                "Index", "ID", "Name", "Rarity",
                "Skill_1 Type", "Skill_1 Sin", "Skill_2 Type", "Skill_2 Sin",
                "Skill_3 Type", "Skill_3 Sin", "Defense Type", "Defense Sin"
            ]
            
            # 테이블 출력
            print(tabulate(table, headers=headers, tablefmt="fancy_grid"))

    def export_to_csv(self, filename, fieldnames, data):
        """
        공통적으로 사용 가능한 CSV 내보내기 메서드
        :param filename: 내보낼 파일 이름 (예: 'identity.csv', 'skill.csv')
        :param fieldnames: CSV의 헤더 리스트
        :param data: CSV로 내보낼 데이터 (리스트 형태)
        """
        try:
            # 현재작업 디렉토 기준으 'csv'폴더 없으면 생성
            if not os.path.exists('./csv'):
                os.makedirs('./csv')
            # 파일 경로 설정 (상대경로 'csv' 폴더 안에 저장)
            file_path = os.path.join('./csv', filename)
            with open(file_path, mode='w', newline='', encoding='utf-8') as file:
                writer = csv.DictWriter(file, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(data)
            print(f"Data successfully exported to {filename}")
        except Exception as e:
            print(f"Error exporting to CSV: {e}")

    def get_identity_data(self):
        """
        Identity 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "ID": identity.identity_id,
                "Name": identity.name,
                "Rarity": identity.rarity,
                "Skill_1_Type": identity.skill_1_type,
                "Skill_1_Sin_Affinity": identity.skill_1_sin_affinity,
                "Skill_2_Type": identity.skill_2_type,
                "Skill_2_Sin_Affinity": identity.skill_2_sin_affinity,
                "Skill_3_Type": identity.skill_3_type,
                "Skill_3_Sin_Affinity": identity.skill_3_sin_affinity,
                "Defense_Type": identity.defense_type,
                "Defense_Sin_Affinity": identity.defense_sin_affinity,
            }
            for identity in self.identities
        ]
        
    def export_identities_data(self, filename="identity.csv"):
        """Identity 데이터를 CSV로 내보내는 메서드"""
        identity_fieldnames = [
            "ID", "Name", "Rarity",
            "Skill_1_Type", "Skill_1_Sin_Affinity",
            "Skill_2_Type", "Skill_2_Sin_Affinity",
            "Skill_3_Type", "Skill_3_Sin_Affinity",
            "Defense_Type", "Defense_Sin_Affinity"
        ]
        identity_data = self.get_identity_data()  # Identity 데이터 가져오기
        self.export_to_csv(filename, identity_fieldnames, identity_data)  # CSV로 내보내기

async def main():
    data_container = DataContainer()
    scraper = Scraper(container=data_container, base_url="https://limbus.kusoge.xyz")
    await scraper.scrape()

await main()

Data successfully exported to identities.csv


In [40]:
import os
import csv
from playwright.async_api import async_playwright
from tabulate import tabulate

# Sin Color 이름 변환을 위한 매핑
resource_mapping = {
    "AZURE": "Gloom",
    "VIOLET": "Envy",
    "AMBER": "Sloth",
    "SHAMROCK": "Gluttony",
    "CRIMSON": "Wrath",
    "SCARLET": "Lust",
    "INDIGO": "Pride"
}

def convert_resource(resource_name):
    """Sin Color 이름을 변환하는 함수. 매핑되지 않은 경우 원래 이름을 반환."""
    return resource_mapping.get(resource_name, resource_name)

class Scraper:
    """웹 페이지 크롤링을 위한 클래스"""
    def __init__(self, container, base_url, img_url):
        self.container = container
        self.base_url = base_url
        self.img_url = img_url

    async def handle_request(self, route, request):
        """이미지와 폰트 리소스 로드를 차단."""
        if request.resource_type in ["image", "font"]:
            await route.abort()
        else:
            await route.continue_()
            
    async def scrape(self):
        """크롤링 메인 함수"""
        async with async_playwright() as p:
            browser = await p.chromium.launch(headless=False)
            page = await browser.new_page()
            await page.route("**/*", self.handle_request)

            # Step 1: /identity 페이지에서 기본 정보 크롤링
            #await self.crawl_identity_page(page)

            # Step 2: /identity/{identity_id} 상세 페이지 크롤링
            #identity_ids_to_load = ["10104", "10310", "10710","10808", "10911", "11005"]
            identity_ids_to_load = ["10104", "10911"]#,"10310", "10710","10808", "11005"]
            self.container.load_identities_by_id('./csv/identities.csv', identity_ids_to_load)
            await self.crawl_identity_details_for_all(page)
            #self.container.display_identities()
            #self.container.export_identities_data("identities.csv")

            await browser.close()

    async def get_identity_id_from_row(self, row):
        """테이블 행(row)에서 인격 ID를 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_url = await identity_cell.get_attribute('href')
            identity_id = identity_url.split('/identity/')[-1]
            return identity_id
        return "Unknown"

    async def get_identity_name_from_row(self, row):
        """테이블 행(row)에서 인격 이름을 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_name = (await identity_cell.inner_text()).replace("\n", " ").strip()
            return identity_name
        return "Unknown"
        
    async def crawl_identity_page(self, page):
        """/identity 페이지로 이동하여 인격 기본 정보를 추출."""
        await page.goto(self.base_url + "/identity")
        await self.extract_identity_page(page)

    async def extract_identity_page(self, page):
        """/identity 페이지의 모든 테이블 행에서 인격 정보를 추출."""
        identity_rows = await page.query_selector_all('tr')
        if len(identity_rows) > 1:
            for idx, identity_row in enumerate(identity_rows):
                if idx == 0:
                    continue # 0번째요소이름을 보여주는 열은 건너뜀
                await self.scrape_identity_name_and_identity_id(identity_row)
                await self.extract_rarity(identity_row)
                for i in range(1, 4):  # 스킬1~3 정보 추출
                    await self.extract_atk_and_resource(identity_row, i)
                await self.extract_defense_and_resource(identity_row)

    async def scrape_identity_name_and_identity_id(self, row):
        """인격 번호와 이름을 추출하여 DataContainer에 저장."""
        try:
            identity_id = await self.get_identity_id_from_row(row)
            identity_name = await self.get_identity_name_from_row(row)
            identity = Identity(identity_id)
            self.container.add_identity(identity)
            self.container.update_identity_name(identity_id, identity_name)
        except Exception as e:
            print(f"Error extracting identity name and identity id: {e}")

    async def get_element_attribute(self, element, selector, attribute):
        """요소에서 지정된 속성(attribute)을 추출."""
        try:
            target_element = await element.query_selector(selector)
            if target_element:
                return await target_element.get_attribute(attribute)
        except Exception as e:
            print(f"Error getting element attribute: {e}")
        return None
    
    async def extract_rarity(self, row):
        """테이블 행(row)에서 희귀도를 추출하여 업데이트."""
        try:
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
                sibling = await parent.evaluate_handle('node => node.nextElementSibling')
                if sibling:
                    rarity_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/rarity_"]', 'src')
                    if rarity_img_src:
                        rarity_level = rarity_img_src.split('_')[-1].split('.')[0]  # rarity_1.png -> 1
                        identity_id = await self.get_identity_id_from_row(row)
                        self.container.update_identity_rarity(identity_id, rarity_level)
                    else:
                        print("Rarity image not found in the next sibling TD.")
        except Exception as e: 
            print(f"Error extracting rarity: {e}")

        return "Unknown"

    async def extract_atk_and_resource(self, row, skill_number):
        """ 공격 타입(ATK Type)과 자원(Attribute) 추출 함수 - 스킬 번호에 따라 처리 """
        try:
            #print(f"Attempting to locate skill{skill_number} type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 해당 스킬 번호에 맞는 형제로 이동
                sibling = parent
                for _ in range(skill_number + 1):  # 스킬 번호에 맞게 이동
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                atk_type, resource = "Unknown", "Unknown"
        
                if sibling:
                    # 공격 타입 이미지 (atk_type) 찾기
                    atk_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/atk_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if atk_type_img_src:
                        atk_type = atk_type_img_src.split('_')[-1].split('.')[0]  # atk_type_SLASH.png -> SLASH
                        #print(f"Extracted attack type: {atk_type}")
                        self.container.update_skill(identity_id, skill_number, atk_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted resource: {resource}")
                        
                        # 리소스를 변환
                        resource = convert_resource(resource)
                        #print(f"Converted resource: {resource}")
                        self.container.update_sin_affinity(identity_id, skill_number, resource)
                        
        except Exception as e:
            print(f"Error extracting skill{skill_number} type and resource: {e}")

    async def extract_defense_and_resource(self, row):
        """ 방어 타입(Def Type)과 자원(Attribute) 추출 함수 """
        try:
            #print("Attempting to locate defense type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 다섯 번째 형제 td로 이동 (방어 관련 정보가 있는 <td>)
                sibling = parent
                for _ in range(5):  # 방어 정보는 5번째 형제에 있음
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                def_type, def_resource = "Unknown", "Unknown"
        
                if sibling:
                    # 방어 타입 이미지 (def_type) 찾기
                    def_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/def_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if def_type_img_src:
                        def_type = def_type_img_src.split('_')[-1].split('.')[0]  # def_type_GUARD.png -> GUARD
                        #print(f"Extracted defense type: {def_type}")
                        self.container.update_skill(identity_id, 'defense', def_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        def_resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted defense resource: {def_resource}")
                        
                        # 방어 리소스를 변환
                        def_resource = convert_resource(def_resource)
                        #print(f"Converted defense resource: {def_resource}")
                        self.container.update_sin_affinity(identity_id, 'defense', def_resource)
 
        except Exception as e:
            print(f"Error extracting defense type and resource: {e}")

    async def crawl_identity_details_for_all(self, page):
        """모든 인격의 세부 정보를 크롤링."""
        for identity in self.container.identities:
            await page.goto(self.base_url + "/identity/"+identity.identity_id)
            print(f"Identity_id: {identity.identity_id}")
            await self.initialize_uptie_and_level(page)
            
    async def initialize_uptie_and_level(self, page):
        await self.uptie_click(page, 1)
        await self.move_slider_to_value(page, 1)
    
    async def uptie_click(self, page, phase):
        fab_button_selector = ".q-btn.q-btn--fab"
        setup_uptie_button = f'a:has(img[src="{self.img_url}/ui/uptie_{phase}.png"])'
        await page.click(fab_button_selector)
        #print(f"[{setup_uptie_button}] 상위 동기화 버튼 클릭 완료")
        await page.click(setup_uptie_button)
        #print(f"{phase} 동기화 버튼 클릭 완료")

    async def move_slider_to_value(self, page, target_value):
        """
        슬라이더를 주어진 값으로 이동시킵니다.
        target_value는 슬라이더의 최소값부터 최대값 사이여야 합니다.
        """
        slider_selector = ".q-slider"
        # 슬라이더의 범위 확인
        min_value = int(await page.get_attribute(slider_selector, "aria-valuemin"))
        max_value = int(await page.get_attribute(slider_selector, "aria-valuemax"))
    
        if target_value < min_value or target_value > max_value:
            raise ValueError(f"Target value {target_value} is out of bounds ({min_value}, {max_value})")
    
        # 슬라이더 thumb 요소와 위치 가져오기
        slider_thumb = await page.query_selector(f"{slider_selector} .q-slider__thumb")
        thumb_box = await slider_thumb.bounding_box()
        
        # 슬라이더의 현재 위치 및 목표 위치 계산
        track = await page.query_selector(f"{slider_selector} .q-slider__track")
        track_box = await track.bounding_box()
        
        # 목표 x 위치 계산 (슬라이더 트랙 내에서 비율 기반)
        target_percentage = (target_value - min_value) / (max_value - min_value)
        target_x = track_box["x"] + target_percentage * track_box["width"]
        current_x = thumb_box["x"] + thumb_box["width"] / 2
        y = thumb_box["y"] + thumb_box["height"] / 2
    
        # 슬라이더를 드래그하여 목표 위치로 이동
        await page.mouse.move(current_x, y)
        await page.mouse.down()
        await page.mouse.move(target_x, y, steps=10)  # 부드럽게 이동
        await page.mouse.up()
    
        print(f"슬라이더를 {target_value}로 이동 완료.")
    
        # 슬라이더 값 확인 및 강제 이벤트 트리거
        await page.evaluate(
            f"""
            (slider) => {{
                slider.dispatchEvent(new Event('input'));  // oninput 이벤트 트리거
                slider.dispatchEvent(new Event('change'));  // onchange 이벤트 트리거
            }}
            """,
            await page.query_selector(slider_selector)
        )
        #await asyncio.sleep(0.5)  # 값 업데이트를 위한 대기 시간
    
        # 이동 후 값 확인
        current_value = await page.get_attribute(slider_selector, "aria-valuenow")
        if current_value != str(target_value):
            print(f"경고: 슬라이더 값이 예상과 다릅니다. 현재 값: {current_value}, 예상 값: {target_value}")

class Identity:
    """인격 정보를 저장하는 클래스"""
    def __init__(self, identity_id, name=None, rarity=None,\
                 skill_1_type=None, skill_1_sin_affinity=None,\
                 skill_2_type=None, skill_2_sin_affinity=None,\
                 skill_3_type=None, skill_3_sin_affinity=None,\
                 defense_type=None, defense_sin_affinity=None):
        self.identity_id = identity_id
        self.name = name
        self.rarity = rarity
        # 선언만 하는 부분 
        self.skill_1_type = skill_1_type 
        self.skill_1_sin_affinity = skill_1_sin_affinity 
        self.skill_2_type = skill_2_type 
        self.skill_2_sin_affinity = skill_2_sin_affinity 
        self.skill_3_type = skill_3_type 
        self.skill_3_sin_affinity = skill_3_sin_affinity 
        self.defense_type = defense_type 
        self.defense_sin_affinity = defense_sin_affinity
        self.level_stats = []  # LevelStat 객체 리스트
        self.uptie_stats = []  # UptieStat 객체 리스트
        self.skills = []  # Skill 객체 리스트
        self.attack_types = {}
        self.stagger_thresholds = {}
        self.coin_effect = []  # CoinEffect 객체 리스트
        self.buff_debuff_info = []  # BuffDebuffInfo 객체 리스트
        self.passive_info = []  # PassiveInfo 객체 리스트

    def update_name(self, name):
        self.name = name

    def update_rarity(self, rarity):
        self.rarity = rarity

    def update_details(self, page):
        # 세부 정보 업데이트 로직 (가정)
        pass
    #def init_uptie(self, page):
    #    init_uptie = 

# LevelStat 객체 정의
class LevelStat:
    def __init__(self, level, hp=None, defense=None):
        self.level = level
        self.hp = hp
        self.defense = defense
    def update_level(self, level):
        self.level = level
    def update_hp(self, hp):
        self.hp = hp
    def update_defense(self, defense):
        self.defense = defense

# UptieStat 객체 정의
class UptieStat:
    def __init__(self, uptie, speed=None):
        self.uptie = uptie
        self.speed = speed
    def update_speed(self, speed):
        self.speed = speed
    
class DataContainer:
    """크롤링 데이터를 관리하는 클래스"""
    def __init__(self):
        self.identities = []

    def update_identity_name(self, identity_id, identity_name):
        """특정 ID의 이름을 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_name(identity_name)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_identity_rarity(self, identity_id, rarity):
        """특정 ID의 희귀도를 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_rarity(rarity)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_skill(self, identity_id, skill_number, new_type):
        """
        특정 identity의 skill_type을 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_type: 새 skill_type
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_type = new_type
                elif skill_number == 2:
                    identity.skill_2_type = new_type
                elif skill_number == 3:
                    identity.skill_3_type = new_type
                elif skill_number == "defense":
                    identity.defense_type = new_type
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def update_sin_affinity(self, identity_id, skill_number, new_affinity):
        """
        특정 identity의 sin_affinity를 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_affinity: 새 sin_affinity
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_sin_affinity = new_affinity
                elif skill_number == 2:
                    identity.skill_2_sin_affinity = new_affinity
                elif skill_number == 3:
                    identity.skill_3_sin_affinity = new_affinity
                elif skill_number == "defense":
                    identity.defense_sin_affinity = new_affinity
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def add_identity(self, identity):
        """새로운 인격 데이터를 추가."""
        self.identities.append(identity)
        
    def display_identities(self):
        """Identity, ID, Name, Rarity 정보를 출력"""
        if not self.identities:
            print("No identities found.")
        else:
            # 데이터를 리스트로 변환
            table = []
            for idx, identity in enumerate(self.identities, 1):
                table.append([
                    idx, identity.identity_id, identity.name, identity.rarity,
                    identity.skill_1_type, identity.skill_1_sin_affinity,
                    identity.skill_2_type, identity.skill_2_sin_affinity,
                    identity.skill_3_type, identity.skill_3_sin_affinity,
                    identity.defense_type, identity.defense_sin_affinity
                ])
            
            # 테이블 헤더 정의
            headers = [
                "Index", "ID", "Name", "Rarity",
                "Skill_1 Type", "Skill_1 Sin", "Skill_2 Type", "Skill_2 Sin",
                "Skill_3 Type", "Skill_3 Sin", "Defense Type", "Defense Sin"
            ]
            
            # 테이블 출력
            print(tabulate(table, headers=headers, tablefmt="fancy_grid"))

    def export_to_csv(self, filename, fieldnames, data):
        """
        공통적으로 사용 가능한 CSV 내보내기 메서드
        :param filename: 내보낼 파일 이름 (예: 'identity.csv', 'skill.csv')
        :param fieldnames: CSV의 헤더 리스트
        :param data: CSV로 내보낼 데이터 (리스트 형태)
        """
        try:
            # 현재작업 디렉토 기준으 'csv'폴더 없으면 생성
            if not os.path.exists('./csv'):
                os.makedirs('./csv')
            # 파일 경로 설정 (상대경로 'csv' 폴더 안에 저장)
            file_path = os.path.join('./csv', filename)
            with open(file_path, mode='w', newline='', encoding='utf-8') as file:
                writer = csv.DictWriter(file, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(data)
            print(f"Data successfully exported to {filename}")
        except Exception as e:
            print(f"Error exporting to CSV: {e}")

    def get_identity_data(self):
        """
        Identity 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "ID": identity.identity_id,
                "Name": identity.name,
                "Rarity": identity.rarity,
                "Skill_1_Type": identity.skill_1_type,
                "Skill_1_Sin_Affinity": identity.skill_1_sin_affinity,
                "Skill_2_Type": identity.skill_2_type,
                "Skill_2_Sin_Affinity": identity.skill_2_sin_affinity,
                "Skill_3_Type": identity.skill_3_type,
                "Skill_3_Sin_Affinity": identity.skill_3_sin_affinity,
                "Defense_Type": identity.defense_type,
                "Defense_Sin_Affinity": identity.defense_sin_affinity,
            }
            for identity in self.identities
        ]
        
    def export_identities_data(self, filename="identity.csv"):
        """Identity 데이터를 CSV로 내보내는 메서드"""
        identity_fieldnames = [
            "ID", "Name", "Rarity",
            "Skill_1_Type", "Skill_1_Sin_Affinity",
            "Skill_2_Type", "Skill_2_Sin_Affinity",
            "Skill_3_Type", "Skill_3_Sin_Affinity",
            "Defense_Type", "Defense_Sin_Affinity"
        ]
        identity_data = self.get_identity_data()  # Identity 데이터 가져오기
        self.export_to_csv(filename, identity_fieldnames, identity_data)  # CSV로 내보내기

    def load_identities_by_id(self, filename, identity_ids):
        """지정된 identity_ids 목록에 해당하는 Identity 데이터를 CSV에서 읽어옴."""
        identity_ids_set = set(identity_ids) # 중복 제거를 위 Set 사용
        with open(filename, mode='r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                if row["ID"] in identity_ids_set:
                    identity = Identity(
                        identity_id=row["ID"],
                        name=row["Name"],
                        rarity=row["Rarity"],
                        skill_1_type=row["Skill_1_Type"],
                        skill_1_sin_affinity=row["Skill_1_Sin_Affinity"],
                        skill_2_type=row["Skill_2_Type"],
                        skill_2_sin_affinity=row["Skill_2_Sin_Affinity"],
                        skill_3_type=row["Skill_3_Type"],
                        skill_3_sin_affinity=row["Skill_3_Sin_Affinity"],
                        defense_type=row["Defense_Type"],
                        defense_sin_affinity=row["Defense_Sin_Affinity"]
                    )
                    self.identities.append(identity)


async def main():
    data_container = DataContainer()
    scraper = Scraper(container=data_container, base_url="https://limbus.kusoge.xyz", img_url="https://img.kusoge.xyz/limbus/img")
    await scraper.scrape()

await main()

Identity_id: 10104
[a:has(img[src="https://img.kusoge.xyz/limbus/img/ui/uptie_1.png"])] 상위 동기화 버튼 클릭 완료
1 동기화 버튼 클릭 완료
슬라이더를 1로 이동 완료.
Identity_id: 10911
[a:has(img[src="https://img.kusoge.xyz/limbus/img/ui/uptie_1.png"])] 상위 동기화 버튼 클릭 완료
1 동기화 버튼 클릭 완료
슬라이더를 1로 이동 완료.


In [16]:
import os
import csv
from playwright.async_api import async_playwright
from tabulate import tabulate

# Sin Color 이름 변환을 위한 매핑
resource_mapping = {
    "AZURE": "Gloom",
    "VIOLET": "Envy",
    "AMBER": "Sloth",
    "SHAMROCK": "Gluttony",
    "CRIMSON": "Wrath",
    "SCARLET": "Lust",
    "INDIGO": "Pride"
}

def convert_resource(resource_name):
    """Sin Color 이름을 변환하는 함수. 매핑되지 않은 경우 원래 이름을 반환."""
    return resource_mapping.get(resource_name, resource_name)

class Scraper:
    """웹 페이지 크롤링을 위한 클래스"""
    def __init__(self, container, base_url, img_url):
        self.container = container
        self.base_url = base_url
        self.img_url = img_url

    async def handle_request(self, route, request):
        """이미지와 폰트 리소스 로드를 차단."""
        if request.resource_type in ["image", "font"]:
            await route.abort()
        else:
            await route.continue_()
            
    async def scrape(self):
        """크롤링 메인 함수"""
        async with async_playwright() as p:
            browser = await p.chromium.launch(headless=True)
            page = await browser.new_page()
            await page.route("**/*", self.handle_request)
            # Step 1: /identity 페이지에서 기본 정보 크롤링
            #await self.crawl_identity_page(page)

            # Step 2: /identity/{identity_id} 상세 페이지 크롤링
            #identity_ids_to_load = ["10104", "10310", "10710","10808", "10911", "11005"]
            identity_ids_to_load = ["10104", "10911"]#,"10310", "10710","10808", "11005"]
            self.container.load_identities_by_id('./csv/identities.csv', identity_ids_to_load)
            await self.crawl_identity_details_for_all(page)
            #self.container.display_identities()
            #self.container.export_identities_data("identities.csv")

            await browser.close()

    async def get_identity_id_from_row(self, row):
        """테이블 행(row)에서 인격 ID를 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_url = await identity_cell.get_attribute('href')
            identity_id = identity_url.split('/identity/')[-1]
            return identity_id
        return "Unknown"

    async def get_identity_name_from_row(self, row):
        """테이블 행(row)에서 인격 이름을 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_name = (await identity_cell.inner_text()).replace("\n", " ").strip()
            return identity_name
        return "Unknown"
        
    async def crawl_identity_page(self, page):
        """/identity 페이지로 이동하여 인격 기본 정보를 추출."""
        await page.goto(self.base_url + "/identity")
        await self.extract_identity_page(page)

    async def extract_identity_page(self, page):
        """/identity 페이지의 모든 테이블 행에서 인격 정보를 추출."""
        identity_rows = await page.query_selector_all('tr')
        if len(identity_rows) > 1:
            for idx, identity_row in enumerate(identity_rows):
                if idx == 0:
                    continue # 0번째요소이름을 보여주는 열은 건너뜀
                await self.scrape_identity_name_and_identity_id(identity_row)
                await self.extract_rarity(identity_row)
                for i in range(1, 4):  # 스킬1~3 정보 추출
                    await self.extract_atk_and_resource(identity_row, i)
                await self.extract_defense_and_resource(identity_row)

    async def scrape_identity_name_and_identity_id(self, row):
        """인격 번호와 이름을 추출하여 DataContainer에 저장."""
        try:
            identity_id = await self.get_identity_id_from_row(row)
            identity_name = await self.get_identity_name_from_row(row)
            identity = Identity(identity_id)
            self.container.add_identity(identity)
            self.container.update_identity_name(identity_id, identity_name)
        except Exception as e:
            print(f"Error extracting identity name and identity id: {e}")

    async def get_element_attribute(self, element, selector, attribute):
        """요소에서 지정된 속성(attribute)을 추출."""
        try:
            target_element = await element.query_selector(selector)
            if target_element:
                return await target_element.get_attribute(attribute)
        except Exception as e:
            print(f"Error getting element attribute: {e}")
        return None
    
    async def extract_rarity(self, row):
        """테이블 행(row)에서 희귀도를 추출하여 업데이트."""
        try:
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
                sibling = await parent.evaluate_handle('node => node.nextElementSibling')
                if sibling:
                    rarity_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/rarity_"]', 'src')
                    if rarity_img_src:
                        rarity_level = rarity_img_src.split('_')[-1].split('.')[0]  # rarity_1.png -> 1
                        identity_id = await self.get_identity_id_from_row(row)
                        self.container.update_identity_rarity(identity_id, rarity_level)
                    else:
                        print("Rarity image not found in the next sibling TD.")
        except Exception as e: 
            print(f"Error extracting rarity: {e}")

        return "Unknown"

    async def extract_atk_and_resource(self, row, skill_number):
        """ 공격 타입(ATK Type)과 자원(Attribute) 추출 함수 - 스킬 번호에 따라 처리 """
        try:
            #print(f"Attempting to locate skill{skill_number} type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 해당 스킬 번호에 맞는 형제로 이동
                sibling = parent
                for _ in range(skill_number + 1):  # 스킬 번호에 맞게 이동
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                atk_type, resource = "Unknown", "Unknown"
        
                if sibling:
                    # 공격 타입 이미지 (atk_type) 찾기
                    atk_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/atk_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if atk_type_img_src:
                        atk_type = atk_type_img_src.split('_')[-1].split('.')[0]  # atk_type_SLASH.png -> SLASH
                        #print(f"Extracted attack type: {atk_type}")
                        self.container.update_skill(identity_id, skill_number, atk_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted resource: {resource}")
                        
                        # 리소스를 변환
                        resource = convert_resource(resource)
                        #print(f"Converted resource: {resource}")
                        self.container.update_sin_affinity(identity_id, skill_number, resource)
                        
        except Exception as e:
            print(f"Error extracting skill{skill_number} type and resource: {e}")

    async def extract_defense_and_resource(self, row):
        """ 방어 타입(Def Type)과 자원(Attribute) 추출 함수 """
        try:
            #print("Attempting to locate defense type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 다섯 번째 형제 td로 이동 (방어 관련 정보가 있는 <td>)
                sibling = parent
                for _ in range(5):  # 방어 정보는 5번째 형제에 있음
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                def_type, def_resource = "Unknown", "Unknown"
        
                if sibling:
                    # 방어 타입 이미지 (def_type) 찾기
                    def_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/def_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if def_type_img_src:
                        def_type = def_type_img_src.split('_')[-1].split('.')[0]  # def_type_GUARD.png -> GUARD
                        #print(f"Extracted defense type: {def_type}")
                        self.container.update_skill(identity_id, 'defense', def_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        def_resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted defense resource: {def_resource}")
                        
                        # 방어 리소스를 변환
                        def_resource = convert_resource(def_resource)
                        #print(f"Converted defense resource: {def_resource}")
                        self.container.update_sin_affinity(identity_id, 'defense', def_resource)
 
        except Exception as e:
            print(f"Error extracting defense type and resource: {e}")

    async def crawl_identity_details_for_all(self, page):
        """모든 인격의 세부 정보를 크롤링."""
        for identity in self.container.identities:
            #await page.wait_for_load_state("networkidle")
            await page.goto(self.base_url + "/identity/"+identity.identity_id)
            print(f"Identity_id: {identity.identity_id}")
            await self.initialize_uptie_and_level(page)
            for i in range(1, identity.level_max+1):
                await self.move_slider_to_value(page,i)
                await self.extract_status(page,i)
            
    async def initialize_uptie_and_level(self, page):
        await self.uptie_click(page, 1)
        await self.move_slider_to_value(page, 1)
    
    async def uptie_click(self, page, phase):
        fab_button_selector = ".q-btn.q-btn--fab"
        setup_uptie_button = f'a:has(img[src="{self.img_url}/ui/uptie_{phase}.png"])'
        await page.click(fab_button_selector)
        #print(f"[{setup_uptie_button}] 상위 동기화 버튼 클릭 완료")
        await page.click(setup_uptie_button)
        #print(f"{phase} 동기화 버튼 클릭 완료")

    async def move_slider_to_value(self, page, target_value):
        """
        슬라이더를 주어진 값으로 이동시킵니다.
        target_value는 슬라이더의 최소값부터 최대값 사이여야 합니다.
        """
        slider_selector = ".q-slider"
        # 슬라이더의 범위 확인
        min_value = int(await page.get_attribute(slider_selector, "aria-valuemin"))
        max_value = int(await page.get_attribute(slider_selector, "aria-valuemax"))
    
        if target_value < min_value or target_value > max_value:
            raise ValueError(f"Target value {target_value} is out of bounds ({min_value}, {max_value})")
    
        # 슬라이더 thumb 요소와 위치 가져오기
        slider_thumb = await page.query_selector(f"{slider_selector} .q-slider__thumb")
        thumb_box = await slider_thumb.bounding_box()
        
        # 슬라이더의 현재 위치 및 목표 위치 계산
        track = await page.query_selector(f"{slider_selector} .q-slider__track")
        track_box = await track.bounding_box()
        
        # 목표 x 위치 계산 (슬라이더 트랙 내에서 비율 기반)
        target_percentage = (target_value - min_value) / (max_value - min_value)
        target_x = track_box["x"] + target_percentage * track_box["width"]
        current_x = thumb_box["x"] + thumb_box["width"] / 2
        y = thumb_box["y"] + thumb_box["height"] / 2
    
        # 슬라이더를 드래그하여 목표 위치로 이동
        await page.mouse.move(current_x, y)
        await page.mouse.down()
        await page.mouse.move(target_x, y, steps=10)  # 부드럽게 이동
        await page.mouse.up()
    
        print(f"슬라이더를 {target_value}로 이동 완료.")
    
        # 슬라이더 값 확인 및 강제 이벤트 트리거
        await page.evaluate(
            f"""
            (slider) => {{
                slider.dispatchEvent(new Event('input'));  // oninput 이벤트 트리거
                slider.dispatchEvent(new Event('change'));  // onchange 이벤트 트리거
            }}
            """,
            await page.query_selector(slider_selector)
        )
    
        # 이동 후 값 확인
        current_value = await page.get_attribute(slider_selector, "aria-valuenow")
        if current_value != str(target_value):
            print(f"경고: 슬라이더 값이 예상과 다릅니다. 현재 값: {current_value}, 예상 값: {target_value}")

    async def get_second_status_element(self, page):
        """
        두 번째 'Status' 요소에 접근하여 반환하는 메소드
        """
        status_elements = await page.query_selector_all("text=Status")
        if len(status_elements) > 1:
            second_status_element = status_elements[1]
            #print("두 번째 'Status' 요소에 접근합니다.")
            return second_status_element
        else:
            #print("두 번째 'Status' 요소가 존재하지 않습니다.")
            return None

    async def extract_status(self, page, level):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 레벨에 따라 speed는 변하 않는다.
        hp_value = await self.extract_hp(second_status_element)
        #speed_value = await self.extract_speed(second_status_element)
        defense_value = await self.extract_defense(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Level={level}, HP={hp_value}, Defense={defense_value}")
    
    async def extract_hp(self, status_element):
        # HP 추출
        hp_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].children[0].nextElementSibling.textContent.trim()"
        )
        return hp_value
    
    async def extract_defense(self, status_element):
        # Defense 추출
        defense_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return defense_value

    async def extract_speed(self, status_element):
        # Speed 추출
        speed_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return speed_value




class Identity:
    """인격 정보를 저장하는 클래스"""
    def __init__(self, identity_id, name=None, rarity=None,\
                 skill_1_type=None, skill_1_sin_affinity=None,\
                 skill_2_type=None, skill_2_sin_affinity=None,\
                 skill_3_type=None, skill_3_sin_affinity=None,\
                 defense_type=None, defense_sin_affinity=None):
        self.identity_id = identity_id
        self.name = name
        self.rarity = rarity
        # 선언만 하는 부분 
        self.skill_1_type = skill_1_type 
        self.skill_1_sin_affinity = skill_1_sin_affinity 
        self.skill_2_type = skill_2_type 
        self.skill_2_sin_affinity = skill_2_sin_affinity 
        self.skill_3_type = skill_3_type 
        self.skill_3_sin_affinity = skill_3_sin_affinity 
        self.defense_type = defense_type 
        self.defense_sin_affinity = defense_sin_affinity
        self.level_max = 50
        self.uptie_max = 4
        self.level_stats = []  # LevelStat 객체 리스트
        self.skills = []  # Skill 객체 리스트
        self.attack_types = {}
        self.stagger_thresholds = {}
        self.coin_effect = []  # CoinEffect 객체 리스트
        self.buff_debuff_info = []  # BuffDebuffInfo 객체 리스트
        self.passive_info = []  # PassiveInfo 객체 리스트

    def update_name(self, name):
        self.name = name

    def update_rarity(self, rarity):
        self.rarity = rarity

    def add_level_stat(self, level, hp, speed, defense):
        self.level_stats.append(LevelStat(level, hp, speed, defense))

    def update_details(self, page):
        # 세부 정보 업데이트 로직 (가정)
        pass
    #def init_uptie(self, page):
    #    init_uptie = 

# LevelStat 객체 정의
class LevelStat:
    def __init__(self, level, hp=None, speed=None, defense=None):
        self.level = level
        self.hp = hp
        self.speed = speed
        self.defense = defense
    def update_level(self, level):
        self.level = level
    def update_hp(self, hp):
        self.hp = hp
    def update_speed(self, speed):
        self.speed = speed
    def update_defense(self, defense):
        self.defense = defense
    
class DataContainer:
    """크롤링 데이터를 관리하는 클래스"""
    def __init__(self):
        self.identities = []

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.level_max

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.uptie_max

    def update_identity_name(self, identity_id, identity_name):
        """특정 ID의 이름을 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_name(identity_name)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_identity_rarity(self, identity_id, rarity):
        """특정 ID의 희귀도를 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_rarity(rarity)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_skill(self, identity_id, skill_number, new_type):
        """
        특정 identity의 skill_type을 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_type: 새 skill_type
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_type = new_type
                elif skill_number == 2:
                    identity.skill_2_type = new_type
                elif skill_number == 3:
                    identity.skill_3_type = new_type
                elif skill_number == "defense":
                    identity.defense_type = new_type
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def update_sin_affinity(self, identity_id, skill_number, new_affinity):
        """
        특정 identity의 sin_affinity를 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_affinity: 새 sin_affinity
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_sin_affinity = new_affinity
                elif skill_number == 2:
                    identity.skill_2_sin_affinity = new_affinity
                elif skill_number == 3:
                    identity.skill_3_sin_affinity = new_affinity
                elif skill_number == "defense":
                    identity.defense_sin_affinity = new_affinity
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def add_identity(self, identity):
        """새로운 인격 데이터를 추가."""
        self.identities.append(identity)
        
    def display_identities(self):
        """Identity, ID, Name, Rarity 정보를 출력"""
        if not self.identities:
            print("No identities found.")
        else:
            # 데이터를 리스트로 변환
            table = []
            for idx, identity in enumerate(self.identities, 1):
                table.append([
                    idx, identity.identity_id, identity.name, identity.rarity,
                    identity.skill_1_type, identity.skill_1_sin_affinity,
                    identity.skill_2_type, identity.skill_2_sin_affinity,
                    identity.skill_3_type, identity.skill_3_sin_affinity,
                    identity.defense_type, identity.defense_sin_affinity
                ])
            
            # 테이블 헤더 정의
            headers = [
                "Index", "ID", "Name", "Rarity",
                "Skill_1 Type", "Skill_1 Sin", "Skill_2 Type", "Skill_2 Sin",
                "Skill_3 Type", "Skill_3 Sin", "Defense Type", "Defense Sin"
            ]
            
            # 테이블 출력
            print(tabulate(table, headers=headers, tablefmt="fancy_grid"))

    def export_to_csv(self, filename, fieldnames, data):
        """
        공통적으로 사용 가능한 CSV 내보내기 메서드
        :param filename: 내보낼 파일 이름 (예: 'identity.csv', 'skill.csv')
        :param fieldnames: CSV의 헤더 리스트
        :param data: CSV로 내보낼 데이터 (리스트 형태)
        """
        try:
            # 현재작업 디렉토 기준으 'csv'폴더 없으면 생성
            if not os.path.exists('./csv'):
                os.makedirs('./csv')
            # 파일 경로 설정 (상대경로 'csv' 폴더 안에 저장)
            file_path = os.path.join('./csv', filename)
            with open(file_path, mode='w', newline='', encoding='utf-8') as file:
                writer = csv.DictWriter(file, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(data)
            print(f"Data successfully exported to {filename}")
        except Exception as e:
            print(f"Error exporting to CSV: {e}")

    def get_identity_data(self):
        """
        Identity 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "ID": identity.identity_id,
                "Name": identity.name,
                "Rarity": identity.rarity,
                "Skill_1_Type": identity.skill_1_type,
                "Skill_1_Sin_Affinity": identity.skill_1_sin_affinity,
                "Skill_2_Type": identity.skill_2_type,
                "Skill_2_Sin_Affinity": identity.skill_2_sin_affinity,
                "Skill_3_Type": identity.skill_3_type,
                "Skill_3_Sin_Affinity": identity.skill_3_sin_affinity,
                "Defense_Type": identity.defense_type,
                "Defense_Sin_Affinity": identity.defense_sin_affinity,
            }
            for identity in self.identities
        ]
        
    def export_identities_data(self, filename="identity.csv"):
        """Identity 데이터를 CSV로 내보내는 메서드"""
        identity_fieldnames = [
            "ID", "Name", "Rarity",
            "Skill_1_Type", "Skill_1_Sin_Affinity",
            "Skill_2_Type", "Skill_2_Sin_Affinity",
            "Skill_3_Type", "Skill_3_Sin_Affinity",
            "Defense_Type", "Defense_Sin_Affinity"
        ]
        identity_data = self.get_identity_data()  # Identity 데이터 가져오기
        self.export_to_csv(filename, identity_fieldnames, identity_data)  # CSV로 내보내기

    def load_identities_by_id(self, filename, identity_ids):
        """지정된 identity_ids 목록에 해당하는 Identity 데이터를 CSV에서 읽어옴."""
        identity_ids_set = set(identity_ids) # 중복 제거를 위 Set 사용
        with open(filename, mode='r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                if row["ID"] in identity_ids_set:
                    identity = Identity(
                        identity_id=row["ID"],
                        name=row["Name"],
                        rarity=row["Rarity"],
                        skill_1_type=row["Skill_1_Type"],
                        skill_1_sin_affinity=row["Skill_1_Sin_Affinity"],
                        skill_2_type=row["Skill_2_Type"],
                        skill_2_sin_affinity=row["Skill_2_Sin_Affinity"],
                        skill_3_type=row["Skill_3_Type"],
                        skill_3_sin_affinity=row["Skill_3_Sin_Affinity"],
                        defense_type=row["Defense_Type"],
                        defense_sin_affinity=row["Defense_Sin_Affinity"]
                    )
                    self.identities.append(identity)

    def get_level_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Level": level_stat.level,
                "HP": level_stat.hp,
                "Speed": level_stat.speed,
                "Defense": level_stat.defense,
            }
            for identity in self.identities
            for level_stat in identity.level_stats
        ]

    def export_level_stats_data(self, filename="levelstat.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        level_stats_fieldnames = ["Identity_ID", "Level", "HP", "Speed", "Defense"]
        level_stats_data = self.get_level_stats_data()  # level_stats 데이터 가져오기
        self.export_to_csv(filename, level_stats_fieldnames, level_stats_data)  # CSV로 내보내기


async def main():
    data_container = DataContainer()
    scraper = Scraper(container=data_container, base_url="https://limbus.kusoge.xyz", img_url="https://img.kusoge.xyz/limbus/img")
    await scraper.scrape()

await main()

Identity_id: 10104
슬라이더를 1로 이동 완료.
슬라이더를 1로 이동 완료.
Level=1, HP=68, Defense=1
슬라이더를 2로 이동 완료.
Level=2, HP=71, Defense=1
슬라이더를 3로 이동 완료.
Level=3, HP=73, Defense=1
슬라이더를 4로 이동 완료.
Level=4, HP=75, Defense=1
슬라이더를 5로 이동 완료.
Level=5, HP=77, Defense=1
슬라이더를 6로 이동 완료.
Level=6, HP=80, Defense=2
슬라이더를 7로 이동 완료.
Level=7, HP=82, Defense=3
슬라이더를 8로 이동 완료.
Level=8, HP=84, Defense=4
슬라이더를 9로 이동 완료.
Level=9, HP=87, Defense=5
슬라이더를 10로 이동 완료.
Level=10, HP=89, Defense=6
슬라이더를 11로 이동 완료.
Level=11, HP=91, Defense=7
슬라이더를 12로 이동 완료.
Level=12, HP=93, Defense=8
슬라이더를 13로 이동 완료.
Level=13, HP=96, Defense=9
슬라이더를 14로 이동 완료.
Level=14, HP=98, Defense=10
슬라이더를 15로 이동 완료.
Level=15, HP=100, Defense=11
슬라이더를 16로 이동 완료.
Level=16, HP=102, Defense=12
슬라이더를 17로 이동 완료.
Level=17, HP=105, Defense=13
슬라이더를 18로 이동 완료.
Level=18, HP=107, Defense=14
슬라이더를 19로 이동 완료.
Level=19, HP=109, Defense=15
슬라이더를 20로 이동 완료.
Level=20, HP=112, Defense=16
슬라이더를 21로 이동 완료.
Level=21, HP=114, Defense=17
슬라이더를 22로 이동 완료.
Level=22, HP=116, Defense=1

In [30]:
import os
import csv
from playwright.async_api import async_playwright
from tabulate import tabulate

# Sin Color 이름 변환을 위한 매핑
resource_mapping = {
    "AZURE": "Gloom",
    "VIOLET": "Envy",
    "AMBER": "Sloth",
    "SHAMROCK": "Gluttony",
    "CRIMSON": "Wrath",
    "SCARLET": "Lust",
    "INDIGO": "Pride"
}

def convert_resource(resource_name):
    """Sin Color 이름을 변환하는 함수. 매핑되지 않은 경우 원래 이름을 반환."""
    return resource_mapping.get(resource_name, resource_name)

class Scraper:
    """웹 페이지 크롤링을 위한 클래스"""
    def __init__(self, container, base_url, img_url):
        self.container = container
        self.base_url = base_url
        self.img_url = img_url

    async def handle_request(self, route, request):
        """이미지와 폰트 리소스 로드를 차단."""
        if request.resource_type in ["image", "font"]:
            await route.abort()
        else:
            await route.continue_()
            
    async def scrape(self):
        """크롤링 메인 함수"""
        async with async_playwright() as p:
            browser = await p.chromium.launch(headless=True)
            page = await browser.new_page()
            await page.route("**/*", self.handle_request)
            # Step 1: /identity 페이지에서 기본 정보 크롤링
            #await self.crawl_identity_page(page)

            # Step 2: /identity/{identity_id} 상세 페이지 크롤링
            #identity_ids_to_load = ["10104", "10310", "10710","10808", "10911", "11005"]
            identity_ids_to_load = ["10104", "10911"]#,"10310", "10710","10808", "11005"]
            self.container.load_identities_by_id('./csv/identities.csv', identity_ids_to_load)
            await self.crawl_identity_details_for_all(page)
            #self.container.display_identities()
            #self.container.export_identities_data("identities.csv")
            #self.container.export_level_stats_data("levelstats.csv")
            #self.container.export_uptie_stats_data("Uptiestats.csv")
            page.set_default_timeout(10000)
            await browser.close()

    async def get_identity_id_from_row(self, row):
        """테이블 행(row)에서 인격 ID를 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_url = await identity_cell.get_attribute('href')
            identity_id = identity_url.split('/identity/')[-1]
            return identity_id
        return "Unknown"

    async def get_identity_name_from_row(self, row):
        """테이블 행(row)에서 인격 이름을 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_name = (await identity_cell.inner_text()).replace("\n", " ").strip()
            return identity_name
        return "Unknown"
        
    async def crawl_identity_page(self, page):
        """/identity 페이지로 이동하여 인격 기본 정보를 추출."""
        await page.goto(self.base_url + "/identity")
        await self.extract_identity_page(page)

    async def extract_identity_page(self, page):
        """/identity 페이지의 모든 테이블 행에서 인격 정보를 추출."""
        identity_rows = await page.query_selector_all('tr')
        if len(identity_rows) > 1:
            for idx, identity_row in enumerate(identity_rows):
                if idx == 0:
                    continue # 0번째요소이름을 보여주는 열은 건너뜀
                await self.scrape_identity_name_and_identity_id(identity_row)
                await self.extract_rarity(identity_row)
                for i in range(1, 4):  # 스킬1~3 정보 추출
                    await self.extract_atk_and_resource(identity_row, i)
                await self.extract_defense_and_resource(identity_row)

    async def scrape_identity_name_and_identity_id(self, row):
        """인격 번호와 이름을 추출하여 DataContainer에 저장."""
        try:
            identity_id = await self.get_identity_id_from_row(row)
            identity_name = await self.get_identity_name_from_row(row)
            identity = Identity(identity_id)
            self.container.add_identity(identity)
            self.container.update_identity_name(identity_id, identity_name)
        except Exception as e:
            print(f"Error extracting identity name and identity id: {e}")

    async def get_element_attribute(self, element, selector, attribute):
        """요소에서 지정된 속성(attribute)을 추출."""
        try:
            target_element = await element.query_selector(selector)
            if target_element:
                return await target_element.get_attribute(attribute)
        except Exception as e:
            print(f"Error getting element attribute: {e}")
        return None
    
    async def extract_rarity(self, row):
        """테이블 행(row)에서 희귀도를 추출하여 업데이트."""
        try:
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
                sibling = await parent.evaluate_handle('node => node.nextElementSibling')
                if sibling:
                    rarity_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/rarity_"]', 'src')
                    if rarity_img_src:
                        rarity_level = rarity_img_src.split('_')[-1].split('.')[0]  # rarity_1.png -> 1
                        identity_id = await self.get_identity_id_from_row(row)
                        self.container.update_identity_rarity(identity_id, rarity_level)
                    else:
                        print("Rarity image not found in the next sibling TD.")
        except Exception as e: 
            print(f"Error extracting rarity: {e}")

        return "Unknown"

    async def extract_atk_and_resource(self, row, skill_number):
        """ 공격 타입(ATK Type)과 자원(Attribute) 추출 함수 - 스킬 번호에 따라 처리 """
        try:
            #print(f"Attempting to locate skill{skill_number} type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 해당 스킬 번호에 맞는 형제로 이동
                sibling = parent
                for _ in range(skill_number + 1):  # 스킬 번호에 맞게 이동
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                atk_type, resource = "Unknown", "Unknown"
        
                if sibling:
                    # 공격 타입 이미지 (atk_type) 찾기
                    atk_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/atk_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if atk_type_img_src:
                        atk_type = atk_type_img_src.split('_')[-1].split('.')[0]  # atk_type_SLASH.png -> SLASH
                        #print(f"Extracted attack type: {atk_type}")
                        self.container.update_skill(identity_id, skill_number, atk_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted resource: {resource}")
                        
                        # 리소스를 변환
                        resource = convert_resource(resource)
                        #print(f"Converted resource: {resource}")
                        self.container.update_sin_affinity(identity_id, skill_number, resource)
                        
        except Exception as e:
            print(f"Error extracting skill{skill_number} type and resource: {e}")

    async def extract_defense_and_resource(self, row):
        """ 방어 타입(Def Type)과 자원(Attribute) 추출 함수 """
        try:
            #print("Attempting to locate defense type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 다섯 번째 형제 td로 이동 (방어 관련 정보가 있는 <td>)
                sibling = parent
                for _ in range(5):  # 방어 정보는 5번째 형제에 있음
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                def_type, def_resource = "Unknown", "Unknown"
        
                if sibling:
                    # 방어 타입 이미지 (def_type) 찾기
                    def_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/def_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if def_type_img_src:
                        def_type = def_type_img_src.split('_')[-1].split('.')[0]  # def_type_GUARD.png -> GUARD
                        #print(f"Extracted defense type: {def_type}")
                        self.container.update_skill(identity_id, 'defense', def_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        def_resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted defense resource: {def_resource}")
                        
                        # 방어 리소스를 변환
                        def_resource = convert_resource(def_resource)
                        #print(f"Converted defense resource: {def_resource}")
                        self.container.update_sin_affinity(identity_id, 'defense', def_resource)
 
        except Exception as e:
            print(f"Error extracting defense type and resource: {e}")

    async def crawl_identity_details_for_all(self, page):
        """모든 인격의 세부 정보를 크롤링."""
        for identity in self.container.identities:
            #await page.wait_for_load_state("networkidle")
            await page.goto(self.base_url + "/identity/"+identity.identity_id)
            print(f"Identity_id: {identity.identity_id}")
            await self.initialize_uptie_and_level(page)
            """ 레벨에 따른 level, hp, defense 다르게 되어 있는 것을 출력 확인 객체로 들어가서 csv로 옮기는 과정까지 진행
            for i in range(1, identity.level_max+1):
                await self.move_slider_to_value(page,i)
                await self.extract_level_status(page, identity.identity_id, i)
            """
            """ 동기화에 따른 speed 다르게 되어 있는 것을 출력 확인 객체로 들어가서 csv로 옮기는 과정까지 진행
            for j in range(1, identity.uptie_max+1):
                await self.uptie_click(page,j)
            """
            
    async def initialize_uptie_and_level(self, page):
        await self.uptie_click(page, 1)
        await self.move_slider_to_value(page, 1)
    
    async def uptie_click(self, page, phase):
        fab_button_selector = ".q-btn.q-btn--fab"
        setup_uptie_button = f'a:has(img[src="{self.img_url}/ui/uptie_{phase}.png"])'
        await page.click(fab_button_selector)
        #print(f"[{setup_uptie_button}] 상위 동기화 버튼 클릭 완료")
        await page.click(setup_uptie_button)
        #print(f"{phase} 동기화 버튼 클릭 완료")

    async def move_slider_to_value(self, page, target_value):
        """
        슬라이더를 주어진 값으로 이동시킵니다.
        target_value는 슬라이더의 최소값부터 최대값 사이여야 합니다.
        """
        slider_selector = ".q-slider"
        # 슬라이더의 범위 확인
        min_value = int(await page.get_attribute(slider_selector, "aria-valuemin"))
        max_value = int(await page.get_attribute(slider_selector, "aria-valuemax"))
    
        if target_value < min_value or target_value > max_value:
            raise ValueError(f"Target value {target_value} is out of bounds ({min_value}, {max_value})")
    
        # 슬라이더 thumb 요소와 위치 가져오기
        slider_thumb = await page.query_selector(f"{slider_selector} .q-slider__thumb")
        thumb_box = await slider_thumb.bounding_box()
        
        # 슬라이더의 현재 위치 및 목표 위치 계산
        track = await page.query_selector(f"{slider_selector} .q-slider__track")
        track_box = await track.bounding_box()
        
        # 목표 x 위치 계산 (슬라이더 트랙 내에서 비율 기반)
        target_percentage = (target_value - min_value) / (max_value - min_value)
        target_x = track_box["x"] + target_percentage * track_box["width"]
        current_x = thumb_box["x"] + thumb_box["width"] / 2
        y = thumb_box["y"] + thumb_box["height"] / 2
    
        # 슬라이더를 드래그하여 목표 위치로 이동
        await page.mouse.move(current_x, y)
        await page.mouse.down()
        await page.mouse.move(target_x, y, steps=10)  # 부드럽게 이동
        await page.mouse.up()
    
        #print(f"슬라이더를 {target_value}로 이동 완료.")
    
        # 슬라이더 값 확인 및 강제 이벤트 트리거
        await page.evaluate(
            f"""
            (slider) => {{
                slider.dispatchEvent(new Event('input'));  // oninput 이벤트 트리거
                slider.dispatchEvent(new Event('change'));  // onchange 이벤트 트리거
            }}
            """,
            await page.query_selector(slider_selector)
        )
    
        # 이동 후 값 확인
        current_value = await page.get_attribute(slider_selector, "aria-valuenow")
        if current_value != str(target_value):
            print(f"경고: 슬라이더 값이 예상과 다릅니다. 현재 값: {current_value}, 예상 값: {target_value}")

    async def get_second_status_element(self, page):
        """
        두 번째 'Status' 요소에 접근하여 반환하는 메소드
        """
        status_elements = await page.query_selector_all("text=Status")
        if len(status_elements) > 1:
            second_status_element = status_elements[1]
            #print("두 번째 'Status' 요소에 접근합니다.")
            return second_status_element
        else:
            #print("두 번째 'Status' 요소가 존재하지 않습니다.")
            return None

    async def extract_level_status(self, page, identity_id, level):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 레벨에 따라 HP와 Defense가 변한다.
        hp_value = await self.extract_hp(second_status_element)
        defense_value = await self.extract_defense(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Level={level}, HP={hp_value}, Defense={defense_value}")
        self.container.add_level_stat(identity_id,level, hp_value, defense_value)
    
    async def extract_hp(self, status_element):
        # HP 추출
        hp_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].children[0].nextElementSibling.textContent.trim()"
        )
        return hp_value
    
    async def extract_defense(self, status_element):
        # Defense 추출
        defense_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return defense_value

    async def extract_uptie_status(self, page, identity_id, uptie):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 uptie에 따라 speed는 변한다.
        speed_value = await self.extract_speed(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Uptie={uptie}, Speed={speed_value}")
        self.container.add_uptie_stat(identity_id,uptie, speed_value)

    async def extract_speed(self, status_element):
        # Speed 추출
        speed_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return speed_value

class Identity:
    """인격 정보를 저장하는 클래스"""
    def __init__(self, identity_id, name=None, rarity=None,\
                 skill_1_type=None, skill_1_sin_affinity=None,\
                 skill_2_type=None, skill_2_sin_affinity=None,\
                 skill_3_type=None, skill_3_sin_affinity=None,\
                 defense_type=None, defense_sin_affinity=None):
        self.identity_id = identity_id
        self.name = name
        self.rarity = rarity
        # 선언만 하는 부분 
        self.skill_1_type = skill_1_type 
        self.skill_1_sin_affinity = skill_1_sin_affinity 
        self.skill_2_type = skill_2_type 
        self.skill_2_sin_affinity = skill_2_sin_affinity 
        self.skill_3_type = skill_3_type 
        self.skill_3_sin_affinity = skill_3_sin_affinity 
        self.defense_type = defense_type 
        self.defense_sin_affinity = defense_sin_affinity
        self.level_max = 50
        self.uptie_max = 4
        self.level_stats = []  # LevelStat 객체 리스트
        self.uptie_stats = [] # UptieStat 객체 리스트
        self.resist_info = None
        self.skills = []  # Skill 객체 리스트
        self.attack_types = {}
        self.stagger_thresholds = {}
        self.coin_effect = []  # CoinEffect 객체 리스트
        self.buff_debuff_info = []  # BuffDebuffInfo 객체 리스트
        self.passive_info = []  # PassiveInfo 객체 리스트

    def update_name(self, name):
        self.name = name

    def update_rarity(self, rarity):
        self.rarity = rarity

    def add_level_stat(self, level, hp, defense):
        self.level_stats.append(LevelStat(level, hp, defense))

    def add_uptie_stat(self, uptie, speed):
        self.uptie_stats.append(UptieStat(uptie, speed))

    def update_details(self, page):
        # 세부 정보 업데이트 로직 (가정)
        pass
    #def init_uptie(self, page):
    #    init_uptie = 

# LevelStat 객체 정의
class LevelStat:
    def __init__(self, level, hp=None, defense=None):
        self.level = level
        self.hp = hp
        self.defense = defense
    def update_level(self, level):
        self.level = level
    def update_hp(self, hp):
        self.hp = hp
    def update_defense(self, defense):
        self.defense = defense

# UptieStat 객체 정의
class UptieStat:
    def __init__(self, uptie, speed=None):
        self.uptie = uptie
        self.speed = speed
    def update_speed(self, speed):
        self.speed = speed
    
class DataContainer:
    """크롤링 데이터를 관리하는 클래스"""
    def __init__(self):
        self.identities = []

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.level_max

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.uptie_max

    def update_identity_name(self, identity_id, identity_name):
        """특정 ID의 이름을 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_name(identity_name)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_identity_rarity(self, identity_id, rarity):
        """특정 ID의 희귀도를 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_rarity(rarity)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_skill(self, identity_id, skill_number, new_type):
        """
        특정 identity의 skill_type을 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_type: 새 skill_type
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_type = new_type
                elif skill_number == 2:
                    identity.skill_2_type = new_type
                elif skill_number == 3:
                    identity.skill_3_type = new_type
                elif skill_number == "defense":
                    identity.defense_type = new_type
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def update_sin_affinity(self, identity_id, skill_number, new_affinity):
        """
        특정 identity의 sin_affinity를 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_affinity: 새 sin_affinity
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_sin_affinity = new_affinity
                elif skill_number == 2:
                    identity.skill_2_sin_affinity = new_affinity
                elif skill_number == 3:
                    identity.skill_3_sin_affinity = new_affinity
                elif skill_number == "defense":
                    identity.defense_sin_affinity = new_affinity
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def add_identity(self, identity):
        """새로운 인격 데이터를 추가."""
        self.identities.append(identity)

    def add_level_stat(self, identity_id, level, hp, defense):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_level_stat(level, hp, defense)

    def add_uptie_stat(self, identity_id, uptie, speed):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_uptie_stat(uptie, speed)
        
    def display_identities(self):
        """Identity, ID, Name, Rarity 정보를 출력"""
        if not self.identities:
            print("No identities found.")
        else:
            # 데이터를 리스트로 변환
            table = []
            for idx, identity in enumerate(self.identities, 1):
                table.append([
                    idx, identity.identity_id, identity.name, identity.rarity,
                    identity.skill_1_type, identity.skill_1_sin_affinity,
                    identity.skill_2_type, identity.skill_2_sin_affinity,
                    identity.skill_3_type, identity.skill_3_sin_affinity,
                    identity.defense_type, identity.defense_sin_affinity
                ])
            
            # 테이블 헤더 정의
            headers = [
                "Index", "ID", "Name", "Rarity",
                "Skill_1 Type", "Skill_1 Sin", "Skill_2 Type", "Skill_2 Sin",
                "Skill_3 Type", "Skill_3 Sin", "Defense Type", "Defense Sin"
            ]
            
            # 테이블 출력
            print(tabulate(table, headers=headers, tablefmt="fancy_grid"))

    def export_to_csv(self, filename, fieldnames, data):
        """
        공통적으로 사용 가능한 CSV 내보내기 메서드
        :param filename: 내보낼 파일 이름 (예: 'identity.csv', 'skill.csv')
        :param fieldnames: CSV의 헤더 리스트
        :param data: CSV로 내보낼 데이터 (리스트 형태)
        """
        try:
            # 현재작업 디렉토 기준으 'csv'폴더 없으면 생성
            if not os.path.exists('./csv'):
                os.makedirs('./csv')
            # 파일 경로 설정 (상대경로 'csv' 폴더 안에 저장)
            file_path = os.path.join('./csv', filename)
            with open(file_path, mode='w', newline='', encoding='utf-8') as file:
                writer = csv.DictWriter(file, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(data)
            print(f"Data successfully exported to {filename}")
        except Exception as e:
            print(f"Error exporting to CSV: {e}")

    def get_identity_data(self):
        """
        Identity 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "ID": identity.identity_id,
                "Name": identity.name,
                "Rarity": identity.rarity,
                "Skill_1_Type": identity.skill_1_type,
                "Skill_1_Sin_Affinity": identity.skill_1_sin_affinity,
                "Skill_2_Type": identity.skill_2_type,
                "Skill_2_Sin_Affinity": identity.skill_2_sin_affinity,
                "Skill_3_Type": identity.skill_3_type,
                "Skill_3_Sin_Affinity": identity.skill_3_sin_affinity,
                "Defense_Type": identity.defense_type,
                "Defense_Sin_Affinity": identity.defense_sin_affinity,
            }
            for identity in self.identities
        ]
        
    def export_identities_data(self, filename="identities.csv"):
        """Identity 데이터를 CSV로 내보내는 메서드"""
        identity_fieldnames = [
            "ID", "Name", "Rarity",
            "Skill_1_Type", "Skill_1_Sin_Affinity",
            "Skill_2_Type", "Skill_2_Sin_Affinity",
            "Skill_3_Type", "Skill_3_Sin_Affinity",
            "Defense_Type", "Defense_Sin_Affinity"
        ]
        identity_data = self.get_identity_data()  # Identity 데이터 가져오기
        self.export_to_csv(filename, identity_fieldnames, identity_data)  # CSV로 내보내기

    def load_identities_by_id(self, filename, identity_ids):
        """지정된 identity_ids 목록에 해당하는 Identity 데이터를 CSV에서 읽어옴."""
        identity_ids_set = set(identity_ids) # 중복 제거를 위 Set 사용
        with open(filename, mode='r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                if row["ID"] in identity_ids_set:
                    identity = Identity(
                        identity_id=row["ID"],
                        name=row["Name"],
                        rarity=row["Rarity"],
                        skill_1_type=row["Skill_1_Type"],
                        skill_1_sin_affinity=row["Skill_1_Sin_Affinity"],
                        skill_2_type=row["Skill_2_Type"],
                        skill_2_sin_affinity=row["Skill_2_Sin_Affinity"],
                        skill_3_type=row["Skill_3_Type"],
                        skill_3_sin_affinity=row["Skill_3_Sin_Affinity"],
                        defense_type=row["Defense_Type"],
                        defense_sin_affinity=row["Defense_Sin_Affinity"]
                    )
                    self.identities.append(identity)

    def get_level_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Level": level_stat.level,
                "HP": level_stat.hp,
                "Defense": level_stat.defense,
            }
            for identity in self.identities
            for level_stat in identity.level_stats
        ]

    def export_level_stats_data(self, filename="levelstats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        level_stats_fieldnames = ["Identity_ID", "Level", "HP", "Defense"]
        level_stats_data = self.get_level_stats_data()  # level_stats 데이터 가져오기
        self.export_to_csv(filename, level_stats_fieldnames, level_stats_data)  # CSV로 내보내기

    def get_uptie_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Uptie": uptie_stat.uptie,
                "Speed": uptie_stat.speed,
            }
            for identity in self.identities
            for uptie_stat in identity.uptie_stats
        ]

    def export_uptie_stats_data(self, filename="Uptiestats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        uptie_stats_fieldnames = ["Identity_ID", "Uptie", "Speed"]
        uptie_stats_data = self.get_uptie_stats_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, uptie_stats_fieldnames, uptie_stats_data)  # CSV로 내보내기


async def main():
    data_container = DataContainer()
    scraper = Scraper(container=data_container, base_url="https://limbus.kusoge.xyz", img_url="https://img.kusoge.xyz/limbus/img")
    await scraper.scrape()

await main()

Identity_id: 10104
Uptie=1, Speed=3-5
Uptie=2, Speed=3-6
Uptie=3, Speed=3-7
Uptie=4, Speed=3-7
Identity_id: 10911
Uptie=1, Speed=2-5
Uptie=2, Speed=3-6
Uptie=3, Speed=4-7
Uptie=4, Speed=4-7
Data successfully exported to Uptiestats.csv


In [57]:
import os
import csv
from playwright.async_api import async_playwright
from tabulate import tabulate

# Sin Color 이름 변환을 위한 매핑
resource_mapping = {
    "AZURE": "Gloom",
    "VIOLET": "Envy",
    "AMBER": "Sloth",
    "SHAMROCK": "Gluttony",
    "CRIMSON": "Wrath",
    "SCARLET": "Lust",
    "INDIGO": "Pride"
}

def convert_resource(resource_name):
    """Sin Color 이름을 변환하는 함수. 매핑되지 않은 경우 원래 이름을 반환."""
    return resource_mapping.get(resource_name, resource_name)

class Scraper:
    """웹 페이지 크롤링을 위한 클래스"""
    def __init__(self, container, base_url, img_url):
        self.container = container
        self.base_url = base_url
        self.img_url = img_url

    async def handle_request(self, route, request):
        """이미지와 폰트 리소스 로드를 차단."""
        if request.resource_type in ["image", "font"]:
            await route.abort()
        else:
            await route.continue_()
            
    async def scrape(self):
        """크롤링 메인 함수"""
        async with async_playwright() as p:
            browser = await p.chromium.launch(headless=True)
            page = await browser.new_page()
            await page.route("**/*", self.handle_request)
            # Step 1: /identity 페이지에서 기본 정보 크롤링
            #await self.crawl_identity_page(page)

            # Step 2: /identity/{identity_id} 상세 페이지 크롤링
            #identity_ids_to_load = ["10104", "10310", "10710","10808", "10911", "11005"]
            identity_ids_to_load = ["10104", "10911","10310", "10710","10808", "11005"]
            self.container.load_identities_by_id('./csv/identities.csv', identity_ids_to_load)
            await self.crawl_identity_details_for_all(page)
            #self.container.display_identities()
            #self.container.export_identities_data("identities.csv")
            #self.container.export_level_stats_data("levelstats.csv")
            #self.container.export_uptie_stats_data("Uptiestats.csv")
            self.container.export_resist_info_data("ResistInfos.csv")
            page.set_default_timeout(10000)
            await browser.close()

    async def get_identity_id_from_row(self, row):
        """테이블 행(row)에서 인격 ID를 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_url = await identity_cell.get_attribute('href')
            identity_id = identity_url.split('/identity/')[-1]
            return identity_id
        return "Unknown"

    async def get_identity_name_from_row(self, row):
        """테이블 행(row)에서 인격 이름을 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_name = (await identity_cell.inner_text()).replace("\n", " ").strip()
            return identity_name
        return "Unknown"
        
    async def crawl_identity_page(self, page):
        """/identity 페이지로 이동하여 인격 기본 정보를 추출."""
        await page.goto(self.base_url + "/identity")
        await self.extract_identity_page(page)

    async def extract_identity_page(self, page):
        """/identity 페이지의 모든 테이블 행에서 인격 정보를 추출."""
        identity_rows = await page.query_selector_all('tr')
        if len(identity_rows) > 1:
            for idx, identity_row in enumerate(identity_rows):
                if idx == 0:
                    continue # 0번째요소이름을 보여주는 열은 건너뜀
                await self.scrape_identity_name_and_identity_id(identity_row)
                await self.extract_rarity(identity_row)
                for i in range(1, 4):  # 스킬1~3 정보 추출
                    await self.extract_atk_and_resource(identity_row, i)
                await self.extract_defense_and_resource(identity_row)

    async def scrape_identity_name_and_identity_id(self, row):
        """인격 번호와 이름을 추출하여 DataContainer에 저장."""
        try:
            identity_id = await self.get_identity_id_from_row(row)
            identity_name = await self.get_identity_name_from_row(row)
            identity = Identity(identity_id)
            self.container.add_identity(identity)
            self.container.update_identity_name(identity_id, identity_name)
        except Exception as e:
            print(f"Error extracting identity name and identity id: {e}")

    async def get_element_attribute(self, element, selector, attribute):
        """요소에서 지정된 속성(attribute)을 추출."""
        try:
            target_element = await element.query_selector(selector)
            if target_element:
                return await target_element.get_attribute(attribute)
        except Exception as e:
            print(f"Error getting element attribute: {e}")
        return None
    
    async def extract_rarity(self, row):
        """테이블 행(row)에서 희귀도를 추출하여 업데이트."""
        try:
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
                sibling = await parent.evaluate_handle('node => node.nextElementSibling')
                if sibling:
                    rarity_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/rarity_"]', 'src')
                    if rarity_img_src:
                        rarity_level = rarity_img_src.split('_')[-1].split('.')[0]  # rarity_1.png -> 1
                        identity_id = await self.get_identity_id_from_row(row)
                        self.container.update_identity_rarity(identity_id, rarity_level)
                    else:
                        print("Rarity image not found in the next sibling TD.")
        except Exception as e: 
            print(f"Error extracting rarity: {e}")

        return "Unknown"

    async def extract_atk_and_resource(self, row, skill_number):
        """ 공격 타입(ATK Type)과 자원(Attribute) 추출 함수 - 스킬 번호에 따라 처리 """
        try:
            #print(f"Attempting to locate skill{skill_number} type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 해당 스킬 번호에 맞는 형제로 이동
                sibling = parent
                for _ in range(skill_number + 1):  # 스킬 번호에 맞게 이동
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                atk_type, resource = "Unknown", "Unknown"
        
                if sibling:
                    # 공격 타입 이미지 (atk_type) 찾기
                    atk_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/atk_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if atk_type_img_src:
                        atk_type = atk_type_img_src.split('_')[-1].split('.')[0]  # atk_type_SLASH.png -> SLASH
                        #print(f"Extracted attack type: {atk_type}")
                        self.container.update_skill(identity_id, skill_number, atk_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted resource: {resource}")
                        
                        # 리소스를 변환
                        resource = convert_resource(resource)
                        #print(f"Converted resource: {resource}")
                        self.container.update_sin_affinity(identity_id, skill_number, resource)
                        
        except Exception as e:
            print(f"Error extracting skill{skill_number} type and resource: {e}")

    async def extract_defense_and_resource(self, row):
        """ 방어 타입(Def Type)과 자원(Attribute) 추출 함수 """
        try:
            #print("Attempting to locate defense type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 다섯 번째 형제 td로 이동 (방어 관련 정보가 있는 <td>)
                sibling = parent
                for _ in range(5):  # 방어 정보는 5번째 형제에 있음
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                def_type, def_resource = "Unknown", "Unknown"
        
                if sibling:
                    # 방어 타입 이미지 (def_type) 찾기
                    def_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/def_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if def_type_img_src:
                        def_type = def_type_img_src.split('_')[-1].split('.')[0]  # def_type_GUARD.png -> GUARD
                        #print(f"Extracted defense type: {def_type}")
                        self.container.update_skill(identity_id, 'defense', def_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        def_resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted defense resource: {def_resource}")
                        
                        # 방어 리소스를 변환
                        def_resource = convert_resource(def_resource)
                        #print(f"Converted defense resource: {def_resource}")
                        self.container.update_sin_affinity(identity_id, 'defense', def_resource)
 
        except Exception as e:
            print(f"Error extracting defense type and resource: {e}")

    async def crawl_identity_details_for_all(self, page):
        """모든 인격의 세부 정보를 크롤링."""
        for identity in self.container.identities:
            #await page.wait_for_load_state("networkidle")
            await page.goto(self.base_url + "/identity/"+identity.identity_id)
            print(f"Identity_id: {identity.identity_id}")
            await self.initialize_uptie_and_level(page)
            """ 레벨에 따른 level, hp, defense 다르게 되어 있는 것을 출력 확인 객체로 들어가서 csv로 옮기는 과정까지 진행
            for i in range(1, identity.level_max+1):
                await self.move_slider_to_value(page,i)
                await self.extract_level_status(page, identity.identity_id, i)
            """
            """ 동기화에 따른 speed 다르게 되어 있는 것을 출력 확인 객체로 들어가서 csv로 옮기는 과정까지 진행
            for j in range(1, identity.uptie_max+1):
                await self.uptie_click(page,j)
                await self.extract_uptie_status(page, identity.identity_id, j)
            """
            """ 공격 저항을 저장하는 것을 확인하고 csv로 들어가는 과정을 확인한다. 위와 다르게 인격당 하나의 열만 있으므로 전체를 확인해본다.
                {"name": "Atk_Type_HIT", "image_src": "https://img.kusoge.xyz/limbus/img/ui/atk_type_HIT.png", "depth": 1},
                {"name": "Atk_Type_PENETRATE", "image_src": "https://img.kusoge.xyz/limbus/img/ui/atk_type_PENETRATE.png", "depth": 1},
                {"name": "Atk_Type_SLASH", "image_src": "https://img.kusoge.xyz/limbus/img/ui/atk_type_SLASH.png", "depth": 1}
                async def extract_resist_type(page, image_src, depth):
                add_resist_info
            """
            resist_type_imgs = [f"{self.img_url}/ui/atk_type_HIT.png",
                                f"{self.img_url}/ui/atk_type_PENETRATE.png",
                                f"{self.img_url}/ui/atk_type_SLASH.png"]
            results = []
            for resist_type_img in resist_type_imgs:
                result = await self.extract_resist_type(page, resist_type_img, 1)
            #print(result)
                results.append(result)
            self.container.add_resist_info(identity.identity_id, results[0], results[1], results[2])
            
    async def initialize_uptie_and_level(self, page):
        await self.uptie_click(page, 1)
        await self.move_slider_to_value(page, 1)
    
    async def uptie_click(self, page, phase):
        fab_button_selector = ".q-btn.q-btn--fab"
        await page.wait_for_selector(fab_button_selector)
        await page.click(fab_button_selector)
        #print(f"[{setup_uptie_button}] 상위 동기화 버튼 클릭 완료")
        setup_uptie_button = f'a:has(img[src="{self.img_url}/ui/uptie_{phase}.png"])'
        await page.wait_for_selector(setup_uptie_button)
        await page.click(setup_uptie_button)
        #print(f"{phase} 동기화 버튼 클릭 완료")

    async def move_slider_to_value(self, page, target_value):
        """
        슬라이더를 주어진 값으로 이동시킵니다.
        target_value는 슬라이더의 최소값부터 최대값 사이여야 합니다.
        """
        slider_selector = ".q-slider"
        # 슬라이더의 범위 확인
        min_value = int(await page.get_attribute(slider_selector, "aria-valuemin"))
        max_value = int(await page.get_attribute(slider_selector, "aria-valuemax"))
    
        if target_value < min_value or target_value > max_value:
            raise ValueError(f"Target value {target_value} is out of bounds ({min_value}, {max_value})")
    
        # 슬라이더 thumb 요소와 위치 가져오기
        slider_thumb = await page.query_selector(f"{slider_selector} .q-slider__thumb")
        thumb_box = await slider_thumb.bounding_box()
        
        # 슬라이더의 현재 위치 및 목표 위치 계산
        track = await page.query_selector(f"{slider_selector} .q-slider__track")
        track_box = await track.bounding_box()
        
        # 목표 x 위치 계산 (슬라이더 트랙 내에서 비율 기반)
        target_percentage = (target_value - min_value) / (max_value - min_value)
        target_x = track_box["x"] + target_percentage * track_box["width"]
        current_x = thumb_box["x"] + thumb_box["width"] / 2
        y = thumb_box["y"] + thumb_box["height"] / 2
    
        # 슬라이더를 드래그하여 목표 위치로 이동
        await page.mouse.move(current_x, y)
        await page.mouse.down()
        await page.mouse.move(target_x, y, steps=10)  # 부드럽게 이동
        await page.mouse.up()
    
        #print(f"슬라이더를 {target_value}로 이동 완료.")
    
        # 슬라이더 값 확인 및 강제 이벤트 트리거
        await page.evaluate(
            f"""
            (slider) => {{
                slider.dispatchEvent(new Event('input'));  // oninput 이벤트 트리거
                slider.dispatchEvent(new Event('change'));  // onchange 이벤트 트리거
            }}
            """,
            await page.query_selector(slider_selector)
        )
    
        # 이동 후 값 확인
        current_value = await page.get_attribute(slider_selector, "aria-valuenow")
        if current_value != str(target_value):
            print(f"경고: 슬라이더 값이 예상과 다릅니다. 현재 값: {current_value}, 예상 값: {target_value}")

    async def get_second_status_element(self, page):
        """
        두 번째 'Status' 요소에 접근하여 반환하는 메소드
        """
        status_elements = await page.query_selector_all("text=Status")
        if len(status_elements) > 1:
            second_status_element = status_elements[1]
            #print("두 번째 'Status' 요소에 접근합니다.")
            return second_status_element
        else:
            #print("두 번째 'Status' 요소가 존재하지 않습니다.")
            return None

    async def extract_level_status(self, page, identity_id, level):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 레벨에 따라 HP와 Defense가 변한다.
        hp_value = await self.extract_hp(second_status_element)
        defense_value = await self.extract_defense(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Level={level}, HP={hp_value}, Defense={defense_value}")
        self.container.add_level_stat(identity_id,level, hp_value, defense_value)
    
    async def extract_hp(self, status_element):
        # HP 추출
        hp_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].children[0].nextElementSibling.textContent.trim()"
        )
        return hp_value
    
    async def extract_defense(self, status_element):
        # Defense 추출
        defense_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return defense_value

    async def extract_uptie_status(self, page, identity_id, uptie):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 uptie에 따라 speed는 변한다.
        speed_value = await self.extract_speed(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Uptie={uptie}, Speed={speed_value}")
        self.container.add_uptie_stat(identity_id,uptie, speed_value)

    async def extract_speed(self, status_element):
        # Speed 추출
        speed_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return speed_value

    async def extract_resist_type(self, page, image_src, depth):
        # 이미지 요소 찾기
        #print(image_src)
        image = await page.query_selector(f'img[src="{image_src}"]')
        #print(image)
        if not image:
            return "Unknown"
    
        # 이미지의 부모 요소를 찾아서 올라감
        parent = image
        for _ in range(depth):
            parent = await parent.evaluate_handle('node => node.parentElement')
            if not parent:
                return "Unknown"
    
        # 부모의 형제 요소 찾기
        sibling = await parent.evaluate_handle('node => node.nextElementSibling')
        
        if sibling:
            value = await sibling.evaluate('node => node.textContent')
            if value:
                return value.strip()
        
        return "Unknown"

class Identity:
    """인격 정보를 저장하는 클래스"""
    def __init__(self, identity_id, name=None, rarity=None,\
                 skill_1_type=None, skill_1_sin_affinity=None,\
                 skill_2_type=None, skill_2_sin_affinity=None,\
                 skill_3_type=None, skill_3_sin_affinity=None,\
                 defense_type=None, defense_sin_affinity=None):
        self.identity_id = identity_id
        self.name = name
        self.rarity = rarity
        # 선언만 하는 부분 
        self.skill_1_type = skill_1_type 
        self.skill_1_sin_affinity = skill_1_sin_affinity 
        self.skill_2_type = skill_2_type 
        self.skill_2_sin_affinity = skill_2_sin_affinity 
        self.skill_3_type = skill_3_type 
        self.skill_3_sin_affinity = skill_3_sin_affinity 
        self.defense_type = defense_type 
        self.defense_sin_affinity = defense_sin_affinity
        self.level_max = 50
        self.uptie_max = 4
        self.level_stats = []  # LevelStat 객체 리스트
        self.uptie_stats = [] # UptieStat 객체 리스트
        self.resist_info = None
        self.skills = []  # Skill 객체 리스트
        self.attack_types = {}
        self.stagger_thresholds = {}
        self.coin_effect = []  # CoinEffect 객체 리스트
        self.buff_debuff_info = []  # BuffDebuffInfo 객체 리스트
        self.passive_info = []  # PassiveInfo 객체 리스트

    def update_name(self, name):
        self.name = name

    def update_rarity(self, rarity):
        self.rarity = rarity

    def add_level_stat(self, level, hp, defense):
        self.level_stats.append(LevelStat(level, hp, defense))

    def add_uptie_stat(self, uptie, speed):
        self.uptie_stats.append(UptieStat(uptie, speed))

    def add_resist_info(self, resist_hit, resist_penetrate, resist_slash):
        self.resist_info = ResistInfo(resist_hit, resist_penetrate, resist_slash)

    def update_details(self, page):
        # 세부 정보 업데이트 로직 (가정)
        pass
    #def init_uptie(self, page):
    #    init_uptie = 

# LevelStat 객체 정의
class LevelStat:
    def __init__(self, level, hp=None, defense=None):
        self.level = level
        self.hp = hp
        self.defense = defense
    def update_level(self, level):
        self.level = level
    def update_hp(self, hp):
        self.hp = hp
    def update_defense(self, defense):
        self.defense = defense

# UptieStat 객체 정의
class UptieStat:
    def __init__(self, uptie, speed=None):
        self.uptie = uptie
        self.speed = speed
    def update_speed(self, speed):
        self.speed = speed

class ResistInfo:
    def __init__(self, resist_hit=None, resist_penetrate=None, resist_slash=None):
        self.resist_hit = resist_hit
        self.resist_penetrate = resist_penetrate
        self.resist_slash = resist_slash

    
class DataContainer:
    """크롤링 데이터를 관리하는 클래스"""
    def __init__(self):
        self.identities = []

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.level_max

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.uptie_max

    def update_identity_name(self, identity_id, identity_name):
        """특정 ID의 이름을 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_name(identity_name)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_identity_rarity(self, identity_id, rarity):
        """특정 ID의 희귀도를 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_rarity(rarity)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_skill(self, identity_id, skill_number, new_type):
        """
        특정 identity의 skill_type을 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_type: 새 skill_type
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_type = new_type
                elif skill_number == 2:
                    identity.skill_2_type = new_type
                elif skill_number == 3:
                    identity.skill_3_type = new_type
                elif skill_number == "defense":
                    identity.defense_type = new_type
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def update_sin_affinity(self, identity_id, skill_number, new_affinity):
        """
        특정 identity의 sin_affinity를 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_affinity: 새 sin_affinity
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_sin_affinity = new_affinity
                elif skill_number == 2:
                    identity.skill_2_sin_affinity = new_affinity
                elif skill_number == 3:
                    identity.skill_3_sin_affinity = new_affinity
                elif skill_number == "defense":
                    identity.defense_sin_affinity = new_affinity
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def add_identity(self, identity):
        """새로운 인격 데이터를 추가."""
        self.identities.append(identity)

    def add_level_stat(self, identity_id, level, hp, defense):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_level_stat(level, hp, defense)

    def add_uptie_stat(self, identity_id, uptie, speed):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_uptie_stat(uptie, speed)

    def add_resist_info(self, identity_id, resist_hit, resist_penetrate, resist_slash):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_resist_info(resist_hit, resist_penetrate, resist_slash)
        
    def display_identities(self):
        """Identity, ID, Name, Rarity 정보를 출력"""
        if not self.identities:
            print("No identities found.")
        else:
            # 데이터를 리스트로 변환
            table = []
            for idx, identity in enumerate(self.identities, 1):
                table.append([
                    idx, identity.identity_id, identity.name, identity.rarity,
                    identity.skill_1_type, identity.skill_1_sin_affinity,
                    identity.skill_2_type, identity.skill_2_sin_affinity,
                    identity.skill_3_type, identity.skill_3_sin_affinity,
                    identity.defense_type, identity.defense_sin_affinity
                ])
            
            # 테이블 헤더 정의
            headers = [
                "Index", "ID", "Name", "Rarity",
                "Skill_1 Type", "Skill_1 Sin", "Skill_2 Type", "Skill_2 Sin",
                "Skill_3 Type", "Skill_3 Sin", "Defense Type", "Defense Sin"
            ]
            
            # 테이블 출력
            print(tabulate(table, headers=headers, tablefmt="fancy_grid"))

    def export_to_csv(self, filename, fieldnames, data):
        """
        공통적으로 사용 가능한 CSV 내보내기 메서드
        :param filename: 내보낼 파일 이름 (예: 'identity.csv', 'skill.csv')
        :param fieldnames: CSV의 헤더 리스트
        :param data: CSV로 내보낼 데이터 (리스트 형태)
        """
        try:
            # 현재작업 디렉토 기준으 'csv'폴더 없으면 생성
            if not os.path.exists('./csv'):
                os.makedirs('./csv')
            # 파일 경로 설정 (상대경로 'csv' 폴더 안에 저장)
            file_path = os.path.join('./csv', filename)
            with open(file_path, mode='w', newline='', encoding='utf-8') as file:
                writer = csv.DictWriter(file, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(data)
            print(f"Data successfully exported to {filename}")
        except Exception as e:
            print(f"Error exporting to CSV: {e}")

    def get_identity_data(self):
        """
        Identity 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "ID": identity.identity_id,
                "Name": identity.name,
                "Rarity": identity.rarity,
                "Skill_1_Type": identity.skill_1_type,
                "Skill_1_Sin_Affinity": identity.skill_1_sin_affinity,
                "Skill_2_Type": identity.skill_2_type,
                "Skill_2_Sin_Affinity": identity.skill_2_sin_affinity,
                "Skill_3_Type": identity.skill_3_type,
                "Skill_3_Sin_Affinity": identity.skill_3_sin_affinity,
                "Defense_Type": identity.defense_type,
                "Defense_Sin_Affinity": identity.defense_sin_affinity,
            }
            for identity in self.identities
        ]
        
    def export_identities_data(self, filename="identities.csv"):
        """Identity 데이터를 CSV로 내보내는 메서드"""
        identity_fieldnames = [
            "ID", "Name", "Rarity",
            "Skill_1_Type", "Skill_1_Sin_Affinity",
            "Skill_2_Type", "Skill_2_Sin_Affinity",
            "Skill_3_Type", "Skill_3_Sin_Affinity",
            "Defense_Type", "Defense_Sin_Affinity"
        ]
        identity_data = self.get_identity_data()  # Identity 데이터 가져오기
        self.export_to_csv(filename, identity_fieldnames, identity_data)  # CSV로 내보내기

    def load_identities_by_id(self, filename, identity_ids):
        """지정된 identity_ids 목록에 해당하는 Identity 데이터를 CSV에서 읽어옴."""
        identity_ids_set = set(identity_ids) # 중복 제거를 위 Set 사용
        with open(filename, mode='r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                if row["ID"] in identity_ids_set:
                    identity = Identity(
                        identity_id=row["ID"],
                        name=row["Name"],
                        rarity=row["Rarity"],
                        skill_1_type=row["Skill_1_Type"],
                        skill_1_sin_affinity=row["Skill_1_Sin_Affinity"],
                        skill_2_type=row["Skill_2_Type"],
                        skill_2_sin_affinity=row["Skill_2_Sin_Affinity"],
                        skill_3_type=row["Skill_3_Type"],
                        skill_3_sin_affinity=row["Skill_3_Sin_Affinity"],
                        defense_type=row["Defense_Type"],
                        defense_sin_affinity=row["Defense_Sin_Affinity"]
                    )
                    self.identities.append(identity)

    def get_level_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Level": level_stat.level,
                "HP": level_stat.hp,
                "Defense": level_stat.defense,
            }
            for identity in self.identities
            for level_stat in identity.level_stats
        ]

    def export_level_stats_data(self, filename="levelstats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        level_stats_fieldnames = ["Identity_ID", "Level", "HP", "Defense"]
        level_stats_data = self.get_level_stats_data()  # level_stats 데이터 가져오기
        self.export_to_csv(filename, level_stats_fieldnames, level_stats_data)  # CSV로 내보내기

    def get_uptie_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Uptie": uptie_stat.uptie,
                "Speed": uptie_stat.speed,
            }
            for identity in self.identities
            for uptie_stat in identity.uptie_stats
        ]

    def export_uptie_stats_data(self, filename="Uptiestats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        uptie_stats_fieldnames = ["Identity_ID", "Uptie", "Speed"]
        uptie_stats_data = self.get_uptie_stats_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, uptie_stats_fieldnames, uptie_stats_data)  # CSV로 내보내기

    def get_resist_info_data(self):
        """
        모든 Identity의 resist_info 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Resist_Hit": identity.resist_info.resist_hit,
                "Resist_Penetrate": identity.resist_info.resist_penetrate,
                "Resist_Slash": identity.resist_info.resist_slash,
            }
            for identity in self.identities
        ]

    def export_resist_info_data(self, filename="ResistInfos.csv"):
        """ResistInfo 데이터를 CSV로 내보내는 메서드"""
        resist_infos_fieldnames = ["Identity_ID", "Resist_Hit", "Resist_Penetrate", "Resist_Slash"]
        resist_infos_data = self.get_resist_info_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, resist_infos_fieldnames, resist_infos_data)  # CSV로 내보내기


async def main():
    data_container = DataContainer()
    scraper = Scraper(container=data_container, base_url="https://limbus.kusoge.xyz", img_url="https://img.kusoge.xyz/limbus/img")
    await scraper.scrape()

await main()

Identity_id: 10104
Identity_id: 10310
Identity_id: 10710
Identity_id: 10808
Identity_id: 10911
Identity_id: 11005
Data successfully exported to ResistInfos.csv


In [8]:
import os
import csv
import re
import numpy as np
import pandas as pd
from playwright.async_api import async_playwright
from tabulate import tabulate

# Sin Color 이름 변환을 위한 매핑
resource_mapping = {
    "AZURE": "Gloom",
    "VIOLET": "Envy",
    "AMBER": "Sloth",
    "SHAMROCK": "Gluttony",
    "CRIMSON": "Wrath",
    "SCARLET": "Lust",
    "INDIGO": "Pride"
}

def convert_resource(resource_name):
    """Sin Color 이름을 변환하는 함수. 매핑되지 않은 경우 원래 이름을 반환."""
    return resource_mapping.get(resource_name, resource_name)

class Scraper:
    """웹 페이지 크롤링을 위한 클래스"""
    def __init__(self, container, base_url, img_url):
        self.container = container
        self.base_url = base_url
        self.img_url = img_url

    async def handle_request(self, route, request):
        """이미지와 폰트 리소스 로드를 차단."""
        if request.resource_type in ["image", "font"]:
            await route.abort()
        else:
            await route.continue_()
            
    async def scrape(self):
        """크롤링 메인 함수"""
        async with async_playwright() as p:
            browser = await p.chromium.launch(headless=True)
            page = await browser.new_page()
            await page.route("**/*", self.handle_request)
            # Step 1: /identity 페이지에서 기본 정보 크롤링
            #await self.crawl_identity_page(page)

            # Step 2: /identity/{identity_id} 상세 페이지 크롤링
            #identity_ids_to_load = ["10104", "10310", "10710","10808", "10911", "11005"]
            identity_ids_to_load = ["10104", "10911","10310", "10710","10808", "11005", "11210"]
            self.container.load_identities_by_id('./csv/identities.csv', identity_ids_to_load)
            await self.crawl_identity_details_for_all(page)
            #self.container.display_identities()
            #self.container.export_identities_data("identities.csv")
            #self.container.export_level_stats_data("Levelstats.csv")
            #self.container.export_uptie_stats_data("Uptiestats.csv")
            #self.container.export_resist_info_data("ResistInfos.csv")
            self.container.export_stagger_threshold_data("StaggerThresholds.csv")
            page.set_default_timeout(10000)
            await browser.close()

    async def get_identity_id_from_row(self, row):
        """테이블 행(row)에서 인격 ID를 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_url = await identity_cell.get_attribute('href')
            identity_id = identity_url.split('/identity/')[-1]
            return identity_id
        return "Unknown"

    async def get_identity_name_from_row(self, row):
        """테이블 행(row)에서 인격 이름을 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_name = (await identity_cell.inner_text()).replace("\n", " ").strip()
            return identity_name
        return "Unknown"
        
    async def crawl_identity_page(self, page):
        """/identity 페이지로 이동하여 인격 기본 정보를 추출."""
        await page.goto(self.base_url + "/identity")
        await self.extract_identity_page(page)

    async def extract_identity_page(self, page):
        """/identity 페이지의 모든 테이블 행에서 인격 정보를 추출."""
        identity_rows = await page.query_selector_all('tr')
        if len(identity_rows) > 1:
            for idx, identity_row in enumerate(identity_rows):
                if idx == 0:
                    continue # 0번째요소이름을 보여주는 열은 건너뜀
                await self.scrape_identity_name_and_identity_id(identity_row)
                await self.extract_rarity(identity_row)
                for i in range(1, 4):  # 스킬1~3 정보 추출
                    await self.extract_atk_and_resource(identity_row, i)
                await self.extract_defense_and_resource(identity_row)

    async def scrape_identity_name_and_identity_id(self, row):
        """인격 번호와 이름을 추출하여 DataContainer에 저장."""
        try:
            identity_id = await self.get_identity_id_from_row(row)
            identity_name = await self.get_identity_name_from_row(row)
            identity = Identity(identity_id)
            self.container.add_identity(identity)
            self.container.update_identity_name(identity_id, identity_name)
        except Exception as e:
            print(f"Error extracting identity name and identity id: {e}")

    async def get_element_attribute(self, element, selector, attribute):
        """요소에서 지정된 속성(attribute)을 추출."""
        try:
            target_element = await element.query_selector(selector)
            if target_element:
                return await target_element.get_attribute(attribute)
        except Exception as e:
            print(f"Error getting element attribute: {e}")
        return None
    
    async def extract_rarity(self, row):
        """테이블 행(row)에서 희귀도를 추출하여 업데이트."""
        try:
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
                sibling = await parent.evaluate_handle('node => node.nextElementSibling')
                if sibling:
                    rarity_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/rarity_"]', 'src')
                    if rarity_img_src:
                        rarity_level = rarity_img_src.split('_')[-1].split('.')[0]  # rarity_1.png -> 1
                        identity_id = await self.get_identity_id_from_row(row)
                        self.container.update_identity_rarity(identity_id, rarity_level)
                    else:
                        print("Rarity image not found in the next sibling TD.")
        except Exception as e: 
            print(f"Error extracting rarity: {e}")

        return "Unknown"

    async def extract_atk_and_resource(self, row, skill_number):
        """ 공격 타입(ATK Type)과 자원(Attribute) 추출 함수 - 스킬 번호에 따라 처리 """
        try:
            #print(f"Attempting to locate skill{skill_number} type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 해당 스킬 번호에 맞는 형제로 이동
                sibling = parent
                for _ in range(skill_number + 1):  # 스킬 번호에 맞게 이동
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                atk_type, resource = "Unknown", "Unknown"
        
                if sibling:
                    # 공격 타입 이미지 (atk_type) 찾기
                    atk_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/atk_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if atk_type_img_src:
                        atk_type = atk_type_img_src.split('_')[-1].split('.')[0]  # atk_type_SLASH.png -> SLASH
                        #print(f"Extracted attack type: {atk_type}")
                        self.container.update_skill(identity_id, skill_number, atk_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted resource: {resource}")
                        
                        # 리소스를 변환
                        resource = convert_resource(resource)
                        #print(f"Converted resource: {resource}")
                        self.container.update_sin_affinity(identity_id, skill_number, resource)
                        
        except Exception as e:
            print(f"Error extracting skill{skill_number} type and resource: {e}")

    async def extract_defense_and_resource(self, row):
        """ 방어 타입(Def Type)과 자원(Attribute) 추출 함수 """
        try:
            #print("Attempting to locate defense type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 다섯 번째 형제 td로 이동 (방어 관련 정보가 있는 <td>)
                sibling = parent
                for _ in range(5):  # 방어 정보는 5번째 형제에 있음
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                def_type, def_resource = "Unknown", "Unknown"
        
                if sibling:
                    # 방어 타입 이미지 (def_type) 찾기
                    def_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/def_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if def_type_img_src:
                        def_type = def_type_img_src.split('_')[-1].split('.')[0]  # def_type_GUARD.png -> GUARD
                        #print(f"Extracted defense type: {def_type}")
                        self.container.update_skill(identity_id, 'defense', def_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        def_resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted defense resource: {def_resource}")
                        
                        # 방어 리소스를 변환
                        def_resource = convert_resource(def_resource)
                        #print(f"Converted defense resource: {def_resource}")
                        self.container.update_sin_affinity(identity_id, 'defense', def_resource)
 
        except Exception as e:
            print(f"Error extracting defense type and resource: {e}")

    async def crawl_identity_details_for_all(self, page):
        """모든 인격의 세부 정보를 크롤링."""
        for identity in self.container.identities:
            await page.wait_for_load_state("networkidle")
            await page.goto(self.base_url + "/identity/"+identity.identity_id)
            """ 레벨과 동기화와 관련 없는 참관타 저항은 밑에서 변경하는 것과 연관성이 적기 때문에 위에서 처리한다.
            print(f"Identity_id: {identity.identity_id}")
            results = []
            resist_type_imgs = [f"{self.img_url}/ui/atk_type_HIT.png",
                                f"{self.img_url}/ui/atk_type_PENETRATE.png",
                                f"{self.img_url}/ui/atk_type_SLASH.png"]
            for resist_type_img in resist_type_imgs:
                result = await self.extract_resist_type(page, resist_type_img, 1)
                results.append(result)
            self.container.add_resist_info(identity.identity_id, results[0], results[1], results[2])
            """
            # Stagger Threshold 값 추출
            stagger_threshold = await self.extract_stagger_threshold_values(page)
            self.container.add_stagger_threshold(identity.identity_id, stagger_threshold[0], stagger_threshold[1], stagger_threshold[2])
            await self.initialize_uptie_and_level(page)
            """ 레벨에 따른 level, hp, defense 다르게 되어 있는 것을 출력 확인 객체로 들어가서 csv로 옮기는 과정까지 진행
            for i in range(1, identity.level_max+1):
                await self.move_slider_to_value(page,i)
                await self.extract_level_status(page, identity.identity_id, i)
            """
            """ 동기화에 따른 speed 다르게 되어 있는 것을 출력 확인 객체로 들어가서 csv로 옮기는 과정까지 진행
            for i in range(1, identity.uptie_max+1):
                await self.uptie_click(page,i)
                await self.extract_uptie_status(page, identity.identity_id, i)
            """
            
    async def initialize_uptie_and_level(self, page):
        await self.uptie_click(page, 1)
        await self.move_slider_to_value(page, 1)
    
    async def uptie_click(self, page, phase):
        fab_button_selector = ".q-btn.q-btn--fab"
        await page.wait_for_selector(fab_button_selector)
        await page.click(fab_button_selector)
        #print(f"[{setup_uptie_button}] 상위 동기화 버튼 클릭 완료")
        setup_uptie_button = f'a:has(img[src="{self.img_url}/ui/uptie_{phase}.png"])'
        await page.wait_for_selector(setup_uptie_button)
        await page.click(setup_uptie_button)
        #print(f"{phase} 동기화 버튼 클릭 완료")

    async def move_slider_to_value(self, page, target_value):
        """
        슬라이더를 주어진 값으로 이동시킵니다.
        target_value는 슬라이더의 최소값부터 최대값 사이여야 합니다.
        """
        slider_selector = ".q-slider"
        # 슬라이더의 범위 확인
        min_value = int(await page.get_attribute(slider_selector, "aria-valuemin"))
        max_value = int(await page.get_attribute(slider_selector, "aria-valuemax"))
    
        if target_value < min_value or target_value > max_value:
            raise ValueError(f"Target value {target_value} is out of bounds ({min_value}, {max_value})")
    
        # 슬라이더 thumb 요소와 위치 가져오기
        slider_thumb = await page.query_selector(f"{slider_selector} .q-slider__thumb")
        thumb_box = await slider_thumb.bounding_box()
        
        # 슬라이더의 현재 위치 및 목표 위치 계산
        track = await page.query_selector(f"{slider_selector} .q-slider__track")
        track_box = await track.bounding_box()
        
        # 목표 x 위치 계산 (슬라이더 트랙 내에서 비율 기반)
        target_percentage = (target_value - min_value) / (max_value - min_value)
        target_x = track_box["x"] + target_percentage * track_box["width"]
        current_x = thumb_box["x"] + thumb_box["width"] / 2
        y = thumb_box["y"] + thumb_box["height"] / 2
    
        # 슬라이더를 드래그하여 목표 위치로 이동
        await page.mouse.move(current_x, y)
        await page.mouse.down()
        await page.mouse.move(target_x, y, steps=10)  # 부드럽게 이동
        await page.mouse.up()
    
        #print(f"슬라이더를 {target_value}로 이동 완료.")
    
        # 슬라이더 값 확인 및 강제 이벤트 트리거
        await page.evaluate(
            f"""
            (slider) => {{
                slider.dispatchEvent(new Event('input'));  // oninput 이벤트 트리거
                slider.dispatchEvent(new Event('change'));  // onchange 이벤트 트리거
            }}
            """,
            await page.query_selector(slider_selector)
        )
    
        # 이동 후 값 확인
        current_value = await page.get_attribute(slider_selector, "aria-valuenow")
        if current_value != str(target_value):
            print(f"경고: 슬라이더 값이 예상과 다릅니다. 현재 값: {current_value}, 예상 값: {target_value}")

    async def get_second_status_element(self, page):
        """
        두 번째 'Status' 요소에 접근하여 반환하는 메소드
        """
        status_elements = await page.query_selector_all("text=Status")
        if len(status_elements) > 1:
            second_status_element = status_elements[1]
            #print("두 번째 'Status' 요소에 접근합니다.")
            return second_status_element
        else:
            #print("두 번째 'Status' 요소가 존재하지 않습니다.")
            return None

    async def extract_level_status(self, page, identity_id, level):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 레벨에 따라 HP와 Defense가 변한다.
        hp_value = await self.extract_hp(second_status_element)
        defense_value = await self.extract_defense(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Level={level}, HP={hp_value}, Defense={defense_value}")
        self.container.add_level_stat(identity_id,level, hp_value, defense_value)
    
    async def extract_hp(self, status_element):
        # HP 추출
        hp_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].children[0].nextElementSibling.textContent.trim()"
        )
        return hp_value
    
    async def extract_defense(self, status_element):
        # Defense 추출
        defense_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return defense_value

    async def extract_uptie_status(self, page, identity_id, uptie):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 uptie에 따라 speed는 변한다.
        speed_value = await self.extract_speed(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Uptie={uptie}, Speed={speed_value}")
        self.container.add_uptie_stat(identity_id,uptie, speed_value)

    async def extract_speed(self, status_element):
        # Speed 추출
        speed_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return speed_value

    async def extract_resist_type(self, page, image_src, depth):
        # 이미지 요소 찾기
        #print(image_src)
        image = await page.query_selector(f'img[src="{image_src}"]')
        #print(image)
        if not image:
            return "Unknown"
    
        # 이미지의 부모 요소를 찾아서 올라감
        parent = image
        for _ in range(depth):
            parent = await parent.evaluate_handle('node => node.parentElement')
            if not parent:
                return "Unknown"
    
        # 부모의 형제 요소 찾기
        sibling = await parent.evaluate_handle('node => node.nextElementSibling')
        
        if sibling:
            value = await sibling.evaluate('node => node.textContent')
            if value:
                return value.strip()
        
        return "Unknown"

    async def extract_stagger_threshold_values(self, page):
        # 'div.row.center.flex-center.q-pt-xs' 요소들 찾기
        elements = await page.query_selector_all('div.row.center.flex-center.q-pt-xs')
        
        # 각 요소의 텍스트 내용 출력하여 'Stagger Threshold' 요소를 찾기
        for element in elements:
            text = await element.text_content()
            
            if text and "Stagger Threshold" in text:
                # 'Stagger Threshold'가 있는 요소를 찾았다면, 그 이후 형제 요소들에서 값을 추출
                parent = await element.evaluate_handle('node => node.parentElement')
                sibling_elements = await parent.query_selector_all('div.row.justify-evenly div')  # Threshold 값이 들어있는 div들
                
                stagger_values = []
                for sibling in sibling_elements:
                    text_content = await sibling.text_content()
                    match = re.search(r'(\d+%)', text_content)
                    if match:
                        stagger_values.append(match.group(1))  # 퍼센트 값만 리스트에 추가
                while len(stagger_values) < 3:
                    stagger_values.append(np.nan)
                #print(stagger_values)
                return stagger_values  # 추출된 값(또는 np.nan 포함)을 반환
        
        return [np.nan, np.nan, np.nan]  # 만약 'Stagger Threshold'를 찾지 못한 경우 기본값 반환

class Identity:
    """인격 정보를 저장하는 클래스"""
    def __init__(self, identity_id, name=None, rarity=None,\
                 skill_1_type=None, skill_1_sin_affinity=None,\
                 skill_2_type=None, skill_2_sin_affinity=None,\
                 skill_3_type=None, skill_3_sin_affinity=None,\
                 defense_type=None, defense_sin_affinity=None):
        self.identity_id = identity_id
        self.name = name
        self.rarity = rarity
        # 선언만 하는 부분 
        self.skill_1_type = skill_1_type 
        self.skill_1_sin_affinity = skill_1_sin_affinity 
        self.skill_2_type = skill_2_type 
        self.skill_2_sin_affinity = skill_2_sin_affinity 
        self.skill_3_type = skill_3_type 
        self.skill_3_sin_affinity = skill_3_sin_affinity 
        self.defense_type = defense_type 
        self.defense_sin_affinity = defense_sin_affinity
        self.level_max = 50
        self.uptie_max = 4
        self.level_stats = []  # LevelStat 객체 리스트
        self.uptie_stats = [] # UptieStat 객체 리스트
        self.resist_info = None
        self.stagger_threshold = None
        self.skills = []  # Skill 객체 리스트
        self.attack_types = {}
        self.coin_effect = []  # CoinEffect 객체 리스트
        self.buff_debuff_info = []  # BuffDebuffInfo 객체 리스트
        self.passive_info = []  # PassiveInfo 객체 리스트

    def update_name(self, name):
        self.name = name

    def update_rarity(self, rarity):
        self.rarity = rarity

    def add_level_stat(self, level, hp, defense):
        self.level_stats.append(LevelStat(level, hp, defense))

    def add_uptie_stat(self, uptie, speed):
        self.uptie_stats.append(UptieStat(uptie, speed))

    def add_resist_info(self, resist_hit, resist_penetrate, resist_slash):
        self.resist_info = ResistInfo(resist_hit, resist_penetrate, resist_slash)

    def add_stagger_threshold(self, first_stagger, second_stagger, third_stagger):
        self.stagger_threshold = StaggerThreshold(first_stagger, second_stagger, third_stagger)

    def update_details(self, page):
        # 세부 정보 업데이트 로직 (가정)
        pass
    #def init_uptie(self, page):
    #    init_uptie = 

# LevelStat 객체 정의
class LevelStat:
    def __init__(self, level, hp=None, defense=None):
        self.level = level
        self.hp = hp
        self.defense = defense
    def update_level(self, level):
        self.level = level
    def update_hp(self, hp):
        self.hp = hp
    def update_defense(self, defense):
        self.defense = defense

# UptieStat 객체 정의
class UptieStat:
    def __init__(self, uptie, speed=None):
        self.uptie = uptie
        self.speed = speed
    def update_speed(self, speed):
        self.speed = speed

# ResistInfo 객체 정의
class ResistInfo:
    def __init__(self, resist_hit=None, resist_penetrate=None, resist_slash=None):
        self.resist_hit = resist_hit
        self.resist_penetrate = resist_penetrate
        self.resist_slash = resist_slash

# ResistInfo 객체 정의
class StaggerThreshold:
    def __init__(self, first_stagger=None, second_stagger=None, third_stagger=None):
        self.first_stagger = first_stagger
        self.second_stagger = second_stagger
        self.third_stagger = third_stagger

    
class DataContainer:
    """크롤링 데이터를 관리하는 클래스"""
    def __init__(self):
        self.identities = []

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.level_max

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.uptie_max

    def update_identity_name(self, identity_id, identity_name):
        """특정 ID의 이름을 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_name(identity_name)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_identity_rarity(self, identity_id, rarity):
        """특정 ID의 희귀도를 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_rarity(rarity)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_skill(self, identity_id, skill_number, new_type):
        """
        특정 identity의 skill_type을 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_type: 새 skill_type
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_type = new_type
                elif skill_number == 2:
                    identity.skill_2_type = new_type
                elif skill_number == 3:
                    identity.skill_3_type = new_type
                elif skill_number == "defense":
                    identity.defense_type = new_type
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def update_sin_affinity(self, identity_id, skill_number, new_affinity):
        """
        특정 identity의 sin_affinity를 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_affinity: 새 sin_affinity
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_sin_affinity = new_affinity
                elif skill_number == 2:
                    identity.skill_2_sin_affinity = new_affinity
                elif skill_number == 3:
                    identity.skill_3_sin_affinity = new_affinity
                elif skill_number == "defense":
                    identity.defense_sin_affinity = new_affinity
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def add_identity(self, identity):
        """새로운 인격 데이터를 추가."""
        self.identities.append(identity)

    def add_level_stat(self, identity_id, level, hp, defense):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_level_stat(level, hp, defense)

    def add_uptie_stat(self, identity_id, uptie, speed):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_uptie_stat(uptie, speed)

    def add_resist_info(self, identity_id, resist_hit, resist_penetrate, resist_slash):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_resist_info(resist_hit, resist_penetrate, resist_slash)

    def add_stagger_threshold(self, identity_id, first_stagger, second_stagger, third_stagger):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_stagger_threshold(first_stagger, second_stagger, third_stagger)
        
    def display_identities(self):
        """Identity, ID, Name, Rarity 정보를 출력"""
        if not self.identities:
            print("No identities found.")
        else:
            # 데이터를 리스트로 변환
            table = []
            for idx, identity in enumerate(self.identities, 1):
                table.append([
                    idx, identity.identity_id, identity.name, identity.rarity,
                    identity.skill_1_type, identity.skill_1_sin_affinity,
                    identity.skill_2_type, identity.skill_2_sin_affinity,
                    identity.skill_3_type, identity.skill_3_sin_affinity,
                    identity.defense_type, identity.defense_sin_affinity
                ])
            
            # 테이블 헤더 정의
            headers = [
                "Index", "ID", "Name", "Rarity",
                "Skill_1 Type", "Skill_1 Sin", "Skill_2 Type", "Skill_2 Sin",
                "Skill_3 Type", "Skill_3 Sin", "Defense Type", "Defense Sin"
            ]
            
            # 테이블 출력
            print(tabulate(table, headers=headers, tablefmt="fancy_grid"))

    def export_to_csv(self, filename, fieldnames, data):
        """
        공통적으로 사용 가능한 CSV 내보내기 메서드
        :param filename: 내보낼 파일 이름 (예: 'identity.csv', 'skill.csv')
        :param fieldnames: CSV의 헤더 리스트
        :param data: CSV로 내보낼 데이터 (리스트 형태)
        """
        try:
            # 현재작업 디렉토 기준으 'csv'폴더 없으면 생성
            if not os.path.exists('./csv'):
                os.makedirs('./csv')
            # 파일 경로 설정 (상대경로 'csv' 폴더 안에 저장)
            file_path = os.path.join('./csv', filename)
            with open(file_path, mode='w', newline='', encoding='utf-8') as file:
                writer = csv.DictWriter(file, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(data)
            print(f"Data successfully exported to {filename}")
        except Exception as e:
            print(f"Error exporting to CSV: {e}")

    def get_identity_data(self):
        """
        Identity 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "ID": identity.identity_id,
                "Name": identity.name,
                "Rarity": identity.rarity,
                "Skill_1_Type": identity.skill_1_type,
                "Skill_1_Sin_Affinity": identity.skill_1_sin_affinity,
                "Skill_2_Type": identity.skill_2_type,
                "Skill_2_Sin_Affinity": identity.skill_2_sin_affinity,
                "Skill_3_Type": identity.skill_3_type,
                "Skill_3_Sin_Affinity": identity.skill_3_sin_affinity,
                "Defense_Type": identity.defense_type,
                "Defense_Sin_Affinity": identity.defense_sin_affinity,
            }
            for identity in self.identities
        ]
        
    def export_identities_data(self, filename="identities.csv"):
        """Identity 데이터를 CSV로 내보내는 메서드"""
        identity_fieldnames = [
            "ID", "Name", "Rarity",
            "Skill_1_Type", "Skill_1_Sin_Affinity",
            "Skill_2_Type", "Skill_2_Sin_Affinity",
            "Skill_3_Type", "Skill_3_Sin_Affinity",
            "Defense_Type", "Defense_Sin_Affinity"
        ]
        identity_data = self.get_identity_data()  # Identity 데이터 가져오기
        self.export_to_csv(filename, identity_fieldnames, identity_data)  # CSV로 내보내기

    def load_identities_by_id(self, filename, identity_ids):
        """지정된 identity_ids 목록에 해당하는 Identity 데이터를 CSV에서 읽어옴."""
        identity_ids_set = set(identity_ids) # 중복 제거를 위 Set 사용
        with open(filename, mode='r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                if row["ID"] in identity_ids_set:
                    identity = Identity(
                        identity_id=row["ID"],
                        name=row["Name"],
                        rarity=row["Rarity"],
                        skill_1_type=row["Skill_1_Type"],
                        skill_1_sin_affinity=row["Skill_1_Sin_Affinity"],
                        skill_2_type=row["Skill_2_Type"],
                        skill_2_sin_affinity=row["Skill_2_Sin_Affinity"],
                        skill_3_type=row["Skill_3_Type"],
                        skill_3_sin_affinity=row["Skill_3_Sin_Affinity"],
                        defense_type=row["Defense_Type"],
                        defense_sin_affinity=row["Defense_Sin_Affinity"]
                    )
                    self.identities.append(identity)

    def get_level_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Level": level_stat.level,
                "HP": level_stat.hp,
                "Defense": level_stat.defense,
            }
            for identity in self.identities
            for level_stat in identity.level_stats
        ]

    def export_level_stats_data(self, filename="levelstats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        level_stats_fieldnames = ["Identity_ID", "Level", "HP", "Defense"]
        level_stats_data = self.get_level_stats_data()  # level_stats 데이터 가져오기
        self.export_to_csv(filename, level_stats_fieldnames, level_stats_data)  # CSV로 내보내기

    def get_uptie_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Uptie": uptie_stat.uptie,
                "Speed": uptie_stat.speed,
            }
            for identity in self.identities
            for uptie_stat in identity.uptie_stats
        ]

    def export_uptie_stats_data(self, filename="Uptiestats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        uptie_stats_fieldnames = ["Identity_ID", "Uptie", "Speed"]
        uptie_stats_data = self.get_uptie_stats_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, uptie_stats_fieldnames, uptie_stats_data)  # CSV로 내보내기

    def get_resist_info_data(self):
        """
        모든 Identity의 resist_info 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Resist_Hit": identity.resist_info.resist_hit,
                "Resist_Penetrate": identity.resist_info.resist_penetrate,
                "Resist_Slash": identity.resist_info.resist_slash,
            }
            for identity in self.identities
        ]

    def export_resist_info_data(self, filename="ResistInfos.csv"):
        """ResistInfo 데이터를 CSV로 내보내는 메서드"""
        resist_infos_fieldnames = ["Identity_ID", "Resist_Hit", "Resist_Penetrate", "Resist_Slash"]
        resist_infos_data = self.get_resist_info_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, resist_infos_fieldnames, resist_infos_data)  # CSV로 내보내기

    def export_uptie_stats_data(self, filename="Uptiestats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        uptie_stats_fieldnames = ["Identity_ID", "Uptie", "Speed"]
        uptie_stats_data = self.get_uptie_stats_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, uptie_stats_fieldnames, uptie_stats_data)  # CSV로 내보내기

    def get_stagger_threshold_data(self):
        """
        모든 Identity의 stagger_threshold 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "First_Stagger": identity.stagger_threshold.first_stagger,
                "Second_Stagger": identity.stagger_threshold.second_stagger,
                "Third_Stagger": identity.stagger_threshold.third_stagger,
            }
            for identity in self.identities
        ]

    def export_stagger_threshold_data(self, filename="StaggerThresholds.csv"):
        """StaggerThresholds 데이터를 CSV로 내보내는 메서드"""
        stagger_thresholds_fieldnames = ["Identity_ID", "First_Stagger", "Second_Stagger", "Third_Stagger"]
        stagger_thresholds_data = self.get_stagger_threshold_data()  # stagger_thresholds 데이터 가져오기
        self.export_to_csv(filename, stagger_thresholds_fieldnames, stagger_thresholds_data)  # CSV로 내보내기


async def main():
    data_container = DataContainer()
    scraper = Scraper(container=data_container, base_url="https://limbus.kusoge.xyz", img_url="https://img.kusoge.xyz/limbus/img")
    await scraper.scrape()

await main()

Data successfully exported to StaggerThresholds.csv


In [2]:
import os
import csv
import re
import numpy as np
import pandas as pd
from playwright.async_api import async_playwright
from tabulate import tabulate

# Sin Color 이름 변환을 위한 매핑
resource_mapping = {
    "AZURE": "Gloom",
    "VIOLET": "Envy",
    "AMBER": "Sloth",
    "SHAMROCK": "Gluttony",
    "CRIMSON": "Wrath",
    "SCARLET": "Lust",
    "INDIGO": "Pride"
}

def convert_resource(resource_name):
    """Sin Color 이름을 변환하는 함수. 매핑되지 않은 경우 원래 이름을 반환."""
    return resource_mapping.get(resource_name, resource_name)

class Scraper:
    """웹 페이지 크롤링을 위한 클래스"""
    def __init__(self, container, base_url, img_url):
        self.container = container
        self.base_url = base_url
        self.img_url = img_url

    async def handle_request(self, route, request):
        """이미지와 폰트 리소스 로드를 차단."""
        if request.resource_type in ["image", "font"]:
            await route.abort()
        else:
            await route.continue_()
            
    async def scrape(self):
        """크롤링 메인 함수"""
        async with async_playwright() as p:
            browser = await p.chromium.launch(headless=True)
            page = await browser.new_page()
            await page.route("**/*", self.handle_request)
            # Step 1: /identity 페이지에서 기본 정보 크롤링
            #await self.crawl_identity_page(page)

            # Step 2: /identity/{identity_id} 상세 페이지 크롤링
            #identity_ids_to_load = ["10104", "10310", "10710","10808", "10911", "11005"]
            identity_ids_to_load = ["10104", "10911"]#,"10310", "10710","10808", "11005", "11210"]
            self.container.load_identities_by_id('./csv/identities.csv', identity_ids_to_load)
            await self.crawl_identity_details_for_all(page)
            #self.container.display_identities()
            #self.container.export_identities_data("identities.csv")
            #self.container.export_level_stats_data("Levelstats.csv")
            #self.container.export_uptie_stats_data("Uptiestats.csv")
            #self.container.export_resist_info_data("ResistInfos.csv")
            #self.container.export_stagger_threshold_data("StaggerThresholds.csv")
            page.set_default_timeout(10000)
            await browser.close()

    async def get_identity_id_from_row(self, row):
        """테이블 행(row)에서 인격 ID를 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_url = await identity_cell.get_attribute('href')
            identity_id = identity_url.split('/identity/')[-1]
            return identity_id
        return "Unknown"

    async def get_identity_name_from_row(self, row):
        """테이블 행(row)에서 인격 이름을 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_name = (await identity_cell.inner_text()).replace("\n", " ").strip()
            return identity_name
        return "Unknown"
        
    async def crawl_identity_page(self, page):
        """/identity 페이지로 이동하여 인격 기본 정보를 추출."""
        await page.goto(self.base_url + "/identity")
        await self.extract_identity_page(page)

    async def extract_identity_page(self, page):
        """/identity 페이지의 모든 테이블 행에서 인격 정보를 추출."""
        identity_rows = await page.query_selector_all('tr')
        if len(identity_rows) > 1:
            for idx, identity_row in enumerate(identity_rows):
                if idx == 0:
                    continue # 0번째요소이름을 보여주는 열은 건너뜀
                await self.scrape_identity_name_and_identity_id(identity_row)
                await self.extract_rarity(identity_row)
                for i in range(1, 4):  # 스킬1~3 정보 추출
                    await self.extract_atk_and_resource(identity_row, i)
                await self.extract_defense_and_resource(identity_row)

    async def scrape_identity_name_and_identity_id(self, row):
        """인격 번호와 이름을 추출하여 DataContainer에 저장."""
        try:
            identity_id = await self.get_identity_id_from_row(row)
            identity_name = await self.get_identity_name_from_row(row)
            identity = Identity(identity_id)
            self.container.add_identity(identity)
            self.container.update_identity_name(identity_id, identity_name)
        except Exception as e:
            print(f"Error extracting identity name and identity id: {e}")

    async def get_element_attribute(self, element, selector, attribute):
        """요소에서 지정된 속성(attribute)을 추출."""
        try:
            target_element = await element.query_selector(selector)
            if target_element:
                return await target_element.get_attribute(attribute)
        except Exception as e:
            print(f"Error getting element attribute: {e}")
        return None
    
    async def extract_rarity(self, row):
        """테이블 행(row)에서 희귀도를 추출하여 업데이트."""
        try:
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
                sibling = await parent.evaluate_handle('node => node.nextElementSibling')
                if sibling:
                    rarity_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/rarity_"]', 'src')
                    if rarity_img_src:
                        rarity_level = rarity_img_src.split('_')[-1].split('.')[0]  # rarity_1.png -> 1
                        identity_id = await self.get_identity_id_from_row(row)
                        self.container.update_identity_rarity(identity_id, rarity_level)
                    else:
                        print("Rarity image not found in the next sibling TD.")
        except Exception as e: 
            print(f"Error extracting rarity: {e}")

        return "Unknown"

    async def extract_atk_and_resource(self, row, skill_number):
        """ 공격 타입(ATK Type)과 자원(Attribute) 추출 함수 - 스킬 번호에 따라 처리 """
        try:
            #print(f"Attempting to locate skill{skill_number} type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 해당 스킬 번호에 맞는 형제로 이동
                sibling = parent
                for _ in range(skill_number + 1):  # 스킬 번호에 맞게 이동
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                atk_type, resource = "Unknown", "Unknown"
        
                if sibling:
                    # 공격 타입 이미지 (atk_type) 찾기
                    atk_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/atk_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if atk_type_img_src:
                        atk_type = atk_type_img_src.split('_')[-1].split('.')[0]  # atk_type_SLASH.png -> SLASH
                        #print(f"Extracted attack type: {atk_type}")
                        self.container.update_skill(identity_id, skill_number, atk_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted resource: {resource}")
                        
                        # 리소스를 변환
                        resource = convert_resource(resource)
                        #print(f"Converted resource: {resource}")
                        self.container.update_sin_affinity(identity_id, skill_number, resource)
                        
        except Exception as e:
            print(f"Error extracting skill{skill_number} type and resource: {e}")

    async def extract_defense_and_resource(self, row):
        """ 방어 타입(Def Type)과 자원(Attribute) 추출 함수 """
        try:
            #print("Attempting to locate defense type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 다섯 번째 형제 td로 이동 (방어 관련 정보가 있는 <td>)
                sibling = parent
                for _ in range(5):  # 방어 정보는 5번째 형제에 있음
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                def_type, def_resource = "Unknown", "Unknown"
        
                if sibling:
                    # 방어 타입 이미지 (def_type) 찾기
                    def_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/def_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if def_type_img_src:
                        def_type = def_type_img_src.split('_')[-1].split('.')[0]  # def_type_GUARD.png -> GUARD
                        #print(f"Extracted defense type: {def_type}")
                        self.container.update_skill(identity_id, 'defense', def_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        def_resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted defense resource: {def_resource}")
                        
                        # 방어 리소스를 변환
                        def_resource = convert_resource(def_resource)
                        #print(f"Converted defense resource: {def_resource}")
                        self.container.update_sin_affinity(identity_id, 'defense', def_resource)
 
        except Exception as e:
            print(f"Error extracting defense type and resource: {e}")

    async def crawl_identity_details_for_all(self, page):
        """모든 인격의 세부 정보를 크롤링."""
        for identity in self.container.identities:
            await page.wait_for_load_state("networkidle")
            await page.goto(self.base_url + "/identity/"+identity.identity_id)
            """ 레벨과 동기화와 관련 없는 참관타 저항은 밑에서 변경하는 것과 연관성이 적기 때문에 위에서 처리한다.
            print(f"Identity_id: {identity.identity_id}")
            results = []
            resist_type_imgs = [f"{self.img_url}/ui/atk_type_HIT.png",
                                f"{self.img_url}/ui/atk_type_PENETRATE.png",
                                f"{self.img_url}/ui/atk_type_SLASH.png"]
            for resist_type_img in resist_type_imgs:
                result = await self.extract_resist_type(page, resist_type_img, 1)
                results.append(result)
            self.container.add_resist_info(identity.identity_id, results[0], results[1], results[2])
            """
            """
            # Stagger Threshold 값 추출
            stagger_threshold = await self.extract_stagger_threshold_values(page)
            self.container.add_stagger_threshold(identity.identity_id, stagger_threshold[0], stagger_threshold[1], stagger_threshold[2])
            """
            await self.initialize_uptie_and_level(page)
            """ 레벨에 따른 level, hp, defense 다르게 되어 있는 것을 출력 확인 객체로 들어가서 csv로 옮기는 과정까지 진행
            for i in range(1, identity.level_max+1):
                await self.move_slider_to_value(page,i)
                await self.extract_level_status(page, identity.identity_id, i)
            """
            """ 동기화에 따른 speed 다르게 되어 있는 것을 출력 확인 객체로 들어가서 csv로 옮기는 과정까지 진행
            for i in range(1, identity.uptie_max+1):
                await self.uptie_click(page,i)
                await self.extract_uptie_status(page, identity.identity_id, i)
            """
            
    async def initialize_uptie_and_level(self, page):
        await self.uptie_click(page, 1)
        await self.move_slider_to_value(page, 1)
    
    async def uptie_click(self, page, phase):
        fab_button_selector = ".q-btn.q-btn--fab"
        await page.wait_for_selector(fab_button_selector)
        await page.click(fab_button_selector)
        #print(f"[{setup_uptie_button}] 상위 동기화 버튼 클릭 완료")
        setup_uptie_button = f'a:has(img[src="{self.img_url}/ui/uptie_{phase}.png"])'
        await page.wait_for_selector(setup_uptie_button)
        await page.click(setup_uptie_button)
        #print(f"{phase} 동기화 버튼 클릭 완료")

    async def move_slider_to_value(self, page, target_value):
        """
        슬라이더를 주어진 값으로 이동시킵니다.
        target_value는 슬라이더의 최소값부터 최대값 사이여야 합니다.
        """
        slider_selector = ".q-slider"
        # 슬라이더의 범위 확인
        min_value = int(await page.get_attribute(slider_selector, "aria-valuemin"))
        max_value = int(await page.get_attribute(slider_selector, "aria-valuemax"))
    
        if target_value < min_value or target_value > max_value:
            raise ValueError(f"Target value {target_value} is out of bounds ({min_value}, {max_value})")
    
        # 슬라이더 thumb 요소와 위치 가져오기
        slider_thumb = await page.query_selector(f"{slider_selector} .q-slider__thumb")
        thumb_box = await slider_thumb.bounding_box()
        
        # 슬라이더의 현재 위치 및 목표 위치 계산
        track = await page.query_selector(f"{slider_selector} .q-slider__track")
        track_box = await track.bounding_box()
        
        # 목표 x 위치 계산 (슬라이더 트랙 내에서 비율 기반)
        target_percentage = (target_value - min_value) / (max_value - min_value)
        target_x = track_box["x"] + target_percentage * track_box["width"]
        current_x = thumb_box["x"] + thumb_box["width"] / 2
        y = thumb_box["y"] + thumb_box["height"] / 2
    
        # 슬라이더를 드래그하여 목표 위치로 이동
        await page.mouse.move(current_x, y)
        await page.mouse.down()
        await page.mouse.move(target_x, y, steps=10)  # 부드럽게 이동
        await page.mouse.up()
    
        #print(f"슬라이더를 {target_value}로 이동 완료.")
    
        # 슬라이더 값 확인 및 강제 이벤트 트리거
        await page.evaluate(
            f"""
            (slider) => {{
                slider.dispatchEvent(new Event('input'));  // oninput 이벤트 트리거
                slider.dispatchEvent(new Event('change'));  // onchange 이벤트 트리거
            }}
            """,
            await page.query_selector(slider_selector)
        )
    
        # 이동 후 값 확인
        current_value = await page.get_attribute(slider_selector, "aria-valuenow")
        if current_value != str(target_value):
            print(f"경고: 슬라이더 값이 예상과 다릅니다. 현재 값: {current_value}, 예상 값: {target_value}")

    async def get_second_status_element(self, page):
        """
        두 번째 'Status' 요소에 접근하여 반환하는 메소드
        """
        status_elements = await page.query_selector_all("text=Status")
        if len(status_elements) > 1:
            second_status_element = status_elements[1]
            #print("두 번째 'Status' 요소에 접근합니다.")
            return second_status_element
        else:
            #print("두 번째 'Status' 요소가 존재하지 않습니다.")
            return None

    async def extract_level_status(self, page, identity_id, level):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 레벨에 따라 HP와 Defense가 변한다.
        hp_value = await self.extract_hp(second_status_element)
        defense_value = await self.extract_defense(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Level={level}, HP={hp_value}, Defense={defense_value}")
        self.container.add_level_stat(identity_id,level, hp_value, defense_value)
    
    async def extract_hp(self, status_element):
        # HP 추출
        hp_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].children[0].nextElementSibling.textContent.trim()"
        )
        return hp_value
    
    async def extract_defense(self, status_element):
        # Defense 추출
        defense_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return defense_value

    async def extract_uptie_status(self, page, identity_id, uptie):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 uptie에 따라 speed는 변한다.
        speed_value = await self.extract_speed(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Uptie={uptie}, Speed={speed_value}")
        self.container.add_uptie_stat(identity_id,uptie, speed_value)

    async def extract_speed(self, status_element):
        # Speed 추출
        speed_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return speed_value

    async def extract_resist_type(self, page, image_src, depth):
        # 이미지 요소 찾기
        #print(image_src)
        image = await page.query_selector(f'img[src="{image_src}"]')
        #print(image)
        if not image:
            return "Unknown"
    
        # 이미지의 부모 요소를 찾아서 올라감
        parent = image
        for _ in range(depth):
            parent = await parent.evaluate_handle('node => node.parentElement')
            if not parent:
                return "Unknown"
    
        # 부모의 형제 요소 찾기
        sibling = await parent.evaluate_handle('node => node.nextElementSibling')
        
        if sibling:
            value = await sibling.evaluate('node => node.textContent')
            if value:
                return value.strip()
        
        return "Unknown"

    async def extract_stagger_threshold_values(self, page):
        # 'div.row.center.flex-center.q-pt-xs' 요소들 찾기
        elements = await page.query_selector_all('div.row.center.flex-center.q-pt-xs')
        
        # 각 요소의 텍스트 내용 출력하여 'Stagger Threshold' 요소를 찾기
        for element in elements:
            text = await element.text_content()
            
            if text and "Stagger Threshold" in text:
                # 'Stagger Threshold'가 있는 요소를 찾았다면, 그 이후 형제 요소들에서 값을 추출
                parent = await element.evaluate_handle('node => node.parentElement')
                sibling_elements = await parent.query_selector_all('div.row.justify-evenly div')  # Threshold 값이 들어있는 div들
                
                stagger_values = []
                for sibling in sibling_elements:
                    text_content = await sibling.text_content()
                    match = re.search(r'(\d+%)', text_content)
                    if match:
                        stagger_values.append(match.group(1))  # 퍼센트 값만 리스트에 추가
                while len(stagger_values) < 3:
                    stagger_values.append(np.nan)
                #print(stagger_values)
                return stagger_values  # 추출된 값(또는 np.nan 포함)을 반환
        
        return [np.nan, np.nan, np.nan]  # 만약 'Stagger Threshold'를 찾지 못한 경우 기본값 반환

class Identity:
    """인격 정보를 저장하는 클래스"""
    def __init__(self, identity_id, name=None, rarity=None,\
                 skill_1_type=None, skill_1_sin_affinity=None,\
                 skill_2_type=None, skill_2_sin_affinity=None,\
                 skill_3_type=None, skill_3_sin_affinity=None,\
                 defense_type=None, defense_sin_affinity=None):
        self.identity_id = identity_id
        self.name = name
        self.rarity = rarity
        # 선언만 하는 부분 
        self.skill_1_type = skill_1_type 
        self.skill_1_sin_affinity = skill_1_sin_affinity 
        self.skill_2_type = skill_2_type 
        self.skill_2_sin_affinity = skill_2_sin_affinity 
        self.skill_3_type = skill_3_type 
        self.skill_3_sin_affinity = skill_3_sin_affinity 
        self.defense_type = defense_type 
        self.defense_sin_affinity = defense_sin_affinity
        self.level_max = 50
        self.uptie_max = 4
        self.level_stats = []  # LevelStat 객체 리스트
        self.uptie_stats = [] # UptieStat 객체 리스트
        self.resist_info = None
        self.stagger_threshold = None
        self.skillset = None  # Skill정보를 담고 있는 skillset 객체 
        self.passive_infos = {}  # 패시브/서브 패시 정보를 담고 있는 딕셔너리
        self.panic_type = {} # 혼란상태 발생시 나타나는 효과를 담고 있는 딕셔너리

    def update_name(self, name):
        self.name = name

    def update_rarity(self, rarity):
        self.rarity = rarity

    def add_level_stat(self, level, hp, defense):
        self.level_stats.append(LevelStat(level, hp, defense))

    def add_uptie_stat(self, uptie, speed):
        self.uptie_stats.append(UptieStat(uptie, speed))

    def add_resist_info(self, resist_hit, resist_penetrate, resist_slash):
        self.resist_info = ResistInfo(resist_hit, resist_penetrate, resist_slash)

    def add_stagger_threshold(self, first_stagger, second_stagger, third_stagger):
        self.stagger_threshold = StaggerThreshold(first_stagger, second_stagger, third_stagger)

    def update_details(self, page):
        # 세부 정보 업데이트 로직 (가정)
        pass
    #def init_uptie(self, page):
    #    init_uptie = 

# LevelStat 객체 정의
class LevelStat:
    def __init__(self, level, hp=None, defense=None):
        self.level = level
        self.hp = hp
        self.defense = defense
    def update_level(self, level):
        self.level = level
    def update_hp(self, hp):
        self.hp = hp
    def update_defense(self, defense):
        self.defense = defense

# UptieStat 객체 정의
class UptieStat:
    def __init__(self, uptie, speed=None):
        self.uptie = uptie
        self.speed = speed
    def update_speed(self, speed):
        self.speed = speed

# ResistInfo 객체 정의
class ResistInfo:
    def __init__(self, resist_hit=None, resist_penetrate=None, resist_slash=None):
        self.resist_hit = resist_hit
        self.resist_penetrate = resist_penetrate
        self.resist_slash = resist_slash

# ResistInfo 객체 정의
class StaggerThreshold:
    def __init__(self, first_stagger=None, second_stagger=None, third_stagger=None):
        self.first_stagger = first_stagger
        self.second_stagger = second_stagger
        self.third_stagger = third_stagger

class SkillSet:
    def __init__(self):
        self.skill_1_count = 0
        self.skill_2_count = 0
        self.skill_3_count = 0
        self.defense_skill_count = 0

        self.skill_1_type = ""
        self.skill_2_type = ""
        self.skill_3_type = ""
        self.defense_skill_type = ""

        self.skill_1_sin_affinity = ""
        self.skill_2_sin_affinity = ""
        self.skill_3_sin_affinity = ""
        self.defense_skill_sin_affinity = ""

        self.skill_1_details = []
        self.skill_2_details = []
        self.skill_3_details = []
        self.defense_skill_details = []

    def add_skill(self, skill_code, skill):
        if skill_code == 1:
            self.skill_1_details.append(skill)
            self.skill_1_count += 1
        elif skill_code == 2:
            self.skill_2_details.append(skill)
            self.skill_2_count += 1
        elif skill_code == 3:
            self.skill_3_details.append(skill)
            self.skill_3_count += 1
        elif skill_code == "defense":
            self.defense_skill_details.append(skill)
            self.defense_skill_count += 1

    def set_skill_type(self, skill_code, skill_type, sin_affinity):
        if skill_code == 1:
            self.skill_1_type = skill_type
            self.skill_1_sin_affinity = sin_affinity
        elif skill_code == 2:
            self.skill_2_type = skill_type
            self.skill_2_sin_affinity = sin_affinity
        elif skill_code == 3:
            self.skill_3_type = skill_type
            self.skill_3_sin_affinity = sin_affinity
        elif skill_type == "defense":
            self.defense_skill_type = skill_type
            self.defense_skill_sin_affinity = sin_affinity

class Skill:
    def __init__(self, skill_code, skill_name, coin_count, init_level, init_remain=None, akt_weight=None):
        self.skill_code = skill_code # "Skill 1"/"Skill 2"/"Skill 3"/"Defense"
        self.skill_name = skill_name # 스킬 이름
        self.coin_count = coin_count # 동전 개수
        self.init_level = init_level # 스킬이 초기 배정받은 레벨, 인격 레벨에 - 1 한 값을 더해주면 스킬의 현 level을 구할 수 있다.
        """
        남아있는 스킬 개수 사용하면, 하나씩 줄어들고 모든 스킬을 사용하면 이 값으로 돌아간다.
        스킬의 남은 숫자없는 스킬은 강화된 스킬로 한번 사용되면 다 조건을 채울 때까 사용 못한다.
        강화된 스킬을 None으로 설정하는 방식과 조건이 채워지면 1로 늘리는 방식이 존재한다.
        스킬을 사용할 수 있는 초기값이다.
        조건을 충족한 강화한 스킬을 이례적인 절차라고 생각하고 None으로 초기값을 설정했다.
        """
        self.init_remain = None 
        self.akt_weight = None #공격 가중치 값이다. 방어스킬에는 없기 때문에 None으로 설정했다.
        self.remain_count = init_remain
        """
        동기화 단계에 따른 정보를 담는다. 현재 4동기화까지 되어 있지만 차후 확장될 것을 생각하여 고정하지 않는다.
        내부에 들어있는 정보로는 uptie_level, skill_power, coin_power 속성으로 들어있다.
        전투, 턴, 스킬, 코인토스 단계로 내려오며 발생하는 효과에 관하여 저장한다.
        """
        self.uptie_infos = []

class UptieInfo:
    def __init__(self, uptie_level, skill_power, coin_power):
        self.uptie_level = uptie_level
        self.skill_power = skill_power
        self.coin_power = coin_power
        self.skill_effects = []

class Effect:
    def __init__(self, condition, description):
        self.condition = condition # 조건 텍스트 "[coin 1 on hit]"
        self.description = description # 효과 설명 텍스 "Inflict +1 Rupture Count
      
class DataContainer:
    """크롤링 데이터를 관리하는 클래스"""
    def __init__(self):
        self.identities = []

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.level_max

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.uptie_max

    def update_identity_name(self, identity_id, identity_name):
        """특정 ID의 이름을 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_name(identity_name)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_identity_rarity(self, identity_id, rarity):
        """특정 ID의 희귀도를 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_rarity(rarity)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_skill(self, identity_id, skill_number, new_type):
        """
        특정 identity의 skill_type을 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_type: 새 skill_type
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_type = new_type
                elif skill_number == 2:
                    identity.skill_2_type = new_type
                elif skill_number == 3:
                    identity.skill_3_type = new_type
                elif skill_number == "defense":
                    identity.defense_type = new_type
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def update_sin_affinity(self, identity_id, skill_number, new_affinity):
        """
        특정 identity의 sin_affinity를 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_affinity: 새 sin_affinity
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_sin_affinity = new_affinity
                elif skill_number == 2:
                    identity.skill_2_sin_affinity = new_affinity
                elif skill_number == 3:
                    identity.skill_3_sin_affinity = new_affinity
                elif skill_number == "defense":
                    identity.defense_sin_affinity = new_affinity
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def add_identity(self, identity):
        """새로운 인격 데이터를 추가."""
        self.identities.append(identity)

    def add_level_stat(self, identity_id, level, hp, defense):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_level_stat(level, hp, defense)

    def add_uptie_stat(self, identity_id, uptie, speed):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_uptie_stat(uptie, speed)

    def add_resist_info(self, identity_id, resist_hit, resist_penetrate, resist_slash):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_resist_info(resist_hit, resist_penetrate, resist_slash)

    def add_stagger_threshold(self, identity_id, first_stagger, second_stagger, third_stagger):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_stagger_threshold(first_stagger, second_stagger, third_stagger)
        
    def display_identities(self):
        """Identity, ID, Name, Rarity 정보를 출력"""
        if not self.identities:
            print("No identities found.")
        else:
            # 데이터를 리스트로 변환
            table = []
            for idx, identity in enumerate(self.identities, 1):
                table.append([
                    idx, identity.identity_id, identity.name, identity.rarity,
                    identity.skill_1_type, identity.skill_1_sin_affinity,
                    identity.skill_2_type, identity.skill_2_sin_affinity,
                    identity.skill_3_type, identity.skill_3_sin_affinity,
                    identity.defense_type, identity.defense_sin_affinity
                ])
            
            # 테이블 헤더 정의
            headers = [
                "Index", "ID", "Name", "Rarity",
                "Skill_1 Type", "Skill_1 Sin", "Skill_2 Type", "Skill_2 Sin",
                "Skill_3 Type", "Skill_3 Sin", "Defense Type", "Defense Sin"
            ]
            
            # 테이블 출력
            print(tabulate(table, headers=headers, tablefmt="fancy_grid"))

    def export_to_csv(self, filename, fieldnames, data):
        """
        공통적으로 사용 가능한 CSV 내보내기 메서드
        :param filename: 내보낼 파일 이름 (예: 'identity.csv', 'skill.csv')
        :param fieldnames: CSV의 헤더 리스트
        :param data: CSV로 내보낼 데이터 (리스트 형태)
        """
        try:
            # 현재작업 디렉토 기준으 'csv'폴더 없으면 생성
            if not os.path.exists('./csv'):
                os.makedirs('./csv')
            # 파일 경로 설정 (상대경로 'csv' 폴더 안에 저장)
            file_path = os.path.join('./csv', filename)
            with open(file_path, mode='w', newline='', encoding='utf-8') as file:
                writer = csv.DictWriter(file, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(data)
            print(f"Data successfully exported to {filename}")
        except Exception as e:
            print(f"Error exporting to CSV: {e}")

    def get_identity_data(self):
        """
        Identity 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "ID": identity.identity_id,
                "Name": identity.name,
                "Rarity": identity.rarity,
                "Skill_1_Type": identity.skill_1_type,
                "Skill_1_Sin_Affinity": identity.skill_1_sin_affinity,
                "Skill_2_Type": identity.skill_2_type,
                "Skill_2_Sin_Affinity": identity.skill_2_sin_affinity,
                "Skill_3_Type": identity.skill_3_type,
                "Skill_3_Sin_Affinity": identity.skill_3_sin_affinity,
                "Defense_Type": identity.defense_type,
                "Defense_Sin_Affinity": identity.defense_sin_affinity,
            }
            for identity in self.identities
        ]
        
    def export_identities_data(self, filename="identities.csv"):
        """Identity 데이터를 CSV로 내보내는 메서드"""
        identity_fieldnames = [
            "ID", "Name", "Rarity",
            "Skill_1_Type", "Skill_1_Sin_Affinity",
            "Skill_2_Type", "Skill_2_Sin_Affinity",
            "Skill_3_Type", "Skill_3_Sin_Affinity",
            "Defense_Type", "Defense_Sin_Affinity"
        ]
        identity_data = self.get_identity_data()  # Identity 데이터 가져오기
        self.export_to_csv(filename, identity_fieldnames, identity_data)  # CSV로 내보내기

    def load_identities_by_id(self, filename, identity_ids):
        """지정된 identity_ids 목록에 해당하는 Identity 데이터를 CSV에서 읽어옴."""
        identity_ids_set = set(identity_ids) # 중복 제거를 위 Set 사용
        with open(filename, mode='r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                if row["ID"] in identity_ids_set:
                    identity = Identity(
                        identity_id=row["ID"],
                        name=row["Name"],
                        rarity=row["Rarity"],
                        skill_1_type=row["Skill_1_Type"],
                        skill_1_sin_affinity=row["Skill_1_Sin_Affinity"],
                        skill_2_type=row["Skill_2_Type"],
                        skill_2_sin_affinity=row["Skill_2_Sin_Affinity"],
                        skill_3_type=row["Skill_3_Type"],
                        skill_3_sin_affinity=row["Skill_3_Sin_Affinity"],
                        defense_type=row["Defense_Type"],
                        defense_sin_affinity=row["Defense_Sin_Affinity"]
                    )
                    self.identities.append(identity)

    def get_level_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Level": level_stat.level,
                "HP": level_stat.hp,
                "Defense": level_stat.defense,
            }
            for identity in self.identities
            for level_stat in identity.level_stats
        ]

    def export_level_stats_data(self, filename="levelstats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        level_stats_fieldnames = ["Identity_ID", "Level", "HP", "Defense"]
        level_stats_data = self.get_level_stats_data()  # level_stats 데이터 가져오기
        self.export_to_csv(filename, level_stats_fieldnames, level_stats_data)  # CSV로 내보내기

    def get_uptie_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Uptie": uptie_stat.uptie,
                "Speed": uptie_stat.speed,
            }
            for identity in self.identities
            for uptie_stat in identity.uptie_stats
        ]

    def export_uptie_stats_data(self, filename="Uptiestats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        uptie_stats_fieldnames = ["Identity_ID", "Uptie", "Speed"]
        uptie_stats_data = self.get_uptie_stats_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, uptie_stats_fieldnames, uptie_stats_data)  # CSV로 내보내기

    def get_resist_info_data(self):
        """
        모든 Identity의 resist_info 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Resist_Hit": identity.resist_info.resist_hit,
                "Resist_Penetrate": identity.resist_info.resist_penetrate,
                "Resist_Slash": identity.resist_info.resist_slash,
            }
            for identity in self.identities
        ]

    def export_resist_info_data(self, filename="ResistInfos.csv"):
        """ResistInfo 데이터를 CSV로 내보내는 메서드"""
        resist_infos_fieldnames = ["Identity_ID", "Resist_Hit", "Resist_Penetrate", "Resist_Slash"]
        resist_infos_data = self.get_resist_info_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, resist_infos_fieldnames, resist_infos_data)  # CSV로 내보내기

    def export_uptie_stats_data(self, filename="Uptiestats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        uptie_stats_fieldnames = ["Identity_ID", "Uptie", "Speed"]
        uptie_stats_data = self.get_uptie_stats_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, uptie_stats_fieldnames, uptie_stats_data)  # CSV로 내보내기

    def get_stagger_threshold_data(self):
        """
        모든 Identity의 stagger_threshold 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "First_Stagger": identity.stagger_threshold.first_stagger,
                "Second_Stagger": identity.stagger_threshold.second_stagger,
                "Third_Stagger": identity.stagger_threshold.third_stagger,
            }
            for identity in self.identities
        ]

    def export_stagger_threshold_data(self, filename="StaggerThresholds.csv"):
        """StaggerThresholds 데이터를 CSV로 내보내는 메서드"""
        stagger_thresholds_fieldnames = ["Identity_ID", "First_Stagger", "Second_Stagger", "Third_Stagger"]
        stagger_thresholds_data = self.get_stagger_threshold_data()  # stagger_thresholds 데이터 가져오기
        self.export_to_csv(filename, stagger_thresholds_fieldnames, stagger_thresholds_data)  # CSV로 내보내기


async def main():
    data_container = DataContainer()
    scraper = Scraper(container=data_container, base_url="https://limbus.kusoge.xyz", img_url="https://img.kusoge.xyz/limbus/img")
    await scraper.scrape()

await main()

In [1]:
import os
import csv
import re
import numpy as np
import pandas as pd
from playwright.async_api import async_playwright
from tabulate import tabulate

# Sin Color 이름 변환을 위한 매핑
resource_mapping = {
    "AZURE": "Gloom",
    "VIOLET": "Envy",
    "AMBER": "Sloth",
    "SHAMROCK": "Gluttony",
    "CRIMSON": "Wrath",
    "SCARLET": "Lust",
    "INDIGO": "Pride"
}

def convert_resource(resource_name):
    """Sin Color 이름을 변환하는 함수. 매핑되지 않은 경우 원래 이름을 반환."""
    return resource_mapping.get(resource_name, resource_name)

class Scraper:
    """웹 페이지 크롤링을 위한 클래스"""
    def __init__(self, container, base_url, img_url):
        self.container = container
        self.base_url = base_url
        self.img_url = img_url

    async def handle_request(self, route, request):
        """이미지와 폰트 리소스 로드를 차단."""
        if request.resource_type in ["image", "font"]:
            await route.abort()
        else:
            await route.continue_()
            
    async def scrape(self):
        """크롤링 메인 함수"""
        async with async_playwright() as p:
            browser = await p.chromium.launch(headless=True)
            page = await browser.new_page()
            await page.route("**/*", self.handle_request)
            # Step 1: /identity 페이지에서 기본 정보 크롤링
            #await self.crawl_identity_page(page)

            # Step 2: /identity/{identity_id} 상세 페이지 크롤링
            #identity_ids_to_load = ["10104", "10310", "10710","10808", "10911", "11005"]
            identity_ids_to_load = ["10104", "10911"]#,"10310", "10710","10808", "11005", "11210"]
            self.container.load_identities_by_id('./csv/identities.csv', identity_ids_to_load)
            await self.crawl_identity_details_for_all(page)
            #self.container.display_identities()
            #self.container.export_identities_data("identities.csv")
            #self.container.export_level_stats_data("Levelstats.csv")
            #self.container.export_uptie_stats_data("Uptiestats.csv")
            #self.container.export_resist_info_data("ResistInfos.csv")
            #self.container.export_stagger_threshold_data("StaggerThresholds.csv")
            page.set_default_timeout(10000)
            await browser.close()

    async def get_identity_id_from_row(self, row):
        """테이블 행(row)에서 인격 ID를 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_url = await identity_cell.get_attribute('href')
            identity_id = identity_url.split('/identity/')[-1]
            return identity_id
        return "Unknown"

    async def get_identity_name_from_row(self, row):
        """테이블 행(row)에서 인격 이름을 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_name = (await identity_cell.inner_text()).replace("\n", " ").strip()
            return identity_name
        return "Unknown"
        
    async def crawl_identity_page(self, page):
        """/identity 페이지로 이동하여 인격 기본 정보를 추출."""
        await page.goto(self.base_url + "/identity")
        await self.extract_identity_page(page)

    async def extract_identity_page(self, page):
        """/identity 페이지의 모든 테이블 행에서 인격 정보를 추출."""
        identity_rows = await page.query_selector_all('tr')
        if len(identity_rows) > 1:
            for idx, identity_row in enumerate(identity_rows):
                if idx == 0:
                    continue # 0번째요소이름을 보여주는 열은 건너뜀
                await self.scrape_identity_name_and_identity_id(identity_row)
                await self.extract_rarity(identity_row)
                for i in range(1, 4):  # 스킬1~3 정보 추출
                    await self.extract_atk_and_resource(identity_row, i)
                await self.extract_defense_and_resource(identity_row)

    async def scrape_identity_name_and_identity_id(self, row):
        """인격 번호와 이름을 추출하여 DataContainer에 저장."""
        try:
            identity_id = await self.get_identity_id_from_row(row)
            identity_name = await self.get_identity_name_from_row(row)
            identity = Identity(identity_id)
            self.container.add_identity(identity)
            self.container.update_identity_name(identity_id, identity_name)
        except Exception as e:
            print(f"Error extracting identity name and identity id: {e}")

    async def get_element_attribute(self, element, selector, attribute):
        """요소에서 지정된 속성(attribute)을 추출."""
        try:
            target_element = await element.query_selector(selector)
            if target_element:
                return await target_element.get_attribute(attribute)
        except Exception as e:
            print(f"Error getting element attribute: {e}")
        return None
    
    async def extract_rarity(self, row):
        """테이블 행(row)에서 희귀도를 추출하여 업데이트."""
        try:
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
                sibling = await parent.evaluate_handle('node => node.nextElementSibling')
                if sibling:
                    rarity_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/rarity_"]', 'src')
                    if rarity_img_src:
                        rarity_level = rarity_img_src.split('_')[-1].split('.')[0]  # rarity_1.png -> 1
                        identity_id = await self.get_identity_id_from_row(row)
                        self.container.update_identity_rarity(identity_id, rarity_level)
                    else:
                        print("Rarity image not found in the next sibling TD.")
        except Exception as e: 
            print(f"Error extracting rarity: {e}")

        return "Unknown"

    async def extract_atk_and_resource(self, row, skill_number):
        """ 공격 타입(ATK Type)과 자원(Attribute) 추출 함수 - 스킬 번호에 따라 처리 """
        try:
            #print(f"Attempting to locate skill{skill_number} type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 해당 스킬 번호에 맞는 형제로 이동
                sibling = parent
                for _ in range(skill_number + 1):  # 스킬 번호에 맞게 이동
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                atk_type, resource = "Unknown", "Unknown"
        
                if sibling:
                    # 공격 타입 이미지 (atk_type) 찾기
                    atk_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/atk_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if atk_type_img_src:
                        atk_type = atk_type_img_src.split('_')[-1].split('.')[0]  # atk_type_SLASH.png -> SLASH
                        #print(f"Extracted attack type: {atk_type}")
                        self.container.update_skill(identity_id, skill_number, atk_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted resource: {resource}")
                        
                        # 리소스를 변환
                        resource = convert_resource(resource)
                        #print(f"Converted resource: {resource}")
                        self.container.update_sin_affinity(identity_id, skill_number, resource)
                        
        except Exception as e:
            print(f"Error extracting skill{skill_number} type and resource: {e}")

    async def extract_defense_and_resource(self, row):
        """ 방어 타입(Def Type)과 자원(Attribute) 추출 함수 """
        try:
            #print("Attempting to locate defense type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 다섯 번째 형제 td로 이동 (방어 관련 정보가 있는 <td>)
                sibling = parent
                for _ in range(5):  # 방어 정보는 5번째 형제에 있음
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                def_type, def_resource = "Unknown", "Unknown"
        
                if sibling:
                    # 방어 타입 이미지 (def_type) 찾기
                    def_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/def_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if def_type_img_src:
                        def_type = def_type_img_src.split('_')[-1].split('.')[0]  # def_type_GUARD.png -> GUARD
                        #print(f"Extracted defense type: {def_type}")
                        self.container.update_skill(identity_id, 'defense', def_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        def_resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted defense resource: {def_resource}")
                        
                        # 방어 리소스를 변환
                        def_resource = convert_resource(def_resource)
                        #print(f"Converted defense resource: {def_resource}")
                        self.container.update_sin_affinity(identity_id, 'defense', def_resource)
 
        except Exception as e:
            print(f"Error extracting defense type and resource: {e}")

    async def crawl_identity_details_for_all(self, page):
        """모든 인격의 세부 정보를 크롤링."""
        for identity in self.container.identities:
            await page.wait_for_load_state("networkidle")
            await page.goto(self.base_url + "/identity/"+identity.identity_id)
            """ 레벨과 동기화와 관련 없는 참관타 저항은 밑에서 변경하는 것과 연관성이 적기 때문에 위에서 처리한다.
            print(f"Identity_id: {identity.identity_id}")
            results = []
            resist_type_imgs = [f"{self.img_url}/ui/atk_type_HIT.png",
                                f"{self.img_url}/ui/atk_type_PENETRATE.png",
                                f"{self.img_url}/ui/atk_type_SLASH.png"]
            for resist_type_img in resist_type_imgs:
                result = await self.extract_resist_type(page, resist_type_img, 1)
                results.append(result)
            self.container.add_resist_info(identity.identity_id, results[0], results[1], results[2])
            """
            """
            # Stagger Threshold 값 추출
            stagger_threshold = await self.extract_stagger_threshold_values(page)
            self.container.add_stagger_threshold(identity.identity_id, stagger_threshold[0], stagger_threshold[1], stagger_threshold[2])
            """
            await self.initialize_uptie_and_level(page)
            """ 레벨에 따른 level, hp, defense 다르게 되어 있는 것을 출력 확인 객체로 들어가서 csv로 옮기는 과정까지 진행
            for i in range(1, identity.level_max+1):
                await self.move_slider_to_value(page,i)
                await self.extract_level_status(page, identity.identity_id, i)
            """
            """ 동기화에 따른 speed 다르게 되어 있는 것을 출력 확인 객체로 들어가서 csv로 옮기는 과정까지 진행
            for i in range(1, identity.uptie_max+1):
                await self.uptie_click(page,i)
                await self.extract_uptie_status(page, identity.identity_id, i)
            """
            
    async def initialize_uptie_and_level(self, page):
        await self.uptie_click(page, 1)
        await self.move_slider_to_value(page, 1)
    
    async def uptie_click(self, page, phase):
        fab_button_selector = ".q-btn.q-btn--fab"
        await page.wait_for_selector(fab_button_selector)
        await page.click(fab_button_selector)
        #print(f"[{setup_uptie_button}] 상위 동기화 버튼 클릭 완료")
        setup_uptie_button = f'a:has(img[src="{self.img_url}/ui/uptie_{phase}.png"])'
        await page.wait_for_selector(setup_uptie_button)
        await page.click(setup_uptie_button)
        #print(f"{phase} 동기화 버튼 클릭 완료")

    async def move_slider_to_value(self, page, target_value):
        """
        슬라이더를 주어진 값으로 이동시킵니다.
        target_value는 슬라이더의 최소값부터 최대값 사이여야 합니다.
        """
        slider_selector = ".q-slider"
        # 슬라이더의 범위 확인
        min_value = int(await page.get_attribute(slider_selector, "aria-valuemin"))
        max_value = int(await page.get_attribute(slider_selector, "aria-valuemax"))
    
        if target_value < min_value or target_value > max_value:
            raise ValueError(f"Target value {target_value} is out of bounds ({min_value}, {max_value})")
    
        # 슬라이더 thumb 요소와 위치 가져오기
        slider_thumb = await page.query_selector(f"{slider_selector} .q-slider__thumb")
        thumb_box = await slider_thumb.bounding_box()
        
        # 슬라이더의 현재 위치 및 목표 위치 계산
        track = await page.query_selector(f"{slider_selector} .q-slider__track")
        track_box = await track.bounding_box()
        
        # 목표 x 위치 계산 (슬라이더 트랙 내에서 비율 기반)
        target_percentage = (target_value - min_value) / (max_value - min_value)
        target_x = track_box["x"] + target_percentage * track_box["width"]
        current_x = thumb_box["x"] + thumb_box["width"] / 2
        y = thumb_box["y"] + thumb_box["height"] / 2
    
        # 슬라이더를 드래그하여 목표 위치로 이동
        await page.mouse.move(current_x, y)
        await page.mouse.down()
        await page.mouse.move(target_x, y, steps=10)  # 부드럽게 이동
        await page.mouse.up()
    
        #print(f"슬라이더를 {target_value}로 이동 완료.")
    
        # 슬라이더 값 확인 및 강제 이벤트 트리거
        await page.evaluate(
            f"""
            (slider) => {{
                slider.dispatchEvent(new Event('input'));  // oninput 이벤트 트리거
                slider.dispatchEvent(new Event('change'));  // onchange 이벤트 트리거
            }}
            """,
            await page.query_selector(slider_selector)
        )
    
        # 이동 후 값 확인
        current_value = await page.get_attribute(slider_selector, "aria-valuenow")
        if current_value != str(target_value):
            print(f"경고: 슬라이더 값이 예상과 다릅니다. 현재 값: {current_value}, 예상 값: {target_value}")

    async def get_second_status_element(self, page):
        """
        두 번째 'Status' 요소에 접근하여 반환하는 메소드
        """
        status_elements = await page.query_selector_all("text=Status")
        if len(status_elements) > 1:
            second_status_element = status_elements[1]
            #print("두 번째 'Status' 요소에 접근합니다.")
            return second_status_element
        else:
            #print("두 번째 'Status' 요소가 존재하지 않습니다.")
            return None

    async def extract_level_status(self, page, identity_id, level):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 레벨에 따라 HP와 Defense가 변한다.
        hp_value = await self.extract_hp(second_status_element)
        defense_value = await self.extract_defense(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Level={level}, HP={hp_value}, Defense={defense_value}")
        self.container.add_level_stat(identity_id,level, hp_value, defense_value)
    
    async def extract_hp(self, status_element):
        # HP 추출
        hp_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].children[0].nextElementSibling.textContent.trim()"
        )
        return hp_value
    
    async def extract_defense(self, status_element):
        # Defense 추출
        defense_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return defense_value

    async def extract_uptie_status(self, page, identity_id, uptie):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 uptie에 따라 speed는 변한다.
        speed_value = await self.extract_speed(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Uptie={uptie}, Speed={speed_value}")
        self.container.add_uptie_stat(identity_id,uptie, speed_value)

    async def extract_speed(self, status_element):
        # Speed 추출
        speed_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return speed_value

    async def extract_resist_type(self, page, image_src, depth):
        # 이미지 요소 찾기
        #print(image_src)
        image = await page.query_selector(f'img[src="{image_src}"]')
        #print(image)
        if not image:
            return "Unknown"
    
        # 이미지의 부모 요소를 찾아서 올라감
        parent = image
        for _ in range(depth):
            parent = await parent.evaluate_handle('node => node.parentElement')
            if not parent:
                return "Unknown"
    
        # 부모의 형제 요소 찾기
        sibling = await parent.evaluate_handle('node => node.nextElementSibling')
        
        if sibling:
            value = await sibling.evaluate('node => node.textContent')
            if value:
                return value.strip()
        
        return "Unknown"

    async def extract_stagger_threshold_values(self, page):
        # 'div.row.center.flex-center.q-pt-xs' 요소들 찾기
        elements = await page.query_selector_all('div.row.center.flex-center.q-pt-xs')
        
        # 각 요소의 텍스트 내용 출력하여 'Stagger Threshold' 요소를 찾기
        for element in elements:
            text = await element.text_content()
            
            if text and "Stagger Threshold" in text:
                # 'Stagger Threshold'가 있는 요소를 찾았다면, 그 이후 형제 요소들에서 값을 추출
                parent = await element.evaluate_handle('node => node.parentElement')
                sibling_elements = await parent.query_selector_all('div.row.justify-evenly div')  # Threshold 값이 들어있는 div들
                
                stagger_values = []
                for sibling in sibling_elements:
                    text_content = await sibling.text_content()
                    match = re.search(r'(\d+%)', text_content)
                    if match:
                        stagger_values.append(match.group(1))  # 퍼센트 값만 리스트에 추가
                while len(stagger_values) < 3:
                    stagger_values.append(np.nan)
                #print(stagger_values)
                return stagger_values  # 추출된 값(또는 np.nan 포함)을 반환
        
        return [np.nan, np.nan, np.nan]  # 만약 'Stagger Threshold'를 찾지 못한 경우 기본값 반환

class Identity:
    """인격 정보를 저장하는 클래스"""
    def __init__(self, identity_id, name=None, rarity=None,\
                 skill_1_type=None, skill_1_sin_affinity=None,\
                 skill_2_type=None, skill_2_sin_affinity=None,\
                 skill_3_type=None, skill_3_sin_affinity=None,\
                 defense_type=None, defense_sin_affinity=None):
        self.identity_id = identity_id
        self.name = name
        self.rarity = rarity
        # 선언만 하는 부분 
        self.skill_1_type = skill_1_type 
        self.skill_1_sin_affinity = skill_1_sin_affinity 
        self.skill_2_type = skill_2_type 
        self.skill_2_sin_affinity = skill_2_sin_affinity 
        self.skill_3_type = skill_3_type 
        self.skill_3_sin_affinity = skill_3_sin_affinity 
        self.defense_type = defense_type 
        self.defense_sin_affinity = defense_sin_affinity
        self.level_max = 50
        self.uptie_max = 4
        self.level_stats = []  # LevelStat 객체 리스트
        self.uptie_stats = [] # UptieStat 객체 리스트
        self.resist_info = None
        self.stagger_threshold = None
        self.skillset = None  # Skill정보를 담고 있는 skillset 객체 
        self.passive_infos = []  # 패시브 객체를 담고 있는 배열
        self.support_passive_infos = [] # 서포트 패시 객체를 담고 있는 배열
        self.panic_type = [] # 혼란상태 발생시 나타나는 효과를 담고 있는 객체 배열

    def update_name(self, name):
        self.name = name

    def update_rarity(self, rarity):
        self.rarity = rarity

    def add_level_stat(self, level, hp, defense):
        self.level_stats.append(LevelStat(level, hp, defense))

    def add_uptie_stat(self, uptie, speed):
        self.uptie_stats.append(UptieStat(uptie, speed))

    def add_resist_info(self, resist_hit, resist_penetrate, resist_slash):
        self.resist_info = ResistInfo(resist_hit, resist_penetrate, resist_slash)

    def add_stagger_threshold(self, first_stagger, second_stagger, third_stagger):
        self.stagger_threshold = StaggerThreshold(first_stagger, second_stagger, third_stagger)

    def update_details(self, page):
        # 세부 정보 업데이트 로직 (가정)
        pass
    #def init_uptie(self, page):
    #    init_uptie = 

# LevelStat 객체 정의
class LevelStat:
    def __init__(self, level, hp=None, defense=None):
        self.level = level
        self.hp = hp
        self.defense = defense
    def update_level(self, level):
        self.level = level
    def update_hp(self, hp):
        self.hp = hp
    def update_defense(self, defense):
        self.defense = defense

# UptieStat 객체 정의
class UptieStat:
    def __init__(self, uptie, speed=None):
        self.uptie = uptie
        self.speed = speed
    def update_speed(self, speed):
        self.speed = speed

# ResistInfo 객체 정의
class ResistInfo:
    def __init__(self, resist_hit=None, resist_penetrate=None, resist_slash=None):
        self.resist_hit = resist_hit
        self.resist_penetrate = resist_penetrate
        self.resist_slash = resist_slash

# ResistInfo 객체 정의
class StaggerThreshold:
    def __init__(self, first_stagger=None, second_stagger=None, third_stagger=None):
        self.first_stagger = first_stagger
        self.second_stagger = second_stagger
        self.third_stagger = third_stagger

class SkillSet:
    def __init__(self):

        self.skill_1_type = ""
        self.skill_2_type = ""
        self.skill_3_type = ""
        self.defense_skill_type = ""

        self.skill_1_sin_affinity = ""
        self.skill_2_sin_affinity = ""
        self.skill_3_sin_affinity = ""
        self.defense_skill_sin_affinity = ""

        self.skill_1_details = []
        self.skill_2_details = []
        self.skill_3_details = []
        self.defense_skill_details = []

    def add_skill(self, skill_code, skill):
        if skill_code == 1:
            self.skill_1_details.append(skill)
        elif skill_code == 2:
            self.skill_2_details.append(skill)
        elif skill_code == 3:
            self.skill_3_details.append(skill)
        elif skill_code == "defense":
            self.defense_skill_details.append(skill)

    def set_skill_type(self, skill_code, skill_type, sin_affinity):
        if skill_code == 1:
            self.skill_1_type = skill_type
            self.skill_1_sin_affinity = sin_affinity
        elif skill_code == 2:
            self.skill_2_type = skill_type
            self.skill_2_sin_affinity = sin_affinity
        elif skill_code == 3:
            self.skill_3_type = skill_type
            self.skill_3_sin_affinity = sin_affinity
        elif skill_type == "defense":
            self.defense_skill_type = skill_type
            self.defense_skill_sin_affinity = sin_affinity

class Skill:
    def __init__(self, skill_code, skill_name, coin_count, init_level, init_remain=None, akt_weight=None):
        self.skill_code = skill_code # "Skill 1"/"Skill 2"/"Skill 3"/"Defense"
        self.skill_name = skill_name # 스킬 이름
        self.coin_count = coin_count # 동전 개수
        self.init_level = init_level # 스킬이 초기 배정받은 레벨, 인격 레벨에 - 1 한 값을 더해주면 스킬의 현 level을 구할 수 있다.
        """
        남아있는 스킬 개수 사용하면, 하나씩 줄어들고 모든 스킬을 사용하면 이 값으로 돌아간다.
        스킬의 남은 숫자없는 스킬은 강화된 스킬로 한번 사용되면 다 조건을 채울 때까 사용 못한다.
        강화된 스킬을 None으로 설정하는 방식과 조건이 채워지면 1로 늘리는 방식이 존재한다.
        스킬을 사용할 수 있는 초기값이다.
        조건을 충족한 강화한 스킬을 이례적인 절차라고 생각하고 None으로 초기값을 설정했다.
        """
        self.init_remain = None 
        self.akt_weight = None #공격 가중치 값이다. 방어스킬에는 없기 때문에 None으로 설정했다.
        self.remain_count = init_remain
        """
        동기화 단계에 따른 정보를 담는다. 현재 4동기화까지 되어 있지만 차후 확장될 것을 생각하여 고정하지 않는다.
        내부에 들어있는 정보로는 uptie_level, skill_power, coin_power 속성으로 들어있다.
        전투, 턴, 스킬, 코인토스 단계로 내려오며 발생하는 효과에 관하여 저장한다.
        """
        self.uptie_infos = []

class UptieInfo:
    def __init__(self, uptie_level, skill_power, coin_power):
        self.uptie_level = uptie_level
        self.skill_power = skill_power
        self.coin_power = coin_power
        self.skill_effects = []

class Effect:
    def __init__(self, condition, description):
        self.condition = condition # 조건 텍스트 "[coin 1 on hit]"
        self.description = description # 효과 설명 텍스 "Inflict +1 Rupture Count

# 추상화된 EffectType 클래스
class EffectType:
    def __init__(self, name, description):
        self.name = name  # Effect 이름
        self.description = description  # Effect 설명

# Passive 클래스
class Passive(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)  # condition이 있는지 여부 확인

# SupportPassive 클래스
class SupportPassive(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)

# PanicType 클래스
class PanicType(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)

# Keyword 클래스
class Keyword:
    def __init__(self, keyword, explain=None, dispellable=None):
        self.keyword = keyword
        self.explain = explain
        self.dispellable = dispellable

      
class DataContainer:
    """크롤링 데이터를 관리하는 클래스"""
    def __init__(self):
        self.identities = []

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.level_max

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.uptie_max

    def update_identity_name(self, identity_id, identity_name):
        """특정 ID의 이름을 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_name(identity_name)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_identity_rarity(self, identity_id, rarity):
        """특정 ID의 희귀도를 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_rarity(rarity)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_skill(self, identity_id, skill_number, new_type):
        """
        특정 identity의 skill_type을 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_type: 새 skill_type
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_type = new_type
                elif skill_number == 2:
                    identity.skill_2_type = new_type
                elif skill_number == 3:
                    identity.skill_3_type = new_type
                elif skill_number == "defense":
                    identity.defense_type = new_type
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def update_sin_affinity(self, identity_id, skill_number, new_affinity):
        """
        특정 identity의 sin_affinity를 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_affinity: 새 sin_affinity
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_sin_affinity = new_affinity
                elif skill_number == 2:
                    identity.skill_2_sin_affinity = new_affinity
                elif skill_number == 3:
                    identity.skill_3_sin_affinity = new_affinity
                elif skill_number == "defense":
                    identity.defense_sin_affinity = new_affinity
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def add_identity(self, identity):
        """새로운 인격 데이터를 추가."""
        self.identities.append(identity)

    def add_level_stat(self, identity_id, level, hp, defense):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_level_stat(level, hp, defense)

    def add_uptie_stat(self, identity_id, uptie, speed):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_uptie_stat(uptie, speed)

    def add_resist_info(self, identity_id, resist_hit, resist_penetrate, resist_slash):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_resist_info(resist_hit, resist_penetrate, resist_slash)

    def add_stagger_threshold(self, identity_id, first_stagger, second_stagger, third_stagger):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_stagger_threshold(first_stagger, second_stagger, third_stagger)
        
    def display_identities(self):
        """Identity, ID, Name, Rarity 정보를 출력"""
        if not self.identities:
            print("No identities found.")
        else:
            # 데이터를 리스트로 변환
            table = []
            for idx, identity in enumerate(self.identities, 1):
                table.append([
                    idx, identity.identity_id, identity.name, identity.rarity,
                    identity.skill_1_type, identity.skill_1_sin_affinity,
                    identity.skill_2_type, identity.skill_2_sin_affinity,
                    identity.skill_3_type, identity.skill_3_sin_affinity,
                    identity.defense_type, identity.defense_sin_affinity
                ])
            
            # 테이블 헤더 정의
            headers = [
                "Index", "ID", "Name", "Rarity",
                "Skill_1 Type", "Skill_1 Sin", "Skill_2 Type", "Skill_2 Sin",
                "Skill_3 Type", "Skill_3 Sin", "Defense Type", "Defense Sin"
            ]
            
            # 테이블 출력
            print(tabulate(table, headers=headers, tablefmt="fancy_grid"))

    def export_to_csv(self, filename, fieldnames, data):
        """
        공통적으로 사용 가능한 CSV 내보내기 메서드
        :param filename: 내보낼 파일 이름 (예: 'identity.csv', 'skill.csv')
        :param fieldnames: CSV의 헤더 리스트
        :param data: CSV로 내보낼 데이터 (리스트 형태)
        """
        try:
            # 현재작업 디렉토 기준으 'csv'폴더 없으면 생성
            if not os.path.exists('./csv'):
                os.makedirs('./csv')
            # 파일 경로 설정 (상대경로 'csv' 폴더 안에 저장)
            file_path = os.path.join('./csv', filename)
            with open(file_path, mode='w', newline='', encoding='utf-8') as file:
                writer = csv.DictWriter(file, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(data)
            print(f"Data successfully exported to {filename}")
        except Exception as e:
            print(f"Error exporting to CSV: {e}")

    def get_identity_data(self):
        """
        Identity 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "ID": identity.identity_id,
                "Name": identity.name,
                "Rarity": identity.rarity,
                "Skill_1_Type": identity.skill_1_type,
                "Skill_1_Sin_Affinity": identity.skill_1_sin_affinity,
                "Skill_2_Type": identity.skill_2_type,
                "Skill_2_Sin_Affinity": identity.skill_2_sin_affinity,
                "Skill_3_Type": identity.skill_3_type,
                "Skill_3_Sin_Affinity": identity.skill_3_sin_affinity,
                "Defense_Type": identity.defense_type,
                "Defense_Sin_Affinity": identity.defense_sin_affinity,
            }
            for identity in self.identities
        ]
        
    def export_identities_data(self, filename="identities.csv"):
        """Identity 데이터를 CSV로 내보내는 메서드"""
        identity_fieldnames = [
            "ID", "Name", "Rarity",
            "Skill_1_Type", "Skill_1_Sin_Affinity",
            "Skill_2_Type", "Skill_2_Sin_Affinity",
            "Skill_3_Type", "Skill_3_Sin_Affinity",
            "Defense_Type", "Defense_Sin_Affinity"
        ]
        identity_data = self.get_identity_data()  # Identity 데이터 가져오기
        self.export_to_csv(filename, identity_fieldnames, identity_data)  # CSV로 내보내기

    def load_identities_by_id(self, filename, identity_ids):
        """지정된 identity_ids 목록에 해당하는 Identity 데이터를 CSV에서 읽어옴."""
        identity_ids_set = set(identity_ids) # 중복 제거를 위 Set 사용
        with open(filename, mode='r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                if row["ID"] in identity_ids_set:
                    identity = Identity(
                        identity_id=row["ID"],
                        name=row["Name"],
                        rarity=row["Rarity"],
                        skill_1_type=row["Skill_1_Type"],
                        skill_1_sin_affinity=row["Skill_1_Sin_Affinity"],
                        skill_2_type=row["Skill_2_Type"],
                        skill_2_sin_affinity=row["Skill_2_Sin_Affinity"],
                        skill_3_type=row["Skill_3_Type"],
                        skill_3_sin_affinity=row["Skill_3_Sin_Affinity"],
                        defense_type=row["Defense_Type"],
                        defense_sin_affinity=row["Defense_Sin_Affinity"]
                    )
                    self.identities.append(identity)

    def get_level_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Level": level_stat.level,
                "HP": level_stat.hp,
                "Defense": level_stat.defense,
            }
            for identity in self.identities
            for level_stat in identity.level_stats
        ]

    def export_level_stats_data(self, filename="levelstats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        level_stats_fieldnames = ["Identity_ID", "Level", "HP", "Defense"]
        level_stats_data = self.get_level_stats_data()  # level_stats 데이터 가져오기
        self.export_to_csv(filename, level_stats_fieldnames, level_stats_data)  # CSV로 내보내기

    def get_uptie_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Uptie": uptie_stat.uptie,
                "Speed": uptie_stat.speed,
            }
            for identity in self.identities
            for uptie_stat in identity.uptie_stats
        ]

    def export_uptie_stats_data(self, filename="Uptiestats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        uptie_stats_fieldnames = ["Identity_ID", "Uptie", "Speed"]
        uptie_stats_data = self.get_uptie_stats_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, uptie_stats_fieldnames, uptie_stats_data)  # CSV로 내보내기

    def get_resist_info_data(self):
        """
        모든 Identity의 resist_info 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Resist_Hit": identity.resist_info.resist_hit,
                "Resist_Penetrate": identity.resist_info.resist_penetrate,
                "Resist_Slash": identity.resist_info.resist_slash,
            }
            for identity in self.identities
        ]

    def export_resist_info_data(self, filename="ResistInfos.csv"):
        """ResistInfo 데이터를 CSV로 내보내는 메서드"""
        resist_infos_fieldnames = ["Identity_ID", "Resist_Hit", "Resist_Penetrate", "Resist_Slash"]
        resist_infos_data = self.get_resist_info_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, resist_infos_fieldnames, resist_infos_data)  # CSV로 내보내기

    def export_uptie_stats_data(self, filename="Uptiestats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        uptie_stats_fieldnames = ["Identity_ID", "Uptie", "Speed"]
        uptie_stats_data = self.get_uptie_stats_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, uptie_stats_fieldnames, uptie_stats_data)  # CSV로 내보내기

    def get_stagger_threshold_data(self):
        """
        모든 Identity의 stagger_threshold 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "First_Stagger": identity.stagger_threshold.first_stagger,
                "Second_Stagger": identity.stagger_threshold.second_stagger,
                "Third_Stagger": identity.stagger_threshold.third_stagger,
            }
            for identity in self.identities
        ]

    def export_stagger_threshold_data(self, filename="StaggerThresholds.csv"):
        """StaggerThresholds 데이터를 CSV로 내보내는 메서드"""
        stagger_thresholds_fieldnames = ["Identity_ID", "First_Stagger", "Second_Stagger", "Third_Stagger"]
        stagger_thresholds_data = self.get_stagger_threshold_data()  # stagger_thresholds 데이터 가져오기
        self.export_to_csv(filename, stagger_thresholds_fieldnames, stagger_thresholds_data)  # CSV로 내보내기


async def main():
    data_container = DataContainer()
    scraper = Scraper(container=data_container, base_url="https://limbus.kusoge.xyz", img_url="https://img.kusoge.xyz/limbus/img")
    await scraper.scrape()

await main()

In [61]:
import os
import csv
import re
import numpy as np
import pandas as pd
from playwright.async_api import async_playwright
from tabulate import tabulate

# Sin Color 이름 변환을 위한 매핑
resource_mapping = {
    "AZURE": "Gloom",
    "VIOLET": "Envy",
    "AMBER": "Sloth",
    "SHAMROCK": "Gluttony",
    "CRIMSON": "Wrath",
    "SCARLET": "Lust",
    "INDIGO": "Pride"
}

def convert_resource(resource_name):
    """Sin Color 이름을 변환하는 함수. 매핑되지 않은 경우 원래 이름을 반환."""
    return resource_mapping.get(resource_name, resource_name)

class Scraper:
    """웹 페이지 크롤링을 위한 클래스"""
    def __init__(self, container, base_url, img_url):
        self.container = container
        self.base_url = base_url
        self.img_url = img_url

    async def handle_request(self, route, request):
        """이미지와 폰트 리소스 로드를 차단."""
        if request.resource_type in ["image", "font"]:
            await route.abort()
        else:
            await route.continue_()
            
    async def scrape(self):
        """크롤링 메인 함수"""
        async with async_playwright() as p:
            browser = await p.chromium.launch(headless=True)
            page = await browser.new_page()
            await page.route("**/*", self.handle_request)
            # Step 1: /identity 페이지에서 기본 정보 크롤링
            #await self.crawl_identity_page(page)

            # Step 2: /identity/{identity_id} 상세 페이지 크롤링
            #identity_ids_to_load = ["10104", "10310", "10710","10808", "10911", "11005"]
            #identity_ids_to_load = ["10104", "10911"]#,"10310", "10710","10808", "11005", "11210"]
            identity_ids_to_load = ["10911"]#,"10310", "10710","10808", "11005", "11210"]
            self.container.load_identities_by_id('./csv/identities.csv', identity_ids_to_load)
            await self.crawl_identity_details_for_all(page)
            #self.container.display_identities()
            #self.container.export_identities_data("identities.csv")
            #self.container.export_level_stats_data("Levelstats.csv")
            #self.container.export_uptie_stats_data("Uptiestats.csv")
            #self.container.export_resist_info_data("ResistInfos.csv")
            #self.container.export_stagger_threshold_data("StaggerThresholds.csv")
            page.set_default_timeout(10000)
            #await page.wait_for_timeout(100000)
            await browser.close()

    async def get_identity_id_from_row(self, row):
        """테이블 행(row)에서 인격 ID를 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_url = await identity_cell.get_attribute('href')
            identity_id = identity_url.split('/identity/')[-1]
            return identity_id
        return "Unknown"

    async def get_identity_name_from_row(self, row):
        """테이블 행(row)에서 인격 이름을 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_name = (await identity_cell.inner_text()).replace("\n", " ").strip()
            return identity_name
        return "Unknown"
        
    async def crawl_identity_page(self, page):
        """/identity 페이지로 이동하여 인격 기본 정보를 추출."""
        await page.goto(self.base_url + "/identity")
        await self.extract_identity_page(page)

    async def extract_identity_page(self, page):
        """/identity 페이지의 모든 테이블 행에서 인격 정보를 추출."""
        identity_rows = await page.query_selector_all('tr')
        if len(identity_rows) > 1:
            for idx, identity_row in enumerate(identity_rows):
                if idx == 0:
                    continue # 0번째요소이름을 보여주는 열은 건너뜀
                await self.scrape_identity_name_and_identity_id(identity_row)
                await self.extract_rarity(identity_row)
                for i in range(1, 4):  # 스킬1~3 정보 추출
                    await self.extract_atk_and_resource(identity_row, i)
                await self.extract_defense_and_resource(identity_row)

    async def scrape_identity_name_and_identity_id(self, row):
        """인격 번호와 이름을 추출하여 DataContainer에 저장."""
        try:
            identity_id = await self.get_identity_id_from_row(row)
            identity_name = await self.get_identity_name_from_row(row)
            identity = Identity(identity_id)
            self.container.add_identity(identity)
            self.container.update_identity_name(identity_id, identity_name)
        except Exception as e:
            print(f"Error extracting identity name and identity id: {e}")

    async def get_element_attribute(self, element, selector, attribute):
        """요소에서 지정된 속성(attribute)을 추출."""
        try:
            target_element = await element.query_selector(selector)
            if target_element:
                return await target_element.get_attribute(attribute)
        except Exception as e:
            print(f"Error getting element attribute: {e}")
        return None
    
    async def extract_rarity(self, row):
        """테이블 행(row)에서 희귀도를 추출하여 업데이트."""
        try:
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
                sibling = await parent.evaluate_handle('node => node.nextElementSibling')
                if sibling:
                    rarity_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/rarity_"]', 'src')
                    if rarity_img_src:
                        rarity_level = rarity_img_src.split('_')[-1].split('.')[0]  # rarity_1.png -> 1
                        identity_id = await self.get_identity_id_from_row(row)
                        self.container.update_identity_rarity(identity_id, rarity_level)
                    else:
                        print("Rarity image not found in the next sibling TD.")
        except Exception as e: 
            print(f"Error extracting rarity: {e}")

        return "Unknown"

    async def extract_atk_and_resource(self, row, skill_number):
        """ 공격 타입(ATK Type)과 자원(Attribute) 추출 함수 - 스킬 번호에 따라 처리 """
        try:
            #print(f"Attempting to locate skill{skill_number} type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 해당 스킬 번호에 맞는 형제로 이동
                sibling = parent
                for _ in range(skill_number + 1):  # 스킬 번호에 맞게 이동
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                atk_type, resource = "Unknown", "Unknown"
        
                if sibling:
                    # 공격 타입 이미지 (atk_type) 찾기
                    atk_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/atk_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if atk_type_img_src:
                        atk_type = atk_type_img_src.split('_')[-1].split('.')[0]  # atk_type_SLASH.png -> SLASH
                        #print(f"Extracted attack type: {atk_type}")
                        self.container.update_skill(identity_id, skill_number, atk_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted resource: {resource}")
                        
                        # 리소스를 변환
                        resource = convert_resource(resource)
                        #print(f"Converted resource: {resource}")
                        self.container.update_sin_affinity(identity_id, skill_number, resource)
                        
        except Exception as e:
            print(f"Error extracting skill{skill_number} type and resource: {e}")

    async def extract_defense_and_resource(self, row):
        """ 방어 타입(Def Type)과 자원(Attribute) 추출 함수 """
        try:
            #print("Attempting to locate defense type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 다섯 번째 형제 td로 이동 (방어 관련 정보가 있는 <td>)
                sibling = parent
                for _ in range(5):  # 방어 정보는 5번째 형제에 있음
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                def_type, def_resource = "Unknown", "Unknown"
        
                if sibling:
                    # 방어 타입 이미지 (def_type) 찾기
                    def_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/def_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if def_type_img_src:
                        def_type = def_type_img_src.split('_')[-1].split('.')[0]  # def_type_GUARD.png -> GUARD
                        #print(f"Extracted defense type: {def_type}")
                        self.container.update_skill(identity_id, 'defense', def_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        def_resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted defense resource: {def_resource}")
                        
                        # 방어 리소스를 변환
                        def_resource = convert_resource(def_resource)
                        #print(f"Converted defense resource: {def_resource}")
                        self.container.update_sin_affinity(identity_id, 'defense', def_resource)
 
        except Exception as e:
            print(f"Error extracting defense type and resource: {e}")

    async def crawl_identity_details_for_all(self, page):
        """모든 인격의 세부 정보를 크롤링."""
        for identity in self.container.identities:
            await page.wait_for_load_state("networkidle")
            await page.goto(self.base_url + "/identity/"+identity.identity_id)
            """ 레벨과 동기화와 관련 없는 참관타 저항은 밑에서 변경하는 것과 연관성이 적기 때문에 위에서 처리한다.
            print(f"Identity_id: {identity.identity_id}")
            results = []
            resist_type_imgs = [f"{self.img_url}/ui/atk_type_HIT.png",
                                f"{self.img_url}/ui/atk_type_PENETRATE.png",
                                f"{self.img_url}/ui/atk_type_SLASH.png"]
            for resist_type_img in resist_type_imgs:
                result = await self.extract_resist_type(page, resist_type_img, 1)
                results.append(result)
            self.container.add_resist_info(identity.identity_id, results[0], results[1], results[2])
            """
            """
            # Stagger Threshold 값 추출
            stagger_threshold = await self.extract_stagger_threshold_values(page)
            self.container.add_stagger_threshold(identity.identity_id, stagger_threshold[0], stagger_threshold[1], stagger_threshold[2])
            """
            uptie_value = await self.initialize_uptie_and_level(page)
            """ 레벨에 따른 level, hp, defense 다르게 되어 있는 것을 출력 확인 객체로 들어가서 csv로 옮기는 과정까지 진행
            for i in range(1, identity.level_max+1):
                await self.move_slider_to_value(page,i)
                await self.extract_level_status(page, identity.identity_id, i)
            """
            """ 동기화에 따른 speed 다르게 되어 있는 것을 출력 확인 객체로 들어가서 csv로 옮기는 과정까지 진행
            for i in range(1, identity.uptie_max+1):
                await self.uptie_click(page,i)
                await self.extract_uptie_status(page, identity.identity_id, i)
            """
            await self.crawl_skills(page, uptie_value)
            
    async def initialize_uptie_and_level(self, page):
        await self.uptie_click(page, 1)
        await self.move_slider_to_value(page, 1)
        return 1
    
    async def uptie_click(self, page, phase):
        fab_button_selector = ".q-btn.q-btn--fab"
        await page.wait_for_selector(fab_button_selector)
        await page.click(fab_button_selector)
        #print(f"[{setup_uptie_button}] 상위 동기화 버튼 클릭 완료")
        setup_uptie_button = f'a:has(img[src="{self.img_url}/ui/uptie_{phase}.png"])'
        await page.wait_for_selector(setup_uptie_button)
        await page.click(setup_uptie_button)
        #print(f"{phase} 동기화 버튼 클릭 완료")

    async def move_slider_to_value(self, page, target_value):
        """
        슬라이더를 주어진 값으로 이동시킵니다.
        target_value는 슬라이더의 최소값부터 최대값 사이여야 합니다.
        """
        slider_selector = ".q-slider"
        # 슬라이더의 범위 확인
        min_value = int(await page.get_attribute(slider_selector, "aria-valuemin"))
        max_value = int(await page.get_attribute(slider_selector, "aria-valuemax"))
    
        if target_value < min_value or target_value > max_value:
            raise ValueError(f"Target value {target_value} is out of bounds ({min_value}, {max_value})")
    
        # 슬라이더 thumb 요소와 위치 가져오기
        slider_thumb = await page.query_selector(f"{slider_selector} .q-slider__thumb")
        thumb_box = await slider_thumb.bounding_box()
        
        # 슬라이더의 현재 위치 및 목표 위치 계산
        track = await page.query_selector(f"{slider_selector} .q-slider__track")
        track_box = await track.bounding_box()
        
        # 목표 x 위치 계산 (슬라이더 트랙 내에서 비율 기반)
        target_percentage = (target_value - min_value) / (max_value - min_value)
        target_x = track_box["x"] + target_percentage * track_box["width"]
        current_x = thumb_box["x"] + thumb_box["width"] / 2
        y = thumb_box["y"] + thumb_box["height"] / 2
    
        # 슬라이더를 드래그하여 목표 위치로 이동
        await page.mouse.move(current_x, y)
        await page.mouse.down()
        await page.mouse.move(target_x, y, steps=10)  # 부드럽게 이동
        await page.mouse.up()
    
        #print(f"슬라이더를 {target_value}로 이동 완료.")
    
        # 슬라이더 값 확인 및 강제 이벤트 트리거
        await page.evaluate(
            f"""
            (slider) => {{
                slider.dispatchEvent(new Event('input'));  // oninput 이벤트 트리거
                slider.dispatchEvent(new Event('change'));  // onchange 이벤트 트리거
            }}
            """,
            await page.query_selector(slider_selector)
        )
    
        # 이동 후 값 확인
        current_value = await page.get_attribute(slider_selector, "aria-valuenow")
        if current_value != str(target_value):
            print(f"경고: 슬라이더 값이 예상과 다릅니다. 현재 값: {current_value}, 예상 값: {target_value}")

    async def get_second_status_element(self, page):
        """
        두 번째 'Status' 요소에 접근하여 반환하는 메소드
        """
        status_elements = await page.query_selector_all("text=Status")
        if len(status_elements) > 1:
            second_status_element = status_elements[1]
            #print("두 번째 'Status' 요소에 접근합니다.")
            return second_status_element
        else:
            #print("두 번째 'Status' 요소가 존재하지 않습니다.")
            return None

    async def extract_level_status(self, page, identity_id, level):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 레벨에 따라 HP와 Defense가 변한다.
        hp_value = await self.extract_hp(second_status_element)
        defense_value = await self.extract_defense(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Level={level}, HP={hp_value}, Defense={defense_value}")
        self.container.add_level_stat(identity_id,level, hp_value, defense_value)
    
    async def extract_hp(self, status_element):
        # HP 추출
        hp_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].children[0].nextElementSibling.textContent.trim()"
        )
        return hp_value
    
    async def extract_defense(self, status_element):
        # Defense 추출
        defense_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return defense_value

    async def extract_uptie_status(self, page, identity_id, uptie):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 uptie에 따라 speed는 변한다.
        speed_value = await self.extract_speed(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Uptie={uptie}, Speed={speed_value}")
        self.container.add_uptie_stat(identity_id,uptie, speed_value)

    async def extract_speed(self, status_element):
        # Speed 추출
        speed_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return speed_value

    async def extract_resist_type(self, page, image_src, depth):
        # 이미지 요소 찾기
        #print(image_src)
        image = await page.query_selector(f'img[src="{image_src}"]')
        #print(image)
        if not image:
            return "Unknown"
    
        # 이미지의 부모 요소를 찾아서 올라감
        parent = image
        for _ in range(depth):
            parent = await parent.evaluate_handle('node => node.parentElement')
            if not parent:
                return "Unknown"
    
        # 부모의 형제 요소 찾기
        sibling = await parent.evaluate_handle('node => node.nextElementSibling')
        
        if sibling:
            value = await sibling.evaluate('node => node.textContent')
            if value:
                return value.strip()
        
        return "Unknown"

    async def extract_stagger_threshold_values(self, page):
        # 'div.row.center.flex-center.q-pt-xs' 요소들 찾기
        elements = await page.query_selector_all('div.row.center.flex-center.q-pt-xs')
        
        # 각 요소의 텍스트 내용 출력하여 'Stagger Threshold' 요소를 찾기
        for element in elements:
            text = await element.text_content()
            
            if text and "Stagger Threshold" in text:
                # 'Stagger Threshold'가 있는 요소를 찾았다면, 그 이후 형제 요소들에서 값을 추출
                parent = await element.evaluate_handle('node => node.parentElement')
                sibling_elements = await parent.query_selector_all('div.row.justify-evenly div')  # Threshold 값이 들어있는 div들
                
                stagger_values = []
                for sibling in sibling_elements:
                    text_content = await sibling.text_content()
                    match = re.search(r'(\d+%)', text_content)
                    if match:
                        stagger_values.append(match.group(1))  # 퍼센트 값만 리스트에 추가
                while len(stagger_values) < 3:
                    stagger_values.append(np.nan)
                #print(stagger_values)
                return stagger_values  # 추출된 값(또는 np.nan 포함)을 반환
        
        return [np.nan, np.nan, np.nan]  # 만약 'Stagger Threshold'를 찾지 못한 경우 기본값 반환

    async def crawl_skills(self, page, uptie_value):
        await self.crawl_skill(page, "skill 1")
        await self.crawl_skill(page, "skill 2")
        #if uptie_value >= 3: # 나중에 스킬 관련된 정보를 전부 긁어오는 것이 확인되면 적용한다. 동기화 3 이상만 스킬3 정보를 긁어온다.
        await self.crawl_skill(page, "skill 3")
        await self.crawl_skill(page, "defense")

    async def crawl_skill(self, page, skill_type):
        skills_elements = await page.query_selector_all('div.skilltag')
        for skill_element in skills_elements:
            skill_text = await skill_element.text_content()
            if skill_text.strip().lower() == skill_type:
                # evaluate로 조부모의 이전 형제 탐색 후 텍스트 가져오기
                left_info = await skill_element.evaluate(
                    """
                    node => {
                        // 조부모로 이동
                        const grandparent = node.parentElement.parentElement;
                
                        // 이전 형제 탐색
                        const previousSibling = grandparent.previousElementSibling;
                        if (!previousSibling) return null; // 이전 형제가 없으면 null 반환
                        // 이전 형제 내부의 .row.justify-around > div > div 탐색
                        const targetDivs = previousSibling.querySelectorAll(".row.justify-around > div");
                        // 1st div: 스킬 위력
                        const skillPower = targetDivs[0] ? targetDivs[0].querySelector('div')?.textContent.trim() : null; 
                        // 3rd div: 코인 파워 값 (3번째 div에서 텍스트 추출)
                        const coinPower = targetDivs[2] ? targetDivs[2].textContent.trim() : null;
                        return { skillPower, coinPower };
                    }
                    """
                )
                print(f"{skill_type}: skillPower:{left_info.get('skillPower')} coinPower:{left_info.get('coinPower')}")

class Identity:
    """인격 정보를 저장하는 클래스"""
    def __init__(self, identity_id, name=None, rarity=None,\
                 skill_1_type=None, skill_1_sin_affinity=None,\
                 skill_2_type=None, skill_2_sin_affinity=None,\
                 skill_3_type=None, skill_3_sin_affinity=None,\
                 defense_type=None, defense_sin_affinity=None):
        self.identity_id = identity_id
        self.name = name
        self.rarity = rarity
        # 선언만 하는 부분 
        self.skill_1_type = skill_1_type 
        self.skill_1_sin_affinity = skill_1_sin_affinity 
        self.skill_2_type = skill_2_type 
        self.skill_2_sin_affinity = skill_2_sin_affinity 
        self.skill_3_type = skill_3_type 
        self.skill_3_sin_affinity = skill_3_sin_affinity 
        self.defense_type = defense_type 
        self.defense_sin_affinity = defense_sin_affinity
        self.level_max = 50
        self.uptie_max = 4
        self.level_stats = []  # LevelStat 객체 리스트
        self.uptie_stats = [] # UptieStat 객체 리스트
        self.resist_info = None
        self.stagger_threshold = None
        self.skillset = SkillSet()  # Skill정보를 담고 있는 skillset 객체 
        self.passive_infos = []  # 패시브 객체를 담고 있는 배열
        self.support_passive_infos = [] # 서포트 패시 객체를 담고 있는 배열
        self.panic_type = [] # 혼란상태 발생시 나타나는 효과를 담고 있는 객체 배열

    def update_name(self, name):
        self.name = name

    def update_rarity(self, rarity):
        self.rarity = rarity

    def add_level_stat(self, level, hp, defense):
        self.level_stats.append(LevelStat(level, hp, defense))

    def add_uptie_stat(self, uptie, speed):
        self.uptie_stats.append(UptieStat(uptie, speed))

    def add_resist_info(self, resist_hit, resist_penetrate, resist_slash):
        self.resist_info = ResistInfo(resist_hit, resist_penetrate, resist_slash)

    def add_stagger_threshold(self, first_stagger, second_stagger, third_stagger):
        self.stagger_threshold = StaggerThreshold(first_stagger, second_stagger, third_stagger)

    def update_details(self, page):
        # 세부 정보 업데이트 로직 (가정)
        pass
    #def init_uptie(self, page):
    #    init_uptie = 

# LevelStat 객체 정의
class LevelStat:
    def __init__(self, level, hp=None, defense=None):
        self.level = level
        self.hp = hp
        self.defense = defense
    def update_level(self, level):
        self.level = level
    def update_hp(self, hp):
        self.hp = hp
    def update_defense(self, defense):
        self.defense = defense

# UptieStat 객체 정의
class UptieStat:
    def __init__(self, uptie, speed=None):
        self.uptie = uptie
        self.speed = speed
    def update_speed(self, speed):
        self.speed = speed

# ResistInfo 객체 정의
class ResistInfo:
    def __init__(self, resist_hit=None, resist_penetrate=None, resist_slash=None):
        self.resist_hit = resist_hit
        self.resist_penetrate = resist_penetrate
        self.resist_slash = resist_slash

# ResistInfo 객체 정의
class StaggerThreshold:
    def __init__(self, first_stagger=None, second_stagger=None, third_stagger=None):
        self.first_stagger = first_stagger
        self.second_stagger = second_stagger
        self.third_stagger = third_stagger

class SkillSet:
    def __init__(self):

        self.skill_1_type = ""
        self.skill_2_type = ""
        self.skill_3_type = ""
        self.defense_skill_type = ""

        self.skill_1_sin_affinity = ""
        self.skill_2_sin_affinity = ""
        self.skill_3_sin_affinity = ""
        self.defense_skill_sin_affinity = ""

        self.skill_1_details = []
        self.skill_2_details = []
        self.skill_3_details = []
        self.defense_skill_details = []

    def add_skill(self, skill_code, skill):
        if skill_code == 1:
            self.skill_1_details.append(skill)
        elif skill_code == 2:
            self.skill_2_details.append(skill)
        elif skill_code == 3:
            self.skill_3_details.append(skill)
        elif skill_code == "defense":
            self.defense_skill_details.append(skill)

    def set_skill_type(self, skill_code, skill_type, sin_affinity):
        if skill_code == 1:
            self.skill_1_type = skill_type
            self.skill_1_sin_affinity = sin_affinity
        elif skill_code == 2:
            self.skill_2_type = skill_type
            self.skill_2_sin_affinity = sin_affinity
        elif skill_code == 3:
            self.skill_3_type = skill_type
            self.skill_3_sin_affinity = sin_affinity
        elif skill_type == "defense":
            self.defense_skill_type = skill_type
            self.defense_skill_sin_affinity = sin_affinity

class Skill:
    def __init__(self, skill_code, skill_name, coin_count, init_level, init_remain=None, akt_weight=None, condition=None):
        self.skill_code = skill_code # "Skill 1"/"Skill 2"/"Skill 3"/"Defense" '''o'''
        self.skill_name = skill_name # 스킬 이름
        self.coin_count = coin_count # 동전 개수
        self.init_level = init_level # 스킬이 초기 배정받은 레벨, 인격 레벨에 - 1 한 값을 더해주면 스킬의 현 level을 구할 수 있다.
        """
        남아있는 스킬 개수 사용하면, 하나씩 줄어들고 모든 스킬을 사용하면 이 값으로 돌아간다.
        스킬의 남은 숫자없는 스킬은 강화된 스킬로 한번 사용되면 다 조건을 채울 때까 사용 못한다.
        강화된 스킬을 None으로 설정하는 방식과 조건이 채워지면 1로 늘리는 방식이 존재한다.
        스킬을 사용할 수 있는 초기값이다.
        조건을 충족한 강화한 스킬을 이례적인 절차라고 생각하고 None으로 초기값을 설정했다.
        """
        self.init_remain = None 
        self.akt_weight = None #공격 가중치 값이다. 방어스킬에는 없기 때문에 None으로 설정했다.
        self.remain_count = init_remain
        """
        동기화 단계에 따른 정보를 담는다. 현재 4동기화까지 되어 있지만 차후 확장될 것을 생각하여 고정하지 않는다.
        내부에 들어있는 정보로는 uptie_level, skill_power, coin_power 속성으로 들어있다.
        전투, 턴, 스킬, 코인토스 단계로 내려오며 발생하는 효과에 관하여 저장한다.
        """
        self.uptie_infos = []  

class UptieInfo:
    def __init__(self, uptie_level, skill_power, coin_power):
        self.uptie_level = uptie_level   #'''o'''
        self.skill_power = skill_power   #'''o'''
        self.coin_power = coin_power     #'''o'''
        self.skill_effects = []

class Effect:
    def __init__(self, condition, description):
        self.condition = condition # 조건 텍스트 "[coin 1 on hit]"
        self.description = description # 효과 설명 텍스 "Inflict +1 Rupture Count

# 추상화된 EffectType 클래스
class EffectType:
    def __init__(self, name, description):
        self.name = name  # Effect 이름
        self.description = description  # Effect 설명

# Passive 클래스
class Passive(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)  # condition이 있는지 여부 확인

# SupportPassive 클래스
class SupportPassive(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)

# PanicType 클래스
class PanicType(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)

# Keyword 클래스
class Keyword:
    def __init__(self, keyword, explain=None, dispellable=None):
        self.keyword = keyword
        self.explain = explain
        self.dispellable = dispellable

      
class DataContainer:
    """크롤링 데이터를 관리하는 클래스"""
    def __init__(self):
        self.identities = []

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.level_max

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.uptie_max

    def update_identity_name(self, identity_id, identity_name):
        """특정 ID의 이름을 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_name(identity_name)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_identity_rarity(self, identity_id, rarity):
        """특정 ID의 희귀도를 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_rarity(rarity)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_skill(self, identity_id, skill_number, new_type):
        """
        특정 identity의 skill_type을 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_type: 새 skill_type
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_type = new_type
                elif skill_number == 2:
                    identity.skill_2_type = new_type
                elif skill_number == 3:
                    identity.skill_3_type = new_type
                elif skill_number == "defense":
                    identity.defense_type = new_type
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def update_sin_affinity(self, identity_id, skill_number, new_affinity):
        """
        특정 identity의 sin_affinity를 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_affinity: 새 sin_affinity
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_sin_affinity = new_affinity
                elif skill_number == 2:
                    identity.skill_2_sin_affinity = new_affinity
                elif skill_number == 3:
                    identity.skill_3_sin_affinity = new_affinity
                elif skill_number == "defense":
                    identity.defense_sin_affinity = new_affinity
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def add_identity(self, identity):
        """새로운 인격 데이터를 추가."""
        self.identities.append(identity)

    def add_level_stat(self, identity_id, level, hp, defense):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_level_stat(level, hp, defense)

    def add_uptie_stat(self, identity_id, uptie, speed):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_uptie_stat(uptie, speed)

    def add_resist_info(self, identity_id, resist_hit, resist_penetrate, resist_slash):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_resist_info(resist_hit, resist_penetrate, resist_slash)

    def add_stagger_threshold(self, identity_id, first_stagger, second_stagger, third_stagger):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_stagger_threshold(first_stagger, second_stagger, third_stagger)
        
    def display_identities(self):
        """Identity, ID, Name, Rarity 정보를 출력"""
        if not self.identities:
            print("No identities found.")
        else:
            # 데이터를 리스트로 변환
            table = []
            for idx, identity in enumerate(self.identities, 1):
                table.append([
                    idx, identity.identity_id, identity.name, identity.rarity,
                    identity.skill_1_type, identity.skill_1_sin_affinity,
                    identity.skill_2_type, identity.skill_2_sin_affinity,
                    identity.skill_3_type, identity.skill_3_sin_affinity,
                    identity.defense_type, identity.defense_sin_affinity
                ])
            
            # 테이블 헤더 정의
            headers = [
                "Index", "ID", "Name", "Rarity",
                "Skill_1 Type", "Skill_1 Sin", "Skill_2 Type", "Skill_2 Sin",
                "Skill_3 Type", "Skill_3 Sin", "Defense Type", "Defense Sin"
            ]
            
            # 테이블 출력
            print(tabulate(table, headers=headers, tablefmt="fancy_grid"))

    def export_to_csv(self, filename, fieldnames, data):
        """
        공통적으로 사용 가능한 CSV 내보내기 메서드
        :param filename: 내보낼 파일 이름 (예: 'identity.csv', 'skill.csv')
        :param fieldnames: CSV의 헤더 리스트
        :param data: CSV로 내보낼 데이터 (리스트 형태)
        """
        try:
            # 현재작업 디렉토 기준으 'csv'폴더 없으면 생성
            if not os.path.exists('./csv'):
                os.makedirs('./csv')
            # 파일 경로 설정 (상대경로 'csv' 폴더 안에 저장)
            file_path = os.path.join('./csv', filename)
            with open(file_path, mode='w', newline='', encoding='utf-8') as file:
                writer = csv.DictWriter(file, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(data)
            print(f"Data successfully exported to {filename}")
        except Exception as e:
            print(f"Error exporting to CSV: {e}")

    def get_identity_data(self):
        """
        Identity 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "ID": identity.identity_id,
                "Name": identity.name,
                "Rarity": identity.rarity,
                "Skill_1_Type": identity.skill_1_type,
                "Skill_1_Sin_Affinity": identity.skill_1_sin_affinity,
                "Skill_2_Type": identity.skill_2_type,
                "Skill_2_Sin_Affinity": identity.skill_2_sin_affinity,
                "Skill_3_Type": identity.skill_3_type,
                "Skill_3_Sin_Affinity": identity.skill_3_sin_affinity,
                "Defense_Type": identity.defense_type,
                "Defense_Sin_Affinity": identity.defense_sin_affinity,
            }
            for identity in self.identities
        ]
        
    def export_identities_data(self, filename="identities.csv"):
        """Identity 데이터를 CSV로 내보내는 메서드"""
        identity_fieldnames = [
            "ID", "Name", "Rarity",
            "Skill_1_Type", "Skill_1_Sin_Affinity",
            "Skill_2_Type", "Skill_2_Sin_Affinity",
            "Skill_3_Type", "Skill_3_Sin_Affinity",
            "Defense_Type", "Defense_Sin_Affinity"
        ]
        identity_data = self.get_identity_data()  # Identity 데이터 가져오기
        self.export_to_csv(filename, identity_fieldnames, identity_data)  # CSV로 내보내기

    def load_identities_by_id(self, filename, identity_ids):
        """지정된 identity_ids 목록에 해당하는 Identity 데이터를 CSV에서 읽어옴."""
        identity_ids_set = set(identity_ids) # 중복 제거를 위 Set 사용
        with open(filename, mode='r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                if row["ID"] in identity_ids_set:
                    identity = Identity(
                        identity_id=row["ID"],
                        name=row["Name"],
                        rarity=row["Rarity"],
                        skill_1_type=row["Skill_1_Type"],
                        skill_1_sin_affinity=row["Skill_1_Sin_Affinity"],
                        skill_2_type=row["Skill_2_Type"],
                        skill_2_sin_affinity=row["Skill_2_Sin_Affinity"],
                        skill_3_type=row["Skill_3_Type"],
                        skill_3_sin_affinity=row["Skill_3_Sin_Affinity"],
                        defense_type=row["Defense_Type"],
                        defense_sin_affinity=row["Defense_Sin_Affinity"]
                    )
                    self.identities.append(identity)

    def get_level_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Level": level_stat.level,
                "HP": level_stat.hp,
                "Defense": level_stat.defense,
            }
            for identity in self.identities
            for level_stat in identity.level_stats
        ]

    def export_level_stats_data(self, filename="levelstats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        level_stats_fieldnames = ["Identity_ID", "Level", "HP", "Defense"]
        level_stats_data = self.get_level_stats_data()  # level_stats 데이터 가져오기
        self.export_to_csv(filename, level_stats_fieldnames, level_stats_data)  # CSV로 내보내기

    def get_uptie_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Uptie": uptie_stat.uptie,
                "Speed": uptie_stat.speed,
            }
            for identity in self.identities
            for uptie_stat in identity.uptie_stats
        ]

    def export_uptie_stats_data(self, filename="Uptiestats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        uptie_stats_fieldnames = ["Identity_ID", "Uptie", "Speed"]
        uptie_stats_data = self.get_uptie_stats_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, uptie_stats_fieldnames, uptie_stats_data)  # CSV로 내보내기

    def get_resist_info_data(self):
        """
        모든 Identity의 resist_info 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Resist_Hit": identity.resist_info.resist_hit,
                "Resist_Penetrate": identity.resist_info.resist_penetrate,
                "Resist_Slash": identity.resist_info.resist_slash,
            }
            for identity in self.identities
        ]

    def export_resist_info_data(self, filename="ResistInfos.csv"):
        """ResistInfo 데이터를 CSV로 내보내는 메서드"""
        resist_infos_fieldnames = ["Identity_ID", "Resist_Hit", "Resist_Penetrate", "Resist_Slash"]
        resist_infos_data = self.get_resist_info_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, resist_infos_fieldnames, resist_infos_data)  # CSV로 내보내기

    def export_uptie_stats_data(self, filename="Uptiestats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        uptie_stats_fieldnames = ["Identity_ID", "Uptie", "Speed"]
        uptie_stats_data = self.get_uptie_stats_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, uptie_stats_fieldnames, uptie_stats_data)  # CSV로 내보내기

    def get_stagger_threshold_data(self):
        """
        모든 Identity의 stagger_threshold 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "First_Stagger": identity.stagger_threshold.first_stagger,
                "Second_Stagger": identity.stagger_threshold.second_stagger,
                "Third_Stagger": identity.stagger_threshold.third_stagger,
            }
            for identity in self.identities
        ]

    def export_stagger_threshold_data(self, filename="StaggerThresholds.csv"):
        """StaggerThresholds 데이터를 CSV로 내보내는 메서드"""
        stagger_thresholds_fieldnames = ["Identity_ID", "First_Stagger", "Second_Stagger", "Third_Stagger"]
        stagger_thresholds_data = self.get_stagger_threshold_data()  # stagger_thresholds 데이터 가져오기
        self.export_to_csv(filename, stagger_thresholds_fieldnames, stagger_thresholds_data)  # CSV로 내보내기


async def main():
    data_container = DataContainer()
    scraper = Scraper(container=data_container, base_url="https://limbus.kusoge.xyz", img_url="https://img.kusoge.xyz/limbus/img")
    await scraper.scrape()

await main() 

skill 1: skillPower:2 coinPower:+4
skill 2: skillPower:3 coinPower:+4
skill 3: skillPower:3 coinPower:+4
skill 3: skillPower:3 coinPower:+5
defense: skillPower:4 coinPower:+4


In [63]:
import os
import csv
import re
import numpy as np
import pandas as pd
from playwright.async_api import async_playwright
from tabulate import tabulate

# Sin Color 이름 변환을 위한 매핑
resource_mapping = {
    "AZURE": "Gloom",
    "VIOLET": "Envy",
    "AMBER": "Sloth",
    "SHAMROCK": "Gluttony",
    "CRIMSON": "Wrath",
    "SCARLET": "Lust",
    "INDIGO": "Pride"
}

def convert_resource(resource_name):
    """Sin Color 이름을 변환하는 함수. 매핑되지 않은 경우 원래 이름을 반환."""
    return resource_mapping.get(resource_name, resource_name)

class Scraper:
    """웹 페이지 크롤링을 위한 클래스"""
    def __init__(self, container, base_url, img_url):
        self.container = container
        self.base_url = base_url
        self.img_url = img_url

    async def handle_request(self, route, request):
        """이미지와 폰트 리소스 로드를 차단."""
        if request.resource_type in ["image", "font"]:
            await route.abort()
        else:
            await route.continue_()
            
    async def scrape(self):
        """크롤링 메인 함수"""
        async with async_playwright() as p:
            browser = await p.chromium.launch(headless=False)
            page = await browser.new_page()
            await page.route("**/*", self.handle_request)
            # Step 1: /identity 페이지에서 기본 정보 크롤링
            #await self.crawl_identity_page(page)

            # Step 2: /identity/{identity_id} 상세 페이지 크롤링
            #identity_ids_to_load = ["10104", "10310", "10710","10808", "10911", "11005"]
            #identity_ids_to_load = ["10104", "10911"]#,"10310", "10710","10808", "11005", "11210"]
            identity_ids_to_load = ["10911"]#,"10310", "10710","10808", "11005", "11210"]
            self.container.load_identities_by_id('./csv/identities.csv', identity_ids_to_load)
            await self.crawl_identity_details_for_all(page)
            #self.container.display_identities()
            #self.container.export_identities_data("identities.csv")
            #self.container.export_level_stats_data("Levelstats.csv")
            #self.container.export_uptie_stats_data("Uptiestats.csv")
            #self.container.export_resist_info_data("ResistInfos.csv")
            #self.container.export_stagger_threshold_data("StaggerThresholds.csv")
            #page.set_default_timeout(10000)
            await page.wait_for_timeout(100000)
            await browser.close()

    async def get_identity_id_from_row(self, row):
        """테이블 행(row)에서 인격 ID를 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_url = await identity_cell.get_attribute('href')
            identity_id = identity_url.split('/identity/')[-1]
            return identity_id
        return "Unknown"

    async def get_identity_name_from_row(self, row):
        """테이블 행(row)에서 인격 이름을 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_name = (await identity_cell.inner_text()).replace("\n", " ").strip()
            return identity_name
        return "Unknown"
        
    async def crawl_identity_page(self, page):
        """/identity 페이지로 이동하여 인격 기본 정보를 추출."""
        await page.goto(self.base_url + "/identity")
        await self.extract_identity_page(page)

    async def extract_identity_page(self, page):
        """/identity 페이지의 모든 테이블 행에서 인격 정보를 추출."""
        identity_rows = await page.query_selector_all('tr')
        if len(identity_rows) > 1:
            for idx, identity_row in enumerate(identity_rows):
                if idx == 0:
                    continue # 0번째요소이름을 보여주는 열은 건너뜀
                await self.scrape_identity_name_and_identity_id(identity_row)
                await self.extract_rarity(identity_row)
                for i in range(1, 4):  # 스킬1~3 정보 추출
                    await self.extract_atk_and_resource(identity_row, i)
                await self.extract_defense_and_resource(identity_row)

    async def scrape_identity_name_and_identity_id(self, row):
        """인격 번호와 이름을 추출하여 DataContainer에 저장."""
        try:
            identity_id = await self.get_identity_id_from_row(row)
            identity_name = await self.get_identity_name_from_row(row)
            identity = Identity(identity_id)
            self.container.add_identity(identity)
            self.container.update_identity_name(identity_id, identity_name)
        except Exception as e:
            print(f"Error extracting identity name and identity id: {e}")

    async def get_element_attribute(self, element, selector, attribute):
        """요소에서 지정된 속성(attribute)을 추출."""
        try:
            target_element = await element.query_selector(selector)
            if target_element:
                return await target_element.get_attribute(attribute)
        except Exception as e:
            print(f"Error getting element attribute: {e}")
        return None
    
    async def extract_rarity(self, row):
        """테이블 행(row)에서 희귀도를 추출하여 업데이트."""
        try:
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
                sibling = await parent.evaluate_handle('node => node.nextElementSibling')
                if sibling:
                    rarity_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/rarity_"]', 'src')
                    if rarity_img_src:
                        rarity_level = rarity_img_src.split('_')[-1].split('.')[0]  # rarity_1.png -> 1
                        identity_id = await self.get_identity_id_from_row(row)
                        self.container.update_identity_rarity(identity_id, rarity_level)
                    else:
                        print("Rarity image not found in the next sibling TD.")
        except Exception as e: 
            print(f"Error extracting rarity: {e}")

        return "Unknown"

    async def extract_atk_and_resource(self, row, skill_number):
        """ 공격 타입(ATK Type)과 자원(Attribute) 추출 함수 - 스킬 번호에 따라 처리 """
        try:
            #print(f"Attempting to locate skill{skill_number} type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 해당 스킬 번호에 맞는 형제로 이동
                sibling = parent
                for _ in range(skill_number + 1):  # 스킬 번호에 맞게 이동
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                atk_type, resource = "Unknown", "Unknown"
        
                if sibling:
                    # 공격 타입 이미지 (atk_type) 찾기
                    atk_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/atk_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if atk_type_img_src:
                        atk_type = atk_type_img_src.split('_')[-1].split('.')[0]  # atk_type_SLASH.png -> SLASH
                        #print(f"Extracted attack type: {atk_type}")
                        self.container.update_skill(identity_id, skill_number, atk_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted resource: {resource}")
                        
                        # 리소스를 변환
                        resource = convert_resource(resource)
                        #print(f"Converted resource: {resource}")
                        self.container.update_sin_affinity(identity_id, skill_number, resource)
                        
        except Exception as e:
            print(f"Error extracting skill{skill_number} type and resource: {e}")

    async def extract_defense_and_resource(self, row):
        """ 방어 타입(Def Type)과 자원(Attribute) 추출 함수 """
        try:
            #print("Attempting to locate defense type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 다섯 번째 형제 td로 이동 (방어 관련 정보가 있는 <td>)
                sibling = parent
                for _ in range(5):  # 방어 정보는 5번째 형제에 있음
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                def_type, def_resource = "Unknown", "Unknown"
        
                if sibling:
                    # 방어 타입 이미지 (def_type) 찾기
                    def_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/def_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if def_type_img_src:
                        def_type = def_type_img_src.split('_')[-1].split('.')[0]  # def_type_GUARD.png -> GUARD
                        #print(f"Extracted defense type: {def_type}")
                        self.container.update_skill(identity_id, 'defense', def_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        def_resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted defense resource: {def_resource}")
                        
                        # 방어 리소스를 변환
                        def_resource = convert_resource(def_resource)
                        #print(f"Converted defense resource: {def_resource}")
                        self.container.update_sin_affinity(identity_id, 'defense', def_resource)
 
        except Exception as e:
            print(f"Error extracting defense type and resource: {e}")

    async def crawl_identity_details_for_all(self, page):
        """모든 인격의 세부 정보를 크롤링."""
        for identity in self.container.identities:
            await page.wait_for_load_state("networkidle")
            await page.goto(self.base_url + "/identity/"+identity.identity_id)
            """ 레벨과 동기화와 관련 없는 참관타 저항은 밑에서 변경하는 것과 연관성이 적기 때문에 위에서 처리한다.
            print(f"Identity_id: {identity.identity_id}")
            results = []
            resist_type_imgs = [f"{self.img_url}/ui/atk_type_HIT.png",
                                f"{self.img_url}/ui/atk_type_PENETRATE.png",
                                f"{self.img_url}/ui/atk_type_SLASH.png"]
            for resist_type_img in resist_type_imgs:
                result = await self.extract_resist_type(page, resist_type_img, 1)
                results.append(result)
            self.container.add_resist_info(identity.identity_id, results[0], results[1], results[2])
            """
            """
            # Stagger Threshold 값 추출
            stagger_threshold = await self.extract_stagger_threshold_values(page)
            self.container.add_stagger_threshold(identity.identity_id, stagger_threshold[0], stagger_threshold[1], stagger_threshold[2])
            """
            uptie_value = await self.initialize_uptie_and_level(page)
            """ 레벨에 따른 level, hp, defense 다르게 되어 있는 것을 출력 확인 객체로 들어가서 csv로 옮기는 과정까지 진행
            for i in range(1, identity.level_max+1):
                await self.move_slider_to_value(page,i)
                await self.extract_level_status(page, identity.identity_id, i)
            """
            """ 동기화에 따른 speed 다르게 되어 있는 것을 출력 확인 객체로 들어가서 csv로 옮기는 과정까지 진행
            for i in range(1, identity.uptie_max+1):
                await self.uptie_click(page,i)
                await self.extract_uptie_status(page, identity.identity_id, i)
            """
            await self.crawl_skills(page, uptie_value)
            
    async def initialize_uptie_and_level(self, page):
        await self.uptie_click(page, 1)
        await self.move_slider_to_value(page, 1)
        return 1
    
    async def uptie_click(self, page, phase):
        fab_button_selector = ".q-btn.q-btn--fab"
        await page.wait_for_selector(fab_button_selector)
        await page.click(fab_button_selector)
        #print(f"[{setup_uptie_button}] 상위 동기화 버튼 클릭 완료")
        setup_uptie_button = f'a:has(img[src="{self.img_url}/ui/uptie_{phase}.png"])'
        await page.wait_for_selector(setup_uptie_button)
        await page.click(setup_uptie_button)
        #print(f"{phase} 동기화 버튼 클릭 완료")

    async def move_slider_to_value(self, page, target_value):
        """
        슬라이더를 주어진 값으로 이동시킵니다.
        target_value는 슬라이더의 최소값부터 최대값 사이여야 합니다.
        """
        slider_selector = ".q-slider"
        # 슬라이더의 범위 확인
        min_value = int(await page.get_attribute(slider_selector, "aria-valuemin"))
        max_value = int(await page.get_attribute(slider_selector, "aria-valuemax"))
    
        if target_value < min_value or target_value > max_value:
            raise ValueError(f"Target value {target_value} is out of bounds ({min_value}, {max_value})")
    
        # 슬라이더 thumb 요소와 위치 가져오기
        slider_thumb = await page.query_selector(f"{slider_selector} .q-slider__thumb")
        thumb_box = await slider_thumb.bounding_box()
        
        # 슬라이더의 현재 위치 및 목표 위치 계산
        track = await page.query_selector(f"{slider_selector} .q-slider__track")
        track_box = await track.bounding_box()
        
        # 목표 x 위치 계산 (슬라이더 트랙 내에서 비율 기반)
        target_percentage = (target_value - min_value) / (max_value - min_value)
        target_x = track_box["x"] + target_percentage * track_box["width"]
        current_x = thumb_box["x"] + thumb_box["width"] / 2
        y = thumb_box["y"] + thumb_box["height"] / 2
    
        # 슬라이더를 드래그하여 목표 위치로 이동
        await page.mouse.move(current_x, y)
        await page.mouse.down()
        await page.mouse.move(target_x, y, steps=10)  # 부드럽게 이동
        await page.mouse.up()
    
        #print(f"슬라이더를 {target_value}로 이동 완료.")
    
        # 슬라이더 값 확인 및 강제 이벤트 트리거
        await page.evaluate(
            f"""
            (slider) => {{
                slider.dispatchEvent(new Event('input'));  // oninput 이벤트 트리거
                slider.dispatchEvent(new Event('change'));  // onchange 이벤트 트리거
            }}
            """,
            await page.query_selector(slider_selector)
        )
    
        # 이동 후 값 확인
        current_value = await page.get_attribute(slider_selector, "aria-valuenow")
        if current_value != str(target_value):
            print(f"경고: 슬라이더 값이 예상과 다릅니다. 현재 값: {current_value}, 예상 값: {target_value}")

    async def get_second_status_element(self, page):
        """
        두 번째 'Status' 요소에 접근하여 반환하는 메소드
        """
        status_elements = await page.query_selector_all("text=Status")
        if len(status_elements) > 1:
            second_status_element = status_elements[1]
            #print("두 번째 'Status' 요소에 접근합니다.")
            return second_status_element
        else:
            #print("두 번째 'Status' 요소가 존재하지 않습니다.")
            return None

    async def extract_level_status(self, page, identity_id, level):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 레벨에 따라 HP와 Defense가 변한다.
        hp_value = await self.extract_hp(second_status_element)
        defense_value = await self.extract_defense(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Level={level}, HP={hp_value}, Defense={defense_value}")
        self.container.add_level_stat(identity_id,level, hp_value, defense_value)
    
    async def extract_hp(self, status_element):
        # HP 추출
        hp_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].children[0].nextElementSibling.textContent.trim()"
        )
        return hp_value
    
    async def extract_defense(self, status_element):
        # Defense 추출
        defense_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return defense_value

    async def extract_uptie_status(self, page, identity_id, uptie):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 uptie에 따라 speed는 변한다.
        speed_value = await self.extract_speed(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Uptie={uptie}, Speed={speed_value}")
        self.container.add_uptie_stat(identity_id,uptie, speed_value)

    async def extract_speed(self, status_element):
        # Speed 추출
        speed_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return speed_value

    async def extract_resist_type(self, page, image_src, depth):
        # 이미지 요소 찾기
        #print(image_src)
        image = await page.query_selector(f'img[src="{image_src}"]')
        #print(image)
        if not image:
            return "Unknown"
    
        # 이미지의 부모 요소를 찾아서 올라감
        parent = image
        for _ in range(depth):
            parent = await parent.evaluate_handle('node => node.parentElement')
            if not parent:
                return "Unknown"
    
        # 부모의 형제 요소 찾기
        sibling = await parent.evaluate_handle('node => node.nextElementSibling')
        
        if sibling:
            value = await sibling.evaluate('node => node.textContent')
            if value:
                return value.strip()
        
        return "Unknown"

    async def extract_stagger_threshold_values(self, page):
        # 'div.row.center.flex-center.q-pt-xs' 요소들 찾기
        elements = await page.query_selector_all('div.row.center.flex-center.q-pt-xs')
        
        # 각 요소의 텍스트 내용 출력하여 'Stagger Threshold' 요소를 찾기
        for element in elements:
            text = await element.text_content()
            
            if text and "Stagger Threshold" in text:
                # 'Stagger Threshold'가 있는 요소를 찾았다면, 그 이후 형제 요소들에서 값을 추출
                parent = await element.evaluate_handle('node => node.parentElement')
                sibling_elements = await parent.query_selector_all('div.row.justify-evenly div')  # Threshold 값이 들어있는 div들
                
                stagger_values = []
                for sibling in sibling_elements:
                    text_content = await sibling.text_content()
                    match = re.search(r'(\d+%)', text_content)
                    if match:
                        stagger_values.append(match.group(1))  # 퍼센트 값만 리스트에 추가
                while len(stagger_values) < 3:
                    stagger_values.append(np.nan)
                #print(stagger_values)
                return stagger_values  # 추출된 값(또는 np.nan 포함)을 반환
        
        return [np.nan, np.nan, np.nan]  # 만약 'Stagger Threshold'를 찾지 못한 경우 기본값 반환

    async def crawl_skills(self, page, uptie_value):
        await self.crawl_skill(page, "skill 1")
        await self.crawl_skill(page, "skill 2")
        #if uptie_value >= 3: # 나중에 스킬 관련된 정보를 전부 긁어오는 것이 확인되면 적용한다. 동기화 3 이상만 스킬3 정보를 긁어온다.
        await self.crawl_skill(page, "skill 3")
        await self.crawl_skill(page, "defense")

    async def crawl_skill(self, page, skill_type):
        skills_elements = await page.query_selector_all('div.skilltag')
        for skill_element in skills_elements:
            skill_text = await skill_element.text_content()
            if skill_text.strip().lower() == skill_type:
                # evaluate로 조부모의 이전 형제 탐색 후 텍스트 가져오기
                left_info = await skill_element.evaluate(
                    """
                    node => {
                        // 조부모로 이동
                        const grandparent = node.parentElement.parentElement;
                
                        // 이전 형제 탐색
                        const previousSibling = grandparent.previousElementSibling;
                        if (!previousSibling) return null; // 이전 형제가 없으면 null 반환
                        // 이전 형제 내부의 .row.justify-around > div > div 탐색
                        const targetDivs = previousSibling.querySelectorAll(".row.justify-around > div");
                        // 1st div: 스킬 위력
                        const skillPower = targetDivs[0] ? targetDivs[0].querySelector('div')?.textContent.trim() : null; 
                        // 3rd div: 코인 파워 값 (3번째 div에서 텍스트 추출)
                        const coinPower = targetDivs[2] ? targetDivs[2].textContent.trim() : null;
                        return { skillPower, coinPower };
                    }
                    """
                )
                print(f"{skill_type}: skillPower:{left_info.get('skillPower')} coinPower:{left_info.get('coinPower')}")

class Identity:
    """인격 정보를 저장하는 클래스"""
    def __init__(self, identity_id, name=None, rarity=None,\
                 skill_1_type=None, skill_1_sin_affinity=None,\
                 skill_2_type=None, skill_2_sin_affinity=None,\
                 skill_3_type=None, skill_3_sin_affinity=None,\
                 defense_type=None, defense_sin_affinity=None):
        self.identity_id = identity_id
        self.name = name
        self.rarity = rarity
        # 선언만 하는 부분 
        self.skill_1_type = skill_1_type 
        self.skill_1_sin_affinity = skill_1_sin_affinity 
        self.skill_2_type = skill_2_type 
        self.skill_2_sin_affinity = skill_2_sin_affinity 
        self.skill_3_type = skill_3_type 
        self.skill_3_sin_affinity = skill_3_sin_affinity 
        self.defense_type = defense_type 
        self.defense_sin_affinity = defense_sin_affinity
        self.level_max = 50
        self.uptie_max = 4
        self.level_stats = []  # LevelStat 객체 리스트
        self.uptie_stats = [] # UptieStat 객체 리스트
        self.resist_info = None
        self.stagger_threshold = None
        self.skillset = SkillSet()  # Skill정보를 담고 있는 skillset 객체 
        self.passive_infos = []  # 패시브 객체를 담고 있는 배열
        self.support_passive_infos = [] # 서포트 패시 객체를 담고 있는 배열
        self.panic_type = [] # 혼란상태 발생시 나타나는 효과를 담고 있는 객체 배열

    def update_name(self, name):
        self.name = name

    def update_rarity(self, rarity):
        self.rarity = rarity

    def add_level_stat(self, level, hp, defense):
        self.level_stats.append(LevelStat(level, hp, defense))

    def add_uptie_stat(self, uptie, speed):
        self.uptie_stats.append(UptieStat(uptie, speed))

    def add_resist_info(self, resist_hit, resist_penetrate, resist_slash):
        self.resist_info = ResistInfo(resist_hit, resist_penetrate, resist_slash)

    def add_stagger_threshold(self, first_stagger, second_stagger, third_stagger):
        self.stagger_threshold = StaggerThreshold(first_stagger, second_stagger, third_stagger)

    def update_details(self, page):
        # 세부 정보 업데이트 로직 (가정)
        pass
    #def init_uptie(self, page):
    #    init_uptie = 

# LevelStat 객체 정의
class LevelStat:
    def __init__(self, level, hp=None, defense=None):
        self.level = level
        self.hp = hp
        self.defense = defense
    def update_level(self, level):
        self.level = level
    def update_hp(self, hp):
        self.hp = hp
    def update_defense(self, defense):
        self.defense = defense

# UptieStat 객체 정의
class UptieStat:
    def __init__(self, uptie, speed=None):
        self.uptie = uptie
        self.speed = speed
    def update_speed(self, speed):
        self.speed = speed

# ResistInfo 객체 정의
class ResistInfo:
    def __init__(self, resist_hit=None, resist_penetrate=None, resist_slash=None):
        self.resist_hit = resist_hit
        self.resist_penetrate = resist_penetrate
        self.resist_slash = resist_slash

# ResistInfo 객체 정의
class StaggerThreshold:
    def __init__(self, first_stagger=None, second_stagger=None, third_stagger=None):
        self.first_stagger = first_stagger
        self.second_stagger = second_stagger
        self.third_stagger = third_stagger

class SkillSet:
    def __init__(self):

        self.skill_1_type = ""
        self.skill_2_type = ""
        self.skill_3_type = ""
        self.defense_skill_type = ""

        self.skill_1_sin_affinity = ""
        self.skill_2_sin_affinity = ""
        self.skill_3_sin_affinity = ""
        self.defense_skill_sin_affinity = ""

        self.skill_1_details = []
        self.skill_2_details = []
        self.skill_3_details = []
        self.defense_skill_details = []

    def add_skill(self, skill_code, skill):
        if skill_code == 1:
            self.skill_1_details.append(skill)
        elif skill_code == 2:
            self.skill_2_details.append(skill)
        elif skill_code == 3:
            self.skill_3_details.append(skill)
        elif skill_code == "defense":
            self.defense_skill_details.append(skill)

    def set_skill_type(self, skill_code, skill_type, sin_affinity):
        if skill_code == 1:
            self.skill_1_type = skill_type
            self.skill_1_sin_affinity = sin_affinity
        elif skill_code == 2:
            self.skill_2_type = skill_type
            self.skill_2_sin_affinity = sin_affinity
        elif skill_code == 3:
            self.skill_3_type = skill_type
            self.skill_3_sin_affinity = sin_affinity
        elif skill_type == "defense":
            self.defense_skill_type = skill_type
            self.defense_skill_sin_affinity = sin_affinity

class Skill:
    def __init__(self, skill_code, skill_name, coin_count, init_level, init_remain=None, akt_weight=None, condition=None):
        self.skill_code = skill_code # "Skill 1"/"Skill 2"/"Skill 3"/"Defense" '''o'''
        self.skill_name = skill_name # 스킬 이름
        self.coin_count = coin_count # 동전 개수
        self.init_level = init_level # 스킬이 초기 배정받은 레벨, 인격 레벨에 - 1 한 값을 더해주면 스킬의 현 level을 구할 수 있다.
        """
        남아있는 스킬 개수 사용하면, 하나씩 줄어들고 모든 스킬을 사용하면 이 값으로 돌아간다.
        스킬의 남은 숫자없는 스킬은 강화된 스킬로 한번 사용되면 다 조건을 채울 때까 사용 못한다.
        강화된 스킬을 None으로 설정하는 방식과 조건이 채워지면 1로 늘리는 방식이 존재한다.
        스킬을 사용할 수 있는 초기값이다.
        조건을 충족한 강화한 스킬을 이례적인 절차라고 생각하고 None으로 초기값을 설정했다.
        """
        self.init_remain = None 
        self.akt_weight = None #공격 가중치 값이다. 방어스킬에는 없기 때문에 None으로 설정했다.
        self.remain_count = init_remain
        """
        동기화 단계에 따른 정보를 담는다. 현재 4동기화까지 되어 있지만 차후 확장될 것을 생각하여 고정하지 않는다.
        내부에 들어있는 정보로는 uptie_level, skill_power, coin_power 속성으로 들어있다.
        전투, 턴, 스킬, 코인토스 단계로 내려오며 발생하는 효과에 관하여 저장한다.
        """
        self.uptie_infos = []  

class UptieInfo:
    def __init__(self, uptie_level, skill_power, coin_power):
        self.uptie_level = uptie_level   #'''o'''
        self.skill_power = skill_power   #'''o'''
        self.coin_power = coin_power     #'''o'''
        self.skill_effects = []

class Effect:
    def __init__(self, condition, description):
        self.condition = condition # 조건 텍스트 "[coin 1 on hit]"
        self.description = description # 효과 설명 텍스 "Inflict +1 Rupture Count

# 추상화된 EffectType 클래스
class EffectType:
    def __init__(self, name, description):
        self.name = name  # Effect 이름
        self.description = description  # Effect 설명

# Passive 클래스
class Passive(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)  # condition이 있는지 여부 확인

# SupportPassive 클래스
class SupportPassive(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)

# PanicType 클래스
class PanicType(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)

# Keyword 클래스
class Keyword:
    def __init__(self, keyword, explain=None, dispellable=None):
        self.keyword = keyword
        self.explain = explain
        self.dispellable = dispellable

      
class DataContainer:
    """크롤링 데이터를 관리하는 클래스"""
    def __init__(self):
        self.identities = []

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.level_max

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.uptie_max

    def update_identity_name(self, identity_id, identity_name):
        """특정 ID의 이름을 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_name(identity_name)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_identity_rarity(self, identity_id, rarity):
        """특정 ID의 희귀도를 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_rarity(rarity)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_skill(self, identity_id, skill_number, new_type):
        """
        특정 identity의 skill_type을 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_type: 새 skill_type
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_type = new_type
                elif skill_number == 2:
                    identity.skill_2_type = new_type
                elif skill_number == 3:
                    identity.skill_3_type = new_type
                elif skill_number == "defense":
                    identity.defense_type = new_type
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def update_sin_affinity(self, identity_id, skill_number, new_affinity):
        """
        특정 identity의 sin_affinity를 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_affinity: 새 sin_affinity
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_sin_affinity = new_affinity
                elif skill_number == 2:
                    identity.skill_2_sin_affinity = new_affinity
                elif skill_number == 3:
                    identity.skill_3_sin_affinity = new_affinity
                elif skill_number == "defense":
                    identity.defense_sin_affinity = new_affinity
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def add_identity(self, identity):
        """새로운 인격 데이터를 추가."""
        self.identities.append(identity)

    def add_level_stat(self, identity_id, level, hp, defense):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_level_stat(level, hp, defense)

    def add_uptie_stat(self, identity_id, uptie, speed):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_uptie_stat(uptie, speed)

    def add_resist_info(self, identity_id, resist_hit, resist_penetrate, resist_slash):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_resist_info(resist_hit, resist_penetrate, resist_slash)

    def add_stagger_threshold(self, identity_id, first_stagger, second_stagger, third_stagger):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_stagger_threshold(first_stagger, second_stagger, third_stagger)
        
    def display_identities(self):
        """Identity, ID, Name, Rarity 정보를 출력"""
        if not self.identities:
            print("No identities found.")
        else:
            # 데이터를 리스트로 변환
            table = []
            for idx, identity in enumerate(self.identities, 1):
                table.append([
                    idx, identity.identity_id, identity.name, identity.rarity,
                    identity.skill_1_type, identity.skill_1_sin_affinity,
                    identity.skill_2_type, identity.skill_2_sin_affinity,
                    identity.skill_3_type, identity.skill_3_sin_affinity,
                    identity.defense_type, identity.defense_sin_affinity
                ])
            
            # 테이블 헤더 정의
            headers = [
                "Index", "ID", "Name", "Rarity",
                "Skill_1 Type", "Skill_1 Sin", "Skill_2 Type", "Skill_2 Sin",
                "Skill_3 Type", "Skill_3 Sin", "Defense Type", "Defense Sin"
            ]
            
            # 테이블 출력
            print(tabulate(table, headers=headers, tablefmt="fancy_grid"))

    def export_to_csv(self, filename, fieldnames, data):
        """
        공통적으로 사용 가능한 CSV 내보내기 메서드
        :param filename: 내보낼 파일 이름 (예: 'identity.csv', 'skill.csv')
        :param fieldnames: CSV의 헤더 리스트
        :param data: CSV로 내보낼 데이터 (리스트 형태)
        """
        try:
            # 현재작업 디렉토 기준으 'csv'폴더 없으면 생성
            if not os.path.exists('./csv'):
                os.makedirs('./csv')
            # 파일 경로 설정 (상대경로 'csv' 폴더 안에 저장)
            file_path = os.path.join('./csv', filename)
            with open(file_path, mode='w', newline='', encoding='utf-8') as file:
                writer = csv.DictWriter(file, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(data)
            print(f"Data successfully exported to {filename}")
        except Exception as e:
            print(f"Error exporting to CSV: {e}")

    def get_identity_data(self):
        """
        Identity 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "ID": identity.identity_id,
                "Name": identity.name,
                "Rarity": identity.rarity,
                "Skill_1_Type": identity.skill_1_type,
                "Skill_1_Sin_Affinity": identity.skill_1_sin_affinity,
                "Skill_2_Type": identity.skill_2_type,
                "Skill_2_Sin_Affinity": identity.skill_2_sin_affinity,
                "Skill_3_Type": identity.skill_3_type,
                "Skill_3_Sin_Affinity": identity.skill_3_sin_affinity,
                "Defense_Type": identity.defense_type,
                "Defense_Sin_Affinity": identity.defense_sin_affinity,
            }
            for identity in self.identities
        ]
        
    def export_identities_data(self, filename="identities.csv"):
        """Identity 데이터를 CSV로 내보내는 메서드"""
        identity_fieldnames = [
            "ID", "Name", "Rarity",
            "Skill_1_Type", "Skill_1_Sin_Affinity",
            "Skill_2_Type", "Skill_2_Sin_Affinity",
            "Skill_3_Type", "Skill_3_Sin_Affinity",
            "Defense_Type", "Defense_Sin_Affinity"
        ]
        identity_data = self.get_identity_data()  # Identity 데이터 가져오기
        self.export_to_csv(filename, identity_fieldnames, identity_data)  # CSV로 내보내기

    def load_identities_by_id(self, filename, identity_ids):
        """지정된 identity_ids 목록에 해당하는 Identity 데이터를 CSV에서 읽어옴."""
        identity_ids_set = set(identity_ids) # 중복 제거를 위 Set 사용
        with open(filename, mode='r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                if row["ID"] in identity_ids_set:
                    identity = Identity(
                        identity_id=row["ID"],
                        name=row["Name"],
                        rarity=row["Rarity"],
                        skill_1_type=row["Skill_1_Type"],
                        skill_1_sin_affinity=row["Skill_1_Sin_Affinity"],
                        skill_2_type=row["Skill_2_Type"],
                        skill_2_sin_affinity=row["Skill_2_Sin_Affinity"],
                        skill_3_type=row["Skill_3_Type"],
                        skill_3_sin_affinity=row["Skill_3_Sin_Affinity"],
                        defense_type=row["Defense_Type"],
                        defense_sin_affinity=row["Defense_Sin_Affinity"]
                    )
                    self.identities.append(identity)

    def get_level_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Level": level_stat.level,
                "HP": level_stat.hp,
                "Defense": level_stat.defense,
            }
            for identity in self.identities
            for level_stat in identity.level_stats
        ]

    def export_level_stats_data(self, filename="levelstats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        level_stats_fieldnames = ["Identity_ID", "Level", "HP", "Defense"]
        level_stats_data = self.get_level_stats_data()  # level_stats 데이터 가져오기
        self.export_to_csv(filename, level_stats_fieldnames, level_stats_data)  # CSV로 내보내기

    def get_uptie_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Uptie": uptie_stat.uptie,
                "Speed": uptie_stat.speed,
            }
            for identity in self.identities
            for uptie_stat in identity.uptie_stats
        ]

    def export_uptie_stats_data(self, filename="Uptiestats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        uptie_stats_fieldnames = ["Identity_ID", "Uptie", "Speed"]
        uptie_stats_data = self.get_uptie_stats_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, uptie_stats_fieldnames, uptie_stats_data)  # CSV로 내보내기

    def get_resist_info_data(self):
        """
        모든 Identity의 resist_info 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Resist_Hit": identity.resist_info.resist_hit,
                "Resist_Penetrate": identity.resist_info.resist_penetrate,
                "Resist_Slash": identity.resist_info.resist_slash,
            }
            for identity in self.identities
        ]

    def export_resist_info_data(self, filename="ResistInfos.csv"):
        """ResistInfo 데이터를 CSV로 내보내는 메서드"""
        resist_infos_fieldnames = ["Identity_ID", "Resist_Hit", "Resist_Penetrate", "Resist_Slash"]
        resist_infos_data = self.get_resist_info_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, resist_infos_fieldnames, resist_infos_data)  # CSV로 내보내기

    def export_uptie_stats_data(self, filename="Uptiestats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        uptie_stats_fieldnames = ["Identity_ID", "Uptie", "Speed"]
        uptie_stats_data = self.get_uptie_stats_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, uptie_stats_fieldnames, uptie_stats_data)  # CSV로 내보내기

    def get_stagger_threshold_data(self):
        """
        모든 Identity의 stagger_threshold 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "First_Stagger": identity.stagger_threshold.first_stagger,
                "Second_Stagger": identity.stagger_threshold.second_stagger,
                "Third_Stagger": identity.stagger_threshold.third_stagger,
            }
            for identity in self.identities
        ]

    def export_stagger_threshold_data(self, filename="StaggerThresholds.csv"):
        """StaggerThresholds 데이터를 CSV로 내보내는 메서드"""
        stagger_thresholds_fieldnames = ["Identity_ID", "First_Stagger", "Second_Stagger", "Third_Stagger"]
        stagger_thresholds_data = self.get_stagger_threshold_data()  # stagger_thresholds 데이터 가져오기
        self.export_to_csv(filename, stagger_thresholds_fieldnames, stagger_thresholds_data)  # CSV로 내보내기


async def main():
    data_container = DataContainer()
    scraper = Scraper(container=data_container, base_url="https://limbus.kusoge.xyz", img_url="https://img.kusoge.xyz/limbus/img")
    await scraper.scrape()

await main() 

skill 1: skillPower:2 coinPower:+4
skill 2: skillPower:3 coinPower:+4
skill 3: skillPower:3 coinPower:+4
skill 3: skillPower:3 coinPower:+5
defense: skillPower:4 coinPower:+4


In [27]:
import os
import csv
import re
import numpy as np
import pandas as pd
from playwright.async_api import async_playwright
from tabulate import tabulate

# Sin Color 이름 변환을 위한 매핑
resource_mapping = {
    "AZURE": "Gloom",
    "VIOLET": "Envy",
    "AMBER": "Sloth",
    "SHAMROCK": "Gluttony",
    "CRIMSON": "Wrath",
    "SCARLET": "Lust",
    "INDIGO": "Pride"
}

def convert_resource(resource_name):
    """Sin Color 이름을 변환하는 함수. 매핑되지 않은 경우 원래 이름을 반환."""
    return resource_mapping.get(resource_name, resource_name)

class Scraper:
    """웹 페이지 크롤링을 위한 클래스"""
    def __init__(self, container, base_url, img_url):
        self.container = container
        self.base_url = base_url
        self.img_url = img_url

    async def handle_request(self, route, request):
        """이미지와 폰트 리소스 로드를 차단."""
        if request.resource_type in ["image", "font"]:
            await route.abort()
        else:
            await route.continue_()
            
    async def scrape(self):
        """크롤링 메인 함수"""
        async with async_playwright() as p:
            browser = await p.chromium.launch(headless=True)
            page = await browser.new_page()
            await page.route("**/*", self.handle_request)
            # Step 1: /identity 페이지에서 기본 정보 크롤링
            #await self.crawl_identity_page(page)

            # Step 2: /identity/{identity_id} 상세 페이지 크롤링
            #identity_ids_to_load = ["10104", "10310", "10710","10808", "10911", "11005"]
            #identity_ids_to_load = ["10104", "10911"]#,"10310", "10710","10808", "11005", "11210"]
            identity_ids_to_load = ["10911"]#,"10310", "10710","10808", "11005", "11210"]
            self.container.load_identities_by_id('./csv/identities.csv', identity_ids_to_load)
            await self.crawl_identity_details_for_all(page)
            #self.container.display_identities()
            #self.container.export_identities_data("identities.csv")
            #self.container.export_level_stats_data("Levelstats.csv")
            #self.container.export_uptie_stats_data("Uptiestats.csv")
            #self.container.export_resist_info_data("ResistInfos.csv")
            #self.container.export_stagger_threshold_data("StaggerThresholds.csv")
            page.set_default_timeout(10000)
            #await page.wait_for_timeout(100000)
            await browser.close()

    async def get_identity_id_from_row(self, row):
        """테이블 행(row)에서 인격 ID를 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_url = await identity_cell.get_attribute('href')
            identity_id = identity_url.split('/identity/')[-1]
            return identity_id
        return "Unknown"

    async def get_identity_name_from_row(self, row):
        """테이블 행(row)에서 인격 이름을 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_name = (await identity_cell.inner_text()).replace("\n", " ").strip()
            return identity_name
        return "Unknown"
        
    async def crawl_identity_page(self, page):
        """/identity 페이지로 이동하여 인격 기본 정보를 추출."""
        await page.goto(self.base_url + "/identity")
        await self.extract_identity_page(page)

    async def extract_identity_page(self, page):
        """/identity 페이지의 모든 테이블 행에서 인격 정보를 추출."""
        identity_rows = await page.query_selector_all('tr')
        if len(identity_rows) > 1:
            for idx, identity_row in enumerate(identity_rows):
                if idx == 0:
                    continue # 0번째요소이름을 보여주는 열은 건너뜀
                await self.scrape_identity_name_and_identity_id(identity_row)
                await self.extract_rarity(identity_row)
                for i in range(1, 4):  # 스킬1~3 정보 추출
                    await self.extract_atk_and_resource(identity_row, i)
                await self.extract_defense_and_resource(identity_row)

    async def scrape_identity_name_and_identity_id(self, row):
        """인격 번호와 이름을 추출하여 DataContainer에 저장."""
        try:
            identity_id = await self.get_identity_id_from_row(row)
            identity_name = await self.get_identity_name_from_row(row)
            identity = Identity(identity_id)
            self.container.add_identity(identity)
            self.container.update_identity_name(identity_id, identity_name)
        except Exception as e:
            print(f"Error extracting identity name and identity id: {e}")

    async def get_element_attribute(self, element, selector, attribute):
        """요소에서 지정된 속성(attribute)을 추출."""
        try:
            target_element = await element.query_selector(selector)
            if target_element:
                return await target_element.get_attribute(attribute)
        except Exception as e:
            print(f"Error getting element attribute: {e}")
        return None
    
    async def extract_rarity(self, row):
        """테이블 행(row)에서 희귀도를 추출하여 업데이트."""
        try:
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
                sibling = await parent.evaluate_handle('node => node.nextElementSibling')
                if sibling:
                    rarity_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/rarity_"]', 'src')
                    if rarity_img_src:
                        rarity_level = rarity_img_src.split('_')[-1].split('.')[0]  # rarity_1.png -> 1
                        identity_id = await self.get_identity_id_from_row(row)
                        self.container.update_identity_rarity(identity_id, rarity_level)
                    else:
                        print("Rarity image not found in the next sibling TD.")
        except Exception as e: 
            print(f"Error extracting rarity: {e}")

        return "Unknown"

    async def extract_atk_and_resource(self, row, skill_number):
        """ 공격 타입(ATK Type)과 자원(Attribute) 추출 함수 - 스킬 번호에 따라 처리 """
        try:
            #print(f"Attempting to locate skill{skill_number} type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 해당 스킬 번호에 맞는 형제로 이동
                sibling = parent
                for _ in range(skill_number + 1):  # 스킬 번호에 맞게 이동
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                atk_type, resource = "Unknown", "Unknown"
        
                if sibling:
                    # 공격 타입 이미지 (atk_type) 찾기
                    atk_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/atk_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if atk_type_img_src:
                        atk_type = atk_type_img_src.split('_')[-1].split('.')[0]  # atk_type_SLASH.png -> SLASH
                        #print(f"Extracted attack type: {atk_type}")
                        self.container.update_skill(identity_id, skill_number, atk_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted resource: {resource}")
                        
                        # 리소스를 변환
                        resource = convert_resource(resource)
                        #print(f"Converted resource: {resource}")
                        self.container.update_sin_affinity(identity_id, skill_number, resource)
                        
        except Exception as e:
            print(f"Error extracting skill{skill_number} type and resource: {e}")

    async def extract_defense_and_resource(self, row):
        """ 방어 타입(Def Type)과 자원(Attribute) 추출 함수 """
        try:
            #print("Attempting to locate defense type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 다섯 번째 형제 td로 이동 (방어 관련 정보가 있는 <td>)
                sibling = parent
                for _ in range(5):  # 방어 정보는 5번째 형제에 있음
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                def_type, def_resource = "Unknown", "Unknown"
        
                if sibling:
                    # 방어 타입 이미지 (def_type) 찾기
                    def_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/def_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if def_type_img_src:
                        def_type = def_type_img_src.split('_')[-1].split('.')[0]  # def_type_GUARD.png -> GUARD
                        #print(f"Extracted defense type: {def_type}")
                        self.container.update_skill(identity_id, 'defense', def_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        def_resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted defense resource: {def_resource}")
                        
                        # 방어 리소스를 변환
                        def_resource = convert_resource(def_resource)
                        #print(f"Converted defense resource: {def_resource}")
                        self.container.update_sin_affinity(identity_id, 'defense', def_resource)
 
        except Exception as e:
            print(f"Error extracting defense type and resource: {e}")

    async def crawl_identity_details_for_all(self, page):
        """모든 인격의 세부 정보를 크롤링."""
        for identity in self.container.identities:
            await page.wait_for_load_state("networkidle")
            await page.goto(self.base_url + "/identity/"+identity.identity_id)
            """ 레벨과 동기화와 관련 없는 참관타 저항은 밑에서 변경하는 것과 연관성이 적기 때문에 위에서 처리한다.
            print(f"Identity_id: {identity.identity_id}")
            results = []
            resist_type_imgs = [f"{self.img_url}/ui/atk_type_HIT.png",
                                f"{self.img_url}/ui/atk_type_PENETRATE.png",
                                f"{self.img_url}/ui/atk_type_SLASH.png"]
            for resist_type_img in resist_type_imgs:
                result = await self.extract_resist_type(page, resist_type_img, 1)
                results.append(result)
            self.container.add_resist_info(identity.identity_id, results[0], results[1], results[2])
            """
            """
            # Stagger Threshold 값 추출
            stagger_threshold = await self.extract_stagger_threshold_values(page)
            self.container.add_stagger_threshold(identity.identity_id, stagger_threshold[0], stagger_threshold[1], stagger_threshold[2])
            """
            uptie_value = await self.initialize_uptie_and_level(page)
            """ 레벨에 따른 level, hp, defense 다르게 되어 있는 것을 출력 확인 객체로 들어가서 csv로 옮기는 과정까지 진행
            for i in range(1, identity.level_max+1):
                await self.move_slider_to_value(page,i)
                await self.extract_level_status(page, identity.identity_id, i)
            """
            """ 동기화에 따른 speed 다르게 되어 있는 것을 출력 확인 객체로 들어가서 csv로 옮기는 과정까지 진행
            for i in range(1, identity.uptie_max+1):
                await self.uptie_click(page,i)
                await self.extract_uptie_status(page, identity.identity_id, i)
            """
            await self.crawl_skills(page, uptie_value)
            
    async def initialize_uptie_and_level(self, page):
        await self.uptie_click(page, 1)
        await self.move_slider_to_value(page, 1)
        return 1
    
    async def uptie_click(self, page, phase):
        fab_button_selector = ".q-btn.q-btn--fab"
        await page.wait_for_selector(fab_button_selector)
        await page.click(fab_button_selector)
        #print(f"[{setup_uptie_button}] 상위 동기화 버튼 클릭 완료")
        setup_uptie_button = f'a:has(img[src="{self.img_url}/ui/uptie_{phase}.png"])'
        await page.wait_for_selector(setup_uptie_button)
        await page.click(setup_uptie_button)
        #print(f"{phase} 동기화 버튼 클릭 완료")

    async def move_slider_to_value(self, page, target_value):
        """
        슬라이더를 주어진 값으로 이동시킵니다.
        target_value는 슬라이더의 최소값부터 최대값 사이여야 합니다.
        """
        slider_selector = ".q-slider"
        # 슬라이더의 범위 확인
        min_value = int(await page.get_attribute(slider_selector, "aria-valuemin"))
        max_value = int(await page.get_attribute(slider_selector, "aria-valuemax"))
    
        if target_value < min_value or target_value > max_value:
            raise ValueError(f"Target value {target_value} is out of bounds ({min_value}, {max_value})")
    
        # 슬라이더 thumb 요소와 위치 가져오기
        slider_thumb = await page.query_selector(f"{slider_selector} .q-slider__thumb")
        thumb_box = await slider_thumb.bounding_box()
        
        # 슬라이더의 현재 위치 및 목표 위치 계산
        track = await page.query_selector(f"{slider_selector} .q-slider__track")
        track_box = await track.bounding_box()
        
        # 목표 x 위치 계산 (슬라이더 트랙 내에서 비율 기반)
        target_percentage = (target_value - min_value) / (max_value - min_value)
        target_x = track_box["x"] + target_percentage * track_box["width"]
        current_x = thumb_box["x"] + thumb_box["width"] / 2
        y = thumb_box["y"] + thumb_box["height"] / 2
    
        # 슬라이더를 드래그하여 목표 위치로 이동
        await page.mouse.move(current_x, y)
        await page.mouse.down()
        await page.mouse.move(target_x, y, steps=10)  # 부드럽게 이동
        await page.mouse.up()
    
        #print(f"슬라이더를 {target_value}로 이동 완료.")
    
        # 슬라이더 값 확인 및 강제 이벤트 트리거
        await page.evaluate(
            f"""
            (slider) => {{
                slider.dispatchEvent(new Event('input'));  // oninput 이벤트 트리거
                slider.dispatchEvent(new Event('change'));  // onchange 이벤트 트리거
            }}
            """,
            await page.query_selector(slider_selector)
        )
    
        # 이동 후 값 확인
        current_value = await page.get_attribute(slider_selector, "aria-valuenow")
        if current_value != str(target_value):
            print(f"경고: 슬라이더 값이 예상과 다릅니다. 현재 값: {current_value}, 예상 값: {target_value}")

    async def get_second_status_element(self, page):
        """
        두 번째 'Status' 요소에 접근하여 반환하는 메소드
        """
        status_elements = await page.query_selector_all("text=Status")
        if len(status_elements) > 1:
            second_status_element = status_elements[1]
            #print("두 번째 'Status' 요소에 접근합니다.")
            return second_status_element
        else:
            #print("두 번째 'Status' 요소가 존재하지 않습니다.")
            return None

    async def extract_level_status(self, page, identity_id, level):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 레벨에 따라 HP와 Defense가 변한다.
        hp_value = await self.extract_hp(second_status_element)
        defense_value = await self.extract_defense(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Level={level}, HP={hp_value}, Defense={defense_value}")
        self.container.add_level_stat(identity_id,level, hp_value, defense_value)
    
    async def extract_hp(self, status_element):
        # HP 추출
        hp_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].children[0].nextElementSibling.textContent.trim()"
        )
        return hp_value
    
    async def extract_defense(self, status_element):
        # Defense 추출
        defense_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return defense_value

    async def extract_uptie_status(self, page, identity_id, uptie):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 uptie에 따라 speed는 변한다.
        speed_value = await self.extract_speed(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Uptie={uptie}, Speed={speed_value}")
        self.container.add_uptie_stat(identity_id,uptie, speed_value)

    async def extract_speed(self, status_element):
        # Speed 추출
        speed_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return speed_value

    async def extract_resist_type(self, page, image_src, depth):
        # 이미지 요소 찾기
        #print(image_src)
        image = await page.query_selector(f'img[src="{image_src}"]')
        #print(image)
        if not image:
            return "Unknown"
    
        # 이미지의 부모 요소를 찾아서 올라감
        parent = image
        for _ in range(depth):
            parent = await parent.evaluate_handle('node => node.parentElement')
            if not parent:
                return "Unknown"
    
        # 부모의 형제 요소 찾기
        sibling = await parent.evaluate_handle('node => node.nextElementSibling')
        
        if sibling:
            value = await sibling.evaluate('node => node.textContent')
            if value:
                return value.strip()
        
        return "Unknown"

    async def extract_stagger_threshold_values(self, page):
        # 'div.row.center.flex-center.q-pt-xs' 요소들 찾기
        elements = await page.query_selector_all('div.row.center.flex-center.q-pt-xs')
        
        # 각 요소의 텍스트 내용 출력하여 'Stagger Threshold' 요소를 찾기
        for element in elements:
            text = await element.text_content()
            
            if text and "Stagger Threshold" in text:
                # 'Stagger Threshold'가 있는 요소를 찾았다면, 그 이후 형제 요소들에서 값을 추출
                parent = await element.evaluate_handle('node => node.parentElement')
                sibling_elements = await parent.query_selector_all('div.row.justify-evenly div')  # Threshold 값이 들어있는 div들
                
                stagger_values = []
                for sibling in sibling_elements:
                    text_content = await sibling.text_content()
                    match = re.search(r'(\d+%)', text_content)
                    if match:
                        stagger_values.append(match.group(1))  # 퍼센트 값만 리스트에 추가
                while len(stagger_values) < 3:
                    stagger_values.append(np.nan)
                #print(stagger_values)
                return stagger_values  # 추출된 값(또는 np.nan 포함)을 반환
        
        return [np.nan, np.nan, np.nan]  # 만약 'Stagger Threshold'를 찾지 못한 경우 기본값 반환

    async def crawl_skills(self, page, uptie_value):
        await self.crawl_skill(page, "skill 1")
        await self.crawl_skill(page, "skill 2")
        #if uptie_value >= 3: # 나중에 스킬 관련된 정보를 전부 긁어오는 것이 확인되면 적용한다. 동기화 3 이상만 스킬3 정보를 긁어온다.
        await self.crawl_skill(page, "skill 3")
        await self.crawl_skill(page, "defense")

    async def crawl_skill(self, page, skill_type):
        skills_elements = await page.query_selector_all('div.skilltag')
        for skill_element in skills_elements:
            skill_text = await skill_element.text_content()
            if skill_text.strip().lower() == skill_type:
                # 스킬 위력과 코인 위력 가져오는 메소드 호출
                skill_data = await self.skill_and_coin_power(skill_element)
                skill_power = skill_data.get("skillPower")
                coin_power = skill_data.get("coinPower")
                coin_count = await self.coin_count(skill_element,".col-auto > .column > .row")
                init_remain = await self.init_remain(skill_element, skill_type, "div.row.justify-start > div.attr.flex.flex-center > div.q-px-sm")
                skill_name = await self.skill_name(skill_element)
                init_level = await self.init_level(skill_element)
                atk_wieght = await self.atk_weight(skill_element, skill_type)
                print(f"{skill_type}:{skill_name} skill_power: {skill_power}, coin_power: {coin_power}, coin_count: {coin_count}, init_remain : {init_remain}, init_level: {init_level}, atk_wieght: {atk_wieght}")

    async def skill_and_coin_power(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                // 조부모로 이동
                const grandparent = node.parentElement.parentElement;
                // 이전 형제 탐색
                const previousSibling = grandparent.previousElementSibling;
                if (!previousSibling) return {{ skillPower: null, coinPower: null }}; // 이전 형제가 없으면 기본값 반환
                // 이전 형제 내부의 .row.justify-around > div > div 탐색
                const targetDivs = previousSibling.querySelectorAll(".row.justify-around > div");
                if (targetDivs.length < 3) return {{ skillPower: null, coinPower: null }}; // 배열 길이 확인
                // 1st div: 스킬 위력
                const skillPower = targetDivs[0]?.querySelector('div')?.textContent?.trim() || null; 
                // 3rd div: 코인 파워 값 (3번째 div에서 텍스트 추출)
                const coinPower = targetDivs[2]?.textContent?.trim() || null;
                return {{ skillPower, coinPower }};
            }}
        """)


    async def coin_count(self, skill_element, hierarchy_selector):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector('{hierarchy_selector}');
                if (!targetElement) return 0;
                return targetElement.childElementCount;
            }}
        """)

    async def init_remain(self, skill_element, skill_type, hierarchy_selector):
        if skill_type == "defense":
            return -1
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector('{hierarchy_selector}');
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def skill_name(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector(".block");
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def init_level(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector("div.text-bold.text-h6.h6.q-pl-xs.q-pr-md");
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def atk_weight(self, skill_element, skill_type):
        if skill_type == "defense":
            return 0
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector("div.text-bold.text-h6.h6.q-pl-xs.q-pr-md");
                const doubleNextSibling = targetElement.nextElementSibling.nextElementSibling;
                if (!doubleNextSibling) return 0;
                return doubleNextSibling.textContent?.trim();
            }}
        """)

class Identity:
    """인격 정보를 저장하는 클래스"""
    def __init__(self, identity_id, name=None, rarity=None,\
                 skill_1_type=None, skill_1_sin_affinity=None,\
                 skill_2_type=None, skill_2_sin_affinity=None,\
                 skill_3_type=None, skill_3_sin_affinity=None,\
                 defense_type=None, defense_sin_affinity=None):
        self.identity_id = identity_id
        self.name = name
        self.rarity = rarity
        # 선언만 하는 부분 
        self.skill_1_type = skill_1_type 
        self.skill_1_sin_affinity = skill_1_sin_affinity 
        self.skill_2_type = skill_2_type 
        self.skill_2_sin_affinity = skill_2_sin_affinity 
        self.skill_3_type = skill_3_type 
        self.skill_3_sin_affinity = skill_3_sin_affinity 
        self.defense_type = defense_type 
        self.defense_sin_affinity = defense_sin_affinity
        self.level_max = 50
        self.uptie_max = 4
        self.level_stats = []  # LevelStat 객체 리스트
        self.uptie_stats = [] # UptieStat 객체 리스트
        self.resist_info = None
        self.stagger_threshold = None
        self.skillset = SkillSet()  # Skill정보를 담고 있는 skillset 객체 
        self.passive_infos = []  # 패시브 객체를 담고 있는 배열
        self.support_passive_infos = [] # 서포트 패시 객체를 담고 있는 배열
        self.panic_type = [] # 혼란상태 발생시 나타나는 효과를 담고 있는 객체 배열

    def update_name(self, name):
        self.name = name

    def update_rarity(self, rarity):
        self.rarity = rarity

    def add_level_stat(self, level, hp, defense):
        self.level_stats.append(LevelStat(level, hp, defense))

    def add_uptie_stat(self, uptie, speed):
        self.uptie_stats.append(UptieStat(uptie, speed))

    def add_resist_info(self, resist_hit, resist_penetrate, resist_slash):
        self.resist_info = ResistInfo(resist_hit, resist_penetrate, resist_slash)

    def add_stagger_threshold(self, first_stagger, second_stagger, third_stagger):
        self.stagger_threshold = StaggerThreshold(first_stagger, second_stagger, third_stagger)

    def update_details(self, page):
        # 세부 정보 업데이트 로직 (가정)
        pass
    #def init_uptie(self, page):
    #    init_uptie = 

# LevelStat 객체 정의
class LevelStat:
    def __init__(self, level, hp=None, defense=None):
        self.level = level
        self.hp = hp
        self.defense = defense
    def update_level(self, level):
        self.level = level
    def update_hp(self, hp):
        self.hp = hp
    def update_defense(self, defense):
        self.defense = defense

# UptieStat 객체 정의
class UptieStat:
    def __init__(self, uptie, speed=None):
        self.uptie = uptie
        self.speed = speed
    def update_speed(self, speed):
        self.speed = speed

# ResistInfo 객체 정의
class ResistInfo:
    def __init__(self, resist_hit=None, resist_penetrate=None, resist_slash=None):
        self.resist_hit = resist_hit
        self.resist_penetrate = resist_penetrate
        self.resist_slash = resist_slash

# ResistInfo 객체 정의
class StaggerThreshold:
    def __init__(self, first_stagger=None, second_stagger=None, third_stagger=None):
        self.first_stagger = first_stagger
        self.second_stagger = second_stagger
        self.third_stagger = third_stagger

class SkillSet:
    def __init__(self):

        self.skill_1_type = ""
        self.skill_2_type = ""
        self.skill_3_type = ""
        self.defense_skill_type = ""

        self.skill_1_sin_affinity = ""
        self.skill_2_sin_affinity = ""
        self.skill_3_sin_affinity = ""
        self.defense_skill_sin_affinity = ""

        self.skill_1_details = []
        self.skill_2_details = []
        self.skill_3_details = []
        self.defense_skill_details = []

    def add_skill(self, skill_code, skill):
        if skill_code == 1:
            self.skill_1_details.append(skill)
        elif skill_code == 2:
            self.skill_2_details.append(skill)
        elif skill_code == 3:
            self.skill_3_details.append(skill)
        elif skill_code == "defense":
            self.defense_skill_details.append(skill)

    def set_skill_type(self, skill_code, skill_type, sin_affinity):
        if skill_code == 1:
            self.skill_1_type = skill_type
            self.skill_1_sin_affinity = sin_affinity
        elif skill_code == 2:
            self.skill_2_type = skill_type
            self.skill_2_sin_affinity = sin_affinity
        elif skill_code == 3:
            self.skill_3_type = skill_type
            self.skill_3_sin_affinity = sin_affinity
        elif skill_type == "defense":
            self.defense_skill_type = skill_type
            self.defense_skill_sin_affinity = sin_affinity

class Skill:
    def __init__(self, skill_code, skill_name, coin_count, init_level, init_remain=None, akt_weight=None, condition=None):
        self.skill_code = skill_code # "Skill 1"/"Skill 2"/"Skill 3"/"Defense" '''o'''
        self.skill_name = skill_name # 스킬 이름 '''o'''
        self.coin_count = coin_count # 동전 개수 '''o'''
        self.init_level = init_level # 스킬이 초기 배정받은 레벨, 인격 레벨에 - 1 한 값을 더해주면 스킬의 현 level을 구할 수 있다. '''o'''
        """
        남아있는 스킬 개수 사용하면, 하나씩 줄어들고 모든 스킬을 사용하면 이 값으로 돌아간다.
        스킬의 남은 숫자없는 스킬은 강화된 스킬로 한번 사용되면 다 조건을 채울 때까 사용 못한다.
        강화된 스킬을 None으로 설정하는 방식과 조건이 채워지면 1로 늘리는 방식이 존재한다.
        스킬을 사용할 수 있는 초기값이다.
        조건을 충족한 강화한 스킬을 이례적인 절차라고 생각하고 None으로 초기값을 설정했다.
        """
        self.init_remain = None #'''o'''
        self.atk_weight = None #공격 가중치 값이다. 방어스킬에는 없기 때문에 None으로 설정했다.
        self.remain_count = init_remain
        """
        동기화 단계에 따른 정보를 담는다. 현재 4동기화까지 되어 있지만 차후 확장될 것을 생각하여 고정하지 않는다.
        내부에 들어있는 정보로는 uptie_level, skill_power, coin_power 속성으로 들어있다.
        전투, 턴, 스킬, 코인토스 단계로 내려오며 발생하는 효과에 관하여 저장한다.
        """
        self.uptie_infos = []  

class UptieInfo:
    def __init__(self, uptie_level, skill_power, coin_power):
        self.uptie_level = uptie_level   #'''o'''
        self.skill_power = skill_power   #'''o'''
        self.coin_power = coin_power     #'''o'''
        self.skill_effects = []

class Effect:
    def __init__(self, condition, description):
        self.condition = condition # 조건 텍스트 "[coin 1 on hit]"
        self.description = description # 효과 설명 텍스 "Inflict +1 Rupture Count

# 추상화된 EffectType 클래스
class EffectType:
    def __init__(self, name, description):
        self.name = name  # Effect 이름
        self.description = description  # Effect 설명

# Passive 클래스
class Passive(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)  # condition이 있는지 여부 확인

# SupportPassive 클래스
class SupportPassive(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)

# PanicType 클래스
class PanicType(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)

# Keyword 클래스
class Keyword:
    def __init__(self, keyword, explain=None, dispellable=None):
        self.keyword = keyword
        self.explain = explain
        self.dispellable = dispellable

      
class DataContainer:
    """크롤링 데이터를 관리하는 클래스"""
    def __init__(self):
        self.identities = []

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.level_max

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.uptie_max

    def update_identity_name(self, identity_id, identity_name):
        """특정 ID의 이름을 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_name(identity_name)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_identity_rarity(self, identity_id, rarity):
        """특정 ID의 희귀도를 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_rarity(rarity)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_skill(self, identity_id, skill_number, new_type):
        """
        특정 identity의 skill_type을 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_type: 새 skill_type
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_type = new_type
                elif skill_number == 2:
                    identity.skill_2_type = new_type
                elif skill_number == 3:
                    identity.skill_3_type = new_type
                elif skill_number == "defense":
                    identity.defense_type = new_type
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def update_sin_affinity(self, identity_id, skill_number, new_affinity):
        """
        특정 identity의 sin_affinity를 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_affinity: 새 sin_affinity
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_sin_affinity = new_affinity
                elif skill_number == 2:
                    identity.skill_2_sin_affinity = new_affinity
                elif skill_number == 3:
                    identity.skill_3_sin_affinity = new_affinity
                elif skill_number == "defense":
                    identity.defense_sin_affinity = new_affinity
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def add_identity(self, identity):
        """새로운 인격 데이터를 추가."""
        self.identities.append(identity)

    def add_level_stat(self, identity_id, level, hp, defense):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_level_stat(level, hp, defense)

    def add_uptie_stat(self, identity_id, uptie, speed):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_uptie_stat(uptie, speed)

    def add_resist_info(self, identity_id, resist_hit, resist_penetrate, resist_slash):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_resist_info(resist_hit, resist_penetrate, resist_slash)

    def add_stagger_threshold(self, identity_id, first_stagger, second_stagger, third_stagger):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_stagger_threshold(first_stagger, second_stagger, third_stagger)
        
    def display_identities(self):
        """Identity, ID, Name, Rarity 정보를 출력"""
        if not self.identities:
            print("No identities found.")
        else:
            # 데이터를 리스트로 변환
            table = []
            for idx, identity in enumerate(self.identities, 1):
                table.append([
                    idx, identity.identity_id, identity.name, identity.rarity,
                    identity.skill_1_type, identity.skill_1_sin_affinity,
                    identity.skill_2_type, identity.skill_2_sin_affinity,
                    identity.skill_3_type, identity.skill_3_sin_affinity,
                    identity.defense_type, identity.defense_sin_affinity
                ])
            
            # 테이블 헤더 정의
            headers = [
                "Index", "ID", "Name", "Rarity",
                "Skill_1 Type", "Skill_1 Sin", "Skill_2 Type", "Skill_2 Sin",
                "Skill_3 Type", "Skill_3 Sin", "Defense Type", "Defense Sin"
            ]
            
            # 테이블 출력
            print(tabulate(table, headers=headers, tablefmt="fancy_grid"))

    def export_to_csv(self, filename, fieldnames, data):
        """
        공통적으로 사용 가능한 CSV 내보내기 메서드
        :param filename: 내보낼 파일 이름 (예: 'identity.csv', 'skill.csv')
        :param fieldnames: CSV의 헤더 리스트
        :param data: CSV로 내보낼 데이터 (리스트 형태)
        """
        try:
            # 현재작업 디렉토 기준으 'csv'폴더 없으면 생성
            if not os.path.exists('./csv'):
                os.makedirs('./csv')
            # 파일 경로 설정 (상대경로 'csv' 폴더 안에 저장)
            file_path = os.path.join('./csv', filename)
            with open(file_path, mode='w', newline='', encoding='utf-8') as file:
                writer = csv.DictWriter(file, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(data)
            print(f"Data successfully exported to {filename}")
        except Exception as e:
            print(f"Error exporting to CSV: {e}")

    def get_identity_data(self):
        """
        Identity 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "ID": identity.identity_id,
                "Name": identity.name,
                "Rarity": identity.rarity,
                "Skill_1_Type": identity.skill_1_type,
                "Skill_1_Sin_Affinity": identity.skill_1_sin_affinity,
                "Skill_2_Type": identity.skill_2_type,
                "Skill_2_Sin_Affinity": identity.skill_2_sin_affinity,
                "Skill_3_Type": identity.skill_3_type,
                "Skill_3_Sin_Affinity": identity.skill_3_sin_affinity,
                "Defense_Type": identity.defense_type,
                "Defense_Sin_Affinity": identity.defense_sin_affinity,
            }
            for identity in self.identities
        ]
        
    def export_identities_data(self, filename="identities.csv"):
        """Identity 데이터를 CSV로 내보내는 메서드"""
        identity_fieldnames = [
            "ID", "Name", "Rarity",
            "Skill_1_Type", "Skill_1_Sin_Affinity",
            "Skill_2_Type", "Skill_2_Sin_Affinity",
            "Skill_3_Type", "Skill_3_Sin_Affinity",
            "Defense_Type", "Defense_Sin_Affinity"
        ]
        identity_data = self.get_identity_data()  # Identity 데이터 가져오기
        self.export_to_csv(filename, identity_fieldnames, identity_data)  # CSV로 내보내기

    def load_identities_by_id(self, filename, identity_ids):
        """지정된 identity_ids 목록에 해당하는 Identity 데이터를 CSV에서 읽어옴."""
        identity_ids_set = set(identity_ids) # 중복 제거를 위 Set 사용
        with open(filename, mode='r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                if row["ID"] in identity_ids_set:
                    identity = Identity(
                        identity_id=row["ID"],
                        name=row["Name"],
                        rarity=row["Rarity"],
                        skill_1_type=row["Skill_1_Type"],
                        skill_1_sin_affinity=row["Skill_1_Sin_Affinity"],
                        skill_2_type=row["Skill_2_Type"],
                        skill_2_sin_affinity=row["Skill_2_Sin_Affinity"],
                        skill_3_type=row["Skill_3_Type"],
                        skill_3_sin_affinity=row["Skill_3_Sin_Affinity"],
                        defense_type=row["Defense_Type"],
                        defense_sin_affinity=row["Defense_Sin_Affinity"]
                    )
                    self.identities.append(identity)

    def get_level_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Level": level_stat.level,
                "HP": level_stat.hp,
                "Defense": level_stat.defense,
            }
            for identity in self.identities
            for level_stat in identity.level_stats
        ]

    def export_level_stats_data(self, filename="levelstats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        level_stats_fieldnames = ["Identity_ID", "Level", "HP", "Defense"]
        level_stats_data = self.get_level_stats_data()  # level_stats 데이터 가져오기
        self.export_to_csv(filename, level_stats_fieldnames, level_stats_data)  # CSV로 내보내기

    def get_uptie_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Uptie": uptie_stat.uptie,
                "Speed": uptie_stat.speed,
            }
            for identity in self.identities
            for uptie_stat in identity.uptie_stats
        ]

    def export_uptie_stats_data(self, filename="Uptiestats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        uptie_stats_fieldnames = ["Identity_ID", "Uptie", "Speed"]
        uptie_stats_data = self.get_uptie_stats_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, uptie_stats_fieldnames, uptie_stats_data)  # CSV로 내보내기

    def get_resist_info_data(self):
        """
        모든 Identity의 resist_info 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Resist_Hit": identity.resist_info.resist_hit,
                "Resist_Penetrate": identity.resist_info.resist_penetrate,
                "Resist_Slash": identity.resist_info.resist_slash,
            }
            for identity in self.identities
        ]

    def export_resist_info_data(self, filename="ResistInfos.csv"):
        """ResistInfo 데이터를 CSV로 내보내는 메서드"""
        resist_infos_fieldnames = ["Identity_ID", "Resist_Hit", "Resist_Penetrate", "Resist_Slash"]
        resist_infos_data = self.get_resist_info_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, resist_infos_fieldnames, resist_infos_data)  # CSV로 내보내기

    def export_uptie_stats_data(self, filename="Uptiestats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        uptie_stats_fieldnames = ["Identity_ID", "Uptie", "Speed"]
        uptie_stats_data = self.get_uptie_stats_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, uptie_stats_fieldnames, uptie_stats_data)  # CSV로 내보내기

    def get_stagger_threshold_data(self):
        """
        모든 Identity의 stagger_threshold 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "First_Stagger": identity.stagger_threshold.first_stagger,
                "Second_Stagger": identity.stagger_threshold.second_stagger,
                "Third_Stagger": identity.stagger_threshold.third_stagger,
            }
            for identity in self.identities
        ]

    def export_stagger_threshold_data(self, filename="StaggerThresholds.csv"):
        """StaggerThresholds 데이터를 CSV로 내보내는 메서드"""
        stagger_thresholds_fieldnames = ["Identity_ID", "First_Stagger", "Second_Stagger", "Third_Stagger"]
        stagger_thresholds_data = self.get_stagger_threshold_data()  # stagger_thresholds 데이터 가져오기
        self.export_to_csv(filename, stagger_thresholds_fieldnames, stagger_thresholds_data)  # CSV로 내보내기


async def main():
    data_container = DataContainer()
    scraper = Scraper(container=data_container, base_url="https://limbus.kusoge.xyz", img_url="https://img.kusoge.xyz/limbus/img")
    await scraper.scrape()

await main() 

skill 1:Begone... skill_power: 2, coin_power: +4, coin_count: 2, init_remain : 3x, init_level: 2, atk_wieght: 1
skill 2:In Finely Ground Mistfall skill_power: 3, coin_power: +4, coin_count: 3, init_remain : 2x, init_level: 3, atk_wieght: 1
skill 3:The Festival Will End skill_power: 3, coin_power: +4, coin_count: 3, init_remain : 1x, init_level: 4, atk_wieght: 1
skill 3:Ascendant Don Quixote Hardblood Arts - The Finale skill_power: 3, coin_power: +5, coin_count: 3, init_remain : 0, init_level: 6, atk_wieght: 3
defense:Don Quixote Hardblood Arts 15: Parasol skill_power: 4, coin_power: +4, coin_count: 2, init_remain : -1, init_level: 3, atk_wieght: 0


In [11]:
import os
import csv
import re
import numpy as np
import pandas as pd
from playwright.async_api import async_playwright
from tabulate import tabulate

# Sin Color 이름 변환을 위한 매핑
resource_mapping = {
    "AZURE": "Gloom",
    "VIOLET": "Envy",
    "AMBER": "Sloth",
    "SHAMROCK": "Gluttony",
    "CRIMSON": "Wrath",
    "SCARLET": "Lust",
    "INDIGO": "Pride"
}

def convert_resource(resource_name):
    """Sin Color 이름을 변환하는 함수. 매핑되지 않은 경우 원래 이름을 반환."""
    return resource_mapping.get(resource_name, resource_name)

class Scraper:
    """웹 페이지 크롤링을 위한 클래스"""
    def __init__(self, container, base_url, img_url):
        self.container = container
        self.base_url = base_url
        self.img_url = img_url

    async def handle_request(self, route, request):
        """이미지와 폰트 리소스 로드를 차단."""
        if request.resource_type in ["image", "font"]:
            await route.abort()
        else:
            await route.continue_()
            
    async def scrape(self):
        """크롤링 메인 함수"""
        async with async_playwright() as p:
            browser = await p.chromium.launch(headless=True)
            page = await browser.new_page()
            await page.route("**/*", self.handle_request)
            # Step 1: /identity 페이지에서 기본 정보 크롤링
            #await self.crawl_identity_page(page)

            # Step 2: /identity/{identity_id} 상세 페이지 크롤링
            #identity_ids_to_load = ["10104", "10310", "10710","10808", "10911", "11005"]
            #identity_ids_to_load = ["10104", "10911"]#,"10310", "10710","10808", "11005", "11210"]
            identity_ids_to_load = ["10911"]#,"10310", "10710","10808", "11005", "11210"]
            self.container.load_identities_by_id('./csv/identities.csv', identity_ids_to_load)
            await self.crawl_identity_details_for_all(page)
            #self.container.display_identities()
            #self.container.export_identities_data("identities.csv")
            #self.container.export_level_stats_data("Levelstats.csv")
            #self.container.export_uptie_stats_data("Uptiestats.csv")
            #self.container.export_resist_info_data("ResistInfos.csv")
            #self.container.export_stagger_threshold_data("StaggerThresholds.csv")
            page.set_default_timeout(10000)
            #await page.wait_for_timeout(100000)
            await browser.close()

    async def get_identity_id_from_row(self, row):
        """테이블 행(row)에서 인격 ID를 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_url = await identity_cell.get_attribute('href')
            identity_id = identity_url.split('/identity/')[-1]
            return identity_id
        return "Unknown"

    async def get_identity_name_from_row(self, row):
        """테이블 행(row)에서 인격 이름을 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_name = (await identity_cell.inner_text()).replace("\n", " ").strip()
            return identity_name
        return "Unknown"
        
    async def crawl_identity_page(self, page):
        """/identity 페이지로 이동하여 인격 기본 정보를 추출."""
        await page.goto(self.base_url + "/identity")
        await self.extract_identity_page(page)

    async def extract_identity_page(self, page):
        """/identity 페이지의 모든 테이블 행에서 인격 정보를 추출."""
        identity_rows = await page.query_selector_all('tr')
        if len(identity_rows) > 1:
            for idx, identity_row in enumerate(identity_rows):
                if idx == 0:
                    continue # 0번째요소이름을 보여주는 열은 건너뜀
                await self.scrape_identity_name_and_identity_id(identity_row)
                await self.extract_rarity(identity_row)
                for i in range(1, 4):  # 스킬1~3 정보 추출
                    await self.extract_atk_and_resource(identity_row, i)
                await self.extract_defense_and_resource(identity_row)

    async def scrape_identity_name_and_identity_id(self, row):
        """인격 번호와 이름을 추출하여 DataContainer에 저장."""
        try:
            identity_id = await self.get_identity_id_from_row(row)
            identity_name = await self.get_identity_name_from_row(row)
            identity = Identity(identity_id)
            self.container.add_identity(identity)
            self.container.update_identity_name(identity_id, identity_name)
        except Exception as e:
            print(f"Error extracting identity name and identity id: {e}")

    async def get_element_attribute(self, element, selector, attribute):
        """요소에서 지정된 속성(attribute)을 추출."""
        try:
            target_element = await element.query_selector(selector)
            if target_element:
                return await target_element.get_attribute(attribute)
        except Exception as e:
            print(f"Error getting element attribute: {e}")
        return None
    
    async def extract_rarity(self, row):
        """테이블 행(row)에서 희귀도를 추출하여 업데이트."""
        try:
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
                sibling = await parent.evaluate_handle('node => node.nextElementSibling')
                if sibling:
                    rarity_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/rarity_"]', 'src')
                    if rarity_img_src:
                        rarity_level = rarity_img_src.split('_')[-1].split('.')[0]  # rarity_1.png -> 1
                        identity_id = await self.get_identity_id_from_row(row)
                        self.container.update_identity_rarity(identity_id, rarity_level)
                    else:
                        print("Rarity image not found in the next sibling TD.")
        except Exception as e: 
            print(f"Error extracting rarity: {e}")

        return "Unknown"

    async def extract_atk_and_resource(self, row, skill_number):
        """ 공격 타입(ATK Type)과 자원(Attribute) 추출 함수 - 스킬 번호에 따라 처리 """
        try:
            #print(f"Attempting to locate skill{skill_number} type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 해당 스킬 번호에 맞는 형제로 이동
                sibling = parent
                for _ in range(skill_number + 1):  # 스킬 번호에 맞게 이동
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                atk_type, resource = "Unknown", "Unknown"
        
                if sibling:
                    # 공격 타입 이미지 (atk_type) 찾기
                    atk_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/atk_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if atk_type_img_src:
                        atk_type = atk_type_img_src.split('_')[-1].split('.')[0]  # atk_type_SLASH.png -> SLASH
                        #print(f"Extracted attack type: {atk_type}")
                        self.container.update_skill(identity_id, skill_number, atk_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted resource: {resource}")
                        
                        # 리소스를 변환
                        resource = convert_resource(resource)
                        #print(f"Converted resource: {resource}")
                        self.container.update_sin_affinity(identity_id, skill_number, resource)
                        
        except Exception as e:
            print(f"Error extracting skill{skill_number} type and resource: {e}")

    async def extract_defense_and_resource(self, row):
        """ 방어 타입(Def Type)과 자원(Attribute) 추출 함수 """
        try:
            #print("Attempting to locate defense type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 다섯 번째 형제 td로 이동 (방어 관련 정보가 있는 <td>)
                sibling = parent
                for _ in range(5):  # 방어 정보는 5번째 형제에 있음
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                def_type, def_resource = "Unknown", "Unknown"
        
                if sibling:
                    # 방어 타입 이미지 (def_type) 찾기
                    def_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/def_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if def_type_img_src:
                        def_type = def_type_img_src.split('_')[-1].split('.')[0]  # def_type_GUARD.png -> GUARD
                        #print(f"Extracted defense type: {def_type}")
                        self.container.update_skill(identity_id, 'defense', def_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        def_resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted defense resource: {def_resource}")
                        
                        # 방어 리소스를 변환
                        def_resource = convert_resource(def_resource)
                        #print(f"Converted defense resource: {def_resource}")
                        self.container.update_sin_affinity(identity_id, 'defense', def_resource)
 
        except Exception as e:
            print(f"Error extracting defense type and resource: {e}")

    async def crawl_identity_details_for_all(self, page):
        """모든 인격의 세부 정보를 크롤링."""
        for identity in self.container.identities:
            await page.wait_for_load_state("networkidle")
            await page.goto(self.base_url + "/identity/"+identity.identity_id)
            """ 레벨과 동기화와 관련 없는 참관타 저항은 밑에서 변경하는 것과 연관성이 적기 때문에 위에서 처리한다.
            print(f"Identity_id: {identity.identity_id}")
            results = []
            resist_type_imgs = [f"{self.img_url}/ui/atk_type_HIT.png",
                                f"{self.img_url}/ui/atk_type_PENETRATE.png",
                                f"{self.img_url}/ui/atk_type_SLASH.png"]
            for resist_type_img in resist_type_imgs:
                result = await self.extract_resist_type(page, resist_type_img, 1)
                results.append(result)
            self.container.add_resist_info(identity.identity_id, results[0], results[1], results[2])
            """
            """
            # Stagger Threshold 값 추출
            stagger_threshold = await self.extract_stagger_threshold_values(page)
            self.container.add_stagger_threshold(identity.identity_id, stagger_threshold[0], stagger_threshold[1], stagger_threshold[2])
            """
            uptie_value = await self.initialize_uptie_and_level(page)
            """ 레벨에 따른 level, hp, defense 다르게 되어 있는 것을 출력 확인 객체로 들어가서 csv로 옮기는 과정까지 진행
            for i in range(1, identity.level_max+1):
                await self.move_slider_to_value(page,i)
                await self.extract_level_status(page, identity.identity_id, i)
            """
            """ 동기화에 따른 speed 다르게 되어 있는 것을 출력 확인 객체로 들어가서 csv로 옮기는 과정까지 진행
            for i in range(1, identity.uptie_max+1):
                await self.uptie_click(page,i)
                await self.extract_uptie_status(page, identity.identity_id, i)
            """
            await self.crawl_skills(page, identity.identity_id, uptie_value)
            
    async def initialize_uptie_and_level(self, page):
        await self.uptie_click(page, 1)
        await self.move_slider_to_value(page, 1)
        return 1
    
    async def uptie_click(self, page, phase):
        fab_button_selector = ".q-btn.q-btn--fab"
        await page.wait_for_selector(fab_button_selector)
        await page.click(fab_button_selector)
        #print(f"[{setup_uptie_button}] 상위 동기화 버튼 클릭 완료")
        setup_uptie_button = f'a:has(img[src="{self.img_url}/ui/uptie_{phase}.png"])'
        await page.wait_for_selector(setup_uptie_button)
        await page.click(setup_uptie_button)
        #print(f"{phase} 동기화 버튼 클릭 완료")

    async def move_slider_to_value(self, page, target_value):
        """
        슬라이더를 주어진 값으로 이동시킵니다.
        target_value는 슬라이더의 최소값부터 최대값 사이여야 합니다.
        """
        slider_selector = ".q-slider"
        # 슬라이더의 범위 확인
        min_value = int(await page.get_attribute(slider_selector, "aria-valuemin"))
        max_value = int(await page.get_attribute(slider_selector, "aria-valuemax"))
    
        if target_value < min_value or target_value > max_value:
            raise ValueError(f"Target value {target_value} is out of bounds ({min_value}, {max_value})")
    
        # 슬라이더 thumb 요소와 위치 가져오기
        slider_thumb = await page.query_selector(f"{slider_selector} .q-slider__thumb")
        thumb_box = await slider_thumb.bounding_box()
        
        # 슬라이더의 현재 위치 및 목표 위치 계산
        track = await page.query_selector(f"{slider_selector} .q-slider__track")
        track_box = await track.bounding_box()
        
        # 목표 x 위치 계산 (슬라이더 트랙 내에서 비율 기반)
        target_percentage = (target_value - min_value) / (max_value - min_value)
        target_x = track_box["x"] + target_percentage * track_box["width"]
        current_x = thumb_box["x"] + thumb_box["width"] / 2
        y = thumb_box["y"] + thumb_box["height"] / 2
    
        # 슬라이더를 드래그하여 목표 위치로 이동
        await page.mouse.move(current_x, y)
        await page.mouse.down()
        await page.mouse.move(target_x, y, steps=10)  # 부드럽게 이동
        await page.mouse.up()
    
        #print(f"슬라이더를 {target_value}로 이동 완료.")
    
        # 슬라이더 값 확인 및 강제 이벤트 트리거
        await page.evaluate(
            f"""
            (slider) => {{
                slider.dispatchEvent(new Event('input'));  // oninput 이벤트 트리거
                slider.dispatchEvent(new Event('change'));  // onchange 이벤트 트리거
            }}
            """,
            await page.query_selector(slider_selector)
        )
    
        # 이동 후 값 확인
        current_value = await page.get_attribute(slider_selector, "aria-valuenow")
        if current_value != str(target_value):
            print(f"경고: 슬라이더 값이 예상과 다릅니다. 현재 값: {current_value}, 예상 값: {target_value}")

    async def get_second_status_element(self, page):
        """
        두 번째 'Status' 요소에 접근하여 반환하는 메소드
        """
        status_elements = await page.query_selector_all("text=Status")
        if len(status_elements) > 1:
            second_status_element = status_elements[1]
            #print("두 번째 'Status' 요소에 접근합니다.")
            return second_status_element
        else:
            #print("두 번째 'Status' 요소가 존재하지 않습니다.")
            return None

    async def extract_level_status(self, page, identity_id, level):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 레벨에 따라 HP와 Defense가 변한다.
        hp_value = await self.extract_hp(second_status_element)
        defense_value = await self.extract_defense(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Level={level}, HP={hp_value}, Defense={defense_value}")
        self.container.add_level_stat(identity_id,level, hp_value, defense_value)
    
    async def extract_hp(self, status_element):
        # HP 추출
        hp_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].children[0].nextElementSibling.textContent.trim()"
        )
        return hp_value
    
    async def extract_defense(self, status_element):
        # Defense 추출
        defense_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return defense_value

    async def extract_uptie_status(self, page, identity_id, uptie):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 uptie에 따라 speed는 변한다.
        speed_value = await self.extract_speed(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Uptie={uptie}, Speed={speed_value}")
        self.container.add_uptie_stat(identity_id,uptie, speed_value)

    async def extract_speed(self, status_element):
        # Speed 추출
        speed_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return speed_value

    async def extract_resist_type(self, page, image_src, depth):
        # 이미지 요소 찾기
        #print(image_src)
        image = await page.query_selector(f'img[src="{image_src}"]')
        #print(image)
        if not image:
            return "Unknown"
    
        # 이미지의 부모 요소를 찾아서 올라감
        parent = image
        for _ in range(depth):
            parent = await parent.evaluate_handle('node => node.parentElement')
            if not parent:
                return "Unknown"
    
        # 부모의 형제 요소 찾기
        sibling = await parent.evaluate_handle('node => node.nextElementSibling')
        
        if sibling:
            value = await sibling.evaluate('node => node.textContent')
            if value:
                return value.strip()
        
        return "Unknown"

    async def extract_stagger_threshold_values(self, page):
        # 'div.row.center.flex-center.q-pt-xs' 요소들 찾기
        elements = await page.query_selector_all('div.row.center.flex-center.q-pt-xs')
        
        # 각 요소의 텍스트 내용 출력하여 'Stagger Threshold' 요소를 찾기
        for element in elements:
            text = await element.text_content()
            
            if text and "Stagger Threshold" in text:
                # 'Stagger Threshold'가 있는 요소를 찾았다면, 그 이후 형제 요소들에서 값을 추출
                parent = await element.evaluate_handle('node => node.parentElement')
                sibling_elements = await parent.query_selector_all('div.row.justify-evenly div')  # Threshold 값이 들어있는 div들
                
                stagger_values = []
                for sibling in sibling_elements:
                    text_content = await sibling.text_content()
                    match = re.search(r'(\d+%)', text_content)
                    if match:
                        stagger_values.append(match.group(1))  # 퍼센트 값만 리스트에 추가
                while len(stagger_values) < 3:
                    stagger_values.append(np.nan)
                #print(stagger_values)
                return stagger_values  # 추출된 값(또는 np.nan 포함)을 반환
        
        return [np.nan, np.nan, np.nan]  # 만약 'Stagger Threshold'를 찾지 못한 경우 기본값 반환

    async def crawl_skills(self, page, identity_id, uptie_value):
        await self.crawl_skill(page, identity_id,"skill 1", uptie_value)
        await self.crawl_skill(page, identity_id,"skill 2", uptie_value)
        #if uptie_value >= 3: # 나중에 스킬 관련된 정보를 전부 긁어오는 것이 확인되면 적용한다. 동기화 3 이상만 스킬3 정보를 긁어온다.
        await self.crawl_skill(page, identity_id,"skill 3", uptie_value)
        await self.crawl_skill(page, identity_id,"defense", uptie_value)

    async def crawl_skill(self, page, identity_id, skill_type, uptie_value):
        skills_elements = await page.query_selector_all('div.skilltag')
        for skill_element in skills_elements:
            skill_text = await skill_element.text_content()
            if skill_text.strip().lower() == skill_type:
                skill_name = await self.skill_name(skill_element)
                coin_count = await self.coin_count(skill_element,".col-auto > .column > .row")
                init_level = await self.init_level(skill_element)
                init_remain = await self.init_remain(skill_element, skill_type, "div.row.justify-start > div.attr.flex.flex-center > div.q-px-sm")
                atk_weight = await self.atk_weight(skill_element, skill_type)
                self.container.add_skill(identity_id, skill_type, skill_name, coin_count, init_level, init_remain, atk_weight)
                #이 밑으로 동기화와 관련된 로직을 추가하면 될 듯.
                skill_data = await self.skill_and_coin_power(skill_element)
                skill_power = skill_data.get("skillPower")
                coin_power = skill_data.get("coinPower")
                self.container.add_skill_uptie_info(identity_id, skill_name, skill_power, coin_power, uptie_value)
                print(f"{skill_type}:{skill_name} skill_power: {skill_power}, coin_power: {coin_power}, coin_count: {coin_count}, init_remain : {init_remain}, init_level: {init_level}, atk_wieght: {atk_weight}")

    async def skill_and_coin_power(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                // 조부모로 이동
                const grandparent = node.parentElement.parentElement;
                // 이전 형제 탐색
                const previousSibling = grandparent.previousElementSibling;
                if (!previousSibling) return {{ skillPower: null, coinPower: null }}; // 이전 형제가 없으면 기본값 반환
                // 이전 형제 내부의 .row.justify-around > div > div 탐색
                const targetDivs = previousSibling.querySelectorAll(".row.justify-around > div");
                if (targetDivs.length < 3) return {{ skillPower: null, coinPower: null }}; // 배열 길이 확인
                // 1st div: 스킬 위력
                const skillPower = targetDivs[0]?.querySelector('div')?.textContent?.trim() || null; 
                // 3rd div: 코인 파워 값 (3번째 div에서 텍스트 추출)
                const coinPower = targetDivs[2]?.textContent?.trim() || null;
                return {{ skillPower, coinPower }};
            }}
        """)


    async def coin_count(self, skill_element, hierarchy_selector):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector('{hierarchy_selector}');
                if (!targetElement) return 0;
                return targetElement.childElementCount;
            }}
        """)

    async def init_remain(self, skill_element, skill_type, hierarchy_selector):
        if skill_type == "defense":
            return -1
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector('{hierarchy_selector}');
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def skill_name(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector(".block");
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def init_level(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector("div.text-bold.text-h6.h6.q-pl-xs.q-pr-md");
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def atk_weight(self, skill_element, skill_type):
        if skill_type == "defense":
            return 0
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector("div.text-bold.text-h6.h6.q-pl-xs.q-pr-md");
                const doubleNextSibling = targetElement.nextElementSibling.nextElementSibling;
                if (!doubleNextSibling) return 0;
                return doubleNextSibling.textContent?.trim();
            }}
        """)

class Identity:
    """인격 정보를 저장하는 클래스"""
    def __init__(self, identity_id, name=None, rarity=None,\
                 skill_1_type=None, skill_1_sin_affinity=None,\
                 skill_2_type=None, skill_2_sin_affinity=None,\
                 skill_3_type=None, skill_3_sin_affinity=None,\
                 defense_type=None, defense_sin_affinity=None):
        self.identity_id = identity_id
        self.name = name
        self.rarity = rarity
        # 선언만 하는 부분 
        self.skill_1_type = skill_1_type 
        self.skill_1_sin_affinity = skill_1_sin_affinity 
        self.skill_2_type = skill_2_type 
        self.skill_2_sin_affinity = skill_2_sin_affinity 
        self.skill_3_type = skill_3_type 
        self.skill_3_sin_affinity = skill_3_sin_affinity 
        self.defense_type = defense_type 
        self.defense_sin_affinity = defense_sin_affinity
        self.level_max = 50
        self.uptie_max = 4
        self.level_stats = []  # LevelStat 객체 리스트
        self.uptie_stats = [] # UptieStat 객체 리스트
        self.resist_info = None
        self.stagger_threshold = None
        self.skillset = SkillSet()  # Skill정보를 담고 있는 skillset 객체 
        self.passive_infos = []  # 패시브 객체를 담고 있는 배열
        self.support_passive_infos = [] # 서포트 패시 객체를 담고 있는 배열
        self.panic_type = [] # 혼란상태 발생시 나타나는 효과를 담고 있는 객체 배열

    def update_name(self, name):
        self.name = name

    def update_rarity(self, rarity):
        self.rarity = rarity

    def add_level_stat(self, level, hp, defense):
        self.level_stats.append(LevelStat(level, hp, defense))

    def add_uptie_stat(self, uptie, speed):
        self.uptie_stats.append(UptieStat(uptie, speed))

    def add_resist_info(self, resist_hit, resist_penetrate, resist_slash):
        self.resist_info = ResistInfo(resist_hit, resist_penetrate, resist_slash)

    def add_stagger_threshold(self, first_stagger, second_stagger, third_stagger):
        self.stagger_threshold = StaggerThreshold(first_stagger, second_stagger, third_stagger)

    def add_skill(self, skill_code, skill_name, coin_count, init_level, init_remain, akt_weight):
        self.skillset.add_skill(skill_code, Skill(skill_code, skill_name, coin_count, init_level, init_remain, akt_weight))
        
    def add_skill_uptie_info(self, skill_name, uptie_level, skill_power, coin_power):
        self.skillset.add_skill_uptie_info(skill_name, uptie_level, skill_power, coin_power)
        
    def update_details(self, page):
        # 세부 정보 업데이트 로직 (가정)
        pass
    #def init_uptie(self, page):
    #    init_uptie = 

# LevelStat 객체 정의
class LevelStat:
    def __init__(self, level, hp=None, defense=None):
        self.level = level
        self.hp = hp
        self.defense = defense
    def update_level(self, level):
        self.level = level
    def update_hp(self, hp):
        self.hp = hp
    def update_defense(self, defense):
        self.defense = defense

# UptieStat 객체 정의
class UptieStat:
    def __init__(self, uptie, speed=None):
        self.uptie = uptie
        self.speed = speed
    def update_speed(self, speed):
        self.speed = speed

# ResistInfo 객체 정의
class ResistInfo:
    def __init__(self, resist_hit=None, resist_penetrate=None, resist_slash=None):
        self.resist_hit = resist_hit
        self.resist_penetrate = resist_penetrate
        self.resist_slash = resist_slash

# ResistInfo 객체 정의
class StaggerThreshold:
    def __init__(self, first_stagger=None, second_stagger=None, third_stagger=None):
        self.first_stagger = first_stagger
        self.second_stagger = second_stagger
        self.third_stagger = third_stagger

class SkillSet:
    def __init__(self):

        self.skill_1_type = ""
        self.skill_2_type = ""
        self.skill_3_type = ""
        self.defense_skill_type = ""

        self.skill_1_sin_affinity = ""
        self.skill_2_sin_affinity = ""
        self.skill_3_sin_affinity = ""
        self.defense_skill_sin_affinity = ""

        self.skill_1_details = []
        self.skill_2_details = []
        self.skill_3_details = []
        self.defense_skill_details = []

    def set_skill_type(self, skill_code, skill_type, sin_affinity):
        if skill_code == "skill 1":
            self.skill_1_type = skill_type
            self.skill_1_sin_affinity = sin_affinity
        elif skill_code == "skill 2":
            self.skill_2_type = skill_type
            self.skill_2_sin_affinity = sin_affinity
        elif skill_code == "skill 3":
            self.skill_3_type = skill_type
            self.skill_3_sin_affinity = sin_affinity
        elif skill_type == "defense":
            self.defense_skill_type = skill_type
            self.defense_skill_sin_affinity = sin_affinity
    
    def add_skill(self, skill_code, skill):
        if skill_code == "skill 1":
            self.skill_1_details.append(skill)
        elif skill_code == "skill 2":
            self.skill_2_details.append(skill)
        elif skill_code == "skill 3":
            self.skill_3_details.append(skill)
        elif skill_code == "defense":
            self.defense_skill_details.append(skill)

    def get_skill_by_name(self, skill_name):
        for skill_list in [self.skill_1_details, self.skill_2_details, self.skill_3_details, self.defense_skill_details]:
            for skill in skill_list:
                result = skill.is_skill_by_name(skill_name)
                if result:
                    return result
        return None

    def add_skill_uptie_info(self, skill_name, uptie_level, skill_power, coin_power):
        skill = self.get_skill_by_name(skill_name)
        skill.add_uptie_info(uptie_level, skill_power, coin_power)
        

class Skill:
    def __init__(self, skill_code, skill_name, coin_count, init_level, init_remain, akt_weight):
        self.skill_code = skill_code # "Skill 1"/"Skill 2"/"Skill 3"/"Defense" '''o'''
        self.skill_name = skill_name # 스킬 이름 '''o'''
        self.coin_count = coin_count # 동전 개수 '''o'''
        self.init_level = init_level # 스킬이 초기 배정받은 레벨, 인격 레벨에 - 1 한 값을 더해주면 스킬의 현 level을 구할 수 있다. '''o'''
        """
        남아있는 스킬 개수 사용하면, 하나씩 줄어들고 모든 스킬을 사용하면 이 값으로 돌아간다.
        스킬의 남은 숫자없는 스킬은 강화된 스킬로 한번 사용되면 다 조건을 채울 때까 사용 못한다.
        강화된 스킬을 None으로 설정하는 방식과 조건이 채워지면 1로 늘리는 방식이 존재한다.
        스킬을 사용할 수 있는 초기값이다.
        조건을 충족한 강화한 스킬을 이례적인 절차라고 생각하고 None으로 초기값을 설정했다.
        """
        self.init_remain = None #'''o'''
        self.atk_weight = None #공격 가중치 값이다. 방어스킬에는 없기 때문에 None으로 설정했다. '''o'''
        self.remain_count = init_remain # 실제 스킬이 작동하며 횟수가 차감되 나중에 모두 차감되면 다음 턴에 다시 초기횟수 리필된다.
        """
        동기화 단계에 따른 정보를 담는다. 현재 4동기화까지 되어 있지만 차후 확장될 것을 생각하여 고정하지 않는다.
        내부에 들어있는 정보로는 uptie_level, skill_power, coin_power 속성으로 들어있다.
        전투, 턴, 스킬, 코인토스 단계로 내려오며 발생하는 효과에 관하여 저장한다.
        """
        self.uptie_info = []  

    def is_skill_by_name(self, skill_name):
        if self.skill_name == skill_name:
            return self
        return None
        
    def add_uptie_info(self, uptie_level, skill_power, coin_power):
        self.uptie_info.append(UptieInfo(uptie_level, skill_power, coin_power))

class UptieInfo:
    def __init__(self, uptie_level, skill_power, coin_power):
        self.uptie_level = uptie_level   #'''o'''
        self.skill_power = skill_power   #'''o'''
        self.coin_power = coin_power     #'''o'''
        self.skill_effects = []

class Effect:
    def __init__(self, condition, description=None):
        self.condition = condition # 조건 텍스트 "[coin 1 on hit]"
        self.description = description # 효과 설명 텍스 "Inflict +1 Rupture Count

# 추상화된 EffectType 클래스
class EffectType:
    def __init__(self, name, description):
        self.name = name  # Effect 이름
        self.description = description  # Effect 설명

# Passive 클래스
class Passive(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)  # condition이 있는지 여부 확인

# SupportPassive 클래스
class SupportPassive(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)

# PanicType 클래스
class PanicType(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)

# Keyword 클래스
class Keyword:
    def __init__(self, keyword, explain=None, dispellable=None):
        self.keyword = keyword
        self.explain = explain
        self.dispellable = dispellable

      
class DataContainer:
    """크롤링 데이터를 관리하는 클래스"""
    def __init__(self):
        self.identities = []

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.level_max

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.uptie_max

    def update_identity_name(self, identity_id, identity_name):
        """특정 ID의 이름을 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_name(identity_name)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_identity_rarity(self, identity_id, rarity):
        """특정 ID의 희귀도를 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_rarity(rarity)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_skill(self, identity_id, skill_number, new_type):
        """
        특정 identity의 skill_type을 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_type: 새 skill_type
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_type = new_type
                elif skill_number == 2:
                    identity.skill_2_type = new_type
                elif skill_number == 3:
                    identity.skill_3_type = new_type
                elif skill_number == "defense":
                    identity.defense_type = new_type
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def update_sin_affinity(self, identity_id, skill_number, new_affinity):
        """
        특정 identity의 sin_affinity를 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_affinity: 새 sin_affinity
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_sin_affinity = new_affinity
                elif skill_number == 2:
                    identity.skill_2_sin_affinity = new_affinity
                elif skill_number == 3:
                    identity.skill_3_sin_affinity = new_affinity
                elif skill_number == "defense":
                    identity.defense_sin_affinity = new_affinity
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def add_identity(self, identity):
        """새로운 인격 데이터를 추가."""
        self.identities.append(identity)

    def add_level_stat(self, identity_id, level, hp, defense):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_level_stat(level, hp, defense)

    def add_uptie_stat(self, identity_id, uptie, speed):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_uptie_stat(uptie, speed)

    def add_resist_info(self, identity_id, resist_hit, resist_penetrate, resist_slash):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_resist_info(resist_hit, resist_penetrate, resist_slash)

    def add_stagger_threshold(self, identity_id, first_stagger, second_stagger, third_stagger):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_stagger_threshold(first_stagger, second_stagger, third_stagger)

    
    def add_skill(self, identity_id, skill_code, skill_name, coin_count, init_level, init_remain, akt_weight):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_code == "skill 1":
                    identity.skillset.set_skill_type(skill_code, identity.skill_1_type, identity.skill_1_sin_affinity)
                elif skill_code == "skill 2":
                    identity.skillset.set_skill_type(skill_code, identity.skill_2_type, identity.skill_2_sin_affinity)
                elif skill_code == "skill 3":
                    identity.skillset.set_skill_type(skill_code, identity.skill_3_type, identity.skill_3_sin_affinity)
                elif skill_code == "defense":
                    identity.skillset.set_skill_type(skill_code, identity.defense_type, identity.defense_sin_affinity)
                identity.add_skill(skill_code, skill_name, coin_count, init_level, init_remain, akt_weight)

    def add_skill_uptie_info(self, identity_id, skill_name, uptie_level, skill_power, coin_power):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_skill_uptie_info(skill_name, uptie_level, skill_power, coin_power)
        
    def display_identities(self):
        """Identity, ID, Name, Rarity 정보를 출력"""
        if not self.identities:
            print("No identities found.")
        else:
            # 데이터를 리스트로 변환
            table = []
            for idx, identity in enumerate(self.identities, 1):
                table.append([
                    idx, identity.identity_id, identity.name, identity.rarity,
                    identity.skill_1_type, identity.skill_1_sin_affinity,
                    identity.skill_2_type, identity.skill_2_sin_affinity,
                    identity.skill_3_type, identity.skill_3_sin_affinity,
                    identity.defense_type, identity.defense_sin_affinity
                ])
            
            # 테이블 헤더 정의
            headers = [
                "Index", "ID", "Name", "Rarity",
                "Skill_1 Type", "Skill_1 Sin", "Skill_2 Type", "Skill_2 Sin",
                "Skill_3 Type", "Skill_3 Sin", "Defense Type", "Defense Sin"
            ]
            
            # 테이블 출력
            print(tabulate(table, headers=headers, tablefmt="fancy_grid"))

    def export_to_csv(self, filename, fieldnames, data):
        """
        공통적으로 사용 가능한 CSV 내보내기 메서드
        :param filename: 내보낼 파일 이름 (예: 'identity.csv', 'skill.csv')
        :param fieldnames: CSV의 헤더 리스트
        :param data: CSV로 내보낼 데이터 (리스트 형태)
        """
        try:
            # 현재작업 디렉토 기준으 'csv'폴더 없으면 생성
            if not os.path.exists('./csv'):
                os.makedirs('./csv')
            # 파일 경로 설정 (상대경로 'csv' 폴더 안에 저장)
            file_path = os.path.join('./csv', filename)
            with open(file_path, mode='w', newline='', encoding='utf-8') as file:
                writer = csv.DictWriter(file, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(data)
            print(f"Data successfully exported to {filename}")
        except Exception as e:
            print(f"Error exporting to CSV: {e}")

    def get_identity_data(self):
        """
        Identity 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "ID": identity.identity_id,
                "Name": identity.name,
                "Rarity": identity.rarity,
                "Skill_1_Type": identity.skill_1_type,
                "Skill_1_Sin_Affinity": identity.skill_1_sin_affinity,
                "Skill_2_Type": identity.skill_2_type,
                "Skill_2_Sin_Affinity": identity.skill_2_sin_affinity,
                "Skill_3_Type": identity.skill_3_type,
                "Skill_3_Sin_Affinity": identity.skill_3_sin_affinity,
                "Defense_Type": identity.defense_type,
                "Defense_Sin_Affinity": identity.defense_sin_affinity,
            }
            for identity in self.identities
        ]
        
    def export_identities_data(self, filename="identities.csv"):
        """Identity 데이터를 CSV로 내보내는 메서드"""
        identity_fieldnames = [
            "ID", "Name", "Rarity",
            "Skill_1_Type", "Skill_1_Sin_Affinity",
            "Skill_2_Type", "Skill_2_Sin_Affinity",
            "Skill_3_Type", "Skill_3_Sin_Affinity",
            "Defense_Type", "Defense_Sin_Affinity"
        ]
        identity_data = self.get_identity_data()  # Identity 데이터 가져오기
        self.export_to_csv(filename, identity_fieldnames, identity_data)  # CSV로 내보내기

    def load_identities_by_id(self, filename, identity_ids):
        """지정된 identity_ids 목록에 해당하는 Identity 데이터를 CSV에서 읽어옴."""
        identity_ids_set = set(identity_ids) # 중복 제거를 위 Set 사용
        with open(filename, mode='r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                if row["ID"] in identity_ids_set:
                    identity = Identity(
                        identity_id=row["ID"],
                        name=row["Name"],
                        rarity=row["Rarity"],
                        skill_1_type=row["Skill_1_Type"],
                        skill_1_sin_affinity=row["Skill_1_Sin_Affinity"],
                        skill_2_type=row["Skill_2_Type"],
                        skill_2_sin_affinity=row["Skill_2_Sin_Affinity"],
                        skill_3_type=row["Skill_3_Type"],
                        skill_3_sin_affinity=row["Skill_3_Sin_Affinity"],
                        defense_type=row["Defense_Type"],
                        defense_sin_affinity=row["Defense_Sin_Affinity"]
                    )
                    self.identities.append(identity)

    def get_level_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Level": level_stat.level,
                "HP": level_stat.hp,
                "Defense": level_stat.defense,
            }
            for identity in self.identities
            for level_stat in identity.level_stats
        ]

    def export_level_stats_data(self, filename="levelstats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        level_stats_fieldnames = ["Identity_ID", "Level", "HP", "Defense"]
        level_stats_data = self.get_level_stats_data()  # level_stats 데이터 가져오기
        self.export_to_csv(filename, level_stats_fieldnames, level_stats_data)  # CSV로 내보내기

    def get_uptie_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Uptie": uptie_stat.uptie,
                "Speed": uptie_stat.speed,
            }
            for identity in self.identities
            for uptie_stat in identity.uptie_stats
        ]

    def export_uptie_stats_data(self, filename="Uptiestats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        uptie_stats_fieldnames = ["Identity_ID", "Uptie", "Speed"]
        uptie_stats_data = self.get_uptie_stats_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, uptie_stats_fieldnames, uptie_stats_data)  # CSV로 내보내기

    def get_resist_info_data(self):
        """
        모든 Identity의 resist_info 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Resist_Hit": identity.resist_info.resist_hit,
                "Resist_Penetrate": identity.resist_info.resist_penetrate,
                "Resist_Slash": identity.resist_info.resist_slash,
            }
            for identity in self.identities
        ]

    def export_resist_info_data(self, filename="ResistInfos.csv"):
        """ResistInfo 데이터를 CSV로 내보내는 메서드"""
        resist_infos_fieldnames = ["Identity_ID", "Resist_Hit", "Resist_Penetrate", "Resist_Slash"]
        resist_infos_data = self.get_resist_info_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, resist_infos_fieldnames, resist_infos_data)  # CSV로 내보내기

    def export_uptie_stats_data(self, filename="Uptiestats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        uptie_stats_fieldnames = ["Identity_ID", "Uptie", "Speed"]
        uptie_stats_data = self.get_uptie_stats_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, uptie_stats_fieldnames, uptie_stats_data)  # CSV로 내보내기

    def get_stagger_threshold_data(self):
        """
        모든 Identity의 stagger_threshold 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "First_Stagger": identity.stagger_threshold.first_stagger,
                "Second_Stagger": identity.stagger_threshold.second_stagger,
                "Third_Stagger": identity.stagger_threshold.third_stagger,
            }
            for identity in self.identities
        ]

    def export_stagger_threshold_data(self, filename="StaggerThresholds.csv"):
        """StaggerThresholds 데이터를 CSV로 내보내는 메서드"""
        stagger_thresholds_fieldnames = ["Identity_ID", "First_Stagger", "Second_Stagger", "Third_Stagger"]
        stagger_thresholds_data = self.get_stagger_threshold_data()  # stagger_thresholds 데이터 가져오기
        self.export_to_csv(filename, stagger_thresholds_fieldnames, stagger_thresholds_data)  # CSV로 내보내기


async def main():
    data_container = DataContainer()
    scraper = Scraper(container=data_container, base_url="https://limbus.kusoge.xyz", img_url="https://img.kusoge.xyz/limbus/img")
    await scraper.scrape()

await main() 

skill 1:Begone... skill_power: 2, coin_power: +4, coin_count: 2, init_remain : 3x, init_level: 2, atk_wieght: 1
skill 2:In Finely Ground Mistfall skill_power: 3, coin_power: +4, coin_count: 3, init_remain : 2x, init_level: 3, atk_wieght: 1
skill 3:The Festival Will End skill_power: 3, coin_power: +4, coin_count: 3, init_remain : 1x, init_level: 4, atk_wieght: 1
skill 3:Ascendant Don Quixote Hardblood Arts - The Finale skill_power: 3, coin_power: +5, coin_count: 3, init_remain : 0, init_level: 6, atk_wieght: 3
defense:Don Quixote Hardblood Arts 15: Parasol skill_power: 4, coin_power: +4, coin_count: 2, init_remain : -1, init_level: 3, atk_wieght: 0


In [13]:
import os
import csv
import re
import numpy as np
import pandas as pd
from playwright.async_api import async_playwright
from tabulate import tabulate

# Sin Color 이름 변환을 위한 매핑
resource_mapping = {
    "AZURE": "Gloom",
    "VIOLET": "Envy",
    "AMBER": "Sloth",
    "SHAMROCK": "Gluttony",
    "CRIMSON": "Wrath",
    "SCARLET": "Lust",
    "INDIGO": "Pride"
}

def convert_resource(resource_name):
    """Sin Color 이름을 변환하는 함수. 매핑되지 않은 경우 원래 이름을 반환."""
    return resource_mapping.get(resource_name, resource_name)

class Scraper:
    """웹 페이지 크롤링을 위한 클래스"""
    def __init__(self, container, base_url, img_url):
        self.container = container
        self.base_url = base_url
        self.img_url = img_url
        self.uptie_level = 1

    async def handle_request(self, route, request):
        """이미지와 폰트 리소스 로드를 차단."""
        if request.resource_type in ["image", "font"]:
            await route.abort()
        else:
            await route.continue_()
            
    async def scrape(self):
        """크롤링 메인 함수"""
        async with async_playwright() as p:
            browser = await p.chromium.launch(headless=True)
            page = await browser.new_page()
            await page.route("**/*", self.handle_request)
            # Step 1: /identity 페이지에서 기본 정보 크롤링
            #await self.crawl_identity_page(page)

            # Step 2: /identity/{identity_id} 상세 페이지 크롤링
            #identity_ids_to_load = ["10104", "10310", "10710","10808", "10911", "11005"]
            #identity_ids_to_load = ["10104", "10911"]#,"10310", "10710","10808", "11005", "11210"]
            identity_ids_to_load = ["10911"]#,"10310", "10710","10808", "11005", "11210"]
            self.container.load_identities_by_id('./csv/identities.csv', identity_ids_to_load)
            await self.crawl_identity_details_for_all(page)
            #self.container.display_identities()
            #self.container.export_identities_data("identities.csv")
            #self.container.export_level_stats_data("Levelstats.csv")
            #self.container.export_uptie_stats_data("Uptiestats.csv")
            #self.container.export_resist_info_data("ResistInfos.csv")
            #self.container.export_stagger_threshold_data("StaggerThresholds.csv")
            page.set_default_timeout(10000)
            #await page.wait_for_timeout(100000)
            await browser.close()

    async def get_identity_id_from_row(self, row):
        """테이블 행(row)에서 인격 ID를 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_url = await identity_cell.get_attribute('href')
            identity_id = identity_url.split('/identity/')[-1]
            return identity_id
        return "Unknown"

    async def get_identity_name_from_row(self, row):
        """테이블 행(row)에서 인격 이름을 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_name = (await identity_cell.inner_text()).replace("\n", " ").strip()
            return identity_name
        return "Unknown"
        
    async def crawl_identity_page(self, page):
        """/identity 페이지로 이동하여 인격 기본 정보를 추출."""
        await page.goto(self.base_url + "/identity")
        await self.extract_identity_page(page)

    async def extract_identity_page(self, page):
        """/identity 페이지의 모든 테이블 행에서 인격 정보를 추출."""
        identity_rows = await page.query_selector_all('tr')
        if len(identity_rows) > 1:
            for idx, identity_row in enumerate(identity_rows):
                if idx == 0:
                    continue # 0번째요소이름을 보여주는 열은 건너뜀
                await self.scrape_identity_name_and_identity_id(identity_row)
                await self.extract_rarity(identity_row)
                for i in range(1, 4):  # 스킬1~3 정보 추출
                    await self.extract_atk_and_resource(identity_row, i)
                await self.extract_defense_and_resource(identity_row)

    async def scrape_identity_name_and_identity_id(self, row):
        """인격 번호와 이름을 추출하여 DataContainer에 저장."""
        try:
            identity_id = await self.get_identity_id_from_row(row)
            identity_name = await self.get_identity_name_from_row(row)
            identity = Identity(identity_id)
            self.container.add_identity(identity)
            self.container.update_identity_name(identity_id, identity_name)
        except Exception as e:
            print(f"Error extracting identity name and identity id: {e}")

    async def get_element_attribute(self, element, selector, attribute):
        """요소에서 지정된 속성(attribute)을 추출."""
        try:
            target_element = await element.query_selector(selector)
            if target_element:
                return await target_element.get_attribute(attribute)
        except Exception as e:
            print(f"Error getting element attribute: {e}")
        return None
    
    async def extract_rarity(self, row):
        """테이블 행(row)에서 희귀도를 추출하여 업데이트."""
        try:
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
                sibling = await parent.evaluate_handle('node => node.nextElementSibling')
                if sibling:
                    rarity_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/rarity_"]', 'src')
                    if rarity_img_src:
                        rarity_level = rarity_img_src.split('_')[-1].split('.')[0]  # rarity_1.png -> 1
                        identity_id = await self.get_identity_id_from_row(row)
                        self.container.update_identity_rarity(identity_id, rarity_level)
                    else:
                        print("Rarity image not found in the next sibling TD.")
        except Exception as e: 
            print(f"Error extracting rarity: {e}")

        return "Unknown"

    async def extract_atk_and_resource(self, row, skill_number):
        """ 공격 타입(ATK Type)과 자원(Attribute) 추출 함수 - 스킬 번호에 따라 처리 """
        try:
            #print(f"Attempting to locate skill{skill_number} type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 해당 스킬 번호에 맞는 형제로 이동
                sibling = parent
                for _ in range(skill_number + 1):  # 스킬 번호에 맞게 이동
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                atk_type, resource = "Unknown", "Unknown"
        
                if sibling:
                    # 공격 타입 이미지 (atk_type) 찾기
                    atk_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/atk_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if atk_type_img_src:
                        atk_type = atk_type_img_src.split('_')[-1].split('.')[0]  # atk_type_SLASH.png -> SLASH
                        #print(f"Extracted attack type: {atk_type}")
                        self.container.update_skill(identity_id, skill_number, atk_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted resource: {resource}")
                        
                        # 리소스를 변환
                        resource = convert_resource(resource)
                        #print(f"Converted resource: {resource}")
                        self.container.update_sin_affinity(identity_id, skill_number, resource)
                        
        except Exception as e:
            print(f"Error extracting skill{skill_number} type and resource: {e}")

    async def extract_defense_and_resource(self, row):
        """ 방어 타입(Def Type)과 자원(Attribute) 추출 함수 """
        try:
            #print("Attempting to locate defense type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 다섯 번째 형제 td로 이동 (방어 관련 정보가 있는 <td>)
                sibling = parent
                for _ in range(5):  # 방어 정보는 5번째 형제에 있음
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                def_type, def_resource = "Unknown", "Unknown"
        
                if sibling:
                    # 방어 타입 이미지 (def_type) 찾기
                    def_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/def_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if def_type_img_src:
                        def_type = def_type_img_src.split('_')[-1].split('.')[0]  # def_type_GUARD.png -> GUARD
                        #print(f"Extracted defense type: {def_type}")
                        self.container.update_skill(identity_id, 'defense', def_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        def_resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted defense resource: {def_resource}")
                        
                        # 방어 리소스를 변환
                        def_resource = convert_resource(def_resource)
                        #print(f"Converted defense resource: {def_resource}")
                        self.container.update_sin_affinity(identity_id, 'defense', def_resource)
 
        except Exception as e:
            print(f"Error extracting defense type and resource: {e}")

    async def crawl_identity_details_for_all(self, page):
        """모든 인격의 세부 정보를 크롤링."""
        for identity in self.container.identities:
            await page.wait_for_load_state("networkidle")
            await page.goto(self.base_url + "/identity/"+identity.identity_id)
            """ 레벨과 동기화와 관련 없는 참관타 저항은 밑에서 변경하는 것과 연관성이 적기 때문에 위에서 처리한다.
            print(f"Identity_id: {identity.identity_id}")
            results = []
            resist_type_imgs = [f"{self.img_url}/ui/atk_type_HIT.png",
                                f"{self.img_url}/ui/atk_type_PENETRATE.png",
                                f"{self.img_url}/ui/atk_type_SLASH.png"]
            for resist_type_img in resist_type_imgs:
                result = await self.extract_resist_type(page, resist_type_img, 1)
                results.append(result)
            self.container.add_resist_info(identity.identity_id, results[0], results[1], results[2])
            """
            """
            # Stagger Threshold 값 추출
            stagger_threshold = await self.extract_stagger_threshold_values(page)
            self.container.add_stagger_threshold(identity.identity_id, stagger_threshold[0], stagger_threshold[1], stagger_threshold[2])
            """
            await self.initialize_uptie_and_level(page)
            """ 레벨에 따른 level, hp, defense 다르게 되어 있는 것을 출력 확인 객체로 들어가서 csv로 옮기는 과정까지 진행
            for i in range(1, identity.level_max+1):
                await self.move_slider_to_value(page,i)
                await self.extract_level_status(page, identity.identity_id, i)
            """
            """ 동기화에 따른 speed 다르게 되어 있는 것을 출력 확인 객체로 들어가서 csv로 옮기는 과정까지 진행
            for i in range(1, identity.uptie_max+1):
                await self.uptie_click(page,i)
                await self.extract_uptie_status(page, identity.identity_id, i)
            """
            await self.crawl_skills(page, identity.identity_id)
            
    async def initialize_uptie_and_level(self, page):
        await self.uptie_click(page, 1)
        await self.move_slider_to_value(page, 1)
        self.uptie_level = 1
        #return 1
    
    async def uptie_click(self, page, phase):
        fab_button_selector = ".q-btn.q-btn--fab"
        await page.wait_for_selector(fab_button_selector)
        await page.click(fab_button_selector)
        #print(f"[{setup_uptie_button}] 상위 동기화 버튼 클릭 완료")
        setup_uptie_button = f'a:has(img[src="{self.img_url}/ui/uptie_{phase}.png"])'
        await page.wait_for_selector(setup_uptie_button)
        await page.click(setup_uptie_button)
        #print(f"{phase} 동기화 버튼 클릭 완료")

    async def move_slider_to_value(self, page, target_value):
        """
        슬라이더를 주어진 값으로 이동시킵니다.
        target_value는 슬라이더의 최소값부터 최대값 사이여야 합니다.
        """
        slider_selector = ".q-slider"
        # 슬라이더의 범위 확인
        min_value = int(await page.get_attribute(slider_selector, "aria-valuemin"))
        max_value = int(await page.get_attribute(slider_selector, "aria-valuemax"))
    
        if target_value < min_value or target_value > max_value:
            raise ValueError(f"Target value {target_value} is out of bounds ({min_value}, {max_value})")
    
        # 슬라이더 thumb 요소와 위치 가져오기
        slider_thumb = await page.query_selector(f"{slider_selector} .q-slider__thumb")
        thumb_box = await slider_thumb.bounding_box()
        
        # 슬라이더의 현재 위치 및 목표 위치 계산
        track = await page.query_selector(f"{slider_selector} .q-slider__track")
        track_box = await track.bounding_box()
        
        # 목표 x 위치 계산 (슬라이더 트랙 내에서 비율 기반)
        target_percentage = (target_value - min_value) / (max_value - min_value)
        target_x = track_box["x"] + target_percentage * track_box["width"]
        current_x = thumb_box["x"] + thumb_box["width"] / 2
        y = thumb_box["y"] + thumb_box["height"] / 2
    
        # 슬라이더를 드래그하여 목표 위치로 이동
        await page.mouse.move(current_x, y)
        await page.mouse.down()
        await page.mouse.move(target_x, y, steps=10)  # 부드럽게 이동
        await page.mouse.up()
    
        #print(f"슬라이더를 {target_value}로 이동 완료.")
    
        # 슬라이더 값 확인 및 강제 이벤트 트리거
        await page.evaluate(
            f"""
            (slider) => {{
                slider.dispatchEvent(new Event('input'));  // oninput 이벤트 트리거
                slider.dispatchEvent(new Event('change'));  // onchange 이벤트 트리거
            }}
            """,
            await page.query_selector(slider_selector)
        )
    
        # 이동 후 값 확인
        current_value = await page.get_attribute(slider_selector, "aria-valuenow")
        if current_value != str(target_value):
            print(f"경고: 슬라이더 값이 예상과 다릅니다. 현재 값: {current_value}, 예상 값: {target_value}")

    async def get_second_status_element(self, page):
        """
        두 번째 'Status' 요소에 접근하여 반환하는 메소드
        """
        status_elements = await page.query_selector_all("text=Status")
        if len(status_elements) > 1:
            second_status_element = status_elements[1]
            #print("두 번째 'Status' 요소에 접근합니다.")
            return second_status_element
        else:
            #print("두 번째 'Status' 요소가 존재하지 않습니다.")
            return None

    async def extract_level_status(self, page, identity_id, level):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 레벨에 따라 HP와 Defense가 변한다.
        hp_value = await self.extract_hp(second_status_element)
        defense_value = await self.extract_defense(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Level={level}, HP={hp_value}, Defense={defense_value}")
        self.container.add_level_stat(identity_id,level, hp_value, defense_value)
    
    async def extract_hp(self, status_element):
        # HP 추출
        hp_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].children[0].nextElementSibling.textContent.trim()"
        )
        return hp_value
    
    async def extract_defense(self, status_element):
        # Defense 추출
        defense_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return defense_value

    async def extract_uptie_status(self, page, identity_id, uptie):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 uptie에 따라 speed는 변한다.
        speed_value = await self.extract_speed(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Uptie={uptie}, Speed={speed_value}")
        self.container.add_uptie_stat(identity_id,uptie, speed_value)

    async def extract_speed(self, status_element):
        # Speed 추출
        speed_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return speed_value

    async def extract_resist_type(self, page, image_src, depth):
        # 이미지 요소 찾기
        #print(image_src)
        image = await page.query_selector(f'img[src="{image_src}"]')
        #print(image)
        if not image:
            return "Unknown"
    
        # 이미지의 부모 요소를 찾아서 올라감
        parent = image
        for _ in range(depth):
            parent = await parent.evaluate_handle('node => node.parentElement')
            if not parent:
                return "Unknown"
    
        # 부모의 형제 요소 찾기
        sibling = await parent.evaluate_handle('node => node.nextElementSibling')
        
        if sibling:
            value = await sibling.evaluate('node => node.textContent')
            if value:
                return value.strip()
        
        return "Unknown"

    async def extract_stagger_threshold_values(self, page):
        # 'div.row.center.flex-center.q-pt-xs' 요소들 찾기
        elements = await page.query_selector_all('div.row.center.flex-center.q-pt-xs')
        
        # 각 요소의 텍스트 내용 출력하여 'Stagger Threshold' 요소를 찾기
        for element in elements:
            text = await element.text_content()
            
            if text and "Stagger Threshold" in text:
                # 'Stagger Threshold'가 있는 요소를 찾았다면, 그 이후 형제 요소들에서 값을 추출
                parent = await element.evaluate_handle('node => node.parentElement')
                sibling_elements = await parent.query_selector_all('div.row.justify-evenly div')  # Threshold 값이 들어있는 div들
                
                stagger_values = []
                for sibling in sibling_elements:
                    text_content = await sibling.text_content()
                    match = re.search(r'(\d+%)', text_content)
                    if match:
                        stagger_values.append(match.group(1))  # 퍼센트 값만 리스트에 추가
                while len(stagger_values) < 3:
                    stagger_values.append(np.nan)
                #print(stagger_values)
                return stagger_values  # 추출된 값(또는 np.nan 포함)을 반환
        
        return [np.nan, np.nan, np.nan]  # 만약 'Stagger Threshold'를 찾지 못한 경우 기본값 반환

    async def crawl_skills(self, page, identity_id):
        await self.crawl_skill(page, identity_id, "skill 1")
        await self.crawl_skill(page, identity_id, "skill 2")
        await self.crawl_skill(page, identity_id, "skill 3")
        await self.crawl_skill(page, identity_id, "defense")


    async def crawl_skill(self, page, identity_id, skill_type):
        skills_elements = await page.query_selector_all('div.skilltag')
        for skill_element in skills_elements:
            skill_text = await skill_element.text_content()
            if skill_text.strip().lower() == skill_type:
                await self.process_skill(skill_element, page, identity_id, skill_type)
        
    async def process_skill(self, skill_element, page, identity_id, skill_type):
        # skill_type이 "skill 3"일 경우 uptie_value가 3 이상일 때만 정보를 저장
        for uptie_value in range(1, 5):  # Uptie levels 1 to 4
            # "skill 3"은 uptie_value가 3 이상일 때만 처리
            if skill_type == "skill 3" and uptie_value < 3:
                continue  # skip if uptie_value is less than 3 for skill 3
    
            # uptie_level에 맞게 클릭
            await self.uptie_click(page, uptie_value)
            if skill_type != "skill 3" and uptie_value > 1:
                # Uptie 정보 갱신
                await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
                continue
            elif skill_type == "skill 3" and uptie_value > 3:
                # Uptie 정보 갱신
                await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
                continue
            # 스킬의 세부 정보를 추출
            skill_details = await self.extract_skill_details(skill_element, skill_type)
            skill_name = skill_details["skill_name"]
    
            # 스킬 정보를 저장
            self.container.add_skill(
                identity_id, skill_type, 
                skill_name, skill_details["coin_count"],
                skill_details["init_level"], skill_details["init_remain"], 
                skill_details["atk_weight"]
            )
            print(f"Added skill: {skill_type}:{skill_name}")
    
            # Uptie 정보 갱신
            await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
    
    async def extract_skill_details(self, skill_element, skill_type):
        # 스킬 세부 정보를 추출하는 함수
        skill_name = await self.skill_name(skill_element)
        coin_count = await self.coin_count(skill_element, ".col-auto > .column > .row")
        init_level = await self.init_level(skill_element)
        init_remain = await self.init_remain(skill_element, skill_type, "div.row.justify-start > div.attr.flex.flex-center > div.q-px-sm")
        atk_weight = await self.atk_weight(skill_element, skill_type)
        return {
            "skill_name": skill_name,
            "coin_count": coin_count,
            "init_level": init_level,
            "init_remain": init_remain,
            "atk_weight": atk_weight
        }
    
    async def update_uptie_info(self, page, skill_element, identity_id, skill_name, skill_type, uptie_value):
        # Uptie 정보를 추출하고 업데이트하는 함수
        uptie_info = await self.extract_uptie_info(skill_element)
        self.container.add_skill_uptie_info(
            identity_id, skill_name, 
            uptie_info["skill_power"], uptie_info["coin_power"], 
            uptie_value
        )
        print(
            f"Updated Uptie for {skill_name} (Uptie Level: {uptie_value}): "
            f"Skill Power: {uptie_info['skill_power']}, Coin Power: {uptie_info['coin_power']}"
        )
        effects = await self.extract_skill_effect(skill_element)

    async def extract_uptie_info(self, skill_element):
        skill_data = await self.skill_and_coin_power(skill_element)
        return {
            "skill_power": skill_data.get("skillPower"),
            "coin_power": skill_data.get("coinPower")
        }

    async def extract_skill_effect(self, skill_element):
        # Create the Effect object
        effect_obj = Effect(condition=None)

        # Extract the condition
        condition_elements = await self.skill_condition(skill_element)
        print(condition_elements)

    async def skill_and_coin_power(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                // 조부모로 이동
                const grandparent = node.parentElement.parentElement;
                // 이전 형제 탐색
                const previousSibling = grandparent.previousElementSibling;
                if (!previousSibling) return {{ skillPower: null, coinPower: null }}; // 이전 형제가 없으면 기본값 반환
                // 이전 형제 내부의 .row.justify-around > div > div 탐색
                const targetDivs = previousSibling.querySelectorAll(".row.justify-around > div");
                if (targetDivs.length < 3) return {{ skillPower: null, coinPower: null }}; // 배열 길이 확인
                // 1st div: 스킬 위력
                const skillPower = targetDivs[0]?.querySelector('div')?.textContent?.trim() || null; 
                // 3rd div: 코인 파워 값 (3번째 div에서 텍스트 추출)
                const coinPower = targetDivs[2]?.textContent?.trim() || null;
                return {{ skillPower, coinPower }};
            }}
        """)


    async def coin_count(self, skill_element, hierarchy_selector):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector('{hierarchy_selector}');
                if (!targetElement) return 0;
                return targetElement.childElementCount;
            }}
        """)

    async def init_remain(self, skill_element, skill_type, hierarchy_selector):
        if skill_type == "defense":
            return -1
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector('{hierarchy_selector}');
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def skill_name(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector(".block");
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def init_level(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector("div.text-bold.text-h6.h6.q-pl-xs.q-pr-md");
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def atk_weight(self, skill_element, skill_type):
        if skill_type == "defense":
            return 0
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector("div.text-bold.text-h6.h6.q-pl-xs.q-pr-md");
                const doubleNextSibling = targetElement.nextElementSibling.nextElementSibling;
                if (!doubleNextSibling) return 0;
                return doubleNextSibling.textContent?.trim();
            }}
        """)

    async def skill_condition(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const doubleNextSibling = parent.nextElementSibling.nextElementSibling;
                if (!doubleNextSibling) return null;
                const targetElements = doubleNextSibling.querySelectorAll("div.col>div>div>span.q-pr-xs");
                if (!targetElements || targetElements.length === 0) return null;
                return Array.from(targetElements).map(el => el.textContent?.trim() || "").filter(text => text.length> 0);
            }}
        """)

class Identity:
    """인격 정보를 저장하는 클래스"""
    def __init__(self, identity_id, name=None, rarity=None,\
                 skill_1_type=None, skill_1_sin_affinity=None,\
                 skill_2_type=None, skill_2_sin_affinity=None,\
                 skill_3_type=None, skill_3_sin_affinity=None,\
                 defense_type=None, defense_sin_affinity=None):
        self.identity_id = identity_id
        self.name = name
        self.rarity = rarity
        # 선언만 하는 부분 
        self.skill_1_type = skill_1_type 
        self.skill_1_sin_affinity = skill_1_sin_affinity 
        self.skill_2_type = skill_2_type 
        self.skill_2_sin_affinity = skill_2_sin_affinity 
        self.skill_3_type = skill_3_type 
        self.skill_3_sin_affinity = skill_3_sin_affinity 
        self.defense_type = defense_type 
        self.defense_sin_affinity = defense_sin_affinity
        self.level_max = 50
        self.uptie_max = 4
        self.level_stats = []  # LevelStat 객체 리스트
        self.uptie_stats = [] # UptieStat 객체 리스트
        self.resist_info = None
        self.stagger_threshold = None
        self.skillset = SkillSet()  # Skill정보를 담고 있는 skillset 객체 
        self.passive_infos = []  # 패시브 객체를 담고 있는 배열
        self.support_passive_infos = [] # 서포트 패시 객체를 담고 있는 배열
        self.panic_type = [] # 혼란상태 발생시 나타나는 효과를 담고 있는 객체 배열

    def update_name(self, name):
        self.name = name

    def update_rarity(self, rarity):
        self.rarity = rarity

    def add_level_stat(self, level, hp, defense):
        self.level_stats.append(LevelStat(level, hp, defense))

    def add_uptie_stat(self, uptie, speed):
        self.uptie_stats.append(UptieStat(uptie, speed))

    def add_resist_info(self, resist_hit, resist_penetrate, resist_slash):
        self.resist_info = ResistInfo(resist_hit, resist_penetrate, resist_slash)

    def add_stagger_threshold(self, first_stagger, second_stagger, third_stagger):
        self.stagger_threshold = StaggerThreshold(first_stagger, second_stagger, third_stagger)

    def add_skill(self, skill_code, skill_name, coin_count, init_level, init_remain, akt_weight):
        self.skillset.add_skill(skill_code, Skill(skill_code, skill_name, coin_count, init_level, init_remain, akt_weight))
        
    def add_skill_uptie_info(self, skill_name, uptie_level, skill_power, coin_power):
        self.skillset.add_skill_uptie_info(skill_name, uptie_level, skill_power, coin_power)
        
    def update_details(self, page):
        # 세부 정보 업데이트 로직 (가정)
        pass
    #def init_uptie(self, page):
    #    init_uptie = 

# LevelStat 객체 정의
class LevelStat:
    def __init__(self, level, hp=None, defense=None):
        self.level = level
        self.hp = hp
        self.defense = defense
    def update_level(self, level):
        self.level = level
    def update_hp(self, hp):
        self.hp = hp
    def update_defense(self, defense):
        self.defense = defense

# UptieStat 객체 정의
class UptieStat:
    def __init__(self, uptie, speed=None):
        self.uptie = uptie
        self.speed = speed
    def update_speed(self, speed):
        self.speed = speed

# ResistInfo 객체 정의
class ResistInfo:
    def __init__(self, resist_hit=None, resist_penetrate=None, resist_slash=None):
        self.resist_hit = resist_hit
        self.resist_penetrate = resist_penetrate
        self.resist_slash = resist_slash

# ResistInfo 객체 정의
class StaggerThreshold:
    def __init__(self, first_stagger=None, second_stagger=None, third_stagger=None):
        self.first_stagger = first_stagger
        self.second_stagger = second_stagger
        self.third_stagger = third_stagger

class SkillSet:
    def __init__(self):

        self.skill_1_type = ""
        self.skill_2_type = ""
        self.skill_3_type = ""
        self.defense_skill_type = ""

        self.skill_1_sin_affinity = ""
        self.skill_2_sin_affinity = ""
        self.skill_3_sin_affinity = ""
        self.defense_skill_sin_affinity = ""

        self.skill_1_details = []
        self.skill_2_details = []
        self.skill_3_details = []
        self.defense_skill_details = []

    def set_skill_type(self, skill_code, skill_type, sin_affinity):
        if skill_code == "skill 1":
            self.skill_1_type = skill_type
            self.skill_1_sin_affinity = sin_affinity
        elif skill_code == "skill 2":
            self.skill_2_type = skill_type
            self.skill_2_sin_affinity = sin_affinity
        elif skill_code == "skill 3":
            self.skill_3_type = skill_type
            self.skill_3_sin_affinity = sin_affinity
        elif skill_type == "defense":
            self.defense_skill_type = skill_type
            self.defense_skill_sin_affinity = sin_affinity
    
    def add_skill(self, skill_code, skill):
        if skill_code == "skill 1":
            self.skill_1_details.append(skill)
        elif skill_code == "skill 2":
            self.skill_2_details.append(skill)
        elif skill_code == "skill 3":
            self.skill_3_details.append(skill)
        elif skill_code == "defense":
            self.defense_skill_details.append(skill)

    def get_skill_by_name(self, skill_name):
        for skill_list in [self.skill_1_details, self.skill_2_details, self.skill_3_details, self.defense_skill_details]:
            for skill in skill_list:
                result = skill.is_skill_by_name(skill_name)
                if result:
                    return result
        return None

    def add_skill_uptie_info(self, skill_name, uptie_level, skill_power, coin_power):
        skill = self.get_skill_by_name(skill_name)
        skill.add_uptie_info(uptie_level, skill_power, coin_power)
        

class Skill:
    def __init__(self, skill_code, skill_name, coin_count, init_level, init_remain, akt_weight):
        self.skill_code = skill_code # "Skill 1"/"Skill 2"/"Skill 3"/"Defense" '''o'''
        self.skill_name = skill_name # 스킬 이름 '''o'''
        self.coin_count = coin_count # 동전 개수 '''o'''
        self.init_level = init_level # 스킬이 초기 배정받은 레벨, 인격 레벨에 - 1 한 값을 더해주면 스킬의 현 level을 구할 수 있다. '''o'''
        """
        남아있는 스킬 개수 사용하면, 하나씩 줄어들고 모든 스킬을 사용하면 이 값으로 돌아간다.
        스킬의 남은 숫자없는 스킬은 강화된 스킬로 한번 사용되면 다 조건을 채울 때까 사용 못한다.
        강화된 스킬을 None으로 설정하는 방식과 조건이 채워지면 1로 늘리는 방식이 존재한다.
        스킬을 사용할 수 있는 초기값이다.
        조건을 충족한 강화한 스킬을 이례적인 절차라고 생각하고 None으로 초기값을 설정했다.
        """
        self.init_remain = None #'''o'''
        self.atk_weight = None #공격 가중치 값이다. 방어스킬에는 없기 때문에 None으로 설정했다. '''o'''
        self.remain_count = init_remain # 실제 스킬이 작동하며 횟수가 차감되 나중에 모두 차감되면 다음 턴에 다시 초기횟수 리필된다.
        """
        동기화 단계에 따른 정보를 담는다. 현재 4동기화까지 되어 있지만 차후 확장될 것을 생각하여 고정하지 않는다.
        내부에 들어있는 정보로는 uptie_level, skill_power, coin_power 속성으로 들어있다.
        전투, 턴, 스킬, 코인토스 단계로 내려오며 발생하는 효과에 관하여 저장한다.
        """
        self.uptie_info = []  

    def is_skill_by_name(self, skill_name):
        if self.skill_name == skill_name:
            return self
        return None
        
    def add_uptie_info(self, uptie_level, skill_power, coin_power):
        self.uptie_info.append(UptieInfo(uptie_level, skill_power, coin_power))

class UptieInfo:
    def __init__(self, uptie_level, skill_power, coin_power):
        self.uptie_level = uptie_level   #'''o'''
        self.skill_power = skill_power   #'''o'''
        self.coin_power = coin_power     #'''o'''
        self.skill_effects = []

class Effect:
    def __init__(self, condition, description=None):
        self.condition = condition # 조건 텍스트 "[coin 1 on hit]"
        self.description = description # 효과 설명 텍스 "Inflict +1 Rupture Count

# 추상화된 EffectType 클래스
class EffectType:
    def __init__(self, name, description):
        self.name = name  # Effect 이름
        self.description = description  # Effect 설명

# Passive 클래스
class Passive(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)  # condition이 있는지 여부 확인

# SupportPassive 클래스
class SupportPassive(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)

# PanicType 클래스
class PanicType(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)

# Keyword 클래스
class Keyword:
    def __init__(self, keyword, explain=None, dispellable=None):
        self.keyword = keyword
        self.explain = explain
        self.dispellable = dispellable

      
class DataContainer:
    """크롤링 데이터를 관리하는 클래스"""
    def __init__(self):
        self.identities = []

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.level_max

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.uptie_max

    def update_identity_name(self, identity_id, identity_name):
        """특정 ID의 이름을 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_name(identity_name)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_identity_rarity(self, identity_id, rarity):
        """특정 ID의 희귀도를 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_rarity(rarity)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_skill(self, identity_id, skill_number, new_type):
        """
        특정 identity의 skill_type을 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_type: 새 skill_type
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_type = new_type
                elif skill_number == 2:
                    identity.skill_2_type = new_type
                elif skill_number == 3:
                    identity.skill_3_type = new_type
                elif skill_number == "defense":
                    identity.defense_type = new_type
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def update_sin_affinity(self, identity_id, skill_number, new_affinity):
        """
        특정 identity의 sin_affinity를 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_affinity: 새 sin_affinity
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_sin_affinity = new_affinity
                elif skill_number == 2:
                    identity.skill_2_sin_affinity = new_affinity
                elif skill_number == 3:
                    identity.skill_3_sin_affinity = new_affinity
                elif skill_number == "defense":
                    identity.defense_sin_affinity = new_affinity
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def add_identity(self, identity):
        """새로운 인격 데이터를 추가."""
        self.identities.append(identity)

    def add_level_stat(self, identity_id, level, hp, defense):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_level_stat(level, hp, defense)

    def add_uptie_stat(self, identity_id, uptie, speed):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_uptie_stat(uptie, speed)

    def add_resist_info(self, identity_id, resist_hit, resist_penetrate, resist_slash):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_resist_info(resist_hit, resist_penetrate, resist_slash)

    def add_stagger_threshold(self, identity_id, first_stagger, second_stagger, third_stagger):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_stagger_threshold(first_stagger, second_stagger, third_stagger)

    
    def add_skill(self, identity_id, skill_code, skill_name, coin_count, init_level, init_remain, akt_weight):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_code == "skill 1":
                    identity.skillset.set_skill_type(skill_code, identity.skill_1_type, identity.skill_1_sin_affinity)
                elif skill_code == "skill 2":
                    identity.skillset.set_skill_type(skill_code, identity.skill_2_type, identity.skill_2_sin_affinity)
                elif skill_code == "skill 3":
                    identity.skillset.set_skill_type(skill_code, identity.skill_3_type, identity.skill_3_sin_affinity)
                elif skill_code == "defense":
                    identity.skillset.set_skill_type(skill_code, identity.defense_type, identity.defense_sin_affinity)
                identity.add_skill(skill_code, skill_name, coin_count, init_level, init_remain, akt_weight)

    def add_skill_uptie_info(self, identity_id, skill_name, uptie_level, skill_power, coin_power):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_skill_uptie_info(skill_name, uptie_level, skill_power, coin_power)
        
    def display_identities(self):
        """Identity, ID, Name, Rarity 정보를 출력"""
        if not self.identities:
            print("No identities found.")
        else:
            # 데이터를 리스트로 변환
            table = []
            for idx, identity in enumerate(self.identities, 1):
                table.append([
                    idx, identity.identity_id, identity.name, identity.rarity,
                    identity.skill_1_type, identity.skill_1_sin_affinity,
                    identity.skill_2_type, identity.skill_2_sin_affinity,
                    identity.skill_3_type, identity.skill_3_sin_affinity,
                    identity.defense_type, identity.defense_sin_affinity
                ])
            
            # 테이블 헤더 정의
            headers = [
                "Index", "ID", "Name", "Rarity",
                "Skill_1 Type", "Skill_1 Sin", "Skill_2 Type", "Skill_2 Sin",
                "Skill_3 Type", "Skill_3 Sin", "Defense Type", "Defense Sin"
            ]
            
            # 테이블 출력
            print(tabulate(table, headers=headers, tablefmt="fancy_grid"))

    def export_to_csv(self, filename, fieldnames, data):
        """
        공통적으로 사용 가능한 CSV 내보내기 메서드
        :param filename: 내보낼 파일 이름 (예: 'identity.csv', 'skill.csv')
        :param fieldnames: CSV의 헤더 리스트
        :param data: CSV로 내보낼 데이터 (리스트 형태)
        """
        try:
            # 현재작업 디렉토 기준으 'csv'폴더 없으면 생성
            if not os.path.exists('./csv'):
                os.makedirs('./csv')
            # 파일 경로 설정 (상대경로 'csv' 폴더 안에 저장)
            file_path = os.path.join('./csv', filename)
            with open(file_path, mode='w', newline='', encoding='utf-8') as file:
                writer = csv.DictWriter(file, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(data)
            print(f"Data successfully exported to {filename}")
        except Exception as e:
            print(f"Error exporting to CSV: {e}")

    def get_identity_data(self):
        """
        Identity 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "ID": identity.identity_id,
                "Name": identity.name,
                "Rarity": identity.rarity,
                "Skill_1_Type": identity.skill_1_type,
                "Skill_1_Sin_Affinity": identity.skill_1_sin_affinity,
                "Skill_2_Type": identity.skill_2_type,
                "Skill_2_Sin_Affinity": identity.skill_2_sin_affinity,
                "Skill_3_Type": identity.skill_3_type,
                "Skill_3_Sin_Affinity": identity.skill_3_sin_affinity,
                "Defense_Type": identity.defense_type,
                "Defense_Sin_Affinity": identity.defense_sin_affinity,
            }
            for identity in self.identities
        ]
        
    def export_identities_data(self, filename="identities.csv"):
        """Identity 데이터를 CSV로 내보내는 메서드"""
        identity_fieldnames = [
            "ID", "Name", "Rarity",
            "Skill_1_Type", "Skill_1_Sin_Affinity",
            "Skill_2_Type", "Skill_2_Sin_Affinity",
            "Skill_3_Type", "Skill_3_Sin_Affinity",
            "Defense_Type", "Defense_Sin_Affinity"
        ]
        identity_data = self.get_identity_data()  # Identity 데이터 가져오기
        self.export_to_csv(filename, identity_fieldnames, identity_data)  # CSV로 내보내기

    def load_identities_by_id(self, filename, identity_ids):
        """지정된 identity_ids 목록에 해당하는 Identity 데이터를 CSV에서 읽어옴."""
        identity_ids_set = set(identity_ids) # 중복 제거를 위 Set 사용
        with open(filename, mode='r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                if row["ID"] in identity_ids_set:
                    identity = Identity(
                        identity_id=row["ID"],
                        name=row["Name"],
                        rarity=row["Rarity"],
                        skill_1_type=row["Skill_1_Type"],
                        skill_1_sin_affinity=row["Skill_1_Sin_Affinity"],
                        skill_2_type=row["Skill_2_Type"],
                        skill_2_sin_affinity=row["Skill_2_Sin_Affinity"],
                        skill_3_type=row["Skill_3_Type"],
                        skill_3_sin_affinity=row["Skill_3_Sin_Affinity"],
                        defense_type=row["Defense_Type"],
                        defense_sin_affinity=row["Defense_Sin_Affinity"]
                    )
                    self.identities.append(identity)

    def get_level_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Level": level_stat.level,
                "HP": level_stat.hp,
                "Defense": level_stat.defense,
            }
            for identity in self.identities
            for level_stat in identity.level_stats
        ]

    def export_level_stats_data(self, filename="levelstats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        level_stats_fieldnames = ["Identity_ID", "Level", "HP", "Defense"]
        level_stats_data = self.get_level_stats_data()  # level_stats 데이터 가져오기
        self.export_to_csv(filename, level_stats_fieldnames, level_stats_data)  # CSV로 내보내기

    def get_uptie_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Uptie": uptie_stat.uptie,
                "Speed": uptie_stat.speed,
            }
            for identity in self.identities
            for uptie_stat in identity.uptie_stats
        ]

    def export_uptie_stats_data(self, filename="Uptiestats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        uptie_stats_fieldnames = ["Identity_ID", "Uptie", "Speed"]
        uptie_stats_data = self.get_uptie_stats_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, uptie_stats_fieldnames, uptie_stats_data)  # CSV로 내보내기

    def get_resist_info_data(self):
        """
        모든 Identity의 resist_info 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Resist_Hit": identity.resist_info.resist_hit,
                "Resist_Penetrate": identity.resist_info.resist_penetrate,
                "Resist_Slash": identity.resist_info.resist_slash,
            }
            for identity in self.identities
        ]

    def export_resist_info_data(self, filename="ResistInfos.csv"):
        """ResistInfo 데이터를 CSV로 내보내는 메서드"""
        resist_infos_fieldnames = ["Identity_ID", "Resist_Hit", "Resist_Penetrate", "Resist_Slash"]
        resist_infos_data = self.get_resist_info_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, resist_infos_fieldnames, resist_infos_data)  # CSV로 내보내기

    def export_uptie_stats_data(self, filename="Uptiestats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        uptie_stats_fieldnames = ["Identity_ID", "Uptie", "Speed"]
        uptie_stats_data = self.get_uptie_stats_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, uptie_stats_fieldnames, uptie_stats_data)  # CSV로 내보내기

    def get_stagger_threshold_data(self):
        """
        모든 Identity의 stagger_threshold 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "First_Stagger": identity.stagger_threshold.first_stagger,
                "Second_Stagger": identity.stagger_threshold.second_stagger,
                "Third_Stagger": identity.stagger_threshold.third_stagger,
            }
            for identity in self.identities
        ]

    def export_stagger_threshold_data(self, filename="StaggerThresholds.csv"):
        """StaggerThresholds 데이터를 CSV로 내보내는 메서드"""
        stagger_thresholds_fieldnames = ["Identity_ID", "First_Stagger", "Second_Stagger", "Third_Stagger"]
        stagger_thresholds_data = self.get_stagger_threshold_data()  # stagger_thresholds 데이터 가져오기
        self.export_to_csv(filename, stagger_thresholds_fieldnames, stagger_thresholds_data)  # CSV로 내보내기


async def main():
    data_container = DataContainer()
    scraper = Scraper(container=data_container, base_url="https://limbus.kusoge.xyz", img_url="https://img.kusoge.xyz/limbus/img")
    await scraper.scrape()

await main() 

Added skill: skill 1:Begone...
Updated Uptie for Begone... (Uptie Level: 1): Skill Power: 2, Coin Power: +4
['[Combat Start]', 'blooming[thornyfall_panic', 'blooming[thornyfall_panic']
Updated Uptie for Begone... (Uptie Level: 2): Skill Power: 2, Coin Power: +4
['[Combat Start]', 'blooming[thornyfall_panic', 'blooming[thornyfall_panic', '[On Use]']
Updated Uptie for Begone... (Uptie Level: 3): Skill Power: 3, Coin Power: +4
['[Combat Start]', 'blooming[thornyfall_panic', 'blooming[thornyfall_panic', '[On Use]']
Updated Uptie for Begone... (Uptie Level: 4): Skill Power: 3, Coin Power: +4
['[Combat Start]', 'blooming[thornyfall_panic', 'blooming[thornyfall_panic', '[On Use]']
Added skill: skill 2:In Finely Ground Mistfall
Updated Uptie for In Finely Ground Mistfall (Uptie Level: 1): Skill Power: 3, Coin Power: +4
['[Combat Start]', 'blooming[thornyfall_panic', 'blooming[thornyfall_panic']
Updated Uptie for In Finely Ground Mistfall (Uptie Level: 2): Skill Power: 3, Coin Power: +4
['[Comb

In [18]:
import os
import csv
import re
import numpy as np
import pandas as pd
from playwright.async_api import async_playwright
from tabulate import tabulate

# Sin Color 이름 변환을 위한 매핑
resource_mapping = {
    "AZURE": "Gloom",
    "VIOLET": "Envy",
    "AMBER": "Sloth",
    "SHAMROCK": "Gluttony",
    "CRIMSON": "Wrath",
    "SCARLET": "Lust",
    "INDIGO": "Pride"
}

def convert_resource(resource_name):
    """Sin Color 이름을 변환하는 함수. 매핑되지 않은 경우 원래 이름을 반환."""
    return resource_mapping.get(resource_name, resource_name)

class Scraper:
    """웹 페이지 크롤링을 위한 클래스"""
    def __init__(self, container, base_url, img_url):
        self.container = container
        self.base_url = base_url
        self.img_url = img_url
        self.uptie_level = 1

    async def handle_request(self, route, request):
        """이미지와 폰트 리소스 로드를 차단."""
        if request.resource_type in ["image", "font"]:
            await route.abort()
        else:
            await route.continue_()
            
    async def scrape(self):
        """크롤링 메인 함수"""
        async with async_playwright() as p:
            browser = await p.chromium.launch(headless=True)
            page = await browser.new_page()
            await page.route("**/*", self.handle_request)
            # Step 1: /identity 페이지에서 기본 정보 크롤링
            #await self.crawl_identity_page(page)

            # Step 2: /identity/{identity_id} 상세 페이지 크롤링
            #identity_ids_to_load = ["10104", "10310", "10710","10808", "10911", "11005"]
            #identity_ids_to_load = ["10104", "10911"]#,"10310", "10710","10808", "11005", "11210"]
            identity_ids_to_load = ["10911"]#,"10310", "10710","10808", "11005", "11210"]
            self.container.load_identities_by_id('./csv/identities.csv', identity_ids_to_load)
            await self.crawl_identity_details_for_all(page)
            #self.container.display_identities()
            #self.container.export_identities_data("identities.csv")
            #self.container.export_level_stats_data("Levelstats.csv")
            #self.container.export_uptie_stats_data("Uptiestats.csv")
            #self.container.export_resist_info_data("ResistInfos.csv")
            #self.container.export_stagger_threshold_data("StaggerThresholds.csv")
            page.set_default_timeout(10000)
            #await page.wait_for_timeout(100000)
            await browser.close()

    async def get_identity_id_from_row(self, row):
        """테이블 행(row)에서 인격 ID를 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_url = await identity_cell.get_attribute('href')
            identity_id = identity_url.split('/identity/')[-1]
            return identity_id
        return "Unknown"

    async def get_identity_name_from_row(self, row):
        """테이블 행(row)에서 인격 이름을 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_name = (await identity_cell.inner_text()).replace("\n", " ").strip()
            return identity_name
        return "Unknown"
        
    async def crawl_identity_page(self, page):
        """/identity 페이지로 이동하여 인격 기본 정보를 추출."""
        await page.goto(self.base_url + "/identity")
        await self.extract_identity_page(page)

    async def extract_identity_page(self, page):
        """/identity 페이지의 모든 테이블 행에서 인격 정보를 추출."""
        identity_rows = await page.query_selector_all('tr')
        if len(identity_rows) > 1:
            for idx, identity_row in enumerate(identity_rows):
                if idx == 0:
                    continue # 0번째요소이름을 보여주는 열은 건너뜀
                await self.scrape_identity_name_and_identity_id(identity_row)
                await self.extract_rarity(identity_row)
                for i in range(1, 4):  # 스킬1~3 정보 추출
                    await self.extract_atk_and_resource(identity_row, i)
                await self.extract_defense_and_resource(identity_row)

    async def scrape_identity_name_and_identity_id(self, row):
        """인격 번호와 이름을 추출하여 DataContainer에 저장."""
        try:
            identity_id = await self.get_identity_id_from_row(row)
            identity_name = await self.get_identity_name_from_row(row)
            identity = Identity(identity_id)
            self.container.add_identity(identity)
            self.container.update_identity_name(identity_id, identity_name)
        except Exception as e:
            print(f"Error extracting identity name and identity id: {e}")

    async def get_element_attribute(self, element, selector, attribute):
        """요소에서 지정된 속성(attribute)을 추출."""
        try:
            target_element = await element.query_selector(selector)
            if target_element:
                return await target_element.get_attribute(attribute)
        except Exception as e:
            print(f"Error getting element attribute: {e}")
        return None
    
    async def extract_rarity(self, row):
        """테이블 행(row)에서 희귀도를 추출하여 업데이트."""
        try:
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
                sibling = await parent.evaluate_handle('node => node.nextElementSibling')
                if sibling:
                    rarity_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/rarity_"]', 'src')
                    if rarity_img_src:
                        rarity_level = rarity_img_src.split('_')[-1].split('.')[0]  # rarity_1.png -> 1
                        identity_id = await self.get_identity_id_from_row(row)
                        self.container.update_identity_rarity(identity_id, rarity_level)
                    else:
                        print("Rarity image not found in the next sibling TD.")
        except Exception as e: 
            print(f"Error extracting rarity: {e}")

        return "Unknown"

    async def extract_atk_and_resource(self, row, skill_number):
        """ 공격 타입(ATK Type)과 자원(Attribute) 추출 함수 - 스킬 번호에 따라 처리 """
        try:
            #print(f"Attempting to locate skill{skill_number} type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 해당 스킬 번호에 맞는 형제로 이동
                sibling = parent
                for _ in range(skill_number + 1):  # 스킬 번호에 맞게 이동
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                atk_type, resource = "Unknown", "Unknown"
        
                if sibling:
                    # 공격 타입 이미지 (atk_type) 찾기
                    atk_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/atk_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if atk_type_img_src:
                        atk_type = atk_type_img_src.split('_')[-1].split('.')[0]  # atk_type_SLASH.png -> SLASH
                        #print(f"Extracted attack type: {atk_type}")
                        self.container.update_skill(identity_id, skill_number, atk_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted resource: {resource}")
                        
                        # 리소스를 변환
                        resource = convert_resource(resource)
                        #print(f"Converted resource: {resource}")
                        self.container.update_sin_affinity(identity_id, skill_number, resource)
                        
        except Exception as e:
            print(f"Error extracting skill{skill_number} type and resource: {e}")

    async def extract_defense_and_resource(self, row):
        """ 방어 타입(Def Type)과 자원(Attribute) 추출 함수 """
        try:
            #print("Attempting to locate defense type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 다섯 번째 형제 td로 이동 (방어 관련 정보가 있는 <td>)
                sibling = parent
                for _ in range(5):  # 방어 정보는 5번째 형제에 있음
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                def_type, def_resource = "Unknown", "Unknown"
        
                if sibling:
                    # 방어 타입 이미지 (def_type) 찾기
                    def_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/def_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if def_type_img_src:
                        def_type = def_type_img_src.split('_')[-1].split('.')[0]  # def_type_GUARD.png -> GUARD
                        #print(f"Extracted defense type: {def_type}")
                        self.container.update_skill(identity_id, 'defense', def_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        def_resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted defense resource: {def_resource}")
                        
                        # 방어 리소스를 변환
                        def_resource = convert_resource(def_resource)
                        #print(f"Converted defense resource: {def_resource}")
                        self.container.update_sin_affinity(identity_id, 'defense', def_resource)
 
        except Exception as e:
            print(f"Error extracting defense type and resource: {e}")

    async def crawl_identity_details_for_all(self, page):
        """모든 인격의 세부 정보를 크롤링."""
        for identity in self.container.identities:
            await page.wait_for_load_state("networkidle")
            await page.goto(self.base_url + "/identity/"+identity.identity_id)
            """ 레벨과 동기화와 관련 없는 참관타 저항은 밑에서 변경하는 것과 연관성이 적기 때문에 위에서 처리한다.
            print(f"Identity_id: {identity.identity_id}")
            results = []
            resist_type_imgs = [f"{self.img_url}/ui/atk_type_HIT.png",
                                f"{self.img_url}/ui/atk_type_PENETRATE.png",
                                f"{self.img_url}/ui/atk_type_SLASH.png"]
            for resist_type_img in resist_type_imgs:
                result = await self.extract_resist_type(page, resist_type_img, 1)
                results.append(result)
            self.container.add_resist_info(identity.identity_id, results[0], results[1], results[2])
            """
            """
            # Stagger Threshold 값 추출
            stagger_threshold = await self.extract_stagger_threshold_values(page)
            self.container.add_stagger_threshold(identity.identity_id, stagger_threshold[0], stagger_threshold[1], stagger_threshold[2])
            """
            await self.initialize_uptie_and_level(page)
            """ 레벨에 따른 level, hp, defense 다르게 되어 있는 것을 출력 확인 객체로 들어가서 csv로 옮기는 과정까지 진행
            for i in range(1, identity.level_max+1):
                await self.move_slider_to_value(page,i)
                await self.extract_level_status(page, identity.identity_id, i)
            """
            """ 동기화에 따른 speed 다르게 되어 있는 것을 출력 확인 객체로 들어가서 csv로 옮기는 과정까지 진행
            for i in range(1, identity.uptie_max+1):
                await self.uptie_click(page,i)
                await self.extract_uptie_status(page, identity.identity_id, i)
            """
            await self.crawl_skills(page, identity.identity_id)
            
    async def initialize_uptie_and_level(self, page):
        await self.uptie_click(page, 1)
        await self.move_slider_to_value(page, 1)
        self.uptie_level = 1
        #return 1
    
    async def uptie_click(self, page, phase):
        fab_button_selector = ".q-btn.q-btn--fab"
        await page.wait_for_selector(fab_button_selector)
        await page.click(fab_button_selector)
        #print(f"[{setup_uptie_button}] 상위 동기화 버튼 클릭 완료")
        setup_uptie_button = f'a:has(img[src="{self.img_url}/ui/uptie_{phase}.png"])'
        await page.wait_for_selector(setup_uptie_button)
        await page.click(setup_uptie_button)
        #print(f"{phase} 동기화 버튼 클릭 완료")

    async def move_slider_to_value(self, page, target_value):
        """
        슬라이더를 주어진 값으로 이동시킵니다.
        target_value는 슬라이더의 최소값부터 최대값 사이여야 합니다.
        """
        slider_selector = ".q-slider"
        # 슬라이더의 범위 확인
        min_value = int(await page.get_attribute(slider_selector, "aria-valuemin"))
        max_value = int(await page.get_attribute(slider_selector, "aria-valuemax"))
    
        if target_value < min_value or target_value > max_value:
            raise ValueError(f"Target value {target_value} is out of bounds ({min_value}, {max_value})")
    
        # 슬라이더 thumb 요소와 위치 가져오기
        slider_thumb = await page.query_selector(f"{slider_selector} .q-slider__thumb")
        thumb_box = await slider_thumb.bounding_box()
        
        # 슬라이더의 현재 위치 및 목표 위치 계산
        track = await page.query_selector(f"{slider_selector} .q-slider__track")
        track_box = await track.bounding_box()
        
        # 목표 x 위치 계산 (슬라이더 트랙 내에서 비율 기반)
        target_percentage = (target_value - min_value) / (max_value - min_value)
        target_x = track_box["x"] + target_percentage * track_box["width"]
        current_x = thumb_box["x"] + thumb_box["width"] / 2
        y = thumb_box["y"] + thumb_box["height"] / 2
    
        # 슬라이더를 드래그하여 목표 위치로 이동
        await page.mouse.move(current_x, y)
        await page.mouse.down()
        await page.mouse.move(target_x, y, steps=10)  # 부드럽게 이동
        await page.mouse.up()
    
        #print(f"슬라이더를 {target_value}로 이동 완료.")
    
        # 슬라이더 값 확인 및 강제 이벤트 트리거
        await page.evaluate(
            f"""
            (slider) => {{
                slider.dispatchEvent(new Event('input'));  // oninput 이벤트 트리거
                slider.dispatchEvent(new Event('change'));  // onchange 이벤트 트리거
            }}
            """,
            await page.query_selector(slider_selector)
        )
    
        # 이동 후 값 확인
        current_value = await page.get_attribute(slider_selector, "aria-valuenow")
        if current_value != str(target_value):
            print(f"경고: 슬라이더 값이 예상과 다릅니다. 현재 값: {current_value}, 예상 값: {target_value}")

    async def get_second_status_element(self, page):
        """
        두 번째 'Status' 요소에 접근하여 반환하는 메소드
        """
        status_elements = await page.query_selector_all("text=Status")
        if len(status_elements) > 1:
            second_status_element = status_elements[1]
            #print("두 번째 'Status' 요소에 접근합니다.")
            return second_status_element
        else:
            #print("두 번째 'Status' 요소가 존재하지 않습니다.")
            return None

    async def extract_level_status(self, page, identity_id, level):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 레벨에 따라 HP와 Defense가 변한다.
        hp_value = await self.extract_hp(second_status_element)
        defense_value = await self.extract_defense(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Level={level}, HP={hp_value}, Defense={defense_value}")
        self.container.add_level_stat(identity_id,level, hp_value, defense_value)
    
    async def extract_hp(self, status_element):
        # HP 추출
        hp_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].children[0].nextElementSibling.textContent.trim()"
        )
        return hp_value
    
    async def extract_defense(self, status_element):
        # Defense 추출
        defense_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return defense_value

    async def extract_uptie_status(self, page, identity_id, uptie):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 uptie에 따라 speed는 변한다.
        speed_value = await self.extract_speed(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Uptie={uptie}, Speed={speed_value}")
        self.container.add_uptie_stat(identity_id,uptie, speed_value)

    async def extract_speed(self, status_element):
        # Speed 추출
        speed_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return speed_value

    async def extract_resist_type(self, page, image_src, depth):
        # 이미지 요소 찾기
        #print(image_src)
        image = await page.query_selector(f'img[src="{image_src}"]')
        #print(image)
        if not image:
            return "Unknown"
    
        # 이미지의 부모 요소를 찾아서 올라감
        parent = image
        for _ in range(depth):
            parent = await parent.evaluate_handle('node => node.parentElement')
            if not parent:
                return "Unknown"
    
        # 부모의 형제 요소 찾기
        sibling = await parent.evaluate_handle('node => node.nextElementSibling')
        
        if sibling:
            value = await sibling.evaluate('node => node.textContent')
            if value:
                return value.strip()
        
        return "Unknown"

    async def extract_stagger_threshold_values(self, page):
        # 'div.row.center.flex-center.q-pt-xs' 요소들 찾기
        elements = await page.query_selector_all('div.row.center.flex-center.q-pt-xs')
        
        # 각 요소의 텍스트 내용 출력하여 'Stagger Threshold' 요소를 찾기
        for element in elements:
            text = await element.text_content()
            
            if text and "Stagger Threshold" in text:
                # 'Stagger Threshold'가 있는 요소를 찾았다면, 그 이후 형제 요소들에서 값을 추출
                parent = await element.evaluate_handle('node => node.parentElement')
                sibling_elements = await parent.query_selector_all('div.row.justify-evenly div')  # Threshold 값이 들어있는 div들
                
                stagger_values = []
                for sibling in sibling_elements:
                    text_content = await sibling.text_content()
                    match = re.search(r'(\d+%)', text_content)
                    if match:
                        stagger_values.append(match.group(1))  # 퍼센트 값만 리스트에 추가
                while len(stagger_values) < 3:
                    stagger_values.append(np.nan)
                #print(stagger_values)
                return stagger_values  # 추출된 값(또는 np.nan 포함)을 반환
        
        return [np.nan, np.nan, np.nan]  # 만약 'Stagger Threshold'를 찾지 못한 경우 기본값 반환

    async def crawl_skills(self, page, identity_id):
        await self.crawl_skill(page, identity_id, "skill 1")
        await self.crawl_skill(page, identity_id, "skill 2")
        await self.crawl_skill(page, identity_id, "skill 3")
        await self.crawl_skill(page, identity_id, "defense")


    async def crawl_skill(self, page, identity_id, skill_type):
        skills_elements = await page.query_selector_all('div.skilltag')
        for skill_element in skills_elements:
            skill_text = await skill_element.text_content()
            if skill_text.strip().lower() == skill_type:
                await self.process_skill(skill_element, page, identity_id, skill_type)
        
    async def process_skill(self, skill_element, page, identity_id, skill_type):
        # skill_type이 "skill 3"일 경우 uptie_value가 3 이상일 때만 정보를 저장
        for uptie_value in range(1, 5):  # Uptie levels 1 to 4
            # "skill 3"은 uptie_value가 3 이상일 때만 처리
            if skill_type == "skill 3" and uptie_value < 3:
                continue  # skip if uptie_value is less than 3 for skill 3
    
            # uptie_level에 맞게 클릭
            await self.uptie_click(page, uptie_value)
            if skill_type != "skill 3" and uptie_value > 1:
                # Uptie 정보 갱신
                await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
                continue
            elif skill_type == "skill 3" and uptie_value > 3:
                # Uptie 정보 갱신
                await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
                continue
            # 스킬의 세부 정보를 추출
            skill_details = await self.extract_skill_details(skill_element, skill_type)
            skill_name = skill_details["skill_name"]
    
            # 스킬 정보를 저장
            self.container.add_skill(
                identity_id, skill_type, 
                skill_name, skill_details["coin_count"],
                skill_details["init_level"], skill_details["init_remain"], 
                skill_details["atk_weight"]
            )
            print(f"Added skill: {skill_type}:{skill_name}")
    
            # Uptie 정보 갱신
            await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
    
    async def extract_skill_details(self, skill_element, skill_type):
        # 스킬 세부 정보를 추출하는 함수
        skill_name = await self.skill_name(skill_element)
        coin_count = await self.coin_count(skill_element, ".col-auto > .column > .row")
        init_level = await self.init_level(skill_element)
        init_remain = await self.init_remain(skill_element, skill_type, "div.row.justify-start > div.attr.flex.flex-center > div.q-px-sm")
        atk_weight = await self.atk_weight(skill_element, skill_type)
        return {
            "skill_name": skill_name,
            "coin_count": coin_count,
            "init_level": init_level,
            "init_remain": init_remain,
            "atk_weight": atk_weight
        }
    
    async def update_uptie_info(self, page, skill_element, identity_id, skill_name, skill_type, uptie_value):
        # Uptie 정보를 추출하고 업데이트하는 함수
        uptie_info = await self.extract_uptie_info(skill_element)
        self.container.add_skill_uptie_info(
            identity_id, skill_name, 
            uptie_info["skill_power"], uptie_info["coin_power"], 
            uptie_value
        )
        print(
            f"Updated Uptie for {skill_name} (Uptie Level: {uptie_value}): "
            f"Skill Power: {uptie_info['skill_power']}, Coin Power: {uptie_info['coin_power']}"
        )
        effects = await self.extract_skill_effect(skill_element)

    async def extract_uptie_info(self, skill_element):
        skill_data = await self.skill_and_coin_power(skill_element)
        return {
            "skill_power": skill_data.get("skillPower"),
            "coin_power": skill_data.get("coinPower")
        }

    async def extract_skill_effect(self, skill_element):
        # Create the Effect object
        effect_obj = Effect(condition=None)

        # Extract the condition
        condition_elements = await self.skill_condition(skill_element)
        print(condition_elements)

    async def skill_and_coin_power(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                // 조부모로 이동
                const grandparent = node.parentElement.parentElement;
                // 이전 형제 탐색
                const previousSibling = grandparent.previousElementSibling;
                if (!previousSibling) return {{ skillPower: null, coinPower: null }}; // 이전 형제가 없으면 기본값 반환
                // 이전 형제 내부의 .row.justify-around > div > div 탐색
                const targetDivs = previousSibling.querySelectorAll(".row.justify-around > div");
                if (targetDivs.length < 3) return {{ skillPower: null, coinPower: null }}; // 배열 길이 확인
                // 1st div: 스킬 위력
                const skillPower = targetDivs[0]?.querySelector('div')?.textContent?.trim() || null; 
                // 3rd div: 코인 파워 값 (3번째 div에서 텍스트 추출)
                const coinPower = targetDivs[2]?.textContent?.trim() || null;
                return {{ skillPower, coinPower }};
            }}
        """)


    async def coin_count(self, skill_element, hierarchy_selector):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector('{hierarchy_selector}');
                if (!targetElement) return 0;
                return targetElement.childElementCount;
            }}
        """)

    async def init_remain(self, skill_element, skill_type, hierarchy_selector):
        if skill_type == "defense":
            return -1
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector('{hierarchy_selector}');
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def skill_name(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector(".block");
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def init_level(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector("div.text-bold.text-h6.h6.q-pl-xs.q-pr-md");
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def atk_weight(self, skill_element, skill_type):
        if skill_type == "defense":
            return 0
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector("div.text-bold.text-h6.h6.q-pl-xs.q-pr-md");
                const doubleNextSibling = targetElement.nextElementSibling.nextElementSibling;
                if (!doubleNextSibling) return 0;
                return doubleNextSibling.textContent?.trim();
            }}
        """)

    async def skill_condition(self, skill_element):
        return await skill_element.evaluate("""
            node => {
                const parent = node.parentElement;
                const doubleNextSibling = parent.nextElementSibling.nextElementSibling;
                if (!doubleNextSibling) return null;
    
                // 모든 매칭되는 요소 선택
                const targetElements = doubleNextSibling.querySelectorAll("div.col>div>div>span.q-pr-xs");
                if (!targetElements || targetElements.length === 0) return null;
    
                // 완전한 대괄호 쌍을 찾는 정규식 (열리고 닫힌 괄호가 있는 텍스트)
                const regex = /\\[[^\\]]+\\]/;
    
                return Array.from(targetElements)
                    .map(el => el.textContent?.trim())
                    .filter(text => text && regex.test(text))  // 완전한 대괄호 텍스트만 필터링
                    .map(text => {
                        const match = text.match(regex);
                        return match ? match[0] : null;  // 첫 번째 완전한 대괄호를 포함한 텍스트 반환
                    })
                    .filter(Boolean); // null 값 제거
            }
        """)


class Identity:
    """인격 정보를 저장하는 클래스"""
    def __init__(self, identity_id, name=None, rarity=None,\
                 skill_1_type=None, skill_1_sin_affinity=None,\
                 skill_2_type=None, skill_2_sin_affinity=None,\
                 skill_3_type=None, skill_3_sin_affinity=None,\
                 defense_type=None, defense_sin_affinity=None):
        self.identity_id = identity_id
        self.name = name
        self.rarity = rarity
        # 선언만 하는 부분 
        self.skill_1_type = skill_1_type 
        self.skill_1_sin_affinity = skill_1_sin_affinity 
        self.skill_2_type = skill_2_type 
        self.skill_2_sin_affinity = skill_2_sin_affinity 
        self.skill_3_type = skill_3_type 
        self.skill_3_sin_affinity = skill_3_sin_affinity 
        self.defense_type = defense_type 
        self.defense_sin_affinity = defense_sin_affinity
        self.level_max = 50
        self.uptie_max = 4
        self.level_stats = []  # LevelStat 객체 리스트
        self.uptie_stats = [] # UptieStat 객체 리스트
        self.resist_info = None
        self.stagger_threshold = None
        self.skillset = SkillSet()  # Skill정보를 담고 있는 skillset 객체 
        self.passive_infos = []  # 패시브 객체를 담고 있는 배열
        self.support_passive_infos = [] # 서포트 패시 객체를 담고 있는 배열
        self.panic_type = [] # 혼란상태 발생시 나타나는 효과를 담고 있는 객체 배열

    def update_name(self, name):
        self.name = name

    def update_rarity(self, rarity):
        self.rarity = rarity

    def add_level_stat(self, level, hp, defense):
        self.level_stats.append(LevelStat(level, hp, defense))

    def add_uptie_stat(self, uptie, speed):
        self.uptie_stats.append(UptieStat(uptie, speed))

    def add_resist_info(self, resist_hit, resist_penetrate, resist_slash):
        self.resist_info = ResistInfo(resist_hit, resist_penetrate, resist_slash)

    def add_stagger_threshold(self, first_stagger, second_stagger, third_stagger):
        self.stagger_threshold = StaggerThreshold(first_stagger, second_stagger, third_stagger)

    def add_skill(self, skill_code, skill_name, coin_count, init_level, init_remain, akt_weight):
        self.skillset.add_skill(skill_code, Skill(skill_code, skill_name, coin_count, init_level, init_remain, akt_weight))
        
    def add_skill_uptie_info(self, skill_name, uptie_level, skill_power, coin_power):
        self.skillset.add_skill_uptie_info(skill_name, uptie_level, skill_power, coin_power)
        
    def update_details(self, page):
        # 세부 정보 업데이트 로직 (가정)
        pass
    #def init_uptie(self, page):
    #    init_uptie = 

# LevelStat 객체 정의
class LevelStat:
    def __init__(self, level, hp=None, defense=None):
        self.level = level
        self.hp = hp
        self.defense = defense
    def update_level(self, level):
        self.level = level
    def update_hp(self, hp):
        self.hp = hp
    def update_defense(self, defense):
        self.defense = defense

# UptieStat 객체 정의
class UptieStat:
    def __init__(self, uptie, speed=None):
        self.uptie = uptie
        self.speed = speed
    def update_speed(self, speed):
        self.speed = speed

# ResistInfo 객체 정의
class ResistInfo:
    def __init__(self, resist_hit=None, resist_penetrate=None, resist_slash=None):
        self.resist_hit = resist_hit
        self.resist_penetrate = resist_penetrate
        self.resist_slash = resist_slash

# ResistInfo 객체 정의
class StaggerThreshold:
    def __init__(self, first_stagger=None, second_stagger=None, third_stagger=None):
        self.first_stagger = first_stagger
        self.second_stagger = second_stagger
        self.third_stagger = third_stagger

class SkillSet:
    def __init__(self):

        self.skill_1_type = ""
        self.skill_2_type = ""
        self.skill_3_type = ""
        self.defense_skill_type = ""

        self.skill_1_sin_affinity = ""
        self.skill_2_sin_affinity = ""
        self.skill_3_sin_affinity = ""
        self.defense_skill_sin_affinity = ""

        self.skill_1_details = []
        self.skill_2_details = []
        self.skill_3_details = []
        self.defense_skill_details = []

    def set_skill_type(self, skill_code, skill_type, sin_affinity):
        if skill_code == "skill 1":
            self.skill_1_type = skill_type
            self.skill_1_sin_affinity = sin_affinity
        elif skill_code == "skill 2":
            self.skill_2_type = skill_type
            self.skill_2_sin_affinity = sin_affinity
        elif skill_code == "skill 3":
            self.skill_3_type = skill_type
            self.skill_3_sin_affinity = sin_affinity
        elif skill_type == "defense":
            self.defense_skill_type = skill_type
            self.defense_skill_sin_affinity = sin_affinity
    
    def add_skill(self, skill_code, skill):
        if skill_code == "skill 1":
            self.skill_1_details.append(skill)
        elif skill_code == "skill 2":
            self.skill_2_details.append(skill)
        elif skill_code == "skill 3":
            self.skill_3_details.append(skill)
        elif skill_code == "defense":
            self.defense_skill_details.append(skill)

    def get_skill_by_name(self, skill_name):
        for skill_list in [self.skill_1_details, self.skill_2_details, self.skill_3_details, self.defense_skill_details]:
            for skill in skill_list:
                result = skill.is_skill_by_name(skill_name)
                if result:
                    return result
        return None

    def add_skill_uptie_info(self, skill_name, uptie_level, skill_power, coin_power):
        skill = self.get_skill_by_name(skill_name)
        skill.add_uptie_info(uptie_level, skill_power, coin_power)
        

class Skill:
    def __init__(self, skill_code, skill_name, coin_count, init_level, init_remain, akt_weight):
        self.skill_code = skill_code # "Skill 1"/"Skill 2"/"Skill 3"/"Defense" '''o'''
        self.skill_name = skill_name # 스킬 이름 '''o'''
        self.coin_count = coin_count # 동전 개수 '''o'''
        self.init_level = init_level # 스킬이 초기 배정받은 레벨, 인격 레벨에 - 1 한 값을 더해주면 스킬의 현 level을 구할 수 있다. '''o'''
        """
        남아있는 스킬 개수 사용하면, 하나씩 줄어들고 모든 스킬을 사용하면 이 값으로 돌아간다.
        스킬의 남은 숫자없는 스킬은 강화된 스킬로 한번 사용되면 다 조건을 채울 때까 사용 못한다.
        강화된 스킬을 None으로 설정하는 방식과 조건이 채워지면 1로 늘리는 방식이 존재한다.
        스킬을 사용할 수 있는 초기값이다.
        조건을 충족한 강화한 스킬을 이례적인 절차라고 생각하고 None으로 초기값을 설정했다.
        """
        self.init_remain = None #'''o'''
        self.atk_weight = None #공격 가중치 값이다. 방어스킬에는 없기 때문에 None으로 설정했다. '''o'''
        self.remain_count = init_remain # 실제 스킬이 작동하며 횟수가 차감되 나중에 모두 차감되면 다음 턴에 다시 초기횟수 리필된다.
        """
        동기화 단계에 따른 정보를 담는다. 현재 4동기화까지 되어 있지만 차후 확장될 것을 생각하여 고정하지 않는다.
        내부에 들어있는 정보로는 uptie_level, skill_power, coin_power 속성으로 들어있다.
        전투, 턴, 스킬, 코인토스 단계로 내려오며 발생하는 효과에 관하여 저장한다.
        """
        self.uptie_info = []  

    def is_skill_by_name(self, skill_name):
        if self.skill_name == skill_name:
            return self
        return None
        
    def add_uptie_info(self, uptie_level, skill_power, coin_power):
        self.uptie_info.append(UptieInfo(uptie_level, skill_power, coin_power))

class UptieInfo:
    def __init__(self, uptie_level, skill_power, coin_power):
        self.uptie_level = uptie_level   #'''o'''
        self.skill_power = skill_power   #'''o'''
        self.coin_power = coin_power     #'''o'''
        self.skill_effects = []

class Effect:
    def __init__(self, condition, description=None):
        self.condition = condition # 조건 텍스트 "[coin 1 on hit]"
        self.description = description # 효과 설명 텍스 "Inflict +1 Rupture Count

# 추상화된 EffectType 클래스
class EffectType:
    def __init__(self, name, description):
        self.name = name  # Effect 이름
        self.description = description  # Effect 설명

# Passive 클래스
class Passive(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)  # condition이 있는지 여부 확인

# SupportPassive 클래스
class SupportPassive(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)

# PanicType 클래스
class PanicType(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)

# Keyword 클래스
class Keyword:
    def __init__(self, keyword, explain=None, dispellable=None):
        self.keyword = keyword
        self.explain = explain
        self.dispellable = dispellable

      
class DataContainer:
    """크롤링 데이터를 관리하는 클래스"""
    def __init__(self):
        self.identities = []

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.level_max

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.uptie_max

    def update_identity_name(self, identity_id, identity_name):
        """특정 ID의 이름을 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_name(identity_name)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_identity_rarity(self, identity_id, rarity):
        """특정 ID의 희귀도를 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_rarity(rarity)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_skill(self, identity_id, skill_number, new_type):
        """
        특정 identity의 skill_type을 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_type: 새 skill_type
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_type = new_type
                elif skill_number == 2:
                    identity.skill_2_type = new_type
                elif skill_number == 3:
                    identity.skill_3_type = new_type
                elif skill_number == "defense":
                    identity.defense_type = new_type
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def update_sin_affinity(self, identity_id, skill_number, new_affinity):
        """
        특정 identity의 sin_affinity를 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_affinity: 새 sin_affinity
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_sin_affinity = new_affinity
                elif skill_number == 2:
                    identity.skill_2_sin_affinity = new_affinity
                elif skill_number == 3:
                    identity.skill_3_sin_affinity = new_affinity
                elif skill_number == "defense":
                    identity.defense_sin_affinity = new_affinity
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def add_identity(self, identity):
        """새로운 인격 데이터를 추가."""
        self.identities.append(identity)

    def add_level_stat(self, identity_id, level, hp, defense):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_level_stat(level, hp, defense)

    def add_uptie_stat(self, identity_id, uptie, speed):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_uptie_stat(uptie, speed)

    def add_resist_info(self, identity_id, resist_hit, resist_penetrate, resist_slash):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_resist_info(resist_hit, resist_penetrate, resist_slash)

    def add_stagger_threshold(self, identity_id, first_stagger, second_stagger, third_stagger):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_stagger_threshold(first_stagger, second_stagger, third_stagger)

    
    def add_skill(self, identity_id, skill_code, skill_name, coin_count, init_level, init_remain, akt_weight):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_code == "skill 1":
                    identity.skillset.set_skill_type(skill_code, identity.skill_1_type, identity.skill_1_sin_affinity)
                elif skill_code == "skill 2":
                    identity.skillset.set_skill_type(skill_code, identity.skill_2_type, identity.skill_2_sin_affinity)
                elif skill_code == "skill 3":
                    identity.skillset.set_skill_type(skill_code, identity.skill_3_type, identity.skill_3_sin_affinity)
                elif skill_code == "defense":
                    identity.skillset.set_skill_type(skill_code, identity.defense_type, identity.defense_sin_affinity)
                identity.add_skill(skill_code, skill_name, coin_count, init_level, init_remain, akt_weight)

    def add_skill_uptie_info(self, identity_id, skill_name, uptie_level, skill_power, coin_power):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_skill_uptie_info(skill_name, uptie_level, skill_power, coin_power)
        
    def display_identities(self):
        """Identity, ID, Name, Rarity 정보를 출력"""
        if not self.identities:
            print("No identities found.")
        else:
            # 데이터를 리스트로 변환
            table = []
            for idx, identity in enumerate(self.identities, 1):
                table.append([
                    idx, identity.identity_id, identity.name, identity.rarity,
                    identity.skill_1_type, identity.skill_1_sin_affinity,
                    identity.skill_2_type, identity.skill_2_sin_affinity,
                    identity.skill_3_type, identity.skill_3_sin_affinity,
                    identity.defense_type, identity.defense_sin_affinity
                ])
            
            # 테이블 헤더 정의
            headers = [
                "Index", "ID", "Name", "Rarity",
                "Skill_1 Type", "Skill_1 Sin", "Skill_2 Type", "Skill_2 Sin",
                "Skill_3 Type", "Skill_3 Sin", "Defense Type", "Defense Sin"
            ]
            
            # 테이블 출력
            print(tabulate(table, headers=headers, tablefmt="fancy_grid"))

    def export_to_csv(self, filename, fieldnames, data):
        """
        공통적으로 사용 가능한 CSV 내보내기 메서드
        :param filename: 내보낼 파일 이름 (예: 'identity.csv', 'skill.csv')
        :param fieldnames: CSV의 헤더 리스트
        :param data: CSV로 내보낼 데이터 (리스트 형태)
        """
        try:
            # 현재작업 디렉토 기준으 'csv'폴더 없으면 생성
            if not os.path.exists('./csv'):
                os.makedirs('./csv')
            # 파일 경로 설정 (상대경로 'csv' 폴더 안에 저장)
            file_path = os.path.join('./csv', filename)
            with open(file_path, mode='w', newline='', encoding='utf-8') as file:
                writer = csv.DictWriter(file, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(data)
            print(f"Data successfully exported to {filename}")
        except Exception as e:
            print(f"Error exporting to CSV: {e}")

    def get_identity_data(self):
        """
        Identity 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "ID": identity.identity_id,
                "Name": identity.name,
                "Rarity": identity.rarity,
                "Skill_1_Type": identity.skill_1_type,
                "Skill_1_Sin_Affinity": identity.skill_1_sin_affinity,
                "Skill_2_Type": identity.skill_2_type,
                "Skill_2_Sin_Affinity": identity.skill_2_sin_affinity,
                "Skill_3_Type": identity.skill_3_type,
                "Skill_3_Sin_Affinity": identity.skill_3_sin_affinity,
                "Defense_Type": identity.defense_type,
                "Defense_Sin_Affinity": identity.defense_sin_affinity,
            }
            for identity in self.identities
        ]
        
    def export_identities_data(self, filename="identities.csv"):
        """Identity 데이터를 CSV로 내보내는 메서드"""
        identity_fieldnames = [
            "ID", "Name", "Rarity",
            "Skill_1_Type", "Skill_1_Sin_Affinity",
            "Skill_2_Type", "Skill_2_Sin_Affinity",
            "Skill_3_Type", "Skill_3_Sin_Affinity",
            "Defense_Type", "Defense_Sin_Affinity"
        ]
        identity_data = self.get_identity_data()  # Identity 데이터 가져오기
        self.export_to_csv(filename, identity_fieldnames, identity_data)  # CSV로 내보내기

    def load_identities_by_id(self, filename, identity_ids):
        """지정된 identity_ids 목록에 해당하는 Identity 데이터를 CSV에서 읽어옴."""
        identity_ids_set = set(identity_ids) # 중복 제거를 위 Set 사용
        with open(filename, mode='r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                if row["ID"] in identity_ids_set:
                    identity = Identity(
                        identity_id=row["ID"],
                        name=row["Name"],
                        rarity=row["Rarity"],
                        skill_1_type=row["Skill_1_Type"],
                        skill_1_sin_affinity=row["Skill_1_Sin_Affinity"],
                        skill_2_type=row["Skill_2_Type"],
                        skill_2_sin_affinity=row["Skill_2_Sin_Affinity"],
                        skill_3_type=row["Skill_3_Type"],
                        skill_3_sin_affinity=row["Skill_3_Sin_Affinity"],
                        defense_type=row["Defense_Type"],
                        defense_sin_affinity=row["Defense_Sin_Affinity"]
                    )
                    self.identities.append(identity)

    def get_level_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Level": level_stat.level,
                "HP": level_stat.hp,
                "Defense": level_stat.defense,
            }
            for identity in self.identities
            for level_stat in identity.level_stats
        ]

    def export_level_stats_data(self, filename="levelstats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        level_stats_fieldnames = ["Identity_ID", "Level", "HP", "Defense"]
        level_stats_data = self.get_level_stats_data()  # level_stats 데이터 가져오기
        self.export_to_csv(filename, level_stats_fieldnames, level_stats_data)  # CSV로 내보내기

    def get_uptie_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Uptie": uptie_stat.uptie,
                "Speed": uptie_stat.speed,
            }
            for identity in self.identities
            for uptie_stat in identity.uptie_stats
        ]

    def export_uptie_stats_data(self, filename="Uptiestats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        uptie_stats_fieldnames = ["Identity_ID", "Uptie", "Speed"]
        uptie_stats_data = self.get_uptie_stats_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, uptie_stats_fieldnames, uptie_stats_data)  # CSV로 내보내기

    def get_resist_info_data(self):
        """
        모든 Identity의 resist_info 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Resist_Hit": identity.resist_info.resist_hit,
                "Resist_Penetrate": identity.resist_info.resist_penetrate,
                "Resist_Slash": identity.resist_info.resist_slash,
            }
            for identity in self.identities
        ]

    def export_resist_info_data(self, filename="ResistInfos.csv"):
        """ResistInfo 데이터를 CSV로 내보내는 메서드"""
        resist_infos_fieldnames = ["Identity_ID", "Resist_Hit", "Resist_Penetrate", "Resist_Slash"]
        resist_infos_data = self.get_resist_info_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, resist_infos_fieldnames, resist_infos_data)  # CSV로 내보내기

    def export_uptie_stats_data(self, filename="Uptiestats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        uptie_stats_fieldnames = ["Identity_ID", "Uptie", "Speed"]
        uptie_stats_data = self.get_uptie_stats_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, uptie_stats_fieldnames, uptie_stats_data)  # CSV로 내보내기

    def get_stagger_threshold_data(self):
        """
        모든 Identity의 stagger_threshold 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "First_Stagger": identity.stagger_threshold.first_stagger,
                "Second_Stagger": identity.stagger_threshold.second_stagger,
                "Third_Stagger": identity.stagger_threshold.third_stagger,
            }
            for identity in self.identities
        ]

    def export_stagger_threshold_data(self, filename="StaggerThresholds.csv"):
        """StaggerThresholds 데이터를 CSV로 내보내는 메서드"""
        stagger_thresholds_fieldnames = ["Identity_ID", "First_Stagger", "Second_Stagger", "Third_Stagger"]
        stagger_thresholds_data = self.get_stagger_threshold_data()  # stagger_thresholds 데이터 가져오기
        self.export_to_csv(filename, stagger_thresholds_fieldnames, stagger_thresholds_data)  # CSV로 내보내기


async def main():
    data_container = DataContainer()
    scraper = Scraper(container=data_container, base_url="https://limbus.kusoge.xyz", img_url="https://img.kusoge.xyz/limbus/img")
    await scraper.scrape()

await main() 

Added skill: skill 1:Begone...
Updated Uptie for Begone... (Uptie Level: 1): Skill Power: 2, Coin Power: +4
['[Combat Start]']
Updated Uptie for Begone... (Uptie Level: 2): Skill Power: 2, Coin Power: +4
['[Combat Start]', '[On Use]']
Updated Uptie for Begone... (Uptie Level: 3): Skill Power: 3, Coin Power: +4
['[Combat Start]', '[On Use]']
Updated Uptie for Begone... (Uptie Level: 4): Skill Power: 3, Coin Power: +4
['[Combat Start]', '[On Use]']
Added skill: skill 2:In Finely Ground Mistfall
Updated Uptie for In Finely Ground Mistfall (Uptie Level: 1): Skill Power: 3, Coin Power: +4
['[Combat Start]']
Updated Uptie for In Finely Ground Mistfall (Uptie Level: 2): Skill Power: 3, Coin Power: +4
['[Combat Start]', '[On Use]']
Updated Uptie for In Finely Ground Mistfall (Uptie Level: 3): Skill Power: 4, Coin Power: +4
['[Combat Start]', '[On Use]']
Updated Uptie for In Finely Ground Mistfall (Uptie Level: 4): Skill Power: 4, Coin Power: +4
['[Combat Start]', '[On Use]']
Added skill: skill

In [54]:
import os
import csv
import re
import numpy as np
import pandas as pd
from playwright.async_api import async_playwright
from tabulate import tabulate

# Sin Color 이름 변환을 위한 매핑
resource_mapping = {
    "AZURE": "Gloom",
    "VIOLET": "Envy",
    "AMBER": "Sloth",
    "SHAMROCK": "Gluttony",
    "CRIMSON": "Wrath",
    "SCARLET": "Lust",
    "INDIGO": "Pride"
}

def convert_resource(resource_name):
    """Sin Color 이름을 변환하는 함수. 매핑되지 않은 경우 원래 이름을 반환."""
    return resource_mapping.get(resource_name, resource_name)

class Scraper:
    """웹 페이지 크롤링을 위한 클래스"""
    def __init__(self, container, base_url, img_url):
        self.container = container
        self.base_url = base_url
        self.img_url = img_url
        self.uptie_level = 1

    async def handle_request(self, route, request):
        """이미지와 폰트 리소스 로드를 차단."""
        if request.resource_type in ["image", "font"]:
            await route.abort()
        else:
            await route.continue_()
            
    async def scrape(self):
        """크롤링 메인 함수"""
        async with async_playwright() as p:
            browser = await p.chromium.launch(headless=True)
            page = await browser.new_page()
            await page.route("**/*", self.handle_request)
            # Step 1: /identity 페이지에서 기본 정보 크롤링
            #await self.crawl_identity_page(page)

            # Step 2: /identity/{identity_id} 상세 페이지 크롤링
            #identity_ids_to_load = ["10104", "10310", "10710","10808", "10911", "11005"]
            #identity_ids_to_load = ["10104", "10911"]#,"10310", "10710","10808", "11005", "11210"]
            identity_ids_to_load = ["10911"]#,"10310", "10710","10808", "11005", "11210"]
            self.container.load_identities_by_id('./csv/identities.csv', identity_ids_to_load)
            await self.crawl_identity_details_for_all(page)
            #self.container.display_identities()
            #self.container.export_identities_data("identities.csv")
            #self.container.export_level_stats_data("Levelstats.csv")
            #self.container.export_uptie_stats_data("Uptiestats.csv")
            #self.container.export_resist_info_data("ResistInfos.csv")
            #self.container.export_stagger_threshold_data("StaggerThresholds.csv")
            page.set_default_timeout(10000)
            #await page.wait_for_timeout(100000)
            await browser.close()

    async def get_identity_id_from_row(self, row):
        """테이블 행(row)에서 인격 ID를 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_url = await identity_cell.get_attribute('href')
            identity_id = identity_url.split('/identity/')[-1]
            return identity_id
        return "Unknown"

    async def get_identity_name_from_row(self, row):
        """테이블 행(row)에서 인격 이름을 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_name = (await identity_cell.inner_text()).replace("\n", " ").strip()
            return identity_name
        return "Unknown"
        
    async def crawl_identity_page(self, page):
        """/identity 페이지로 이동하여 인격 기본 정보를 추출."""
        await page.goto(self.base_url + "/identity")
        await self.extract_identity_page(page)

    async def extract_identity_page(self, page):
        """/identity 페이지의 모든 테이블 행에서 인격 정보를 추출."""
        identity_rows = await page.query_selector_all('tr')
        if len(identity_rows) > 1:
            for idx, identity_row in enumerate(identity_rows):
                if idx == 0:
                    continue # 0번째요소이름을 보여주는 열은 건너뜀
                await self.scrape_identity_name_and_identity_id(identity_row)
                await self.extract_rarity(identity_row)
                for i in range(1, 4):  # 스킬1~3 정보 추출
                    await self.extract_atk_and_resource(identity_row, i)
                await self.extract_defense_and_resource(identity_row)

    async def scrape_identity_name_and_identity_id(self, row):
        """인격 번호와 이름을 추출하여 DataContainer에 저장."""
        try:
            identity_id = await self.get_identity_id_from_row(row)
            identity_name = await self.get_identity_name_from_row(row)
            identity = Identity(identity_id)
            self.container.add_identity(identity)
            self.container.update_identity_name(identity_id, identity_name)
        except Exception as e:
            print(f"Error extracting identity name and identity id: {e}")

    async def get_element_attribute(self, element, selector, attribute):
        """요소에서 지정된 속성(attribute)을 추출."""
        try:
            target_element = await element.query_selector(selector)
            if target_element:
                return await target_element.get_attribute(attribute)
        except Exception as e:
            print(f"Error getting element attribute: {e}")
        return None
    
    async def extract_rarity(self, row):
        """테이블 행(row)에서 희귀도를 추출하여 업데이트."""
        try:
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
                sibling = await parent.evaluate_handle('node => node.nextElementSibling')
                if sibling:
                    rarity_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/rarity_"]', 'src')
                    if rarity_img_src:
                        rarity_level = rarity_img_src.split('_')[-1].split('.')[0]  # rarity_1.png -> 1
                        identity_id = await self.get_identity_id_from_row(row)
                        self.container.update_identity_rarity(identity_id, rarity_level)
                    else:
                        print("Rarity image not found in the next sibling TD.")
        except Exception as e: 
            print(f"Error extracting rarity: {e}")

        return "Unknown"

    async def extract_atk_and_resource(self, row, skill_number):
        """ 공격 타입(ATK Type)과 자원(Attribute) 추출 함수 - 스킬 번호에 따라 처리 """
        try:
            #print(f"Attempting to locate skill{skill_number} type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 해당 스킬 번호에 맞는 형제로 이동
                sibling = parent
                for _ in range(skill_number + 1):  # 스킬 번호에 맞게 이동
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                atk_type, resource = "Unknown", "Unknown"
        
                if sibling:
                    # 공격 타입 이미지 (atk_type) 찾기
                    atk_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/atk_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if atk_type_img_src:
                        atk_type = atk_type_img_src.split('_')[-1].split('.')[0]  # atk_type_SLASH.png -> SLASH
                        #print(f"Extracted attack type: {atk_type}")
                        self.container.update_skill(identity_id, skill_number, atk_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted resource: {resource}")
                        
                        # 리소스를 변환
                        resource = convert_resource(resource)
                        #print(f"Converted resource: {resource}")
                        self.container.update_sin_affinity(identity_id, skill_number, resource)
                        
        except Exception as e:
            print(f"Error extracting skill{skill_number} type and resource: {e}")

    async def extract_defense_and_resource(self, row):
        """ 방어 타입(Def Type)과 자원(Attribute) 추출 함수 """
        try:
            #print("Attempting to locate defense type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 다섯 번째 형제 td로 이동 (방어 관련 정보가 있는 <td>)
                sibling = parent
                for _ in range(5):  # 방어 정보는 5번째 형제에 있음
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                def_type, def_resource = "Unknown", "Unknown"
        
                if sibling:
                    # 방어 타입 이미지 (def_type) 찾기
                    def_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/def_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if def_type_img_src:
                        def_type = def_type_img_src.split('_')[-1].split('.')[0]  # def_type_GUARD.png -> GUARD
                        #print(f"Extracted defense type: {def_type}")
                        self.container.update_skill(identity_id, 'defense', def_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        def_resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted defense resource: {def_resource}")
                        
                        # 방어 리소스를 변환
                        def_resource = convert_resource(def_resource)
                        #print(f"Converted defense resource: {def_resource}")
                        self.container.update_sin_affinity(identity_id, 'defense', def_resource)
 
        except Exception as e:
            print(f"Error extracting defense type and resource: {e}")

    async def crawl_identity_details_for_all(self, page):
        """모든 인격의 세부 정보를 크롤링."""
        for identity in self.container.identities:
            await page.wait_for_load_state("networkidle")
            await page.goto(self.base_url + "/identity/"+identity.identity_id)
            """ 레벨과 동기화와 관련 없는 참관타 저항은 밑에서 변경하는 것과 연관성이 적기 때문에 위에서 처리한다.
            print(f"Identity_id: {identity.identity_id}")
            results = []
            resist_type_imgs = [f"{self.img_url}/ui/atk_type_HIT.png",
                                f"{self.img_url}/ui/atk_type_PENETRATE.png",
                                f"{self.img_url}/ui/atk_type_SLASH.png"]
            for resist_type_img in resist_type_imgs:
                result = await self.extract_resist_type(page, resist_type_img, 1)
                results.append(result)
            self.container.add_resist_info(identity.identity_id, results[0], results[1], results[2])
            """
            """
            # Stagger Threshold 값 추출
            stagger_threshold = await self.extract_stagger_threshold_values(page)
            self.container.add_stagger_threshold(identity.identity_id, stagger_threshold[0], stagger_threshold[1], stagger_threshold[2])
            """
            await self.initialize_uptie_and_level(page)
            """ 레벨에 따른 level, hp, defense 다르게 되어 있는 것을 출력 확인 객체로 들어가서 csv로 옮기는 과정까지 진행
            for i in range(1, identity.level_max+1):
                await self.move_slider_to_value(page,i)
                await self.extract_level_status(page, identity.identity_id, i)
            """
            """ 동기화에 따른 speed 다르게 되어 있는 것을 출력 확인 객체로 들어가서 csv로 옮기는 과정까지 진행
            for i in range(1, identity.uptie_max+1):
                await self.uptie_click(page,i)
                await self.extract_uptie_status(page, identity.identity_id, i)
            """
            await self.crawl_skills(page, identity.identity_id)
            
    async def initialize_uptie_and_level(self, page):
        await self.uptie_click(page, 1)
        await self.move_slider_to_value(page, 1)
        self.uptie_level = 1
        #return 1
    
    async def uptie_click(self, page, phase):
        fab_button_selector = ".q-btn.q-btn--fab"
        await page.wait_for_selector(fab_button_selector)
        await page.click(fab_button_selector)
        #print(f"[{setup_uptie_button}] 상위 동기화 버튼 클릭 완료")
        setup_uptie_button = f'a:has(img[src="{self.img_url}/ui/uptie_{phase}.png"])'
        await page.wait_for_selector(setup_uptie_button)
        await page.click(setup_uptie_button)
        #print(f"{phase} 동기화 버튼 클릭 완료")

    async def move_slider_to_value(self, page, target_value):
        """
        슬라이더를 주어진 값으로 이동시킵니다.
        target_value는 슬라이더의 최소값부터 최대값 사이여야 합니다.
        """
        slider_selector = ".q-slider"
        # 슬라이더의 범위 확인
        min_value = int(await page.get_attribute(slider_selector, "aria-valuemin"))
        max_value = int(await page.get_attribute(slider_selector, "aria-valuemax"))
    
        if target_value < min_value or target_value > max_value:
            raise ValueError(f"Target value {target_value} is out of bounds ({min_value}, {max_value})")
    
        # 슬라이더 thumb 요소와 위치 가져오기
        slider_thumb = await page.query_selector(f"{slider_selector} .q-slider__thumb")
        thumb_box = await slider_thumb.bounding_box()
        
        # 슬라이더의 현재 위치 및 목표 위치 계산
        track = await page.query_selector(f"{slider_selector} .q-slider__track")
        track_box = await track.bounding_box()
        
        # 목표 x 위치 계산 (슬라이더 트랙 내에서 비율 기반)
        target_percentage = (target_value - min_value) / (max_value - min_value)
        target_x = track_box["x"] + target_percentage * track_box["width"]
        current_x = thumb_box["x"] + thumb_box["width"] / 2
        y = thumb_box["y"] + thumb_box["height"] / 2
    
        # 슬라이더를 드래그하여 목표 위치로 이동
        await page.mouse.move(current_x, y)
        await page.mouse.down()
        await page.mouse.move(target_x, y, steps=10)  # 부드럽게 이동
        await page.mouse.up()
    
        #print(f"슬라이더를 {target_value}로 이동 완료.")
    
        # 슬라이더 값 확인 및 강제 이벤트 트리거
        await page.evaluate(
            f"""
            (slider) => {{
                slider.dispatchEvent(new Event('input'));  // oninput 이벤트 트리거
                slider.dispatchEvent(new Event('change'));  // onchange 이벤트 트리거
            }}
            """,
            await page.query_selector(slider_selector)
        )
    
        # 이동 후 값 확인
        current_value = await page.get_attribute(slider_selector, "aria-valuenow")
        if current_value != str(target_value):
            print(f"경고: 슬라이더 값이 예상과 다릅니다. 현재 값: {current_value}, 예상 값: {target_value}")

    async def get_second_status_element(self, page):
        """
        두 번째 'Status' 요소에 접근하여 반환하는 메소드
        """
        status_elements = await page.query_selector_all("text=Status")
        if len(status_elements) > 1:
            second_status_element = status_elements[1]
            #print("두 번째 'Status' 요소에 접근합니다.")
            return second_status_element
        else:
            #print("두 번째 'Status' 요소가 존재하지 않습니다.")
            return None

    async def extract_level_status(self, page, identity_id, level):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 레벨에 따라 HP와 Defense가 변한다.
        hp_value = await self.extract_hp(second_status_element)
        defense_value = await self.extract_defense(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Level={level}, HP={hp_value}, Defense={defense_value}")
        self.container.add_level_stat(identity_id,level, hp_value, defense_value)
    
    async def extract_hp(self, status_element):
        # HP 추출
        hp_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].children[0].nextElementSibling.textContent.trim()"
        )
        return hp_value
    
    async def extract_defense(self, status_element):
        # Defense 추출
        defense_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return defense_value

    async def extract_uptie_status(self, page, identity_id, uptie):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 uptie에 따라 speed는 변한다.
        speed_value = await self.extract_speed(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Uptie={uptie}, Speed={speed_value}")
        self.container.add_uptie_stat(identity_id,uptie, speed_value)

    async def extract_speed(self, status_element):
        # Speed 추출
        speed_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return speed_value

    async def extract_resist_type(self, page, image_src, depth):
        # 이미지 요소 찾기
        #print(image_src)
        image = await page.query_selector(f'img[src="{image_src}"]')
        #print(image)
        if not image:
            return "Unknown"
    
        # 이미지의 부모 요소를 찾아서 올라감
        parent = image
        for _ in range(depth):
            parent = await parent.evaluate_handle('node => node.parentElement')
            if not parent:
                return "Unknown"
    
        # 부모의 형제 요소 찾기
        sibling = await parent.evaluate_handle('node => node.nextElementSibling')
        
        if sibling:
            value = await sibling.evaluate('node => node.textContent')
            if value:
                return value.strip()
        
        return "Unknown"

    async def extract_stagger_threshold_values(self, page):
        # 'div.row.center.flex-center.q-pt-xs' 요소들 찾기
        elements = await page.query_selector_all('div.row.center.flex-center.q-pt-xs')
        
        # 각 요소의 텍스트 내용 출력하여 'Stagger Threshold' 요소를 찾기
        for element in elements:
            text = await element.text_content()
            
            if text and "Stagger Threshold" in text:
                # 'Stagger Threshold'가 있는 요소를 찾았다면, 그 이후 형제 요소들에서 값을 추출
                parent = await element.evaluate_handle('node => node.parentElement')
                sibling_elements = await parent.query_selector_all('div.row.justify-evenly div')  # Threshold 값이 들어있는 div들
                
                stagger_values = []
                for sibling in sibling_elements:
                    text_content = await sibling.text_content()
                    match = re.search(r'(\d+%)', text_content)
                    if match:
                        stagger_values.append(match.group(1))  # 퍼센트 값만 리스트에 추가
                while len(stagger_values) < 3:
                    stagger_values.append(np.nan)
                #print(stagger_values)
                return stagger_values  # 추출된 값(또는 np.nan 포함)을 반환
        
        return [np.nan, np.nan, np.nan]  # 만약 'Stagger Threshold'를 찾지 못한 경우 기본값 반환

    async def crawl_skills(self, page, identity_id):
        await self.crawl_skill(page, identity_id, "skill 1")
        await self.crawl_skill(page, identity_id, "skill 2")
        await self.crawl_skill(page, identity_id, "skill 3")
        await self.crawl_skill(page, identity_id, "defense")


    async def crawl_skill(self, page, identity_id, skill_type):
        skills_elements = await page.query_selector_all('div.skilltag')
        for skill_element in skills_elements:
            skill_text = await skill_element.text_content()
            if skill_text.strip().lower() == skill_type:
                await self.process_skill(skill_element, page, identity_id, skill_type)
        
    async def process_skill(self, skill_element, page, identity_id, skill_type):
        # skill_type이 "skill 3"일 경우 uptie_value가 3 이상일 때만 정보를 저장
        for uptie_value in range(1, 5):  # Uptie levels 1 to 4
            # "skill 3"은 uptie_value가 3 이상일 때만 처리
            if skill_type == "skill 3" and uptie_value < 3:
                continue  # skip if uptie_value is less than 3 for skill 3
    
            # uptie_level에 맞게 클릭
            await self.uptie_click(page, uptie_value)
            if skill_type != "skill 3" and uptie_value > 1:
                # Uptie 정보 갱신
                await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
                continue
            elif skill_type == "skill 3" and uptie_value > 3:
                # Uptie 정보 갱신
                await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
                continue
            # 스킬의 세부 정보를 추출
            skill_details = await self.extract_skill_details(skill_element, skill_type)
            skill_name = skill_details["skill_name"]
    
            # 스킬 정보를 저장
            self.container.add_skill(
                identity_id, skill_type, 
                skill_name, skill_details["coin_count"],
                skill_details["init_level"], skill_details["init_remain"], 
                skill_details["atk_weight"]
            )
            print(f"Added skill: {skill_type}:{skill_name}")
    
            # Uptie 정보 갱신
            await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
    
    async def extract_skill_details(self, skill_element, skill_type):
        # 스킬 세부 정보를 추출하는 함수
        skill_name = await self.skill_name(skill_element)
        coin_count = await self.coin_count(skill_element, ".col-auto > .column > .row")
        init_level = await self.init_level(skill_element)
        init_remain = await self.init_remain(skill_element, skill_type, "div.row.justify-start > div.attr.flex.flex-center > div.q-px-sm")
        atk_weight = await self.atk_weight(skill_element, skill_type)
        return {
            "skill_name": skill_name,
            "coin_count": coin_count,
            "init_level": init_level,
            "init_remain": init_remain,
            "atk_weight": atk_weight
        }
    
    async def update_uptie_info(self, page, skill_element, identity_id, skill_name, skill_type, uptie_value):
        # Uptie 정보를 추출하고 업데이트하는 함수
        uptie_info = await self.extract_uptie_info(skill_element)
        self.container.add_skill_uptie_info(
            identity_id, skill_name, 
            uptie_info["skill_power"], uptie_info["coin_power"], 
            uptie_value
        )
        print(
            f"Updated Uptie for {skill_name} (Uptie Level: {uptie_value}): "
            f"Skill Power: {uptie_info['skill_power']}, Coin Power: {uptie_info['coin_power']}"
        )
        all_text = await self.extract_all_text(skill_element)
        if all_text:
            print("Extracted text values:")
            for text in all_text:
                print(text)
            else:
                print("No text found.")

    async def extract_uptie_info(self, skill_element):
        skill_data = await self.skill_and_coin_power(skill_element)
        return {
            "skill_power": skill_data.get("skillPower"),
            "coin_power": skill_data.get("coinPower")
        }

    async def extract_skill_effect(self, skill_element):
        # Create the Effect object
        effect_obj = Effect(condition=None)
        # Extract effects
        effects = await self.skill_effects(skill_element)
        if effects:
            effect_obj.effects.append(effects)
        return effect_obj

    async def skill_and_coin_power(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                // 조부모로 이동
                const grandparent = node.parentElement.parentElement;
                // 이전 형제 탐색
                const previousSibling = grandparent.previousElementSibling;
                if (!previousSibling) return {{ skillPower: null, coinPower: null }}; // 이전 형제가 없으면 기본값 반환
                // 이전 형제 내부의 .row.justify-around > div > div 탐색
                const targetDivs = previousSibling.querySelectorAll(".row.justify-around > div");
                if (targetDivs.length < 3) return {{ skillPower: null, coinPower: null }}; // 배열 길이 확인
                // 1st div: 스킬 위력
                const skillPower = targetDivs[0]?.querySelector('div')?.textContent?.trim() || null; 
                // 3rd div: 코인 파워 값 (3번째 div에서 텍스트 추출)
                const coinPower = targetDivs[2]?.textContent?.trim() || null;
                return {{ skillPower, coinPower }};
            }}
        """)


    async def coin_count(self, skill_element, hierarchy_selector):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector('{hierarchy_selector}');
                if (!targetElement) return 0;
                return targetElement.childElementCount;
            }}
        """)

    async def init_remain(self, skill_element, skill_type, hierarchy_selector):
        if skill_type == "defense":
            return -1
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector('{hierarchy_selector}');
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def skill_name(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector(".block");
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def init_level(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector("div.text-bold.text-h6.h6.q-pl-xs.q-pr-md");
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def atk_weight(self, skill_element, skill_type):
        if skill_type == "defense":
            return 0
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector("div.text-bold.text-h6.h6.q-pl-xs.q-pr-md");
                const doubleNextSibling = targetElement.nextElementSibling.nextElementSibling;
                if (!doubleNextSibling) return 0;
                return doubleNextSibling.textContent?.trim();
            }}
        """)
        
    async def extract_all_text(self, skill_element):
        return await skill_element.evaluate("""
            node => {
                // 부모 노드에서 시작해 모든 텍스트 노드 추출
                const parent = node.parentElement;
                const doubleNextSibling = parent?.nextElementSibling?.nextElementSibling;
                if (!doubleNextSibling) return null;
    
                // 모든 텍스트 노드 가져오기
                const allTextNodes = doubleNextSibling.querySelectorAll("div.col");
                return Array.from(allTextNodes)
                    .map(el => el.textContent?.trim())
                    .filter(Boolean); // null 또는 빈 텍스트 제거
            }
        """)
'''
    async def skill_effects(self, skill_element):
        return await skill_element.evaluate("""
            node => {
                const parent = node.parentElement;
                const doubleNextSibling = parent.nextElementSibling.nextElementSibling;
                if (!doubleNextSibling) return null;
    
                // effects 텍스트가 포함된 노드 찾기
                const effectElements = doubleNextSibling.querySelectorAll("span, a");
                if (!effectElements || effectElements.length === 0) return null;
    
                return Array.from(effectElements)
                    .map(el => {
                        if (el.tagName === "A") {
                            // 키워드가 포함된 <a> 태그 처리
                            const keywordSpan = el.querySelector("span");
                            const keyword = keywordSpan ? keywordSpan.textContent.trim() : null;
                            return keyword ? `keyword:${keyword}` : null;
                        } else {
                            // 일반 텍스트 처리
                            return el.textContent?.trim();
                        }
                    })
                    .filter((text, index, self) => text && self.indexOf(text) === index) // 중복 제거
                    .join(" "); // 효과를 하나의 문자열로 결합
            }
        """)
        '''


class Identity:
    """인격 정보를 저장하는 클래스"""
    def __init__(self, identity_id, name=None, rarity=None,\
                 skill_1_type=None, skill_1_sin_affinity=None,\
                 skill_2_type=None, skill_2_sin_affinity=None,\
                 skill_3_type=None, skill_3_sin_affinity=None,\
                 defense_type=None, defense_sin_affinity=None):
        self.identity_id = identity_id
        self.name = name
        self.rarity = rarity
        # 선언만 하는 부분 
        self.skill_1_type = skill_1_type 
        self.skill_1_sin_affinity = skill_1_sin_affinity 
        self.skill_2_type = skill_2_type 
        self.skill_2_sin_affinity = skill_2_sin_affinity 
        self.skill_3_type = skill_3_type 
        self.skill_3_sin_affinity = skill_3_sin_affinity 
        self.defense_type = defense_type 
        self.defense_sin_affinity = defense_sin_affinity
        self.level_max = 50
        self.uptie_max = 4
        self.level_stats = []  # LevelStat 객체 리스트
        self.uptie_stats = [] # UptieStat 객체 리스트
        self.resist_info = None
        self.stagger_threshold = None
        self.skillset = SkillSet()  # Skill정보를 담고 있는 skillset 객체 
        self.passive_infos = []  # 패시브 객체를 담고 있는 배열
        self.support_passive_infos = [] # 서포트 패시 객체를 담고 있는 배열
        self.panic_type = [] # 혼란상태 발생시 나타나는 효과를 담고 있는 객체 배열

    def update_name(self, name):
        self.name = name

    def update_rarity(self, rarity):
        self.rarity = rarity

    def add_level_stat(self, level, hp, defense):
        self.level_stats.append(LevelStat(level, hp, defense))

    def add_uptie_stat(self, uptie, speed):
        self.uptie_stats.append(UptieStat(uptie, speed))

    def add_resist_info(self, resist_hit, resist_penetrate, resist_slash):
        self.resist_info = ResistInfo(resist_hit, resist_penetrate, resist_slash)

    def add_stagger_threshold(self, first_stagger, second_stagger, third_stagger):
        self.stagger_threshold = StaggerThreshold(first_stagger, second_stagger, third_stagger)

    def add_skill(self, skill_code, skill_name, coin_count, init_level, init_remain, akt_weight):
        self.skillset.add_skill(skill_code, Skill(skill_code, skill_name, coin_count, init_level, init_remain, akt_weight))
        
    def add_skill_uptie_info(self, skill_name, uptie_level, skill_power, coin_power):
        self.skillset.add_skill_uptie_info(skill_name, uptie_level, skill_power, coin_power)
        
    def update_details(self, page):
        # 세부 정보 업데이트 로직 (가정)
        pass
    #def init_uptie(self, page):
    #    init_uptie = 

# LevelStat 객체 정의
class LevelStat:
    def __init__(self, level, hp=None, defense=None):
        self.level = level
        self.hp = hp
        self.defense = defense
    def update_level(self, level):
        self.level = level
    def update_hp(self, hp):
        self.hp = hp
    def update_defense(self, defense):
        self.defense = defense

# UptieStat 객체 정의
class UptieStat:
    def __init__(self, uptie, speed=None):
        self.uptie = uptie
        self.speed = speed
    def update_speed(self, speed):
        self.speed = speed

# ResistInfo 객체 정의
class ResistInfo:
    def __init__(self, resist_hit=None, resist_penetrate=None, resist_slash=None):
        self.resist_hit = resist_hit
        self.resist_penetrate = resist_penetrate
        self.resist_slash = resist_slash

# ResistInfo 객체 정의
class StaggerThreshold:
    def __init__(self, first_stagger=None, second_stagger=None, third_stagger=None):
        self.first_stagger = first_stagger
        self.second_stagger = second_stagger
        self.third_stagger = third_stagger

class SkillSet:
    def __init__(self):

        self.skill_1_type = ""
        self.skill_2_type = ""
        self.skill_3_type = ""
        self.defense_skill_type = ""

        self.skill_1_sin_affinity = ""
        self.skill_2_sin_affinity = ""
        self.skill_3_sin_affinity = ""
        self.defense_skill_sin_affinity = ""

        self.skill_1_details = []
        self.skill_2_details = []
        self.skill_3_details = []
        self.defense_skill_details = []

    def set_skill_type(self, skill_code, skill_type, sin_affinity):
        if skill_code == "skill 1":
            self.skill_1_type = skill_type
            self.skill_1_sin_affinity = sin_affinity
        elif skill_code == "skill 2":
            self.skill_2_type = skill_type
            self.skill_2_sin_affinity = sin_affinity
        elif skill_code == "skill 3":
            self.skill_3_type = skill_type
            self.skill_3_sin_affinity = sin_affinity
        elif skill_type == "defense":
            self.defense_skill_type = skill_type
            self.defense_skill_sin_affinity = sin_affinity
    
    def add_skill(self, skill_code, skill):
        if skill_code == "skill 1":
            self.skill_1_details.append(skill)
        elif skill_code == "skill 2":
            self.skill_2_details.append(skill)
        elif skill_code == "skill 3":
            self.skill_3_details.append(skill)
        elif skill_code == "defense":
            self.defense_skill_details.append(skill)

    def get_skill_by_name(self, skill_name):
        for skill_list in [self.skill_1_details, self.skill_2_details, self.skill_3_details, self.defense_skill_details]:
            for skill in skill_list:
                result = skill.is_skill_by_name(skill_name)
                if result:
                    return result
        return None

    def add_skill_uptie_info(self, skill_name, uptie_level, skill_power, coin_power):
        skill = self.get_skill_by_name(skill_name)
        skill.add_uptie_info(uptie_level, skill_power, coin_power)
        

class Skill:
    def __init__(self, skill_code, skill_name, coin_count, init_level, init_remain, akt_weight):
        self.skill_code = skill_code # "Skill 1"/"Skill 2"/"Skill 3"/"Defense" '''o'''
        self.skill_name = skill_name # 스킬 이름 '''o'''
        self.coin_count = coin_count # 동전 개수 '''o'''
        self.init_level = init_level # 스킬이 초기 배정받은 레벨, 인격 레벨에 - 1 한 값을 더해주면 스킬의 현 level을 구할 수 있다. '''o'''
        """
        남아있는 스킬 개수 사용하면, 하나씩 줄어들고 모든 스킬을 사용하면 이 값으로 돌아간다.
        스킬의 남은 숫자없는 스킬은 강화된 스킬로 한번 사용되면 다 조건을 채울 때까 사용 못한다.
        강화된 스킬을 None으로 설정하는 방식과 조건이 채워지면 1로 늘리는 방식이 존재한다.
        스킬을 사용할 수 있는 초기값이다.
        조건을 충족한 강화한 스킬을 이례적인 절차라고 생각하고 None으로 초기값을 설정했다.
        """
        self.init_remain = None #'''o'''
        self.atk_weight = None #공격 가중치 값이다. 방어스킬에는 없기 때문에 None으로 설정했다. '''o'''
        self.remain_count = init_remain # 실제 스킬이 작동하며 횟수가 차감되 나중에 모두 차감되면 다음 턴에 다시 초기횟수 리필된다.
        """
        동기화 단계에 따른 정보를 담는다. 현재 4동기화까지 되어 있지만 차후 확장될 것을 생각하여 고정하지 않는다.
        내부에 들어있는 정보로는 uptie_level, skill_power, coin_power 속성으로 들어있다.
        전투, 턴, 스킬, 코인토스 단계로 내려오며 발생하는 효과에 관하여 저장한다.
        """
        self.uptie_info = []  

    def is_skill_by_name(self, skill_name):
        if self.skill_name == skill_name:
            return self
        return None
        
    def add_uptie_info(self, uptie_level, skill_power, coin_power):
        self.uptie_info.append(UptieInfo(uptie_level, skill_power, coin_power))

class UptieInfo:
    def __init__(self, uptie_level, skill_power, coin_power):
        self.uptie_level = uptie_level   #'''o'''
        self.skill_power = skill_power   #'''o'''
        self.coin_power = coin_power     #'''o'''
        self.skill_effects = []

class Effect:
    def __init__(self, condition):
        self.condition = condition # 조건 텍스트 "[coin 1 on hit]"
        self.effects = [] # 효과 설명 텍스 "Inflict +1 Rupture Count

# 추상화된 EffectType 클래스
class EffectType:
    def __init__(self, name, description):
        self.name = name  # Effect 이름
        self.description = description  # Effect 설명

# Passive 클래스
class Passive(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)  # condition이 있는지 여부 확인

# SupportPassive 클래스
class SupportPassive(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)

# PanicType 클래스
class PanicType(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)

# Keyword 클래스
class Keyword:
    def __init__(self, keyword, explain=None, dispellable=None):
        self.keyword = keyword
        self.explain = explain
        self.dispellable = dispellable

      
class DataContainer:
    """크롤링 데이터를 관리하는 클래스"""
    def __init__(self):
        self.identities = []

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.level_max

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.uptie_max

    def update_identity_name(self, identity_id, identity_name):
        """특정 ID의 이름을 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_name(identity_name)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_identity_rarity(self, identity_id, rarity):
        """특정 ID의 희귀도를 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_rarity(rarity)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_skill(self, identity_id, skill_number, new_type):
        """
        특정 identity의 skill_type을 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_type: 새 skill_type
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_type = new_type
                elif skill_number == 2:
                    identity.skill_2_type = new_type
                elif skill_number == 3:
                    identity.skill_3_type = new_type
                elif skill_number == "defense":
                    identity.defense_type = new_type
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def update_sin_affinity(self, identity_id, skill_number, new_affinity):
        """
        특정 identity의 sin_affinity를 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_affinity: 새 sin_affinity
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_sin_affinity = new_affinity
                elif skill_number == 2:
                    identity.skill_2_sin_affinity = new_affinity
                elif skill_number == 3:
                    identity.skill_3_sin_affinity = new_affinity
                elif skill_number == "defense":
                    identity.defense_sin_affinity = new_affinity
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def add_identity(self, identity):
        """새로운 인격 데이터를 추가."""
        self.identities.append(identity)

    def add_level_stat(self, identity_id, level, hp, defense):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_level_stat(level, hp, defense)

    def add_uptie_stat(self, identity_id, uptie, speed):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_uptie_stat(uptie, speed)

    def add_resist_info(self, identity_id, resist_hit, resist_penetrate, resist_slash):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_resist_info(resist_hit, resist_penetrate, resist_slash)

    def add_stagger_threshold(self, identity_id, first_stagger, second_stagger, third_stagger):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_stagger_threshold(first_stagger, second_stagger, third_stagger)

    
    def add_skill(self, identity_id, skill_code, skill_name, coin_count, init_level, init_remain, akt_weight):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_code == "skill 1":
                    identity.skillset.set_skill_type(skill_code, identity.skill_1_type, identity.skill_1_sin_affinity)
                elif skill_code == "skill 2":
                    identity.skillset.set_skill_type(skill_code, identity.skill_2_type, identity.skill_2_sin_affinity)
                elif skill_code == "skill 3":
                    identity.skillset.set_skill_type(skill_code, identity.skill_3_type, identity.skill_3_sin_affinity)
                elif skill_code == "defense":
                    identity.skillset.set_skill_type(skill_code, identity.defense_type, identity.defense_sin_affinity)
                identity.add_skill(skill_code, skill_name, coin_count, init_level, init_remain, akt_weight)

    def add_skill_uptie_info(self, identity_id, skill_name, uptie_level, skill_power, coin_power):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_skill_uptie_info(skill_name, uptie_level, skill_power, coin_power)
        
    def display_identities(self):
        """Identity, ID, Name, Rarity 정보를 출력"""
        if not self.identities:
            print("No identities found.")
        else:
            # 데이터를 리스트로 변환
            table = []
            for idx, identity in enumerate(self.identities, 1):
                table.append([
                    idx, identity.identity_id, identity.name, identity.rarity,
                    identity.skill_1_type, identity.skill_1_sin_affinity,
                    identity.skill_2_type, identity.skill_2_sin_affinity,
                    identity.skill_3_type, identity.skill_3_sin_affinity,
                    identity.defense_type, identity.defense_sin_affinity
                ])
            
            # 테이블 헤더 정의
            headers = [
                "Index", "ID", "Name", "Rarity",
                "Skill_1 Type", "Skill_1 Sin", "Skill_2 Type", "Skill_2 Sin",
                "Skill_3 Type", "Skill_3 Sin", "Defense Type", "Defense Sin"
            ]
            
            # 테이블 출력
            print(tabulate(table, headers=headers, tablefmt="fancy_grid"))

    def export_to_csv(self, filename, fieldnames, data):
        """
        공통적으로 사용 가능한 CSV 내보내기 메서드
        :param filename: 내보낼 파일 이름 (예: 'identity.csv', 'skill.csv')
        :param fieldnames: CSV의 헤더 리스트
        :param data: CSV로 내보낼 데이터 (리스트 형태)
        """
        try:
            # 현재작업 디렉토 기준으 'csv'폴더 없으면 생성
            if not os.path.exists('./csv'):
                os.makedirs('./csv')
            # 파일 경로 설정 (상대경로 'csv' 폴더 안에 저장)
            file_path = os.path.join('./csv', filename)
            with open(file_path, mode='w', newline='', encoding='utf-8') as file:
                writer = csv.DictWriter(file, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(data)
            print(f"Data successfully exported to {filename}")
        except Exception as e:
            print(f"Error exporting to CSV: {e}")

    def get_identity_data(self):
        """
        Identity 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "ID": identity.identity_id,
                "Name": identity.name,
                "Rarity": identity.rarity,
                "Skill_1_Type": identity.skill_1_type,
                "Skill_1_Sin_Affinity": identity.skill_1_sin_affinity,
                "Skill_2_Type": identity.skill_2_type,
                "Skill_2_Sin_Affinity": identity.skill_2_sin_affinity,
                "Skill_3_Type": identity.skill_3_type,
                "Skill_3_Sin_Affinity": identity.skill_3_sin_affinity,
                "Defense_Type": identity.defense_type,
                "Defense_Sin_Affinity": identity.defense_sin_affinity,
            }
            for identity in self.identities
        ]
        
    def export_identities_data(self, filename="identities.csv"):
        """Identity 데이터를 CSV로 내보내는 메서드"""
        identity_fieldnames = [
            "ID", "Name", "Rarity",
            "Skill_1_Type", "Skill_1_Sin_Affinity",
            "Skill_2_Type", "Skill_2_Sin_Affinity",
            "Skill_3_Type", "Skill_3_Sin_Affinity",
            "Defense_Type", "Defense_Sin_Affinity"
        ]
        identity_data = self.get_identity_data()  # Identity 데이터 가져오기
        self.export_to_csv(filename, identity_fieldnames, identity_data)  # CSV로 내보내기

    def load_identities_by_id(self, filename, identity_ids):
        """지정된 identity_ids 목록에 해당하는 Identity 데이터를 CSV에서 읽어옴."""
        identity_ids_set = set(identity_ids) # 중복 제거를 위 Set 사용
        with open(filename, mode='r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                if row["ID"] in identity_ids_set:
                    identity = Identity(
                        identity_id=row["ID"],
                        name=row["Name"],
                        rarity=row["Rarity"],
                        skill_1_type=row["Skill_1_Type"],
                        skill_1_sin_affinity=row["Skill_1_Sin_Affinity"],
                        skill_2_type=row["Skill_2_Type"],
                        skill_2_sin_affinity=row["Skill_2_Sin_Affinity"],
                        skill_3_type=row["Skill_3_Type"],
                        skill_3_sin_affinity=row["Skill_3_Sin_Affinity"],
                        defense_type=row["Defense_Type"],
                        defense_sin_affinity=row["Defense_Sin_Affinity"]
                    )
                    self.identities.append(identity)

    def get_level_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Level": level_stat.level,
                "HP": level_stat.hp,
                "Defense": level_stat.defense,
            }
            for identity in self.identities
            for level_stat in identity.level_stats
        ]

    def export_level_stats_data(self, filename="levelstats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        level_stats_fieldnames = ["Identity_ID", "Level", "HP", "Defense"]
        level_stats_data = self.get_level_stats_data()  # level_stats 데이터 가져오기
        self.export_to_csv(filename, level_stats_fieldnames, level_stats_data)  # CSV로 내보내기

    def get_uptie_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Uptie": uptie_stat.uptie,
                "Speed": uptie_stat.speed,
            }
            for identity in self.identities
            for uptie_stat in identity.uptie_stats
        ]

    def export_uptie_stats_data(self, filename="Uptiestats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        uptie_stats_fieldnames = ["Identity_ID", "Uptie", "Speed"]
        uptie_stats_data = self.get_uptie_stats_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, uptie_stats_fieldnames, uptie_stats_data)  # CSV로 내보내기

    def get_resist_info_data(self):
        """
        모든 Identity의 resist_info 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Resist_Hit": identity.resist_info.resist_hit,
                "Resist_Penetrate": identity.resist_info.resist_penetrate,
                "Resist_Slash": identity.resist_info.resist_slash,
            }
            for identity in self.identities
        ]

    def export_resist_info_data(self, filename="ResistInfos.csv"):
        """ResistInfo 데이터를 CSV로 내보내는 메서드"""
        resist_infos_fieldnames = ["Identity_ID", "Resist_Hit", "Resist_Penetrate", "Resist_Slash"]
        resist_infos_data = self.get_resist_info_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, resist_infos_fieldnames, resist_infos_data)  # CSV로 내보내기

    def export_uptie_stats_data(self, filename="Uptiestats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        uptie_stats_fieldnames = ["Identity_ID", "Uptie", "Speed"]
        uptie_stats_data = self.get_uptie_stats_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, uptie_stats_fieldnames, uptie_stats_data)  # CSV로 내보내기

    def get_stagger_threshold_data(self):
        """
        모든 Identity의 stagger_threshold 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "First_Stagger": identity.stagger_threshold.first_stagger,
                "Second_Stagger": identity.stagger_threshold.second_stagger,
                "Third_Stagger": identity.stagger_threshold.third_stagger,
            }
            for identity in self.identities
        ]

    def export_stagger_threshold_data(self, filename="StaggerThresholds.csv"):
        """StaggerThresholds 데이터를 CSV로 내보내는 메서드"""
        stagger_thresholds_fieldnames = ["Identity_ID", "First_Stagger", "Second_Stagger", "Third_Stagger"]
        stagger_thresholds_data = self.get_stagger_threshold_data()  # stagger_thresholds 데이터 가져오기
        self.export_to_csv(filename, stagger_thresholds_fieldnames, stagger_thresholds_data)  # CSV로 내보내기


async def main():
    data_container = DataContainer()
    scraper = Scraper(container=data_container, base_url="https://limbus.kusoge.xyz", img_url="https://img.kusoge.xyz/limbus/img")
    await scraper.scrape()

await main() 

Added skill: skill 1:Begone...
Updated Uptie for Begone... (Uptie Level: 1): Skill Power: 2, Coin Power: +4
Extracted text values:
[Combat Start] On self and 1 ally with the highest max HP: consume up to 20 Bloodfeast and apply 1 blooming[thornyfall_panicRodionFirst] for every 10 Bloodfeast this unit consumed (2 times per turn)- Prioritizes <Bloodfiend> allies- If the affected ally is a <Bloodfiend>, this unit applies 1 additional blooming[thornyfall_panicRodionFirst]- If this unit failed to consume Bloodfeast, gain 5 Bleed[On Hit] Inflict 1 Bleed[On Hit] Inflict 1 Rupture
No text found.
Updated Uptie for Begone... (Uptie Level: 2): Skill Power: 2, Coin Power: +4
Extracted text values:
[Combat Start] On self and 1 ally with the highest max HP: consume up to 20 Bloodfeast and apply 1 blooming[thornyfall_panicRodionFirst] for every 10 Bloodfeast this unit consumed (2 times per turn)- Prioritizes <Bloodfiend> allies- If the affected ally is a <Bloodfiend>, this unit applies 1 additional b

In [45]:
import os
import csv
import re
import numpy as np
import pandas as pd
from playwright.async_api import async_playwright
from tabulate import tabulate

# Sin Color 이름 변환을 위한 매핑
resource_mapping = {
    "AZURE": "Gloom",
    "VIOLET": "Envy",
    "AMBER": "Sloth",
    "SHAMROCK": "Gluttony",
    "CRIMSON": "Wrath",
    "SCARLET": "Lust",
    "INDIGO": "Pride"
}

def convert_resource(resource_name):
    """Sin Color 이름을 변환하는 함수. 매핑되지 않은 경우 원래 이름을 반환."""
    return resource_mapping.get(resource_name, resource_name)

class Scraper:
    """웹 페이지 크롤링을 위한 클래스"""
    def __init__(self, container, base_url, img_url):
        self.container = container
        self.base_url = base_url
        self.img_url = img_url
        self.uptie_level = 1

    async def handle_request(self, route, request):
        """이미지와 폰트 리소스 로드를 차단."""
        if request.resource_type in ["image", "font"]:
            await route.abort()
        else:
            await route.continue_()
            
    async def scrape(self):
        """크롤링 메인 함수"""
        async with async_playwright() as p:
            browser = await p.chromium.launch(headless=True)
            page = await browser.new_page()
            await page.route("**/*", self.handle_request)
            # Step 1: /identity 페이지에서 기본 정보 크롤링
            #await self.crawl_identity_page(page)

            # Step 2: /identity/{identity_id} 상세 페이지 크롤링
            #identity_ids_to_load = ["10104", "10310", "10710","10808", "10911", "11005"]
            #identity_ids_to_load = ["10104", "10911"]#,"10310", "10710","10808", "11005", "11210"]
            identity_ids_to_load = ["10911"]#,"10310", "10710","10808", "11005", "11210"]
            self.container.load_identities_by_id('./csv/identities.csv', identity_ids_to_load)
            await self.crawl_identity_details_for_all(page)
            #self.container.display_identities()
            #self.container.export_identities_data("identities.csv")
            #self.container.export_level_stats_data("Levelstats.csv")
            #self.container.export_uptie_stats_data("Uptiestats.csv")
            #self.container.export_resist_info_data("ResistInfos.csv")
            #self.container.export_stagger_threshold_data("StaggerThresholds.csv")
            page.set_default_timeout(10000)
            #await page.wait_for_timeout(100000)
            await browser.close()

    async def get_identity_id_from_row(self, row):
        """테이블 행(row)에서 인격 ID를 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_url = await identity_cell.get_attribute('href')
            identity_id = identity_url.split('/identity/')[-1]
            return identity_id
        return "Unknown"

    async def get_identity_name_from_row(self, row):
        """테이블 행(row)에서 인격 이름을 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_name = (await identity_cell.inner_text()).replace("\n", " ").strip()
            return identity_name
        return "Unknown"
        
    async def crawl_identity_page(self, page):
        """/identity 페이지로 이동하여 인격 기본 정보를 추출."""
        await page.goto(self.base_url + "/identity")
        await self.extract_identity_page(page)

    async def extract_identity_page(self, page):
        """/identity 페이지의 모든 테이블 행에서 인격 정보를 추출."""
        identity_rows = await page.query_selector_all('tr')
        if len(identity_rows) > 1:
            for idx, identity_row in enumerate(identity_rows):
                if idx == 0:
                    continue # 0번째요소이름을 보여주는 열은 건너뜀
                await self.scrape_identity_name_and_identity_id(identity_row)
                await self.extract_rarity(identity_row)
                for i in range(1, 4):  # 스킬1~3 정보 추출
                    await self.extract_atk_and_resource(identity_row, i)
                await self.extract_defense_and_resource(identity_row)

    async def scrape_identity_name_and_identity_id(self, row):
        """인격 번호와 이름을 추출하여 DataContainer에 저장."""
        try:
            identity_id = await self.get_identity_id_from_row(row)
            identity_name = await self.get_identity_name_from_row(row)
            identity = Identity(identity_id)
            self.container.add_identity(identity)
            self.container.update_identity_name(identity_id, identity_name)
        except Exception as e:
            print(f"Error extracting identity name and identity id: {e}")

    async def get_element_attribute(self, element, selector, attribute):
        """요소에서 지정된 속성(attribute)을 추출."""
        try:
            target_element = await element.query_selector(selector)
            if target_element:
                return await target_element.get_attribute(attribute)
        except Exception as e:
            print(f"Error getting element attribute: {e}")
        return None
    
    async def extract_rarity(self, row):
        """테이블 행(row)에서 희귀도를 추출하여 업데이트."""
        try:
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
                sibling = await parent.evaluate_handle('node => node.nextElementSibling')
                if sibling:
                    rarity_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/rarity_"]', 'src')
                    if rarity_img_src:
                        rarity_level = rarity_img_src.split('_')[-1].split('.')[0]  # rarity_1.png -> 1
                        identity_id = await self.get_identity_id_from_row(row)
                        self.container.update_identity_rarity(identity_id, rarity_level)
                    else:
                        print("Rarity image not found in the next sibling TD.")
        except Exception as e: 
            print(f"Error extracting rarity: {e}")

        return "Unknown"

    async def extract_atk_and_resource(self, row, skill_number):
        """ 공격 타입(ATK Type)과 자원(Attribute) 추출 함수 - 스킬 번호에 따라 처리 """
        try:
            #print(f"Attempting to locate skill{skill_number} type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 해당 스킬 번호에 맞는 형제로 이동
                sibling = parent
                for _ in range(skill_number + 1):  # 스킬 번호에 맞게 이동
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                atk_type, resource = "Unknown", "Unknown"
        
                if sibling:
                    # 공격 타입 이미지 (atk_type) 찾기
                    atk_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/atk_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if atk_type_img_src:
                        atk_type = atk_type_img_src.split('_')[-1].split('.')[0]  # atk_type_SLASH.png -> SLASH
                        #print(f"Extracted attack type: {atk_type}")
                        self.container.update_skill(identity_id, skill_number, atk_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted resource: {resource}")
                        
                        # 리소스를 변환
                        resource = convert_resource(resource)
                        #print(f"Converted resource: {resource}")
                        self.container.update_sin_affinity(identity_id, skill_number, resource)
                        
        except Exception as e:
            print(f"Error extracting skill{skill_number} type and resource: {e}")

    async def extract_defense_and_resource(self, row):
        """ 방어 타입(Def Type)과 자원(Attribute) 추출 함수 """
        try:
            #print("Attempting to locate defense type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 다섯 번째 형제 td로 이동 (방어 관련 정보가 있는 <td>)
                sibling = parent
                for _ in range(5):  # 방어 정보는 5번째 형제에 있음
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                def_type, def_resource = "Unknown", "Unknown"
        
                if sibling:
                    # 방어 타입 이미지 (def_type) 찾기
                    def_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/def_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if def_type_img_src:
                        def_type = def_type_img_src.split('_')[-1].split('.')[0]  # def_type_GUARD.png -> GUARD
                        #print(f"Extracted defense type: {def_type}")
                        self.container.update_skill(identity_id, 'defense', def_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        def_resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted defense resource: {def_resource}")
                        
                        # 방어 리소스를 변환
                        def_resource = convert_resource(def_resource)
                        #print(f"Converted defense resource: {def_resource}")
                        self.container.update_sin_affinity(identity_id, 'defense', def_resource)
 
        except Exception as e:
            print(f"Error extracting defense type and resource: {e}")

    async def crawl_identity_details_for_all(self, page):
        """모든 인격의 세부 정보를 크롤링."""
        for identity in self.container.identities:
            await page.wait_for_load_state("networkidle")
            await page.goto(self.base_url + "/identity/"+identity.identity_id)
            """ 레벨과 동기화와 관련 없는 참관타 저항은 밑에서 변경하는 것과 연관성이 적기 때문에 위에서 처리한다.
            print(f"Identity_id: {identity.identity_id}")
            results = []
            resist_type_imgs = [f"{self.img_url}/ui/atk_type_HIT.png",
                                f"{self.img_url}/ui/atk_type_PENETRATE.png",
                                f"{self.img_url}/ui/atk_type_SLASH.png"]
            for resist_type_img in resist_type_imgs:
                result = await self.extract_resist_type(page, resist_type_img, 1)
                results.append(result)
            self.container.add_resist_info(identity.identity_id, results[0], results[1], results[2])
            """
            """
            # Stagger Threshold 값 추출
            stagger_threshold = await self.extract_stagger_threshold_values(page)
            self.container.add_stagger_threshold(identity.identity_id, stagger_threshold[0], stagger_threshold[1], stagger_threshold[2])
            """
            await self.initialize_uptie_and_level(page)
            """ 레벨에 따른 level, hp, defense 다르게 되어 있는 것을 출력 확인 객체로 들어가서 csv로 옮기는 과정까지 진행
            for i in range(1, identity.level_max+1):
                await self.move_slider_to_value(page,i)
                await self.extract_level_status(page, identity.identity_id, i)
            """
            """ 동기화에 따른 speed 다르게 되어 있는 것을 출력 확인 객체로 들어가서 csv로 옮기는 과정까지 진행
            for i in range(1, identity.uptie_max+1):
                await self.uptie_click(page,i)
                await self.extract_uptie_status(page, identity.identity_id, i)
            """
            await self.crawl_skills(page, identity.identity_id)
            
    async def initialize_uptie_and_level(self, page):
        await self.uptie_click(page, 1)
        await self.move_slider_to_value(page, 1)
        self.uptie_level = 1
        #return 1
    
    async def uptie_click(self, page, phase):
        fab_button_selector = ".q-btn.q-btn--fab"
        await page.wait_for_selector(fab_button_selector)
        await page.click(fab_button_selector)
        #print(f"[{setup_uptie_button}] 상위 동기화 버튼 클릭 완료")
        setup_uptie_button = f'a:has(img[src="{self.img_url}/ui/uptie_{phase}.png"])'
        await page.wait_for_selector(setup_uptie_button)
        await page.click(setup_uptie_button)
        #print(f"{phase} 동기화 버튼 클릭 완료")

    async def move_slider_to_value(self, page, target_value):
        """
        슬라이더를 주어진 값으로 이동시킵니다.
        target_value는 슬라이더의 최소값부터 최대값 사이여야 합니다.
        """
        slider_selector = ".q-slider"
        # 슬라이더의 범위 확인
        min_value = int(await page.get_attribute(slider_selector, "aria-valuemin"))
        max_value = int(await page.get_attribute(slider_selector, "aria-valuemax"))
    
        if target_value < min_value or target_value > max_value:
            raise ValueError(f"Target value {target_value} is out of bounds ({min_value}, {max_value})")
    
        # 슬라이더 thumb 요소와 위치 가져오기
        slider_thumb = await page.query_selector(f"{slider_selector} .q-slider__thumb")
        thumb_box = await slider_thumb.bounding_box()
        
        # 슬라이더의 현재 위치 및 목표 위치 계산
        track = await page.query_selector(f"{slider_selector} .q-slider__track")
        track_box = await track.bounding_box()
        
        # 목표 x 위치 계산 (슬라이더 트랙 내에서 비율 기반)
        target_percentage = (target_value - min_value) / (max_value - min_value)
        target_x = track_box["x"] + target_percentage * track_box["width"]
        current_x = thumb_box["x"] + thumb_box["width"] / 2
        y = thumb_box["y"] + thumb_box["height"] / 2
    
        # 슬라이더를 드래그하여 목표 위치로 이동
        await page.mouse.move(current_x, y)
        await page.mouse.down()
        await page.mouse.move(target_x, y, steps=10)  # 부드럽게 이동
        await page.mouse.up()
    
        #print(f"슬라이더를 {target_value}로 이동 완료.")
    
        # 슬라이더 값 확인 및 강제 이벤트 트리거
        await page.evaluate(
            f"""
            (slider) => {{
                slider.dispatchEvent(new Event('input'));  // oninput 이벤트 트리거
                slider.dispatchEvent(new Event('change'));  // onchange 이벤트 트리거
            }}
            """,
            await page.query_selector(slider_selector)
        )
    
        # 이동 후 값 확인
        current_value = await page.get_attribute(slider_selector, "aria-valuenow")
        if current_value != str(target_value):
            print(f"경고: 슬라이더 값이 예상과 다릅니다. 현재 값: {current_value}, 예상 값: {target_value}")

    async def get_second_status_element(self, page):
        """
        두 번째 'Status' 요소에 접근하여 반환하는 메소드
        """
        status_elements = await page.query_selector_all("text=Status")
        if len(status_elements) > 1:
            second_status_element = status_elements[1]
            #print("두 번째 'Status' 요소에 접근합니다.")
            return second_status_element
        else:
            #print("두 번째 'Status' 요소가 존재하지 않습니다.")
            return None

    async def extract_level_status(self, page, identity_id, level):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 레벨에 따라 HP와 Defense가 변한다.
        hp_value = await self.extract_hp(second_status_element)
        defense_value = await self.extract_defense(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Level={level}, HP={hp_value}, Defense={defense_value}")
        self.container.add_level_stat(identity_id,level, hp_value, defense_value)
    
    async def extract_hp(self, status_element):
        # HP 추출
        hp_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].children[0].nextElementSibling.textContent.trim()"
        )
        return hp_value
    
    async def extract_defense(self, status_element):
        # Defense 추출
        defense_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return defense_value

    async def extract_uptie_status(self, page, identity_id, uptie):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 uptie에 따라 speed는 변한다.
        speed_value = await self.extract_speed(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Uptie={uptie}, Speed={speed_value}")
        self.container.add_uptie_stat(identity_id,uptie, speed_value)

    async def extract_speed(self, status_element):
        # Speed 추출
        speed_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return speed_value

    async def extract_resist_type(self, page, image_src, depth):
        # 이미지 요소 찾기
        #print(image_src)
        image = await page.query_selector(f'img[src="{image_src}"]')
        #print(image)
        if not image:
            return "Unknown"
    
        # 이미지의 부모 요소를 찾아서 올라감
        parent = image
        for _ in range(depth):
            parent = await parent.evaluate_handle('node => node.parentElement')
            if not parent:
                return "Unknown"
    
        # 부모의 형제 요소 찾기
        sibling = await parent.evaluate_handle('node => node.nextElementSibling')
        
        if sibling:
            value = await sibling.evaluate('node => node.textContent')
            if value:
                return value.strip()
        
        return "Unknown"

    async def extract_stagger_threshold_values(self, page):
        # 'div.row.center.flex-center.q-pt-xs' 요소들 찾기
        elements = await page.query_selector_all('div.row.center.flex-center.q-pt-xs')
        
        # 각 요소의 텍스트 내용 출력하여 'Stagger Threshold' 요소를 찾기
        for element in elements:
            text = await element.text_content()
            
            if text and "Stagger Threshold" in text:
                # 'Stagger Threshold'가 있는 요소를 찾았다면, 그 이후 형제 요소들에서 값을 추출
                parent = await element.evaluate_handle('node => node.parentElement')
                sibling_elements = await parent.query_selector_all('div.row.justify-evenly div')  # Threshold 값이 들어있는 div들
                
                stagger_values = []
                for sibling in sibling_elements:
                    text_content = await sibling.text_content()
                    match = re.search(r'(\d+%)', text_content)
                    if match:
                        stagger_values.append(match.group(1))  # 퍼센트 값만 리스트에 추가
                while len(stagger_values) < 3:
                    stagger_values.append(np.nan)
                #print(stagger_values)
                return stagger_values  # 추출된 값(또는 np.nan 포함)을 반환
        
        return [np.nan, np.nan, np.nan]  # 만약 'Stagger Threshold'를 찾지 못한 경우 기본값 반환

    async def crawl_skills(self, page, identity_id):
        await self.crawl_skill(page, identity_id, "skill 1")
        await self.crawl_skill(page, identity_id, "skill 2")
        await self.crawl_skill(page, identity_id, "skill 3")
        await self.crawl_skill(page, identity_id, "defense")


    async def crawl_skill(self, page, identity_id, skill_type):
        skills_elements = await page.query_selector_all('div.skilltag')
        for skill_element in skills_elements:
            skill_text = await skill_element.text_content()
            if skill_text.strip().lower() == skill_type:
                await self.process_skill(skill_element, page, identity_id, skill_type)
        
    async def process_skill(self, skill_element, page, identity_id, skill_type):
        # skill_type이 "skill 3"일 경우 uptie_value가 3 이상일 때만 정보를 저장
        for uptie_value in range(1, 5):  # Uptie levels 1 to 4
            # "skill 3"은 uptie_value가 3 이상일 때만 처리
            if skill_type == "skill 3" and uptie_value < 3:
                continue  # skip if uptie_value is less than 3 for skill 3
    
            # uptie_level에 맞게 클릭
            await self.uptie_click(page, uptie_value)
            if skill_type != "skill 3" and uptie_value > 1:
                # Uptie 정보 갱신
                await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
                continue
            elif skill_type == "skill 3" and uptie_value > 3:
                # Uptie 정보 갱신
                await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
                continue
            # 스킬의 세부 정보를 추출
            skill_details = await self.extract_skill_details(skill_element, skill_type)
            skill_name = skill_details["skill_name"]
    
            # 스킬 정보를 저장
            self.container.add_skill(
                identity_id, skill_type, 
                skill_name, skill_details["coin_count"],
                skill_details["init_level"], skill_details["init_remain"], 
                skill_details["atk_weight"]
            )
            print(f"Added skill: {skill_type}:{skill_name}")
    
            # Uptie 정보 갱신
            await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
    
    async def extract_skill_details(self, skill_element, skill_type):
        # 스킬 세부 정보를 추출하는 함수
        skill_name = await self.skill_name(skill_element)
        coin_count = await self.coin_count(skill_element, ".col-auto > .column > .row")
        init_level = await self.init_level(skill_element)
        init_remain = await self.init_remain(skill_element, skill_type, "div.row.justify-start > div.attr.flex.flex-center > div.q-px-sm")
        atk_weight = await self.atk_weight(skill_element, skill_type)
        return {
            "skill_name": skill_name,
            "coin_count": coin_count,
            "init_level": init_level,
            "init_remain": init_remain,
            "atk_weight": atk_weight
        }
    
    async def update_uptie_info(self, page, skill_element, identity_id, skill_name, skill_type, uptie_value):
        # Uptie 정보를 추출하고 업데이트하는 함수
        uptie_info = await self.extract_uptie_info(skill_element)
        self.container.add_skill_uptie_info(
            identity_id, skill_name, 
            uptie_info["skill_power"], uptie_info["coin_power"], 
            uptie_value
        )
        print(
            f"Updated Uptie for {skill_name} (Uptie Level: {uptie_value}): "
            f"Skill Power: {uptie_info['skill_power']}, Coin Power: {uptie_info['coin_power']}"
        )
        all_text = await self.extract_all_text(skill_element)
        if all_text:
            print("Extracted text:")
            print(all_text)
        else:
            print("No text found.")

    async def extract_uptie_info(self, skill_element):
        skill_data = await self.skill_and_coin_power(skill_element)
        return {
            "skill_power": skill_data.get("skillPower"),
            "coin_power": skill_data.get("coinPower")
        }

    async def extract_skill_effect(self, skill_element):
        # Create the Effect object
        effect_obj = Effect(condition=None)
        # Extract effects
        effects = await self.skill_effects(skill_element)
        if effects:
            effect_obj.effects.append(effects)
        return effect_obj

    async def skill_and_coin_power(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                // 조부모로 이동
                const grandparent = node.parentElement.parentElement;
                // 이전 형제 탐색
                const previousSibling = grandparent.previousElementSibling;
                if (!previousSibling) return {{ skillPower: null, coinPower: null }}; // 이전 형제가 없으면 기본값 반환
                // 이전 형제 내부의 .row.justify-around > div > div 탐색
                const targetDivs = previousSibling.querySelectorAll(".row.justify-around > div");
                if (targetDivs.length < 3) return {{ skillPower: null, coinPower: null }}; // 배열 길이 확인
                // 1st div: 스킬 위력
                const skillPower = targetDivs[0]?.querySelector('div')?.textContent?.trim() || null; 
                // 3rd div: 코인 파워 값 (3번째 div에서 텍스트 추출)
                const coinPower = targetDivs[2]?.textContent?.trim() || null;
                return {{ skillPower, coinPower }};
            }}
        """)


    async def coin_count(self, skill_element, hierarchy_selector):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector('{hierarchy_selector}');
                if (!targetElement) return 0;
                return targetElement.childElementCount;
            }}
        """)

    async def init_remain(self, skill_element, skill_type, hierarchy_selector):
        if skill_type == "defense":
            return -1
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector('{hierarchy_selector}');
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def skill_name(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector(".block");
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def init_level(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector("div.text-bold.text-h6.h6.q-pl-xs.q-pr-md");
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def atk_weight(self, skill_element, skill_type):
        if skill_type == "defense":
            return 0
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector("div.text-bold.text-h6.h6.q-pl-xs.q-pr-md");
                const doubleNextSibling = targetElement.nextElementSibling.nextElementSibling;
                if (!doubleNextSibling) return 0;
                return doubleNextSibling.textContent?.trim();
            }}
        """)
        
    async def extract_all_text(self, skill_element):
        return await skill_element.evaluate("""
            node => {
                const parent = node.parentElement;
                const doubleNextSibling = parent?.nextElementSibling?.nextElementSibling;
                if (!doubleNextSibling) return null;
    
                // 모든 텍스트 노드 추출
                const textNodes = doubleNextSibling.querySelectorAll("div.col");
                const texts = Array.from(textNodes)
                    .map(el => el.textContent?.trim()) // 텍스트 추출
                    .filter(text => text && text.length > 0); // 빈 텍스트 제거
    
                // 중복 제거를 Set으로 더 간단하게 처리
                const uniqueTexts = [...new Set(texts)];
    
                // 줄바꿈으로 텍스트 결합
                return uniqueTexts.join("\\n");
            }
        """)
'''
    async def skill_effects(self, skill_element):
        return await skill_element.evaluate("""
            node => {
                const parent = node.parentElement;
                const doubleNextSibling = parent.nextElementSibling.nextElementSibling;
                if (!doubleNextSibling) return null;
    
                // effects 텍스트가 포함된 노드 찾기
                const effectElements = doubleNextSibling.querySelectorAll("span, a");
                if (!effectElements || effectElements.length === 0) return null;
    
                return Array.from(effectElements)
                    .map(el => {
                        if (el.tagName === "A") {
                            // 키워드가 포함된 <a> 태그 처리
                            const keywordSpan = el.querySelector("span");
                            const keyword = keywordSpan ? keywordSpan.textContent.trim() : null;
                            return keyword ? `keyword:${keyword}` : null;
                        } else {
                            // 일반 텍스트 처리
                            return el.textContent?.trim();
                        }
                    })
                    .filter((text, index, self) => text && self.indexOf(text) === index) // 중복 제거
                    .join(" "); // 효과를 하나의 문자열로 결합
            }
        """)
        '''


class Identity:
    """인격 정보를 저장하는 클래스"""
    def __init__(self, identity_id, name=None, rarity=None,\
                 skill_1_type=None, skill_1_sin_affinity=None,\
                 skill_2_type=None, skill_2_sin_affinity=None,\
                 skill_3_type=None, skill_3_sin_affinity=None,\
                 defense_type=None, defense_sin_affinity=None):
        self.identity_id = identity_id
        self.name = name
        self.rarity = rarity
        # 선언만 하는 부분 
        self.skill_1_type = skill_1_type 
        self.skill_1_sin_affinity = skill_1_sin_affinity 
        self.skill_2_type = skill_2_type 
        self.skill_2_sin_affinity = skill_2_sin_affinity 
        self.skill_3_type = skill_3_type 
        self.skill_3_sin_affinity = skill_3_sin_affinity 
        self.defense_type = defense_type 
        self.defense_sin_affinity = defense_sin_affinity
        self.level_max = 50
        self.uptie_max = 4
        self.level_stats = []  # LevelStat 객체 리스트
        self.uptie_stats = [] # UptieStat 객체 리스트
        self.resist_info = None
        self.stagger_threshold = None
        self.skillset = SkillSet()  # Skill정보를 담고 있는 skillset 객체 
        self.passive_infos = []  # 패시브 객체를 담고 있는 배열
        self.support_passive_infos = [] # 서포트 패시 객체를 담고 있는 배열
        self.panic_type = [] # 혼란상태 발생시 나타나는 효과를 담고 있는 객체 배열

    def update_name(self, name):
        self.name = name

    def update_rarity(self, rarity):
        self.rarity = rarity

    def add_level_stat(self, level, hp, defense):
        self.level_stats.append(LevelStat(level, hp, defense))

    def add_uptie_stat(self, uptie, speed):
        self.uptie_stats.append(UptieStat(uptie, speed))

    def add_resist_info(self, resist_hit, resist_penetrate, resist_slash):
        self.resist_info = ResistInfo(resist_hit, resist_penetrate, resist_slash)

    def add_stagger_threshold(self, first_stagger, second_stagger, third_stagger):
        self.stagger_threshold = StaggerThreshold(first_stagger, second_stagger, third_stagger)

    def add_skill(self, skill_code, skill_name, coin_count, init_level, init_remain, akt_weight):
        self.skillset.add_skill(skill_code, Skill(skill_code, skill_name, coin_count, init_level, init_remain, akt_weight))
        
    def add_skill_uptie_info(self, skill_name, uptie_level, skill_power, coin_power):
        self.skillset.add_skill_uptie_info(skill_name, uptie_level, skill_power, coin_power)
        
    def update_details(self, page):
        # 세부 정보 업데이트 로직 (가정)
        pass
    #def init_uptie(self, page):
    #    init_uptie = 

# LevelStat 객체 정의
class LevelStat:
    def __init__(self, level, hp=None, defense=None):
        self.level = level
        self.hp = hp
        self.defense = defense
    def update_level(self, level):
        self.level = level
    def update_hp(self, hp):
        self.hp = hp
    def update_defense(self, defense):
        self.defense = defense

# UptieStat 객체 정의
class UptieStat:
    def __init__(self, uptie, speed=None):
        self.uptie = uptie
        self.speed = speed
    def update_speed(self, speed):
        self.speed = speed

# ResistInfo 객체 정의
class ResistInfo:
    def __init__(self, resist_hit=None, resist_penetrate=None, resist_slash=None):
        self.resist_hit = resist_hit
        self.resist_penetrate = resist_penetrate
        self.resist_slash = resist_slash

# ResistInfo 객체 정의
class StaggerThreshold:
    def __init__(self, first_stagger=None, second_stagger=None, third_stagger=None):
        self.first_stagger = first_stagger
        self.second_stagger = second_stagger
        self.third_stagger = third_stagger

class SkillSet:
    def __init__(self):

        self.skill_1_type = ""
        self.skill_2_type = ""
        self.skill_3_type = ""
        self.defense_skill_type = ""

        self.skill_1_sin_affinity = ""
        self.skill_2_sin_affinity = ""
        self.skill_3_sin_affinity = ""
        self.defense_skill_sin_affinity = ""

        self.skill_1_details = []
        self.skill_2_details = []
        self.skill_3_details = []
        self.defense_skill_details = []

    def set_skill_type(self, skill_code, skill_type, sin_affinity):
        if skill_code == "skill 1":
            self.skill_1_type = skill_type
            self.skill_1_sin_affinity = sin_affinity
        elif skill_code == "skill 2":
            self.skill_2_type = skill_type
            self.skill_2_sin_affinity = sin_affinity
        elif skill_code == "skill 3":
            self.skill_3_type = skill_type
            self.skill_3_sin_affinity = sin_affinity
        elif skill_type == "defense":
            self.defense_skill_type = skill_type
            self.defense_skill_sin_affinity = sin_affinity
    
    def add_skill(self, skill_code, skill):
        if skill_code == "skill 1":
            self.skill_1_details.append(skill)
        elif skill_code == "skill 2":
            self.skill_2_details.append(skill)
        elif skill_code == "skill 3":
            self.skill_3_details.append(skill)
        elif skill_code == "defense":
            self.defense_skill_details.append(skill)

    def get_skill_by_name(self, skill_name):
        for skill_list in [self.skill_1_details, self.skill_2_details, self.skill_3_details, self.defense_skill_details]:
            for skill in skill_list:
                result = skill.is_skill_by_name(skill_name)
                if result:
                    return result
        return None

    def add_skill_uptie_info(self, skill_name, uptie_level, skill_power, coin_power):
        skill = self.get_skill_by_name(skill_name)
        skill.add_uptie_info(uptie_level, skill_power, coin_power)
        

class Skill:
    def __init__(self, skill_code, skill_name, coin_count, init_level, init_remain, akt_weight):
        self.skill_code = skill_code # "Skill 1"/"Skill 2"/"Skill 3"/"Defense" '''o'''
        self.skill_name = skill_name # 스킬 이름 '''o'''
        self.coin_count = coin_count # 동전 개수 '''o'''
        self.init_level = init_level # 스킬이 초기 배정받은 레벨, 인격 레벨에 - 1 한 값을 더해주면 스킬의 현 level을 구할 수 있다. '''o'''
        """
        남아있는 스킬 개수 사용하면, 하나씩 줄어들고 모든 스킬을 사용하면 이 값으로 돌아간다.
        스킬의 남은 숫자없는 스킬은 강화된 스킬로 한번 사용되면 다 조건을 채울 때까 사용 못한다.
        강화된 스킬을 None으로 설정하는 방식과 조건이 채워지면 1로 늘리는 방식이 존재한다.
        스킬을 사용할 수 있는 초기값이다.
        조건을 충족한 강화한 스킬을 이례적인 절차라고 생각하고 None으로 초기값을 설정했다.
        """
        self.init_remain = None #'''o'''
        self.atk_weight = None #공격 가중치 값이다. 방어스킬에는 없기 때문에 None으로 설정했다. '''o'''
        self.remain_count = init_remain # 실제 스킬이 작동하며 횟수가 차감되 나중에 모두 차감되면 다음 턴에 다시 초기횟수 리필된다.
        """
        동기화 단계에 따른 정보를 담는다. 현재 4동기화까지 되어 있지만 차후 확장될 것을 생각하여 고정하지 않는다.
        내부에 들어있는 정보로는 uptie_level, skill_power, coin_power 속성으로 들어있다.
        전투, 턴, 스킬, 코인토스 단계로 내려오며 발생하는 효과에 관하여 저장한다.
        """
        self.uptie_info = []  

    def is_skill_by_name(self, skill_name):
        if self.skill_name == skill_name:
            return self
        return None
        
    def add_uptie_info(self, uptie_level, skill_power, coin_power):
        self.uptie_info.append(UptieInfo(uptie_level, skill_power, coin_power))

class UptieInfo:
    def __init__(self, uptie_level, skill_power, coin_power):
        self.uptie_level = uptie_level   #'''o'''
        self.skill_power = skill_power   #'''o'''
        self.coin_power = coin_power     #'''o'''
        self.skill_effects = []

class Effect:
    def __init__(self, condition):
        self.condition = condition # 조건 텍스트 "[coin 1 on hit]"
        self.effects = [] # 효과 설명 텍스 "Inflict +1 Rupture Count

# 추상화된 EffectType 클래스
class EffectType:
    def __init__(self, name, description):
        self.name = name  # Effect 이름
        self.description = description  # Effect 설명

# Passive 클래스
class Passive(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)  # condition이 있는지 여부 확인

# SupportPassive 클래스
class SupportPassive(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)

# PanicType 클래스
class PanicType(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)

# Keyword 클래스
class Keyword:
    def __init__(self, keyword, explain=None, dispellable=None):
        self.keyword = keyword
        self.explain = explain
        self.dispellable = dispellable

      
class DataContainer:
    """크롤링 데이터를 관리하는 클래스"""
    def __init__(self):
        self.identities = []

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.level_max

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.uptie_max

    def update_identity_name(self, identity_id, identity_name):
        """특정 ID의 이름을 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_name(identity_name)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_identity_rarity(self, identity_id, rarity):
        """특정 ID의 희귀도를 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_rarity(rarity)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_skill(self, identity_id, skill_number, new_type):
        """
        특정 identity의 skill_type을 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_type: 새 skill_type
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_type = new_type
                elif skill_number == 2:
                    identity.skill_2_type = new_type
                elif skill_number == 3:
                    identity.skill_3_type = new_type
                elif skill_number == "defense":
                    identity.defense_type = new_type
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def update_sin_affinity(self, identity_id, skill_number, new_affinity):
        """
        특정 identity의 sin_affinity를 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_affinity: 새 sin_affinity
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_sin_affinity = new_affinity
                elif skill_number == 2:
                    identity.skill_2_sin_affinity = new_affinity
                elif skill_number == 3:
                    identity.skill_3_sin_affinity = new_affinity
                elif skill_number == "defense":
                    identity.defense_sin_affinity = new_affinity
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def add_identity(self, identity):
        """새로운 인격 데이터를 추가."""
        self.identities.append(identity)

    def add_level_stat(self, identity_id, level, hp, defense):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_level_stat(level, hp, defense)

    def add_uptie_stat(self, identity_id, uptie, speed):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_uptie_stat(uptie, speed)

    def add_resist_info(self, identity_id, resist_hit, resist_penetrate, resist_slash):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_resist_info(resist_hit, resist_penetrate, resist_slash)

    def add_stagger_threshold(self, identity_id, first_stagger, second_stagger, third_stagger):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_stagger_threshold(first_stagger, second_stagger, third_stagger)

    
    def add_skill(self, identity_id, skill_code, skill_name, coin_count, init_level, init_remain, akt_weight):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_code == "skill 1":
                    identity.skillset.set_skill_type(skill_code, identity.skill_1_type, identity.skill_1_sin_affinity)
                elif skill_code == "skill 2":
                    identity.skillset.set_skill_type(skill_code, identity.skill_2_type, identity.skill_2_sin_affinity)
                elif skill_code == "skill 3":
                    identity.skillset.set_skill_type(skill_code, identity.skill_3_type, identity.skill_3_sin_affinity)
                elif skill_code == "defense":
                    identity.skillset.set_skill_type(skill_code, identity.defense_type, identity.defense_sin_affinity)
                identity.add_skill(skill_code, skill_name, coin_count, init_level, init_remain, akt_weight)

    def add_skill_uptie_info(self, identity_id, skill_name, uptie_level, skill_power, coin_power):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_skill_uptie_info(skill_name, uptie_level, skill_power, coin_power)
        
    def display_identities(self):
        """Identity, ID, Name, Rarity 정보를 출력"""
        if not self.identities:
            print("No identities found.")
        else:
            # 데이터를 리스트로 변환
            table = []
            for idx, identity in enumerate(self.identities, 1):
                table.append([
                    idx, identity.identity_id, identity.name, identity.rarity,
                    identity.skill_1_type, identity.skill_1_sin_affinity,
                    identity.skill_2_type, identity.skill_2_sin_affinity,
                    identity.skill_3_type, identity.skill_3_sin_affinity,
                    identity.defense_type, identity.defense_sin_affinity
                ])
            
            # 테이블 헤더 정의
            headers = [
                "Index", "ID", "Name", "Rarity",
                "Skill_1 Type", "Skill_1 Sin", "Skill_2 Type", "Skill_2 Sin",
                "Skill_3 Type", "Skill_3 Sin", "Defense Type", "Defense Sin"
            ]
            
            # 테이블 출력
            print(tabulate(table, headers=headers, tablefmt="fancy_grid"))

    def export_to_csv(self, filename, fieldnames, data):
        """
        공통적으로 사용 가능한 CSV 내보내기 메서드
        :param filename: 내보낼 파일 이름 (예: 'identity.csv', 'skill.csv')
        :param fieldnames: CSV의 헤더 리스트
        :param data: CSV로 내보낼 데이터 (리스트 형태)
        """
        try:
            # 현재작업 디렉토 기준으 'csv'폴더 없으면 생성
            if not os.path.exists('./csv'):
                os.makedirs('./csv')
            # 파일 경로 설정 (상대경로 'csv' 폴더 안에 저장)
            file_path = os.path.join('./csv', filename)
            with open(file_path, mode='w', newline='', encoding='utf-8') as file:
                writer = csv.DictWriter(file, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(data)
            print(f"Data successfully exported to {filename}")
        except Exception as e:
            print(f"Error exporting to CSV: {e}")

    def get_identity_data(self):
        """
        Identity 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "ID": identity.identity_id,
                "Name": identity.name,
                "Rarity": identity.rarity,
                "Skill_1_Type": identity.skill_1_type,
                "Skill_1_Sin_Affinity": identity.skill_1_sin_affinity,
                "Skill_2_Type": identity.skill_2_type,
                "Skill_2_Sin_Affinity": identity.skill_2_sin_affinity,
                "Skill_3_Type": identity.skill_3_type,
                "Skill_3_Sin_Affinity": identity.skill_3_sin_affinity,
                "Defense_Type": identity.defense_type,
                "Defense_Sin_Affinity": identity.defense_sin_affinity,
            }
            for identity in self.identities
        ]
        
    def export_identities_data(self, filename="identities.csv"):
        """Identity 데이터를 CSV로 내보내는 메서드"""
        identity_fieldnames = [
            "ID", "Name", "Rarity",
            "Skill_1_Type", "Skill_1_Sin_Affinity",
            "Skill_2_Type", "Skill_2_Sin_Affinity",
            "Skill_3_Type", "Skill_3_Sin_Affinity",
            "Defense_Type", "Defense_Sin_Affinity"
        ]
        identity_data = self.get_identity_data()  # Identity 데이터 가져오기
        self.export_to_csv(filename, identity_fieldnames, identity_data)  # CSV로 내보내기

    def load_identities_by_id(self, filename, identity_ids):
        """지정된 identity_ids 목록에 해당하는 Identity 데이터를 CSV에서 읽어옴."""
        identity_ids_set = set(identity_ids) # 중복 제거를 위 Set 사용
        with open(filename, mode='r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                if row["ID"] in identity_ids_set:
                    identity = Identity(
                        identity_id=row["ID"],
                        name=row["Name"],
                        rarity=row["Rarity"],
                        skill_1_type=row["Skill_1_Type"],
                        skill_1_sin_affinity=row["Skill_1_Sin_Affinity"],
                        skill_2_type=row["Skill_2_Type"],
                        skill_2_sin_affinity=row["Skill_2_Sin_Affinity"],
                        skill_3_type=row["Skill_3_Type"],
                        skill_3_sin_affinity=row["Skill_3_Sin_Affinity"],
                        defense_type=row["Defense_Type"],
                        defense_sin_affinity=row["Defense_Sin_Affinity"]
                    )
                    self.identities.append(identity)

    def get_level_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Level": level_stat.level,
                "HP": level_stat.hp,
                "Defense": level_stat.defense,
            }
            for identity in self.identities
            for level_stat in identity.level_stats
        ]

    def export_level_stats_data(self, filename="levelstats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        level_stats_fieldnames = ["Identity_ID", "Level", "HP", "Defense"]
        level_stats_data = self.get_level_stats_data()  # level_stats 데이터 가져오기
        self.export_to_csv(filename, level_stats_fieldnames, level_stats_data)  # CSV로 내보내기

    def get_uptie_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Uptie": uptie_stat.uptie,
                "Speed": uptie_stat.speed,
            }
            for identity in self.identities
            for uptie_stat in identity.uptie_stats
        ]

    def export_uptie_stats_data(self, filename="Uptiestats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        uptie_stats_fieldnames = ["Identity_ID", "Uptie", "Speed"]
        uptie_stats_data = self.get_uptie_stats_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, uptie_stats_fieldnames, uptie_stats_data)  # CSV로 내보내기

    def get_resist_info_data(self):
        """
        모든 Identity의 resist_info 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Resist_Hit": identity.resist_info.resist_hit,
                "Resist_Penetrate": identity.resist_info.resist_penetrate,
                "Resist_Slash": identity.resist_info.resist_slash,
            }
            for identity in self.identities
        ]

    def export_resist_info_data(self, filename="ResistInfos.csv"):
        """ResistInfo 데이터를 CSV로 내보내는 메서드"""
        resist_infos_fieldnames = ["Identity_ID", "Resist_Hit", "Resist_Penetrate", "Resist_Slash"]
        resist_infos_data = self.get_resist_info_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, resist_infos_fieldnames, resist_infos_data)  # CSV로 내보내기

    def export_uptie_stats_data(self, filename="Uptiestats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        uptie_stats_fieldnames = ["Identity_ID", "Uptie", "Speed"]
        uptie_stats_data = self.get_uptie_stats_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, uptie_stats_fieldnames, uptie_stats_data)  # CSV로 내보내기

    def get_stagger_threshold_data(self):
        """
        모든 Identity의 stagger_threshold 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "First_Stagger": identity.stagger_threshold.first_stagger,
                "Second_Stagger": identity.stagger_threshold.second_stagger,
                "Third_Stagger": identity.stagger_threshold.third_stagger,
            }
            for identity in self.identities
        ]

    def export_stagger_threshold_data(self, filename="StaggerThresholds.csv"):
        """StaggerThresholds 데이터를 CSV로 내보내는 메서드"""
        stagger_thresholds_fieldnames = ["Identity_ID", "First_Stagger", "Second_Stagger", "Third_Stagger"]
        stagger_thresholds_data = self.get_stagger_threshold_data()  # stagger_thresholds 데이터 가져오기
        self.export_to_csv(filename, stagger_thresholds_fieldnames, stagger_thresholds_data)  # CSV로 내보내기


async def main():
    data_container = DataContainer()
    scraper = Scraper(container=data_container, base_url="https://limbus.kusoge.xyz", img_url="https://img.kusoge.xyz/limbus/img")
    await scraper.scrape()

await main() 

Added skill: skill 1:Begone...
Updated Uptie for Begone... (Uptie Level: 1): Skill Power: 2, Coin Power: +4
Extracted text:
[Combat Start] On self and 1 ally with the highest max HP: consume up to 20 Bloodfeast and apply 1 blooming[thornyfall_panicRodionFirst] for every 10 Bloodfeast this unit consumed (2 times per turn)- Prioritizes <Bloodfiend> allies- If the affected ally is a <Bloodfiend>, this unit applies 1 additional blooming[thornyfall_panicRodionFirst]- If this unit failed to consume Bloodfeast, gain 5 Bleed[On Hit] Inflict 1 Bleed[On Hit] Inflict 1 Rupture
Updated Uptie for Begone... (Uptie Level: 2): Skill Power: 2, Coin Power: +4
Extracted text:
[Combat Start] On self and 1 ally with the highest max HP: consume up to 20 Bloodfeast and apply 1 blooming[thornyfall_panicRodionFirst] for every 10 Bloodfeast this unit consumed (2 times per turn)- Prioritizes <Bloodfiend> allies- If the affected ally is a <Bloodfiend>, this unit applies 1 additional blooming[thornyfall_panicRodio

In [4]:
import os
import csv
import re
import numpy as np
import pandas as pd
from playwright.async_api import async_playwright
from tabulate import tabulate

# Sin Color 이름 변환을 위한 매핑
resource_mapping = {
    "AZURE": "Gloom",
    "VIOLET": "Envy",
    "AMBER": "Sloth",
    "SHAMROCK": "Gluttony",
    "CRIMSON": "Wrath",
    "SCARLET": "Lust",
    "INDIGO": "Pride"
}

def convert_resource(resource_name):
    """Sin Color 이름을 변환하는 함수. 매핑되지 않은 경우 원래 이름을 반환."""
    return resource_mapping.get(resource_name, resource_name)

class Scraper:
    """웹 페이지 크롤링을 위한 클래스"""
    def __init__(self, container, base_url, img_url):
        self.container = container
        self.base_url = base_url
        self.img_url = img_url
        self.uptie_level = 1

    async def handle_request(self, route, request):
        """이미지와 폰트 리소스 로드를 차단."""
        if request.resource_type in ["image", "font"]:
            await route.abort()
        else:
            await route.continue_()
            
    async def scrape(self):
        """크롤링 메인 함수"""
        async with async_playwright() as p:
            browser = await p.chromium.launch(headless=True)
            page = await browser.new_page()
            await page.route("**/*", self.handle_request)
            # Step 1: /identity 페이지에서 기본 정보 크롤링
            #await self.crawl_identity_page(page)

            # Step 2: /identity/{identity_id} 상세 페이지 크롤링
            #identity_ids_to_load = ["10104", "10310", "10710","10808", "10911", "11005"]
            #identity_ids_to_load = ["10104", "10911"]#,"10310", "10710","10808", "11005", "11210"]
            identity_ids_to_load = ["10911"]#,"10310", "10710","10808", "11005", "11210"]
            self.container.load_identities_by_id('./csv/identities.csv', identity_ids_to_load)
            await self.crawl_identity_details_for_all(page)
            #self.container.display_identities()
            #self.container.export_identities_data("identities.csv")
            #self.container.export_level_stats_data("Levelstats.csv")
            #self.container.export_uptie_stats_data("Uptiestats.csv")
            #self.container.export_resist_info_data("ResistInfos.csv")
            #self.container.export_stagger_threshold_data("StaggerThresholds.csv")
            page.set_default_timeout(10000)
            #await page.wait_for_timeout(100000)
            await browser.close()

    async def get_identity_id_from_row(self, row):
        """테이블 행(row)에서 인격 ID를 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_url = await identity_cell.get_attribute('href')
            identity_id = identity_url.split('/identity/')[-1]
            return identity_id
        return "Unknown"

    async def get_identity_name_from_row(self, row):
        """테이블 행(row)에서 인격 이름을 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_name = (await identity_cell.inner_text()).replace("\n", " ").strip()
            return identity_name
        return "Unknown"
        
    async def crawl_identity_page(self, page):
        """/identity 페이지로 이동하여 인격 기본 정보를 추출."""
        await page.goto(self.base_url + "/identity")
        await self.extract_identity_page(page)

    async def extract_identity_page(self, page):
        """/identity 페이지의 모든 테이블 행에서 인격 정보를 추출."""
        identity_rows = await page.query_selector_all('tr')
        if len(identity_rows) > 1:
            for idx, identity_row in enumerate(identity_rows):
                if idx == 0:
                    continue # 0번째요소이름을 보여주는 열은 건너뜀
                await self.scrape_identity_name_and_identity_id(identity_row)
                await self.extract_rarity(identity_row)
                for i in range(1, 4):  # 스킬1~3 정보 추출
                    await self.extract_atk_and_resource(identity_row, i)
                await self.extract_defense_and_resource(identity_row)

    async def scrape_identity_name_and_identity_id(self, row):
        """인격 번호와 이름을 추출하여 DataContainer에 저장."""
        try:
            identity_id = await self.get_identity_id_from_row(row)
            identity_name = await self.get_identity_name_from_row(row)
            identity = Identity(identity_id)
            self.container.add_identity(identity)
            self.container.update_identity_name(identity_id, identity_name)
        except Exception as e:
            print(f"Error extracting identity name and identity id: {e}")

    async def get_element_attribute(self, element, selector, attribute):
        """요소에서 지정된 속성(attribute)을 추출."""
        try:
            target_element = await element.query_selector(selector)
            if target_element:
                return await target_element.get_attribute(attribute)
        except Exception as e:
            print(f"Error getting element attribute: {e}")
        return None
    
    async def extract_rarity(self, row):
        """테이블 행(row)에서 희귀도를 추출하여 업데이트."""
        try:
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
                sibling = await parent.evaluate_handle('node => node.nextElementSibling')
                if sibling:
                    rarity_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/rarity_"]', 'src')
                    if rarity_img_src:
                        rarity_level = rarity_img_src.split('_')[-1].split('.')[0]  # rarity_1.png -> 1
                        identity_id = await self.get_identity_id_from_row(row)
                        self.container.update_identity_rarity(identity_id, rarity_level)
                    else:
                        print("Rarity image not found in the next sibling TD.")
        except Exception as e: 
            print(f"Error extracting rarity: {e}")

        return "Unknown"

    async def extract_atk_and_resource(self, row, skill_number):
        """ 공격 타입(ATK Type)과 자원(Attribute) 추출 함수 - 스킬 번호에 따라 처리 """
        try:
            #print(f"Attempting to locate skill{skill_number} type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 해당 스킬 번호에 맞는 형제로 이동
                sibling = parent
                for _ in range(skill_number + 1):  # 스킬 번호에 맞게 이동
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                atk_type, resource = "Unknown", "Unknown"
        
                if sibling:
                    # 공격 타입 이미지 (atk_type) 찾기
                    atk_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/atk_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if atk_type_img_src:
                        atk_type = atk_type_img_src.split('_')[-1].split('.')[0]  # atk_type_SLASH.png -> SLASH
                        #print(f"Extracted attack type: {atk_type}")
                        self.container.update_skill(identity_id, skill_number, atk_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted resource: {resource}")
                        
                        # 리소스를 변환
                        resource = convert_resource(resource)
                        #print(f"Converted resource: {resource}")
                        self.container.update_sin_affinity(identity_id, skill_number, resource)
                        
        except Exception as e:
            print(f"Error extracting skill{skill_number} type and resource: {e}")

    async def extract_defense_and_resource(self, row):
        """ 방어 타입(Def Type)과 자원(Attribute) 추출 함수 """
        try:
            #print("Attempting to locate defense type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 다섯 번째 형제 td로 이동 (방어 관련 정보가 있는 <td>)
                sibling = parent
                for _ in range(5):  # 방어 정보는 5번째 형제에 있음
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                def_type, def_resource = "Unknown", "Unknown"
        
                if sibling:
                    # 방어 타입 이미지 (def_type) 찾기
                    def_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/def_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if def_type_img_src:
                        def_type = def_type_img_src.split('_')[-1].split('.')[0]  # def_type_GUARD.png -> GUARD
                        #print(f"Extracted defense type: {def_type}")
                        self.container.update_skill(identity_id, 'defense', def_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        def_resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted defense resource: {def_resource}")
                        
                        # 방어 리소스를 변환
                        def_resource = convert_resource(def_resource)
                        #print(f"Converted defense resource: {def_resource}")
                        self.container.update_sin_affinity(identity_id, 'defense', def_resource)
 
        except Exception as e:
            print(f"Error extracting defense type and resource: {e}")

    async def crawl_identity_details_for_all(self, page):
        """모든 인격의 세부 정보를 크롤링."""
        for identity in self.container.identities:
            await page.wait_for_load_state("networkidle")
            await page.goto(self.base_url + "/identity/"+identity.identity_id)
            """ 레벨과 동기화와 관련 없는 참관타 저항은 밑에서 변경하는 것과 연관성이 적기 때문에 위에서 처리한다.
            print(f"Identity_id: {identity.identity_id}")
            results = []
            resist_type_imgs = [f"{self.img_url}/ui/atk_type_HIT.png",
                                f"{self.img_url}/ui/atk_type_PENETRATE.png",
                                f"{self.img_url}/ui/atk_type_SLASH.png"]
            for resist_type_img in resist_type_imgs:
                result = await self.extract_resist_type(page, resist_type_img, 1)
                results.append(result)
            self.container.add_resist_info(identity.identity_id, results[0], results[1], results[2])
            """
            """
            # Stagger Threshold 값 추출
            stagger_threshold = await self.extract_stagger_threshold_values(page)
            self.container.add_stagger_threshold(identity.identity_id, stagger_threshold[0], stagger_threshold[1], stagger_threshold[2])
            """
            await self.initialize_uptie_and_level(page)
            """ 레벨에 따른 level, hp, defense 다르게 되어 있는 것을 출력 확인 객체로 들어가서 csv로 옮기는 과정까지 진행
            for i in range(1, identity.level_max+1):
                await self.move_slider_to_value(page,i)
                await self.extract_level_status(page, identity.identity_id, i)
            """
            """ 동기화에 따른 speed 다르게 되어 있는 것을 출력 확인 객체로 들어가서 csv로 옮기는 과정까지 진행
            for i in range(1, identity.uptie_max+1):
                await self.uptie_click(page,i)
                await self.extract_uptie_status(page, identity.identity_id, i)
            """
            await self.crawl_skills(page, identity.identity_id)
            
    async def initialize_uptie_and_level(self, page):
        await self.uptie_click(page, 1)
        await self.move_slider_to_value(page, 1)
        self.uptie_level = 1
        #return 1
    
    async def uptie_click(self, page, phase):
        fab_button_selector = ".q-btn.q-btn--fab"
        await page.wait_for_selector(fab_button_selector)
        await page.click(fab_button_selector)
        #print(f"[{setup_uptie_button}] 상위 동기화 버튼 클릭 완료")
        setup_uptie_button = f'a:has(img[src="{self.img_url}/ui/uptie_{phase}.png"])'
        await page.wait_for_selector(setup_uptie_button)
        await page.click(setup_uptie_button)
        #print(f"{phase} 동기화 버튼 클릭 완료")

    async def move_slider_to_value(self, page, target_value):
        """
        슬라이더를 주어진 값으로 이동시킵니다.
        target_value는 슬라이더의 최소값부터 최대값 사이여야 합니다.
        """
        slider_selector = ".q-slider"
        # 슬라이더의 범위 확인
        min_value = int(await page.get_attribute(slider_selector, "aria-valuemin"))
        max_value = int(await page.get_attribute(slider_selector, "aria-valuemax"))
    
        if target_value < min_value or target_value > max_value:
            raise ValueError(f"Target value {target_value} is out of bounds ({min_value}, {max_value})")
    
        # 슬라이더 thumb 요소와 위치 가져오기
        slider_thumb = await page.query_selector(f"{slider_selector} .q-slider__thumb")
        thumb_box = await slider_thumb.bounding_box()
        
        # 슬라이더의 현재 위치 및 목표 위치 계산
        track = await page.query_selector(f"{slider_selector} .q-slider__track")
        track_box = await track.bounding_box()
        
        # 목표 x 위치 계산 (슬라이더 트랙 내에서 비율 기반)
        target_percentage = (target_value - min_value) / (max_value - min_value)
        target_x = track_box["x"] + target_percentage * track_box["width"]
        current_x = thumb_box["x"] + thumb_box["width"] / 2
        y = thumb_box["y"] + thumb_box["height"] / 2
    
        # 슬라이더를 드래그하여 목표 위치로 이동
        await page.mouse.move(current_x, y)
        await page.mouse.down()
        await page.mouse.move(target_x, y, steps=10)  # 부드럽게 이동
        await page.mouse.up()
    
        #print(f"슬라이더를 {target_value}로 이동 완료.")
    
        # 슬라이더 값 확인 및 강제 이벤트 트리거
        await page.evaluate(
            f"""
            (slider) => {{
                slider.dispatchEvent(new Event('input'));  // oninput 이벤트 트리거
                slider.dispatchEvent(new Event('change'));  // onchange 이벤트 트리거
            }}
            """,
            await page.query_selector(slider_selector)
        )
    
        # 이동 후 값 확인
        current_value = await page.get_attribute(slider_selector, "aria-valuenow")
        if current_value != str(target_value):
            print(f"경고: 슬라이더 값이 예상과 다릅니다. 현재 값: {current_value}, 예상 값: {target_value}")

    async def get_second_status_element(self, page):
        """
        두 번째 'Status' 요소에 접근하여 반환하는 메소드
        """
        status_elements = await page.query_selector_all("text=Status")
        if len(status_elements) > 1:
            second_status_element = status_elements[1]
            #print("두 번째 'Status' 요소에 접근합니다.")
            return second_status_element
        else:
            #print("두 번째 'Status' 요소가 존재하지 않습니다.")
            return None

    async def extract_level_status(self, page, identity_id, level):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 레벨에 따라 HP와 Defense가 변한다.
        hp_value = await self.extract_hp(second_status_element)
        defense_value = await self.extract_defense(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Level={level}, HP={hp_value}, Defense={defense_value}")
        self.container.add_level_stat(identity_id,level, hp_value, defense_value)
    
    async def extract_hp(self, status_element):
        # HP 추출
        hp_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].children[0].nextElementSibling.textContent.trim()"
        )
        return hp_value
    
    async def extract_defense(self, status_element):
        # Defense 추출
        defense_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return defense_value

    async def extract_uptie_status(self, page, identity_id, uptie):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 uptie에 따라 speed는 변한다.
        speed_value = await self.extract_speed(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Uptie={uptie}, Speed={speed_value}")
        self.container.add_uptie_stat(identity_id,uptie, speed_value)

    async def extract_speed(self, status_element):
        # Speed 추출
        speed_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return speed_value

    async def extract_resist_type(self, page, image_src, depth):
        # 이미지 요소 찾기
        #print(image_src)
        image = await page.query_selector(f'img[src="{image_src}"]')
        #print(image)
        if not image:
            return "Unknown"
    
        # 이미지의 부모 요소를 찾아서 올라감
        parent = image
        for _ in range(depth):
            parent = await parent.evaluate_handle('node => node.parentElement')
            if not parent:
                return "Unknown"
    
        # 부모의 형제 요소 찾기
        sibling = await parent.evaluate_handle('node => node.nextElementSibling')
        
        if sibling:
            value = await sibling.evaluate('node => node.textContent')
            if value:
                return value.strip()
        
        return "Unknown"

    async def extract_stagger_threshold_values(self, page):
        # 'div.row.center.flex-center.q-pt-xs' 요소들 찾기
        elements = await page.query_selector_all('div.row.center.flex-center.q-pt-xs')
        
        # 각 요소의 텍스트 내용 출력하여 'Stagger Threshold' 요소를 찾기
        for element in elements:
            text = await element.text_content()
            
            if text and "Stagger Threshold" in text:
                # 'Stagger Threshold'가 있는 요소를 찾았다면, 그 이후 형제 요소들에서 값을 추출
                parent = await element.evaluate_handle('node => node.parentElement')
                sibling_elements = await parent.query_selector_all('div.row.justify-evenly div')  # Threshold 값이 들어있는 div들
                
                stagger_values = []
                for sibling in sibling_elements:
                    text_content = await sibling.text_content()
                    match = re.search(r'(\d+%)', text_content)
                    if match:
                        stagger_values.append(match.group(1))  # 퍼센트 값만 리스트에 추가
                while len(stagger_values) < 3:
                    stagger_values.append(np.nan)
                #print(stagger_values)
                return stagger_values  # 추출된 값(또는 np.nan 포함)을 반환
        
        return [np.nan, np.nan, np.nan]  # 만약 'Stagger Threshold'를 찾지 못한 경우 기본값 반환

    async def crawl_skills(self, page, identity_id):
        await self.crawl_skill(page, identity_id, "skill 1")
        await self.crawl_skill(page, identity_id, "skill 2")
        await self.crawl_skill(page, identity_id, "skill 3")
        await self.crawl_skill(page, identity_id, "defense")


    async def crawl_skill(self, page, identity_id, skill_type):
        skills_elements = await page.query_selector_all('div.skilltag')
        for skill_element in skills_elements:
            skill_text = await skill_element.text_content()
            if skill_text.strip().lower() == skill_type:
                await self.process_skill(skill_element, page, identity_id, skill_type)
        
    async def process_skill(self, skill_element, page, identity_id, skill_type):
        # skill_type이 "skill 3"일 경우 uptie_value가 3 이상일 때만 정보를 저장
        for uptie_value in range(1, 5):  # Uptie levels 1 to 4
            # "skill 3"은 uptie_value가 3 이상일 때만 처리
            if skill_type == "skill 3" and uptie_value < 3:
                continue  # skip if uptie_value is less than 3 for skill 3
    
            # uptie_level에 맞게 클릭
            await self.uptie_click(page, uptie_value)
            if skill_type != "skill 3" and uptie_value > 1:
                # Uptie 정보 갱신
                await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
                continue
            elif skill_type == "skill 3" and uptie_value > 3:
                # Uptie 정보 갱신
                await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
                continue
            # 스킬의 세부 정보를 추출
            skill_details = await self.extract_skill_details(skill_element, skill_type)
            skill_name = skill_details["skill_name"]
    
            # 스킬 정보를 저장
            self.container.add_skill(
                identity_id, skill_type, 
                skill_name, skill_details["coin_count"],
                skill_details["init_level"], skill_details["init_remain"], 
                skill_details["atk_weight"]
            )
            print(f"Added skill: {skill_type}:{skill_name}")
    
            # Uptie 정보 갱신
            await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
    
    async def extract_skill_details(self, skill_element, skill_type):
        # 스킬 세부 정보를 추출하는 함수
        skill_name = await self.skill_name(skill_element)
        coin_count = await self.coin_count(skill_element, ".col-auto > .column > .row")
        init_level = await self.init_level(skill_element)
        init_remain = await self.init_remain(skill_element, skill_type, "div.row.justify-start > div.attr.flex.flex-center > div.q-px-sm")
        atk_weight = await self.atk_weight(skill_element, skill_type)
        return {
            "skill_name": skill_name,
            "coin_count": coin_count,
            "init_level": init_level,
            "init_remain": init_remain,
            "atk_weight": atk_weight
        }
    
    async def update_uptie_info(self, page, skill_element, identity_id, skill_name, skill_type, uptie_value):
        # Uptie 정보를 추출하고 업데이트하는 함수
        uptie_info = await self.extract_uptie_info(skill_element)
        self.container.add_skill_uptie_info(
            identity_id, skill_name, 
            uptie_info["skill_power"], uptie_info["coin_power"], 
            uptie_value
        )
        print(
            f"Updated Uptie for {skill_name} (Uptie Level: {uptie_value}): "
            f"Skill Power: {uptie_info['skill_power']}, Coin Power: {uptie_info['coin_power']}"
        )
        effect = await self.skill_effects(skill_element)
        print(effect)

    async def extract_uptie_info(self, skill_element):
        skill_data = await self.skill_and_coin_power(skill_element)
        return {
            "skill_power": skill_data.get("skillPower"),
            "coin_power": skill_data.get("coinPower")
        }

    async def extract_skill_effect(self, skill_element):
        # Create the Effect object
        effect_obj = Effect(condition=None)
        # Extract effects
        effects = await self.skill_effects(skill_element)
        if effects:
            effect_obj.effects.append(effects)
        return effect_obj

    async def skill_and_coin_power(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                // 조부모로 이동
                const grandparent = node.parentElement.parentElement;
                // 이전 형제 탐색
                const previousSibling = grandparent.previousElementSibling;
                if (!previousSibling) return {{ skillPower: null, coinPower: null }}; // 이전 형제가 없으면 기본값 반환
                // 이전 형제 내부의 .row.justify-around > div > div 탐색
                const targetDivs = previousSibling.querySelectorAll(".row.justify-around > div");
                if (targetDivs.length < 3) return {{ skillPower: null, coinPower: null }}; // 배열 길이 확인
                // 1st div: 스킬 위력
                const skillPower = targetDivs[0]?.querySelector('div')?.textContent?.trim() || null; 
                // 3rd div: 코인 파워 값 (3번째 div에서 텍스트 추출)
                const coinPower = targetDivs[2]?.textContent?.trim() || null;
                return {{ skillPower, coinPower }};
            }}
        """)


    async def coin_count(self, skill_element, hierarchy_selector):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector('{hierarchy_selector}');
                if (!targetElement) return 0;
                return targetElement.childElementCount;
            }}
        """)

    async def init_remain(self, skill_element, skill_type, hierarchy_selector):
        if skill_type == "defense":
            return -1
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector('{hierarchy_selector}');
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def skill_name(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector(".block");
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def init_level(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector("div.text-bold.text-h6.h6.q-pl-xs.q-pr-md");
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def atk_weight(self, skill_element, skill_type):
        if skill_type == "defense":
            return 0
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector("div.text-bold.text-h6.h6.q-pl-xs.q-pr-md");
                const doubleNextSibling = targetElement.nextElementSibling.nextElementSibling;
                if (!doubleNextSibling) return 0;
                return doubleNextSibling.textContent?.trim();
            }}
        """)

    async def skill_effects(self, skill_element):
        try:
            return await skill_element.evaluate("""
                node => {
                    const parent = node.parentElement;
                    const doubleNextSibling = parent.nextElementSibling.nextElementSibling;
                    if (!doubleNextSibling) return null;
        
                    const textNodes = doubleNextSibling.querySelectorAll("div.col");
                    const texts = Array.from(textNodes)
                        .map(el => el.innerHTML?.trim())
                        .filter(text => text && text.length > 0);
    
                    const updatedTexts = texts.map(text => {
                        return text.replace(/<a[^>]*>(.*?)<\\/a>/g, (match, p1) => {
                            return `${p1}`;
                        });
                    });
        
                    return Array.from(updatedTexts)
                        .map(el => {
                            return el.trim();
                        })
                        .filter((text, index, self) => text && self.indexOf(text) === index) // 중복 제거
                        .join(" "); // 효과를 하나의 문자열로 결합
                }
            """)
        except playwright._impl._errors.TargetClosedError:
            print("Target page, context, or browser has been closed.")
            return None



class Identity:
    """인격 정보를 저장하는 클래스"""
    def __init__(self, identity_id, name=None, rarity=None,\
                 skill_1_type=None, skill_1_sin_affinity=None,\
                 skill_2_type=None, skill_2_sin_affinity=None,\
                 skill_3_type=None, skill_3_sin_affinity=None,\
                 defense_type=None, defense_sin_affinity=None):
        self.identity_id = identity_id
        self.name = name
        self.rarity = rarity
        # 선언만 하는 부분 
        self.skill_1_type = skill_1_type 
        self.skill_1_sin_affinity = skill_1_sin_affinity 
        self.skill_2_type = skill_2_type 
        self.skill_2_sin_affinity = skill_2_sin_affinity 
        self.skill_3_type = skill_3_type 
        self.skill_3_sin_affinity = skill_3_sin_affinity 
        self.defense_type = defense_type 
        self.defense_sin_affinity = defense_sin_affinity
        self.level_max = 50
        self.uptie_max = 4
        self.level_stats = []  # LevelStat 객체 리스트
        self.uptie_stats = [] # UptieStat 객체 리스트
        self.resist_info = None
        self.stagger_threshold = None
        self.skillset = SkillSet()  # Skill정보를 담고 있는 skillset 객체 
        self.passive_infos = []  # 패시브 객체를 담고 있는 배열
        self.support_passive_infos = [] # 서포트 패시 객체를 담고 있는 배열
        self.panic_type = [] # 혼란상태 발생시 나타나는 효과를 담고 있는 객체 배열

    def update_name(self, name):
        self.name = name

    def update_rarity(self, rarity):
        self.rarity = rarity

    def add_level_stat(self, level, hp, defense):
        self.level_stats.append(LevelStat(level, hp, defense))

    def add_uptie_stat(self, uptie, speed):
        self.uptie_stats.append(UptieStat(uptie, speed))

    def add_resist_info(self, resist_hit, resist_penetrate, resist_slash):
        self.resist_info = ResistInfo(resist_hit, resist_penetrate, resist_slash)

    def add_stagger_threshold(self, first_stagger, second_stagger, third_stagger):
        self.stagger_threshold = StaggerThreshold(first_stagger, second_stagger, third_stagger)

    def add_skill(self, skill_code, skill_name, coin_count, init_level, init_remain, akt_weight):
        self.skillset.add_skill(skill_code, Skill(skill_code, skill_name, coin_count, init_level, init_remain, akt_weight))
        
    def add_skill_uptie_info(self, skill_name, uptie_level, skill_power, coin_power):
        self.skillset.add_skill_uptie_info(skill_name, uptie_level, skill_power, coin_power)
        
    def update_details(self, page):
        # 세부 정보 업데이트 로직 (가정)
        pass
    #def init_uptie(self, page):
    #    init_uptie = 

# LevelStat 객체 정의
class LevelStat:
    def __init__(self, level, hp=None, defense=None):
        self.level = level
        self.hp = hp
        self.defense = defense
    def update_level(self, level):
        self.level = level
    def update_hp(self, hp):
        self.hp = hp
    def update_defense(self, defense):
        self.defense = defense

# UptieStat 객체 정의
class UptieStat:
    def __init__(self, uptie, speed=None):
        self.uptie = uptie
        self.speed = speed
    def update_speed(self, speed):
        self.speed = speed

# ResistInfo 객체 정의
class ResistInfo:
    def __init__(self, resist_hit=None, resist_penetrate=None, resist_slash=None):
        self.resist_hit = resist_hit
        self.resist_penetrate = resist_penetrate
        self.resist_slash = resist_slash

# ResistInfo 객체 정의
class StaggerThreshold:
    def __init__(self, first_stagger=None, second_stagger=None, third_stagger=None):
        self.first_stagger = first_stagger
        self.second_stagger = second_stagger
        self.third_stagger = third_stagger

class SkillSet:
    def __init__(self):

        self.skill_1_type = ""
        self.skill_2_type = ""
        self.skill_3_type = ""
        self.defense_skill_type = ""

        self.skill_1_sin_affinity = ""
        self.skill_2_sin_affinity = ""
        self.skill_3_sin_affinity = ""
        self.defense_skill_sin_affinity = ""

        self.skill_1_details = []
        self.skill_2_details = []
        self.skill_3_details = []
        self.defense_skill_details = []

    def set_skill_type(self, skill_code, skill_type, sin_affinity):
        if skill_code == "skill 1":
            self.skill_1_type = skill_type
            self.skill_1_sin_affinity = sin_affinity
        elif skill_code == "skill 2":
            self.skill_2_type = skill_type
            self.skill_2_sin_affinity = sin_affinity
        elif skill_code == "skill 3":
            self.skill_3_type = skill_type
            self.skill_3_sin_affinity = sin_affinity
        elif skill_type == "defense":
            self.defense_skill_type = skill_type
            self.defense_skill_sin_affinity = sin_affinity
    
    def add_skill(self, skill_code, skill):
        if skill_code == "skill 1":
            self.skill_1_details.append(skill)
        elif skill_code == "skill 2":
            self.skill_2_details.append(skill)
        elif skill_code == "skill 3":
            self.skill_3_details.append(skill)
        elif skill_code == "defense":
            self.defense_skill_details.append(skill)

    def get_skill_by_name(self, skill_name):
        for skill_list in [self.skill_1_details, self.skill_2_details, self.skill_3_details, self.defense_skill_details]:
            for skill in skill_list:
                result = skill.is_skill_by_name(skill_name)
                if result:
                    return result
        return None

    def add_skill_uptie_info(self, skill_name, uptie_level, skill_power, coin_power):
        skill = self.get_skill_by_name(skill_name)
        skill.add_uptie_info(uptie_level, skill_power, coin_power)
        

class Skill:
    def __init__(self, skill_code, skill_name, coin_count, init_level, init_remain, akt_weight):
        self.skill_code = skill_code # "Skill 1"/"Skill 2"/"Skill 3"/"Defense" '''o'''
        self.skill_name = skill_name # 스킬 이름 '''o'''
        self.coin_count = coin_count # 동전 개수 '''o'''
        self.init_level = init_level # 스킬이 초기 배정받은 레벨, 인격 레벨에 - 1 한 값을 더해주면 스킬의 현 level을 구할 수 있다. '''o'''
        """
        남아있는 스킬 개수 사용하면, 하나씩 줄어들고 모든 스킬을 사용하면 이 값으로 돌아간다.
        스킬의 남은 숫자없는 스킬은 강화된 스킬로 한번 사용되면 다 조건을 채울 때까 사용 못한다.
        강화된 스킬을 None으로 설정하는 방식과 조건이 채워지면 1로 늘리는 방식이 존재한다.
        스킬을 사용할 수 있는 초기값이다.
        조건을 충족한 강화한 스킬을 이례적인 절차라고 생각하고 None으로 초기값을 설정했다.
        """
        self.init_remain = None #'''o'''
        self.atk_weight = None #공격 가중치 값이다. 방어스킬에는 없기 때문에 None으로 설정했다. '''o'''
        self.remain_count = init_remain # 실제 스킬이 작동하며 횟수가 차감되 나중에 모두 차감되면 다음 턴에 다시 초기횟수 리필된다.
        """
        동기화 단계에 따른 정보를 담는다. 현재 4동기화까지 되어 있지만 차후 확장될 것을 생각하여 고정하지 않는다.
        내부에 들어있는 정보로는 uptie_level, skill_power, coin_power 속성으로 들어있다.
        전투, 턴, 스킬, 코인토스 단계로 내려오며 발생하는 효과에 관하여 저장한다.
        """
        self.uptie_info = []  

    def is_skill_by_name(self, skill_name):
        if self.skill_name == skill_name:
            return self
        return None
        
    def add_uptie_info(self, uptie_level, skill_power, coin_power):
        self.uptie_info.append(UptieInfo(uptie_level, skill_power, coin_power))

class UptieInfo:
    def __init__(self, uptie_level, skill_power, coin_power):
        self.uptie_level = uptie_level   #'''o'''
        self.skill_power = skill_power   #'''o'''
        self.coin_power = coin_power     #'''o'''
        self.skill_effects = []

class Effect:
    def __init__(self, condition):
        self.condition = condition # 조건 텍스트 "[coin 1 on hit]"
        self.effects = [] # 효과 설명 텍스 "Inflict +1 Rupture Count

# 추상화된 EffectType 클래스
class EffectType:
    def __init__(self, name, description):
        self.name = name  # Effect 이름
        self.description = description  # Effect 설명

# Passive 클래스
class Passive(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)  # condition이 있는지 여부 확인

# SupportPassive 클래스
class SupportPassive(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)

# PanicType 클래스
class PanicType(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)

# Keyword 클래스
class Keyword:
    def __init__(self, keyword, explain=None, dispellable=None):
        self.keyword = keyword
        self.explain = explain
        self.dispellable = dispellable

      
class DataContainer:
    """크롤링 데이터를 관리하는 클래스"""
    def __init__(self):
        self.identities = []

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.level_max

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.uptie_max

    def update_identity_name(self, identity_id, identity_name):
        """특정 ID의 이름을 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_name(identity_name)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_identity_rarity(self, identity_id, rarity):
        """특정 ID의 희귀도를 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_rarity(rarity)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_skill(self, identity_id, skill_number, new_type):
        """
        특정 identity의 skill_type을 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_type: 새 skill_type
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_type = new_type
                elif skill_number == 2:
                    identity.skill_2_type = new_type
                elif skill_number == 3:
                    identity.skill_3_type = new_type
                elif skill_number == "defense":
                    identity.defense_type = new_type
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def update_sin_affinity(self, identity_id, skill_number, new_affinity):
        """
        특정 identity의 sin_affinity를 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_affinity: 새 sin_affinity
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_sin_affinity = new_affinity
                elif skill_number == 2:
                    identity.skill_2_sin_affinity = new_affinity
                elif skill_number == 3:
                    identity.skill_3_sin_affinity = new_affinity
                elif skill_number == "defense":
                    identity.defense_sin_affinity = new_affinity
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def add_identity(self, identity):
        """새로운 인격 데이터를 추가."""
        self.identities.append(identity)

    def add_level_stat(self, identity_id, level, hp, defense):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_level_stat(level, hp, defense)

    def add_uptie_stat(self, identity_id, uptie, speed):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_uptie_stat(uptie, speed)

    def add_resist_info(self, identity_id, resist_hit, resist_penetrate, resist_slash):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_resist_info(resist_hit, resist_penetrate, resist_slash)

    def add_stagger_threshold(self, identity_id, first_stagger, second_stagger, third_stagger):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_stagger_threshold(first_stagger, second_stagger, third_stagger)

    
    def add_skill(self, identity_id, skill_code, skill_name, coin_count, init_level, init_remain, akt_weight):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_code == "skill 1":
                    identity.skillset.set_skill_type(skill_code, identity.skill_1_type, identity.skill_1_sin_affinity)
                elif skill_code == "skill 2":
                    identity.skillset.set_skill_type(skill_code, identity.skill_2_type, identity.skill_2_sin_affinity)
                elif skill_code == "skill 3":
                    identity.skillset.set_skill_type(skill_code, identity.skill_3_type, identity.skill_3_sin_affinity)
                elif skill_code == "defense":
                    identity.skillset.set_skill_type(skill_code, identity.defense_type, identity.defense_sin_affinity)
                identity.add_skill(skill_code, skill_name, coin_count, init_level, init_remain, akt_weight)

    def add_skill_uptie_info(self, identity_id, skill_name, uptie_level, skill_power, coin_power):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_skill_uptie_info(skill_name, uptie_level, skill_power, coin_power)
        
    def display_identities(self):
        """Identity, ID, Name, Rarity 정보를 출력"""
        if not self.identities:
            print("No identities found.")
        else:
            # 데이터를 리스트로 변환
            table = []
            for idx, identity in enumerate(self.identities, 1):
                table.append([
                    idx, identity.identity_id, identity.name, identity.rarity,
                    identity.skill_1_type, identity.skill_1_sin_affinity,
                    identity.skill_2_type, identity.skill_2_sin_affinity,
                    identity.skill_3_type, identity.skill_3_sin_affinity,
                    identity.defense_type, identity.defense_sin_affinity
                ])
            
            # 테이블 헤더 정의
            headers = [
                "Index", "ID", "Name", "Rarity",
                "Skill_1 Type", "Skill_1 Sin", "Skill_2 Type", "Skill_2 Sin",
                "Skill_3 Type", "Skill_3 Sin", "Defense Type", "Defense Sin"
            ]
            
            # 테이블 출력
            print(tabulate(table, headers=headers, tablefmt="fancy_grid"))

    def export_to_csv(self, filename, fieldnames, data):
        """
        공통적으로 사용 가능한 CSV 내보내기 메서드
        :param filename: 내보낼 파일 이름 (예: 'identity.csv', 'skill.csv')
        :param fieldnames: CSV의 헤더 리스트
        :param data: CSV로 내보낼 데이터 (리스트 형태)
        """
        try:
            # 현재작업 디렉토 기준으 'csv'폴더 없으면 생성
            if not os.path.exists('./csv'):
                os.makedirs('./csv')
            # 파일 경로 설정 (상대경로 'csv' 폴더 안에 저장)
            file_path = os.path.join('./csv', filename)
            with open(file_path, mode='w', newline='', encoding='utf-8') as file:
                writer = csv.DictWriter(file, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(data)
            print(f"Data successfully exported to {filename}")
        except Exception as e:
            print(f"Error exporting to CSV: {e}")

    def get_identity_data(self):
        """
        Identity 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "ID": identity.identity_id,
                "Name": identity.name,
                "Rarity": identity.rarity,
                "Skill_1_Type": identity.skill_1_type,
                "Skill_1_Sin_Affinity": identity.skill_1_sin_affinity,
                "Skill_2_Type": identity.skill_2_type,
                "Skill_2_Sin_Affinity": identity.skill_2_sin_affinity,
                "Skill_3_Type": identity.skill_3_type,
                "Skill_3_Sin_Affinity": identity.skill_3_sin_affinity,
                "Defense_Type": identity.defense_type,
                "Defense_Sin_Affinity": identity.defense_sin_affinity,
            }
            for identity in self.identities
        ]
        
    def export_identities_data(self, filename="identities.csv"):
        """Identity 데이터를 CSV로 내보내는 메서드"""
        identity_fieldnames = [
            "ID", "Name", "Rarity",
            "Skill_1_Type", "Skill_1_Sin_Affinity",
            "Skill_2_Type", "Skill_2_Sin_Affinity",
            "Skill_3_Type", "Skill_3_Sin_Affinity",
            "Defense_Type", "Defense_Sin_Affinity"
        ]
        identity_data = self.get_identity_data()  # Identity 데이터 가져오기
        self.export_to_csv(filename, identity_fieldnames, identity_data)  # CSV로 내보내기

    def load_identities_by_id(self, filename, identity_ids):
        """지정된 identity_ids 목록에 해당하는 Identity 데이터를 CSV에서 읽어옴."""
        identity_ids_set = set(identity_ids) # 중복 제거를 위 Set 사용
        with open(filename, mode='r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                if row["ID"] in identity_ids_set:
                    identity = Identity(
                        identity_id=row["ID"],
                        name=row["Name"],
                        rarity=row["Rarity"],
                        skill_1_type=row["Skill_1_Type"],
                        skill_1_sin_affinity=row["Skill_1_Sin_Affinity"],
                        skill_2_type=row["Skill_2_Type"],
                        skill_2_sin_affinity=row["Skill_2_Sin_Affinity"],
                        skill_3_type=row["Skill_3_Type"],
                        skill_3_sin_affinity=row["Skill_3_Sin_Affinity"],
                        defense_type=row["Defense_Type"],
                        defense_sin_affinity=row["Defense_Sin_Affinity"]
                    )
                    self.identities.append(identity)

    def get_level_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Level": level_stat.level,
                "HP": level_stat.hp,
                "Defense": level_stat.defense,
            }
            for identity in self.identities
            for level_stat in identity.level_stats
        ]

    def export_level_stats_data(self, filename="levelstats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        level_stats_fieldnames = ["Identity_ID", "Level", "HP", "Defense"]
        level_stats_data = self.get_level_stats_data()  # level_stats 데이터 가져오기
        self.export_to_csv(filename, level_stats_fieldnames, level_stats_data)  # CSV로 내보내기

    def get_uptie_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Uptie": uptie_stat.uptie,
                "Speed": uptie_stat.speed,
            }
            for identity in self.identities
            for uptie_stat in identity.uptie_stats
        ]

    def export_uptie_stats_data(self, filename="Uptiestats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        uptie_stats_fieldnames = ["Identity_ID", "Uptie", "Speed"]
        uptie_stats_data = self.get_uptie_stats_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, uptie_stats_fieldnames, uptie_stats_data)  # CSV로 내보내기

    def get_resist_info_data(self):
        """
        모든 Identity의 resist_info 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Resist_Hit": identity.resist_info.resist_hit,
                "Resist_Penetrate": identity.resist_info.resist_penetrate,
                "Resist_Slash": identity.resist_info.resist_slash,
            }
            for identity in self.identities
        ]

    def export_resist_info_data(self, filename="ResistInfos.csv"):
        """ResistInfo 데이터를 CSV로 내보내는 메서드"""
        resist_infos_fieldnames = ["Identity_ID", "Resist_Hit", "Resist_Penetrate", "Resist_Slash"]
        resist_infos_data = self.get_resist_info_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, resist_infos_fieldnames, resist_infos_data)  # CSV로 내보내기

    def export_uptie_stats_data(self, filename="Uptiestats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        uptie_stats_fieldnames = ["Identity_ID", "Uptie", "Speed"]
        uptie_stats_data = self.get_uptie_stats_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, uptie_stats_fieldnames, uptie_stats_data)  # CSV로 내보내기

    def get_stagger_threshold_data(self):
        """
        모든 Identity의 stagger_threshold 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "First_Stagger": identity.stagger_threshold.first_stagger,
                "Second_Stagger": identity.stagger_threshold.second_stagger,
                "Third_Stagger": identity.stagger_threshold.third_stagger,
            }
            for identity in self.identities
        ]

    def export_stagger_threshold_data(self, filename="StaggerThresholds.csv"):
        """StaggerThresholds 데이터를 CSV로 내보내는 메서드"""
        stagger_thresholds_fieldnames = ["Identity_ID", "First_Stagger", "Second_Stagger", "Third_Stagger"]
        stagger_thresholds_data = self.get_stagger_threshold_data()  # stagger_thresholds 데이터 가져오기
        self.export_to_csv(filename, stagger_thresholds_fieldnames, stagger_thresholds_data)  # CSV로 내보내기


async def main():
    data_container = DataContainer()
    scraper = Scraper(container=data_container, base_url="https://limbus.kusoge.xyz", img_url="https://img.kusoge.xyz/limbus/img")
    await scraper.scrape()

await main() 

<>:609: SyntaxWarning: invalid escape sequence '\/'
/var/folders/27/r1qdd08x0fx8nvdhy823ntqw0000gn/T/ipykernel_89764/2705730669.py:609: SyntaxWarning: invalid escape sequence '\/'
  return await skill_element.evaluate("""


Added skill: skill 1:Begone...
Updated Uptie for Begone... (Uptie Level: 1): Skill Power: 2, Coin Power: +4
<div data-v-045a300e=""><!--[--><div data-v-045a300e=""><span class="q-pr-xs startbattle">[Combat Start]</span><span> </span><span>On</span><span> </span><span>self</span><span> </span><span>and</span><span> </span><span>1</span><span> </span><span>ally</span><span> </span><span>with</span><span> </span><span>the</span><span> </span><span>highest</span><span> </span><span>max</span><span> </span><span>HP</span><span>: </span><span>consume</span><span> </span><span>up</span><span> </span><span>to</span><span> </span><span>20</span><span> </span><span class="q-focus-helper"></span><span class="q-btn__content text-center col items-center q-anchor--skip justify-center row"><div data-v-25b93422="" class="q-avatar" left="" style="font-size: 20px;"><div class="q-avatar__content row flex-center overflow-hidden"><img data-v-25b93422="" src="https://img.kusoge.xyz/limbus/img/icon/BloodDinn

In [41]:
import os
import csv
import re
import numpy as np
import pandas as pd
from playwright.async_api import async_playwright
from tabulate import tabulate

# Sin Color 이름 변환을 위한 매핑
resource_mapping = {
    "AZURE": "Gloom",
    "VIOLET": "Envy",
    "AMBER": "Sloth",
    "SHAMROCK": "Gluttony",
    "CRIMSON": "Wrath",
    "SCARLET": "Lust",
    "INDIGO": "Pride"
}

def convert_resource(resource_name):
    """Sin Color 이름을 변환하는 함수. 매핑되지 않은 경우 원래 이름을 반환."""
    return resource_mapping.get(resource_name, resource_name)

class Scraper:
    """웹 페이지 크롤링을 위한 클래스"""
    def __init__(self, container, base_url, img_url):
        self.container = container
        self.base_url = base_url
        self.img_url = img_url
        self.uptie_level = 1

    async def handle_request(self, route, request):
        """이미지와 폰트 리소스 로드를 차단."""
        if request.resource_type in ["image", "font"]:
            await route.abort()
        else:
            await route.continue_()
            
    async def scrape(self):
        """크롤링 메인 함수"""
        async with async_playwright() as p:
            browser = await p.chromium.launch(headless=True)
            page = await browser.new_page()
            await page.route("**/*", self.handle_request)
            # Step 1: /identity 페이지에서 기본 정보 크롤링
            #await self.crawl_identity_page(page)

            # Step 2: /identity/{identity_id} 상세 페이지 크롤링
            #identity_ids_to_load = ["10104", "10310", "10710","10808", "10911", "11005"]
            #identity_ids_to_load = ["10104", "10911"]#,"10310", "10710","10808", "11005", "11210"]
            identity_ids_to_load = ["10911"]#,"10310", "10710","10808", "11005", "11210"]
            self.container.load_identities_by_id('./csv/identities.csv', identity_ids_to_load)
            await self.crawl_identity_details_for_all(page)
            #self.container.display_identities()
            #self.container.export_identities_data("identities.csv")
            #self.container.export_level_stats_data("Levelstats.csv")
            #self.container.export_uptie_stats_data("Uptiestats.csv")
            #self.container.export_resist_info_data("ResistInfos.csv")
            #self.container.export_stagger_threshold_data("StaggerThresholds.csv")
            #page.set_default_timeout(10000)
            await page.wait_for_timeout(100000)
            await browser.close()

    async def get_identity_id_from_row(self, row):
        """테이블 행(row)에서 인격 ID를 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_url = await identity_cell.get_attribute('href')
            identity_id = identity_url.split('/identity/')[-1]
            return identity_id
        return "Unknown"

    async def get_identity_name_from_row(self, row):
        """테이블 행(row)에서 인격 이름을 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_name = (await identity_cell.inner_text()).replace("\n", " ").strip()
            return identity_name
        return "Unknown"
        
    async def crawl_identity_page(self, page):
        """/identity 페이지로 이동하여 인격 기본 정보를 추출."""
        await page.goto(self.base_url + "/identity")
        await self.extract_identity_page(page)

    async def extract_identity_page(self, page):
        """/identity 페이지의 모든 테이블 행에서 인격 정보를 추출."""
        identity_rows = await page.query_selector_all('tr')
        if len(identity_rows) > 1:
            for idx, identity_row in enumerate(identity_rows):
                if idx == 0:
                    continue # 0번째요소이름을 보여주는 열은 건너뜀
                await self.scrape_identity_name_and_identity_id(identity_row)
                await self.extract_rarity(identity_row)
                for i in range(1, 4):  # 스킬1~3 정보 추출
                    await self.extract_atk_and_resource(identity_row, i)
                await self.extract_defense_and_resource(identity_row)

    async def scrape_identity_name_and_identity_id(self, row):
        """인격 번호와 이름을 추출하여 DataContainer에 저장."""
        try:
            identity_id = await self.get_identity_id_from_row(row)
            identity_name = await self.get_identity_name_from_row(row)
            identity = Identity(identity_id)
            self.container.add_identity(identity)
            self.container.update_identity_name(identity_id, identity_name)
        except Exception as e:
            print(f"Error extracting identity name and identity id: {e}")

    async def get_element_attribute(self, element, selector, attribute):
        """요소에서 지정된 속성(attribute)을 추출."""
        try:
            target_element = await element.query_selector(selector)
            if target_element:
                return await target_element.get_attribute(attribute)
        except Exception as e:
            print(f"Error getting element attribute: {e}")
        return None
    
    async def extract_rarity(self, row):
        """테이블 행(row)에서 희귀도를 추출하여 업데이트."""
        try:
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
                sibling = await parent.evaluate_handle('node => node.nextElementSibling')
                if sibling:
                    rarity_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/rarity_"]', 'src')
                    if rarity_img_src:
                        rarity_level = rarity_img_src.split('_')[-1].split('.')[0]  # rarity_1.png -> 1
                        identity_id = await self.get_identity_id_from_row(row)
                        self.container.update_identity_rarity(identity_id, rarity_level)
                    else:
                        print("Rarity image not found in the next sibling TD.")
        except Exception as e: 
            print(f"Error extracting rarity: {e}")

        return "Unknown"

    async def extract_atk_and_resource(self, row, skill_number):
        """ 공격 타입(ATK Type)과 자원(Attribute) 추출 함수 - 스킬 번호에 따라 처리 """
        try:
            #print(f"Attempting to locate skill{skill_number} type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 해당 스킬 번호에 맞는 형제로 이동
                sibling = parent
                for _ in range(skill_number + 1):  # 스킬 번호에 맞게 이동
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                atk_type, resource = "Unknown", "Unknown"
        
                if sibling:
                    # 공격 타입 이미지 (atk_type) 찾기
                    atk_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/atk_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if atk_type_img_src:
                        atk_type = atk_type_img_src.split('_')[-1].split('.')[0]  # atk_type_SLASH.png -> SLASH
                        #print(f"Extracted attack type: {atk_type}")
                        self.container.update_skill(identity_id, skill_number, atk_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted resource: {resource}")
                        
                        # 리소스를 변환
                        resource = convert_resource(resource)
                        #print(f"Converted resource: {resource}")
                        self.container.update_sin_affinity(identity_id, skill_number, resource)
                        
        except Exception as e:
            print(f"Error extracting skill{skill_number} type and resource: {e}")

    async def extract_defense_and_resource(self, row):
        """ 방어 타입(Def Type)과 자원(Attribute) 추출 함수 """
        try:
            #print("Attempting to locate defense type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 다섯 번째 형제 td로 이동 (방어 관련 정보가 있는 <td>)
                sibling = parent
                for _ in range(5):  # 방어 정보는 5번째 형제에 있음
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                def_type, def_resource = "Unknown", "Unknown"
        
                if sibling:
                    # 방어 타입 이미지 (def_type) 찾기
                    def_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/def_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if def_type_img_src:
                        def_type = def_type_img_src.split('_')[-1].split('.')[0]  # def_type_GUARD.png -> GUARD
                        #print(f"Extracted defense type: {def_type}")
                        self.container.update_skill(identity_id, 'defense', def_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        def_resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted defense resource: {def_resource}")
                        
                        # 방어 리소스를 변환
                        def_resource = convert_resource(def_resource)
                        #print(f"Converted defense resource: {def_resource}")
                        self.container.update_sin_affinity(identity_id, 'defense', def_resource)
 
        except Exception as e:
            print(f"Error extracting defense type and resource: {e}")

    async def crawl_identity_details_for_all(self, page):
        """모든 인격의 세부 정보를 크롤링."""
        for identity in self.container.identities:
            await page.wait_for_load_state("networkidle")
            await page.goto(self.base_url + "/identity/"+identity.identity_id)
            """ 레벨과 동기화와 관련 없는 참관타 저항은 밑에서 변경하는 것과 연관성이 적기 때문에 위에서 처리한다.
            print(f"Identity_id: {identity.identity_id}")
            results = []
            resist_type_imgs = [f"{self.img_url}/ui/atk_type_HIT.png",
                                f"{self.img_url}/ui/atk_type_PENETRATE.png",
                                f"{self.img_url}/ui/atk_type_SLASH.png"]
            for resist_type_img in resist_type_imgs:
                result = await self.extract_resist_type(page, resist_type_img, 1)
                results.append(result)
            self.container.add_resist_info(identity.identity_id, results[0], results[1], results[2])
            """
            """
            # Stagger Threshold 값 추출
            stagger_threshold = await self.extract_stagger_threshold_values(page)
            self.container.add_stagger_threshold(identity.identity_id, stagger_threshold[0], stagger_threshold[1], stagger_threshold[2])
            """
            await self.initialize_uptie_and_level(page)
            """ 레벨에 따른 level, hp, defense 다르게 되어 있는 것을 출력 확인 객체로 들어가서 csv로 옮기는 과정까지 진행
            for i in range(1, identity.level_max+1):
                await self.move_slider_to_value(page,i)
                await self.extract_level_status(page, identity.identity_id, i)
            """
            """ 동기화에 따른 speed 다르게 되어 있는 것을 출력 확인 객체로 들어가서 csv로 옮기는 과정까지 진행
            for i in range(1, identity.uptie_max+1):
                await self.uptie_click(page,i)
                await self.extract_uptie_status(page, identity.identity_id, i)
            """
            await self.crawl_skills(page, identity.identity_id)
            
    async def initialize_uptie_and_level(self, page):
        await self.uptie_click(page, 1)
        await self.move_slider_to_value(page, 1)
        self.uptie_level = 1
        #return 1
    
    async def uptie_click(self, page, phase):
        fab_button_selector = ".q-btn.q-btn--fab"
        await page.wait_for_selector(fab_button_selector)
        await page.click(fab_button_selector)
        #print(f"[{setup_uptie_button}] 상위 동기화 버튼 클릭 완료")
        setup_uptie_button = f'a:has(img[src="{self.img_url}/ui/uptie_{phase}.png"])'
        await page.wait_for_selector(setup_uptie_button)
        await page.click(setup_uptie_button)
        #print(f"{phase} 동기화 버튼 클릭 완료")

    async def move_slider_to_value(self, page, target_value):
        """
        슬라이더를 주어진 값으로 이동시킵니다.
        target_value는 슬라이더의 최소값부터 최대값 사이여야 합니다.
        """
        slider_selector = ".q-slider"
        # 슬라이더의 범위 확인
        min_value = int(await page.get_attribute(slider_selector, "aria-valuemin"))
        max_value = int(await page.get_attribute(slider_selector, "aria-valuemax"))
    
        if target_value < min_value or target_value > max_value:
            raise ValueError(f"Target value {target_value} is out of bounds ({min_value}, {max_value})")
    
        # 슬라이더 thumb 요소와 위치 가져오기
        slider_thumb = await page.query_selector(f"{slider_selector} .q-slider__thumb")
        thumb_box = await slider_thumb.bounding_box()
        
        # 슬라이더의 현재 위치 및 목표 위치 계산
        track = await page.query_selector(f"{slider_selector} .q-slider__track")
        track_box = await track.bounding_box()
        
        # 목표 x 위치 계산 (슬라이더 트랙 내에서 비율 기반)
        target_percentage = (target_value - min_value) / (max_value - min_value)
        target_x = track_box["x"] + target_percentage * track_box["width"]
        current_x = thumb_box["x"] + thumb_box["width"] / 2
        y = thumb_box["y"] + thumb_box["height"] / 2
    
        # 슬라이더를 드래그하여 목표 위치로 이동
        await page.mouse.move(current_x, y)
        await page.mouse.down()
        await page.mouse.move(target_x, y, steps=10)  # 부드럽게 이동
        await page.mouse.up()
    
        #print(f"슬라이더를 {target_value}로 이동 완료.")
    
        # 슬라이더 값 확인 및 강제 이벤트 트리거
        await page.evaluate(
            f"""
            (slider) => {{
                slider.dispatchEvent(new Event('input'));  // oninput 이벤트 트리거
                slider.dispatchEvent(new Event('change'));  // onchange 이벤트 트리거
            }}
            """,
            await page.query_selector(slider_selector)
        )
    
        # 이동 후 값 확인
        current_value = await page.get_attribute(slider_selector, "aria-valuenow")
        if current_value != str(target_value):
            print(f"경고: 슬라이더 값이 예상과 다릅니다. 현재 값: {current_value}, 예상 값: {target_value}")

    async def get_second_status_element(self, page):
        """
        두 번째 'Status' 요소에 접근하여 반환하는 메소드
        """
        status_elements = await page.query_selector_all("text=Status")
        if len(status_elements) > 1:
            second_status_element = status_elements[1]
            #print("두 번째 'Status' 요소에 접근합니다.")
            return second_status_element
        else:
            #print("두 번째 'Status' 요소가 존재하지 않습니다.")
            return None

    async def extract_level_status(self, page, identity_id, level):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 레벨에 따라 HP와 Defense가 변한다.
        hp_value = await self.extract_hp(second_status_element)
        defense_value = await self.extract_defense(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Level={level}, HP={hp_value}, Defense={defense_value}")
        self.container.add_level_stat(identity_id,level, hp_value, defense_value)
    
    async def extract_hp(self, status_element):
        # HP 추출
        hp_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].children[0].nextElementSibling.textContent.trim()"
        )
        return hp_value
    
    async def extract_defense(self, status_element):
        # Defense 추출
        defense_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return defense_value

    async def extract_uptie_status(self, page, identity_id, uptie):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 uptie에 따라 speed는 변한다.
        speed_value = await self.extract_speed(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Uptie={uptie}, Speed={speed_value}")
        self.container.add_uptie_stat(identity_id,uptie, speed_value)

    async def extract_speed(self, status_element):
        # Speed 추출
        speed_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return speed_value

    async def extract_resist_type(self, page, image_src, depth):
        # 이미지 요소 찾기
        #print(image_src)
        image = await page.query_selector(f'img[src="{image_src}"]')
        #print(image)
        if not image:
            return "Unknown"
    
        # 이미지의 부모 요소를 찾아서 올라감
        parent = image
        for _ in range(depth):
            parent = await parent.evaluate_handle('node => node.parentElement')
            if not parent:
                return "Unknown"
    
        # 부모의 형제 요소 찾기
        sibling = await parent.evaluate_handle('node => node.nextElementSibling')
        
        if sibling:
            value = await sibling.evaluate('node => node.textContent')
            if value:
                return value.strip()
        
        return "Unknown"

    async def extract_stagger_threshold_values(self, page):
        # 'div.row.center.flex-center.q-pt-xs' 요소들 찾기
        elements = await page.query_selector_all('div.row.center.flex-center.q-pt-xs')
        
        # 각 요소의 텍스트 내용 출력하여 'Stagger Threshold' 요소를 찾기
        for element in elements:
            text = await element.text_content()
            
            if text and "Stagger Threshold" in text:
                # 'Stagger Threshold'가 있는 요소를 찾았다면, 그 이후 형제 요소들에서 값을 추출
                parent = await element.evaluate_handle('node => node.parentElement')
                sibling_elements = await parent.query_selector_all('div.row.justify-evenly div')  # Threshold 값이 들어있는 div들
                
                stagger_values = []
                for sibling in sibling_elements:
                    text_content = await sibling.text_content()
                    match = re.search(r'(\d+%)', text_content)
                    if match:
                        stagger_values.append(match.group(1))  # 퍼센트 값만 리스트에 추가
                while len(stagger_values) < 3:
                    stagger_values.append(np.nan)
                #print(stagger_values)
                return stagger_values  # 추출된 값(또는 np.nan 포함)을 반환
        
        return [np.nan, np.nan, np.nan]  # 만약 'Stagger Threshold'를 찾지 못한 경우 기본값 반환

    async def crawl_skills(self, page, identity_id):
        await self.crawl_skill(page, identity_id, "skill 1")
        await self.crawl_skill(page, identity_id, "skill 2")
        await self.crawl_skill(page, identity_id, "skill 3")
        await self.crawl_skill(page, identity_id, "defense")


    async def crawl_skill(self, page, identity_id, skill_type):
        skills_elements = await page.query_selector_all('div.skilltag')
        for skill_element in skills_elements:
            skill_text = await skill_element.text_content()
            if skill_text.strip().lower() == skill_type:
                await self.process_skill(skill_element, page, identity_id, skill_type)
        
    async def process_skill(self, skill_element, page, identity_id, skill_type):
        # skill_type이 "skill 3"일 경우 uptie_value가 3 이상일 때만 정보를 저장
        for uptie_value in range(1, 5):  # Uptie levels 1 to 4
            # "skill 3"은 uptie_value가 3 이상일 때만 처리
            if skill_type == "skill 3" and uptie_value < 3:
                continue  # skip if uptie_value is less than 3 for skill 3
    
            # uptie_level에 맞게 클릭
            await self.uptie_click(page, uptie_value)
            if skill_type != "skill 3" and uptie_value > 1:
                # Uptie 정보 갱신
                await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
                continue
            elif skill_type == "skill 3" and uptie_value > 3:
                # Uptie 정보 갱신
                await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
                continue
            # 스킬의 세부 정보를 추출
            skill_details = await self.extract_skill_details(skill_element, skill_type)
            skill_name = skill_details["skill_name"]
    
            # 스킬 정보를 저장
            self.container.add_skill(
                identity_id, skill_type, 
                skill_name, skill_details["coin_count"],
                skill_details["init_level"], skill_details["init_remain"], 
                skill_details["atk_weight"]
            )
            print(f"Added skill: {skill_type}:{skill_name}")
    
            # Uptie 정보 갱신
            await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
    
    async def extract_skill_details(self, skill_element, skill_type):
        # 스킬 세부 정보를 추출하는 함수
        skill_name = await self.skill_name(skill_element)
        coin_count = await self.coin_count(skill_element, ".col-auto > .column > .row")
        init_level = await self.init_level(skill_element)
        init_remain = await self.init_remain(skill_element, skill_type, "div.row.justify-start > div.attr.flex.flex-center > div.q-px-sm")
        atk_weight = await self.atk_weight(skill_element, skill_type)
        return {
            "skill_name": skill_name,
            "coin_count": coin_count,
            "init_level": init_level,
            "init_remain": init_remain,
            "atk_weight": atk_weight
        }
    
    async def update_uptie_info(self, page, skill_element, identity_id, skill_name, skill_type, uptie_value):
        # Uptie 정보를 추출하고 업데이트하는 함수
        uptie_info = await self.extract_uptie_info(skill_element)
        self.container.add_skill_uptie_info(
            identity_id, skill_name, 
            uptie_info["skill_power"], uptie_info["coin_power"], 
            uptie_value
        )
        print(
            f"Updated Uptie for {skill_name} (Uptie Level: {uptie_value}): "
            f"Skill Power: {uptie_info['skill_power']}, Coin Power: {uptie_info['coin_power']}"
        )
        effect = await self.skill_effects(skill_element)
        print(effect)

    async def extract_uptie_info(self, skill_element):
        skill_data = await self.skill_and_coin_power(skill_element)
        return {
            "skill_power": skill_data.get("skillPower"),
            "coin_power": skill_data.get("coinPower")
        }

    async def extract_skill_effect(self, skill_element):
        # Create the Effect object
        effect_obj = Effect(condition=None)
        # Extract effects
        effects = await self.skill_effects(skill_element)
        if effects:
            effect_obj.effects.append(effects)
        return effect_obj

    async def skill_and_coin_power(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                // 조부모로 이동
                const grandparent = node.parentElement.parentElement;
                // 이전 형제 탐색
                const previousSibling = grandparent.previousElementSibling;
                if (!previousSibling) return {{ skillPower: null, coinPower: null }}; // 이전 형제가 없으면 기본값 반환
                // 이전 형제 내부의 .row.justify-around > div > div 탐색
                const targetDivs = previousSibling.querySelectorAll(".row.justify-around > div");
                if (targetDivs.length < 3) return {{ skillPower: null, coinPower: null }}; // 배열 길이 확인
                // 1st div: 스킬 위력
                const skillPower = targetDivs[0]?.querySelector('div')?.textContent?.trim() || null; 
                // 3rd div: 코인 파워 값 (3번째 div에서 텍스트 추출)
                const coinPower = targetDivs[2]?.textContent?.trim() || null;
                return {{ skillPower, coinPower }};
            }}
        """)


    async def coin_count(self, skill_element, hierarchy_selector):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector('{hierarchy_selector}');
                if (!targetElement) return 0;
                return targetElement.childElementCount;
            }}
        """)

    async def init_remain(self, skill_element, skill_type, hierarchy_selector):
        if skill_type == "defense":
            return -1
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector('{hierarchy_selector}');
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def skill_name(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector(".block");
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def init_level(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector("div.text-bold.text-h6.h6.q-pl-xs.q-pr-md");
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def atk_weight(self, skill_element, skill_type):
        if skill_type == "defense":
            return 0
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector("div.text-bold.text-h6.h6.q-pl-xs.q-pr-md");
                const doubleNextSibling = targetElement.nextElementSibling.nextElementSibling;
                if (!doubleNextSibling) return 0;
                return doubleNextSibling.textContent?.trim();
            }}
        """)

    async def skill_effects(self, skill_element):
        return await skill_element.evaluate("""
            node => {
                const parent = node.parentElement;
                const doubleNextSibling = parent.nextElementSibling?.nextElementSibling;
                if (!doubleNextSibling) return null;
    
                // doubleNextSibling에서 첫 번째 div.col 요소 찾기
                const textNode = doubleNextSibling.querySelector("div.col");
                if (!textNode) return null;
    
                const results = {
                    keywords: [], // 키워드 저장
                    conditions: [], // 조건 저장
                    effects: [] // 효과 저장
                };
    
                // 1. 조건 추출 (대괄호 텍스트)
                const conditionElements = textNode.querySelectorAll("span");
                conditionElements.forEach(span => {
                    const text = span.textContent?.trim();
                    if (text?.startsWith("[") && text?.endsWith("]")) {
                        results.conditions.push(text);
                    }
                });
    
                // 2. 키워드 추출
                const keywordElements = textNode.querySelectorAll('a[href^="/en/keyword/"]');
                    keywordElements.forEach(a => {
                    const span = a.querySelector('span.q-btn__content.text-center.col.items-center.q-anchor--skip.justify-center.row>span');
                    if (span) {
                        const keywordText = span.textContent.trim();
                        if (keywordText && !results.keywords.includes(keywordText)) {
                            results.keywords.push(keywordText); // 키워드 저장
                        }
                    }
                });
    
                // 3. 효과 추출
                const effectElements = textNode.querySelectorAll("span");
                effectElements.forEach(span => {
                    const text = span.textContent?.trim();
                    if (
                        text &&
                        !text.startsWith("[") 
                        //&& 조건이 아님
                        //!results.keywords.includes(text) && // 키워드가 아님
                        //!results.effects.includes(text) // 이미 저장된 효과가 아님
                    ) {
                        if (results.keywords.includes(text)) {
                            results.effects.push('keyword:'+text
                        } else {
                            results.effects.push(text);
                        }
                    }
                });
    
                return results;
            }
        """)

class Identity:
    """인격 정보를 저장하는 클래스"""
    def __init__(self, identity_id, name=None, rarity=None,\
                 skill_1_type=None, skill_1_sin_affinity=None,\
                 skill_2_type=None, skill_2_sin_affinity=None,\
                 skill_3_type=None, skill_3_sin_affinity=None,\
                 defense_type=None, defense_sin_affinity=None):
        self.identity_id = identity_id
        self.name = name
        self.rarity = rarity
        # 선언만 하는 부분 
        self.skill_1_type = skill_1_type 
        self.skill_1_sin_affinity = skill_1_sin_affinity 
        self.skill_2_type = skill_2_type 
        self.skill_2_sin_affinity = skill_2_sin_affinity 
        self.skill_3_type = skill_3_type 
        self.skill_3_sin_affinity = skill_3_sin_affinity 
        self.defense_type = defense_type 
        self.defense_sin_affinity = defense_sin_affinity
        self.level_max = 50
        self.uptie_max = 4
        self.level_stats = []  # LevelStat 객체 리스트
        self.uptie_stats = [] # UptieStat 객체 리스트
        self.resist_info = None
        self.stagger_threshold = None
        self.skillset = SkillSet()  # Skill정보를 담고 있는 skillset 객체 
        self.passive_infos = []  # 패시브 객체를 담고 있는 배열
        self.support_passive_infos = [] # 서포트 패시 객체를 담고 있는 배열
        self.panic_type = [] # 혼란상태 발생시 나타나는 효과를 담고 있는 객체 배열

    def update_name(self, name):
        self.name = name

    def update_rarity(self, rarity):
        self.rarity = rarity

    def add_level_stat(self, level, hp, defense):
        self.level_stats.append(LevelStat(level, hp, defense))

    def add_uptie_stat(self, uptie, speed):
        self.uptie_stats.append(UptieStat(uptie, speed))

    def add_resist_info(self, resist_hit, resist_penetrate, resist_slash):
        self.resist_info = ResistInfo(resist_hit, resist_penetrate, resist_slash)

    def add_stagger_threshold(self, first_stagger, second_stagger, third_stagger):
        self.stagger_threshold = StaggerThreshold(first_stagger, second_stagger, third_stagger)

    def add_skill(self, skill_code, skill_name, coin_count, init_level, init_remain, akt_weight):
        self.skillset.add_skill(skill_code, Skill(skill_code, skill_name, coin_count, init_level, init_remain, akt_weight))
        
    def add_skill_uptie_info(self, skill_name, uptie_level, skill_power, coin_power):
        self.skillset.add_skill_uptie_info(skill_name, uptie_level, skill_power, coin_power)
        
    def update_details(self, page):
        # 세부 정보 업데이트 로직 (가정)
        pass
    #def init_uptie(self, page):
    #    init_uptie = 

# LevelStat 객체 정의
class LevelStat:
    def __init__(self, level, hp=None, defense=None):
        self.level = level
        self.hp = hp
        self.defense = defense
    def update_level(self, level):
        self.level = level
    def update_hp(self, hp):
        self.hp = hp
    def update_defense(self, defense):
        self.defense = defense

# UptieStat 객체 정의
class UptieStat:
    def __init__(self, uptie, speed=None):
        self.uptie = uptie
        self.speed = speed
    def update_speed(self, speed):
        self.speed = speed

# ResistInfo 객체 정의
class ResistInfo:
    def __init__(self, resist_hit=None, resist_penetrate=None, resist_slash=None):
        self.resist_hit = resist_hit
        self.resist_penetrate = resist_penetrate
        self.resist_slash = resist_slash

# ResistInfo 객체 정의
class StaggerThreshold:
    def __init__(self, first_stagger=None, second_stagger=None, third_stagger=None):
        self.first_stagger = first_stagger
        self.second_stagger = second_stagger
        self.third_stagger = third_stagger

class SkillSet:
    def __init__(self):

        self.skill_1_type = ""
        self.skill_2_type = ""
        self.skill_3_type = ""
        self.defense_skill_type = ""

        self.skill_1_sin_affinity = ""
        self.skill_2_sin_affinity = ""
        self.skill_3_sin_affinity = ""
        self.defense_skill_sin_affinity = ""

        self.skill_1_details = []
        self.skill_2_details = []
        self.skill_3_details = []
        self.defense_skill_details = []

    def set_skill_type(self, skill_code, skill_type, sin_affinity):
        if skill_code == "skill 1":
            self.skill_1_type = skill_type
            self.skill_1_sin_affinity = sin_affinity
        elif skill_code == "skill 2":
            self.skill_2_type = skill_type
            self.skill_2_sin_affinity = sin_affinity
        elif skill_code == "skill 3":
            self.skill_3_type = skill_type
            self.skill_3_sin_affinity = sin_affinity
        elif skill_type == "defense":
            self.defense_skill_type = skill_type
            self.defense_skill_sin_affinity = sin_affinity
    
    def add_skill(self, skill_code, skill):
        if skill_code == "skill 1":
            self.skill_1_details.append(skill)
        elif skill_code == "skill 2":
            self.skill_2_details.append(skill)
        elif skill_code == "skill 3":
            self.skill_3_details.append(skill)
        elif skill_code == "defense":
            self.defense_skill_details.append(skill)

    def get_skill_by_name(self, skill_name):
        for skill_list in [self.skill_1_details, self.skill_2_details, self.skill_3_details, self.defense_skill_details]:
            for skill in skill_list:
                result = skill.is_skill_by_name(skill_name)
                if result:
                    return result
        return None

    def add_skill_uptie_info(self, skill_name, uptie_level, skill_power, coin_power):
        skill = self.get_skill_by_name(skill_name)
        skill.add_uptie_info(uptie_level, skill_power, coin_power)
        

class Skill:
    def __init__(self, skill_code, skill_name, coin_count, init_level, init_remain, akt_weight):
        self.skill_code = skill_code # "Skill 1"/"Skill 2"/"Skill 3"/"Defense" '''o'''
        self.skill_name = skill_name # 스킬 이름 '''o'''
        self.coin_count = coin_count # 동전 개수 '''o'''
        self.init_level = init_level # 스킬이 초기 배정받은 레벨, 인격 레벨에 - 1 한 값을 더해주면 스킬의 현 level을 구할 수 있다. '''o'''
        """
        남아있는 스킬 개수 사용하면, 하나씩 줄어들고 모든 스킬을 사용하면 이 값으로 돌아간다.
        스킬의 남은 숫자없는 스킬은 강화된 스킬로 한번 사용되면 다 조건을 채울 때까 사용 못한다.
        강화된 스킬을 None으로 설정하는 방식과 조건이 채워지면 1로 늘리는 방식이 존재한다.
        스킬을 사용할 수 있는 초기값이다.
        조건을 충족한 강화한 스킬을 이례적인 절차라고 생각하고 None으로 초기값을 설정했다.
        """
        self.init_remain = None #'''o'''
        self.atk_weight = None #공격 가중치 값이다. 방어스킬에는 없기 때문에 None으로 설정했다. '''o'''
        self.remain_count = init_remain # 실제 스킬이 작동하며 횟수가 차감되 나중에 모두 차감되면 다음 턴에 다시 초기횟수 리필된다.
        """
        동기화 단계에 따른 정보를 담는다. 현재 4동기화까지 되어 있지만 차후 확장될 것을 생각하여 고정하지 않는다.
        내부에 들어있는 정보로는 uptie_level, skill_power, coin_power 속성으로 들어있다.
        전투, 턴, 스킬, 코인토스 단계로 내려오며 발생하는 효과에 관하여 저장한다.
        """
        self.uptie_info = []  

    def is_skill_by_name(self, skill_name):
        if self.skill_name == skill_name:
            return self
        return None
        
    def add_uptie_info(self, uptie_level, skill_power, coin_power):
        self.uptie_info.append(UptieInfo(uptie_level, skill_power, coin_power))

class UptieInfo:
    def __init__(self, uptie_level, skill_power, coin_power):
        self.uptie_level = uptie_level   #'''o'''
        self.skill_power = skill_power   #'''o'''
        self.coin_power = coin_power     #'''o'''
        self.skill_effects = []

class Effect:
    def __init__(self, condition):
        self.condition = condition # 조건 텍스트 "[coin 1 on hit]"
        self.effects = [] # 효과 설명 텍스 "Inflict +1 Rupture Count

# 추상화된 EffectType 클래스
class EffectType:
    def __init__(self, name, description):
        self.name = name  # Effect 이름
        self.description = description  # Effect 설명

# Passive 클래스
class Passive(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)  # condition이 있는지 여부 확인

# SupportPassive 클래스
class SupportPassive(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)

# PanicType 클래스
class PanicType(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)

# Keyword 클래스
class Keyword:
    def __init__(self, keyword, explain=None, dispellable=None):
        self.keyword = keyword
        self.explain = explain
        self.dispellable = dispellable

      
class DataContainer:
    """크롤링 데이터를 관리하는 클래스"""
    def __init__(self):
        self.identities = []

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.level_max

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.uptie_max

    def update_identity_name(self, identity_id, identity_name):
        """특정 ID의 이름을 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_name(identity_name)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_identity_rarity(self, identity_id, rarity):
        """특정 ID의 희귀도를 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_rarity(rarity)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_skill(self, identity_id, skill_number, new_type):
        """
        특정 identity의 skill_type을 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_type: 새 skill_type
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_type = new_type
                elif skill_number == 2:
                    identity.skill_2_type = new_type
                elif skill_number == 3:
                    identity.skill_3_type = new_type
                elif skill_number == "defense":
                    identity.defense_type = new_type
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def update_sin_affinity(self, identity_id, skill_number, new_affinity):
        """
        특정 identity의 sin_affinity를 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_affinity: 새 sin_affinity
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_sin_affinity = new_affinity
                elif skill_number == 2:
                    identity.skill_2_sin_affinity = new_affinity
                elif skill_number == 3:
                    identity.skill_3_sin_affinity = new_affinity
                elif skill_number == "defense":
                    identity.defense_sin_affinity = new_affinity
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def add_identity(self, identity):
        """새로운 인격 데이터를 추가."""
        self.identities.append(identity)

    def add_level_stat(self, identity_id, level, hp, defense):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_level_stat(level, hp, defense)

    def add_uptie_stat(self, identity_id, uptie, speed):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_uptie_stat(uptie, speed)

    def add_resist_info(self, identity_id, resist_hit, resist_penetrate, resist_slash):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_resist_info(resist_hit, resist_penetrate, resist_slash)

    def add_stagger_threshold(self, identity_id, first_stagger, second_stagger, third_stagger):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_stagger_threshold(first_stagger, second_stagger, third_stagger)

    
    def add_skill(self, identity_id, skill_code, skill_name, coin_count, init_level, init_remain, akt_weight):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_code == "skill 1":
                    identity.skillset.set_skill_type(skill_code, identity.skill_1_type, identity.skill_1_sin_affinity)
                elif skill_code == "skill 2":
                    identity.skillset.set_skill_type(skill_code, identity.skill_2_type, identity.skill_2_sin_affinity)
                elif skill_code == "skill 3":
                    identity.skillset.set_skill_type(skill_code, identity.skill_3_type, identity.skill_3_sin_affinity)
                elif skill_code == "defense":
                    identity.skillset.set_skill_type(skill_code, identity.defense_type, identity.defense_sin_affinity)
                identity.add_skill(skill_code, skill_name, coin_count, init_level, init_remain, akt_weight)

    def add_skill_uptie_info(self, identity_id, skill_name, uptie_level, skill_power, coin_power):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_skill_uptie_info(skill_name, uptie_level, skill_power, coin_power)
        
    def display_identities(self):
        """Identity, ID, Name, Rarity 정보를 출력"""
        if not self.identities:
            print("No identities found.")
        else:
            # 데이터를 리스트로 변환
            table = []
            for idx, identity in enumerate(self.identities, 1):
                table.append([
                    idx, identity.identity_id, identity.name, identity.rarity,
                    identity.skill_1_type, identity.skill_1_sin_affinity,
                    identity.skill_2_type, identity.skill_2_sin_affinity,
                    identity.skill_3_type, identity.skill_3_sin_affinity,
                    identity.defense_type, identity.defense_sin_affinity
                ])
            
            # 테이블 헤더 정의
            headers = [
                "Index", "ID", "Name", "Rarity",
                "Skill_1 Type", "Skill_1 Sin", "Skill_2 Type", "Skill_2 Sin",
                "Skill_3 Type", "Skill_3 Sin", "Defense Type", "Defense Sin"
            ]
            
            # 테이블 출력
            print(tabulate(table, headers=headers, tablefmt="fancy_grid"))

    def export_to_csv(self, filename, fieldnames, data):
        """
        공통적으로 사용 가능한 CSV 내보내기 메서드
        :param filename: 내보낼 파일 이름 (예: 'identity.csv', 'skill.csv')
        :param fieldnames: CSV의 헤더 리스트
        :param data: CSV로 내보낼 데이터 (리스트 형태)
        """
        try:
            # 현재작업 디렉토 기준으 'csv'폴더 없으면 생성
            if not os.path.exists('./csv'):
                os.makedirs('./csv')
            # 파일 경로 설정 (상대경로 'csv' 폴더 안에 저장)
            file_path = os.path.join('./csv', filename)
            with open(file_path, mode='w', newline='', encoding='utf-8') as file:
                writer = csv.DictWriter(file, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(data)
            print(f"Data successfully exported to {filename}")
        except Exception as e:
            print(f"Error exporting to CSV: {e}")

    def get_identity_data(self):
        """
        Identity 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "ID": identity.identity_id,
                "Name": identity.name,
                "Rarity": identity.rarity,
                "Skill_1_Type": identity.skill_1_type,
                "Skill_1_Sin_Affinity": identity.skill_1_sin_affinity,
                "Skill_2_Type": identity.skill_2_type,
                "Skill_2_Sin_Affinity": identity.skill_2_sin_affinity,
                "Skill_3_Type": identity.skill_3_type,
                "Skill_3_Sin_Affinity": identity.skill_3_sin_affinity,
                "Defense_Type": identity.defense_type,
                "Defense_Sin_Affinity": identity.defense_sin_affinity,
            }
            for identity in self.identities
        ]
        
    def export_identities_data(self, filename="identities.csv"):
        """Identity 데이터를 CSV로 내보내는 메서드"""
        identity_fieldnames = [
            "ID", "Name", "Rarity",
            "Skill_1_Type", "Skill_1_Sin_Affinity",
            "Skill_2_Type", "Skill_2_Sin_Affinity",
            "Skill_3_Type", "Skill_3_Sin_Affinity",
            "Defense_Type", "Defense_Sin_Affinity"
        ]
        identity_data = self.get_identity_data()  # Identity 데이터 가져오기
        self.export_to_csv(filename, identity_fieldnames, identity_data)  # CSV로 내보내기

    def load_identities_by_id(self, filename, identity_ids):
        """지정된 identity_ids 목록에 해당하는 Identity 데이터를 CSV에서 읽어옴."""
        identity_ids_set = set(identity_ids) # 중복 제거를 위 Set 사용
        with open(filename, mode='r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                if row["ID"] in identity_ids_set:
                    identity = Identity(
                        identity_id=row["ID"],
                        name=row["Name"],
                        rarity=row["Rarity"],
                        skill_1_type=row["Skill_1_Type"],
                        skill_1_sin_affinity=row["Skill_1_Sin_Affinity"],
                        skill_2_type=row["Skill_2_Type"],
                        skill_2_sin_affinity=row["Skill_2_Sin_Affinity"],
                        skill_3_type=row["Skill_3_Type"],
                        skill_3_sin_affinity=row["Skill_3_Sin_Affinity"],
                        defense_type=row["Defense_Type"],
                        defense_sin_affinity=row["Defense_Sin_Affinity"]
                    )
                    self.identities.append(identity)

    def get_level_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Level": level_stat.level,
                "HP": level_stat.hp,
                "Defense": level_stat.defense,
            }
            for identity in self.identities
            for level_stat in identity.level_stats
        ]

    def export_level_stats_data(self, filename="levelstats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        level_stats_fieldnames = ["Identity_ID", "Level", "HP", "Defense"]
        level_stats_data = self.get_level_stats_data()  # level_stats 데이터 가져오기
        self.export_to_csv(filename, level_stats_fieldnames, level_stats_data)  # CSV로 내보내기

    def get_uptie_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Uptie": uptie_stat.uptie,
                "Speed": uptie_stat.speed,
            }
            for identity in self.identities
            for uptie_stat in identity.uptie_stats
        ]

    def export_uptie_stats_data(self, filename="Uptiestats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        uptie_stats_fieldnames = ["Identity_ID", "Uptie", "Speed"]
        uptie_stats_data = self.get_uptie_stats_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, uptie_stats_fieldnames, uptie_stats_data)  # CSV로 내보내기

    def get_resist_info_data(self):
        """
        모든 Identity의 resist_info 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Resist_Hit": identity.resist_info.resist_hit,
                "Resist_Penetrate": identity.resist_info.resist_penetrate,
                "Resist_Slash": identity.resist_info.resist_slash,
            }
            for identity in self.identities
        ]

    def export_resist_info_data(self, filename="ResistInfos.csv"):
        """ResistInfo 데이터를 CSV로 내보내는 메서드"""
        resist_infos_fieldnames = ["Identity_ID", "Resist_Hit", "Resist_Penetrate", "Resist_Slash"]
        resist_infos_data = self.get_resist_info_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, resist_infos_fieldnames, resist_infos_data)  # CSV로 내보내기

    def export_uptie_stats_data(self, filename="Uptiestats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        uptie_stats_fieldnames = ["Identity_ID", "Uptie", "Speed"]
        uptie_stats_data = self.get_uptie_stats_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, uptie_stats_fieldnames, uptie_stats_data)  # CSV로 내보내기

    def get_stagger_threshold_data(self):
        """
        모든 Identity의 stagger_threshold 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "First_Stagger": identity.stagger_threshold.first_stagger,
                "Second_Stagger": identity.stagger_threshold.second_stagger,
                "Third_Stagger": identity.stagger_threshold.third_stagger,
            }
            for identity in self.identities
        ]

    def export_stagger_threshold_data(self, filename="StaggerThresholds.csv"):
        """StaggerThresholds 데이터를 CSV로 내보내는 메서드"""
        stagger_thresholds_fieldnames = ["Identity_ID", "First_Stagger", "Second_Stagger", "Third_Stagger"]
        stagger_thresholds_data = self.get_stagger_threshold_data()  # stagger_thresholds 데이터 가져오기
        self.export_to_csv(filename, stagger_thresholds_fieldnames, stagger_thresholds_data)  # CSV로 내보내기


async def main():
    data_container = DataContainer()
    scraper = Scraper(container=data_container, base_url="https://limbus.kusoge.xyz", img_url="https://img.kusoge.xyz/limbus/img")
    await scraper.scrape()

await main() 

Added skill: skill 1:Begone...
Updated Uptie for Begone... (Uptie Level: 1): Skill Power: 2, Coin Power: +4


Error: ElementHandle.evaluate: SyntaxError: missing ) after argument list
    at eval (<anonymous>)
    at UtilityScript.evaluate (<anonymous>:234:30)
    at UtilityScript.<anonymous> (<anonymous>:1:44)

In [44]:
import os
import csv
import re
import numpy as np
import pandas as pd
from playwright.async_api import async_playwright
from tabulate import tabulate

# Sin Color 이름 변환을 위한 매핑
resource_mapping = {
    "AZURE": "Gloom",
    "VIOLET": "Envy",
    "AMBER": "Sloth",
    "SHAMROCK": "Gluttony",
    "CRIMSON": "Wrath",
    "SCARLET": "Lust",
    "INDIGO": "Pride"
}

def convert_resource(resource_name):
    """Sin Color 이름을 변환하는 함수. 매핑되지 않은 경우 원래 이름을 반환."""
    return resource_mapping.get(resource_name, resource_name)

class Scraper:
    """웹 페이지 크롤링을 위한 클래스"""
    def __init__(self, container, base_url, img_url):
        self.container = container
        self.base_url = base_url
        self.img_url = img_url
        self.uptie_level = 1

    async def handle_request(self, route, request):
        """이미지와 폰트 리소스 로드를 차단."""
        if request.resource_type in ["image", "font"]:
            await route.abort()
        else:
            await route.continue_()
            
    async def scrape(self):
        """크롤링 메인 함수"""
        async with async_playwright() as p:
            browser = await p.chromium.launch(headless=True)
            page = await browser.new_page()
            await page.route("**/*", self.handle_request)
            # Step 1: /identity 페이지에서 기본 정보 크롤링
            #await self.crawl_identity_page(page)

            # Step 2: /identity/{identity_id} 상세 페이지 크롤링
            #identity_ids_to_load = ["10104", "10310", "10710","10808", "10911", "11005"]
            #identity_ids_to_load = ["10104", "10911"]#,"10310", "10710","10808", "11005", "11210"]
            identity_ids_to_load = ["10911"]#,"10310", "10710","10808", "11005", "11210"]
            self.container.load_identities_by_id('./csv/identities.csv', identity_ids_to_load)
            await self.crawl_identity_details_for_all(page)
            #self.container.display_identities()
            #self.container.export_identities_data("identities.csv")
            #self.container.export_level_stats_data("Levelstats.csv")
            #self.container.export_uptie_stats_data("Uptiestats.csv")
            #self.container.export_resist_info_data("ResistInfos.csv")
            #self.container.export_stagger_threshold_data("StaggerThresholds.csv")
            #page.set_default_timeout(10000)
            await page.wait_for_timeout(100000)
            await browser.close()

    async def get_identity_id_from_row(self, row):
        """테이블 행(row)에서 인격 ID를 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_url = await identity_cell.get_attribute('href')
            identity_id = identity_url.split('/identity/')[-1]
            return identity_id
        return "Unknown"

    async def get_identity_name_from_row(self, row):
        """테이블 행(row)에서 인격 이름을 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_name = (await identity_cell.inner_text()).replace("\n", " ").strip()
            return identity_name
        return "Unknown"
        
    async def crawl_identity_page(self, page):
        """/identity 페이지로 이동하여 인격 기본 정보를 추출."""
        await page.goto(self.base_url + "/identity")
        await self.extract_identity_page(page)

    async def extract_identity_page(self, page):
        """/identity 페이지의 모든 테이블 행에서 인격 정보를 추출."""
        identity_rows = await page.query_selector_all('tr')
        if len(identity_rows) > 1:
            for idx, identity_row in enumerate(identity_rows):
                if idx == 0:
                    continue # 0번째요소이름을 보여주는 열은 건너뜀
                await self.scrape_identity_name_and_identity_id(identity_row)
                await self.extract_rarity(identity_row)
                for i in range(1, 4):  # 스킬1~3 정보 추출
                    await self.extract_atk_and_resource(identity_row, i)
                await self.extract_defense_and_resource(identity_row)

    async def scrape_identity_name_and_identity_id(self, row):
        """인격 번호와 이름을 추출하여 DataContainer에 저장."""
        try:
            identity_id = await self.get_identity_id_from_row(row)
            identity_name = await self.get_identity_name_from_row(row)
            identity = Identity(identity_id)
            self.container.add_identity(identity)
            self.container.update_identity_name(identity_id, identity_name)
        except Exception as e:
            print(f"Error extracting identity name and identity id: {e}")

    async def get_element_attribute(self, element, selector, attribute):
        """요소에서 지정된 속성(attribute)을 추출."""
        try:
            target_element = await element.query_selector(selector)
            if target_element:
                return await target_element.get_attribute(attribute)
        except Exception as e:
            print(f"Error getting element attribute: {e}")
        return None
    
    async def extract_rarity(self, row):
        """테이블 행(row)에서 희귀도를 추출하여 업데이트."""
        try:
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
                sibling = await parent.evaluate_handle('node => node.nextElementSibling')
                if sibling:
                    rarity_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/rarity_"]', 'src')
                    if rarity_img_src:
                        rarity_level = rarity_img_src.split('_')[-1].split('.')[0]  # rarity_1.png -> 1
                        identity_id = await self.get_identity_id_from_row(row)
                        self.container.update_identity_rarity(identity_id, rarity_level)
                    else:
                        print("Rarity image not found in the next sibling TD.")
        except Exception as e: 
            print(f"Error extracting rarity: {e}")

        return "Unknown"

    async def extract_atk_and_resource(self, row, skill_number):
        """ 공격 타입(ATK Type)과 자원(Attribute) 추출 함수 - 스킬 번호에 따라 처리 """
        try:
            #print(f"Attempting to locate skill{skill_number} type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 해당 스킬 번호에 맞는 형제로 이동
                sibling = parent
                for _ in range(skill_number + 1):  # 스킬 번호에 맞게 이동
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                atk_type, resource = "Unknown", "Unknown"
        
                if sibling:
                    # 공격 타입 이미지 (atk_type) 찾기
                    atk_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/atk_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if atk_type_img_src:
                        atk_type = atk_type_img_src.split('_')[-1].split('.')[0]  # atk_type_SLASH.png -> SLASH
                        #print(f"Extracted attack type: {atk_type}")
                        self.container.update_skill(identity_id, skill_number, atk_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted resource: {resource}")
                        
                        # 리소스를 변환
                        resource = convert_resource(resource)
                        #print(f"Converted resource: {resource}")
                        self.container.update_sin_affinity(identity_id, skill_number, resource)
                        
        except Exception as e:
            print(f"Error extracting skill{skill_number} type and resource: {e}")

    async def extract_defense_and_resource(self, row):
        """ 방어 타입(Def Type)과 자원(Attribute) 추출 함수 """
        try:
            #print("Attempting to locate defense type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 다섯 번째 형제 td로 이동 (방어 관련 정보가 있는 <td>)
                sibling = parent
                for _ in range(5):  # 방어 정보는 5번째 형제에 있음
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                def_type, def_resource = "Unknown", "Unknown"
        
                if sibling:
                    # 방어 타입 이미지 (def_type) 찾기
                    def_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/def_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if def_type_img_src:
                        def_type = def_type_img_src.split('_')[-1].split('.')[0]  # def_type_GUARD.png -> GUARD
                        #print(f"Extracted defense type: {def_type}")
                        self.container.update_skill(identity_id, 'defense', def_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        def_resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted defense resource: {def_resource}")
                        
                        # 방어 리소스를 변환
                        def_resource = convert_resource(def_resource)
                        #print(f"Converted defense resource: {def_resource}")
                        self.container.update_sin_affinity(identity_id, 'defense', def_resource)
 
        except Exception as e:
            print(f"Error extracting defense type and resource: {e}")

    async def crawl_identity_details_for_all(self, page):
        """모든 인격의 세부 정보를 크롤링."""
        for identity in self.container.identities:
            await page.wait_for_load_state("networkidle")
            await page.goto(self.base_url + "/identity/"+identity.identity_id)
            """ 레벨과 동기화와 관련 없는 참관타 저항은 밑에서 변경하는 것과 연관성이 적기 때문에 위에서 처리한다.
            print(f"Identity_id: {identity.identity_id}")
            results = []
            resist_type_imgs = [f"{self.img_url}/ui/atk_type_HIT.png",
                                f"{self.img_url}/ui/atk_type_PENETRATE.png",
                                f"{self.img_url}/ui/atk_type_SLASH.png"]
            for resist_type_img in resist_type_imgs:
                result = await self.extract_resist_type(page, resist_type_img, 1)
                results.append(result)
            self.container.add_resist_info(identity.identity_id, results[0], results[1], results[2])
            """
            """
            # Stagger Threshold 값 추출
            stagger_threshold = await self.extract_stagger_threshold_values(page)
            self.container.add_stagger_threshold(identity.identity_id, stagger_threshold[0], stagger_threshold[1], stagger_threshold[2])
            """
            await self.initialize_uptie_and_level(page)
            """ 레벨에 따른 level, hp, defense 다르게 되어 있는 것을 출력 확인 객체로 들어가서 csv로 옮기는 과정까지 진행
            for i in range(1, identity.level_max+1):
                await self.move_slider_to_value(page,i)
                await self.extract_level_status(page, identity.identity_id, i)
            """
            """ 동기화에 따른 speed 다르게 되어 있는 것을 출력 확인 객체로 들어가서 csv로 옮기는 과정까지 진행
            for i in range(1, identity.uptie_max+1):
                await self.uptie_click(page,i)
                await self.extract_uptie_status(page, identity.identity_id, i)
            """
            await self.crawl_skills(page, identity.identity_id)
            
    async def initialize_uptie_and_level(self, page):
        await self.uptie_click(page, 1)
        await self.move_slider_to_value(page, 1)
        self.uptie_level = 1
        #return 1
    
    async def uptie_click(self, page, phase):
        fab_button_selector = ".q-btn.q-btn--fab"
        await page.wait_for_selector(fab_button_selector)
        await page.click(fab_button_selector)
        #print(f"[{setup_uptie_button}] 상위 동기화 버튼 클릭 완료")
        setup_uptie_button = f'a:has(img[src="{self.img_url}/ui/uptie_{phase}.png"])'
        await page.wait_for_selector(setup_uptie_button)
        await page.click(setup_uptie_button)
        #print(f"{phase} 동기화 버튼 클릭 완료")

    async def move_slider_to_value(self, page, target_value):
        """
        슬라이더를 주어진 값으로 이동시킵니다.
        target_value는 슬라이더의 최소값부터 최대값 사이여야 합니다.
        """
        slider_selector = ".q-slider"
        # 슬라이더의 범위 확인
        min_value = int(await page.get_attribute(slider_selector, "aria-valuemin"))
        max_value = int(await page.get_attribute(slider_selector, "aria-valuemax"))
    
        if target_value < min_value or target_value > max_value:
            raise ValueError(f"Target value {target_value} is out of bounds ({min_value}, {max_value})")
    
        # 슬라이더 thumb 요소와 위치 가져오기
        slider_thumb = await page.query_selector(f"{slider_selector} .q-slider__thumb")
        thumb_box = await slider_thumb.bounding_box()
        
        # 슬라이더의 현재 위치 및 목표 위치 계산
        track = await page.query_selector(f"{slider_selector} .q-slider__track")
        track_box = await track.bounding_box()
        
        # 목표 x 위치 계산 (슬라이더 트랙 내에서 비율 기반)
        target_percentage = (target_value - min_value) / (max_value - min_value)
        target_x = track_box["x"] + target_percentage * track_box["width"]
        current_x = thumb_box["x"] + thumb_box["width"] / 2
        y = thumb_box["y"] + thumb_box["height"] / 2
    
        # 슬라이더를 드래그하여 목표 위치로 이동
        await page.mouse.move(current_x, y)
        await page.mouse.down()
        await page.mouse.move(target_x, y, steps=10)  # 부드럽게 이동
        await page.mouse.up()
    
        #print(f"슬라이더를 {target_value}로 이동 완료.")
    
        # 슬라이더 값 확인 및 강제 이벤트 트리거
        await page.evaluate(
            f"""
            (slider) => {{
                slider.dispatchEvent(new Event('input'));  // oninput 이벤트 트리거
                slider.dispatchEvent(new Event('change'));  // onchange 이벤트 트리거
            }}
            """,
            await page.query_selector(slider_selector)
        )
    
        # 이동 후 값 확인
        current_value = await page.get_attribute(slider_selector, "aria-valuenow")
        if current_value != str(target_value):
            print(f"경고: 슬라이더 값이 예상과 다릅니다. 현재 값: {current_value}, 예상 값: {target_value}")

    async def get_second_status_element(self, page):
        """
        두 번째 'Status' 요소에 접근하여 반환하는 메소드
        """
        status_elements = await page.query_selector_all("text=Status")
        if len(status_elements) > 1:
            second_status_element = status_elements[1]
            #print("두 번째 'Status' 요소에 접근합니다.")
            return second_status_element
        else:
            #print("두 번째 'Status' 요소가 존재하지 않습니다.")
            return None

    async def extract_level_status(self, page, identity_id, level):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 레벨에 따라 HP와 Defense가 변한다.
        hp_value = await self.extract_hp(second_status_element)
        defense_value = await self.extract_defense(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Level={level}, HP={hp_value}, Defense={defense_value}")
        self.container.add_level_stat(identity_id,level, hp_value, defense_value)
    
    async def extract_hp(self, status_element):
        # HP 추출
        hp_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].children[0].nextElementSibling.textContent.trim()"
        )
        return hp_value
    
    async def extract_defense(self, status_element):
        # Defense 추출
        defense_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return defense_value

    async def extract_uptie_status(self, page, identity_id, uptie):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 uptie에 따라 speed는 변한다.
        speed_value = await self.extract_speed(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Uptie={uptie}, Speed={speed_value}")
        self.container.add_uptie_stat(identity_id,uptie, speed_value)

    async def extract_speed(self, status_element):
        # Speed 추출
        speed_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return speed_value

    async def extract_resist_type(self, page, image_src, depth):
        # 이미지 요소 찾기
        #print(image_src)
        image = await page.query_selector(f'img[src="{image_src}"]')
        #print(image)
        if not image:
            return "Unknown"
    
        # 이미지의 부모 요소를 찾아서 올라감
        parent = image
        for _ in range(depth):
            parent = await parent.evaluate_handle('node => node.parentElement')
            if not parent:
                return "Unknown"
    
        # 부모의 형제 요소 찾기
        sibling = await parent.evaluate_handle('node => node.nextElementSibling')
        
        if sibling:
            value = await sibling.evaluate('node => node.textContent')
            if value:
                return value.strip()
        
        return "Unknown"

    async def extract_stagger_threshold_values(self, page):
        # 'div.row.center.flex-center.q-pt-xs' 요소들 찾기
        elements = await page.query_selector_all('div.row.center.flex-center.q-pt-xs')
        
        # 각 요소의 텍스트 내용 출력하여 'Stagger Threshold' 요소를 찾기
        for element in elements:
            text = await element.text_content()
            
            if text and "Stagger Threshold" in text:
                # 'Stagger Threshold'가 있는 요소를 찾았다면, 그 이후 형제 요소들에서 값을 추출
                parent = await element.evaluate_handle('node => node.parentElement')
                sibling_elements = await parent.query_selector_all('div.row.justify-evenly div')  # Threshold 값이 들어있는 div들
                
                stagger_values = []
                for sibling in sibling_elements:
                    text_content = await sibling.text_content()
                    match = re.search(r'(\d+%)', text_content)
                    if match:
                        stagger_values.append(match.group(1))  # 퍼센트 값만 리스트에 추가
                while len(stagger_values) < 3:
                    stagger_values.append(np.nan)
                #print(stagger_values)
                return stagger_values  # 추출된 값(또는 np.nan 포함)을 반환
        
        return [np.nan, np.nan, np.nan]  # 만약 'Stagger Threshold'를 찾지 못한 경우 기본값 반환

    async def crawl_skills(self, page, identity_id):
        await self.crawl_skill(page, identity_id, "skill 1")
        await self.crawl_skill(page, identity_id, "skill 2")
        await self.crawl_skill(page, identity_id, "skill 3")
        await self.crawl_skill(page, identity_id, "defense")


    async def crawl_skill(self, page, identity_id, skill_type):
        skills_elements = await page.query_selector_all('div.skilltag')
        for skill_element in skills_elements:
            skill_text = await skill_element.text_content()
            if skill_text.strip().lower() == skill_type:
                await self.process_skill(skill_element, page, identity_id, skill_type)
        
    async def process_skill(self, skill_element, page, identity_id, skill_type):
        # skill_type이 "skill 3"일 경우 uptie_value가 3 이상일 때만 정보를 저장
        for uptie_value in range(1, 5):  # Uptie levels 1 to 4
            # "skill 3"은 uptie_value가 3 이상일 때만 처리
            if skill_type == "skill 3" and uptie_value < 3:
                continue  # skip if uptie_value is less than 3 for skill 3
    
            # uptie_level에 맞게 클릭
            await self.uptie_click(page, uptie_value)
            if skill_type != "skill 3" and uptie_value > 1:
                # Uptie 정보 갱신
                await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
                continue
            elif skill_type == "skill 3" and uptie_value > 3:
                # Uptie 정보 갱신
                await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
                continue
            # 스킬의 세부 정보를 추출
            skill_details = await self.extract_skill_details(skill_element, skill_type)
            skill_name = skill_details["skill_name"]
    
            # 스킬 정보를 저장
            self.container.add_skill(
                identity_id, skill_type, 
                skill_name, skill_details["coin_count"],
                skill_details["init_level"], skill_details["init_remain"], 
                skill_details["atk_weight"]
            )
            print(f"Added skill: {skill_type}:{skill_name}")
    
            # Uptie 정보 갱신
            await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
    
    async def extract_skill_details(self, skill_element, skill_type):
        # 스킬 세부 정보를 추출하는 함수
        skill_name = await self.skill_name(skill_element)
        coin_count = await self.coin_count(skill_element, ".col-auto > .column > .row")
        init_level = await self.init_level(skill_element)
        init_remain = await self.init_remain(skill_element, skill_type, "div.row.justify-start > div.attr.flex.flex-center > div.q-px-sm")
        atk_weight = await self.atk_weight(skill_element, skill_type)
        return {
            "skill_name": skill_name,
            "coin_count": coin_count,
            "init_level": init_level,
            "init_remain": init_remain,
            "atk_weight": atk_weight
        }
    
    async def update_uptie_info(self, page, skill_element, identity_id, skill_name, skill_type, uptie_value):
        # Uptie 정보를 추출하고 업데이트하는 함수
        uptie_info = await self.extract_uptie_info(skill_element)
        self.container.add_skill_uptie_info(
            identity_id, skill_name, 
            uptie_info["skill_power"], uptie_info["coin_power"], 
            uptie_value
        )
        print(
            f"Updated Uptie for {skill_name} (Uptie Level: {uptie_value}): "
            f"Skill Power: {uptie_info['skill_power']}, Coin Power: {uptie_info['coin_power']}"
        )
        effect = await self.skill_effects(skill_element)
        print(effect)

    async def extract_uptie_info(self, skill_element):
        skill_data = await self.skill_and_coin_power(skill_element)
        return {
            "skill_power": skill_data.get("skillPower"),
            "coin_power": skill_data.get("coinPower")
        }

    async def extract_skill_effect(self, skill_element):
        # Create the Effect object
        effect_obj = Effect(condition=None)
        # Extract effects
        effects = await self.skill_effects(skill_element)
        if effects:
            effect_obj.effects.append(effects)
        return effect_obj

    async def skill_and_coin_power(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                // 조부모로 이동
                const grandparent = node.parentElement.parentElement;
                // 이전 형제 탐색
                const previousSibling = grandparent.previousElementSibling;
                if (!previousSibling) return {{ skillPower: null, coinPower: null }}; // 이전 형제가 없으면 기본값 반환
                // 이전 형제 내부의 .row.justify-around > div > div 탐색
                const targetDivs = previousSibling.querySelectorAll(".row.justify-around > div");
                if (targetDivs.length < 3) return {{ skillPower: null, coinPower: null }}; // 배열 길이 확인
                // 1st div: 스킬 위력
                const skillPower = targetDivs[0]?.querySelector('div')?.textContent?.trim() || null; 
                // 3rd div: 코인 파워 값 (3번째 div에서 텍스트 추출)
                const coinPower = targetDivs[2]?.textContent?.trim() || null;
                return {{ skillPower, coinPower }};
            }}
        """)


    async def coin_count(self, skill_element, hierarchy_selector):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector('{hierarchy_selector}');
                if (!targetElement) return 0;
                return targetElement.childElementCount;
            }}
        """)

    async def init_remain(self, skill_element, skill_type, hierarchy_selector):
        if skill_type == "defense":
            return -1
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector('{hierarchy_selector}');
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def skill_name(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector(".block");
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def init_level(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector("div.text-bold.text-h6.h6.q-pl-xs.q-pr-md");
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def atk_weight(self, skill_element, skill_type):
        if skill_type == "defense":
            return 0
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector("div.text-bold.text-h6.h6.q-pl-xs.q-pr-md");
                const doubleNextSibling = targetElement.nextElementSibling.nextElementSibling;
                if (!doubleNextSibling) return 0;
                return doubleNextSibling.textContent?.trim();
            }}
        """)

    async def skill_effects(self, skill_element):
        return await skill_element.evaluate("""
            node => {
                const parent = node.parentElement;
                const doubleNextSibling = parent.nextElementSibling?.nextElementSibling;
                if (!doubleNextSibling) return null;
        
                // doubleNextSibling에서 첫 번째 div.col 요소 찾기
                const textNode = doubleNextSibling.querySelector("div.col");
                if (!textNode) return null;
        
                const results = {
                    keywords: [], // 키워드 저장
                    conditions: [], // 조건 저장
                    effects: [] // 효과 저장
                };
        
                // 1. 조건 추출 (대괄호 텍스트)
                const conditionElements = textNode.querySelectorAll("span");
                conditionElements.forEach(span => {
                    const text = span.textContent?.trim();
                    if (text?.startsWith("[") && text?.endsWith("]")) {
                        results.conditions.push(text);
                    }
                });
        
                // 2. 키워드 추출
                const keywordElements = textNode.querySelectorAll('a[href^="/en/keyword/"]');
                keywordElements.forEach(a => {
                    const span = a.querySelector('span.q-btn__content.text-center.col.items-center.q-anchor--skip.justify-center.row>span');
                    if (span) {
                        const keywordText = span.textContent.trim();
                        if (keywordText && !results.keywords.includes(keywordText)) {
                            results.keywords.push(keywordText); // 키워드 저장
                        }
                    }
                });
        
                // 3. 효과 추출
                const effectElements = textNode.querySelectorAll("span");
                effectElements.forEach(span => {
                    const text = span.textContent?.trim();
                    if (text && !text.startsWith("[")) { // 조건이 아님
                        if (results.keywords.includes(text)) {
                            results.effects.push('keyword:' + text);
                        } else {
                            results.effects.push(text);
                        }
                    }
                });
        
                return results;
            }
        """)


class Identity:
    """인격 정보를 저장하는 클래스"""
    def __init__(self, identity_id, name=None, rarity=None,\
                 skill_1_type=None, skill_1_sin_affinity=None,\
                 skill_2_type=None, skill_2_sin_affinity=None,\
                 skill_3_type=None, skill_3_sin_affinity=None,\
                 defense_type=None, defense_sin_affinity=None):
        self.identity_id = identity_id
        self.name = name
        self.rarity = rarity
        # 선언만 하는 부분 
        self.skill_1_type = skill_1_type 
        self.skill_1_sin_affinity = skill_1_sin_affinity 
        self.skill_2_type = skill_2_type 
        self.skill_2_sin_affinity = skill_2_sin_affinity 
        self.skill_3_type = skill_3_type 
        self.skill_3_sin_affinity = skill_3_sin_affinity 
        self.defense_type = defense_type 
        self.defense_sin_affinity = defense_sin_affinity
        self.level_max = 50
        self.uptie_max = 4
        self.level_stats = []  # LevelStat 객체 리스트
        self.uptie_stats = [] # UptieStat 객체 리스트
        self.resist_info = None
        self.stagger_threshold = None
        self.skillset = SkillSet()  # Skill정보를 담고 있는 skillset 객체 
        self.passive_infos = []  # 패시브 객체를 담고 있는 배열
        self.support_passive_infos = [] # 서포트 패시 객체를 담고 있는 배열
        self.panic_type = [] # 혼란상태 발생시 나타나는 효과를 담고 있는 객체 배열

    def update_name(self, name):
        self.name = name

    def update_rarity(self, rarity):
        self.rarity = rarity

    def add_level_stat(self, level, hp, defense):
        self.level_stats.append(LevelStat(level, hp, defense))

    def add_uptie_stat(self, uptie, speed):
        self.uptie_stats.append(UptieStat(uptie, speed))

    def add_resist_info(self, resist_hit, resist_penetrate, resist_slash):
        self.resist_info = ResistInfo(resist_hit, resist_penetrate, resist_slash)

    def add_stagger_threshold(self, first_stagger, second_stagger, third_stagger):
        self.stagger_threshold = StaggerThreshold(first_stagger, second_stagger, third_stagger)

    def add_skill(self, skill_code, skill_name, coin_count, init_level, init_remain, akt_weight):
        self.skillset.add_skill(skill_code, Skill(skill_code, skill_name, coin_count, init_level, init_remain, akt_weight))
        
    def add_skill_uptie_info(self, skill_name, uptie_level, skill_power, coin_power):
        self.skillset.add_skill_uptie_info(skill_name, uptie_level, skill_power, coin_power)
        
    def update_details(self, page):
        # 세부 정보 업데이트 로직 (가정)
        pass
    #def init_uptie(self, page):
    #    init_uptie = 

# LevelStat 객체 정의
class LevelStat:
    def __init__(self, level, hp=None, defense=None):
        self.level = level
        self.hp = hp
        self.defense = defense
    def update_level(self, level):
        self.level = level
    def update_hp(self, hp):
        self.hp = hp
    def update_defense(self, defense):
        self.defense = defense

# UptieStat 객체 정의
class UptieStat:
    def __init__(self, uptie, speed=None):
        self.uptie = uptie
        self.speed = speed
    def update_speed(self, speed):
        self.speed = speed

# ResistInfo 객체 정의
class ResistInfo:
    def __init__(self, resist_hit=None, resist_penetrate=None, resist_slash=None):
        self.resist_hit = resist_hit
        self.resist_penetrate = resist_penetrate
        self.resist_slash = resist_slash

# ResistInfo 객체 정의
class StaggerThreshold:
    def __init__(self, first_stagger=None, second_stagger=None, third_stagger=None):
        self.first_stagger = first_stagger
        self.second_stagger = second_stagger
        self.third_stagger = third_stagger

class SkillSet:
    def __init__(self):

        self.skill_1_type = ""
        self.skill_2_type = ""
        self.skill_3_type = ""
        self.defense_skill_type = ""

        self.skill_1_sin_affinity = ""
        self.skill_2_sin_affinity = ""
        self.skill_3_sin_affinity = ""
        self.defense_skill_sin_affinity = ""

        self.skill_1_details = []
        self.skill_2_details = []
        self.skill_3_details = []
        self.defense_skill_details = []

    def set_skill_type(self, skill_code, skill_type, sin_affinity):
        if skill_code == "skill 1":
            self.skill_1_type = skill_type
            self.skill_1_sin_affinity = sin_affinity
        elif skill_code == "skill 2":
            self.skill_2_type = skill_type
            self.skill_2_sin_affinity = sin_affinity
        elif skill_code == "skill 3":
            self.skill_3_type = skill_type
            self.skill_3_sin_affinity = sin_affinity
        elif skill_type == "defense":
            self.defense_skill_type = skill_type
            self.defense_skill_sin_affinity = sin_affinity
    
    def add_skill(self, skill_code, skill):
        if skill_code == "skill 1":
            self.skill_1_details.append(skill)
        elif skill_code == "skill 2":
            self.skill_2_details.append(skill)
        elif skill_code == "skill 3":
            self.skill_3_details.append(skill)
        elif skill_code == "defense":
            self.defense_skill_details.append(skill)

    def get_skill_by_name(self, skill_name):
        for skill_list in [self.skill_1_details, self.skill_2_details, self.skill_3_details, self.defense_skill_details]:
            for skill in skill_list:
                result = skill.is_skill_by_name(skill_name)
                if result:
                    return result
        return None

    def add_skill_uptie_info(self, skill_name, uptie_level, skill_power, coin_power):
        skill = self.get_skill_by_name(skill_name)
        skill.add_uptie_info(uptie_level, skill_power, coin_power)
        

class Skill:
    def __init__(self, skill_code, skill_name, coin_count, init_level, init_remain, akt_weight):
        self.skill_code = skill_code # "Skill 1"/"Skill 2"/"Skill 3"/"Defense" '''o'''
        self.skill_name = skill_name # 스킬 이름 '''o'''
        self.coin_count = coin_count # 동전 개수 '''o'''
        self.init_level = init_level # 스킬이 초기 배정받은 레벨, 인격 레벨에 - 1 한 값을 더해주면 스킬의 현 level을 구할 수 있다. '''o'''
        """
        남아있는 스킬 개수 사용하면, 하나씩 줄어들고 모든 스킬을 사용하면 이 값으로 돌아간다.
        스킬의 남은 숫자없는 스킬은 강화된 스킬로 한번 사용되면 다 조건을 채울 때까 사용 못한다.
        강화된 스킬을 None으로 설정하는 방식과 조건이 채워지면 1로 늘리는 방식이 존재한다.
        스킬을 사용할 수 있는 초기값이다.
        조건을 충족한 강화한 스킬을 이례적인 절차라고 생각하고 None으로 초기값을 설정했다.
        """
        self.init_remain = None #'''o'''
        self.atk_weight = None #공격 가중치 값이다. 방어스킬에는 없기 때문에 None으로 설정했다. '''o'''
        self.remain_count = init_remain # 실제 스킬이 작동하며 횟수가 차감되 나중에 모두 차감되면 다음 턴에 다시 초기횟수 리필된다.
        """
        동기화 단계에 따른 정보를 담는다. 현재 4동기화까지 되어 있지만 차후 확장될 것을 생각하여 고정하지 않는다.
        내부에 들어있는 정보로는 uptie_level, skill_power, coin_power 속성으로 들어있다.
        전투, 턴, 스킬, 코인토스 단계로 내려오며 발생하는 효과에 관하여 저장한다.
        """
        self.uptie_info = []  

    def is_skill_by_name(self, skill_name):
        if self.skill_name == skill_name:
            return self
        return None
        
    def add_uptie_info(self, uptie_level, skill_power, coin_power):
        self.uptie_info.append(UptieInfo(uptie_level, skill_power, coin_power))

class UptieInfo:
    def __init__(self, uptie_level, skill_power, coin_power):
        self.uptie_level = uptie_level   #'''o'''
        self.skill_power = skill_power   #'''o'''
        self.coin_power = coin_power     #'''o'''
        self.skill_effects = []

class Effect:
    def __init__(self, condition):
        self.condition = condition # 조건 텍스트 "[coin 1 on hit]"
        self.effects = [] # 효과 설명 텍스 "Inflict +1 Rupture Count

# 추상화된 EffectType 클래스
class EffectType:
    def __init__(self, name, description):
        self.name = name  # Effect 이름
        self.description = description  # Effect 설명

# Passive 클래스
class Passive(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)  # condition이 있는지 여부 확인

# SupportPassive 클래스
class SupportPassive(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)

# PanicType 클래스
class PanicType(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)

# Keyword 클래스
class Keyword:
    def __init__(self, keyword, explain=None, dispellable=None):
        self.keyword = keyword
        self.explain = explain
        self.dispellable = dispellable

      
class DataContainer:
    """크롤링 데이터를 관리하는 클래스"""
    def __init__(self):
        self.identities = []

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.level_max

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.uptie_max

    def update_identity_name(self, identity_id, identity_name):
        """특정 ID의 이름을 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_name(identity_name)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_identity_rarity(self, identity_id, rarity):
        """특정 ID의 희귀도를 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_rarity(rarity)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_skill(self, identity_id, skill_number, new_type):
        """
        특정 identity의 skill_type을 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_type: 새 skill_type
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_type = new_type
                elif skill_number == 2:
                    identity.skill_2_type = new_type
                elif skill_number == 3:
                    identity.skill_3_type = new_type
                elif skill_number == "defense":
                    identity.defense_type = new_type
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def update_sin_affinity(self, identity_id, skill_number, new_affinity):
        """
        특정 identity의 sin_affinity를 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_affinity: 새 sin_affinity
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_sin_affinity = new_affinity
                elif skill_number == 2:
                    identity.skill_2_sin_affinity = new_affinity
                elif skill_number == 3:
                    identity.skill_3_sin_affinity = new_affinity
                elif skill_number == "defense":
                    identity.defense_sin_affinity = new_affinity
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def add_identity(self, identity):
        """새로운 인격 데이터를 추가."""
        self.identities.append(identity)

    def add_level_stat(self, identity_id, level, hp, defense):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_level_stat(level, hp, defense)

    def add_uptie_stat(self, identity_id, uptie, speed):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_uptie_stat(uptie, speed)

    def add_resist_info(self, identity_id, resist_hit, resist_penetrate, resist_slash):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_resist_info(resist_hit, resist_penetrate, resist_slash)

    def add_stagger_threshold(self, identity_id, first_stagger, second_stagger, third_stagger):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_stagger_threshold(first_stagger, second_stagger, third_stagger)

    
    def add_skill(self, identity_id, skill_code, skill_name, coin_count, init_level, init_remain, akt_weight):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_code == "skill 1":
                    identity.skillset.set_skill_type(skill_code, identity.skill_1_type, identity.skill_1_sin_affinity)
                elif skill_code == "skill 2":
                    identity.skillset.set_skill_type(skill_code, identity.skill_2_type, identity.skill_2_sin_affinity)
                elif skill_code == "skill 3":
                    identity.skillset.set_skill_type(skill_code, identity.skill_3_type, identity.skill_3_sin_affinity)
                elif skill_code == "defense":
                    identity.skillset.set_skill_type(skill_code, identity.defense_type, identity.defense_sin_affinity)
                identity.add_skill(skill_code, skill_name, coin_count, init_level, init_remain, akt_weight)

    def add_skill_uptie_info(self, identity_id, skill_name, uptie_level, skill_power, coin_power):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_skill_uptie_info(skill_name, uptie_level, skill_power, coin_power)
        
    def display_identities(self):
        """Identity, ID, Name, Rarity 정보를 출력"""
        if not self.identities:
            print("No identities found.")
        else:
            # 데이터를 리스트로 변환
            table = []
            for idx, identity in enumerate(self.identities, 1):
                table.append([
                    idx, identity.identity_id, identity.name, identity.rarity,
                    identity.skill_1_type, identity.skill_1_sin_affinity,
                    identity.skill_2_type, identity.skill_2_sin_affinity,
                    identity.skill_3_type, identity.skill_3_sin_affinity,
                    identity.defense_type, identity.defense_sin_affinity
                ])
            
            # 테이블 헤더 정의
            headers = [
                "Index", "ID", "Name", "Rarity",
                "Skill_1 Type", "Skill_1 Sin", "Skill_2 Type", "Skill_2 Sin",
                "Skill_3 Type", "Skill_3 Sin", "Defense Type", "Defense Sin"
            ]
            
            # 테이블 출력
            print(tabulate(table, headers=headers, tablefmt="fancy_grid"))

    def export_to_csv(self, filename, fieldnames, data):
        """
        공통적으로 사용 가능한 CSV 내보내기 메서드
        :param filename: 내보낼 파일 이름 (예: 'identity.csv', 'skill.csv')
        :param fieldnames: CSV의 헤더 리스트
        :param data: CSV로 내보낼 데이터 (리스트 형태)
        """
        try:
            # 현재작업 디렉토 기준으 'csv'폴더 없으면 생성
            if not os.path.exists('./csv'):
                os.makedirs('./csv')
            # 파일 경로 설정 (상대경로 'csv' 폴더 안에 저장)
            file_path = os.path.join('./csv', filename)
            with open(file_path, mode='w', newline='', encoding='utf-8') as file:
                writer = csv.DictWriter(file, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(data)
            print(f"Data successfully exported to {filename}")
        except Exception as e:
            print(f"Error exporting to CSV: {e}")

    def get_identity_data(self):
        """
        Identity 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "ID": identity.identity_id,
                "Name": identity.name,
                "Rarity": identity.rarity,
                "Skill_1_Type": identity.skill_1_type,
                "Skill_1_Sin_Affinity": identity.skill_1_sin_affinity,
                "Skill_2_Type": identity.skill_2_type,
                "Skill_2_Sin_Affinity": identity.skill_2_sin_affinity,
                "Skill_3_Type": identity.skill_3_type,
                "Skill_3_Sin_Affinity": identity.skill_3_sin_affinity,
                "Defense_Type": identity.defense_type,
                "Defense_Sin_Affinity": identity.defense_sin_affinity,
            }
            for identity in self.identities
        ]
        
    def export_identities_data(self, filename="identities.csv"):
        """Identity 데이터를 CSV로 내보내는 메서드"""
        identity_fieldnames = [
            "ID", "Name", "Rarity",
            "Skill_1_Type", "Skill_1_Sin_Affinity",
            "Skill_2_Type", "Skill_2_Sin_Affinity",
            "Skill_3_Type", "Skill_3_Sin_Affinity",
            "Defense_Type", "Defense_Sin_Affinity"
        ]
        identity_data = self.get_identity_data()  # Identity 데이터 가져오기
        self.export_to_csv(filename, identity_fieldnames, identity_data)  # CSV로 내보내기

    def load_identities_by_id(self, filename, identity_ids):
        """지정된 identity_ids 목록에 해당하는 Identity 데이터를 CSV에서 읽어옴."""
        identity_ids_set = set(identity_ids) # 중복 제거를 위 Set 사용
        with open(filename, mode='r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                if row["ID"] in identity_ids_set:
                    identity = Identity(
                        identity_id=row["ID"],
                        name=row["Name"],
                        rarity=row["Rarity"],
                        skill_1_type=row["Skill_1_Type"],
                        skill_1_sin_affinity=row["Skill_1_Sin_Affinity"],
                        skill_2_type=row["Skill_2_Type"],
                        skill_2_sin_affinity=row["Skill_2_Sin_Affinity"],
                        skill_3_type=row["Skill_3_Type"],
                        skill_3_sin_affinity=row["Skill_3_Sin_Affinity"],
                        defense_type=row["Defense_Type"],
                        defense_sin_affinity=row["Defense_Sin_Affinity"]
                    )
                    self.identities.append(identity)

    def get_level_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Level": level_stat.level,
                "HP": level_stat.hp,
                "Defense": level_stat.defense,
            }
            for identity in self.identities
            for level_stat in identity.level_stats
        ]

    def export_level_stats_data(self, filename="levelstats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        level_stats_fieldnames = ["Identity_ID", "Level", "HP", "Defense"]
        level_stats_data = self.get_level_stats_data()  # level_stats 데이터 가져오기
        self.export_to_csv(filename, level_stats_fieldnames, level_stats_data)  # CSV로 내보내기

    def get_uptie_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Uptie": uptie_stat.uptie,
                "Speed": uptie_stat.speed,
            }
            for identity in self.identities
            for uptie_stat in identity.uptie_stats
        ]

    def export_uptie_stats_data(self, filename="Uptiestats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        uptie_stats_fieldnames = ["Identity_ID", "Uptie", "Speed"]
        uptie_stats_data = self.get_uptie_stats_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, uptie_stats_fieldnames, uptie_stats_data)  # CSV로 내보내기

    def get_resist_info_data(self):
        """
        모든 Identity의 resist_info 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Resist_Hit": identity.resist_info.resist_hit,
                "Resist_Penetrate": identity.resist_info.resist_penetrate,
                "Resist_Slash": identity.resist_info.resist_slash,
            }
            for identity in self.identities
        ]

    def export_resist_info_data(self, filename="ResistInfos.csv"):
        """ResistInfo 데이터를 CSV로 내보내는 메서드"""
        resist_infos_fieldnames = ["Identity_ID", "Resist_Hit", "Resist_Penetrate", "Resist_Slash"]
        resist_infos_data = self.get_resist_info_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, resist_infos_fieldnames, resist_infos_data)  # CSV로 내보내기

    def export_uptie_stats_data(self, filename="Uptiestats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        uptie_stats_fieldnames = ["Identity_ID", "Uptie", "Speed"]
        uptie_stats_data = self.get_uptie_stats_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, uptie_stats_fieldnames, uptie_stats_data)  # CSV로 내보내기

    def get_stagger_threshold_data(self):
        """
        모든 Identity의 stagger_threshold 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "First_Stagger": identity.stagger_threshold.first_stagger,
                "Second_Stagger": identity.stagger_threshold.second_stagger,
                "Third_Stagger": identity.stagger_threshold.third_stagger,
            }
            for identity in self.identities
        ]

    def export_stagger_threshold_data(self, filename="StaggerThresholds.csv"):
        """StaggerThresholds 데이터를 CSV로 내보내는 메서드"""
        stagger_thresholds_fieldnames = ["Identity_ID", "First_Stagger", "Second_Stagger", "Third_Stagger"]
        stagger_thresholds_data = self.get_stagger_threshold_data()  # stagger_thresholds 데이터 가져오기
        self.export_to_csv(filename, stagger_thresholds_fieldnames, stagger_thresholds_data)  # CSV로 내보내기


async def main():
    data_container = DataContainer()
    scraper = Scraper(container=data_container, base_url="https://limbus.kusoge.xyz", img_url="https://img.kusoge.xyz/limbus/img")
    await scraper.scrape()

await main() 

Added skill: skill 1:Begone...
Updated Uptie for Begone... (Uptie Level: 1): Skill Power: 2, Coin Power: +4
{'keywords': ['Bloodfeast', 'Bleed', 'Rupture'], 'conditions': ['[Combat Start]', '[On Hit]', '[On Hit]'], 'effects': ['On', 'self', 'and', '1', 'ally', 'with', 'the', 'highest', 'max', 'HP', ':', 'consume', 'up', 'to', '20', 'keyword:Bloodfeast', 'keyword:Bloodfeast', 'and', 'apply', '1', 'blooming[thornyfall_panic', 'RodionFirst', ']', 'for', 'every', '10', 'keyword:Bloodfeast', 'keyword:Bloodfeast', 'this', 'unit', 'consumed', '(', '2', 'times', 'per', 'turn', ')', '-', 'Prioritizes', '<', 'Bloodfiend', '>', 'allies', '-', 'If', 'the', 'affected', 'ally', 'is', 'a', '<', 'Bloodfiend', '>,', 'this', 'unit', 'applies', '1', 'additional', 'blooming[thornyfall_panic', 'RodionFirst', ']', '-', 'If', 'this', 'unit', 'failed', 'to', 'consume', 'keyword:Bloodfeast', 'keyword:Bloodfeast', ',', 'gain', '5', 'keyword:Bleed', 'keyword:Bleed', 'Inflict', '1', 'keyword:Bleed', 'keyword:Blee

CancelledError: 

In [70]:
import os
import csv
import re
import numpy as np
import pandas as pd
from playwright.async_api import async_playwright
from tabulate import tabulate

# Sin Color 이름 변환을 위한 매핑
resource_mapping = {
    "AZURE": "Gloom",
    "VIOLET": "Envy",
    "AMBER": "Sloth",
    "SHAMROCK": "Gluttony",
    "CRIMSON": "Wrath",
    "SCARLET": "Lust",
    "INDIGO": "Pride"
}

def convert_resource(resource_name):
    """Sin Color 이름을 변환하는 함수. 매핑되지 않은 경우 원래 이름을 반환."""
    return resource_mapping.get(resource_name, resource_name)

class Scraper:
    """웹 페이지 크롤링을 위한 클래스"""
    def __init__(self, container, base_url, img_url):
        self.container = container
        self.base_url = base_url
        self.img_url = img_url
        self.uptie_level = 1

    async def handle_request(self, route, request):
        """이미지와 폰트 리소스 로드를 차단."""
        if request.resource_type in ["image", "font"]:
            await route.abort()
        else:
            await route.continue_()
            
    async def scrape(self):
        """크롤링 메인 함수"""
        async with async_playwright() as p:
            browser = await p.chromium.launch(headless=True)
            page = await browser.new_page()
            await page.route("**/*", self.handle_request)
            # Step 1: /identity 페이지에서 기본 정보 크롤링
            #await self.crawl_identity_page(page)

            # Step 2: /identity/{identity_id} 상세 페이지 크롤링
            #identity_ids_to_load = ["10104", "10310", "10710","10808", "10911", "11005"]
            #identity_ids_to_load = ["10104", "10911"]#,"10310", "10710","10808", "11005", "11210"]
            identity_ids_to_load = ["10911"]#,"10310", "10710","10808", "11005", "11210"]
            self.container.load_identities_by_id('./csv/identities.csv', identity_ids_to_load)
            await self.crawl_identity_details_for_all(page)
            #self.container.display_identities()
            #self.container.export_identities_data("identities.csv")
            #self.container.export_level_stats_data("Levelstats.csv")
            #self.container.export_uptie_stats_data("Uptiestats.csv")
            #self.container.export_resist_info_data("ResistInfos.csv")
            #self.container.export_stagger_threshold_data("StaggerThresholds.csv")
            #page.set_default_timeout(10000)
            await page.wait_for_timeout(100000)
            await browser.close()

    async def get_identity_id_from_row(self, row):
        """테이블 행(row)에서 인격 ID를 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_url = await identity_cell.get_attribute('href')
            identity_id = identity_url.split('/identity/')[-1]
            return identity_id
        return "Unknown"

    async def get_identity_name_from_row(self, row):
        """테이블 행(row)에서 인격 이름을 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_name = (await identity_cell.inner_text()).replace("\n", " ").strip()
            return identity_name
        return "Unknown"
        
    async def crawl_identity_page(self, page):
        """/identity 페이지로 이동하여 인격 기본 정보를 추출."""
        await page.goto(self.base_url + "/identity")
        await self.extract_identity_page(page)

    async def extract_identity_page(self, page):
        """/identity 페이지의 모든 테이블 행에서 인격 정보를 추출."""
        identity_rows = await page.query_selector_all('tr')
        if len(identity_rows) > 1:
            for idx, identity_row in enumerate(identity_rows):
                if idx == 0:
                    continue # 0번째요소이름을 보여주는 열은 건너뜀
                await self.scrape_identity_name_and_identity_id(identity_row)
                await self.extract_rarity(identity_row)
                for i in range(1, 4):  # 스킬1~3 정보 추출
                    await self.extract_atk_and_resource(identity_row, i)
                await self.extract_defense_and_resource(identity_row)

    async def scrape_identity_name_and_identity_id(self, row):
        """인격 번호와 이름을 추출하여 DataContainer에 저장."""
        try:
            identity_id = await self.get_identity_id_from_row(row)
            identity_name = await self.get_identity_name_from_row(row)
            identity = Identity(identity_id)
            self.container.add_identity(identity)
            self.container.update_identity_name(identity_id, identity_name)
        except Exception as e:
            print(f"Error extracting identity name and identity id: {e}")

    async def get_element_attribute(self, element, selector, attribute):
        """요소에서 지정된 속성(attribute)을 추출."""
        try:
            target_element = await element.query_selector(selector)
            if target_element:
                return await target_element.get_attribute(attribute)
        except Exception as e:
            print(f"Error getting element attribute: {e}")
        return None
    
    async def extract_rarity(self, row):
        """테이블 행(row)에서 희귀도를 추출하여 업데이트."""
        try:
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
                sibling = await parent.evaluate_handle('node => node.nextElementSibling')
                if sibling:
                    rarity_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/rarity_"]', 'src')
                    if rarity_img_src:
                        rarity_level = rarity_img_src.split('_')[-1].split('.')[0]  # rarity_1.png -> 1
                        identity_id = await self.get_identity_id_from_row(row)
                        self.container.update_identity_rarity(identity_id, rarity_level)
                    else:
                        print("Rarity image not found in the next sibling TD.")
        except Exception as e: 
            print(f"Error extracting rarity: {e}")

        return "Unknown"

    async def extract_atk_and_resource(self, row, skill_number):
        """ 공격 타입(ATK Type)과 자원(Attribute) 추출 함수 - 스킬 번호에 따라 처리 """
        try:
            #print(f"Attempting to locate skill{skill_number} type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 해당 스킬 번호에 맞는 형제로 이동
                sibling = parent
                for _ in range(skill_number + 1):  # 스킬 번호에 맞게 이동
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                atk_type, resource = "Unknown", "Unknown"
        
                if sibling:
                    # 공격 타입 이미지 (atk_type) 찾기
                    atk_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/atk_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if atk_type_img_src:
                        atk_type = atk_type_img_src.split('_')[-1].split('.')[0]  # atk_type_SLASH.png -> SLASH
                        #print(f"Extracted attack type: {atk_type}")
                        self.container.update_skill(identity_id, skill_number, atk_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted resource: {resource}")
                        
                        # 리소스를 변환
                        resource = convert_resource(resource)
                        #print(f"Converted resource: {resource}")
                        self.container.update_sin_affinity(identity_id, skill_number, resource)
                        
        except Exception as e:
            print(f"Error extracting skill{skill_number} type and resource: {e}")

    async def extract_defense_and_resource(self, row):
        """ 방어 타입(Def Type)과 자원(Attribute) 추출 함수 """
        try:
            #print("Attempting to locate defense type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 다섯 번째 형제 td로 이동 (방어 관련 정보가 있는 <td>)
                sibling = parent
                for _ in range(5):  # 방어 정보는 5번째 형제에 있음
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                def_type, def_resource = "Unknown", "Unknown"
        
                if sibling:
                    # 방어 타입 이미지 (def_type) 찾기
                    def_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/def_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if def_type_img_src:
                        def_type = def_type_img_src.split('_')[-1].split('.')[0]  # def_type_GUARD.png -> GUARD
                        #print(f"Extracted defense type: {def_type}")
                        self.container.update_skill(identity_id, 'defense', def_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        def_resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted defense resource: {def_resource}")
                        
                        # 방어 리소스를 변환
                        def_resource = convert_resource(def_resource)
                        #print(f"Converted defense resource: {def_resource}")
                        self.container.update_sin_affinity(identity_id, 'defense', def_resource)
 
        except Exception as e:
            print(f"Error extracting defense type and resource: {e}")

    async def crawl_identity_details_for_all(self, page):
        """모든 인격의 세부 정보를 크롤링."""
        for identity in self.container.identities:
            await page.wait_for_load_state("networkidle")
            await page.goto(self.base_url + "/identity/"+identity.identity_id)
            """ 레벨과 동기화와 관련 없는 참관타 저항은 밑에서 변경하는 것과 연관성이 적기 때문에 위에서 처리한다.
            print(f"Identity_id: {identity.identity_id}")
            results = []
            resist_type_imgs = [f"{self.img_url}/ui/atk_type_HIT.png",
                                f"{self.img_url}/ui/atk_type_PENETRATE.png",
                                f"{self.img_url}/ui/atk_type_SLASH.png"]
            for resist_type_img in resist_type_imgs:
                result = await self.extract_resist_type(page, resist_type_img, 1)
                results.append(result)
            self.container.add_resist_info(identity.identity_id, results[0], results[1], results[2])
            """
            """
            # Stagger Threshold 값 추출
            stagger_threshold = await self.extract_stagger_threshold_values(page)
            self.container.add_stagger_threshold(identity.identity_id, stagger_threshold[0], stagger_threshold[1], stagger_threshold[2])
            """
            await self.initialize_uptie_and_level(page)
            """ 레벨에 따른 level, hp, defense 다르게 되어 있는 것을 출력 확인 객체로 들어가서 csv로 옮기는 과정까지 진행
            for i in range(1, identity.level_max+1):
                await self.move_slider_to_value(page,i)
                await self.extract_level_status(page, identity.identity_id, i)
            """
            """ 동기화에 따른 speed 다르게 되어 있는 것을 출력 확인 객체로 들어가서 csv로 옮기는 과정까지 진행
            for i in range(1, identity.uptie_max+1):
                await self.uptie_click(page,i)
                await self.extract_uptie_status(page, identity.identity_id, i)
            """
            await self.crawl_skills(page, identity.identity_id)
            
    async def initialize_uptie_and_level(self, page):
        await self.uptie_click(page, 1)
        await self.move_slider_to_value(page, 1)
        self.uptie_level = 1
        #return 1
    
    async def uptie_click(self, page, phase):
        fab_button_selector = ".q-btn.q-btn--fab"
        await page.wait_for_selector(fab_button_selector)
        await page.click(fab_button_selector)
        #print(f"[{setup_uptie_button}] 상위 동기화 버튼 클릭 완료")
        setup_uptie_button = f'a:has(img[src="{self.img_url}/ui/uptie_{phase}.png"])'
        await page.wait_for_selector(setup_uptie_button)
        await page.click(setup_uptie_button)
        #print(f"{phase} 동기화 버튼 클릭 완료")

    async def move_slider_to_value(self, page, target_value):
        """
        슬라이더를 주어진 값으로 이동시킵니다.
        target_value는 슬라이더의 최소값부터 최대값 사이여야 합니다.
        """
        slider_selector = ".q-slider"
        # 슬라이더의 범위 확인
        min_value = int(await page.get_attribute(slider_selector, "aria-valuemin"))
        max_value = int(await page.get_attribute(slider_selector, "aria-valuemax"))
    
        if target_value < min_value or target_value > max_value:
            raise ValueError(f"Target value {target_value} is out of bounds ({min_value}, {max_value})")
    
        # 슬라이더 thumb 요소와 위치 가져오기
        slider_thumb = await page.query_selector(f"{slider_selector} .q-slider__thumb")
        thumb_box = await slider_thumb.bounding_box()
        
        # 슬라이더의 현재 위치 및 목표 위치 계산
        track = await page.query_selector(f"{slider_selector} .q-slider__track")
        track_box = await track.bounding_box()
        
        # 목표 x 위치 계산 (슬라이더 트랙 내에서 비율 기반)
        target_percentage = (target_value - min_value) / (max_value - min_value)
        target_x = track_box["x"] + target_percentage * track_box["width"]
        current_x = thumb_box["x"] + thumb_box["width"] / 2
        y = thumb_box["y"] + thumb_box["height"] / 2
    
        # 슬라이더를 드래그하여 목표 위치로 이동
        await page.mouse.move(current_x, y)
        await page.mouse.down()
        await page.mouse.move(target_x, y, steps=10)  # 부드럽게 이동
        await page.mouse.up()
    
        #print(f"슬라이더를 {target_value}로 이동 완료.")
    
        # 슬라이더 값 확인 및 강제 이벤트 트리거
        await page.evaluate(
            f"""
            (slider) => {{
                slider.dispatchEvent(new Event('input'));  // oninput 이벤트 트리거
                slider.dispatchEvent(new Event('change'));  // onchange 이벤트 트리거
            }}
            """,
            await page.query_selector(slider_selector)
        )
    
        # 이동 후 값 확인
        current_value = await page.get_attribute(slider_selector, "aria-valuenow")
        if current_value != str(target_value):
            print(f"경고: 슬라이더 값이 예상과 다릅니다. 현재 값: {current_value}, 예상 값: {target_value}")

    async def get_second_status_element(self, page):
        """
        두 번째 'Status' 요소에 접근하여 반환하는 메소드
        """
        status_elements = await page.query_selector_all("text=Status")
        if len(status_elements) > 1:
            second_status_element = status_elements[1]
            #print("두 번째 'Status' 요소에 접근합니다.")
            return second_status_element
        else:
            #print("두 번째 'Status' 요소가 존재하지 않습니다.")
            return None

    async def extract_level_status(self, page, identity_id, level):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 레벨에 따라 HP와 Defense가 변한다.
        hp_value = await self.extract_hp(second_status_element)
        defense_value = await self.extract_defense(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Level={level}, HP={hp_value}, Defense={defense_value}")
        self.container.add_level_stat(identity_id,level, hp_value, defense_value)
    
    async def extract_hp(self, status_element):
        # HP 추출
        hp_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].children[0].nextElementSibling.textContent.trim()"
        )
        return hp_value
    
    async def extract_defense(self, status_element):
        # Defense 추출
        defense_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return defense_value

    async def extract_uptie_status(self, page, identity_id, uptie):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 uptie에 따라 speed는 변한다.
        speed_value = await self.extract_speed(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Uptie={uptie}, Speed={speed_value}")
        self.container.add_uptie_stat(identity_id,uptie, speed_value)

    async def extract_speed(self, status_element):
        # Speed 추출
        speed_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return speed_value

    async def extract_resist_type(self, page, image_src, depth):
        # 이미지 요소 찾기
        #print(image_src)
        image = await page.query_selector(f'img[src="{image_src}"]')
        #print(image)
        if not image:
            return "Unknown"
    
        # 이미지의 부모 요소를 찾아서 올라감
        parent = image
        for _ in range(depth):
            parent = await parent.evaluate_handle('node => node.parentElement')
            if not parent:
                return "Unknown"
    
        # 부모의 형제 요소 찾기
        sibling = await parent.evaluate_handle('node => node.nextElementSibling')
        
        if sibling:
            value = await sibling.evaluate('node => node.textContent')
            if value:
                return value.strip()
        
        return "Unknown"

    async def extract_stagger_threshold_values(self, page):
        # 'div.row.center.flex-center.q-pt-xs' 요소들 찾기
        elements = await page.query_selector_all('div.row.center.flex-center.q-pt-xs')
        
        # 각 요소의 텍스트 내용 출력하여 'Stagger Threshold' 요소를 찾기
        for element in elements:
            text = await element.text_content()
            
            if text and "Stagger Threshold" in text:
                # 'Stagger Threshold'가 있는 요소를 찾았다면, 그 이후 형제 요소들에서 값을 추출
                parent = await element.evaluate_handle('node => node.parentElement')
                sibling_elements = await parent.query_selector_all('div.row.justify-evenly div')  # Threshold 값이 들어있는 div들
                
                stagger_values = []
                for sibling in sibling_elements:
                    text_content = await sibling.text_content()
                    match = re.search(r'(\d+%)', text_content)
                    if match:
                        stagger_values.append(match.group(1))  # 퍼센트 값만 리스트에 추가
                while len(stagger_values) < 3:
                    stagger_values.append(np.nan)
                #print(stagger_values)
                return stagger_values  # 추출된 값(또는 np.nan 포함)을 반환
        
        return [np.nan, np.nan, np.nan]  # 만약 'Stagger Threshold'를 찾지 못한 경우 기본값 반환

    async def crawl_skills(self, page, identity_id):
        await self.crawl_skill(page, identity_id, "skill 1")
        await self.crawl_skill(page, identity_id, "skill 2")
        await self.crawl_skill(page, identity_id, "skill 3")
        await self.crawl_skill(page, identity_id, "defense")


    async def crawl_skill(self, page, identity_id, skill_type):
        skills_elements = await page.query_selector_all('div.skilltag')
        for skill_element in skills_elements:
            skill_text = await skill_element.text_content()
            if skill_text.strip().lower() == skill_type:
                await self.process_skill(skill_element, page, identity_id, skill_type)
        
    async def process_skill(self, skill_element, page, identity_id, skill_type):
        # skill_type이 "skill 3"일 경우 uptie_value가 3 이상일 때만 정보를 저장
        for uptie_value in range(1, 5):  # Uptie levels 1 to 4
            # "skill 3"은 uptie_value가 3 이상일 때만 처리
            if skill_type == "skill 3" and uptie_value < 3:
                continue  # skip if uptie_value is less than 3 for skill 3
    
            # uptie_level에 맞게 클릭
            await self.uptie_click(page, uptie_value)
            if skill_type != "skill 3" and uptie_value > 1:
                # Uptie 정보 갱신
                await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
                continue
            elif skill_type == "skill 3" and uptie_value > 3:
                # Uptie 정보 갱신
                await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
                continue
            # 스킬의 세부 정보를 추출
            skill_details = await self.extract_skill_details(skill_element, skill_type)
            skill_name = skill_details["skill_name"]
    
            # 스킬 정보를 저장
            self.container.add_skill(
                identity_id, skill_type, 
                skill_name, skill_details["coin_count"],
                skill_details["init_level"], skill_details["init_remain"], 
                skill_details["atk_weight"]
            )
            print(f"Added skill: {skill_type}:{skill_name}")
    
            # Uptie 정보 갱신
            await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
    
    async def extract_skill_details(self, skill_element, skill_type):
        # 스킬 세부 정보를 추출하는 함수
        skill_name = await self.skill_name(skill_element)
        coin_count = await self.coin_count(skill_element, ".col-auto > .column > .row")
        init_level = await self.init_level(skill_element)
        init_remain = await self.init_remain(skill_element, skill_type, "div.row.justify-start > div.attr.flex.flex-center > div.q-px-sm")
        atk_weight = await self.atk_weight(skill_element, skill_type)
        return {
            "skill_name": skill_name,
            "coin_count": coin_count,
            "init_level": init_level,
            "init_remain": init_remain,
            "atk_weight": atk_weight
        }
    
    async def update_uptie_info(self, page, skill_element, identity_id, skill_name, skill_type, uptie_value):
        # Uptie 정보를 추출하고 업데이트하는 함수
        uptie_info = await self.extract_uptie_info(skill_element)
        self.container.add_skill_uptie_info(
            identity_id, skill_name, 
            uptie_info["skill_power"], uptie_info["coin_power"], 
            uptie_value
        )
        print(
            f"Updated Uptie for {skill_name} (Uptie Level: {uptie_value}): "
            f"Skill Power: {uptie_info['skill_power']}, Coin Power: {uptie_info['coin_power']}"
        )
        effect = await self.skill_effects(skill_element)
        print(effect)

    async def extract_uptie_info(self, skill_element):
        skill_data = await self.skill_and_coin_power(skill_element)
        return {
            "skill_power": skill_data.get("skillPower"),
            "coin_power": skill_data.get("coinPower")
        }

    async def extract_skill_effect(self, skill_element):
        # Create the Effect object
        effect_obj = Effect(condition=None)
        # Extract effects
        effects = await self.skill_effects(skill_element)
        if effects:
            effect_obj.effects.append(effects)
        return effect_obj

    async def skill_and_coin_power(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                // 조부모로 이동
                const grandparent = node.parentElement.parentElement;
                // 이전 형제 탐색
                const previousSibling = grandparent.previousElementSibling;
                if (!previousSibling) return {{ skillPower: null, coinPower: null }}; // 이전 형제가 없으면 기본값 반환
                // 이전 형제 내부의 .row.justify-around > div > div 탐색
                const targetDivs = previousSibling.querySelectorAll(".row.justify-around > div");
                if (targetDivs.length < 3) return {{ skillPower: null, coinPower: null }}; // 배열 길이 확인
                // 1st div: 스킬 위력
                const skillPower = targetDivs[0]?.querySelector('div')?.textContent?.trim() || null; 
                // 3rd div: 코인 파워 값 (3번째 div에서 텍스트 추출)
                const coinPower = targetDivs[2]?.textContent?.trim() || null;
                return {{ skillPower, coinPower }};
            }}
        """)


    async def coin_count(self, skill_element, hierarchy_selector):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector('{hierarchy_selector}');
                if (!targetElement) return 0;
                return targetElement.childElementCount;
            }}
        """)

    async def init_remain(self, skill_element, skill_type, hierarchy_selector):
        if skill_type == "defense":
            return -1
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector('{hierarchy_selector}');
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def skill_name(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector(".block");
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def init_level(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector("div.text-bold.text-h6.h6.q-pl-xs.q-pr-md");
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def atk_weight(self, skill_element, skill_type):
        if skill_type == "defense":
            return 0
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector("div.text-bold.text-h6.h6.q-pl-xs.q-pr-md");
                const doubleNextSibling = targetElement.nextElementSibling.nextElementSibling;
                if (!doubleNextSibling) return 0;
                return doubleNextSibling.textContent?.trim();
            }}
        """)

    async def skill_effects(self, skill_element):
        return await skill_element.evaluate("""
            node => {
                const parent = node.parentElement;
                const doubleNextSibling = parent.nextElementSibling?.nextElementSibling;
                if (!doubleNextSibling) return null;
    
                // doubleNextSibling에서 첫 번째 div.col 요소 찾기
                const textNode = doubleNextSibling.querySelector("div.col");
                if (!textNode) return null;
    
                const results = {
                    keywords: [], // 키워드 저장
                    conditions: [], // 조건 저장
                    effectsByCondition: {} // 조건별 효과
                };
    
                // 1. 조건 추출 (대괄호 텍스트)
                const conditionElements = textNode.querySelectorAll("span");
                conditionElements.forEach(span => {
                    let text = span.textContent?.trim();
                    if (text?.startsWith("[") && text?.endsWith("]")) {
                        // 부모의 전형제 노드 탐색
                        const sibling = span.parentElement.previousElementSibling;
                        if (sibling) {
                            const emblemImage = sibling.querySelector("img.emblem");
                            console.log("Sibling element found:", sibling); // 디버깅용
                            if (emblemImage) {
                                const src = emblemImage.getAttribute("src");
                                console.log("Image source:", src); // 디버깅용
                                const match = src.match(/ui_num_(\\d+)\\.png$/); // 정규식 수정
                                if (match) {
                                    const number = match[1]; // 숫자 추출
                                    text = `[Coin ${number} ${text.replace("[", "").replace("]", "")}]`;
                                    console.log("Updated condition:", text); // 디버깅용
                                }
                            }
                        }
                        results.conditions.push(text);
                        results.effectsByCondition[text] = []; // 조건별 효과를 초기화
                    }
                });
    
                // 2. 키워드 추출
                const keywordElements = textNode.querySelectorAll('a[href^="/en/keyword/"]');
                keywordElements.forEach(a => {
                    const span = a.querySelector('span.q-btn__content.text-center.col.items-center.q-anchor--skip.justify-center.row>span');
                    if (span) {
                        const keywordText = span.textContent.trim();
                        if (keywordText && !results.keywords.includes(keywordText)) {
                            results.keywords.push(keywordText); // 키워드 저장
                        }
                    }
                });
    
                // 3. 효과 추출
                const effectElements = textNode.querySelectorAll("span");
                let currentCondition = null;
                let currentEffect = []; // 초기화
                effectElements.forEach(span => {
                    let text = span.textContent?.trim();
                    if (!text) return;
    
                    // 조건 발견 시 currentCondition 변경
                    if (text.startsWith("[") && text.endsWith("]")) {
                        // 이전 조건의 효과 저장
                        if (currentCondition && currentEffect.length > 0) {
                            // 부모의 전형제 노드 탐색
                            const sibling = span.parentElement.previousElementSibling;
                            if (sibling) {
                                const emblemImage = sibling.querySelector("img.emblem");
                                if (emblemImage) {
                                    const src = emblemImage.getAttribute("src");
                                    const match = src.match(/ui_num_(\\d+)\\.png$/); // 정규식 수정
                                    if (match) {
                                        const number = match[1]; // 숫자 추출
                                        text = `[Coin ${number} ${text.replace("[", "").replace("]", "")}]`;
                                    }
                                }
                            }
                            results.effectsByCondition[currentCondition].push(currentEffect.join(' '));
                            currentEffect = []; // 초기화
                        }
                        currentCondition = text;
                        return;
                    }
    
                    // 현재 조건의 효과 처리
                    if (currentCondition) {
                        if (results.keywords.includes(text)) {
                            const keywordEffect = 'keyword:' + text;
                            if (!currentEffect.includes(keywordEffect)) {
                                currentEffect.push(keywordEffect);
                            }
                        } else {
                            currentEffect.push(text);
                        }
                    }
                });
    
                // 마지막 효과 저장
                if (currentCondition && currentEffect.length > 0) {
                    results.effectsByCondition[currentCondition].push(currentEffect.join(' '));
                }
    
                return results;
            }
        """)



class Identity:
    """인격 정보를 저장하는 클래스"""
    def __init__(self, identity_id, name=None, rarity=None,\
                 skill_1_type=None, skill_1_sin_affinity=None,\
                 skill_2_type=None, skill_2_sin_affinity=None,\
                 skill_3_type=None, skill_3_sin_affinity=None,\
                 defense_type=None, defense_sin_affinity=None):
        self.identity_id = identity_id
        self.name = name
        self.rarity = rarity
        # 선언만 하는 부분 
        self.skill_1_type = skill_1_type 
        self.skill_1_sin_affinity = skill_1_sin_affinity 
        self.skill_2_type = skill_2_type 
        self.skill_2_sin_affinity = skill_2_sin_affinity 
        self.skill_3_type = skill_3_type 
        self.skill_3_sin_affinity = skill_3_sin_affinity 
        self.defense_type = defense_type 
        self.defense_sin_affinity = defense_sin_affinity
        self.level_max = 50
        self.uptie_max = 4
        self.level_stats = []  # LevelStat 객체 리스트
        self.uptie_stats = [] # UptieStat 객체 리스트
        self.resist_info = None
        self.stagger_threshold = None
        self.skillset = SkillSet()  # Skill정보를 담고 있는 skillset 객체 
        self.passive_infos = []  # 패시브 객체를 담고 있는 배열
        self.support_passive_infos = [] # 서포트 패시 객체를 담고 있는 배열
        self.panic_type = [] # 혼란상태 발생시 나타나는 효과를 담고 있는 객체 배열

    def update_name(self, name):
        self.name = name

    def update_rarity(self, rarity):
        self.rarity = rarity

    def add_level_stat(self, level, hp, defense):
        self.level_stats.append(LevelStat(level, hp, defense))

    def add_uptie_stat(self, uptie, speed):
        self.uptie_stats.append(UptieStat(uptie, speed))

    def add_resist_info(self, resist_hit, resist_penetrate, resist_slash):
        self.resist_info = ResistInfo(resist_hit, resist_penetrate, resist_slash)

    def add_stagger_threshold(self, first_stagger, second_stagger, third_stagger):
        self.stagger_threshold = StaggerThreshold(first_stagger, second_stagger, third_stagger)

    def add_skill(self, skill_code, skill_name, coin_count, init_level, init_remain, akt_weight):
        self.skillset.add_skill(skill_code, Skill(skill_code, skill_name, coin_count, init_level, init_remain, akt_weight))
        
    def add_skill_uptie_info(self, skill_name, uptie_level, skill_power, coin_power):
        self.skillset.add_skill_uptie_info(skill_name, uptie_level, skill_power, coin_power)
        
    def update_details(self, page):
        # 세부 정보 업데이트 로직 (가정)
        pass
    #def init_uptie(self, page):
    #    init_uptie = 

# LevelStat 객체 정의
class LevelStat:
    def __init__(self, level, hp=None, defense=None):
        self.level = level
        self.hp = hp
        self.defense = defense
    def update_level(self, level):
        self.level = level
    def update_hp(self, hp):
        self.hp = hp
    def update_defense(self, defense):
        self.defense = defense

# UptieStat 객체 정의
class UptieStat:
    def __init__(self, uptie, speed=None):
        self.uptie = uptie
        self.speed = speed
    def update_speed(self, speed):
        self.speed = speed

# ResistInfo 객체 정의
class ResistInfo:
    def __init__(self, resist_hit=None, resist_penetrate=None, resist_slash=None):
        self.resist_hit = resist_hit
        self.resist_penetrate = resist_penetrate
        self.resist_slash = resist_slash

# ResistInfo 객체 정의
class StaggerThreshold:
    def __init__(self, first_stagger=None, second_stagger=None, third_stagger=None):
        self.first_stagger = first_stagger
        self.second_stagger = second_stagger
        self.third_stagger = third_stagger

class SkillSet:
    def __init__(self):

        self.skill_1_type = ""
        self.skill_2_type = ""
        self.skill_3_type = ""
        self.defense_skill_type = ""

        self.skill_1_sin_affinity = ""
        self.skill_2_sin_affinity = ""
        self.skill_3_sin_affinity = ""
        self.defense_skill_sin_affinity = ""

        self.skill_1_details = []
        self.skill_2_details = []
        self.skill_3_details = []
        self.defense_skill_details = []

    def set_skill_type(self, skill_code, skill_type, sin_affinity):
        if skill_code == "skill 1":
            self.skill_1_type = skill_type
            self.skill_1_sin_affinity = sin_affinity
        elif skill_code == "skill 2":
            self.skill_2_type = skill_type
            self.skill_2_sin_affinity = sin_affinity
        elif skill_code == "skill 3":
            self.skill_3_type = skill_type
            self.skill_3_sin_affinity = sin_affinity
        elif skill_type == "defense":
            self.defense_skill_type = skill_type
            self.defense_skill_sin_affinity = sin_affinity
    
    def add_skill(self, skill_code, skill):
        if skill_code == "skill 1":
            self.skill_1_details.append(skill)
        elif skill_code == "skill 2":
            self.skill_2_details.append(skill)
        elif skill_code == "skill 3":
            self.skill_3_details.append(skill)
        elif skill_code == "defense":
            self.defense_skill_details.append(skill)

    def get_skill_by_name(self, skill_name):
        for skill_list in [self.skill_1_details, self.skill_2_details, self.skill_3_details, self.defense_skill_details]:
            for skill in skill_list:
                result = skill.is_skill_by_name(skill_name)
                if result:
                    return result
        return None

    def add_skill_uptie_info(self, skill_name, uptie_level, skill_power, coin_power):
        skill = self.get_skill_by_name(skill_name)
        skill.add_uptie_info(uptie_level, skill_power, coin_power)
        

class Skill:
    def __init__(self, skill_code, skill_name, coin_count, init_level, init_remain, akt_weight):
        self.skill_code = skill_code # "Skill 1"/"Skill 2"/"Skill 3"/"Defense" '''o'''
        self.skill_name = skill_name # 스킬 이름 '''o'''
        self.coin_count = coin_count # 동전 개수 '''o'''
        self.init_level = init_level # 스킬이 초기 배정받은 레벨, 인격 레벨에 - 1 한 값을 더해주면 스킬의 현 level을 구할 수 있다. '''o'''
        """
        남아있는 스킬 개수 사용하면, 하나씩 줄어들고 모든 스킬을 사용하면 이 값으로 돌아간다.
        스킬의 남은 숫자없는 스킬은 강화된 스킬로 한번 사용되면 다 조건을 채울 때까 사용 못한다.
        강화된 스킬을 None으로 설정하는 방식과 조건이 채워지면 1로 늘리는 방식이 존재한다.
        스킬을 사용할 수 있는 초기값이다.
        조건을 충족한 강화한 스킬을 이례적인 절차라고 생각하고 None으로 초기값을 설정했다.
        """
        self.init_remain = None #'''o'''
        self.atk_weight = None #공격 가중치 값이다. 방어스킬에는 없기 때문에 None으로 설정했다. '''o'''
        self.remain_count = init_remain # 실제 스킬이 작동하며 횟수가 차감되 나중에 모두 차감되면 다음 턴에 다시 초기횟수 리필된다.
        """
        동기화 단계에 따른 정보를 담는다. 현재 4동기화까지 되어 있지만 차후 확장될 것을 생각하여 고정하지 않는다.
        내부에 들어있는 정보로는 uptie_level, skill_power, coin_power 속성으로 들어있다.
        전투, 턴, 스킬, 코인토스 단계로 내려오며 발생하는 효과에 관하여 저장한다.
        """
        self.uptie_info = []  

    def is_skill_by_name(self, skill_name):
        if self.skill_name == skill_name:
            return self
        return None
        
    def add_uptie_info(self, uptie_level, skill_power, coin_power):
        self.uptie_info.append(UptieInfo(uptie_level, skill_power, coin_power))

class UptieInfo:
    def __init__(self, uptie_level, skill_power, coin_power):
        self.uptie_level = uptie_level   #'''o'''
        self.skill_power = skill_power   #'''o'''
        self.coin_power = coin_power     #'''o'''
        self.skill_effects = []

class Effect:
    def __init__(self, condition):
        self.condition = condition # 조건 텍스트 "[coin 1 on hit]"
        self.effects = [] # 효과 설명 텍스 "Inflict +1 Rupture Count

# 추상화된 EffectType 클래스
class EffectType:
    def __init__(self, name, description):
        self.name = name  # Effect 이름
        self.description = description  # Effect 설명

# Passive 클래스
class Passive(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)  # condition이 있는지 여부 확인

# SupportPassive 클래스
class SupportPassive(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)

# PanicType 클래스
class PanicType(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)

# Keyword 클래스
class Keyword:
    def __init__(self, keyword, explain=None, dispellable=None):
        self.keyword = keyword
        self.explain = explain
        self.dispellable = dispellable

      
class DataContainer:
    """크롤링 데이터를 관리하는 클래스"""
    def __init__(self):
        self.identities = []

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.level_max

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.uptie_max

    def update_identity_name(self, identity_id, identity_name):
        """특정 ID의 이름을 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_name(identity_name)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_identity_rarity(self, identity_id, rarity):
        """특정 ID의 희귀도를 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_rarity(rarity)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_skill(self, identity_id, skill_number, new_type):
        """
        특정 identity의 skill_type을 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_type: 새 skill_type
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_type = new_type
                elif skill_number == 2:
                    identity.skill_2_type = new_type
                elif skill_number == 3:
                    identity.skill_3_type = new_type
                elif skill_number == "defense":
                    identity.defense_type = new_type
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def update_sin_affinity(self, identity_id, skill_number, new_affinity):
        """
        특정 identity의 sin_affinity를 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_affinity: 새 sin_affinity
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_sin_affinity = new_affinity
                elif skill_number == 2:
                    identity.skill_2_sin_affinity = new_affinity
                elif skill_number == 3:
                    identity.skill_3_sin_affinity = new_affinity
                elif skill_number == "defense":
                    identity.defense_sin_affinity = new_affinity
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def add_identity(self, identity):
        """새로운 인격 데이터를 추가."""
        self.identities.append(identity)

    def add_level_stat(self, identity_id, level, hp, defense):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_level_stat(level, hp, defense)

    def add_uptie_stat(self, identity_id, uptie, speed):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_uptie_stat(uptie, speed)

    def add_resist_info(self, identity_id, resist_hit, resist_penetrate, resist_slash):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_resist_info(resist_hit, resist_penetrate, resist_slash)

    def add_stagger_threshold(self, identity_id, first_stagger, second_stagger, third_stagger):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_stagger_threshold(first_stagger, second_stagger, third_stagger)

    
    def add_skill(self, identity_id, skill_code, skill_name, coin_count, init_level, init_remain, akt_weight):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_code == "skill 1":
                    identity.skillset.set_skill_type(skill_code, identity.skill_1_type, identity.skill_1_sin_affinity)
                elif skill_code == "skill 2":
                    identity.skillset.set_skill_type(skill_code, identity.skill_2_type, identity.skill_2_sin_affinity)
                elif skill_code == "skill 3":
                    identity.skillset.set_skill_type(skill_code, identity.skill_3_type, identity.skill_3_sin_affinity)
                elif skill_code == "defense":
                    identity.skillset.set_skill_type(skill_code, identity.defense_type, identity.defense_sin_affinity)
                identity.add_skill(skill_code, skill_name, coin_count, init_level, init_remain, akt_weight)

    def add_skill_uptie_info(self, identity_id, skill_name, uptie_level, skill_power, coin_power):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_skill_uptie_info(skill_name, uptie_level, skill_power, coin_power)
        
    def display_identities(self):
        """Identity, ID, Name, Rarity 정보를 출력"""
        if not self.identities:
            print("No identities found.")
        else:
            # 데이터를 리스트로 변환
            table = []
            for idx, identity in enumerate(self.identities, 1):
                table.append([
                    idx, identity.identity_id, identity.name, identity.rarity,
                    identity.skill_1_type, identity.skill_1_sin_affinity,
                    identity.skill_2_type, identity.skill_2_sin_affinity,
                    identity.skill_3_type, identity.skill_3_sin_affinity,
                    identity.defense_type, identity.defense_sin_affinity
                ])
            
            # 테이블 헤더 정의
            headers = [
                "Index", "ID", "Name", "Rarity",
                "Skill_1 Type", "Skill_1 Sin", "Skill_2 Type", "Skill_2 Sin",
                "Skill_3 Type", "Skill_3 Sin", "Defense Type", "Defense Sin"
            ]
            
            # 테이블 출력
            print(tabulate(table, headers=headers, tablefmt="fancy_grid"))

    def export_to_csv(self, filename, fieldnames, data):
        """
        공통적으로 사용 가능한 CSV 내보내기 메서드
        :param filename: 내보낼 파일 이름 (예: 'identity.csv', 'skill.csv')
        :param fieldnames: CSV의 헤더 리스트
        :param data: CSV로 내보낼 데이터 (리스트 형태)
        """
        try:
            # 현재작업 디렉토 기준으 'csv'폴더 없으면 생성
            if not os.path.exists('./csv'):
                os.makedirs('./csv')
            # 파일 경로 설정 (상대경로 'csv' 폴더 안에 저장)
            file_path = os.path.join('./csv', filename)
            with open(file_path, mode='w', newline='', encoding='utf-8') as file:
                writer = csv.DictWriter(file, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(data)
            print(f"Data successfully exported to {filename}")
        except Exception as e:
            print(f"Error exporting to CSV: {e}")

    def get_identity_data(self):
        """
        Identity 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "ID": identity.identity_id,
                "Name": identity.name,
                "Rarity": identity.rarity,
                "Skill_1_Type": identity.skill_1_type,
                "Skill_1_Sin_Affinity": identity.skill_1_sin_affinity,
                "Skill_2_Type": identity.skill_2_type,
                "Skill_2_Sin_Affinity": identity.skill_2_sin_affinity,
                "Skill_3_Type": identity.skill_3_type,
                "Skill_3_Sin_Affinity": identity.skill_3_sin_affinity,
                "Defense_Type": identity.defense_type,
                "Defense_Sin_Affinity": identity.defense_sin_affinity,
            }
            for identity in self.identities
        ]
        
    def export_identities_data(self, filename="identities.csv"):
        """Identity 데이터를 CSV로 내보내는 메서드"""
        identity_fieldnames = [
            "ID", "Name", "Rarity",
            "Skill_1_Type", "Skill_1_Sin_Affinity",
            "Skill_2_Type", "Skill_2_Sin_Affinity",
            "Skill_3_Type", "Skill_3_Sin_Affinity",
            "Defense_Type", "Defense_Sin_Affinity"
        ]
        identity_data = self.get_identity_data()  # Identity 데이터 가져오기
        self.export_to_csv(filename, identity_fieldnames, identity_data)  # CSV로 내보내기

    def load_identities_by_id(self, filename, identity_ids):
        """지정된 identity_ids 목록에 해당하는 Identity 데이터를 CSV에서 읽어옴."""
        identity_ids_set = set(identity_ids) # 중복 제거를 위 Set 사용
        with open(filename, mode='r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                if row["ID"] in identity_ids_set:
                    identity = Identity(
                        identity_id=row["ID"],
                        name=row["Name"],
                        rarity=row["Rarity"],
                        skill_1_type=row["Skill_1_Type"],
                        skill_1_sin_affinity=row["Skill_1_Sin_Affinity"],
                        skill_2_type=row["Skill_2_Type"],
                        skill_2_sin_affinity=row["Skill_2_Sin_Affinity"],
                        skill_3_type=row["Skill_3_Type"],
                        skill_3_sin_affinity=row["Skill_3_Sin_Affinity"],
                        defense_type=row["Defense_Type"],
                        defense_sin_affinity=row["Defense_Sin_Affinity"]
                    )
                    self.identities.append(identity)

    def get_level_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Level": level_stat.level,
                "HP": level_stat.hp,
                "Defense": level_stat.defense,
            }
            for identity in self.identities
            for level_stat in identity.level_stats
        ]

    def export_level_stats_data(self, filename="levelstats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        level_stats_fieldnames = ["Identity_ID", "Level", "HP", "Defense"]
        level_stats_data = self.get_level_stats_data()  # level_stats 데이터 가져오기
        self.export_to_csv(filename, level_stats_fieldnames, level_stats_data)  # CSV로 내보내기

    def get_uptie_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Uptie": uptie_stat.uptie,
                "Speed": uptie_stat.speed,
            }
            for identity in self.identities
            for uptie_stat in identity.uptie_stats
        ]

    def export_uptie_stats_data(self, filename="Uptiestats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        uptie_stats_fieldnames = ["Identity_ID", "Uptie", "Speed"]
        uptie_stats_data = self.get_uptie_stats_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, uptie_stats_fieldnames, uptie_stats_data)  # CSV로 내보내기

    def get_resist_info_data(self):
        """
        모든 Identity의 resist_info 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Resist_Hit": identity.resist_info.resist_hit,
                "Resist_Penetrate": identity.resist_info.resist_penetrate,
                "Resist_Slash": identity.resist_info.resist_slash,
            }
            for identity in self.identities
        ]

    def export_resist_info_data(self, filename="ResistInfos.csv"):
        """ResistInfo 데이터를 CSV로 내보내는 메서드"""
        resist_infos_fieldnames = ["Identity_ID", "Resist_Hit", "Resist_Penetrate", "Resist_Slash"]
        resist_infos_data = self.get_resist_info_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, resist_infos_fieldnames, resist_infos_data)  # CSV로 내보내기

    def export_uptie_stats_data(self, filename="Uptiestats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        uptie_stats_fieldnames = ["Identity_ID", "Uptie", "Speed"]
        uptie_stats_data = self.get_uptie_stats_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, uptie_stats_fieldnames, uptie_stats_data)  # CSV로 내보내기

    def get_stagger_threshold_data(self):
        """
        모든 Identity의 stagger_threshold 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "First_Stagger": identity.stagger_threshold.first_stagger,
                "Second_Stagger": identity.stagger_threshold.second_stagger,
                "Third_Stagger": identity.stagger_threshold.third_stagger,
            }
            for identity in self.identities
        ]

    def export_stagger_threshold_data(self, filename="StaggerThresholds.csv"):
        """StaggerThresholds 데이터를 CSV로 내보내는 메서드"""
        stagger_thresholds_fieldnames = ["Identity_ID", "First_Stagger", "Second_Stagger", "Third_Stagger"]
        stagger_thresholds_data = self.get_stagger_threshold_data()  # stagger_thresholds 데이터 가져오기
        self.export_to_csv(filename, stagger_thresholds_fieldnames, stagger_thresholds_data)  # CSV로 내보내기


async def main():
    data_container = DataContainer()
    scraper = Scraper(container=data_container, base_url="https://limbus.kusoge.xyz", img_url="https://img.kusoge.xyz/limbus/img")
    await scraper.scrape()

await main() 

Added skill: skill 1:Begone...
Updated Uptie for Begone... (Uptie Level: 1): Skill Power: 2, Coin Power: +4
{'keywords': ['Bloodfeast', 'Bleed', 'Rupture'], 'conditions': ['[Combat Start]', '[Coin 2 On Hit]', '[Coin 2 On Hit]'], 'effectsByCondition': {'[Combat Start]': ['On self and 1 ally with the highest max HP : consume up to 20 keyword:Bloodfeast and apply 1 blooming[thornyfall_panic RodionFirst ] for every 10 this unit consumed ( 2 times per turn ) - Prioritizes < Bloodfiend > allies - If the affected ally is a < Bloodfiend >, this unit applies 1 additional blooming[thornyfall_panic RodionFirst ] - If this unit failed to consume , gain 5 keyword:Bleed'], '[Coin 2 On Hit]': ['Inflict 1 keyword:Bleed', 'Inflict 1 keyword:Rupture']}}
Updated Uptie for Begone... (Uptie Level: 2): Skill Power: 2, Coin Power: +4
{'keywords': ['Bloodfeast', 'Bleed', 'Rupture'], 'conditions': ['[Combat Start]', '[On Use]', '[Coin 2 On Hit]', '[Coin 2 On Hit]'], 'effectsByCondition': {'[Combat Start]': ['O

In [25]:
import os
import csv
import re
import numpy as np
import pandas as pd
from playwright.async_api import async_playwright
from tabulate import tabulate

# Sin Color 이름 변환을 위한 매핑
resource_mapping = {
    "AZURE": "Gloom",
    "VIOLET": "Envy",
    "AMBER": "Sloth",
    "SHAMROCK": "Gluttony",
    "CRIMSON": "Wrath",
    "SCARLET": "Lust",
    "INDIGO": "Pride"
}

def convert_resource(resource_name):
    """Sin Color 이름을 변환하는 함수. 매핑되지 않은 경우 원래 이름을 반환."""
    return resource_mapping.get(resource_name, resource_name)

class Scraper:
    """웹 페이지 크롤링을 위한 클래스"""
    def __init__(self, container, base_url, img_url):
        self.container = container
        self.base_url = base_url
        self.img_url = img_url
        self.uptie_level = 1

    async def handle_request(self, route, request):
        """이미지와 폰트 리소스 로드를 차단."""
        if request.resource_type in ["image", "font"]:
            await route.abort()
        else:
            await route.continue_()
            
    async def scrape(self):
        """크롤링 메인 함수"""
        async with async_playwright() as p:
            browser = await p.chromium.launch(headless=True)
            page = await browser.new_page()
            await page.route("**/*", self.handle_request)
            # Step 1: /identity 페이지에서 기본 정보 크롤링
            #await self.crawl_identity_page(page)

            # Step 2: /identity/{identity_id} 상세 페이지 크롤링
            #identity_ids_to_load = ["10104", "10310", "10710","10808", "10911", "11005"]
            #identity_ids_to_load = ["10104", "10911"]#,"10310", "10710","10808", "11005", "11210"]
            identity_ids_to_load = ["10911","10310"]#, "10710","10808", "11005", "11210"]
            self.container.load_identities_by_id('./csv/identities.csv', identity_ids_to_load)
            await self.crawl_identity_details_for_all(page)
            #self.container.display_identities()
            #self.container.export_identities_data("identities.csv")
            #self.container.export_level_stats_data("Levelstats.csv")
            #self.container.export_uptie_stats_data("Uptiestats.csv")
            #self.container.export_resist_info_data("ResistInfos.csv")
            #self.container.export_stagger_threshold_data("StaggerThresholds.csv")
            #page.set_default_timeout(10000)
            await page.wait_for_timeout(100000)
            await browser.close()

    async def get_identity_id_from_row(self, row):
        """테이블 행(row)에서 인격 ID를 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_url = await identity_cell.get_attribute('href')
            identity_id = identity_url.split('/identity/')[-1]
            return identity_id
        return "Unknown"

    async def get_identity_name_from_row(self, row):
        """테이블 행(row)에서 인격 이름을 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_name = (await identity_cell.inner_text()).replace("\n", " ").strip()
            return identity_name
        return "Unknown"
        
    async def crawl_identity_page(self, page):
        """/identity 페이지로 이동하여 인격 기본 정보를 추출."""
        await page.goto(self.base_url + "/identity")
        await self.extract_identity_page(page)

    async def extract_identity_page(self, page):
        """/identity 페이지의 모든 테이블 행에서 인격 정보를 추출."""
        identity_rows = await page.query_selector_all('tr')
        if len(identity_rows) > 1:
            for idx, identity_row in enumerate(identity_rows):
                if idx == 0:
                    continue # 0번째요소이름을 보여주는 열은 건너뜀
                await self.scrape_identity_name_and_identity_id(identity_row)
                await self.extract_rarity(identity_row)
                for i in range(1, 4):  # 스킬1~3 정보 추출
                    await self.extract_atk_and_resource(identity_row, i)
                await self.extract_defense_and_resource(identity_row)

    async def scrape_identity_name_and_identity_id(self, row):
        """인격 번호와 이름을 추출하여 DataContainer에 저장."""
        try:
            identity_id = await self.get_identity_id_from_row(row)
            identity_name = await self.get_identity_name_from_row(row)
            identity = Identity(identity_id)
            self.container.add_identity(identity)
            self.container.update_identity_name(identity_id, identity_name)
        except Exception as e:
            print(f"Error extracting identity name and identity id: {e}")

    async def get_element_attribute(self, element, selector, attribute):
        """요소에서 지정된 속성(attribute)을 추출."""
        try:
            target_element = await element.query_selector(selector)
            if target_element:
                return await target_element.get_attribute(attribute)
        except Exception as e:
            print(f"Error getting element attribute: {e}")
        return None
    
    async def extract_rarity(self, row):
        """테이블 행(row)에서 희귀도를 추출하여 업데이트."""
        try:
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
                sibling = await parent.evaluate_handle('node => node.nextElementSibling')
                if sibling:
                    rarity_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/rarity_"]', 'src')
                    if rarity_img_src:
                        rarity_level = rarity_img_src.split('_')[-1].split('.')[0]  # rarity_1.png -> 1
                        identity_id = await self.get_identity_id_from_row(row)
                        self.container.update_identity_rarity(identity_id, rarity_level)
                    else:
                        print("Rarity image not found in the next sibling TD.")
        except Exception as e: 
            print(f"Error extracting rarity: {e}")

        return "Unknown"

    async def extract_atk_and_resource(self, row, skill_number):
        """ 공격 타입(ATK Type)과 자원(Attribute) 추출 함수 - 스킬 번호에 따라 처리 """
        try:
            #print(f"Attempting to locate skill{skill_number} type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 해당 스킬 번호에 맞는 형제로 이동
                sibling = parent
                for _ in range(skill_number + 1):  # 스킬 번호에 맞게 이동
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                atk_type, resource = "Unknown", "Unknown"
        
                if sibling:
                    # 공격 타입 이미지 (atk_type) 찾기
                    atk_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/atk_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if atk_type_img_src:
                        atk_type = atk_type_img_src.split('_')[-1].split('.')[0]  # atk_type_SLASH.png -> SLASH
                        #print(f"Extracted attack type: {atk_type}")
                        self.container.update_skill(identity_id, skill_number, atk_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted resource: {resource}")
                        
                        # 리소스를 변환
                        resource = convert_resource(resource)
                        #print(f"Converted resource: {resource}")
                        self.container.update_sin_affinity(identity_id, skill_number, resource)
                        
        except Exception as e:
            print(f"Error extracting skill{skill_number} type and resource: {e}")

    async def extract_defense_and_resource(self, row):
        """ 방어 타입(Def Type)과 자원(Attribute) 추출 함수 """
        try:
            #print("Attempting to locate defense type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 다섯 번째 형제 td로 이동 (방어 관련 정보가 있는 <td>)
                sibling = parent
                for _ in range(5):  # 방어 정보는 5번째 형제에 있음
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                def_type, def_resource = "Unknown", "Unknown"
        
                if sibling:
                    # 방어 타입 이미지 (def_type) 찾기
                    def_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/def_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if def_type_img_src:
                        def_type = def_type_img_src.split('_')[-1].split('.')[0]  # def_type_GUARD.png -> GUARD
                        #print(f"Extracted defense type: {def_type}")
                        self.container.update_skill(identity_id, 'defense', def_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        def_resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted defense resource: {def_resource}")
                        
                        # 방어 리소스를 변환
                        def_resource = convert_resource(def_resource)
                        #print(f"Converted defense resource: {def_resource}")
                        self.container.update_sin_affinity(identity_id, 'defense', def_resource)
 
        except Exception as e:
            print(f"Error extracting defense type and resource: {e}")

    async def crawl_identity_details_for_all(self, page):
        """모든 인격의 세부 정보를 크롤링."""
        for identity in self.container.identities:
            await page.wait_for_load_state("networkidle")
            await page.goto(self.base_url + "/identity/"+identity.identity_id)
            """ 레벨과 동기화와 관련 없는 참관타 저항은 밑에서 변경하는 것과 연관성이 적기 때문에 위에서 처리한다.
            print(f"Identity_id: {identity.identity_id}")
            results = []
            resist_type_imgs = [f"{self.img_url}/ui/atk_type_HIT.png",
                                f"{self.img_url}/ui/atk_type_PENETRATE.png",
                                f"{self.img_url}/ui/atk_type_SLASH.png"]
            for resist_type_img in resist_type_imgs:
                result = await self.extract_resist_type(page, resist_type_img, 1)
                results.append(result)
            self.container.add_resist_info(identity.identity_id, results[0], results[1], results[2])
            """
            """
            # Stagger Threshold 값 추출
            stagger_threshold = await self.extract_stagger_threshold_values(page)
            self.container.add_stagger_threshold(identity.identity_id, stagger_threshold[0], stagger_threshold[1], stagger_threshold[2])
            """
            await self.initialize_uptie_and_level(page)
            """ 레벨에 따른 level, hp, defense 다르게 되어 있는 것을 출력 확인 객체로 들어가서 csv로 옮기는 과정까지 진행
            for i in range(1, identity.level_max+1):
                await self.move_slider_to_value(page,i)
                await self.extract_level_status(page, identity.identity_id, i)
            """
            """ 동기화에 따른 speed 다르게 되어 있는 것을 출력 확인 객체로 들어가서 csv로 옮기는 과정까지 진행
            for i in range(1, identity.uptie_max+1):
                await self.uptie_click(page,i)
                await self.extract_uptie_status(page, identity.identity_id, i)
            """
            await self.crawl_skills(page, identity.identity_id)
            
    async def initialize_uptie_and_level(self, page):
        await self.uptie_click(page, 1)
        await self.move_slider_to_value(page, 1)
        self.uptie_level = 1
        #return 1
    
    async def uptie_click(self, page, phase):
        fab_button_selector = ".q-btn.q-btn--fab"
        await page.wait_for_selector(fab_button_selector)
        await page.click(fab_button_selector)
        #print(f"[{setup_uptie_button}] 상위 동기화 버튼 클릭 완료")
        setup_uptie_button = f'a:has(img[src="{self.img_url}/ui/uptie_{phase}.png"])'
        await page.wait_for_selector(setup_uptie_button)
        await page.click(setup_uptie_button)
        #print(f"{phase} 동기화 버튼 클릭 완료")

    async def move_slider_to_value(self, page, target_value):
        """
        슬라이더를 주어진 값으로 이동시킵니다.
        target_value는 슬라이더의 최소값부터 최대값 사이여야 합니다.
        """
        slider_selector = ".q-slider"
        # 슬라이더의 범위 확인
        min_value = int(await page.get_attribute(slider_selector, "aria-valuemin"))
        max_value = int(await page.get_attribute(slider_selector, "aria-valuemax"))
    
        if target_value < min_value or target_value > max_value:
            raise ValueError(f"Target value {target_value} is out of bounds ({min_value}, {max_value})")
    
        # 슬라이더 thumb 요소와 위치 가져오기
        slider_thumb = await page.query_selector(f"{slider_selector} .q-slider__thumb")
        thumb_box = await slider_thumb.bounding_box()
        
        # 슬라이더의 현재 위치 및 목표 위치 계산
        track = await page.query_selector(f"{slider_selector} .q-slider__track")
        track_box = await track.bounding_box()
        
        # 목표 x 위치 계산 (슬라이더 트랙 내에서 비율 기반)
        target_percentage = (target_value - min_value) / (max_value - min_value)
        target_x = track_box["x"] + target_percentage * track_box["width"]
        current_x = thumb_box["x"] + thumb_box["width"] / 2
        y = thumb_box["y"] + thumb_box["height"] / 2
    
        # 슬라이더를 드래그하여 목표 위치로 이동
        await page.mouse.move(current_x, y)
        await page.mouse.down()
        await page.mouse.move(target_x, y, steps=10)  # 부드럽게 이동
        await page.mouse.up()
    
        #print(f"슬라이더를 {target_value}로 이동 완료.")
    
        # 슬라이더 값 확인 및 강제 이벤트 트리거
        await page.evaluate(
            f"""
            (slider) => {{
                slider.dispatchEvent(new Event('input'));  // oninput 이벤트 트리거
                slider.dispatchEvent(new Event('change'));  // onchange 이벤트 트리거
            }}
            """,
            await page.query_selector(slider_selector)
        )
    
        # 이동 후 값 확인
        current_value = await page.get_attribute(slider_selector, "aria-valuenow")
        if current_value != str(target_value):
            print(f"경고: 슬라이더 값이 예상과 다릅니다. 현재 값: {current_value}, 예상 값: {target_value}")

    async def get_second_status_element(self, page):
        """
        두 번째 'Status' 요소에 접근하여 반환하는 메소드
        """
        status_elements = await page.query_selector_all("text=Status")
        if len(status_elements) > 1:
            second_status_element = status_elements[1]
            #print("두 번째 'Status' 요소에 접근합니다.")
            return second_status_element
        else:
            #print("두 번째 'Status' 요소가 존재하지 않습니다.")
            return None

    async def extract_level_status(self, page, identity_id, level):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 레벨에 따라 HP와 Defense가 변한다.
        hp_value = await self.extract_hp(second_status_element)
        defense_value = await self.extract_defense(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Level={level}, HP={hp_value}, Defense={defense_value}")
        self.container.add_level_stat(identity_id,level, hp_value, defense_value)
    
    async def extract_hp(self, status_element):
        # HP 추출
        hp_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].children[0].nextElementSibling.textContent.trim()"
        )
        return hp_value
    
    async def extract_defense(self, status_element):
        # Defense 추출
        defense_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return defense_value

    async def extract_uptie_status(self, page, identity_id, uptie):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 uptie에 따라 speed는 변한다.
        speed_value = await self.extract_speed(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Uptie={uptie}, Speed={speed_value}")
        self.container.add_uptie_stat(identity_id,uptie, speed_value)

    async def extract_speed(self, status_element):
        # Speed 추출
        speed_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return speed_value

    async def extract_resist_type(self, page, image_src, depth):
        # 이미지 요소 찾기
        #print(image_src)
        image = await page.query_selector(f'img[src="{image_src}"]')
        #print(image)
        if not image:
            return "Unknown"
    
        # 이미지의 부모 요소를 찾아서 올라감
        parent = image
        for _ in range(depth):
            parent = await parent.evaluate_handle('node => node.parentElement')
            if not parent:
                return "Unknown"
    
        # 부모의 형제 요소 찾기
        sibling = await parent.evaluate_handle('node => node.nextElementSibling')
        
        if sibling:
            value = await sibling.evaluate('node => node.textContent')
            if value:
                return value.strip()
        
        return "Unknown"

    async def extract_stagger_threshold_values(self, page):
        # 'div.row.center.flex-center.q-pt-xs' 요소들 찾기
        elements = await page.query_selector_all('div.row.center.flex-center.q-pt-xs')
        
        # 각 요소의 텍스트 내용 출력하여 'Stagger Threshold' 요소를 찾기
        for element in elements:
            text = await element.text_content()
            
            if text and "Stagger Threshold" in text:
                # 'Stagger Threshold'가 있는 요소를 찾았다면, 그 이후 형제 요소들에서 값을 추출
                parent = await element.evaluate_handle('node => node.parentElement')
                sibling_elements = await parent.query_selector_all('div.row.justify-evenly div')  # Threshold 값이 들어있는 div들
                
                stagger_values = []
                for sibling in sibling_elements:
                    text_content = await sibling.text_content()
                    match = re.search(r'(\d+%)', text_content)
                    if match:
                        stagger_values.append(match.group(1))  # 퍼센트 값만 리스트에 추가
                while len(stagger_values) < 3:
                    stagger_values.append(np.nan)
                #print(stagger_values)
                return stagger_values  # 추출된 값(또는 np.nan 포함)을 반환
        
        return [np.nan, np.nan, np.nan]  # 만약 'Stagger Threshold'를 찾지 못한 경우 기본값 반환

    async def crawl_skills(self, page, identity_id):
        await self.crawl_skill(page, identity_id, "skill 1")
        await self.crawl_skill(page, identity_id, "skill 2")
        await self.crawl_skill(page, identity_id, "skill 3")
        await self.crawl_skill(page, identity_id, "defense")


    async def crawl_skill(self, page, identity_id, skill_type):
        skills_elements = await page.query_selector_all('div.skilltag')
        for skill_element in skills_elements:
            skill_text = await skill_element.text_content()
            if skill_text.strip().lower() == skill_type:
                await self.process_skill(skill_element, page, identity_id, skill_type)
        
    async def process_skill(self, skill_element, page, identity_id, skill_type):
        # skill_type이 "skill 3"일 경우 uptie_value가 3 이상일 때만 정보를 저장
        for uptie_value in range(1, 5):  # Uptie levels 1 to 4
            # "skill 3"은 uptie_value가 3 이상일 때만 처리
            if skill_type == "skill 3" and uptie_value < 3:
                continue  # skip if uptie_value is less than 3 for skill 3
    
            # uptie_level에 맞게 클릭
            await self.uptie_click(page, uptie_value)
            if skill_type != "skill 3" and uptie_value > 1:
                # Uptie 정보 갱신
                await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
                continue
            elif skill_type == "skill 3" and uptie_value > 3:
                # Uptie 정보 갱신
                await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
                continue
            # 스킬의 세부 정보를 추출
            skill_details = await self.extract_skill_details(skill_element, skill_type)
            skill_name = skill_details["skill_name"]
    
            # 스킬 정보를 저장
            self.container.add_skill(
                identity_id, skill_type, 
                skill_name, skill_details["coin_count"],
                skill_details["init_level"], skill_details["init_remain"], 
                skill_details["atk_weight"]
            )
            print(f"Added skill: {skill_type}:{skill_name}")
    
            # Uptie 정보 갱신
            await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
    
    async def extract_skill_details(self, skill_element, skill_type):
        # 스킬 세부 정보를 추출하는 함수
        skill_name = await self.skill_name(skill_element)
        coin_count = await self.coin_count(skill_element, ".col-auto > .column > .row")
        init_level = await self.init_level(skill_element)
        init_remain = await self.init_remain(skill_element, skill_type, "div.row.justify-start > div.attr.flex.flex-center > div.q-px-sm")
        atk_weight = await self.atk_weight(skill_element, skill_type)
        return {
            "skill_name": skill_name,
            "coin_count": coin_count,
            "init_level": init_level,
            "init_remain": init_remain,
            "atk_weight": atk_weight
        }
    
    async def update_uptie_info(self, page, skill_element, identity_id, skill_name, skill_type, uptie_value):
        # Uptie 정보를 추출하고 업데이트하는 함수
        uptie_info = await self.extract_uptie_info(skill_element)
        effect_data = await self.skill_effects(skill_element)
        self.container.add_skill_uptie_info(
            identity_id, skill_name, 
            uptie_info["skill_power"], uptie_info["coin_power"], 
            uptie_value, effect_data.get("effectsByCondition")
        )
        print(
            f"Updated Uptie for {skill_name} (Uptie Level: {uptie_value}): "
            f"Skill Power: {uptie_info['skill_power']}, Coin Power: {uptie_info['coin_power']} "
            f"Skill Effect: {effect_data.get("effectsByCondition")}"
        )

    async def extract_uptie_info(self, skill_element):
        skill_data = await self.skill_and_coin_power(skill_element)
        return {
            "skill_power": skill_data.get("skillPower"),
            "coin_power": skill_data.get("coinPower")
        }

    async def extract_skill_effect(self, skill_element):
        # Create the Effect object
        effect_obj = Effect(condition=None)
        # Extract effects
        effects = await self.skill_effects(skill_element)
        if effects:
            effect_obj.effects.append(effects)
        return effect_obj

    async def skill_and_coin_power(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                // 조부모로 이동
                const grandparent = node.parentElement.parentElement;
                // 이전 형제 탐색
                const previousSibling = grandparent.previousElementSibling;
                if (!previousSibling) return {{ skillPower: null, coinPower: null }}; // 이전 형제가 없으면 기본값 반환
                // 이전 형제 내부의 .row.justify-around > div > div 탐색
                const targetDivs = previousSibling.querySelectorAll(".row.justify-around > div");
                if (targetDivs.length < 3) return {{ skillPower: null, coinPower: null }}; // 배열 길이 확인
                // 1st div: 스킬 위력
                const skillPower = targetDivs[0]?.querySelector('div')?.textContent?.trim() || null; 
                // 3rd div: 코인 파워 값 (3번째 div에서 텍스트 추출)
                const coinPower = targetDivs[2]?.textContent?.trim() || null;
                return {{ skillPower, coinPower }};
            }}
        """)


    async def coin_count(self, skill_element, hierarchy_selector):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector('{hierarchy_selector}');
                if (!targetElement) return 0;
                return targetElement.childElementCount;
            }}
        """)

    async def init_remain(self, skill_element, skill_type, hierarchy_selector):
        if skill_type == "defense":
            return -1
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector('{hierarchy_selector}');
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def skill_name(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector(".block");
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def init_level(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector("div.text-bold.text-h6.h6.q-pl-xs.q-pr-md");
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def atk_weight(self, skill_element, skill_type):
        if skill_type == "defense":
            return 0
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector("div.text-bold.text-h6.h6.q-pl-xs.q-pr-md");
                const doubleNextSibling = targetElement.nextElementSibling.nextElementSibling;
                if (!doubleNextSibling) return 0;
                return doubleNextSibling.textContent?.trim();
            }}
        """)

    async def skill_effects(self, skill_element):
        return await skill_element.evaluate("""
            node => {
                const parent = node.parentElement;
                const doubleNextSibling = parent.nextElementSibling?.nextElementSibling;
                if (!doubleNextSibling) return null;
    
                // doubleNextSibling에서 첫 번째 div.col 요소 찾기
                const textNode = doubleNextSibling.querySelector("div.col");
                if (!textNode) return null;
    
                const results = {
                    keywords: [], // 키워드 저장
                    conditions: [], // 조건 저장
                    effectsByCondition: {} // 조건별 효과
                };
    
                // 1. 조건 추출 (대괄호 텍스트)
                const conditionElements = textNode.querySelectorAll("span");
                conditionElements.forEach(span => {
                    let text = span.textContent?.trim();
                    if (text?.startsWith("[") && text?.endsWith("]")) {
                        // 부모의 전형제 노드 탐색
                        const sibling = span.parentElement.previousElementSibling;
                        if (sibling) {
                            const emblemImage = sibling.querySelector("img.emblem");
                            console.log("Sibling element found:", sibling); // 디버깅용
                            if (emblemImage) {
                                const src = emblemImage.getAttribute("src");
                                console.log("Image source:", src); // 디버깅용
                                const match = src.match(/ui_num_(\\d+)\\.png$/); // 정규식 수정
                                if (match) {
                                    const number = match[1]; // 숫자 추출
                                    text = `[Coin ${number} ${text.replace("[", "").replace("]", "")}]`;
                                    console.log("Updated condition:", text); // 디버깅용
                                }
                            }
                        }
                        results.conditions.push(text);
                        results.effectsByCondition[text] = []; // 조건별 효과를 초기화
                    }
                });
    
                // 2. 키워드 추출
                const keywordElements = textNode.querySelectorAll('a[href^="/en/keyword/"]');
                keywordElements.forEach(a => {
                    const span = a.querySelector('span.q-btn__content.text-center.col.items-center.q-anchor--skip.justify-center.row>span');
                    if (span) {
                        const keywordText = span.textContent.trim();
                        if (keywordText && !results.keywords.includes(keywordText)) {
                            results.keywords.push(keywordText); // 키워드 저장
                        }
                    }
                });
    
                // 3. 효과 추출
                const effectElements = textNode.querySelectorAll("span");
                let currentCondition = null;
                let currentEffect = []; // 초기화
                effectElements.forEach(span => {
                    let text = span.textContent?.trim();
                    if (!text) return;
    
                    // 조건 발견 시 currentCondition 변경
                    if (text.startsWith("[") && text.endsWith("]")) {
                        // 이전 조건의 효과 저장
                        if (currentCondition && currentEffect.length > 0) {
                            // 부모의 전형제 노드 탐색
                            const sibling = span.parentElement.previousElementSibling;
                            if (sibling) {
                                const emblemImage = sibling.querySelector("img.emblem");
                                if (emblemImage) {
                                    const src = emblemImage.getAttribute("src");
                                    const match = src.match(/ui_num_(\\d+)\\.png$/); // 정규식 수정
                                    if (match) {
                                        const number = match[1]; // 숫자 추출
                                        text = `[Coin ${number} ${text.replace("[", "").replace("]", "")}]`;
                                    }
                                }
                            }
                            results.effectsByCondition[currentCondition].push(currentEffect.join(' '));
                            currentEffect = []; // 초기화
                        }
                        currentCondition = text;
                        return;
                    }
    
                    // 현재 조건의 효과 처리
                    if (currentCondition) {
                        if (results.keywords.includes(text)) {
                            const keywordEffect = 'keyword:' + text;
                            if (!currentEffect.includes(keywordEffect)) {
                                currentEffect.push(keywordEffect);
                            }
                        } else {
                            currentEffect.push(text);
                        }
                    }
                });
    
                // 마지막 효과 저장
                if (currentCondition && currentEffect.length > 0) {
                    results.effectsByCondition[currentCondition].push(currentEffect.join(' '));
                }
    
                return results;
            }
        """)



class Identity:
    """인격 정보를 저장하는 클래스"""
    def __init__(self, identity_id, name=None, rarity=None,\
                 skill_1_type=None, skill_1_sin_affinity=None,\
                 skill_2_type=None, skill_2_sin_affinity=None,\
                 skill_3_type=None, skill_3_sin_affinity=None,\
                 defense_type=None, defense_sin_affinity=None):
        self.identity_id = identity_id
        self.name = name
        self.rarity = rarity
        # 선언만 하는 부분 
        self.skill_1_type = skill_1_type 
        self.skill_1_sin_affinity = skill_1_sin_affinity 
        self.skill_2_type = skill_2_type 
        self.skill_2_sin_affinity = skill_2_sin_affinity 
        self.skill_3_type = skill_3_type 
        self.skill_3_sin_affinity = skill_3_sin_affinity 
        self.defense_type = defense_type 
        self.defense_sin_affinity = defense_sin_affinity
        self.level_max = 50
        self.uptie_max = 4
        self.level_stats = []  # LevelStat 객체 리스트
        self.uptie_stats = [] # UptieStat 객체 리스트
        self.resist_info = None
        self.stagger_threshold = None
        self.skillset = SkillSet()  # Skill정보를 담고 있는 skillset 객체 
        self.passive_infos = []  # 패시브 객체를 담고 있는 배열
        self.support_passive_infos = [] # 서포트 패시 객체를 담고 있는 배열
        self.panic_type = [] # 혼란상태 발생시 나타나는 효과를 담고 있는 객체 배열

    def update_name(self, name):
        self.name = name

    def update_rarity(self, rarity):
        self.rarity = rarity

    def add_level_stat(self, level, hp, defense):
        self.level_stats.append(LevelStat(level, hp, defense))

    def add_uptie_stat(self, uptie, speed):
        self.uptie_stats.append(UptieStat(uptie, speed))

    def add_resist_info(self, resist_hit, resist_penetrate, resist_slash):
        self.resist_info = ResistInfo(resist_hit, resist_penetrate, resist_slash)

    def add_stagger_threshold(self, first_stagger, second_stagger, third_stagger):
        self.stagger_threshold = StaggerThreshold(first_stagger, second_stagger, third_stagger)

    def add_skill(self, skill_code, skill_name, coin_count, init_level, init_remain, akt_weight):
        self.skillset.add_skill(skill_code, Skill(skill_code, skill_name, coin_count, init_level, init_remain, akt_weight))
        
    def add_skill_uptie_info(self, skill_name, uptie_level, skill_power, coin_power, effect_by_condition):
        self.skillset.add_skill_uptie_info(skill_name, uptie_level, skill_power, coin_power, effect_by_condition)
        
    def update_details(self, page):
        # 세부 정보 업데이트 로직 (가정)
        pass
    #def init_uptie(self, page):
    #    init_uptie = 

# LevelStat 객체 정의
class LevelStat:
    def __init__(self, level, hp=None, defense=None):
        self.level = level
        self.hp = hp
        self.defense = defense
    def update_level(self, level):
        self.level = level
    def update_hp(self, hp):
        self.hp = hp
    def update_defense(self, defense):
        self.defense = defense

# UptieStat 객체 정의
class UptieStat:
    def __init__(self, uptie, speed=None):
        self.uptie = uptie
        self.speed = speed
    def update_speed(self, speed):
        self.speed = speed

# ResistInfo 객체 정의
class ResistInfo:
    def __init__(self, resist_hit=None, resist_penetrate=None, resist_slash=None):
        self.resist_hit = resist_hit
        self.resist_penetrate = resist_penetrate
        self.resist_slash = resist_slash

# ResistInfo 객체 정의
class StaggerThreshold:
    def __init__(self, first_stagger=None, second_stagger=None, third_stagger=None):
        self.first_stagger = first_stagger
        self.second_stagger = second_stagger
        self.third_stagger = third_stagger

class SkillSet:
    def __init__(self):

        self.skill_1_type = ""
        self.skill_2_type = ""
        self.skill_3_type = ""
        self.defense_skill_type = ""

        self.skill_1_sin_affinity = ""
        self.skill_2_sin_affinity = ""
        self.skill_3_sin_affinity = ""
        self.defense_skill_sin_affinity = ""

        self.skill_1_details = []
        self.skill_2_details = []
        self.skill_3_details = []
        self.defense_skill_details = []

    def set_skill_type(self, skill_code, skill_type, sin_affinity):
        if skill_code == "skill 1":
            self.skill_1_type = skill_type
            self.skill_1_sin_affinity = sin_affinity
        elif skill_code == "skill 2":
            self.skill_2_type = skill_type
            self.skill_2_sin_affinity = sin_affinity
        elif skill_code == "skill 3":
            self.skill_3_type = skill_type
            self.skill_3_sin_affinity = sin_affinity
        elif skill_type == "defense":
            self.defense_skill_type = skill_type
            self.defense_skill_sin_affinity = sin_affinity
    
    def add_skill(self, skill_code, skill):
        if skill_code == "skill 1":
            self.skill_1_details.append(skill)
            skill.skill_type = self.skill_1_type
            skill.sin_affinity = self.skill_1_sin_affinity
        elif skill_code == "skill 2":
            self.skill_2_details.append(skill)
            skill.skill_type = self.skill_2_type
            skill.sin_affinity = self.skill_2_sin_affinity
        elif skill_code == "skill 3":
            self.skill_3_details.append(skill)
            skill.skill_type = self.skill_3_type
            skill.sin_affinity = self.skill_3_sin_affinity
        elif skill_code == "defense":
            self.defense_skill_details.append(skill)
            skill.skill_type = self.defense_skill_type 
            skill.sin_affinity = self.defense_skill_sin_affinity

    def get_skill_by_name(self, skill_name):
        for skill_list in [self.skill_1_details, self.skill_2_details, self.skill_3_details, self.defense_skill_details]:
            for skill in skill_list:
                result = skill.is_skill_by_name(skill_name)
                if result:
                    return result
        return None

    def add_skill_uptie_info(self, skill_name, uptie_level, skill_power, coin_power, effect_by_condition):
        skill = self.get_skill_by_name(skill_name)
        skill.add_uptie_info(uptie_level, skill_power, coin_power, effect_by_condition)
        

class Skill:
    def __init__(self, skill_code, skill_name, coin_count, init_level, init_remain, akt_weight):
        self.skill_code = skill_code # "Skill 1"/"Skill 2"/"Skill 3"/"Defense" '''o'''
        self.skill_type = None
        self.sin_affinity = None
        self.skill_name = skill_name # 스킬 이름 '''o'''
        self.coin_count = coin_count # 동전 개수 '''o'''
        self.init_level = init_level # 스킬이 초기 배정받은 레벨, 인격 레벨에 - 1 한 값을 더해주면 스킬의 현 level을 구할 수 있다. '''o'''
        self.init_remain = None #'''o'''
        self.atk_weight = None #공격 가중치 값이다. 방어스킬에는 없기 때문에 None으로 설정했다. '''o'''
        self.remain_count = init_remain # 실제 스킬이 작동하며 횟수가 차감되 나중에 모두 차감되면 다음 턴에 다시 초기횟수 리필된다.
        self.uptie_info = []  

    def is_skill_by_name(self, skill_name):
        if self.skill_name == skill_name:
            return self
        return None
        
    def add_uptie_info(self, uptie_level, skill_power, coin_power, effects_by_condition):
        self.uptie_info.append(UptieInfo(uptie_level, skill_power, coin_power, effects_by_condition))

class UptieInfo:
    def __init__(self, uptie_level, skill_power, coin_power, effects_by_condition):
        self.uptie_level = uptie_level   #'''o'''
        self.skill_power = skill_power   #'''o'''
        self.coin_power = coin_power     #'''o'''
        self.skill_effect = effects_by_condition

# 추상화된 EffectType 클래스
class EffectType:
    def __init__(self, name, description):
        self.name = name  # Effect 이름
        self.description = description  # Effect 설명

# Passive 클래스
class Passive(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)  # condition이 있는지 여부 확인

# SupportPassive 클래스
class SupportPassive(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)

# PanicType 클래스
class PanicType(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)

# Keyword 클래스
class Keyword:
    def __init__(self, keyword, explain=None, dispellable=None):
        self.keyword = keyword
        self.explain = explain
        self.dispellable = dispellable

      
class DataContainer:
    """크롤링 데이터를 관리하는 클래스"""
    def __init__(self):
        self.identities = []

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.level_max

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.uptie_max

    def update_identity_name(self, identity_id, identity_name):
        """특정 ID의 이름을 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_name(identity_name)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_identity_rarity(self, identity_id, rarity):
        """특정 ID의 희귀도를 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_rarity(rarity)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_skill(self, identity_id, skill_number, new_type):
        """
        특정 identity의 skill_type을 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_type: 새 skill_type
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_type = new_type
                elif skill_number == 2:
                    identity.skill_2_type = new_type
                elif skill_number == 3:
                    identity.skill_3_type = new_type
                elif skill_number == "defense":
                    identity.defense_type = new_type
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def update_sin_affinity(self, identity_id, skill_number, new_affinity):
        """
        특정 identity의 sin_affinity를 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_affinity: 새 sin_affinity
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_sin_affinity = new_affinity
                elif skill_number == 2:
                    identity.skill_2_sin_affinity = new_affinity
                elif skill_number == 3:
                    identity.skill_3_sin_affinity = new_affinity
                elif skill_number == "defense":
                    identity.defense_sin_affinity = new_affinity
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def add_identity(self, identity):
        """새로운 인격 데이터를 추가."""
        self.identities.append(identity)

    def add_level_stat(self, identity_id, level, hp, defense):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_level_stat(level, hp, defense)

    def add_uptie_stat(self, identity_id, uptie, speed):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_uptie_stat(uptie, speed)

    def add_resist_info(self, identity_id, resist_hit, resist_penetrate, resist_slash):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_resist_info(resist_hit, resist_penetrate, resist_slash)

    def add_stagger_threshold(self, identity_id, first_stagger, second_stagger, third_stagger):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_stagger_threshold(first_stagger, second_stagger, third_stagger)

    
    def add_skill(self, identity_id, skill_code, skill_name, coin_count, init_level, init_remain, atk_weight):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_code == "skill 1":
                    identity.skillset.set_skill_type(skill_code, identity.skill_1_type, identity.skill_1_sin_affinity)
                elif skill_code == "skill 2":
                    identity.skillset.set_skill_type(skill_code, identity.skill_2_type, identity.skill_2_sin_affinity)
                elif skill_code == "skill 3":
                    identity.skillset.set_skill_type(skill_code, identity.skill_3_type, identity.skill_3_sin_affinity)
                elif skill_code == "defense":
                    identity.skillset.set_skill_type(skill_code, identity.defense_type, identity.defense_sin_affinity)
                identity.add_skill(skill_code, skill_name, coin_count, init_level, init_remain, akt_weight)

    def add_skill_uptie_info(self, identity_id, skill_name, uptie_level, skill_power, coin_power, effects_by_condition):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_skill_uptie_info(skill_name, uptie_level, skill_power, coin_power, effects_by_condition)
        
    def display_identities(self):
        """Identity, ID, Name, Rarity 정보를 출력"""
        if not self.identities:
            print("No identities found.")
        else:
            # 데이터를 리스트로 변환
            table = []
            for idx, identity in enumerate(self.identities, 1):
                table.append([
                    idx, identity.identity_id, identity.name, identity.rarity,
                    identity.skill_1_type, identity.skill_1_sin_affinity,
                    identity.skill_2_type, identity.skill_2_sin_affinity,
                    identity.skill_3_type, identity.skill_3_sin_affinity,
                    identity.defense_type, identity.defense_sin_affinity
                ])
            
            # 테이블 헤더 정의
            headers = [
                "Index", "ID", "Name", "Rarity",
                "Skill_1 Type", "Skill_1 Sin", "Skill_2 Type", "Skill_2 Sin",
                "Skill_3 Type", "Skill_3 Sin", "Defense Type", "Defense Sin"
            ]
            
            # 테이블 출력
            print(tabulate(table, headers=headers, tablefmt="fancy_grid"))

    def export_to_csv(self, filename, fieldnames, data):
        """
        공통적으로 사용 가능한 CSV 내보내기 메서드
        :param filename: 내보낼 파일 이름 (예: 'identity.csv', 'skill.csv')
        :param fieldnames: CSV의 헤더 리스트
        :param data: CSV로 내보낼 데이터 (리스트 형태)
        """
        try:
            # 현재작업 디렉토 기준으 'csv'폴더 없으면 생성
            if not os.path.exists('./csv'):
                os.makedirs('./csv')
            # 파일 경로 설정 (상대경로 'csv' 폴더 안에 저장)
            file_path = os.path.join('./csv', filename)
            with open(file_path, mode='w', newline='', encoding='utf-8') as file:
                writer = csv.DictWriter(file, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(data)
            print(f"Data successfully exported to {filename}")
        except Exception as e:
            print(f"Error exporting to CSV: {e}")

    def get_identity_data(self):
        """
        Identity 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "ID": identity.identity_id,
                "Name": identity.name,
                "Rarity": identity.rarity,
                "Skill_1_Type": identity.skill_1_type,
                "Skill_1_Sin_Affinity": identity.skill_1_sin_affinity,
                "Skill_2_Type": identity.skill_2_type,
                "Skill_2_Sin_Affinity": identity.skill_2_sin_affinity,
                "Skill_3_Type": identity.skill_3_type,
                "Skill_3_Sin_Affinity": identity.skill_3_sin_affinity,
                "Defense_Type": identity.defense_type,
                "Defense_Sin_Affinity": identity.defense_sin_affinity,
            }
            for identity in self.identities
        ]
        
    def export_identities_data(self, filename="identities.csv"):
        """Identity 데이터를 CSV로 내보내는 메서드"""
        identity_fieldnames = [
            "ID", "Name", "Rarity",
            "Skill_1_Type", "Skill_1_Sin_Affinity",
            "Skill_2_Type", "Skill_2_Sin_Affinity",
            "Skill_3_Type", "Skill_3_Sin_Affinity",
            "Defense_Type", "Defense_Sin_Affinity"
        ]
        identity_data = self.get_identity_data()  # Identity 데이터 가져오기
        self.export_to_csv(filename, identity_fieldnames, identity_data)  # CSV로 내보내기

    def load_identities_by_id(self, filename, identity_ids):
        """지정된 identity_ids 목록에 해당하는 Identity 데이터를 CSV에서 읽어옴."""
        identity_ids_set = set(identity_ids) # 중복 제거를 위 Set 사용
        with open(filename, mode='r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                if row["ID"] in identity_ids_set:
                    identity = Identity(
                        identity_id=row["ID"],
                        name=row["Name"],
                        rarity=row["Rarity"],
                        skill_1_type=row["Skill_1_Type"],
                        skill_1_sin_affinity=row["Skill_1_Sin_Affinity"],
                        skill_2_type=row["Skill_2_Type"],
                        skill_2_sin_affinity=row["Skill_2_Sin_Affinity"],
                        skill_3_type=row["Skill_3_Type"],
                        skill_3_sin_affinity=row["Skill_3_Sin_Affinity"],
                        defense_type=row["Defense_Type"],
                        defense_sin_affinity=row["Defense_Sin_Affinity"]
                    )
                    self.identities.append(identity)

    def get_level_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Level": level_stat.level,
                "HP": level_stat.hp,
                "Defense": level_stat.defense,
            }
            for identity in self.identities
            for level_stat in identity.level_stats
        ]

    def export_level_stats_data(self, filename="levelstats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        level_stats_fieldnames = ["Identity_ID", "Level", "HP", "Defense"]
        level_stats_data = self.get_level_stats_data()  # level_stats 데이터 가져오기
        self.export_to_csv(filename, level_stats_fieldnames, level_stats_data)  # CSV로 내보내기

    def get_uptie_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Uptie": uptie_stat.uptie,
                "Speed": uptie_stat.speed,
            }
            for identity in self.identities
            for uptie_stat in identity.uptie_stats
        ]

    def export_uptie_stats_data(self, filename="Uptiestats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        uptie_stats_fieldnames = ["Identity_ID", "Uptie", "Speed"]
        uptie_stats_data = self.get_uptie_stats_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, uptie_stats_fieldnames, uptie_stats_data)  # CSV로 내보내기

    def get_resist_info_data(self):
        """
        모든 Identity의 resist_info 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Resist_Hit": identity.resist_info.resist_hit,
                "Resist_Penetrate": identity.resist_info.resist_penetrate,
                "Resist_Slash": identity.resist_info.resist_slash,
            }
            for identity in self.identities
        ]

    def export_resist_info_data(self, filename="ResistInfos.csv"):
        """ResistInfo 데이터를 CSV로 내보내는 메서드"""
        resist_infos_fieldnames = ["Identity_ID", "Resist_Hit", "Resist_Penetrate", "Resist_Slash"]
        resist_infos_data = self.get_resist_info_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, resist_infos_fieldnames, resist_infos_data)  # CSV로 내보내기

    def export_uptie_stats_data(self, filename="Uptiestats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        uptie_stats_fieldnames = ["Identity_ID", "Uptie", "Speed"]
        uptie_stats_data = self.get_uptie_stats_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, uptie_stats_fieldnames, uptie_stats_data)  # CSV로 내보내기

    def get_stagger_threshold_data(self):
        """
        모든 Identity의 stagger_threshold 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "First_Stagger": identity.stagger_threshold.first_stagger,
                "Second_Stagger": identity.stagger_threshold.second_stagger,
                "Third_Stagger": identity.stagger_threshold.third_stagger,
            }
            for identity in self.identities
        ]

    def export_stagger_threshold_data(self, filename="StaggerThresholds.csv"):
        """StaggerThresholds 데이터를 CSV로 내보내는 메서드"""
        stagger_thresholds_fieldnames = ["Identity_ID", "First_Stagger", "Second_Stagger", "Third_Stagger"]
        stagger_thresholds_data = self.get_stagger_threshold_data()  # stagger_thresholds 데이터 가져오기
        self.export_to_csv(filename, stagger_thresholds_fieldnames, stagger_thresholds_data)  # CSV로 내보내기


async def main():
    data_container = DataContainer()
    scraper = Scraper(container=data_container, base_url="https://limbus.kusoge.xyz", img_url="https://img.kusoge.xyz/limbus/img")
    await scraper.scrape()

await main() 

NameError: name 'akt_weight' is not defined

In [28]:
import os
import csv
import re
import json
import numpy as np
import pandas as pd
from playwright.async_api import async_playwright
from tabulate import tabulate

# Sin Color 이름 변환을 위한 매핑
resource_mapping = {
    "AZURE": "Gloom",
    "VIOLET": "Envy",
    "AMBER": "Sloth",
    "SHAMROCK": "Gluttony",
    "CRIMSON": "Wrath",
    "SCARLET": "Lust",
    "INDIGO": "Pride"
}

def convert_resource(resource_name):
    """Sin Color 이름을 변환하는 함수. 매핑되지 않은 경우 원래 이름을 반환."""
    return resource_mapping.get(resource_name, resource_name)

class Scraper:
    """웹 페이지 크롤링을 위한 클래스"""
    def __init__(self, container, base_url, img_url):
        self.container = container
        self.base_url = base_url
        self.img_url = img_url
        self.uptie_level = 1

    async def handle_request(self, route, request):
        """이미지와 폰트 리소스 로드를 차단."""
        if request.resource_type in ["image", "font"]:
            await route.abort()
        else:
            await route.continue_()
            
    async def scrape(self):
        """크롤링 메인 함수"""
        async with async_playwright() as p:
            browser = await p.chromium.launch(headless=True)
            page = await browser.new_page()
            await page.route("**/*", self.handle_request)
            # Step 1: /identity 페이지에서 기본 정보 크롤링
            #await self.crawl_identity_page(page)

            # Step 2: /identity/{identity_id} 상세 페이지 크롤링
            #identity_ids_to_load = ["10104", "10310", "10710","10808", "10911", "11005"]
            #identity_ids_to_load = ["10104", "10911"]#,"10310", "10710","10808", "11005", "11210"]
            identity_ids_to_load = ["10911","10310"]#, "10710","10808", "11005", "11210"]
            self.container.load_identities_by_id('./csv/identities.csv', identity_ids_to_load)
            await self.crawl_identity_details_for_all(page)
            #self.container.display_identities()
            #self.container.export_identities_data("identities.csv")
            #self.container.export_level_stats_data("Levelstats.csv")
            #self.container.export_uptie_stats_data("Uptiestats.csv")
            #self.container.export_resist_info_data("ResistInfos.csv")
            #self.container.export_stagger_threshold_data("StaggerThresholds.csv")
            self.container.export_identity_skillset_data("IdentitiesSkills.csv")
            #page.set_default_timeout(10000)
            await page.wait_for_timeout(100000)
            await browser.close()

    async def get_identity_id_from_row(self, row):
        """테이블 행(row)에서 인격 ID를 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_url = await identity_cell.get_attribute('href')
            identity_id = identity_url.split('/identity/')[-1]
            return identity_id
        return "Unknown"

    async def get_identity_name_from_row(self, row):
        """테이블 행(row)에서 인격 이름을 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_name = (await identity_cell.inner_text()).replace("\n", " ").strip()
            return identity_name
        return "Unknown"
        
    async def crawl_identity_page(self, page):
        """/identity 페이지로 이동하여 인격 기본 정보를 추출."""
        await page.goto(self.base_url + "/identity")
        await self.extract_identity_page(page)

    async def extract_identity_page(self, page):
        """/identity 페이지의 모든 테이블 행에서 인격 정보를 추출."""
        identity_rows = await page.query_selector_all('tr')
        if len(identity_rows) > 1:
            for idx, identity_row in enumerate(identity_rows):
                if idx == 0:
                    continue # 0번째요소이름을 보여주는 열은 건너뜀
                await self.scrape_identity_name_and_identity_id(identity_row)
                await self.extract_rarity(identity_row)
                for i in range(1, 4):  # 스킬1~3 정보 추출
                    await self.extract_atk_and_resource(identity_row, i)
                await self.extract_defense_and_resource(identity_row)

    async def scrape_identity_name_and_identity_id(self, row):
        """인격 번호와 이름을 추출하여 DataContainer에 저장."""
        try:
            identity_id = await self.get_identity_id_from_row(row)
            identity_name = await self.get_identity_name_from_row(row)
            identity = Identity(identity_id)
            self.container.add_identity(identity)
            self.container.update_identity_name(identity_id, identity_name)
        except Exception as e:
            print(f"Error extracting identity name and identity id: {e}")

    async def get_element_attribute(self, element, selector, attribute):
        """요소에서 지정된 속성(attribute)을 추출."""
        try:
            target_element = await element.query_selector(selector)
            if target_element:
                return await target_element.get_attribute(attribute)
        except Exception as e:
            print(f"Error getting element attribute: {e}")
        return None
    
    async def extract_rarity(self, row):
        """테이블 행(row)에서 희귀도를 추출하여 업데이트."""
        try:
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
                sibling = await parent.evaluate_handle('node => node.nextElementSibling')
                if sibling:
                    rarity_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/rarity_"]', 'src')
                    if rarity_img_src:
                        rarity_level = rarity_img_src.split('_')[-1].split('.')[0]  # rarity_1.png -> 1
                        identity_id = await self.get_identity_id_from_row(row)
                        self.container.update_identity_rarity(identity_id, rarity_level)
                    else:
                        print("Rarity image not found in the next sibling TD.")
        except Exception as e: 
            print(f"Error extracting rarity: {e}")

        return "Unknown"

    async def extract_atk_and_resource(self, row, skill_number):
        """ 공격 타입(ATK Type)과 자원(Attribute) 추출 함수 - 스킬 번호에 따라 처리 """
        try:
            #print(f"Attempting to locate skill{skill_number} type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 해당 스킬 번호에 맞는 형제로 이동
                sibling = parent
                for _ in range(skill_number + 1):  # 스킬 번호에 맞게 이동
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                atk_type, resource = "Unknown", "Unknown"
        
                if sibling:
                    # 공격 타입 이미지 (atk_type) 찾기
                    atk_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/atk_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if atk_type_img_src:
                        atk_type = atk_type_img_src.split('_')[-1].split('.')[0]  # atk_type_SLASH.png -> SLASH
                        #print(f"Extracted attack type: {atk_type}")
                        self.container.update_skill(identity_id, skill_number, atk_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted resource: {resource}")
                        
                        # 리소스를 변환
                        resource = convert_resource(resource)
                        #print(f"Converted resource: {resource}")
                        self.container.update_sin_affinity(identity_id, skill_number, resource)
                        
        except Exception as e:
            print(f"Error extracting skill{skill_number} type and resource: {e}")

    async def extract_defense_and_resource(self, row):
        """ 방어 타입(Def Type)과 자원(Attribute) 추출 함수 """
        try:
            #print("Attempting to locate defense type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 다섯 번째 형제 td로 이동 (방어 관련 정보가 있는 <td>)
                sibling = parent
                for _ in range(5):  # 방어 정보는 5번째 형제에 있음
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                def_type, def_resource = "Unknown", "Unknown"
        
                if sibling:
                    # 방어 타입 이미지 (def_type) 찾기
                    def_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/def_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if def_type_img_src:
                        def_type = def_type_img_src.split('_')[-1].split('.')[0]  # def_type_GUARD.png -> GUARD
                        #print(f"Extracted defense type: {def_type}")
                        self.container.update_skill(identity_id, 'defense', def_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        def_resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted defense resource: {def_resource}")
                        
                        # 방어 리소스를 변환
                        def_resource = convert_resource(def_resource)
                        #print(f"Converted defense resource: {def_resource}")
                        self.container.update_sin_affinity(identity_id, 'defense', def_resource)
 
        except Exception as e:
            print(f"Error extracting defense type and resource: {e}")

    async def crawl_identity_details_for_all(self, page):
        """모든 인격의 세부 정보를 크롤링."""
        for identity in self.container.identities:
            await page.wait_for_load_state("networkidle")
            await page.goto(self.base_url + "/identity/"+identity.identity_id)
            """ 레벨과 동기화와 관련 없는 참관타 저항은 밑에서 변경하는 것과 연관성이 적기 때문에 위에서 처리한다.
            print(f"Identity_id: {identity.identity_id}")
            results = []
            resist_type_imgs = [f"{self.img_url}/ui/atk_type_HIT.png",
                                f"{self.img_url}/ui/atk_type_PENETRATE.png",
                                f"{self.img_url}/ui/atk_type_SLASH.png"]
            for resist_type_img in resist_type_imgs:
                result = await self.extract_resist_type(page, resist_type_img, 1)
                results.append(result)
            self.container.add_resist_info(identity.identity_id, results[0], results[1], results[2])
            """
            """
            # Stagger Threshold 값 추출
            stagger_threshold = await self.extract_stagger_threshold_values(page)
            self.container.add_stagger_threshold(identity.identity_id, stagger_threshold[0], stagger_threshold[1], stagger_threshold[2])
            """
            await self.initialize_uptie_and_level(page)
            """ 레벨에 따른 level, hp, defense 다르게 되어 있는 것을 출력 확인 객체로 들어가서 csv로 옮기는 과정까지 진행
            for i in range(1, identity.level_max+1):
                await self.move_slider_to_value(page,i)
                await self.extract_level_status(page, identity.identity_id, i)
            """
            """ 동기화에 따른 speed 다르게 되어 있는 것을 출력 확인 객체로 들어가서 csv로 옮기는 과정까지 진행
            for i in range(1, identity.uptie_max+1):
                await self.uptie_click(page,i)
                await self.extract_uptie_status(page, identity.identity_id, i)
            """
            await self.crawl_skills(page, identity.identity_id)
            
    async def initialize_uptie_and_level(self, page):
        await self.uptie_click(page, 1)
        await self.move_slider_to_value(page, 1)
        self.uptie_level = 1
        #return 1
    
    async def uptie_click(self, page, phase):
        fab_button_selector = ".q-btn.q-btn--fab"
        await page.wait_for_selector(fab_button_selector)
        await page.click(fab_button_selector)
        #print(f"[{setup_uptie_button}] 상위 동기화 버튼 클릭 완료")
        setup_uptie_button = f'a:has(img[src="{self.img_url}/ui/uptie_{phase}.png"])'
        await page.wait_for_selector(setup_uptie_button)
        await page.click(setup_uptie_button)
        #print(f"{phase} 동기화 버튼 클릭 완료")

    async def move_slider_to_value(self, page, target_value):
        """
        슬라이더를 주어진 값으로 이동시킵니다.
        target_value는 슬라이더의 최소값부터 최대값 사이여야 합니다.
        """
        slider_selector = ".q-slider"
        # 슬라이더의 범위 확인
        min_value = int(await page.get_attribute(slider_selector, "aria-valuemin"))
        max_value = int(await page.get_attribute(slider_selector, "aria-valuemax"))
    
        if target_value < min_value or target_value > max_value:
            raise ValueError(f"Target value {target_value} is out of bounds ({min_value}, {max_value})")
    
        # 슬라이더 thumb 요소와 위치 가져오기
        slider_thumb = await page.query_selector(f"{slider_selector} .q-slider__thumb")
        thumb_box = await slider_thumb.bounding_box()
        
        # 슬라이더의 현재 위치 및 목표 위치 계산
        track = await page.query_selector(f"{slider_selector} .q-slider__track")
        track_box = await track.bounding_box()
        
        # 목표 x 위치 계산 (슬라이더 트랙 내에서 비율 기반)
        target_percentage = (target_value - min_value) / (max_value - min_value)
        target_x = track_box["x"] + target_percentage * track_box["width"]
        current_x = thumb_box["x"] + thumb_box["width"] / 2
        y = thumb_box["y"] + thumb_box["height"] / 2
    
        # 슬라이더를 드래그하여 목표 위치로 이동
        await page.mouse.move(current_x, y)
        await page.mouse.down()
        await page.mouse.move(target_x, y, steps=10)  # 부드럽게 이동
        await page.mouse.up()
    
        #print(f"슬라이더를 {target_value}로 이동 완료.")
    
        # 슬라이더 값 확인 및 강제 이벤트 트리거
        await page.evaluate(
            f"""
            (slider) => {{
                slider.dispatchEvent(new Event('input'));  // oninput 이벤트 트리거
                slider.dispatchEvent(new Event('change'));  // onchange 이벤트 트리거
            }}
            """,
            await page.query_selector(slider_selector)
        )
    
        # 이동 후 값 확인
        current_value = await page.get_attribute(slider_selector, "aria-valuenow")
        if current_value != str(target_value):
            print(f"경고: 슬라이더 값이 예상과 다릅니다. 현재 값: {current_value}, 예상 값: {target_value}")

    async def get_second_status_element(self, page):
        """
        두 번째 'Status' 요소에 접근하여 반환하는 메소드
        """
        status_elements = await page.query_selector_all("text=Status")
        if len(status_elements) > 1:
            second_status_element = status_elements[1]
            #print("두 번째 'Status' 요소에 접근합니다.")
            return second_status_element
        else:
            #print("두 번째 'Status' 요소가 존재하지 않습니다.")
            return None

    async def extract_level_status(self, page, identity_id, level):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 레벨에 따라 HP와 Defense가 변한다.
        hp_value = await self.extract_hp(second_status_element)
        defense_value = await self.extract_defense(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Level={level}, HP={hp_value}, Defense={defense_value}")
        self.container.add_level_stat(identity_id,level, hp_value, defense_value)
    
    async def extract_hp(self, status_element):
        # HP 추출
        hp_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].children[0].nextElementSibling.textContent.trim()"
        )
        return hp_value
    
    async def extract_defense(self, status_element):
        # Defense 추출
        defense_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return defense_value

    async def extract_uptie_status(self, page, identity_id, uptie):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 uptie에 따라 speed는 변한다.
        speed_value = await self.extract_speed(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Uptie={uptie}, Speed={speed_value}")
        self.container.add_uptie_stat(identity_id,uptie, speed_value)

    async def extract_speed(self, status_element):
        # Speed 추출
        speed_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return speed_value

    async def extract_resist_type(self, page, image_src, depth):
        # 이미지 요소 찾기
        #print(image_src)
        image = await page.query_selector(f'img[src="{image_src}"]')
        #print(image)
        if not image:
            return "Unknown"
    
        # 이미지의 부모 요소를 찾아서 올라감
        parent = image
        for _ in range(depth):
            parent = await parent.evaluate_handle('node => node.parentElement')
            if not parent:
                return "Unknown"
    
        # 부모의 형제 요소 찾기
        sibling = await parent.evaluate_handle('node => node.nextElementSibling')
        
        if sibling:
            value = await sibling.evaluate('node => node.textContent')
            if value:
                return value.strip()
        
        return "Unknown"

    async def extract_stagger_threshold_values(self, page):
        # 'div.row.center.flex-center.q-pt-xs' 요소들 찾기
        elements = await page.query_selector_all('div.row.center.flex-center.q-pt-xs')
        
        # 각 요소의 텍스트 내용 출력하여 'Stagger Threshold' 요소를 찾기
        for element in elements:
            text = await element.text_content()
            
            if text and "Stagger Threshold" in text:
                # 'Stagger Threshold'가 있는 요소를 찾았다면, 그 이후 형제 요소들에서 값을 추출
                parent = await element.evaluate_handle('node => node.parentElement')
                sibling_elements = await parent.query_selector_all('div.row.justify-evenly div')  # Threshold 값이 들어있는 div들
                
                stagger_values = []
                for sibling in sibling_elements:
                    text_content = await sibling.text_content()
                    match = re.search(r'(\d+%)', text_content)
                    if match:
                        stagger_values.append(match.group(1))  # 퍼센트 값만 리스트에 추가
                while len(stagger_values) < 3:
                    stagger_values.append(np.nan)
                #print(stagger_values)
                return stagger_values  # 추출된 값(또는 np.nan 포함)을 반환
        
        return [np.nan, np.nan, np.nan]  # 만약 'Stagger Threshold'를 찾지 못한 경우 기본값 반환

    async def crawl_skills(self, page, identity_id):
        await self.crawl_skill(page, identity_id, "skill 1")
        await self.crawl_skill(page, identity_id, "skill 2")
        await self.crawl_skill(page, identity_id, "skill 3")
        await self.crawl_skill(page, identity_id, "defense")


    async def crawl_skill(self, page, identity_id, skill_type):
        skills_elements = await page.query_selector_all('div.skilltag')
        for skill_element in skills_elements:
            skill_text = await skill_element.text_content()
            if skill_text.strip().lower() == skill_type:
                await self.process_skill(skill_element, page, identity_id, skill_type)
        
    async def process_skill(self, skill_element, page, identity_id, skill_type):
        # skill_type이 "skill 3"일 경우 uptie_value가 3 이상일 때만 정보를 저장
        for uptie_value in range(1, 5):  # Uptie levels 1 to 4
            # "skill 3"은 uptie_value가 3 이상일 때만 처리
            if skill_type == "skill 3" and uptie_value < 3:
                continue  # skip if uptie_value is less than 3 for skill 3
    
            # uptie_level에 맞게 클릭
            await self.uptie_click(page, uptie_value)
            if skill_type != "skill 3" and uptie_value > 1:
                # Uptie 정보 갱신
                await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
                continue
            elif skill_type == "skill 3" and uptie_value > 3:
                # Uptie 정보 갱신
                await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
                continue
            # 스킬의 세부 정보를 추출
            skill_details = await self.extract_skill_details(skill_element, skill_type)
            skill_name = skill_details["skill_name"]
    
            # 스킬 정보를 저장
            self.container.add_skill(
                identity_id, skill_type, 
                skill_name, skill_details["coin_count"],
                skill_details["init_level"], skill_details["init_remain"], 
                skill_details["atk_weight"]
            )
            print(f"Added skill: {skill_type}:{skill_name}")
    
            # Uptie 정보 갱신
            await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
    
    async def extract_skill_details(self, skill_element, skill_type):
        # 스킬 세부 정보를 추출하는 함수
        skill_name = await self.skill_name(skill_element)
        coin_count = await self.coin_count(skill_element, ".col-auto > .column > .row")
        init_level = await self.init_level(skill_element)
        init_remain = await self.init_remain(skill_element, skill_type, "div.row.justify-start > div.attr.flex.flex-center > div.q-px-sm")
        atk_weight = await self.atk_weight(skill_element, skill_type)
        return {
            "skill_name": skill_name,
            "coin_count": coin_count,
            "init_level": init_level,
            "init_remain": init_remain,
            "atk_weight": atk_weight
        }
    
    async def update_uptie_info(self, page, skill_element, identity_id, skill_name, skill_type, uptie_value):
        # Uptie 정보를 추출하고 업데이트하는 함수
        uptie_info = await self.extract_uptie_info(skill_element)
        effect_data = await self.skill_effects(skill_element)
        self.container.add_skill_uptie_info(
            identity_id, skill_name, 
            uptie_value, uptie_info["skill_power"], uptie_info["coin_power"], 
            effect_data.get("effectsByCondition")
        )
        print(
            f"Updated Uptie for {skill_name} (Uptie Level: {uptie_value}): "
            f"Skill Power: {uptie_info['skill_power']}, Coin Power: {uptie_info['coin_power']} "
            f"Skill Effect: {effect_data.get("effectsByCondition")}"
        )

    async def extract_uptie_info(self, skill_element):
        skill_data = await self.skill_and_coin_power(skill_element)
        return {
            "skill_power": skill_data.get("skillPower"),
            "coin_power": skill_data.get("coinPower")
        }

    async def extract_skill_effect(self, skill_element):
        # Create the Effect object
        effect_obj = Effect(condition=None)
        # Extract effects
        effects = await self.skill_effects(skill_element)
        if effects:
            effect_obj.effects.append(effects)
        return effect_obj

    async def skill_and_coin_power(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                // 조부모로 이동
                const grandparent = node.parentElement.parentElement;
                // 이전 형제 탐색
                const previousSibling = grandparent.previousElementSibling;
                if (!previousSibling) return {{ skillPower: null, coinPower: null }}; // 이전 형제가 없으면 기본값 반환
                // 이전 형제 내부의 .row.justify-around > div > div 탐색
                const targetDivs = previousSibling.querySelectorAll(".row.justify-around > div");
                if (targetDivs.length < 3) return {{ skillPower: null, coinPower: null }}; // 배열 길이 확인
                // 1st div: 스킬 위력
                const skillPower = targetDivs[0]?.querySelector('div')?.textContent?.trim() || null; 
                // 3rd div: 코인 파워 값 (3번째 div에서 텍스트 추출)
                const coinPower = targetDivs[2]?.textContent?.trim() || null;
                return {{ skillPower, coinPower }};
            }}
        """)


    async def coin_count(self, skill_element, hierarchy_selector):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector('{hierarchy_selector}');
                if (!targetElement) return 0;
                return targetElement.childElementCount;
            }}
        """)

    async def init_remain(self, skill_element, skill_type, hierarchy_selector):
        if skill_type == "defense":
            return -1
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector('{hierarchy_selector}');
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def skill_name(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector(".block");
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def init_level(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector("div.text-bold.text-h6.h6.q-pl-xs.q-pr-md");
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def atk_weight(self, skill_element, skill_type):
        if skill_type == "defense":
            return 0
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector("div.text-bold.text-h6.h6.q-pl-xs.q-pr-md");
                const doubleNextSibling = targetElement.nextElementSibling.nextElementSibling;
                if (!doubleNextSibling) return 0;
                return doubleNextSibling.textContent?.trim();
            }}
        """)

    async def skill_effects(self, skill_element):
        return await skill_element.evaluate("""
            node => {
                const parent = node.parentElement;
                const doubleNextSibling = parent.nextElementSibling?.nextElementSibling;
                if (!doubleNextSibling) return null;
    
                // doubleNextSibling에서 첫 번째 div.col 요소 찾기
                const textNode = doubleNextSibling.querySelector("div.col");
                if (!textNode) return null;
    
                const results = {
                    keywords: [], // 키워드 저장
                    conditions: [], // 조건 저장
                    effectsByCondition: {} // 조건별 효과
                };
    
                // 1. 조건 추출 (대괄호 텍스트)
                const conditionElements = textNode.querySelectorAll("span");
                conditionElements.forEach(span => {
                    let text = span.textContent?.trim();
                    if (text?.startsWith("[") && text?.endsWith("]")) {
                        // 부모의 전형제 노드 탐색
                        const sibling = span.parentElement.previousElementSibling;
                        if (sibling) {
                            const emblemImage = sibling.querySelector("img.emblem");
                            console.log("Sibling element found:", sibling); // 디버깅용
                            if (emblemImage) {
                                const src = emblemImage.getAttribute("src");
                                console.log("Image source:", src); // 디버깅용
                                const match = src.match(/ui_num_(\\d+)\\.png$/); // 정규식 수정
                                if (match) {
                                    const number = match[1]; // 숫자 추출
                                    text = `[Coin ${number} ${text.replace("[", "").replace("]", "")}]`;
                                    console.log("Updated condition:", text); // 디버깅용
                                }
                            }
                        }
                        results.conditions.push(text);
                        results.effectsByCondition[text] = []; // 조건별 효과를 초기화
                    }
                });
    
                // 2. 키워드 추출
                const keywordElements = textNode.querySelectorAll('a[href^="/en/keyword/"]');
                keywordElements.forEach(a => {
                    const span = a.querySelector('span.q-btn__content.text-center.col.items-center.q-anchor--skip.justify-center.row>span');
                    if (span) {
                        const keywordText = span.textContent.trim();
                        if (keywordText && !results.keywords.includes(keywordText)) {
                            results.keywords.push(keywordText); // 키워드 저장
                        }
                    }
                });
    
                // 3. 효과 추출
                const effectElements = textNode.querySelectorAll("span");
                let currentCondition = null;
                let currentEffect = []; // 초기화
                effectElements.forEach(span => {
                    let text = span.textContent?.trim();
                    if (!text) return;
    
                    // 조건 발견 시 currentCondition 변경
                    if (text.startsWith("[") && text.endsWith("]")) {
                        // 이전 조건의 효과 저장
                        if (currentCondition && currentEffect.length > 0) {
                            // 부모의 전형제 노드 탐색
                            const sibling = span.parentElement.previousElementSibling;
                            if (sibling) {
                                const emblemImage = sibling.querySelector("img.emblem");
                                if (emblemImage) {
                                    const src = emblemImage.getAttribute("src");
                                    const match = src.match(/ui_num_(\\d+)\\.png$/); // 정규식 수정
                                    if (match) {
                                        const number = match[1]; // 숫자 추출
                                        text = `[Coin ${number} ${text.replace("[", "").replace("]", "")}]`;
                                    }
                                }
                            }
                            results.effectsByCondition[currentCondition].push(currentEffect.join(' '));
                            currentEffect = []; // 초기화
                        }
                        currentCondition = text;
                        return;
                    }
    
                    // 현재 조건의 효과 처리
                    if (currentCondition) {
                        if (results.keywords.includes(text)) {
                            const keywordEffect = 'keyword:' + text;
                            if (!currentEffect.includes(keywordEffect)) {
                                currentEffect.push(keywordEffect);
                            }
                        } else {
                            currentEffect.push(text);
                        }
                    }
                });
    
                // 마지막 효과 저장
                if (currentCondition && currentEffect.length > 0) {
                    results.effectsByCondition[currentCondition].push(currentEffect.join(' '));
                }
    
                return results;
            }
        """)



class Identity:
    """인격 정보를 저장하는 클래스"""
    def __init__(self, identity_id, name=None, rarity=None,\
                 skill_1_type=None, skill_1_sin_affinity=None,\
                 skill_2_type=None, skill_2_sin_affinity=None,\
                 skill_3_type=None, skill_3_sin_affinity=None,\
                 defense_type=None, defense_sin_affinity=None):
        self.identity_id = identity_id
        self.name = name
        self.rarity = rarity
        # 선언만 하는 부분 
        self.skill_1_type = skill_1_type 
        self.skill_1_sin_affinity = skill_1_sin_affinity 
        self.skill_2_type = skill_2_type 
        self.skill_2_sin_affinity = skill_2_sin_affinity 
        self.skill_3_type = skill_3_type 
        self.skill_3_sin_affinity = skill_3_sin_affinity 
        self.defense_type = defense_type 
        self.defense_sin_affinity = defense_sin_affinity
        self.level_max = 50
        self.uptie_max = 4
        self.level_stats = []  # LevelStat 객체 리스트
        self.uptie_stats = [] # UptieStat 객체 리스트
        self.resist_info = None
        self.stagger_threshold = None
        self.skillset = SkillSet()  # Skill정보를 담고 있는 skillset 객체 
        self.passive_infos = []  # 패시브 객체를 담고 있는 배열
        self.support_passive_infos = [] # 서포트 패시 객체를 담고 있는 배열
        self.panic_type = [] # 혼란상태 발생시 나타나는 효과를 담고 있는 객체 배열

    def update_name(self, name):
        self.name = name

    def update_rarity(self, rarity):
        self.rarity = rarity

    def add_level_stat(self, level, hp, defense):
        self.level_stats.append(LevelStat(level, hp, defense))

    def add_uptie_stat(self, uptie, speed):
        self.uptie_stats.append(UptieStat(uptie, speed))

    def add_resist_info(self, resist_hit, resist_penetrate, resist_slash):
        self.resist_info = ResistInfo(resist_hit, resist_penetrate, resist_slash)

    def add_stagger_threshold(self, first_stagger, second_stagger, third_stagger):
        self.stagger_threshold = StaggerThreshold(first_stagger, second_stagger, third_stagger)

    def add_skill(self, skill_code, skill_name, coin_count, init_level, init_remain, atk_weight):
        self.skillset.add_skill(skill_code, Skill(skill_code, skill_name, coin_count, init_level, init_remain, atk_weight))
        
    def add_skill_uptie_info(self, skill_name, uptie_level, skill_power, coin_power, effect_by_condition):
        self.skillset.add_skill_uptie_info(skill_name, uptie_level, skill_power, coin_power, effect_by_condition)
        
    def update_details(self, page):
        # 세부 정보 업데이트 로직 (가정)
        pass
    #def init_uptie(self, page):
    #    init_uptie = 

# LevelStat 객체 정의
class LevelStat:
    def __init__(self, level, hp=None, defense=None):
        self.level = level
        self.hp = hp
        self.defense = defense
    def update_level(self, level):
        self.level = level
    def update_hp(self, hp):
        self.hp = hp
    def update_defense(self, defense):
        self.defense = defense

# UptieStat 객체 정의
class UptieStat:
    def __init__(self, uptie, speed=None):
        self.uptie = uptie
        self.speed = speed
    def update_speed(self, speed):
        self.speed = speed

# ResistInfo 객체 정의
class ResistInfo:
    def __init__(self, resist_hit=None, resist_penetrate=None, resist_slash=None):
        self.resist_hit = resist_hit
        self.resist_penetrate = resist_penetrate
        self.resist_slash = resist_slash

# ResistInfo 객체 정의
class StaggerThreshold:
    def __init__(self, first_stagger=None, second_stagger=None, third_stagger=None):
        self.first_stagger = first_stagger
        self.second_stagger = second_stagger
        self.third_stagger = third_stagger

class SkillSet:
    def __init__(self):

        self.skill_1_type = ""
        self.skill_2_type = ""
        self.skill_3_type = ""
        self.defense_skill_type = ""

        self.skill_1_sin_affinity = ""
        self.skill_2_sin_affinity = ""
        self.skill_3_sin_affinity = ""
        self.defense_skill_sin_affinity = ""

        self.skill_1_details = []
        self.skill_2_details = []
        self.skill_3_details = []
        self.defense_skill_details = []

    def set_skill_type(self, skill_code, skill_type, sin_affinity):
        if skill_code == "skill 1":
            self.skill_1_type = skill_type
            self.skill_1_sin_affinity = sin_affinity
        elif skill_code == "skill 2":
            self.skill_2_type = skill_type
            self.skill_2_sin_affinity = sin_affinity
        elif skill_code == "skill 3":
            self.skill_3_type = skill_type
            self.skill_3_sin_affinity = sin_affinity
        elif skill_code == "defense":
            self.defense_skill_type = skill_type
            self.defense_skill_sin_affinity = sin_affinity
    
    def add_skill(self, skill_code, skill):
        if skill_code == "skill 1":
            self.skill_1_details.append(skill)
            skill.skill_type = self.skill_1_type
            skill.sin_affinity = self.skill_1_sin_affinity
        elif skill_code == "skill 2":
            self.skill_2_details.append(skill)
            skill.skill_type = self.skill_2_type
            skill.sin_affinity = self.skill_2_sin_affinity
        elif skill_code == "skill 3":
            self.skill_3_details.append(skill)
            skill.skill_type = self.skill_3_type
            skill.sin_affinity = self.skill_3_sin_affinity
        elif skill_code == "defense":
            self.defense_skill_details.append(skill)
            skill.skill_type = self.defense_skill_type 
            skill.sin_affinity = self.defense_skill_sin_affinity

    def get_skill_by_name(self, skill_name):
        for skill_list in [self.skill_1_details, self.skill_2_details, self.skill_3_details, self.defense_skill_details]:
            for skill in skill_list:
                result = skill.is_skill_by_name(skill_name)
                if result:
                    return result
        return None

    def add_skill_uptie_info(self, skill_name, uptie_level, skill_power, coin_power, effect_by_condition):
        skill = self.get_skill_by_name(skill_name)
        skill.add_uptie_info(uptie_level, skill_power, coin_power, effect_by_condition)
        

class Skill:
    def __init__(self, skill_code, skill_name, coin_count, init_level, init_remain, atk_weight):
        self.skill_code = skill_code # "Skill 1"/"Skill 2"/"Skill 3"/"Defense" '''o'''
        self.skill_type = None
        self.sin_affinity = None
        self.skill_name = skill_name # 스킬 이름 '''o'''
        self.coin_count = coin_count # 동전 개수 '''o'''
        self.init_level = init_level # 스킬이 초기 배정받은 레벨, 인격 레벨에 - 1 한 값을 더해주면 스킬의 현 level을 구할 수 있다. '''o'''
        self.init_remain = init_remain #'''o'''
        self.atk_weight = atk_weight #공격 가중치 값이다. 방어스킬에는 없기 때문에 None으로 설정했다. '''o'''
        self.remain_count = init_remain # 실제 스킬이 작동하며 횟수가 차감되 나중에 모두 차감되면 다음 턴에 다시 초기횟수 리필된다.
        self.uptie_info = []  

    def is_skill_by_name(self, skill_name):
        if self.skill_name == skill_name:
            return self
        return None
        
    def add_uptie_info(self, uptie_level, skill_power, coin_power, effects_by_condition):
        self.uptie_info.append(UptieInfo(uptie_level, skill_power, coin_power, effects_by_condition))

class UptieInfo:
    def __init__(self, uptie_level, skill_power, coin_power, effects_by_condition):
        self.uptie_level = uptie_level   #'''o'''
        self.skill_power = skill_power   #'''o'''
        self.coin_power = coin_power     #'''o'''
        self.skill_effect = effects_by_condition

# 추상화된 EffectType 클래스
class EffectType:
    def __init__(self, name, description):
        self.name = name  # Effect 이름
        self.description = description  # Effect 설명

# Passive 클래스
class Passive(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)  # condition이 있는지 여부 확인

# SupportPassive 클래스
class SupportPassive(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)

# PanicType 클래스
class PanicType(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)

# Keyword 클래스
class Keyword:
    def __init__(self, keyword, explain=None, dispellable=None):
        self.keyword = keyword
        self.explain = explain
        self.dispellable = dispellable

      
class DataContainer:
    """크롤링 데이터를 관리하는 클래스"""
    def __init__(self):
        self.identities = []

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.level_max

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.uptie_max

    def update_identity_name(self, identity_id, identity_name):
        """특정 ID의 이름을 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_name(identity_name)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_identity_rarity(self, identity_id, rarity):
        """특정 ID의 희귀도를 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_rarity(rarity)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_skill(self, identity_id, skill_number, new_type):
        """
        특정 identity의 skill_type을 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_type: 새 skill_type
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_type = new_type
                elif skill_number == 2:
                    identity.skill_2_type = new_type
                elif skill_number == 3:
                    identity.skill_3_type = new_type
                elif skill_number == "defense":
                    identity.defense_type = new_type
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def update_sin_affinity(self, identity_id, skill_number, new_affinity):
        """
        특정 identity의 sin_affinity를 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_affinity: 새 sin_affinity
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_sin_affinity = new_affinity
                elif skill_number == 2:
                    identity.skill_2_sin_affinity = new_affinity
                elif skill_number == 3:
                    identity.skill_3_sin_affinity = new_affinity
                elif skill_number == "defense":
                    identity.defense_sin_affinity = new_affinity
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def add_identity(self, identity):
        """새로운 인격 데이터를 추가."""
        self.identities.append(identity)

    def add_level_stat(self, identity_id, level, hp, defense):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_level_stat(level, hp, defense)

    def add_uptie_stat(self, identity_id, uptie, speed):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_uptie_stat(uptie, speed)

    def add_resist_info(self, identity_id, resist_hit, resist_penetrate, resist_slash):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_resist_info(resist_hit, resist_penetrate, resist_slash)

    def add_stagger_threshold(self, identity_id, first_stagger, second_stagger, third_stagger):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_stagger_threshold(first_stagger, second_stagger, third_stagger)

    
    def add_skill(self, identity_id, skill_code, skill_name, coin_count, init_level, init_remain, atk_weight):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_code == "skill 1":
                    identity.skillset.set_skill_type(skill_code, identity.skill_1_type, identity.skill_1_sin_affinity)
                elif skill_code == "skill 2":
                    identity.skillset.set_skill_type(skill_code, identity.skill_2_type, identity.skill_2_sin_affinity)
                elif skill_code == "skill 3":
                    identity.skillset.set_skill_type(skill_code, identity.skill_3_type, identity.skill_3_sin_affinity)
                elif skill_code == "defense":
                    identity.skillset.set_skill_type(skill_code, identity.defense_type, identity.defense_sin_affinity)
                identity.add_skill(skill_code, skill_name, coin_count, init_level, init_remain, atk_weight)

    def add_skill_uptie_info(self, identity_id, skill_name, uptie_level, skill_power, coin_power, effects_by_condition):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_skill_uptie_info(skill_name, uptie_level, skill_power, coin_power, effects_by_condition)
        
    def display_identities(self):
        """Identity, ID, Name, Rarity 정보를 출력"""
        if not self.identities:
            print("No identities found.")
        else:
            # 데이터를 리스트로 변환
            table = []
            for idx, identity in enumerate(self.identities, 1):
                table.append([
                    idx, identity.identity_id, identity.name, identity.rarity,
                    identity.skill_1_type, identity.skill_1_sin_affinity,
                    identity.skill_2_type, identity.skill_2_sin_affinity,
                    identity.skill_3_type, identity.skill_3_sin_affinity,
                    identity.defense_type, identity.defense_sin_affinity
                ])
            
            # 테이블 헤더 정의
            headers = [
                "Index", "ID", "Name", "Rarity",
                "Skill_1 Type", "Skill_1 Sin", "Skill_2 Type", "Skill_2 Sin",
                "Skill_3 Type", "Skill_3 Sin", "Defense Type", "Defense Sin"
            ]
            
            # 테이블 출력
            print(tabulate(table, headers=headers, tablefmt="fancy_grid"))

    def export_to_csv(self, filename, fieldnames, data):
        """
        공통적으로 사용 가능한 CSV 내보내기 메서드
        :param filename: 내보낼 파일 이름 (예: 'identity.csv', 'skill.csv')
        :param fieldnames: CSV의 헤더 리스트
        :param data: CSV로 내보낼 데이터 (리스트 형태)
        """
        try:
            # 현재작업 디렉토 기준으 'csv'폴더 없으면 생성
            if not os.path.exists('./csv'):
                os.makedirs('./csv')
            # 파일 경로 설정 (상대경로 'csv' 폴더 안에 저장)
            file_path = os.path.join('./csv', filename)
            with open(file_path, mode='w', newline='', encoding='utf-8') as file:
                writer = csv.DictWriter(file, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(data)
            print(f"Data successfully exported to {filename}")
        except Exception as e:
            print(f"Error exporting to CSV: {e}")

    def get_identity_data(self):
        """
        Identity 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "ID": identity.identity_id,
                "Name": identity.name,
                "Rarity": identity.rarity,
                "Skill_1_Type": identity.skill_1_type,
                "Skill_1_Sin_Affinity": identity.skill_1_sin_affinity,
                "Skill_2_Type": identity.skill_2_type,
                "Skill_2_Sin_Affinity": identity.skill_2_sin_affinity,
                "Skill_3_Type": identity.skill_3_type,
                "Skill_3_Sin_Affinity": identity.skill_3_sin_affinity,
                "Defense_Type": identity.defense_type,
                "Defense_Sin_Affinity": identity.defense_sin_affinity,
            }
            for identity in self.identities
        ]
        
    def export_identities_data(self, filename="identities.csv"):
        """Identity 데이터를 CSV로 내보내는 메서드"""
        identity_fieldnames = [
            "ID", "Name", "Rarity",
            "Skill_1_Type", "Skill_1_Sin_Affinity",
            "Skill_2_Type", "Skill_2_Sin_Affinity",
            "Skill_3_Type", "Skill_3_Sin_Affinity",
            "Defense_Type", "Defense_Sin_Affinity"
        ]
        identity_data = self.get_identity_data()  # Identity 데이터 가져오기
        self.export_to_csv(filename, identity_fieldnames, identity_data)  # CSV로 내보내기

    def load_identities_by_id(self, filename, identity_ids):
        """지정된 identity_ids 목록에 해당하는 Identity 데이터를 CSV에서 읽어옴."""
        identity_ids_set = set(identity_ids) # 중복 제거를 위 Set 사용
        with open(filename, mode='r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                if row["ID"] in identity_ids_set:
                    identity = Identity(
                        identity_id=row["ID"],
                        name=row["Name"],
                        rarity=row["Rarity"],
                        skill_1_type=row["Skill_1_Type"],
                        skill_1_sin_affinity=row["Skill_1_Sin_Affinity"],
                        skill_2_type=row["Skill_2_Type"],
                        skill_2_sin_affinity=row["Skill_2_Sin_Affinity"],
                        skill_3_type=row["Skill_3_Type"],
                        skill_3_sin_affinity=row["Skill_3_Sin_Affinity"],
                        defense_type=row["Defense_Type"],
                        defense_sin_affinity=row["Defense_Sin_Affinity"]
                    )
                    self.identities.append(identity)

    def get_level_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Level": level_stat.level,
                "HP": level_stat.hp,
                "Defense": level_stat.defense,
            }
            for identity in self.identities
            for level_stat in identity.level_stats
        ]

    def export_level_stats_data(self, filename="levelstats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        level_stats_fieldnames = ["Identity_ID", "Level", "HP", "Defense"]
        level_stats_data = self.get_level_stats_data()  # level_stats 데이터 가져오기
        self.export_to_csv(filename, level_stats_fieldnames, level_stats_data)  # CSV로 내보내기

    def get_uptie_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Uptie": uptie_stat.uptie,
                "Speed": uptie_stat.speed,
            }
            for identity in self.identities
            for uptie_stat in identity.uptie_stats
        ]

    def export_uptie_stats_data(self, filename="Uptiestats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        uptie_stats_fieldnames = ["Identity_ID", "Uptie", "Speed"]
        uptie_stats_data = self.get_uptie_stats_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, uptie_stats_fieldnames, uptie_stats_data)  # CSV로 내보내기

    def get_resist_info_data(self):
        """
        모든 Identity의 resist_info 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Resist_Hit": identity.resist_info.resist_hit,
                "Resist_Penetrate": identity.resist_info.resist_penetrate,
                "Resist_Slash": identity.resist_info.resist_slash,
            }
            for identity in self.identities
        ]

    def export_resist_info_data(self, filename="ResistInfos.csv"):
        """ResistInfo 데이터를 CSV로 내보내는 메서드"""
        resist_infos_fieldnames = ["Identity_ID", "Resist_Hit", "Resist_Penetrate", "Resist_Slash"]
        resist_infos_data = self.get_resist_info_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, resist_infos_fieldnames, resist_infos_data)  # CSV로 내보내기

    def export_uptie_stats_data(self, filename="Uptiestats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        uptie_stats_fieldnames = ["Identity_ID", "Uptie", "Speed"]
        uptie_stats_data = self.get_uptie_stats_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, uptie_stats_fieldnames, uptie_stats_data)  # CSV로 내보내기

    def get_stagger_threshold_data(self):
        """
        모든 Identity의 stagger_threshold 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "First_Stagger": identity.stagger_threshold.first_stagger,
                "Second_Stagger": identity.stagger_threshold.second_stagger,
                "Third_Stagger": identity.stagger_threshold.third_stagger,
            }
            for identity in self.identities
        ]

    def export_stagger_threshold_data(self, filename="StaggerThresholds.csv"):
        """StaggerThresholds 데이터를 CSV로 내보내는 메서드"""
        stagger_thresholds_fieldnames = ["Identity_ID", "First_Stagger", "Second_Stagger", "Third_Stagger"]
        stagger_thresholds_data = self.get_stagger_threshold_data()  # stagger_thresholds 데이터 가져오기
        self.export_to_csv(filename, stagger_thresholds_fieldnames, stagger_thresholds_data)  # CSV로 내보내기

    def get_skillset_data(self):
        """모든 Identity의 skillset 데이터를 export_to_csv을 포맷으로 변환"""
        return [
            {
                "Identity_ID": identity.identity_id,
                "Skill_Code": skill.skill_code,
                "Skill_Type": skill.skill_type,
                "Sin_Affinity": skill.sin_affinity,
                "Skill_Name": skill.skill_name,
                "Coin_Count": skill.coin_count,
                "Init_Level": skill.init_level,
                "Init_Remain": skill.init_remain,
                "Atk_Weight": skill.atk_weight,
                "Uptie_Level": uptie_info.uptie_level,
                "Skill_Power": uptie_info.skill_power,
                "Coin_Power": uptie_info.coin_power,
                "Effect_Data": json.dumps(uptie_info.skill_effect)
            }
            for identity in self.identities
            for skill_list in [identity.skillset.skill_1_details, identity.skillset.skill_2_details, identity.skillset.skill_3_details, identity.skillset.defense_skill_details]
            for skill in skill_list
            for uptie_info in skill.uptie_info
        ]

    def export_identity_skillset_data(self, filename="IdentitiesSkills.csv"):
        """IdentitiesSkills 데이터를 CSV로 내보내는 메서드"""
        identities_skills_fieldnames = ["Identity_ID", "Skill_Code", "Skill_Type", "Sin_Affinity", "Skill_Name", "Coin_Count", "Init_Level", "Init_Remain", "Atk_Weight", "Uptie_Level", "Skill_Power", "Coin_Power", "Effect_Data"]
        identities_skills_data = self.get_skillset_data()
        self.export_to_csv(filename, identities_skills_fieldnames, identities_skills_data)
            


async def main():
    data_container = DataContainer()
    scraper = Scraper(container=data_container, base_url="https://limbus.kusoge.xyz", img_url="https://img.kusoge.xyz/limbus/img")
    await scraper.scrape()

await main() 

Added skill: skill 1:Enough is Enough
Updated Uptie for Enough is Enough (Uptie Level: 1): Skill Power: 3, Coin Power: +3 Skill Effect: {'[On Use]': ['Consume up to 50 keyword:Bloodfeast and gain 1 keyword:Hardblood for every 5 this unit consumed - If this unit failed to consume , gain 5 keyword:Bleed'], '[Coin 2 On Hit]': ['Inflict 1 keyword:Bleed']}
Updated Uptie for Enough is Enough (Uptie Level: 2): Skill Power: 3, Coin Power: +3 Skill Effect: {'[On Use]': ['Consume up to 50 keyword:Bloodfeast and gain 1 keyword:Hardblood for every 5 this unit consumed - If this unit failed to consume , gain 5 keyword:Bleed', 'If the target has 6 + keyword:Bleed , Coin Power + 1'], '[Coin 2 On Hit]': ['Inflict 2 keyword:Bleed']}
Updated Uptie for Enough is Enough (Uptie Level: 3): Skill Power: 3, Coin Power: +4 Skill Effect: {'[On Use]': ['Consume up to 50 keyword:Bloodfeast and gain 1 keyword:Hardblood for every 5 this unit consumed - If this unit failed to consume , gain 5 keyword:Bleed', 'If the

In [5]:
import os
import csv
import re
import json
import numpy as np
import pandas as pd
from playwright.async_api import async_playwright
from tabulate import tabulate

# Sin Color 이름 변환을 위한 매핑
resource_mapping = {
    "AZURE": "Gloom",
    "VIOLET": "Envy",
    "AMBER": "Sloth",
    "SHAMROCK": "Gluttony",
    "CRIMSON": "Wrath",
    "SCARLET": "Lust",
    "INDIGO": "Pride"
}

def convert_resource(resource_name):
    """Sin Color 이름을 변환하는 함수. 매핑되지 않은 경우 원래 이름을 반환."""
    return resource_mapping.get(resource_name, resource_name)

class Scraper:
    """웹 페이지 크롤링을 위한 클래스"""
    def __init__(self, container, base_url, img_url):
        self.container = container
        self.base_url = base_url
        self.img_url = img_url
        self.uptie_level = 1

    async def handle_request(self, route, request):
        """이미지와 폰트 리소스 로드를 차단."""
        if request.resource_type in ["image", "font"]:
            await route.abort()
        else:
            await route.continue_()
            
    async def scrape(self):
        """크롤링 메인 함수"""
        async with async_playwright() as p:
            browser = await p.chromium.launch(headless=True)
            page = await browser.new_page()
            await page.route("**/*", self.handle_request)
            # Step 1: /identity 페이지에서 기본 정보 크롤링
            await self.crawl_identity_page(page)

            # Step 2: /identity/{identity_id} 상세 페이지 크롤링
            #identity_ids_to_load = ["10104", "10310", "10710","10808", "10911", "11005"]
            #identity_ids_to_load = ["10104", "10911"]#,"10310", "10710","10808", "11005", "11210"]
            #identity_ids_to_load = ["10911","10310"]#, "10710","10808", "11005", "11210"]
            #self.container.load_identities_by_id('./csv/identities.csv', identity_ids_to_load)
            #await self.crawl_identity_details_for_all(page)
            #self.container.display_identities()
            self.container.export_identities_data("identities.csv")
            #self.container.export_level_stats_data("Levelstats.csv")
            #self.container.export_uptie_stats_data("Uptiestats.csv")
            #self.container.export_resist_info_data("ResistInfos.csv")
            #self.container.export_stagger_threshold_data("StaggerThresholds.csv")
            #self.container.export_identity_skillset_data("IdentitiesSkills.csv")
            page.set_default_timeout(10000)
            #await page.wait_for_timeout(100000)
            await browser.close()

    async def get_identity_id_from_row(self, row):
        """테이블 행(row)에서 인격 ID를 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_url = await identity_cell.get_attribute('href')
            identity_id = identity_url.split('/identity/')[-1]
            return identity_id
        return "Unknown"

    async def get_identity_name_from_row(self, row):
        """테이블 행(row)에서 인격 이름을 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_name = (await identity_cell.inner_text()).replace("\n", " ").strip()
            return identity_name
        return "Unknown"
        
    async def crawl_identity_page(self, page):
        """/identity 페이지로 이동하여 인격 기본 정보를 추출."""
        await page.goto(self.base_url + "/identity")
        await self.extract_identity_page(page)

    async def extract_identity_page(self, page):
        """/identity 페이지의 모든 테이블 행에서 인격 정보를 추출."""
        identity_rows = await page.query_selector_all('tr')
        if len(identity_rows) > 1:
            for idx, identity_row in enumerate(identity_rows):
                if idx == 0:
                    continue # 0번째요소이름을 보여주는 열은 건너뜀
                await self.scrape_identity_name_and_identity_id(identity_row)
                await self.extract_rarity(identity_row)
                for i in range(1, 4):  # 스킬1~3 정보 추출
                    await self.extract_atk_and_resource(identity_row, i)
                await self.extract_defense_and_resource(identity_row)

    async def scrape_identity_name_and_identity_id(self, row):
        """인격 번호와 이름을 추출하여 DataContainer에 저장."""
        try:
            identity_id = await self.get_identity_id_from_row(row)
            identity_name = await self.get_identity_name_from_row(row)
            identity = Identity(identity_id)
            self.container.add_identity(identity)
            self.container.update_identity_name(identity_id, identity_name)
        except Exception as e:
            print(f"Error extracting identity name and identity id: {e}")

    async def get_element_attribute(self, element, selector, attribute):
        """요소에서 지정된 속성(attribute)을 추출."""
        try:
            target_element = await element.query_selector(selector)
            if target_element:
                return await target_element.get_attribute(attribute)
        except Exception as e:
            print(f"Error getting element attribute: {e}")
        return None
    
    async def extract_rarity(self, row):
        """테이블 행(row)에서 희귀도를 추출하여 업데이트."""
        try:
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
                sibling = await parent.evaluate_handle('node => node.nextElementSibling')
                if sibling:
                    rarity_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/rarity_"]', 'src')
                    if rarity_img_src:
                        rarity_level = rarity_img_src.split('_')[-1].split('.')[0]  # rarity_1.png -> 1
                        identity_id = await self.get_identity_id_from_row(row)
                        self.container.update_identity_rarity(identity_id, rarity_level)
                    else:
                        print("Rarity image not found in the next sibling TD.")
        except Exception as e: 
            print(f"Error extracting rarity: {e}")

        return "Unknown"

    async def extract_atk_and_resource(self, row, skill_number):
        """ 공격 타입(ATK Type)과 자원(Attribute) 추출 함수 - 스킬 번호에 따라 처리 """
        try:
            #print(f"Attempting to locate skill{skill_number} type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 해당 스킬 번호에 맞는 형제로 이동
                sibling = parent
                for _ in range(skill_number + 1):  # 스킬 번호에 맞게 이동
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                atk_type, resource = "Unknown", "Unknown"
        
                if sibling:
                    # 공격 타입 이미지 (atk_type) 찾기
                    atk_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/atk_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if atk_type_img_src:
                        atk_type = atk_type_img_src.split('_')[-1].split('.')[0]  # atk_type_SLASH.png -> SLASH
                        #print(f"Extracted attack type: {atk_type}")
                        self.container.update_skill(identity_id, skill_number, atk_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted resource: {resource}")
                        
                        # 리소스를 변환
                        resource = convert_resource(resource)
                        #print(f"Converted resource: {resource}")
                        self.container.update_sin_affinity(identity_id, skill_number, resource)
                        
        except Exception as e:
            print(f"Error extracting skill{skill_number} type and resource: {e}")

    async def extract_defense_and_resource(self, row):
        """ 방어 타입(Def Type)과 자원(Attribute) 추출 함수 """
        try:
            #print("Attempting to locate defense type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 다섯 번째 형제 td로 이동 (방어 관련 정보가 있는 <td>)
                sibling = parent
                for _ in range(5):  # 방어 정보는 5번째 형제에 있음
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                def_type, def_resource = "Unknown", "Unknown"
        
                if sibling:
                    # 방어 타입 이미지 (def_type) 찾기
                    def_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/def_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if def_type_img_src:
                        def_type = def_type_img_src.split('_')[-1].split('.')[0]  # def_type_GUARD.png -> GUARD
                        #print(f"Extracted defense type: {def_type}")
                        self.container.update_skill(identity_id, 'defense', def_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        def_resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted defense resource: {def_resource}")
                        
                        # 방어 리소스를 변환
                        def_resource = convert_resource(def_resource)
                        #print(f"Converted defense resource: {def_resource}")
                        self.container.update_sin_affinity(identity_id, 'defense', def_resource)
 
        except Exception as e:
            print(f"Error extracting defense type and resource: {e}")

    async def crawl_identity_details_for_all(self, page):
        """모든 인격의 세부 정보를 크롤링."""
        for identity in self.container.identities:
            await page.wait_for_load_state("networkidle")
            await page.goto(self.base_url + "/identity/"+identity.identity_id)
            """ 레벨과 동기화와 관련 없는 참관타 저항은 밑에서 변경하는 것과 연관성이 적기 때문에 위에서 처리한다.
            print(f"Identity_id: {identity.identity_id}")
            results = []
            resist_type_imgs = [f"{self.img_url}/ui/atk_type_HIT.png",
                                f"{self.img_url}/ui/atk_type_PENETRATE.png",
                                f"{self.img_url}/ui/atk_type_SLASH.png"]
            for resist_type_img in resist_type_imgs:
                result = await self.extract_resist_type(page, resist_type_img, 1)
                results.append(result)
            self.container.add_resist_info(identity.identity_id, results[0], results[1], results[2])
            """
            """
            # Stagger Threshold 값 추출
            stagger_threshold = await self.extract_stagger_threshold_values(page)
            self.container.add_stagger_threshold(identity.identity_id, stagger_threshold[0], stagger_threshold[1], stagger_threshold[2])
            """
            await self.initialize_uptie_and_level(page)
            """ 레벨에 따른 level, hp, defense 다르게 되어 있는 것을 출력 확인 객체로 들어가서 csv로 옮기는 과정까지 진행
            for i in range(1, identity.level_max+1):
                await self.move_slider_to_value(page,i)
                await self.extract_level_status(page, identity.identity_id, i)
            """
            """ 동기화에 따른 speed 다르게 되어 있는 것을 출력 확인 객체로 들어가서 csv로 옮기는 과정까지 진행
            for i in range(1, identity.uptie_max+1):
                await self.uptie_click(page,i)
                await self.extract_uptie_status(page, identity.identity_id, i)
            """
            """
                인격에 대한 skill 정보를 가져오는 메소드 위에서 출력하는 csv파일로 인격이 가지고 있는 스킬 정보를 내보낸다.
            await self.crawl_skills(page, identity.identity_id)
            """
            
    async def initialize_uptie_and_level(self, page):
        await self.uptie_click(page, 1)
        await self.move_slider_to_value(page, 1)
        self.uptie_level = 1
        #return 1
    
    async def uptie_click(self, page, phase):
        fab_button_selector = ".q-btn.q-btn--fab"
        await page.wait_for_selector(fab_button_selector)
        await page.click(fab_button_selector)
        #print(f"[{setup_uptie_button}] 상위 동기화 버튼 클릭 완료")
        setup_uptie_button = f'a:has(img[src="{self.img_url}/ui/uptie_{phase}.png"])'
        await page.wait_for_selector(setup_uptie_button)
        await page.click(setup_uptie_button)
        #print(f"{phase} 동기화 버튼 클릭 완료")

    async def move_slider_to_value(self, page, target_value):
        """
        슬라이더를 주어진 값으로 이동시킵니다.
        target_value는 슬라이더의 최소값부터 최대값 사이여야 합니다.
        """
        slider_selector = ".q-slider"
        # 슬라이더의 범위 확인
        min_value = int(await page.get_attribute(slider_selector, "aria-valuemin"))
        max_value = int(await page.get_attribute(slider_selector, "aria-valuemax"))
    
        if target_value < min_value or target_value > max_value:
            raise ValueError(f"Target value {target_value} is out of bounds ({min_value}, {max_value})")
    
        # 슬라이더 thumb 요소와 위치 가져오기
        slider_thumb = await page.query_selector(f"{slider_selector} .q-slider__thumb")
        thumb_box = await slider_thumb.bounding_box()
        
        # 슬라이더의 현재 위치 및 목표 위치 계산
        track = await page.query_selector(f"{slider_selector} .q-slider__track")
        track_box = await track.bounding_box()
        
        # 목표 x 위치 계산 (슬라이더 트랙 내에서 비율 기반)
        target_percentage = (target_value - min_value) / (max_value - min_value)
        target_x = track_box["x"] + target_percentage * track_box["width"]
        current_x = thumb_box["x"] + thumb_box["width"] / 2
        y = thumb_box["y"] + thumb_box["height"] / 2
    
        # 슬라이더를 드래그하여 목표 위치로 이동
        await page.mouse.move(current_x, y)
        await page.mouse.down()
        await page.mouse.move(target_x, y, steps=10)  # 부드럽게 이동
        await page.mouse.up()
    
        #print(f"슬라이더를 {target_value}로 이동 완료.")
    
        # 슬라이더 값 확인 및 강제 이벤트 트리거
        await page.evaluate(
            f"""
            (slider) => {{
                slider.dispatchEvent(new Event('input'));  // oninput 이벤트 트리거
                slider.dispatchEvent(new Event('change'));  // onchange 이벤트 트리거
            }}
            """,
            await page.query_selector(slider_selector)
        )
    
        # 이동 후 값 확인
        current_value = await page.get_attribute(slider_selector, "aria-valuenow")
        if current_value != str(target_value):
            print(f"경고: 슬라이더 값이 예상과 다릅니다. 현재 값: {current_value}, 예상 값: {target_value}")

    async def get_second_status_element(self, page):
        """
        두 번째 'Status' 요소에 접근하여 반환하는 메소드
        """
        status_elements = await page.query_selector_all("text=Status")
        if len(status_elements) > 1:
            second_status_element = status_elements[1]
            #print("두 번째 'Status' 요소에 접근합니다.")
            return second_status_element
        else:
            #print("두 번째 'Status' 요소가 존재하지 않습니다.")
            return None

    async def extract_level_status(self, page, identity_id, level):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 레벨에 따라 HP와 Defense가 변한다.
        hp_value = await self.extract_hp(second_status_element)
        defense_value = await self.extract_defense(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Level={level}, HP={hp_value}, Defense={defense_value}")
        self.container.add_level_stat(identity_id,level, hp_value, defense_value)
    
    async def extract_hp(self, status_element):
        # HP 추출
        hp_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].children[0].nextElementSibling.textContent.trim()"
        )
        return hp_value
    
    async def extract_defense(self, status_element):
        # Defense 추출
        defense_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return defense_value

    async def extract_uptie_status(self, page, identity_id, uptie):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 uptie에 따라 speed는 변한다.
        speed_value = await self.extract_speed(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Uptie={uptie}, Speed={speed_value}")
        self.container.add_uptie_stat(identity_id,uptie, speed_value)

    async def extract_speed(self, status_element):
        # Speed 추출
        speed_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return speed_value

    async def extract_resist_type(self, page, image_src, depth):
        # 이미지 요소 찾기
        #print(image_src)
        image = await page.query_selector(f'img[src="{image_src}"]')
        #print(image)
        if not image:
            return "Unknown"
    
        # 이미지의 부모 요소를 찾아서 올라감
        parent = image
        for _ in range(depth):
            parent = await parent.evaluate_handle('node => node.parentElement')
            if not parent:
                return "Unknown"
    
        # 부모의 형제 요소 찾기
        sibling = await parent.evaluate_handle('node => node.nextElementSibling')
        
        if sibling:
            value = await sibling.evaluate('node => node.textContent')
            if value:
                return value.strip()
        
        return "Unknown"

    async def extract_stagger_threshold_values(self, page):
        # 'div.row.center.flex-center.q-pt-xs' 요소들 찾기
        elements = await page.query_selector_all('div.row.center.flex-center.q-pt-xs')
        
        # 각 요소의 텍스트 내용 출력하여 'Stagger Threshold' 요소를 찾기
        for element in elements:
            text = await element.text_content()
            
            if text and "Stagger Threshold" in text:
                # 'Stagger Threshold'가 있는 요소를 찾았다면, 그 이후 형제 요소들에서 값을 추출
                parent = await element.evaluate_handle('node => node.parentElement')
                sibling_elements = await parent.query_selector_all('div.row.justify-evenly div')  # Threshold 값이 들어있는 div들
                
                stagger_values = []
                for sibling in sibling_elements:
                    text_content = await sibling.text_content()
                    match = re.search(r'(\d+%)', text_content)
                    if match:
                        stagger_values.append(match.group(1))  # 퍼센트 값만 리스트에 추가
                while len(stagger_values) < 3:
                    stagger_values.append(np.nan)
                #print(stagger_values)
                return stagger_values  # 추출된 값(또는 np.nan 포함)을 반환
        
        return [np.nan, np.nan, np.nan]  # 만약 'Stagger Threshold'를 찾지 못한 경우 기본값 반환

    async def crawl_skills(self, page, identity_id):
        await self.crawl_skill(page, identity_id, "skill 1")
        await self.crawl_skill(page, identity_id, "skill 2")
        await self.crawl_skill(page, identity_id, "skill 3")
        await self.crawl_skill(page, identity_id, "defense")


    async def crawl_skill(self, page, identity_id, skill_type):
        skills_elements = await page.query_selector_all('div.skilltag')
        for skill_element in skills_elements:
            skill_text = await skill_element.text_content()
            if skill_text.strip().lower() == skill_type:
                await self.process_skill(skill_element, page, identity_id, skill_type)
        
    async def process_skill(self, skill_element, page, identity_id, skill_type):
        # skill_type이 "skill 3"일 경우 uptie_value가 3 이상일 때만 정보를 저장
        for uptie_value in range(1, 5):  # Uptie levels 1 to 4
            # "skill 3"은 uptie_value가 3 이상일 때만 처리
            if skill_type == "skill 3" and uptie_value < 3:
                continue  # skip if uptie_value is less than 3 for skill 3
    
            # uptie_level에 맞게 클릭
            await self.uptie_click(page, uptie_value)
            if skill_type != "skill 3" and uptie_value > 1:
                # Uptie 정보 갱신
                await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
                continue
            elif skill_type == "skill 3" and uptie_value > 3:
                # Uptie 정보 갱신
                await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
                continue
            # 스킬의 세부 정보를 추출
            skill_details = await self.extract_skill_details(skill_element, skill_type)
            skill_name = skill_details["skill_name"]
    
            # 스킬 정보를 저장
            self.container.add_skill(
                identity_id, skill_type, 
                skill_name, skill_details["coin_count"],
                skill_details["init_level"], skill_details["init_remain"], 
                skill_details["atk_weight"]
            )
            print(f"Added skill: {skill_type}:{skill_name}")
    
            # Uptie 정보 갱신
            await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
    
    async def extract_skill_details(self, skill_element, skill_type):
        # 스킬 세부 정보를 추출하는 함수
        skill_name = await self.skill_name(skill_element)
        coin_count = await self.coin_count(skill_element, ".col-auto > .column > .row")
        init_level = await self.init_level(skill_element)
        init_remain = await self.init_remain(skill_element, skill_type, "div.row.justify-start > div.attr.flex.flex-center > div.q-px-sm")
        atk_weight = await self.atk_weight(skill_element, skill_type)
        return {
            "skill_name": skill_name,
            "coin_count": coin_count,
            "init_level": init_level,
            "init_remain": init_remain,
            "atk_weight": atk_weight
        }
    
    async def update_uptie_info(self, page, skill_element, identity_id, skill_name, skill_type, uptie_value):
        # Uptie 정보를 추출하고 업데이트하는 함수
        uptie_info = await self.extract_uptie_info(skill_element)
        effect_data = await self.skill_effects(skill_element)
        self.container.add_skill_uptie_info(
            identity_id, skill_name, 
            uptie_value, uptie_info["skill_power"], uptie_info["coin_power"], 
            effect_data.get("effectsByCondition")
        )
        print(
            f"Updated Uptie for {skill_name} (Uptie Level: {uptie_value}): "
            f"Skill Power: {uptie_info['skill_power']}, Coin Power: {uptie_info['coin_power']} "
            f"Skill Effect: {effect_data.get("effectsByCondition")}"
        )

    async def extract_uptie_info(self, skill_element):
        skill_data = await self.skill_and_coin_power(skill_element)
        return {
            "skill_power": skill_data.get("skillPower"),
            "coin_power": skill_data.get("coinPower")
        }

    async def skill_and_coin_power(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                // 조부모로 이동
                const grandparent = node.parentElement.parentElement;
                // 이전 형제 탐색
                const previousSibling = grandparent.previousElementSibling;
                if (!previousSibling) return {{ skillPower: null, coinPower: null }}; // 이전 형제가 없으면 기본값 반환
                // 이전 형제 내부의 .row.justify-around > div > div 탐색
                const targetDivs = previousSibling.querySelectorAll(".row.justify-around > div");
                if (targetDivs.length < 3) return {{ skillPower: null, coinPower: null }}; // 배열 길이 확인
                // 1st div: 스킬 위력
                const skillPower = targetDivs[0]?.querySelector('div')?.textContent?.trim() || null; 
                // 3rd div: 코인 파워 값 (3번째 div에서 텍스트 추출)
                const coinPower = targetDivs[2]?.textContent?.trim() || null;
                return {{ skillPower, coinPower }};
            }}
        """)


    async def coin_count(self, skill_element, hierarchy_selector):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector('{hierarchy_selector}');
                if (!targetElement) return 0;
                return targetElement.childElementCount;
            }}
        """)

    async def init_remain(self, skill_element, skill_type, hierarchy_selector):
        if skill_type == "defense":
            return -1
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector('{hierarchy_selector}');
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def skill_name(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector(".block");
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def init_level(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector("div.text-bold.text-h6.h6.q-pl-xs.q-pr-md");
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def atk_weight(self, skill_element, skill_type):
        if skill_type == "defense":
            return 0
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector("div.text-bold.text-h6.h6.q-pl-xs.q-pr-md");
                const doubleNextSibling = targetElement.nextElementSibling.nextElementSibling;
                if (!doubleNextSibling) return 0;
                return doubleNextSibling.textContent?.trim();
            }}
        """)

    async def skill_effects(self, skill_element):
        return await skill_element.evaluate("""
            node => {
                const parent = node.parentElement;
                const doubleNextSibling = parent.nextElementSibling?.nextElementSibling;
                if (!doubleNextSibling) return null;
    
                // doubleNextSibling에서 첫 번째 div.col 요소 찾기
                const textNode = doubleNextSibling.querySelector("div.col");
                if (!textNode) return null;
    
                const results = {
                    keywords: [], // 키워드 저장
                    conditions: [], // 조건 저장
                    effectsByCondition: {} // 조건별 효과
                };
    
                // 1. 조건 추출 (대괄호 텍스트)
                const conditionElements = textNode.querySelectorAll("span");
                conditionElements.forEach(span => {
                    let text = span.textContent?.trim();
                    if (text?.startsWith("[") && text?.endsWith("]")) {
                        // 부모의 전형제 노드 탐색
                        const sibling = span.parentElement.previousElementSibling;
                        if (sibling) {
                            const emblemImage = sibling.querySelector("img.emblem");
                            console.log("Sibling element found:", sibling); // 디버깅용
                            if (emblemImage) {
                                const src = emblemImage.getAttribute("src");
                                console.log("Image source:", src); // 디버깅용
                                const match = src.match(/ui_num_(\\d+)\\.png$/); // 정규식 수정
                                if (match) {
                                    const number = match[1]; // 숫자 추출
                                    text = `[Coin ${number} ${text.replace("[", "").replace("]", "")}]`;
                                    console.log("Updated condition:", text); // 디버깅용
                                }
                            }
                        }
                        results.conditions.push(text);
                        results.effectsByCondition[text] = []; // 조건별 효과를 초기화
                    }
                });
    
                // 2. 키워드 추출
                const keywordElements = textNode.querySelectorAll('a[href^="/en/keyword/"]');
                keywordElements.forEach(a => {
                    const span = a.querySelector('span.q-btn__content.text-center.col.items-center.q-anchor--skip.justify-center.row>span');
                    if (span) {
                        const keywordText = span.textContent.trim();
                        if (keywordText && !results.keywords.includes(keywordText)) {
                            results.keywords.push(keywordText); // 키워드 저장
                        }
                    }
                });
    
                // 3. 효과 추출
                const effectElements = textNode.querySelectorAll("span");
                let currentCondition = null;
                let currentEffect = []; // 초기화
                effectElements.forEach(span => {
                    let text = span.textContent?.trim();
                    if (!text) return;
    
                    // 조건 발견 시 currentCondition 변경
                    if (text.startsWith("[") && text.endsWith("]")) {
                        // 이전 조건의 효과 저장
                        if (currentCondition && currentEffect.length > 0) {
                            // 부모의 전형제 노드 탐색
                            const sibling = span.parentElement.previousElementSibling;
                            if (sibling) {
                                const emblemImage = sibling.querySelector("img.emblem");
                                if (emblemImage) {
                                    const src = emblemImage.getAttribute("src");
                                    const match = src.match(/ui_num_(\\d+)\\.png$/); // 정규식 수정
                                    if (match) {
                                        const number = match[1]; // 숫자 추출
                                        text = `[Coin ${number} ${text.replace("[", "").replace("]", "")}]`;
                                    }
                                }
                            }
                            results.effectsByCondition[currentCondition].push(currentEffect.join(' '));
                            currentEffect = []; // 초기화
                        }
                        currentCondition = text;
                        return;
                    }
    
                    // 현재 조건의 효과 처리
                    if (currentCondition) {
                        if (results.keywords.includes(text)) {
                            const keywordEffect = 'keyword:' + text;
                            if (!currentEffect.includes(keywordEffect)) {
                                currentEffect.push(keywordEffect);
                            }
                        } else {
                            currentEffect.push(text);
                        }
                    }
                });
    
                // 마지막 효과 저장
                if (currentCondition && currentEffect.length > 0) {
                    results.effectsByCondition[currentCondition].push(currentEffect.join(' '));
                }
    
                return results;
            }
        """)



class Identity:
    """인격 정보를 저장하는 클래스"""
    def __init__(self, identity_id, name=None, rarity=None,\
                 skill_1_type=None, skill_1_sin_affinity=None,\
                 skill_2_type=None, skill_2_sin_affinity=None,\
                 skill_3_type=None, skill_3_sin_affinity=None,\
                 defense_type=None, defense_sin_affinity=None):
        self.identity_id = identity_id
        self.name = name
        self.rarity = rarity
        # 선언만 하는 부분 
        self.skill_1_type = skill_1_type 
        self.skill_1_sin_affinity = skill_1_sin_affinity 
        self.skill_2_type = skill_2_type 
        self.skill_2_sin_affinity = skill_2_sin_affinity 
        self.skill_3_type = skill_3_type 
        self.skill_3_sin_affinity = skill_3_sin_affinity 
        self.defense_type = defense_type 
        self.defense_sin_affinity = defense_sin_affinity
        self.level_max = 50
        self.uptie_max = 4
        self.level_stats = []  # LevelStat 객체 리스트
        self.uptie_stats = [] # UptieStat 객체 리스트
        self.resist_info = None
        self.stagger_threshold = None
        self.skillset = SkillSet()  # Skill정보를 담고 있는 skillset 객체 
        self.passive_infos = []  # 패시브 객체를 담고 있는 배열
        self.support_passive_infos = [] # 서포트 패시 객체를 담고 있는 배열
        self.panic_type = [] # 혼란상태 발생시 나타나는 효과를 담고 있는 객체 배열

    def update_name(self, name):
        self.name = name

    def update_rarity(self, rarity):
        self.rarity = rarity

    def add_level_stat(self, level, hp, defense):
        self.level_stats.append(LevelStat(level, hp, defense))

    def add_uptie_stat(self, uptie, speed):
        self.uptie_stats.append(UptieStat(uptie, speed))

    def add_resist_info(self, resist_hit, resist_penetrate, resist_slash):
        self.resist_info = ResistInfo(resist_hit, resist_penetrate, resist_slash)

    def add_stagger_threshold(self, first_stagger, second_stagger, third_stagger):
        self.stagger_threshold = StaggerThreshold(first_stagger, second_stagger, third_stagger)

    def add_skill(self, skill_code, skill_name, coin_count, init_level, init_remain, atk_weight):
        self.skillset.add_skill(skill_code, Skill(skill_code, skill_name, coin_count, init_level, init_remain, atk_weight))
        
    def add_skill_uptie_info(self, skill_name, uptie_level, skill_power, coin_power, effect_by_condition):
        self.skillset.add_skill_uptie_info(skill_name, uptie_level, skill_power, coin_power, effect_by_condition)
        
    def update_details(self, page):
        # 세부 정보 업데이트 로직 (가정)
        pass
    #def init_uptie(self, page):
    #    init_uptie = 

# LevelStat 객체 정의
class LevelStat:
    def __init__(self, level, hp=None, defense=None):
        self.level = level
        self.hp = hp
        self.defense = defense
    def update_level(self, level):
        self.level = level
    def update_hp(self, hp):
        self.hp = hp
    def update_defense(self, defense):
        self.defense = defense

# UptieStat 객체 정의
class UptieStat:
    def __init__(self, uptie, speed=None):
        self.uptie = uptie
        self.speed = speed
    def update_speed(self, speed):
        self.speed = speed

# ResistInfo 객체 정의
class ResistInfo:
    def __init__(self, resist_hit=None, resist_penetrate=None, resist_slash=None):
        self.resist_hit = resist_hit
        self.resist_penetrate = resist_penetrate
        self.resist_slash = resist_slash

# ResistInfo 객체 정의
class StaggerThreshold:
    def __init__(self, first_stagger=None, second_stagger=None, third_stagger=None):
        self.first_stagger = first_stagger
        self.second_stagger = second_stagger
        self.third_stagger = third_stagger

class SkillSet:
    def __init__(self):

        self.skill_1_type = ""
        self.skill_2_type = ""
        self.skill_3_type = ""
        self.defense_skill_type = ""

        self.skill_1_sin_affinity = ""
        self.skill_2_sin_affinity = ""
        self.skill_3_sin_affinity = ""
        self.defense_skill_sin_affinity = ""

        self.skill_1_details = []
        self.skill_2_details = []
        self.skill_3_details = []
        self.defense_skill_details = []

    def set_skill_type(self, skill_code, skill_type, sin_affinity):
        if skill_code == "skill 1":
            self.skill_1_type = skill_type
            self.skill_1_sin_affinity = sin_affinity
        elif skill_code == "skill 2":
            self.skill_2_type = skill_type
            self.skill_2_sin_affinity = sin_affinity
        elif skill_code == "skill 3":
            self.skill_3_type = skill_type
            self.skill_3_sin_affinity = sin_affinity
        elif skill_code == "defense":
            self.defense_skill_type = skill_type
            self.defense_skill_sin_affinity = sin_affinity
    
    def add_skill(self, skill_code, skill):
        if skill_code == "skill 1":
            self.skill_1_details.append(skill)
            skill.skill_type = self.skill_1_type
            skill.sin_affinity = self.skill_1_sin_affinity
        elif skill_code == "skill 2":
            self.skill_2_details.append(skill)
            skill.skill_type = self.skill_2_type
            skill.sin_affinity = self.skill_2_sin_affinity
        elif skill_code == "skill 3":
            self.skill_3_details.append(skill)
            skill.skill_type = self.skill_3_type
            skill.sin_affinity = self.skill_3_sin_affinity
        elif skill_code == "defense":
            self.defense_skill_details.append(skill)
            skill.skill_type = self.defense_skill_type 
            skill.sin_affinity = self.defense_skill_sin_affinity

    def get_skill_by_name(self, skill_name):
        for skill_list in [self.skill_1_details, self.skill_2_details, self.skill_3_details, self.defense_skill_details]:
            for skill in skill_list:
                result = skill.is_skill_by_name(skill_name)
                if result:
                    return result
        return None

    def add_skill_uptie_info(self, skill_name, uptie_level, skill_power, coin_power, effect_by_condition):
        skill = self.get_skill_by_name(skill_name)
        skill.add_uptie_info(uptie_level, skill_power, coin_power, effect_by_condition)
        

class Skill:
    def __init__(self, skill_code, skill_name, coin_count, init_level, init_remain, atk_weight):
        self.skill_code = skill_code # "Skill 1"/"Skill 2"/"Skill 3"/"Defense" '''o'''
        self.skill_type = None
        self.sin_affinity = None
        self.skill_name = skill_name # 스킬 이름 '''o'''
        self.coin_count = coin_count # 동전 개수 '''o'''
        self.init_level = init_level # 스킬이 초기 배정받은 레벨, 인격 레벨에 - 1 한 값을 더해주면 스킬의 현 level을 구할 수 있다. '''o'''
        self.init_remain = init_remain #'''o'''
        self.atk_weight = atk_weight #공격 가중치 값이다. 방어스킬에는 없기 때문에 None으로 설정했다. '''o'''
        self.remain_count = init_remain # 실제 스킬이 작동하며 횟수가 차감되 나중에 모두 차감되면 다음 턴에 다시 초기횟수 리필된다.
        self.uptie_info = []  

    def is_skill_by_name(self, skill_name):
        if self.skill_name == skill_name:
            return self
        return None
        
    def add_uptie_info(self, uptie_level, skill_power, coin_power, effects_by_condition):
        self.uptie_info.append(UptieInfo(uptie_level, skill_power, coin_power, effects_by_condition))

class UptieInfo:
    def __init__(self, uptie_level, skill_power, coin_power, effects_by_condition):
        self.uptie_level = uptie_level   #'''o'''
        self.skill_power = skill_power   #'''o'''
        self.coin_power = coin_power     #'''o'''
        self.skill_effect = effects_by_condition

# 추상화된 EffectType 클래스
class EffectType:
    def __init__(self, name, description):
        self.name = name  # Effect 이름
        self.description = description  # Effect 설명

# Passive 클래스
class Passive(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)  # condition이 있는지 여부 확인

# SupportPassive 클래스
class SupportPassive(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)

# PanicType 클래스
class PanicType(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)

# Keyword 클래스
class Keyword:
    def __init__(self, keyword, explain=None, dispellable=None):
        self.keyword = keyword
        self.explain = explain
        self.dispellable = dispellable

      
class DataContainer:
    """크롤링 데이터를 관리하는 클래스"""
    def __init__(self):
        self.identities = []

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.level_max

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.uptie_max

    def update_identity_name(self, identity_id, identity_name):
        """특정 ID의 이름을 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_name(identity_name)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_identity_rarity(self, identity_id, rarity):
        """특정 ID의 희귀도를 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_rarity(rarity)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_skill(self, identity_id, skill_number, new_type):
        """
        특정 identity의 skill_type을 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_type: 새 skill_type
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_type = new_type
                elif skill_number == 2:
                    identity.skill_2_type = new_type
                elif skill_number == 3:
                    identity.skill_3_type = new_type
                elif skill_number == "defense":
                    identity.defense_type = new_type
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def update_sin_affinity(self, identity_id, skill_number, new_affinity):
        """
        특정 identity의 sin_affinity를 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_affinity: 새 sin_affinity
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_sin_affinity = new_affinity
                elif skill_number == 2:
                    identity.skill_2_sin_affinity = new_affinity
                elif skill_number == 3:
                    identity.skill_3_sin_affinity = new_affinity
                elif skill_number == "defense":
                    identity.defense_sin_affinity = new_affinity
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def add_identity(self, identity):
        """새로운 인격 데이터를 추가."""
        self.identities.append(identity)

    def add_level_stat(self, identity_id, level, hp, defense):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_level_stat(level, hp, defense)

    def add_uptie_stat(self, identity_id, uptie, speed):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_uptie_stat(uptie, speed)

    def add_resist_info(self, identity_id, resist_hit, resist_penetrate, resist_slash):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_resist_info(resist_hit, resist_penetrate, resist_slash)

    def add_stagger_threshold(self, identity_id, first_stagger, second_stagger, third_stagger):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_stagger_threshold(first_stagger, second_stagger, third_stagger)

    
    def add_skill(self, identity_id, skill_code, skill_name, coin_count, init_level, init_remain, atk_weight):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_code == "skill 1":
                    identity.skillset.set_skill_type(skill_code, identity.skill_1_type, identity.skill_1_sin_affinity)
                elif skill_code == "skill 2":
                    identity.skillset.set_skill_type(skill_code, identity.skill_2_type, identity.skill_2_sin_affinity)
                elif skill_code == "skill 3":
                    identity.skillset.set_skill_type(skill_code, identity.skill_3_type, identity.skill_3_sin_affinity)
                elif skill_code == "defense":
                    identity.skillset.set_skill_type(skill_code, identity.defense_type, identity.defense_sin_affinity)
                identity.add_skill(skill_code, skill_name, coin_count, init_level, init_remain, atk_weight)

    def add_skill_uptie_info(self, identity_id, skill_name, uptie_level, skill_power, coin_power, effects_by_condition):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_skill_uptie_info(skill_name, uptie_level, skill_power, coin_power, effects_by_condition)
        
    def display_identities(self):
        """Identity, ID, Name, Rarity 정보를 출력"""
        if not self.identities:
            print("No identities found.")
        else:
            # 데이터를 리스트로 변환
            table = []
            for idx, identity in enumerate(self.identities, 1):
                table.append([
                    idx, identity.identity_id, identity.name, identity.rarity,
                    identity.skill_1_type, identity.skill_1_sin_affinity,
                    identity.skill_2_type, identity.skill_2_sin_affinity,
                    identity.skill_3_type, identity.skill_3_sin_affinity,
                    identity.defense_type, identity.defense_sin_affinity
                ])
            
            # 테이블 헤더 정의
            headers = [
                "Index", "ID", "Name", "Rarity",
                "Skill_1 Type", "Skill_1 Sin", "Skill_2 Type", "Skill_2 Sin",
                "Skill_3 Type", "Skill_3 Sin", "Defense Type", "Defense Sin"
            ]
            
            # 테이블 출력
            print(tabulate(table, headers=headers, tablefmt="fancy_grid"))

    def export_to_csv(self, filename, fieldnames, data):
        """
        공통적으로 사용 가능한 CSV 내보내기 메서드
        :param filename: 내보낼 파일 이름 (예: 'identity.csv', 'skill.csv')
        :param fieldnames: CSV의 헤더 리스트
        :param data: CSV로 내보낼 데이터 (리스트 형태)
        """
        try:
            # 현재작업 디렉토 기준으 'csv'폴더 없으면 생성
            if not os.path.exists('./csv'):
                os.makedirs('./csv')
            # 파일 경로 설정 (상대경로 'csv' 폴더 안에 저장)
            file_path = os.path.join('./csv', filename)
            with open(file_path, mode='w', newline='', encoding='utf-8') as file:
                writer = csv.DictWriter(file, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(data)
            print(f"Data successfully exported to {filename}")
        except Exception as e:
            print(f"Error exporting to CSV: {e}")

    def get_identity_data(self):
        """
        Identity 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "ID": identity.identity_id,
                "Name": identity.name,
                "Rarity": identity.rarity,
                "Skill_1_Type": identity.skill_1_type,
                "Skill_1_Sin_Affinity": identity.skill_1_sin_affinity,
                "Skill_2_Type": identity.skill_2_type,
                "Skill_2_Sin_Affinity": identity.skill_2_sin_affinity,
                "Skill_3_Type": identity.skill_3_type,
                "Skill_3_Sin_Affinity": identity.skill_3_sin_affinity,
                "Defense_Type": identity.defense_type,
                "Defense_Sin_Affinity": identity.defense_sin_affinity,
            }
            for identity in self.identities
        ]
        
    def export_identities_data(self, filename="identities.csv"):
        """Identity 데이터를 CSV로 내보내는 메서드"""
        identity_fieldnames = [
            "ID", "Name", "Rarity",
            "Skill_1_Type", "Skill_1_Sin_Affinity",
            "Skill_2_Type", "Skill_2_Sin_Affinity",
            "Skill_3_Type", "Skill_3_Sin_Affinity",
            "Defense_Type", "Defense_Sin_Affinity"
        ]
        identity_data = self.get_identity_data()  # Identity 데이터 가져오기
        self.export_to_csv(filename, identity_fieldnames, identity_data)  # CSV로 내보내기

    def load_identities_by_id(self, filename, identity_ids):
        """지정된 identity_ids 목록에 해당하는 Identity 데이터를 CSV에서 읽어옴."""
        identity_ids_set = set(identity_ids) # 중복 제거를 위 Set 사용
        with open(filename, mode='r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                if row["ID"] in identity_ids_set:
                    identity = Identity(
                        identity_id=row["ID"],
                        name=row["Name"],
                        rarity=row["Rarity"],
                        skill_1_type=row["Skill_1_Type"],
                        skill_1_sin_affinity=row["Skill_1_Sin_Affinity"],
                        skill_2_type=row["Skill_2_Type"],
                        skill_2_sin_affinity=row["Skill_2_Sin_Affinity"],
                        skill_3_type=row["Skill_3_Type"],
                        skill_3_sin_affinity=row["Skill_3_Sin_Affinity"],
                        defense_type=row["Defense_Type"],
                        defense_sin_affinity=row["Defense_Sin_Affinity"]
                    )
                    self.identities.append(identity)

    def get_level_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Level": level_stat.level,
                "HP": level_stat.hp,
                "Defense": level_stat.defense,
            }
            for identity in self.identities
            for level_stat in identity.level_stats
        ]

    def export_level_stats_data(self, filename="levelstats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        level_stats_fieldnames = ["Identity_ID", "Level", "HP", "Defense"]
        level_stats_data = self.get_level_stats_data()  # level_stats 데이터 가져오기
        self.export_to_csv(filename, level_stats_fieldnames, level_stats_data)  # CSV로 내보내기

    def get_uptie_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Uptie": uptie_stat.uptie,
                "Speed": uptie_stat.speed,
            }
            for identity in self.identities
            for uptie_stat in identity.uptie_stats
        ]

    def export_uptie_stats_data(self, filename="Uptiestats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        uptie_stats_fieldnames = ["Identity_ID", "Uptie", "Speed"]
        uptie_stats_data = self.get_uptie_stats_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, uptie_stats_fieldnames, uptie_stats_data)  # CSV로 내보내기

    def get_resist_info_data(self):
        """
        모든 Identity의 resist_info 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Resist_Hit": identity.resist_info.resist_hit,
                "Resist_Penetrate": identity.resist_info.resist_penetrate,
                "Resist_Slash": identity.resist_info.resist_slash,
            }
            for identity in self.identities
        ]

    def export_resist_info_data(self, filename="ResistInfos.csv"):
        """ResistInfo 데이터를 CSV로 내보내는 메서드"""
        resist_infos_fieldnames = ["Identity_ID", "Resist_Hit", "Resist_Penetrate", "Resist_Slash"]
        resist_infos_data = self.get_resist_info_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, resist_infos_fieldnames, resist_infos_data)  # CSV로 내보내기

    def export_uptie_stats_data(self, filename="Uptiestats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        uptie_stats_fieldnames = ["Identity_ID", "Uptie", "Speed"]
        uptie_stats_data = self.get_uptie_stats_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, uptie_stats_fieldnames, uptie_stats_data)  # CSV로 내보내기

    def get_stagger_threshold_data(self):
        """
        모든 Identity의 stagger_threshold 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "First_Stagger": identity.stagger_threshold.first_stagger,
                "Second_Stagger": identity.stagger_threshold.second_stagger,
                "Third_Stagger": identity.stagger_threshold.third_stagger,
            }
            for identity in self.identities
        ]

    def export_stagger_threshold_data(self, filename="StaggerThresholds.csv"):
        """StaggerThresholds 데이터를 CSV로 내보내는 메서드"""
        stagger_thresholds_fieldnames = ["Identity_ID", "First_Stagger", "Second_Stagger", "Third_Stagger"]
        stagger_thresholds_data = self.get_stagger_threshold_data()  # stagger_thresholds 데이터 가져오기
        self.export_to_csv(filename, stagger_thresholds_fieldnames, stagger_thresholds_data)  # CSV로 내보내기

    def get_skillset_data(self):
        """모든 Identity의 skillset 데이터를 export_to_csv을 포맷으로 변환"""
        return [
            {
                "Identity_ID": identity.identity_id,
                "Skill_Code": skill.skill_code,
                "Skill_Type": skill.skill_type,
                "Sin_Affinity": skill.sin_affinity,
                "Skill_Name": skill.skill_name,
                "Coin_Count": skill.coin_count,
                "Init_Level": skill.init_level,
                "Init_Remain": skill.init_remain,
                "Atk_Weight": skill.atk_weight,
                "Uptie_Level": uptie_info.uptie_level,
                "Skill_Power": uptie_info.skill_power,
                "Coin_Power": uptie_info.coin_power,
                "Effect_Data": json.dumps(uptie_info.skill_effect)
            }
            for identity in self.identities
            for skill_list in [identity.skillset.skill_1_details, identity.skillset.skill_2_details, identity.skillset.skill_3_details, identity.skillset.defense_skill_details]
            for skill in skill_list
            for uptie_info in skill.uptie_info
        ]

    def export_identity_skillset_data(self, filename="IdentitiesSkills.csv"):
        """IdentitiesSkills 데이터를 CSV로 내보내는 메서드"""
        identities_skills_fieldnames = ["Identity_ID", "Skill_Code", "Skill_Type", "Sin_Affinity", "Skill_Name", "Coin_Count", "Init_Level", "Init_Remain", "Atk_Weight", "Uptie_Level", "Skill_Power", "Coin_Power", "Effect_Data"]
        identities_skills_data = self.get_skillset_data()
        self.export_to_csv(filename, identities_skills_fieldnames, identities_skills_data)
            


async def main():
    data_container = DataContainer()
    scraper = Scraper(container=data_container, base_url="https://limbus.kusoge.xyz", img_url="https://img.kusoge.xyz/limbus/img")
    await scraper.scrape()

await main() 

Future exception was never retrieved
future: <Future finished exception=TargetClosedError('Target page, context or browser has been closed')>
playwright._impl._errors.TargetClosedError: Target page, context or browser has been closed


CancelledError: 

In [13]:
import os
import csv
import re
import json
import numpy as np
import pandas as pd
from playwright.async_api import async_playwright
from tabulate import tabulate

# Sin Color 이름 변환을 위한 매핑
resource_mapping = {
    "AZURE": "Gloom",
    "VIOLET": "Envy",
    "AMBER": "Sloth",
    "SHAMROCK": "Gluttony",
    "CRIMSON": "Wrath",
    "SCARLET": "Lust",
    "INDIGO": "Pride"
}

def convert_resource(resource_name):
    """Sin Color 이름을 변환하는 함수. 매핑되지 않은 경우 원래 이름을 반환."""
    return resource_mapping.get(resource_name, resource_name)

class Scraper:
    """웹 페이지 크롤링을 위한 클래스"""
    def __init__(self, container, base_url, img_url):
        self.container = container
        self.base_url = base_url
        self.img_url = img_url
        self.uptie_level = 1

    async def handle_request(self, route, request):
        """이미지와 폰트 리소스 로드를 차단."""
        if request.resource_type in ["image", "font"]:
            await route.abort()
        else:
            await route.continue_()
            
    async def scrape(self):
        """크롤링 메인 함수"""
        async with async_playwright() as p:
            browser = await p.chromium.launch(headless=True)
            page = await browser.new_page()
            await page.route("**/*", self.handle_request)
            # Step 1: /identity 페이지에서 기본 정보 크롤링
            #await self.crawl_identity_page(page)

            # Step 2: /identity/{identity_id} 상세 페이지 크롤링
            #identity_ids_to_load = ["10104", "10310", "10710","10808", "10911", "11005"]
            #identity_ids_to_load = ["10104", "10911"]#,"10310", "10710","10808", "11005", "11210"]
            #identity_ids_to_load = ["10911","10310"]#, "10710","10808", "11005", "11210"]
            identity_ids_to_load = ["10611","10711"]#, "10710","10808", "11005", "11210"]
            self.container.load_identities_by_id('./csv/identities.csv', identity_ids_to_load)
            await self.crawl_identity_details_for_all(page)
            #self.container.display_identities()
            #self.container.export_identities_data("identities.csv")
            #self.container.export_level_stats_data("Levelstats.csv")
            #self.container.export_uptie_stats_data("Uptiestats.csv")
            #self.container.export_resist_info_data("ResistInfos.csv")
            #self.container.export_stagger_threshold_data("StaggerThresholds.csv")
            #self.container.export_identity_skillset_data("IdentitiesSkills.csv")
            page.set_default_timeout(10000)
            #await page.wait_for_timeout(100000)
            await browser.close()

    async def get_identity_id_from_row(self, row):
        """테이블 행(row)에서 인격 ID를 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_url = await identity_cell.get_attribute('href')
            identity_id = identity_url.split('/identity/')[-1]
            return identity_id
        return "Unknown"

    async def get_identity_name_from_row(self, row):
        """테이블 행(row)에서 인격 이름을 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_name = (await identity_cell.inner_text()).replace("\n", " ").strip()
            return identity_name
        return "Unknown"
        
    async def crawl_identity_page(self, page):
        """/identity 페이지로 이동하여 인격 기본 정보를 추출."""
        await page.goto(self.base_url + "/identity")
        await self.extract_identity_page(page)

    async def extract_identity_page(self, page):
        """/identity 페이지의 모든 테이블 행에서 인격 정보를 추출."""
        identity_rows = await page.query_selector_all('tr')
        if len(identity_rows) > 1:
            for idx, identity_row in enumerate(identity_rows):
                if idx == 0:
                    continue # 0번째요소이름을 보여주는 열은 건너뜀
                await self.scrape_identity_name_and_identity_id(identity_row)
                await self.extract_rarity(identity_row)
                for i in range(1, 4):  # 스킬1~3 정보 추출
                    await self.extract_atk_and_resource(identity_row, i)
                await self.extract_defense_and_resource(identity_row)

    async def scrape_identity_name_and_identity_id(self, row):
        """인격 번호와 이름을 추출하여 DataContainer에 저장."""
        try:
            identity_id = await self.get_identity_id_from_row(row)
            identity_name = await self.get_identity_name_from_row(row)
            identity = Identity(identity_id)
            self.container.add_identity(identity)
            self.container.update_identity_name(identity_id, identity_name)
        except Exception as e:
            print(f"Error extracting identity name and identity id: {e}")

    async def get_element_attribute(self, element, selector, attribute):
        """요소에서 지정된 속성(attribute)을 추출."""
        try:
            target_element = await element.query_selector(selector)
            if target_element:
                return await target_element.get_attribute(attribute)
        except Exception as e:
            print(f"Error getting element attribute: {e}")
        return None
    
    async def extract_rarity(self, row):
        """테이블 행(row)에서 희귀도를 추출하여 업데이트."""
        try:
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
                sibling = await parent.evaluate_handle('node => node.nextElementSibling')
                if sibling:
                    rarity_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/rarity_"]', 'src')
                    if rarity_img_src:
                        rarity_level = rarity_img_src.split('_')[-1].split('.')[0]  # rarity_1.png -> 1
                        identity_id = await self.get_identity_id_from_row(row)
                        self.container.update_identity_rarity(identity_id, rarity_level)
                    else:
                        print("Rarity image not found in the next sibling TD.")
        except Exception as e: 
            print(f"Error extracting rarity: {e}")

        return "Unknown"

    async def extract_atk_and_resource(self, row, skill_number):
        """ 공격 타입(ATK Type)과 자원(Attribute) 추출 함수 - 스킬 번호에 따라 처리 """
        try:
            #print(f"Attempting to locate skill{skill_number} type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 해당 스킬 번호에 맞는 형제로 이동
                sibling = parent
                for _ in range(skill_number + 1):  # 스킬 번호에 맞게 이동
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                atk_type, resource = "Unknown", "Unknown"
        
                if sibling:
                    # 공격 타입 이미지 (atk_type) 찾기
                    atk_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/atk_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if atk_type_img_src:
                        atk_type = atk_type_img_src.split('_')[-1].split('.')[0]  # atk_type_SLASH.png -> SLASH
                        #print(f"Extracted attack type: {atk_type}")
                        self.container.update_skill(identity_id, skill_number, atk_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted resource: {resource}")
                        
                        # 리소스를 변환
                        resource = convert_resource(resource)
                        #print(f"Converted resource: {resource}")
                        self.container.update_sin_affinity(identity_id, skill_number, resource)
                        
        except Exception as e:
            print(f"Error extracting skill{skill_number} type and resource: {e}")

    async def extract_defense_and_resource(self, row):
        """ 방어 타입(Def Type)과 자원(Attribute) 추출 함수 """
        try:
            #print("Attempting to locate defense type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 다섯 번째 형제 td로 이동 (방어 관련 정보가 있는 <td>)
                sibling = parent
                for _ in range(5):  # 방어 정보는 5번째 형제에 있음
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                def_type, def_resource = "Unknown", "Unknown"
        
                if sibling:
                    # 방어 타입 이미지 (def_type) 찾기
                    def_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/def_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if def_type_img_src:
                        def_type = def_type_img_src.split('_')[-1].split('.')[0]  # def_type_GUARD.png -> GUARD
                        #print(f"Extracted defense type: {def_type}")
                        self.container.update_skill(identity_id, 'defense', def_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        def_resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted defense resource: {def_resource}")
                        
                        # 방어 리소스를 변환
                        def_resource = convert_resource(def_resource)
                        #print(f"Converted defense resource: {def_resource}")
                        self.container.update_sin_affinity(identity_id, 'defense', def_resource)
 
        except Exception as e:
            print(f"Error extracting defense type and resource: {e}")

    async def crawl_identity_details_for_all(self, page):
        """모든 인격의 세부 정보를 크롤링."""
        for identity in self.container.identities:
            await page.wait_for_load_state("networkidle")
            await page.goto(self.base_url + "/identity/"+identity.identity_id)
            """ 레벨과 동기화와 관련 없는 참관타 저항은 밑에서 변경하는 것과 연관성이 적기 때문에 위에서 처리한다.
            print(f"Identity_id: {identity.identity_id}")
            results = []
            resist_type_imgs = [f"{self.img_url}/ui/atk_type_HIT.png",
                                f"{self.img_url}/ui/atk_type_PENETRATE.png",
                                f"{self.img_url}/ui/atk_type_SLASH.png"]
            for resist_type_img in resist_type_imgs:
                result = await self.extract_resist_type(page, resist_type_img, 1)
                results.append(result)
            self.container.add_resist_info(identity.identity_id, results[0], results[1], results[2])
            """
            """
            # Stagger Threshold 값 추출
            stagger_threshold = await self.extract_stagger_threshold_values(page)
            self.container.add_stagger_threshold(identity.identity_id, stagger_threshold[0], stagger_threshold[1], stagger_threshold[2])
            """
            await self.initialize_uptie_and_level(page)
            """ 레벨에 따른 level, hp, defense 다르게 되어 있는 것을 출력 확인 객체로 들어가서 csv로 옮기는 과정까지 진행
            for i in range(1, identity.level_max+1):
                await self.move_slider_to_value(page,i)
                await self.extract_level_status(page, identity.identity_id, i)
            """
            """ 동기화에 따른 speed 다르게 되어 있는 것을 출력 확인 객체로 들어가서 csv로 옮기는 과정까지 진행
            for i in range(1, identity.uptie_max+1):
                await self.uptie_click(page,i)
                await self.extract_uptie_status(page, identity.identity_id, i)
            """
            """
                인격에 대한 skill 정보를 가져오는 메소드 위에서 출력하는 csv파일로 인격이 가지고 있는 스킬 정보를 내보낸다.
            await self.crawl_skills(page, identity.identity_id)
            """
            await self.crawl_influence_behavior(page, identity.identity_id)
            
    async def initialize_uptie_and_level(self, page):
        await self.uptie_click(page, 1)
        await self.move_slider_to_value(page, 1)
        self.uptie_level = 1
        #return 1
    
    async def uptie_click(self, page, phase):
        fab_button_selector = ".q-btn.q-btn--fab"
        await page.wait_for_selector(fab_button_selector)
        await page.click(fab_button_selector)
        #print(f"[{setup_uptie_button}] 상위 동기화 버튼 클릭 완료")
        setup_uptie_button = f'a:has(img[src="{self.img_url}/ui/uptie_{phase}.png"])'
        await page.wait_for_selector(setup_uptie_button)
        await page.click(setup_uptie_button)
        #print(f"{phase} 동기화 버튼 클릭 완료")

    async def move_slider_to_value(self, page, target_value):
        """
        슬라이더를 주어진 값으로 이동시킵니다.
        target_value는 슬라이더의 최소값부터 최대값 사이여야 합니다.
        """
        slider_selector = ".q-slider"
        # 슬라이더의 범위 확인
        min_value = int(await page.get_attribute(slider_selector, "aria-valuemin"))
        max_value = int(await page.get_attribute(slider_selector, "aria-valuemax"))
    
        if target_value < min_value or target_value > max_value:
            raise ValueError(f"Target value {target_value} is out of bounds ({min_value}, {max_value})")
    
        # 슬라이더 thumb 요소와 위치 가져오기
        slider_thumb = await page.query_selector(f"{slider_selector} .q-slider__thumb")
        thumb_box = await slider_thumb.bounding_box()
        
        # 슬라이더의 현재 위치 및 목표 위치 계산
        track = await page.query_selector(f"{slider_selector} .q-slider__track")
        track_box = await track.bounding_box()
        
        # 목표 x 위치 계산 (슬라이더 트랙 내에서 비율 기반)
        target_percentage = (target_value - min_value) / (max_value - min_value)
        target_x = track_box["x"] + target_percentage * track_box["width"]
        current_x = thumb_box["x"] + thumb_box["width"] / 2
        y = thumb_box["y"] + thumb_box["height"] / 2
    
        # 슬라이더를 드래그하여 목표 위치로 이동
        await page.mouse.move(current_x, y)
        await page.mouse.down()
        await page.mouse.move(target_x, y, steps=10)  # 부드럽게 이동
        await page.mouse.up()
    
        #print(f"슬라이더를 {target_value}로 이동 완료.")
    
        # 슬라이더 값 확인 및 강제 이벤트 트리거
        await page.evaluate(
            f"""
            (slider) => {{
                slider.dispatchEvent(new Event('input'));  // oninput 이벤트 트리거
                slider.dispatchEvent(new Event('change'));  // onchange 이벤트 트리거
            }}
            """,
            await page.query_selector(slider_selector)
        )
    
        # 이동 후 값 확인
        current_value = await page.get_attribute(slider_selector, "aria-valuenow")
        if current_value != str(target_value):
            print(f"경고: 슬라이더 값이 예상과 다릅니다. 현재 값: {current_value}, 예상 값: {target_value}")

    async def get_second_status_element(self, page):
        """
        두 번째 'Status' 요소에 접근하여 반환하는 메소드
        """
        status_elements = await page.query_selector_all("text=Status")
        if len(status_elements) > 1:
            second_status_element = status_elements[1]
            #print("두 번째 'Status' 요소에 접근합니다.")
            return second_status_element
        else:
            #print("두 번째 'Status' 요소가 존재하지 않습니다.")
            return None

    async def extract_level_status(self, page, identity_id, level):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 레벨에 따라 HP와 Defense가 변한다.
        hp_value = await self.extract_hp(second_status_element)
        defense_value = await self.extract_defense(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Level={level}, HP={hp_value}, Defense={defense_value}")
        self.container.add_level_stat(identity_id,level, hp_value, defense_value)
    
    async def extract_hp(self, status_element):
        # HP 추출
        hp_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].children[0].nextElementSibling.textContent.trim()"
        )
        return hp_value
    
    async def extract_defense(self, status_element):
        # Defense 추출
        defense_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return defense_value

    async def extract_uptie_status(self, page, identity_id, uptie):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 uptie에 따라 speed는 변한다.
        speed_value = await self.extract_speed(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Uptie={uptie}, Speed={speed_value}")
        self.container.add_uptie_stat(identity_id,uptie, speed_value)

    async def extract_speed(self, status_element):
        # Speed 추출
        speed_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return speed_value

    async def extract_resist_type(self, page, image_src, depth):
        # 이미지 요소 찾기
        #print(image_src)
        image = await page.query_selector(f'img[src="{image_src}"]')
        #print(image)
        if not image:
            return "Unknown"
    
        # 이미지의 부모 요소를 찾아서 올라감
        parent = image
        for _ in range(depth):
            parent = await parent.evaluate_handle('node => node.parentElement')
            if not parent:
                return "Unknown"
    
        # 부모의 형제 요소 찾기
        sibling = await parent.evaluate_handle('node => node.nextElementSibling')
        
        if sibling:
            value = await sibling.evaluate('node => node.textContent')
            if value:
                return value.strip()
        
        return "Unknown"

    async def extract_stagger_threshold_values(self, page):
        # 'div.row.center.flex-center.q-pt-xs' 요소들 찾기
        elements = await page.query_selector_all('div.row.center.flex-center.q-pt-xs')
        
        # 각 요소의 텍스트 내용 출력하여 'Stagger Threshold' 요소를 찾기
        for element in elements:
            text = await element.text_content()
            
            if text and "Stagger Threshold" in text:
                # 'Stagger Threshold'가 있는 요소를 찾았다면, 그 이후 형제 요소들에서 값을 추출
                parent = await element.evaluate_handle('node => node.parentElement')
                sibling_elements = await parent.query_selector_all('div.row.justify-evenly div')  # Threshold 값이 들어있는 div들
                
                stagger_values = []
                for sibling in sibling_elements:
                    text_content = await sibling.text_content()
                    match = re.search(r'(\d+%)', text_content)
                    if match:
                        stagger_values.append(match.group(1))  # 퍼센트 값만 리스트에 추가
                while len(stagger_values) < 3:
                    stagger_values.append(np.nan)
                #print(stagger_values)
                return stagger_values  # 추출된 값(또는 np.nan 포함)을 반환
        
        return [np.nan, np.nan, np.nan]  # 만약 'Stagger Threshold'를 찾지 못한 경우 기본값 반환

    async def crawl_skills(self, page, identity_id):
        await self.crawl_skill(page, identity_id, "skill 1")
        await self.crawl_skill(page, identity_id, "skill 2")
        await self.crawl_skill(page, identity_id, "skill 3")
        await self.crawl_skill(page, identity_id, "defense")


    async def crawl_skill(self, page, identity_id, skill_type):
        skills_elements = await page.query_selector_all('div.skilltag')
        for skill_element in skills_elements:
            skill_text = await skill_element.text_content()
            if skill_text.strip().lower() == skill_type:
                await self.process_skill(skill_element, page, identity_id, skill_type)
        
    async def process_skill(self, skill_element, page, identity_id, skill_type):
        # skill_type이 "skill 3"일 경우 uptie_value가 3 이상일 때만 정보를 저장
        for uptie_value in range(1, 5):  # Uptie levels 1 to 4
            # "skill 3"은 uptie_value가 3 이상일 때만 처리
            if skill_type == "skill 3" and uptie_value < 3:
                continue  # skip if uptie_value is less than 3 for skill 3
    
            # uptie_level에 맞게 클릭
            await self.uptie_click(page, uptie_value)
            if skill_type != "skill 3" and uptie_value > 1:
                # Uptie 정보 갱신
                await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
                continue
            elif skill_type == "skill 3" and uptie_value > 3:
                # Uptie 정보 갱신
                await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
                continue
            # 스킬의 세부 정보를 추출
            skill_details = await self.extract_skill_details(skill_element, skill_type)
            skill_name = skill_details["skill_name"]
    
            # 스킬 정보를 저장
            self.container.add_skill(
                identity_id, skill_type, 
                skill_name, skill_details["coin_count"],
                skill_details["init_level"], skill_details["init_remain"], 
                skill_details["atk_weight"]
            )
            print(f"Added skill: {skill_type}:{skill_name}")
    
            # Uptie 정보 갱신
            await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
    
    async def extract_skill_details(self, skill_element, skill_type):
        # 스킬 세부 정보를 추출하는 함수
        skill_name = await self.skill_name(skill_element)
        coin_count = await self.coin_count(skill_element, ".col-auto > .column > .row")
        init_level = await self.init_level(skill_element)
        init_remain = await self.init_remain(skill_element, skill_type, "div.row.justify-start > div.attr.flex.flex-center > div.q-px-sm")
        atk_weight = await self.atk_weight(skill_element, skill_type)
        return {
            "skill_name": skill_name,
            "coin_count": coin_count,
            "init_level": init_level,
            "init_remain": init_remain,
            "atk_weight": atk_weight
        }
    
    async def update_uptie_info(self, page, skill_element, identity_id, skill_name, skill_type, uptie_value):
        # Uptie 정보를 추출하고 업데이트하는 함수
        uptie_info = await self.extract_uptie_info(skill_element)
        effect_data = await self.skill_effects(skill_element)
        self.container.add_skill_uptie_info(
            identity_id, skill_name, 
            uptie_value, uptie_info["skill_power"], uptie_info["coin_power"], 
            effect_data.get("effectsByCondition")
        )
        print(
            f"Updated Uptie for {skill_name} (Uptie Level: {uptie_value}): "
            f"Skill Power: {uptie_info['skill_power']}, Coin Power: {uptie_info['coin_power']} "
            f"Skill Effect: {effect_data.get("effectsByCondition")}"
        )

    async def extract_uptie_info(self, skill_element):
        skill_data = await self.skill_and_coin_power(skill_element)
        return {
            "skill_power": skill_data.get("skillPower"),
            "coin_power": skill_data.get("coinPower")
        }
        
    async def crawl_influence_behavior(self, page, identity_id):
        influence_behaviors = await page.query_selector_all('div[align="right"]')
        # 텍스트를 기준으 추출하고 메소드로 파라미터 들어가 정보를 받는다.
        for influence_behavior in influence_behaviors:
            active_type = await influence_behavior.inner_text()
            active_type = active_type.strip()
            print(active_type)

    async def skill_and_coin_power(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                // 조부모로 이동
                const grandparent = node.parentElement.parentElement;
                // 이전 형제 탐색
                const previousSibling = grandparent.previousElementSibling;
                if (!previousSibling) return {{ skillPower: null, coinPower: null }}; // 이전 형제가 없으면 기본값 반환
                // 이전 형제 내부의 .row.justify-around > div > div 탐색
                const targetDivs = previousSibling.querySelectorAll(".row.justify-around > div");
                if (targetDivs.length < 3) return {{ skillPower: null, coinPower: null }}; // 배열 길이 확인
                // 1st div: 스킬 위력
                const skillPower = targetDivs[0]?.querySelector('div')?.textContent?.trim() || null; 
                // 3rd div: 코인 파워 값 (3번째 div에서 텍스트 추출)
                const coinPower = targetDivs[2]?.textContent?.trim() || null;
                return {{ skillPower, coinPower }};
            }}
        """)


    async def coin_count(self, skill_element, hierarchy_selector):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector('{hierarchy_selector}');
                if (!targetElement) return 0;
                return targetElement.childElementCount;
            }}
        """)

    async def init_remain(self, skill_element, skill_type, hierarchy_selector):
        if skill_type == "defense":
            return -1
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector('{hierarchy_selector}');
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def skill_name(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector(".block");
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def init_level(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector("div.text-bold.text-h6.h6.q-pl-xs.q-pr-md");
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def atk_weight(self, skill_element, skill_type):
        if skill_type == "defense":
            return 0
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector("div.text-bold.text-h6.h6.q-pl-xs.q-pr-md");
                const doubleNextSibling = targetElement.nextElementSibling.nextElementSibling;
                if (!doubleNextSibling) return 0;
                return doubleNextSibling.textContent?.trim();
            }}
        """)

    async def skill_effects(self, skill_element):
        return await skill_element.evaluate("""
            node => {
                const parent = node.parentElement;
                const doubleNextSibling = parent.nextElementSibling?.nextElementSibling;
                if (!doubleNextSibling) return null;
    
                // doubleNextSibling에서 첫 번째 div.col 요소 찾기
                const textNode = doubleNextSibling.querySelector("div.col");
                if (!textNode) return null;
    
                const results = {
                    keywords: [], // 키워드 저장
                    conditions: [], // 조건 저장
                    effectsByCondition: {} // 조건별 효과
                };
    
                // 1. 조건 추출 (대괄호 텍스트)
                const conditionElements = textNode.querySelectorAll("span");
                conditionElements.forEach(span => {
                    let text = span.textContent?.trim();
                    if (text?.startsWith("[") && text?.endsWith("]")) {
                        // 부모의 전형제 노드 탐색
                        const sibling = span.parentElement.previousElementSibling;
                        if (sibling) {
                            const emblemImage = sibling.querySelector("img.emblem");
                            console.log("Sibling element found:", sibling); // 디버깅용
                            if (emblemImage) {
                                const src = emblemImage.getAttribute("src");
                                console.log("Image source:", src); // 디버깅용
                                const match = src.match(/ui_num_(\\d+)\\.png$/); // 정규식 수정
                                if (match) {
                                    const number = match[1]; // 숫자 추출
                                    text = `[Coin ${number} ${text.replace("[", "").replace("]", "")}]`;
                                    console.log("Updated condition:", text); // 디버깅용
                                }
                            }
                        }
                        results.conditions.push(text);
                        results.effectsByCondition[text] = []; // 조건별 효과를 초기화
                    }
                });
    
                // 2. 키워드 추출
                const keywordElements = textNode.querySelectorAll('a[href^="/en/keyword/"]');
                keywordElements.forEach(a => {
                    const span = a.querySelector('span.q-btn__content.text-center.col.items-center.q-anchor--skip.justify-center.row>span');
                    if (span) {
                        const keywordText = span.textContent.trim();
                        if (keywordText && !results.keywords.includes(keywordText)) {
                            results.keywords.push(keywordText); // 키워드 저장
                        }
                    }
                });
    
                // 3. 효과 추출
                const effectElements = textNode.querySelectorAll("span");
                let currentCondition = null;
                let currentEffect = []; // 초기화
                effectElements.forEach(span => {
                    let text = span.textContent?.trim();
                    if (!text) return;
    
                    // 조건 발견 시 currentCondition 변경
                    if (text.startsWith("[") && text.endsWith("]")) {
                        // 이전 조건의 효과 저장
                        if (currentCondition && currentEffect.length > 0) {
                            // 부모의 전형제 노드 탐색
                            const sibling = span.parentElement.previousElementSibling;
                            if (sibling) {
                                const emblemImage = sibling.querySelector("img.emblem");
                                if (emblemImage) {
                                    const src = emblemImage.getAttribute("src");
                                    const match = src.match(/ui_num_(\\d+)\\.png$/); // 정규식 수정
                                    if (match) {
                                        const number = match[1]; // 숫자 추출
                                        text = `[Coin ${number} ${text.replace("[", "").replace("]", "")}]`;
                                    }
                                }
                            }
                            results.effectsByCondition[currentCondition].push(currentEffect.join(' '));
                            currentEffect = []; // 초기화
                        }
                        currentCondition = text;
                        return;
                    }
    
                    // 현재 조건의 효과 처리
                    if (currentCondition) {
                        if (results.keywords.includes(text)) {
                            const keywordEffect = 'keyword:' + text;
                            if (!currentEffect.includes(keywordEffect)) {
                                currentEffect.push(keywordEffect);
                            }
                        } else {
                            currentEffect.push(text);
                        }
                    }
                });
    
                // 마지막 효과 저장
                if (currentCondition && currentEffect.length > 0) {
                    results.effectsByCondition[currentCondition].push(currentEffect.join(' '));
                }
    
                return results;
            }
        """)



class Identity:
    """인격 정보를 저장하는 클래스"""
    def __init__(self, identity_id, name=None, rarity=None,\
                 skill_1_type=None, skill_1_sin_affinity=None,\
                 skill_2_type=None, skill_2_sin_affinity=None,\
                 skill_3_type=None, skill_3_sin_affinity=None,\
                 defense_type=None, defense_sin_affinity=None):
        self.identity_id = identity_id
        self.name = name
        self.rarity = rarity
        # 선언만 하는 부분 
        self.skill_1_type = skill_1_type 
        self.skill_1_sin_affinity = skill_1_sin_affinity 
        self.skill_2_type = skill_2_type 
        self.skill_2_sin_affinity = skill_2_sin_affinity 
        self.skill_3_type = skill_3_type 
        self.skill_3_sin_affinity = skill_3_sin_affinity 
        self.defense_type = defense_type 
        self.defense_sin_affinity = defense_sin_affinity
        self.level_max = 50
        self.uptie_max = 4
        self.level_stats = []  # LevelStat 객체 리스트
        self.uptie_stats = [] # UptieStat 객체 리스트
        self.resist_info = None
        self.stagger_threshold = None
        self.skillset = SkillSet()  # Skill정보를 담고 있는 skillset 객체 
        self.passive_infos = []  # 패시브 객체를 담고 있는 배열
        self.support_passive_infos = [] # 서포트 패시 객체를 담고 있는 배열
        self.panic_type = [] # 혼란상태 발생시 나타나는 효과를 담고 있는 객체 배열

    def update_name(self, name):
        self.name = name

    def update_rarity(self, rarity):
        self.rarity = rarity

    def add_level_stat(self, level, hp, defense):
        self.level_stats.append(LevelStat(level, hp, defense))

    def add_uptie_stat(self, uptie, speed):
        self.uptie_stats.append(UptieStat(uptie, speed))

    def add_resist_info(self, resist_hit, resist_penetrate, resist_slash):
        self.resist_info = ResistInfo(resist_hit, resist_penetrate, resist_slash)

    def add_stagger_threshold(self, first_stagger, second_stagger, third_stagger):
        self.stagger_threshold = StaggerThreshold(first_stagger, second_stagger, third_stagger)

    def add_skill(self, skill_code, skill_name, coin_count, init_level, init_remain, atk_weight):
        self.skillset.add_skill(skill_code, Skill(skill_code, skill_name, coin_count, init_level, init_remain, atk_weight))
        
    def add_skill_uptie_info(self, skill_name, uptie_level, skill_power, coin_power, effect_by_condition):
        self.skillset.add_skill_uptie_info(skill_name, uptie_level, skill_power, coin_power, effect_by_condition)
        
    def update_details(self, page):
        # 세부 정보 업데이트 로직 (가정)
        pass
    #def init_uptie(self, page):
    #    init_uptie = 

# LevelStat 객체 정의
class LevelStat:
    def __init__(self, level, hp=None, defense=None):
        self.level = level
        self.hp = hp
        self.defense = defense
    def update_level(self, level):
        self.level = level
    def update_hp(self, hp):
        self.hp = hp
    def update_defense(self, defense):
        self.defense = defense

# UptieStat 객체 정의
class UptieStat:
    def __init__(self, uptie, speed=None):
        self.uptie = uptie
        self.speed = speed
    def update_speed(self, speed):
        self.speed = speed

# ResistInfo 객체 정의
class ResistInfo:
    def __init__(self, resist_hit=None, resist_penetrate=None, resist_slash=None):
        self.resist_hit = resist_hit
        self.resist_penetrate = resist_penetrate
        self.resist_slash = resist_slash

# ResistInfo 객체 정의
class StaggerThreshold:
    def __init__(self, first_stagger=None, second_stagger=None, third_stagger=None):
        self.first_stagger = first_stagger
        self.second_stagger = second_stagger
        self.third_stagger = third_stagger

class SkillSet:
    def __init__(self):

        self.skill_1_type = ""
        self.skill_2_type = ""
        self.skill_3_type = ""
        self.defense_skill_type = ""

        self.skill_1_sin_affinity = ""
        self.skill_2_sin_affinity = ""
        self.skill_3_sin_affinity = ""
        self.defense_skill_sin_affinity = ""

        self.skill_1_details = []
        self.skill_2_details = []
        self.skill_3_details = []
        self.defense_skill_details = []

    def set_skill_type(self, skill_code, skill_type, sin_affinity):
        if skill_code == "skill 1":
            self.skill_1_type = skill_type
            self.skill_1_sin_affinity = sin_affinity
        elif skill_code == "skill 2":
            self.skill_2_type = skill_type
            self.skill_2_sin_affinity = sin_affinity
        elif skill_code == "skill 3":
            self.skill_3_type = skill_type
            self.skill_3_sin_affinity = sin_affinity
        elif skill_code == "defense":
            self.defense_skill_type = skill_type
            self.defense_skill_sin_affinity = sin_affinity
    
    def add_skill(self, skill_code, skill):
        if skill_code == "skill 1":
            self.skill_1_details.append(skill)
            skill.skill_type = self.skill_1_type
            skill.sin_affinity = self.skill_1_sin_affinity
        elif skill_code == "skill 2":
            self.skill_2_details.append(skill)
            skill.skill_type = self.skill_2_type
            skill.sin_affinity = self.skill_2_sin_affinity
        elif skill_code == "skill 3":
            self.skill_3_details.append(skill)
            skill.skill_type = self.skill_3_type
            skill.sin_affinity = self.skill_3_sin_affinity
        elif skill_code == "defense":
            self.defense_skill_details.append(skill)
            skill.skill_type = self.defense_skill_type 
            skill.sin_affinity = self.defense_skill_sin_affinity

    def get_skill_by_name(self, skill_name):
        for skill_list in [self.skill_1_details, self.skill_2_details, self.skill_3_details, self.defense_skill_details]:
            for skill in skill_list:
                result = skill.is_skill_by_name(skill_name)
                if result:
                    return result
        return None

    def add_skill_uptie_info(self, skill_name, uptie_level, skill_power, coin_power, effect_by_condition):
        skill = self.get_skill_by_name(skill_name)
        skill.add_uptie_info(uptie_level, skill_power, coin_power, effect_by_condition)
        

class Skill:
    def __init__(self, skill_code, skill_name, coin_count, init_level, init_remain, atk_weight):
        self.skill_code = skill_code # "Skill 1"/"Skill 2"/"Skill 3"/"Defense" '''o'''
        self.skill_type = None
        self.sin_affinity = None
        self.skill_name = skill_name # 스킬 이름 '''o'''
        self.coin_count = coin_count # 동전 개수 '''o'''
        self.init_level = init_level # 스킬이 초기 배정받은 레벨, 인격 레벨에 - 1 한 값을 더해주면 스킬의 현 level을 구할 수 있다. '''o'''
        self.init_remain = init_remain #'''o'''
        self.atk_weight = atk_weight #공격 가중치 값이다. 방어스킬에는 없기 때문에 None으로 설정했다. '''o'''
        self.remain_count = init_remain # 실제 스킬이 작동하며 횟수가 차감되 나중에 모두 차감되면 다음 턴에 다시 초기횟수 리필된다.
        self.uptie_info = []  

    def is_skill_by_name(self, skill_name):
        if self.skill_name == skill_name:
            return self
        return None
        
    def add_uptie_info(self, uptie_level, skill_power, coin_power, effects_by_condition):
        self.uptie_info.append(UptieInfo(uptie_level, skill_power, coin_power, effects_by_condition))

class UptieInfo:
    def __init__(self, uptie_level, skill_power, coin_power, effects_by_condition):
        self.uptie_level = uptie_level   #'''o'''
        self.skill_power = skill_power   #'''o'''
        self.coin_power = coin_power     #'''o'''
        self.skill_effect = effects_by_condition

# 추상화된 EffectType 클래스
class EffectType:
    def __init__(self, name, description):
        self.name = name  # Effect 이름
        self.description = description  # Effect 설명

# Passive 클래스
class Passive(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)  # condition이 있는지 여부 확인

# SupportPassive 클래스
class SupportPassive(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)

# PanicType 클래스
class PanicType(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)

# Keyword 클래스
class Keyword:
    def __init__(self, keyword, explain=None, dispellable=None):
        self.keyword = keyword
        self.explain = explain
        self.dispellable = dispellable

      
class DataContainer:
    """크롤링 데이터를 관리하는 클래스"""
    def __init__(self):
        self.identities = []

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.level_max

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.uptie_max

    def update_identity_name(self, identity_id, identity_name):
        """특정 ID의 이름을 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_name(identity_name)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_identity_rarity(self, identity_id, rarity):
        """특정 ID의 희귀도를 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_rarity(rarity)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_skill(self, identity_id, skill_number, new_type):
        """
        특정 identity의 skill_type을 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_type: 새 skill_type
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_type = new_type
                elif skill_number == 2:
                    identity.skill_2_type = new_type
                elif skill_number == 3:
                    identity.skill_3_type = new_type
                elif skill_number == "defense":
                    identity.defense_type = new_type
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def update_sin_affinity(self, identity_id, skill_number, new_affinity):
        """
        특정 identity의 sin_affinity를 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_affinity: 새 sin_affinity
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_sin_affinity = new_affinity
                elif skill_number == 2:
                    identity.skill_2_sin_affinity = new_affinity
                elif skill_number == 3:
                    identity.skill_3_sin_affinity = new_affinity
                elif skill_number == "defense":
                    identity.defense_sin_affinity = new_affinity
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def add_identity(self, identity):
        """새로운 인격 데이터를 추가."""
        self.identities.append(identity)

    def add_level_stat(self, identity_id, level, hp, defense):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_level_stat(level, hp, defense)

    def add_uptie_stat(self, identity_id, uptie, speed):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_uptie_stat(uptie, speed)

    def add_resist_info(self, identity_id, resist_hit, resist_penetrate, resist_slash):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_resist_info(resist_hit, resist_penetrate, resist_slash)

    def add_stagger_threshold(self, identity_id, first_stagger, second_stagger, third_stagger):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_stagger_threshold(first_stagger, second_stagger, third_stagger)

    
    def add_skill(self, identity_id, skill_code, skill_name, coin_count, init_level, init_remain, atk_weight):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_code == "skill 1":
                    identity.skillset.set_skill_type(skill_code, identity.skill_1_type, identity.skill_1_sin_affinity)
                elif skill_code == "skill 2":
                    identity.skillset.set_skill_type(skill_code, identity.skill_2_type, identity.skill_2_sin_affinity)
                elif skill_code == "skill 3":
                    identity.skillset.set_skill_type(skill_code, identity.skill_3_type, identity.skill_3_sin_affinity)
                elif skill_code == "defense":
                    identity.skillset.set_skill_type(skill_code, identity.defense_type, identity.defense_sin_affinity)
                identity.add_skill(skill_code, skill_name, coin_count, init_level, init_remain, atk_weight)

    def add_skill_uptie_info(self, identity_id, skill_name, uptie_level, skill_power, coin_power, effects_by_condition):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_skill_uptie_info(skill_name, uptie_level, skill_power, coin_power, effects_by_condition)
        
    def display_identities(self):
        """Identity, ID, Name, Rarity 정보를 출력"""
        if not self.identities:
            print("No identities found.")
        else:
            # 데이터를 리스트로 변환
            table = []
            for idx, identity in enumerate(self.identities, 1):
                table.append([
                    idx, identity.identity_id, identity.name, identity.rarity,
                    identity.skill_1_type, identity.skill_1_sin_affinity,
                    identity.skill_2_type, identity.skill_2_sin_affinity,
                    identity.skill_3_type, identity.skill_3_sin_affinity,
                    identity.defense_type, identity.defense_sin_affinity
                ])
            
            # 테이블 헤더 정의
            headers = [
                "Index", "ID", "Name", "Rarity",
                "Skill_1 Type", "Skill_1 Sin", "Skill_2 Type", "Skill_2 Sin",
                "Skill_3 Type", "Skill_3 Sin", "Defense Type", "Defense Sin"
            ]
            
            # 테이블 출력
            print(tabulate(table, headers=headers, tablefmt="fancy_grid"))

    def export_to_csv(self, filename, fieldnames, data):
        """
        공통적으로 사용 가능한 CSV 내보내기 메서드
        :param filename: 내보낼 파일 이름 (예: 'identity.csv', 'skill.csv')
        :param fieldnames: CSV의 헤더 리스트
        :param data: CSV로 내보낼 데이터 (리스트 형태)
        """
        try:
            # 현재작업 디렉토 기준으 'csv'폴더 없으면 생성
            if not os.path.exists('./csv'):
                os.makedirs('./csv')
            # 파일 경로 설정 (상대경로 'csv' 폴더 안에 저장)
            file_path = os.path.join('./csv', filename)
            with open(file_path, mode='w', newline='', encoding='utf-8') as file:
                writer = csv.DictWriter(file, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(data)
            print(f"Data successfully exported to {filename}")
        except Exception as e:
            print(f"Error exporting to CSV: {e}")

    def get_identity_data(self):
        """
        Identity 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "ID": identity.identity_id,
                "Name": identity.name,
                "Rarity": identity.rarity,
                "Skill_1_Type": identity.skill_1_type,
                "Skill_1_Sin_Affinity": identity.skill_1_sin_affinity,
                "Skill_2_Type": identity.skill_2_type,
                "Skill_2_Sin_Affinity": identity.skill_2_sin_affinity,
                "Skill_3_Type": identity.skill_3_type,
                "Skill_3_Sin_Affinity": identity.skill_3_sin_affinity,
                "Defense_Type": identity.defense_type,
                "Defense_Sin_Affinity": identity.defense_sin_affinity,
            }
            for identity in self.identities
        ]
        
    def export_identities_data(self, filename="identities.csv"):
        """Identity 데이터를 CSV로 내보내는 메서드"""
        identity_fieldnames = [
            "ID", "Name", "Rarity",
            "Skill_1_Type", "Skill_1_Sin_Affinity",
            "Skill_2_Type", "Skill_2_Sin_Affinity",
            "Skill_3_Type", "Skill_3_Sin_Affinity",
            "Defense_Type", "Defense_Sin_Affinity"
        ]
        identity_data = self.get_identity_data()  # Identity 데이터 가져오기
        self.export_to_csv(filename, identity_fieldnames, identity_data)  # CSV로 내보내기

    def load_identities_by_id(self, filename, identity_ids):
        """지정된 identity_ids 목록에 해당하는 Identity 데이터를 CSV에서 읽어옴."""
        identity_ids_set = set(identity_ids) # 중복 제거를 위 Set 사용
        with open(filename, mode='r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                if row["ID"] in identity_ids_set:
                    identity = Identity(
                        identity_id=row["ID"],
                        name=row["Name"],
                        rarity=row["Rarity"],
                        skill_1_type=row["Skill_1_Type"],
                        skill_1_sin_affinity=row["Skill_1_Sin_Affinity"],
                        skill_2_type=row["Skill_2_Type"],
                        skill_2_sin_affinity=row["Skill_2_Sin_Affinity"],
                        skill_3_type=row["Skill_3_Type"],
                        skill_3_sin_affinity=row["Skill_3_Sin_Affinity"],
                        defense_type=row["Defense_Type"],
                        defense_sin_affinity=row["Defense_Sin_Affinity"]
                    )
                    self.identities.append(identity)

    def get_level_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Level": level_stat.level,
                "HP": level_stat.hp,
                "Defense": level_stat.defense,
            }
            for identity in self.identities
            for level_stat in identity.level_stats
        ]

    def export_level_stats_data(self, filename="levelstats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        level_stats_fieldnames = ["Identity_ID", "Level", "HP", "Defense"]
        level_stats_data = self.get_level_stats_data()  # level_stats 데이터 가져오기
        self.export_to_csv(filename, level_stats_fieldnames, level_stats_data)  # CSV로 내보내기

    def get_uptie_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Uptie": uptie_stat.uptie,
                "Speed": uptie_stat.speed,
            }
            for identity in self.identities
            for uptie_stat in identity.uptie_stats
        ]

    def export_uptie_stats_data(self, filename="Uptiestats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        uptie_stats_fieldnames = ["Identity_ID", "Uptie", "Speed"]
        uptie_stats_data = self.get_uptie_stats_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, uptie_stats_fieldnames, uptie_stats_data)  # CSV로 내보내기

    def get_resist_info_data(self):
        """
        모든 Identity의 resist_info 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Resist_Hit": identity.resist_info.resist_hit,
                "Resist_Penetrate": identity.resist_info.resist_penetrate,
                "Resist_Slash": identity.resist_info.resist_slash,
            }
            for identity in self.identities
        ]

    def export_resist_info_data(self, filename="ResistInfos.csv"):
        """ResistInfo 데이터를 CSV로 내보내는 메서드"""
        resist_infos_fieldnames = ["Identity_ID", "Resist_Hit", "Resist_Penetrate", "Resist_Slash"]
        resist_infos_data = self.get_resist_info_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, resist_infos_fieldnames, resist_infos_data)  # CSV로 내보내기

    def export_uptie_stats_data(self, filename="Uptiestats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        uptie_stats_fieldnames = ["Identity_ID", "Uptie", "Speed"]
        uptie_stats_data = self.get_uptie_stats_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, uptie_stats_fieldnames, uptie_stats_data)  # CSV로 내보내기

    def get_stagger_threshold_data(self):
        """
        모든 Identity의 stagger_threshold 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "First_Stagger": identity.stagger_threshold.first_stagger,
                "Second_Stagger": identity.stagger_threshold.second_stagger,
                "Third_Stagger": identity.stagger_threshold.third_stagger,
            }
            for identity in self.identities
        ]

    def export_stagger_threshold_data(self, filename="StaggerThresholds.csv"):
        """StaggerThresholds 데이터를 CSV로 내보내는 메서드"""
        stagger_thresholds_fieldnames = ["Identity_ID", "First_Stagger", "Second_Stagger", "Third_Stagger"]
        stagger_thresholds_data = self.get_stagger_threshold_data()  # stagger_thresholds 데이터 가져오기
        self.export_to_csv(filename, stagger_thresholds_fieldnames, stagger_thresholds_data)  # CSV로 내보내기

    def get_skillset_data(self):
        """모든 Identity의 skillset 데이터를 export_to_csv을 포맷으로 변환"""
        return [
            {
                "Identity_ID": identity.identity_id,
                "Skill_Code": skill.skill_code,
                "Skill_Type": skill.skill_type,
                "Sin_Affinity": skill.sin_affinity,
                "Skill_Name": skill.skill_name,
                "Coin_Count": skill.coin_count,
                "Init_Level": skill.init_level,
                "Init_Remain": skill.init_remain,
                "Atk_Weight": skill.atk_weight,
                "Uptie_Level": uptie_info.uptie_level,
                "Skill_Power": uptie_info.skill_power,
                "Coin_Power": uptie_info.coin_power,
                "Effect_Data": json.dumps(uptie_info.skill_effect)
            }
            for identity in self.identities
            for skill_list in [identity.skillset.skill_1_details, identity.skillset.skill_2_details, identity.skillset.skill_3_details, identity.skillset.defense_skill_details]
            for skill in skill_list
            for uptie_info in skill.uptie_info
        ]

    def export_identity_skillset_data(self, filename="IdentitiesSkills.csv"):
        """IdentitiesSkills 데이터를 CSV로 내보내는 메서드"""
        identities_skills_fieldnames = ["Identity_ID", "Skill_Code", "Skill_Type", "Sin_Affinity", "Skill_Name", "Coin_Count", "Init_Level", "Init_Remain", "Atk_Weight", "Uptie_Level", "Skill_Power", "Coin_Power", "Effect_Data"]
        identities_skills_data = self.get_skillset_data()
        self.export_to_csv(filename, identities_skills_fieldnames, identities_skills_data)
            


async def main():
    data_container = DataContainer()
    scraper = Scraper(container=data_container, base_url="https://limbus.kusoge.xyz", img_url="https://img.kusoge.xyz/limbus/img")
    await scraper.scrape()

await main() 

/opt/miniconda3/lib/python3.12/site-packages/pyee/base.py:63: RuntimeWarning: coroutine 'Page.query_selector_all' was never awaited
  ] = dict()
/opt/miniconda3/lib/python3.12/site-packages/pyee/base.py:63: RuntimeWarning: coroutine 'ElementHandle.inner_text' was never awaited
  ] = dict()


SKILL 1
SKILL 2
SKILL 3
SKILL 2
SKILL 3
SKILL 1
DEFENSE
DEFENSE
PASSIVE
PASSIVE
PASSIVE
SUPPORT PASSIVE
PANIC TYPE
SKILL 1
SKILL 2
SKILL 3
SKILL 3
DEFENSE
DEFENSE
PASSIVE
PASSIVE
SUPPORT PASSIVE
PANIC TYPE


In [35]:
import os
import csv
import re
import json
import numpy as np
import pandas as pd
from playwright.async_api import async_playwright
from tabulate import tabulate

# Sin Color 이름 변환을 위한 매핑
resource_mapping = {
    "AZURE": "Gloom",
    "VIOLET": "Envy",
    "AMBER": "Sloth",
    "SHAMROCK": "Gluttony",
    "CRIMSON": "Wrath",
    "SCARLET": "Lust",
    "INDIGO": "Pride"
}

def convert_resource(resource_name):
    """Sin Color 이름을 변환하는 함수. 매핑되지 않은 경우 원래 이름을 반환."""
    return resource_mapping.get(resource_name, resource_name)

def find_sin_for_color(url):
    # URL에 color_name을 추출
    color_name = url.split('/')[-1].split('.')[0].replace("attr_", "")
    # 해당 색상이 resource_mapping에 있는 확인
    for color, sin in resource_mapping.items():
        if color_name in color:
            return sin
    return None


class Scraper:
    """웹 페이지 크롤링을 위한 클래스"""
    def __init__(self, container, base_url, img_url):
        self.container = container
        self.base_url = base_url
        self.img_url = img_url
        self.uptie_level = 1

    async def handle_request(self, route, request):
        """이미지와 폰트 리소스 로드를 차단."""
        if request.resource_type in ["image", "font"]:
            await route.abort()
        else:
            await route.continue_()
            
    async def scrape(self):
        """크롤링 메인 함수"""
        async with async_playwright() as p:
            browser = await p.chromium.launch(headless=True)
            page = await browser.new_page()
            await page.route("**/*", self.handle_request)
            # Step 1: /identity 페이지에서 기본 정보 크롤링
            #await self.crawl_identity_page(page)

            # Step 2: /identity/{identity_id} 상세 페이지 크롤링
            #identity_ids_to_load = ["10104", "10310", "10710","10808", "10911", "11005"]
            #identity_ids_to_load = ["10104", "10911"]#,"10310", "10710","10808", "11005", "11210"]
            #identity_ids_to_load = ["10911","10310"]#, "10710","10808", "11005", "11210"]
            identity_ids_to_load = ["10911"]#, "10611"]#,"10711"]#, "10710","10808", "11005", "11210"]
            self.container.load_identities_by_id('./csv/identities.csv', identity_ids_to_load)
            await self.crawl_identity_details_for_all(page)
            #self.container.display_identities()
            #self.container.export_identities_data("identities.csv")
            #self.container.export_level_stats_data("Levelstats.csv")
            #self.container.export_uptie_stats_data("Uptiestats.csv")
            #self.container.export_resist_info_data("ResistInfos.csv")
            #self.container.export_stagger_threshold_data("StaggerThresholds.csv")
            #self.container.export_identity_skillset_data("IdentitiesSkills.csv")
            page.set_default_timeout(10000)
            #await page.wait_for_timeout(100000)
            await browser.close()

    async def get_identity_id_from_row(self, row):
        """테이블 행(row)에서 인격 ID를 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_url = await identity_cell.get_attribute('href')
            identity_id = identity_url.split('/identity/')[-1]
            return identity_id
        return "Unknown"

    async def get_identity_name_from_row(self, row):
        """테이블 행(row)에서 인격 이름을 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_name = (await identity_cell.inner_text()).replace("\n", " ").strip()
            return identity_name
        return "Unknown"
        
    async def crawl_identity_page(self, page):
        """/identity 페이지로 이동하여 인격 기본 정보를 추출."""
        await page.goto(self.base_url + "/identity")
        await self.extract_identity_page(page)

    async def extract_identity_page(self, page):
        """/identity 페이지의 모든 테이블 행에서 인격 정보를 추출."""
        identity_rows = await page.query_selector_all('tr')
        if len(identity_rows) > 1:
            for idx, identity_row in enumerate(identity_rows):
                if idx == 0:
                    continue # 0번째요소이름을 보여주는 열은 건너뜀
                await self.scrape_identity_name_and_identity_id(identity_row)
                await self.extract_rarity(identity_row)
                for i in range(1, 4):  # 스킬1~3 정보 추출
                    await self.extract_atk_and_resource(identity_row, i)
                await self.extract_defense_and_resource(identity_row)

    async def scrape_identity_name_and_identity_id(self, row):
        """인격 번호와 이름을 추출하여 DataContainer에 저장."""
        try:
            identity_id = await self.get_identity_id_from_row(row)
            identity_name = await self.get_identity_name_from_row(row)
            identity = Identity(identity_id)
            self.container.add_identity(identity)
            self.container.update_identity_name(identity_id, identity_name)
        except Exception as e:
            print(f"Error extracting identity name and identity id: {e}")

    async def get_element_attribute(self, element, selector, attribute):
        """요소에서 지정된 속성(attribute)을 추출."""
        try:
            target_element = await element.query_selector(selector)
            if target_element:
                return await target_element.get_attribute(attribute)
        except Exception as e:
            print(f"Error getting element attribute: {e}")
        return None
    
    async def extract_rarity(self, row):
        """테이블 행(row)에서 희귀도를 추출하여 업데이트."""
        try:
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
                sibling = await parent.evaluate_handle('node => node.nextElementSibling')
                if sibling:
                    rarity_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/rarity_"]', 'src')
                    if rarity_img_src:
                        rarity_level = rarity_img_src.split('_')[-1].split('.')[0]  # rarity_1.png -> 1
                        identity_id = await self.get_identity_id_from_row(row)
                        self.container.update_identity_rarity(identity_id, rarity_level)
                    else:
                        print("Rarity image not found in the next sibling TD.")
        except Exception as e: 
            print(f"Error extracting rarity: {e}")

        return "Unknown"

    async def extract_atk_and_resource(self, row, skill_number):
        """ 공격 타입(ATK Type)과 자원(Attribute) 추출 함수 - 스킬 번호에 따라 처리 """
        try:
            #print(f"Attempting to locate skill{skill_number} type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 해당 스킬 번호에 맞는 형제로 이동
                sibling = parent
                for _ in range(skill_number + 1):  # 스킬 번호에 맞게 이동
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                atk_type, resource = "Unknown", "Unknown"
        
                if sibling:
                    # 공격 타입 이미지 (atk_type) 찾기
                    atk_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/atk_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if atk_type_img_src:
                        atk_type = atk_type_img_src.split('_')[-1].split('.')[0]  # atk_type_SLASH.png -> SLASH
                        #print(f"Extracted attack type: {atk_type}")
                        self.container.update_skill(identity_id, skill_number, atk_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted resource: {resource}")
                        
                        # 리소스를 변환
                        resource = convert_resource(resource)
                        #print(f"Converted resource: {resource}")
                        self.container.update_sin_affinity(identity_id, skill_number, resource)
                        
        except Exception as e:
            print(f"Error extracting skill{skill_number} type and resource: {e}")

    async def extract_defense_and_resource(self, row):
        """ 방어 타입(Def Type)과 자원(Attribute) 추출 함수 """
        try:
            #print("Attempting to locate defense type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 다섯 번째 형제 td로 이동 (방어 관련 정보가 있는 <td>)
                sibling = parent
                for _ in range(5):  # 방어 정보는 5번째 형제에 있음
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                def_type, def_resource = "Unknown", "Unknown"
        
                if sibling:
                    # 방어 타입 이미지 (def_type) 찾기
                    def_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/def_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if def_type_img_src:
                        def_type = def_type_img_src.split('_')[-1].split('.')[0]  # def_type_GUARD.png -> GUARD
                        #print(f"Extracted defense type: {def_type}")
                        self.container.update_skill(identity_id, 'defense', def_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        def_resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted defense resource: {def_resource}")
                        
                        # 방어 리소스를 변환
                        def_resource = convert_resource(def_resource)
                        #print(f"Converted defense resource: {def_resource}")
                        self.container.update_sin_affinity(identity_id, 'defense', def_resource)
 
        except Exception as e:
            print(f"Error extracting defense type and resource: {e}")

    async def crawl_identity_details_for_all(self, page):
        """모든 인격의 세부 정보를 크롤링."""
        for identity in self.container.identities:
            await page.wait_for_load_state("networkidle")
            await page.goto(self.base_url + "/identity/"+identity.identity_id)
            """ 레벨과 동기화와 관련 없는 참관타 저항은 밑에서 변경하는 것과 연관성이 적기 때문에 위에서 처리한다.
            print(f"Identity_id: {identity.identity_id}")
            results = []
            resist_type_imgs = [f"{self.img_url}/ui/atk_type_HIT.png",
                                f"{self.img_url}/ui/atk_type_PENETRATE.png",
                                f"{self.img_url}/ui/atk_type_SLASH.png"]
            for resist_type_img in resist_type_imgs:
                result = await self.extract_resist_type(page, resist_type_img, 1)
                results.append(result)
            self.container.add_resist_info(identity.identity_id, results[0], results[1], results[2])
            """
            """
            # Stagger Threshold 값 추출
            stagger_threshold = await self.extract_stagger_threshold_values(page)
            self.container.add_stagger_threshold(identity.identity_id, stagger_threshold[0], stagger_threshold[1], stagger_threshold[2])
            """
            await self.initialize_uptie_and_level(page)
            """ 레벨에 따른 level, hp, defense 다르게 되어 있는 것을 출력 확인 객체로 들어가서 csv로 옮기는 과정까지 진행
            for i in range(1, identity.level_max+1):
                await self.move_slider_to_value(page,i)
                await self.extract_level_status(page, identity.identity_id, i)
            """
            """ 동기화에 따른 speed 다르게 되어 있는 것을 출력 확인 객체로 들어가서 csv로 옮기는 과정까지 진행
            for i in range(1, identity.uptie_max+1):
                await self.uptie_click(page,i)
                await self.extract_uptie_status(page, identity.identity_id, i)
            """
            """
                인격에 대한 skill 정보를 가져오는 메소드 위에서 출력하는 csv파일로 인격이 가지고 있는 스킬 정보를 내보낸다.
            await self.crawl_skills(page, identity.identity_id)
            """
            await self.crawl_influence_behavior(page, identity.identity_id)
            
    async def initialize_uptie_and_level(self, page):
        await self.uptie_click(page, 1)
        await self.move_slider_to_value(page, 1)
        self.uptie_level = 1
        #return 1
    
    async def uptie_click(self, page, phase):
        fab_button_selector = ".q-btn.q-btn--fab"
        await page.wait_for_selector(fab_button_selector)
        await page.click(fab_button_selector)
        #print(f"[{setup_uptie_button}] 상위 동기화 버튼 클릭 완료")
        setup_uptie_button = f'a:has(img[src="{self.img_url}/ui/uptie_{phase}.png"])'
        await page.wait_for_selector(setup_uptie_button)
        await page.click(setup_uptie_button)
        #print(f"{phase} 동기화 버튼 클릭 완료")

    async def move_slider_to_value(self, page, target_value):
        """
        슬라이더를 주어진 값으로 이동시킵니다.
        target_value는 슬라이더의 최소값부터 최대값 사이여야 합니다.
        """
        slider_selector = ".q-slider"
        # 슬라이더의 범위 확인
        min_value = int(await page.get_attribute(slider_selector, "aria-valuemin"))
        max_value = int(await page.get_attribute(slider_selector, "aria-valuemax"))
    
        if target_value < min_value or target_value > max_value:
            raise ValueError(f"Target value {target_value} is out of bounds ({min_value}, {max_value})")
    
        # 슬라이더 thumb 요소와 위치 가져오기
        slider_thumb = await page.query_selector(f"{slider_selector} .q-slider__thumb")
        thumb_box = await slider_thumb.bounding_box()
        
        # 슬라이더의 현재 위치 및 목표 위치 계산
        track = await page.query_selector(f"{slider_selector} .q-slider__track")
        track_box = await track.bounding_box()
        
        # 목표 x 위치 계산 (슬라이더 트랙 내에서 비율 기반)
        target_percentage = (target_value - min_value) / (max_value - min_value)
        target_x = track_box["x"] + target_percentage * track_box["width"]
        current_x = thumb_box["x"] + thumb_box["width"] / 2
        y = thumb_box["y"] + thumb_box["height"] / 2
    
        # 슬라이더를 드래그하여 목표 위치로 이동
        await page.mouse.move(current_x, y)
        await page.mouse.down()
        await page.mouse.move(target_x, y, steps=10)  # 부드럽게 이동
        await page.mouse.up()
    
        #print(f"슬라이더를 {target_value}로 이동 완료.")
    
        # 슬라이더 값 확인 및 강제 이벤트 트리거
        await page.evaluate(
            f"""
            (slider) => {{
                slider.dispatchEvent(new Event('input'));  // oninput 이벤트 트리거
                slider.dispatchEvent(new Event('change'));  // onchange 이벤트 트리거
            }}
            """,
            await page.query_selector(slider_selector)
        )
    
        # 이동 후 값 확인
        current_value = await page.get_attribute(slider_selector, "aria-valuenow")
        if current_value != str(target_value):
            print(f"경고: 슬라이더 값이 예상과 다릅니다. 현재 값: {current_value}, 예상 값: {target_value}")

    async def get_second_status_element(self, page):
        """
        두 번째 'Status' 요소에 접근하여 반환하는 메소드
        """
        status_elements = await page.query_selector_all("text=Status")
        if len(status_elements) > 1:
            second_status_element = status_elements[1]
            #print("두 번째 'Status' 요소에 접근합니다.")
            return second_status_element
        else:
            #print("두 번째 'Status' 요소가 존재하지 않습니다.")
            return None

    async def extract_level_status(self, page, identity_id, level):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 레벨에 따라 HP와 Defense가 변한다.
        hp_value = await self.extract_hp(second_status_element)
        defense_value = await self.extract_defense(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Level={level}, HP={hp_value}, Defense={defense_value}")
        self.container.add_level_stat(identity_id,level, hp_value, defense_value)
    
    async def extract_hp(self, status_element):
        # HP 추출
        hp_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].children[0].nextElementSibling.textContent.trim()"
        )
        return hp_value
    
    async def extract_defense(self, status_element):
        # Defense 추출
        defense_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return defense_value

    async def extract_uptie_status(self, page, identity_id, uptie):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 uptie에 따라 speed는 변한다.
        speed_value = await self.extract_speed(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Uptie={uptie}, Speed={speed_value}")
        self.container.add_uptie_stat(identity_id,uptie, speed_value)

    async def extract_speed(self, status_element):
        # Speed 추출
        speed_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return speed_value

    async def extract_resist_type(self, page, image_src, depth):
        # 이미지 요소 찾기
        #print(image_src)
        image = await page.query_selector(f'img[src="{image_src}"]')
        #print(image)
        if not image:
            return "Unknown"
    
        # 이미지의 부모 요소를 찾아서 올라감
        parent = image
        for _ in range(depth):
            parent = await parent.evaluate_handle('node => node.parentElement')
            if not parent:
                return "Unknown"
    
        # 부모의 형제 요소 찾기
        sibling = await parent.evaluate_handle('node => node.nextElementSibling')
        
        if sibling:
            value = await sibling.evaluate('node => node.textContent')
            if value:
                return value.strip()
        
        return "Unknown"

    async def extract_stagger_threshold_values(self, page):
        # 'div.row.center.flex-center.q-pt-xs' 요소들 찾기
        elements = await page.query_selector_all('div.row.center.flex-center.q-pt-xs')
        
        # 각 요소의 텍스트 내용 출력하여 'Stagger Threshold' 요소를 찾기
        for element in elements:
            text = await element.text_content()
            
            if text and "Stagger Threshold" in text:
                # 'Stagger Threshold'가 있는 요소를 찾았다면, 그 이후 형제 요소들에서 값을 추출
                parent = await element.evaluate_handle('node => node.parentElement')
                sibling_elements = await parent.query_selector_all('div.row.justify-evenly div')  # Threshold 값이 들어있는 div들
                
                stagger_values = []
                for sibling in sibling_elements:
                    text_content = await sibling.text_content()
                    match = re.search(r'(\d+%)', text_content)
                    if match:
                        stagger_values.append(match.group(1))  # 퍼센트 값만 리스트에 추가
                while len(stagger_values) < 3:
                    stagger_values.append(np.nan)
                #print(stagger_values)
                return stagger_values  # 추출된 값(또는 np.nan 포함)을 반환
        
        return [np.nan, np.nan, np.nan]  # 만약 'Stagger Threshold'를 찾지 못한 경우 기본값 반환

    async def crawl_skills(self, page, identity_id):
        await self.crawl_skill(page, identity_id, "skill 1")
        await self.crawl_skill(page, identity_id, "skill 2")
        await self.crawl_skill(page, identity_id, "skill 3")
        await self.crawl_skill(page, identity_id, "defense")


    async def crawl_skill(self, page, identity_id, skill_type):
        skills_elements = await page.query_selector_all('div.skilltag')
        for skill_element in skills_elements:
            skill_text = await skill_element.text_content()
            if skill_text.strip().lower() == skill_type:
                await self.process_skill(skill_element, page, identity_id, skill_type)
        
    async def process_skill(self, skill_element, page, identity_id, skill_type):
        # skill_type이 "skill 3"일 경우 uptie_value가 3 이상일 때만 정보를 저장
        for uptie_value in range(1, 5):  # Uptie levels 1 to 4
            # "skill 3"은 uptie_value가 3 이상일 때만 처리
            if skill_type == "skill 3" and uptie_value < 3:
                continue  # skip if uptie_value is less than 3 for skill 3
    
            # uptie_level에 맞게 클릭
            await self.uptie_click(page, uptie_value)
            if skill_type != "skill 3" and uptie_value > 1:
                # Uptie 정보 갱신
                await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
                continue
            elif skill_type == "skill 3" and uptie_value > 3:
                # Uptie 정보 갱신
                await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
                continue
            # 스킬의 세부 정보를 추출
            skill_details = await self.extract_skill_details(skill_element, skill_type)
            skill_name = skill_details["skill_name"]
    
            # 스킬 정보를 저장
            self.container.add_skill(
                identity_id, skill_type, 
                skill_name, skill_details["coin_count"],
                skill_details["init_level"], skill_details["init_remain"], 
                skill_details["atk_weight"]
            )
            print(f"Added skill: {skill_type}:{skill_name}")
    
            # Uptie 정보 갱신
            await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
    
    async def extract_skill_details(self, skill_element, skill_type):
        # 스킬 세부 정보를 추출하는 함수
        skill_name = await self.skill_name(skill_element)
        coin_count = await self.coin_count(skill_element, ".col-auto > .column > .row")
        init_level = await self.init_level(skill_element)
        init_remain = await self.init_remain(skill_element, skill_type, "div.row.justify-start > div.attr.flex.flex-center > div.q-px-sm")
        atk_weight = await self.atk_weight(skill_element, skill_type)
        return {
            "skill_name": skill_name,
            "coin_count": coin_count,
            "init_level": init_level,
            "init_remain": init_remain,
            "atk_weight": atk_weight
        }
    
    async def update_uptie_info(self, page, skill_element, identity_id, skill_name, skill_type, uptie_value):
        # Uptie 정보를 추출하고 업데이트하는 함수
        uptie_info = await self.extract_uptie_info(skill_element)
        effect_data = await self.skill_effects(skill_element)
        self.container.add_skill_uptie_info(
            identity_id, skill_name, 
            uptie_value, uptie_info["skill_power"], uptie_info["coin_power"], 
            effect_data.get("effectsByCondition")
        )
        print(
            f"Updated Uptie for {skill_name} (Uptie Level: {uptie_value}): "
            f"Skill Power: {uptie_info['skill_power']}, Coin Power: {uptie_info['coin_power']} "
            f"Skill Effect: {effect_data.get("effectsByCondition")}"
        )

    async def extract_uptie_info(self, skill_element):
        skill_data = await self.skill_and_coin_power(skill_element)
        return {
            "skill_power": skill_data.get("skillPower"),
            "coin_power": skill_data.get("coinPower")
        }
        
    async def crawl_influence_behavior(self, page, identity_id):
        influence_behaviors = await page.query_selector_all('div[align="right"]')
        # 텍스트를 기준으 추출하고 메소드로 파라미터 들어가 정보를 받는다.
        for influence_behavior in influence_behaviors:
            active_type = await influence_behavior.inner_text()
            active_type = active_type.strip()

            if active_type == "PASSIVE":
                """
                # PASSIVE에 대한 처리
                print(f"Processing PASSIVE logic for: {active_type}")
                # CALL method for PASSIVE
                """
                passive_data = await self.passive_effects(influence_behavior)
            elif active_type == "SUPPORT PASSIVE":
                """
                # SUPPORT PASSIVE에 대한 처리
                print(f"Processing SUPPORT PASSIVE logic for: {active_type}")
                # CALL method for SUPPORT PASSIVE
                """
                support_passive_data = await self.support_passive_effects(influence_behavior)
            elif active_type == "PANIC TYPE":
                """
                # PANIC TYPE에 대한 처리
                print(f"Processing PANIC TYPE logic for:{active_type}")
                # CALL method for PANIC TYPE
                """
                panic_type_data = await self.panic_effects(influence_behavior)

    async def skill_and_coin_power(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                // 조부모로 이동
                const grandparent = node.parentElement.parentElement;
                // 이전 형제 탐색
                const previousSibling = grandparent.previousElementSibling;
                if (!previousSibling) return {{ skillPower: null, coinPower: null }}; // 이전 형제가 없으면 기본값 반환
                // 이전 형제 내부의 .row.justify-around > div > div 탐색
                const targetDivs = previousSibling.querySelectorAll(".row.justify-around > div");
                if (targetDivs.length < 3) return {{ skillPower: null, coinPower: null }}; // 배열 길이 확인
                // 1st div: 스킬 위력
                const skillPower = targetDivs[0]?.querySelector('div')?.textContent?.trim() || null; 
                // 3rd div: 코인 파워 값 (3번째 div에서 텍스트 추출)
                const coinPower = targetDivs[2]?.textContent?.trim() || null;
                return {{ skillPower, coinPower }};
            }}
        """)


    async def coin_count(self, skill_element, hierarchy_selector):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector('{hierarchy_selector}');
                if (!targetElement) return 0;
                return targetElement.childElementCount;
            }}
        """)

    async def init_remain(self, skill_element, skill_type, hierarchy_selector):
        if skill_type == "defense":
            return -1
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector('{hierarchy_selector}');
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def skill_name(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector(".block");
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def init_level(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector("div.text-bold.text-h6.h6.q-pl-xs.q-pr-md");
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def atk_weight(self, skill_element, skill_type):
        if skill_type == "defense":
            return 0
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector("div.text-bold.text-h6.h6.q-pl-xs.q-pr-md");
                const doubleNextSibling = targetElement.nextElementSibling.nextElementSibling;
                if (!doubleNextSibling) return 0;
                return doubleNextSibling.textContent?.trim();
            }}
        """)

    async def skill_effects(self, skill_element):
        return await skill_element.evaluate("""
            node => {
                const parent = node.parentElement;
                const doubleNextSibling = parent.nextElementSibling?.nextElementSibling;
                if (!doubleNextSibling) return null;
    
                // doubleNextSibling에서 첫 번째 div.col 요소 찾기
                const textNode = doubleNextSibling.querySelector("div.col");
                if (!textNode) return null;
    
                const results = {
                    keywords: [], // 키워드 저장
                    conditions: [], // 조건 저장
                    effectsByCondition: {} // 조건별 효과
                };
    
                // 1. 조건 추출 (대괄호 텍스트)
                const conditionElements = textNode.querySelectorAll("span");
                conditionElements.forEach(span => {
                    let text = span.textContent?.trim();
                    if (text?.startsWith("[") && text?.endsWith("]")) {
                        // 부모의 전형제 노드 탐색
                        const sibling = span.parentElement.previousElementSibling;
                        if (sibling) {
                            const emblemImage = sibling.querySelector("img.emblem");
                            console.log("Sibling element found:", sibling); // 디버깅용
                            if (emblemImage) {
                                const src = emblemImage.getAttribute("src");
                                console.log("Image source:", src); // 디버깅용
                                const match = src.match(/ui_num_(\\d+)\\.png$/); // 정규식 수정
                                if (match) {
                                    const number = match[1]; // 숫자 추출
                                    text = `[Coin ${number} ${text.replace("[", "").replace("]", "")}]`;
                                    console.log("Updated condition:", text); // 디버깅용
                                }
                            }
                        }
                        results.conditions.push(text);
                        results.effectsByCondition[text] = []; // 조건별 효과를 초기화
                    }
                });
    
                // 2. 키워드 추출
                const keywordElements = textNode.querySelectorAll('a[href^="/en/keyword/"]');
                keywordElements.forEach(a => {
                    const span = a.querySelector('span.q-btn__content.text-center.col.items-center.q-anchor--skip.justify-center.row>span');
                    if (span) {
                        const keywordText = span.textContent.trim();
                        if (keywordText && !results.keywords.includes(keywordText)) {
                            results.keywords.push(keywordText); // 키워드 저장
                        }
                    }
                });
    
                // 3. 효과 추출
                const effectElements = textNode.querySelectorAll("span");
                let currentCondition = null;
                let currentEffect = []; // 초기화
                effectElements.forEach(span => {
                    let text = span.textContent?.trim();
                    if (!text) return;
    
                    // 조건 발견 시 currentCondition 변경
                    if (text.startsWith("[") && text.endsWith("]")) {
                        // 이전 조건의 효과 저장
                        if (currentCondition && currentEffect.length > 0) {
                            // 부모의 전형제 노드 탐색
                            const sibling = span.parentElement.previousElementSibling;
                            if (sibling) {
                                const emblemImage = sibling.querySelector("img.emblem");
                                if (emblemImage) {
                                    const src = emblemImage.getAttribute("src");
                                    const match = src.match(/ui_num_(\\d+)\\.png$/); // 정규식 수정
                                    if (match) {
                                        const number = match[1]; // 숫자 추출
                                        text = `[Coin ${number} ${text.replace("[", "").replace("]", "")}]`;
                                    }
                                }
                            }
                            results.effectsByCondition[currentCondition].push(currentEffect.join(' '));
                            currentEffect = []; // 초기화
                        }
                        currentCondition = text;
                        return;
                    }
    
                    // 현재 조건의 효과 처리
                    if (currentCondition) {
                        if (results.keywords.includes(text)) {
                            const keywordEffect = 'keyword:' + text;
                            if (!currentEffect.includes(keywordEffect)) {
                                currentEffect.push(keywordEffect);
                            }
                        } else {
                            currentEffect.push(text);
                        }
                    }
                });
    
                // 마지막 효과 저장
                if (currentCondition && currentEffect.length > 0) {
                    results.effectsByCondition[currentCondition].push(currentEffect.join(' '));
                }
    
                return results;
            }
        """)

    async def passive_effects(self, passive_element):
        passive_name = await self.behavior_name(passive_element)
        #print(passive_name)
    async def support_passive_effects(self, support_passive_element):
        support_passive_name = await self.behavior_name(support_passive_element)
        #print(support_passive_name)
    async def panic_effects(self, panic_element):
        panic_type = await self.behavior_name(panic_element)
        #print(panic_type)

    async def parent_prev_sibling(self, element):
        parent_element = await element.evaluate_handle('element => element.parentElement')
        if parent_element:
            #print("Parent element found")
            # 부모 안에서 특정 클래스 가진 요소 찾기
            previous_sibling = await parent_element.evaluate_handle('element => element.previousElementSibling')
            return previous_sibling
        return None

    async def behavior_name(self, element):
        parent_prev_sibling = await self.parent_prev_sibling(element)
        passive_name_element = await parent_prev_sibling.query_selector('span.block')
        if passive_name_element:
            passive_name = await passive_name_element.inner_text()
            return passive_name
        else:
            #print("Passive name element not found")
            return None
        
class Identity:
    """인격 정보를 저장하는 클래스"""
    def __init__(self, identity_id, name=None, rarity=None,\
                 skill_1_type=None, skill_1_sin_affinity=None,\
                 skill_2_type=None, skill_2_sin_affinity=None,\
                 skill_3_type=None, skill_3_sin_affinity=None,\
                 defense_type=None, defense_sin_affinity=None):
        self.identity_id = identity_id
        self.name = name
        self.rarity = rarity
        # 선언만 하는 부분 
        self.skill_1_type = skill_1_type 
        self.skill_1_sin_affinity = skill_1_sin_affinity 
        self.skill_2_type = skill_2_type 
        self.skill_2_sin_affinity = skill_2_sin_affinity 
        self.skill_3_type = skill_3_type 
        self.skill_3_sin_affinity = skill_3_sin_affinity 
        self.defense_type = defense_type 
        self.defense_sin_affinity = defense_sin_affinity
        self.level_max = 50
        self.uptie_max = 4
        self.level_stats = []  # LevelStat 객체 리스트
        self.uptie_stats = [] # UptieStat 객체 리스트
        self.resist_info = None
        self.stagger_threshold = None
        self.skillset = SkillSet()  # Skill정보를 담고 있는 skillset 객체 
        self.passive_infos = []  # 패시브 객체를 담고 있는 배열
        self.support_passive_infos = [] # 서포트 패시 객체를 담고 있는 배열
        self.panic_type = [] # 혼란상태 발생시 나타나는 효과를 담고 있는 객체 배열1

    def update_name(self, name):
        self.name = name

    def update_rarity(self, rarity):
        self.rarity = rarity

    def add_level_stat(self, level, hp, defense):
        self.level_stats.append(LevelStat(level, hp, defense))

    def add_uptie_stat(self, uptie, speed):
        self.uptie_stats.append(UptieStat(uptie, speed))

    def add_resist_info(self, resist_hit, resist_penetrate, resist_slash):
        self.resist_info = ResistInfo(resist_hit, resist_penetrate, resist_slash)

    def add_stagger_threshold(self, first_stagger, second_stagger, third_stagger):
        self.stagger_threshold = StaggerThreshold(first_stagger, second_stagger, third_stagger)

    def add_skill(self, skill_code, skill_name, coin_count, init_level, init_remain, atk_weight):
        self.skillset.add_skill(skill_code, Skill(skill_code, skill_name, coin_count, init_level, init_remain, atk_weight))
        
    def add_skill_uptie_info(self, skill_name, uptie_level, skill_power, coin_power, effect_by_condition):
        self.skillset.add_skill_uptie_info(skill_name, uptie_level, skill_power, coin_power, effect_by_condition)
        
    def update_details(self, page):
        # 세부 정보 업데이트 로직 (가정)
        pass
    #def init_uptie(self, page):
    #    init_uptie = 

# LevelStat 객체 정의
class LevelStat:
    def __init__(self, level, hp=None, defense=None):
        self.level = level
        self.hp = hp
        self.defense = defense
    def update_level(self, level):
        self.level = level
    def update_hp(self, hp):
        self.hp = hp
    def update_defense(self, defense):
        self.defense = defense

# UptieStat 객체 정의
class UptieStat:
    def __init__(self, uptie, speed=None):
        self.uptie = uptie
        self.speed = speed
    def update_speed(self, speed):
        self.speed = speed

# ResistInfo 객체 정의
class ResistInfo:
    def __init__(self, resist_hit=None, resist_penetrate=None, resist_slash=None):
        self.resist_hit = resist_hit
        self.resist_penetrate = resist_penetrate
        self.resist_slash = resist_slash

# ResistInfo 객체 정의
class StaggerThreshold:
    def __init__(self, first_stagger=None, second_stagger=None, third_stagger=None):
        self.first_stagger = first_stagger
        self.second_stagger = second_stagger
        self.third_stagger = third_stagger

class SkillSet:
    def __init__(self):

        self.skill_1_type = ""
        self.skill_2_type = ""
        self.skill_3_type = ""
        self.defense_skill_type = ""

        self.skill_1_sin_affinity = ""
        self.skill_2_sin_affinity = ""
        self.skill_3_sin_affinity = ""
        self.defense_skill_sin_affinity = ""

        self.skill_1_details = []
        self.skill_2_details = []
        self.skill_3_details = []
        self.defense_skill_details = []

    def set_skill_type(self, skill_code, skill_type, sin_affinity):
        if skill_code == "skill 1":
            self.skill_1_type = skill_type
            self.skill_1_sin_affinity = sin_affinity
        elif skill_code == "skill 2":
            self.skill_2_type = skill_type
            self.skill_2_sin_affinity = sin_affinity
        elif skill_code == "skill 3":
            self.skill_3_type = skill_type
            self.skill_3_sin_affinity = sin_affinity
        elif skill_code == "defense":
            self.defense_skill_type = skill_type
            self.defense_skill_sin_affinity = sin_affinity
    
    def add_skill(self, skill_code, skill):
        if skill_code == "skill 1":
            self.skill_1_details.append(skill)
            skill.skill_type = self.skill_1_type
            skill.sin_affinity = self.skill_1_sin_affinity
        elif skill_code == "skill 2":
            self.skill_2_details.append(skill)
            skill.skill_type = self.skill_2_type
            skill.sin_affinity = self.skill_2_sin_affinity
        elif skill_code == "skill 3":
            self.skill_3_details.append(skill)
            skill.skill_type = self.skill_3_type
            skill.sin_affinity = self.skill_3_sin_affinity
        elif skill_code == "defense":
            self.defense_skill_details.append(skill)
            skill.skill_type = self.defense_skill_type 
            skill.sin_affinity = self.defense_skill_sin_affinity

    def get_skill_by_name(self, skill_name):
        for skill_list in [self.skill_1_details, self.skill_2_details, self.skill_3_details, self.defense_skill_details]:
            for skill in skill_list:
                result = skill.is_skill_by_name(skill_name)
                if result:
                    return result
        return None

    def add_skill_uptie_info(self, skill_name, uptie_level, skill_power, coin_power, effect_by_condition):
        skill = self.get_skill_by_name(skill_name)
        skill.add_uptie_info(uptie_level, skill_power, coin_power, effect_by_condition)
        

class Skill:
    def __init__(self, skill_code, skill_name, coin_count, init_level, init_remain, atk_weight):
        self.skill_code = skill_code # "Skill 1"/"Skill 2"/"Skill 3"/"Defense" '''o'''
        self.skill_type = None
        self.sin_affinity = None
        self.skill_name = skill_name # 스킬 이름 '''o'''
        self.coin_count = coin_count # 동전 개수 '''o'''
        self.init_level = init_level # 스킬이 초기 배정받은 레벨, 인격 레벨에 - 1 한 값을 더해주면 스킬의 현 level을 구할 수 있다. '''o'''
        self.init_remain = init_remain #'''o'''
        self.atk_weight = atk_weight #공격 가중치 값이다. 방어스킬에는 없기 때문에 None으로 설정했다. '''o'''
        self.remain_count = init_remain # 실제 스킬이 작동하며 횟수가 차감되 나중에 모두 차감되면 다음 턴에 다시 초기횟수 리필된다.
        self.uptie_info = []  

    def is_skill_by_name(self, skill_name):
        if self.skill_name == skill_name:
            return self
        return None
        
    def add_uptie_info(self, uptie_level, skill_power, coin_power, effects_by_condition):
        self.uptie_info.append(UptieInfo(uptie_level, skill_power, coin_power, effects_by_condition))

class UptieInfo:
    def __init__(self, uptie_level, skill_power, coin_power, effects_by_condition):
        self.uptie_level = uptie_level   #'''o'''
        self.skill_power = skill_power   #'''o'''
        self.coin_power = coin_power     #'''o'''
        self.skill_effect = effects_by_condition

# 추상화된 EffectType 클래스
class EffectType:
    def __init__(self, name, description):
        self.name = name  # Effect 이름
        self.description = description  # Effect 설명

# Passive 클래스
class Passive(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)  # condition이 있는지 여부 확인

# SupportPassive 클래스
class SupportPassive(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)

# PanicType 클래스
class PanicType(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)

# Keyword 클래스
class Keyword:
    def __init__(self, keyword, explain=None, dispellable=None):
        self.keyword = keyword
        self.explain = explain
        self.dispellable = dispellable

      
class DataContainer:
    """크롤링 데이터를 관리하는 클래스"""
    def __init__(self):
        self.identities = []

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.level_max

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.uptie_max

    def update_identity_name(self, identity_id, identity_name):
        """특정 ID의 이름을 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_name(identity_name)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_identity_rarity(self, identity_id, rarity):
        """특정 ID의 희귀도를 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_rarity(rarity)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_skill(self, identity_id, skill_number, new_type):
        """
        특정 identity의 skill_type을 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_type: 새 skill_type
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_type = new_type
                elif skill_number == 2:
                    identity.skill_2_type = new_type
                elif skill_number == 3:
                    identity.skill_3_type = new_type
                elif skill_number == "defense":
                    identity.defense_type = new_type
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def update_sin_affinity(self, identity_id, skill_number, new_affinity):
        """
        특정 identity의 sin_affinity를 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_affinity: 새 sin_affinity
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_sin_affinity = new_affinity
                elif skill_number == 2:
                    identity.skill_2_sin_affinity = new_affinity
                elif skill_number == 3:
                    identity.skill_3_sin_affinity = new_affinity
                elif skill_number == "defense":
                    identity.defense_sin_affinity = new_affinity
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def add_identity(self, identity):
        """새로운 인격 데이터를 추가."""
        self.identities.append(identity)

    def add_level_stat(self, identity_id, level, hp, defense):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_level_stat(level, hp, defense)

    def add_uptie_stat(self, identity_id, uptie, speed):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_uptie_stat(uptie, speed)

    def add_resist_info(self, identity_id, resist_hit, resist_penetrate, resist_slash):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_resist_info(resist_hit, resist_penetrate, resist_slash)

    def add_stagger_threshold(self, identity_id, first_stagger, second_stagger, third_stagger):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_stagger_threshold(first_stagger, second_stagger, third_stagger)

    
    def add_skill(self, identity_id, skill_code, skill_name, coin_count, init_level, init_remain, atk_weight):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_code == "skill 1":
                    identity.skillset.set_skill_type(skill_code, identity.skill_1_type, identity.skill_1_sin_affinity)
                elif skill_code == "skill 2":
                    identity.skillset.set_skill_type(skill_code, identity.skill_2_type, identity.skill_2_sin_affinity)
                elif skill_code == "skill 3":
                    identity.skillset.set_skill_type(skill_code, identity.skill_3_type, identity.skill_3_sin_affinity)
                elif skill_code == "defense":
                    identity.skillset.set_skill_type(skill_code, identity.defense_type, identity.defense_sin_affinity)
                identity.add_skill(skill_code, skill_name, coin_count, init_level, init_remain, atk_weight)

    def add_skill_uptie_info(self, identity_id, skill_name, uptie_level, skill_power, coin_power, effects_by_condition):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_skill_uptie_info(skill_name, uptie_level, skill_power, coin_power, effects_by_condition)
        
    def display_identities(self):
        """Identity, ID, Name, Rarity 정보를 출력"""
        if not self.identities:
            print("No identities found.")
        else:
            # 데이터를 리스트로 변환
            table = []
            for idx, identity in enumerate(self.identities, 1):
                table.append([
                    idx, identity.identity_id, identity.name, identity.rarity,
                    identity.skill_1_type, identity.skill_1_sin_affinity,
                    identity.skill_2_type, identity.skill_2_sin_affinity,
                    identity.skill_3_type, identity.skill_3_sin_affinity,
                    identity.defense_type, identity.defense_sin_affinity
                ])
            
            # 테이블 헤더 정의
            headers = [
                "Index", "ID", "Name", "Rarity",
                "Skill_1 Type", "Skill_1 Sin", "Skill_2 Type", "Skill_2 Sin",
                "Skill_3 Type", "Skill_3 Sin", "Defense Type", "Defense Sin"
            ]
            
            # 테이블 출력
            print(tabulate(table, headers=headers, tablefmt="fancy_grid"))

    def export_to_csv(self, filename, fieldnames, data):
        """
        공통적으로 사용 가능한 CSV 내보내기 메서드
        :param filename: 내보낼 파일 이름 (예: 'identity.csv', 'skill.csv')
        :param fieldnames: CSV의 헤더 리스트
        :param data: CSV로 내보낼 데이터 (리스트 형태)
        """
        try:
            # 현재작업 디렉토 기준으 'csv'폴더 없으면 생성
            if not os.path.exists('./csv'):
                os.makedirs('./csv')
            # 파일 경로 설정 (상대경로 'csv' 폴더 안에 저장)
            file_path = os.path.join('./csv', filename)
            with open(file_path, mode='w', newline='', encoding='utf-8') as file:
                writer = csv.DictWriter(file, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(data)
            print(f"Data successfully exported to {filename}")
        except Exception as e:
            print(f"Error exporting to CSV: {e}")

    def get_identity_data(self):
        """
        Identity 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "ID": identity.identity_id,
                "Name": identity.name,
                "Rarity": identity.rarity,
                "Skill_1_Type": identity.skill_1_type,
                "Skill_1_Sin_Affinity": identity.skill_1_sin_affinity,
                "Skill_2_Type": identity.skill_2_type,
                "Skill_2_Sin_Affinity": identity.skill_2_sin_affinity,
                "Skill_3_Type": identity.skill_3_type,
                "Skill_3_Sin_Affinity": identity.skill_3_sin_affinity,
                "Defense_Type": identity.defense_type,
                "Defense_Sin_Affinity": identity.defense_sin_affinity,
            }
            for identity in self.identities
        ]
        
    def export_identities_data(self, filename="identities.csv"):
        """Identity 데이터를 CSV로 내보내는 메서드"""
        identity_fieldnames = [
            "ID", "Name", "Rarity",
            "Skill_1_Type", "Skill_1_Sin_Affinity",
            "Skill_2_Type", "Skill_2_Sin_Affinity",
            "Skill_3_Type", "Skill_3_Sin_Affinity",
            "Defense_Type", "Defense_Sin_Affinity"
        ]
        identity_data = self.get_identity_data()  # Identity 데이터 가져오기
        self.export_to_csv(filename, identity_fieldnames, identity_data)  # CSV로 내보내기

    def load_identities_by_id(self, filename, identity_ids):
        """지정된 identity_ids 목록에 해당하는 Identity 데이터를 CSV에서 읽어옴."""
        identity_ids_set = set(identity_ids) # 중복 제거를 위 Set 사용
        with open(filename, mode='r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                if row["ID"] in identity_ids_set:
                    identity = Identity(
                        identity_id=row["ID"],
                        name=row["Name"],
                        rarity=row["Rarity"],
                        skill_1_type=row["Skill_1_Type"],
                        skill_1_sin_affinity=row["Skill_1_Sin_Affinity"],
                        skill_2_type=row["Skill_2_Type"],
                        skill_2_sin_affinity=row["Skill_2_Sin_Affinity"],
                        skill_3_type=row["Skill_3_Type"],
                        skill_3_sin_affinity=row["Skill_3_Sin_Affinity"],
                        defense_type=row["Defense_Type"],
                        defense_sin_affinity=row["Defense_Sin_Affinity"]
                    )
                    self.identities.append(identity)

    def get_level_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Level": level_stat.level,
                "HP": level_stat.hp,
                "Defense": level_stat.defense,
            }
            for identity in self.identities
            for level_stat in identity.level_stats
        ]

    def export_level_stats_data(self, filename="levelstats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        level_stats_fieldnames = ["Identity_ID", "Level", "HP", "Defense"]
        level_stats_data = self.get_level_stats_data()  # level_stats 데이터 가져오기
        self.export_to_csv(filename, level_stats_fieldnames, level_stats_data)  # CSV로 내보내기

    def get_uptie_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Uptie": uptie_stat.uptie,
                "Speed": uptie_stat.speed,
            }
            for identity in self.identities
            for uptie_stat in identity.uptie_stats
        ]

    def export_uptie_stats_data(self, filename="Uptiestats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        uptie_stats_fieldnames = ["Identity_ID", "Uptie", "Speed"]
        uptie_stats_data = self.get_uptie_stats_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, uptie_stats_fieldnames, uptie_stats_data)  # CSV로 내보내기

    def get_resist_info_data(self):
        """
        모든 Identity의 resist_info 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Resist_Hit": identity.resist_info.resist_hit,
                "Resist_Penetrate": identity.resist_info.resist_penetrate,
                "Resist_Slash": identity.resist_info.resist_slash,
            }
            for identity in self.identities
        ]

    def export_resist_info_data(self, filename="ResistInfos.csv"):
        """ResistInfo 데이터를 CSV로 내보내는 메서드"""
        resist_infos_fieldnames = ["Identity_ID", "Resist_Hit", "Resist_Penetrate", "Resist_Slash"]
        resist_infos_data = self.get_resist_info_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, resist_infos_fieldnames, resist_infos_data)  # CSV로 내보내기

    def export_uptie_stats_data(self, filename="Uptiestats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        uptie_stats_fieldnames = ["Identity_ID", "Uptie", "Speed"]
        uptie_stats_data = self.get_uptie_stats_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, uptie_stats_fieldnames, uptie_stats_data)  # CSV로 내보내기

    def get_stagger_threshold_data(self):
        """
        모든 Identity의 stagger_threshold 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "First_Stagger": identity.stagger_threshold.first_stagger,
                "Second_Stagger": identity.stagger_threshold.second_stagger,
                "Third_Stagger": identity.stagger_threshold.third_stagger,
            }
            for identity in self.identities
        ]

    def export_stagger_threshold_data(self, filename="StaggerThresholds.csv"):
        """StaggerThresholds 데이터를 CSV로 내보내는 메서드"""
        stagger_thresholds_fieldnames = ["Identity_ID", "First_Stagger", "Second_Stagger", "Third_Stagger"]
        stagger_thresholds_data = self.get_stagger_threshold_data()  # stagger_thresholds 데이터 가져오기
        self.export_to_csv(filename, stagger_thresholds_fieldnames, stagger_thresholds_data)  # CSV로 내보내기

    def get_skillset_data(self):
        """모든 Identity의 skillset 데이터를 export_to_csv을 포맷으로 변환"""
        return [
            {
                "Identity_ID": identity.identity_id,
                "Skill_Code": skill.skill_code,
                "Skill_Type": skill.skill_type,
                "Sin_Affinity": skill.sin_affinity,
                "Skill_Name": skill.skill_name,
                "Coin_Count": skill.coin_count,
                "Init_Level": skill.init_level,
                "Init_Remain": skill.init_remain,
                "Atk_Weight": skill.atk_weight,
                "Uptie_Level": uptie_info.uptie_level,
                "Skill_Power": uptie_info.skill_power,
                "Coin_Power": uptie_info.coin_power,
                "Effect_Data": json.dumps(uptie_info.skill_effect)
            }
            for identity in self.identities
            for skill_list in [identity.skillset.skill_1_details, identity.skillset.skill_2_details, identity.skillset.skill_3_details, identity.skillset.defense_skill_details]
            for skill in skill_list
            for uptie_info in skill.uptie_info
        ]

    def export_identity_skillset_data(self, filename="IdentitiesSkills.csv"):
        """IdentitiesSkills 데이터를 CSV로 내보내는 메서드"""
        identities_skills_fieldnames = ["Identity_ID", "Skill_Code", "Skill_Type", "Sin_Affinity", "Skill_Name", "Coin_Count", "Init_Level", "Init_Remain", "Atk_Weight", "Uptie_Level", "Skill_Power", "Coin_Power", "Effect_Data"]
        identities_skills_data = self.get_skillset_data()
        self.export_to_csv(filename, identities_skills_fieldnames, identities_skills_data)
            


async def main():
    data_container = DataContainer()
    scraper = Scraper(container=data_container, base_url="https://limbus.kusoge.xyz", img_url="https://img.kusoge.xyz/limbus/img")
    await scraper.scrape()

await main() 

Bloodfeast
Hardblood Thorn
"Blossom with Blood..."
Panic


In [85]:
import os
import csv
import re
import json
import numpy as np
import pandas as pd
from playwright.async_api import async_playwright
from tabulate import tabulate

# Sin Color 이름 변환을 위한 매핑
resource_mapping = {
    "AZURE": "Gloom",
    "VIOLET": "Envy",
    "AMBER": "Sloth",
    "SHAMROCK": "Gluttony",
    "CRIMSON": "Wrath",
    "SCARLET": "Lust",
    "INDIGO": "Pride"
}

def convert_resource(resource_name):
    """Sin Color 이름을 변환하는 함수. 매핑되지 않은 경우 원래 이름을 반환."""
    return resource_mapping.get(resource_name, resource_name)

def find_sin_for_color(url):
    # URL에 color_name을 추출
    color_name = url.split('/')[-1].split('.')[0].replace("attr_", "")
    # 해당 색상이 resource_mapping에 있는 확인
    for color, sin in resource_mapping.items():
        if color_name in color:
            return sin
    return None


class Scraper:
    """웹 페이지 크롤링을 위한 클래스"""
    def __init__(self, container, base_url, img_url):
        self.container = container
        self.base_url = base_url
        self.img_url = img_url
        self.uptie_level = 1

    async def handle_request(self, route, request):
        """이미지와 폰트 리소스 로드를 차단."""
        if request.resource_type =="image":
            #print(f"{request.url}")
            img_url = request.url
            if  re.match(f"{self.img_url}/ui/attr.*\\.png", img_url):
                await route.continue_()
            else:
                await route.fulfill(
                    status=200,
                    content_type="image/png",
                    body=b""
                )
        elif request.resource_type == "font":
            await route.abort()
        else:
            await route.continue_()
            
    async def scrape(self):
        """크롤링 메인 함수"""
        async with async_playwright() as p:
            browser = await p.chromium.launch(headless=False)
            page = await browser.new_page()
            #await page.evaluate('window.scrollTo(0, document.body.scrollHeight)')
            await page.route("**/*", self.handle_request)
            # Step 1: /identity 페이지에서 기본 정보 크롤링
            #await self.crawl_identity_page(page)

            # Step 2: /identity/{identity_id} 상세 페이지 크롤링
            #identity_ids_to_load = ["10104", "10310", "10710","10808", "10911", "11005"]
            #identity_ids_to_load = ["10104", "10911"]#,"10310", "10710","10808", "11005", "11210"]
            #identity_ids_to_load = ["10911","10310"]#, "10710","10808", "11005", "11210"]
            identity_ids_to_load = ["10911"]#, "10611"]#,"10711"]#, "10710","10808", "11005", "11210"]
            self.container.load_identities_by_id('./csv/identities.csv', identity_ids_to_load)
            await self.crawl_identity_details_for_all(page)
            #self.container.display_identities()
            #self.container.export_identities_data("identities.csv")
            #self.container.export_level_stats_data("Levelstats.csv")
            #self.container.export_uptie_stats_data("Uptiestats.csv")
            #self.container.export_resist_info_data("ResistInfos.csv")
            #self.container.export_stagger_threshold_data("StaggerThresholds.csv")
            #self.container.export_identity_skillset_data("IdentitiesSkills.csv")
            #page.set_default_timeout(10000)
            await page.wait_for_timeout(100000)
            await browser.close()

    async def get_identity_id_from_row(self, row):
        """테이블 행(row)에서 인격 ID를 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_url = await identity_cell.get_attribute('href')
            identity_id = identity_url.split('/identity/')[-1]
            return identity_id
        return "Unknown"

    async def get_identity_name_from_row(self, row):
        """테이블 행(row)에서 인격 이름을 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_name = (await identity_cell.inner_text()).replace("\n", " ").strip()
            return identity_name
        return "Unknown"
        
    async def crawl_identity_page(self, page):
        """/identity 페이지로 이동하여 인격 기본 정보를 추출."""
        await page.goto(self.base_url + "/identity")
        await self.extract_identity_page(page)

    async def extract_identity_page(self, page):
        """/identity 페이지의 모든 테이블 행에서 인격 정보를 추출."""
        identity_rows = await page.query_selector_all('tr')
        if len(identity_rows) > 1:
            for idx, identity_row in enumerate(identity_rows):
                if idx == 0:
                    continue # 0번째요소이름을 보여주는 열은 건너뜀
                await self.scrape_identity_name_and_identity_id(identity_row)
                await self.extract_rarity(identity_row)
                for i in range(1, 4):  # 스킬1~3 정보 추출
                    await self.extract_atk_and_resource(identity_row, i)
                await self.extract_defense_and_resource(identity_row)

    async def scrape_identity_name_and_identity_id(self, row):
        """인격 번호와 이름을 추출하여 DataContainer에 저장."""
        try:
            identity_id = await self.get_identity_id_from_row(row)
            identity_name = await self.get_identity_name_from_row(row)
            identity = Identity(identity_id)
            self.container.add_identity(identity)
            self.container.update_identity_name(identity_id, identity_name)
        except Exception as e:
            print(f"Error extracting identity name and identity id: {e}")

    async def get_element_attribute(self, element, selector, attribute):
        """요소에서 지정된 속성(attribute)을 추출."""
        try:
            target_element = await element.query_selector(selector)
            if target_element:
                return await target_element.get_attribute(attribute)
        except Exception as e:
            print(f"Error getting element attribute: {e}")
        return None
    
    async def extract_rarity(self, row):
        """테이블 행(row)에서 희귀도를 추출하여 업데이트."""
        try:
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
                sibling = await parent.evaluate_handle('node => node.nextElementSibling')
                if sibling:
                    rarity_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/rarity_"]', 'src')
                    if rarity_img_src:
                        rarity_level = rarity_img_src.split('_')[-1].split('.')[0]  # rarity_1.png -> 1
                        identity_id = await self.get_identity_id_from_row(row)
                        self.container.update_identity_rarity(identity_id, rarity_level)
                    else:
                        print("Rarity image not found in the next sibling TD.")
        except Exception as e: 
            print(f"Error extracting rarity: {e}")

        return "Unknown"

    async def extract_atk_and_resource(self, row, skill_number):
        """ 공격 타입(ATK Type)과 자원(Attribute) 추출 함수 - 스킬 번호에 따라 처리 """
        try:
            #print(f"Attempting to locate skill{skill_number} type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 해당 스킬 번호에 맞는 형제로 이동
                sibling = parent
                for _ in range(skill_number + 1):  # 스킬 번호에 맞게 이동
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                atk_type, resource = "Unknown", "Unknown"
        
                if sibling:
                    # 공격 타입 이미지 (atk_type) 찾기
                    atk_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/atk_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if atk_type_img_src:
                        atk_type = atk_type_img_src.split('_')[-1].split('.')[0]  # atk_type_SLASH.png -> SLASH
                        #print(f"Extracted attack type: {atk_type}")
                        self.container.update_skill(identity_id, skill_number, atk_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted resource: {resource}")
                        
                        # 리소스를 변환
                        resource = convert_resource(resource)
                        #print(f"Converted resource: {resource}")
                        self.container.update_sin_affinity(identity_id, skill_number, resource)
                        
        except Exception as e:
            print(f"Error extracting skill{skill_number} type and resource: {e}")

    async def extract_defense_and_resource(self, row):
        """ 방어 타입(Def Type)과 자원(Attribute) 추출 함수 """
        try:
            #print("Attempting to locate defense type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 다섯 번째 형제 td로 이동 (방어 관련 정보가 있는 <td>)
                sibling = parent
                for _ in range(5):  # 방어 정보는 5번째 형제에 있음
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                def_type, def_resource = "Unknown", "Unknown"
        
                if sibling:
                    # 방어 타입 이미지 (def_type) 찾기
                    def_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/def_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if def_type_img_src:
                        def_type = def_type_img_src.split('_')[-1].split('.')[0]  # def_type_GUARD.png -> GUARD
                        #print(f"Extracted defense type: {def_type}")
                        self.container.update_skill(identity_id, 'defense', def_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        def_resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted defense resource: {def_resource}")
                        
                        # 방어 리소스를 변환
                        def_resource = convert_resource(def_resource)
                        #print(f"Converted defense resource: {def_resource}")
                        self.container.update_sin_affinity(identity_id, 'defense', def_resource)
 
        except Exception as e:
            print(f"Error extracting defense type and resource: {e}")

    async def crawl_identity_details_for_all(self, page):
        """모든 인격의 세부 정보를 크롤링."""
        for identity in self.container.identities:
            await page.wait_for_load_state("networkidle")
            await page.goto(self.base_url + "/identity/"+identity.identity_id)
            """ 레벨과 동기화와 관련 없는 참관타 저항은 밑에서 변경하는 것과 연관성이 적기 때문에 위에서 처리한다.
            print(f"Identity_id: {identity.identity_id}")
            results = []
            resist_type_imgs = [f"{self.img_url}/ui/atk_type_HIT.png",
                                f"{self.img_url}/ui/atk_type_PENETRATE.png",
                                f"{self.img_url}/ui/atk_type_SLASH.png"]
            for resist_type_img in resist_type_imgs:
                result = await self.extract_resist_type(page, resist_type_img, 1)
                results.append(result)
            self.container.add_resist_info(identity.identity_id, results[0], results[1], results[2])
            """
            """
            # Stagger Threshold 값 추출
            stagger_threshold = await self.extract_stagger_threshold_values(page)
            self.container.add_stagger_threshold(identity.identity_id, stagger_threshold[0], stagger_threshold[1], stagger_threshold[2])
            """
            await self.initialize_uptie_and_level(page)
            """ 레벨에 따른 level, hp, defense 다르게 되어 있는 것을 출력 확인 객체로 들어가서 csv로 옮기는 과정까지 진행
            for i in range(1, identity.level_max+1):
                await self.move_slider_to_value(page,i)
                await self.extract_level_status(page, identity.identity_id, i)
            """
            """ 동기화에 따른 speed 다르게 되어 있는 것을 출력 확인 객체로 들어가서 csv로 옮기는 과정까지 진행
            for i in range(1, identity.uptie_max+1):
                await self.uptie_click(page,i)
                await self.extract_uptie_status(page, identity.identity_id, i)
            """
            """
                인격에 대한 skill 정보를 가져오는 메소드 위에서 출력하는 csv파일로 인격이 가지고 있는 스킬 정보를 내보낸다.
            await self.crawl_skills(page, identity.identity_id)
            """
            await self.crawl_influence_behavior(page, identity.identity_id)
            
    async def initialize_uptie_and_level(self, page):
        await self.uptie_click(page, 1)
        await self.move_slider_to_value(page, 1)
        self.uptie_level = 1
        #return 1
    
    async def uptie_click(self, page, phase):
        fab_button_selector = ".q-btn.q-btn--fab"
        await page.wait_for_selector(fab_button_selector)
        await page.click(fab_button_selector)
        #print(f"[{setup_uptie_button}] 상위 동기화 버튼 클릭 완료")
        setup_uptie_button = f'a:has(img[src="{self.img_url}/ui/uptie_{phase}.png"])'
        await page.wait_for_selector(setup_uptie_button)
        await page.click(setup_uptie_button)
        #print(f"{phase} 동기화 버튼 클릭 완료")

    async def move_slider_to_value(self, page, target_value):
        """
        슬라이더를 주어진 값으로 이동시킵니다.
        target_value는 슬라이더의 최소값부터 최대값 사이여야 합니다.
        """
        slider_selector = ".q-slider"
        # 슬라이더의 범위 확인
        min_value = int(await page.get_attribute(slider_selector, "aria-valuemin"))
        max_value = int(await page.get_attribute(slider_selector, "aria-valuemax"))
    
        if target_value < min_value or target_value > max_value:
            raise ValueError(f"Target value {target_value} is out of bounds ({min_value}, {max_value})")
    
        # 슬라이더 thumb 요소와 위치 가져오기
        slider_thumb = await page.query_selector(f"{slider_selector} .q-slider__thumb")
        thumb_box = await slider_thumb.bounding_box()
        
        # 슬라이더의 현재 위치 및 목표 위치 계산
        track = await page.query_selector(f"{slider_selector} .q-slider__track")
        track_box = await track.bounding_box()
        
        # 목표 x 위치 계산 (슬라이더 트랙 내에서 비율 기반)
        target_percentage = (target_value - min_value) / (max_value - min_value)
        target_x = track_box["x"] + target_percentage * track_box["width"]
        current_x = thumb_box["x"] + thumb_box["width"] / 2
        y = thumb_box["y"] + thumb_box["height"] / 2
    
        # 슬라이더를 드래그하여 목표 위치로 이동
        await page.mouse.move(current_x, y)
        await page.mouse.down()
        await page.mouse.move(target_x, y, steps=10)  # 부드럽게 이동
        await page.mouse.up()
    
        #print(f"슬라이더를 {target_value}로 이동 완료.")
    
        # 슬라이더 값 확인 및 강제 이벤트 트리거
        await page.evaluate(
            f"""
            (slider) => {{
                slider.dispatchEvent(new Event('input'));  // oninput 이벤트 트리거
                slider.dispatchEvent(new Event('change'));  // onchange 이벤트 트리거
            }}
            """,
            await page.query_selector(slider_selector)
        )
    
        # 이동 후 값 확인
        current_value = await page.get_attribute(slider_selector, "aria-valuenow")
        if current_value != str(target_value):
            print(f"경고: 슬라이더 값이 예상과 다릅니다. 현재 값: {current_value}, 예상 값: {target_value}")

    async def get_second_status_element(self, page):
        """
        두 번째 'Status' 요소에 접근하여 반환하는 메소드
        """
        status_elements = await page.query_selector_all("text=Status")
        if len(status_elements) > 1:
            second_status_element = status_elements[1]
            #print("두 번째 'Status' 요소에 접근합니다.")
            return second_status_element
        else:
            #print("두 번째 'Status' 요소가 존재하지 않습니다.")
            return None

    async def extract_level_status(self, page, identity_id, level):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 레벨에 따라 HP와 Defense가 변한다.
        hp_value = await self.extract_hp(second_status_element)
        defense_value = await self.extract_defense(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Level={level}, HP={hp_value}, Defense={defense_value}")
        self.container.add_level_stat(identity_id,level, hp_value, defense_value)
    
    async def extract_hp(self, status_element):
        # HP 추출
        hp_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].children[0].nextElementSibling.textContent.trim()"
        )
        return hp_value
    
    async def extract_defense(self, status_element):
        # Defense 추출
        defense_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return defense_value

    async def extract_uptie_status(self, page, identity_id, uptie):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 uptie에 따라 speed는 변한다.
        speed_value = await self.extract_speed(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Uptie={uptie}, Speed={speed_value}")
        self.container.add_uptie_stat(identity_id,uptie, speed_value)

    async def extract_speed(self, status_element):
        # Speed 추출
        speed_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return speed_value

    async def extract_resist_type(self, page, image_src, depth):
        # 이미지 요소 찾기
        #print(image_src)
        image = await page.query_selector(f'img[src="{image_src}"]')
        #print(image)
        if not image:
            return "Unknown"
    
        # 이미지의 부모 요소를 찾아서 올라감
        parent = image
        for _ in range(depth):
            parent = await parent.evaluate_handle('node => node.parentElement')
            if not parent:
                return "Unknown"
    
        # 부모의 형제 요소 찾기
        sibling = await parent.evaluate_handle('node => node.nextElementSibling')
        
        if sibling:
            value = await sibling.evaluate('node => node.textContent')
            if value:
                return value.strip()
        
        return "Unknown"

    async def extract_stagger_threshold_values(self, page):
        # 'div.row.center.flex-center.q-pt-xs' 요소들 찾기
        elements = await page.query_selector_all('div.row.center.flex-center.q-pt-xs')
        
        # 각 요소의 텍스트 내용 출력하여 'Stagger Threshold' 요소를 찾기
        for element in elements:
            text = await element.text_content()
            
            if text and "Stagger Threshold" in text:
                # 'Stagger Threshold'가 있는 요소를 찾았다면, 그 이후 형제 요소들에서 값을 추출
                parent = await element.evaluate_handle('node => node.parentElement')
                sibling_elements = await parent.query_selector_all('div.row.justify-evenly div')  # Threshold 값이 들어있는 div들
                
                stagger_values = []
                for sibling in sibling_elements:
                    text_content = await sibling.text_content()
                    match = re.search(r'(\d+%)', text_content)
                    if match:
                        stagger_values.append(match.group(1))  # 퍼센트 값만 리스트에 추가
                while len(stagger_values) < 3:
                    stagger_values.append(np.nan)
                #print(stagger_values)
                return stagger_values  # 추출된 값(또는 np.nan 포함)을 반환
        
        return [np.nan, np.nan, np.nan]  # 만약 'Stagger Threshold'를 찾지 못한 경우 기본값 반환

    async def crawl_skills(self, page, identity_id):
        await self.crawl_skill(page, identity_id, "skill 1")
        await self.crawl_skill(page, identity_id, "skill 2")
        await self.crawl_skill(page, identity_id, "skill 3")
        await self.crawl_skill(page, identity_id, "defense")


    async def crawl_skill(self, page, identity_id, skill_type):
        skills_elements = await page.query_selector_all('div.skilltag')
        for skill_element in skills_elements:
            skill_text = await skill_element.text_content()
            if skill_text.strip().lower() == skill_type:
                await self.process_skill(skill_element, page, identity_id, skill_type)
        
    async def process_skill(self, skill_element, page, identity_id, skill_type):
        # skill_type이 "skill 3"일 경우 uptie_value가 3 이상일 때만 정보를 저장
        for uptie_value in range(1, 5):  # Uptie levels 1 to 4
            # "skill 3"은 uptie_value가 3 이상일 때만 처리
            if skill_type == "skill 3" and uptie_value < 3:
                continue  # skip if uptie_value is less than 3 for skill 3
    
            # uptie_level에 맞게 클릭
            await self.uptie_click(page, uptie_value)
            if skill_type != "skill 3" and uptie_value > 1:
                # Uptie 정보 갱신
                await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
                continue
            elif skill_type == "skill 3" and uptie_value > 3:
                # Uptie 정보 갱신
                await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
                continue
            # 스킬의 세부 정보를 추출
            skill_details = await self.extract_skill_details(skill_element, skill_type)
            skill_name = skill_details["skill_name"]
    
            # 스킬 정보를 저장
            self.container.add_skill(
                identity_id, skill_type, 
                skill_name, skill_details["coin_count"],
                skill_details["init_level"], skill_details["init_remain"], 
                skill_details["atk_weight"]
            )
            print(f"Added skill: {skill_type}:{skill_name}")
    
            # Uptie 정보 갱신
            await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
    
    async def extract_skill_details(self, skill_element, skill_type):
        # 스킬 세부 정보를 추출하는 함수
        skill_name = await self.skill_name(skill_element)
        coin_count = await self.coin_count(skill_element, ".col-auto > .column > .row")
        init_level = await self.init_level(skill_element)
        init_remain = await self.init_remain(skill_element, skill_type, "div.row.justify-start > div.attr.flex.flex-center > div.q-px-sm")
        atk_weight = await self.atk_weight(skill_element, skill_type)
        return {
            "skill_name": skill_name,
            "coin_count": coin_count,
            "init_level": init_level,
            "init_remain": init_remain,
            "atk_weight": atk_weight
        }
    
    async def update_uptie_info(self, page, skill_element, identity_id, skill_name, skill_type, uptie_value):
        # Uptie 정보를 추출하고 업데이트하는 함수
        uptie_info = await self.extract_uptie_info(skill_element)
        effect_data = await self.skill_effects(skill_element)
        self.container.add_skill_uptie_info(
            identity_id, skill_name, 
            uptie_value, uptie_info["skill_power"], uptie_info["coin_power"], 
            effect_data.get("effectsByCondition")
        )
        print(
            f"Updated Uptie for {skill_name} (Uptie Level: {uptie_value}): "
            f"Skill Power: {uptie_info['skill_power']}, Coin Power: {uptie_info['coin_power']} "
            f"Skill Effect: {effect_data.get("effectsByCondition")}"
        )

    async def extract_uptie_info(self, skill_element):
        skill_data = await self.skill_and_coin_power(skill_element)
        return {
            "skill_power": skill_data.get("skillPower"),
            "coin_power": skill_data.get("coinPower")
        }
        
    async def crawl_influence_behavior(self, page, identity_id):
        influence_behaviors = await page.query_selector_all('div[align="right"]')
        # 텍스트를 기준으 추출하고 메소드로 파라미터 들어가 정보를 받는다.
        for influence_behavior in influence_behaviors:
            active_type = await influence_behavior.inner_text()
            active_type = active_type.strip()

            if active_type == "PASSIVE":
                """
                # PASSIVE에 대한 처리
                print(f"Processing PASSIVE logic for: {active_type}")
                # CALL method for PASSIVE
                """
                passive_data = await self.passive_effects(influence_behavior)
            elif active_type == "SUPPORT PASSIVE":
                """
                # SUPPORT PASSIVE에 대한 처리
                print(f"Processing SUPPORT PASSIVE logic for: {active_type}")
                # CALL method for SUPPORT PASSIVE
                """
                support_passive_data = await self.support_passive_effects(influence_behavior)
            elif active_type == "PANIC TYPE":
                """
                # PANIC TYPE에 대한 처리
                print(f"Processing PANIC TYPE logic for:{active_type}")
                # CALL method for PANIC TYPE
                """
                panic_type_data = await self.panic_effects(influence_behavior)

    async def skill_and_coin_power(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                // 조부모로 이동
                const grandparent = node.parentElement.parentElement;
                // 이전 형제 탐색
                const previousSibling = grandparent.previousElementSibling;
                if (!previousSibling) return {{ skillPower: null, coinPower: null }}; // 이전 형제가 없으면 기본값 반환
                // 이전 형제 내부의 .row.justify-around > div > div 탐색
                const targetDivs = previousSibling.querySelectorAll(".row.justify-around > div");
                if (targetDivs.length < 3) return {{ skillPower: null, coinPower: null }}; // 배열 길이 확인
                // 1st div: 스킬 위력
                const skillPower = targetDivs[0]?.querySelector('div')?.textContent?.trim() || null; 
                // 3rd div: 코인 파워 값 (3번째 div에서 텍스트 추출)
                const coinPower = targetDivs[2]?.textContent?.trim() || null;
                return {{ skillPower, coinPower }};
            }}
        """)


    async def coin_count(self, skill_element, hierarchy_selector):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector('{hierarchy_selector}');
                if (!targetElement) return 0;
                return targetElement.childElementCount;
            }}
        """)

    async def init_remain(self, skill_element, skill_type, hierarchy_selector):
        if skill_type == "defense":
            return -1
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector('{hierarchy_selector}');
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def skill_name(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector(".block");
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def init_level(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector("div.text-bold.text-h6.h6.q-pl-xs.q-pr-md");
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def atk_weight(self, skill_element, skill_type):
        if skill_type == "defense":
            return 0
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector("div.text-bold.text-h6.h6.q-pl-xs.q-pr-md");
                const doubleNextSibling = targetElement.nextElementSibling.nextElementSibling;
                if (!doubleNextSibling) return 0;
                return doubleNextSibling.textContent?.trim();
            }}
        """)

    async def skill_effects(self, skill_element):
        return await skill_element.evaluate("""
            node => {
                const parent = node.parentElement;
                const doubleNextSibling = parent.nextElementSibling?.nextElementSibling;
                if (!doubleNextSibling) return null;
    
                // doubleNextSibling에서 첫 번째 div.col 요소 찾기
                const textNode = doubleNextSibling.querySelector("div.col");
                if (!textNode) return null;
    
                const results = {
                    keywords: [], // 키워드 저장
                    conditions: [], // 조건 저장
                    effectsByCondition: {} // 조건별 효과
                };
    
                // 1. 조건 추출 (대괄호 텍스트)
                const conditionElements = textNode.querySelectorAll("span");
                conditionElements.forEach(span => {
                    let text = span.textContent?.trim();
                    if (text?.startsWith("[") && text?.endsWith("]")) {
                        // 부모의 전형제 노드 탐색
                        const sibling = span.parentElement.previousElementSibling;
                        if (sibling) {
                            const emblemImage = sibling.querySelector("img.emblem");
                            console.log("Sibling element found:", sibling); // 디버깅용
                            if (emblemImage) {
                                const src = emblemImage.getAttribute("src");
                                console.log("Image source:", src); // 디버깅용
                                const match = src.match(/ui_num_(\\d+)\\.png$/); // 정규식 수정
                                if (match) {
                                    const number = match[1]; // 숫자 추출
                                    text = `[Coin ${number} ${text.replace("[", "").replace("]", "")}]`;
                                    console.log("Updated condition:", text); // 디버깅용
                                }
                            }
                        }
                        results.conditions.push(text);
                        results.effectsByCondition[text] = []; // 조건별 효과를 초기화
                    }
                });
    
                // 2. 키워드 추출
                const keywordElements = textNode.querySelectorAll('a[href^="/en/keyword/"]');
                keywordElements.forEach(a => {
                    const span = a.querySelector('span.q-btn__content.text-center.col.items-center.q-anchor--skip.justify-center.row>span');
                    if (span) {
                        const keywordText = span.textContent.trim();
                        if (keywordText && !results.keywords.includes(keywordText)) {
                            results.keywords.push(keywordText); // 키워드 저장
                        }
                    }
                });
    
                // 3. 효과 추출
                const effectElements = textNode.querySelectorAll("span");
                let currentCondition = null;
                let currentEffect = []; // 초기화
                effectElements.forEach(span => {
                    let text = span.textContent?.trim();
                    if (!text) return;
    
                    // 조건 발견 시 currentCondition 변경
                    if (text.startsWith("[") && text.endsWith("]")) {
                        // 이전 조건의 효과 저장
                        if (currentCondition && currentEffect.length > 0) {
                            // 부모의 전형제 노드 탐색
                            const sibling = span.parentElement.previousElementSibling;
                            if (sibling) {
                                const emblemImage = sibling.querySelector("img.emblem");
                                if (emblemImage) {
                                    const src = emblemImage.getAttribute("src");
                                    const match = src.match(/ui_num_(\\d+)\\.png$/); // 정규식 수정
                                    if (match) {
                                        const number = match[1]; // 숫자 추출
                                        text = `[Coin ${number} ${text.replace("[", "").replace("]", "")}]`;
                                    }
                                }
                            }
                            results.effectsByCondition[currentCondition].push(currentEffect.join(' '));
                            currentEffect = []; // 초기화
                        }
                        currentCondition = text;
                        return;
                    }
    
                    // 현재 조건의 효과 처리
                    if (currentCondition) {
                        if (results.keywords.includes(text)) {
                            const keywordEffect = 'keyword:' + text;
                            if (!currentEffect.includes(keywordEffect)) {
                                currentEffect.push(keywordEffect);
                            }
                        } else {
                            currentEffect.push(text);
                        }
                    }
                });
    
                // 마지막 효과 저장
                if (currentCondition && currentEffect.length > 0) {
                    results.effectsByCondition[currentCondition].push(currentEffect.join(' '));
                }
    
                return results;
            }
        """)

    async def passive_effects(self, passive_element):
        passive_name = await self.behavior_name(passive_element)
        behavior_conditions = await self.behavior_condition(passive_element)
        print(passive_name)
        print(behavior_conditions)
    async def support_passive_effects(self, support_passive_element):
        support_passive_name = await self.behavior_name(support_passive_element)
        #print(support_passive_name)
    async def panic_effects(self, panic_element):
        panic_type = await self.behavior_name(panic_element)
        # print(panic_type)

    async def navigate_element_by_steps(self, element, parent_steps=0, prev_sibling_steps=0, next_sibling_steps=0):
        """
        주어진 `element`에서 부모, 이전 형제, 또는 다음 형제로 지정된 횟수만큼 이동하여 ElementHandle 반환.
        
        Args:
            element (ElementHandle): 탐색 기준이 되는 요소.
            parent_steps (int): 부모로 이동할 횟수 (0이면 이동하지 않음).
            prev_sibling_steps (int): 이전 형제로 이동할 횟수 (0이면 이동하지 않음).
            next_sibling_steps (int): 다음 형제로 이동할 횟수 (0이면 이동하지 않음).
        
        Returns:
            ElementHandle or None: 탐색된 요소를 반환하거나 없으면 None.
        """
        try:
            current_element = element
    
            # 부모로 이동
            for _ in range(parent_steps):
                current_element = await current_element.evaluate_handle('el => el.parentElement')
                if not current_element:
                    print("Parent element not found during navigation.")
                    return None
    
            # 이전 형제로 이동
            for _ in range(prev_sibling_steps):
                current_element = await current_element.evaluate_handle('el => el.previousElementSibling')
                if not current_element:
                    print("Previous sibling element not found during navigation.")
                    return None
    
            # 다음 형제로 이동
            for _ in range(next_sibling_steps):
                current_element = await current_element.evaluate_handle('el => el.nextElementSibling')
                if not current_element:
                    print("Next sibling element not found during navigation.")
                    return None
    
            return current_element
    
        except Exception as e:
            print(f"Error navigating elements: {e}")
            return None

    async def behavior_name(self, element):
        parent_prev_sibling = await self.navigate_element_by_steps(element, parent_steps=1, prev_sibling_steps=1, next_sibling_steps=0)
        passive_name_element = await parent_prev_sibling.query_selector('span.block')
        if passive_name_element:
            passive_name = await passive_name_element.inner_text()
            return passive_name
        else:
            #print("Passive name element not found")
            return None

    async def behavior_condition(self, element):
        try:
            conditions = []
            parent_prev_sibling = await self.navigate_element_by_steps(element, parent_steps=1, prev_sibling_steps=1, next_sibling_steps=0)
            img_elements = await parent_prev_sibling.query_selector_all('img')
            if not img_elements:
                print("No <img> elements found.")
                return conditions
    
            for img_element in img_elements:
                # 각 이미지의 src 속성 가져오기
                img_src = await img_element.get_attribute('src')
                print(img_src)
    
                sin_name = None
                for color_key in resource_mapping.keys():
                    if f"{self.img_url}/ui/attr_{color_key}.png" in img_src:
                        sin_name = resource_mapping[color_key]
                        break
    
                if not sin_name:
                    print(f"No matching Sin Color found for {img_src}.")
                    continue  # No Sin Color found, skip this img_element
    
                # 해당 이미지와 관련된 조건 텍스트 추출
                # 두 번째 부모 요소에서 시작하여 다음 형제를 찾아 조건 텍스트를 추출
                condition_element = await self.navigate_element_by_steps(img_element, parent_steps=3, prev_sibling_steps=0, next_sibling_steps=0)
                if condition_element:
                    # condition_element가 None이 아닌 경우에만 진행
                    condition_text_element = await condition_element.query_selector('.q-pl-xs')
                    if condition_text_element:
                        condition_text = await condition_text_element.inner_text()
                        condition_text = condition_text.strip()
                    else:
                        print(f"No condition text found for {img_src}.")
                        condition_text = ""  # condition_text_element가 None이면 빈 문자열로 처리
                else:
                    print(f"No condition element found for {img_src}.")
                    condition_text = ""  # condition_element가 None이면 빈 문자열로 처리
    
                # 결과를 배열에 추가
                conditions.append(f"{sin_name} {condition_text}")
            return conditions
        except Exception as e:
            print(f"Error extracting conditions: {e}")
            return []


class Identity:
    """인격 정보를 저장하는 클래스"""
    def __init__(self, identity_id, name=None, rarity=None,\
                 skill_1_type=None, skill_1_sin_affinity=None,\
                 skill_2_type=None, skill_2_sin_affinity=None,\
                 skill_3_type=None, skill_3_sin_affinity=None,\
                 defense_type=None, defense_sin_affinity=None):
        self.identity_id = identity_id
        self.name = name
        self.rarity = rarity
        # 선언만 하는 부분 
        self.skill_1_type = skill_1_type 
        self.skill_1_sin_affinity = skill_1_sin_affinity 
        self.skill_2_type = skill_2_type 
        self.skill_2_sin_affinity = skill_2_sin_affinity 
        self.skill_3_type = skill_3_type 
        self.skill_3_sin_affinity = skill_3_sin_affinity 
        self.defense_type = defense_type 
        self.defense_sin_affinity = defense_sin_affinity
        self.level_max = 50
        self.uptie_max = 4
        self.level_stats = []  # LevelStat 객체 리스트
        self.uptie_stats = [] # UptieStat 객체 리스트
        self.resist_info = None
        self.stagger_threshold = None
        self.skillset = SkillSet()  # Skill정보를 담고 있는 skillset 객체 
        self.passive_infos = []  # 패시브 객체를 담고 있는 배열
        self.support_passive_infos = [] # 서포트 패시 객체를 담고 있는 배열
        self.panic_type = [] # 혼란상태 발생시 나타나는 효과를 담고 있는 객체 배열1

    def update_name(self, name):
        self.name = name

    def update_rarity(self, rarity):
        self.rarity = rarity

    def add_level_stat(self, level, hp, defense):
        self.level_stats.append(LevelStat(level, hp, defense))

    def add_uptie_stat(self, uptie, speed):
        self.uptie_stats.append(UptieStat(uptie, speed))

    def add_resist_info(self, resist_hit, resist_penetrate, resist_slash):
        self.resist_info = ResistInfo(resist_hit, resist_penetrate, resist_slash)

    def add_stagger_threshold(self, first_stagger, second_stagger, third_stagger):
        self.stagger_threshold = StaggerThreshold(first_stagger, second_stagger, third_stagger)

    def add_skill(self, skill_code, skill_name, coin_count, init_level, init_remain, atk_weight):
        self.skillset.add_skill(skill_code, Skill(skill_code, skill_name, coin_count, init_level, init_remain, atk_weight))
        
    def add_skill_uptie_info(self, skill_name, uptie_level, skill_power, coin_power, effect_by_condition):
        self.skillset.add_skill_uptie_info(skill_name, uptie_level, skill_power, coin_power, effect_by_condition)
        
    def update_details(self, page):
        # 세부 정보 업데이트 로직 (가정)
        pass
    #def init_uptie(self, page):
    #    init_uptie = 

# LevelStat 객체 정의
class LevelStat:
    def __init__(self, level, hp=None, defense=None):
        self.level = level
        self.hp = hp
        self.defense = defense
    def update_level(self, level):
        self.level = level
    def update_hp(self, hp):
        self.hp = hp
    def update_defense(self, defense):
        self.defense = defense

# UptieStat 객체 정의
class UptieStat:
    def __init__(self, uptie, speed=None):
        self.uptie = uptie
        self.speed = speed
    def update_speed(self, speed):
        self.speed = speed

# ResistInfo 객체 정의
class ResistInfo:
    def __init__(self, resist_hit=None, resist_penetrate=None, resist_slash=None):
        self.resist_hit = resist_hit
        self.resist_penetrate = resist_penetrate
        self.resist_slash = resist_slash

# ResistInfo 객체 정의
class StaggerThreshold:
    def __init__(self, first_stagger=None, second_stagger=None, third_stagger=None):
        self.first_stagger = first_stagger
        self.second_stagger = second_stagger
        self.third_stagger = third_stagger

class SkillSet:
    def __init__(self):

        self.skill_1_type = ""
        self.skill_2_type = ""
        self.skill_3_type = ""
        self.defense_skill_type = ""

        self.skill_1_sin_affinity = ""
        self.skill_2_sin_affinity = ""
        self.skill_3_sin_affinity = ""
        self.defense_skill_sin_affinity = ""

        self.skill_1_details = []
        self.skill_2_details = []
        self.skill_3_details = []
        self.defense_skill_details = []

    def set_skill_type(self, skill_code, skill_type, sin_affinity):
        if skill_code == "skill 1":
            self.skill_1_type = skill_type
            self.skill_1_sin_affinity = sin_affinity
        elif skill_code == "skill 2":
            self.skill_2_type = skill_type
            self.skill_2_sin_affinity = sin_affinity
        elif skill_code == "skill 3":
            self.skill_3_type = skill_type
            self.skill_3_sin_affinity = sin_affinity
        elif skill_code == "defense":
            self.defense_skill_type = skill_type
            self.defense_skill_sin_affinity = sin_affinity
    
    def add_skill(self, skill_code, skill):
        if skill_code == "skill 1":
            self.skill_1_details.append(skill)
            skill.skill_type = self.skill_1_type
            skill.sin_affinity = self.skill_1_sin_affinity
        elif skill_code == "skill 2":
            self.skill_2_details.append(skill)
            skill.skill_type = self.skill_2_type
            skill.sin_affinity = self.skill_2_sin_affinity
        elif skill_code == "skill 3":
            self.skill_3_details.append(skill)
            skill.skill_type = self.skill_3_type
            skill.sin_affinity = self.skill_3_sin_affinity
        elif skill_code == "defense":
            self.defense_skill_details.append(skill)
            skill.skill_type = self.defense_skill_type 
            skill.sin_affinity = self.defense_skill_sin_affinity

    def get_skill_by_name(self, skill_name):
        for skill_list in [self.skill_1_details, self.skill_2_details, self.skill_3_details, self.defense_skill_details]:
            for skill in skill_list:
                result = skill.is_skill_by_name(skill_name)
                if result:
                    return result
        return None

    def add_skill_uptie_info(self, skill_name, uptie_level, skill_power, coin_power, effect_by_condition):
        skill = self.get_skill_by_name(skill_name)
        skill.add_uptie_info(uptie_level, skill_power, coin_power, effect_by_condition)
        

class Skill:
    def __init__(self, skill_code, skill_name, coin_count, init_level, init_remain, atk_weight):
        self.skill_code = skill_code # "Skill 1"/"Skill 2"/"Skill 3"/"Defense" '''o'''
        self.skill_type = None
        self.sin_affinity = None
        self.skill_name = skill_name # 스킬 이름 '''o'''
        self.coin_count = coin_count # 동전 개수 '''o'''
        self.init_level = init_level # 스킬이 초기 배정받은 레벨, 인격 레벨에 - 1 한 값을 더해주면 스킬의 현 level을 구할 수 있다. '''o'''
        self.init_remain = init_remain #'''o'''
        self.atk_weight = atk_weight #공격 가중치 값이다. 방어스킬에는 없기 때문에 None으로 설정했다. '''o'''
        self.remain_count = init_remain # 실제 스킬이 작동하며 횟수가 차감되 나중에 모두 차감되면 다음 턴에 다시 초기횟수 리필된다.
        self.uptie_info = []  

    def is_skill_by_name(self, skill_name):
        if self.skill_name == skill_name:
            return self
        return None
        
    def add_uptie_info(self, uptie_level, skill_power, coin_power, effects_by_condition):
        self.uptie_info.append(UptieInfo(uptie_level, skill_power, coin_power, effects_by_condition))

class UptieInfo:
    def __init__(self, uptie_level, skill_power, coin_power, effects_by_condition):
        self.uptie_level = uptie_level   #'''o'''
        self.skill_power = skill_power   #'''o'''
        self.coin_power = coin_power     #'''o'''
        self.skill_effect = effects_by_condition

# 추상화된 EffectType 클래스
class EffectType:
    def __init__(self, name, description):
        self.name = name  # Effect 이름
        self.description = description  # Effect 설명

# Passive 클래스
class Passive(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)  # condition이 있는지 여부 확인

# SupportPassive 클래스
class SupportPassive(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)

# PanicType 클래스
class PanicType(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)

# Keyword 클래스
class Keyword:
    def __init__(self, keyword, explain=None, dispellable=None):
        self.keyword = keyword
        self.explain = explain
        self.dispellable = dispellable

      
class DataContainer:
    """크롤링 데이터를 관리하는 클래스"""
    def __init__(self):
        self.identities = []

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.level_max

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.uptie_max

    def update_identity_name(self, identity_id, identity_name):
        """특정 ID의 이름을 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_name(identity_name)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_identity_rarity(self, identity_id, rarity):
        """특정 ID의 희귀도를 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_rarity(rarity)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_skill(self, identity_id, skill_number, new_type):
        """
        특정 identity의 skill_type을 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_type: 새 skill_type
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_type = new_type
                elif skill_number == 2:
                    identity.skill_2_type = new_type
                elif skill_number == 3:
                    identity.skill_3_type = new_type
                elif skill_number == "defense":
                    identity.defense_type = new_type
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def update_sin_affinity(self, identity_id, skill_number, new_affinity):
        """
        특정 identity의 sin_affinity를 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_affinity: 새 sin_affinity
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_sin_affinity = new_affinity
                elif skill_number == 2:
                    identity.skill_2_sin_affinity = new_affinity
                elif skill_number == 3:
                    identity.skill_3_sin_affinity = new_affinity
                elif skill_number == "defense":
                    identity.defense_sin_affinity = new_affinity
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def add_identity(self, identity):
        """새로운 인격 데이터를 추가."""
        self.identities.append(identity)

    def add_level_stat(self, identity_id, level, hp, defense):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_level_stat(level, hp, defense)

    def add_uptie_stat(self, identity_id, uptie, speed):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_uptie_stat(uptie, speed)

    def add_resist_info(self, identity_id, resist_hit, resist_penetrate, resist_slash):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_resist_info(resist_hit, resist_penetrate, resist_slash)

    def add_stagger_threshold(self, identity_id, first_stagger, second_stagger, third_stagger):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_stagger_threshold(first_stagger, second_stagger, third_stagger)

    
    def add_skill(self, identity_id, skill_code, skill_name, coin_count, init_level, init_remain, atk_weight):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_code == "skill 1":
                    identity.skillset.set_skill_type(skill_code, identity.skill_1_type, identity.skill_1_sin_affinity)
                elif skill_code == "skill 2":
                    identity.skillset.set_skill_type(skill_code, identity.skill_2_type, identity.skill_2_sin_affinity)
                elif skill_code == "skill 3":
                    identity.skillset.set_skill_type(skill_code, identity.skill_3_type, identity.skill_3_sin_affinity)
                elif skill_code == "defense":
                    identity.skillset.set_skill_type(skill_code, identity.defense_type, identity.defense_sin_affinity)
                identity.add_skill(skill_code, skill_name, coin_count, init_level, init_remain, atk_weight)

    def add_skill_uptie_info(self, identity_id, skill_name, uptie_level, skill_power, coin_power, effects_by_condition):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_skill_uptie_info(skill_name, uptie_level, skill_power, coin_power, effects_by_condition)
        
    def display_identities(self):
        """Identity, ID, Name, Rarity 정보를 출력"""
        if not self.identities:
            print("No identities found.")
        else:
            # 데이터를 리스트로 변환
            table = []
            for idx, identity in enumerate(self.identities, 1):
                table.append([
                    idx, identity.identity_id, identity.name, identity.rarity,
                    identity.skill_1_type, identity.skill_1_sin_affinity,
                    identity.skill_2_type, identity.skill_2_sin_affinity,
                    identity.skill_3_type, identity.skill_3_sin_affinity,
                    identity.defense_type, identity.defense_sin_affinity
                ])
            
            # 테이블 헤더 정의
            headers = [
                "Index", "ID", "Name", "Rarity",
                "Skill_1 Type", "Skill_1 Sin", "Skill_2 Type", "Skill_2 Sin",
                "Skill_3 Type", "Skill_3 Sin", "Defense Type", "Defense Sin"
            ]
            
            # 테이블 출력
            print(tabulate(table, headers=headers, tablefmt="fancy_grid"))

    def export_to_csv(self, filename, fieldnames, data):
        """
        공통적으로 사용 가능한 CSV 내보내기 메서드
        :param filename: 내보낼 파일 이름 (예: 'identity.csv', 'skill.csv')
        :param fieldnames: CSV의 헤더 리스트
        :param data: CSV로 내보낼 데이터 (리스트 형태)
        """
        try:
            # 현재작업 디렉토 기준으 'csv'폴더 없으면 생성
            if not os.path.exists('./csv'):
                os.makedirs('./csv')
            # 파일 경로 설정 (상대경로 'csv' 폴더 안에 저장)
            file_path = os.path.join('./csv', filename)
            with open(file_path, mode='w', newline='', encoding='utf-8') as file:
                writer = csv.DictWriter(file, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(data)
            print(f"Data successfully exported to {filename}")
        except Exception as e:
            print(f"Error exporting to CSV: {e}")

    def get_identity_data(self):
        """
        Identity 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "ID": identity.identity_id,
                "Name": identity.name,
                "Rarity": identity.rarity,
                "Skill_1_Type": identity.skill_1_type,
                "Skill_1_Sin_Affinity": identity.skill_1_sin_affinity,
                "Skill_2_Type": identity.skill_2_type,
                "Skill_2_Sin_Affinity": identity.skill_2_sin_affinity,
                "Skill_3_Type": identity.skill_3_type,
                "Skill_3_Sin_Affinity": identity.skill_3_sin_affinity,
                "Defense_Type": identity.defense_type,
                "Defense_Sin_Affinity": identity.defense_sin_affinity,
            }
            for identity in self.identities
        ]
        
    def export_identities_data(self, filename="identities.csv"):
        """Identity 데이터를 CSV로 내보내는 메서드"""
        identity_fieldnames = [
            "ID", "Name", "Rarity",
            "Skill_1_Type", "Skill_1_Sin_Affinity",
            "Skill_2_Type", "Skill_2_Sin_Affinity",
            "Skill_3_Type", "Skill_3_Sin_Affinity",
            "Defense_Type", "Defense_Sin_Affinity"
        ]
        identity_data = self.get_identity_data()  # Identity 데이터 가져오기
        self.export_to_csv(filename, identity_fieldnames, identity_data)  # CSV로 내보내기

    def load_identities_by_id(self, filename, identity_ids):
        """지정된 identity_ids 목록에 해당하는 Identity 데이터를 CSV에서 읽어옴."""
        identity_ids_set = set(identity_ids) # 중복 제거를 위 Set 사용
        with open(filename, mode='r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                if row["ID"] in identity_ids_set:
                    identity = Identity(
                        identity_id=row["ID"],
                        name=row["Name"],
                        rarity=row["Rarity"],
                        skill_1_type=row["Skill_1_Type"],
                        skill_1_sin_affinity=row["Skill_1_Sin_Affinity"],
                        skill_2_type=row["Skill_2_Type"],
                        skill_2_sin_affinity=row["Skill_2_Sin_Affinity"],
                        skill_3_type=row["Skill_3_Type"],
                        skill_3_sin_affinity=row["Skill_3_Sin_Affinity"],
                        defense_type=row["Defense_Type"],
                        defense_sin_affinity=row["Defense_Sin_Affinity"]
                    )
                    self.identities.append(identity)

    def get_level_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Level": level_stat.level,
                "HP": level_stat.hp,
                "Defense": level_stat.defense,
            }
            for identity in self.identities
            for level_stat in identity.level_stats
        ]

    def export_level_stats_data(self, filename="levelstats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        level_stats_fieldnames = ["Identity_ID", "Level", "HP", "Defense"]
        level_stats_data = self.get_level_stats_data()  # level_stats 데이터 가져오기
        self.export_to_csv(filename, level_stats_fieldnames, level_stats_data)  # CSV로 내보내기

    def get_uptie_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Uptie": uptie_stat.uptie,
                "Speed": uptie_stat.speed,
            }
            for identity in self.identities
            for uptie_stat in identity.uptie_stats
        ]

    def export_uptie_stats_data(self, filename="Uptiestats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        uptie_stats_fieldnames = ["Identity_ID", "Uptie", "Speed"]
        uptie_stats_data = self.get_uptie_stats_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, uptie_stats_fieldnames, uptie_stats_data)  # CSV로 내보내기

    def get_resist_info_data(self):
        """
        모든 Identity의 resist_info 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Resist_Hit": identity.resist_info.resist_hit,
                "Resist_Penetrate": identity.resist_info.resist_penetrate,
                "Resist_Slash": identity.resist_info.resist_slash,
            }
            for identity in self.identities
        ]

    def export_resist_info_data(self, filename="ResistInfos.csv"):
        """ResistInfo 데이터를 CSV로 내보내는 메서드"""
        resist_infos_fieldnames = ["Identity_ID", "Resist_Hit", "Resist_Penetrate", "Resist_Slash"]
        resist_infos_data = self.get_resist_info_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, resist_infos_fieldnames, resist_infos_data)  # CSV로 내보내기

    def export_uptie_stats_data(self, filename="Uptiestats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        uptie_stats_fieldnames = ["Identity_ID", "Uptie", "Speed"]
        uptie_stats_data = self.get_uptie_stats_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, uptie_stats_fieldnames, uptie_stats_data)  # CSV로 내보내기

    def get_stagger_threshold_data(self):
        """
        모든 Identity의 stagger_threshold 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "First_Stagger": identity.stagger_threshold.first_stagger,
                "Second_Stagger": identity.stagger_threshold.second_stagger,
                "Third_Stagger": identity.stagger_threshold.third_stagger,
            }
            for identity in self.identities
        ]

    def export_stagger_threshold_data(self, filename="StaggerThresholds.csv"):
        """StaggerThresholds 데이터를 CSV로 내보내는 메서드"""
        stagger_thresholds_fieldnames = ["Identity_ID", "First_Stagger", "Second_Stagger", "Third_Stagger"]
        stagger_thresholds_data = self.get_stagger_threshold_data()  # stagger_thresholds 데이터 가져오기
        self.export_to_csv(filename, stagger_thresholds_fieldnames, stagger_thresholds_data)  # CSV로 내보내기

    def get_skillset_data(self):
        """모든 Identity의 skillset 데이터를 export_to_csv을 포맷으로 변환"""
        return [
            {
                "Identity_ID": identity.identity_id,
                "Skill_Code": skill.skill_code,
                "Skill_Type": skill.skill_type,
                "Sin_Affinity": skill.sin_affinity,
                "Skill_Name": skill.skill_name,
                "Coin_Count": skill.coin_count,
                "Init_Level": skill.init_level,
                "Init_Remain": skill.init_remain,
                "Atk_Weight": skill.atk_weight,
                "Uptie_Level": uptie_info.uptie_level,
                "Skill_Power": uptie_info.skill_power,
                "Coin_Power": uptie_info.coin_power,
                "Effect_Data": json.dumps(uptie_info.skill_effect)
            }
            for identity in self.identities
            for skill_list in [identity.skillset.skill_1_details, identity.skillset.skill_2_details, identity.skillset.skill_3_details, identity.skillset.defense_skill_details]
            for skill in skill_list
            for uptie_info in skill.uptie_info
        ]

    def export_identity_skillset_data(self, filename="IdentitiesSkills.csv"):
        """IdentitiesSkills 데이터를 CSV로 내보내는 메서드"""
        identities_skills_fieldnames = ["Identity_ID", "Skill_Code", "Skill_Type", "Sin_Affinity", "Skill_Name", "Coin_Count", "Init_Level", "Init_Remain", "Atk_Weight", "Uptie_Level", "Skill_Power", "Coin_Power", "Effect_Data"]
        identities_skills_data = self.get_skillset_data()
        self.export_to_csv(filename, identities_skills_fieldnames, identities_skills_data)
            


async def main():
    data_container = DataContainer()
    scraper = Scraper(container=data_container, base_url="https://limbus.kusoge.xyz", img_url="https://img.kusoge.xyz/limbus/img")
    await scraper.scrape()

await main() 

https://img.kusoge.xyz/limbus/img/icon/BloodDinner.png
No matching Sin Color found for https://img.kusoge.xyz/limbus/img/icon/BloodDinner.png.
https://img.kusoge.xyz/limbus/img/icon/Laceration.png
No matching Sin Color found for https://img.kusoge.xyz/limbus/img/icon/Laceration.png.
Bloodfeast
[]
https://img.kusoge.xyz/limbus/img/ui/attr_SCARLET.png
https://img.kusoge.xyz/limbus/img/ui/attr_VIOLET.png
https://img.kusoge.xyz/limbus/img/icon/Laceration.png
No matching Sin Color found for https://img.kusoge.xyz/limbus/img/icon/Laceration.png.
https://img.kusoge.xyz/limbus/img/icon/BloodDinner.png
No matching Sin Color found for https://img.kusoge.xyz/limbus/img/icon/BloodDinner.png.
Hardblood Thorn
['Lust x3 Owned', 'Envy x2 Owned']


In [13]:
import os
import csv
import re
import json
import numpy as np
import pandas as pd
from playwright.async_api import async_playwright
from tabulate import tabulate

# Sin Color 이름 변환을 위한 매핑
resource_mapping = {
    "AZURE": "Gloom",
    "VIOLET": "Envy",
    "AMBER": "Sloth",
    "SHAMROCK": "Gluttony",
    "CRIMSON": "Wrath",
    "SCARLET": "Lust",
    "INDIGO": "Pride"
}

def convert_resource(resource_name):
    """Sin Color 이름을 변환하는 함수. 매핑되지 않은 경우 원래 이름을 반환."""
    return resource_mapping.get(resource_name, resource_name)

def find_sin_for_color(url):
    # URL에 color_name을 추출
    color_name = url.split('/')[-1].split('.')[0].replace("attr_", "")
    # 해당 색상이 resource_mapping에 있는 확인
    for color, sin in resource_mapping.items():
        if color_name in color:
            return sin
    return None


class Scraper:
    """웹 페이지 크롤링을 위한 클래스"""
    def __init__(self, container, base_url, img_url):
        self.container = container
        self.base_url = base_url
        self.img_url = img_url
        self.uptie_level = 1

    async def handle_request(self, route, request):
        """이미지와 폰트 리소스 로드를 차단."""
        if request.resource_type =="image":
            #print(f"{request.url}")
            img_url = request.url
            if  re.match(f"{self.img_url}/ui/attr.*\\.png", img_url):
                await route.continue_()
            else:
                await route.fulfill(
                    status=200,
                    content_type="image/png",
                    body=b""
                )
        elif request.resource_type == "font":
            await route.abort()
        else:
            await route.continue_()
            
    async def scrape(self):
        """크롤링 메인 함수"""
        async with async_playwright() as p:
            browser = await p.chromium.launch(headless=True)
            page = await browser.new_page()
            #await page.evaluate('window.scrollTo(0, document.body.scrollHeight)')
            await page.route("**/*", self.handle_request)
            # Step 1: /identity 페이지에서 기본 정보 크롤링
            #await self.crawl_identity_page(page)

            # Step 2: /identity/{identity_id} 상세 페이지 크롤링
            #identity_ids_to_load = ["10104", "10310", "10710","10808", "10911", "11005"]
            #identity_ids_to_load = ["10104", "10911"]#,"10310", "10710","10808", "11005", "11210"]
            #identity_ids_to_load = ["10911","10310"]#, "10710","10808", "11005", "11210"]
            identity_ids_to_load = ["10911"]#, "10611"]#,"10711"]#, "10710","10808", "11005", "11210"]
            self.container.load_identities_by_id('./csv/identities.csv', identity_ids_to_load)
            await self.crawl_identity_details_for_all(page)
            #self.container.display_identities()
            #self.container.export_identities_data("identities.csv")
            #self.container.export_level_stats_data("Levelstats.csv")
            #self.container.export_uptie_stats_data("Uptiestats.csv")
            #self.container.export_resist_info_data("ResistInfos.csv")
            #self.container.export_stagger_threshold_data("StaggerThresholds.csv")
            #self.container.export_identity_skillset_data("IdentitiesSkills.csv")
            page.set_default_timeout(10000)
            #await page.wait_for_timeout(100000)
            await browser.close()

    async def get_identity_id_from_row(self, row):
        """테이블 행(row)에서 인격 ID를 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_url = await identity_cell.get_attribute('href')
            identity_id = identity_url.split('/identity/')[-1]
            return identity_id
        return "Unknown"

    async def get_identity_name_from_row(self, row):
        """테이블 행(row)에서 인격 이름을 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_name = (await identity_cell.inner_text()).replace("\n", " ").strip()
            return identity_name
        return "Unknown"
        
    async def crawl_identity_page(self, page):
        """/identity 페이지로 이동하여 인격 기본 정보를 추출."""
        await page.goto(self.base_url + "/identity")
        await self.extract_identity_page(page)

    async def extract_identity_page(self, page):
        """/identity 페이지의 모든 테이블 행에서 인격 정보를 추출."""
        identity_rows = await page.query_selector_all('tr')
        if len(identity_rows) > 1:
            for idx, identity_row in enumerate(identity_rows):
                if idx == 0:
                    continue # 0번째요소이름을 보여주는 열은 건너뜀
                await self.scrape_identity_name_and_identity_id(identity_row)
                await self.extract_rarity(identity_row)
                for i in range(1, 4):  # 스킬1~3 정보 추출
                    await self.extract_atk_and_resource(identity_row, i)
                await self.extract_defense_and_resource(identity_row)

    async def scrape_identity_name_and_identity_id(self, row):
        """인격 번호와 이름을 추출하여 DataContainer에 저장."""
        try:
            identity_id = await self.get_identity_id_from_row(row)
            identity_name = await self.get_identity_name_from_row(row)
            identity = Identity(identity_id)
            self.container.add_identity(identity)
            self.container.update_identity_name(identity_id, identity_name)
        except Exception as e:
            print(f"Error extracting identity name and identity id: {e}")

    async def get_element_attribute(self, element, selector, attribute):
        """요소에서 지정된 속성(attribute)을 추출."""
        try:
            target_element = await element.query_selector(selector)
            if target_element:
                return await target_element.get_attribute(attribute)
        except Exception as e:
            print(f"Error getting element attribute: {e}")
        return None
    
    async def extract_rarity(self, row):
        """테이블 행(row)에서 희귀도를 추출하여 업데이트."""
        try:
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
                sibling = await parent.evaluate_handle('node => node.nextElementSibling')
                if sibling:
                    rarity_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/rarity_"]', 'src')
                    if rarity_img_src:
                        rarity_level = rarity_img_src.split('_')[-1].split('.')[0]  # rarity_1.png -> 1
                        identity_id = await self.get_identity_id_from_row(row)
                        self.container.update_identity_rarity(identity_id, rarity_level)
                    else:
                        print("Rarity image not found in the next sibling TD.")
        except Exception as e: 
            print(f"Error extracting rarity: {e}")

        return "Unknown"

    async def extract_atk_and_resource(self, row, skill_number):
        """ 공격 타입(ATK Type)과 자원(Attribute) 추출 함수 - 스킬 번호에 따라 처리 """
        try:
            #print(f"Attempting to locate skill{skill_number} type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 해당 스킬 번호에 맞는 형제로 이동
                sibling = parent
                for _ in range(skill_number + 1):  # 스킬 번호에 맞게 이동
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                atk_type, resource = "Unknown", "Unknown"
        
                if sibling:
                    # 공격 타입 이미지 (atk_type) 찾기
                    atk_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/atk_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if atk_type_img_src:
                        atk_type = atk_type_img_src.split('_')[-1].split('.')[0]  # atk_type_SLASH.png -> SLASH
                        #print(f"Extracted attack type: {atk_type}")
                        self.container.update_skill(identity_id, skill_number, atk_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted resource: {resource}")
                        
                        # 리소스를 변환
                        resource = convert_resource(resource)
                        #print(f"Converted resource: {resource}")
                        self.container.update_sin_affinity(identity_id, skill_number, resource)
                        
        except Exception as e:
            print(f"Error extracting skill{skill_number} type and resource: {e}")

    async def extract_defense_and_resource(self, row):
        """ 방어 타입(Def Type)과 자원(Attribute) 추출 함수 """
        try:
            #print("Attempting to locate defense type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 다섯 번째 형제 td로 이동 (방어 관련 정보가 있는 <td>)
                sibling = parent
                for _ in range(5):  # 방어 정보는 5번째 형제에 있음
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                def_type, def_resource = "Unknown", "Unknown"
        
                if sibling:
                    # 방어 타입 이미지 (def_type) 찾기
                    def_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/def_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if def_type_img_src:
                        def_type = def_type_img_src.split('_')[-1].split('.')[0]  # def_type_GUARD.png -> GUARD
                        #print(f"Extracted defense type: {def_type}")
                        self.container.update_skill(identity_id, 'defense', def_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        def_resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted defense resource: {def_resource}")
                        
                        # 방어 리소스를 변환
                        def_resource = convert_resource(def_resource)
                        #print(f"Converted defense resource: {def_resource}")
                        self.container.update_sin_affinity(identity_id, 'defense', def_resource)
 
        except Exception as e:
            print(f"Error extracting defense type and resource: {e}")

    async def crawl_identity_details_for_all(self, page):
        """모든 인격의 세부 정보를 크롤링."""
        for identity in self.container.identities:
            await page.wait_for_load_state("networkidle")
            await page.goto(self.base_url + "/identity/"+identity.identity_id)
            """ 레벨과 동기화와 관련 없는 참관타 저항은 밑에서 변경하는 것과 연관성이 적기 때문에 위에서 처리한다.
            print(f"Identity_id: {identity.identity_id}")
            results = []
            resist_type_imgs = [f"{self.img_url}/ui/atk_type_HIT.png",
                                f"{self.img_url}/ui/atk_type_PENETRATE.png",
                                f"{self.img_url}/ui/atk_type_SLASH.png"]
            for resist_type_img in resist_type_imgs:
                result = await self.extract_resist_type(page, resist_type_img, 1)
                results.append(result)
            self.container.add_resist_info(identity.identity_id, results[0], results[1], results[2])
            """
            """
            # Stagger Threshold 값 추출
            stagger_threshold = await self.extract_stagger_threshold_values(page)
            self.container.add_stagger_threshold(identity.identity_id, stagger_threshold[0], stagger_threshold[1], stagger_threshold[2])
            """
            await self.initialize_uptie_and_level(page)
            """ 레벨에 따른 level, hp, defense 다르게 되어 있는 것을 출력 확인 객체로 들어가서 csv로 옮기는 과정까지 진행
            for i in range(1, identity.level_max+1):
                await self.move_slider_to_value(page,i)
                await self.extract_level_status(page, identity.identity_id, i)
            """
            """ 동기화에 따른 speed 다르게 되어 있는 것을 출력 확인 객체로 들어가서 csv로 옮기는 과정까지 진행
            for i in range(1, identity.uptie_max+1):
                await self.uptie_click(page,i)
                await self.extract_uptie_status(page, identity.identity_id, i)
            """
            """
                인격에 대한 skill 정보를 가져오는 메소드 위에서 출력하는 csv파일로 인격이 가지고 있는 스킬 정보를 내보낸다.
            await self.crawl_skills(page, identity.identity_id)
            """
            await self.crawl_influence_behavior(page, identity.identity_id)
            
    async def initialize_uptie_and_level(self, page):
        await self.uptie_click(page, 1)
        await self.move_slider_to_value(page, 1)
        self.uptie_level = 1
        #return 1
    
    async def uptie_click(self, page, phase):
        fab_button_selector = ".q-btn.q-btn--fab"
        await page.wait_for_selector(fab_button_selector)
        await page.click(fab_button_selector)
        #print(f"[{setup_uptie_button}] 상위 동기화 버튼 클릭 완료")
        setup_uptie_button = f'a:has(img[src="{self.img_url}/ui/uptie_{phase}.png"])'
        await page.wait_for_selector(setup_uptie_button)
        await page.click(setup_uptie_button)
        #print(f"{phase} 동기화 버튼 클릭 완료")

    async def move_slider_to_value(self, page, target_value):
        """
        슬라이더를 주어진 값으로 이동시킵니다.
        target_value는 슬라이더의 최소값부터 최대값 사이여야 합니다.
        """
        slider_selector = ".q-slider"
        # 슬라이더의 범위 확인
        min_value = int(await page.get_attribute(slider_selector, "aria-valuemin"))
        max_value = int(await page.get_attribute(slider_selector, "aria-valuemax"))
    
        if target_value < min_value or target_value > max_value:
            raise ValueError(f"Target value {target_value} is out of bounds ({min_value}, {max_value})")
    
        # 슬라이더 thumb 요소와 위치 가져오기
        slider_thumb = await page.query_selector(f"{slider_selector} .q-slider__thumb")
        thumb_box = await slider_thumb.bounding_box()
        
        # 슬라이더의 현재 위치 및 목표 위치 계산
        track = await page.query_selector(f"{slider_selector} .q-slider__track")
        track_box = await track.bounding_box()
        
        # 목표 x 위치 계산 (슬라이더 트랙 내에서 비율 기반)
        target_percentage = (target_value - min_value) / (max_value - min_value)
        target_x = track_box["x"] + target_percentage * track_box["width"]
        current_x = thumb_box["x"] + thumb_box["width"] / 2
        y = thumb_box["y"] + thumb_box["height"] / 2
    
        # 슬라이더를 드래그하여 목표 위치로 이동
        await page.mouse.move(current_x, y)
        await page.mouse.down()
        await page.mouse.move(target_x, y, steps=10)  # 부드럽게 이동
        await page.mouse.up()
    
        #print(f"슬라이더를 {target_value}로 이동 완료.")
    
        # 슬라이더 값 확인 및 강제 이벤트 트리거
        await page.evaluate(
            f"""
            (slider) => {{
                slider.dispatchEvent(new Event('input'));  // oninput 이벤트 트리거
                slider.dispatchEvent(new Event('change'));  // onchange 이벤트 트리거
            }}
            """,
            await page.query_selector(slider_selector)
        )
    
        # 이동 후 값 확인
        current_value = await page.get_attribute(slider_selector, "aria-valuenow")
        if current_value != str(target_value):
            print(f"경고: 슬라이더 값이 예상과 다릅니다. 현재 값: {current_value}, 예상 값: {target_value}")

    async def get_second_status_element(self, page):
        """
        두 번째 'Status' 요소에 접근하여 반환하는 메소드
        """
        status_elements = await page.query_selector_all("text=Status")
        if len(status_elements) > 1:
            second_status_element = status_elements[1]
            #print("두 번째 'Status' 요소에 접근합니다.")
            return second_status_element
        else:
            #print("두 번째 'Status' 요소가 존재하지 않습니다.")
            return None

    async def extract_level_status(self, page, identity_id, level):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 레벨에 따라 HP와 Defense가 변한다.
        hp_value = await self.extract_hp(second_status_element)
        defense_value = await self.extract_defense(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Level={level}, HP={hp_value}, Defense={defense_value}")
        self.container.add_level_stat(identity_id,level, hp_value, defense_value)
    
    async def extract_hp(self, status_element):
        # HP 추출
        hp_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].children[0].nextElementSibling.textContent.trim()"
        )
        return hp_value
    
    async def extract_defense(self, status_element):
        # Defense 추출
        defense_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return defense_value

    async def extract_uptie_status(self, page, identity_id, uptie):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 uptie에 따라 speed는 변한다.
        speed_value = await self.extract_speed(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Uptie={uptie}, Speed={speed_value}")
        self.container.add_uptie_stat(identity_id,uptie, speed_value)

    async def extract_speed(self, status_element):
        # Speed 추출
        speed_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return speed_value

    async def extract_resist_type(self, page, image_src, depth):
        # 이미지 요소 찾기
        #print(image_src)
        image = await page.query_selector(f'img[src="{image_src}"]')
        #print(image)
        if not image:
            return "Unknown"
    
        # 이미지의 부모 요소를 찾아서 올라감
        parent = image
        for _ in range(depth):
            parent = await parent.evaluate_handle('node => node.parentElement')
            if not parent:
                return "Unknown"
    
        # 부모의 형제 요소 찾기
        sibling = await parent.evaluate_handle('node => node.nextElementSibling')
        
        if sibling:
            value = await sibling.evaluate('node => node.textContent')
            if value:
                return value.strip()
        
        return "Unknown"

    async def extract_stagger_threshold_values(self, page):
        # 'div.row.center.flex-center.q-pt-xs' 요소들 찾기
        elements = await page.query_selector_all('div.row.center.flex-center.q-pt-xs')
        
        # 각 요소의 텍스트 내용 출력하여 'Stagger Threshold' 요소를 찾기
        for element in elements:
            text = await element.text_content()
            
            if text and "Stagger Threshold" in text:
                # 'Stagger Threshold'가 있는 요소를 찾았다면, 그 이후 형제 요소들에서 값을 추출
                parent = await element.evaluate_handle('node => node.parentElement')
                sibling_elements = await parent.query_selector_all('div.row.justify-evenly div')  # Threshold 값이 들어있는 div들
                
                stagger_values = []
                for sibling in sibling_elements:
                    text_content = await sibling.text_content()
                    match = re.search(r'(\d+%)', text_content)
                    if match:
                        stagger_values.append(match.group(1))  # 퍼센트 값만 리스트에 추가
                while len(stagger_values) < 3:
                    stagger_values.append(np.nan)
                #print(stagger_values)
                return stagger_values  # 추출된 값(또는 np.nan 포함)을 반환
        
        return [np.nan, np.nan, np.nan]  # 만약 'Stagger Threshold'를 찾지 못한 경우 기본값 반환

    async def crawl_skills(self, page, identity_id):
        await self.crawl_skill(page, identity_id, "skill 1")
        await self.crawl_skill(page, identity_id, "skill 2")
        await self.crawl_skill(page, identity_id, "skill 3")
        await self.crawl_skill(page, identity_id, "defense")


    async def crawl_skill(self, page, identity_id, skill_type):
        skills_elements = await page.query_selector_all('div.skilltag')
        for skill_element in skills_elements:
            skill_text = await skill_element.text_content()
            if skill_text.strip().lower() == skill_type:
                await self.process_skill(skill_element, page, identity_id, skill_type)
        
    async def process_skill(self, skill_element, page, identity_id, skill_type):
        # skill_type이 "skill 3"일 경우 uptie_value가 3 이상일 때만 정보를 저장
        for uptie_value in range(1, 5):  # Uptie levels 1 to 4
            # "skill 3"은 uptie_value가 3 이상일 때만 처리
            if skill_type == "skill 3" and uptie_value < 3:
                continue  # skip if uptie_value is less than 3 for skill 3
    
            # uptie_level에 맞게 클릭
            await self.uptie_click(page, uptie_value)
            if skill_type != "skill 3" and uptie_value > 1:
                # Uptie 정보 갱신
                await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
                continue
            elif skill_type == "skill 3" and uptie_value > 3:
                # Uptie 정보 갱신
                await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
                continue
            # 스킬의 세부 정보를 추출
            skill_details = await self.extract_skill_details(skill_element, skill_type)
            skill_name = skill_details["skill_name"]
    
            # 스킬 정보를 저장
            self.container.add_skill(
                identity_id, skill_type, 
                skill_name, skill_details["coin_count"],
                skill_details["init_level"], skill_details["init_remain"], 
                skill_details["atk_weight"]
            )
            print(f"Added skill: {skill_type}:{skill_name}")
    
            # Uptie 정보 갱신
            await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
    
    async def extract_skill_details(self, skill_element, skill_type):
        # 스킬 세부 정보를 추출하는 함수
        skill_name = await self.skill_name(skill_element)
        coin_count = await self.coin_count(skill_element, ".col-auto > .column > .row")
        init_level = await self.init_level(skill_element)
        init_remain = await self.init_remain(skill_element, skill_type, "div.row.justify-start > div.attr.flex.flex-center > div.q-px-sm")
        atk_weight = await self.atk_weight(skill_element, skill_type)
        return {
            "skill_name": skill_name,
            "coin_count": coin_count,
            "init_level": init_level,
            "init_remain": init_remain,
            "atk_weight": atk_weight
        }
    
    async def update_uptie_info(self, page, skill_element, identity_id, skill_name, skill_type, uptie_value):
        # Uptie 정보를 추출하고 업데이트하는 함수
        uptie_info = await self.extract_uptie_info(skill_element)
        effect_data = await self.skill_effects(skill_element)
        self.container.add_skill_uptie_info(
            identity_id, skill_name, 
            uptie_value, uptie_info["skill_power"], uptie_info["coin_power"], 
            effect_data.get("effectsByCondition")
        )
        print(
            f"Updated Uptie for {skill_name} (Uptie Level: {uptie_value}): "
            f"Skill Power: {uptie_info['skill_power']}, Coin Power: {uptie_info['coin_power']} "
            f"Skill Effect: {effect_data.get("effectsByCondition")}"
        )

    async def extract_uptie_info(self, skill_element):
        skill_data = await self.skill_and_coin_power(skill_element)
        return {
            "skill_power": skill_data.get("skillPower"),
            "coin_power": skill_data.get("coinPower")
        }
        
    async def crawl_influence_behavior(self, page, identity_id):
        influence_behaviors = await page.query_selector_all('div[align="right"]')
        # 텍스트를 기준으 추출하고 메소드로 파라미터 들어가 정보를 받는다.
        for influence_behavior in influence_behaviors:
            active_type = await influence_behavior.inner_text()
            active_type = active_type.strip()

            if active_type == "PASSIVE":
                """
                # PASSIVE에 대한 처리
                print(f"Processing PASSIVE logic for: {active_type}")
                # CALL method for PASSIVE
                """
                passive_data = await self.passive_effects(influence_behavior)
            elif active_type == "SUPPORT PASSIVE":
                """
                # SUPPORT PASSIVE에 대한 처리
                print(f"Processing SUPPORT PASSIVE logic for: {active_type}")
                # CALL method for SUPPORT PASSIVE
                """
                support_passive_data = await self.support_passive_effects(influence_behavior)
            elif active_type == "PANIC TYPE":
                """
                # PANIC TYPE에 대한 처리
                print(f"Processing PANIC TYPE logic for:{active_type}")
                # CALL method for PANIC TYPE
                """
                panic_type_data = await self.panic_effects(influence_behavior)

    async def skill_and_coin_power(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                // 조부모로 이동
                const grandparent = node.parentElement.parentElement;
                // 이전 형제 탐색
                const previousSibling = grandparent.previousElementSibling;
                if (!previousSibling) return {{ skillPower: null, coinPower: null }}; // 이전 형제가 없으면 기본값 반환
                // 이전 형제 내부의 .row.justify-around > div > div 탐색
                const targetDivs = previousSibling.querySelectorAll(".row.justify-around > div");
                if (targetDivs.length < 3) return {{ skillPower: null, coinPower: null }}; // 배열 길이 확인
                // 1st div: 스킬 위력
                const skillPower = targetDivs[0]?.querySelector('div')?.textContent?.trim() || null; 
                // 3rd div: 코인 파워 값 (3번째 div에서 텍스트 추출)
                const coinPower = targetDivs[2]?.textContent?.trim() || null;
                return {{ skillPower, coinPower }};
            }}
        """)


    async def coin_count(self, skill_element, hierarchy_selector):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector('{hierarchy_selector}');
                if (!targetElement) return 0;
                return targetElement.childElementCount;
            }}
        """)

    async def init_remain(self, skill_element, skill_type, hierarchy_selector):
        if skill_type == "defense":
            return -1
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector('{hierarchy_selector}');
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def skill_name(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector(".block");
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def init_level(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector("div.text-bold.text-h6.h6.q-pl-xs.q-pr-md");
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def atk_weight(self, skill_element, skill_type):
        if skill_type == "defense":
            return 0
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector("div.text-bold.text-h6.h6.q-pl-xs.q-pr-md");
                const doubleNextSibling = targetElement.nextElementSibling.nextElementSibling;
                if (!doubleNextSibling) return 0;
                return doubleNextSibling.textContent?.trim();
            }}
        """)

    async def skill_effects(self, skill_element):
        return await skill_element.evaluate("""
            node => {
                const parent = node.parentElement;
                const doubleNextSibling = parent.nextElementSibling?.nextElementSibling;
                if (!doubleNextSibling) return null;
    
                // doubleNextSibling에서 첫 번째 div.col 요소 찾기
                const textNode = doubleNextSibling.querySelector("div.col");
                if (!textNode) return null;
    
                const results = {
                    keywords: [], // 키워드 저장
                    conditions: [], // 조건 저장
                    effectsByCondition: {} // 조건별 효과
                };
    
                // 1. 조건 추출 (대괄호 텍스트)
                const conditionElements = textNode.querySelectorAll("span");
                conditionElements.forEach(span => {
                    let text = span.textContent?.trim();
                    if (text?.startsWith("[") && text?.endsWith("]")) {
                        // 부모의 전형제 노드 탐색
                        const sibling = span.parentElement.previousElementSibling;
                        if (sibling) {
                            const emblemImage = sibling.querySelector("img.emblem");
                            console.log("Sibling element found:", sibling); // 디버깅용
                            if (emblemImage) {
                                const src = emblemImage.getAttribute("src");
                                console.log("Image source:", src); // 디버깅용
                                const match = src.match(/ui_num_(\\d+)\\.png$/); // 정규식 수정
                                if (match) {
                                    const number = match[1]; // 숫자 추출
                                    text = `[Coin ${number} ${text.replace("[", "").replace("]", "")}]`;
                                    console.log("Updated condition:", text); // 디버깅용
                                }
                            }
                        }
                        results.conditions.push(text);
                        results.effectsByCondition[text] = []; // 조건별 효과를 초기화
                    }
                });
    
                // 2. 키워드 추출
                const keywordElements = textNode.querySelectorAll('a[href^="/en/keyword/"]');
                keywordElements.forEach(a => {
                    const span = a.querySelector('span.q-btn__content.text-center.col.items-center.q-anchor--skip.justify-center.row>span');
                    if (span) {
                        const keywordText = span.textContent.trim();
                        if (keywordText && !results.keywords.includes(keywordText)) {
                            results.keywords.push(keywordText); // 키워드 저장
                        }
                    }
                });
    
                // 3. 효과 추출
                const effectElements = textNode.querySelectorAll("span");
                let currentCondition = null;
                let currentEffect = []; // 초기화
                effectElements.forEach(span => {
                    let text = span.textContent?.trim();
                    if (!text) return;
    
                    // 조건 발견 시 currentCondition 변경
                    if (text.startsWith("[") && text.endsWith("]")) {
                        // 이전 조건의 효과 저장
                        if (currentCondition && currentEffect.length > 0) {
                            // 부모의 전형제 노드 탐색
                            const sibling = span.parentElement.previousElementSibling;
                            if (sibling) {
                                const emblemImage = sibling.querySelector("img.emblem");
                                if (emblemImage) {
                                    const src = emblemImage.getAttribute("src");
                                    const match = src.match(/ui_num_(\\d+)\\.png$/); // 정규식 수정
                                    if (match) {
                                        const number = match[1]; // 숫자 추출
                                        text = `[Coin ${number} ${text.replace("[", "").replace("]", "")}]`;
                                    }
                                }
                            }
                            results.effectsByCondition[currentCondition].push(currentEffect.join(' '));
                            currentEffect = []; // 초기화
                        }
                        currentCondition = text;
                        return;
                    }
    
                    // 현재 조건의 효과 처리
                    if (currentCondition) {
                        if (results.keywords.includes(text)) {
                            const keywordEffect = 'keyword:' + text;
                            if (!currentEffect.includes(keywordEffect)) {
                                currentEffect.push(keywordEffect);
                            }
                        } else {
                            currentEffect.push(text);
                        }
                    }
                });
    
                // 마지막 효과 저장
                if (currentCondition && currentEffect.length > 0) {
                    results.effectsByCondition[currentCondition].push(currentEffect.join(' '));
                }
    
                return results;
            }
        """)

    async def passive_effects(self, passive_element):
        passive_name = await self.behavior_name(passive_element)
        passive_conditions = await self.behavior_condition(passive_element)
        passive_description = await self.behavior_description(passive_element)
        print(passive_name)
        print(passive_conditions)
        print(passive_description)
    async def support_passive_effects(self, support_passive_element):
        support_passive_name = await self.behavior_name(support_passive_element)
        support_passive_conditions = await self.behavior_condition(support_passive_element)
        support_passive_description = await self.behavior_description(support_passive_element)
        print(support_passive_name)
        print(support_passive_conditions)
        print(support_passive_description)
    async def panic_effects(self, panic_element):
        panic_type = await self.behavior_name(panic_element)
        panic_conditions = await self.behavior_condition(panic_element)
        panic_description = await self.panic_description(panic_element)
        print(panic_type)
        print(panic_conditions)
        print(panic_description)

    async def navigate_element_by_steps(self, element, parent_steps=0, prev_sibling_steps=0, next_sibling_steps=0):
        """
        주어진 `element`에서 부모, 이전 형제, 또는 다음 형제로 지정된 횟수만큼 이동하여 ElementHandle 반환.
        
        Args:
            element (ElementHandle): 탐색 기준이 되는 요소.
            parent_steps (int): 부모로 이동할 횟수 (0이면 이동하지 않음).
            prev_sibling_steps (int): 이전 형제로 이동할 횟수 (0이면 이동하지 않음).
            next_sibling_steps (int): 다음 형제로 이동할 횟수 (0이면 이동하지 않음).
        
        Returns:
            ElementHandle or None: 탐색된 요소를 반환하거나 없으면 None.
        """
        try:
            current_element = element
    
            # 부모로 이동
            for _ in range(parent_steps):
                current_element = await current_element.evaluate_handle('el => el.parentElement')
                if not current_element:
                    print("Parent element not found during navigation.")
                    return None
    
            # 이전 형제로 이동
            for _ in range(prev_sibling_steps):
                current_element = await current_element.evaluate_handle('el => el.previousElementSibling')
                if not current_element:
                    print("Previous sibling element not found during navigation.")
                    return None
    
            # 다음 형제로 이동
            for _ in range(next_sibling_steps):
                current_element = await current_element.evaluate_handle('el => el.nextElementSibling')
                if not current_element:
                    print("Next sibling element not found during navigation.")
                    return None
    
            return current_element
    
        except Exception as e:
            print(f"Error navigating elements: {e}")
            return None

    async def get_last_sibling(self, element):
        try:
            # 같은 부 아 마지막 형제를 가져오는 JavaScript 실행
            last_sibling = await element.evaluate_handle('el => el.parentElement.lastElementChild')
            return last_sibling
        except Exception as e:
            print(f"Error getting last sibling: {e}")
            return None

    async def behavior_name(self, element):
        parent_prev_sibling = await self.navigate_element_by_steps(element, parent_steps=1, prev_sibling_steps=1, next_sibling_steps=0)
        passive_name_element = await parent_prev_sibling.query_selector('span.block')
        if passive_name_element:
            passive_name = await passive_name_element.inner_text()
            return passive_name
        else:
            #print("Passive name element not found")
            return None

    async def behavior_condition(self, element):
        try:
            conditions = []
            parent_prev_sibling = await self.navigate_element_by_steps(element, parent_steps=1, prev_sibling_steps=1, next_sibling_steps=0)
            img_elements = await parent_prev_sibling.query_selector_all('img')
            if not img_elements:
                #print("No <img> elements found.")
                return conditions
    
            for img_element in img_elements:
                # 각 이미지의 src 속성 가져오기
                img_src = await img_element.get_attribute('src')
                #print(img_src)
    
                sin_name = None
                for color_key in resource_mapping.keys():
                    if f"{self.img_url}/ui/attr_{color_key}.png" in img_src:
                        sin_name = resource_mapping[color_key]
                        break
    
                if not sin_name:
                    #print(f"No matching Sin Color found for {img_src}.")
                    continue  # No Sin Color found, skip this img_element
    
                # 해당 이미지와 관련된 조건 텍스트 추출
                # 두 번째 부모 요소에서 시작하여 다음 형제를 찾아 조건 텍스트를 추출
                condition_element = await self.navigate_element_by_steps(img_element, parent_steps=3, prev_sibling_steps=0, next_sibling_steps=0)
                if condition_element:
                    # condition_element가 None이 아닌 경우에만 진행
                    condition_text_element = await condition_element.query_selector('.q-pl-xs')
                    if condition_text_element:
                        condition_text = await condition_text_element.inner_text()
                        condition_text = condition_text.strip()
                    else:
                        print(f"No condition text found for {img_src}.")
                        condition_text = ""  # condition_text_element가 None이면 빈 문자열로 처리
                else:
                    print(f"No condition element found for {img_src}.")
                    condition_text = ""  # condition_element가 None이면 빈 문자열로 처리
    
                # 결과를 배열에 추가
                conditions.append(f"{sin_name} {condition_text}")
            return conditions
        except Exception as e:
            print(f"Error extracting conditions: {e}")
            return 

    async def behavior_description(self, element):
        try:
            description_texts = []
            parent_prev_sibling = await self.navigate_element_by_steps(element, parent_steps=1, prev_sibling_steps=1, next_sibling_steps=0)
            behavior_title = await parent_prev_sibling.query_selector('.q-pb-sm')
            behavior_description_element = await self.get_last_sibling(behavior_title)
            descriptionElements = await behavior_description_element.query_selector_all("span");
            for descriptionElement in descriptionElements:
                description_text = await descriptionElement.inner_text()
                stripped_text = description_text.strip()
                if stripped_text:
                    description_texts.append(stripped_text)
            return description_texts
        except Exception as e:
            print(f"Error extracting description: {e}")
            return

    async def panic_description(self, element):
        try:
            description_texts = []
            parent_prev_sibling = await self.navigate_element_by_steps(element, parent_steps=1, prev_sibling_steps=1, next_sibling_steps=0)
            behavior_title = await parent_prev_sibling.query_selector('.q-pb-sm')
            behavior_description_element = await self.get_last_sibling(behavior_title)
            descriptionElements = await behavior_description_element.query_selector_all("div");
            for descriptionElement in descriptionElements:
                description_text = await descriptionElement.inner_text()
                description_texts.append(description_text.strip())
            return description_texts
        except Exception as e:
            print(f"Error extracting description: {e}")
            return

class Identity:
    """인격 정보를 저장하는 클래스"""
    def __init__(self, identity_id, name=None, rarity=None,\
                 skill_1_type=None, skill_1_sin_affinity=None,\
                 skill_2_type=None, skill_2_sin_affinity=None,\
                 skill_3_type=None, skill_3_sin_affinity=None,\
                 defense_type=None, defense_sin_affinity=None):
        self.identity_id = identity_id
        self.name = name
        self.rarity = rarity
        # 선언만 하는 부분 
        self.skill_1_type = skill_1_type 
        self.skill_1_sin_affinity = skill_1_sin_affinity 
        self.skill_2_type = skill_2_type 
        self.skill_2_sin_affinity = skill_2_sin_affinity 
        self.skill_3_type = skill_3_type 
        self.skill_3_sin_affinity = skill_3_sin_affinity 
        self.defense_type = defense_type 
        self.defense_sin_affinity = defense_sin_affinity
        self.level_max = 50
        self.uptie_max = 4
        self.level_stats = []  # LevelStat 객체 리스트
        self.uptie_stats = [] # UptieStat 객체 리스트
        self.resist_info = None
        self.stagger_threshold = None
        self.skillset = SkillSet()  # Skill정보를 담고 있는 skillset 객체 
        self.passive_infos = []  # 패시브 객체를 담고 있는 배열
        self.support_passive_infos = [] # 서포트 패시 객체를 담고 있는 배열
        self.panic_type = [] # 혼란상태 발생시 나타나는 효과를 담고 있는 객체 배열1

    def update_name(self, name):
        self.name = name

    def update_rarity(self, rarity):
        self.rarity = rarity

    def add_level_stat(self, level, hp, defense):
        self.level_stats.append(LevelStat(level, hp, defense))

    def add_uptie_stat(self, uptie, speed):
        self.uptie_stats.append(UptieStat(uptie, speed))

    def add_resist_info(self, resist_hit, resist_penetrate, resist_slash):
        self.resist_info = ResistInfo(resist_hit, resist_penetrate, resist_slash)

    def add_stagger_threshold(self, first_stagger, second_stagger, third_stagger):
        self.stagger_threshold = StaggerThreshold(first_stagger, second_stagger, third_stagger)

    def add_skill(self, skill_code, skill_name, coin_count, init_level, init_remain, atk_weight):
        self.skillset.add_skill(skill_code, Skill(skill_code, skill_name, coin_count, init_level, init_remain, atk_weight))
        
    def add_skill_uptie_info(self, skill_name, uptie_level, skill_power, coin_power, effect_by_condition):
        self.skillset.add_skill_uptie_info(skill_name, uptie_level, skill_power, coin_power, effect_by_condition)
        
    def update_details(self, page):
        # 세부 정보 업데이트 로직 (가정)
        pass
    #def init_uptie(self, page):
    #    init_uptie = 

# LevelStat 객체 정의
class LevelStat:
    def __init__(self, level, hp=None, defense=None):
        self.level = level
        self.hp = hp
        self.defense = defense
    def update_level(self, level):
        self.level = level
    def update_hp(self, hp):
        self.hp = hp
    def update_defense(self, defense):
        self.defense = defense

# UptieStat 객체 정의
class UptieStat:
    def __init__(self, uptie, speed=None):
        self.uptie = uptie
        self.speed = speed
    def update_speed(self, speed):
        self.speed = speed

# ResistInfo 객체 정의
class ResistInfo:
    def __init__(self, resist_hit=None, resist_penetrate=None, resist_slash=None):
        self.resist_hit = resist_hit
        self.resist_penetrate = resist_penetrate
        self.resist_slash = resist_slash

# ResistInfo 객체 정의
class StaggerThreshold:
    def __init__(self, first_stagger=None, second_stagger=None, third_stagger=None):
        self.first_stagger = first_stagger
        self.second_stagger = second_stagger
        self.third_stagger = third_stagger

class SkillSet:
    def __init__(self):

        self.skill_1_type = ""
        self.skill_2_type = ""
        self.skill_3_type = ""
        self.defense_skill_type = ""

        self.skill_1_sin_affinity = ""
        self.skill_2_sin_affinity = ""
        self.skill_3_sin_affinity = ""
        self.defense_skill_sin_affinity = ""

        self.skill_1_details = []
        self.skill_2_details = []
        self.skill_3_details = []
        self.defense_skill_details = []

    def set_skill_type(self, skill_code, skill_type, sin_affinity):
        if skill_code == "skill 1":
            self.skill_1_type = skill_type
            self.skill_1_sin_affinity = sin_affinity
        elif skill_code == "skill 2":
            self.skill_2_type = skill_type
            self.skill_2_sin_affinity = sin_affinity
        elif skill_code == "skill 3":
            self.skill_3_type = skill_type
            self.skill_3_sin_affinity = sin_affinity
        elif skill_code == "defense":
            self.defense_skill_type = skill_type
            self.defense_skill_sin_affinity = sin_affinity
    
    def add_skill(self, skill_code, skill):
        if skill_code == "skill 1":
            self.skill_1_details.append(skill)
            skill.skill_type = self.skill_1_type
            skill.sin_affinity = self.skill_1_sin_affinity
        elif skill_code == "skill 2":
            self.skill_2_details.append(skill)
            skill.skill_type = self.skill_2_type
            skill.sin_affinity = self.skill_2_sin_affinity
        elif skill_code == "skill 3":
            self.skill_3_details.append(skill)
            skill.skill_type = self.skill_3_type
            skill.sin_affinity = self.skill_3_sin_affinity
        elif skill_code == "defense":
            self.defense_skill_details.append(skill)
            skill.skill_type = self.defense_skill_type 
            skill.sin_affinity = self.defense_skill_sin_affinity

    def get_skill_by_name(self, skill_name):
        for skill_list in [self.skill_1_details, self.skill_2_details, self.skill_3_details, self.defense_skill_details]:
            for skill in skill_list:
                result = skill.is_skill_by_name(skill_name)
                if result:
                    return result
        return None

    def add_skill_uptie_info(self, skill_name, uptie_level, skill_power, coin_power, effect_by_condition):
        skill = self.get_skill_by_name(skill_name)
        skill.add_uptie_info(uptie_level, skill_power, coin_power, effect_by_condition)
        

class Skill:
    def __init__(self, skill_code, skill_name, coin_count, init_level, init_remain, atk_weight):
        self.skill_code = skill_code # "Skill 1"/"Skill 2"/"Skill 3"/"Defense" '''o'''
        self.skill_type = None
        self.sin_affinity = None
        self.skill_name = skill_name # 스킬 이름 '''o'''
        self.coin_count = coin_count # 동전 개수 '''o'''
        self.init_level = init_level # 스킬이 초기 배정받은 레벨, 인격 레벨에 - 1 한 값을 더해주면 스킬의 현 level을 구할 수 있다. '''o'''
        self.init_remain = init_remain #'''o'''
        self.atk_weight = atk_weight #공격 가중치 값이다. 방어스킬에는 없기 때문에 None으로 설정했다. '''o'''
        self.remain_count = init_remain # 실제 스킬이 작동하며 횟수가 차감되 나중에 모두 차감되면 다음 턴에 다시 초기횟수 리필된다.
        self.uptie_info = []  

    def is_skill_by_name(self, skill_name):
        if self.skill_name == skill_name:
            return self
        return None
        
    def add_uptie_info(self, uptie_level, skill_power, coin_power, effects_by_condition):
        self.uptie_info.append(UptieInfo(uptie_level, skill_power, coin_power, effects_by_condition))

class UptieInfo:
    def __init__(self, uptie_level, skill_power, coin_power, effects_by_condition):
        self.uptie_level = uptie_level   #'''o'''
        self.skill_power = skill_power   #'''o'''
        self.coin_power = coin_power     #'''o'''
        self.skill_effect = effects_by_condition

# 추상화된 EffectType 클래스
class EffectType:
    def __init__(self, name, description):
        self.name = name  # Effect 이름
        self.description = description  # Effect 설명

# Passive 클래스
class Passive(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)  # condition이 있는지 여부 확인

# SupportPassive 클래스
class SupportPassive(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)

# PanicType 클래스
class PanicType(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)

# Keyword 클래스
class Keyword:
    def __init__(self, keyword, explain=None, dispellable=None):
        self.keyword = keyword
        self.explain = explain
        self.dispellable = dispellable

      
class DataContainer:
    """크롤링 데이터를 관리하는 클래스"""
    def __init__(self):
        self.identities = []

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.level_max

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.uptie_max

    def update_identity_name(self, identity_id, identity_name):
        """특정 ID의 이름을 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_name(identity_name)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_identity_rarity(self, identity_id, rarity):
        """특정 ID의 희귀도를 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_rarity(rarity)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_skill(self, identity_id, skill_number, new_type):
        """
        특정 identity의 skill_type을 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_type: 새 skill_type
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_type = new_type
                elif skill_number == 2:
                    identity.skill_2_type = new_type
                elif skill_number == 3:
                    identity.skill_3_type = new_type
                elif skill_number == "defense":
                    identity.defense_type = new_type
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def update_sin_affinity(self, identity_id, skill_number, new_affinity):
        """
        특정 identity의 sin_affinity를 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_affinity: 새 sin_affinity
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_sin_affinity = new_affinity
                elif skill_number == 2:
                    identity.skill_2_sin_affinity = new_affinity
                elif skill_number == 3:
                    identity.skill_3_sin_affinity = new_affinity
                elif skill_number == "defense":
                    identity.defense_sin_affinity = new_affinity
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def add_identity(self, identity):
        """새로운 인격 데이터를 추가."""
        self.identities.append(identity)

    def add_level_stat(self, identity_id, level, hp, defense):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_level_stat(level, hp, defense)

    def add_uptie_stat(self, identity_id, uptie, speed):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_uptie_stat(uptie, speed)

    def add_resist_info(self, identity_id, resist_hit, resist_penetrate, resist_slash):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_resist_info(resist_hit, resist_penetrate, resist_slash)

    def add_stagger_threshold(self, identity_id, first_stagger, second_stagger, third_stagger):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_stagger_threshold(first_stagger, second_stagger, third_stagger)

    
    def add_skill(self, identity_id, skill_code, skill_name, coin_count, init_level, init_remain, atk_weight):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_code == "skill 1":
                    identity.skillset.set_skill_type(skill_code, identity.skill_1_type, identity.skill_1_sin_affinity)
                elif skill_code == "skill 2":
                    identity.skillset.set_skill_type(skill_code, identity.skill_2_type, identity.skill_2_sin_affinity)
                elif skill_code == "skill 3":
                    identity.skillset.set_skill_type(skill_code, identity.skill_3_type, identity.skill_3_sin_affinity)
                elif skill_code == "defense":
                    identity.skillset.set_skill_type(skill_code, identity.defense_type, identity.defense_sin_affinity)
                identity.add_skill(skill_code, skill_name, coin_count, init_level, init_remain, atk_weight)

    def add_skill_uptie_info(self, identity_id, skill_name, uptie_level, skill_power, coin_power, effects_by_condition):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_skill_uptie_info(skill_name, uptie_level, skill_power, coin_power, effects_by_condition)
        
    def display_identities(self):
        """Identity, ID, Name, Rarity 정보를 출력"""
        if not self.identities:
            print("No identities found.")
        else:
            # 데이터를 리스트로 변환
            table = []
            for idx, identity in enumerate(self.identities, 1):
                table.append([
                    idx, identity.identity_id, identity.name, identity.rarity,
                    identity.skill_1_type, identity.skill_1_sin_affinity,
                    identity.skill_2_type, identity.skill_2_sin_affinity,
                    identity.skill_3_type, identity.skill_3_sin_affinity,
                    identity.defense_type, identity.defense_sin_affinity
                ])
            
            # 테이블 헤더 정의
            headers = [
                "Index", "ID", "Name", "Rarity",
                "Skill_1 Type", "Skill_1 Sin", "Skill_2 Type", "Skill_2 Sin",
                "Skill_3 Type", "Skill_3 Sin", "Defense Type", "Defense Sin"
            ]
            
            # 테이블 출력
            print(tabulate(table, headers=headers, tablefmt="fancy_grid"))

    def export_to_csv(self, filename, fieldnames, data):
        """
        공통적으로 사용 가능한 CSV 내보내기 메서드
        :param filename: 내보낼 파일 이름 (예: 'identity.csv', 'skill.csv')
        :param fieldnames: CSV의 헤더 리스트
        :param data: CSV로 내보낼 데이터 (리스트 형태)
        """
        try:
            # 현재작업 디렉토 기준으 'csv'폴더 없으면 생성
            if not os.path.exists('./csv'):
                os.makedirs('./csv')
            # 파일 경로 설정 (상대경로 'csv' 폴더 안에 저장)
            file_path = os.path.join('./csv', filename)
            with open(file_path, mode='w', newline='', encoding='utf-8') as file:
                writer = csv.DictWriter(file, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(data)
            print(f"Data successfully exported to {filename}")
        except Exception as e:
            print(f"Error exporting to CSV: {e}")

    def get_identity_data(self):
        """
        Identity 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "ID": identity.identity_id,
                "Name": identity.name,
                "Rarity": identity.rarity,
                "Skill_1_Type": identity.skill_1_type,
                "Skill_1_Sin_Affinity": identity.skill_1_sin_affinity,
                "Skill_2_Type": identity.skill_2_type,
                "Skill_2_Sin_Affinity": identity.skill_2_sin_affinity,
                "Skill_3_Type": identity.skill_3_type,
                "Skill_3_Sin_Affinity": identity.skill_3_sin_affinity,
                "Defense_Type": identity.defense_type,
                "Defense_Sin_Affinity": identity.defense_sin_affinity,
            }
            for identity in self.identities
        ]
        
    def export_identities_data(self, filename="identities.csv"):
        """Identity 데이터를 CSV로 내보내는 메서드"""
        identity_fieldnames = [
            "ID", "Name", "Rarity",
            "Skill_1_Type", "Skill_1_Sin_Affinity",
            "Skill_2_Type", "Skill_2_Sin_Affinity",
            "Skill_3_Type", "Skill_3_Sin_Affinity",
            "Defense_Type", "Defense_Sin_Affinity"
        ]
        identity_data = self.get_identity_data()  # Identity 데이터 가져오기
        self.export_to_csv(filename, identity_fieldnames, identity_data)  # CSV로 내보내기

    def load_identities_by_id(self, filename, identity_ids):
        """지정된 identity_ids 목록에 해당하는 Identity 데이터를 CSV에서 읽어옴."""
        identity_ids_set = set(identity_ids) # 중복 제거를 위 Set 사용
        with open(filename, mode='r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                if row["ID"] in identity_ids_set:
                    identity = Identity(
                        identity_id=row["ID"],
                        name=row["Name"],
                        rarity=row["Rarity"],
                        skill_1_type=row["Skill_1_Type"],
                        skill_1_sin_affinity=row["Skill_1_Sin_Affinity"],
                        skill_2_type=row["Skill_2_Type"],
                        skill_2_sin_affinity=row["Skill_2_Sin_Affinity"],
                        skill_3_type=row["Skill_3_Type"],
                        skill_3_sin_affinity=row["Skill_3_Sin_Affinity"],
                        defense_type=row["Defense_Type"],
                        defense_sin_affinity=row["Defense_Sin_Affinity"]
                    )
                    self.identities.append(identity)

    def get_level_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Level": level_stat.level,
                "HP": level_stat.hp,
                "Defense": level_stat.defense,
            }
            for identity in self.identities
            for level_stat in identity.level_stats
        ]

    def export_level_stats_data(self, filename="levelstats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        level_stats_fieldnames = ["Identity_ID", "Level", "HP", "Defense"]
        level_stats_data = self.get_level_stats_data()  # level_stats 데이터 가져오기
        self.export_to_csv(filename, level_stats_fieldnames, level_stats_data)  # CSV로 내보내기

    def get_uptie_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Uptie": uptie_stat.uptie,
                "Speed": uptie_stat.speed,
            }
            for identity in self.identities
            for uptie_stat in identity.uptie_stats
        ]

    def export_uptie_stats_data(self, filename="Uptiestats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        uptie_stats_fieldnames = ["Identity_ID", "Uptie", "Speed"]
        uptie_stats_data = self.get_uptie_stats_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, uptie_stats_fieldnames, uptie_stats_data)  # CSV로 내보내기

    def get_resist_info_data(self):
        """
        모든 Identity의 resist_info 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Resist_Hit": identity.resist_info.resist_hit,
                "Resist_Penetrate": identity.resist_info.resist_penetrate,
                "Resist_Slash": identity.resist_info.resist_slash,
            }
            for identity in self.identities
        ]

    def export_resist_info_data(self, filename="ResistInfos.csv"):
        """ResistInfo 데이터를 CSV로 내보내는 메서드"""
        resist_infos_fieldnames = ["Identity_ID", "Resist_Hit", "Resist_Penetrate", "Resist_Slash"]
        resist_infos_data = self.get_resist_info_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, resist_infos_fieldnames, resist_infos_data)  # CSV로 내보내기

    def export_uptie_stats_data(self, filename="Uptiestats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        uptie_stats_fieldnames = ["Identity_ID", "Uptie", "Speed"]
        uptie_stats_data = self.get_uptie_stats_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, uptie_stats_fieldnames, uptie_stats_data)  # CSV로 내보내기

    def get_stagger_threshold_data(self):
        """
        모든 Identity의 stagger_threshold 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "First_Stagger": identity.stagger_threshold.first_stagger,
                "Second_Stagger": identity.stagger_threshold.second_stagger,
                "Third_Stagger": identity.stagger_threshold.third_stagger,
            }
            for identity in self.identities
        ]

    def export_stagger_threshold_data(self, filename="StaggerThresholds.csv"):
        """StaggerThresholds 데이터를 CSV로 내보내는 메서드"""
        stagger_thresholds_fieldnames = ["Identity_ID", "First_Stagger", "Second_Stagger", "Third_Stagger"]
        stagger_thresholds_data = self.get_stagger_threshold_data()  # stagger_thresholds 데이터 가져오기
        self.export_to_csv(filename, stagger_thresholds_fieldnames, stagger_thresholds_data)  # CSV로 내보내기

    def get_skillset_data(self):
        """모든 Identity의 skillset 데이터를 export_to_csv을 포맷으로 변환"""
        return [
            {
                "Identity_ID": identity.identity_id,
                "Skill_Code": skill.skill_code,
                "Skill_Type": skill.skill_type,
                "Sin_Affinity": skill.sin_affinity,
                "Skill_Name": skill.skill_name,
                "Coin_Count": skill.coin_count,
                "Init_Level": skill.init_level,
                "Init_Remain": skill.init_remain,
                "Atk_Weight": skill.atk_weight,
                "Uptie_Level": uptie_info.uptie_level,
                "Skill_Power": uptie_info.skill_power,
                "Coin_Power": uptie_info.coin_power,
                "Effect_Data": json.dumps(uptie_info.skill_effect)
            }
            for identity in self.identities
            for skill_list in [identity.skillset.skill_1_details, identity.skillset.skill_2_details, identity.skillset.skill_3_details, identity.skillset.defense_skill_details]
            for skill in skill_list
            for uptie_info in skill.uptie_info
        ]

    def export_identity_skillset_data(self, filename="IdentitiesSkills.csv"):
        """IdentitiesSkills 데이터를 CSV로 내보내는 메서드"""
        identities_skills_fieldnames = ["Identity_ID", "Skill_Code", "Skill_Type", "Sin_Affinity", "Skill_Name", "Coin_Count", "Init_Level", "Init_Remain", "Atk_Weight", "Uptie_Level", "Skill_Power", "Coin_Power", "Effect_Data"]
        identities_skills_data = self.get_skillset_data()
        self.export_to_csv(filename, identities_skills_fieldnames, identities_skills_data)
            


async def main():
    data_container = DataContainer()
    scraper = Scraper(container=data_container, base_url="https://limbus.kusoge.xyz", img_url="https://img.kusoge.xyz/limbus/img")
    await scraper.scrape()

await main() 

Bloodfeast
[]
['If', 'this', 'unit', 'is', 'on', 'field', ',', 'or', 'is', 'one', 'of', 'the', 'units', 'that', 'can', 'appear', 'on', 'this', 'Stage', ',', 'increase', 'Bloodfeast', 'Bloodfeast', 'value', 'by', 'the', 'amount', 'of', 'Bleed', 'Bleed', 'damage', 'collectively', 'received', 'by', 'every', 'unit', '.', 'When', 'this', 'unit', 'enters', 'the', 'field', ',', 'the', 'sleeping', 'blood', 'drenching', 'the', 'battlefield', 'will', 'fill', 'the', 'surface', '.']
Hardblood Thorn
['Lust x3 Owned', 'Envy x2 Owned']
['When', 'an', 'other', 'ally', 'takes', 'Bleed', 'Bleed', 'damage', 'or', 'consumes', 'Bloodfeast', 'Bloodfeast', ',', 'gain', '1', 'blooming[thornyfall_panic', 'RodionFirst', '] (', '3', 'times', 'per', 'turn', ')', 'Heal', 'HP', 'on', 'self', 'by', '20', '%', 'of', 'the', 'damage', 'dealt', 'with', 'base', 'Skills', '(', 'max', '10', 'per', 'Skill', ')']
"Blossom with Blood..."
['Lust x3 Owned', 'Envy x3 Owned']
['To', '1', 'ally', 'with', 'the', 'highest', 'Bleed',

In [20]:
import os
import csv
import re
import json
import numpy as np
import pandas as pd
from playwright.async_api import async_playwright
from tabulate import tabulate

# Sin Color 이름 변환을 위한 매핑
resource_mapping = {
    "AZURE": "Gloom",
    "VIOLET": "Envy",
    "AMBER": "Sloth",
    "SHAMROCK": "Gluttony",
    "CRIMSON": "Wrath",
    "SCARLET": "Lust",
    "INDIGO": "Pride"
}

def convert_resource(resource_name):
    """Sin Color 이름을 변환하는 함수. 매핑되지 않은 경우 원래 이름을 반환."""
    return resource_mapping.get(resource_name, resource_name)

def find_sin_for_color(url):
    # URL에 color_name을 추출
    color_name = url.split('/')[-1].split('.')[0].replace("attr_", "")
    # 해당 색상이 resource_mapping에 있는 확인
    for color, sin in resource_mapping.items():
        if color_name in color:
            return sin
    return None


class Scraper:
    """웹 페이지 크롤링을 위한 클래스"""
    def __init__(self, container, base_url, img_url):
        self.container = container
        self.base_url = base_url
        self.img_url = img_url
        self.uptie_level = 1

    async def handle_request(self, route, request):
        """이미지와 폰트 리소스 로드를 차단."""
        if request.resource_type =="image":
            #print(f"{request.url}")
            img_url = request.url
            if  re.match(f"{self.img_url}/ui/attr.*\\.png", img_url):
                await route.continue_()
            else:
                await route.fulfill(
                    status=200,
                    content_type="image/png",
                    body=b""
                )
        elif request.resource_type == "font":
            await route.abort()
        else:
            await route.continue_()
            
    async def scrape(self):
        """크롤링 메인 함수"""
        async with async_playwright() as p:
            browser = await p.chromium.launch(headless=True)
            page = await browser.new_page()
            #await page.evaluate('window.scrollTo(0, document.body.scrollHeight)')
            await page.route("**/*", self.handle_request)
            # Step 1: /identity 페이지에서 기본 정보 크롤링
            #await self.crawl_identity_page(page)

            # Step 2: /identity/{identity_id} 상세 페이지 크롤링
            #identity_ids_to_load = ["10104", "10310", "10710","10808", "10911", "11005"]
            #identity_ids_to_load = ["10104", "10911"]#,"10310", "10710","10808", "11005", "11210"]
            #identity_ids_to_load = ["10911","10310"]#, "10710","10808", "11005", "11210"]
            identity_ids_to_load = ["10911"]#, "10611"]#,"10711"]#, "10710","10808", "11005", "11210"]
            self.container.load_identities_by_id('./csv/identities.csv', identity_ids_to_load)
            await self.crawl_identity_details_for_all(page)
            #self.container.display_identities()
            #self.container.export_identities_data("identities.csv")
            #self.container.export_level_stats_data("Levelstats.csv")
            #self.container.export_uptie_stats_data("Uptiestats.csv")
            #self.container.export_resist_info_data("ResistInfos.csv")
            #self.container.export_stagger_threshold_data("StaggerThresholds.csv")
            #self.container.export_identity_skillset_data("IdentitiesSkills.csv")
            page.set_default_timeout(10000)
            #await page.wait_for_timeout(100000)
            await browser.close()

    async def get_identity_id_from_row(self, row):
        """테이블 행(row)에서 인격 ID를 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_url = await identity_cell.get_attribute('href')
            identity_id = identity_url.split('/identity/')[-1]
            return identity_id
        return "Unknown"

    async def get_identity_name_from_row(self, row):
        """테이블 행(row)에서 인격 이름을 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_name = (await identity_cell.inner_text()).replace("\n", " ").strip()
            return identity_name
        return "Unknown"
        
    async def crawl_identity_page(self, page):
        """/identity 페이지로 이동하여 인격 기본 정보를 추출."""
        await page.goto(self.base_url + "/identity")
        await self.extract_identity_page(page)

    async def extract_identity_page(self, page):
        """/identity 페이지의 모든 테이블 행에서 인격 정보를 추출."""
        identity_rows = await page.query_selector_all('tr')
        if len(identity_rows) > 1:
            for idx, identity_row in enumerate(identity_rows):
                if idx == 0:
                    continue # 0번째요소이름을 보여주는 열은 건너뜀
                await self.scrape_identity_name_and_identity_id(identity_row)
                await self.extract_rarity(identity_row)
                for i in range(1, 4):  # 스킬1~3 정보 추출
                    await self.extract_atk_and_resource(identity_row, i)
                await self.extract_defense_and_resource(identity_row)

    async def scrape_identity_name_and_identity_id(self, row):
        """인격 번호와 이름을 추출하여 DataContainer에 저장."""
        try:
            identity_id = await self.get_identity_id_from_row(row)
            identity_name = await self.get_identity_name_from_row(row)
            identity = Identity(identity_id)
            self.container.add_identity(identity)
            self.container.update_identity_name(identity_id, identity_name)
        except Exception as e:
            print(f"Error extracting identity name and identity id: {e}")

    async def get_element_attribute(self, element, selector, attribute):
        """요소에서 지정된 속성(attribute)을 추출."""
        try:
            target_element = await element.query_selector(selector)
            if target_element:
                return await target_element.get_attribute(attribute)
        except Exception as e:
            print(f"Error getting element attribute: {e}")
        return None
    
    async def extract_rarity(self, row):
        """테이블 행(row)에서 희귀도를 추출하여 업데이트."""
        try:
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
                sibling = await parent.evaluate_handle('node => node.nextElementSibling')
                if sibling:
                    rarity_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/rarity_"]', 'src')
                    if rarity_img_src:
                        rarity_level = rarity_img_src.split('_')[-1].split('.')[0]  # rarity_1.png -> 1
                        identity_id = await self.get_identity_id_from_row(row)
                        self.container.update_identity_rarity(identity_id, rarity_level)
                    else:
                        print("Rarity image not found in the next sibling TD.")
        except Exception as e: 
            print(f"Error extracting rarity: {e}")

        return "Unknown"

    async def extract_atk_and_resource(self, row, skill_number):
        """ 공격 타입(ATK Type)과 자원(Attribute) 추출 함수 - 스킬 번호에 따라 처리 """
        try:
            #print(f"Attempting to locate skill{skill_number} type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 해당 스킬 번호에 맞는 형제로 이동
                sibling = parent
                for _ in range(skill_number + 1):  # 스킬 번호에 맞게 이동
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                atk_type, resource = "Unknown", "Unknown"
        
                if sibling:
                    # 공격 타입 이미지 (atk_type) 찾기
                    atk_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/atk_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if atk_type_img_src:
                        atk_type = atk_type_img_src.split('_')[-1].split('.')[0]  # atk_type_SLASH.png -> SLASH
                        #print(f"Extracted attack type: {atk_type}")
                        self.container.update_skill(identity_id, skill_number, atk_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted resource: {resource}")
                        
                        # 리소스를 변환
                        resource = convert_resource(resource)
                        #print(f"Converted resource: {resource}")
                        self.container.update_sin_affinity(identity_id, skill_number, resource)
                        
        except Exception as e:
            print(f"Error extracting skill{skill_number} type and resource: {e}")

    async def extract_defense_and_resource(self, row):
        """ 방어 타입(Def Type)과 자원(Attribute) 추출 함수 """
        try:
            #print("Attempting to locate defense type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 다섯 번째 형제 td로 이동 (방어 관련 정보가 있는 <td>)
                sibling = parent
                for _ in range(5):  # 방어 정보는 5번째 형제에 있음
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                def_type, def_resource = "Unknown", "Unknown"
        
                if sibling:
                    # 방어 타입 이미지 (def_type) 찾기
                    def_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/def_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if def_type_img_src:
                        def_type = def_type_img_src.split('_')[-1].split('.')[0]  # def_type_GUARD.png -> GUARD
                        #print(f"Extracted defense type: {def_type}")
                        self.container.update_skill(identity_id, 'defense', def_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        def_resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted defense resource: {def_resource}")
                        
                        # 방어 리소스를 변환
                        def_resource = convert_resource(def_resource)
                        #print(f"Converted defense resource: {def_resource}")
                        self.container.update_sin_affinity(identity_id, 'defense', def_resource)
 
        except Exception as e:
            print(f"Error extracting defense type and resource: {e}")

    async def crawl_identity_details_for_all(self, page):
        """모든 인격의 세부 정보를 크롤링."""
        for identity in self.container.identities:
            await page.wait_for_load_state("networkidle")
            await page.goto(self.base_url + "/identity/"+identity.identity_id)
            """ 레벨과 동기화와 관련 없는 참관타 저항은 밑에서 변경하는 것과 연관성이 적기 때문에 위에서 처리한다.
            print(f"Identity_id: {identity.identity_id}")
            results = []
            resist_type_imgs = [f"{self.img_url}/ui/atk_type_HIT.png",
                                f"{self.img_url}/ui/atk_type_PENETRATE.png",
                                f"{self.img_url}/ui/atk_type_SLASH.png"]
            for resist_type_img in resist_type_imgs:
                result = await self.extract_resist_type(page, resist_type_img, 1)
                results.append(result)
            self.container.add_resist_info(identity.identity_id, results[0], results[1], results[2])
            """
            """
            # Stagger Threshold 값 추출
            stagger_threshold = await self.extract_stagger_threshold_values(page)
            self.container.add_stagger_threshold(identity.identity_id, stagger_threshold[0], stagger_threshold[1], stagger_threshold[2])
            """
            await self.initialize_uptie_and_level(page)
            """ 레벨에 따른 level, hp, defense 다르게 되어 있는 것을 출력 확인 객체로 들어가서 csv로 옮기는 과정까지 진행
            for i in range(1, identity.level_max+1):
                await self.move_slider_to_value(page,i)
                await self.extract_level_status(page, identity.identity_id, i)
            """
            """ 동기화에 따른 speed 다르게 되어 있는 것을 출력 확인 객체로 들어가서 csv로 옮기는 과정까지 진행
            for i in range(1, identity.uptie_max+1):
                await self.uptie_click(page,i)
                await self.extract_uptie_status(page, identity.identity_id, i)
            """
            """
                인격에 대한 skill 정보를 가져오는 메소드 위에서 출력하는 csv파일로 인격이 가지고 있는 스킬 정보를 내보낸다.
            await self.crawl_skills(page, identity.identity_id)
            """
            await self.crawl_influence_behavior(page, identity.identity_id)
            
    async def initialize_uptie_and_level(self, page):
        await self.uptie_click(page, 1)
        await self.move_slider_to_value(page, 1)
        self.uptie_level = 1
        #return 1
    
    async def uptie_click(self, page, phase):
        fab_button_selector = ".q-btn.q-btn--fab"
        await page.wait_for_selector(fab_button_selector)
        await page.click(fab_button_selector)
        #print(f"[{setup_uptie_button}] 상위 동기화 버튼 클릭 완료")
        setup_uptie_button = f'a:has(img[src="{self.img_url}/ui/uptie_{phase}.png"])'
        await page.wait_for_selector(setup_uptie_button)
        await page.click(setup_uptie_button)
        #print(f"{phase} 동기화 버튼 클릭 완료")

    async def move_slider_to_value(self, page, target_value):
        """
        슬라이더를 주어진 값으로 이동시킵니다.
        target_value는 슬라이더의 최소값부터 최대값 사이여야 합니다.
        """
        slider_selector = ".q-slider"
        # 슬라이더의 범위 확인
        min_value = int(await page.get_attribute(slider_selector, "aria-valuemin"))
        max_value = int(await page.get_attribute(slider_selector, "aria-valuemax"))
    
        if target_value < min_value or target_value > max_value:
            raise ValueError(f"Target value {target_value} is out of bounds ({min_value}, {max_value})")
    
        # 슬라이더 thumb 요소와 위치 가져오기
        slider_thumb = await page.query_selector(f"{slider_selector} .q-slider__thumb")
        thumb_box = await slider_thumb.bounding_box()
        
        # 슬라이더의 현재 위치 및 목표 위치 계산
        track = await page.query_selector(f"{slider_selector} .q-slider__track")
        track_box = await track.bounding_box()
        
        # 목표 x 위치 계산 (슬라이더 트랙 내에서 비율 기반)
        target_percentage = (target_value - min_value) / (max_value - min_value)
        target_x = track_box["x"] + target_percentage * track_box["width"]
        current_x = thumb_box["x"] + thumb_box["width"] / 2
        y = thumb_box["y"] + thumb_box["height"] / 2
    
        # 슬라이더를 드래그하여 목표 위치로 이동
        await page.mouse.move(current_x, y)
        await page.mouse.down()
        await page.mouse.move(target_x, y, steps=10)  # 부드럽게 이동
        await page.mouse.up()
    
        #print(f"슬라이더를 {target_value}로 이동 완료.")
    
        # 슬라이더 값 확인 및 강제 이벤트 트리거
        await page.evaluate(
            f"""
            (slider) => {{
                slider.dispatchEvent(new Event('input'));  // oninput 이벤트 트리거
                slider.dispatchEvent(new Event('change'));  // onchange 이벤트 트리거
            }}
            """,
            await page.query_selector(slider_selector)
        )
    
        # 이동 후 값 확인
        current_value = await page.get_attribute(slider_selector, "aria-valuenow")
        if current_value != str(target_value):
            print(f"경고: 슬라이더 값이 예상과 다릅니다. 현재 값: {current_value}, 예상 값: {target_value}")

    async def get_second_status_element(self, page):
        """
        두 번째 'Status' 요소에 접근하여 반환하는 메소드
        """
        status_elements = await page.query_selector_all("text=Status")
        if len(status_elements) > 1:
            second_status_element = status_elements[1]
            #print("두 번째 'Status' 요소에 접근합니다.")
            return second_status_element
        else:
            #print("두 번째 'Status' 요소가 존재하지 않습니다.")
            return None

    async def extract_level_status(self, page, identity_id, level):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 레벨에 따라 HP와 Defense가 변한다.
        hp_value = await self.extract_hp(second_status_element)
        defense_value = await self.extract_defense(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Level={level}, HP={hp_value}, Defense={defense_value}")
        self.container.add_level_stat(identity_id,level, hp_value, defense_value)
    
    async def extract_hp(self, status_element):
        # HP 추출
        hp_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].children[0].nextElementSibling.textContent.trim()"
        )
        return hp_value
    
    async def extract_defense(self, status_element):
        # Defense 추출
        defense_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return defense_value

    async def extract_uptie_status(self, page, identity_id, uptie):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 uptie에 따라 speed는 변한다.
        speed_value = await self.extract_speed(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Uptie={uptie}, Speed={speed_value}")
        self.container.add_uptie_stat(identity_id,uptie, speed_value)

    async def extract_speed(self, status_element):
        # Speed 추출
        speed_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return speed_value

    async def extract_resist_type(self, page, image_src, depth):
        # 이미지 요소 찾기
        #print(image_src)
        image = await page.query_selector(f'img[src="{image_src}"]')
        #print(image)
        if not image:
            return "Unknown"
    
        # 이미지의 부모 요소를 찾아서 올라감
        parent = image
        for _ in range(depth):
            parent = await parent.evaluate_handle('node => node.parentElement')
            if not parent:
                return "Unknown"
    
        # 부모의 형제 요소 찾기
        sibling = await parent.evaluate_handle('node => node.nextElementSibling')
        
        if sibling:
            value = await sibling.evaluate('node => node.textContent')
            if value:
                return value.strip()
        
        return "Unknown"

    async def extract_stagger_threshold_values(self, page):
        # 'div.row.center.flex-center.q-pt-xs' 요소들 찾기
        elements = await page.query_selector_all('div.row.center.flex-center.q-pt-xs')
        
        # 각 요소의 텍스트 내용 출력하여 'Stagger Threshold' 요소를 찾기
        for element in elements:
            text = await element.text_content()
            
            if text and "Stagger Threshold" in text:
                # 'Stagger Threshold'가 있는 요소를 찾았다면, 그 이후 형제 요소들에서 값을 추출
                parent = await element.evaluate_handle('node => node.parentElement')
                sibling_elements = await parent.query_selector_all('div.row.justify-evenly div')  # Threshold 값이 들어있는 div들
                
                stagger_values = []
                for sibling in sibling_elements:
                    text_content = await sibling.text_content()
                    match = re.search(r'(\d+%)', text_content)
                    if match:
                        stagger_values.append(match.group(1))  # 퍼센트 값만 리스트에 추가
                while len(stagger_values) < 3:
                    stagger_values.append(np.nan)
                #print(stagger_values)
                return stagger_values  # 추출된 값(또는 np.nan 포함)을 반환
        
        return [np.nan, np.nan, np.nan]  # 만약 'Stagger Threshold'를 찾지 못한 경우 기본값 반환

    async def crawl_skills(self, page, identity_id):
        await self.crawl_skill(page, identity_id, "skill 1")
        await self.crawl_skill(page, identity_id, "skill 2")
        await self.crawl_skill(page, identity_id, "skill 3")
        await self.crawl_skill(page, identity_id, "defense")


    async def crawl_skill(self, page, identity_id, skill_type):
        skills_elements = await page.query_selector_all('div.skilltag')
        for skill_element in skills_elements:
            skill_text = await skill_element.text_content()
            if skill_text.strip().lower() == skill_type:
                await self.process_skill(skill_element, page, identity_id, skill_type)
        
    async def process_skill(self, skill_element, page, identity_id, skill_type):
        # skill_type이 "skill 3"일 경우 uptie_value가 3 이상일 때만 정보를 저장
        for uptie_value in range(1, 5):  # Uptie levels 1 to 4
            # "skill 3"은 uptie_value가 3 이상일 때만 처리
            if skill_type == "skill 3" and uptie_value < 3:
                continue  # skip if uptie_value is less than 3 for skill 3
    
            # uptie_level에 맞게 클릭
            await self.uptie_click(page, uptie_value)
            if skill_type != "skill 3" and uptie_value > 1:
                # Uptie 정보 갱신
                await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
                continue
            elif skill_type == "skill 3" and uptie_value > 3:
                # Uptie 정보 갱신
                await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
                continue
            # 스킬의 세부 정보를 추출
            skill_details = await self.extract_skill_details(skill_element, skill_type)
            skill_name = skill_details["skill_name"]
    
            # 스킬 정보를 저장
            self.container.add_skill(
                identity_id, skill_type, 
                skill_name, skill_details["coin_count"],
                skill_details["init_level"], skill_details["init_remain"], 
                skill_details["atk_weight"]
            )
            print(f"Added skill: {skill_type}:{skill_name}")
    
            # Uptie 정보 갱신
            await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
    
    async def extract_skill_details(self, skill_element, skill_type):
        # 스킬 세부 정보를 추출하는 함수
        skill_name = await self.skill_name(skill_element)
        coin_count = await self.coin_count(skill_element, ".col-auto > .column > .row")
        init_level = await self.init_level(skill_element)
        init_remain = await self.init_remain(skill_element, skill_type, "div.row.justify-start > div.attr.flex.flex-center > div.q-px-sm")
        atk_weight = await self.atk_weight(skill_element, skill_type)
        return {
            "skill_name": skill_name,
            "coin_count": coin_count,
            "init_level": init_level,
            "init_remain": init_remain,
            "atk_weight": atk_weight
        }
    
    async def update_uptie_info(self, page, skill_element, identity_id, skill_name, skill_type, uptie_value):
        # Uptie 정보를 추출하고 업데이트하는 함수
        uptie_info = await self.extract_uptie_info(skill_element)
        effect_data = await self.skill_effects(skill_element)
        self.container.add_skill_uptie_info(
            identity_id, skill_name, 
            uptie_value, uptie_info["skill_power"], uptie_info["coin_power"], 
            effect_data.get("effectsByCondition")
        )
        print(
            f"Updated Uptie for {skill_name} (Uptie Level: {uptie_value}): "
            f"Skill Power: {uptie_info['skill_power']}, Coin Power: {uptie_info['coin_power']} "
            f"Skill Effect: {effect_data.get("effectsByCondition")}"
        )

    async def extract_uptie_info(self, skill_element):
        skill_data = await self.skill_and_coin_power(skill_element)
        return {
            "skill_power": skill_data.get("skillPower"),
            "coin_power": skill_data.get("coinPower")
        }
        
    async def crawl_influence_behavior(self, page, identity_id):
        influence_behaviors = await page.query_selector_all('div[align="right"]')
        # 텍스트를 기준으 추출하고 메소드로 파라미터 들어가 정보를 받는다.
        for influence_behavior in influence_behaviors:
            active_type = await influence_behavior.inner_text()
            active_type = active_type.strip()

            if active_type == "PASSIVE":
                """
                # PASSIVE에 대한 처리
                print(f"Processing PASSIVE logic for: {active_type}")
                # CALL method for PASSIVE
                """
                passive_data = await self.passive_effects(influence_behavior)
            elif active_type == "SUPPORT PASSIVE":
                """
                # SUPPORT PASSIVE에 대한 처리
                print(f"Processing SUPPORT PASSIVE logic for: {active_type}")
                # CALL method for SUPPORT PASSIVE
                """
                support_passive_data = await self.support_passive_effects(influence_behavior)
            elif active_type == "PANIC TYPE":
                """
                # PANIC TYPE에 대한 처리
                print(f"Processing PANIC TYPE logic for:{active_type}")
                # CALL method for PANIC TYPE
                """
                panic_type_data = await self.panic_effects(influence_behavior)

    async def skill_and_coin_power(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                // 조부모로 이동
                const grandparent = node.parentElement.parentElement;
                // 이전 형제 탐색
                const previousSibling = grandparent.previousElementSibling;
                if (!previousSibling) return {{ skillPower: null, coinPower: null }}; // 이전 형제가 없으면 기본값 반환
                // 이전 형제 내부의 .row.justify-around > div > div 탐색
                const targetDivs = previousSibling.querySelectorAll(".row.justify-around > div");
                if (targetDivs.length < 3) return {{ skillPower: null, coinPower: null }}; // 배열 길이 확인
                // 1st div: 스킬 위력
                const skillPower = targetDivs[0]?.querySelector('div')?.textContent?.trim() || null; 
                // 3rd div: 코인 파워 값 (3번째 div에서 텍스트 추출)
                const coinPower = targetDivs[2]?.textContent?.trim() || null;
                return {{ skillPower, coinPower }};
            }}
        """)


    async def coin_count(self, skill_element, hierarchy_selector):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector('{hierarchy_selector}');
                if (!targetElement) return 0;
                return targetElement.childElementCount;
            }}
        """)

    async def init_remain(self, skill_element, skill_type, hierarchy_selector):
        if skill_type == "defense":
            return -1
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector('{hierarchy_selector}');
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def skill_name(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector(".block");
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def init_level(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector("div.text-bold.text-h6.h6.q-pl-xs.q-pr-md");
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def atk_weight(self, skill_element, skill_type):
        if skill_type == "defense":
            return 0
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector("div.text-bold.text-h6.h6.q-pl-xs.q-pr-md");
                const doubleNextSibling = targetElement.nextElementSibling.nextElementSibling;
                if (!doubleNextSibling) return 0;
                return doubleNextSibling.textContent?.trim();
            }}
        """)

    async def skill_effects(self, skill_element):
        return await skill_element.evaluate("""
            node => {
                const parent = node.parentElement;
                const doubleNextSibling = parent.nextElementSibling?.nextElementSibling;
                if (!doubleNextSibling) return null;
    
                // doubleNextSibling에서 첫 번째 div.col 요소 찾기
                const textNode = doubleNextSibling.querySelector("div.col");
                if (!textNode) return null;
    
                const results = {
                    keywords: [], // 키워드 저장
                    conditions: [], // 조건 저장
                    effectsByCondition: {} // 조건별 효과
                };
    
                // 1. 조건 추출 (대괄호 텍스트)
                const conditionElements = textNode.querySelectorAll("span");
                conditionElements.forEach(span => {
                    let text = span.textContent?.trim();
                    if (text?.startsWith("[") && text?.endsWith("]")) {
                        // 부모의 전형제 노드 탐색
                        const sibling = span.parentElement.previousElementSibling;
                        if (sibling) {
                            const emblemImage = sibling.querySelector("img.emblem");
                            console.log("Sibling element found:", sibling); // 디버깅용
                            if (emblemImage) {
                                const src = emblemImage.getAttribute("src");
                                console.log("Image source:", src); // 디버깅용
                                const match = src.match(/ui_num_(\\d+)\\.png$/); // 정규식 수정
                                if (match) {
                                    const number = match[1]; // 숫자 추출
                                    text = `[Coin ${number} ${text.replace("[", "").replace("]", "")}]`;
                                    console.log("Updated condition:", text); // 디버깅용
                                }
                            }
                        }
                        results.conditions.push(text);
                        results.effectsByCondition[text] = []; // 조건별 효과를 초기화
                    }
                });
    
                // 2. 키워드 추출
                const keywordElements = textNode.querySelectorAll('a[href^="/en/keyword/"]');
                keywordElements.forEach(a => {
                    const span = a.querySelector('span.q-btn__content.text-center.col.items-center.q-anchor--skip.justify-center.row>span');
                    if (span) {
                        const keywordText = span.textContent.trim();
                        if (keywordText && !results.keywords.includes(keywordText)) {
                            results.keywords.push(keywordText); // 키워드 저장
                        }
                    }
                });
    
                // 3. 효과 추출
                const effectElements = textNode.querySelectorAll("span");
                let currentCondition = null;
                let currentEffect = []; // 초기화
                effectElements.forEach(span => {
                    let text = span.textContent?.trim();
                    if (!text) return;
    
                    // 조건 발견 시 currentCondition 변경
                    if (text.startsWith("[") && text.endsWith("]")) {
                        // 이전 조건의 효과 저장
                        if (currentCondition && currentEffect.length > 0) {
                            // 부모의 전형제 노드 탐색
                            const sibling = span.parentElement.previousElementSibling;
                            if (sibling) {
                                const emblemImage = sibling.querySelector("img.emblem");
                                if (emblemImage) {
                                    const src = emblemImage.getAttribute("src");
                                    const match = src.match(/ui_num_(\\d+)\\.png$/); // 정규식 수정
                                    if (match) {
                                        const number = match[1]; // 숫자 추출
                                        text = `[Coin ${number} ${text.replace("[", "").replace("]", "")}]`;
                                    }
                                }
                            }
                            results.effectsByCondition[currentCondition].push(currentEffect.join(' '));
                            currentEffect = []; // 초기화
                        }
                        currentCondition = text;
                        return;
                    }
    
                    // 현재 조건의 효과 처리
                    if (currentCondition) {
                        if (results.keywords.includes(text)) {
                            const keywordEffect = 'keyword:' + text;
                            if (!currentEffect.includes(keywordEffect)) {
                                currentEffect.push(keywordEffect);
                            }
                        } else {
                            currentEffect.push(text);
                        }
                    }
                });
    
                // 마지막 효과 저장
                if (currentCondition && currentEffect.length > 0) {
                    results.effectsByCondition[currentCondition].push(currentEffect.join(' '));
                }
    
                return results;
            }
        """)

    async def passive_effects(self, passive_element):
        passive_name = await self.behavior_name(passive_element)
        passive_conditions = await self.behavior_condition(passive_element)
        passive_description = await self.behavior_description(passive_element)
        print(passive_name)
        print(passive_conditions)
        print(passive_description)
    async def support_passive_effects(self, support_passive_element):
        support_passive_name = await self.behavior_name(support_passive_element)
        support_passive_conditions = await self.behavior_condition(support_passive_element)
        support_passive_description = await self.behavior_description(support_passive_element)
        print(support_passive_name)
        print(support_passive_conditions)
        print(support_passive_description)
    async def panic_effects(self, panic_element):
        panic_type = await self.behavior_name(panic_element)
        panic_conditions = await self.behavior_condition(panic_element)
        panic_description = await self.panic_description(panic_element)
        print(panic_type)
        print(panic_conditions)
        print(panic_description)

    async def navigate_element_by_steps(self, element, parent_steps=0, prev_sibling_steps=0, next_sibling_steps=0):
        """
        주어진 `element`에서 부모, 이전 형제, 또는 다음 형제로 지정된 횟수만큼 이동하여 ElementHandle 반환.
        
        Args:
            element (ElementHandle): 탐색 기준이 되는 요소.
            parent_steps (int): 부모로 이동할 횟수 (0이면 이동하지 않음).
            prev_sibling_steps (int): 이전 형제로 이동할 횟수 (0이면 이동하지 않음).
            next_sibling_steps (int): 다음 형제로 이동할 횟수 (0이면 이동하지 않음).
        
        Returns:
            ElementHandle or None: 탐색된 요소를 반환하거나 없으면 None.
        """
        try:
            current_element = element
    
            # 부모로 이동
            for _ in range(parent_steps):
                current_element = await current_element.evaluate_handle('el => el.parentElement')
                if not current_element:
                    print("Parent element not found during navigation.")
                    return None
    
            # 이전 형제로 이동
            for _ in range(prev_sibling_steps):
                current_element = await current_element.evaluate_handle('el => el.previousElementSibling')
                if not current_element:
                    print("Previous sibling element not found during navigation.")
                    return None
    
            # 다음 형제로 이동
            for _ in range(next_sibling_steps):
                current_element = await current_element.evaluate_handle('el => el.nextElementSibling')
                if not current_element:
                    print("Next sibling element not found during navigation.")
                    return None
    
            return current_element
    
        except Exception as e:
            print(f"Error navigating elements: {e}")
            return None

    async def get_last_sibling(self, element):
        try:
            # 같은 부 아 마지막 형제를 가져오는 JavaScript 실행
            last_sibling = await element.evaluate_handle('el => el.parentElement.lastElementChild')
            return last_sibling
        except Exception as e:
            print(f"Error getting last sibling: {e}")
            return None

    async def behavior_name(self, element):
        parent_prev_sibling = await self.navigate_element_by_steps(element, parent_steps=1, prev_sibling_steps=1, next_sibling_steps=0)
        passive_name_element = await parent_prev_sibling.query_selector('span.block')
        if passive_name_element:
            passive_name = await passive_name_element.inner_text()
            return passive_name
        else:
            #print("Passive name element not found")
            return None

    async def behavior_condition(self, element):
        try:
            conditions = []
            parent_prev_sibling = await self.navigate_element_by_steps(element, parent_steps=1, prev_sibling_steps=1, next_sibling_steps=0)
            img_elements = await parent_prev_sibling.query_selector_all('img')
            if not img_elements:
                #print("No <img> elements found.")
                return conditions
    
            for img_element in img_elements:
                # 각 이미지의 src 속성 가져오기
                img_src = await img_element.get_attribute('src')
                #print(img_src)
    
                sin_name = None
                for color_key in resource_mapping.keys():
                    if f"{self.img_url}/ui/attr_{color_key}.png" in img_src:
                        sin_name = resource_mapping[color_key]
                        break
    
                if not sin_name:
                    #print(f"No matching Sin Color found for {img_src}.")
                    continue  # No Sin Color found, skip this img_element
    
                # 해당 이미지와 관련된 조건 텍스트 추출
                # 두 번째 부모 요소에서 시작하여 다음 형제를 찾아 조건 텍스트를 추출
                condition_element = await self.navigate_element_by_steps(img_element, parent_steps=3, prev_sibling_steps=0, next_sibling_steps=0)
                if condition_element:
                    # condition_element가 None이 아닌 경우에만 진행
                    condition_text_element = await condition_element.query_selector('.q-pl-xs')
                    if condition_text_element:
                        condition_text = await condition_text_element.inner_text()
                        condition_text = condition_text.strip()
                    else:
                        print(f"No condition text found for {img_src}.")
                        condition_text = ""  # condition_text_element가 None이면 빈 문자열로 처리
                else:
                    print(f"No condition element found for {img_src}.")
                    condition_text = ""  # condition_element가 None이면 빈 문자열로 처리
    
                # 결과를 배열에 추가
                conditions.append(f"{sin_name} {condition_text}")
            return conditions
        except Exception as e:
            print(f"Error extracting conditions: {e}")
            return 
            
    async def behavior_description(self, element):
        try:
            description_texts = []
            keywords = []
    
            # Navigate to the behavior description parent
            parent_prev_sibling = await self.navigate_element_by_steps(element, parent_steps=1, prev_sibling_steps=1, next_sibling_steps=0)
            behavior_title = await parent_prev_sibling.query_selector('.q-pb-sm')
            behavior_description_element = await self.get_last_sibling(behavior_title)
    
            # Fetch keyword elements
            keyword_elements = await behavior_description_element.query_selector_all('a[href^="/en/keyword/"]')
            for keyword_element in keyword_elements:
                keyword_span = await keyword_element.query_selector(
                    'span.q-btn__content.text-center.col.items-center.q-anchor--skip.justify-center.row > span'
                )
                if keyword_span:
                    keyword_text = await keyword_span.inner_text()
                    stripped_text = keyword_text.strip()
                    if stripped_text and stripped_text not in keywords:
                        keywords.append(stripped_text)
    
            # Fetch description elements
            descriptionElements = await behavior_description_element.query_selector_all("span")
            seen_texts = set()  # Use a set to track already processed texts
    
            for descriptionElement in descriptionElements:
                description_text = await descriptionElement.inner_text()
                stripped_text = description_text.strip()
                if stripped_text and stripped_text not in seen_texts:
                    seen_texts.add(stripped_text)  # Mark text as seen
                    if stripped_text in keywords:
                        description_texts.append(f"keyword:{stripped_text}")
                    else:
                        description_texts.append(stripped_text)
    
            return description_texts
        except Exception as e:
            print(f"Error extracting description: {e}")
            return

    async def panic_description(self, element):
        try:
            description_texts = []
            keywords = []
    
            # Navigate to the behavior description parent
            parent_prev_sibling = await self.navigate_element_by_steps(element, parent_steps=1, prev_sibling_steps=1, next_sibling_steps=0)
            behavior_title = await parent_prev_sibling.query_selector('.q-pb-sm')
            behavior_description_element = await self.get_last_sibling(behavior_title)
    
            # Fetch keyword elements
            keyword_elements = await behavior_description_element.query_selector_all('a[href^="/en/keyword/"]')
            for keyword_element in keyword_elements:
                keyword_span = await keyword_element.query_selector(
                    'span.q-btn__content.text-center.col.items-center.q-anchor--skip.justify-center.row > span'
                )
                if keyword_span:
                    keyword_text = await keyword_span.inner_text()
                    stripped_text = keyword_text.strip()
                    if stripped_text and stripped_text not in keywords:
                        keywords.append(stripped_text)
    
            # Fetch description elements
            descriptionElements = await behavior_description_element.query_selector_all("div")
            seen_texts = set()  # Use a set to track already processed texts
    
            for descriptionElement in descriptionElements:
                description_text = await descriptionElement.inner_text()
                stripped_text = description_text.strip()
                if stripped_text and stripped_text not in seen_texts:
                    seen_texts.add(stripped_text)  # Mark text as seen
                    if stripped_text in keywords:
                        description_texts.append(f"keyword:{stripped_text}")
                    else:
                        description_texts.append(stripped_text)
    
            return description_texts
        except Exception as e:
            print(f"Error extracting description: {e}")
            return

class Identity:
    """인격 정보를 저장하는 클래스"""
    def __init__(self, identity_id, name=None, rarity=None,\
                 skill_1_type=None, skill_1_sin_affinity=None,\
                 skill_2_type=None, skill_2_sin_affinity=None,\
                 skill_3_type=None, skill_3_sin_affinity=None,\
                 defense_type=None, defense_sin_affinity=None):
        self.identity_id = identity_id
        self.name = name
        self.rarity = rarity
        # 선언만 하는 부분 
        self.skill_1_type = skill_1_type 
        self.skill_1_sin_affinity = skill_1_sin_affinity 
        self.skill_2_type = skill_2_type 
        self.skill_2_sin_affinity = skill_2_sin_affinity 
        self.skill_3_type = skill_3_type 
        self.skill_3_sin_affinity = skill_3_sin_affinity 
        self.defense_type = defense_type 
        self.defense_sin_affinity = defense_sin_affinity
        self.level_max = 50
        self.uptie_max = 4
        self.level_stats = []  # LevelStat 객체 리스트
        self.uptie_stats = [] # UptieStat 객체 리스트
        self.resist_info = None
        self.stagger_threshold = None
        self.skillset = SkillSet()  # Skill정보를 담고 있는 skillset 객체 
        self.passive_infos = []  # 패시브 객체를 담고 있는 배열
        self.support_passive_infos = [] # 서포트 패시 객체를 담고 있는 배열
        self.panic_type = [] # 혼란상태 발생시 나타나는 효과를 담고 있는 객체 배열1

    def update_name(self, name):
        self.name = name

    def update_rarity(self, rarity):
        self.rarity = rarity

    def add_level_stat(self, level, hp, defense):
        self.level_stats.append(LevelStat(level, hp, defense))

    def add_uptie_stat(self, uptie, speed):
        self.uptie_stats.append(UptieStat(uptie, speed))

    def add_resist_info(self, resist_hit, resist_penetrate, resist_slash):
        self.resist_info = ResistInfo(resist_hit, resist_penetrate, resist_slash)

    def add_stagger_threshold(self, first_stagger, second_stagger, third_stagger):
        self.stagger_threshold = StaggerThreshold(first_stagger, second_stagger, third_stagger)

    def add_skill(self, skill_code, skill_name, coin_count, init_level, init_remain, atk_weight):
        self.skillset.add_skill(skill_code, Skill(skill_code, skill_name, coin_count, init_level, init_remain, atk_weight))
        
    def add_skill_uptie_info(self, skill_name, uptie_level, skill_power, coin_power, effect_by_condition):
        self.skillset.add_skill_uptie_info(skill_name, uptie_level, skill_power, coin_power, effect_by_condition)
        
    def update_details(self, page):
        # 세부 정보 업데이트 로직 (가정)
        pass
    #def init_uptie(self, page):
    #    init_uptie = 

# LevelStat 객체 정의
class LevelStat:
    def __init__(self, level, hp=None, defense=None):
        self.level = level
        self.hp = hp
        self.defense = defense
    def update_level(self, level):
        self.level = level
    def update_hp(self, hp):
        self.hp = hp
    def update_defense(self, defense):
        self.defense = defense

# UptieStat 객체 정의
class UptieStat:
    def __init__(self, uptie, speed=None):
        self.uptie = uptie
        self.speed = speed
    def update_speed(self, speed):
        self.speed = speed

# ResistInfo 객체 정의
class ResistInfo:
    def __init__(self, resist_hit=None, resist_penetrate=None, resist_slash=None):
        self.resist_hit = resist_hit
        self.resist_penetrate = resist_penetrate
        self.resist_slash = resist_slash

# ResistInfo 객체 정의
class StaggerThreshold:
    def __init__(self, first_stagger=None, second_stagger=None, third_stagger=None):
        self.first_stagger = first_stagger
        self.second_stagger = second_stagger
        self.third_stagger = third_stagger

class SkillSet:
    def __init__(self):

        self.skill_1_type = ""
        self.skill_2_type = ""
        self.skill_3_type = ""
        self.defense_skill_type = ""

        self.skill_1_sin_affinity = ""
        self.skill_2_sin_affinity = ""
        self.skill_3_sin_affinity = ""
        self.defense_skill_sin_affinity = ""

        self.skill_1_details = []
        self.skill_2_details = []
        self.skill_3_details = []
        self.defense_skill_details = []

    def set_skill_type(self, skill_code, skill_type, sin_affinity):
        if skill_code == "skill 1":
            self.skill_1_type = skill_type
            self.skill_1_sin_affinity = sin_affinity
        elif skill_code == "skill 2":
            self.skill_2_type = skill_type
            self.skill_2_sin_affinity = sin_affinity
        elif skill_code == "skill 3":
            self.skill_3_type = skill_type
            self.skill_3_sin_affinity = sin_affinity
        elif skill_code == "defense":
            self.defense_skill_type = skill_type
            self.defense_skill_sin_affinity = sin_affinity
    
    def add_skill(self, skill_code, skill):
        if skill_code == "skill 1":
            self.skill_1_details.append(skill)
            skill.skill_type = self.skill_1_type
            skill.sin_affinity = self.skill_1_sin_affinity
        elif skill_code == "skill 2":
            self.skill_2_details.append(skill)
            skill.skill_type = self.skill_2_type
            skill.sin_affinity = self.skill_2_sin_affinity
        elif skill_code == "skill 3":
            self.skill_3_details.append(skill)
            skill.skill_type = self.skill_3_type
            skill.sin_affinity = self.skill_3_sin_affinity
        elif skill_code == "defense":
            self.defense_skill_details.append(skill)
            skill.skill_type = self.defense_skill_type 
            skill.sin_affinity = self.defense_skill_sin_affinity

    def get_skill_by_name(self, skill_name):
        for skill_list in [self.skill_1_details, self.skill_2_details, self.skill_3_details, self.defense_skill_details]:
            for skill in skill_list:
                result = skill.is_skill_by_name(skill_name)
                if result:
                    return result
        return None

    def add_skill_uptie_info(self, skill_name, uptie_level, skill_power, coin_power, effect_by_condition):
        skill = self.get_skill_by_name(skill_name)
        skill.add_uptie_info(uptie_level, skill_power, coin_power, effect_by_condition)
        

class Skill:
    def __init__(self, skill_code, skill_name, coin_count, init_level, init_remain, atk_weight):
        self.skill_code = skill_code # "Skill 1"/"Skill 2"/"Skill 3"/"Defense" '''o'''
        self.skill_type = None
        self.sin_affinity = None
        self.skill_name = skill_name # 스킬 이름 '''o'''
        self.coin_count = coin_count # 동전 개수 '''o'''
        self.init_level = init_level # 스킬이 초기 배정받은 레벨, 인격 레벨에 - 1 한 값을 더해주면 스킬의 현 level을 구할 수 있다. '''o'''
        self.init_remain = init_remain #'''o'''
        self.atk_weight = atk_weight #공격 가중치 값이다. 방어스킬에는 없기 때문에 None으로 설정했다. '''o'''
        self.remain_count = init_remain # 실제 스킬이 작동하며 횟수가 차감되 나중에 모두 차감되면 다음 턴에 다시 초기횟수 리필된다.
        self.uptie_info = []  

    def is_skill_by_name(self, skill_name):
        if self.skill_name == skill_name:
            return self
        return None
        
    def add_uptie_info(self, uptie_level, skill_power, coin_power, effects_by_condition):
        self.uptie_info.append(UptieInfo(uptie_level, skill_power, coin_power, effects_by_condition))

class UptieInfo:
    def __init__(self, uptie_level, skill_power, coin_power, effects_by_condition):
        self.uptie_level = uptie_level   #'''o'''
        self.skill_power = skill_power   #'''o'''
        self.coin_power = coin_power     #'''o'''
        self.skill_effect = effects_by_condition

# 추상화된 EffectType 클래스
class EffectType:
    def __init__(self, name, description):
        self.name = name  # Effect 이름
        self.description = description  # Effect 설명

# Passive 클래스
class Passive(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)  # condition이 있는지 여부 확인

# SupportPassive 클래스
class SupportPassive(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)

# PanicType 클래스
class PanicType(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)

# Keyword 클래스
class Keyword:
    def __init__(self, keyword, explain=None, dispellable=None):
        self.keyword = keyword
        self.explain = explain
        self.dispellable = dispellable

      
class DataContainer:
    """크롤링 데이터를 관리하는 클래스"""
    def __init__(self):
        self.identities = []

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.level_max

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.uptie_max

    def update_identity_name(self, identity_id, identity_name):
        """특정 ID의 이름을 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_name(identity_name)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_identity_rarity(self, identity_id, rarity):
        """특정 ID의 희귀도를 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_rarity(rarity)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_skill(self, identity_id, skill_number, new_type):
        """
        특정 identity의 skill_type을 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_type: 새 skill_type
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_type = new_type
                elif skill_number == 2:
                    identity.skill_2_type = new_type
                elif skill_number == 3:
                    identity.skill_3_type = new_type
                elif skill_number == "defense":
                    identity.defense_type = new_type
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def update_sin_affinity(self, identity_id, skill_number, new_affinity):
        """
        특정 identity의 sin_affinity를 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_affinity: 새 sin_affinity
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_sin_affinity = new_affinity
                elif skill_number == 2:
                    identity.skill_2_sin_affinity = new_affinity
                elif skill_number == 3:
                    identity.skill_3_sin_affinity = new_affinity
                elif skill_number == "defense":
                    identity.defense_sin_affinity = new_affinity
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def add_identity(self, identity):
        """새로운 인격 데이터를 추가."""
        self.identities.append(identity)

    def add_level_stat(self, identity_id, level, hp, defense):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_level_stat(level, hp, defense)

    def add_uptie_stat(self, identity_id, uptie, speed):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_uptie_stat(uptie, speed)

    def add_resist_info(self, identity_id, resist_hit, resist_penetrate, resist_slash):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_resist_info(resist_hit, resist_penetrate, resist_slash)

    def add_stagger_threshold(self, identity_id, first_stagger, second_stagger, third_stagger):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_stagger_threshold(first_stagger, second_stagger, third_stagger)

    
    def add_skill(self, identity_id, skill_code, skill_name, coin_count, init_level, init_remain, atk_weight):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_code == "skill 1":
                    identity.skillset.set_skill_type(skill_code, identity.skill_1_type, identity.skill_1_sin_affinity)
                elif skill_code == "skill 2":
                    identity.skillset.set_skill_type(skill_code, identity.skill_2_type, identity.skill_2_sin_affinity)
                elif skill_code == "skill 3":
                    identity.skillset.set_skill_type(skill_code, identity.skill_3_type, identity.skill_3_sin_affinity)
                elif skill_code == "defense":
                    identity.skillset.set_skill_type(skill_code, identity.defense_type, identity.defense_sin_affinity)
                identity.add_skill(skill_code, skill_name, coin_count, init_level, init_remain, atk_weight)

    def add_skill_uptie_info(self, identity_id, skill_name, uptie_level, skill_power, coin_power, effects_by_condition):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_skill_uptie_info(skill_name, uptie_level, skill_power, coin_power, effects_by_condition)
        
    def display_identities(self):
        """Identity, ID, Name, Rarity 정보를 출력"""
        if not self.identities:
            print("No identities found.")
        else:
            # 데이터를 리스트로 변환
            table = []
            for idx, identity in enumerate(self.identities, 1):
                table.append([
                    idx, identity.identity_id, identity.name, identity.rarity,
                    identity.skill_1_type, identity.skill_1_sin_affinity,
                    identity.skill_2_type, identity.skill_2_sin_affinity,
                    identity.skill_3_type, identity.skill_3_sin_affinity,
                    identity.defense_type, identity.defense_sin_affinity
                ])
            
            # 테이블 헤더 정의
            headers = [
                "Index", "ID", "Name", "Rarity",
                "Skill_1 Type", "Skill_1 Sin", "Skill_2 Type", "Skill_2 Sin",
                "Skill_3 Type", "Skill_3 Sin", "Defense Type", "Defense Sin"
            ]
            
            # 테이블 출력
            print(tabulate(table, headers=headers, tablefmt="fancy_grid"))

    def export_to_csv(self, filename, fieldnames, data):
        """
        공통적으로 사용 가능한 CSV 내보내기 메서드
        :param filename: 내보낼 파일 이름 (예: 'identity.csv', 'skill.csv')
        :param fieldnames: CSV의 헤더 리스트
        :param data: CSV로 내보낼 데이터 (리스트 형태)
        """
        try:
            # 현재작업 디렉토 기준으 'csv'폴더 없으면 생성
            if not os.path.exists('./csv'):
                os.makedirs('./csv')
            # 파일 경로 설정 (상대경로 'csv' 폴더 안에 저장)
            file_path = os.path.join('./csv', filename)
            with open(file_path, mode='w', newline='', encoding='utf-8') as file:
                writer = csv.DictWriter(file, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(data)
            print(f"Data successfully exported to {filename}")
        except Exception as e:
            print(f"Error exporting to CSV: {e}")

    def get_identity_data(self):
        """
        Identity 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "ID": identity.identity_id,
                "Name": identity.name,
                "Rarity": identity.rarity,
                "Skill_1_Type": identity.skill_1_type,
                "Skill_1_Sin_Affinity": identity.skill_1_sin_affinity,
                "Skill_2_Type": identity.skill_2_type,
                "Skill_2_Sin_Affinity": identity.skill_2_sin_affinity,
                "Skill_3_Type": identity.skill_3_type,
                "Skill_3_Sin_Affinity": identity.skill_3_sin_affinity,
                "Defense_Type": identity.defense_type,
                "Defense_Sin_Affinity": identity.defense_sin_affinity,
            }
            for identity in self.identities
        ]
        
    def export_identities_data(self, filename="identities.csv"):
        """Identity 데이터를 CSV로 내보내는 메서드"""
        identity_fieldnames = [
            "ID", "Name", "Rarity",
            "Skill_1_Type", "Skill_1_Sin_Affinity",
            "Skill_2_Type", "Skill_2_Sin_Affinity",
            "Skill_3_Type", "Skill_3_Sin_Affinity",
            "Defense_Type", "Defense_Sin_Affinity"
        ]
        identity_data = self.get_identity_data()  # Identity 데이터 가져오기
        self.export_to_csv(filename, identity_fieldnames, identity_data)  # CSV로 내보내기

    def load_identities_by_id(self, filename, identity_ids):
        """지정된 identity_ids 목록에 해당하는 Identity 데이터를 CSV에서 읽어옴."""
        identity_ids_set = set(identity_ids) # 중복 제거를 위 Set 사용
        with open(filename, mode='r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                if row["ID"] in identity_ids_set:
                    identity = Identity(
                        identity_id=row["ID"],
                        name=row["Name"],
                        rarity=row["Rarity"],
                        skill_1_type=row["Skill_1_Type"],
                        skill_1_sin_affinity=row["Skill_1_Sin_Affinity"],
                        skill_2_type=row["Skill_2_Type"],
                        skill_2_sin_affinity=row["Skill_2_Sin_Affinity"],
                        skill_3_type=row["Skill_3_Type"],
                        skill_3_sin_affinity=row["Skill_3_Sin_Affinity"],
                        defense_type=row["Defense_Type"],
                        defense_sin_affinity=row["Defense_Sin_Affinity"]
                    )
                    self.identities.append(identity)

    def get_level_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Level": level_stat.level,
                "HP": level_stat.hp,
                "Defense": level_stat.defense,
            }
            for identity in self.identities
            for level_stat in identity.level_stats
        ]

    def export_level_stats_data(self, filename="levelstats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        level_stats_fieldnames = ["Identity_ID", "Level", "HP", "Defense"]
        level_stats_data = self.get_level_stats_data()  # level_stats 데이터 가져오기
        self.export_to_csv(filename, level_stats_fieldnames, level_stats_data)  # CSV로 내보내기

    def get_uptie_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Uptie": uptie_stat.uptie,
                "Speed": uptie_stat.speed,
            }
            for identity in self.identities
            for uptie_stat in identity.uptie_stats
        ]

    def export_uptie_stats_data(self, filename="Uptiestats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        uptie_stats_fieldnames = ["Identity_ID", "Uptie", "Speed"]
        uptie_stats_data = self.get_uptie_stats_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, uptie_stats_fieldnames, uptie_stats_data)  # CSV로 내보내기

    def get_resist_info_data(self):
        """
        모든 Identity의 resist_info 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Resist_Hit": identity.resist_info.resist_hit,
                "Resist_Penetrate": identity.resist_info.resist_penetrate,
                "Resist_Slash": identity.resist_info.resist_slash,
            }
            for identity in self.identities
        ]

    def export_resist_info_data(self, filename="ResistInfos.csv"):
        """ResistInfo 데이터를 CSV로 내보내는 메서드"""
        resist_infos_fieldnames = ["Identity_ID", "Resist_Hit", "Resist_Penetrate", "Resist_Slash"]
        resist_infos_data = self.get_resist_info_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, resist_infos_fieldnames, resist_infos_data)  # CSV로 내보내기

    def export_uptie_stats_data(self, filename="Uptiestats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        uptie_stats_fieldnames = ["Identity_ID", "Uptie", "Speed"]
        uptie_stats_data = self.get_uptie_stats_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, uptie_stats_fieldnames, uptie_stats_data)  # CSV로 내보내기

    def get_stagger_threshold_data(self):
        """
        모든 Identity의 stagger_threshold 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "First_Stagger": identity.stagger_threshold.first_stagger,
                "Second_Stagger": identity.stagger_threshold.second_stagger,
                "Third_Stagger": identity.stagger_threshold.third_stagger,
            }
            for identity in self.identities
        ]

    def export_stagger_threshold_data(self, filename="StaggerThresholds.csv"):
        """StaggerThresholds 데이터를 CSV로 내보내는 메서드"""
        stagger_thresholds_fieldnames = ["Identity_ID", "First_Stagger", "Second_Stagger", "Third_Stagger"]
        stagger_thresholds_data = self.get_stagger_threshold_data()  # stagger_thresholds 데이터 가져오기
        self.export_to_csv(filename, stagger_thresholds_fieldnames, stagger_thresholds_data)  # CSV로 내보내기

    def get_skillset_data(self):
        """모든 Identity의 skillset 데이터를 export_to_csv을 포맷으로 변환"""
        return [
            {
                "Identity_ID": identity.identity_id,
                "Skill_Code": skill.skill_code,
                "Skill_Type": skill.skill_type,
                "Sin_Affinity": skill.sin_affinity,
                "Skill_Name": skill.skill_name,
                "Coin_Count": skill.coin_count,
                "Init_Level": skill.init_level,
                "Init_Remain": skill.init_remain,
                "Atk_Weight": skill.atk_weight,
                "Uptie_Level": uptie_info.uptie_level,
                "Skill_Power": uptie_info.skill_power,
                "Coin_Power": uptie_info.coin_power,
                "Effect_Data": json.dumps(uptie_info.skill_effect)
            }
            for identity in self.identities
            for skill_list in [identity.skillset.skill_1_details, identity.skillset.skill_2_details, identity.skillset.skill_3_details, identity.skillset.defense_skill_details]
            for skill in skill_list
            for uptie_info in skill.uptie_info
        ]

    def export_identity_skillset_data(self, filename="IdentitiesSkills.csv"):
        """IdentitiesSkills 데이터를 CSV로 내보내는 메서드"""
        identities_skills_fieldnames = ["Identity_ID", "Skill_Code", "Skill_Type", "Sin_Affinity", "Skill_Name", "Coin_Count", "Init_Level", "Init_Remain", "Atk_Weight", "Uptie_Level", "Skill_Power", "Coin_Power", "Effect_Data"]
        identities_skills_data = self.get_skillset_data()
        self.export_to_csv(filename, identities_skills_fieldnames, identities_skills_data)
            


async def main():
    data_container = DataContainer()
    scraper = Scraper(container=data_container, base_url="https://limbus.kusoge.xyz", img_url="https://img.kusoge.xyz/limbus/img")
    await scraper.scrape()

await main() 

Bloodfeast
[]
['If', 'this', 'unit', 'is', 'on', 'field', ',', 'or', 'one', 'of', 'the', 'units', 'that', 'can', 'appear', 'Stage', 'increase', 'keyword:Bloodfeast', 'value', 'by', 'amount', 'keyword:Bleed', 'damage', 'collectively', 'received', 'every', '.', 'When', 'enters', 'sleeping', 'blood', 'drenching', 'battlefield', 'will', 'fill', 'surface']
Hardblood Thorn
['Lust x3 Owned', 'Envy x2 Owned']
['When', 'an', 'other', 'ally', 'takes', 'keyword:Bleed', 'damage', 'or', 'consumes', 'keyword:Bloodfeast', ',', 'gain', '1', 'blooming[thornyfall_panic', 'RodionFirst', '] (', '3', 'times', 'per', 'turn', ')', 'Heal', 'HP', 'on', 'self', 'by', '20', '%', 'of', 'the', 'dealt', 'with', 'base', 'Skills', '(', 'max', '10', 'Skill']
"Blossom with Blood..."
['Lust x3 Owned', 'Envy x3 Owned']
['To', '1', 'ally', 'with', 'the', 'highest', 'keyword:Bleed', 'Potency', 'at', 'Turn', 'End', ':', 'reduce', "'", 's', 'by', '6', 'max', ',', 'and', 'apply', '(', 'reduced', '/', '2', ')', 'blooming[thorn

In [68]:
import os
import csv
import re
import json
import numpy as np
import pandas as pd
from playwright.async_api import async_playwright
from tabulate import tabulate

# Sin Color 이름 변환을 위한 매핑
resource_mapping = {
    "AZURE": "Gloom",
    "VIOLET": "Envy",
    "AMBER": "Sloth",
    "SHAMROCK": "Gluttony",
    "CRIMSON": "Wrath",
    "SCARLET": "Lust",
    "INDIGO": "Pride"
}

def convert_resource(resource_name):
    """Sin Color 이름을 변환하는 함수. 매핑되지 않은 경우 원래 이름을 반환."""
    return resource_mapping.get(resource_name, resource_name)

def find_sin_for_color(url):
    # URL에 color_name을 추출
    color_name = url.split('/')[-1].split('.')[0].replace("attr_", "")
    # 해당 색상이 resource_mapping에 있는 확인
    for color, sin in resource_mapping.items():
        if color_name in color:
            return sin
    return None


class Scraper:
    """웹 페이지 크롤링을 위한 클래스"""
    def __init__(self, container, base_url, img_url):
        self.container = container
        self.base_url = base_url
        self.img_url = img_url
        self.uptie_level = 1

    async def handle_request(self, route, request):
        """이미지와 폰트 리소스 로드를 차단."""
        if request.resource_type =="image":
            #print(f"{request.url}")
            img_url = request.url
            if  re.match(f"{self.img_url}/ui/attr.*\\.png", img_url):
                await route.continue_()
            else:
                await route.fulfill(
                    status=200,
                    content_type="image/png",
                    body=b""
                )
        elif request.resource_type == "font":
            await route.abort()
        else:
            await route.continue_()
            
    async def scrape(self):
        """크롤링 메인 함수"""
        async with async_playwright() as p:
            browser = await p.chromium.launch(headless=True)
            page = await browser.new_page()
            #await page.evaluate('window.scrollTo(0, document.body.scrollHeight)')
            await page.route("**/*", self.handle_request)
            # Step 1: /identity 페이지에서 기본 정보 크롤링
            #await self.crawl_identity_page(page)

            # Step 2: /identity/{identity_id} 상세 페이지 크롤링
            #identity_ids_to_load = ["10104", "10310", "10710","10808", "10911", "11005"]
            #identity_ids_to_load = ["10104", "10911"]#,"10310", "10710","10808", "11005", "11210"]
            #identity_ids_to_load = ["10911","10310"]#, "10710","10808", "11005", "11210"]
            identity_ids_to_load = ["10911"]#, "10611"]#,"10711"]#, "10710","10808", "11005", "11210"]
            self.container.load_identities_by_id('./csv/identities.csv', identity_ids_to_load)
            await self.crawl_identity_details_for_all(page)
            #self.container.display_identities()
            #self.container.export_identities_data("identities.csv")
            #self.container.export_level_stats_data("Levelstats.csv")
            #self.container.export_uptie_stats_data("Uptiestats.csv")
            #self.container.export_resist_info_data("ResistInfos.csv")
            #self.container.export_stagger_threshold_data("StaggerThresholds.csv")
            #self.container.export_identity_skillset_data("IdentitiesSkills.csv")
            page.set_default_timeout(10000)
            #await page.wait_for_timeout(100000)
            await browser.close()

    async def get_identity_id_from_row(self, row):
        """테이블 행(row)에서 인격 ID를 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_url = await identity_cell.get_attribute('href')
            identity_id = identity_url.split('/identity/')[-1]
            return identity_id
        return "Unknown"

    async def get_identity_name_from_row(self, row):
        """테이블 행(row)에서 인격 이름을 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_name = (await identity_cell.inner_text()).replace("\n", " ").strip()
            return identity_name
        return "Unknown"
        
    async def crawl_identity_page(self, page):
        """/identity 페이지로 이동하여 인격 기본 정보를 추출."""
        await page.goto(self.base_url + "/identity")
        await self.extract_identity_page(page)

    async def extract_identity_page(self, page):
        """/identity 페이지의 모든 테이블 행에서 인격 정보를 추출."""
        identity_rows = await page.query_selector_all('tr')
        if len(identity_rows) > 1:
            for idx, identity_row in enumerate(identity_rows):
                if idx == 0:
                    continue # 0번째요소이름을 보여주는 열은 건너뜀
                await self.scrape_identity_name_and_identity_id(identity_row)
                await self.extract_rarity(identity_row)
                for i in range(1, 4):  # 스킬1~3 정보 추출
                    await self.extract_atk_and_resource(identity_row, i)
                await self.extract_defense_and_resource(identity_row)

    async def scrape_identity_name_and_identity_id(self, row):
        """인격 번호와 이름을 추출하여 DataContainer에 저장."""
        try:
            identity_id = await self.get_identity_id_from_row(row)
            identity_name = await self.get_identity_name_from_row(row)
            identity = Identity(identity_id)
            self.container.add_identity(identity)
            self.container.update_identity_name(identity_id, identity_name)
        except Exception as e:
            print(f"Error extracting identity name and identity id: {e}")

    async def get_element_attribute(self, element, selector, attribute):
        """요소에서 지정된 속성(attribute)을 추출."""
        try:
            target_element = await element.query_selector(selector)
            if target_element:
                return await target_element.get_attribute(attribute)
        except Exception as e:
            print(f"Error getting element attribute: {e}")
        return None
    
    async def extract_rarity(self, row):
        """테이블 행(row)에서 희귀도를 추출하여 업데이트."""
        try:
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
                sibling = await parent.evaluate_handle('node => node.nextElementSibling')
                if sibling:
                    rarity_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/rarity_"]', 'src')
                    if rarity_img_src:
                        rarity_level = rarity_img_src.split('_')[-1].split('.')[0]  # rarity_1.png -> 1
                        identity_id = await self.get_identity_id_from_row(row)
                        self.container.update_identity_rarity(identity_id, rarity_level)
                    else:
                        print("Rarity image not found in the next sibling TD.")
        except Exception as e: 
            print(f"Error extracting rarity: {e}")

        return "Unknown"

    async def extract_atk_and_resource(self, row, skill_number):
        """ 공격 타입(ATK Type)과 자원(Attribute) 추출 함수 - 스킬 번호에 따라 처리 """
        try:
            #print(f"Attempting to locate skill{skill_number} type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 해당 스킬 번호에 맞는 형제로 이동
                sibling = parent
                for _ in range(skill_number + 1):  # 스킬 번호에 맞게 이동
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                atk_type, resource = "Unknown", "Unknown"
        
                if sibling:
                    # 공격 타입 이미지 (atk_type) 찾기
                    atk_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/atk_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if atk_type_img_src:
                        atk_type = atk_type_img_src.split('_')[-1].split('.')[0]  # atk_type_SLASH.png -> SLASH
                        #print(f"Extracted attack type: {atk_type}")
                        self.container.update_skill(identity_id, skill_number, atk_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted resource: {resource}")
                        
                        # 리소스를 변환
                        resource = convert_resource(resource)
                        #print(f"Converted resource: {resource}")
                        self.container.update_sin_affinity(identity_id, skill_number, resource)
                        
        except Exception as e:
            print(f"Error extracting skill{skill_number} type and resource: {e}")

    async def extract_defense_and_resource(self, row):
        """ 방어 타입(Def Type)과 자원(Attribute) 추출 함수 """
        try:
            #print("Attempting to locate defense type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 다섯 번째 형제 td로 이동 (방어 관련 정보가 있는 <td>)
                sibling = parent
                for _ in range(5):  # 방어 정보는 5번째 형제에 있음
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                def_type, def_resource = "Unknown", "Unknown"
        
                if sibling:
                    # 방어 타입 이미지 (def_type) 찾기
                    def_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/def_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if def_type_img_src:
                        def_type = def_type_img_src.split('_')[-1].split('.')[0]  # def_type_GUARD.png -> GUARD
                        #print(f"Extracted defense type: {def_type}")
                        self.container.update_skill(identity_id, 'defense', def_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        def_resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted defense resource: {def_resource}")
                        
                        # 방어 리소스를 변환
                        def_resource = convert_resource(def_resource)
                        #print(f"Converted defense resource: {def_resource}")
                        self.container.update_sin_affinity(identity_id, 'defense', def_resource)
 
        except Exception as e:
            print(f"Error extracting defense type and resource: {e}")

    async def crawl_identity_details_for_all(self, page):
        """모든 인격의 세부 정보를 크롤링."""
        for identity in self.container.identities:
            await page.wait_for_load_state("networkidle")
            await page.goto(self.base_url + "/identity/"+identity.identity_id)
            """ 레벨과 동기화와 관련 없는 참관타 저항은 밑에서 변경하는 것과 연관성이 적기 때문에 위에서 처리한다.
            print(f"Identity_id: {identity.identity_id}")
            results = []
            resist_type_imgs = [f"{self.img_url}/ui/atk_type_HIT.png",
                                f"{self.img_url}/ui/atk_type_PENETRATE.png",
                                f"{self.img_url}/ui/atk_type_SLASH.png"]
            for resist_type_img in resist_type_imgs:
                result = await self.extract_resist_type(page, resist_type_img, 1)
                results.append(result)
            self.container.add_resist_info(identity.identity_id, results[0], results[1], results[2])
            """
            """
            # Stagger Threshold 값 추출
            stagger_threshold = await self.extract_stagger_threshold_values(page)
            self.container.add_stagger_threshold(identity.identity_id, stagger_threshold[0], stagger_threshold[1], stagger_threshold[2])
            """
            await self.initialize_uptie_and_level(page)
            """ 레벨에 따른 level, hp, defense 다르게 되어 있는 것을 출력 확인 객체로 들어가서 csv로 옮기는 과정까지 진행
            for i in range(1, identity.level_max+1):
                await self.move_slider_to_value(page,i)
                await self.extract_level_status(page, identity.identity_id, i)
            """
            """ 동기화에 따른 speed 다르게 되어 있는 것을 출력 확인 객체로 들어가서 csv로 옮기는 과정까지 진행
            for i in range(1, identity.uptie_max+1):
                await self.uptie_click(page,i)
                await self.extract_uptie_status(page, identity.identity_id, i)
            """
            """
                인격에 대한 skill 정보를 가져오는 메소드 위에서 출력하는 csv파일로 인격이 가지고 있는 스킬 정보를 내보낸다.
            await self.crawl_skills(page, identity.identity_id)
            """
            await self.crawl_influence_behavior(page, identity.identity_id)
            
    async def initialize_uptie_and_level(self, page):
        await self.uptie_click(page, 1)
        await self.move_slider_to_value(page, 1)
        self.uptie_level = 1
        #return 1
    
    async def uptie_click(self, page, phase):
        fab_button_selector = ".q-btn.q-btn--fab"
        await page.wait_for_selector(fab_button_selector)
        await page.click(fab_button_selector)
        #print(f"[{setup_uptie_button}] 상위 동기화 버튼 클릭 완료")
        setup_uptie_button = f'a:has(img[src="{self.img_url}/ui/uptie_{phase}.png"])'
        await page.wait_for_selector(setup_uptie_button)
        await page.click(setup_uptie_button)
        #print(f"{phase} 동기화 버튼 클릭 완료")

    async def move_slider_to_value(self, page, target_value):
        """
        슬라이더를 주어진 값으로 이동시킵니다.
        target_value는 슬라이더의 최소값부터 최대값 사이여야 합니다.
        """
        slider_selector = ".q-slider"
        # 슬라이더의 범위 확인
        min_value = int(await page.get_attribute(slider_selector, "aria-valuemin"))
        max_value = int(await page.get_attribute(slider_selector, "aria-valuemax"))
    
        if target_value < min_value or target_value > max_value:
            raise ValueError(f"Target value {target_value} is out of bounds ({min_value}, {max_value})")
    
        # 슬라이더 thumb 요소와 위치 가져오기
        slider_thumb = await page.query_selector(f"{slider_selector} .q-slider__thumb")
        thumb_box = await slider_thumb.bounding_box()
        
        # 슬라이더의 현재 위치 및 목표 위치 계산
        track = await page.query_selector(f"{slider_selector} .q-slider__track")
        track_box = await track.bounding_box()
        
        # 목표 x 위치 계산 (슬라이더 트랙 내에서 비율 기반)
        target_percentage = (target_value - min_value) / (max_value - min_value)
        target_x = track_box["x"] + target_percentage * track_box["width"]
        current_x = thumb_box["x"] + thumb_box["width"] / 2
        y = thumb_box["y"] + thumb_box["height"] / 2
    
        # 슬라이더를 드래그하여 목표 위치로 이동
        await page.mouse.move(current_x, y)
        await page.mouse.down()
        await page.mouse.move(target_x, y, steps=10)  # 부드럽게 이동
        await page.mouse.up()
    
        #print(f"슬라이더를 {target_value}로 이동 완료.")
    
        # 슬라이더 값 확인 및 강제 이벤트 트리거
        await page.evaluate(
            f"""
            (slider) => {{
                slider.dispatchEvent(new Event('input'));  // oninput 이벤트 트리거
                slider.dispatchEvent(new Event('change'));  // onchange 이벤트 트리거
            }}
            """,
            await page.query_selector(slider_selector)
        )
    
        # 이동 후 값 확인
        current_value = await page.get_attribute(slider_selector, "aria-valuenow")
        if current_value != str(target_value):
            print(f"경고: 슬라이더 값이 예상과 다릅니다. 현재 값: {current_value}, 예상 값: {target_value}")

    async def get_second_status_element(self, page):
        """
        두 번째 'Status' 요소에 접근하여 반환하는 메소드
        """
        status_elements = await page.query_selector_all("text=Status")
        if len(status_elements) > 1:
            second_status_element = status_elements[1]
            #print("두 번째 'Status' 요소에 접근합니다.")
            return second_status_element
        else:
            #print("두 번째 'Status' 요소가 존재하지 않습니다.")
            return None

    async def extract_level_status(self, page, identity_id, level):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 레벨에 따라 HP와 Defense가 변한다.
        hp_value = await self.extract_hp(second_status_element)
        defense_value = await self.extract_defense(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Level={level}, HP={hp_value}, Defense={defense_value}")
        self.container.add_level_stat(identity_id,level, hp_value, defense_value)
    
    async def extract_hp(self, status_element):
        # HP 추출
        hp_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].children[0].nextElementSibling.textContent.trim()"
        )
        return hp_value
    
    async def extract_defense(self, status_element):
        # Defense 추출
        defense_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return defense_value

    async def extract_uptie_status(self, page, identity_id, uptie):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 uptie에 따라 speed는 변한다.
        speed_value = await self.extract_speed(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Uptie={uptie}, Speed={speed_value}")
        self.container.add_uptie_stat(identity_id,uptie, speed_value)

    async def extract_speed(self, status_element):
        # Speed 추출
        speed_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return speed_value

    async def extract_resist_type(self, page, image_src, depth):
        # 이미지 요소 찾기
        #print(image_src)
        image = await page.query_selector(f'img[src="{image_src}"]')
        #print(image)
        if not image:
            return "Unknown"
    
        # 이미지의 부모 요소를 찾아서 올라감
        parent = image
        for _ in range(depth):
            parent = await parent.evaluate_handle('node => node.parentElement')
            if not parent:
                return "Unknown"
    
        # 부모의 형제 요소 찾기
        sibling = await parent.evaluate_handle('node => node.nextElementSibling')
        
        if sibling:
            value = await sibling.evaluate('node => node.textContent')
            if value:
                return value.strip()
        
        return "Unknown"

    async def extract_stagger_threshold_values(self, page):
        # 'div.row.center.flex-center.q-pt-xs' 요소들 찾기
        elements = await page.query_selector_all('div.row.center.flex-center.q-pt-xs')
        
        # 각 요소의 텍스트 내용 출력하여 'Stagger Threshold' 요소를 찾기
        for element in elements:
            text = await element.text_content()
            
            if text and "Stagger Threshold" in text:
                # 'Stagger Threshold'가 있는 요소를 찾았다면, 그 이후 형제 요소들에서 값을 추출
                parent = await element.evaluate_handle('node => node.parentElement')
                sibling_elements = await parent.query_selector_all('div.row.justify-evenly div')  # Threshold 값이 들어있는 div들
                
                stagger_values = []
                for sibling in sibling_elements:
                    text_content = await sibling.text_content()
                    match = re.search(r'(\d+%)', text_content)
                    if match:
                        stagger_values.append(match.group(1))  # 퍼센트 값만 리스트에 추가
                while len(stagger_values) < 3:
                    stagger_values.append(np.nan)
                #print(stagger_values)
                return stagger_values  # 추출된 값(또는 np.nan 포함)을 반환
        
        return [np.nan, np.nan, np.nan]  # 만약 'Stagger Threshold'를 찾지 못한 경우 기본값 반환

    async def crawl_skills(self, page, identity_id):
        await self.crawl_skill(page, identity_id, "skill 1")
        await self.crawl_skill(page, identity_id, "skill 2")
        await self.crawl_skill(page, identity_id, "skill 3")
        await self.crawl_skill(page, identity_id, "defense")


    async def crawl_skill(self, page, identity_id, skill_type):
        skills_elements = await page.query_selector_all('div.skilltag')
        for skill_element in skills_elements:
            skill_text = await skill_element.text_content()
            if skill_text.strip().lower() == skill_type:
                await self.process_skill(skill_element, page, identity_id, skill_type)
        
    async def process_skill(self, skill_element, page, identity_id, skill_type):
        # skill_type이 "skill 3"일 경우 uptie_value가 3 이상일 때만 정보를 저장
        for uptie_value in range(1, 5):  # Uptie levels 1 to 4
            # "skill 3"은 uptie_value가 3 이상일 때만 처리
            if skill_type == "skill 3" and uptie_value < 3:
                continue  # skip if uptie_value is less than 3 for skill 3
    
            # uptie_level에 맞게 클릭
            await self.uptie_click(page, uptie_value)
            if skill_type != "skill 3" and uptie_value > 1:
                # Uptie 정보 갱신
                await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
                continue
            elif skill_type == "skill 3" and uptie_value > 3:
                # Uptie 정보 갱신
                await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
                continue
            # 스킬의 세부 정보를 추출
            skill_details = await self.extract_skill_details(skill_element, skill_type)
            skill_name = skill_details["skill_name"]
    
            # 스킬 정보를 저장
            self.container.add_skill(
                identity_id, skill_type, 
                skill_name, skill_details["coin_count"],
                skill_details["init_level"], skill_details["init_remain"], 
                skill_details["atk_weight"]
            )
            print(f"Added skill: {skill_type}:{skill_name}")
    
            # Uptie 정보 갱신
            await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
    
    async def extract_skill_details(self, skill_element, skill_type):
        # 스킬 세부 정보를 추출하는 함수
        skill_name = await self.skill_name(skill_element)
        coin_count = await self.coin_count(skill_element, ".col-auto > .column > .row")
        init_level = await self.init_level(skill_element)
        init_remain = await self.init_remain(skill_element, skill_type, "div.row.justify-start > div.attr.flex.flex-center > div.q-px-sm")
        atk_weight = await self.atk_weight(skill_element, skill_type)
        return {
            "skill_name": skill_name,
            "coin_count": coin_count,
            "init_level": init_level,
            "init_remain": init_remain,
            "atk_weight": atk_weight
        }
    
    async def update_uptie_info(self, page, skill_element, identity_id, skill_name, skill_type, uptie_value):
        # Uptie 정보를 추출하고 업데이트하는 함수
        uptie_info = await self.extract_uptie_info(skill_element)
        effect_data = await self.skill_effects(skill_element)
        self.container.add_skill_uptie_info(
            identity_id, skill_name, 
            uptie_value, uptie_info["skill_power"], uptie_info["coin_power"], 
            effect_data.get("effectsByCondition")
        )
        print(
            f"Updated Uptie for {skill_name} (Uptie Level: {uptie_value}): "
            f"Skill Power: {uptie_info['skill_power']}, Coin Power: {uptie_info['coin_power']} "
            f"Skill Effect: {effect_data.get("effectsByCondition")}"
        )

    async def extract_uptie_info(self, skill_element):
        skill_data = await self.skill_and_coin_power(skill_element)
        return {
            "skill_power": skill_data.get("skillPower"),
            "coin_power": skill_data.get("coinPower")
        }
        
    async def crawl_influence_behavior(self, page, identity_id):
        influence_behaviors = await page.query_selector_all('div[align="right"]')
        # 텍스트를 기준으 추출하고 메소드로 파라미터 들어가 정보를 받는다.
        for influence_behavior in influence_behaviors:
            active_type = await influence_behavior.inner_text()
            active_type = active_type.strip()

            if active_type == "PASSIVE":
                """
                # PASSIVE에 대한 처리
                print(f"Processing PASSIVE logic for: {active_type}")
                # CALL method for PASSIVE
                """
                passive_data = await self.passive_effects(influence_behavior)
            elif active_type == "SUPPORT PASSIVE":
                """
                # SUPPORT PASSIVE에 대한 처리
                print(f"Processing SUPPORT PASSIVE logic for: {active_type}")
                # CALL method for SUPPORT PASSIVE
                """
                support_passive_data = await self.support_passive_effects(influence_behavior)
            elif active_type == "PANIC TYPE":
                """
                # PANIC TYPE에 대한 처리
                print(f"Processing PANIC TYPE logic for:{active_type}")
                # CALL method for PANIC TYPE
                """
                panic_type_data = await self.panic_effects(influence_behavior)

    async def skill_and_coin_power(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                // 조부모로 이동
                const grandparent = node.parentElement.parentElement;
                // 이전 형제 탐색
                const previousSibling = grandparent.previousElementSibling;
                if (!previousSibling) return {{ skillPower: null, coinPower: null }}; // 이전 형제가 없으면 기본값 반환
                // 이전 형제 내부의 .row.justify-around > div > div 탐색
                const targetDivs = previousSibling.querySelectorAll(".row.justify-around > div");
                if (targetDivs.length < 3) return {{ skillPower: null, coinPower: null }}; // 배열 길이 확인
                // 1st div: 스킬 위력
                const skillPower = targetDivs[0]?.querySelector('div')?.textContent?.trim() || null; 
                // 3rd div: 코인 파워 값 (3번째 div에서 텍스트 추출)
                const coinPower = targetDivs[2]?.textContent?.trim() || null;
                return {{ skillPower, coinPower }};
            }}
        """)


    async def coin_count(self, skill_element, hierarchy_selector):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector('{hierarchy_selector}');
                if (!targetElement) return 0;
                return targetElement.childElementCount;
            }}
        """)

    async def init_remain(self, skill_element, skill_type, hierarchy_selector):
        if skill_type == "defense":
            return -1
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector('{hierarchy_selector}');
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def skill_name(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector(".block");
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def init_level(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector("div.text-bold.text-h6.h6.q-pl-xs.q-pr-md");
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def atk_weight(self, skill_element, skill_type):
        if skill_type == "defense":
            return 0
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector("div.text-bold.text-h6.h6.q-pl-xs.q-pr-md");
                const doubleNextSibling = targetElement.nextElementSibling.nextElementSibling;
                if (!doubleNextSibling) return 0;
                return doubleNextSibling.textContent?.trim();
            }}
        """)

    async def skill_effects(self, skill_element):
        return await skill_element.evaluate("""
            node => {
                const parent = node.parentElement;
                const doubleNextSibling = parent.nextElementSibling?.nextElementSibling;
                if (!doubleNextSibling) return null;
    
                // doubleNextSibling에서 첫 번째 div.col 요소 찾기
                const textNode = doubleNextSibling.querySelector("div.col");
                if (!textNode) return null;
    
                const results = {
                    keywords: [], // 키워드 저장
                    conditions: [], // 조건 저장
                    effectsByCondition: {} // 조건별 효과
                };
    
                // 1. 조건 추출 (대괄호 텍스트)
                const conditionElements = textNode.querySelectorAll("span");
                conditionElements.forEach(span => {
                    let text = span.textContent?.trim();
                    if (text?.startsWith("[") && text?.endsWith("]")) {
                        // 부모의 전형제 노드 탐색
                        const sibling = span.parentElement.previousElementSibling;
                        if (sibling) {
                            const emblemImage = sibling.querySelector("img.emblem");
                            console.log("Sibling element found:", sibling); // 디버깅용
                            if (emblemImage) {
                                const src = emblemImage.getAttribute("src");
                                console.log("Image source:", src); // 디버깅용
                                const match = src.match(/ui_num_(\\d+)\\.png$/); // 정규식 수정
                                if (match) {
                                    const number = match[1]; // 숫자 추출
                                    text = `[Coin ${number} ${text.replace("[", "").replace("]", "")}]`;
                                    console.log("Updated condition:", text); // 디버깅용
                                }
                            }
                        }
                        results.conditions.push(text);
                        results.effectsByCondition[text] = []; // 조건별 효과를 초기화
                    }
                });
    
                // 2. 키워드 추출
                const keywordElements = textNode.querySelectorAll('a[href^="/en/keyword/"]');
                keywordElements.forEach(a => {
                    const span = a.querySelector('span.q-btn__content.text-center.col.items-center.q-anchor--skip.justify-center.row>span');
                    if (span) {
                        const keywordText = span.textContent.trim();
                        if (keywordText && !results.keywords.includes(keywordText)) {
                            results.keywords.push(keywordText); // 키워드 저장
                        }
                    }
                });
    
                // 3. 효과 추출
                const effectElements = textNode.querySelectorAll("span");
                let currentCondition = null;
                let currentEffect = []; // 초기화
                effectElements.forEach(span => {
                    let text = span.textContent?.trim();
                    if (!text) return;
    
                    // 조건 발견 시 currentCondition 변경
                    if (text.startsWith("[") && text.endsWith("]")) {
                        // 이전 조건의 효과 저장
                        if (currentCondition && currentEffect.length > 0) {
                            // 부모의 전형제 노드 탐색
                            const sibling = span.parentElement.previousElementSibling;
                            if (sibling) {
                                const emblemImage = sibling.querySelector("img.emblem");
                                if (emblemImage) {
                                    const src = emblemImage.getAttribute("src");
                                    const match = src.match(/ui_num_(\\d+)\\.png$/); // 정규식 수정
                                    if (match) {
                                        const number = match[1]; // 숫자 추출
                                        text = `[Coin ${number} ${text.replace("[", "").replace("]", "")}]`;
                                    }
                                }
                            }
                            results.effectsByCondition[currentCondition].push(currentEffect.join(' '));
                            currentEffect = []; // 초기화
                        }
                        currentCondition = text;
                        return;
                    }
    
                    // 현재 조건의 효과 처리
                    if (currentCondition) {
                        if (results.keywords.includes(text)) {
                            const keywordEffect = 'keyword:' + text;
                            if (!currentEffect.includes(keywordEffect)) {
                                currentEffect.push(keywordEffect);
                            }
                        } else {
                            currentEffect.push(text);
                        }
                    }
                });
    
                // 마지막 효과 저장
                if (currentCondition && currentEffect.length > 0) {
                    results.effectsByCondition[currentCondition].push(currentEffect.join(' '));
                }
    
                return results;
            }
        """)

    async def passive_effects(self, passive_element):
        passive_name = await self.behavior_name(passive_element)
        passive_conditions = await self.behavior_condition(passive_element)
        passive_description = await self.behavior_description(passive_element)
        print(passive_name)
        print(passive_conditions)
        print(passive_description)
    async def support_passive_effects(self, support_passive_element):
        support_passive_name = await self.behavior_name(support_passive_element)
        support_passive_conditions = await self.behavior_condition(support_passive_element)
        support_passive_description = await self.behavior_description(support_passive_element)
        print(support_passive_name)
        print(support_passive_conditions)
        print(support_passive_description)
    async def panic_effects(self, panic_element):
        panic_type = await self.behavior_name(panic_element)
        panic_conditions = await self.behavior_condition(panic_element)
        panic_description = await self.panic_description(panic_element)
        print(panic_type)
        print(panic_conditions)
        print(panic_description)

    async def navigate_element_by_steps(self, element, parent_steps=0, prev_sibling_steps=0, next_sibling_steps=0):
        """
        주어진 `element`에서 부모, 이전 형제, 또는 다음 형제로 지정된 횟수만큼 이동하여 ElementHandle 반환.
        
        Args:
            element (ElementHandle): 탐색 기준이 되는 요소.
            parent_steps (int): 부모로 이동할 횟수 (0이면 이동하지 않음).
            prev_sibling_steps (int): 이전 형제로 이동할 횟수 (0이면 이동하지 않음).
            next_sibling_steps (int): 다음 형제로 이동할 횟수 (0이면 이동하지 않음).
        
        Returns:
            ElementHandle or None: 탐색된 요소를 반환하거나 없으면 None.
        """
        try:
            current_element = element
    
            # 부모로 이동
            for _ in range(parent_steps):
                current_element = await current_element.evaluate_handle('el => el.parentElement')
                if not current_element:
                    print("Parent element not found during navigation.")
                    return None
    
            # 이전 형제로 이동
            for _ in range(prev_sibling_steps):
                current_element = await current_element.evaluate_handle('el => el.previousElementSibling')
                if not current_element:
                    print("Previous sibling element not found during navigation.")
                    return None
    
            # 다음 형제로 이동
            for _ in range(next_sibling_steps):
                current_element = await current_element.evaluate_handle('el => el.nextElementSibling')
                if not current_element:
                    print("Next sibling element not found during navigation.")
                    return None
    
            return current_element
    
        except Exception as e:
            print(f"Error navigating elements: {e}")
            return None

    async def get_last_sibling(self, element):
        try:
            # 같은 부 아 마지막 형제를 가져오는 JavaScript 실행
            last_sibling = await element.evaluate_handle('el => el.parentElement.lastElementChild')
            return last_sibling
        except Exception as e:
            print(f"Error getting last sibling: {e}")
            return None

    async def behavior_name(self, element):
        parent_prev_sibling = await self.navigate_element_by_steps(element, parent_steps=1, prev_sibling_steps=1, next_sibling_steps=0)
        passive_name_element = await parent_prev_sibling.query_selector('span.block')
        if passive_name_element:
            passive_name = await passive_name_element.inner_text()
            return passive_name
        else:
            #print("Passive name element not found")
            return None

    async def behavior_condition(self, element):
        try:
            conditions = []
            parent_prev_sibling = await self.navigate_element_by_steps(element, parent_steps=1, prev_sibling_steps=1, next_sibling_steps=0)
            img_elements = await parent_prev_sibling.query_selector_all('img')
            if not img_elements:
                #print("No <img> elements found.")
                return conditions
    
            for img_element in img_elements:
                # 각 이미지의 src 속성 가져오기
                img_src = await img_element.get_attribute('src')
                #print(img_src)
    
                sin_name = None
                for color_key in resource_mapping.keys():
                    if f"{self.img_url}/ui/attr_{color_key}.png" in img_src:
                        sin_name = resource_mapping[color_key]
                        break
    
                if not sin_name:
                    #print(f"No matching Sin Color found for {img_src}.")
                    continue  # No Sin Color found, skip this img_element
    
                # 해당 이미지와 관련된 조건 텍스트 추출
                # 두 번째 부모 요소에서 시작하여 다음 형제를 찾아 조건 텍스트를 추출
                condition_element = await self.navigate_element_by_steps(img_element, parent_steps=3, prev_sibling_steps=0, next_sibling_steps=0)
                if condition_element:
                    # condition_element가 None이 아닌 경우에만 진행
                    condition_text_element = await condition_element.query_selector('.q-pl-xs')
                    if condition_text_element:
                        condition_text = await condition_text_element.inner_text()
                        condition_text = condition_text.strip()
                    else:
                        print(f"No condition text found for {img_src}.")
                        condition_text = ""  # condition_text_element가 None이면 빈 문자열로 처리
                else:
                    print(f"No condition element found for {img_src}.")
                    condition_text = ""  # condition_element가 None이면 빈 문자열로 처리
    
                # 결과를 배열에 추가
                conditions.append(f"{sin_name} {condition_text}")
            return conditions
        except Exception as e:
            print(f"Error extracting conditions: {e}")
            return 
            
    async def behavior_description(self, element):
        parent_prev_sibling = await self.navigate_element_by_steps(element, parent_steps=1, prev_sibling_steps=1, next_sibling_steps=0)
        return await parent_prev_sibling.evaluate("""
                node => {
                    const behavior_title_node = node.querySelector('.q-pb-sm')
                    const behavior_title_parent = behavior_title_node.parentElement
                    const behavior_description = behavior_title_parent.lastElementChild
                    if (!behavior_description) return null;

                    const results = {
                        keywords: [], // 키워드 저장
                        description: [] // 설명 저장
                    };
                    // 1. 키워드 추출
                    const keywordElements = behavior_description.querySelectorAll('a[href^="/en/keyword/"]');
                    if (!keywordElements) return null;
                    keywordElements.forEach(a => {
                        const span = a.querySelector('span.q-btn__content.text-center.col.items-center.q-anchor--skip.justify-center.row>span');
                        if (span) {
                            const keywordText = span.textContent.trim();
                            if (keywordText && !results.keywords.includes(keywordText)) {
                                results.keywords.push(keywordText); // 키워드 저장
                            }
                        }
                    });
        
                    // 2. 설명 추출
                    const descriptionElements = behavior_description.querySelectorAll("span");
                    let currentEffect = []; // 초기화
                    descriptionElements.forEach(span => {
                        let text = span.textContent?.trim();
                        if (!text) return;
                        if (results.keywords.includes(text)) {
                            const keywordEffect = 'keyword:' + text;
                            if (!currentEffect.includes(keywordEffect)) {
                                currentEffect.push(keywordEffect);
                            }
                        } else {
                            currentEffect.push(text);
                        }
                    });
        
                    // 마지막 효과 저장
                    if (currentEffect.length > 0) {
                        results.description.push(currentEffect.join(' '));
                    }
        
                    return results;
                }
            """)

    async def panic_description(self, element):
        try:
            description_texts = []
            keywords = []
    
            # Navigate to the behavior description parent
            parent_prev_sibling = await self.navigate_element_by_steps(element, parent_steps=1, prev_sibling_steps=1, next_sibling_steps=0)
            behavior_title = await parent_prev_sibling.query_selector('.q-pb-sm')
            behavior_description_element = await self.get_last_sibling(behavior_title)
    
            # Fetch keyword elements
            keyword_elements = await behavior_description_element.query_selector_all('a[href^="/en/keyword/"]')
            for keyword_element in keyword_elements:
                keyword_span = await keyword_element.query_selector(
                    'span.q-btn__content.text-center.col.items-center.q-anchor--skip.justify-center.row > span'
                )
                if keyword_span:
                    keyword_text = await keyword_span.inner_text()
                    stripped_text = keyword_text.strip()
                    if stripped_text and stripped_text not in keywords:
                        keywords.append(stripped_text)
    
            # Fetch description elements
            descriptionElements = await behavior_description_element.query_selector_all("div")
            seen_texts = []
            # 수정된 부분: inner_text()의 원본을 출력해보고 strip()을 조정하여 문제를 파악합니다.
            # 키워드를 포함한 텍스트에서 마지막 문장이 누락되지 않도록 조정
            for descriptionElement in descriptionElements:
                description_text = await descriptionElement.inner_text()
                #print(f"Original text: '{description_text}'")  # 원본 텍스트 출력
                stripped_text = description_text.strip()  # 앞뒤 공백만 제거
                stripped_text = re.sub(r'\s+', ' ', stripped_text)
                description_texts.append(stripped_text)
            normalize_text = await self.combine_and_process_texts(description_texts)
            return normalize_text
        except Exception as e:
            print(f"Error extracting description: {e}")
            return

    async def combine_and_process_texts(self, texts):
        """
        배열 형태의 텍스트 데이터를 합치고 형식을 처리한 문자열을 반환합니다.
        Args:
            texts (list of list): 텍스트 배열 목록
        Returns:
            str: 형식이 처리된 최종 문자열
        """
        processed_sentences = []
        
        for text_array in texts:
            # 빈 문자열을 제외한 나머지 텍스트를 이어붙이기
            sentence = ''.join([word for word in text_array if word != ''])
            sentence = re.sub(r'\\](\\s*)', ']', sentence)
            processed_sentences.append(sentence)
    
        # 문장을 공백으로 연결하여 반환
        return ''.join(processed_sentences)


class Identity:
    """인격 정보를 저장하는 클래스"""
    def __init__(self, identity_id, name=None, rarity=None,\
                 skill_1_type=None, skill_1_sin_affinity=None,\
                 skill_2_type=None, skill_2_sin_affinity=None,\
                 skill_3_type=None, skill_3_sin_affinity=None,\
                 defense_type=None, defense_sin_affinity=None):
        self.identity_id = identity_id
        self.name = name
        self.rarity = rarity
        # 선언만 하는 부분 
        self.skill_1_type = skill_1_type 
        self.skill_1_sin_affinity = skill_1_sin_affinity 
        self.skill_2_type = skill_2_type 
        self.skill_2_sin_affinity = skill_2_sin_affinity 
        self.skill_3_type = skill_3_type 
        self.skill_3_sin_affinity = skill_3_sin_affinity 
        self.defense_type = defense_type 
        self.defense_sin_affinity = defense_sin_affinity
        self.level_max = 50
        self.uptie_max = 4
        self.level_stats = []  # LevelStat 객체 리스트
        self.uptie_stats = [] # UptieStat 객체 리스트
        self.resist_info = None
        self.stagger_threshold = None
        self.skillset = SkillSet()  # Skill정보를 담고 있는 skillset 객체 
        self.passive_infos = []  # 패시브 객체를 담고 있는 배열
        self.support_passive_infos = [] # 서포트 패시 객체를 담고 있는 배열
        self.panic_type = [] # 혼란상태 발생시 나타나는 효과를 담고 있는 객체 배열1

    def update_name(self, name):
        self.name = name

    def update_rarity(self, rarity):
        self.rarity = rarity

    def add_level_stat(self, level, hp, defense):
        self.level_stats.append(LevelStat(level, hp, defense))

    def add_uptie_stat(self, uptie, speed):
        self.uptie_stats.append(UptieStat(uptie, speed))

    def add_resist_info(self, resist_hit, resist_penetrate, resist_slash):
        self.resist_info = ResistInfo(resist_hit, resist_penetrate, resist_slash)

    def add_stagger_threshold(self, first_stagger, second_stagger, third_stagger):
        self.stagger_threshold = StaggerThreshold(first_stagger, second_stagger, third_stagger)

    def add_skill(self, skill_code, skill_name, coin_count, init_level, init_remain, atk_weight):
        self.skillset.add_skill(skill_code, Skill(skill_code, skill_name, coin_count, init_level, init_remain, atk_weight))
        
    def add_skill_uptie_info(self, skill_name, uptie_level, skill_power, coin_power, effect_by_condition):
        self.skillset.add_skill_uptie_info(skill_name, uptie_level, skill_power, coin_power, effect_by_condition)
        
    def update_details(self, page):
        # 세부 정보 업데이트 로직 (가정)
        pass
    #def init_uptie(self, page):
    #    init_uptie = 

# LevelStat 객체 정의
class LevelStat:
    def __init__(self, level, hp=None, defense=None):
        self.level = level
        self.hp = hp
        self.defense = defense
    def update_level(self, level):
        self.level = level
    def update_hp(self, hp):
        self.hp = hp
    def update_defense(self, defense):
        self.defense = defense

# UptieStat 객체 정의
class UptieStat:
    def __init__(self, uptie, speed=None):
        self.uptie = uptie
        self.speed = speed
    def update_speed(self, speed):
        self.speed = speed

# ResistInfo 객체 정의
class ResistInfo:
    def __init__(self, resist_hit=None, resist_penetrate=None, resist_slash=None):
        self.resist_hit = resist_hit
        self.resist_penetrate = resist_penetrate
        self.resist_slash = resist_slash

# ResistInfo 객체 정의
class StaggerThreshold:
    def __init__(self, first_stagger=None, second_stagger=None, third_stagger=None):
        self.first_stagger = first_stagger
        self.second_stagger = second_stagger
        self.third_stagger = third_stagger

class SkillSet:
    def __init__(self):

        self.skill_1_type = ""
        self.skill_2_type = ""
        self.skill_3_type = ""
        self.defense_skill_type = ""

        self.skill_1_sin_affinity = ""
        self.skill_2_sin_affinity = ""
        self.skill_3_sin_affinity = ""
        self.defense_skill_sin_affinity = ""

        self.skill_1_details = []
        self.skill_2_details = []
        self.skill_3_details = []
        self.defense_skill_details = []

    def set_skill_type(self, skill_code, skill_type, sin_affinity):
        if skill_code == "skill 1":
            self.skill_1_type = skill_type
            self.skill_1_sin_affinity = sin_affinity
        elif skill_code == "skill 2":
            self.skill_2_type = skill_type
            self.skill_2_sin_affinity = sin_affinity
        elif skill_code == "skill 3":
            self.skill_3_type = skill_type
            self.skill_3_sin_affinity = sin_affinity
        elif skill_code == "defense":
            self.defense_skill_type = skill_type
            self.defense_skill_sin_affinity = sin_affinity
    
    def add_skill(self, skill_code, skill):
        if skill_code == "skill 1":
            self.skill_1_details.append(skill)
            skill.skill_type = self.skill_1_type
            skill.sin_affinity = self.skill_1_sin_affinity
        elif skill_code == "skill 2":
            self.skill_2_details.append(skill)
            skill.skill_type = self.skill_2_type
            skill.sin_affinity = self.skill_2_sin_affinity
        elif skill_code == "skill 3":
            self.skill_3_details.append(skill)
            skill.skill_type = self.skill_3_type
            skill.sin_affinity = self.skill_3_sin_affinity
        elif skill_code == "defense":
            self.defense_skill_details.append(skill)
            skill.skill_type = self.defense_skill_type 
            skill.sin_affinity = self.defense_skill_sin_affinity

    def get_skill_by_name(self, skill_name):
        for skill_list in [self.skill_1_details, self.skill_2_details, self.skill_3_details, self.defense_skill_details]:
            for skill in skill_list:
                result = skill.is_skill_by_name(skill_name)
                if result:
                    return result
        return None

    def add_skill_uptie_info(self, skill_name, uptie_level, skill_power, coin_power, effect_by_condition):
        skill = self.get_skill_by_name(skill_name)
        skill.add_uptie_info(uptie_level, skill_power, coin_power, effect_by_condition)
        

class Skill:
    def __init__(self, skill_code, skill_name, coin_count, init_level, init_remain, atk_weight):
        self.skill_code = skill_code # "Skill 1"/"Skill 2"/"Skill 3"/"Defense" '''o'''
        self.skill_type = None
        self.sin_affinity = None
        self.skill_name = skill_name # 스킬 이름 '''o'''
        self.coin_count = coin_count # 동전 개수 '''o'''
        self.init_level = init_level # 스킬이 초기 배정받은 레벨, 인격 레벨에 - 1 한 값을 더해주면 스킬의 현 level을 구할 수 있다. '''o'''
        self.init_remain = init_remain #'''o'''
        self.atk_weight = atk_weight #공격 가중치 값이다. 방어스킬에는 없기 때문에 None으로 설정했다. '''o'''
        self.remain_count = init_remain # 실제 스킬이 작동하며 횟수가 차감되 나중에 모두 차감되면 다음 턴에 다시 초기횟수 리필된다.
        self.uptie_info = []  

    def is_skill_by_name(self, skill_name):
        if self.skill_name == skill_name:
            return self
        return None
        
    def add_uptie_info(self, uptie_level, skill_power, coin_power, effects_by_condition):
        self.uptie_info.append(UptieInfo(uptie_level, skill_power, coin_power, effects_by_condition))

class UptieInfo:
    def __init__(self, uptie_level, skill_power, coin_power, effects_by_condition):
        self.uptie_level = uptie_level   #'''o'''
        self.skill_power = skill_power   #'''o'''
        self.coin_power = coin_power     #'''o'''
        self.skill_effect = effects_by_condition

# 추상화된 EffectType 클래스
class EffectType:
    def __init__(self, name, description):
        self.name = name  # Effect 이름
        self.description = description  # Effect 설명

# Passive 클래스
class Passive(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)  # condition이 있는지 여부 확인

# SupportPassive 클래스
class SupportPassive(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)

# PanicType 클래스
class PanicType(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)

# Keyword 클래스
class Keyword:
    def __init__(self, keyword, explain=None, dispellable=None):
        self.keyword = keyword
        self.explain = explain
        self.dispellable = dispellable

      
class DataContainer:
    """크롤링 데이터를 관리하는 클래스"""
    def __init__(self):
        self.identities = []

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.level_max

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.uptie_max

    def update_identity_name(self, identity_id, identity_name):
        """특정 ID의 이름을 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_name(identity_name)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_identity_rarity(self, identity_id, rarity):
        """특정 ID의 희귀도를 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_rarity(rarity)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_skill(self, identity_id, skill_number, new_type):
        """
        특정 identity의 skill_type을 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_type: 새 skill_type
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_type = new_type
                elif skill_number == 2:
                    identity.skill_2_type = new_type
                elif skill_number == 3:
                    identity.skill_3_type = new_type
                elif skill_number == "defense":
                    identity.defense_type = new_type
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def update_sin_affinity(self, identity_id, skill_number, new_affinity):
        """
        특정 identity의 sin_affinity를 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_affinity: 새 sin_affinity
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_sin_affinity = new_affinity
                elif skill_number == 2:
                    identity.skill_2_sin_affinity = new_affinity
                elif skill_number == 3:
                    identity.skill_3_sin_affinity = new_affinity
                elif skill_number == "defense":
                    identity.defense_sin_affinity = new_affinity
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def add_identity(self, identity):
        """새로운 인격 데이터를 추가."""
        self.identities.append(identity)

    def add_level_stat(self, identity_id, level, hp, defense):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_level_stat(level, hp, defense)

    def add_uptie_stat(self, identity_id, uptie, speed):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_uptie_stat(uptie, speed)

    def add_resist_info(self, identity_id, resist_hit, resist_penetrate, resist_slash):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_resist_info(resist_hit, resist_penetrate, resist_slash)

    def add_stagger_threshold(self, identity_id, first_stagger, second_stagger, third_stagger):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_stagger_threshold(first_stagger, second_stagger, third_stagger)

    
    def add_skill(self, identity_id, skill_code, skill_name, coin_count, init_level, init_remain, atk_weight):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_code == "skill 1":
                    identity.skillset.set_skill_type(skill_code, identity.skill_1_type, identity.skill_1_sin_affinity)
                elif skill_code == "skill 2":
                    identity.skillset.set_skill_type(skill_code, identity.skill_2_type, identity.skill_2_sin_affinity)
                elif skill_code == "skill 3":
                    identity.skillset.set_skill_type(skill_code, identity.skill_3_type, identity.skill_3_sin_affinity)
                elif skill_code == "defense":
                    identity.skillset.set_skill_type(skill_code, identity.defense_type, identity.defense_sin_affinity)
                identity.add_skill(skill_code, skill_name, coin_count, init_level, init_remain, atk_weight)

    def add_skill_uptie_info(self, identity_id, skill_name, uptie_level, skill_power, coin_power, effects_by_condition):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_skill_uptie_info(skill_name, uptie_level, skill_power, coin_power, effects_by_condition)
        
    def display_identities(self):
        """Identity, ID, Name, Rarity 정보를 출력"""
        if not self.identities:
            print("No identities found.")
        else:
            # 데이터를 리스트로 변환
            table = []
            for idx, identity in enumerate(self.identities, 1):
                table.append([
                    idx, identity.identity_id, identity.name, identity.rarity,
                    identity.skill_1_type, identity.skill_1_sin_affinity,
                    identity.skill_2_type, identity.skill_2_sin_affinity,
                    identity.skill_3_type, identity.skill_3_sin_affinity,
                    identity.defense_type, identity.defense_sin_affinity
                ])
            
            # 테이블 헤더 정의
            headers = [
                "Index", "ID", "Name", "Rarity",
                "Skill_1 Type", "Skill_1 Sin", "Skill_2 Type", "Skill_2 Sin",
                "Skill_3 Type", "Skill_3 Sin", "Defense Type", "Defense Sin"
            ]
            
            # 테이블 출력
            print(tabulate(table, headers=headers, tablefmt="fancy_grid"))

    def export_to_csv(self, filename, fieldnames, data):
        """
        공통적으로 사용 가능한 CSV 내보내기 메서드
        :param filename: 내보낼 파일 이름 (예: 'identity.csv', 'skill.csv')
        :param fieldnames: CSV의 헤더 리스트
        :param data: CSV로 내보낼 데이터 (리스트 형태)
        """
        try:
            # 현재작업 디렉토 기준으 'csv'폴더 없으면 생성
            if not os.path.exists('./csv'):
                os.makedirs('./csv')
            # 파일 경로 설정 (상대경로 'csv' 폴더 안에 저장)
            file_path = os.path.join('./csv', filename)
            with open(file_path, mode='w', newline='', encoding='utf-8') as file:
                writer = csv.DictWriter(file, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(data)
            print(f"Data successfully exported to {filename}")
        except Exception as e:
            print(f"Error exporting to CSV: {e}")

    def get_identity_data(self):
        """
        Identity 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "ID": identity.identity_id,
                "Name": identity.name,
                "Rarity": identity.rarity,
                "Skill_1_Type": identity.skill_1_type,
                "Skill_1_Sin_Affinity": identity.skill_1_sin_affinity,
                "Skill_2_Type": identity.skill_2_type,
                "Skill_2_Sin_Affinity": identity.skill_2_sin_affinity,
                "Skill_3_Type": identity.skill_3_type,
                "Skill_3_Sin_Affinity": identity.skill_3_sin_affinity,
                "Defense_Type": identity.defense_type,
                "Defense_Sin_Affinity": identity.defense_sin_affinity,
            }
            for identity in self.identities
        ]
        
    def export_identities_data(self, filename="identities.csv"):
        """Identity 데이터를 CSV로 내보내는 메서드"""
        identity_fieldnames = [
            "ID", "Name", "Rarity",
            "Skill_1_Type", "Skill_1_Sin_Affinity",
            "Skill_2_Type", "Skill_2_Sin_Affinity",
            "Skill_3_Type", "Skill_3_Sin_Affinity",
            "Defense_Type", "Defense_Sin_Affinity"
        ]
        identity_data = self.get_identity_data()  # Identity 데이터 가져오기
        self.export_to_csv(filename, identity_fieldnames, identity_data)  # CSV로 내보내기

    def load_identities_by_id(self, filename, identity_ids):
        """지정된 identity_ids 목록에 해당하는 Identity 데이터를 CSV에서 읽어옴."""
        identity_ids_set = set(identity_ids) # 중복 제거를 위 Set 사용
        with open(filename, mode='r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                if row["ID"] in identity_ids_set:
                    identity = Identity(
                        identity_id=row["ID"],
                        name=row["Name"],
                        rarity=row["Rarity"],
                        skill_1_type=row["Skill_1_Type"],
                        skill_1_sin_affinity=row["Skill_1_Sin_Affinity"],
                        skill_2_type=row["Skill_2_Type"],
                        skill_2_sin_affinity=row["Skill_2_Sin_Affinity"],
                        skill_3_type=row["Skill_3_Type"],
                        skill_3_sin_affinity=row["Skill_3_Sin_Affinity"],
                        defense_type=row["Defense_Type"],
                        defense_sin_affinity=row["Defense_Sin_Affinity"]
                    )
                    self.identities.append(identity)

    def get_level_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Level": level_stat.level,
                "HP": level_stat.hp,
                "Defense": level_stat.defense,
            }
            for identity in self.identities
            for level_stat in identity.level_stats
        ]

    def export_level_stats_data(self, filename="levelstats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        level_stats_fieldnames = ["Identity_ID", "Level", "HP", "Defense"]
        level_stats_data = self.get_level_stats_data()  # level_stats 데이터 가져오기
        self.export_to_csv(filename, level_stats_fieldnames, level_stats_data)  # CSV로 내보내기

    def get_uptie_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Uptie": uptie_stat.uptie,
                "Speed": uptie_stat.speed,
            }
            for identity in self.identities
            for uptie_stat in identity.uptie_stats
        ]

    def export_uptie_stats_data(self, filename="Uptiestats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        uptie_stats_fieldnames = ["Identity_ID", "Uptie", "Speed"]
        uptie_stats_data = self.get_uptie_stats_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, uptie_stats_fieldnames, uptie_stats_data)  # CSV로 내보내기

    def get_resist_info_data(self):
        """
        모든 Identity의 resist_info 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Resist_Hit": identity.resist_info.resist_hit,
                "Resist_Penetrate": identity.resist_info.resist_penetrate,
                "Resist_Slash": identity.resist_info.resist_slash,
            }
            for identity in self.identities
        ]

    def export_resist_info_data(self, filename="ResistInfos.csv"):
        """ResistInfo 데이터를 CSV로 내보내는 메서드"""
        resist_infos_fieldnames = ["Identity_ID", "Resist_Hit", "Resist_Penetrate", "Resist_Slash"]
        resist_infos_data = self.get_resist_info_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, resist_infos_fieldnames, resist_infos_data)  # CSV로 내보내기

    def export_uptie_stats_data(self, filename="Uptiestats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        uptie_stats_fieldnames = ["Identity_ID", "Uptie", "Speed"]
        uptie_stats_data = self.get_uptie_stats_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, uptie_stats_fieldnames, uptie_stats_data)  # CSV로 내보내기

    def get_stagger_threshold_data(self):
        """
        모든 Identity의 stagger_threshold 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "First_Stagger": identity.stagger_threshold.first_stagger,
                "Second_Stagger": identity.stagger_threshold.second_stagger,
                "Third_Stagger": identity.stagger_threshold.third_stagger,
            }
            for identity in self.identities
        ]

    def export_stagger_threshold_data(self, filename="StaggerThresholds.csv"):
        """StaggerThresholds 데이터를 CSV로 내보내는 메서드"""
        stagger_thresholds_fieldnames = ["Identity_ID", "First_Stagger", "Second_Stagger", "Third_Stagger"]
        stagger_thresholds_data = self.get_stagger_threshold_data()  # stagger_thresholds 데이터 가져오기
        self.export_to_csv(filename, stagger_thresholds_fieldnames, stagger_thresholds_data)  # CSV로 내보내기

    def get_skillset_data(self):
        """모든 Identity의 skillset 데이터를 export_to_csv을 포맷으로 변환"""
        return [
            {
                "Identity_ID": identity.identity_id,
                "Skill_Code": skill.skill_code,
                "Skill_Type": skill.skill_type,
                "Sin_Affinity": skill.sin_affinity,
                "Skill_Name": skill.skill_name,
                "Coin_Count": skill.coin_count,
                "Init_Level": skill.init_level,
                "Init_Remain": skill.init_remain,
                "Atk_Weight": skill.atk_weight,
                "Uptie_Level": uptie_info.uptie_level,
                "Skill_Power": uptie_info.skill_power,
                "Coin_Power": uptie_info.coin_power,
                "Effect_Data": json.dumps(uptie_info.skill_effect)
            }
            for identity in self.identities
            for skill_list in [identity.skillset.skill_1_details, identity.skillset.skill_2_details, identity.skillset.skill_3_details, identity.skillset.defense_skill_details]
            for skill in skill_list
            for uptie_info in skill.uptie_info
        ]

    def export_identity_skillset_data(self, filename="IdentitiesSkills.csv"):
        """IdentitiesSkills 데이터를 CSV로 내보내는 메서드"""
        identities_skills_fieldnames = ["Identity_ID", "Skill_Code", "Skill_Type", "Sin_Affinity", "Skill_Name", "Coin_Count", "Init_Level", "Init_Remain", "Atk_Weight", "Uptie_Level", "Skill_Power", "Coin_Power", "Effect_Data"]
        identities_skills_data = self.get_skillset_data()
        self.export_to_csv(filename, identities_skills_fieldnames, identities_skills_data)
            


async def main():
    data_container = DataContainer()
    scraper = Scraper(container=data_container, base_url="https://limbus.kusoge.xyz", img_url="https://img.kusoge.xyz/limbus/img")
    await scraper.scrape()

await main() 

Bloodfeast
[]
{'keywords': ['Bloodfeast', 'Bleed'], 'description': ['If this unit is on field , or is one of the units that can appear on this Stage , increase keyword:Bloodfeast value by the amount of keyword:Bleed damage collectively received by every unit . When this unit enters the field , the sleeping blood drenching the battlefield will fill the surface .']}
Hardblood Thorn
['Lust x3 Owned', 'Envy x2 Owned']
{'keywords': ['Bleed', 'Bloodfeast'], 'description': ['When an other ally takes keyword:Bleed damage or consumes keyword:Bloodfeast , gain 1 blooming[thornyfall_panic RodionFirst ] ( 3 times per turn ) Heal HP on self by 20 % of the damage dealt with base Skills ( max 10 per Skill )']}
"Blossom with Blood..."
['Lust x3 Owned', 'Envy x3 Owned']
{'keywords': ['Bleed'], 'description': ["To 1 ally with the highest keyword:Bleed Potency at Turn End : reduce the ally ' s Potency by 6 max , and apply ( Potency reduced / 2 ) blooming[thornyfall_panic RodionFirst ] ( rounded down )"]}

In [70]:
import os
import csv
import re
import json
import numpy as np
import pandas as pd
from playwright.async_api import async_playwright
from tabulate import tabulate

# Sin Color 이름 변환을 위한 매핑
resource_mapping = {
    "AZURE": "Gloom",
    "VIOLET": "Envy",
    "AMBER": "Sloth",
    "SHAMROCK": "Gluttony",
    "CRIMSON": "Wrath",
    "SCARLET": "Lust",
    "INDIGO": "Pride"
}

def convert_resource(resource_name):
    """Sin Color 이름을 변환하는 함수. 매핑되지 않은 경우 원래 이름을 반환."""
    return resource_mapping.get(resource_name, resource_name)

def find_sin_for_color(url):
    # URL에 color_name을 추출
    color_name = url.split('/')[-1].split('.')[0].replace("attr_", "")
    # 해당 색상이 resource_mapping에 있는 확인
    for color, sin in resource_mapping.items():
        if color_name in color:
            return sin
    return None


class Scraper:
    """웹 페이지 크롤링을 위한 클래스"""
    def __init__(self, container, base_url, img_url):
        self.container = container
        self.base_url = base_url
        self.img_url = img_url
        self.uptie_level = 1

    async def handle_request(self, route, request):
        """이미지와 폰트 리소스 로드를 차단."""
        if request.resource_type =="image":
            #print(f"{request.url}")
            img_url = request.url
            if  re.match(f"{self.img_url}/ui/attr.*\\.png", img_url):
                await route.continue_()
            else:
                await route.fulfill(
                    status=200,
                    content_type="image/png",
                    body=b""
                )
        elif request.resource_type == "font":
            await route.abort()
        else:
            await route.continue_()
            
    async def scrape(self):
        """크롤링 메인 함수"""
        async with async_playwright() as p:
            browser = await p.chromium.launch(headless=True)
            page = await browser.new_page()
            #await page.evaluate('window.scrollTo(0, document.body.scrollHeight)')
            await page.route("**/*", self.handle_request)
            # Step 1: /identity 페이지에서 기본 정보 크롤링
            #await self.crawl_identity_page(page)

            # Step 2: /identity/{identity_id} 상세 페이지 크롤링
            #identity_ids_to_load = ["10104", "10310", "10710","10808", "10911", "11005"]
            #identity_ids_to_load = ["10104", "10911"]#,"10310", "10710","10808", "11005", "11210"]
            #identity_ids_to_load = ["10911","10310"]#, "10710","10808", "11005", "11210"]
            identity_ids_to_load = ["10911"]#, "10611"]#,"10711"]#, "10710","10808", "11005", "11210"]
            self.container.load_identities_by_id('./csv/identities.csv', identity_ids_to_load)
            await self.crawl_identity_details_for_all(page)
            #self.container.display_identities()
            #self.container.export_identities_data("identities.csv")
            #self.container.export_level_stats_data("Levelstats.csv")
            #self.container.export_uptie_stats_data("Uptiestats.csv")
            #self.container.export_resist_info_data("ResistInfos.csv")
            #self.container.export_stagger_threshold_data("StaggerThresholds.csv")
            #self.container.export_identity_skillset_data("IdentitiesSkills.csv")
            page.set_default_timeout(10000)
            #await page.wait_for_timeout(100000)
            await browser.close()

    async def get_identity_id_from_row(self, row):
        """테이블 행(row)에서 인격 ID를 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_url = await identity_cell.get_attribute('href')
            identity_id = identity_url.split('/identity/')[-1]
            return identity_id
        return "Unknown"

    async def get_identity_name_from_row(self, row):
        """테이블 행(row)에서 인격 이름을 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_name = (await identity_cell.inner_text()).replace("\n", " ").strip()
            return identity_name
        return "Unknown"
        
    async def crawl_identity_page(self, page):
        """/identity 페이지로 이동하여 인격 기본 정보를 추출."""
        await page.goto(self.base_url + "/identity")
        await self.extract_identity_page(page)

    async def extract_identity_page(self, page):
        """/identity 페이지의 모든 테이블 행에서 인격 정보를 추출."""
        identity_rows = await page.query_selector_all('tr')
        if len(identity_rows) > 1:
            for idx, identity_row in enumerate(identity_rows):
                if idx == 0:
                    continue # 0번째요소이름을 보여주는 열은 건너뜀
                await self.scrape_identity_name_and_identity_id(identity_row)
                await self.extract_rarity(identity_row)
                for i in range(1, 4):  # 스킬1~3 정보 추출
                    await self.extract_atk_and_resource(identity_row, i)
                await self.extract_defense_and_resource(identity_row)

    async def scrape_identity_name_and_identity_id(self, row):
        """인격 번호와 이름을 추출하여 DataContainer에 저장."""
        try:
            identity_id = await self.get_identity_id_from_row(row)
            identity_name = await self.get_identity_name_from_row(row)
            identity = Identity(identity_id)
            self.container.add_identity(identity)
            self.container.update_identity_name(identity_id, identity_name)
        except Exception as e:
            print(f"Error extracting identity name and identity id: {e}")

    async def get_element_attribute(self, element, selector, attribute):
        """요소에서 지정된 속성(attribute)을 추출."""
        try:
            target_element = await element.query_selector(selector)
            if target_element:
                return await target_element.get_attribute(attribute)
        except Exception as e:
            print(f"Error getting element attribute: {e}")
        return None
    
    async def extract_rarity(self, row):
        """테이블 행(row)에서 희귀도를 추출하여 업데이트."""
        try:
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
                sibling = await parent.evaluate_handle('node => node.nextElementSibling')
                if sibling:
                    rarity_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/rarity_"]', 'src')
                    if rarity_img_src:
                        rarity_level = rarity_img_src.split('_')[-1].split('.')[0]  # rarity_1.png -> 1
                        identity_id = await self.get_identity_id_from_row(row)
                        self.container.update_identity_rarity(identity_id, rarity_level)
                    else:
                        print("Rarity image not found in the next sibling TD.")
        except Exception as e: 
            print(f"Error extracting rarity: {e}")

        return "Unknown"

    async def extract_atk_and_resource(self, row, skill_number):
        """ 공격 타입(ATK Type)과 자원(Attribute) 추출 함수 - 스킬 번호에 따라 처리 """
        try:
            #print(f"Attempting to locate skill{skill_number} type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 해당 스킬 번호에 맞는 형제로 이동
                sibling = parent
                for _ in range(skill_number + 1):  # 스킬 번호에 맞게 이동
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                atk_type, resource = "Unknown", "Unknown"
        
                if sibling:
                    # 공격 타입 이미지 (atk_type) 찾기
                    atk_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/atk_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if atk_type_img_src:
                        atk_type = atk_type_img_src.split('_')[-1].split('.')[0]  # atk_type_SLASH.png -> SLASH
                        #print(f"Extracted attack type: {atk_type}")
                        self.container.update_skill(identity_id, skill_number, atk_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted resource: {resource}")
                        
                        # 리소스를 변환
                        resource = convert_resource(resource)
                        #print(f"Converted resource: {resource}")
                        self.container.update_sin_affinity(identity_id, skill_number, resource)
                        
        except Exception as e:
            print(f"Error extracting skill{skill_number} type and resource: {e}")

    async def extract_defense_and_resource(self, row):
        """ 방어 타입(Def Type)과 자원(Attribute) 추출 함수 """
        try:
            #print("Attempting to locate defense type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 다섯 번째 형제 td로 이동 (방어 관련 정보가 있는 <td>)
                sibling = parent
                for _ in range(5):  # 방어 정보는 5번째 형제에 있음
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                def_type, def_resource = "Unknown", "Unknown"
        
                if sibling:
                    # 방어 타입 이미지 (def_type) 찾기
                    def_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/def_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if def_type_img_src:
                        def_type = def_type_img_src.split('_')[-1].split('.')[0]  # def_type_GUARD.png -> GUARD
                        #print(f"Extracted defense type: {def_type}")
                        self.container.update_skill(identity_id, 'defense', def_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        def_resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted defense resource: {def_resource}")
                        
                        # 방어 리소스를 변환
                        def_resource = convert_resource(def_resource)
                        #print(f"Converted defense resource: {def_resource}")
                        self.container.update_sin_affinity(identity_id, 'defense', def_resource)
 
        except Exception as e:
            print(f"Error extracting defense type and resource: {e}")

    async def crawl_identity_details_for_all(self, page):
        """모든 인격의 세부 정보를 크롤링."""
        for identity in self.container.identities:
            await page.wait_for_load_state("networkidle")
            await page.goto(self.base_url + "/identity/"+identity.identity_id)
            """ 레벨과 동기화와 관련 없는 참관타 저항은 밑에서 변경하는 것과 연관성이 적기 때문에 위에서 처리한다.
            print(f"Identity_id: {identity.identity_id}")
            results = []
            resist_type_imgs = [f"{self.img_url}/ui/atk_type_HIT.png",
                                f"{self.img_url}/ui/atk_type_PENETRATE.png",
                                f"{self.img_url}/ui/atk_type_SLASH.png"]
            for resist_type_img in resist_type_imgs:
                result = await self.extract_resist_type(page, resist_type_img, 1)
                results.append(result)
            self.container.add_resist_info(identity.identity_id, results[0], results[1], results[2])
            """
            """
            # Stagger Threshold 값 추출
            stagger_threshold = await self.extract_stagger_threshold_values(page)
            self.container.add_stagger_threshold(identity.identity_id, stagger_threshold[0], stagger_threshold[1], stagger_threshold[2])
            """
            await self.initialize_uptie_and_level(page)
            """ 레벨에 따른 level, hp, defense 다르게 되어 있는 것을 출력 확인 객체로 들어가서 csv로 옮기는 과정까지 진행
            for i in range(1, identity.level_max+1):
                await self.move_slider_to_value(page,i)
                await self.extract_level_status(page, identity.identity_id, i)
            """
            """ 동기화에 따른 speed 다르게 되어 있는 것을 출력 확인 객체로 들어가서 csv로 옮기는 과정까지 진행
            for i in range(1, identity.uptie_max+1):
                await self.uptie_click(page,i)
                await self.extract_uptie_status(page, identity.identity_id, i)
            """
            """
                인격에 대한 skill 정보를 가져오는 메소드 위에서 출력하는 csv파일로 인격이 가지고 있는 스킬 정보를 내보낸다.
            await self.crawl_skills(page, identity.identity_id)
            """
            await self.crawl_influence_behavior(page, identity.identity_id)
            
    async def initialize_uptie_and_level(self, page):
        await self.uptie_click(page, 1)
        await self.move_slider_to_value(page, 1)
        self.uptie_level = 1
        #return 1
    
    async def uptie_click(self, page, phase):
        fab_button_selector = ".q-btn.q-btn--fab"
        await page.wait_for_selector(fab_button_selector)
        await page.click(fab_button_selector)
        #print(f"[{setup_uptie_button}] 상위 동기화 버튼 클릭 완료")
        setup_uptie_button = f'a:has(img[src="{self.img_url}/ui/uptie_{phase}.png"])'
        await page.wait_for_selector(setup_uptie_button)
        await page.click(setup_uptie_button)
        #print(f"{phase} 동기화 버튼 클릭 완료")

    async def move_slider_to_value(self, page, target_value):
        """
        슬라이더를 주어진 값으로 이동시킵니다.
        target_value는 슬라이더의 최소값부터 최대값 사이여야 합니다.
        """
        slider_selector = ".q-slider"
        # 슬라이더의 범위 확인
        min_value = int(await page.get_attribute(slider_selector, "aria-valuemin"))
        max_value = int(await page.get_attribute(slider_selector, "aria-valuemax"))
    
        if target_value < min_value or target_value > max_value:
            raise ValueError(f"Target value {target_value} is out of bounds ({min_value}, {max_value})")
    
        # 슬라이더 thumb 요소와 위치 가져오기
        slider_thumb = await page.query_selector(f"{slider_selector} .q-slider__thumb")
        thumb_box = await slider_thumb.bounding_box()
        
        # 슬라이더의 현재 위치 및 목표 위치 계산
        track = await page.query_selector(f"{slider_selector} .q-slider__track")
        track_box = await track.bounding_box()
        
        # 목표 x 위치 계산 (슬라이더 트랙 내에서 비율 기반)
        target_percentage = (target_value - min_value) / (max_value - min_value)
        target_x = track_box["x"] + target_percentage * track_box["width"]
        current_x = thumb_box["x"] + thumb_box["width"] / 2
        y = thumb_box["y"] + thumb_box["height"] / 2
    
        # 슬라이더를 드래그하여 목표 위치로 이동
        await page.mouse.move(current_x, y)
        await page.mouse.down()
        await page.mouse.move(target_x, y, steps=10)  # 부드럽게 이동
        await page.mouse.up()
    
        #print(f"슬라이더를 {target_value}로 이동 완료.")
    
        # 슬라이더 값 확인 및 강제 이벤트 트리거
        await page.evaluate(
            f"""
            (slider) => {{
                slider.dispatchEvent(new Event('input'));  // oninput 이벤트 트리거
                slider.dispatchEvent(new Event('change'));  // onchange 이벤트 트리거
            }}
            """,
            await page.query_selector(slider_selector)
        )
    
        # 이동 후 값 확인
        current_value = await page.get_attribute(slider_selector, "aria-valuenow")
        if current_value != str(target_value):
            print(f"경고: 슬라이더 값이 예상과 다릅니다. 현재 값: {current_value}, 예상 값: {target_value}")

    async def get_second_status_element(self, page):
        """
        두 번째 'Status' 요소에 접근하여 반환하는 메소드
        """
        status_elements = await page.query_selector_all("text=Status")
        if len(status_elements) > 1:
            second_status_element = status_elements[1]
            #print("두 번째 'Status' 요소에 접근합니다.")
            return second_status_element
        else:
            #print("두 번째 'Status' 요소가 존재하지 않습니다.")
            return None

    async def extract_level_status(self, page, identity_id, level):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 레벨에 따라 HP와 Defense가 변한다.
        hp_value = await self.extract_hp(second_status_element)
        defense_value = await self.extract_defense(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Level={level}, HP={hp_value}, Defense={defense_value}")
        self.container.add_level_stat(identity_id,level, hp_value, defense_value)
    
    async def extract_hp(self, status_element):
        # HP 추출
        hp_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].children[0].nextElementSibling.textContent.trim()"
        )
        return hp_value
    
    async def extract_defense(self, status_element):
        # Defense 추출
        defense_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return defense_value

    async def extract_uptie_status(self, page, identity_id, uptie):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 uptie에 따라 speed는 변한다.
        speed_value = await self.extract_speed(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Uptie={uptie}, Speed={speed_value}")
        self.container.add_uptie_stat(identity_id,uptie, speed_value)

    async def extract_speed(self, status_element):
        # Speed 추출
        speed_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return speed_value

    async def extract_resist_type(self, page, image_src, depth):
        # 이미지 요소 찾기
        #print(image_src)
        image = await page.query_selector(f'img[src="{image_src}"]')
        #print(image)
        if not image:
            return "Unknown"
    
        # 이미지의 부모 요소를 찾아서 올라감
        parent = image
        for _ in range(depth):
            parent = await parent.evaluate_handle('node => node.parentElement')
            if not parent:
                return "Unknown"
    
        # 부모의 형제 요소 찾기
        sibling = await parent.evaluate_handle('node => node.nextElementSibling')
        
        if sibling:
            value = await sibling.evaluate('node => node.textContent')
            if value:
                return value.strip()
        
        return "Unknown"

    async def extract_stagger_threshold_values(self, page):
        # 'div.row.center.flex-center.q-pt-xs' 요소들 찾기
        elements = await page.query_selector_all('div.row.center.flex-center.q-pt-xs')
        
        # 각 요소의 텍스트 내용 출력하여 'Stagger Threshold' 요소를 찾기
        for element in elements:
            text = await element.text_content()
            
            if text and "Stagger Threshold" in text:
                # 'Stagger Threshold'가 있는 요소를 찾았다면, 그 이후 형제 요소들에서 값을 추출
                parent = await element.evaluate_handle('node => node.parentElement')
                sibling_elements = await parent.query_selector_all('div.row.justify-evenly div')  # Threshold 값이 들어있는 div들
                
                stagger_values = []
                for sibling in sibling_elements:
                    text_content = await sibling.text_content()
                    match = re.search(r'(\d+%)', text_content)
                    if match:
                        stagger_values.append(match.group(1))  # 퍼센트 값만 리스트에 추가
                while len(stagger_values) < 3:
                    stagger_values.append(np.nan)
                #print(stagger_values)
                return stagger_values  # 추출된 값(또는 np.nan 포함)을 반환
        
        return [np.nan, np.nan, np.nan]  # 만약 'Stagger Threshold'를 찾지 못한 경우 기본값 반환

    async def crawl_skills(self, page, identity_id):
        await self.crawl_skill(page, identity_id, "skill 1")
        await self.crawl_skill(page, identity_id, "skill 2")
        await self.crawl_skill(page, identity_id, "skill 3")
        await self.crawl_skill(page, identity_id, "defense")


    async def crawl_skill(self, page, identity_id, skill_type):
        skills_elements = await page.query_selector_all('div.skilltag')
        for skill_element in skills_elements:
            skill_text = await skill_element.text_content()
            if skill_text.strip().lower() == skill_type:
                await self.process_skill(skill_element, page, identity_id, skill_type)
        
    async def process_skill(self, skill_element, page, identity_id, skill_type):
        # skill_type이 "skill 3"일 경우 uptie_value가 3 이상일 때만 정보를 저장
        for uptie_value in range(1, 5):  # Uptie levels 1 to 4
            # "skill 3"은 uptie_value가 3 이상일 때만 처리
            if skill_type == "skill 3" and uptie_value < 3:
                continue  # skip if uptie_value is less than 3 for skill 3
    
            # uptie_level에 맞게 클릭
            await self.uptie_click(page, uptie_value)
            if skill_type != "skill 3" and uptie_value > 1:
                # Uptie 정보 갱신
                await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
                continue
            elif skill_type == "skill 3" and uptie_value > 3:
                # Uptie 정보 갱신
                await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
                continue
            # 스킬의 세부 정보를 추출
            skill_details = await self.extract_skill_details(skill_element, skill_type)
            skill_name = skill_details["skill_name"]
    
            # 스킬 정보를 저장
            self.container.add_skill(
                identity_id, skill_type, 
                skill_name, skill_details["coin_count"],
                skill_details["init_level"], skill_details["init_remain"], 
                skill_details["atk_weight"]
            )
            print(f"Added skill: {skill_type}:{skill_name}")
    
            # Uptie 정보 갱신
            await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
    
    async def extract_skill_details(self, skill_element, skill_type):
        # 스킬 세부 정보를 추출하는 함수
        skill_name = await self.skill_name(skill_element)
        coin_count = await self.coin_count(skill_element, ".col-auto > .column > .row")
        init_level = await self.init_level(skill_element)
        init_remain = await self.init_remain(skill_element, skill_type, "div.row.justify-start > div.attr.flex.flex-center > div.q-px-sm")
        atk_weight = await self.atk_weight(skill_element, skill_type)
        return {
            "skill_name": skill_name,
            "coin_count": coin_count,
            "init_level": init_level,
            "init_remain": init_remain,
            "atk_weight": atk_weight
        }
    
    async def update_uptie_info(self, page, skill_element, identity_id, skill_name, skill_type, uptie_value):
        # Uptie 정보를 추출하고 업데이트하는 함수
        uptie_info = await self.extract_uptie_info(skill_element)
        effect_data = await self.skill_effects(skill_element)
        self.container.add_skill_uptie_info(
            identity_id, skill_name, 
            uptie_value, uptie_info["skill_power"], uptie_info["coin_power"], 
            effect_data.get("effectsByCondition")
        )
        print(
            f"Updated Uptie for {skill_name} (Uptie Level: {uptie_value}): "
            f"Skill Power: {uptie_info['skill_power']}, Coin Power: {uptie_info['coin_power']} "
            f"Skill Effect: {effect_data.get("effectsByCondition")}"
        )

    async def extract_uptie_info(self, skill_element):
        skill_data = await self.skill_and_coin_power(skill_element)
        return {
            "skill_power": skill_data.get("skillPower"),
            "coin_power": skill_data.get("coinPower")
        }
        
    async def crawl_influence_behavior(self, page, identity_id):
        influence_behaviors = await page.query_selector_all('div[align="right"]')
        # 텍스트를 기준으 추출하고 메소드로 파라미터 들어가 정보를 받는다.
        for influence_behavior in influence_behaviors:
            active_type = await influence_behavior.inner_text()
            active_type = active_type.strip()

            if active_type == "PASSIVE":
                """
                # PASSIVE에 대한 처리
                print(f"Processing PASSIVE logic for: {active_type}")
                # CALL method for PASSIVE
                """
                passive_data = await self.passive_effects(influence_behavior)
            elif active_type == "SUPPORT PASSIVE":
                """
                # SUPPORT PASSIVE에 대한 처리
                print(f"Processing SUPPORT PASSIVE logic for: {active_type}")
                # CALL method for SUPPORT PASSIVE
                """
                support_passive_data = await self.support_passive_effects(influence_behavior)
            elif active_type == "PANIC TYPE":
                """
                # PANIC TYPE에 대한 처리
                print(f"Processing PANIC TYPE logic for:{active_type}")
                # CALL method for PANIC TYPE
                """
                panic_type_data = await self.panic_effects(influence_behavior)

    async def skill_and_coin_power(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                // 조부모로 이동
                const grandparent = node.parentElement.parentElement;
                // 이전 형제 탐색
                const previousSibling = grandparent.previousElementSibling;
                if (!previousSibling) return {{ skillPower: null, coinPower: null }}; // 이전 형제가 없으면 기본값 반환
                // 이전 형제 내부의 .row.justify-around > div > div 탐색
                const targetDivs = previousSibling.querySelectorAll(".row.justify-around > div");
                if (targetDivs.length < 3) return {{ skillPower: null, coinPower: null }}; // 배열 길이 확인
                // 1st div: 스킬 위력
                const skillPower = targetDivs[0]?.querySelector('div')?.textContent?.trim() || null; 
                // 3rd div: 코인 파워 값 (3번째 div에서 텍스트 추출)
                const coinPower = targetDivs[2]?.textContent?.trim() || null;
                return {{ skillPower, coinPower }};
            }}
        """)


    async def coin_count(self, skill_element, hierarchy_selector):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector('{hierarchy_selector}');
                if (!targetElement) return 0;
                return targetElement.childElementCount;
            }}
        """)

    async def init_remain(self, skill_element, skill_type, hierarchy_selector):
        if skill_type == "defense":
            return -1
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector('{hierarchy_selector}');
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def skill_name(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector(".block");
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def init_level(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector("div.text-bold.text-h6.h6.q-pl-xs.q-pr-md");
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def atk_weight(self, skill_element, skill_type):
        if skill_type == "defense":
            return 0
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector("div.text-bold.text-h6.h6.q-pl-xs.q-pr-md");
                const doubleNextSibling = targetElement.nextElementSibling.nextElementSibling;
                if (!doubleNextSibling) return 0;
                return doubleNextSibling.textContent?.trim();
            }}
        """)

    async def skill_effects(self, skill_element):
        return await skill_element.evaluate("""
            node => {
                const parent = node.parentElement;
                const doubleNextSibling = parent.nextElementSibling?.nextElementSibling;
                if (!doubleNextSibling) return null;
    
                // doubleNextSibling에서 첫 번째 div.col 요소 찾기
                const textNode = doubleNextSibling.querySelector("div.col");
                if (!textNode) return null;
    
                const results = {
                    keywords: [], // 키워드 저장
                    conditions: [], // 조건 저장
                    effectsByCondition: {} // 조건별 효과
                };
    
                // 1. 조건 추출 (대괄호 텍스트)
                const conditionElements = textNode.querySelectorAll("span");
                conditionElements.forEach(span => {
                    let text = span.textContent?.trim();
                    if (text?.startsWith("[") && text?.endsWith("]")) {
                        // 부모의 전형제 노드 탐색
                        const sibling = span.parentElement.previousElementSibling;
                        if (sibling) {
                            const emblemImage = sibling.querySelector("img.emblem");
                            console.log("Sibling element found:", sibling); // 디버깅용
                            if (emblemImage) {
                                const src = emblemImage.getAttribute("src");
                                console.log("Image source:", src); // 디버깅용
                                const match = src.match(/ui_num_(\\d+)\\.png$/); // 정규식 수정
                                if (match) {
                                    const number = match[1]; // 숫자 추출
                                    text = `[Coin ${number} ${text.replace("[", "").replace("]", "")}]`;
                                    console.log("Updated condition:", text); // 디버깅용
                                }
                            }
                        }
                        results.conditions.push(text);
                        results.effectsByCondition[text] = []; // 조건별 효과를 초기화
                    }
                });
    
                // 2. 키워드 추출
                const keywordElements = textNode.querySelectorAll('a[href^="/en/keyword/"]');
                keywordElements.forEach(a => {
                    const span = a.querySelector('span.q-btn__content.text-center.col.items-center.q-anchor--skip.justify-center.row>span');
                    if (span) {
                        const keywordText = span.textContent.trim();
                        if (keywordText && !results.keywords.includes(keywordText)) {
                            results.keywords.push(keywordText); // 키워드 저장
                        }
                    }
                });
    
                // 3. 효과 추출
                const effectElements = textNode.querySelectorAll("span");
                let currentCondition = null;
                let currentEffect = []; // 초기화
                effectElements.forEach(span => {
                    let text = span.textContent?.trim();
                    if (!text) return;
    
                    // 조건 발견 시 currentCondition 변경
                    if (text.startsWith("[") && text.endsWith("]")) {
                        // 이전 조건의 효과 저장
                        if (currentCondition && currentEffect.length > 0) {
                            // 부모의 전형제 노드 탐색
                            const sibling = span.parentElement.previousElementSibling;
                            if (sibling) {
                                const emblemImage = sibling.querySelector("img.emblem");
                                if (emblemImage) {
                                    const src = emblemImage.getAttribute("src");
                                    const match = src.match(/ui_num_(\\d+)\\.png$/); // 정규식 수정
                                    if (match) {
                                        const number = match[1]; // 숫자 추출
                                        text = `[Coin ${number} ${text.replace("[", "").replace("]", "")}]`;
                                    }
                                }
                            }
                            results.effectsByCondition[currentCondition].push(currentEffect.join(' '));
                            currentEffect = []; // 초기화
                        }
                        currentCondition = text;
                        return;
                    }
    
                    // 현재 조건의 효과 처리
                    if (currentCondition) {
                        if (results.keywords.includes(text)) {
                            const keywordEffect = 'keyword:' + text;
                            if (!currentEffect.includes(keywordEffect)) {
                                currentEffect.push(keywordEffect);
                            }
                        } else {
                            currentEffect.push(text);
                        }
                    }
                });
    
                // 마지막 효과 저장
                if (currentCondition && currentEffect.length > 0) {
                    results.effectsByCondition[currentCondition].push(currentEffect.join(' '));
                }
    
                return results;
            }
        """)

    async def passive_effects(self, passive_element):
        passive_name = await self.behavior_name(passive_element)
        passive_conditions = await self.behavior_condition(passive_element)
        passive_description = await self.behavior_description(passive_element)
        print(passive_name)
        print(passive_conditions)
        print(passive_description)
    async def support_passive_effects(self, support_passive_element):
        support_passive_name = await self.behavior_name(support_passive_element)
        support_passive_conditions = await self.behavior_condition(support_passive_element)
        support_passive_description = await self.behavior_description(support_passive_element)
        print(support_passive_name)
        print(support_passive_conditions)
        print(support_passive_description)
    async def panic_effects(self, panic_element):
        panic_type = await self.behavior_name(panic_element)
        panic_conditions = await self.behavior_condition(panic_element)
        panic_description = await self.panic_description(panic_element)
        print(panic_type)
        print(panic_conditions)
        print(panic_description)

    async def navigate_element_by_steps(self, element, parent_steps=0, prev_sibling_steps=0, next_sibling_steps=0):
        """
        주어진 `element`에서 부모, 이전 형제, 또는 다음 형제로 지정된 횟수만큼 이동하여 ElementHandle 반환.
        
        Args:
            element (ElementHandle): 탐색 기준이 되는 요소.
            parent_steps (int): 부모로 이동할 횟수 (0이면 이동하지 않음).
            prev_sibling_steps (int): 이전 형제로 이동할 횟수 (0이면 이동하지 않음).
            next_sibling_steps (int): 다음 형제로 이동할 횟수 (0이면 이동하지 않음).
        
        Returns:
            ElementHandle or None: 탐색된 요소를 반환하거나 없으면 None.
        """
        try:
            current_element = element
    
            # 부모로 이동
            for _ in range(parent_steps):
                current_element = await current_element.evaluate_handle('el => el.parentElement')
                if not current_element:
                    print("Parent element not found during navigation.")
                    return None
    
            # 이전 형제로 이동
            for _ in range(prev_sibling_steps):
                current_element = await current_element.evaluate_handle('el => el.previousElementSibling')
                if not current_element:
                    print("Previous sibling element not found during navigation.")
                    return None
    
            # 다음 형제로 이동
            for _ in range(next_sibling_steps):
                current_element = await current_element.evaluate_handle('el => el.nextElementSibling')
                if not current_element:
                    print("Next sibling element not found during navigation.")
                    return None
    
            return current_element
    
        except Exception as e:
            print(f"Error navigating elements: {e}")
            return None

    async def get_last_sibling(self, element):
        try:
            # 같은 부 아 마지막 형제를 가져오는 JavaScript 실행
            last_sibling = await element.evaluate_handle('el => el.parentElement.lastElementChild')
            return last_sibling
        except Exception as e:
            print(f"Error getting last sibling: {e}")
            return None

    async def behavior_name(self, element):
        parent_prev_sibling = await self.navigate_element_by_steps(element, parent_steps=1, prev_sibling_steps=1, next_sibling_steps=0)
        passive_name_element = await parent_prev_sibling.query_selector('span.block')
        if passive_name_element:
            passive_name = await passive_name_element.inner_text()
            return passive_name
        else:
            #print("Passive name element not found")
            return None

    async def behavior_condition(self, element):
        try:
            conditions = []
            parent_prev_sibling = await self.navigate_element_by_steps(element, parent_steps=1, prev_sibling_steps=1, next_sibling_steps=0)
            img_elements = await parent_prev_sibling.query_selector_all('img')
            if not img_elements:
                #print("No <img> elements found.")
                return conditions
    
            for img_element in img_elements:
                # 각 이미지의 src 속성 가져오기
                img_src = await img_element.get_attribute('src')
                #print(img_src)
    
                sin_name = None
                for color_key in resource_mapping.keys():
                    if f"{self.img_url}/ui/attr_{color_key}.png" in img_src:
                        sin_name = resource_mapping[color_key]
                        break
    
                if not sin_name:
                    #print(f"No matching Sin Color found for {img_src}.")
                    continue  # No Sin Color found, skip this img_element
    
                # 해당 이미지와 관련된 조건 텍스트 추출
                # 두 번째 부모 요소에서 시작하여 다음 형제를 찾아 조건 텍스트를 추출
                condition_element = await self.navigate_element_by_steps(img_element, parent_steps=3, prev_sibling_steps=0, next_sibling_steps=0)
                if condition_element:
                    # condition_element가 None이 아닌 경우에만 진행
                    condition_text_element = await condition_element.query_selector('.q-pl-xs')
                    if condition_text_element:
                        condition_text = await condition_text_element.inner_text()
                        condition_text = condition_text.strip()
                    else:
                        print(f"No condition text found for {img_src}.")
                        condition_text = ""  # condition_text_element가 None이면 빈 문자열로 처리
                else:
                    print(f"No condition element found for {img_src}.")
                    condition_text = ""  # condition_element가 None이면 빈 문자열로 처리
    
                # 결과를 배열에 추가
                conditions.append(f"{sin_name} {condition_text}")
            return conditions
        except Exception as e:
            print(f"Error extracting conditions: {e}")
            return 
            
    async def behavior_description(self, element):
        parent_prev_sibling = await self.navigate_element_by_steps(element, parent_steps=1, prev_sibling_steps=1, next_sibling_steps=0)
        return await parent_prev_sibling.evaluate("""
                node => {
                    const behavior_title_node = node.querySelector('.q-pb-sm')
                    const behavior_title_parent = behavior_title_node.parentElement
                    const behavior_description = behavior_title_parent.lastElementChild
                    if (!behavior_description) return null;

                    const results = {
                        keywords: [], // 키워드 저장
                        description: [] // 설명 저장
                    };
                    // 1. 키워드 추출
                    const keywordElements = behavior_description.querySelectorAll('a[href^="/en/keyword/"]');
                    if (!keywordElements) return null;
                    keywordElements.forEach(a => {
                        const span = a.querySelector('span.q-btn__content.text-center.col.items-center.q-anchor--skip.justify-center.row>span');
                        if (span) {
                            const keywordText = span.textContent.trim();
                            if (keywordText && !results.keywords.includes(keywordText)) {
                                results.keywords.push(keywordText); // 키워드 저장
                            }
                        }
                    });
        
                    // 2. 설명 추출
                    const descriptionElements = behavior_description.querySelectorAll("span");
                    let currentEffect = []; // 초기화
                    descriptionElements.forEach(span => {
                        let text = span.textContent?.trim();
                        if (!text) return;
                        // 공백과 기호 처리
                        text = text.replace(/\\s*([.,:;!?()\\[\\]])\\s*/g, '$1'); // 기호 앞뒤 공백 제거
                        text = text.replace(/\\s+/g, ' '); // 다중 공백을 단일 공백으로 변환
                        if (results.keywords.includes(text)) {
                            const keywordEffect = 'keyword:' + text;
                            if (!currentEffect.includes(keywordEffect)) {
                                currentEffect.push(keywordEffect);
                            }
                        } else {
                            currentEffect.push(text);
                        }
                    });
        
                    // 마지막 효과 저장
                    if (currentEffect.length > 0) {
                        results.description.push(currentEffect.join(' '));
                    }
        
                    return results;
                }
            """)

    async def panic_description(self, element):
        try:
            description_texts = []
            keywords = []
    
            # Navigate to the behavior description parent
            parent_prev_sibling = await self.navigate_element_by_steps(element, parent_steps=1, prev_sibling_steps=1, next_sibling_steps=0)
            behavior_title = await parent_prev_sibling.query_selector('.q-pb-sm')
            behavior_description_element = await self.get_last_sibling(behavior_title)
    
            # Fetch keyword elements
            keyword_elements = await behavior_description_element.query_selector_all('a[href^="/en/keyword/"]')
            for keyword_element in keyword_elements:
                keyword_span = await keyword_element.query_selector(
                    'span.q-btn__content.text-center.col.items-center.q-anchor--skip.justify-center.row > span'
                )
                if keyword_span:
                    keyword_text = await keyword_span.inner_text()
                    stripped_text = keyword_text.strip()
                    if stripped_text and stripped_text not in keywords:
                        keywords.append(stripped_text)
    
            # Fetch description elements
            descriptionElements = await behavior_description_element.query_selector_all("div")
            seen_texts = []
            # 수정된 부분: inner_text()의 원본을 출력해보고 strip()을 조정하여 문제를 파악합니다.
            # 키워드를 포함한 텍스트에서 마지막 문장이 누락되지 않도록 조정
            for descriptionElement in descriptionElements:
                description_text = await descriptionElement.inner_text()
                #print(f"Original text: '{description_text}'")  # 원본 텍스트 출력
                stripped_text = description_text.strip()  # 앞뒤 공백만 제거
                stripped_text = re.sub(r'\s+', ' ', stripped_text)
                description_texts.append(stripped_text)
            normalize_text = await self.combine_and_process_texts(description_texts)
            return normalize_text
        except Exception as e:
            print(f"Error extracting description: {e}")
            return

    async def combine_and_process_texts(self, texts):
        """
        배열 형태의 텍스트 데이터를 합치고 형식을 처리한 문자열을 반환합니다.
        Args:
            texts (list of list): 텍스트 배열 목록
        Returns:
            str: 형식이 처리된 최종 문자열
        """
        processed_sentences = []
        
        for text_array in texts:
            # 빈 문자열을 제외한 나머지 텍스트를 이어붙이기
            sentence = ''.join([word for word in text_array if word != ''])
            sentence = re.sub(r'\\](\\s*)', ']', sentence)
            processed_sentences.append(sentence)
    
        # 문장을 공백으로 연결하여 반환
        return ''.join(processed_sentences)


class Identity:
    """인격 정보를 저장하는 클래스"""
    def __init__(self, identity_id, name=None, rarity=None,\
                 skill_1_type=None, skill_1_sin_affinity=None,\
                 skill_2_type=None, skill_2_sin_affinity=None,\
                 skill_3_type=None, skill_3_sin_affinity=None,\
                 defense_type=None, defense_sin_affinity=None):
        self.identity_id = identity_id
        self.name = name
        self.rarity = rarity
        # 선언만 하는 부분 
        self.skill_1_type = skill_1_type 
        self.skill_1_sin_affinity = skill_1_sin_affinity 
        self.skill_2_type = skill_2_type 
        self.skill_2_sin_affinity = skill_2_sin_affinity 
        self.skill_3_type = skill_3_type 
        self.skill_3_sin_affinity = skill_3_sin_affinity 
        self.defense_type = defense_type 
        self.defense_sin_affinity = defense_sin_affinity
        self.level_max = 50
        self.uptie_max = 4
        self.level_stats = []  # LevelStat 객체 리스트
        self.uptie_stats = [] # UptieStat 객체 리스트
        self.resist_info = None
        self.stagger_threshold = None
        self.skillset = SkillSet()  # Skill정보를 담고 있는 skillset 객체 
        self.passive_infos = []  # 패시브 객체를 담고 있는 배열
        self.support_passive_infos = [] # 서포트 패시 객체를 담고 있는 배열
        self.panic_type = [] # 혼란상태 발생시 나타나는 효과를 담고 있는 객체 배열1

    def update_name(self, name):
        self.name = name

    def update_rarity(self, rarity):
        self.rarity = rarity

    def add_level_stat(self, level, hp, defense):
        self.level_stats.append(LevelStat(level, hp, defense))

    def add_uptie_stat(self, uptie, speed):
        self.uptie_stats.append(UptieStat(uptie, speed))

    def add_resist_info(self, resist_hit, resist_penetrate, resist_slash):
        self.resist_info = ResistInfo(resist_hit, resist_penetrate, resist_slash)

    def add_stagger_threshold(self, first_stagger, second_stagger, third_stagger):
        self.stagger_threshold = StaggerThreshold(first_stagger, second_stagger, third_stagger)

    def add_skill(self, skill_code, skill_name, coin_count, init_level, init_remain, atk_weight):
        self.skillset.add_skill(skill_code, Skill(skill_code, skill_name, coin_count, init_level, init_remain, atk_weight))
        
    def add_skill_uptie_info(self, skill_name, uptie_level, skill_power, coin_power, effect_by_condition):
        self.skillset.add_skill_uptie_info(skill_name, uptie_level, skill_power, coin_power, effect_by_condition)
        
    def update_details(self, page):
        # 세부 정보 업데이트 로직 (가정)
        pass
    #def init_uptie(self, page):
    #    init_uptie = 

# LevelStat 객체 정의
class LevelStat:
    def __init__(self, level, hp=None, defense=None):
        self.level = level
        self.hp = hp
        self.defense = defense
    def update_level(self, level):
        self.level = level
    def update_hp(self, hp):
        self.hp = hp
    def update_defense(self, defense):
        self.defense = defense

# UptieStat 객체 정의
class UptieStat:
    def __init__(self, uptie, speed=None):
        self.uptie = uptie
        self.speed = speed
    def update_speed(self, speed):
        self.speed = speed

# ResistInfo 객체 정의
class ResistInfo:
    def __init__(self, resist_hit=None, resist_penetrate=None, resist_slash=None):
        self.resist_hit = resist_hit
        self.resist_penetrate = resist_penetrate
        self.resist_slash = resist_slash

# ResistInfo 객체 정의
class StaggerThreshold:
    def __init__(self, first_stagger=None, second_stagger=None, third_stagger=None):
        self.first_stagger = first_stagger
        self.second_stagger = second_stagger
        self.third_stagger = third_stagger

class SkillSet:
    def __init__(self):

        self.skill_1_type = ""
        self.skill_2_type = ""
        self.skill_3_type = ""
        self.defense_skill_type = ""

        self.skill_1_sin_affinity = ""
        self.skill_2_sin_affinity = ""
        self.skill_3_sin_affinity = ""
        self.defense_skill_sin_affinity = ""

        self.skill_1_details = []
        self.skill_2_details = []
        self.skill_3_details = []
        self.defense_skill_details = []

    def set_skill_type(self, skill_code, skill_type, sin_affinity):
        if skill_code == "skill 1":
            self.skill_1_type = skill_type
            self.skill_1_sin_affinity = sin_affinity
        elif skill_code == "skill 2":
            self.skill_2_type = skill_type
            self.skill_2_sin_affinity = sin_affinity
        elif skill_code == "skill 3":
            self.skill_3_type = skill_type
            self.skill_3_sin_affinity = sin_affinity
        elif skill_code == "defense":
            self.defense_skill_type = skill_type
            self.defense_skill_sin_affinity = sin_affinity
    
    def add_skill(self, skill_code, skill):
        if skill_code == "skill 1":
            self.skill_1_details.append(skill)
            skill.skill_type = self.skill_1_type
            skill.sin_affinity = self.skill_1_sin_affinity
        elif skill_code == "skill 2":
            self.skill_2_details.append(skill)
            skill.skill_type = self.skill_2_type
            skill.sin_affinity = self.skill_2_sin_affinity
        elif skill_code == "skill 3":
            self.skill_3_details.append(skill)
            skill.skill_type = self.skill_3_type
            skill.sin_affinity = self.skill_3_sin_affinity
        elif skill_code == "defense":
            self.defense_skill_details.append(skill)
            skill.skill_type = self.defense_skill_type 
            skill.sin_affinity = self.defense_skill_sin_affinity

    def get_skill_by_name(self, skill_name):
        for skill_list in [self.skill_1_details, self.skill_2_details, self.skill_3_details, self.defense_skill_details]:
            for skill in skill_list:
                result = skill.is_skill_by_name(skill_name)
                if result:
                    return result
        return None

    def add_skill_uptie_info(self, skill_name, uptie_level, skill_power, coin_power, effect_by_condition):
        skill = self.get_skill_by_name(skill_name)
        skill.add_uptie_info(uptie_level, skill_power, coin_power, effect_by_condition)
        

class Skill:
    def __init__(self, skill_code, skill_name, coin_count, init_level, init_remain, atk_weight):
        self.skill_code = skill_code # "Skill 1"/"Skill 2"/"Skill 3"/"Defense" '''o'''
        self.skill_type = None
        self.sin_affinity = None
        self.skill_name = skill_name # 스킬 이름 '''o'''
        self.coin_count = coin_count # 동전 개수 '''o'''
        self.init_level = init_level # 스킬이 초기 배정받은 레벨, 인격 레벨에 - 1 한 값을 더해주면 스킬의 현 level을 구할 수 있다. '''o'''
        self.init_remain = init_remain #'''o'''
        self.atk_weight = atk_weight #공격 가중치 값이다. 방어스킬에는 없기 때문에 None으로 설정했다. '''o'''
        self.remain_count = init_remain # 실제 스킬이 작동하며 횟수가 차감되 나중에 모두 차감되면 다음 턴에 다시 초기횟수 리필된다.
        self.uptie_info = []  

    def is_skill_by_name(self, skill_name):
        if self.skill_name == skill_name:
            return self
        return None
        
    def add_uptie_info(self, uptie_level, skill_power, coin_power, effects_by_condition):
        self.uptie_info.append(UptieInfo(uptie_level, skill_power, coin_power, effects_by_condition))

class UptieInfo:
    def __init__(self, uptie_level, skill_power, coin_power, effects_by_condition):
        self.uptie_level = uptie_level   #'''o'''
        self.skill_power = skill_power   #'''o'''
        self.coin_power = coin_power     #'''o'''
        self.skill_effect = effects_by_condition

# 추상화된 EffectType 클래스
class EffectType:
    def __init__(self, name, description):
        self.name = name  # Effect 이름
        self.description = description  # Effect 설명

# Passive 클래스
class Passive(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)  # condition이 있는지 여부 확인

# SupportPassive 클래스
class SupportPassive(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)

# PanicType 클래스
class PanicType(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)

# Keyword 클래스
class Keyword:
    def __init__(self, keyword, explain=None, dispellable=None):
        self.keyword = keyword
        self.explain = explain
        self.dispellable = dispellable

      
class DataContainer:
    """크롤링 데이터를 관리하는 클래스"""
    def __init__(self):
        self.identities = []

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.level_max

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.uptie_max

    def update_identity_name(self, identity_id, identity_name):
        """특정 ID의 이름을 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_name(identity_name)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_identity_rarity(self, identity_id, rarity):
        """특정 ID의 희귀도를 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_rarity(rarity)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_skill(self, identity_id, skill_number, new_type):
        """
        특정 identity의 skill_type을 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_type: 새 skill_type
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_type = new_type
                elif skill_number == 2:
                    identity.skill_2_type = new_type
                elif skill_number == 3:
                    identity.skill_3_type = new_type
                elif skill_number == "defense":
                    identity.defense_type = new_type
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def update_sin_affinity(self, identity_id, skill_number, new_affinity):
        """
        특정 identity의 sin_affinity를 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_affinity: 새 sin_affinity
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_sin_affinity = new_affinity
                elif skill_number == 2:
                    identity.skill_2_sin_affinity = new_affinity
                elif skill_number == 3:
                    identity.skill_3_sin_affinity = new_affinity
                elif skill_number == "defense":
                    identity.defense_sin_affinity = new_affinity
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def add_identity(self, identity):
        """새로운 인격 데이터를 추가."""
        self.identities.append(identity)

    def add_level_stat(self, identity_id, level, hp, defense):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_level_stat(level, hp, defense)

    def add_uptie_stat(self, identity_id, uptie, speed):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_uptie_stat(uptie, speed)

    def add_resist_info(self, identity_id, resist_hit, resist_penetrate, resist_slash):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_resist_info(resist_hit, resist_penetrate, resist_slash)

    def add_stagger_threshold(self, identity_id, first_stagger, second_stagger, third_stagger):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_stagger_threshold(first_stagger, second_stagger, third_stagger)

    
    def add_skill(self, identity_id, skill_code, skill_name, coin_count, init_level, init_remain, atk_weight):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_code == "skill 1":
                    identity.skillset.set_skill_type(skill_code, identity.skill_1_type, identity.skill_1_sin_affinity)
                elif skill_code == "skill 2":
                    identity.skillset.set_skill_type(skill_code, identity.skill_2_type, identity.skill_2_sin_affinity)
                elif skill_code == "skill 3":
                    identity.skillset.set_skill_type(skill_code, identity.skill_3_type, identity.skill_3_sin_affinity)
                elif skill_code == "defense":
                    identity.skillset.set_skill_type(skill_code, identity.defense_type, identity.defense_sin_affinity)
                identity.add_skill(skill_code, skill_name, coin_count, init_level, init_remain, atk_weight)

    def add_skill_uptie_info(self, identity_id, skill_name, uptie_level, skill_power, coin_power, effects_by_condition):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_skill_uptie_info(skill_name, uptie_level, skill_power, coin_power, effects_by_condition)
        
    def display_identities(self):
        """Identity, ID, Name, Rarity 정보를 출력"""
        if not self.identities:
            print("No identities found.")
        else:
            # 데이터를 리스트로 변환
            table = []
            for idx, identity in enumerate(self.identities, 1):
                table.append([
                    idx, identity.identity_id, identity.name, identity.rarity,
                    identity.skill_1_type, identity.skill_1_sin_affinity,
                    identity.skill_2_type, identity.skill_2_sin_affinity,
                    identity.skill_3_type, identity.skill_3_sin_affinity,
                    identity.defense_type, identity.defense_sin_affinity
                ])
            
            # 테이블 헤더 정의
            headers = [
                "Index", "ID", "Name", "Rarity",
                "Skill_1 Type", "Skill_1 Sin", "Skill_2 Type", "Skill_2 Sin",
                "Skill_3 Type", "Skill_3 Sin", "Defense Type", "Defense Sin"
            ]
            
            # 테이블 출력
            print(tabulate(table, headers=headers, tablefmt="fancy_grid"))

    def export_to_csv(self, filename, fieldnames, data):
        """
        공통적으로 사용 가능한 CSV 내보내기 메서드
        :param filename: 내보낼 파일 이름 (예: 'identity.csv', 'skill.csv')
        :param fieldnames: CSV의 헤더 리스트
        :param data: CSV로 내보낼 데이터 (리스트 형태)
        """
        try:
            # 현재작업 디렉토 기준으 'csv'폴더 없으면 생성
            if not os.path.exists('./csv'):
                os.makedirs('./csv')
            # 파일 경로 설정 (상대경로 'csv' 폴더 안에 저장)
            file_path = os.path.join('./csv', filename)
            with open(file_path, mode='w', newline='', encoding='utf-8') as file:
                writer = csv.DictWriter(file, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(data)
            print(f"Data successfully exported to {filename}")
        except Exception as e:
            print(f"Error exporting to CSV: {e}")

    def get_identity_data(self):
        """
        Identity 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "ID": identity.identity_id,
                "Name": identity.name,
                "Rarity": identity.rarity,
                "Skill_1_Type": identity.skill_1_type,
                "Skill_1_Sin_Affinity": identity.skill_1_sin_affinity,
                "Skill_2_Type": identity.skill_2_type,
                "Skill_2_Sin_Affinity": identity.skill_2_sin_affinity,
                "Skill_3_Type": identity.skill_3_type,
                "Skill_3_Sin_Affinity": identity.skill_3_sin_affinity,
                "Defense_Type": identity.defense_type,
                "Defense_Sin_Affinity": identity.defense_sin_affinity,
            }
            for identity in self.identities
        ]
        
    def export_identities_data(self, filename="identities.csv"):
        """Identity 데이터를 CSV로 내보내는 메서드"""
        identity_fieldnames = [
            "ID", "Name", "Rarity",
            "Skill_1_Type", "Skill_1_Sin_Affinity",
            "Skill_2_Type", "Skill_2_Sin_Affinity",
            "Skill_3_Type", "Skill_3_Sin_Affinity",
            "Defense_Type", "Defense_Sin_Affinity"
        ]
        identity_data = self.get_identity_data()  # Identity 데이터 가져오기
        self.export_to_csv(filename, identity_fieldnames, identity_data)  # CSV로 내보내기

    def load_identities_by_id(self, filename, identity_ids):
        """지정된 identity_ids 목록에 해당하는 Identity 데이터를 CSV에서 읽어옴."""
        identity_ids_set = set(identity_ids) # 중복 제거를 위 Set 사용
        with open(filename, mode='r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                if row["ID"] in identity_ids_set:
                    identity = Identity(
                        identity_id=row["ID"],
                        name=row["Name"],
                        rarity=row["Rarity"],
                        skill_1_type=row["Skill_1_Type"],
                        skill_1_sin_affinity=row["Skill_1_Sin_Affinity"],
                        skill_2_type=row["Skill_2_Type"],
                        skill_2_sin_affinity=row["Skill_2_Sin_Affinity"],
                        skill_3_type=row["Skill_3_Type"],
                        skill_3_sin_affinity=row["Skill_3_Sin_Affinity"],
                        defense_type=row["Defense_Type"],
                        defense_sin_affinity=row["Defense_Sin_Affinity"]
                    )
                    self.identities.append(identity)

    def get_level_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Level": level_stat.level,
                "HP": level_stat.hp,
                "Defense": level_stat.defense,
            }
            for identity in self.identities
            for level_stat in identity.level_stats
        ]

    def export_level_stats_data(self, filename="levelstats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        level_stats_fieldnames = ["Identity_ID", "Level", "HP", "Defense"]
        level_stats_data = self.get_level_stats_data()  # level_stats 데이터 가져오기
        self.export_to_csv(filename, level_stats_fieldnames, level_stats_data)  # CSV로 내보내기

    def get_uptie_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Uptie": uptie_stat.uptie,
                "Speed": uptie_stat.speed,
            }
            for identity in self.identities
            for uptie_stat in identity.uptie_stats
        ]

    def export_uptie_stats_data(self, filename="Uptiestats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        uptie_stats_fieldnames = ["Identity_ID", "Uptie", "Speed"]
        uptie_stats_data = self.get_uptie_stats_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, uptie_stats_fieldnames, uptie_stats_data)  # CSV로 내보내기

    def get_resist_info_data(self):
        """
        모든 Identity의 resist_info 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Resist_Hit": identity.resist_info.resist_hit,
                "Resist_Penetrate": identity.resist_info.resist_penetrate,
                "Resist_Slash": identity.resist_info.resist_slash,
            }
            for identity in self.identities
        ]

    def export_resist_info_data(self, filename="ResistInfos.csv"):
        """ResistInfo 데이터를 CSV로 내보내는 메서드"""
        resist_infos_fieldnames = ["Identity_ID", "Resist_Hit", "Resist_Penetrate", "Resist_Slash"]
        resist_infos_data = self.get_resist_info_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, resist_infos_fieldnames, resist_infos_data)  # CSV로 내보내기

    def export_uptie_stats_data(self, filename="Uptiestats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        uptie_stats_fieldnames = ["Identity_ID", "Uptie", "Speed"]
        uptie_stats_data = self.get_uptie_stats_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, uptie_stats_fieldnames, uptie_stats_data)  # CSV로 내보내기

    def get_stagger_threshold_data(self):
        """
        모든 Identity의 stagger_threshold 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "First_Stagger": identity.stagger_threshold.first_stagger,
                "Second_Stagger": identity.stagger_threshold.second_stagger,
                "Third_Stagger": identity.stagger_threshold.third_stagger,
            }
            for identity in self.identities
        ]

    def export_stagger_threshold_data(self, filename="StaggerThresholds.csv"):
        """StaggerThresholds 데이터를 CSV로 내보내는 메서드"""
        stagger_thresholds_fieldnames = ["Identity_ID", "First_Stagger", "Second_Stagger", "Third_Stagger"]
        stagger_thresholds_data = self.get_stagger_threshold_data()  # stagger_thresholds 데이터 가져오기
        self.export_to_csv(filename, stagger_thresholds_fieldnames, stagger_thresholds_data)  # CSV로 내보내기

    def get_skillset_data(self):
        """모든 Identity의 skillset 데이터를 export_to_csv을 포맷으로 변환"""
        return [
            {
                "Identity_ID": identity.identity_id,
                "Skill_Code": skill.skill_code,
                "Skill_Type": skill.skill_type,
                "Sin_Affinity": skill.sin_affinity,
                "Skill_Name": skill.skill_name,
                "Coin_Count": skill.coin_count,
                "Init_Level": skill.init_level,
                "Init_Remain": skill.init_remain,
                "Atk_Weight": skill.atk_weight,
                "Uptie_Level": uptie_info.uptie_level,
                "Skill_Power": uptie_info.skill_power,
                "Coin_Power": uptie_info.coin_power,
                "Effect_Data": json.dumps(uptie_info.skill_effect)
            }
            for identity in self.identities
            for skill_list in [identity.skillset.skill_1_details, identity.skillset.skill_2_details, identity.skillset.skill_3_details, identity.skillset.defense_skill_details]
            for skill in skill_list
            for uptie_info in skill.uptie_info
        ]

    def export_identity_skillset_data(self, filename="IdentitiesSkills.csv"):
        """IdentitiesSkills 데이터를 CSV로 내보내는 메서드"""
        identities_skills_fieldnames = ["Identity_ID", "Skill_Code", "Skill_Type", "Sin_Affinity", "Skill_Name", "Coin_Count", "Init_Level", "Init_Remain", "Atk_Weight", "Uptie_Level", "Skill_Power", "Coin_Power", "Effect_Data"]
        identities_skills_data = self.get_skillset_data()
        self.export_to_csv(filename, identities_skills_fieldnames, identities_skills_data)
            


async def main():
    data_container = DataContainer()
    scraper = Scraper(container=data_container, base_url="https://limbus.kusoge.xyz", img_url="https://img.kusoge.xyz/limbus/img")
    await scraper.scrape()

await main() 

Bloodfeast
[]
{'keywords': ['Bloodfeast', 'Bleed'], 'description': ['If this unit is on field , or is one of the units that can appear on this Stage , increase keyword:Bloodfeast value by the amount of keyword:Bleed damage collectively received by every unit . When this unit enters the field , the sleeping blood drenching the battlefield will fill the surface .']}
Hardblood Thorn
['Lust x3 Owned', 'Envy x2 Owned']
{'keywords': ['Bleed', 'Bloodfeast'], 'description': ['When an other ally takes keyword:Bleed damage or consumes keyword:Bloodfeast , gain 1 blooming[thornyfall_panic RodionFirst ]( 3 times per turn ) Heal HP on self by 20 % of the damage dealt with base Skills ( max 10 per Skill )']}
"Blossom with Blood..."
['Lust x3 Owned', 'Envy x3 Owned']
{'keywords': ['Bleed'], 'description': ["To 1 ally with the highest keyword:Bleed Potency at Turn End : reduce the ally ' s Potency by 6 max , and apply ( Potency reduced / 2 ) blooming[thornyfall_panic RodionFirst ]( rounded down )"]}
P

In [72]:
import os
import csv
import re
import json
import numpy as np
import pandas as pd
from playwright.async_api import async_playwright
from tabulate import tabulate

# Sin Color 이름 변환을 위한 매핑
resource_mapping = {
    "AZURE": "Gloom",
    "VIOLET": "Envy",
    "AMBER": "Sloth",
    "SHAMROCK": "Gluttony",
    "CRIMSON": "Wrath",
    "SCARLET": "Lust",
    "INDIGO": "Pride"
}

def convert_resource(resource_name):
    """Sin Color 이름을 변환하는 함수. 매핑되지 않은 경우 원래 이름을 반환."""
    return resource_mapping.get(resource_name, resource_name)

def find_sin_for_color(url):
    # URL에 color_name을 추출
    color_name = url.split('/')[-1].split('.')[0].replace("attr_", "")
    # 해당 색상이 resource_mapping에 있는 확인
    for color, sin in resource_mapping.items():
        if color_name in color:
            return sin
    return None


class Scraper:
    """웹 페이지 크롤링을 위한 클래스"""
    def __init__(self, container, base_url, img_url):
        self.container = container
        self.base_url = base_url
        self.img_url = img_url
        self.uptie_level = 1

    async def handle_request(self, route, request):
        """이미지와 폰트 리소스 로드를 차단."""
        if request.resource_type =="image":
            #print(f"{request.url}")
            img_url = request.url
            if  re.match(f"{self.img_url}/ui/attr.*\\.png", img_url):
                await route.continue_()
            else:
                await route.fulfill(
                    status=200,
                    content_type="image/png",
                    body=b""
                )
        elif request.resource_type == "font":
            await route.abort()
        else:
            await route.continue_()
            
    async def scrape(self):
        """크롤링 메인 함수"""
        async with async_playwright() as p:
            browser = await p.chromium.launch(headless=True)
            page = await browser.new_page()
            #await page.evaluate('window.scrollTo(0, document.body.scrollHeight)')
            await page.route("**/*", self.handle_request)
            # Step 1: /identity 페이지에서 기본 정보 크롤링
            #await self.crawl_identity_page(page)

            # Step 2: /identity/{identity_id} 상세 페이지 크롤링
            #identity_ids_to_load = ["10104", "10310", "10710","10808", "10911", "11005"]
            #identity_ids_to_load = ["10104", "10911"]#,"10310", "10710","10808", "11005", "11210"]
            #identity_ids_to_load = ["10911","10310"]#, "10710","10808", "11005", "11210"]
            identity_ids_to_load = ["10911"]#, "10611"]#,"10711"]#, "10710","10808", "11005", "11210"]
            self.container.load_identities_by_id('./csv/identities.csv', identity_ids_to_load)
            await self.crawl_identity_details_for_all(page)
            #self.container.display_identities()
            #self.container.export_identities_data("identities.csv")
            #self.container.export_level_stats_data("Levelstats.csv")
            #self.container.export_uptie_stats_data("Uptiestats.csv")
            #self.container.export_resist_info_data("ResistInfos.csv")
            #self.container.export_stagger_threshold_data("StaggerThresholds.csv")
            #self.container.export_identity_skillset_data("IdentitiesSkills.csv")
            page.set_default_timeout(10000)
            #await page.wait_for_timeout(100000)
            await browser.close()

    async def get_identity_id_from_row(self, row):
        """테이블 행(row)에서 인격 ID를 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_url = await identity_cell.get_attribute('href')
            identity_id = identity_url.split('/identity/')[-1]
            return identity_id
        return "Unknown"

    async def get_identity_name_from_row(self, row):
        """테이블 행(row)에서 인격 이름을 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_name = (await identity_cell.inner_text()).replace("\n", " ").strip()
            return identity_name
        return "Unknown"
        
    async def crawl_identity_page(self, page):
        """/identity 페이지로 이동하여 인격 기본 정보를 추출."""
        await page.goto(self.base_url + "/identity")
        await self.extract_identity_page(page)

    async def extract_identity_page(self, page):
        """/identity 페이지의 모든 테이블 행에서 인격 정보를 추출."""
        identity_rows = await page.query_selector_all('tr')
        if len(identity_rows) > 1:
            for idx, identity_row in enumerate(identity_rows):
                if idx == 0:
                    continue # 0번째요소이름을 보여주는 열은 건너뜀
                await self.scrape_identity_name_and_identity_id(identity_row)
                await self.extract_rarity(identity_row)
                for i in range(1, 4):  # 스킬1~3 정보 추출
                    await self.extract_atk_and_resource(identity_row, i)
                await self.extract_defense_and_resource(identity_row)

    async def scrape_identity_name_and_identity_id(self, row):
        """인격 번호와 이름을 추출하여 DataContainer에 저장."""
        try:
            identity_id = await self.get_identity_id_from_row(row)
            identity_name = await self.get_identity_name_from_row(row)
            identity = Identity(identity_id)
            self.container.add_identity(identity)
            self.container.update_identity_name(identity_id, identity_name)
        except Exception as e:
            print(f"Error extracting identity name and identity id: {e}")

    async def get_element_attribute(self, element, selector, attribute):
        """요소에서 지정된 속성(attribute)을 추출."""
        try:
            target_element = await element.query_selector(selector)
            if target_element:
                return await target_element.get_attribute(attribute)
        except Exception as e:
            print(f"Error getting element attribute: {e}")
        return None
    
    async def extract_rarity(self, row):
        """테이블 행(row)에서 희귀도를 추출하여 업데이트."""
        try:
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
                sibling = await parent.evaluate_handle('node => node.nextElementSibling')
                if sibling:
                    rarity_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/rarity_"]', 'src')
                    if rarity_img_src:
                        rarity_level = rarity_img_src.split('_')[-1].split('.')[0]  # rarity_1.png -> 1
                        identity_id = await self.get_identity_id_from_row(row)
                        self.container.update_identity_rarity(identity_id, rarity_level)
                    else:
                        print("Rarity image not found in the next sibling TD.")
        except Exception as e: 
            print(f"Error extracting rarity: {e}")

        return "Unknown"

    async def extract_atk_and_resource(self, row, skill_number):
        """ 공격 타입(ATK Type)과 자원(Attribute) 추출 함수 - 스킬 번호에 따라 처리 """
        try:
            #print(f"Attempting to locate skill{skill_number} type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 해당 스킬 번호에 맞는 형제로 이동
                sibling = parent
                for _ in range(skill_number + 1):  # 스킬 번호에 맞게 이동
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                atk_type, resource = "Unknown", "Unknown"
        
                if sibling:
                    # 공격 타입 이미지 (atk_type) 찾기
                    atk_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/atk_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if atk_type_img_src:
                        atk_type = atk_type_img_src.split('_')[-1].split('.')[0]  # atk_type_SLASH.png -> SLASH
                        #print(f"Extracted attack type: {atk_type}")
                        self.container.update_skill(identity_id, skill_number, atk_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted resource: {resource}")
                        
                        # 리소스를 변환
                        resource = convert_resource(resource)
                        #print(f"Converted resource: {resource}")
                        self.container.update_sin_affinity(identity_id, skill_number, resource)
                        
        except Exception as e:
            print(f"Error extracting skill{skill_number} type and resource: {e}")

    async def extract_defense_and_resource(self, row):
        """ 방어 타입(Def Type)과 자원(Attribute) 추출 함수 """
        try:
            #print("Attempting to locate defense type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 다섯 번째 형제 td로 이동 (방어 관련 정보가 있는 <td>)
                sibling = parent
                for _ in range(5):  # 방어 정보는 5번째 형제에 있음
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                def_type, def_resource = "Unknown", "Unknown"
        
                if sibling:
                    # 방어 타입 이미지 (def_type) 찾기
                    def_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/def_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if def_type_img_src:
                        def_type = def_type_img_src.split('_')[-1].split('.')[0]  # def_type_GUARD.png -> GUARD
                        #print(f"Extracted defense type: {def_type}")
                        self.container.update_skill(identity_id, 'defense', def_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        def_resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted defense resource: {def_resource}")
                        
                        # 방어 리소스를 변환
                        def_resource = convert_resource(def_resource)
                        #print(f"Converted defense resource: {def_resource}")
                        self.container.update_sin_affinity(identity_id, 'defense', def_resource)
 
        except Exception as e:
            print(f"Error extracting defense type and resource: {e}")

    async def crawl_identity_details_for_all(self, page):
        """모든 인격의 세부 정보를 크롤링."""
        for identity in self.container.identities:
            await page.wait_for_load_state("networkidle")
            await page.goto(self.base_url + "/identity/"+identity.identity_id)
            """ 레벨과 동기화와 관련 없는 참관타 저항은 밑에서 변경하는 것과 연관성이 적기 때문에 위에서 처리한다.
            print(f"Identity_id: {identity.identity_id}")
            results = []
            resist_type_imgs = [f"{self.img_url}/ui/atk_type_HIT.png",
                                f"{self.img_url}/ui/atk_type_PENETRATE.png",
                                f"{self.img_url}/ui/atk_type_SLASH.png"]
            for resist_type_img in resist_type_imgs:
                result = await self.extract_resist_type(page, resist_type_img, 1)
                results.append(result)
            self.container.add_resist_info(identity.identity_id, results[0], results[1], results[2])
            """
            """
            # Stagger Threshold 값 추출
            stagger_threshold = await self.extract_stagger_threshold_values(page)
            self.container.add_stagger_threshold(identity.identity_id, stagger_threshold[0], stagger_threshold[1], stagger_threshold[2])
            """
            await self.initialize_uptie_and_level(page)
            """ 레벨에 따른 level, hp, defense 다르게 되어 있는 것을 출력 확인 객체로 들어가서 csv로 옮기는 과정까지 진행
            for i in range(1, identity.level_max+1):
                await self.move_slider_to_value(page,i)
                await self.extract_level_status(page, identity.identity_id, i)
            """
            """ 동기화에 따른 speed 다르게 되어 있는 것을 출력 확인 객체로 들어가서 csv로 옮기는 과정까지 진행
            for i in range(1, identity.uptie_max+1):
                await self.uptie_click(page,i)
                await self.extract_uptie_status(page, identity.identity_id, i)
            """
            """
                인격에 대한 skill 정보를 가져오는 메소드 위에서 출력하는 csv파일로 인격이 가지고 있는 스킬 정보를 내보낸다.
            await self.crawl_skills(page, identity.identity_id)
            """
            await self.crawl_influence_behavior(page, identity.identity_id)
            
    async def initialize_uptie_and_level(self, page):
        await self.uptie_click(page, 1)
        await self.move_slider_to_value(page, 1)
        self.uptie_level = 1
        #return 1
    
    async def uptie_click(self, page, phase):
        fab_button_selector = ".q-btn.q-btn--fab"
        await page.wait_for_selector(fab_button_selector)
        await page.click(fab_button_selector)
        #print(f"[{setup_uptie_button}] 상위 동기화 버튼 클릭 완료")
        setup_uptie_button = f'a:has(img[src="{self.img_url}/ui/uptie_{phase}.png"])'
        await page.wait_for_selector(setup_uptie_button)
        await page.click(setup_uptie_button)
        #print(f"{phase} 동기화 버튼 클릭 완료")

    async def move_slider_to_value(self, page, target_value):
        """
        슬라이더를 주어진 값으로 이동시킵니다.
        target_value는 슬라이더의 최소값부터 최대값 사이여야 합니다.
        """
        slider_selector = ".q-slider"
        # 슬라이더의 범위 확인
        min_value = int(await page.get_attribute(slider_selector, "aria-valuemin"))
        max_value = int(await page.get_attribute(slider_selector, "aria-valuemax"))
    
        if target_value < min_value or target_value > max_value:
            raise ValueError(f"Target value {target_value} is out of bounds ({min_value}, {max_value})")
    
        # 슬라이더 thumb 요소와 위치 가져오기
        slider_thumb = await page.query_selector(f"{slider_selector} .q-slider__thumb")
        thumb_box = await slider_thumb.bounding_box()
        
        # 슬라이더의 현재 위치 및 목표 위치 계산
        track = await page.query_selector(f"{slider_selector} .q-slider__track")
        track_box = await track.bounding_box()
        
        # 목표 x 위치 계산 (슬라이더 트랙 내에서 비율 기반)
        target_percentage = (target_value - min_value) / (max_value - min_value)
        target_x = track_box["x"] + target_percentage * track_box["width"]
        current_x = thumb_box["x"] + thumb_box["width"] / 2
        y = thumb_box["y"] + thumb_box["height"] / 2
    
        # 슬라이더를 드래그하여 목표 위치로 이동
        await page.mouse.move(current_x, y)
        await page.mouse.down()
        await page.mouse.move(target_x, y, steps=10)  # 부드럽게 이동
        await page.mouse.up()
    
        #print(f"슬라이더를 {target_value}로 이동 완료.")
    
        # 슬라이더 값 확인 및 강제 이벤트 트리거
        await page.evaluate(
            f"""
            (slider) => {{
                slider.dispatchEvent(new Event('input'));  // oninput 이벤트 트리거
                slider.dispatchEvent(new Event('change'));  // onchange 이벤트 트리거
            }}
            """,
            await page.query_selector(slider_selector)
        )
    
        # 이동 후 값 확인
        current_value = await page.get_attribute(slider_selector, "aria-valuenow")
        if current_value != str(target_value):
            print(f"경고: 슬라이더 값이 예상과 다릅니다. 현재 값: {current_value}, 예상 값: {target_value}")

    async def get_second_status_element(self, page):
        """
        두 번째 'Status' 요소에 접근하여 반환하는 메소드
        """
        status_elements = await page.query_selector_all("text=Status")
        if len(status_elements) > 1:
            second_status_element = status_elements[1]
            #print("두 번째 'Status' 요소에 접근합니다.")
            return second_status_element
        else:
            #print("두 번째 'Status' 요소가 존재하지 않습니다.")
            return None

    async def extract_level_status(self, page, identity_id, level):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 레벨에 따라 HP와 Defense가 변한다.
        hp_value = await self.extract_hp(second_status_element)
        defense_value = await self.extract_defense(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Level={level}, HP={hp_value}, Defense={defense_value}")
        self.container.add_level_stat(identity_id,level, hp_value, defense_value)
    
    async def extract_hp(self, status_element):
        # HP 추출
        hp_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].children[0].nextElementSibling.textContent.trim()"
        )
        return hp_value
    
    async def extract_defense(self, status_element):
        # Defense 추출
        defense_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return defense_value

    async def extract_uptie_status(self, page, identity_id, uptie):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 uptie에 따라 speed는 변한다.
        speed_value = await self.extract_speed(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Uptie={uptie}, Speed={speed_value}")
        self.container.add_uptie_stat(identity_id,uptie, speed_value)

    async def extract_speed(self, status_element):
        # Speed 추출
        speed_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return speed_value

    async def extract_resist_type(self, page, image_src, depth):
        # 이미지 요소 찾기
        #print(image_src)
        image = await page.query_selector(f'img[src="{image_src}"]')
        #print(image)
        if not image:
            return "Unknown"
    
        # 이미지의 부모 요소를 찾아서 올라감
        parent = image
        for _ in range(depth):
            parent = await parent.evaluate_handle('node => node.parentElement')
            if not parent:
                return "Unknown"
    
        # 부모의 형제 요소 찾기
        sibling = await parent.evaluate_handle('node => node.nextElementSibling')
        
        if sibling:
            value = await sibling.evaluate('node => node.textContent')
            if value:
                return value.strip()
        
        return "Unknown"

    async def extract_stagger_threshold_values(self, page):
        # 'div.row.center.flex-center.q-pt-xs' 요소들 찾기
        elements = await page.query_selector_all('div.row.center.flex-center.q-pt-xs')
        
        # 각 요소의 텍스트 내용 출력하여 'Stagger Threshold' 요소를 찾기
        for element in elements:
            text = await element.text_content()
            
            if text and "Stagger Threshold" in text:
                # 'Stagger Threshold'가 있는 요소를 찾았다면, 그 이후 형제 요소들에서 값을 추출
                parent = await element.evaluate_handle('node => node.parentElement')
                sibling_elements = await parent.query_selector_all('div.row.justify-evenly div')  # Threshold 값이 들어있는 div들
                
                stagger_values = []
                for sibling in sibling_elements:
                    text_content = await sibling.text_content()
                    match = re.search(r'(\d+%)', text_content)
                    if match:
                        stagger_values.append(match.group(1))  # 퍼센트 값만 리스트에 추가
                while len(stagger_values) < 3:
                    stagger_values.append(np.nan)
                #print(stagger_values)
                return stagger_values  # 추출된 값(또는 np.nan 포함)을 반환
        
        return [np.nan, np.nan, np.nan]  # 만약 'Stagger Threshold'를 찾지 못한 경우 기본값 반환

    async def crawl_skills(self, page, identity_id):
        await self.crawl_skill(page, identity_id, "skill 1")
        await self.crawl_skill(page, identity_id, "skill 2")
        await self.crawl_skill(page, identity_id, "skill 3")
        await self.crawl_skill(page, identity_id, "defense")


    async def crawl_skill(self, page, identity_id, skill_type):
        skills_elements = await page.query_selector_all('div.skilltag')
        for skill_element in skills_elements:
            skill_text = await skill_element.text_content()
            if skill_text.strip().lower() == skill_type:
                await self.process_skill(skill_element, page, identity_id, skill_type)
        
    async def process_skill(self, skill_element, page, identity_id, skill_type):
        # skill_type이 "skill 3"일 경우 uptie_value가 3 이상일 때만 정보를 저장
        for uptie_value in range(1, 5):  # Uptie levels 1 to 4
            # "skill 3"은 uptie_value가 3 이상일 때만 처리
            if skill_type == "skill 3" and uptie_value < 3:
                continue  # skip if uptie_value is less than 3 for skill 3
    
            # uptie_level에 맞게 클릭
            await self.uptie_click(page, uptie_value)
            if skill_type != "skill 3" and uptie_value > 1:
                # Uptie 정보 갱신
                await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
                continue
            elif skill_type == "skill 3" and uptie_value > 3:
                # Uptie 정보 갱신
                await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
                continue
            # 스킬의 세부 정보를 추출
            skill_details = await self.extract_skill_details(skill_element, skill_type)
            skill_name = skill_details["skill_name"]
    
            # 스킬 정보를 저장
            self.container.add_skill(
                identity_id, skill_type, 
                skill_name, skill_details["coin_count"],
                skill_details["init_level"], skill_details["init_remain"], 
                skill_details["atk_weight"]
            )
            print(f"Added skill: {skill_type}:{skill_name}")
    
            # Uptie 정보 갱신
            await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
    
    async def extract_skill_details(self, skill_element, skill_type):
        # 스킬 세부 정보를 추출하는 함수
        skill_name = await self.skill_name(skill_element)
        coin_count = await self.coin_count(skill_element, ".col-auto > .column > .row")
        init_level = await self.init_level(skill_element)
        init_remain = await self.init_remain(skill_element, skill_type, "div.row.justify-start > div.attr.flex.flex-center > div.q-px-sm")
        atk_weight = await self.atk_weight(skill_element, skill_type)
        return {
            "skill_name": skill_name,
            "coin_count": coin_count,
            "init_level": init_level,
            "init_remain": init_remain,
            "atk_weight": atk_weight
        }
    
    async def update_uptie_info(self, page, skill_element, identity_id, skill_name, skill_type, uptie_value):
        # Uptie 정보를 추출하고 업데이트하는 함수
        uptie_info = await self.extract_uptie_info(skill_element)
        effect_data = await self.skill_effects(skill_element)
        self.container.add_skill_uptie_info(
            identity_id, skill_name, 
            uptie_value, uptie_info["skill_power"], uptie_info["coin_power"], 
            effect_data.get("effectsByCondition")
        )
        print(
            f"Updated Uptie for {skill_name} (Uptie Level: {uptie_value}): "
            f"Skill Power: {uptie_info['skill_power']}, Coin Power: {uptie_info['coin_power']} "
            f"Skill Effect: {effect_data.get("effectsByCondition")}"
        )

    async def extract_uptie_info(self, skill_element):
        skill_data = await self.skill_and_coin_power(skill_element)
        return {
            "skill_power": skill_data.get("skillPower"),
            "coin_power": skill_data.get("coinPower")
        }
        
    async def crawl_influence_behavior(self, page, identity_id):
        influence_behaviors = await page.query_selector_all('div[align="right"]')
        # 텍스트를 기준으 추출하고 메소드로 파라미터 들어가 정보를 받는다.
        for influence_behavior in influence_behaviors:
            active_type = await influence_behavior.inner_text()
            active_type = active_type.strip()

            if active_type == "PASSIVE":
                """
                # PASSIVE에 대한 처리
                print(f"Processing PASSIVE logic for: {active_type}")
                # CALL method for PASSIVE
                """
                passive_data = await self.passive_effects(influence_behavior)
            elif active_type == "SUPPORT PASSIVE":
                """
                # SUPPORT PASSIVE에 대한 처리
                print(f"Processing SUPPORT PASSIVE logic for: {active_type}")
                # CALL method for SUPPORT PASSIVE
                """
                support_passive_data = await self.support_passive_effects(influence_behavior)
            elif active_type == "PANIC TYPE":
                """
                # PANIC TYPE에 대한 처리
                print(f"Processing PANIC TYPE logic for:{active_type}")
                # CALL method for PANIC TYPE
                """
                panic_type_data = await self.panic_effects(influence_behavior)

    async def skill_and_coin_power(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                // 조부모로 이동
                const grandparent = node.parentElement.parentElement;
                // 이전 형제 탐색
                const previousSibling = grandparent.previousElementSibling;
                if (!previousSibling) return {{ skillPower: null, coinPower: null }}; // 이전 형제가 없으면 기본값 반환
                // 이전 형제 내부의 .row.justify-around > div > div 탐색
                const targetDivs = previousSibling.querySelectorAll(".row.justify-around > div");
                if (targetDivs.length < 3) return {{ skillPower: null, coinPower: null }}; // 배열 길이 확인
                // 1st div: 스킬 위력
                const skillPower = targetDivs[0]?.querySelector('div')?.textContent?.trim() || null; 
                // 3rd div: 코인 파워 값 (3번째 div에서 텍스트 추출)
                const coinPower = targetDivs[2]?.textContent?.trim() || null;
                return {{ skillPower, coinPower }};
            }}
        """)


    async def coin_count(self, skill_element, hierarchy_selector):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector('{hierarchy_selector}');
                if (!targetElement) return 0;
                return targetElement.childElementCount;
            }}
        """)

    async def init_remain(self, skill_element, skill_type, hierarchy_selector):
        if skill_type == "defense":
            return -1
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector('{hierarchy_selector}');
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def skill_name(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector(".block");
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def init_level(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector("div.text-bold.text-h6.h6.q-pl-xs.q-pr-md");
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def atk_weight(self, skill_element, skill_type):
        if skill_type == "defense":
            return 0
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector("div.text-bold.text-h6.h6.q-pl-xs.q-pr-md");
                const doubleNextSibling = targetElement.nextElementSibling.nextElementSibling;
                if (!doubleNextSibling) return 0;
                return doubleNextSibling.textContent?.trim();
            }}
        """)

    async def skill_effects(self, skill_element):
        return await skill_element.evaluate("""
            node => {
                const parent = node.parentElement;
                const doubleNextSibling = parent.nextElementSibling?.nextElementSibling;
                if (!doubleNextSibling) return null;
    
                // doubleNextSibling에서 첫 번째 div.col 요소 찾기
                const textNode = doubleNextSibling.querySelector("div.col");
                if (!textNode) return null;
    
                const results = {
                    keywords: [], // 키워드 저장
                    conditions: [], // 조건 저장
                    effectsByCondition: {} // 조건별 효과
                };
    
                // 1. 조건 추출 (대괄호 텍스트)
                const conditionElements = textNode.querySelectorAll("span");
                conditionElements.forEach(span => {
                    let text = span.textContent?.trim();
                    if (text?.startsWith("[") && text?.endsWith("]")) {
                        // 부모의 전형제 노드 탐색
                        const sibling = span.parentElement.previousElementSibling;
                        if (sibling) {
                            const emblemImage = sibling.querySelector("img.emblem");
                            console.log("Sibling element found:", sibling); // 디버깅용
                            if (emblemImage) {
                                const src = emblemImage.getAttribute("src");
                                console.log("Image source:", src); // 디버깅용
                                const match = src.match(/ui_num_(\\d+)\\.png$/); // 정규식 수정
                                if (match) {
                                    const number = match[1]; // 숫자 추출
                                    text = `[Coin ${number} ${text.replace("[", "").replace("]", "")}]`;
                                    console.log("Updated condition:", text); // 디버깅용
                                }
                            }
                        }
                        results.conditions.push(text);
                        results.effectsByCondition[text] = []; // 조건별 효과를 초기화
                    }
                });
    
                // 2. 키워드 추출
                const keywordElements = textNode.querySelectorAll('a[href^="/en/keyword/"]');
                keywordElements.forEach(a => {
                    const span = a.querySelector('span.q-btn__content.text-center.col.items-center.q-anchor--skip.justify-center.row>span');
                    if (span) {
                        const keywordText = span.textContent.trim();
                        if (keywordText && !results.keywords.includes(keywordText)) {
                            results.keywords.push(keywordText); // 키워드 저장
                        }
                    }
                });
    
                // 3. 효과 추출
                const effectElements = textNode.querySelectorAll("span");
                let currentCondition = null;
                let currentEffect = []; // 초기화
                effectElements.forEach(span => {
                    let text = span.textContent?.trim();
                    if (!text) return;
    
                    // 조건 발견 시 currentCondition 변경
                    if (text.startsWith("[") && text.endsWith("]")) {
                        // 이전 조건의 효과 저장
                        if (currentCondition && currentEffect.length > 0) {
                            // 부모의 전형제 노드 탐색
                            const sibling = span.parentElement.previousElementSibling;
                            if (sibling) {
                                const emblemImage = sibling.querySelector("img.emblem");
                                if (emblemImage) {
                                    const src = emblemImage.getAttribute("src");
                                    const match = src.match(/ui_num_(\\d+)\\.png$/); // 정규식 수정
                                    if (match) {
                                        const number = match[1]; // 숫자 추출
                                        text = `[Coin ${number} ${text.replace("[", "").replace("]", "")}]`;
                                    }
                                }
                            }
                            results.effectsByCondition[currentCondition].push(currentEffect.join(' '));
                            currentEffect = []; // 초기화
                        }
                        currentCondition = text;
                        return;
                    }
    
                    // 현재 조건의 효과 처리
                    if (currentCondition) {
                        if (results.keywords.includes(text)) {
                            const keywordEffect = 'keyword:' + text;
                            if (!currentEffect.includes(keywordEffect)) {
                                currentEffect.push(keywordEffect);
                            }
                        } else {
                            currentEffect.push(text);
                        }
                    }
                });
    
                // 마지막 효과 저장
                if (currentCondition && currentEffect.length > 0) {
                    results.effectsByCondition[currentCondition].push(currentEffect.join(' '));
                }
    
                return results;
            }
        """)

    async def passive_effects(self, passive_element):
        passive_name = await self.behavior_name(passive_element)
        passive_conditions = await self.behavior_condition(passive_element)
        passive_description = await self.behavior_description(passive_element)
        print(passive_name)
        print(passive_conditions)
        print(passive_description)
    async def support_passive_effects(self, support_passive_element):
        support_passive_name = await self.behavior_name(support_passive_element)
        support_passive_conditions = await self.behavior_condition(support_passive_element)
        support_passive_description = await self.behavior_description(support_passive_element)
        print(support_passive_name)
        print(support_passive_conditions)
        print(support_passive_description)
    async def panic_effects(self, panic_element):
        panic_type = await self.behavior_name(panic_element)
        panic_conditions = await self.behavior_condition(panic_element)
        panic_description = await self.panic_description(panic_element)
        print(panic_type)
        print(panic_conditions)
        print(panic_description)

    async def navigate_element_by_steps(self, element, parent_steps=0, prev_sibling_steps=0, next_sibling_steps=0):
        """
        주어진 `element`에서 부모, 이전 형제, 또는 다음 형제로 지정된 횟수만큼 이동하여 ElementHandle 반환.
        
        Args:
            element (ElementHandle): 탐색 기준이 되는 요소.
            parent_steps (int): 부모로 이동할 횟수 (0이면 이동하지 않음).
            prev_sibling_steps (int): 이전 형제로 이동할 횟수 (0이면 이동하지 않음).
            next_sibling_steps (int): 다음 형제로 이동할 횟수 (0이면 이동하지 않음).
        
        Returns:
            ElementHandle or None: 탐색된 요소를 반환하거나 없으면 None.
        """
        try:
            current_element = element
    
            # 부모로 이동
            for _ in range(parent_steps):
                current_element = await current_element.evaluate_handle('el => el.parentElement')
                if not current_element:
                    print("Parent element not found during navigation.")
                    return None
    
            # 이전 형제로 이동
            for _ in range(prev_sibling_steps):
                current_element = await current_element.evaluate_handle('el => el.previousElementSibling')
                if not current_element:
                    print("Previous sibling element not found during navigation.")
                    return None
    
            # 다음 형제로 이동
            for _ in range(next_sibling_steps):
                current_element = await current_element.evaluate_handle('el => el.nextElementSibling')
                if not current_element:
                    print("Next sibling element not found during navigation.")
                    return None
    
            return current_element
    
        except Exception as e:
            print(f"Error navigating elements: {e}")
            return None

    async def get_last_sibling(self, element):
        try:
            # 같은 부 아 마지막 형제를 가져오는 JavaScript 실행
            last_sibling = await element.evaluate_handle('el => el.parentElement.lastElementChild')
            return last_sibling
        except Exception as e:
            print(f"Error getting last sibling: {e}")
            return None

    async def behavior_name(self, element):
        parent_prev_sibling = await self.navigate_element_by_steps(element, parent_steps=1, prev_sibling_steps=1, next_sibling_steps=0)
        passive_name_element = await parent_prev_sibling.query_selector('span.block')
        if passive_name_element:
            passive_name = await passive_name_element.inner_text()
            return passive_name
        else:
            #print("Passive name element not found")
            return None

    async def behavior_condition(self, element):
        try:
            conditions = []
            parent_prev_sibling = await self.navigate_element_by_steps(element, parent_steps=1, prev_sibling_steps=1, next_sibling_steps=0)
            img_elements = await parent_prev_sibling.query_selector_all('img')
            if not img_elements:
                #print("No <img> elements found.")
                return conditions
    
            for img_element in img_elements:
                # 각 이미지의 src 속성 가져오기
                img_src = await img_element.get_attribute('src')
                #print(img_src)
    
                sin_name = None
                for color_key in resource_mapping.keys():
                    if f"{self.img_url}/ui/attr_{color_key}.png" in img_src:
                        sin_name = resource_mapping[color_key]
                        break
    
                if not sin_name:
                    #print(f"No matching Sin Color found for {img_src}.")
                    continue  # No Sin Color found, skip this img_element
    
                # 해당 이미지와 관련된 조건 텍스트 추출
                # 두 번째 부모 요소에서 시작하여 다음 형제를 찾아 조건 텍스트를 추출
                condition_element = await self.navigate_element_by_steps(img_element, parent_steps=3, prev_sibling_steps=0, next_sibling_steps=0)
                if condition_element:
                    # condition_element가 None이 아닌 경우에만 진행
                    condition_text_element = await condition_element.query_selector('.q-pl-xs')
                    if condition_text_element:
                        condition_text = await condition_text_element.inner_text()
                        condition_text = condition_text.strip()
                    else:
                        print(f"No condition text found for {img_src}.")
                        condition_text = ""  # condition_text_element가 None이면 빈 문자열로 처리
                else:
                    print(f"No condition element found for {img_src}.")
                    condition_text = ""  # condition_element가 None이면 빈 문자열로 처리
    
                # 결과를 배열에 추가
                conditions.append(f"{sin_name} {condition_text}")
            return conditions
        except Exception as e:
            print(f"Error extracting conditions: {e}")
            return 
            
    async def behavior_description(self, element):
        parent_prev_sibling = await self.navigate_element_by_steps(element, parent_steps=1, prev_sibling_steps=1, next_sibling_steps=0)
        return await parent_prev_sibling.evaluate("""
                node => {
                    const behavior_title_node = node.querySelector('.q-pb-sm')
                    const behavior_title_parent = behavior_title_node.parentElement
                    const behavior_description = behavior_title_parent.lastElementChild
                    if (!behavior_description) return null;

                    const results = {
                        keywords: [], // 키워드 저장
                        description: [] // 설명 저장
                    };
                    // 1. 키워드 추출
                    const keywordElements = behavior_description.querySelectorAll('a[href^="/en/keyword/"]');
                    if (!keywordElements) return null;
                    keywordElements.forEach(a => {
                        const span = a.querySelector('span.q-btn__content.text-center.col.items-center.q-anchor--skip.justify-center.row>span');
                        if (span) {
                            const keywordText = span.textContent.trim();
                            if (keywordText && !results.keywords.includes(keywordText)) {
                                results.keywords.push(keywordText); // 키워드 저장
                            }
                        }
                    });
        
                    // 2. 설명 추출
                    const descriptionElements = behavior_description.querySelectorAll("span");
                    let currentEffect = []; // 초기화
                    descriptionElements.forEach(span => {
                        let text = span.textContent?.trim();
                        if (!text) return;
                        // 공백과 기호 처리
                        text = text.replace(/\\s*([.,:;!?()\\[\\]])\\s*/g, '$1'); // 기호 앞뒤 공백 제거
                        text = text.replace(/\\s+/g, ''); // 다중 공백을 단일 공백으로 변환
                        if (results.keywords.includes(text)) {
                            const keywordEffect = 'keyword:' + text;
                            if (!currentEffect.includes(keywordEffect)) {
                                currentEffect.push(keywordEffect);
                            }
                        } else {
                            currentEffect.push(text);
                        }
                    });
        
                    // 마지막 효과 저장
                    if (currentEffect.length > 0) {
                        results.description.push(currentEffect.join(''));
                    }
        
                    return results;
                }
            """)

    async def panic_description(self, element):
        try:
            description_texts = []
            keywords = []
    
            # Navigate to the behavior description parent
            parent_prev_sibling = await self.navigate_element_by_steps(element, parent_steps=1, prev_sibling_steps=1, next_sibling_steps=0)
            behavior_title = await parent_prev_sibling.query_selector('.q-pb-sm')
            behavior_description_element = await self.get_last_sibling(behavior_title)
    
            # Fetch keyword elements
            keyword_elements = await behavior_description_element.query_selector_all('a[href^="/en/keyword/"]')
            for keyword_element in keyword_elements:
                keyword_span = await keyword_element.query_selector(
                    'span.q-btn__content.text-center.col.items-center.q-anchor--skip.justify-center.row > span'
                )
                if keyword_span:
                    keyword_text = await keyword_span.inner_text()
                    stripped_text = keyword_text.strip()
                    if stripped_text and stripped_text not in keywords:
                        keywords.append(stripped_text)
    
            # Fetch description elements
            descriptionElements = await behavior_description_element.query_selector_all("div")
            seen_texts = []
            # 수정된 부분: inner_text()의 원본을 출력해보고 strip()을 조정하여 문제를 파악합니다.
            # 키워드를 포함한 텍스트에서 마지막 문장이 누락되지 않도록 조정
            for descriptionElement in descriptionElements:
                description_text = await descriptionElement.inner_text()
                #print(f"Original text: '{description_text}'")  # 원본 텍스트 출력
                stripped_text = description_text.strip()  # 앞뒤 공백만 제거
                stripped_text = re.sub(r'\s+', ' ', stripped_text)
                description_texts.append(stripped_text)
            normalize_text = await self.combine_and_process_texts(description_texts)
            return normalize_text
        except Exception as e:
            print(f"Error extracting description: {e}")
            return

    async def combine_and_process_texts(self, texts):
        """
        배열 형태의 텍스트 데이터를 합치고 형식을 처리한 문자열을 반환합니다.
        Args:
            texts (list of list): 텍스트 배열 목록
        Returns:
            str: 형식이 처리된 최종 문자열
        """
        processed_sentences = []
        
        for text_array in texts:
            # 빈 문자열을 제외한 나머지 텍스트를 이어붙이기
            sentence = ''.join([word for word in text_array if word != ''])
            sentence = re.sub(r'\\](\\s*)', ']', sentence)
            processed_sentences.append(sentence)
    
        # 문장을 공백으로 연결하여 반환
        return ''.join(processed_sentences)


class Identity:
    """인격 정보를 저장하는 클래스"""
    def __init__(self, identity_id, name=None, rarity=None,\
                 skill_1_type=None, skill_1_sin_affinity=None,\
                 skill_2_type=None, skill_2_sin_affinity=None,\
                 skill_3_type=None, skill_3_sin_affinity=None,\
                 defense_type=None, defense_sin_affinity=None):
        self.identity_id = identity_id
        self.name = name
        self.rarity = rarity
        # 선언만 하는 부분 
        self.skill_1_type = skill_1_type 
        self.skill_1_sin_affinity = skill_1_sin_affinity 
        self.skill_2_type = skill_2_type 
        self.skill_2_sin_affinity = skill_2_sin_affinity 
        self.skill_3_type = skill_3_type 
        self.skill_3_sin_affinity = skill_3_sin_affinity 
        self.defense_type = defense_type 
        self.defense_sin_affinity = defense_sin_affinity
        self.level_max = 50
        self.uptie_max = 4
        self.level_stats = []  # LevelStat 객체 리스트
        self.uptie_stats = [] # UptieStat 객체 리스트
        self.resist_info = None
        self.stagger_threshold = None
        self.skillset = SkillSet()  # Skill정보를 담고 있는 skillset 객체 
        self.passive_infos = []  # 패시브 객체를 담고 있는 배열
        self.support_passive_infos = [] # 서포트 패시 객체를 담고 있는 배열
        self.panic_type = [] # 혼란상태 발생시 나타나는 효과를 담고 있는 객체 배열1

    def update_name(self, name):
        self.name = name

    def update_rarity(self, rarity):
        self.rarity = rarity

    def add_level_stat(self, level, hp, defense):
        self.level_stats.append(LevelStat(level, hp, defense))

    def add_uptie_stat(self, uptie, speed):
        self.uptie_stats.append(UptieStat(uptie, speed))

    def add_resist_info(self, resist_hit, resist_penetrate, resist_slash):
        self.resist_info = ResistInfo(resist_hit, resist_penetrate, resist_slash)

    def add_stagger_threshold(self, first_stagger, second_stagger, third_stagger):
        self.stagger_threshold = StaggerThreshold(first_stagger, second_stagger, third_stagger)

    def add_skill(self, skill_code, skill_name, coin_count, init_level, init_remain, atk_weight):
        self.skillset.add_skill(skill_code, Skill(skill_code, skill_name, coin_count, init_level, init_remain, atk_weight))
        
    def add_skill_uptie_info(self, skill_name, uptie_level, skill_power, coin_power, effect_by_condition):
        self.skillset.add_skill_uptie_info(skill_name, uptie_level, skill_power, coin_power, effect_by_condition)
        
    def update_details(self, page):
        # 세부 정보 업데이트 로직 (가정)
        pass
    #def init_uptie(self, page):
    #    init_uptie = 

# LevelStat 객체 정의
class LevelStat:
    def __init__(self, level, hp=None, defense=None):
        self.level = level
        self.hp = hp
        self.defense = defense
    def update_level(self, level):
        self.level = level
    def update_hp(self, hp):
        self.hp = hp
    def update_defense(self, defense):
        self.defense = defense

# UptieStat 객체 정의
class UptieStat:
    def __init__(self, uptie, speed=None):
        self.uptie = uptie
        self.speed = speed
    def update_speed(self, speed):
        self.speed = speed

# ResistInfo 객체 정의
class ResistInfo:
    def __init__(self, resist_hit=None, resist_penetrate=None, resist_slash=None):
        self.resist_hit = resist_hit
        self.resist_penetrate = resist_penetrate
        self.resist_slash = resist_slash

# ResistInfo 객체 정의
class StaggerThreshold:
    def __init__(self, first_stagger=None, second_stagger=None, third_stagger=None):
        self.first_stagger = first_stagger
        self.second_stagger = second_stagger
        self.third_stagger = third_stagger

class SkillSet:
    def __init__(self):

        self.skill_1_type = ""
        self.skill_2_type = ""
        self.skill_3_type = ""
        self.defense_skill_type = ""

        self.skill_1_sin_affinity = ""
        self.skill_2_sin_affinity = ""
        self.skill_3_sin_affinity = ""
        self.defense_skill_sin_affinity = ""

        self.skill_1_details = []
        self.skill_2_details = []
        self.skill_3_details = []
        self.defense_skill_details = []

    def set_skill_type(self, skill_code, skill_type, sin_affinity):
        if skill_code == "skill 1":
            self.skill_1_type = skill_type
            self.skill_1_sin_affinity = sin_affinity
        elif skill_code == "skill 2":
            self.skill_2_type = skill_type
            self.skill_2_sin_affinity = sin_affinity
        elif skill_code == "skill 3":
            self.skill_3_type = skill_type
            self.skill_3_sin_affinity = sin_affinity
        elif skill_code == "defense":
            self.defense_skill_type = skill_type
            self.defense_skill_sin_affinity = sin_affinity
    
    def add_skill(self, skill_code, skill):
        if skill_code == "skill 1":
            self.skill_1_details.append(skill)
            skill.skill_type = self.skill_1_type
            skill.sin_affinity = self.skill_1_sin_affinity
        elif skill_code == "skill 2":
            self.skill_2_details.append(skill)
            skill.skill_type = self.skill_2_type
            skill.sin_affinity = self.skill_2_sin_affinity
        elif skill_code == "skill 3":
            self.skill_3_details.append(skill)
            skill.skill_type = self.skill_3_type
            skill.sin_affinity = self.skill_3_sin_affinity
        elif skill_code == "defense":
            self.defense_skill_details.append(skill)
            skill.skill_type = self.defense_skill_type 
            skill.sin_affinity = self.defense_skill_sin_affinity

    def get_skill_by_name(self, skill_name):
        for skill_list in [self.skill_1_details, self.skill_2_details, self.skill_3_details, self.defense_skill_details]:
            for skill in skill_list:
                result = skill.is_skill_by_name(skill_name)
                if result:
                    return result
        return None

    def add_skill_uptie_info(self, skill_name, uptie_level, skill_power, coin_power, effect_by_condition):
        skill = self.get_skill_by_name(skill_name)
        skill.add_uptie_info(uptie_level, skill_power, coin_power, effect_by_condition)
        

class Skill:
    def __init__(self, skill_code, skill_name, coin_count, init_level, init_remain, atk_weight):
        self.skill_code = skill_code # "Skill 1"/"Skill 2"/"Skill 3"/"Defense" '''o'''
        self.skill_type = None
        self.sin_affinity = None
        self.skill_name = skill_name # 스킬 이름 '''o'''
        self.coin_count = coin_count # 동전 개수 '''o'''
        self.init_level = init_level # 스킬이 초기 배정받은 레벨, 인격 레벨에 - 1 한 값을 더해주면 스킬의 현 level을 구할 수 있다. '''o'''
        self.init_remain = init_remain #'''o'''
        self.atk_weight = atk_weight #공격 가중치 값이다. 방어스킬에는 없기 때문에 None으로 설정했다. '''o'''
        self.remain_count = init_remain # 실제 스킬이 작동하며 횟수가 차감되 나중에 모두 차감되면 다음 턴에 다시 초기횟수 리필된다.
        self.uptie_info = []  

    def is_skill_by_name(self, skill_name):
        if self.skill_name == skill_name:
            return self
        return None
        
    def add_uptie_info(self, uptie_level, skill_power, coin_power, effects_by_condition):
        self.uptie_info.append(UptieInfo(uptie_level, skill_power, coin_power, effects_by_condition))

class UptieInfo:
    def __init__(self, uptie_level, skill_power, coin_power, effects_by_condition):
        self.uptie_level = uptie_level   #'''o'''
        self.skill_power = skill_power   #'''o'''
        self.coin_power = coin_power     #'''o'''
        self.skill_effect = effects_by_condition

# 추상화된 EffectType 클래스
class EffectType:
    def __init__(self, name, description):
        self.name = name  # Effect 이름
        self.description = description  # Effect 설명

# Passive 클래스
class Passive(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)  # condition이 있는지 여부 확인

# SupportPassive 클래스
class SupportPassive(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)

# PanicType 클래스
class PanicType(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)

# Keyword 클래스
class Keyword:
    def __init__(self, keyword, explain=None, dispellable=None):
        self.keyword = keyword
        self.explain = explain
        self.dispellable = dispellable

      
class DataContainer:
    """크롤링 데이터를 관리하는 클래스"""
    def __init__(self):
        self.identities = []

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.level_max

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.uptie_max

    def update_identity_name(self, identity_id, identity_name):
        """특정 ID의 이름을 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_name(identity_name)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_identity_rarity(self, identity_id, rarity):
        """특정 ID의 희귀도를 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_rarity(rarity)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_skill(self, identity_id, skill_number, new_type):
        """
        특정 identity의 skill_type을 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_type: 새 skill_type
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_type = new_type
                elif skill_number == 2:
                    identity.skill_2_type = new_type
                elif skill_number == 3:
                    identity.skill_3_type = new_type
                elif skill_number == "defense":
                    identity.defense_type = new_type
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def update_sin_affinity(self, identity_id, skill_number, new_affinity):
        """
        특정 identity의 sin_affinity를 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_affinity: 새 sin_affinity
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_sin_affinity = new_affinity
                elif skill_number == 2:
                    identity.skill_2_sin_affinity = new_affinity
                elif skill_number == 3:
                    identity.skill_3_sin_affinity = new_affinity
                elif skill_number == "defense":
                    identity.defense_sin_affinity = new_affinity
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def add_identity(self, identity):
        """새로운 인격 데이터를 추가."""
        self.identities.append(identity)

    def add_level_stat(self, identity_id, level, hp, defense):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_level_stat(level, hp, defense)

    def add_uptie_stat(self, identity_id, uptie, speed):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_uptie_stat(uptie, speed)

    def add_resist_info(self, identity_id, resist_hit, resist_penetrate, resist_slash):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_resist_info(resist_hit, resist_penetrate, resist_slash)

    def add_stagger_threshold(self, identity_id, first_stagger, second_stagger, third_stagger):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_stagger_threshold(first_stagger, second_stagger, third_stagger)

    
    def add_skill(self, identity_id, skill_code, skill_name, coin_count, init_level, init_remain, atk_weight):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_code == "skill 1":
                    identity.skillset.set_skill_type(skill_code, identity.skill_1_type, identity.skill_1_sin_affinity)
                elif skill_code == "skill 2":
                    identity.skillset.set_skill_type(skill_code, identity.skill_2_type, identity.skill_2_sin_affinity)
                elif skill_code == "skill 3":
                    identity.skillset.set_skill_type(skill_code, identity.skill_3_type, identity.skill_3_sin_affinity)
                elif skill_code == "defense":
                    identity.skillset.set_skill_type(skill_code, identity.defense_type, identity.defense_sin_affinity)
                identity.add_skill(skill_code, skill_name, coin_count, init_level, init_remain, atk_weight)

    def add_skill_uptie_info(self, identity_id, skill_name, uptie_level, skill_power, coin_power, effects_by_condition):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_skill_uptie_info(skill_name, uptie_level, skill_power, coin_power, effects_by_condition)
        
    def display_identities(self):
        """Identity, ID, Name, Rarity 정보를 출력"""
        if not self.identities:
            print("No identities found.")
        else:
            # 데이터를 리스트로 변환
            table = []
            for idx, identity in enumerate(self.identities, 1):
                table.append([
                    idx, identity.identity_id, identity.name, identity.rarity,
                    identity.skill_1_type, identity.skill_1_sin_affinity,
                    identity.skill_2_type, identity.skill_2_sin_affinity,
                    identity.skill_3_type, identity.skill_3_sin_affinity,
                    identity.defense_type, identity.defense_sin_affinity
                ])
            
            # 테이블 헤더 정의
            headers = [
                "Index", "ID", "Name", "Rarity",
                "Skill_1 Type", "Skill_1 Sin", "Skill_2 Type", "Skill_2 Sin",
                "Skill_3 Type", "Skill_3 Sin", "Defense Type", "Defense Sin"
            ]
            
            # 테이블 출력
            print(tabulate(table, headers=headers, tablefmt="fancy_grid"))

    def export_to_csv(self, filename, fieldnames, data):
        """
        공통적으로 사용 가능한 CSV 내보내기 메서드
        :param filename: 내보낼 파일 이름 (예: 'identity.csv', 'skill.csv')
        :param fieldnames: CSV의 헤더 리스트
        :param data: CSV로 내보낼 데이터 (리스트 형태)
        """
        try:
            # 현재작업 디렉토 기준으 'csv'폴더 없으면 생성
            if not os.path.exists('./csv'):
                os.makedirs('./csv')
            # 파일 경로 설정 (상대경로 'csv' 폴더 안에 저장)
            file_path = os.path.join('./csv', filename)
            with open(file_path, mode='w', newline='', encoding='utf-8') as file:
                writer = csv.DictWriter(file, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(data)
            print(f"Data successfully exported to {filename}")
        except Exception as e:
            print(f"Error exporting to CSV: {e}")

    def get_identity_data(self):
        """
        Identity 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "ID": identity.identity_id,
                "Name": identity.name,
                "Rarity": identity.rarity,
                "Skill_1_Type": identity.skill_1_type,
                "Skill_1_Sin_Affinity": identity.skill_1_sin_affinity,
                "Skill_2_Type": identity.skill_2_type,
                "Skill_2_Sin_Affinity": identity.skill_2_sin_affinity,
                "Skill_3_Type": identity.skill_3_type,
                "Skill_3_Sin_Affinity": identity.skill_3_sin_affinity,
                "Defense_Type": identity.defense_type,
                "Defense_Sin_Affinity": identity.defense_sin_affinity,
            }
            for identity in self.identities
        ]
        
    def export_identities_data(self, filename="identities.csv"):
        """Identity 데이터를 CSV로 내보내는 메서드"""
        identity_fieldnames = [
            "ID", "Name", "Rarity",
            "Skill_1_Type", "Skill_1_Sin_Affinity",
            "Skill_2_Type", "Skill_2_Sin_Affinity",
            "Skill_3_Type", "Skill_3_Sin_Affinity",
            "Defense_Type", "Defense_Sin_Affinity"
        ]
        identity_data = self.get_identity_data()  # Identity 데이터 가져오기
        self.export_to_csv(filename, identity_fieldnames, identity_data)  # CSV로 내보내기

    def load_identities_by_id(self, filename, identity_ids):
        """지정된 identity_ids 목록에 해당하는 Identity 데이터를 CSV에서 읽어옴."""
        identity_ids_set = set(identity_ids) # 중복 제거를 위 Set 사용
        with open(filename, mode='r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                if row["ID"] in identity_ids_set:
                    identity = Identity(
                        identity_id=row["ID"],
                        name=row["Name"],
                        rarity=row["Rarity"],
                        skill_1_type=row["Skill_1_Type"],
                        skill_1_sin_affinity=row["Skill_1_Sin_Affinity"],
                        skill_2_type=row["Skill_2_Type"],
                        skill_2_sin_affinity=row["Skill_2_Sin_Affinity"],
                        skill_3_type=row["Skill_3_Type"],
                        skill_3_sin_affinity=row["Skill_3_Sin_Affinity"],
                        defense_type=row["Defense_Type"],
                        defense_sin_affinity=row["Defense_Sin_Affinity"]
                    )
                    self.identities.append(identity)

    def get_level_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Level": level_stat.level,
                "HP": level_stat.hp,
                "Defense": level_stat.defense,
            }
            for identity in self.identities
            for level_stat in identity.level_stats
        ]

    def export_level_stats_data(self, filename="levelstats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        level_stats_fieldnames = ["Identity_ID", "Level", "HP", "Defense"]
        level_stats_data = self.get_level_stats_data()  # level_stats 데이터 가져오기
        self.export_to_csv(filename, level_stats_fieldnames, level_stats_data)  # CSV로 내보내기

    def get_uptie_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Uptie": uptie_stat.uptie,
                "Speed": uptie_stat.speed,
            }
            for identity in self.identities
            for uptie_stat in identity.uptie_stats
        ]

    def export_uptie_stats_data(self, filename="Uptiestats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        uptie_stats_fieldnames = ["Identity_ID", "Uptie", "Speed"]
        uptie_stats_data = self.get_uptie_stats_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, uptie_stats_fieldnames, uptie_stats_data)  # CSV로 내보내기

    def get_resist_info_data(self):
        """
        모든 Identity의 resist_info 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Resist_Hit": identity.resist_info.resist_hit,
                "Resist_Penetrate": identity.resist_info.resist_penetrate,
                "Resist_Slash": identity.resist_info.resist_slash,
            }
            for identity in self.identities
        ]

    def export_resist_info_data(self, filename="ResistInfos.csv"):
        """ResistInfo 데이터를 CSV로 내보내는 메서드"""
        resist_infos_fieldnames = ["Identity_ID", "Resist_Hit", "Resist_Penetrate", "Resist_Slash"]
        resist_infos_data = self.get_resist_info_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, resist_infos_fieldnames, resist_infos_data)  # CSV로 내보내기

    def export_uptie_stats_data(self, filename="Uptiestats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        uptie_stats_fieldnames = ["Identity_ID", "Uptie", "Speed"]
        uptie_stats_data = self.get_uptie_stats_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, uptie_stats_fieldnames, uptie_stats_data)  # CSV로 내보내기

    def get_stagger_threshold_data(self):
        """
        모든 Identity의 stagger_threshold 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "First_Stagger": identity.stagger_threshold.first_stagger,
                "Second_Stagger": identity.stagger_threshold.second_stagger,
                "Third_Stagger": identity.stagger_threshold.third_stagger,
            }
            for identity in self.identities
        ]

    def export_stagger_threshold_data(self, filename="StaggerThresholds.csv"):
        """StaggerThresholds 데이터를 CSV로 내보내는 메서드"""
        stagger_thresholds_fieldnames = ["Identity_ID", "First_Stagger", "Second_Stagger", "Third_Stagger"]
        stagger_thresholds_data = self.get_stagger_threshold_data()  # stagger_thresholds 데이터 가져오기
        self.export_to_csv(filename, stagger_thresholds_fieldnames, stagger_thresholds_data)  # CSV로 내보내기

    def get_skillset_data(self):
        """모든 Identity의 skillset 데이터를 export_to_csv을 포맷으로 변환"""
        return [
            {
                "Identity_ID": identity.identity_id,
                "Skill_Code": skill.skill_code,
                "Skill_Type": skill.skill_type,
                "Sin_Affinity": skill.sin_affinity,
                "Skill_Name": skill.skill_name,
                "Coin_Count": skill.coin_count,
                "Init_Level": skill.init_level,
                "Init_Remain": skill.init_remain,
                "Atk_Weight": skill.atk_weight,
                "Uptie_Level": uptie_info.uptie_level,
                "Skill_Power": uptie_info.skill_power,
                "Coin_Power": uptie_info.coin_power,
                "Effect_Data": json.dumps(uptie_info.skill_effect)
            }
            for identity in self.identities
            for skill_list in [identity.skillset.skill_1_details, identity.skillset.skill_2_details, identity.skillset.skill_3_details, identity.skillset.defense_skill_details]
            for skill in skill_list
            for uptie_info in skill.uptie_info
        ]

    def export_identity_skillset_data(self, filename="IdentitiesSkills.csv"):
        """IdentitiesSkills 데이터를 CSV로 내보내는 메서드"""
        identities_skills_fieldnames = ["Identity_ID", "Skill_Code", "Skill_Type", "Sin_Affinity", "Skill_Name", "Coin_Count", "Init_Level", "Init_Remain", "Atk_Weight", "Uptie_Level", "Skill_Power", "Coin_Power", "Effect_Data"]
        identities_skills_data = self.get_skillset_data()
        self.export_to_csv(filename, identities_skills_fieldnames, identities_skills_data)
            


async def main():
    data_container = DataContainer()
    scraper = Scraper(container=data_container, base_url="https://limbus.kusoge.xyz", img_url="https://img.kusoge.xyz/limbus/img")
    await scraper.scrape()

await main() 

Bloodfeast
[]
{'keywords': ['Bloodfeast', 'Bleed'], 'description': ['Ifthisunitisonfield,orisoneoftheunitsthatcanappearonthisStage,increasekeyword:Bloodfeastvaluebytheamountofkeyword:Bleeddamagecollectivelyreceivedbyeveryunit.Whenthisunitentersthefield,thesleepingblooddrenchingthebattlefieldwillfillthesurface.']}
Hardblood Thorn
['Lust x3 Owned', 'Envy x2 Owned']
{'keywords': ['Bleed', 'Bloodfeast'], 'description': ['Whenanotherallytakeskeyword:Bleeddamageorconsumeskeyword:Bloodfeast,gain1blooming[thornyfall_panicRodionFirst](3timesperturn)HealHPonselfby20%ofthedamagedealtwithbaseSkills(max10perSkill)']}
"Blossom with Blood..."
['Lust x3 Owned', 'Envy x3 Owned']
{'keywords': ['Bleed'], 'description': ["To1allywiththehighestkeyword:BleedPotencyatTurnEnd:reducetheally'sPotencyby6max,andapply(Potencyreduced/2)blooming[thornyfall_panicRodionFirst](roundeddown)"]}
Panic
[]
Panic Effects :Does not act for this turn.


In [92]:
import os
import csv
import re
import json
import numpy as np
import pandas as pd
from playwright.async_api import async_playwright
from tabulate import tabulate

# Sin Color 이름 변환을 위한 매핑
resource_mapping = {
    "AZURE": "Gloom",
    "VIOLET": "Envy",
    "AMBER": "Sloth",
    "SHAMROCK": "Gluttony",
    "CRIMSON": "Wrath",
    "SCARLET": "Lust",
    "INDIGO": "Pride"
}

def convert_resource(resource_name):
    """Sin Color 이름을 변환하는 함수. 매핑되지 않은 경우 원래 이름을 반환."""
    return resource_mapping.get(resource_name, resource_name)

def find_sin_for_color(url):
    # URL에 color_name을 추출
    color_name = url.split('/')[-1].split('.')[0].replace("attr_", "")
    # 해당 색상이 resource_mapping에 있는 확인
    for color, sin in resource_mapping.items():
        if color_name in color:
            return sin
    return None


class Scraper:
    """웹 페이지 크롤링을 위한 클래스"""
    def __init__(self, container, base_url, img_url):
        self.container = container
        self.base_url = base_url
        self.img_url = img_url
        self.uptie_level = 1

    async def handle_request(self, route, request):
        """이미지와 폰트 리소스 로드를 차단."""
        if request.resource_type =="image":
            #print(f"{request.url}")
            img_url = request.url
            if  re.match(f"{self.img_url}/ui/attr.*\\.png", img_url):
                await route.continue_()
            else:
                await route.fulfill(
                    status=200,
                    content_type="image/png",
                    body=b""
                )
        elif request.resource_type == "font":
            await route.abort()
        else:
            await route.continue_()
            
    async def scrape(self):
        """크롤링 메인 함수"""
        async with async_playwright() as p:
            browser = await p.chromium.launch(headless=True)
            page = await browser.new_page()
            #await page.evaluate('window.scrollTo(0, document.body.scrollHeight)')
            await page.route("**/*", self.handle_request)
            # Step 1: /identity 페이지에서 기본 정보 크롤링
            #await self.crawl_identity_page(page)

            # Step 2: /identity/{identity_id} 상세 페이지 크롤링
            #identity_ids_to_load = ["10104", "10310", "10710","10808", "10911", "11005"]
            #identity_ids_to_load = ["10104", "10911"]#,"10310", "10710","10808", "11005", "11210"]
            #identity_ids_to_load = ["10911","10310"]#, "10710","10808", "11005", "11210"]
            identity_ids_to_load = ["10911"]#, "10611"]#,"10711"]#, "10710","10808", "11005", "11210"]
            self.container.load_identities_by_id('./csv/identities.csv', identity_ids_to_load)
            await self.crawl_identity_details_for_all(page)
            #self.container.display_identities()
            #self.container.export_identities_data("identities.csv")
            #self.container.export_level_stats_data("Levelstats.csv")
            #self.container.export_uptie_stats_data("Uptiestats.csv")
            #self.container.export_resist_info_data("ResistInfos.csv")
            #self.container.export_stagger_threshold_data("StaggerThresholds.csv")
            #self.container.export_identity_skillset_data("IdentitiesSkills.csv")
            page.set_default_timeout(10000)
            #await page.wait_for_timeout(100000)
            await browser.close()

    async def get_identity_id_from_row(self, row):
        """테이블 행(row)에서 인격 ID를 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_url = await identity_cell.get_attribute('href')
            identity_id = identity_url.split('/identity/')[-1]
            return identity_id
        return "Unknown"

    async def get_identity_name_from_row(self, row):
        """테이블 행(row)에서 인격 이름을 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_name = (await identity_cell.inner_text()).replace("\n", " ").strip()
            return identity_name
        return "Unknown"
        
    async def crawl_identity_page(self, page):
        """/identity 페이지로 이동하여 인격 기본 정보를 추출."""
        await page.goto(self.base_url + "/identity")
        await self.extract_identity_page(page)

    async def extract_identity_page(self, page):
        """/identity 페이지의 모든 테이블 행에서 인격 정보를 추출."""
        identity_rows = await page.query_selector_all('tr')
        if len(identity_rows) > 1:
            for idx, identity_row in enumerate(identity_rows):
                if idx == 0:
                    continue # 0번째요소이름을 보여주는 열은 건너뜀
                await self.scrape_identity_name_and_identity_id(identity_row)
                await self.extract_rarity(identity_row)
                for i in range(1, 4):  # 스킬1~3 정보 추출
                    await self.extract_atk_and_resource(identity_row, i)
                await self.extract_defense_and_resource(identity_row)

    async def scrape_identity_name_and_identity_id(self, row):
        """인격 번호와 이름을 추출하여 DataContainer에 저장."""
        try:
            identity_id = await self.get_identity_id_from_row(row)
            identity_name = await self.get_identity_name_from_row(row)
            identity = Identity(identity_id)
            self.container.add_identity(identity)
            self.container.update_identity_name(identity_id, identity_name)
        except Exception as e:
            print(f"Error extracting identity name and identity id: {e}")

    async def get_element_attribute(self, element, selector, attribute):
        """요소에서 지정된 속성(attribute)을 추출."""
        try:
            target_element = await element.query_selector(selector)
            if target_element:
                return await target_element.get_attribute(attribute)
        except Exception as e:
            print(f"Error getting element attribute: {e}")
        return None
    
    async def extract_rarity(self, row):
        """테이블 행(row)에서 희귀도를 추출하여 업데이트."""
        try:
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
                sibling = await parent.evaluate_handle('node => node.nextElementSibling')
                if sibling:
                    rarity_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/rarity_"]', 'src')
                    if rarity_img_src:
                        rarity_level = rarity_img_src.split('_')[-1].split('.')[0]  # rarity_1.png -> 1
                        identity_id = await self.get_identity_id_from_row(row)
                        self.container.update_identity_rarity(identity_id, rarity_level)
                    else:
                        print("Rarity image not found in the next sibling TD.")
        except Exception as e: 
            print(f"Error extracting rarity: {e}")

        return "Unknown"

    async def extract_atk_and_resource(self, row, skill_number):
        """ 공격 타입(ATK Type)과 자원(Attribute) 추출 함수 - 스킬 번호에 따라 처리 """
        try:
            #print(f"Attempting to locate skill{skill_number} type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 해당 스킬 번호에 맞는 형제로 이동
                sibling = parent
                for _ in range(skill_number + 1):  # 스킬 번호에 맞게 이동
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                atk_type, resource = "Unknown", "Unknown"
        
                if sibling:
                    # 공격 타입 이미지 (atk_type) 찾기
                    atk_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/atk_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if atk_type_img_src:
                        atk_type = atk_type_img_src.split('_')[-1].split('.')[0]  # atk_type_SLASH.png -> SLASH
                        #print(f"Extracted attack type: {atk_type}")
                        self.container.update_skill(identity_id, skill_number, atk_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted resource: {resource}")
                        
                        # 리소스를 변환
                        resource = convert_resource(resource)
                        #print(f"Converted resource: {resource}")
                        self.container.update_sin_affinity(identity_id, skill_number, resource)
                        
        except Exception as e:
            print(f"Error extracting skill{skill_number} type and resource: {e}")

    async def extract_defense_and_resource(self, row):
        """ 방어 타입(Def Type)과 자원(Attribute) 추출 함수 """
        try:
            #print("Attempting to locate defense type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 다섯 번째 형제 td로 이동 (방어 관련 정보가 있는 <td>)
                sibling = parent
                for _ in range(5):  # 방어 정보는 5번째 형제에 있음
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                def_type, def_resource = "Unknown", "Unknown"
        
                if sibling:
                    # 방어 타입 이미지 (def_type) 찾기
                    def_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/def_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if def_type_img_src:
                        def_type = def_type_img_src.split('_')[-1].split('.')[0]  # def_type_GUARD.png -> GUARD
                        #print(f"Extracted defense type: {def_type}")
                        self.container.update_skill(identity_id, 'defense', def_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        def_resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted defense resource: {def_resource}")
                        
                        # 방어 리소스를 변환
                        def_resource = convert_resource(def_resource)
                        #print(f"Converted defense resource: {def_resource}")
                        self.container.update_sin_affinity(identity_id, 'defense', def_resource)
 
        except Exception as e:
            print(f"Error extracting defense type and resource: {e}")

    async def crawl_identity_details_for_all(self, page):
        """모든 인격의 세부 정보를 크롤링."""
        for identity in self.container.identities:
            await page.wait_for_load_state("networkidle")
            await page.goto(self.base_url + "/identity/"+identity.identity_id)
            """ 레벨과 동기화와 관련 없는 참관타 저항은 밑에서 변경하는 것과 연관성이 적기 때문에 위에서 처리한다.
            print(f"Identity_id: {identity.identity_id}")
            results = []
            resist_type_imgs = [f"{self.img_url}/ui/atk_type_HIT.png",
                                f"{self.img_url}/ui/atk_type_PENETRATE.png",
                                f"{self.img_url}/ui/atk_type_SLASH.png"]
            for resist_type_img in resist_type_imgs:
                result = await self.extract_resist_type(page, resist_type_img, 1)
                results.append(result)
            self.container.add_resist_info(identity.identity_id, results[0], results[1], results[2])
            """
            """
            # Stagger Threshold 값 추출
            stagger_threshold = await self.extract_stagger_threshold_values(page)
            self.container.add_stagger_threshold(identity.identity_id, stagger_threshold[0], stagger_threshold[1], stagger_threshold[2])
            """
            await self.initialize_uptie_and_level(page)
            """ 레벨에 따른 level, hp, defense 다르게 되어 있는 것을 출력 확인 객체로 들어가서 csv로 옮기는 과정까지 진행
            for i in range(1, identity.level_max+1):
                await self.move_slider_to_value(page,i)
                await self.extract_level_status(page, identity.identity_id, i)
            """
            """ 동기화에 따른 speed 다르게 되어 있는 것을 출력 확인 객체로 들어가서 csv로 옮기는 과정까지 진행
            for i in range(1, identity.uptie_max+1):
                await self.uptie_click(page,i)
                await self.extract_uptie_status(page, identity.identity_id, i)
            """
            """
                인격에 대한 skill 정보를 가져오는 메소드 위에서 출력하는 csv파일로 인격이 가지고 있는 스킬 정보를 내보낸다.
            await self.crawl_skills(page, identity.identity_id)
            """
            await self.crawl_influence_behavior(page, identity.identity_id)
            
    async def initialize_uptie_and_level(self, page):
        await self.uptie_click(page, 1)
        await self.move_slider_to_value(page, 1)
        self.uptie_level = 1
        #return 1
    
    async def uptie_click(self, page, phase):
        fab_button_selector = ".q-btn.q-btn--fab"
        await page.wait_for_selector(fab_button_selector)
        await page.click(fab_button_selector)
        #print(f"[{setup_uptie_button}] 상위 동기화 버튼 클릭 완료")
        setup_uptie_button = f'a:has(img[src="{self.img_url}/ui/uptie_{phase}.png"])'
        await page.wait_for_selector(setup_uptie_button)
        await page.click(setup_uptie_button)
        #print(f"{phase} 동기화 버튼 클릭 완료")

    async def move_slider_to_value(self, page, target_value):
        """
        슬라이더를 주어진 값으로 이동시킵니다.
        target_value는 슬라이더의 최소값부터 최대값 사이여야 합니다.
        """
        slider_selector = ".q-slider"
        # 슬라이더의 범위 확인
        min_value = int(await page.get_attribute(slider_selector, "aria-valuemin"))
        max_value = int(await page.get_attribute(slider_selector, "aria-valuemax"))
    
        if target_value < min_value or target_value > max_value:
            raise ValueError(f"Target value {target_value} is out of bounds ({min_value}, {max_value})")
    
        # 슬라이더 thumb 요소와 위치 가져오기
        slider_thumb = await page.query_selector(f"{slider_selector} .q-slider__thumb")
        thumb_box = await slider_thumb.bounding_box()
        
        # 슬라이더의 현재 위치 및 목표 위치 계산
        track = await page.query_selector(f"{slider_selector} .q-slider__track")
        track_box = await track.bounding_box()
        
        # 목표 x 위치 계산 (슬라이더 트랙 내에서 비율 기반)
        target_percentage = (target_value - min_value) / (max_value - min_value)
        target_x = track_box["x"] + target_percentage * track_box["width"]
        current_x = thumb_box["x"] + thumb_box["width"] / 2
        y = thumb_box["y"] + thumb_box["height"] / 2
    
        # 슬라이더를 드래그하여 목표 위치로 이동
        await page.mouse.move(current_x, y)
        await page.mouse.down()
        await page.mouse.move(target_x, y, steps=10)  # 부드럽게 이동
        await page.mouse.up()
    
        #print(f"슬라이더를 {target_value}로 이동 완료.")
    
        # 슬라이더 값 확인 및 강제 이벤트 트리거
        await page.evaluate(
            f"""
            (slider) => {{
                slider.dispatchEvent(new Event('input'));  // oninput 이벤트 트리거
                slider.dispatchEvent(new Event('change'));  // onchange 이벤트 트리거
            }}
            """,
            await page.query_selector(slider_selector)
        )
    
        # 이동 후 값 확인
        current_value = await page.get_attribute(slider_selector, "aria-valuenow")
        if current_value != str(target_value):
            print(f"경고: 슬라이더 값이 예상과 다릅니다. 현재 값: {current_value}, 예상 값: {target_value}")

    async def get_second_status_element(self, page):
        """
        두 번째 'Status' 요소에 접근하여 반환하는 메소드
        """
        status_elements = await page.query_selector_all("text=Status")
        if len(status_elements) > 1:
            second_status_element = status_elements[1]
            #print("두 번째 'Status' 요소에 접근합니다.")
            return second_status_element
        else:
            #print("두 번째 'Status' 요소가 존재하지 않습니다.")
            return None

    async def extract_level_status(self, page, identity_id, level):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 레벨에 따라 HP와 Defense가 변한다.
        hp_value = await self.extract_hp(second_status_element)
        defense_value = await self.extract_defense(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Level={level}, HP={hp_value}, Defense={defense_value}")
        self.container.add_level_stat(identity_id,level, hp_value, defense_value)
    
    async def extract_hp(self, status_element):
        # HP 추출
        hp_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].children[0].nextElementSibling.textContent.trim()"
        )
        return hp_value
    
    async def extract_defense(self, status_element):
        # Defense 추출
        defense_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return defense_value

    async def extract_uptie_status(self, page, identity_id, uptie):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 uptie에 따라 speed는 변한다.
        speed_value = await self.extract_speed(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Uptie={uptie}, Speed={speed_value}")
        self.container.add_uptie_stat(identity_id,uptie, speed_value)

    async def extract_speed(self, status_element):
        # Speed 추출
        speed_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return speed_value

    async def extract_resist_type(self, page, image_src, depth):
        # 이미지 요소 찾기
        #print(image_src)
        image = await page.query_selector(f'img[src="{image_src}"]')
        #print(image)
        if not image:
            return "Unknown"
    
        # 이미지의 부모 요소를 찾아서 올라감
        parent = image
        for _ in range(depth):
            parent = await parent.evaluate_handle('node => node.parentElement')
            if not parent:
                return "Unknown"
    
        # 부모의 형제 요소 찾기
        sibling = await parent.evaluate_handle('node => node.nextElementSibling')
        
        if sibling:
            value = await sibling.evaluate('node => node.textContent')
            if value:
                return value.strip()
        
        return "Unknown"

    async def extract_stagger_threshold_values(self, page):
        # 'div.row.center.flex-center.q-pt-xs' 요소들 찾기
        elements = await page.query_selector_all('div.row.center.flex-center.q-pt-xs')
        
        # 각 요소의 텍스트 내용 출력하여 'Stagger Threshold' 요소를 찾기
        for element in elements:
            text = await element.text_content()
            
            if text and "Stagger Threshold" in text:
                # 'Stagger Threshold'가 있는 요소를 찾았다면, 그 이후 형제 요소들에서 값을 추출
                parent = await element.evaluate_handle('node => node.parentElement')
                sibling_elements = await parent.query_selector_all('div.row.justify-evenly div')  # Threshold 값이 들어있는 div들
                
                stagger_values = []
                for sibling in sibling_elements:
                    text_content = await sibling.text_content()
                    match = re.search(r'(\d+%)', text_content)
                    if match:
                        stagger_values.append(match.group(1))  # 퍼센트 값만 리스트에 추가
                while len(stagger_values) < 3:
                    stagger_values.append(np.nan)
                #print(stagger_values)
                return stagger_values  # 추출된 값(또는 np.nan 포함)을 반환
        
        return [np.nan, np.nan, np.nan]  # 만약 'Stagger Threshold'를 찾지 못한 경우 기본값 반환

    async def crawl_skills(self, page, identity_id):
        await self.crawl_skill(page, identity_id, "skill 1")
        await self.crawl_skill(page, identity_id, "skill 2")
        await self.crawl_skill(page, identity_id, "skill 3")
        await self.crawl_skill(page, identity_id, "defense")


    async def crawl_skill(self, page, identity_id, skill_type):
        skills_elements = await page.query_selector_all('div.skilltag')
        for skill_element in skills_elements:
            skill_text = await skill_element.text_content()
            if skill_text.strip().lower() == skill_type:
                await self.process_skill(skill_element, page, identity_id, skill_type)
        
    async def process_skill(self, skill_element, page, identity_id, skill_type):
        # skill_type이 "skill 3"일 경우 uptie_value가 3 이상일 때만 정보를 저장
        for uptie_value in range(1, 5):  # Uptie levels 1 to 4
            # "skill 3"은 uptie_value가 3 이상일 때만 처리
            if skill_type == "skill 3" and uptie_value < 3:
                continue  # skip if uptie_value is less than 3 for skill 3
    
            # uptie_level에 맞게 클릭
            await self.uptie_click(page, uptie_value)
            if skill_type != "skill 3" and uptie_value > 1:
                # Uptie 정보 갱신
                await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
                continue
            elif skill_type == "skill 3" and uptie_value > 3:
                # Uptie 정보 갱신
                await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
                continue
            # 스킬의 세부 정보를 추출
            skill_details = await self.extract_skill_details(skill_element, skill_type)
            skill_name = skill_details["skill_name"]
    
            # 스킬 정보를 저장
            self.container.add_skill(
                identity_id, skill_type, 
                skill_name, skill_details["coin_count"],
                skill_details["init_level"], skill_details["init_remain"], 
                skill_details["atk_weight"]
            )
            print(f"Added skill: {skill_type}:{skill_name}")
    
            # Uptie 정보 갱신
            await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
    
    async def extract_skill_details(self, skill_element, skill_type):
        # 스킬 세부 정보를 추출하는 함수
        skill_name = await self.skill_name(skill_element)
        coin_count = await self.coin_count(skill_element, ".col-auto > .column > .row")
        init_level = await self.init_level(skill_element)
        init_remain = await self.init_remain(skill_element, skill_type, "div.row.justify-start > div.attr.flex.flex-center > div.q-px-sm")
        atk_weight = await self.atk_weight(skill_element, skill_type)
        return {
            "skill_name": skill_name,
            "coin_count": coin_count,
            "init_level": init_level,
            "init_remain": init_remain,
            "atk_weight": atk_weight
        }
    
    async def update_uptie_info(self, page, skill_element, identity_id, skill_name, skill_type, uptie_value):
        # Uptie 정보를 추출하고 업데이트하는 함수
        uptie_info = await self.extract_uptie_info(skill_element)
        effect_data = await self.skill_effects(skill_element)
        self.container.add_skill_uptie_info(
            identity_id, skill_name, 
            uptie_value, uptie_info["skill_power"], uptie_info["coin_power"], 
            effect_data.get("effectsByCondition")
        )
        print(
            f"Updated Uptie for {skill_name} (Uptie Level: {uptie_value}): "
            f"Skill Power: {uptie_info['skill_power']}, Coin Power: {uptie_info['coin_power']} "
            f"Skill Effect: {effect_data.get("effectsByCondition")}"
        )

    async def extract_uptie_info(self, skill_element):
        skill_data = await self.skill_and_coin_power(skill_element)
        return {
            "skill_power": skill_data.get("skillPower"),
            "coin_power": skill_data.get("coinPower")
        }
        
    async def crawl_influence_behavior(self, page, identity_id):
        influence_behaviors = await page.query_selector_all('div[align="right"]')
        # 텍스트를 기준으 추출하고 메소드로 파라미터 들어가 정보를 받는다.
        for influence_behavior in influence_behaviors:
            active_type = await influence_behavior.inner_text()
            active_type = active_type.strip()

            if active_type == "PASSIVE":
                """
                # PASSIVE에 대한 처리
                print(f"Processing PASSIVE logic for: {active_type}")
                # CALL method for PASSIVE
                """
                passive_data = await self.passive_effects(influence_behavior)
            elif active_type == "SUPPORT PASSIVE":
                """
                # SUPPORT PASSIVE에 대한 처리
                print(f"Processing SUPPORT PASSIVE logic for: {active_type}")
                # CALL method for SUPPORT PASSIVE
                """
                support_passive_data = await self.support_passive_effects(influence_behavior)
            elif active_type == "PANIC TYPE":
                """
                # PANIC TYPE에 대한 처리
                print(f"Processing PANIC TYPE logic for:{active_type}")
                # CALL method for PANIC TYPE
                """
                panic_type_data = await self.panic_effects(influence_behavior)

    async def skill_and_coin_power(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                // 조부모로 이동
                const grandparent = node.parentElement.parentElement;
                // 이전 형제 탐색
                const previousSibling = grandparent.previousElementSibling;
                if (!previousSibling) return {{ skillPower: null, coinPower: null }}; // 이전 형제가 없으면 기본값 반환
                // 이전 형제 내부의 .row.justify-around > div > div 탐색
                const targetDivs = previousSibling.querySelectorAll(".row.justify-around > div");
                if (targetDivs.length < 3) return {{ skillPower: null, coinPower: null }}; // 배열 길이 확인
                // 1st div: 스킬 위력
                const skillPower = targetDivs[0]?.querySelector('div')?.textContent?.trim() || null; 
                // 3rd div: 코인 파워 값 (3번째 div에서 텍스트 추출)
                const coinPower = targetDivs[2]?.textContent?.trim() || null;
                return {{ skillPower, coinPower }};
            }}
        """)


    async def coin_count(self, skill_element, hierarchy_selector):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector('{hierarchy_selector}');
                if (!targetElement) return 0;
                return targetElement.childElementCount;
            }}
        """)

    async def init_remain(self, skill_element, skill_type, hierarchy_selector):
        if skill_type == "defense":
            return -1
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector('{hierarchy_selector}');
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def skill_name(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector(".block");
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def init_level(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector("div.text-bold.text-h6.h6.q-pl-xs.q-pr-md");
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def atk_weight(self, skill_element, skill_type):
        if skill_type == "defense":
            return 0
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector("div.text-bold.text-h6.h6.q-pl-xs.q-pr-md");
                const doubleNextSibling = targetElement.nextElementSibling.nextElementSibling;
                if (!doubleNextSibling) return 0;
                return doubleNextSibling.textContent?.trim();
            }}
        """)

    async def skill_effects(self, skill_element):
        return await skill_element.evaluate("""
            node => {
                const parent = node.parentElement;
                const doubleNextSibling = parent.nextElementSibling?.nextElementSibling;
                if (!doubleNextSibling) return null;
    
                // doubleNextSibling에서 첫 번째 div.col 요소 찾기
                const textNode = doubleNextSibling.querySelector("div.col");
                if (!textNode) return null;
    
                const results = {
                    keywords: [], // 키워드 저장
                    conditions: [], // 조건 저장
                    effectsByCondition: {} // 조건별 효과
                };
    
                // 1. 조건 추출 (대괄호 텍스트)
                const conditionElements = textNode.querySelectorAll("span");
                conditionElements.forEach(span => {
                    let text = span.textContent?.trim();
                    if (text?.startsWith("[") && text?.endsWith("]")) {
                        // 부모의 전형제 노드 탐색
                        const sibling = span.parentElement.previousElementSibling;
                        if (sibling) {
                            const emblemImage = sibling.querySelector("img.emblem");
                            console.log("Sibling element found:", sibling); // 디버깅용
                            if (emblemImage) {
                                const src = emblemImage.getAttribute("src");
                                console.log("Image source:", src); // 디버깅용
                                const match = src.match(/ui_num_(\\d+)\\.png$/); // 정규식 수정
                                if (match) {
                                    const number = match[1]; // 숫자 추출
                                    text = `[Coin ${number} ${text.replace("[", "").replace("]", "")}]`;
                                    console.log("Updated condition:", text); // 디버깅용
                                }
                            }
                        }
                        results.conditions.push(text);
                        results.effectsByCondition[text] = []; // 조건별 효과를 초기화
                    }
                });
    
                // 2. 키워드 추출
                const keywordElements = textNode.querySelectorAll('a[href^="/en/keyword/"]');
                keywordElements.forEach(a => {
                    const span = a.querySelector('span.q-btn__content.text-center.col.items-center.q-anchor--skip.justify-center.row>span');
                    if (span) {
                        const keywordText = span.textContent.trim();
                        if (keywordText && !results.keywords.includes(keywordText)) {
                            results.keywords.push(keywordText); // 키워드 저장
                        }
                    }
                });
    
                // 3. 효과 추출
                const effectElements = textNode.querySelectorAll("span");
                let currentCondition = null;
                let currentEffect = []; // 초기화
                effectElements.forEach(span => {
                    let text = span.textContent?.trim();
                    if (!text) return;
    
                    // 조건 발견 시 currentCondition 변경
                    if (text.startsWith("[") && text.endsWith("]")) {
                        // 이전 조건의 효과 저장
                        if (currentCondition && currentEffect.length > 0) {
                            // 부모의 전형제 노드 탐색
                            const sibling = span.parentElement.previousElementSibling;
                            if (sibling) {
                                const emblemImage = sibling.querySelector("img.emblem");
                                if (emblemImage) {
                                    const src = emblemImage.getAttribute("src");
                                    const match = src.match(/ui_num_(\\d+)\\.png$/); // 정규식 수정
                                    if (match) {
                                        const number = match[1]; // 숫자 추출
                                        text = `[Coin ${number} ${text.replace("[", "").replace("]", "")}]`;
                                    }
                                }
                            }
                            results.effectsByCondition[currentCondition].push(currentEffect.join(' '));
                            currentEffect = []; // 초기화
                        }
                        currentCondition = text;
                        return;
                    }
    
                    // 현재 조건의 효과 처리
                    if (currentCondition) {
                        if (results.keywords.includes(text)) {
                            const keywordEffect = 'keyword:' + text;
                            if (!currentEffect.includes(keywordEffect)) {
                                currentEffect.push(keywordEffect);
                            }
                        } else {
                            currentEffect.push(text);
                        }
                    }
                });
    
                // 마지막 효과 저장
                if (currentCondition && currentEffect.length > 0) {
                    results.effectsByCondition[currentCondition].push(currentEffect.join(' '));
                }
    
                return results;
            }
        """)

    async def passive_effects(self, passive_element):
        passive_name = await self.behavior_name(passive_element)
        passive_conditions = await self.behavior_condition(passive_element)
        passive_description = await self.behavior_description(passive_element, "span")
        print(passive_name)
        print(passive_conditions)
        print(passive_description)
    async def support_passive_effects(self, support_passive_element):
        support_passive_name = await self.behavior_name(support_passive_element)
        support_passive_conditions = await self.behavior_condition(support_passive_element)
        support_passive_description = await self.behavior_description(support_passive_element, "span")
        print(support_passive_name)
        print(support_passive_conditions)
        print(support_passive_description)
    async def panic_effects(self, panic_element):
        panic_type = await self.behavior_name(panic_element)
        panic_conditions = await self.behavior_condition(panic_element)
        panic_description = await self.behavior_description(panic_element, "div")
        print(panic_type)
        print(panic_conditions)
        print(panic_description)

    async def navigate_element_by_steps(self, element, parent_steps=0, prev_sibling_steps=0, next_sibling_steps=0):
        """
        주어진 `element`에서 부모, 이전 형제, 또는 다음 형제로 지정된 횟수만큼 이동하여 ElementHandle 반환.
        
        Args:
            element (ElementHandle): 탐색 기준이 되는 요소.
            parent_steps (int): 부모로 이동할 횟수 (0이면 이동하지 않음).
            prev_sibling_steps (int): 이전 형제로 이동할 횟수 (0이면 이동하지 않음).
            next_sibling_steps (int): 다음 형제로 이동할 횟수 (0이면 이동하지 않음).
        
        Returns:
            ElementHandle or None: 탐색된 요소를 반환하거나 없으면 None.
        """
        try:
            current_element = element
    
            # 부모로 이동
            for _ in range(parent_steps):
                current_element = await current_element.evaluate_handle('el => el.parentElement')
                if not current_element:
                    print("Parent element not found during navigation.")
                    return None
    
            # 이전 형제로 이동
            for _ in range(prev_sibling_steps):
                current_element = await current_element.evaluate_handle('el => el.previousElementSibling')
                if not current_element:
                    print("Previous sibling element not found during navigation.")
                    return None
    
            # 다음 형제로 이동
            for _ in range(next_sibling_steps):
                current_element = await current_element.evaluate_handle('el => el.nextElementSibling')
                if not current_element:
                    print("Next sibling element not found during navigation.")
                    return None
    
            return current_element
    
        except Exception as e:
            print(f"Error navigating elements: {e}")
            return None

    async def get_last_sibling(self, element):
        try:
            # 같은 부 아 마지막 형제를 가져오는 JavaScript 실행
            last_sibling = await element.evaluate_handle('el => el.parentElement.lastElementChild')
            return last_sibling
        except Exception as e:
            print(f"Error getting last sibling: {e}")
            return None

    async def behavior_name(self, element):
        parent_prev_sibling = await self.navigate_element_by_steps(element, parent_steps=1, prev_sibling_steps=1, next_sibling_steps=0)
        passive_name_element = await parent_prev_sibling.query_selector('span.block')
        if passive_name_element:
            passive_name = await passive_name_element.inner_text()
            return passive_name
        else:
            #print("Passive name element not found")
            return None

    async def behavior_condition(self, element):
        try:
            conditions = []
            parent_prev_sibling = await self.navigate_element_by_steps(element, parent_steps=1, prev_sibling_steps=1, next_sibling_steps=0)
            img_elements = await parent_prev_sibling.query_selector_all('img')
            if not img_elements:
                #print("No <img> elements found.")
                return conditions
    
            for img_element in img_elements:
                # 각 이미지의 src 속성 가져오기
                img_src = await img_element.get_attribute('src')
                #print(img_src)
    
                sin_name = None
                for color_key in resource_mapping.keys():
                    if f"{self.img_url}/ui/attr_{color_key}.png" in img_src:
                        sin_name = resource_mapping[color_key]
                        break
    
                if not sin_name:
                    #print(f"No matching Sin Color found for {img_src}.")
                    continue  # No Sin Color found, skip this img_element
    
                # 해당 이미지와 관련된 조건 텍스트 추출
                # 두 번째 부모 요소에서 시작하여 다음 형제를 찾아 조건 텍스트를 추출
                condition_element = await self.navigate_element_by_steps(img_element, parent_steps=3, prev_sibling_steps=0, next_sibling_steps=0)
                if condition_element:
                    # condition_element가 None이 아닌 경우에만 진행
                    condition_text_element = await condition_element.query_selector('.q-pl-xs')
                    if condition_text_element:
                        condition_text = await condition_text_element.inner_text()
                        condition_text = condition_text.strip()
                    else:
                        print(f"No condition text found for {img_src}.")
                        condition_text = ""  # condition_text_element가 None이면 빈 문자열로 처리
                else:
                    print(f"No condition element found for {img_src}.")
                    condition_text = ""  # condition_element가 None이면 빈 문자열로 처리
    
                # 결과를 배열에 추가
                conditions.append(f"{sin_name} {condition_text}")
            return conditions
        except Exception as e:
            print(f"Error extracting conditions: {e}")
            return 
            
    async def behavior_description(self, element, tag_type):
        parent_prev_sibling = await self.navigate_element_by_steps(
            element, parent_steps=1, prev_sibling_steps=1, next_sibling_steps=0
        )
        return await parent_prev_sibling.evaluate(f"""
            (node) => {{
                const behavior_title_node = node.querySelector('.q-pb-sm');
                const behavior_title_parent = behavior_title_node.parentElement;
                const behavior_description = behavior_title_parent.lastElementChild;
                if (!behavior_description) return null;
    
                const results = {{
                    keywords: [], // 키워드 저장
                    description: [] // 설명 저장
                }};
                
                // 1. 키워드 추출
                const keywordElements = behavior_description.querySelectorAll('a[href^="/en/keyword/"]');
                if (!keywordElements) return null;
                keywordElements.forEach(a => {{
                    const span = a.querySelector('span.q-btn__content.text-center.col.items-center.q-anchor--skip.justify-center.row>span');
                    if (span) {{
                        const keywordText = span.textContent.trim();
                        if (keywordText && !results.keywords.includes(keywordText)) {{
                            results.keywords.push(keywordText); // 키워드 저장
                        }}
                    }}
                }});
    
                // 2. 설명 추출
                const descriptionElements = behavior_description.querySelectorAll("{tag_type}");
                let currentEffect = []; // 초기화
                descriptionElements.forEach(span => {{
                    let text = span.textContent?.trim();
                    console.log("origin text:", text);
                    if (!text) return;
                    // 공백과 기호 처리
                    text = text.replace(/\\s*([.,:;!?()\\[\\]])\\s*/g, '$1'); // 기호 앞뒤 공백 제거
                    console.log("after symbol processing:", text);
                    console.log("after spacing character: ", text);
                    if (results.keywords.includes(text)) {{
                        const keywordEffect = 'keyword:' + text;
                        if (!currentEffect.includes(keywordEffect)) {{
                            currentEffect.push(keywordEffect);
                        }}
                    }} else {{
                        currentEffect.push(text);
                    }}
                    console.log("current effect:", currentEffect);
                }});
    
                // 마지막 효과 저장
                if (currentEffect.length > 0) {{
                    const finalDescription = currentEffect.join(' ');
                    console.log("After process:", finalDescription);
                    results.description.push(finalDescription);
                }}
                let processedText = results.description
                    .map(word => word.trim())
                    .join(" ")
                    .replace(/\\s+([,.:])/g, "$1")
                    .replace(/\\s+'/g, "'")
                    .replace(/'\\s+/g, "'")
                    .replace(/\\[\\s+/g, "[")
                    .replace(/\\s+\\]/g, "]")
                    .replace(/\\(\\s+/g, "(")
                    .replace(/\\s+\\)/g, ")")
                    .replace(/\\s+%/g, "%")
                    .replace(/\\s+\\//g, "/")
                    .replace(/\\/\\s+/g, "/")
                    .replace(/\\s+\\/\\s+/g, "/");
                results.description = processedText;
                return results;
            }}
        """)

    async def combine_and_process_texts(self, texts):
        """
        배열 형태의 텍스트 데이터를 합치고 형식을 처리한 문자열을 반환합니다.
        Args:
            texts (list of list): 텍스트 배열 목록
        Returns:
            str: 형식이 처리된 최종 문자열
        """
        processed_sentences = []
        
        for text_array in texts:
            # 빈 문자열을 제외한 나머지 텍스트를 이어붙이기
            sentence = ''.join([word for word in text_array if word != ''])
            sentence = re.sub(r'\\](\\s*)', ']', sentence)
            processed_sentences.append(sentence)
    
        # 문장을 공백으로 연결하여 반환
        return ''.join(processed_sentences)


class Identity:
    """인격 정보를 저장하는 클래스"""
    def __init__(self, identity_id, name=None, rarity=None,\
                 skill_1_type=None, skill_1_sin_affinity=None,\
                 skill_2_type=None, skill_2_sin_affinity=None,\
                 skill_3_type=None, skill_3_sin_affinity=None,\
                 defense_type=None, defense_sin_affinity=None):
        self.identity_id = identity_id
        self.name = name
        self.rarity = rarity
        # 선언만 하는 부분 
        self.skill_1_type = skill_1_type 
        self.skill_1_sin_affinity = skill_1_sin_affinity 
        self.skill_2_type = skill_2_type 
        self.skill_2_sin_affinity = skill_2_sin_affinity 
        self.skill_3_type = skill_3_type 
        self.skill_3_sin_affinity = skill_3_sin_affinity 
        self.defense_type = defense_type 
        self.defense_sin_affinity = defense_sin_affinity
        self.level_max = 50
        self.uptie_max = 4
        self.level_stats = []  # LevelStat 객체 리스트
        self.uptie_stats = [] # UptieStat 객체 리스트
        self.resist_info = None
        self.stagger_threshold = None
        self.skillset = SkillSet()  # Skill정보를 담고 있는 skillset 객체 
        self.passive_infos = []  # 패시브 객체를 담고 있는 배열
        self.support_passive_infos = [] # 서포트 패시 객체를 담고 있는 배열
        self.panic_type = [] # 혼란상태 발생시 나타나는 효과를 담고 있는 객체 배열1

    def update_name(self, name):
        self.name = name

    def update_rarity(self, rarity):
        self.rarity = rarity

    def add_level_stat(self, level, hp, defense):
        self.level_stats.append(LevelStat(level, hp, defense))

    def add_uptie_stat(self, uptie, speed):
        self.uptie_stats.append(UptieStat(uptie, speed))

    def add_resist_info(self, resist_hit, resist_penetrate, resist_slash):
        self.resist_info = ResistInfo(resist_hit, resist_penetrate, resist_slash)

    def add_stagger_threshold(self, first_stagger, second_stagger, third_stagger):
        self.stagger_threshold = StaggerThreshold(first_stagger, second_stagger, third_stagger)

    def add_skill(self, skill_code, skill_name, coin_count, init_level, init_remain, atk_weight):
        self.skillset.add_skill(skill_code, Skill(skill_code, skill_name, coin_count, init_level, init_remain, atk_weight))
        
    def add_skill_uptie_info(self, skill_name, uptie_level, skill_power, coin_power, effect_by_condition):
        self.skillset.add_skill_uptie_info(skill_name, uptie_level, skill_power, coin_power, effect_by_condition)
        
    def update_details(self, page):
        # 세부 정보 업데이트 로직 (가정)
        pass
    #def init_uptie(self, page):
    #    init_uptie = 

# LevelStat 객체 정의
class LevelStat:
    def __init__(self, level, hp=None, defense=None):
        self.level = level
        self.hp = hp
        self.defense = defense
    def update_level(self, level):
        self.level = level
    def update_hp(self, hp):
        self.hp = hp
    def update_defense(self, defense):
        self.defense = defense

# UptieStat 객체 정의
class UptieStat:
    def __init__(self, uptie, speed=None):
        self.uptie = uptie
        self.speed = speed
    def update_speed(self, speed):
        self.speed = speed

# ResistInfo 객체 정의
class ResistInfo:
    def __init__(self, resist_hit=None, resist_penetrate=None, resist_slash=None):
        self.resist_hit = resist_hit
        self.resist_penetrate = resist_penetrate
        self.resist_slash = resist_slash

# ResistInfo 객체 정의
class StaggerThreshold:
    def __init__(self, first_stagger=None, second_stagger=None, third_stagger=None):
        self.first_stagger = first_stagger
        self.second_stagger = second_stagger
        self.third_stagger = third_stagger

class SkillSet:
    def __init__(self):

        self.skill_1_type = ""
        self.skill_2_type = ""
        self.skill_3_type = ""
        self.defense_skill_type = ""

        self.skill_1_sin_affinity = ""
        self.skill_2_sin_affinity = ""
        self.skill_3_sin_affinity = ""
        self.defense_skill_sin_affinity = ""

        self.skill_1_details = []
        self.skill_2_details = []
        self.skill_3_details = []
        self.defense_skill_details = []

    def set_skill_type(self, skill_code, skill_type, sin_affinity):
        if skill_code == "skill 1":
            self.skill_1_type = skill_type
            self.skill_1_sin_affinity = sin_affinity
        elif skill_code == "skill 2":
            self.skill_2_type = skill_type
            self.skill_2_sin_affinity = sin_affinity
        elif skill_code == "skill 3":
            self.skill_3_type = skill_type
            self.skill_3_sin_affinity = sin_affinity
        elif skill_code == "defense":
            self.defense_skill_type = skill_type
            self.defense_skill_sin_affinity = sin_affinity
    
    def add_skill(self, skill_code, skill):
        if skill_code == "skill 1":
            self.skill_1_details.append(skill)
            skill.skill_type = self.skill_1_type
            skill.sin_affinity = self.skill_1_sin_affinity
        elif skill_code == "skill 2":
            self.skill_2_details.append(skill)
            skill.skill_type = self.skill_2_type
            skill.sin_affinity = self.skill_2_sin_affinity
        elif skill_code == "skill 3":
            self.skill_3_details.append(skill)
            skill.skill_type = self.skill_3_type
            skill.sin_affinity = self.skill_3_sin_affinity
        elif skill_code == "defense":
            self.defense_skill_details.append(skill)
            skill.skill_type = self.defense_skill_type 
            skill.sin_affinity = self.defense_skill_sin_affinity

    def get_skill_by_name(self, skill_name):
        for skill_list in [self.skill_1_details, self.skill_2_details, self.skill_3_details, self.defense_skill_details]:
            for skill in skill_list:
                result = skill.is_skill_by_name(skill_name)
                if result:
                    return result
        return None

    def add_skill_uptie_info(self, skill_name, uptie_level, skill_power, coin_power, effect_by_condition):
        skill = self.get_skill_by_name(skill_name)
        skill.add_uptie_info(uptie_level, skill_power, coin_power, effect_by_condition)
        

class Skill:
    def __init__(self, skill_code, skill_name, coin_count, init_level, init_remain, atk_weight):
        self.skill_code = skill_code # "Skill 1"/"Skill 2"/"Skill 3"/"Defense" '''o'''
        self.skill_type = None
        self.sin_affinity = None
        self.skill_name = skill_name # 스킬 이름 '''o'''
        self.coin_count = coin_count # 동전 개수 '''o'''
        self.init_level = init_level # 스킬이 초기 배정받은 레벨, 인격 레벨에 - 1 한 값을 더해주면 스킬의 현 level을 구할 수 있다. '''o'''
        self.init_remain = init_remain #'''o'''
        self.atk_weight = atk_weight #공격 가중치 값이다. 방어스킬에는 없기 때문에 None으로 설정했다. '''o'''
        self.remain_count = init_remain # 실제 스킬이 작동하며 횟수가 차감되 나중에 모두 차감되면 다음 턴에 다시 초기횟수 리필된다.
        self.uptie_info = []  

    def is_skill_by_name(self, skill_name):
        if self.skill_name == skill_name:
            return self
        return None
        
    def add_uptie_info(self, uptie_level, skill_power, coin_power, effects_by_condition):
        self.uptie_info.append(UptieInfo(uptie_level, skill_power, coin_power, effects_by_condition))

class UptieInfo:
    def __init__(self, uptie_level, skill_power, coin_power, effects_by_condition):
        self.uptie_level = uptie_level   #'''o'''
        self.skill_power = skill_power   #'''o'''
        self.coin_power = coin_power     #'''o'''
        self.skill_effect = effects_by_condition

# 추상화된 EffectType 클래스
class EffectType:
    def __init__(self, name, description):
        self.name = name  # Effect 이름
        self.description = description  # Effect 설명

# Passive 클래스
class Passive(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)  # condition이 있는지 여부 확인

# SupportPassive 클래스
class SupportPassive(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)

# PanicType 클래스
class PanicType(EffectType):
    def __init__(self, name, description, condition=None):
        super().__init__(name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)

# Keyword 클래스
class Keyword:
    def __init__(self, keyword, explain=None, dispellable=None):
        self.keyword = keyword
        self.explain = explain
        self.dispellable = dispellable

      
class DataContainer:
    """크롤링 데이터를 관리하는 클래스"""
    def __init__(self):
        self.identities = []

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.level_max

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.uptie_max

    def update_identity_name(self, identity_id, identity_name):
        """특정 ID의 이름을 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_name(identity_name)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_identity_rarity(self, identity_id, rarity):
        """특정 ID의 희귀도를 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_rarity(rarity)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_skill(self, identity_id, skill_number, new_type):
        """
        특정 identity의 skill_type을 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_type: 새 skill_type
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_type = new_type
                elif skill_number == 2:
                    identity.skill_2_type = new_type
                elif skill_number == 3:
                    identity.skill_3_type = new_type
                elif skill_number == "defense":
                    identity.defense_type = new_type
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def update_sin_affinity(self, identity_id, skill_number, new_affinity):
        """
        특정 identity의 sin_affinity를 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_affinity: 새 sin_affinity
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_sin_affinity = new_affinity
                elif skill_number == 2:
                    identity.skill_2_sin_affinity = new_affinity
                elif skill_number == 3:
                    identity.skill_3_sin_affinity = new_affinity
                elif skill_number == "defense":
                    identity.defense_sin_affinity = new_affinity
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def add_identity(self, identity):
        """새로운 인격 데이터를 추가."""
        self.identities.append(identity)

    def add_level_stat(self, identity_id, level, hp, defense):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_level_stat(level, hp, defense)

    def add_uptie_stat(self, identity_id, uptie, speed):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_uptie_stat(uptie, speed)

    def add_resist_info(self, identity_id, resist_hit, resist_penetrate, resist_slash):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_resist_info(resist_hit, resist_penetrate, resist_slash)

    def add_stagger_threshold(self, identity_id, first_stagger, second_stagger, third_stagger):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_stagger_threshold(first_stagger, second_stagger, third_stagger)

    
    def add_skill(self, identity_id, skill_code, skill_name, coin_count, init_level, init_remain, atk_weight):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_code == "skill 1":
                    identity.skillset.set_skill_type(skill_code, identity.skill_1_type, identity.skill_1_sin_affinity)
                elif skill_code == "skill 2":
                    identity.skillset.set_skill_type(skill_code, identity.skill_2_type, identity.skill_2_sin_affinity)
                elif skill_code == "skill 3":
                    identity.skillset.set_skill_type(skill_code, identity.skill_3_type, identity.skill_3_sin_affinity)
                elif skill_code == "defense":
                    identity.skillset.set_skill_type(skill_code, identity.defense_type, identity.defense_sin_affinity)
                identity.add_skill(skill_code, skill_name, coin_count, init_level, init_remain, atk_weight)

    def add_skill_uptie_info(self, identity_id, skill_name, uptie_level, skill_power, coin_power, effects_by_condition):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_skill_uptie_info(skill_name, uptie_level, skill_power, coin_power, effects_by_condition)
        
    def display_identities(self):
        """Identity, ID, Name, Rarity 정보를 출력"""
        if not self.identities:
            print("No identities found.")
        else:
            # 데이터를 리스트로 변환
            table = []
            for idx, identity in enumerate(self.identities, 1):
                table.append([
                    idx, identity.identity_id, identity.name, identity.rarity,
                    identity.skill_1_type, identity.skill_1_sin_affinity,
                    identity.skill_2_type, identity.skill_2_sin_affinity,
                    identity.skill_3_type, identity.skill_3_sin_affinity,
                    identity.defense_type, identity.defense_sin_affinity
                ])
            
            # 테이블 헤더 정의
            headers = [
                "Index", "ID", "Name", "Rarity",
                "Skill_1 Type", "Skill_1 Sin", "Skill_2 Type", "Skill_2 Sin",
                "Skill_3 Type", "Skill_3 Sin", "Defense Type", "Defense Sin"
            ]
            
            # 테이블 출력
            print(tabulate(table, headers=headers, tablefmt="fancy_grid"))

    def export_to_csv(self, filename, fieldnames, data):
        """
        공통적으로 사용 가능한 CSV 내보내기 메서드
        :param filename: 내보낼 파일 이름 (예: 'identity.csv', 'skill.csv')
        :param fieldnames: CSV의 헤더 리스트
        :param data: CSV로 내보낼 데이터 (리스트 형태)
        """
        try:
            # 현재작업 디렉토 기준으 'csv'폴더 없으면 생성
            if not os.path.exists('./csv'):
                os.makedirs('./csv')
            # 파일 경로 설정 (상대경로 'csv' 폴더 안에 저장)
            file_path = os.path.join('./csv', filename)
            with open(file_path, mode='w', newline='', encoding='utf-8') as file:
                writer = csv.DictWriter(file, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(data)
            print(f"Data successfully exported to {filename}")
        except Exception as e:
            print(f"Error exporting to CSV: {e}")

    def get_identity_data(self):
        """
        Identity 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "ID": identity.identity_id,
                "Name": identity.name,
                "Rarity": identity.rarity,
                "Skill_1_Type": identity.skill_1_type,
                "Skill_1_Sin_Affinity": identity.skill_1_sin_affinity,
                "Skill_2_Type": identity.skill_2_type,
                "Skill_2_Sin_Affinity": identity.skill_2_sin_affinity,
                "Skill_3_Type": identity.skill_3_type,
                "Skill_3_Sin_Affinity": identity.skill_3_sin_affinity,
                "Defense_Type": identity.defense_type,
                "Defense_Sin_Affinity": identity.defense_sin_affinity,
            }
            for identity in self.identities
        ]
        
    def export_identities_data(self, filename="identities.csv"):
        """Identity 데이터를 CSV로 내보내는 메서드"""
        identity_fieldnames = [
            "ID", "Name", "Rarity",
            "Skill_1_Type", "Skill_1_Sin_Affinity",
            "Skill_2_Type", "Skill_2_Sin_Affinity",
            "Skill_3_Type", "Skill_3_Sin_Affinity",
            "Defense_Type", "Defense_Sin_Affinity"
        ]
        identity_data = self.get_identity_data()  # Identity 데이터 가져오기
        self.export_to_csv(filename, identity_fieldnames, identity_data)  # CSV로 내보내기

    def load_identities_by_id(self, filename, identity_ids):
        """지정된 identity_ids 목록에 해당하는 Identity 데이터를 CSV에서 읽어옴."""
        identity_ids_set = set(identity_ids) # 중복 제거를 위 Set 사용
        with open(filename, mode='r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                if row["ID"] in identity_ids_set:
                    identity = Identity(
                        identity_id=row["ID"],
                        name=row["Name"],
                        rarity=row["Rarity"],
                        skill_1_type=row["Skill_1_Type"],
                        skill_1_sin_affinity=row["Skill_1_Sin_Affinity"],
                        skill_2_type=row["Skill_2_Type"],
                        skill_2_sin_affinity=row["Skill_2_Sin_Affinity"],
                        skill_3_type=row["Skill_3_Type"],
                        skill_3_sin_affinity=row["Skill_3_Sin_Affinity"],
                        defense_type=row["Defense_Type"],
                        defense_sin_affinity=row["Defense_Sin_Affinity"]
                    )
                    self.identities.append(identity)

    def get_level_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Level": level_stat.level,
                "HP": level_stat.hp,
                "Defense": level_stat.defense,
            }
            for identity in self.identities
            for level_stat in identity.level_stats
        ]

    def export_level_stats_data(self, filename="levelstats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        level_stats_fieldnames = ["Identity_ID", "Level", "HP", "Defense"]
        level_stats_data = self.get_level_stats_data()  # level_stats 데이터 가져오기
        self.export_to_csv(filename, level_stats_fieldnames, level_stats_data)  # CSV로 내보내기

    def get_uptie_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Uptie": uptie_stat.uptie,
                "Speed": uptie_stat.speed,
            }
            for identity in self.identities
            for uptie_stat in identity.uptie_stats
        ]

    def export_uptie_stats_data(self, filename="Uptiestats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        uptie_stats_fieldnames = ["Identity_ID", "Uptie", "Speed"]
        uptie_stats_data = self.get_uptie_stats_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, uptie_stats_fieldnames, uptie_stats_data)  # CSV로 내보내기

    def get_resist_info_data(self):
        """
        모든 Identity의 resist_info 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Resist_Hit": identity.resist_info.resist_hit,
                "Resist_Penetrate": identity.resist_info.resist_penetrate,
                "Resist_Slash": identity.resist_info.resist_slash,
            }
            for identity in self.identities
        ]

    def export_resist_info_data(self, filename="ResistInfos.csv"):
        """ResistInfo 데이터를 CSV로 내보내는 메서드"""
        resist_infos_fieldnames = ["Identity_ID", "Resist_Hit", "Resist_Penetrate", "Resist_Slash"]
        resist_infos_data = self.get_resist_info_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, resist_infos_fieldnames, resist_infos_data)  # CSV로 내보내기

    def export_uptie_stats_data(self, filename="Uptiestats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        uptie_stats_fieldnames = ["Identity_ID", "Uptie", "Speed"]
        uptie_stats_data = self.get_uptie_stats_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, uptie_stats_fieldnames, uptie_stats_data)  # CSV로 내보내기

    def get_stagger_threshold_data(self):
        """
        모든 Identity의 stagger_threshold 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "First_Stagger": identity.stagger_threshold.first_stagger,
                "Second_Stagger": identity.stagger_threshold.second_stagger,
                "Third_Stagger": identity.stagger_threshold.third_stagger,
            }
            for identity in self.identities
        ]

    def export_stagger_threshold_data(self, filename="StaggerThresholds.csv"):
        """StaggerThresholds 데이터를 CSV로 내보내는 메서드"""
        stagger_thresholds_fieldnames = ["Identity_ID", "First_Stagger", "Second_Stagger", "Third_Stagger"]
        stagger_thresholds_data = self.get_stagger_threshold_data()  # stagger_thresholds 데이터 가져오기
        self.export_to_csv(filename, stagger_thresholds_fieldnames, stagger_thresholds_data)  # CSV로 내보내기

    def get_skillset_data(self):
        """모든 Identity의 skillset 데이터를 export_to_csv을 포맷으로 변환"""
        return [
            {
                "Identity_ID": identity.identity_id,
                "Skill_Code": skill.skill_code,
                "Skill_Type": skill.skill_type,
                "Sin_Affinity": skill.sin_affinity,
                "Skill_Name": skill.skill_name,
                "Coin_Count": skill.coin_count,
                "Init_Level": skill.init_level,
                "Init_Remain": skill.init_remain,
                "Atk_Weight": skill.atk_weight,
                "Uptie_Level": uptie_info.uptie_level,
                "Skill_Power": uptie_info.skill_power,
                "Coin_Power": uptie_info.coin_power,
                "Effect_Data": json.dumps(uptie_info.skill_effect)
            }
            for identity in self.identities
            for skill_list in [identity.skillset.skill_1_details, identity.skillset.skill_2_details, identity.skillset.skill_3_details, identity.skillset.defense_skill_details]
            for skill in skill_list
            for uptie_info in skill.uptie_info
        ]

    def export_identity_skillset_data(self, filename="IdentitiesSkills.csv"):
        """IdentitiesSkills 데이터를 CSV로 내보내는 메서드"""
        identities_skills_fieldnames = ["Identity_ID", "Skill_Code", "Skill_Type", "Sin_Affinity", "Skill_Name", "Coin_Count", "Init_Level", "Init_Remain", "Atk_Weight", "Uptie_Level", "Skill_Power", "Coin_Power", "Effect_Data"]
        identities_skills_data = self.get_skillset_data()
        self.export_to_csv(filename, identities_skills_fieldnames, identities_skills_data)
            


async def main():
    data_container = DataContainer()
    scraper = Scraper(container=data_container, base_url="https://limbus.kusoge.xyz", img_url="https://img.kusoge.xyz/limbus/img")
    await scraper.scrape()

await main() 

Bloodfeast
[]
{'keywords': ['Bloodfeast', 'Bleed'], 'description': 'If this unit is on field, or is one of the units that can appear on this Stage, increase keyword:Bloodfeast value by the amount of keyword:Bleed damage collectively received by every unit. When this unit enters the field, the sleeping blood drenching the battlefield will fill the surface.'}
Hardblood Thorn
['Lust x3 Owned', 'Envy x2 Owned']
{'keywords': ['Bleed', 'Bloodfeast'], 'description': 'When an other ally takes keyword:Bleed damage or consumes keyword:Bloodfeast, gain 1 blooming[thornyfall_panic RodionFirst](3 times per turn) Heal HP on self by 20% of the damage dealt with base Skills (max 10 per Skill)'}
"Blossom with Blood..."
['Lust x3 Owned', 'Envy x3 Owned']
{'keywords': ['Bleed'], 'description': "To 1 ally with the highest keyword:Bleed Potency at Turn End: reduce the ally's Potency by 6 max, and apply (Potency reduced/2) blooming[thornyfall_panic RodionFirst](rounded down)"}
Panic
[]
{'keywords': [], 'des

In [93]:
import os
import csv
import re
import json
import numpy as np
import pandas as pd
from playwright.async_api import async_playwright
from tabulate import tabulate

# Sin Color 이름 변환을 위한 매핑
resource_mapping = {
    "AZURE": "Gloom",
    "VIOLET": "Envy",
    "AMBER": "Sloth",
    "SHAMROCK": "Gluttony",
    "CRIMSON": "Wrath",
    "SCARLET": "Lust",
    "INDIGO": "Pride"
}

def convert_resource(resource_name):
    """Sin Color 이름을 변환하는 함수. 매핑되지 않은 경우 원래 이름을 반환."""
    return resource_mapping.get(resource_name, resource_name)

def find_sin_for_color(url):
    # URL에 color_name을 추출
    color_name = url.split('/')[-1].split('.')[0].replace("attr_", "")
    # 해당 색상이 resource_mapping에 있는 확인
    for color, sin in resource_mapping.items():
        if color_name in color:
            return sin
    return None


class Scraper:
    """웹 페이지 크롤링을 위한 클래스"""
    def __init__(self, container, base_url, img_url):
        self.container = container
        self.base_url = base_url
        self.img_url = img_url
        self.uptie_level = 1

    async def handle_request(self, route, request):
        """이미지와 폰트 리소스 로드를 차단."""
        if request.resource_type =="image":
            #print(f"{request.url}")
            img_url = request.url
            if  re.match(f"{self.img_url}/ui/attr.*\\.png", img_url):
                await route.continue_()
            else:
                await route.fulfill(
                    status=200,
                    content_type="image/png",
                    body=b""
                )
        elif request.resource_type == "font":
            await route.abort()
        else:
            await route.continue_()
            
    async def scrape(self):
        """크롤링 메인 함수"""
        async with async_playwright() as p:
            browser = await p.chromium.launch(headless=True)
            page = await browser.new_page()
            #await page.evaluate('window.scrollTo(0, document.body.scrollHeight)')
            await page.route("**/*", self.handle_request)
            # Step 1: /identity 페이지에서 기본 정보 크롤링
            #await self.crawl_identity_page(page)

            # Step 2: /identity/{identity_id} 상세 페이지 크롤링
            #identity_ids_to_load = ["10104", "10310", "10710","10808", "10911", "11005"]
            #identity_ids_to_load = ["10104", "10911"]#,"10310", "10710","10808", "11005", "11210"]
            #identity_ids_to_load = ["10911","10310"]#, "10710","10808", "11005", "11210"]
            identity_ids_to_load = ["10911"]#, "10611"]#,"10711"]#, "10710","10808", "11005", "11210"]
            self.container.load_identities_by_id('./csv/identities.csv', identity_ids_to_load)
            await self.crawl_identity_details_for_all(page)
            #self.container.display_identities()
            #self.container.export_identities_data("identities.csv")
            #self.container.export_level_stats_data("Levelstats.csv")
            #self.container.export_uptie_stats_data("Uptiestats.csv")
            #self.container.export_resist_info_data("ResistInfos.csv")
            #self.container.export_stagger_threshold_data("StaggerThresholds.csv")
            #self.container.export_identity_skillset_data("IdentitiesSkills.csv")
            page.set_default_timeout(10000)
            #await page.wait_for_timeout(100000)
            await browser.close()

    async def get_identity_id_from_row(self, row):
        """테이블 행(row)에서 인격 ID를 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_url = await identity_cell.get_attribute('href')
            identity_id = identity_url.split('/identity/')[-1]
            return identity_id
        return "Unknown"

    async def get_identity_name_from_row(self, row):
        """테이블 행(row)에서 인격 이름을 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_name = (await identity_cell.inner_text()).replace("\n", " ").strip()
            return identity_name
        return "Unknown"
        
    async def crawl_identity_page(self, page):
        """/identity 페이지로 이동하여 인격 기본 정보를 추출."""
        await page.goto(self.base_url + "/identity")
        await self.extract_identity_page(page)

    async def extract_identity_page(self, page):
        """/identity 페이지의 모든 테이블 행에서 인격 정보를 추출."""
        identity_rows = await page.query_selector_all('tr')
        if len(identity_rows) > 1:
            for idx, identity_row in enumerate(identity_rows):
                if idx == 0:
                    continue # 0번째요소이름을 보여주는 열은 건너뜀
                await self.scrape_identity_name_and_identity_id(identity_row)
                await self.extract_rarity(identity_row)
                for i in range(1, 4):  # 스킬1~3 정보 추출
                    await self.extract_atk_and_resource(identity_row, i)
                await self.extract_defense_and_resource(identity_row)

    async def scrape_identity_name_and_identity_id(self, row):
        """인격 번호와 이름을 추출하여 DataContainer에 저장."""
        try:
            identity_id = await self.get_identity_id_from_row(row)
            identity_name = await self.get_identity_name_from_row(row)
            identity = Identity(identity_id)
            self.container.add_identity(identity)
            self.container.update_identity_name(identity_id, identity_name)
        except Exception as e:
            print(f"Error extracting identity name and identity id: {e}")

    async def get_element_attribute(self, element, selector, attribute):
        """요소에서 지정된 속성(attribute)을 추출."""
        try:
            target_element = await element.query_selector(selector)
            if target_element:
                return await target_element.get_attribute(attribute)
        except Exception as e:
            print(f"Error getting element attribute: {e}")
        return None
    
    async def extract_rarity(self, row):
        """테이블 행(row)에서 희귀도를 추출하여 업데이트."""
        try:
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
                sibling = await parent.evaluate_handle('node => node.nextElementSibling')
                if sibling:
                    rarity_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/rarity_"]', 'src')
                    if rarity_img_src:
                        rarity_level = rarity_img_src.split('_')[-1].split('.')[0]  # rarity_1.png -> 1
                        identity_id = await self.get_identity_id_from_row(row)
                        self.container.update_identity_rarity(identity_id, rarity_level)
                    else:
                        print("Rarity image not found in the next sibling TD.")
        except Exception as e: 
            print(f"Error extracting rarity: {e}")

        return "Unknown"

    async def extract_atk_and_resource(self, row, skill_number):
        """ 공격 타입(ATK Type)과 자원(Attribute) 추출 함수 - 스킬 번호에 따라 처리 """
        try:
            #print(f"Attempting to locate skill{skill_number} type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 해당 스킬 번호에 맞는 형제로 이동
                sibling = parent
                for _ in range(skill_number + 1):  # 스킬 번호에 맞게 이동
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                atk_type, resource = "Unknown", "Unknown"
        
                if sibling:
                    # 공격 타입 이미지 (atk_type) 찾기
                    atk_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/atk_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if atk_type_img_src:
                        atk_type = atk_type_img_src.split('_')[-1].split('.')[0]  # atk_type_SLASH.png -> SLASH
                        #print(f"Extracted attack type: {atk_type}")
                        self.container.update_skill(identity_id, skill_number, atk_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted resource: {resource}")
                        
                        # 리소스를 변환
                        resource = convert_resource(resource)
                        #print(f"Converted resource: {resource}")
                        self.container.update_sin_affinity(identity_id, skill_number, resource)
                        
        except Exception as e:
            print(f"Error extracting skill{skill_number} type and resource: {e}")

    async def extract_defense_and_resource(self, row):
        """ 방어 타입(Def Type)과 자원(Attribute) 추출 함수 """
        try:
            #print("Attempting to locate defense type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 다섯 번째 형제 td로 이동 (방어 관련 정보가 있는 <td>)
                sibling = parent
                for _ in range(5):  # 방어 정보는 5번째 형제에 있음
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                def_type, def_resource = "Unknown", "Unknown"
        
                if sibling:
                    # 방어 타입 이미지 (def_type) 찾기
                    def_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/def_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if def_type_img_src:
                        def_type = def_type_img_src.split('_')[-1].split('.')[0]  # def_type_GUARD.png -> GUARD
                        #print(f"Extracted defense type: {def_type}")
                        self.container.update_skill(identity_id, 'defense', def_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        def_resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted defense resource: {def_resource}")
                        
                        # 방어 리소스를 변환
                        def_resource = convert_resource(def_resource)
                        #print(f"Converted defense resource: {def_resource}")
                        self.container.update_sin_affinity(identity_id, 'defense', def_resource)
 
        except Exception as e:
            print(f"Error extracting defense type and resource: {e}")

    async def crawl_identity_details_for_all(self, page):
        """모든 인격의 세부 정보를 크롤링."""
        for identity in self.container.identities:
            await page.wait_for_load_state("networkidle")
            await page.goto(self.base_url + "/identity/"+identity.identity_id)
            """ 레벨과 동기화와 관련 없는 참관타 저항은 밑에서 변경하는 것과 연관성이 적기 때문에 위에서 처리한다.
            print(f"Identity_id: {identity.identity_id}")
            results = []
            resist_type_imgs = [f"{self.img_url}/ui/atk_type_HIT.png",
                                f"{self.img_url}/ui/atk_type_PENETRATE.png",
                                f"{self.img_url}/ui/atk_type_SLASH.png"]
            for resist_type_img in resist_type_imgs:
                result = await self.extract_resist_type(page, resist_type_img, 1)
                results.append(result)
            self.container.add_resist_info(identity.identity_id, results[0], results[1], results[2])
            """
            """
            # Stagger Threshold 값 추출
            stagger_threshold = await self.extract_stagger_threshold_values(page)
            self.container.add_stagger_threshold(identity.identity_id, stagger_threshold[0], stagger_threshold[1], stagger_threshold[2])
            """
            await self.initialize_uptie_and_level(page)
            """ 레벨에 따른 level, hp, defense 다르게 되어 있는 것을 출력 확인 객체로 들어가서 csv로 옮기는 과정까지 진행
            for i in range(1, identity.level_max+1):
                await self.move_slider_to_value(page,i)
                await self.extract_level_status(page, identity.identity_id, i)
            """
            """ 동기화에 따른 speed 다르게 되어 있는 것을 출력 확인 객체로 들어가서 csv로 옮기는 과정까지 진행
            for i in range(1, identity.uptie_max+1):
                await self.uptie_click(page,i)
                await self.extract_uptie_status(page, identity.identity_id, i)
            """
            """
                인격에 대한 skill 정보를 가져오는 메소드 위에서 출력하는 csv파일로 인격이 가지고 있는 스킬 정보를 내보낸다.
            await self.crawl_skills(page, identity.identity_id)
            """
            await self.crawl_influence_behavior(page, identity.identity_id)
            
    async def initialize_uptie_and_level(self, page):
        await self.uptie_click(page, 1)
        await self.move_slider_to_value(page, 1)
        self.uptie_level = 1
        #return 1
    
    async def uptie_click(self, page, phase):
        fab_button_selector = ".q-btn.q-btn--fab"
        await page.wait_for_selector(fab_button_selector)
        await page.click(fab_button_selector)
        #print(f"[{setup_uptie_button}] 상위 동기화 버튼 클릭 완료")
        setup_uptie_button = f'a:has(img[src="{self.img_url}/ui/uptie_{phase}.png"])'
        await page.wait_for_selector(setup_uptie_button)
        await page.click(setup_uptie_button)
        #print(f"{phase} 동기화 버튼 클릭 완료")

    async def move_slider_to_value(self, page, target_value):
        """
        슬라이더를 주어진 값으로 이동시킵니다.
        target_value는 슬라이더의 최소값부터 최대값 사이여야 합니다.
        """
        slider_selector = ".q-slider"
        # 슬라이더의 범위 확인
        min_value = int(await page.get_attribute(slider_selector, "aria-valuemin"))
        max_value = int(await page.get_attribute(slider_selector, "aria-valuemax"))
    
        if target_value < min_value or target_value > max_value:
            raise ValueError(f"Target value {target_value} is out of bounds ({min_value}, {max_value})")
    
        # 슬라이더 thumb 요소와 위치 가져오기
        slider_thumb = await page.query_selector(f"{slider_selector} .q-slider__thumb")
        thumb_box = await slider_thumb.bounding_box()
        
        # 슬라이더의 현재 위치 및 목표 위치 계산
        track = await page.query_selector(f"{slider_selector} .q-slider__track")
        track_box = await track.bounding_box()
        
        # 목표 x 위치 계산 (슬라이더 트랙 내에서 비율 기반)
        target_percentage = (target_value - min_value) / (max_value - min_value)
        target_x = track_box["x"] + target_percentage * track_box["width"]
        current_x = thumb_box["x"] + thumb_box["width"] / 2
        y = thumb_box["y"] + thumb_box["height"] / 2
    
        # 슬라이더를 드래그하여 목표 위치로 이동
        await page.mouse.move(current_x, y)
        await page.mouse.down()
        await page.mouse.move(target_x, y, steps=10)  # 부드럽게 이동
        await page.mouse.up()
    
        #print(f"슬라이더를 {target_value}로 이동 완료.")
    
        # 슬라이더 값 확인 및 강제 이벤트 트리거
        await page.evaluate(
            f"""
            (slider) => {{
                slider.dispatchEvent(new Event('input'));  // oninput 이벤트 트리거
                slider.dispatchEvent(new Event('change'));  // onchange 이벤트 트리거
            }}
            """,
            await page.query_selector(slider_selector)
        )
    
        # 이동 후 값 확인
        current_value = await page.get_attribute(slider_selector, "aria-valuenow")
        if current_value != str(target_value):
            print(f"경고: 슬라이더 값이 예상과 다릅니다. 현재 값: {current_value}, 예상 값: {target_value}")

    async def get_second_status_element(self, page):
        """
        두 번째 'Status' 요소에 접근하여 반환하는 메소드
        """
        status_elements = await page.query_selector_all("text=Status")
        if len(status_elements) > 1:
            second_status_element = status_elements[1]
            #print("두 번째 'Status' 요소에 접근합니다.")
            return second_status_element
        else:
            #print("두 번째 'Status' 요소가 존재하지 않습니다.")
            return None

    async def extract_level_status(self, page, identity_id, level):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 레벨에 따라 HP와 Defense가 변한다.
        hp_value = await self.extract_hp(second_status_element)
        defense_value = await self.extract_defense(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Level={level}, HP={hp_value}, Defense={defense_value}")
        self.container.add_level_stat(identity_id,level, hp_value, defense_value)
    
    async def extract_hp(self, status_element):
        # HP 추출
        hp_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].children[0].nextElementSibling.textContent.trim()"
        )
        return hp_value
    
    async def extract_defense(self, status_element):
        # Defense 추출
        defense_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return defense_value

    async def extract_uptie_status(self, page, identity_id, uptie):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 uptie에 따라 speed는 변한다.
        speed_value = await self.extract_speed(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Uptie={uptie}, Speed={speed_value}")
        self.container.add_uptie_stat(identity_id,uptie, speed_value)

    async def extract_speed(self, status_element):
        # Speed 추출
        speed_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return speed_value

    async def extract_resist_type(self, page, image_src, depth):
        # 이미지 요소 찾기
        #print(image_src)
        image = await page.query_selector(f'img[src="{image_src}"]')
        #print(image)
        if not image:
            return "Unknown"
    
        # 이미지의 부모 요소를 찾아서 올라감
        parent = image
        for _ in range(depth):
            parent = await parent.evaluate_handle('node => node.parentElement')
            if not parent:
                return "Unknown"
    
        # 부모의 형제 요소 찾기
        sibling = await parent.evaluate_handle('node => node.nextElementSibling')
        
        if sibling:
            value = await sibling.evaluate('node => node.textContent')
            if value:
                return value.strip()
        
        return "Unknown"

    async def extract_stagger_threshold_values(self, page):
        # 'div.row.center.flex-center.q-pt-xs' 요소들 찾기
        elements = await page.query_selector_all('div.row.center.flex-center.q-pt-xs')
        
        # 각 요소의 텍스트 내용 출력하여 'Stagger Threshold' 요소를 찾기
        for element in elements:
            text = await element.text_content()
            
            if text and "Stagger Threshold" in text:
                # 'Stagger Threshold'가 있는 요소를 찾았다면, 그 이후 형제 요소들에서 값을 추출
                parent = await element.evaluate_handle('node => node.parentElement')
                sibling_elements = await parent.query_selector_all('div.row.justify-evenly div')  # Threshold 값이 들어있는 div들
                
                stagger_values = []
                for sibling in sibling_elements:
                    text_content = await sibling.text_content()
                    match = re.search(r'(\d+%)', text_content)
                    if match:
                        stagger_values.append(match.group(1))  # 퍼센트 값만 리스트에 추가
                while len(stagger_values) < 3:
                    stagger_values.append(np.nan)
                #print(stagger_values)
                return stagger_values  # 추출된 값(또는 np.nan 포함)을 반환
        
        return [np.nan, np.nan, np.nan]  # 만약 'Stagger Threshold'를 찾지 못한 경우 기본값 반환

    async def crawl_skills(self, page, identity_id):
        await self.crawl_skill(page, identity_id, "skill 1")
        await self.crawl_skill(page, identity_id, "skill 2")
        await self.crawl_skill(page, identity_id, "skill 3")
        await self.crawl_skill(page, identity_id, "defense")


    async def crawl_skill(self, page, identity_id, skill_type):
        skills_elements = await page.query_selector_all('div.skilltag')
        for skill_element in skills_elements:
            skill_text = await skill_element.text_content()
            if skill_text.strip().lower() == skill_type:
                await self.process_skill(skill_element, page, identity_id, skill_type)
        
    async def process_skill(self, skill_element, page, identity_id, skill_type):
        # skill_type이 "skill 3"일 경우 uptie_value가 3 이상일 때만 정보를 저장
        for uptie_value in range(1, 5):  # Uptie levels 1 to 4
            # "skill 3"은 uptie_value가 3 이상일 때만 처리
            if skill_type == "skill 3" and uptie_value < 3:
                continue  # skip if uptie_value is less than 3 for skill 3
    
            # uptie_level에 맞게 클릭
            await self.uptie_click(page, uptie_value)
            if skill_type != "skill 3" and uptie_value > 1:
                # Uptie 정보 갱신
                await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
                continue
            elif skill_type == "skill 3" and uptie_value > 3:
                # Uptie 정보 갱신
                await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
                continue
            # 스킬의 세부 정보를 추출
            skill_details = await self.extract_skill_details(skill_element, skill_type)
            skill_name = skill_details["skill_name"]
    
            # 스킬 정보를 저장
            self.container.add_skill(
                identity_id, skill_type, 
                skill_name, skill_details["coin_count"],
                skill_details["init_level"], skill_details["init_remain"], 
                skill_details["atk_weight"]
            )
            print(f"Added skill: {skill_type}:{skill_name}")
    
            # Uptie 정보 갱신
            await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
    
    async def extract_skill_details(self, skill_element, skill_type):
        # 스킬 세부 정보를 추출하는 함수
        skill_name = await self.skill_name(skill_element)
        coin_count = await self.coin_count(skill_element, ".col-auto > .column > .row")
        init_level = await self.init_level(skill_element)
        init_remain = await self.init_remain(skill_element, skill_type, "div.row.justify-start > div.attr.flex.flex-center > div.q-px-sm")
        atk_weight = await self.atk_weight(skill_element, skill_type)
        return {
            "skill_name": skill_name,
            "coin_count": coin_count,
            "init_level": init_level,
            "init_remain": init_remain,
            "atk_weight": atk_weight
        }
    
    async def update_uptie_info(self, page, skill_element, identity_id, skill_name, skill_type, uptie_value):
        # Uptie 정보를 추출하고 업데이트하는 함수
        uptie_info = await self.extract_uptie_info(skill_element)
        effect_data = await self.skill_effects(skill_element)
        self.container.add_skill_uptie_info(
            identity_id, skill_name, 
            uptie_value, uptie_info["skill_power"], uptie_info["coin_power"], 
            effect_data.get("effectsByCondition")
        )
        print(
            f"Updated Uptie for {skill_name} (Uptie Level: {uptie_value}): "
            f"Skill Power: {uptie_info['skill_power']}, Coin Power: {uptie_info['coin_power']} "
            f"Skill Effect: {effect_data.get("effectsByCondition")}"
        )

    async def extract_uptie_info(self, skill_element):
        skill_data = await self.skill_and_coin_power(skill_element)
        return {
            "skill_power": skill_data.get("skillPower"),
            "coin_power": skill_data.get("coinPower")
        }
        
    async def crawl_influence_behavior(self, page, identity_id):
        influence_behaviors = await page.query_selector_all('div[align="right"]')
        # 텍스트를 기준으 추출하고 메소드로 파라미터 들어가 정보를 받는다.
        for influence_behavior in influence_behaviors:
            active_type = await influence_behavior.inner_text()
            active_type = active_type.strip()

            if active_type == "PASSIVE":
                """
                # PASSIVE에 대한 처리
                print(f"Processing PASSIVE logic for: {active_type}")
                # CALL method for PASSIVE
                """
                passive_data = await self.passive_effects(influence_behavior)
            elif active_type == "SUPPORT PASSIVE":
                """
                # SUPPORT PASSIVE에 대한 처리
                print(f"Processing SUPPORT PASSIVE logic for: {active_type}")
                # CALL method for SUPPORT PASSIVE
                """
                support_passive_data = await self.support_passive_effects(influence_behavior)
            elif active_type == "PANIC TYPE":
                """
                # PANIC TYPE에 대한 처리
                print(f"Processing PANIC TYPE logic for:{active_type}")
                # CALL method for PANIC TYPE
                """
                panic_type_data = await self.panic_effects(influence_behavior)

    async def skill_and_coin_power(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                // 조부모로 이동
                const grandparent = node.parentElement.parentElement;
                // 이전 형제 탐색
                const previousSibling = grandparent.previousElementSibling;
                if (!previousSibling) return {{ skillPower: null, coinPower: null }}; // 이전 형제가 없으면 기본값 반환
                // 이전 형제 내부의 .row.justify-around > div > div 탐색
                const targetDivs = previousSibling.querySelectorAll(".row.justify-around > div");
                if (targetDivs.length < 3) return {{ skillPower: null, coinPower: null }}; // 배열 길이 확인
                // 1st div: 스킬 위력
                const skillPower = targetDivs[0]?.querySelector('div')?.textContent?.trim() || null; 
                // 3rd div: 코인 파워 값 (3번째 div에서 텍스트 추출)
                const coinPower = targetDivs[2]?.textContent?.trim() || null;
                return {{ skillPower, coinPower }};
            }}
        """)


    async def coin_count(self, skill_element, hierarchy_selector):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector('{hierarchy_selector}');
                if (!targetElement) return 0;
                return targetElement.childElementCount;
            }}
        """)

    async def init_remain(self, skill_element, skill_type, hierarchy_selector):
        if skill_type == "defense":
            return -1
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector('{hierarchy_selector}');
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def skill_name(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector(".block");
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def init_level(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector("div.text-bold.text-h6.h6.q-pl-xs.q-pr-md");
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def atk_weight(self, skill_element, skill_type):
        if skill_type == "defense":
            return 0
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector("div.text-bold.text-h6.h6.q-pl-xs.q-pr-md");
                const doubleNextSibling = targetElement.nextElementSibling.nextElementSibling;
                if (!doubleNextSibling) return 0;
                return doubleNextSibling.textContent?.trim();
            }}
        """)

    async def skill_effects(self, skill_element):
        return await skill_element.evaluate("""
            node => {
                const parent = node.parentElement;
                const doubleNextSibling = parent.nextElementSibling?.nextElementSibling;
                if (!doubleNextSibling) return null;
    
                // doubleNextSibling에서 첫 번째 div.col 요소 찾기
                const textNode = doubleNextSibling.querySelector("div.col");
                if (!textNode) return null;
    
                const results = {
                    keywords: [], // 키워드 저장
                    conditions: [], // 조건 저장
                    effectsByCondition: {} // 조건별 효과
                };
    
                // 1. 조건 추출 (대괄호 텍스트)
                const conditionElements = textNode.querySelectorAll("span");
                conditionElements.forEach(span => {
                    let text = span.textContent?.trim();
                    if (text?.startsWith("[") && text?.endsWith("]")) {
                        // 부모의 전형제 노드 탐색
                        const sibling = span.parentElement.previousElementSibling;
                        if (sibling) {
                            const emblemImage = sibling.querySelector("img.emblem");
                            console.log("Sibling element found:", sibling); // 디버깅용
                            if (emblemImage) {
                                const src = emblemImage.getAttribute("src");
                                console.log("Image source:", src); // 디버깅용
                                const match = src.match(/ui_num_(\\d+)\\.png$/); // 정규식 수정
                                if (match) {
                                    const number = match[1]; // 숫자 추출
                                    text = `[Coin ${number} ${text.replace("[", "").replace("]", "")}]`;
                                    console.log("Updated condition:", text); // 디버깅용
                                }
                            }
                        }
                        results.conditions.push(text);
                        results.effectsByCondition[text] = []; // 조건별 효과를 초기화
                    }
                });
    
                // 2. 키워드 추출
                const keywordElements = textNode.querySelectorAll('a[href^="/en/keyword/"]');
                keywordElements.forEach(a => {
                    const span = a.querySelector('span.q-btn__content.text-center.col.items-center.q-anchor--skip.justify-center.row>span');
                    if (span) {
                        const keywordText = span.textContent.trim();
                        if (keywordText && !results.keywords.includes(keywordText)) {
                            results.keywords.push(keywordText); // 키워드 저장
                        }
                    }
                });
    
                // 3. 효과 추출
                const effectElements = textNode.querySelectorAll("span");
                let currentCondition = null;
                let currentEffect = []; // 초기화
                effectElements.forEach(span => {
                    let text = span.textContent?.trim();
                    if (!text) return;
    
                    // 조건 발견 시 currentCondition 변경
                    if (text.startsWith("[") && text.endsWith("]")) {
                        // 이전 조건의 효과 저장
                        if (currentCondition && currentEffect.length > 0) {
                            // 부모의 전형제 노드 탐색
                            const sibling = span.parentElement.previousElementSibling;
                            if (sibling) {
                                const emblemImage = sibling.querySelector("img.emblem");
                                if (emblemImage) {
                                    const src = emblemImage.getAttribute("src");
                                    const match = src.match(/ui_num_(\\d+)\\.png$/); // 정규식 수정
                                    if (match) {
                                        const number = match[1]; // 숫자 추출
                                        text = `[Coin ${number} ${text.replace("[", "").replace("]", "")}]`;
                                    }
                                }
                            }
                            results.effectsByCondition[currentCondition].push(currentEffect.join(' '));
                            currentEffect = []; // 초기화
                        }
                        currentCondition = text;
                        return;
                    }
    
                    // 현재 조건의 효과 처리
                    if (currentCondition) {
                        if (results.keywords.includes(text)) {
                            const keywordEffect = 'keyword:' + text;
                            if (!currentEffect.includes(keywordEffect)) {
                                currentEffect.push(keywordEffect);
                            }
                        } else {
                            currentEffect.push(text);
                        }
                    }
                });
    
                // 마지막 효과 저장
                if (currentCondition && currentEffect.length > 0) {
                    results.effectsByCondition[currentCondition].push(currentEffect.join(' '));
                }
    
                return results;
            }
        """)

    async def passive_effects(self, passive_element):
        passive_name = await self.behavior_name(passive_element)
        passive_conditions = await self.behavior_condition(passive_element)
        passive_description = await self.behavior_description(passive_element, "span")
        print(passive_name)
        print(passive_conditions)
        print(passive_description)
    async def support_passive_effects(self, support_passive_element):
        support_passive_name = await self.behavior_name(support_passive_element)
        support_passive_conditions = await self.behavior_condition(support_passive_element)
        support_passive_description = await self.behavior_description(support_passive_element, "span")
        print(support_passive_name)
        print(support_passive_conditions)
        print(support_passive_description)
    async def panic_effects(self, panic_element):
        panic_type = await self.behavior_name(panic_element)
        panic_conditions = await self.behavior_condition(panic_element)
        panic_description = await self.behavior_description(panic_element, "div")
        print(panic_type)
        print(panic_conditions)
        print(panic_description)

    async def navigate_element_by_steps(self, element, parent_steps=0, prev_sibling_steps=0, next_sibling_steps=0):
        """
        주어진 `element`에서 부모, 이전 형제, 또는 다음 형제로 지정된 횟수만큼 이동하여 ElementHandle 반환.
        
        Args:
            element (ElementHandle): 탐색 기준이 되는 요소.
            parent_steps (int): 부모로 이동할 횟수 (0이면 이동하지 않음).
            prev_sibling_steps (int): 이전 형제로 이동할 횟수 (0이면 이동하지 않음).
            next_sibling_steps (int): 다음 형제로 이동할 횟수 (0이면 이동하지 않음).
        
        Returns:
            ElementHandle or None: 탐색된 요소를 반환하거나 없으면 None.
        """
        try:
            current_element = element
    
            # 부모로 이동
            for _ in range(parent_steps):
                current_element = await current_element.evaluate_handle('el => el.parentElement')
                if not current_element:
                    print("Parent element not found during navigation.")
                    return None
    
            # 이전 형제로 이동
            for _ in range(prev_sibling_steps):
                current_element = await current_element.evaluate_handle('el => el.previousElementSibling')
                if not current_element:
                    print("Previous sibling element not found during navigation.")
                    return None
    
            # 다음 형제로 이동
            for _ in range(next_sibling_steps):
                current_element = await current_element.evaluate_handle('el => el.nextElementSibling')
                if not current_element:
                    print("Next sibling element not found during navigation.")
                    return None
    
            return current_element
    
        except Exception as e:
            print(f"Error navigating elements: {e}")
            return None

    async def get_last_sibling(self, element):
        try:
            # 같은 부 아 마지막 형제를 가져오는 JavaScript 실행
            last_sibling = await element.evaluate_handle('el => el.parentElement.lastElementChild')
            return last_sibling
        except Exception as e:
            print(f"Error getting last sibling: {e}")
            return None

    async def behavior_name(self, element):
        parent_prev_sibling = await self.navigate_element_by_steps(element, parent_steps=1, prev_sibling_steps=1, next_sibling_steps=0)
        passive_name_element = await parent_prev_sibling.query_selector('span.block')
        if passive_name_element:
            passive_name = await passive_name_element.inner_text()
            return passive_name
        else:
            #print("Passive name element not found")
            return None

    async def behavior_condition(self, element):
        try:
            conditions = []
            parent_prev_sibling = await self.navigate_element_by_steps(element, parent_steps=1, prev_sibling_steps=1, next_sibling_steps=0)
            img_elements = await parent_prev_sibling.query_selector_all('img')
            if not img_elements:
                #print("No <img> elements found.")
                return conditions
    
            for img_element in img_elements:
                # 각 이미지의 src 속성 가져오기
                img_src = await img_element.get_attribute('src')
                #print(img_src)
    
                sin_name = None
                for color_key in resource_mapping.keys():
                    if f"{self.img_url}/ui/attr_{color_key}.png" in img_src:
                        sin_name = resource_mapping[color_key]
                        break
    
                if not sin_name:
                    #print(f"No matching Sin Color found for {img_src}.")
                    continue  # No Sin Color found, skip this img_element
    
                # 해당 이미지와 관련된 조건 텍스트 추출
                # 두 번째 부모 요소에서 시작하여 다음 형제를 찾아 조건 텍스트를 추출
                condition_element = await self.navigate_element_by_steps(img_element, parent_steps=3, prev_sibling_steps=0, next_sibling_steps=0)
                if condition_element:
                    # condition_element가 None이 아닌 경우에만 진행
                    condition_text_element = await condition_element.query_selector('.q-pl-xs')
                    if condition_text_element:
                        condition_text = await condition_text_element.inner_text()
                        condition_text = condition_text.strip()
                    else:
                        print(f"No condition text found for {img_src}.")
                        condition_text = ""  # condition_text_element가 None이면 빈 문자열로 처리
                else:
                    print(f"No condition element found for {img_src}.")
                    condition_text = ""  # condition_element가 None이면 빈 문자열로 처리
    
                # 결과를 배열에 추가
                conditions.append(f"{sin_name} {condition_text}")
            return conditions
        except Exception as e:
            print(f"Error extracting conditions: {e}")
            return 
            
    async def behavior_description(self, element, tag_type):
        parent_prev_sibling = await self.navigate_element_by_steps(
            element, parent_steps=1, prev_sibling_steps=1, next_sibling_steps=0
        )
        return await parent_prev_sibling.evaluate(f"""
            (node) => {{
                const behavior_title_node = node.querySelector('.q-pb-sm');
                const behavior_title_parent = behavior_title_node.parentElement;
                const behavior_description = behavior_title_parent.lastElementChild;
                if (!behavior_description) return null;
    
                const results = {{
                    keywords: [], // 키워드 저장
                    description: [] // 설명 저장
                }};
                
                // 1. 키워드 추출
                const keywordElements = behavior_description.querySelectorAll('a[href^="/en/keyword/"]');
                if (!keywordElements) return null;
                keywordElements.forEach(a => {{
                    const span = a.querySelector('span.q-btn__content.text-center.col.items-center.q-anchor--skip.justify-center.row>span');
                    if (span) {{
                        const keywordText = span.textContent.trim();
                        if (keywordText && !results.keywords.includes(keywordText)) {{
                            results.keywords.push(keywordText); // 키워드 저장
                        }}
                    }}
                }});
    
                // 2. 설명 추출
                const descriptionElements = behavior_description.querySelectorAll("{tag_type}");
                let currentEffect = []; // 초기화
                descriptionElements.forEach(span => {{
                    let text = span.textContent?.trim();
                    console.log("origin text:", text);
                    if (!text) return;
                    // 공백과 기호 처리
                    text = text.replace(/\\s*([.,:;!?()\\[\\]])\\s*/g, '$1'); // 기호 앞뒤 공백 제거
                    console.log("after symbol processing:", text);
                    console.log("after spacing character: ", text);
                    if (results.keywords.includes(text)) {{
                        const keywordEffect = 'keyword:' + text;
                        if (!currentEffect.includes(keywordEffect)) {{
                            currentEffect.push(keywordEffect);
                        }}
                    }} else {{
                        currentEffect.push(text);
                    }}
                    console.log("current effect:", currentEffect);
                }});
    
                // 마지막 효과 저장
                if (currentEffect.length > 0) {{
                    const finalDescription = currentEffect.join(' ');
                    console.log("After process:", finalDescription);
                    results.description.push(finalDescription);
                }}
                let processedText = results.description
                    .map(word => word.trim())
                    .join(" ")
                    .replace(/\\s+([,.:])/g, "$1")
                    .replace(/\\s+'/g, "'")
                    .replace(/'\\s+/g, "'")
                    .replace(/\\[\\s+/g, "[")
                    .replace(/\\s+\\]/g, "]")
                    .replace(/\\(\\s+/g, "(")
                    .replace(/\\s+\\)/g, ")")
                    .replace(/\\s+%/g, "%")
                    .replace(/\\s+\\//g, "/")
                    .replace(/\\/\\s+/g, "/")
                    .replace(/\\s+\\/\\s+/g, "/");
                results.description = processedText;
                return results;
            }}
        """)

    async def combine_and_process_texts(self, texts):
        """
        배열 형태의 텍스트 데이터를 합치고 형식을 처리한 문자열을 반환합니다.
        Args:
            texts (list of list): 텍스트 배열 목록
        Returns:
            str: 형식이 처리된 최종 문자열
        """
        processed_sentences = []
        
        for text_array in texts:
            # 빈 문자열을 제외한 나머지 텍스트를 이어붙이기
            sentence = ''.join([word for word in text_array if word != ''])
            sentence = re.sub(r'\\](\\s*)', ']', sentence)
            processed_sentences.append(sentence)
    
        # 문장을 공백으로 연결하여 반환
        return ''.join(processed_sentences)


class Identity:
    """인격 정보를 저장하는 클래스"""
    def __init__(self, identity_id, name=None, rarity=None,\
                 skill_1_type=None, skill_1_sin_affinity=None,\
                 skill_2_type=None, skill_2_sin_affinity=None,\
                 skill_3_type=None, skill_3_sin_affinity=None,\
                 defense_type=None, defense_sin_affinity=None):
        self.identity_id = identity_id
        self.name = name
        self.rarity = rarity
        # 선언만 하는 부분 
        self.skill_1_type = skill_1_type 
        self.skill_1_sin_affinity = skill_1_sin_affinity 
        self.skill_2_type = skill_2_type 
        self.skill_2_sin_affinity = skill_2_sin_affinity 
        self.skill_3_type = skill_3_type 
        self.skill_3_sin_affinity = skill_3_sin_affinity 
        self.defense_type = defense_type 
        self.defense_sin_affinity = defense_sin_affinity
        self.level_max = 50
        self.uptie_max = 4
        self.level_stats = []  # LevelStat 객체 리스트
        self.uptie_stats = [] # UptieStat 객체 리스트
        self.resist_info = None
        self.stagger_threshold = None
        self.skillset = SkillSet()  # Skill정보를 담고 있는 skillset 객체 
        self.passive_infos = []  # 패시브 객체를 담고 있는 배열
        self.support_passive_infos = [] # 서포트 패시 객체를 담고 있는 배열
        self.panic_type = [] # 혼란상태 발생시 나타나는 효과를 담고 있는 객체 배열1

    def update_name(self, name):
        self.name = name

    def update_rarity(self, rarity):
        self.rarity = rarity

    def add_level_stat(self, level, hp, defense):
        self.level_stats.append(LevelStat(level, hp, defense))

    def add_uptie_stat(self, uptie, speed):
        self.uptie_stats.append(UptieStat(uptie, speed))

    def add_resist_info(self, resist_hit, resist_penetrate, resist_slash):
        self.resist_info = ResistInfo(resist_hit, resist_penetrate, resist_slash)

    def add_stagger_threshold(self, first_stagger, second_stagger, third_stagger):
        self.stagger_threshold = StaggerThreshold(first_stagger, second_stagger, third_stagger)

    def add_skill(self, skill_code, skill_name, coin_count, init_level, init_remain, atk_weight):
        self.skillset.add_skill(skill_code, Skill(skill_code, skill_name, coin_count, init_level, init_remain, atk_weight))
        
    def add_skill_uptie_info(self, skill_name, uptie_level, skill_power, coin_power, effect_by_condition):
        self.skillset.add_skill_uptie_info(skill_name, uptie_level, skill_power, coin_power, effect_by_condition)
        
    def update_details(self, page):
        # 세부 정보 업데이트 로직 (가정)
        pass
    #def init_uptie(self, page):
    #    init_uptie = 

# LevelStat 객체 정의
class LevelStat:
    def __init__(self, level, hp=None, defense=None):
        self.level = level
        self.hp = hp
        self.defense = defense
    def update_level(self, level):
        self.level = level
    def update_hp(self, hp):
        self.hp = hp
    def update_defense(self, defense):
        self.defense = defense

# UptieStat 객체 정의
class UptieStat:
    def __init__(self, uptie, speed=None):
        self.uptie = uptie
        self.speed = speed
    def update_speed(self, speed):
        self.speed = speed

# ResistInfo 객체 정의
class ResistInfo:
    def __init__(self, resist_hit=None, resist_penetrate=None, resist_slash=None):
        self.resist_hit = resist_hit
        self.resist_penetrate = resist_penetrate
        self.resist_slash = resist_slash

# ResistInfo 객체 정의
class StaggerThreshold:
    def __init__(self, first_stagger=None, second_stagger=None, third_stagger=None):
        self.first_stagger = first_stagger
        self.second_stagger = second_stagger
        self.third_stagger = third_stagger

class SkillSet:
    def __init__(self):

        self.skill_1_type = ""
        self.skill_2_type = ""
        self.skill_3_type = ""
        self.defense_skill_type = ""

        self.skill_1_sin_affinity = ""
        self.skill_2_sin_affinity = ""
        self.skill_3_sin_affinity = ""
        self.defense_skill_sin_affinity = ""

        self.skill_1_details = []
        self.skill_2_details = []
        self.skill_3_details = []
        self.defense_skill_details = []

    def set_skill_type(self, skill_code, skill_type, sin_affinity):
        if skill_code == "skill 1":
            self.skill_1_type = skill_type
            self.skill_1_sin_affinity = sin_affinity
        elif skill_code == "skill 2":
            self.skill_2_type = skill_type
            self.skill_2_sin_affinity = sin_affinity
        elif skill_code == "skill 3":
            self.skill_3_type = skill_type
            self.skill_3_sin_affinity = sin_affinity
        elif skill_code == "defense":
            self.defense_skill_type = skill_type
            self.defense_skill_sin_affinity = sin_affinity
    
    def add_skill(self, skill_code, skill):
        if skill_code == "skill 1":
            self.skill_1_details.append(skill)
            skill.skill_type = self.skill_1_type
            skill.sin_affinity = self.skill_1_sin_affinity
        elif skill_code == "skill 2":
            self.skill_2_details.append(skill)
            skill.skill_type = self.skill_2_type
            skill.sin_affinity = self.skill_2_sin_affinity
        elif skill_code == "skill 3":
            self.skill_3_details.append(skill)
            skill.skill_type = self.skill_3_type
            skill.sin_affinity = self.skill_3_sin_affinity
        elif skill_code == "defense":
            self.defense_skill_details.append(skill)
            skill.skill_type = self.defense_skill_type 
            skill.sin_affinity = self.defense_skill_sin_affinity

    def get_skill_by_name(self, skill_name):
        for skill_list in [self.skill_1_details, self.skill_2_details, self.skill_3_details, self.defense_skill_details]:
            for skill in skill_list:
                result = skill.is_skill_by_name(skill_name)
                if result:
                    return result
        return None

    def add_skill_uptie_info(self, skill_name, uptie_level, skill_power, coin_power, effect_by_condition):
        skill = self.get_skill_by_name(skill_name)
        skill.add_uptie_info(uptie_level, skill_power, coin_power, effect_by_condition)
        

class Skill:
    def __init__(self, skill_code, skill_name, coin_count, init_level, init_remain, atk_weight):
        self.skill_code = skill_code # "Skill 1"/"Skill 2"/"Skill 3"/"Defense" '''o'''
        self.skill_type = None
        self.sin_affinity = None
        self.skill_name = skill_name # 스킬 이름 '''o'''
        self.coin_count = coin_count # 동전 개수 '''o'''
        self.init_level = init_level # 스킬이 초기 배정받은 레벨, 인격 레벨에 - 1 한 값을 더해주면 스킬의 현 level을 구할 수 있다. '''o'''
        self.init_remain = init_remain #'''o'''
        self.atk_weight = atk_weight #공격 가중치 값이다. 방어스킬에는 없기 때문에 None으로 설정했다. '''o'''
        self.remain_count = init_remain # 실제 스킬이 작동하며 횟수가 차감되 나중에 모두 차감되면 다음 턴에 다시 초기횟수 리필된다.
        self.uptie_info = []  

    def is_skill_by_name(self, skill_name):
        if self.skill_name == skill_name:
            return self
        return None
        
    def add_uptie_info(self, uptie_level, skill_power, coin_power, effects_by_condition):
        self.uptie_info.append(UptieInfo(uptie_level, skill_power, coin_power, effects_by_condition))

class UptieInfo:
    def __init__(self, uptie_level, skill_power, coin_power, effects_by_condition):
        self.uptie_level = uptie_level   #'''o'''
        self.skill_power = skill_power   #'''o'''
        self.coin_power = coin_power     #'''o'''
        self.skill_effect = effects_by_condition

# 추상화된 EffectType 클래스
class EffectType:
    def __init__(self, effect_type, name, description):
        self.effect_type = effect_type # Effect 유형
        self.name = name  # Effect 이름
        self.description = description  # Effect 설명

# Passive 클래스
class Passive(EffectType):
    def __init__(self, effect_type, name, description, condition=None):
        super().__init__(effect_type, name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)  # condition이 있는지 여부 확인

# SupportPassive 클래스
class SupportPassive(EffectType):
    def __init__(self, effect_type, name, description, condition=None):
        super().__init__(effect_type, name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)

# PanicType 클래스
class PanicType(EffectType):
    def __init__(self, effect_type, name, description, condition=None):
        super().__init__(effect_type, name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)

# Keyword 클래스
class Keyword:
    def __init__(self, keyword, explain=None, dispellable=None):
        self.keyword = keyword
        self.explain = explain
        self.dispellable = dispellable

      
class DataContainer:
    """크롤링 데이터를 관리하는 클래스"""
    def __init__(self):
        self.identities = []

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.level_max

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.uptie_max

    def update_identity_name(self, identity_id, identity_name):
        """특정 ID의 이름을 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_name(identity_name)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_identity_rarity(self, identity_id, rarity):
        """특정 ID의 희귀도를 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_rarity(rarity)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_skill(self, identity_id, skill_number, new_type):
        """
        특정 identity의 skill_type을 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_type: 새 skill_type
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_type = new_type
                elif skill_number == 2:
                    identity.skill_2_type = new_type
                elif skill_number == 3:
                    identity.skill_3_type = new_type
                elif skill_number == "defense":
                    identity.defense_type = new_type
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def update_sin_affinity(self, identity_id, skill_number, new_affinity):
        """
        특정 identity의 sin_affinity를 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_affinity: 새 sin_affinity
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_sin_affinity = new_affinity
                elif skill_number == 2:
                    identity.skill_2_sin_affinity = new_affinity
                elif skill_number == 3:
                    identity.skill_3_sin_affinity = new_affinity
                elif skill_number == "defense":
                    identity.defense_sin_affinity = new_affinity
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def add_identity(self, identity):
        """새로운 인격 데이터를 추가."""
        self.identities.append(identity)

    def add_level_stat(self, identity_id, level, hp, defense):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_level_stat(level, hp, defense)

    def add_uptie_stat(self, identity_id, uptie, speed):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_uptie_stat(uptie, speed)

    def add_resist_info(self, identity_id, resist_hit, resist_penetrate, resist_slash):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_resist_info(resist_hit, resist_penetrate, resist_slash)

    def add_stagger_threshold(self, identity_id, first_stagger, second_stagger, third_stagger):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_stagger_threshold(first_stagger, second_stagger, third_stagger)

    
    def add_skill(self, identity_id, skill_code, skill_name, coin_count, init_level, init_remain, atk_weight):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_code == "skill 1":
                    identity.skillset.set_skill_type(skill_code, identity.skill_1_type, identity.skill_1_sin_affinity)
                elif skill_code == "skill 2":
                    identity.skillset.set_skill_type(skill_code, identity.skill_2_type, identity.skill_2_sin_affinity)
                elif skill_code == "skill 3":
                    identity.skillset.set_skill_type(skill_code, identity.skill_3_type, identity.skill_3_sin_affinity)
                elif skill_code == "defense":
                    identity.skillset.set_skill_type(skill_code, identity.defense_type, identity.defense_sin_affinity)
                identity.add_skill(skill_code, skill_name, coin_count, init_level, init_remain, atk_weight)

    def add_skill_uptie_info(self, identity_id, skill_name, uptie_level, skill_power, coin_power, effects_by_condition):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_skill_uptie_info(skill_name, uptie_level, skill_power, coin_power, effects_by_condition)
        
    def display_identities(self):
        """Identity, ID, Name, Rarity 정보를 출력"""
        if not self.identities:
            print("No identities found.")
        else:
            # 데이터를 리스트로 변환
            table = []
            for idx, identity in enumerate(self.identities, 1):
                table.append([
                    idx, identity.identity_id, identity.name, identity.rarity,
                    identity.skill_1_type, identity.skill_1_sin_affinity,
                    identity.skill_2_type, identity.skill_2_sin_affinity,
                    identity.skill_3_type, identity.skill_3_sin_affinity,
                    identity.defense_type, identity.defense_sin_affinity
                ])
            
            # 테이블 헤더 정의
            headers = [
                "Index", "ID", "Name", "Rarity",
                "Skill_1 Type", "Skill_1 Sin", "Skill_2 Type", "Skill_2 Sin",
                "Skill_3 Type", "Skill_3 Sin", "Defense Type", "Defense Sin"
            ]
            
            # 테이블 출력
            print(tabulate(table, headers=headers, tablefmt="fancy_grid"))

    def export_to_csv(self, filename, fieldnames, data):
        """
        공통적으로 사용 가능한 CSV 내보내기 메서드
        :param filename: 내보낼 파일 이름 (예: 'identity.csv', 'skill.csv')
        :param fieldnames: CSV의 헤더 리스트
        :param data: CSV로 내보낼 데이터 (리스트 형태)
        """
        try:
            # 현재작업 디렉토 기준으 'csv'폴더 없으면 생성
            if not os.path.exists('./csv'):
                os.makedirs('./csv')
            # 파일 경로 설정 (상대경로 'csv' 폴더 안에 저장)
            file_path = os.path.join('./csv', filename)
            with open(file_path, mode='w', newline='', encoding='utf-8') as file:
                writer = csv.DictWriter(file, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(data)
            print(f"Data successfully exported to {filename}")
        except Exception as e:
            print(f"Error exporting to CSV: {e}")

    def get_identity_data(self):
        """
        Identity 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "ID": identity.identity_id,
                "Name": identity.name,
                "Rarity": identity.rarity,
                "Skill_1_Type": identity.skill_1_type,
                "Skill_1_Sin_Affinity": identity.skill_1_sin_affinity,
                "Skill_2_Type": identity.skill_2_type,
                "Skill_2_Sin_Affinity": identity.skill_2_sin_affinity,
                "Skill_3_Type": identity.skill_3_type,
                "Skill_3_Sin_Affinity": identity.skill_3_sin_affinity,
                "Defense_Type": identity.defense_type,
                "Defense_Sin_Affinity": identity.defense_sin_affinity,
            }
            for identity in self.identities
        ]
        
    def export_identities_data(self, filename="identities.csv"):
        """Identity 데이터를 CSV로 내보내는 메서드"""
        identity_fieldnames = [
            "ID", "Name", "Rarity",
            "Skill_1_Type", "Skill_1_Sin_Affinity",
            "Skill_2_Type", "Skill_2_Sin_Affinity",
            "Skill_3_Type", "Skill_3_Sin_Affinity",
            "Defense_Type", "Defense_Sin_Affinity"
        ]
        identity_data = self.get_identity_data()  # Identity 데이터 가져오기
        self.export_to_csv(filename, identity_fieldnames, identity_data)  # CSV로 내보내기

    def load_identities_by_id(self, filename, identity_ids):
        """지정된 identity_ids 목록에 해당하는 Identity 데이터를 CSV에서 읽어옴."""
        identity_ids_set = set(identity_ids) # 중복 제거를 위 Set 사용
        with open(filename, mode='r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                if row["ID"] in identity_ids_set:
                    identity = Identity(
                        identity_id=row["ID"],
                        name=row["Name"],
                        rarity=row["Rarity"],
                        skill_1_type=row["Skill_1_Type"],
                        skill_1_sin_affinity=row["Skill_1_Sin_Affinity"],
                        skill_2_type=row["Skill_2_Type"],
                        skill_2_sin_affinity=row["Skill_2_Sin_Affinity"],
                        skill_3_type=row["Skill_3_Type"],
                        skill_3_sin_affinity=row["Skill_3_Sin_Affinity"],
                        defense_type=row["Defense_Type"],
                        defense_sin_affinity=row["Defense_Sin_Affinity"]
                    )
                    self.identities.append(identity)

    def get_level_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Level": level_stat.level,
                "HP": level_stat.hp,
                "Defense": level_stat.defense,
            }
            for identity in self.identities
            for level_stat in identity.level_stats
        ]

    def export_level_stats_data(self, filename="levelstats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        level_stats_fieldnames = ["Identity_ID", "Level", "HP", "Defense"]
        level_stats_data = self.get_level_stats_data()  # level_stats 데이터 가져오기
        self.export_to_csv(filename, level_stats_fieldnames, level_stats_data)  # CSV로 내보내기

    def get_uptie_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Uptie": uptie_stat.uptie,
                "Speed": uptie_stat.speed,
            }
            for identity in self.identities
            for uptie_stat in identity.uptie_stats
        ]

    def export_uptie_stats_data(self, filename="Uptiestats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        uptie_stats_fieldnames = ["Identity_ID", "Uptie", "Speed"]
        uptie_stats_data = self.get_uptie_stats_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, uptie_stats_fieldnames, uptie_stats_data)  # CSV로 내보내기

    def get_resist_info_data(self):
        """
        모든 Identity의 resist_info 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Resist_Hit": identity.resist_info.resist_hit,
                "Resist_Penetrate": identity.resist_info.resist_penetrate,
                "Resist_Slash": identity.resist_info.resist_slash,
            }
            for identity in self.identities
        ]

    def export_resist_info_data(self, filename="ResistInfos.csv"):
        """ResistInfo 데이터를 CSV로 내보내는 메서드"""
        resist_infos_fieldnames = ["Identity_ID", "Resist_Hit", "Resist_Penetrate", "Resist_Slash"]
        resist_infos_data = self.get_resist_info_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, resist_infos_fieldnames, resist_infos_data)  # CSV로 내보내기

    def export_uptie_stats_data(self, filename="Uptiestats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        uptie_stats_fieldnames = ["Identity_ID", "Uptie", "Speed"]
        uptie_stats_data = self.get_uptie_stats_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, uptie_stats_fieldnames, uptie_stats_data)  # CSV로 내보내기

    def get_stagger_threshold_data(self):
        """
        모든 Identity의 stagger_threshold 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "First_Stagger": identity.stagger_threshold.first_stagger,
                "Second_Stagger": identity.stagger_threshold.second_stagger,
                "Third_Stagger": identity.stagger_threshold.third_stagger,
            }
            for identity in self.identities
        ]

    def export_stagger_threshold_data(self, filename="StaggerThresholds.csv"):
        """StaggerThresholds 데이터를 CSV로 내보내는 메서드"""
        stagger_thresholds_fieldnames = ["Identity_ID", "First_Stagger", "Second_Stagger", "Third_Stagger"]
        stagger_thresholds_data = self.get_stagger_threshold_data()  # stagger_thresholds 데이터 가져오기
        self.export_to_csv(filename, stagger_thresholds_fieldnames, stagger_thresholds_data)  # CSV로 내보내기

    def get_skillset_data(self):
        """모든 Identity의 skillset 데이터를 export_to_csv을 포맷으로 변환"""
        return [
            {
                "Identity_ID": identity.identity_id,
                "Skill_Code": skill.skill_code,
                "Skill_Type": skill.skill_type,
                "Sin_Affinity": skill.sin_affinity,
                "Skill_Name": skill.skill_name,
                "Coin_Count": skill.coin_count,
                "Init_Level": skill.init_level,
                "Init_Remain": skill.init_remain,
                "Atk_Weight": skill.atk_weight,
                "Uptie_Level": uptie_info.uptie_level,
                "Skill_Power": uptie_info.skill_power,
                "Coin_Power": uptie_info.coin_power,
                "Effect_Data": json.dumps(uptie_info.skill_effect)
            }
            for identity in self.identities
            for skill_list in [identity.skillset.skill_1_details, identity.skillset.skill_2_details, identity.skillset.skill_3_details, identity.skillset.defense_skill_details]
            for skill in skill_list
            for uptie_info in skill.uptie_info
        ]

    def export_identity_skillset_data(self, filename="IdentitiesSkills.csv"):
        """IdentitiesSkills 데이터를 CSV로 내보내는 메서드"""
        identities_skills_fieldnames = ["Identity_ID", "Skill_Code", "Skill_Type", "Sin_Affinity", "Skill_Name", "Coin_Count", "Init_Level", "Init_Remain", "Atk_Weight", "Uptie_Level", "Skill_Power", "Coin_Power", "Effect_Data"]
        identities_skills_data = self.get_skillset_data()
        self.export_to_csv(filename, identities_skills_fieldnames, identities_skills_data)
            


async def main():
    data_container = DataContainer()
    scraper = Scraper(container=data_container, base_url="https://limbus.kusoge.xyz", img_url="https://img.kusoge.xyz/limbus/img")
    await scraper.scrape()

await main() 

Bloodfeast
[]
{'keywords': ['Bloodfeast', 'Bleed'], 'description': 'If this unit is on field, or is one of the units that can appear on this Stage, increase keyword:Bloodfeast value by the amount of keyword:Bleed damage collectively received by every unit. When this unit enters the field, the sleeping blood drenching the battlefield will fill the surface.'}
Hardblood Thorn
['Lust x3 Owned', 'Envy x2 Owned']
{'keywords': ['Bleed', 'Bloodfeast'], 'description': 'When an other ally takes keyword:Bleed damage or consumes keyword:Bloodfeast, gain 1 blooming[thornyfall_panic RodionFirst](3 times per turn) Heal HP on self by 20% of the damage dealt with base Skills (max 10 per Skill)'}
"Blossom with Blood..."
['Lust x3 Owned', 'Envy x3 Owned']
{'keywords': ['Bleed'], 'description': "To 1 ally with the highest keyword:Bleed Potency at Turn End: reduce the ally's Potency by 6 max, and apply (Potency reduced/2) blooming[thornyfall_panic RodionFirst](rounded down)"}
Panic
[]
{'keywords': [], 'des

In [1]:
import os
import csv
import re
import json
import numpy as np
import pandas as pd
from playwright.async_api import async_playwright
from tabulate import tabulate

# Sin Color 이름 변환을 위한 매핑
resource_mapping = {
    "AZURE": "Gloom",
    "VIOLET": "Envy",
    "AMBER": "Sloth",
    "SHAMROCK": "Gluttony",
    "CRIMSON": "Wrath",
    "SCARLET": "Lust",
    "INDIGO": "Pride"
}

def convert_resource(resource_name):
    """Sin Color 이름을 변환하는 함수. 매핑되지 않은 경우 원래 이름을 반환."""
    return resource_mapping.get(resource_name, resource_name)

def find_sin_for_color(url):
    # URL에 color_name을 추출
    color_name = url.split('/')[-1].split('.')[0].replace("attr_", "")
    # 해당 색상이 resource_mapping에 있는 확인
    for color, sin in resource_mapping.items():
        if color_name in color:
            return sin
    return None


class Scraper:
    """웹 페이지 크롤링을 위한 클래스"""
    def __init__(self, container, base_url, img_url):
        self.container = container
        self.base_url = base_url
        self.img_url = img_url
        self.uptie_level = 1

    async def handle_request(self, route, request):
        """이미지와 폰트 리소스 로드를 차단."""
        if request.resource_type =="image":
            #print(f"{request.url}")
            img_url = request.url
            if  re.match(f"{self.img_url}/ui/attr.*\\.png", img_url):
                await route.continue_()
            else:
                await route.fulfill(
                    status=200,
                    content_type="image/png",
                    body=b""
                )
        elif request.resource_type == "font":
            await route.abort()
        else:
            await route.continue_()
            
    async def scrape(self):
        """크롤링 메인 함수"""
        async with async_playwright() as p:
            browser = await p.chromium.launch(headless=True)
            page = await browser.new_page()
            #await page.evaluate('window.scrollTo(0, document.body.scrollHeight)')
            await page.route("**/*", self.handle_request)
            # Step 1: /identity 페이지에서 기본 정보 크롤링
            #await self.crawl_identity_page(page)

            # Step 2: /identity/{identity_id} 상세 페이지 크롤링
            #identity_ids_to_load = ["10104", "10310", "10710","10808", "10911", "11005"]
            #identity_ids_to_load = ["10104", "10911"]#,"10310", "10710","10808", "11005", "11210"]
            #identity_ids_to_load = ["10911","10310"]#, "10710","10808", "11005", "11210"]
            identity_ids_to_load = ["10911", "10611", "10711"]#, "10710","10808", "11005", "11210"]
            self.container.load_identities_by_id('./csv/identities.csv', identity_ids_to_load)
            await self.crawl_identity_details_for_all(page)
            #self.container.display_identities()
            #self.container.export_identities_data("identities.csv")
            #self.container.export_level_stats_data("Levelstats.csv")
            #self.container.export_uptie_stats_data("Uptiestats.csv")
            #self.container.export_resist_info_data("ResistInfos.csv")
            #self.container.export_stagger_threshold_data("StaggerThresholds.csv")
            #self.container.export_identity_skillset_data("IdentitiesSkills.csv")
            self.container.export_identities_behaviors_data("IdentitiesBehaviors.csv")
            page.set_default_timeout(10000)
            #await page.wait_for_timeout(100000)
            await browser.close()

    async def get_identity_id_from_row(self, row):
        """테이블 행(row)에서 인격 ID를 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_url = await identity_cell.get_attribute('href')
            identity_id = identity_url.split('/identity/')[-1]
            return identity_id
        return "Unknown"

    async def get_identity_name_from_row(self, row):
        """테이블 행(row)에서 인격 이름을 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_name = (await identity_cell.inner_text()).replace("\n", " ").strip()
            return identity_name
        return "Unknown"
        
    async def crawl_identity_page(self, page):
        """/identity 페이지로 이동하여 인격 기본 정보를 추출."""
        await page.goto(self.base_url + "/identity")
        await self.extract_identity_page(page)

    async def extract_identity_page(self, page):
        """/identity 페이지의 모든 테이블 행에서 인격 정보를 추출."""
        identity_rows = await page.query_selector_all('tr')
        if len(identity_rows) > 1:
            for idx, identity_row in enumerate(identity_rows):
                if idx == 0:
                    continue # 0번째요소이름을 보여주는 열은 건너뜀
                await self.scrape_identity_name_and_identity_id(identity_row)
                await self.extract_rarity(identity_row)
                for i in range(1, 4):  # 스킬1~3 정보 추출
                    await self.extract_atk_and_resource(identity_row, i)
                await self.extract_defense_and_resource(identity_row)

    async def scrape_identity_name_and_identity_id(self, row):
        """인격 번호와 이름을 추출하여 DataContainer에 저장."""
        try:
            identity_id = await self.get_identity_id_from_row(row)
            identity_name = await self.get_identity_name_from_row(row)
            identity = Identity(identity_id)
            self.container.add_identity(identity)
            self.container.update_identity_name(identity_id, identity_name)
        except Exception as e:
            print(f"Error extracting identity name and identity id: {e}")

    async def get_element_attribute(self, element, selector, attribute):
        """요소에서 지정된 속성(attribute)을 추출."""
        try:
            target_element = await element.query_selector(selector)
            if target_element:
                return await target_element.get_attribute(attribute)
        except Exception as e:
            print(f"Error getting element attribute: {e}")
        return None
    
    async def extract_rarity(self, row):
        """테이블 행(row)에서 희귀도를 추출하여 업데이트."""
        try:
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
                sibling = await parent.evaluate_handle('node => node.nextElementSibling')
                if sibling:
                    rarity_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/rarity_"]', 'src')
                    if rarity_img_src:
                        rarity_level = rarity_img_src.split('_')[-1].split('.')[0]  # rarity_1.png -> 1
                        identity_id = await self.get_identity_id_from_row(row)
                        self.container.update_identity_rarity(identity_id, rarity_level)
                    else:
                        print("Rarity image not found in the next sibling TD.")
        except Exception as e: 
            print(f"Error extracting rarity: {e}")

        return "Unknown"

    async def extract_atk_and_resource(self, row, skill_number):
        """ 공격 타입(ATK Type)과 자원(Attribute) 추출 함수 - 스킬 번호에 따라 처리 """
        try:
            #print(f"Attempting to locate skill{skill_number} type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 해당 스킬 번호에 맞는 형제로 이동
                sibling = parent
                for _ in range(skill_number + 1):  # 스킬 번호에 맞게 이동
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                atk_type, resource = "Unknown", "Unknown"
        
                if sibling:
                    # 공격 타입 이미지 (atk_type) 찾기
                    atk_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/atk_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if atk_type_img_src:
                        atk_type = atk_type_img_src.split('_')[-1].split('.')[0]  # atk_type_SLASH.png -> SLASH
                        #print(f"Extracted attack type: {atk_type}")
                        self.container.update_skill(identity_id, skill_number, atk_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted resource: {resource}")
                        
                        # 리소스를 변환
                        resource = convert_resource(resource)
                        #print(f"Converted resource: {resource}")
                        self.container.update_sin_affinity(identity_id, skill_number, resource)
                        
        except Exception as e:
            print(f"Error extracting skill{skill_number} type and resource: {e}")

    async def extract_defense_and_resource(self, row):
        """ 방어 타입(Def Type)과 자원(Attribute) 추출 함수 """
        try:
            #print("Attempting to locate defense type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 다섯 번째 형제 td로 이동 (방어 관련 정보가 있는 <td>)
                sibling = parent
                for _ in range(5):  # 방어 정보는 5번째 형제에 있음
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                def_type, def_resource = "Unknown", "Unknown"
        
                if sibling:
                    # 방어 타입 이미지 (def_type) 찾기
                    def_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/def_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if def_type_img_src:
                        def_type = def_type_img_src.split('_')[-1].split('.')[0]  # def_type_GUARD.png -> GUARD
                        #print(f"Extracted defense type: {def_type}")
                        self.container.update_skill(identity_id, 'defense', def_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        def_resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted defense resource: {def_resource}")
                        
                        # 방어 리소스를 변환
                        def_resource = convert_resource(def_resource)
                        #print(f"Converted defense resource: {def_resource}")
                        self.container.update_sin_affinity(identity_id, 'defense', def_resource)
 
        except Exception as e:
            print(f"Error extracting defense type and resource: {e}")

    async def crawl_identity_details_for_all(self, page):
        """모든 인격의 세부 정보를 크롤링."""
        for identity in self.container.identities:
            await page.wait_for_load_state("networkidle")
            await page.goto(self.base_url + "/identity/"+identity.identity_id)
            """ 레벨과 동기화와 관련 없는 참관타 저항은 밑에서 변경하는 것과 연관성이 적기 때문에 위에서 처리한다.
            print(f"Identity_id: {identity.identity_id}")
            results = []
            resist_type_imgs = [f"{self.img_url}/ui/atk_type_HIT.png",
                                f"{self.img_url}/ui/atk_type_PENETRATE.png",
                                f"{self.img_url}/ui/atk_type_SLASH.png"]
            for resist_type_img in resist_type_imgs:
                result = await self.extract_resist_type(page, resist_type_img, 1)
                results.append(result)
            self.container.add_resist_info(identity.identity_id, results[0], results[1], results[2])
            """
            """
            # Stagger Threshold 값 추출
            stagger_threshold = await self.extract_stagger_threshold_values(page)
            self.container.add_stagger_threshold(identity.identity_id, stagger_threshold[0], stagger_threshold[1], stagger_threshold[2])
            """
            await self.initialize_uptie_and_level(page)
            """ 레벨에 따른 level, hp, defense 다르게 되어 있는 것을 출력 확인 객체로 들어가서 csv로 옮기는 과정까지 진행
            for i in range(1, identity.level_max+1):
                await self.move_slider_to_value(page,i)
                await self.extract_level_status(page, identity.identity_id, i)
            """
            """ 동기화에 따른 speed 다르게 되어 있는 것을 출력 확인 객체로 들어가서 csv로 옮기는 과정까지 진행
            for i in range(1, identity.uptie_max+1):
                await self.uptie_click(page,i)
                await self.extract_uptie_status(page, identity.identity_id, i)
            """
            """
                인격에 대한 skill 정보를 가져오는 메소드 위에서 출력하는 csv파일로 인격이 가지고 있는 스킬 정보를 내보낸다.
            await self.crawl_skills(page, identity.identity_id)
            """
            await self.crawl_influence_behavior(page, identity.identity_id)
            
    async def initialize_uptie_and_level(self, page):
        await self.uptie_click(page, 1)
        await self.move_slider_to_value(page, 1)
        self.uptie_level = 1
        #return 1
    
    async def uptie_click(self, page, phase):
        fab_button_selector = ".q-btn.q-btn--fab"
        await page.wait_for_selector(fab_button_selector)
        await page.click(fab_button_selector)
        #print(f"[{setup_uptie_button}] 상위 동기화 버튼 클릭 완료")
        setup_uptie_button = f'a:has(img[src="{self.img_url}/ui/uptie_{phase}.png"])'
        await page.wait_for_selector(setup_uptie_button)
        await page.click(setup_uptie_button)
        #print(f"{phase} 동기화 버튼 클릭 완료")

    async def move_slider_to_value(self, page, target_value):
        """
        슬라이더를 주어진 값으로 이동시킵니다.
        target_value는 슬라이더의 최소값부터 최대값 사이여야 합니다.
        """
        slider_selector = ".q-slider"
        # 슬라이더의 범위 확인
        min_value = int(await page.get_attribute(slider_selector, "aria-valuemin"))
        max_value = int(await page.get_attribute(slider_selector, "aria-valuemax"))
    
        if target_value < min_value or target_value > max_value:
            raise ValueError(f"Target value {target_value} is out of bounds ({min_value}, {max_value})")
    
        # 슬라이더 thumb 요소와 위치 가져오기
        slider_thumb = await page.query_selector(f"{slider_selector} .q-slider__thumb")
        thumb_box = await slider_thumb.bounding_box()
        
        # 슬라이더의 현재 위치 및 목표 위치 계산
        track = await page.query_selector(f"{slider_selector} .q-slider__track")
        track_box = await track.bounding_box()
        
        # 목표 x 위치 계산 (슬라이더 트랙 내에서 비율 기반)
        target_percentage = (target_value - min_value) / (max_value - min_value)
        target_x = track_box["x"] + target_percentage * track_box["width"]
        current_x = thumb_box["x"] + thumb_box["width"] / 2
        y = thumb_box["y"] + thumb_box["height"] / 2
    
        # 슬라이더를 드래그하여 목표 위치로 이동
        await page.mouse.move(current_x, y)
        await page.mouse.down()
        await page.mouse.move(target_x, y, steps=10)  # 부드럽게 이동
        await page.mouse.up()
    
        #print(f"슬라이더를 {target_value}로 이동 완료.")
    
        # 슬라이더 값 확인 및 강제 이벤트 트리거
        await page.evaluate(
            f"""
            (slider) => {{
                slider.dispatchEvent(new Event('input'));  // oninput 이벤트 트리거
                slider.dispatchEvent(new Event('change'));  // onchange 이벤트 트리거
            }}
            """,
            await page.query_selector(slider_selector)
        )
    
        # 이동 후 값 확인
        current_value = await page.get_attribute(slider_selector, "aria-valuenow")
        if current_value != str(target_value):
            print(f"경고: 슬라이더 값이 예상과 다릅니다. 현재 값: {current_value}, 예상 값: {target_value}")

    async def get_second_status_element(self, page):
        """
        두 번째 'Status' 요소에 접근하여 반환하는 메소드
        """
        status_elements = await page.query_selector_all("text=Status")
        if len(status_elements) > 1:
            second_status_element = status_elements[1]
            #print("두 번째 'Status' 요소에 접근합니다.")
            return second_status_element
        else:
            #print("두 번째 'Status' 요소가 존재하지 않습니다.")
            return None

    async def extract_level_status(self, page, identity_id, level):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 레벨에 따라 HP와 Defense가 변한다.
        hp_value = await self.extract_hp(second_status_element)
        defense_value = await self.extract_defense(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Level={level}, HP={hp_value}, Defense={defense_value}")
        self.container.add_level_stat(identity_id,level, hp_value, defense_value)
    
    async def extract_hp(self, status_element):
        # HP 추출
        hp_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].children[0].nextElementSibling.textContent.trim()"
        )
        return hp_value
    
    async def extract_defense(self, status_element):
        # Defense 추출
        defense_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return defense_value

    async def extract_uptie_status(self, page, identity_id, uptie):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 uptie에 따라 speed는 변한다.
        speed_value = await self.extract_speed(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Uptie={uptie}, Speed={speed_value}")
        self.container.add_uptie_stat(identity_id,uptie, speed_value)

    async def extract_speed(self, status_element):
        # Speed 추출
        speed_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return speed_value

    async def extract_resist_type(self, page, image_src, depth):
        # 이미지 요소 찾기
        #print(image_src)
        image = await page.query_selector(f'img[src="{image_src}"]')
        #print(image)
        if not image:
            return "Unknown"
    
        # 이미지의 부모 요소를 찾아서 올라감
        parent = image
        for _ in range(depth):
            parent = await parent.evaluate_handle('node => node.parentElement')
            if not parent:
                return "Unknown"
    
        # 부모의 형제 요소 찾기
        sibling = await parent.evaluate_handle('node => node.nextElementSibling')
        
        if sibling:
            value = await sibling.evaluate('node => node.textContent')
            if value:
                return value.strip()
        
        return "Unknown"

    async def extract_stagger_threshold_values(self, page):
        # 'div.row.center.flex-center.q-pt-xs' 요소들 찾기
        elements = await page.query_selector_all('div.row.center.flex-center.q-pt-xs')
        
        # 각 요소의 텍스트 내용 출력하여 'Stagger Threshold' 요소를 찾기
        for element in elements:
            text = await element.text_content()
            
            if text and "Stagger Threshold" in text:
                # 'Stagger Threshold'가 있는 요소를 찾았다면, 그 이후 형제 요소들에서 값을 추출
                parent = await element.evaluate_handle('node => node.parentElement')
                sibling_elements = await parent.query_selector_all('div.row.justify-evenly div')  # Threshold 값이 들어있는 div들
                
                stagger_values = []
                for sibling in sibling_elements:
                    text_content = await sibling.text_content()
                    match = re.search(r'(\d+%)', text_content)
                    if match:
                        stagger_values.append(match.group(1))  # 퍼센트 값만 리스트에 추가
                while len(stagger_values) < 3:
                    stagger_values.append(np.nan)
                #print(stagger_values)
                return stagger_values  # 추출된 값(또는 np.nan 포함)을 반환
        
        return [np.nan, np.nan, np.nan]  # 만약 'Stagger Threshold'를 찾지 못한 경우 기본값 반환

    async def crawl_skills(self, page, identity_id):
        await self.crawl_skill(page, identity_id, "skill 1")
        await self.crawl_skill(page, identity_id, "skill 2")
        await self.crawl_skill(page, identity_id, "skill 3")
        await self.crawl_skill(page, identity_id, "defense")


    async def crawl_skill(self, page, identity_id, skill_type):
        skills_elements = await page.query_selector_all('div.skilltag')
        for skill_element in skills_elements:
            skill_text = await skill_element.text_content()
            if skill_text.strip().lower() == skill_type:
                await self.process_skill(skill_element, page, identity_id, skill_type)
        
    async def process_skill(self, skill_element, page, identity_id, skill_type):
        # skill_type이 "skill 3"일 경우 uptie_value가 3 이상일 때만 정보를 저장
        for uptie_value in range(1, 5):  # Uptie levels 1 to 4
            # "skill 3"은 uptie_value가 3 이상일 때만 처리
            if skill_type == "skill 3" and uptie_value < 3:
                continue  # skip if uptie_value is less than 3 for skill 3
    
            # uptie_level에 맞게 클릭
            await self.uptie_click(page, uptie_value)
            if skill_type != "skill 3" and uptie_value > 1:
                # Uptie 정보 갱신
                await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
                continue
            elif skill_type == "skill 3" and uptie_value > 3:
                # Uptie 정보 갱신
                await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
                continue
            # 스킬의 세부 정보를 추출
            skill_details = await self.extract_skill_details(skill_element, skill_type)
            skill_name = skill_details["skill_name"]
    
            # 스킬 정보를 저장
            self.container.add_skill(
                identity_id, skill_type, 
                skill_name, skill_details["coin_count"],
                skill_details["init_level"], skill_details["init_remain"], 
                skill_details["atk_weight"]
            )
            print(f"Added skill: {skill_type}:{skill_name}")
    
            # Uptie 정보 갱신
            await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
    
    async def extract_skill_details(self, skill_element, skill_type):
        # 스킬 세부 정보를 추출하는 함수
        skill_name = await self.skill_name(skill_element)
        coin_count = await self.coin_count(skill_element, ".col-auto > .column > .row")
        init_level = await self.init_level(skill_element)
        init_remain = await self.init_remain(skill_element, skill_type, "div.row.justify-start > div.attr.flex.flex-center > div.q-px-sm")
        atk_weight = await self.atk_weight(skill_element, skill_type)
        return {
            "skill_name": skill_name,
            "coin_count": coin_count,
            "init_level": init_level,
            "init_remain": init_remain,
            "atk_weight": atk_weight
        }
    
    async def update_uptie_info(self, page, skill_element, identity_id, skill_name, skill_type, uptie_value):
        # Uptie 정보를 추출하고 업데이트하는 함수
        uptie_info = await self.extract_uptie_info(skill_element)
        effect_data = await self.skill_effects(skill_element)
        self.container.add_skill_uptie_info(
            identity_id, skill_name, 
            uptie_value, uptie_info["skill_power"], uptie_info["coin_power"], 
            effect_data.get("effectsByCondition")
        )
        print(
            f"Updated Uptie for {skill_name} (Uptie Level: {uptie_value}): "
            f"Skill Power: {uptie_info['skill_power']}, Coin Power: {uptie_info['coin_power']} "
            f"Skill Effect: {effect_data.get("effectsByCondition")}"
        )

    async def extract_uptie_info(self, skill_element):
        skill_data = await self.skill_and_coin_power(skill_element)
        return {
            "skill_power": skill_data.get("skillPower"),
            "coin_power": skill_data.get("coinPower")
        }
        
    async def crawl_influence_behavior(self, page, identity_id):
        influence_behaviors = await page.query_selector_all('div[align="right"]')
        # 텍스트를 기준으 추출하고 메소드로 파라미터 들어가 정보를 받는다.
        for influence_behavior in influence_behaviors:
            active_type = await influence_behavior.inner_text()
            active_type = active_type.strip()

            if active_type == "PASSIVE":
                """
                # PASSIVE에 대한 처리
                print(f"Processing PASSIVE logic for: {active_type}")
                # CALL method for PASSIVE
                """
                await self.passive_effects(influence_behavior, identity_id)
            elif active_type == "SUPPORT PASSIVE":
                """
                # SUPPORT PASSIVE에 대한 처리
                print(f"Processing SUPPORT PASSIVE logic for: {active_type}")
                # CALL method for SUPPORT PASSIVE
                """
                await self.support_passive_effects(influence_behavior, identity_id)
            elif active_type == "PANIC TYPE":
                """
                # PANIC TYPE에 대한 처리
                print(f"Processing PANIC TYPE logic for:{active_type}")
                # CALL method for PANIC TYPE
                """
                await self.panic_effects(influence_behavior, identity_id)

    async def skill_and_coin_power(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                // 조부모로 이동
                const grandparent = node.parentElement.parentElement;
                // 이전 형제 탐색
                const previousSibling = grandparent.previousElementSibling;
                if (!previousSibling) return {{ skillPower: null, coinPower: null }}; // 이전 형제가 없으면 기본값 반환
                // 이전 형제 내부의 .row.justify-around > div > div 탐색
                const targetDivs = previousSibling.querySelectorAll(".row.justify-around > div");
                if (targetDivs.length < 3) return {{ skillPower: null, coinPower: null }}; // 배열 길이 확인
                // 1st div: 스킬 위력
                const skillPower = targetDivs[0]?.querySelector('div')?.textContent?.trim() || null; 
                // 3rd div: 코인 파워 값 (3번째 div에서 텍스트 추출)
                const coinPower = targetDivs[2]?.textContent?.trim() || null;
                return {{ skillPower, coinPower }};
            }}
        """)


    async def coin_count(self, skill_element, hierarchy_selector):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector('{hierarchy_selector}');
                if (!targetElement) return 0;
                return targetElement.childElementCount;
            }}
        """)

    async def init_remain(self, skill_element, skill_type, hierarchy_selector):
        if skill_type == "defense":
            return -1
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector('{hierarchy_selector}');
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def skill_name(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector(".block");
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def init_level(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector("div.text-bold.text-h6.h6.q-pl-xs.q-pr-md");
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def atk_weight(self, skill_element, skill_type):
        if skill_type == "defense":
            return 0
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector("div.text-bold.text-h6.h6.q-pl-xs.q-pr-md");
                const doubleNextSibling = targetElement.nextElementSibling.nextElementSibling;
                if (!doubleNextSibling) return 0;
                return doubleNextSibling.textContent?.trim();
            }}
        """)

    async def skill_effects(self, skill_element):
        return await skill_element.evaluate("""
            node => {
                const parent = node.parentElement;
                const doubleNextSibling = parent.nextElementSibling?.nextElementSibling;
                if (!doubleNextSibling) return null;
    
                // doubleNextSibling에서 첫 번째 div.col 요소 찾기
                const textNode = doubleNextSibling.querySelector("div.col");
                if (!textNode) return null;
    
                const results = {
                    keywords: [], // 키워드 저장
                    conditions: [], // 조건 저장
                    effectsByCondition: {} // 조건별 효과
                };
    
                // 1. 조건 추출 (대괄호 텍스트)
                const conditionElements = textNode.querySelectorAll("span");
                conditionElements.forEach(span => {
                    let text = span.textContent?.trim();
                    if (text?.startsWith("[") && text?.endsWith("]")) {
                        // 부모의 전형제 노드 탐색
                        const sibling = span.parentElement.previousElementSibling;
                        if (sibling) {
                            const emblemImage = sibling.querySelector("img.emblem");
                            console.log("Sibling element found:", sibling); // 디버깅용
                            if (emblemImage) {
                                const src = emblemImage.getAttribute("src");
                                console.log("Image source:", src); // 디버깅용
                                const match = src.match(/ui_num_(\\d+)\\.png$/); // 정규식 수정
                                if (match) {
                                    const number = match[1]; // 숫자 추출
                                    text = `[Coin ${number} ${text.replace("[", "").replace("]", "")}]`;
                                    console.log("Updated condition:", text); // 디버깅용
                                }
                            }
                        }
                        results.conditions.push(text);
                        results.effectsByCondition[text] = []; // 조건별 효과를 초기화
                    }
                });
    
                // 2. 키워드 추출
                const keywordElements = textNode.querySelectorAll('a[href^="/en/keyword/"]');
                keywordElements.forEach(a => {
                    const span = a.querySelector('span.q-btn__content.text-center.col.items-center.q-anchor--skip.justify-center.row>span');
                    if (span) {
                        const keywordText = span.textContent.trim();
                        if (keywordText && !results.keywords.includes(keywordText)) {
                            results.keywords.push(keywordText); // 키워드 저장
                        }
                    }
                });
    
                // 3. 효과 추출
                const effectElements = textNode.querySelectorAll("span");
                let currentCondition = null;
                let currentEffect = []; // 초기화
                effectElements.forEach(span => {
                    let text = span.textContent?.trim();
                    if (!text) return;
    
                    // 조건 발견 시 currentCondition 변경
                    if (text.startsWith("[") && text.endsWith("]")) {
                        // 이전 조건의 효과 저장
                        if (currentCondition && currentEffect.length > 0) {
                            // 부모의 전형제 노드 탐색
                            const sibling = span.parentElement.previousElementSibling;
                            if (sibling) {
                                const emblemImage = sibling.querySelector("img.emblem");
                                if (emblemImage) {
                                    const src = emblemImage.getAttribute("src");
                                    const match = src.match(/ui_num_(\\d+)\\.png$/); // 정규식 수정
                                    if (match) {
                                        const number = match[1]; // 숫자 추출
                                        text = `[Coin ${number} ${text.replace("[", "").replace("]", "")}]`;
                                    }
                                }
                            }
                            results.effectsByCondition[currentCondition].push(currentEffect.join(' '));
                            currentEffect = []; // 초기화
                        }
                        currentCondition = text;
                        return;
                    }
    
                    // 현재 조건의 효과 처리
                    if (currentCondition) {
                        if (results.keywords.includes(text)) {
                            const keywordEffect = 'keyword:' + text;
                            if (!currentEffect.includes(keywordEffect)) {
                                currentEffect.push(keywordEffect);
                            }
                        } else {
                            currentEffect.push(text);
                        }
                    }
                });
    
                // 마지막 효과 저장
                if (currentCondition && currentEffect.length > 0) {
                    results.effectsByCondition[currentCondition].push(currentEffect.join(' '));
                }
    
                return results;
            }
        """)

    async def passive_effects(self, passive_element, identity_id):
        passive_name = await self.behavior_name(passive_element)
        passive_conditions = await self.behavior_condition(passive_element)
        passive_description = await self.behavior_description(passive_element, "span")
        #print(passive_name)
        #print(passive_conditions)
        #print(passive_description)
        self.container.add_influence_behavior(identity_id, "passive", passive_name, passive_conditions, passive_description.get("description"))
        keywords = passive_description.get("keywords")
    async def support_passive_effects(self, support_passive_element, identity_id):
        support_passive_name = await self.behavior_name(support_passive_element)
        support_passive_conditions = await self.behavior_condition(support_passive_element)
        support_passive_description = await self.behavior_description(support_passive_element, "span")
        #print(support_passive_name)
        #print(support_passive_conditions)
        #print(support_passive_description)
        self.container.add_influence_behavior(identity_id, "support passive", support_passive_name, support_passive_conditions, support_passive_description.get("description"))
    async def panic_effects(self, panic_element, identity_id):
        panic_name = await self.behavior_name(panic_element)
        panic_conditions = await self.behavior_condition(panic_element)
        panic_description = await self.behavior_description(panic_element, "div")
        #print(panic_type)
        #print(panic_conditions)
        #print(panic_description)
        self.container.add_influence_behavior(identity_id, "panic type", panic_name, panic_conditions, panic_description.get("description"))

    async def navigate_element_by_steps(self, element, parent_steps=0, prev_sibling_steps=0, next_sibling_steps=0):
        """
        주어진 `element`에서 부모, 이전 형제, 또는 다음 형제로 지정된 횟수만큼 이동하여 ElementHandle 반환.
        
        Args:
            element (ElementHandle): 탐색 기준이 되는 요소.
            parent_steps (int): 부모로 이동할 횟수 (0이면 이동하지 않음).
            prev_sibling_steps (int): 이전 형제로 이동할 횟수 (0이면 이동하지 않음).
            next_sibling_steps (int): 다음 형제로 이동할 횟수 (0이면 이동하지 않음).
        
        Returns:
            ElementHandle or None: 탐색된 요소를 반환하거나 없으면 None.
        """
        try:
            current_element = element
    
            # 부모로 이동
            for _ in range(parent_steps):
                current_element = await current_element.evaluate_handle('el => el.parentElement')
                if not current_element:
                    print("Parent element not found during navigation.")
                    return None
    
            # 이전 형제로 이동
            for _ in range(prev_sibling_steps):
                current_element = await current_element.evaluate_handle('el => el.previousElementSibling')
                if not current_element:
                    print("Previous sibling element not found during navigation.")
                    return None
    
            # 다음 형제로 이동
            for _ in range(next_sibling_steps):
                current_element = await current_element.evaluate_handle('el => el.nextElementSibling')
                if not current_element:
                    print("Next sibling element not found during navigation.")
                    return None
    
            return current_element
    
        except Exception as e:
            print(f"Error navigating elements: {e}")
            return None

    async def get_last_sibling(self, element):
        try:
            # 같은 부 아 마지막 형제를 가져오는 JavaScript 실행
            last_sibling = await element.evaluate_handle('el => el.parentElement.lastElementChild')
            return last_sibling
        except Exception as e:
            print(f"Error getting last sibling: {e}")
            return None

    async def behavior_name(self, element):
        parent_prev_sibling = await self.navigate_element_by_steps(element, parent_steps=1, prev_sibling_steps=1, next_sibling_steps=0)
        passive_name_element = await parent_prev_sibling.query_selector('span.block')
        if passive_name_element:
            passive_name = await passive_name_element.inner_text()
            return passive_name
        else:
            #print("Passive name element not found")
            return None

    async def behavior_condition(self, element):
        try:
            conditions = []
            parent_prev_sibling = await self.navigate_element_by_steps(element, parent_steps=1, prev_sibling_steps=1, next_sibling_steps=0)
            img_elements = await parent_prev_sibling.query_selector_all('img')
            if not img_elements:
                #print("No <img> elements found.")
                return conditions
    
            for img_element in img_elements:
                # 각 이미지의 src 속성 가져오기
                img_src = await img_element.get_attribute('src')
                #print(img_src)
    
                sin_name = None
                for color_key in resource_mapping.keys():
                    if f"{self.img_url}/ui/attr_{color_key}.png" in img_src:
                        sin_name = resource_mapping[color_key]
                        break
    
                if not sin_name:
                    #print(f"No matching Sin Color found for {img_src}.")
                    continue  # No Sin Color found, skip this img_element
    
                # 해당 이미지와 관련된 조건 텍스트 추출
                # 두 번째 부모 요소에서 시작하여 다음 형제를 찾아 조건 텍스트를 추출
                condition_element = await self.navigate_element_by_steps(img_element, parent_steps=3, prev_sibling_steps=0, next_sibling_steps=0)
                if condition_element:
                    # condition_element가 None이 아닌 경우에만 진행
                    condition_text_element = await condition_element.query_selector('.q-pl-xs')
                    if condition_text_element:
                        condition_text = await condition_text_element.inner_text()
                        condition_text = condition_text.strip()
                    else:
                        print(f"No condition text found for {img_src}.")
                        condition_text = ""  # condition_text_element가 None이면 빈 문자열로 처리
                else:
                    print(f"No condition element found for {img_src}.")
                    condition_text = ""  # condition_element가 None이면 빈 문자열로 처리
    
                # 결과를 배열에 추가
                conditions.append(f"{sin_name} {condition_text}")
            return conditions
        except Exception as e:
            print(f"Error extracting conditions: {e}")
            return 
            
    async def behavior_description(self, element, tag_type):
        parent_prev_sibling = await self.navigate_element_by_steps(
            element, parent_steps=1, prev_sibling_steps=1, next_sibling_steps=0
        )
        return await parent_prev_sibling.evaluate(f"""
            (node) => {{
                const behavior_title_node = node.querySelector('.q-pb-sm');
                const behavior_title_parent = behavior_title_node.parentElement;
                const behavior_description = behavior_title_parent.lastElementChild;
                if (!behavior_description) return null;
    
                const results = {{
                    keywords: [], // 키워드 저장
                    description: [] // 설명 저장
                }};
                
                // 1. 키워드 추출
                const keywordElements = behavior_description.querySelectorAll('a[href^="/en/keyword/"]');
                if (!keywordElements) return null;
                keywordElements.forEach(a => {{
                    const span = a.querySelector('span.q-btn__content.text-center.col.items-center.q-anchor--skip.justify-center.row>span');
                    if (span) {{
                        const keywordText = span.textContent.trim();
                        if (keywordText && !results.keywords.includes(keywordText)) {{
                            results.keywords.push(keywordText); // 키워드 저장
                        }}
                    }}
                }});
    
                // 2. 설명 추출
                const descriptionElements = behavior_description.querySelectorAll("{tag_type}");
                let currentEffect = []; // 초기화
                descriptionElements.forEach(span => {{
                    let text = span.textContent?.trim();
                    console.log("origin text:", text);
                    if (!text) return;
                    // 공백과 기호 처리
                    text = text.replace(/\\s*([.,:;!?()\\[\\]])\\s*/g, '$1'); // 기호 앞뒤 공백 제거
                    console.log("after symbol processing:", text);
                    console.log("after spacing character: ", text);
                    if (results.keywords.includes(text)) {{
                        const keywordEffect = 'keyword:' + text;
                        if (!currentEffect.includes(keywordEffect)) {{
                            currentEffect.push(keywordEffect);
                        }}
                    }} else {{
                        currentEffect.push(text);
                    }}
                    console.log("current effect:", currentEffect);
                }});
    
                // 마지막 효과 저장
                if (currentEffect.length > 0) {{
                    const finalDescription = currentEffect.join(' ');
                    console.log("After process:", finalDescription);
                    results.description.push(finalDescription);
                }}
                let processedText = results.description
                    .map(word => word.trim())
                    .join(" ")
                    .replace(/\\s+([,.:])/g, "$1")
                    .replace(/\\s+'/g, "'")
                    .replace(/'\\s+/g, "'")
                    .replace(/\\[\\s+/g, "[")
                    .replace(/\\s+\\]/g, "]")
                    .replace(/\\(\\s+/g, "(")
                    .replace(/\\s+\\)/g, ")")
                    .replace(/\\s+%/g, "%")
                    .replace(/\\s+\\//g, "/")
                    .replace(/\\/\\s+/g, "/")
                    .replace(/\\s+\\/\\s+/g, "/");
                results.description = processedText;
                return results;
            }}
        """)

class Identity:
    """인격 정보를 저장하는 클래스"""
    def __init__(self, identity_id, name=None, rarity=None,\
                 skill_1_type=None, skill_1_sin_affinity=None,\
                 skill_2_type=None, skill_2_sin_affinity=None,\
                 skill_3_type=None, skill_3_sin_affinity=None,\
                 defense_type=None, defense_sin_affinity=None):
        self.identity_id = identity_id
        self.name = name
        self.rarity = rarity
        # 선언만 하는 부분 
        self.skill_1_type = skill_1_type 
        self.skill_1_sin_affinity = skill_1_sin_affinity 
        self.skill_2_type = skill_2_type 
        self.skill_2_sin_affinity = skill_2_sin_affinity 
        self.skill_3_type = skill_3_type 
        self.skill_3_sin_affinity = skill_3_sin_affinity 
        self.defense_type = defense_type 
        self.defense_sin_affinity = defense_sin_affinity
        self.level_max = 50
        self.uptie_max = 4
        self.level_stats = []  # LevelStat 객체 리스트
        self.uptie_stats = [] # UptieStat 객체 리스트
        self.resist_info = None
        self.stagger_threshold = None
        self.skillset = SkillSet()  # Skill정보를 담고 있는 skillset 객체 
        self.passive_infos = []  # 패시브 객체를 담고 있는 배열
        self.support_passive_infos = [] # 서포트 패시 객체를 담고 있는 배열
        self.panic_type = [] # 혼란상태 발생시 나타나는 효과를 담고 있는 객체 배열1

    def update_name(self, name):
        self.name = name

    def update_rarity(self, rarity):
        self.rarity = rarity

    def add_level_stat(self, level, hp, defense):
        self.level_stats.append(LevelStat(level, hp, defense))

    def add_uptie_stat(self, uptie, speed):
        self.uptie_stats.append(UptieStat(uptie, speed))

    def add_resist_info(self, resist_hit, resist_penetrate, resist_slash):
        self.resist_info = ResistInfo(resist_hit, resist_penetrate, resist_slash)

    def add_stagger_threshold(self, first_stagger, second_stagger, third_stagger):
        self.stagger_threshold = StaggerThreshold(first_stagger, second_stagger, third_stagger)

    def add_skill(self, skill_code, skill_name, coin_count, init_level, init_remain, atk_weight):
        self.skillset.add_skill(skill_code, Skill(skill_code, skill_name, coin_count, init_level, init_remain, atk_weight))
        
    def add_skill_uptie_info(self, skill_name, uptie_level, skill_power, coin_power, effect_by_condition):
        self.skillset.add_skill_uptie_info(skill_name, uptie_level, skill_power, coin_power, effect_by_condition)

    def add_influence_behavior(self, behavior_type, behavior_name, condition, description):
        if behavior_type == "passive":
            self.passive_infos.append(Passive(behavior_type, behavior_name, condition, description))
        elif behavior_type == "support passive":
            self.support_passive_infos.append(SupportPassive(behavior_type, behavior_name, condition, description))
        elif behavior_type == "panic type":
            self.panic_type.append(PanicType(behavior_type, behavior_name, condition, description))
        
    def update_details(self, page):
        # 세부 정보 업데이트 로직 (가정)
        pass
    #def init_uptie(self, page):
    #    init_uptie = 

# LevelStat 객체 정의
class LevelStat:
    def __init__(self, level, hp=None, defense=None):
        self.level = level
        self.hp = hp
        self.defense = defense
    def update_level(self, level):
        self.level = level
    def update_hp(self, hp):
        self.hp = hp
    def update_defense(self, defense):
        self.defense = defense

# UptieStat 객체 정의
class UptieStat:
    def __init__(self, uptie, speed=None):
        self.uptie = uptie
        self.speed = speed
    def update_speed(self, speed):
        self.speed = speed

# ResistInfo 객체 정의
class ResistInfo:
    def __init__(self, resist_hit=None, resist_penetrate=None, resist_slash=None):
        self.resist_hit = resist_hit
        self.resist_penetrate = resist_penetrate
        self.resist_slash = resist_slash

# ResistInfo 객체 정의
class StaggerThreshold:
    def __init__(self, first_stagger=None, second_stagger=None, third_stagger=None):
        self.first_stagger = first_stagger
        self.second_stagger = second_stagger
        self.third_stagger = third_stagger

class SkillSet:
    def __init__(self):

        self.skill_1_type = ""
        self.skill_2_type = ""
        self.skill_3_type = ""
        self.defense_skill_type = ""

        self.skill_1_sin_affinity = ""
        self.skill_2_sin_affinity = ""
        self.skill_3_sin_affinity = ""
        self.defense_skill_sin_affinity = ""

        self.skill_1_details = []
        self.skill_2_details = []
        self.skill_3_details = []
        self.defense_skill_details = []

    def set_skill_type(self, skill_code, skill_type, sin_affinity):
        if skill_code == "skill 1":
            self.skill_1_type = skill_type
            self.skill_1_sin_affinity = sin_affinity
        elif skill_code == "skill 2":
            self.skill_2_type = skill_type
            self.skill_2_sin_affinity = sin_affinity
        elif skill_code == "skill 3":
            self.skill_3_type = skill_type
            self.skill_3_sin_affinity = sin_affinity
        elif skill_code == "defense":
            self.defense_skill_type = skill_type
            self.defense_skill_sin_affinity = sin_affinity
    
    def add_skill(self, skill_code, skill):
        if skill_code == "skill 1":
            self.skill_1_details.append(skill)
            skill.skill_type = self.skill_1_type
            skill.sin_affinity = self.skill_1_sin_affinity
        elif skill_code == "skill 2":
            self.skill_2_details.append(skill)
            skill.skill_type = self.skill_2_type
            skill.sin_affinity = self.skill_2_sin_affinity
        elif skill_code == "skill 3":
            self.skill_3_details.append(skill)
            skill.skill_type = self.skill_3_type
            skill.sin_affinity = self.skill_3_sin_affinity
        elif skill_code == "defense":
            self.defense_skill_details.append(skill)
            skill.skill_type = self.defense_skill_type 
            skill.sin_affinity = self.defense_skill_sin_affinity

    def get_skill_by_name(self, skill_name):
        for skill_list in [self.skill_1_details, self.skill_2_details, self.skill_3_details, self.defense_skill_details]:
            for skill in skill_list:
                result = skill.is_skill_by_name(skill_name)
                if result:
                    return result
        return None

    def add_skill_uptie_info(self, skill_name, uptie_level, skill_power, coin_power, effect_by_condition):
        skill = self.get_skill_by_name(skill_name)
        skill.add_uptie_info(uptie_level, skill_power, coin_power, effect_by_condition)
        

class Skill:
    def __init__(self, skill_code, skill_name, coin_count, init_level, init_remain, atk_weight):
        self.skill_code = skill_code # "Skill 1"/"Skill 2"/"Skill 3"/"Defense" '''o'''
        self.skill_type = None
        self.sin_affinity = None
        self.skill_name = skill_name # 스킬 이름 '''o'''
        self.coin_count = coin_count # 동전 개수 '''o'''
        self.init_level = init_level # 스킬이 초기 배정받은 레벨, 인격 레벨에 - 1 한 값을 더해주면 스킬의 현 level을 구할 수 있다. '''o'''
        self.init_remain = init_remain #'''o'''
        self.atk_weight = atk_weight #공격 가중치 값이다. 방어스킬에는 없기 때문에 None으로 설정했다. '''o'''
        self.remain_count = init_remain # 실제 스킬이 작동하며 횟수가 차감되 나중에 모두 차감되면 다음 턴에 다시 초기횟수 리필된다.
        self.uptie_info = []  

    def is_skill_by_name(self, skill_name):
        if self.skill_name == skill_name:
            return self
        return None
        
    def add_uptie_info(self, uptie_level, skill_power, coin_power, effects_by_condition):
        self.uptie_info.append(UptieInfo(uptie_level, skill_power, coin_power, effects_by_condition))

class UptieInfo:
    def __init__(self, uptie_level, skill_power, coin_power, effects_by_condition):
        self.uptie_level = uptie_level   #'''o'''
        self.skill_power = skill_power   #'''o'''
        self.coin_power = coin_power     #'''o'''
        self.skill_effect = effects_by_condition

# 추상화된 EffectType 클래스
class EffectType:
    def __init__(self, behavior_type, name, description):
        self.behavior_type = behavior_type
        self.name = name  # Effect 이름
        self.description = description  # Effect 설명

# Passive 클래스
class Passive(EffectType):
    def __init__(self, behavior_type, name, condition, description):
        super().__init__(behavior_type, name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)  # condition이 있는지 여부 확인

# SupportPassive 클래스
class SupportPassive(EffectType):
    def __init__(self,behavior_type, name, condition, description):
        super().__init__(behavior_type, name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)

# PanicType 클래스
class PanicType(EffectType):
    def __init__(self, behavior_type, name, condition, description):
        super().__init__(behavior_type, name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)

# Keyword 클래스
class Keyword:
    def __init__(self, keyword, explain=None, dispellable=None):
        self.keyword = keyword
        self.explain = explain
        self.dispellable = dispellable

      
class DataContainer:
    """크롤링 데이터를 관리하는 클래스"""
    def __init__(self):
        self.identities = []

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.level_max

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.uptie_max

    def update_identity_name(self, identity_id, identity_name):
        """특정 ID의 이름을 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_name(identity_name)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_identity_rarity(self, identity_id, rarity):
        """특정 ID의 희귀도를 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_rarity(rarity)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_skill(self, identity_id, skill_number, new_type):
        """
        특정 identity의 skill_type을 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_type: 새 skill_type
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_type = new_type
                elif skill_number == 2:
                    identity.skill_2_type = new_type
                elif skill_number == 3:
                    identity.skill_3_type = new_type
                elif skill_number == "defense":
                    identity.defense_type = new_type
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def update_sin_affinity(self, identity_id, skill_number, new_affinity):
        """
        특정 identity의 sin_affinity를 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_affinity: 새 sin_affinity
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_sin_affinity = new_affinity
                elif skill_number == 2:
                    identity.skill_2_sin_affinity = new_affinity
                elif skill_number == 3:
                    identity.skill_3_sin_affinity = new_affinity
                elif skill_number == "defense":
                    identity.defense_sin_affinity = new_affinity
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def add_identity(self, identity):
        """새로운 인격 데이터를 추가."""
        self.identities.append(identity)

    def add_level_stat(self, identity_id, level, hp, defense):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_level_stat(level, hp, defense)

    def add_uptie_stat(self, identity_id, uptie, speed):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_uptie_stat(uptie, speed)

    def add_resist_info(self, identity_id, resist_hit, resist_penetrate, resist_slash):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_resist_info(resist_hit, resist_penetrate, resist_slash)

    def add_stagger_threshold(self, identity_id, first_stagger, second_stagger, third_stagger):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_stagger_threshold(first_stagger, second_stagger, third_stagger)

    
    def add_skill(self, identity_id, skill_code, skill_name, coin_count, init_level, init_remain, atk_weight):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_code == "skill 1":
                    identity.skillset.set_skill_type(skill_code, identity.skill_1_type, identity.skill_1_sin_affinity)
                elif skill_code == "skill 2":
                    identity.skillset.set_skill_type(skill_code, identity.skill_2_type, identity.skill_2_sin_affinity)
                elif skill_code == "skill 3":
                    identity.skillset.set_skill_type(skill_code, identity.skill_3_type, identity.skill_3_sin_affinity)
                elif skill_code == "defense":
                    identity.skillset.set_skill_type(skill_code, identity.defense_type, identity.defense_sin_affinity)
                identity.add_skill(skill_code, skill_name, coin_count, init_level, init_remain, atk_weight)

    def add_skill_uptie_info(self, identity_id, skill_name, uptie_level, skill_power, coin_power, effects_by_condition):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_skill_uptie_info(skill_name, uptie_level, skill_power, coin_power, effects_by_condition)

    def add_influence_behavior(self, identity_id, behavior_type, behavior_name, condition, description):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_influence_behavior(behavior_type, behavior_name, condition, description)
        
    def display_identities(self):
        """Identity, ID, Name, Rarity 정보를 출력"""
        if not self.identities:
            print("No identities found.")
        else:
            # 데이터를 리스트로 변환
            table = []
            for idx, identity in enumerate(self.identities, 1):
                table.append([
                    idx, identity.identity_id, identity.name, identity.rarity,
                    identity.skill_1_type, identity.skill_1_sin_affinity,
                    identity.skill_2_type, identity.skill_2_sin_affinity,
                    identity.skill_3_type, identity.skill_3_sin_affinity,
                    identity.defense_type, identity.defense_sin_affinity
                ])
            
            # 테이블 헤더 정의
            headers = [
                "Index", "ID", "Name", "Rarity",
                "Skill_1 Type", "Skill_1 Sin", "Skill_2 Type", "Skill_2 Sin",
                "Skill_3 Type", "Skill_3 Sin", "Defense Type", "Defense Sin"
            ]
            
            # 테이블 출력
            print(tabulate(table, headers=headers, tablefmt="fancy_grid"))

    def export_to_csv(self, filename, fieldnames, data):
        """
        공통적으로 사용 가능한 CSV 내보내기 메서드
        :param filename: 내보낼 파일 이름 (예: 'identity.csv', 'skill.csv')
        :param fieldnames: CSV의 헤더 리스트
        :param data: CSV로 내보낼 데이터 (리스트 형태)
        """
        try:
            # 현재작업 디렉토 기준으 'csv'폴더 없으면 생성
            if not os.path.exists('./csv'):
                os.makedirs('./csv')
            # 파일 경로 설정 (상대경로 'csv' 폴더 안에 저장)
            file_path = os.path.join('./csv', filename)
            with open(file_path, mode='w', newline='', encoding='utf-8') as file:
                writer = csv.DictWriter(file, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(data)
            print(f"Data successfully exported to {filename}")
        except Exception as e:
            print(f"Error exporting to CSV: {e}")

    def get_identity_data(self):
        """
        Identity 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "ID": identity.identity_id,
                "Name": identity.name,
                "Rarity": identity.rarity,
                "Skill_1_Type": identity.skill_1_type,
                "Skill_1_Sin_Affinity": identity.skill_1_sin_affinity,
                "Skill_2_Type": identity.skill_2_type,
                "Skill_2_Sin_Affinity": identity.skill_2_sin_affinity,
                "Skill_3_Type": identity.skill_3_type,
                "Skill_3_Sin_Affinity": identity.skill_3_sin_affinity,
                "Defense_Type": identity.defense_type,
                "Defense_Sin_Affinity": identity.defense_sin_affinity,
            }
            for identity in self.identities
        ]
        
    def export_identities_data(self, filename="identities.csv"):
        """Identity 데이터를 CSV로 내보내는 메서드"""
        identity_fieldnames = [
            "ID", "Name", "Rarity",
            "Skill_1_Type", "Skill_1_Sin_Affinity",
            "Skill_2_Type", "Skill_2_Sin_Affinity",
            "Skill_3_Type", "Skill_3_Sin_Affinity",
            "Defense_Type", "Defense_Sin_Affinity"
        ]
        identity_data = self.get_identity_data()  # Identity 데이터 가져오기
        self.export_to_csv(filename, identity_fieldnames, identity_data)  # CSV로 내보내기

    def load_identities_by_id(self, filename, identity_ids):
        """지정된 identity_ids 목록에 해당하는 Identity 데이터를 CSV에서 읽어옴."""
        identity_ids_set = set(identity_ids) # 중복 제거를 위 Set 사용
        with open(filename, mode='r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                if row["ID"] in identity_ids_set:
                    identity = Identity(
                        identity_id=row["ID"],
                        name=row["Name"],
                        rarity=row["Rarity"],
                        skill_1_type=row["Skill_1_Type"],
                        skill_1_sin_affinity=row["Skill_1_Sin_Affinity"],
                        skill_2_type=row["Skill_2_Type"],
                        skill_2_sin_affinity=row["Skill_2_Sin_Affinity"],
                        skill_3_type=row["Skill_3_Type"],
                        skill_3_sin_affinity=row["Skill_3_Sin_Affinity"],
                        defense_type=row["Defense_Type"],
                        defense_sin_affinity=row["Defense_Sin_Affinity"]
                    )
                    self.identities.append(identity)

    def get_level_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Level": level_stat.level,
                "HP": level_stat.hp,
                "Defense": level_stat.defense,
            }
            for identity in self.identities
            for level_stat in identity.level_stats
        ]

    def export_level_stats_data(self, filename="levelstats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        level_stats_fieldnames = ["Identity_ID", "Level", "HP", "Defense"]
        level_stats_data = self.get_level_stats_data()  # level_stats 데이터 가져오기
        self.export_to_csv(filename, level_stats_fieldnames, level_stats_data)  # CSV로 내보내기

    def get_uptie_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Uptie": uptie_stat.uptie,
                "Speed": uptie_stat.speed,
            }
            for identity in self.identities
            for uptie_stat in identity.uptie_stats
        ]

    def export_uptie_stats_data(self, filename="Uptiestats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        uptie_stats_fieldnames = ["Identity_ID", "Uptie", "Speed"]
        uptie_stats_data = self.get_uptie_stats_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, uptie_stats_fieldnames, uptie_stats_data)  # CSV로 내보내기

    def get_resist_info_data(self):
        """
        모든 Identity의 resist_info 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Resist_Hit": identity.resist_info.resist_hit,
                "Resist_Penetrate": identity.resist_info.resist_penetrate,
                "Resist_Slash": identity.resist_info.resist_slash,
            }
            for identity in self.identities
        ]

    def export_resist_info_data(self, filename="ResistInfos.csv"):
        """ResistInfo 데이터를 CSV로 내보내는 메서드"""
        resist_infos_fieldnames = ["Identity_ID", "Resist_Hit", "Resist_Penetrate", "Resist_Slash"]
        resist_infos_data = self.get_resist_info_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, resist_infos_fieldnames, resist_infos_data)  # CSV로 내보내기

    def export_uptie_stats_data(self, filename="Uptiestats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        uptie_stats_fieldnames = ["Identity_ID", "Uptie", "Speed"]
        uptie_stats_data = self.get_uptie_stats_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, uptie_stats_fieldnames, uptie_stats_data)  # CSV로 내보내기

    def get_stagger_threshold_data(self):
        """
        모든 Identity의 stagger_threshold 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "First_Stagger": identity.stagger_threshold.first_stagger,
                "Second_Stagger": identity.stagger_threshold.second_stagger,
                "Third_Stagger": identity.stagger_threshold.third_stagger,
            }
            for identity in self.identities
        ]

    def export_stagger_threshold_data(self, filename="StaggerThresholds.csv"):
        """StaggerThresholds 데이터를 CSV로 내보내는 메서드"""
        stagger_thresholds_fieldnames = ["Identity_ID", "First_Stagger", "Second_Stagger", "Third_Stagger"]
        stagger_thresholds_data = self.get_stagger_threshold_data()  # stagger_thresholds 데이터 가져오기
        self.export_to_csv(filename, stagger_thresholds_fieldnames, stagger_thresholds_data)  # CSV로 내보내기

    def get_skillset_data(self):
        """모든 Identity의 skillset 데이터를 export_to_csv을 포맷으로 변환"""
        return [
            {
                "Identity_ID": identity.identity_id,
                "Skill_Code": skill.skill_code,
                "Skill_Type": skill.skill_type,
                "Sin_Affinity": skill.sin_affinity,
                "Skill_Name": skill.skill_name,
                "Coin_Count": skill.coin_count,
                "Init_Level": skill.init_level,
                "Init_Remain": skill.init_remain,
                "Atk_Weight": skill.atk_weight,
                "Uptie_Level": uptie_info.uptie_level,
                "Skill_Power": uptie_info.skill_power,
                "Coin_Power": uptie_info.coin_power,
                "Effect_Data": json.dumps(uptie_info.skill_effect)
            }
            for identity in self.identities
            for skill_list in [identity.skillset.skill_1_details, identity.skillset.skill_2_details, identity.skillset.skill_3_details, identity.skillset.defense_skill_details]
            for skill in skill_list
            for uptie_info in skill.uptie_info
        ]

    def export_identity_skillset_data(self, filename="IdentitiesSkills.csv"):
        """IdentitiesSkills 데이터를 CSV로 내보내는 메서드"""
        identities_skills_fieldnames = ["Identity_ID", "Skill_Code", "Skill_Type", "Sin_Affinity", "Skill_Name", "Coin_Count", "Init_Level", "Init_Remain", "Atk_Weight", "Uptie_Level", "Skill_Power", "Coin_Power", "Effect_Data"]
        identities_skills_data = self.get_skillset_data()
        self.export_to_csv(filename, identities_skills_fieldnames, identities_skills_data)

    def get_identities_behaviors_data(self):
        """모든 Identity의 influence behavior 데이터를 export_to_csv 포맷으로 변환"""
        return [
            {
                "Identity_ID": identity.identity_id,
                "Effect_Type": effect_type,
                "Behavior_Name": effect.name,
                "Behavior_Condition": effect.condition if effect.condition else np.nan,
                "Behavior_Description" : effect.description
            }
            for identity in self.identities
            for effect_list, effect_type in [(identity.passive_infos, 'Passive'), (identity.support_passive_infos, 'Support Passive'), (identity.panic_type, 'Panic Type')]
            for effect in effect_list
        ]

    def export_identities_behaviors_data(self, filename="IdentitiesBehaviors.csv"):
        """IdentitiesBehaviors 데이터를 CSV로 내보내는 메서드"""
        identities_behaviors_fieldnames = ["Identity_ID","Effect_Type","Behavior_Name","Behavior_Condition","Behavior_Description"]
        identities_behaviors_data = self.get_identities_behaviors_data()
        self.export_to_csv(filename, identities_behaviors_fieldnames, identities_behaviors_data)
            
async def main():
    data_container = DataContainer()
    scraper = Scraper(container=data_container, base_url="https://limbus.kusoge.xyz", img_url="https://img.kusoge.xyz/limbus/img")
    await scraper.scrape()

await main() 

TimeoutError: Timeout 30000ms exceeded.

In [108]:
import os
import csv
import re
import json
import numpy as np
import pandas as pd
from playwright.async_api import async_playwright
from tabulate import tabulate

# Sin Color 이름 변환을 위한 매핑
resource_mapping = {
    "AZURE": "Gloom",
    "VIOLET": "Envy",
    "AMBER": "Sloth",
    "SHAMROCK": "Gluttony",
    "CRIMSON": "Wrath",
    "SCARLET": "Lust",
    "INDIGO": "Pride"
}

def convert_resource(resource_name):
    """Sin Color 이름을 변환하는 함수. 매핑되지 않은 경우 원래 이름을 반환."""
    return resource_mapping.get(resource_name, resource_name)

def find_sin_for_color(url):
    # URL에 color_name을 추출
    color_name = url.split('/')[-1].split('.')[0].replace("attr_", "")
    # 해당 색상이 resource_mapping에 있는 확인
    for color, sin in resource_mapping.items():
        if color_name in color:
            return sin
    return None


class Scraper:
    """웹 페이지 크롤링을 위한 클래스"""
    def __init__(self, container, base_url, img_url):
        self.container = container
        self.base_url = base_url
        self.img_url = img_url
        self.uptie_level = 1

    async def handle_request(self, route, request):
        """이미지와 폰트 리소스 로드를 차단."""
        if request.resource_type =="image":
            #print(f"{request.url}")
            img_url = request.url
            if  re.match(f"{self.img_url}/ui/attr.*\\.png", img_url):
                await route.continue_()
            else:
                await route.fulfill(
                    status=200,
                    content_type="image/png",
                    body=b""
                )
        elif request.resource_type == "font":
            await route.abort()
        else:
            await route.continue_()
            
    async def scrape(self):
        """크롤링 메인 함수"""
        async with async_playwright() as p:
            browser = await p.chromium.launch(headless=False)
            page = await browser.new_page()
            #await page.evaluate('window.scrollTo(0, document.body.scrollHeight)')
            await page.route("**/*", self.handle_request)
            # Step 1: /identity 페이지에서 기본 정보 크롤링
            #await self.crawl_identity_page(page)

            # Step 2: /identity/{identity_id} 상세 페이지 크롤링
            #identity_ids_to_load = ["10104", "10310", "10710","10808", "10911", "11005"]
            #identity_ids_to_load = ["10104", "10911"]#,"10310", "10710","10808", "11005", "11210"]
            #identity_ids_to_load = ["10911","10310"]#, "10710","10808", "11005", "11210"]
            identity_ids_to_load = ["10911", "10611", "10711"]#, "10710","10808", "11005", "11210"]
            self.container.load_identities_by_id('./csv/identities.csv', identity_ids_to_load)
            await self.crawl_identity_details_for_all(page)
            #self.container.display_identities()
            #self.container.export_identities_data("identities.csv")
            #self.container.export_level_stats_data("Levelstats.csv")
            #self.container.export_uptie_stats_data("Uptiestats.csv")
            #self.container.export_resist_info_data("ResistInfos.csv")
            #self.container.export_stagger_threshold_data("StaggerThresholds.csv")
            #self.container.export_identity_skillset_data("IdentitiesSkills.csv")
            self.container.export_identities_behaviors_data("IdentitiesBehaviors.csv")
            page.set_default_timeout(10000)
            #await page.wait_for_timeout(100000)
            await browser.close()

    async def get_identity_id_from_row(self, row):
        """테이블 행(row)에서 인격 ID를 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_url = await identity_cell.get_attribute('href')
            identity_id = identity_url.split('/identity/')[-1]
            return identity_id
        return "Unknown"

    async def get_identity_name_from_row(self, row):
        """테이블 행(row)에서 인격 이름을 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_name = (await identity_cell.inner_text()).replace("\n", " ").strip()
            return identity_name
        return "Unknown"
        
    async def crawl_identity_page(self, page):
        """/identity 페이지로 이동하여 인격 기본 정보를 추출."""
        await page.goto(self.base_url + "/identity")
        await self.extract_identity_page(page)

    async def extract_identity_page(self, page):
        """/identity 페이지의 모든 테이블 행에서 인격 정보를 추출."""
        identity_rows = await page.query_selector_all('tr')
        if len(identity_rows) > 1:
            for idx, identity_row in enumerate(identity_rows):
                if idx == 0:
                    continue # 0번째요소이름을 보여주는 열은 건너뜀
                await self.scrape_identity_name_and_identity_id(identity_row)
                await self.extract_rarity(identity_row)
                for i in range(1, 4):  # 스킬1~3 정보 추출
                    await self.extract_atk_and_resource(identity_row, i)
                await self.extract_defense_and_resource(identity_row)

    async def scrape_identity_name_and_identity_id(self, row):
        """인격 번호와 이름을 추출하여 DataContainer에 저장."""
        try:
            identity_id = await self.get_identity_id_from_row(row)
            identity_name = await self.get_identity_name_from_row(row)
            identity = Identity(identity_id)
            self.container.add_identity(identity)
            self.container.update_identity_name(identity_id, identity_name)
        except Exception as e:
            print(f"Error extracting identity name and identity id: {e}")

    async def get_element_attribute(self, element, selector, attribute):
        """요소에서 지정된 속성(attribute)을 추출."""
        try:
            target_element = await element.query_selector(selector)
            if target_element:
                return await target_element.get_attribute(attribute)
        except Exception as e:
            print(f"Error getting element attribute: {e}")
        return None
    
    async def extract_rarity(self, row):
        """테이블 행(row)에서 희귀도를 추출하여 업데이트."""
        try:
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
                sibling = await parent.evaluate_handle('node => node.nextElementSibling')
                if sibling:
                    rarity_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/rarity_"]', 'src')
                    if rarity_img_src:
                        rarity_level = rarity_img_src.split('_')[-1].split('.')[0]  # rarity_1.png -> 1
                        identity_id = await self.get_identity_id_from_row(row)
                        self.container.update_identity_rarity(identity_id, rarity_level)
                    else:
                        print("Rarity image not found in the next sibling TD.")
        except Exception as e: 
            print(f"Error extracting rarity: {e}")

        return "Unknown"

    async def extract_atk_and_resource(self, row, skill_number):
        """ 공격 타입(ATK Type)과 자원(Attribute) 추출 함수 - 스킬 번호에 따라 처리 """
        try:
            #print(f"Attempting to locate skill{skill_number} type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 해당 스킬 번호에 맞는 형제로 이동
                sibling = parent
                for _ in range(skill_number + 1):  # 스킬 번호에 맞게 이동
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                atk_type, resource = "Unknown", "Unknown"
        
                if sibling:
                    # 공격 타입 이미지 (atk_type) 찾기
                    atk_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/atk_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if atk_type_img_src:
                        atk_type = atk_type_img_src.split('_')[-1].split('.')[0]  # atk_type_SLASH.png -> SLASH
                        #print(f"Extracted attack type: {atk_type}")
                        self.container.update_skill(identity_id, skill_number, atk_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted resource: {resource}")
                        
                        # 리소스를 변환
                        resource = convert_resource(resource)
                        #print(f"Converted resource: {resource}")
                        self.container.update_sin_affinity(identity_id, skill_number, resource)
                        
        except Exception as e:
            print(f"Error extracting skill{skill_number} type and resource: {e}")

    async def extract_defense_and_resource(self, row):
        """ 방어 타입(Def Type)과 자원(Attribute) 추출 함수 """
        try:
            #print("Attempting to locate defense type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 다섯 번째 형제 td로 이동 (방어 관련 정보가 있는 <td>)
                sibling = parent
                for _ in range(5):  # 방어 정보는 5번째 형제에 있음
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                def_type, def_resource = "Unknown", "Unknown"
        
                if sibling:
                    # 방어 타입 이미지 (def_type) 찾기
                    def_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/def_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if def_type_img_src:
                        def_type = def_type_img_src.split('_')[-1].split('.')[0]  # def_type_GUARD.png -> GUARD
                        #print(f"Extracted defense type: {def_type}")
                        self.container.update_skill(identity_id, 'defense', def_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        def_resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted defense resource: {def_resource}")
                        
                        # 방어 리소스를 변환
                        def_resource = convert_resource(def_resource)
                        #print(f"Converted defense resource: {def_resource}")
                        self.container.update_sin_affinity(identity_id, 'defense', def_resource)
 
        except Exception as e:
            print(f"Error extracting defense type and resource: {e}")

    async def crawl_identity_details_for_all(self, page):
        """모든 인격의 세부 정보를 크롤링."""
        for identity in self.container.identities:
            await page.wait_for_load_state("networkidle")
            await page.goto(self.base_url + "/identity/"+identity.identity_id)
            """ 레벨과 동기화와 관련 없는 참관타 저항은 밑에서 변경하는 것과 연관성이 적기 때문에 위에서 처리한다.
            print(f"Identity_id: {identity.identity_id}")
            results = []
            resist_type_imgs = [f"{self.img_url}/ui/atk_type_HIT.png",
                                f"{self.img_url}/ui/atk_type_PENETRATE.png",
                                f"{self.img_url}/ui/atk_type_SLASH.png"]
            for resist_type_img in resist_type_imgs:
                result = await self.extract_resist_type(page, resist_type_img, 1)
                results.append(result)
            self.container.add_resist_info(identity.identity_id, results[0], results[1], results[2])
            """
            """
            # Stagger Threshold 값 추출
            stagger_threshold = await self.extract_stagger_threshold_values(page)
            self.container.add_stagger_threshold(identity.identity_id, stagger_threshold[0], stagger_threshold[1], stagger_threshold[2])
            """
            await self.initialize_uptie_and_level(page)
            """ 레벨에 따른 level, hp, defense 다르게 되어 있는 것을 출력 확인 객체로 들어가서 csv로 옮기는 과정까지 진행
            for i in range(1, identity.level_max+1):
                await self.move_slider_to_value(page,i)
                await self.extract_level_status(page, identity.identity_id, i)
            """
            """ 동기화에 따른 speed 다르게 되어 있는 것을 출력 확인 객체로 들어가서 csv로 옮기는 과정까지 진행
            for i in range(1, identity.uptie_max+1):
                await self.uptie_click(page,i)
                await self.extract_uptie_status(page, identity.identity_id, i)
            """
            """
                인격에 대한 skill 정보를 가져오는 메소드 위에서 출력하는 csv파일로 인격이 가지고 있는 스킬 정보를 내보낸다.
            await self.crawl_skills(page, identity.identity_id)
            """
            await self.crawl_influence_behavior(page, identity.identity_id)
            
    async def initialize_uptie_and_level(self, page):
        await self.uptie_click(page, 1)
        await self.move_slider_to_value(page, 1)
        self.uptie_level = 1
        #return 1
    
    async def uptie_click(self, page, phase):
        fab_button_selector = ".q-btn.q-btn--fab"
        await page.wait_for_selector(fab_button_selector)
        await page.click(fab_button_selector)
        #print(f"[{setup_uptie_button}] 상위 동기화 버튼 클릭 완료")
        setup_uptie_button = f'a:has(img[src="{self.img_url}/ui/uptie_{phase}.png"])'
        await page.wait_for_selector(setup_uptie_button)
        await page.click(setup_uptie_button)
        #print(f"{phase} 동기화 버튼 클릭 완료")

    async def move_slider_to_value(self, page, target_value):
        """
        슬라이더를 주어진 값으로 이동시킵니다.
        target_value는 슬라이더의 최소값부터 최대값 사이여야 합니다.
        """
        slider_selector = ".q-slider"
        # 슬라이더의 범위 확인
        min_value = int(await page.get_attribute(slider_selector, "aria-valuemin"))
        max_value = int(await page.get_attribute(slider_selector, "aria-valuemax"))
    
        if target_value < min_value or target_value > max_value:
            raise ValueError(f"Target value {target_value} is out of bounds ({min_value}, {max_value})")
    
        # 슬라이더 thumb 요소와 위치 가져오기
        slider_thumb = await page.query_selector(f"{slider_selector} .q-slider__thumb")
        thumb_box = await slider_thumb.bounding_box()
        
        # 슬라이더의 현재 위치 및 목표 위치 계산
        track = await page.query_selector(f"{slider_selector} .q-slider__track")
        track_box = await track.bounding_box()
        
        # 목표 x 위치 계산 (슬라이더 트랙 내에서 비율 기반)
        target_percentage = (target_value - min_value) / (max_value - min_value)
        target_x = track_box["x"] + target_percentage * track_box["width"]
        current_x = thumb_box["x"] + thumb_box["width"] / 2
        y = thumb_box["y"] + thumb_box["height"] / 2
    
        # 슬라이더를 드래그하여 목표 위치로 이동
        await page.mouse.move(current_x, y)
        await page.mouse.down()
        await page.mouse.move(target_x, y, steps=10)  # 부드럽게 이동
        await page.mouse.up()
    
        #print(f"슬라이더를 {target_value}로 이동 완료.")
    
        # 슬라이더 값 확인 및 강제 이벤트 트리거
        await page.evaluate(
            f"""
            (slider) => {{
                slider.dispatchEvent(new Event('input'));  // oninput 이벤트 트리거
                slider.dispatchEvent(new Event('change'));  // onchange 이벤트 트리거
            }}
            """,
            await page.query_selector(slider_selector)
        )
    
        # 이동 후 값 확인
        current_value = await page.get_attribute(slider_selector, "aria-valuenow")
        if current_value != str(target_value):
            print(f"경고: 슬라이더 값이 예상과 다릅니다. 현재 값: {current_value}, 예상 값: {target_value}")

    async def get_second_status_element(self, page):
        """
        두 번째 'Status' 요소에 접근하여 반환하는 메소드
        """
        status_elements = await page.query_selector_all("text=Status")
        if len(status_elements) > 1:
            second_status_element = status_elements[1]
            #print("두 번째 'Status' 요소에 접근합니다.")
            return second_status_element
        else:
            #print("두 번째 'Status' 요소가 존재하지 않습니다.")
            return None

    async def extract_level_status(self, page, identity_id, level):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 레벨에 따라 HP와 Defense가 변한다.
        hp_value = await self.extract_hp(second_status_element)
        defense_value = await self.extract_defense(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Level={level}, HP={hp_value}, Defense={defense_value}")
        self.container.add_level_stat(identity_id,level, hp_value, defense_value)
    
    async def extract_hp(self, status_element):
        # HP 추출
        hp_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].children[0].nextElementSibling.textContent.trim()"
        )
        return hp_value
    
    async def extract_defense(self, status_element):
        # Defense 추출
        defense_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return defense_value

    async def extract_uptie_status(self, page, identity_id, uptie):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 uptie에 따라 speed는 변한다.
        speed_value = await self.extract_speed(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Uptie={uptie}, Speed={speed_value}")
        self.container.add_uptie_stat(identity_id,uptie, speed_value)

    async def extract_speed(self, status_element):
        # Speed 추출
        speed_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return speed_value

    async def extract_resist_type(self, page, image_src, depth):
        # 이미지 요소 찾기
        #print(image_src)
        image = await page.query_selector(f'img[src="{image_src}"]')
        #print(image)
        if not image:
            return "Unknown"
    
        # 이미지의 부모 요소를 찾아서 올라감
        parent = image
        for _ in range(depth):
            parent = await parent.evaluate_handle('node => node.parentElement')
            if not parent:
                return "Unknown"
    
        # 부모의 형제 요소 찾기
        sibling = await parent.evaluate_handle('node => node.nextElementSibling')
        
        if sibling:
            value = await sibling.evaluate('node => node.textContent')
            if value:
                return value.strip()
        
        return "Unknown"

    async def extract_stagger_threshold_values(self, page):
        # 'div.row.center.flex-center.q-pt-xs' 요소들 찾기
        elements = await page.query_selector_all('div.row.center.flex-center.q-pt-xs')
        
        # 각 요소의 텍스트 내용 출력하여 'Stagger Threshold' 요소를 찾기
        for element in elements:
            text = await element.text_content()
            
            if text and "Stagger Threshold" in text:
                # 'Stagger Threshold'가 있는 요소를 찾았다면, 그 이후 형제 요소들에서 값을 추출
                parent = await element.evaluate_handle('node => node.parentElement')
                sibling_elements = await parent.query_selector_all('div.row.justify-evenly div')  # Threshold 값이 들어있는 div들
                
                stagger_values = []
                for sibling in sibling_elements:
                    text_content = await sibling.text_content()
                    match = re.search(r'(\d+%)', text_content)
                    if match:
                        stagger_values.append(match.group(1))  # 퍼센트 값만 리스트에 추가
                while len(stagger_values) < 3:
                    stagger_values.append(np.nan)
                #print(stagger_values)
                return stagger_values  # 추출된 값(또는 np.nan 포함)을 반환
        
        return [np.nan, np.nan, np.nan]  # 만약 'Stagger Threshold'를 찾지 못한 경우 기본값 반환

    async def crawl_skills(self, page, identity_id):
        await self.crawl_skill(page, identity_id, "skill 1")
        await self.crawl_skill(page, identity_id, "skill 2")
        await self.crawl_skill(page, identity_id, "skill 3")
        await self.crawl_skill(page, identity_id, "defense")


    async def crawl_skill(self, page, identity_id, skill_type):
        skills_elements = await page.query_selector_all('div.skilltag')
        for skill_element in skills_elements:
            skill_text = await skill_element.text_content()
            if skill_text.strip().lower() == skill_type:
                await self.process_skill(skill_element, page, identity_id, skill_type)
        
    async def process_skill(self, skill_element, page, identity_id, skill_type):
        # skill_type이 "skill 3"일 경우 uptie_value가 3 이상일 때만 정보를 저장
        for uptie_value in range(1, 5):  # Uptie levels 1 to 4
            # "skill 3"은 uptie_value가 3 이상일 때만 처리
            if skill_type == "skill 3" and uptie_value < 3:
                continue  # skip if uptie_value is less than 3 for skill 3
    
            # uptie_level에 맞게 클릭
            await self.uptie_click(page, uptie_value)
            if skill_type != "skill 3" and uptie_value > 1:
                # Uptie 정보 갱신
                await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
                continue
            elif skill_type == "skill 3" and uptie_value > 3:
                # Uptie 정보 갱신
                await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
                continue
            # 스킬의 세부 정보를 추출
            skill_details = await self.extract_skill_details(skill_element, skill_type)
            skill_name = skill_details["skill_name"]
    
            # 스킬 정보를 저장
            self.container.add_skill(
                identity_id, skill_type, 
                skill_name, skill_details["coin_count"],
                skill_details["init_level"], skill_details["init_remain"], 
                skill_details["atk_weight"]
            )
            print(f"Added skill: {skill_type}:{skill_name}")
    
            # Uptie 정보 갱신
            await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
    
    async def extract_skill_details(self, skill_element, skill_type):
        # 스킬 세부 정보를 추출하는 함수
        skill_name = await self.skill_name(skill_element)
        coin_count = await self.coin_count(skill_element, ".col-auto > .column > .row")
        init_level = await self.init_level(skill_element)
        init_remain = await self.init_remain(skill_element, skill_type, "div.row.justify-start > div.attr.flex.flex-center > div.q-px-sm")
        atk_weight = await self.atk_weight(skill_element, skill_type)
        return {
            "skill_name": skill_name,
            "coin_count": coin_count,
            "init_level": init_level,
            "init_remain": init_remain,
            "atk_weight": atk_weight
        }
    
    async def update_uptie_info(self, page, skill_element, identity_id, skill_name, skill_type, uptie_value):
        # Uptie 정보를 추출하고 업데이트하는 함수
        uptie_info = await self.extract_uptie_info(skill_element)
        effect_data = await self.skill_effects(skill_element)
        self.container.add_skill_uptie_info(
            identity_id, skill_name, 
            uptie_value, uptie_info["skill_power"], uptie_info["coin_power"], 
            effect_data.get("effectsByCondition")
        )
        print(
            f"Updated Uptie for {skill_name} (Uptie Level: {uptie_value}): "
            f"Skill Power: {uptie_info['skill_power']}, Coin Power: {uptie_info['coin_power']} "
            f"Skill Effect: {effect_data.get("effectsByCondition")}"
        )

    async def extract_uptie_info(self, skill_element):
        skill_data = await self.skill_and_coin_power(skill_element)
        return {
            "skill_power": skill_data.get("skillPower"),
            "coin_power": skill_data.get("coinPower")
        }
        
    async def crawl_influence_behavior(self, page, identity_id):
        influence_behaviors = await page.query_selector_all('div[align="right"]')
        # 텍스트를 기준으 추출하고 메소드로 파라미터 들어가 정보를 받는다.
        for influence_behavior in influence_behaviors:
            active_type = await influence_behavior.inner_text()
            active_type = active_type.strip()

            if active_type == "PASSIVE":
                """
                # PASSIVE에 대한 처리
                print(f"Processing PASSIVE logic for: {active_type}")
                # CALL method for PASSIVE
                """
                await self.passive_effects(influence_behavior, identity_id)
            elif active_type == "SUPPORT PASSIVE":
                """
                # SUPPORT PASSIVE에 대한 처리
                print(f"Processing SUPPORT PASSIVE logic for: {active_type}")
                # CALL method for SUPPORT PASSIVE
                """
                await self.support_passive_effects(influence_behavior, identity_id)
            elif active_type == "PANIC TYPE":
                """
                # PANIC TYPE에 대한 처리
                print(f"Processing PANIC TYPE logic for:{active_type}")
                # CALL method for PANIC TYPE
                """
                await self.panic_effects(influence_behavior, identity_id)

    async def skill_and_coin_power(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                // 조부모로 이동
                const grandparent = node.parentElement.parentElement;
                // 이전 형제 탐색
                const previousSibling = grandparent.previousElementSibling;
                if (!previousSibling) return {{ skillPower: null, coinPower: null }}; // 이전 형제가 없으면 기본값 반환
                // 이전 형제 내부의 .row.justify-around > div > div 탐색
                const targetDivs = previousSibling.querySelectorAll(".row.justify-around > div");
                if (targetDivs.length < 3) return {{ skillPower: null, coinPower: null }}; // 배열 길이 확인
                // 1st div: 스킬 위력
                const skillPower = targetDivs[0]?.querySelector('div')?.textContent?.trim() || null; 
                // 3rd div: 코인 파워 값 (3번째 div에서 텍스트 추출)
                const coinPower = targetDivs[2]?.textContent?.trim() || null;
                return {{ skillPower, coinPower }};
            }}
        """)


    async def coin_count(self, skill_element, hierarchy_selector):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector('{hierarchy_selector}');
                if (!targetElement) return 0;
                return targetElement.childElementCount;
            }}
        """)

    async def init_remain(self, skill_element, skill_type, hierarchy_selector):
        if skill_type == "defense":
            return -1
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector('{hierarchy_selector}');
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def skill_name(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector(".block");
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def init_level(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector("div.text-bold.text-h6.h6.q-pl-xs.q-pr-md");
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def atk_weight(self, skill_element, skill_type):
        if skill_type == "defense":
            return 0
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector("div.text-bold.text-h6.h6.q-pl-xs.q-pr-md");
                const doubleNextSibling = targetElement.nextElementSibling.nextElementSibling;
                if (!doubleNextSibling) return 0;
                return doubleNextSibling.textContent?.trim();
            }}
        """)

    async def skill_effects(self, skill_element):
        return await skill_element.evaluate("""
            node => {
                const parent = node.parentElement;
                const doubleNextSibling = parent.nextElementSibling?.nextElementSibling;
                if (!doubleNextSibling) return null;
    
                // doubleNextSibling에서 첫 번째 div.col 요소 찾기
                const textNode = doubleNextSibling.querySelector("div.col");
                if (!textNode) return null;
    
                const results = {
                    keywords: [], // 키워드 저장
                    conditions: [], // 조건 저장
                    effectsByCondition: {} // 조건별 효과
                };
    
                // 1. 조건 추출 (대괄호 텍스트)
                const conditionElements = textNode.querySelectorAll("span");
                conditionElements.forEach(span => {
                    let text = span.textContent?.trim();
                    if (text?.startsWith("[") && text?.endsWith("]")) {
                        // 부모의 전형제 노드 탐색
                        const sibling = span.parentElement.previousElementSibling;
                        if (sibling) {
                            const emblemImage = sibling.querySelector("img.emblem");
                            console.log("Sibling element found:", sibling); // 디버깅용
                            if (emblemImage) {
                                const src = emblemImage.getAttribute("src");
                                console.log("Image source:", src); // 디버깅용
                                const match = src.match(/ui_num_(\\d+)\\.png$/); // 정규식 수정
                                if (match) {
                                    const number = match[1]; // 숫자 추출
                                    text = `[Coin ${number} ${text.replace("[", "").replace("]", "")}]`;
                                    console.log("Updated condition:", text); // 디버깅용
                                }
                            }
                        }
                        results.conditions.push(text);
                        results.effectsByCondition[text] = []; // 조건별 효과를 초기화
                    }
                });
    
                // 2. 키워드 추출
                const keywordElements = textNode.querySelectorAll('a[href^="/en/keyword/"]');
                keywordElements.forEach(a => {
                    const span = a.querySelector('span.q-btn__content.text-center.col.items-center.q-anchor--skip.justify-center.row>span');
                    if (span) {
                        const keywordText = span.textContent.trim();
                        if (keywordText && !results.keywords.includes(keywordText)) {
                            results.keywords.push(keywordText); // 키워드 저장
                        }
                    }
                });
    
                // 3. 효과 추출
                const effectElements = textNode.querySelectorAll("span");
                let currentCondition = null;
                let currentEffect = []; // 초기화
                effectElements.forEach(span => {
                    let text = span.textContent?.trim();
                    if (!text) return;
    
                    // 조건 발견 시 currentCondition 변경
                    if (text.startsWith("[") && text.endsWith("]")) {
                        // 이전 조건의 효과 저장
                        if (currentCondition && currentEffect.length > 0) {
                            // 부모의 전형제 노드 탐색
                            const sibling = span.parentElement.previousElementSibling;
                            if (sibling) {
                                const emblemImage = sibling.querySelector("img.emblem");
                                if (emblemImage) {
                                    const src = emblemImage.getAttribute("src");
                                    const match = src.match(/ui_num_(\\d+)\\.png$/); // 정규식 수정
                                    if (match) {
                                        const number = match[1]; // 숫자 추출
                                        text = `[Coin ${number} ${text.replace("[", "").replace("]", "")}]`;
                                    }
                                }
                            }
                            results.effectsByCondition[currentCondition].push(currentEffect.join(' '));
                            currentEffect = []; // 초기화
                        }
                        currentCondition = text;
                        return;
                    }
    
                    // 현재 조건의 효과 처리
                    if (currentCondition) {
                        if (results.keywords.includes(text)) {
                            const keywordEffect = 'keyword:' + text;
                            if (!currentEffect.includes(keywordEffect)) {
                                currentEffect.push(keywordEffect);
                            }
                        } else {
                            currentEffect.push(text);
                        }
                    }
                });
    
                // 마지막 효과 저장
                if (currentCondition && currentEffect.length > 0) {
                    results.effectsByCondition[currentCondition].push(currentEffect.join(' '));
                }
    
                return results;
            }
        """)

    async def passive_effects(self, passive_element, identity_id):
        passive_name = await self.behavior_name(passive_element)
        passive_conditions = await self.behavior_condition(passive_element)
        passive_description = await self.behavior_description(passive_element, "span")
        #print(passive_name)
        #print(passive_conditions)
        #print(passive_description)
        self.container.add_influence_behavior(identity_id, "passive", passive_name, passive_conditions, passive_description.get("description"))
    async def support_passive_effects(self, support_passive_element, identity_id):
        support_passive_name = await self.behavior_name(support_passive_element)
        support_passive_conditions = await self.behavior_condition(support_passive_element)
        support_passive_description = await self.behavior_description(support_passive_element, "span")
        #print(support_passive_name)
        #print(support_passive_conditions)
        #print(support_passive_description)
        self.container.add_influence_behavior(identity_id, "support passive", support_passive_name, support_passive_conditions, support_passive_description.get("description"))
    async def panic_effects(self, panic_element, identity_id):
        panic_name = await self.behavior_name(panic_element)
        panic_conditions = await self.behavior_condition(panic_element)
        panic_description = await self.behavior_description(panic_element, "div")
        #print(panic_type)
        #print(panic_conditions)
        #print(panic_description)
        self.container.add_influence_behavior(identity_id, "panic type", panic_name, panic_conditions, panic_description.get("description"))

    async def navigate_element_by_steps(self, element, parent_steps=0, prev_sibling_steps=0, next_sibling_steps=0):
        """
        주어진 `element`에서 부모, 이전 형제, 또는 다음 형제로 지정된 횟수만큼 이동하여 ElementHandle 반환.
        
        Args:
            element (ElementHandle): 탐색 기준이 되는 요소.
            parent_steps (int): 부모로 이동할 횟수 (0이면 이동하지 않음).
            prev_sibling_steps (int): 이전 형제로 이동할 횟수 (0이면 이동하지 않음).
            next_sibling_steps (int): 다음 형제로 이동할 횟수 (0이면 이동하지 않음).
        
        Returns:
            ElementHandle or None: 탐색된 요소를 반환하거나 없으면 None.
        """
        try:
            current_element = element
    
            # 부모로 이동
            for _ in range(parent_steps):
                current_element = await current_element.evaluate_handle('el => el.parentElement')
                if not current_element:
                    print("Parent element not found during navigation.")
                    return None
    
            # 이전 형제로 이동
            for _ in range(prev_sibling_steps):
                current_element = await current_element.evaluate_handle('el => el.previousElementSibling')
                if not current_element:
                    print("Previous sibling element not found during navigation.")
                    return None
    
            # 다음 형제로 이동
            for _ in range(next_sibling_steps):
                current_element = await current_element.evaluate_handle('el => el.nextElementSibling')
                if not current_element:
                    print("Next sibling element not found during navigation.")
                    return None
    
            return current_element
    
        except Exception as e:
            print(f"Error navigating elements: {e}")
            return None

    async def get_last_sibling(self, element):
        try:
            # 같은 부 아 마지막 형제를 가져오는 JavaScript 실행
            last_sibling = await element.evaluate_handle('el => el.parentElement.lastElementChild')
            return last_sibling
        except Exception as e:
            print(f"Error getting last sibling: {e}")
            return None

    async def behavior_name(self, element):
        parent_prev_sibling = await self.navigate_element_by_steps(element, parent_steps=1, prev_sibling_steps=1, next_sibling_steps=0)
        passive_name_element = await parent_prev_sibling.query_selector('span.block')
        if passive_name_element:
            passive_name = await passive_name_element.inner_text()
            return passive_name
        else:
            #print("Passive name element not found")
            return None

    async def behavior_condition(self, element):
        try:
            conditions = []
            parent_prev_sibling = await self.navigate_element_by_steps(element, parent_steps=1, prev_sibling_steps=1, next_sibling_steps=0)
            img_elements = await parent_prev_sibling.query_selector_all('img')
            if not img_elements:
                #print("No <img> elements found.")
                return conditions
    
            for img_element in img_elements:
                # 각 이미지의 src 속성 가져오기
                img_src = await img_element.get_attribute('src')
                #print(img_src)
    
                sin_name = None
                for color_key in resource_mapping.keys():
                    if f"{self.img_url}/ui/attr_{color_key}.png" in img_src:
                        sin_name = resource_mapping[color_key]
                        break
    
                if not sin_name:
                    #print(f"No matching Sin Color found for {img_src}.")
                    continue  # No Sin Color found, skip this img_element
    
                # 해당 이미지와 관련된 조건 텍스트 추출
                # 두 번째 부모 요소에서 시작하여 다음 형제를 찾아 조건 텍스트를 추출
                condition_element = await self.navigate_element_by_steps(img_element, parent_steps=3, prev_sibling_steps=0, next_sibling_steps=0)
                if condition_element:
                    # condition_element가 None이 아닌 경우에만 진행
                    condition_text_element = await condition_element.query_selector('.q-pl-xs')
                    if condition_text_element:
                        condition_text = await condition_text_element.inner_text()
                        condition_text = condition_text.strip()
                    else:
                        print(f"No condition text found for {img_src}.")
                        condition_text = ""  # condition_text_element가 None이면 빈 문자열로 처리
                else:
                    print(f"No condition element found for {img_src}.")
                    condition_text = ""  # condition_element가 None이면 빈 문자열로 처리
    
                # 결과를 배열에 추가
                conditions.append(f"{sin_name} {condition_text}")
            return conditions
        except Exception as e:
            print(f"Error extracting conditions: {e}")
            return 
            
    async def behavior_description(self, element, tag_type):
        parent_prev_sibling = await self.navigate_element_by_steps(
            element, parent_steps=1, prev_sibling_steps=1, next_sibling_steps=0
        )
        return await parent_prev_sibling.evaluate(f"""
            (node) => {{
                const behavior_title_node = node.querySelector('.q-pb-sm');
                const behavior_title_parent = behavior_title_node.parentElement;
                const behavior_description = behavior_title_parent.lastElementChild;
                if (!behavior_description) return null;
    
                const results = {{
                    keywords: [], // 키워드 저장
                    description: [] // 설명 저장
                }};
                
                // 1. 키워드 추출
                const keywordElements = behavior_description.querySelectorAll('a[href^="/en/keyword/"]');
                if (!keywordElements) return null;
                keywordElements.forEach(a => {{
                    const span = a.querySelector('span.q-btn__content.text-center.col.items-center.q-anchor--skip.justify-center.row>span');
                    if (span) {{
                        const keywordText = span.textContent.trim();
                        if (keywordText && !results.keywords.includes(keywordText)) {{
                            results.keywords.push(keywordText); // 키워드 저장
                        }}
                    }}
                }});
    
                // 2. 설명 추출
                const descriptionElements = behavior_description.querySelectorAll("{tag_type}");
                let currentEffect = []; // 초기화
                descriptionElements.forEach(span => {{
                    let text = span.textContent?.trim();
                    console.log("origin text:", text);
                    if (!text) return;
                    // 공백과 기호 처리
                    text = text.replace(/\\s*([.,:;!?()\\[\\]])\\s*/g, '$1'); // 기호 앞뒤 공백 제거
                    console.log("after symbol processing:", text);
                    console.log("after spacing character: ", text);
                    if (results.keywords.includes(text)) {{
                        const keywordEffect = 'keyword:' + text;
                        if (!currentEffect.includes(keywordEffect)) {{
                            currentEffect.push(keywordEffect);
                        }}
                    }} else {{
                        currentEffect.push(text);
                    }}
                    console.log("current effect:", currentEffect);
                }});
    
                // 마지막 효과 저장
                if (currentEffect.length > 0) {{
                    const finalDescription = currentEffect.join(' ');
                    console.log("After process:", finalDescription);
                    results.description.push(finalDescription);
                }}
                let processedText = results.description
                    .map(word => word.trim())
                    .join(" ")
                    .replace(/\\s+([,.:])/g, "$1")
                    .replace(/\\s+'/g, "'")
                    .replace(/'\\s+/g, "'")
                    .replace(/\\[\\s+/g, "[")
                    .replace(/\\s+\\]/g, "]")
                    .replace(/\\(\\s+/g, "(")
                    .replace(/\\s+\\)/g, ")")
                    .replace(/\\s+%/g, "%")
                    .replace(/\\s+\\//g, "/")
                    .replace(/\\/\\s+/g, "/")
                    .replace(/\\s+\\/\\s+/g, "/");
                results.description = processedText;
                return results;
            }}
        """)

class Identity:
    """인격 정보를 저장하는 클래스"""
    def __init__(self, identity_id, name=None, rarity=None,\
                 skill_1_type=None, skill_1_sin_affinity=None,\
                 skill_2_type=None, skill_2_sin_affinity=None,\
                 skill_3_type=None, skill_3_sin_affinity=None,\
                 defense_type=None, defense_sin_affinity=None):
        self.identity_id = identity_id
        self.name = name
        self.rarity = rarity
        # 선언만 하는 부분 
        self.skill_1_type = skill_1_type 
        self.skill_1_sin_affinity = skill_1_sin_affinity 
        self.skill_2_type = skill_2_type 
        self.skill_2_sin_affinity = skill_2_sin_affinity 
        self.skill_3_type = skill_3_type 
        self.skill_3_sin_affinity = skill_3_sin_affinity 
        self.defense_type = defense_type 
        self.defense_sin_affinity = defense_sin_affinity
        self.level_max = 50
        self.uptie_max = 4
        self.level_stats = []  # LevelStat 객체 리스트
        self.uptie_stats = [] # UptieStat 객체 리스트
        self.resist_info = None
        self.stagger_threshold = None
        self.skillset = SkillSet()  # Skill정보를 담고 있는 skillset 객체 
        self.passive_infos = []  # 패시브 객체를 담고 있는 배열
        self.support_passive_infos = [] # 서포트 패시 객체를 담고 있는 배열
        self.panic_type = [] # 혼란상태 발생시 나타나는 효과를 담고 있는 객체 배열1

    def update_name(self, name):
        self.name = name

    def update_rarity(self, rarity):
        self.rarity = rarity

    def add_level_stat(self, level, hp, defense):
        self.level_stats.append(LevelStat(level, hp, defense))

    def add_uptie_stat(self, uptie, speed):
        self.uptie_stats.append(UptieStat(uptie, speed))

    def add_resist_info(self, resist_hit, resist_penetrate, resist_slash):
        self.resist_info = ResistInfo(resist_hit, resist_penetrate, resist_slash)

    def add_stagger_threshold(self, first_stagger, second_stagger, third_stagger):
        self.stagger_threshold = StaggerThreshold(first_stagger, second_stagger, third_stagger)

    def add_skill(self, skill_code, skill_name, coin_count, init_level, init_remain, atk_weight):
        self.skillset.add_skill(skill_code, Skill(skill_code, skill_name, coin_count, init_level, init_remain, atk_weight))
        
    def add_skill_uptie_info(self, skill_name, uptie_level, skill_power, coin_power, effect_by_condition):
        self.skillset.add_skill_uptie_info(skill_name, uptie_level, skill_power, coin_power, effect_by_condition)

    def add_influence_behavior(self, behavior_type, behavior_name, condition, description):
        if behavior_type == "passive":
            self.passive_infos.append(Passive(behavior_type, behavior_name, condition, description))
        elif behavior_type == "support passive":
            self.support_passive_infos.append(SupportPassive(behavior_type, behavior_name, condition, description))
        elif behavior_type == "panic type":
            self.panic_type.append(PanicType(behavior_type, behavior_name, condition, description))
        
    def update_details(self, page):
        # 세부 정보 업데이트 로직 (가정)
        pass
    #def init_uptie(self, page):
    #    init_uptie = 

# LevelStat 객체 정의
class LevelStat:
    def __init__(self, level, hp=None, defense=None):
        self.level = level
        self.hp = hp
        self.defense = defense
    def update_level(self, level):
        self.level = level
    def update_hp(self, hp):
        self.hp = hp
    def update_defense(self, defense):
        self.defense = defense

# UptieStat 객체 정의
class UptieStat:
    def __init__(self, uptie, speed=None):
        self.uptie = uptie
        self.speed = speed
    def update_speed(self, speed):
        self.speed = speed

# ResistInfo 객체 정의
class ResistInfo:
    def __init__(self, resist_hit=None, resist_penetrate=None, resist_slash=None):
        self.resist_hit = resist_hit
        self.resist_penetrate = resist_penetrate
        self.resist_slash = resist_slash

# ResistInfo 객체 정의
class StaggerThreshold:
    def __init__(self, first_stagger=None, second_stagger=None, third_stagger=None):
        self.first_stagger = first_stagger
        self.second_stagger = second_stagger
        self.third_stagger = third_stagger

class SkillSet:
    def __init__(self):

        self.skill_1_type = ""
        self.skill_2_type = ""
        self.skill_3_type = ""
        self.defense_skill_type = ""

        self.skill_1_sin_affinity = ""
        self.skill_2_sin_affinity = ""
        self.skill_3_sin_affinity = ""
        self.defense_skill_sin_affinity = ""

        self.skill_1_details = []
        self.skill_2_details = []
        self.skill_3_details = []
        self.defense_skill_details = []

    def set_skill_type(self, skill_code, skill_type, sin_affinity):
        if skill_code == "skill 1":
            self.skill_1_type = skill_type
            self.skill_1_sin_affinity = sin_affinity
        elif skill_code == "skill 2":
            self.skill_2_type = skill_type
            self.skill_2_sin_affinity = sin_affinity
        elif skill_code == "skill 3":
            self.skill_3_type = skill_type
            self.skill_3_sin_affinity = sin_affinity
        elif skill_code == "defense":
            self.defense_skill_type = skill_type
            self.defense_skill_sin_affinity = sin_affinity
    
    def add_skill(self, skill_code, skill):
        if skill_code == "skill 1":
            self.skill_1_details.append(skill)
            skill.skill_type = self.skill_1_type
            skill.sin_affinity = self.skill_1_sin_affinity
        elif skill_code == "skill 2":
            self.skill_2_details.append(skill)
            skill.skill_type = self.skill_2_type
            skill.sin_affinity = self.skill_2_sin_affinity
        elif skill_code == "skill 3":
            self.skill_3_details.append(skill)
            skill.skill_type = self.skill_3_type
            skill.sin_affinity = self.skill_3_sin_affinity
        elif skill_code == "defense":
            self.defense_skill_details.append(skill)
            skill.skill_type = self.defense_skill_type 
            skill.sin_affinity = self.defense_skill_sin_affinity

    def get_skill_by_name(self, skill_name):
        for skill_list in [self.skill_1_details, self.skill_2_details, self.skill_3_details, self.defense_skill_details]:
            for skill in skill_list:
                result = skill.is_skill_by_name(skill_name)
                if result:
                    return result
        return None

    def add_skill_uptie_info(self, skill_name, uptie_level, skill_power, coin_power, effect_by_condition):
        skill = self.get_skill_by_name(skill_name)
        skill.add_uptie_info(uptie_level, skill_power, coin_power, effect_by_condition)
        

class Skill:
    def __init__(self, skill_code, skill_name, coin_count, init_level, init_remain, atk_weight):
        self.skill_code = skill_code # "Skill 1"/"Skill 2"/"Skill 3"/"Defense" '''o'''
        self.skill_type = None
        self.sin_affinity = None
        self.skill_name = skill_name # 스킬 이름 '''o'''
        self.coin_count = coin_count # 동전 개수 '''o'''
        self.init_level = init_level # 스킬이 초기 배정받은 레벨, 인격 레벨에 - 1 한 값을 더해주면 스킬의 현 level을 구할 수 있다. '''o'''
        self.init_remain = init_remain #'''o'''
        self.atk_weight = atk_weight #공격 가중치 값이다. 방어스킬에는 없기 때문에 None으로 설정했다. '''o'''
        self.remain_count = init_remain # 실제 스킬이 작동하며 횟수가 차감되 나중에 모두 차감되면 다음 턴에 다시 초기횟수 리필된다.
        self.uptie_info = []  

    def is_skill_by_name(self, skill_name):
        if self.skill_name == skill_name:
            return self
        return None
        
    def add_uptie_info(self, uptie_level, skill_power, coin_power, effects_by_condition):
        self.uptie_info.append(UptieInfo(uptie_level, skill_power, coin_power, effects_by_condition))

class UptieInfo:
    def __init__(self, uptie_level, skill_power, coin_power, effects_by_condition):
        self.uptie_level = uptie_level   #'''o'''
        self.skill_power = skill_power   #'''o'''
        self.coin_power = coin_power     #'''o'''
        self.skill_effect = effects_by_condition

# 추상화된 EffectType 클래스
class EffectType:
    def __init__(self, behavior_type, name, description):
        self.behavior_type = behavior_type
        self.name = name  # Effect 이름
        self.description = description  # Effect 설명

# Passive 클래스
class Passive(EffectType):
    def __init__(self, behavior_type, name, condition, description):
        super().__init__(behavior_type, name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)  # condition이 있는지 여부 확인

# SupportPassive 클래스
class SupportPassive(EffectType):
    def __init__(self,behavior_type, name, condition, description):
        super().__init__(behavior_type, name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)

# PanicType 클래스
class PanicType(EffectType):
    def __init__(self, behavior_type, name, condition, description):
        super().__init__(behavior_type, name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)

# Keyword 클래스
class Keyword:
    def __init__(self, keyword, explain=None, dispellable=None):
        self.keyword = keyword
        self.explain = explain
        self.dispellable = dispellable

      
class DataContainer:
    """크롤링 데이터를 관리하는 클래스"""
    def __init__(self):
        self.identities = []

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.level_max

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.uptie_max

    def update_identity_name(self, identity_id, identity_name):
        """특정 ID의 이름을 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_name(identity_name)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_identity_rarity(self, identity_id, rarity):
        """특정 ID의 희귀도를 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_rarity(rarity)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_skill(self, identity_id, skill_number, new_type):
        """
        특정 identity의 skill_type을 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_type: 새 skill_type
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_type = new_type
                elif skill_number == 2:
                    identity.skill_2_type = new_type
                elif skill_number == 3:
                    identity.skill_3_type = new_type
                elif skill_number == "defense":
                    identity.defense_type = new_type
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def update_sin_affinity(self, identity_id, skill_number, new_affinity):
        """
        특정 identity의 sin_affinity를 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_affinity: 새 sin_affinity
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_sin_affinity = new_affinity
                elif skill_number == 2:
                    identity.skill_2_sin_affinity = new_affinity
                elif skill_number == 3:
                    identity.skill_3_sin_affinity = new_affinity
                elif skill_number == "defense":
                    identity.defense_sin_affinity = new_affinity
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def add_identity(self, identity):
        """새로운 인격 데이터를 추가."""
        self.identities.append(identity)

    def add_level_stat(self, identity_id, level, hp, defense):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_level_stat(level, hp, defense)

    def add_uptie_stat(self, identity_id, uptie, speed):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_uptie_stat(uptie, speed)

    def add_resist_info(self, identity_id, resist_hit, resist_penetrate, resist_slash):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_resist_info(resist_hit, resist_penetrate, resist_slash)

    def add_stagger_threshold(self, identity_id, first_stagger, second_stagger, third_stagger):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_stagger_threshold(first_stagger, second_stagger, third_stagger)

    
    def add_skill(self, identity_id, skill_code, skill_name, coin_count, init_level, init_remain, atk_weight):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_code == "skill 1":
                    identity.skillset.set_skill_type(skill_code, identity.skill_1_type, identity.skill_1_sin_affinity)
                elif skill_code == "skill 2":
                    identity.skillset.set_skill_type(skill_code, identity.skill_2_type, identity.skill_2_sin_affinity)
                elif skill_code == "skill 3":
                    identity.skillset.set_skill_type(skill_code, identity.skill_3_type, identity.skill_3_sin_affinity)
                elif skill_code == "defense":
                    identity.skillset.set_skill_type(skill_code, identity.defense_type, identity.defense_sin_affinity)
                identity.add_skill(skill_code, skill_name, coin_count, init_level, init_remain, atk_weight)

    def add_skill_uptie_info(self, identity_id, skill_name, uptie_level, skill_power, coin_power, effects_by_condition):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_skill_uptie_info(skill_name, uptie_level, skill_power, coin_power, effects_by_condition)

    def add_influence_behavior(self, identity_id, behavior_type, behavior_name, condition, description):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_influence_behavior(behavior_type, behavior_name, condition, description)
        
    def display_identities(self):
        """Identity, ID, Name, Rarity 정보를 출력"""
        if not self.identities:
            print("No identities found.")
        else:
            # 데이터를 리스트로 변환
            table = []
            for idx, identity in enumerate(self.identities, 1):
                table.append([
                    idx, identity.identity_id, identity.name, identity.rarity,
                    identity.skill_1_type, identity.skill_1_sin_affinity,
                    identity.skill_2_type, identity.skill_2_sin_affinity,
                    identity.skill_3_type, identity.skill_3_sin_affinity,
                    identity.defense_type, identity.defense_sin_affinity
                ])
            
            # 테이블 헤더 정의
            headers = [
                "Index", "ID", "Name", "Rarity",
                "Skill_1 Type", "Skill_1 Sin", "Skill_2 Type", "Skill_2 Sin",
                "Skill_3 Type", "Skill_3 Sin", "Defense Type", "Defense Sin"
            ]
            
            # 테이블 출력
            print(tabulate(table, headers=headers, tablefmt="fancy_grid"))

    def export_to_csv(self, filename, fieldnames, data):
        """
        공통적으로 사용 가능한 CSV 내보내기 메서드
        :param filename: 내보낼 파일 이름 (예: 'identity.csv', 'skill.csv')
        :param fieldnames: CSV의 헤더 리스트
        :param data: CSV로 내보낼 데이터 (리스트 형태)
        """
        try:
            # 현재작업 디렉토 기준으 'csv'폴더 없으면 생성
            if not os.path.exists('./csv'):
                os.makedirs('./csv')
            # 파일 경로 설정 (상대경로 'csv' 폴더 안에 저장)
            file_path = os.path.join('./csv', filename)
            with open(file_path, mode='w', newline='', encoding='utf-8') as file:
                writer = csv.DictWriter(file, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(data)
            print(f"Data successfully exported to {filename}")
        except Exception as e:
            print(f"Error exporting to CSV: {e}")

    def get_identity_data(self):
        """
        Identity 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "ID": identity.identity_id,
                "Name": identity.name,
                "Rarity": identity.rarity,
                "Skill_1_Type": identity.skill_1_type,
                "Skill_1_Sin_Affinity": identity.skill_1_sin_affinity,
                "Skill_2_Type": identity.skill_2_type,
                "Skill_2_Sin_Affinity": identity.skill_2_sin_affinity,
                "Skill_3_Type": identity.skill_3_type,
                "Skill_3_Sin_Affinity": identity.skill_3_sin_affinity,
                "Defense_Type": identity.defense_type,
                "Defense_Sin_Affinity": identity.defense_sin_affinity,
            }
            for identity in self.identities
        ]
        
    def export_identities_data(self, filename="identities.csv"):
        """Identity 데이터를 CSV로 내보내는 메서드"""
        identity_fieldnames = [
            "ID", "Name", "Rarity",
            "Skill_1_Type", "Skill_1_Sin_Affinity",
            "Skill_2_Type", "Skill_2_Sin_Affinity",
            "Skill_3_Type", "Skill_3_Sin_Affinity",
            "Defense_Type", "Defense_Sin_Affinity"
        ]
        identity_data = self.get_identity_data()  # Identity 데이터 가져오기
        self.export_to_csv(filename, identity_fieldnames, identity_data)  # CSV로 내보내기

    def load_identities_by_id(self, filename, identity_ids):
        """지정된 identity_ids 목록에 해당하는 Identity 데이터를 CSV에서 읽어옴."""
        identity_ids_set = set(identity_ids) # 중복 제거를 위 Set 사용
        with open(filename, mode='r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                if row["ID"] in identity_ids_set:
                    identity = Identity(
                        identity_id=row["ID"],
                        name=row["Name"],
                        rarity=row["Rarity"],
                        skill_1_type=row["Skill_1_Type"],
                        skill_1_sin_affinity=row["Skill_1_Sin_Affinity"],
                        skill_2_type=row["Skill_2_Type"],
                        skill_2_sin_affinity=row["Skill_2_Sin_Affinity"],
                        skill_3_type=row["Skill_3_Type"],
                        skill_3_sin_affinity=row["Skill_3_Sin_Affinity"],
                        defense_type=row["Defense_Type"],
                        defense_sin_affinity=row["Defense_Sin_Affinity"]
                    )
                    self.identities.append(identity)

    def get_level_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Level": level_stat.level,
                "HP": level_stat.hp,
                "Defense": level_stat.defense,
            }
            for identity in self.identities
            for level_stat in identity.level_stats
        ]

    def export_level_stats_data(self, filename="levelstats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        level_stats_fieldnames = ["Identity_ID", "Level", "HP", "Defense"]
        level_stats_data = self.get_level_stats_data()  # level_stats 데이터 가져오기
        self.export_to_csv(filename, level_stats_fieldnames, level_stats_data)  # CSV로 내보내기

    def get_uptie_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Uptie": uptie_stat.uptie,
                "Speed": uptie_stat.speed,
            }
            for identity in self.identities
            for uptie_stat in identity.uptie_stats
        ]

    def export_uptie_stats_data(self, filename="Uptiestats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        uptie_stats_fieldnames = ["Identity_ID", "Uptie", "Speed"]
        uptie_stats_data = self.get_uptie_stats_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, uptie_stats_fieldnames, uptie_stats_data)  # CSV로 내보내기

    def get_resist_info_data(self):
        """
        모든 Identity의 resist_info 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Resist_Hit": identity.resist_info.resist_hit,
                "Resist_Penetrate": identity.resist_info.resist_penetrate,
                "Resist_Slash": identity.resist_info.resist_slash,
            }
            for identity in self.identities
        ]

    def export_resist_info_data(self, filename="ResistInfos.csv"):
        """ResistInfo 데이터를 CSV로 내보내는 메서드"""
        resist_infos_fieldnames = ["Identity_ID", "Resist_Hit", "Resist_Penetrate", "Resist_Slash"]
        resist_infos_data = self.get_resist_info_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, resist_infos_fieldnames, resist_infos_data)  # CSV로 내보내기

    def export_uptie_stats_data(self, filename="Uptiestats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        uptie_stats_fieldnames = ["Identity_ID", "Uptie", "Speed"]
        uptie_stats_data = self.get_uptie_stats_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, uptie_stats_fieldnames, uptie_stats_data)  # CSV로 내보내기

    def get_stagger_threshold_data(self):
        """
        모든 Identity의 stagger_threshold 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "First_Stagger": identity.stagger_threshold.first_stagger,
                "Second_Stagger": identity.stagger_threshold.second_stagger,
                "Third_Stagger": identity.stagger_threshold.third_stagger,
            }
            for identity in self.identities
        ]

    def export_stagger_threshold_data(self, filename="StaggerThresholds.csv"):
        """StaggerThresholds 데이터를 CSV로 내보내는 메서드"""
        stagger_thresholds_fieldnames = ["Identity_ID", "First_Stagger", "Second_Stagger", "Third_Stagger"]
        stagger_thresholds_data = self.get_stagger_threshold_data()  # stagger_thresholds 데이터 가져오기
        self.export_to_csv(filename, stagger_thresholds_fieldnames, stagger_thresholds_data)  # CSV로 내보내기

    def get_skillset_data(self):
        """모든 Identity의 skillset 데이터를 export_to_csv을 포맷으로 변환"""
        return [
            {
                "Identity_ID": identity.identity_id,
                "Skill_Code": skill.skill_code,
                "Skill_Type": skill.skill_type,
                "Sin_Affinity": skill.sin_affinity,
                "Skill_Name": skill.skill_name,
                "Coin_Count": skill.coin_count,
                "Init_Level": skill.init_level,
                "Init_Remain": skill.init_remain,
                "Atk_Weight": skill.atk_weight,
                "Uptie_Level": uptie_info.uptie_level,
                "Skill_Power": uptie_info.skill_power,
                "Coin_Power": uptie_info.coin_power,
                "Effect_Data": json.dumps(uptie_info.skill_effect)
            }
            for identity in self.identities
            for skill_list in [identity.skillset.skill_1_details, identity.skillset.skill_2_details, identity.skillset.skill_3_details, identity.skillset.defense_skill_details]
            for skill in skill_list
            for uptie_info in skill.uptie_info
        ]

    def export_identity_skillset_data(self, filename="IdentitiesSkills.csv"):
        """IdentitiesSkills 데이터를 CSV로 내보내는 메서드"""
        identities_skills_fieldnames = ["Identity_ID", "Skill_Code", "Skill_Type", "Sin_Affinity", "Skill_Name", "Coin_Count", "Init_Level", "Init_Remain", "Atk_Weight", "Uptie_Level", "Skill_Power", "Coin_Power", "Effect_Data"]
        identities_skills_data = self.get_skillset_data()
        self.export_to_csv(filename, identities_skills_fieldnames, identities_skills_data)

    def get_identities_behaviors_data(self):
        """모든 Identity의 influence behavior 데이터를 export_to_csv 포맷으로 변환"""
        return [
            {
                "Identity_ID": identity.identity_id,
                "Effect_Type": effect_type,
                "Behavior_Name": effect.name,
                "Behavior_Condition": effect.condition if effect.condition else np.nan,
                "Behavior_Description" : effect.description
            }
            for identity in self.identities
            for effect_list, effect_type in [(identity.passive_infos, 'Passive'), (identity.support_passive_infos, 'Support Passive'), (identity.panic_type, 'Panic Type')]
            for effect in effect_list
        ]

    def export_identities_behaviors_data(self, filename="IdentitiesBehaviors.csv"):
        """IdentitiesBehaviors 데이터를 CSV로 내보내는 메서드"""
        identities_behaviors_fieldnames = ["Identity_ID","Effect_Type","Behavior_Name","Behavior_Condition","Behavior_Description"]
        identities_behaviors_data = self.get_identities_behaviors_data()
        self.export_to_csv(filename, identities_behaviors_fieldnames, identities_behaviors_data)
            
async def main():
    data_container = DataContainer()
    scraper = Scraper(container=data_container, base_url="https://limbus.kusoge.xyz", img_url="https://img.kusoge.xyz/limbus/img")
    await scraper.scrape()

await main() 

Data successfully exported to IdentitiesBehaviors.csv


In [5]:
import os
import csv
import re
import json
import numpy as np
import pandas as pd
from playwright.async_api import async_playwright
from tabulate import tabulate

# Sin Color 이름 변환을 위한 매핑
resource_mapping = {
    "AZURE": "Gloom",
    "VIOLET": "Envy",
    "AMBER": "Sloth",
    "SHAMROCK": "Gluttony",
    "CRIMSON": "Wrath",
    "SCARLET": "Lust",
    "INDIGO": "Pride"
}

def convert_resource(resource_name):
    """Sin Color 이름을 변환하는 함수. 매핑되지 않은 경우 원래 이름을 반환."""
    return resource_mapping.get(resource_name, resource_name)

def find_sin_for_color(url):
    # URL에 color_name을 추출
    color_name = url.split('/')[-1].split('.')[0].replace("attr_", "")
    # 해당 색상이 resource_mapping에 있는 확인
    for color, sin in resource_mapping.items():
        if color_name in color:
            return sin
    return None


class Scraper:
    """웹 페이지 크롤링을 위한 클래스"""
    def __init__(self, container, base_url, img_url):
        self.container = container
        self.base_url = base_url
        self.img_url = img_url
        self.uptie_level = 1
        self.keywords_set = set()

    async def handle_request(self, route, request):
        """이미지와 폰트 리소스 로드를 차단."""
        if request.resource_type =="image":
            #print(f"{request.url}")
            img_url = request.url
            if  re.match(f"{self.img_url}/ui/attr.*\\.png", img_url):
                await route.continue_()
            else:
                await route.fulfill(
                    status=200,
                    content_type="image/png",
                    body=b""
                )
        elif request.resource_type == "font":
            await route.abort()
        else:
            await route.continue_()
            
    async def scrape(self):
        """크롤링 메인 함수"""
        async with async_playwright() as p:
            browser = await p.chromium.launch(headless=True)
            page = await browser.new_page()
            #await page.evaluate('window.scrollTo(0, document.body.scrollHeight)')
            await page.route("**/*", self.handle_request)
            # Step 1: /identity 페이지에서 기본 정보 크롤링
            #await self.crawl_identity_page(page)

            # Step 2: /identity/{identity_id} 상세 페이지 크롤링
            #identity_ids_to_load = ["10104", "10310", "10710","10808", "10911", "11005"]
            #identity_ids_to_load = ["10104", "10911"]#,"10310", "10710","10808", "11005", "11210"]
            #identity_ids_to_load = ["10911","10310"]#, "10710","10808", "11005", "11210"]
            identity_ids_to_load = ["10911", "10611", "10711"]#, "10710","10808", "11005", "11210"]
            self.container.load_identities_by_id('./csv/identities.csv', identity_ids_to_load)
            await self.crawl_identity_details_for_all(page)
            #self.container.display_identities()
            #self.container.export_identities_data("identities.csv")
            #self.container.export_level_stats_data("Levelstats.csv")
            #self.container.export_uptie_stats_data("Uptiestats.csv")
            #self.container.export_resist_info_data("ResistInfos.csv")
            #self.container.export_stagger_threshold_data("StaggerThresholds.csv")
            #self.container.export_identity_skillset_data("IdentitiesSkills.csv")
            #self.container.export_identities_behaviors_data("IdentitiesBehaviors.csv")
            print(self.keywords_set)
            page.set_default_timeout(10000)
            #await page.wait_for_timeout(100000)
            await browser.close()

    async def get_identity_id_from_row(self, row):
        """테이블 행(row)에서 인격 ID를 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_url = await identity_cell.get_attribute('href')
            identity_id = identity_url.split('/identity/')[-1]
            return identity_id
        return "Unknown"

    async def get_identity_name_from_row(self, row):
        """테이블 행(row)에서 인격 이름을 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_name = (await identity_cell.inner_text()).replace("\n", " ").strip()
            return identity_name
        return "Unknown"
        
    async def crawl_identity_page(self, page):
        """/identity 페이지로 이동하여 인격 기본 정보를 추출."""
        await page.goto(self.base_url + "/identity")
        await self.extract_identity_page(page)

    async def extract_identity_page(self, page):
        """/identity 페이지의 모든 테이블 행에서 인격 정보를 추출."""
        identity_rows = await page.query_selector_all('tr')
        if len(identity_rows) > 1:
            for idx, identity_row in enumerate(identity_rows):
                if idx == 0:
                    continue # 0번째요소이름을 보여주는 열은 건너뜀
                await self.scrape_identity_name_and_identity_id(identity_row)
                await self.extract_rarity(identity_row)
                for i in range(1, 4):  # 스킬1~3 정보 추출
                    await self.extract_atk_and_resource(identity_row, i)
                await self.extract_defense_and_resource(identity_row)

    async def scrape_identity_name_and_identity_id(self, row):
        """인격 번호와 이름을 추출하여 DataContainer에 저장."""
        try:
            identity_id = await self.get_identity_id_from_row(row)
            identity_name = await self.get_identity_name_from_row(row)
            identity = Identity(identity_id)
            self.container.add_identity(identity)
            self.container.update_identity_name(identity_id, identity_name)
        except Exception as e:
            print(f"Error extracting identity name and identity id: {e}")

    async def get_element_attribute(self, element, selector, attribute):
        """요소에서 지정된 속성(attribute)을 추출."""
        try:
            target_element = await element.query_selector(selector)
            if target_element:
                return await target_element.get_attribute(attribute)
        except Exception as e:
            print(f"Error getting element attribute: {e}")
        return None
    
    async def extract_rarity(self, row):
        """테이블 행(row)에서 희귀도를 추출하여 업데이트."""
        try:
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
                sibling = await parent.evaluate_handle('node => node.nextElementSibling')
                if sibling:
                    rarity_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/rarity_"]', 'src')
                    if rarity_img_src:
                        rarity_level = rarity_img_src.split('_')[-1].split('.')[0]  # rarity_1.png -> 1
                        identity_id = await self.get_identity_id_from_row(row)
                        self.container.update_identity_rarity(identity_id, rarity_level)
                    else:
                        print("Rarity image not found in the next sibling TD.")
        except Exception as e: 
            print(f"Error extracting rarity: {e}")

        return "Unknown"

    async def extract_atk_and_resource(self, row, skill_number):
        """ 공격 타입(ATK Type)과 자원(Attribute) 추출 함수 - 스킬 번호에 따라 처리 """
        try:
            #print(f"Attempting to locate skill{skill_number} type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 해당 스킬 번호에 맞는 형제로 이동
                sibling = parent
                for _ in range(skill_number + 1):  # 스킬 번호에 맞게 이동
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                atk_type, resource = "Unknown", "Unknown"
        
                if sibling:
                    # 공격 타입 이미지 (atk_type) 찾기
                    atk_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/atk_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if atk_type_img_src:
                        atk_type = atk_type_img_src.split('_')[-1].split('.')[0]  # atk_type_SLASH.png -> SLASH
                        #print(f"Extracted attack type: {atk_type}")
                        self.container.update_skill(identity_id, skill_number, atk_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted resource: {resource}")
                        
                        # 리소스를 변환
                        resource = convert_resource(resource)
                        #print(f"Converted resource: {resource}")
                        self.container.update_sin_affinity(identity_id, skill_number, resource)
                        
        except Exception as e:
            print(f"Error extracting skill{skill_number} type and resource: {e}")

    async def extract_defense_and_resource(self, row):
        """ 방어 타입(Def Type)과 자원(Attribute) 추출 함수 """
        try:
            #print("Attempting to locate defense type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 다섯 번째 형제 td로 이동 (방어 관련 정보가 있는 <td>)
                sibling = parent
                for _ in range(5):  # 방어 정보는 5번째 형제에 있음
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                def_type, def_resource = "Unknown", "Unknown"
        
                if sibling:
                    # 방어 타입 이미지 (def_type) 찾기
                    def_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/def_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if def_type_img_src:
                        def_type = def_type_img_src.split('_')[-1].split('.')[0]  # def_type_GUARD.png -> GUARD
                        #print(f"Extracted defense type: {def_type}")
                        self.container.update_skill(identity_id, 'defense', def_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        def_resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted defense resource: {def_resource}")
                        
                        # 방어 리소스를 변환
                        def_resource = convert_resource(def_resource)
                        #print(f"Converted defense resource: {def_resource}")
                        self.container.update_sin_affinity(identity_id, 'defense', def_resource)
 
        except Exception as e:
            print(f"Error extracting defense type and resource: {e}")

    async def crawl_identity_details_for_all(self, page):
        """모든 인격의 세부 정보를 크롤링."""
        for identity in self.container.identities:
            await page.wait_for_load_state("networkidle")
            await page.goto(self.base_url + "/identity/"+identity.identity_id)
            """ 레벨과 동기화와 관련 없는 참관타 저항은 밑에서 변경하는 것과 연관성이 적기 때문에 위에서 처리한다.
            print(f"Identity_id: {identity.identity_id}")
            results = []
            resist_type_imgs = [f"{self.img_url}/ui/atk_type_HIT.png",
                                f"{self.img_url}/ui/atk_type_PENETRATE.png",
                                f"{self.img_url}/ui/atk_type_SLASH.png"]
            for resist_type_img in resist_type_imgs:
                result = await self.extract_resist_type(page, resist_type_img, 1)
                results.append(result)
            self.container.add_resist_info(identity.identity_id, results[0], results[1], results[2])
            """
            """
            # Stagger Threshold 값 추출
            stagger_threshold = await self.extract_stagger_threshold_values(page)
            self.container.add_stagger_threshold(identity.identity_id, stagger_threshold[0], stagger_threshold[1], stagger_threshold[2])
            """
            await self.initialize_uptie_and_level(page)
            """ 레벨에 따른 level, hp, defense 다르게 되어 있는 것을 출력 확인 객체로 들어가서 csv로 옮기는 과정까지 진행
            for i in range(1, identity.level_max+1):
                await self.move_slider_to_value(page,i)
                await self.extract_level_status(page, identity.identity_id, i)
            """
            """ 동기화에 따른 speed 다르게 되어 있는 것을 출력 확인 객체로 들어가서 csv로 옮기는 과정까지 진행
            for i in range(1, identity.uptie_max+1):
                await self.uptie_click(page,i)
                await self.extract_uptie_status(page, identity.identity_id, i)
            """
            """
                인격에 대한 skill 정보를 가져오는 메소드 위에서 출력하는 csv파일로 인격이 가지고 있는 스킬 정보를 내보낸다.
            await self.crawl_skills(page, identity.identity_id)
            """
            await self.crawl_influence_behavior(page, identity.identity_id)
            
    async def initialize_uptie_and_level(self, page):
        await self.uptie_click(page, 1)
        await self.move_slider_to_value(page, 1)
        self.uptie_level = 1
        #return 1
    
    async def uptie_click(self, page, phase):
        fab_button_selector = ".q-btn.q-btn--fab"
        await page.wait_for_selector(fab_button_selector)
        await page.click(fab_button_selector)
        #print(f"[{setup_uptie_button}] 상위 동기화 버튼 클릭 완료")
        setup_uptie_button = f'a:has(img[src="{self.img_url}/ui/uptie_{phase}.png"])'
        await page.wait_for_selector(setup_uptie_button)
        await page.click(setup_uptie_button)
        #print(f"{phase} 동기화 버튼 클릭 완료")

    async def move_slider_to_value(self, page, target_value):
        """
        슬라이더를 주어진 값으로 이동시킵니다.
        target_value는 슬라이더의 최소값부터 최대값 사이여야 합니다.
        """
        slider_selector = ".q-slider"
        # 슬라이더의 범위 확인
        min_value = int(await page.get_attribute(slider_selector, "aria-valuemin"))
        max_value = int(await page.get_attribute(slider_selector, "aria-valuemax"))
    
        if target_value < min_value or target_value > max_value:
            raise ValueError(f"Target value {target_value} is out of bounds ({min_value}, {max_value})")
    
        # 슬라이더 thumb 요소와 위치 가져오기
        slider_thumb = await page.query_selector(f"{slider_selector} .q-slider__thumb")
        thumb_box = await slider_thumb.bounding_box()
        
        # 슬라이더의 현재 위치 및 목표 위치 계산
        track = await page.query_selector(f"{slider_selector} .q-slider__track")
        track_box = await track.bounding_box()
        
        # 목표 x 위치 계산 (슬라이더 트랙 내에서 비율 기반)
        target_percentage = (target_value - min_value) / (max_value - min_value)
        target_x = track_box["x"] + target_percentage * track_box["width"]
        current_x = thumb_box["x"] + thumb_box["width"] / 2
        y = thumb_box["y"] + thumb_box["height"] / 2
    
        # 슬라이더를 드래그하여 목표 위치로 이동
        await page.mouse.move(current_x, y)
        await page.mouse.down()
        await page.mouse.move(target_x, y, steps=10)  # 부드럽게 이동
        await page.mouse.up()
    
        #print(f"슬라이더를 {target_value}로 이동 완료.")
    
        # 슬라이더 값 확인 및 강제 이벤트 트리거
        await page.evaluate(
            f"""
            (slider) => {{
                slider.dispatchEvent(new Event('input'));  // oninput 이벤트 트리거
                slider.dispatchEvent(new Event('change'));  // onchange 이벤트 트리거
            }}
            """,
            await page.query_selector(slider_selector)
        )
    
        # 이동 후 값 확인
        current_value = await page.get_attribute(slider_selector, "aria-valuenow")
        if current_value != str(target_value):
            print(f"경고: 슬라이더 값이 예상과 다릅니다. 현재 값: {current_value}, 예상 값: {target_value}")

    async def get_second_status_element(self, page):
        """
        두 번째 'Status' 요소에 접근하여 반환하는 메소드
        """
        status_elements = await page.query_selector_all("text=Status")
        if len(status_elements) > 1:
            second_status_element = status_elements[1]
            #print("두 번째 'Status' 요소에 접근합니다.")
            return second_status_element
        else:
            #print("두 번째 'Status' 요소가 존재하지 않습니다.")
            return None

    async def extract_level_status(self, page, identity_id, level):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 레벨에 따라 HP와 Defense가 변한다.
        hp_value = await self.extract_hp(second_status_element)
        defense_value = await self.extract_defense(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Level={level}, HP={hp_value}, Defense={defense_value}")
        self.container.add_level_stat(identity_id,level, hp_value, defense_value)
    
    async def extract_hp(self, status_element):
        # HP 추출
        hp_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].children[0].nextElementSibling.textContent.trim()"
        )
        return hp_value
    
    async def extract_defense(self, status_element):
        # Defense 추출
        defense_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return defense_value

    async def extract_uptie_status(self, page, identity_id, uptie):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 uptie에 따라 speed는 변한다.
        speed_value = await self.extract_speed(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Uptie={uptie}, Speed={speed_value}")
        self.container.add_uptie_stat(identity_id,uptie, speed_value)

    async def extract_speed(self, status_element):
        # Speed 추출
        speed_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return speed_value

    async def extract_resist_type(self, page, image_src, depth):
        # 이미지 요소 찾기
        #print(image_src)
        image = await page.query_selector(f'img[src="{image_src}"]')
        #print(image)
        if not image:
            return "Unknown"
    
        # 이미지의 부모 요소를 찾아서 올라감
        parent = image
        for _ in range(depth):
            parent = await parent.evaluate_handle('node => node.parentElement')
            if not parent:
                return "Unknown"
    
        # 부모의 형제 요소 찾기
        sibling = await parent.evaluate_handle('node => node.nextElementSibling')
        
        if sibling:
            value = await sibling.evaluate('node => node.textContent')
            if value:
                return value.strip()
        
        return "Unknown"

    async def extract_stagger_threshold_values(self, page):
        # 'div.row.center.flex-center.q-pt-xs' 요소들 찾기
        elements = await page.query_selector_all('div.row.center.flex-center.q-pt-xs')
        
        # 각 요소의 텍스트 내용 출력하여 'Stagger Threshold' 요소를 찾기
        for element in elements:
            text = await element.text_content()
            
            if text and "Stagger Threshold" in text:
                # 'Stagger Threshold'가 있는 요소를 찾았다면, 그 이후 형제 요소들에서 값을 추출
                parent = await element.evaluate_handle('node => node.parentElement')
                sibling_elements = await parent.query_selector_all('div.row.justify-evenly div')  # Threshold 값이 들어있는 div들
                
                stagger_values = []
                for sibling in sibling_elements:
                    text_content = await sibling.text_content()
                    match = re.search(r'(\d+%)', text_content)
                    if match:
                        stagger_values.append(match.group(1))  # 퍼센트 값만 리스트에 추가
                while len(stagger_values) < 3:
                    stagger_values.append(np.nan)
                #print(stagger_values)
                return stagger_values  # 추출된 값(또는 np.nan 포함)을 반환
        
        return [np.nan, np.nan, np.nan]  # 만약 'Stagger Threshold'를 찾지 못한 경우 기본값 반환

    async def crawl_skills(self, page, identity_id):
        await self.crawl_skill(page, identity_id, "skill 1")
        await self.crawl_skill(page, identity_id, "skill 2")
        await self.crawl_skill(page, identity_id, "skill 3")
        await self.crawl_skill(page, identity_id, "defense")


    async def crawl_skill(self, page, identity_id, skill_type):
        skills_elements = await page.query_selector_all('div.skilltag')
        for skill_element in skills_elements:
            skill_text = await skill_element.text_content()
            if skill_text.strip().lower() == skill_type:
                await self.process_skill(skill_element, page, identity_id, skill_type)
        
    async def process_skill(self, skill_element, page, identity_id, skill_type):
        # skill_type이 "skill 3"일 경우 uptie_value가 3 이상일 때만 정보를 저장
        for uptie_value in range(1, 5):  # Uptie levels 1 to 4
            # "skill 3"은 uptie_value가 3 이상일 때만 처리
            if skill_type == "skill 3" and uptie_value < 3:
                continue  # skip if uptie_value is less than 3 for skill 3
    
            # uptie_level에 맞게 클릭
            await self.uptie_click(page, uptie_value)
            if skill_type != "skill 3" and uptie_value > 1:
                # Uptie 정보 갱신
                await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
                continue
            elif skill_type == "skill 3" and uptie_value > 3:
                # Uptie 정보 갱신
                await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
                continue
            # 스킬의 세부 정보를 추출
            skill_details = await self.extract_skill_details(skill_element, skill_type)
            skill_name = skill_details["skill_name"]
    
            # 스킬 정보를 저장
            self.container.add_skill(
                identity_id, skill_type, 
                skill_name, skill_details["coin_count"],
                skill_details["init_level"], skill_details["init_remain"], 
                skill_details["atk_weight"]
            )
            print(f"Added skill: {skill_type}:{skill_name}")
    
            # Uptie 정보 갱신
            await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
    
    async def extract_skill_details(self, skill_element, skill_type):
        # 스킬 세부 정보를 추출하는 함수
        skill_name = await self.skill_name(skill_element)
        coin_count = await self.coin_count(skill_element, ".col-auto > .column > .row")
        init_level = await self.init_level(skill_element)
        init_remain = await self.init_remain(skill_element, skill_type, "div.row.justify-start > div.attr.flex.flex-center > div.q-px-sm")
        atk_weight = await self.atk_weight(skill_element, skill_type)
        return {
            "skill_name": skill_name,
            "coin_count": coin_count,
            "init_level": init_level,
            "init_remain": init_remain,
            "atk_weight": atk_weight
        }
    
    async def update_uptie_info(self, page, skill_element, identity_id, skill_name, skill_type, uptie_value):
        # Uptie 정보를 추출하고 업데이트하는 함수
        uptie_info = await self.extract_uptie_info(skill_element)
        effect_data = await self.skill_effects(skill_element)
        self.container.add_skill_uptie_info(
            identity_id, skill_name, 
            uptie_value, uptie_info["skill_power"], uptie_info["coin_power"], 
            effect_data.get("effectsByCondition")
        )
        keywords = effect_data.get("keywords")
        if keywords:
            self.keywords_set.update(keywords)
        print(
            f"Updated Uptie for {skill_name} (Uptie Level: {uptie_value}): "
            f"Skill Power: {uptie_info['skill_power']}, Coin Power: {uptie_info['coin_power']} "
            f"Skill Effect: {effect_data.get("effectsByCondition")}"
        )

    async def extract_uptie_info(self, skill_element):
        skill_data = await self.skill_and_coin_power(skill_element)
        return {
            "skill_power": skill_data.get("skillPower"),
            "coin_power": skill_data.get("coinPower")
        }
        
    async def crawl_influence_behavior(self, page, identity_id):
        influence_behaviors = await page.query_selector_all('div[align="right"]')
        # 텍스트를 기준으 추출하고 메소드로 파라미터 들어가 정보를 받는다.
        for influence_behavior in influence_behaviors:
            active_type = await influence_behavior.inner_text()
            active_type = active_type.strip()

            if active_type == "PASSIVE":
                """
                # PASSIVE에 대한 처리
                print(f"Processing PASSIVE logic for: {active_type}")
                # CALL method for PASSIVE
                """
                await self.passive_effects(influence_behavior, identity_id)
            elif active_type == "SUPPORT PASSIVE":
                """
                # SUPPORT PASSIVE에 대한 처리
                print(f"Processing SUPPORT PASSIVE logic for: {active_type}")
                # CALL method for SUPPORT PASSIVE
                """
                await self.support_passive_effects(influence_behavior, identity_id)
            elif active_type == "PANIC TYPE":
                """
                # PANIC TYPE에 대한 처리
                print(f"Processing PANIC TYPE logic for:{active_type}")
                # CALL method for PANIC TYPE
                """
                await self.panic_effects(influence_behavior, identity_id)

    async def skill_and_coin_power(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                // 조부모로 이동
                const grandparent = node.parentElement.parentElement;
                // 이전 형제 탐색
                const previousSibling = grandparent.previousElementSibling;
                if (!previousSibling) return {{ skillPower: null, coinPower: null }}; // 이전 형제가 없으면 기본값 반환
                // 이전 형제 내부의 .row.justify-around > div > div 탐색
                const targetDivs = previousSibling.querySelectorAll(".row.justify-around > div");
                if (targetDivs.length < 3) return {{ skillPower: null, coinPower: null }}; // 배열 길이 확인
                // 1st div: 스킬 위력
                const skillPower = targetDivs[0]?.querySelector('div')?.textContent?.trim() || null; 
                // 3rd div: 코인 파워 값 (3번째 div에서 텍스트 추출)
                const coinPower = targetDivs[2]?.textContent?.trim() || null;
                return {{ skillPower, coinPower }};
            }}
        """)


    async def coin_count(self, skill_element, hierarchy_selector):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector('{hierarchy_selector}');
                if (!targetElement) return 0;
                return targetElement.childElementCount;
            }}
        """)

    async def init_remain(self, skill_element, skill_type, hierarchy_selector):
        if skill_type == "defense":
            return -1
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector('{hierarchy_selector}');
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def skill_name(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector(".block");
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def init_level(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector("div.text-bold.text-h6.h6.q-pl-xs.q-pr-md");
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def atk_weight(self, skill_element, skill_type):
        if skill_type == "defense":
            return 0
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector("div.text-bold.text-h6.h6.q-pl-xs.q-pr-md");
                const doubleNextSibling = targetElement.nextElementSibling.nextElementSibling;
                if (!doubleNextSibling) return 0;
                return doubleNextSibling.textContent?.trim();
            }}
        """)

    async def skill_effects(self, skill_element):
        return await skill_element.evaluate("""
            node => {
                const parent = node.parentElement;
                const doubleNextSibling = parent.nextElementSibling?.nextElementSibling;
                if (!doubleNextSibling) return null;
    
                // doubleNextSibling에서 첫 번째 div.col 요소 찾기
                const textNode = doubleNextSibling.querySelector("div.col");
                if (!textNode) return null;
    
                const results = {
                    keywords: [], // 키워드 저장
                    conditions: [], // 조건 저장
                    effectsByCondition: {} // 조건별 효과
                };
    
                // 1. 조건 추출 (대괄호 텍스트)
                const conditionElements = textNode.querySelectorAll("span");
                conditionElements.forEach(span => {
                    let text = span.textContent?.trim();
                    if (text?.startsWith("[") && text?.endsWith("]")) {
                        // 부모의 전형제 노드 탐색
                        const sibling = span.parentElement.previousElementSibling;
                        if (sibling) {
                            const emblemImage = sibling.querySelector("img.emblem");
                            console.log("Sibling element found:", sibling); // 디버깅용
                            if (emblemImage) {
                                const src = emblemImage.getAttribute("src");
                                console.log("Image source:", src); // 디버깅용
                                const match = src.match(/ui_num_(\\d+)\\.png$/); // 정규식 수정
                                if (match) {
                                    const number = match[1]; // 숫자 추출
                                    text = `[Coin ${number} ${text.replace("[", "").replace("]", "")}]`;
                                    console.log("Updated condition:", text); // 디버깅용
                                }
                            }
                        }
                        results.conditions.push(text);
                        results.effectsByCondition[text] = []; // 조건별 효과를 초기화
                    }
                });
    
                // 2. 키워드 추출
                const keywordElements = textNode.querySelectorAll('a[href^="/en/keyword/"]');
                keywordElements.forEach(a => {
                    const span = a.querySelector('span.q-btn__content.text-center.col.items-center.q-anchor--skip.justify-center.row>span');
                    if (span) {
                        const keywordText = span.textContent.trim();
                        if (keywordText && !results.keywords.includes(keywordText)) {
                            results.keywords.push(keywordText); // 키워드 저장
                        }
                    }
                });
    
                // 3. 효과 추출
                const effectElements = textNode.querySelectorAll("span");
                let currentCondition = null;
                let currentEffect = []; // 초기화
                effectElements.forEach(span => {
                    let text = span.textContent?.trim();
                    if (!text) return;
    
                    // 조건 발견 시 currentCondition 변경
                    if (text.startsWith("[") && text.endsWith("]")) {
                        // 이전 조건의 효과 저장
                        if (currentCondition && currentEffect.length > 0) {
                            // 부모의 전형제 노드 탐색
                            const sibling = span.parentElement.previousElementSibling;
                            if (sibling) {
                                const emblemImage = sibling.querySelector("img.emblem");
                                if (emblemImage) {
                                    const src = emblemImage.getAttribute("src");
                                    const match = src.match(/ui_num_(\\d+)\\.png$/); // 정규식 수정
                                    if (match) {
                                        const number = match[1]; // 숫자 추출
                                        text = `[Coin ${number} ${text.replace("[", "").replace("]", "")}]`;
                                    }
                                }
                            }
                            results.effectsByCondition[currentCondition].push(currentEffect.join(' '));
                            currentEffect = []; // 초기화
                        }
                        currentCondition = text;
                        return;
                    }
    
                    // 현재 조건의 효과 처리
                    if (currentCondition) {
                        if (results.keywords.includes(text)) {
                            const keywordEffect = 'keyword:' + text;
                            if (!currentEffect.includes(keywordEffect)) {
                                currentEffect.push(keywordEffect);
                            }
                        } else {
                            currentEffect.push(text);
                        }
                    }
                });
    
                // 마지막 효과 저장
                if (currentCondition && currentEffect.length > 0) {
                    let processedText = currentEffect
                        .map(word => word.trim())
                        .join(" ")
                        .replace(/\\s+([,.:])/g, "$1")
                        .replace(/\\s+'/g, "'")
                        .replace(/'\\s+/g, "'")
                        .replace(/\\[\\s+/g, "[")
                        .replace(/\\s+\\]/g, "]")
                        .replace(/\\(\\s+/g, "(")
                        .replace(/\\s+\\)/g, ")")
                        .replace(/\\s+%/g, "%")
                        .replace(/\\s+\\//g, "/")
                        .replace(/\\/\\s+/g, "/")
                        .replace(/\\s+\\/\\s+/g, "/");
                    results.effectsByCondition[currentCondition].push(processedText);
                }
    
                return results;
            }
        """)

    async def passive_effects(self, passive_element, identity_id):
        passive_name = await self.behavior_name(passive_element)
        passive_conditions = await self.behavior_condition(passive_element)
        passive_description = await self.behavior_description(passive_element, "span")
        #print(passive_name)
        #print(passive_conditions)
        print(passive_description)
        self.container.add_influence_behavior(identity_id, "passive", passive_name, passive_conditions, passive_description.get("description"))
        keywords = passive_description.get("keywords")
        if keywords:
            self.keywords_set.update(keywords)
    async def support_passive_effects(self, support_passive_element, identity_id):
        support_passive_name = await self.behavior_name(support_passive_element)
        support_passive_conditions = await self.behavior_condition(support_passive_element)
        support_passive_description = await self.behavior_description(support_passive_element, "span")
        #print(support_passive_name)
        #print(support_passive_conditions)
        print(support_passive_description)
        self.container.add_influence_behavior(identity_id, "support passive", support_passive_name, support_passive_conditions, support_passive_description.get("description"))
        keywords = support_passive_description.get("keywords")
        if keywords:
            self.keywords_set.update(keywords)
    async def panic_effects(self, panic_element, identity_id):
        panic_name = await self.behavior_name(panic_element)
        panic_conditions = await self.behavior_condition(panic_element)
        panic_description = await self.behavior_description(panic_element, "div")
        #print(panic_type)
        #print(panic_conditions)
        #print(panic_description)
        self.container.add_influence_behavior(identity_id, "panic type", panic_name, panic_conditions, panic_description.get("description"))
        keywords = panic_description.get("keywords")
        if keywords:
            self.keywords_set.update(keywords)

    async def navigate_element_by_steps(self, element, parent_steps=0, prev_sibling_steps=0, next_sibling_steps=0):
        """
        주어진 `element`에서 부모, 이전 형제, 또는 다음 형제로 지정된 횟수만큼 이동하여 ElementHandle 반환.
        
        Args:
            element (ElementHandle): 탐색 기준이 되는 요소.
            parent_steps (int): 부모로 이동할 횟수 (0이면 이동하지 않음).
            prev_sibling_steps (int): 이전 형제로 이동할 횟수 (0이면 이동하지 않음).
            next_sibling_steps (int): 다음 형제로 이동할 횟수 (0이면 이동하지 않음).
        
        Returns:
            ElementHandle or None: 탐색된 요소를 반환하거나 없으면 None.
        """
        try:
            current_element = element
    
            # 부모로 이동
            for _ in range(parent_steps):
                current_element = await current_element.evaluate_handle('el => el.parentElement')
                if not current_element:
                    print("Parent element not found during navigation.")
                    return None
    
            # 이전 형제로 이동
            for _ in range(prev_sibling_steps):
                current_element = await current_element.evaluate_handle('el => el.previousElementSibling')
                if not current_element:
                    print("Previous sibling element not found during navigation.")
                    return None
    
            # 다음 형제로 이동
            for _ in range(next_sibling_steps):
                current_element = await current_element.evaluate_handle('el => el.nextElementSibling')
                if not current_element:
                    print("Next sibling element not found during navigation.")
                    return None
    
            return current_element
    
        except Exception as e:
            print(f"Error navigating elements: {e}")
            return None

    async def get_last_sibling(self, element):
        try:
            # 같은 부 아 마지막 형제를 가져오는 JavaScript 실행
            last_sibling = await element.evaluate_handle('el => el.parentElement.lastElementChild')
            return last_sibling
        except Exception as e:
            print(f"Error getting last sibling: {e}")
            return None

    async def behavior_name(self, element):
        parent_prev_sibling = await self.navigate_element_by_steps(element, parent_steps=1, prev_sibling_steps=1, next_sibling_steps=0)
        passive_name_element = await parent_prev_sibling.query_selector('span.block')
        if passive_name_element:
            passive_name = await passive_name_element.inner_text()
            return passive_name
        else:
            #print("Passive name element not found")
            return None

    async def behavior_condition(self, element):
        try:
            conditions = []
            parent_prev_sibling = await self.navigate_element_by_steps(element, parent_steps=1, prev_sibling_steps=1, next_sibling_steps=0)
            img_elements = await parent_prev_sibling.query_selector_all('img')
            if not img_elements:
                #print("No <img> elements found.")
                return conditions
    
            for img_element in img_elements:
                # 각 이미지의 src 속성 가져오기
                img_src = await img_element.get_attribute('src')
                #print(img_src)
    
                sin_name = None
                for color_key in resource_mapping.keys():
                    if f"{self.img_url}/ui/attr_{color_key}.png" in img_src:
                        sin_name = resource_mapping[color_key]
                        break
    
                if not sin_name:
                    #print(f"No matching Sin Color found for {img_src}.")
                    continue  # No Sin Color found, skip this img_element
    
                # 해당 이미지와 관련된 조건 텍스트 추출
                # 두 번째 부모 요소에서 시작하여 다음 형제를 찾아 조건 텍스트를 추출
                condition_element = await self.navigate_element_by_steps(img_element, parent_steps=3, prev_sibling_steps=0, next_sibling_steps=0)
                if condition_element:
                    # condition_element가 None이 아닌 경우에만 진행
                    condition_text_element = await condition_element.query_selector('.q-pl-xs')
                    if condition_text_element:
                        condition_text = await condition_text_element.inner_text()
                        condition_text = condition_text.strip()
                    else:
                        print(f"No condition text found for {img_src}.")
                        condition_text = ""  # condition_text_element가 None이면 빈 문자열로 처리
                else:
                    print(f"No condition element found for {img_src}.")
                    condition_text = ""  # condition_element가 None이면 빈 문자열로 처리
    
                # 결과를 배열에 추가
                conditions.append(f"{sin_name} {condition_text}")
            return conditions
        except Exception as e:
            print(f"Error extracting conditions: {e}")
            return 
            
    async def behavior_description(self, element, tag_type):
        parent_prev_sibling = await self.navigate_element_by_steps(
            element, parent_steps=1, prev_sibling_steps=1, next_sibling_steps=0
        )
        return await parent_prev_sibling.evaluate(f"""
            (node) => {{
                const behavior_title_node = node.querySelector('.q-pb-sm');
                const behavior_title_parent = behavior_title_node.parentElement;
                const behavior_description = behavior_title_parent.lastElementChild;
                if (!behavior_description) return null;
    
                const results = {{
                    keywords: [], // 키워드 저장
                    description: [] // 설명 저장
                }};
                
                // 1. 키워드 추출
                const keywordElements = behavior_description.querySelectorAll('a[href^="/en/keyword/"]');
                if (!keywordElements) return null;
                keywordElements.forEach(a => {{
                    const span = a.querySelector('span.q-btn__content.text-center.col.items-center.q-anchor--skip.justify-center.row>span');
                    if (span) {{
                        const keywordText = span.textContent.trim();
                        if (keywordText && !results.keywords.includes(keywordText)) {{
                            results.keywords.push(keywordText); // 키워드 저장
                        }}
                    }}
                }});
    
                // 2. 설명 추출
                const descriptionElements = behavior_description.querySelectorAll("{tag_type}");
                let currentEffect = []; // 초기화
                descriptionElements.forEach(span => {{
                    let text = span.textContent?.trim();
                    console.log("origin text:", text);
                    if (!text) return;
                    // 공백과 기호 처리
                    text = text.replace(/\\s*([.,:;!?()\\[\\]])\\s*/g, '$1'); // 기호 앞뒤 공백 제거
                    console.log("after symbol processing:", text);
                    console.log("after spacing character: ", text);
                    if (results.keywords.includes(text)) {{
                        const keywordEffect = 'keyword:' + text;
                        if (!currentEffect.includes(keywordEffect)) {{
                            currentEffect.push(keywordEffect);
                        }}
                    }} else {{
                        currentEffect.push(text);
                    }}
                    console.log("current effect:", currentEffect);
                }});
    
                // 마지막 효과 저장
                if (currentEffect.length > 0) {{
                    const finalDescription = currentEffect.join(' ');
                    console.log("After process:", finalDescription);
                    results.description.push(finalDescription);
                }}
                let processedText = results.description
                    .map(word => word.trim())
                    .join(" ")
                    .replace(/\\s+([,.:])/g, "$1")
                    .replace(/\\s+'/g, "'")
                    .replace(/'\\s+/g, "'")
                    .replace(/\\[\\s+/g, "[")
                    .replace(/\\s+\\]/g, "]")
                    .replace(/\\(\\s+/g, "(")
                    .replace(/\\s+\\)/g, ")")
                    .replace(/\\s+%/g, "%")
                    .replace(/\\s+\\//g, "/")
                    .replace(/\\/\\s+/g, "/")
                    .replace(/\\s+\\/\\s+/g, "/");
                results.description = processedText;
                return results;
            }}
        """)

class Identity:
    """인격 정보를 저장하는 클래스"""
    def __init__(self, identity_id, name=None, rarity=None,\
                 skill_1_type=None, skill_1_sin_affinity=None,\
                 skill_2_type=None, skill_2_sin_affinity=None,\
                 skill_3_type=None, skill_3_sin_affinity=None,\
                 defense_type=None, defense_sin_affinity=None):
        self.identity_id = identity_id
        self.name = name
        self.rarity = rarity
        # 선언만 하는 부분 
        self.skill_1_type = skill_1_type 
        self.skill_1_sin_affinity = skill_1_sin_affinity 
        self.skill_2_type = skill_2_type 
        self.skill_2_sin_affinity = skill_2_sin_affinity 
        self.skill_3_type = skill_3_type 
        self.skill_3_sin_affinity = skill_3_sin_affinity 
        self.defense_type = defense_type 
        self.defense_sin_affinity = defense_sin_affinity
        self.level_max = 50
        self.uptie_max = 4
        self.level_stats = []  # LevelStat 객체 리스트
        self.uptie_stats = [] # UptieStat 객체 리스트
        self.resist_info = None
        self.stagger_threshold = None
        self.skillset = SkillSet()  # Skill정보를 담고 있는 skillset 객체 
        self.passive_infos = []  # 패시브 객체를 담고 있는 배열
        self.support_passive_infos = [] # 서포트 패시 객체를 담고 있는 배열
        self.panic_type = [] # 혼란상태 발생시 나타나는 효과를 담고 있는 객체 배열1

    def update_name(self, name):
        self.name = name

    def update_rarity(self, rarity):
        self.rarity = rarity

    def add_level_stat(self, level, hp, defense):
        self.level_stats.append(LevelStat(level, hp, defense))

    def add_uptie_stat(self, uptie, speed):
        self.uptie_stats.append(UptieStat(uptie, speed))

    def add_resist_info(self, resist_hit, resist_penetrate, resist_slash):
        self.resist_info = ResistInfo(resist_hit, resist_penetrate, resist_slash)

    def add_stagger_threshold(self, first_stagger, second_stagger, third_stagger):
        self.stagger_threshold = StaggerThreshold(first_stagger, second_stagger, third_stagger)

    def add_skill(self, skill_code, skill_name, coin_count, init_level, init_remain, atk_weight):
        self.skillset.add_skill(skill_code, Skill(skill_code, skill_name, coin_count, init_level, init_remain, atk_weight))
        
    def add_skill_uptie_info(self, skill_name, uptie_level, skill_power, coin_power, effect_by_condition):
        self.skillset.add_skill_uptie_info(skill_name, uptie_level, skill_power, coin_power, effect_by_condition)

    def add_influence_behavior(self, behavior_type, behavior_name, condition, description):
        if behavior_type == "passive":
            self.passive_infos.append(Passive(behavior_type, behavior_name, condition, description))
        elif behavior_type == "support passive":
            self.support_passive_infos.append(SupportPassive(behavior_type, behavior_name, condition, description))
        elif behavior_type == "panic type":
            self.panic_type.append(PanicType(behavior_type, behavior_name, condition, description))
        
    def update_details(self, page):
        # 세부 정보 업데이트 로직 (가정)
        pass
    #def init_uptie(self, page):
    #    init_uptie = 

# LevelStat 객체 정의
class LevelStat:
    def __init__(self, level, hp=None, defense=None):
        self.level = level
        self.hp = hp
        self.defense = defense
    def update_level(self, level):
        self.level = level
    def update_hp(self, hp):
        self.hp = hp
    def update_defense(self, defense):
        self.defense = defense

# UptieStat 객체 정의
class UptieStat:
    def __init__(self, uptie, speed=None):
        self.uptie = uptie
        self.speed = speed
    def update_speed(self, speed):
        self.speed = speed

# ResistInfo 객체 정의
class ResistInfo:
    def __init__(self, resist_hit=None, resist_penetrate=None, resist_slash=None):
        self.resist_hit = resist_hit
        self.resist_penetrate = resist_penetrate
        self.resist_slash = resist_slash

# ResistInfo 객체 정의
class StaggerThreshold:
    def __init__(self, first_stagger=None, second_stagger=None, third_stagger=None):
        self.first_stagger = first_stagger
        self.second_stagger = second_stagger
        self.third_stagger = third_stagger

class SkillSet:
    def __init__(self):

        self.skill_1_type = ""
        self.skill_2_type = ""
        self.skill_3_type = ""
        self.defense_skill_type = ""

        self.skill_1_sin_affinity = ""
        self.skill_2_sin_affinity = ""
        self.skill_3_sin_affinity = ""
        self.defense_skill_sin_affinity = ""

        self.skill_1_details = []
        self.skill_2_details = []
        self.skill_3_details = []
        self.defense_skill_details = []

    def set_skill_type(self, skill_code, skill_type, sin_affinity):
        if skill_code == "skill 1":
            self.skill_1_type = skill_type
            self.skill_1_sin_affinity = sin_affinity
        elif skill_code == "skill 2":
            self.skill_2_type = skill_type
            self.skill_2_sin_affinity = sin_affinity
        elif skill_code == "skill 3":
            self.skill_3_type = skill_type
            self.skill_3_sin_affinity = sin_affinity
        elif skill_code == "defense":
            self.defense_skill_type = skill_type
            self.defense_skill_sin_affinity = sin_affinity
    
    def add_skill(self, skill_code, skill):
        if skill_code == "skill 1":
            self.skill_1_details.append(skill)
            skill.skill_type = self.skill_1_type
            skill.sin_affinity = self.skill_1_sin_affinity
        elif skill_code == "skill 2":
            self.skill_2_details.append(skill)
            skill.skill_type = self.skill_2_type
            skill.sin_affinity = self.skill_2_sin_affinity
        elif skill_code == "skill 3":
            self.skill_3_details.append(skill)
            skill.skill_type = self.skill_3_type
            skill.sin_affinity = self.skill_3_sin_affinity
        elif skill_code == "defense":
            self.defense_skill_details.append(skill)
            skill.skill_type = self.defense_skill_type 
            skill.sin_affinity = self.defense_skill_sin_affinity

    def get_skill_by_name(self, skill_name):
        for skill_list in [self.skill_1_details, self.skill_2_details, self.skill_3_details, self.defense_skill_details]:
            for skill in skill_list:
                result = skill.is_skill_by_name(skill_name)
                if result:
                    return result
        return None

    def add_skill_uptie_info(self, skill_name, uptie_level, skill_power, coin_power, effect_by_condition):
        skill = self.get_skill_by_name(skill_name)
        skill.add_uptie_info(uptie_level, skill_power, coin_power, effect_by_condition)
        

class Skill:
    def __init__(self, skill_code, skill_name, coin_count, init_level, init_remain, atk_weight):
        self.skill_code = skill_code # "Skill 1"/"Skill 2"/"Skill 3"/"Defense" '''o'''
        self.skill_type = None
        self.sin_affinity = None
        self.skill_name = skill_name # 스킬 이름 '''o'''
        self.coin_count = coin_count # 동전 개수 '''o'''
        self.init_level = init_level # 스킬이 초기 배정받은 레벨, 인격 레벨에 - 1 한 값을 더해주면 스킬의 현 level을 구할 수 있다. '''o'''
        self.init_remain = init_remain #'''o'''
        self.atk_weight = atk_weight #공격 가중치 값이다. 방어스킬에는 없기 때문에 None으로 설정했다. '''o'''
        self.remain_count = init_remain # 실제 스킬이 작동하며 횟수가 차감되 나중에 모두 차감되면 다음 턴에 다시 초기횟수 리필된다.
        self.uptie_info = []  

    def is_skill_by_name(self, skill_name):
        if self.skill_name == skill_name:
            return self
        return None
        
    def add_uptie_info(self, uptie_level, skill_power, coin_power, effects_by_condition):
        self.uptie_info.append(UptieInfo(uptie_level, skill_power, coin_power, effects_by_condition))

class UptieInfo:
    def __init__(self, uptie_level, skill_power, coin_power, effects_by_condition):
        self.uptie_level = uptie_level   #'''o'''
        self.skill_power = skill_power   #'''o'''
        self.coin_power = coin_power     #'''o'''
        self.skill_effect = effects_by_condition

# 추상화된 EffectType 클래스
class EffectType:
    def __init__(self, behavior_type, name, description):
        self.behavior_type = behavior_type
        self.name = name  # Effect 이름
        self.description = description  # Effect 설명

# Passive 클래스
class Passive(EffectType):
    def __init__(self, behavior_type, name, condition, description):
        super().__init__(behavior_type, name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)  # condition이 있는지 여부 확인

# SupportPassive 클래스
class SupportPassive(EffectType):
    def __init__(self,behavior_type, name, condition, description):
        super().__init__(behavior_type, name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)

# PanicType 클래스
class PanicType(EffectType):
    def __init__(self, behavior_type, name, condition, description):
        super().__init__(behavior_type, name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)

# Keyword 클래스
class Keyword:
    def __init__(self, keyword, explain=None, dispellable=None):
        self.keyword = keyword
        self.explain = explain
        self.dispellable = dispellable

      
class DataContainer:
    """크롤링 데이터를 관리하는 클래스"""
    def __init__(self):
        self.identities = []

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.level_max

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.uptie_max

    def update_identity_name(self, identity_id, identity_name):
        """특정 ID의 이름을 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_name(identity_name)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_identity_rarity(self, identity_id, rarity):
        """특정 ID의 희귀도를 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_rarity(rarity)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_skill(self, identity_id, skill_number, new_type):
        """
        특정 identity의 skill_type을 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_type: 새 skill_type
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_type = new_type
                elif skill_number == 2:
                    identity.skill_2_type = new_type
                elif skill_number == 3:
                    identity.skill_3_type = new_type
                elif skill_number == "defense":
                    identity.defense_type = new_type
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def update_sin_affinity(self, identity_id, skill_number, new_affinity):
        """
        특정 identity의 sin_affinity를 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_affinity: 새 sin_affinity
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_sin_affinity = new_affinity
                elif skill_number == 2:
                    identity.skill_2_sin_affinity = new_affinity
                elif skill_number == 3:
                    identity.skill_3_sin_affinity = new_affinity
                elif skill_number == "defense":
                    identity.defense_sin_affinity = new_affinity
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def add_identity(self, identity):
        """새로운 인격 데이터를 추가."""
        self.identities.append(identity)

    def add_level_stat(self, identity_id, level, hp, defense):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_level_stat(level, hp, defense)

    def add_uptie_stat(self, identity_id, uptie, speed):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_uptie_stat(uptie, speed)

    def add_resist_info(self, identity_id, resist_hit, resist_penetrate, resist_slash):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_resist_info(resist_hit, resist_penetrate, resist_slash)

    def add_stagger_threshold(self, identity_id, first_stagger, second_stagger, third_stagger):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_stagger_threshold(first_stagger, second_stagger, third_stagger)

    
    def add_skill(self, identity_id, skill_code, skill_name, coin_count, init_level, init_remain, atk_weight):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_code == "skill 1":
                    identity.skillset.set_skill_type(skill_code, identity.skill_1_type, identity.skill_1_sin_affinity)
                elif skill_code == "skill 2":
                    identity.skillset.set_skill_type(skill_code, identity.skill_2_type, identity.skill_2_sin_affinity)
                elif skill_code == "skill 3":
                    identity.skillset.set_skill_type(skill_code, identity.skill_3_type, identity.skill_3_sin_affinity)
                elif skill_code == "defense":
                    identity.skillset.set_skill_type(skill_code, identity.defense_type, identity.defense_sin_affinity)
                identity.add_skill(skill_code, skill_name, coin_count, init_level, init_remain, atk_weight)

    def add_skill_uptie_info(self, identity_id, skill_name, uptie_level, skill_power, coin_power, effects_by_condition):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_skill_uptie_info(skill_name, uptie_level, skill_power, coin_power, effects_by_condition)

    def add_influence_behavior(self, identity_id, behavior_type, behavior_name, condition, description):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_influence_behavior(behavior_type, behavior_name, condition, description)
        
    def display_identities(self):
        """Identity, ID, Name, Rarity 정보를 출력"""
        if not self.identities:
            print("No identities found.")
        else:
            # 데이터를 리스트로 변환
            table = []
            for idx, identity in enumerate(self.identities, 1):
                table.append([
                    idx, identity.identity_id, identity.name, identity.rarity,
                    identity.skill_1_type, identity.skill_1_sin_affinity,
                    identity.skill_2_type, identity.skill_2_sin_affinity,
                    identity.skill_3_type, identity.skill_3_sin_affinity,
                    identity.defense_type, identity.defense_sin_affinity
                ])
            
            # 테이블 헤더 정의
            headers = [
                "Index", "ID", "Name", "Rarity",
                "Skill_1 Type", "Skill_1 Sin", "Skill_2 Type", "Skill_2 Sin",
                "Skill_3 Type", "Skill_3 Sin", "Defense Type", "Defense Sin"
            ]
            
            # 테이블 출력
            print(tabulate(table, headers=headers, tablefmt="fancy_grid"))

    def export_to_csv(self, filename, fieldnames, data):
        """
        공통적으로 사용 가능한 CSV 내보내기 메서드
        :param filename: 내보낼 파일 이름 (예: 'identity.csv', 'skill.csv')
        :param fieldnames: CSV의 헤더 리스트
        :param data: CSV로 내보낼 데이터 (리스트 형태)
        """
        try:
            # 현재작업 디렉토 기준으 'csv'폴더 없으면 생성
            if not os.path.exists('./csv'):
                os.makedirs('./csv')
            # 파일 경로 설정 (상대경로 'csv' 폴더 안에 저장)
            file_path = os.path.join('./csv', filename)
            with open(file_path, mode='w', newline='', encoding='utf-8') as file:
                writer = csv.DictWriter(file, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(data)
            print(f"Data successfully exported to {filename}")
        except Exception as e:
            print(f"Error exporting to CSV: {e}")

    def get_identity_data(self):
        """
        Identity 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "ID": identity.identity_id,
                "Name": identity.name,
                "Rarity": identity.rarity,
                "Skill_1_Type": identity.skill_1_type,
                "Skill_1_Sin_Affinity": identity.skill_1_sin_affinity,
                "Skill_2_Type": identity.skill_2_type,
                "Skill_2_Sin_Affinity": identity.skill_2_sin_affinity,
                "Skill_3_Type": identity.skill_3_type,
                "Skill_3_Sin_Affinity": identity.skill_3_sin_affinity,
                "Defense_Type": identity.defense_type,
                "Defense_Sin_Affinity": identity.defense_sin_affinity,
            }
            for identity in self.identities
        ]
        
    def export_identities_data(self, filename="identities.csv"):
        """Identity 데이터를 CSV로 내보내는 메서드"""
        identity_fieldnames = [
            "ID", "Name", "Rarity",
            "Skill_1_Type", "Skill_1_Sin_Affinity",
            "Skill_2_Type", "Skill_2_Sin_Affinity",
            "Skill_3_Type", "Skill_3_Sin_Affinity",
            "Defense_Type", "Defense_Sin_Affinity"
        ]
        identity_data = self.get_identity_data()  # Identity 데이터 가져오기
        self.export_to_csv(filename, identity_fieldnames, identity_data)  # CSV로 내보내기

    def load_identities_by_id(self, filename, identity_ids):
        """지정된 identity_ids 목록에 해당하는 Identity 데이터를 CSV에서 읽어옴."""
        identity_ids_set = set(identity_ids) # 중복 제거를 위 Set 사용
        with open(filename, mode='r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                if row["ID"] in identity_ids_set:
                    identity = Identity(
                        identity_id=row["ID"],
                        name=row["Name"],
                        rarity=row["Rarity"],
                        skill_1_type=row["Skill_1_Type"],
                        skill_1_sin_affinity=row["Skill_1_Sin_Affinity"],
                        skill_2_type=row["Skill_2_Type"],
                        skill_2_sin_affinity=row["Skill_2_Sin_Affinity"],
                        skill_3_type=row["Skill_3_Type"],
                        skill_3_sin_affinity=row["Skill_3_Sin_Affinity"],
                        defense_type=row["Defense_Type"],
                        defense_sin_affinity=row["Defense_Sin_Affinity"]
                    )
                    self.identities.append(identity)

    def get_level_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Level": level_stat.level,
                "HP": level_stat.hp,
                "Defense": level_stat.defense,
            }
            for identity in self.identities
            for level_stat in identity.level_stats
        ]

    def export_level_stats_data(self, filename="levelstats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        level_stats_fieldnames = ["Identity_ID", "Level", "HP", "Defense"]
        level_stats_data = self.get_level_stats_data()  # level_stats 데이터 가져오기
        self.export_to_csv(filename, level_stats_fieldnames, level_stats_data)  # CSV로 내보내기

    def get_uptie_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Uptie": uptie_stat.uptie,
                "Speed": uptie_stat.speed,
            }
            for identity in self.identities
            for uptie_stat in identity.uptie_stats
        ]

    def export_uptie_stats_data(self, filename="Uptiestats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        uptie_stats_fieldnames = ["Identity_ID", "Uptie", "Speed"]
        uptie_stats_data = self.get_uptie_stats_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, uptie_stats_fieldnames, uptie_stats_data)  # CSV로 내보내기

    def get_resist_info_data(self):
        """
        모든 Identity의 resist_info 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Resist_Hit": identity.resist_info.resist_hit,
                "Resist_Penetrate": identity.resist_info.resist_penetrate,
                "Resist_Slash": identity.resist_info.resist_slash,
            }
            for identity in self.identities
        ]

    def export_resist_info_data(self, filename="ResistInfos.csv"):
        """ResistInfo 데이터를 CSV로 내보내는 메서드"""
        resist_infos_fieldnames = ["Identity_ID", "Resist_Hit", "Resist_Penetrate", "Resist_Slash"]
        resist_infos_data = self.get_resist_info_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, resist_infos_fieldnames, resist_infos_data)  # CSV로 내보내기

    def export_uptie_stats_data(self, filename="Uptiestats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        uptie_stats_fieldnames = ["Identity_ID", "Uptie", "Speed"]
        uptie_stats_data = self.get_uptie_stats_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, uptie_stats_fieldnames, uptie_stats_data)  # CSV로 내보내기

    def get_stagger_threshold_data(self):
        """
        모든 Identity의 stagger_threshold 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "First_Stagger": identity.stagger_threshold.first_stagger,
                "Second_Stagger": identity.stagger_threshold.second_stagger,
                "Third_Stagger": identity.stagger_threshold.third_stagger,
            }
            for identity in self.identities
        ]

    def export_stagger_threshold_data(self, filename="StaggerThresholds.csv"):
        """StaggerThresholds 데이터를 CSV로 내보내는 메서드"""
        stagger_thresholds_fieldnames = ["Identity_ID", "First_Stagger", "Second_Stagger", "Third_Stagger"]
        stagger_thresholds_data = self.get_stagger_threshold_data()  # stagger_thresholds 데이터 가져오기
        self.export_to_csv(filename, stagger_thresholds_fieldnames, stagger_thresholds_data)  # CSV로 내보내기

    def get_skillset_data(self):
        """모든 Identity의 skillset 데이터를 export_to_csv을 포맷으로 변환"""
        return [
            {
                "Identity_ID": identity.identity_id,
                "Skill_Code": skill.skill_code,
                "Skill_Type": skill.skill_type,
                "Sin_Affinity": skill.sin_affinity,
                "Skill_Name": skill.skill_name,
                "Coin_Count": skill.coin_count,
                "Init_Level": skill.init_level,
                "Init_Remain": skill.init_remain,
                "Atk_Weight": skill.atk_weight,
                "Uptie_Level": uptie_info.uptie_level,
                "Skill_Power": uptie_info.skill_power,
                "Coin_Power": uptie_info.coin_power,
                "Effect_Data": json.dumps(uptie_info.skill_effect)
            }
            for identity in self.identities
            for skill_list in [identity.skillset.skill_1_details, identity.skillset.skill_2_details, identity.skillset.skill_3_details, identity.skillset.defense_skill_details]
            for skill in skill_list
            for uptie_info in skill.uptie_info
        ]

    def export_identity_skillset_data(self, filename="IdentitiesSkills.csv"):
        """IdentitiesSkills 데이터를 CSV로 내보내는 메서드"""
        identities_skills_fieldnames = ["Identity_ID", "Skill_Code", "Skill_Type", "Sin_Affinity", "Skill_Name", "Coin_Count", "Init_Level", "Init_Remain", "Atk_Weight", "Uptie_Level", "Skill_Power", "Coin_Power", "Effect_Data"]
        identities_skills_data = self.get_skillset_data()
        self.export_to_csv(filename, identities_skills_fieldnames, identities_skills_data)

    def get_identities_behaviors_data(self):
        """모든 Identity의 influence behavior 데이터를 export_to_csv 포맷으로 변환"""
        return [
            {
                "Identity_ID": identity.identity_id,
                "Effect_Type": effect_type,
                "Behavior_Name": effect.name,
                "Behavior_Condition": effect.condition if effect.condition else np.nan,
                "Behavior_Description" : effect.description
            }
            for identity in self.identities
            for effect_list, effect_type in [(identity.passive_infos, 'Passive'), (identity.support_passive_infos, 'Support Passive'), (identity.panic_type, 'Panic Type')]
            for effect in effect_list
        ]

    def export_identities_behaviors_data(self, filename="IdentitiesBehaviors.csv"):
        """IdentitiesBehaviors 데이터를 CSV로 내보내는 메서드"""
        identities_behaviors_fieldnames = ["Identity_ID","Effect_Type","Behavior_Name","Behavior_Condition","Behavior_Description"]
        identities_behaviors_data = self.get_identities_behaviors_data()
        self.export_to_csv(filename, identities_behaviors_fieldnames, identities_behaviors_data)
            
async def main():
    data_container = DataContainer()
    scraper = Scraper(container=data_container, base_url="https://limbus.kusoge.xyz", img_url="https://img.kusoge.xyz/limbus/img")
    await scraper.scrape()

await main() 

{'keywords': ['Ammo'], 'description': "When flipping a Coin that consumes keyword:Ammo: even when this unit is out of, the attack does not get canceled. However, the affected Coins'On Hit effects do not activate in such cases."}
{'keywords': ['Overwatch Assignment'], 'description': "If the Full - Stop Office Fixer Heathcliff is in keyword:Overwatch Assignment state, or has been Deployed as a Backup unit, empower Full - Stop Office Rep. Hong Lu's Skill 2 and Skill 3"}
{'keywords': ['Overwatch Assignment', 'Poise'], 'description': "If the Full - Stop Office Fixer Heathcliff is in keyword:Overwatch Assignment state, or has been Deployed as a Backup unit, empower Full - Stop Office Rep. Hong Lu's Skill 2 and Skill 3 Combat Start: at 20 + keyword:Poise Potency, gain + 1 Count"}
{'keywords': ['Ammo'], 'description': 'Ally Identity with the most keyword:Ammo deals + 10% more damage with Skills that spend (Does not activate when out of)'}
{'keywords': ['Pierce Power Up', 'Ammo', 'Overwatch Ass

In [41]:
import os
import csv
import re
import json
import numpy as np
import pandas as pd
from playwright.async_api import async_playwright
from tabulate import tabulate

# Sin Color 이름 변환을 위한 매핑
resource_mapping = {
    "AZURE": "Gloom",
    "VIOLET": "Envy",
    "AMBER": "Sloth",
    "SHAMROCK": "Gluttony",
    "CRIMSON": "Wrath",
    "SCARLET": "Lust",
    "INDIGO": "Pride"
}

def convert_resource(resource_name):
    """Sin Color 이름을 변환하는 함수. 매핑되지 않은 경우 원래 이름을 반환."""
    return resource_mapping.get(resource_name, resource_name)

def find_sin_for_color(url):
    # URL에 color_name을 추출
    color_name = url.split('/')[-1].split('.')[0].replace("attr_", "")
    # 해당 색상이 resource_mapping에 있는 확인
    for color, sin in resource_mapping.items():
        if color_name in color:
            return sin
    return None


class Scraper:
    """웹 페이지 크롤링을 위한 클래스"""
    def __init__(self, container, base_url, img_url):
        self.container = container
        self.base_url = base_url
        self.img_url = img_url
        self.uptie_level = 1
        self.keywords_set = set()

    async def handle_request(self, route, request):
        """이미지와 폰트 리소스 로드를 차단."""
        if request.resource_type =="image":
            #print(f"{request.url}")
            img_url = request.url
            if  re.match(f"{self.img_url}/ui/attr.*\\.png", img_url):
                await route.continue_()
            else:
                await route.fulfill(
                    status=200,
                    content_type="image/png",
                    body=b""
                )
        elif request.resource_type == "font":
            await route.abort()
        else:
            await route.continue_()
            
    async def scrape(self):
        """크롤링 메인 함수"""
        async with async_playwright() as p:
            browser = await p.chromium.launch(headless=False)
            page = await browser.new_page()
            #await page.evaluate('window.scrollTo(0, document.body.scrollHeight)')
            await page.route("**/*", self.handle_request)
            await page.wait_for_load_state("domcontentloaded")
            # Step 1: /identity 페이지에서 기본 정보 크롤링
            #await self.crawl_identity_page(page)

            # Step 2: /identity/{identity_id} 상세 페이지 크롤링
            #identity_ids_to_load = ["10104", "10310", "10710","10808", "10911", "11005"]
            #identity_ids_to_load = ["10104", "10911"]#,"10310", "10710","10808", "11005", "11210"]
            #identity_ids_to_load = ["10911","10310"]#, "10710","10808", "11005", "11210"]
            identity_ids_to_load = ["10911", "10611", "10711"]#, "10710","10808", "11005", "11210"]
            self.container.load_identities_by_id('./csv/identities.csv', identity_ids_to_load)
            await self.crawl_identity_details_for_all(page)
            #self.container.display_identities()
            #self.container.export_identities_data("identities.csv")
            #self.container.export_level_stats_data("Levelstats.csv")
            #self.container.export_uptie_stats_data("Uptiestats.csv")
            #self.container.export_resist_info_data("ResistInfos.csv")
            #self.container.export_stagger_threshold_data("StaggerThresholds.csv")
            #self.container.export_identity_skillset_data("IdentitiesSkills.csv")
            #self.container.export_identities_behaviors_data("IdentitiesBehaviors.csv")
            await self.crawl_keywords_details_for_collect(page)
            self.container.export_keywords_data("Keywords.csv")
            page.set_default_timeout(10000)
            #await page.wait_for_timeout(100000)
            await browser.close()

    async def get_identity_id_from_row(self, row):
        """테이블 행(row)에서 인격 ID를 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_url = await identity_cell.get_attribute('href')
            identity_id = identity_url.split('/identity/')[-1]
            return identity_id
        return "Unknown"

    async def get_identity_name_from_row(self, row):
        """테이블 행(row)에서 인격 이름을 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_name = (await identity_cell.inner_text()).replace("\n", " ").strip()
            return identity_name
        return "Unknown"
        
    async def crawl_identity_page(self, page):
        """/identity 페이지로 이동하여 인격 기본 정보를 추출."""
        await page.goto(self.base_url + "/identity")
        await self.extract_identity_page(page)

    async def extract_identity_page(self, page):
        """/identity 페이지의 모든 테이블 행에서 인격 정보를 추출."""
        identity_rows = await page.query_selector_all('tr')
        if len(identity_rows) > 1:
            for idx, identity_row in enumerate(identity_rows):
                if idx == 0:
                    continue # 0번째요소이름을 보여주는 열은 건너뜀
                await self.scrape_identity_name_and_identity_id(identity_row)
                await self.extract_rarity(identity_row)
                for i in range(1, 4):  # 스킬1~3 정보 추출
                    await self.extract_atk_and_resource(identity_row, i)
                await self.extract_defense_and_resource(identity_row)

    async def scrape_identity_name_and_identity_id(self, row):
        """인격 번호와 이름을 추출하여 DataContainer에 저장."""
        try:
            identity_id = await self.get_identity_id_from_row(row)
            identity_name = await self.get_identity_name_from_row(row)
            identity = Identity(identity_id)
            self.container.add_identity(identity)
            self.container.update_identity_name(identity_id, identity_name)
        except Exception as e:
            print(f"Error extracting identity name and identity id: {e}")

    async def get_element_attribute(self, element, selector, attribute):
        """요소에서 지정된 속성(attribute)을 추출."""
        try:
            target_element = await element.query_selector(selector)
            if target_element:
                return await target_element.get_attribute(attribute)
        except Exception as e:
            print(f"Error getting element attribute: {e}")
        return None
    
    async def extract_rarity(self, row):
        """테이블 행(row)에서 희귀도를 추출하여 업데이트."""
        try:
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
                sibling = await parent.evaluate_handle('node => node.nextElementSibling')
                if sibling:
                    rarity_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/rarity_"]', 'src')
                    if rarity_img_src:
                        rarity_level = rarity_img_src.split('_')[-1].split('.')[0]  # rarity_1.png -> 1
                        identity_id = await self.get_identity_id_from_row(row)
                        self.container.update_identity_rarity(identity_id, rarity_level)
                    else:
                        print("Rarity image not found in the next sibling TD.")
        except Exception as e: 
            print(f"Error extracting rarity: {e}")

        return "Unknown"

    async def extract_atk_and_resource(self, row, skill_number):
        """ 공격 타입(ATK Type)과 자원(Attribute) 추출 함수 - 스킬 번호에 따라 처리 """
        try:
            #print(f"Attempting to locate skill{skill_number} type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 해당 스킬 번호에 맞는 형제로 이동
                sibling = parent
                for _ in range(skill_number + 1):  # 스킬 번호에 맞게 이동
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                atk_type, resource = "Unknown", "Unknown"
        
                if sibling:
                    # 공격 타입 이미지 (atk_type) 찾기
                    atk_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/atk_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if atk_type_img_src:
                        atk_type = atk_type_img_src.split('_')[-1].split('.')[0]  # atk_type_SLASH.png -> SLASH
                        #print(f"Extracted attack type: {atk_type}")
                        self.container.update_skill(identity_id, skill_number, atk_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted resource: {resource}")
                        
                        # 리소스를 변환
                        resource = convert_resource(resource)
                        #print(f"Converted resource: {resource}")
                        self.container.update_sin_affinity(identity_id, skill_number, resource)
                        
        except Exception as e:
            print(f"Error extracting skill{skill_number} type and resource: {e}")

    async def extract_defense_and_resource(self, row):
        """ 방어 타입(Def Type)과 자원(Attribute) 추출 함수 """
        try:
            #print("Attempting to locate defense type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 다섯 번째 형제 td로 이동 (방어 관련 정보가 있는 <td>)
                sibling = parent
                for _ in range(5):  # 방어 정보는 5번째 형제에 있음
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                def_type, def_resource = "Unknown", "Unknown"
        
                if sibling:
                    # 방어 타입 이미지 (def_type) 찾기
                    def_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/def_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if def_type_img_src:
                        def_type = def_type_img_src.split('_')[-1].split('.')[0]  # def_type_GUARD.png -> GUARD
                        #print(f"Extracted defense type: {def_type}")
                        self.container.update_skill(identity_id, 'defense', def_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        def_resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted defense resource: {def_resource}")
                        
                        # 방어 리소스를 변환
                        def_resource = convert_resource(def_resource)
                        #print(f"Converted defense resource: {def_resource}")
                        self.container.update_sin_affinity(identity_id, 'defense', def_resource)
 
        except Exception as e:
            print(f"Error extracting defense type and resource: {e}")

    async def crawl_identity_details_for_all(self, page):
        """모든 인격의 세부 정보를 크롤링."""
        for identity in self.container.identities:
            await page.goto(self.base_url + "/identity/"+identity.identity_id)
            """ 레벨과 동기화와 관련 없는 참관타 저항은 밑에서 변경하는 것과 연관성이 적기 때문에 위에서 처리한다.
            print(f"Identity_id: {identity.identity_id}")
            results = []
            resist_type_imgs = [f"{self.img_url}/ui/atk_type_HIT.png",
                                f"{self.img_url}/ui/atk_type_PENETRATE.png",
                                f"{self.img_url}/ui/atk_type_SLASH.png"]
            for resist_type_img in resist_type_imgs:
                result = await self.extract_resist_type(page, resist_type_img, 1)
                results.append(result)
            self.container.add_resist_info(identity.identity_id, results[0], results[1], results[2])
            """
            """
            # Stagger Threshold 값 추출
            stagger_threshold = await self.extract_stagger_threshold_values(page)
            self.container.add_stagger_threshold(identity.identity_id, stagger_threshold[0], stagger_threshold[1], stagger_threshold[2])
            """
            await self.initialize_uptie_and_level(page)
            """ 레벨에 따른 level, hp, defense 다르게 되어 있는 것을 출력 확인 객체로 들어가서 csv로 옮기는 과정까지 진행
            for i in range(1, identity.level_max+1):
                await self.move_slider_to_value(page,i)
                await self.extract_level_status(page, identity.identity_id, i)
            """
            """ 동기화에 따른 speed 다르게 되어 있는 것을 출력 확인 객체로 들어가서 csv로 옮기는 과정까지 진행
            for i in range(1, identity.uptie_max+1):
                await self.uptie_click(page,i)
                await self.extract_uptie_status(page, identity.identity_id, i)
            """
            """
                인격에 대한 skill 정보를 가져오는 메소드 위에서 출력하는 csv파일로 인격이 가지고 있는 스킬 정보를 내보낸다.
            await self.crawl_skills(page, identity.identity_id)
            """
            await self.crawl_influence_behavior(page, identity.identity_id)

    async def crawl_keywords_details_for_collect(self, page):
        """모아진 키워드의 세부 정보를 크롤링."""
        for keyword in self.keywords_set:
            await page.goto(self.base_url + "/keyword/"+keyword)
            await self.crawl_keyword_info(page, keyword)
            
    async def initialize_uptie_and_level(self, page):
        await self.uptie_click(page, 1)
        await self.move_slider_to_value(page, 1)
        self.uptie_level = 1
        #return 1
    
    async def uptie_click(self, page, phase):
        fab_button_selector = ".q-btn.q-btn--fab"
        await page.wait_for_selector(fab_button_selector)
        await page.click(fab_button_selector)
        #print(f"[{setup_uptie_button}] 상위 동기화 버튼 클릭 완료")
        setup_uptie_button = f'a:has(img[src="{self.img_url}/ui/uptie_{phase}.png"])'
        await page.wait_for_selector(setup_uptie_button)
        await page.click(setup_uptie_button)
        #print(f"{phase} 동기화 버튼 클릭 완료")

    async def move_slider_to_value(self, page, target_value):
        """
        슬라이더를 주어진 값으로 이동시킵니다.
        target_value는 슬라이더의 최소값부터 최대값 사이여야 합니다.
        """
        slider_selector = ".q-slider"
        # 슬라이더의 범위 확인
        min_value = int(await page.get_attribute(slider_selector, "aria-valuemin"))
        max_value = int(await page.get_attribute(slider_selector, "aria-valuemax"))
    
        if target_value < min_value or target_value > max_value:
            raise ValueError(f"Target value {target_value} is out of bounds ({min_value}, {max_value})")
    
        # 슬라이더 thumb 요소와 위치 가져오기
        slider_thumb = await page.query_selector(f"{slider_selector} .q-slider__thumb")
        thumb_box = await slider_thumb.bounding_box()
        
        # 슬라이더의 현재 위치 및 목표 위치 계산
        track = await page.query_selector(f"{slider_selector} .q-slider__track")
        track_box = await track.bounding_box()
        
        # 목표 x 위치 계산 (슬라이더 트랙 내에서 비율 기반)
        target_percentage = (target_value - min_value) / (max_value - min_value)
        target_x = track_box["x"] + target_percentage * track_box["width"]
        current_x = thumb_box["x"] + thumb_box["width"] / 2
        y = thumb_box["y"] + thumb_box["height"] / 2
    
        # 슬라이더를 드래그하여 목표 위치로 이동
        await page.mouse.move(current_x, y)
        await page.mouse.down()
        await page.mouse.move(target_x, y, steps=10)  # 부드럽게 이동
        await page.mouse.up()
    
        #print(f"슬라이더를 {target_value}로 이동 완료.")
    
        # 슬라이더 값 확인 및 강제 이벤트 트리거
        await page.evaluate(
            f"""
            (slider) => {{
                slider.dispatchEvent(new Event('input'));  // oninput 이벤트 트리거
                slider.dispatchEvent(new Event('change'));  // onchange 이벤트 트리거
            }}
            """,
            await page.query_selector(slider_selector)
        )
    
        # 이동 후 값 확인
        current_value = await page.get_attribute(slider_selector, "aria-valuenow")
        if current_value != str(target_value):
            print(f"경고: 슬라이더 값이 예상과 다릅니다. 현재 값: {current_value}, 예상 값: {target_value}")

    async def get_second_status_element(self, page):
        """
        두 번째 'Status' 요소에 접근하여 반환하는 메소드
        """
        status_elements = await page.query_selector_all("text=Status")
        if len(status_elements) > 1:
            second_status_element = status_elements[1]
            #print("두 번째 'Status' 요소에 접근합니다.")
            return second_status_element
        else:
            #print("두 번째 'Status' 요소가 존재하지 않습니다.")
            return None

    async def extract_level_status(self, page, identity_id, level):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 레벨에 따라 HP와 Defense가 변한다.
        hp_value = await self.extract_hp(second_status_element)
        defense_value = await self.extract_defense(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Level={level}, HP={hp_value}, Defense={defense_value}")
        self.container.add_level_stat(identity_id,level, hp_value, defense_value)
    
    async def extract_hp(self, status_element):
        # HP 추출
        hp_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].children[0].nextElementSibling.textContent.trim()"
        )
        return hp_value
    
    async def extract_defense(self, status_element):
        # Defense 추출
        defense_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return defense_value

    async def extract_uptie_status(self, page, identity_id, uptie):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 uptie에 따라 speed는 변한다.
        speed_value = await self.extract_speed(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Uptie={uptie}, Speed={speed_value}")
        self.container.add_uptie_stat(identity_id,uptie, speed_value)

    async def extract_speed(self, status_element):
        # Speed 추출
        speed_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return speed_value

    async def extract_resist_type(self, page, image_src, depth):
        # 이미지 요소 찾기
        #print(image_src)
        image = await page.query_selector(f'img[src="{image_src}"]')
        #print(image)
        if not image:
            return "Unknown"
    
        # 이미지의 부모 요소를 찾아서 올라감
        parent = image
        for _ in range(depth):
            parent = await parent.evaluate_handle('node => node.parentElement')
            if not parent:
                return "Unknown"
    
        # 부모의 형제 요소 찾기
        sibling = await parent.evaluate_handle('node => node.nextElementSibling')
        
        if sibling:
            value = await sibling.evaluate('node => node.textContent')
            if value:
                return value.strip()
        
        return "Unknown"

    async def extract_stagger_threshold_values(self, page):
        # 'div.row.center.flex-center.q-pt-xs' 요소들 찾기
        elements = await page.query_selector_all('div.row.center.flex-center.q-pt-xs')
        
        # 각 요소의 텍스트 내용 출력하여 'Stagger Threshold' 요소를 찾기
        for element in elements:
            text = await element.text_content()
            
            if text and "Stagger Threshold" in text:
                # 'Stagger Threshold'가 있는 요소를 찾았다면, 그 이후 형제 요소들에서 값을 추출
                parent = await element.evaluate_handle('node => node.parentElement')
                sibling_elements = await parent.query_selector_all('div.row.justify-evenly div')  # Threshold 값이 들어있는 div들
                
                stagger_values = []
                for sibling in sibling_elements:
                    text_content = await sibling.text_content()
                    match = re.search(r'(\d+%)', text_content)
                    if match:
                        stagger_values.append(match.group(1))  # 퍼센트 값만 리스트에 추가
                while len(stagger_values) < 3:
                    stagger_values.append(np.nan)
                #print(stagger_values)
                return stagger_values  # 추출된 값(또는 np.nan 포함)을 반환
        
        return [np.nan, np.nan, np.nan]  # 만약 'Stagger Threshold'를 찾지 못한 경우 기본값 반환

    async def crawl_skills(self, page, identity_id):
        await self.crawl_skill(page, identity_id, "skill 1")
        await self.crawl_skill(page, identity_id, "skill 2")
        await self.crawl_skill(page, identity_id, "skill 3")
        await self.crawl_skill(page, identity_id, "defense")


    async def crawl_skill(self, page, identity_id, skill_type):
        skills_elements = await page.query_selector_all('div.skilltag')
        for skill_element in skills_elements:
            skill_text = await skill_element.text_content()
            if skill_text.strip().lower() == skill_type:
                await self.process_skill(skill_element, page, identity_id, skill_type)
        
    async def process_skill(self, skill_element, page, identity_id, skill_type):
        # skill_type이 "skill 3"일 경우 uptie_value가 3 이상일 때만 정보를 저장
        for uptie_value in range(1, 5):  # Uptie levels 1 to 4
            # "skill 3"은 uptie_value가 3 이상일 때만 처리
            if skill_type == "skill 3" and uptie_value < 3:
                continue  # skip if uptie_value is less than 3 for skill 3
    
            # uptie_level에 맞게 클릭
            await self.uptie_click(page, uptie_value)
            if skill_type != "skill 3" and uptie_value > 1:
                # Uptie 정보 갱신
                await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
                continue
            elif skill_type == "skill 3" and uptie_value > 3:
                # Uptie 정보 갱신
                await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
                continue
            # 스킬의 세부 정보를 추출
            skill_details = await self.extract_skill_details(skill_element, skill_type)
            skill_name = skill_details["skill_name"]
    
            # 스킬 정보를 저장
            self.container.add_skill(
                identity_id, skill_type, 
                skill_name, skill_details["coin_count"],
                skill_details["init_level"], skill_details["init_remain"], 
                skill_details["atk_weight"]
            )
            print(f"Added skill: {skill_type}:{skill_name}")
    
            # Uptie 정보 갱신
            await self.update_uptie_info(page, skill_element, identity_id, skill_name, skill_type, uptie_value)
    
    async def extract_skill_details(self, skill_element, skill_type):
        # 스킬 세부 정보를 추출하는 함수
        skill_name = await self.skill_name(skill_element)
        coin_count = await self.coin_count(skill_element, ".col-auto > .column > .row")
        init_level = await self.init_level(skill_element)
        init_remain = await self.init_remain(skill_element, skill_type, "div.row.justify-start > div.attr.flex.flex-center > div.q-px-sm")
        atk_weight = await self.atk_weight(skill_element, skill_type)
        return {
            "skill_name": skill_name,
            "coin_count": coin_count,
            "init_level": init_level,
            "init_remain": init_remain,
            "atk_weight": atk_weight
        }
    
    async def update_uptie_info(self, page, skill_element, identity_id, skill_name, skill_type, uptie_value):
        # Uptie 정보를 추출하고 업데이트하는 함수
        uptie_info = await self.extract_uptie_info(skill_element)
        effect_data = await self.skill_effects(skill_element)
        self.container.add_skill_uptie_info(
            identity_id, skill_name, 
            uptie_value, uptie_info["skill_power"], uptie_info["coin_power"], 
            effect_data.get("effectsByCondition")
        )
        keywords = effect_data.get("keywords")
        if keywords:
            self.keywords_set.update(keywords)
        print(
            f"Updated Uptie for {skill_name} (Uptie Level: {uptie_value}): "
            f"Skill Power: {uptie_info['skill_power']}, Coin Power: {uptie_info['coin_power']} "
            f"Skill Effect: {effect_data.get("effectsByCondition")}"
        )

    async def extract_uptie_info(self, skill_element):
        skill_data = await self.skill_and_coin_power(skill_element)
        return {
            "skill_power": skill_data.get("skillPower"),
            "coin_power": skill_data.get("coinPower")
        }
        
    async def crawl_influence_behavior(self, page, identity_id):
        influence_behaviors = await page.query_selector_all('div[align="right"]')
        # 텍스트를 기준으 추출하고 메소드로 파라미터 들어가 정보를 받는다.
        for influence_behavior in influence_behaviors:
            active_type = await influence_behavior.inner_text()
            active_type = active_type.strip()

            if active_type == "PASSIVE":
                """
                # PASSIVE에 대한 처리
                print(f"Processing PASSIVE logic for: {active_type}")
                # CALL method for PASSIVE
                """
                await self.passive_effects(influence_behavior, identity_id)
            elif active_type == "SUPPORT PASSIVE":
                """
                # SUPPORT PASSIVE에 대한 처리
                print(f"Processing SUPPORT PASSIVE logic for: {active_type}")
                # CALL method for SUPPORT PASSIVE
                """
                await self.support_passive_effects(influence_behavior, identity_id)
            elif active_type == "PANIC TYPE":
                """
                # PANIC TYPE에 대한 처리
                print(f"Processing PANIC TYPE logic for:{active_type}")
                # CALL method for PANIC TYPE
                """
                await self.panic_effects(influence_behavior, identity_id)

    async def skill_and_coin_power(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                // 조부모로 이동
                const grandparent = node.parentElement.parentElement;
                // 이전 형제 탐색
                const previousSibling = grandparent.previousElementSibling;
                if (!previousSibling) return {{ skillPower: null, coinPower: null }}; // 이전 형제가 없으면 기본값 반환
                // 이전 형제 내부의 .row.justify-around > div > div 탐색
                const targetDivs = previousSibling.querySelectorAll(".row.justify-around > div");
                if (targetDivs.length < 3) return {{ skillPower: null, coinPower: null }}; // 배열 길이 확인
                // 1st div: 스킬 위력
                const skillPower = targetDivs[0]?.querySelector('div')?.textContent?.trim() || null; 
                // 3rd div: 코인 파워 값 (3번째 div에서 텍스트 추출)
                const coinPower = targetDivs[2]?.textContent?.trim() || null;
                return {{ skillPower, coinPower }};
            }}
        """)


    async def coin_count(self, skill_element, hierarchy_selector):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector('{hierarchy_selector}');
                if (!targetElement) return 0;
                return targetElement.childElementCount;
            }}
        """)

    async def init_remain(self, skill_element, skill_type, hierarchy_selector):
        if skill_type == "defense":
            return -1
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector('{hierarchy_selector}');
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def skill_name(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector(".block");
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def init_level(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector("div.text-bold.text-h6.h6.q-pl-xs.q-pr-md");
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def atk_weight(self, skill_element, skill_type):
        if skill_type == "defense":
            return 0
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector("div.text-bold.text-h6.h6.q-pl-xs.q-pr-md");
                const doubleNextSibling = targetElement.nextElementSibling.nextElementSibling;
                if (!doubleNextSibling) return 0;
                return doubleNextSibling.textContent?.trim();
            }}
        """)

    async def skill_effects(self, skill_element):
        return await skill_element.evaluate("""
            node => {
                const parent = node.parentElement;
                const doubleNextSibling = parent.nextElementSibling?.nextElementSibling;
                if (!doubleNextSibling) return null;
    
                // doubleNextSibling에서 첫 번째 div.col 요소 찾기
                const textNode = doubleNextSibling.querySelector("div.col");
                if (!textNode) return null;
    
                const results = {
                    keywords: [], // 키워드 저장
                    conditions: [], // 조건 저장
                    effectsByCondition: {} // 조건별 효과
                };
    
                // 1. 조건 추출 (대괄호 텍스트)
                const conditionElements = textNode.querySelectorAll("span");
                conditionElements.forEach(span => {
                    let text = span.textContent?.trim();
                    if (text?.startsWith("[") && text?.endsWith("]")) {
                        // 부모의 전형제 노드 탐색
                        const sibling = span.parentElement.previousElementSibling;
                        if (sibling) {
                            const emblemImage = sibling.querySelector("img.emblem");
                            console.log("Sibling element found:", sibling); // 디버깅용
                            if (emblemImage) {
                                const src = emblemImage.getAttribute("src");
                                console.log("Image source:", src); // 디버깅용
                                const match = src.match(/ui_num_(\\d+)\\.png$/); // 정규식 수정
                                if (match) {
                                    const number = match[1]; // 숫자 추출
                                    text = `[Coin ${number} ${text.replace("[", "").replace("]", "")}]`;
                                    console.log("Updated condition:", text); // 디버깅용
                                }
                            }
                        }
                        results.conditions.push(text);
                        results.effectsByCondition[text] = []; // 조건별 효과를 초기화
                    }
                });
    
                // 2. 키워드 추출
                const keywordElements = textNode.querySelectorAll('a[href^="/en/keyword/"]');
                if (!keywordElements) return null;

                // 키워드 매핑 객체 초기화
                if (!results.keywordMapping) {
                    results.keywordMapping = {}; //텍스트: 키 형태로 저장
                }

                keywordElements.forEach(a => {
                    const span = a.querySelector('span.q-btn__content.text-center.col.items-center.q-anchor--skip.justify-center.row>span');
                    if (span) {
                        const keywordText = span.textContent.trim();
                        const href = a.getAttribute('href');
                        const keywordKey = href.split('/').pop();
                        if (keywordKey && !Object.values(results.keywordMapping).includes(keywordKey)) {
                            results.keywordMapping[keywordText] = keywordKey;
                        }
                    }
                });
    
                // 3. 효과 추출
                const effectElements = textNode.querySelectorAll("span");
                let currentCondition = null;
                let currentEffect = []; // 초기화
                let keywordList = new Set(Object.values(results.keywordMapping));
                effectElements.forEach(span => {
                    let text = span.textContent?.trim();
                    if (!text) return;
    
                    // 조건 발견 시 currentCondition 변경
                    if (text.startsWith("[") && text.endsWith("]")) {
                        // 이전 조건의 효과 저장
                        if (currentCondition && currentEffect.length > 0) {
                            // 부모의 전형제 노드 탐색
                            const sibling = span.parentElement.previousElementSibling;
                            if (sibling) {
                                const emblemImage = sibling.querySelector("img.emblem");
                                if (emblemImage) {
                                    const src = emblemImage.getAttribute("src");
                                    const match = src.match(/ui_num_(\\d+)\\.png$/); // 정규식 수정
                                    if (match) {
                                        const number = match[1]; // 숫자 추출
                                        text = `[Coin ${number} ${text.replace("[", "").replace("]", "")}]`;
                                    }
                                }
                            }
                            results.effectsByCondition[currentCondition].push(currentEffect.join(' '));
                            currentEffect = []; // 초기화
                        }
                        currentCondition = text;
                        return;
                    }
    
                    // 현재 조건의 효과 처리
                    if (currentCondition) {
                        if (results.keywordMapping[text]) {
                            const keywordEffect = `keyword:${results.keywordMapping[text]}`;
                            keywordList.add(results.keywordMapping[text]);
                            if (!currentEffect.includes(keywordEffect)) {
                                currentEffect.push(keywordEffect);
                            }
                        } else {
                            currentEffect.push(text);
                        }
                    }
                });

                results.keywords = Array.from(keywordList);
                delete results.keywordMapping;
                // 마지막 효과 저장
                if (currentCondition && currentEffect.length > 0) {
                    let processedText = currentEffect
                        .map(word => word.trim())
                        .join(" ")
                        .replace(/\\s+([,.:])/g, "$1")
                        .replace(/\\s+'/g, "'")
                        .replace(/'\\s+/g, "'")
                        .replace(/\\[\\s+/g, "[")
                        .replace(/\\s+\\]/g, "]")
                        .replace(/\\(\\s+/g, "(")
                        .replace(/\\s+\\)/g, ")")
                        .replace(/\\s+%/g, "%")
                        .replace(/\\s+\\//g, "/")
                        .replace(/\\/\\s+/g, "/")
                        .replace(/\\s+\\/\\s+/g, "/");
                    results.effectsByCondition[currentCondition].push(processedText);
                }
    
                return results;
            }
        """)

    async def passive_effects(self, passive_element, identity_id):
        passive_name = await self.behavior_name(passive_element)
        passive_conditions = await self.behavior_condition(passive_element)
        passive_description = await self.behavior_description(passive_element, "span")
        #print(passive_name)
        #print(passive_conditions)
        #print(passive_description)
        self.container.add_influence_behavior(identity_id, "passive", passive_name, passive_conditions, passive_description.get("description"))
        keywords = passive_description.get("keywords")
        if keywords:
            self.keywords_set.update(keywords)
    async def support_passive_effects(self, support_passive_element, identity_id):
        support_passive_name = await self.behavior_name(support_passive_element)
        support_passive_conditions = await self.behavior_condition(support_passive_element)
        support_passive_description = await self.behavior_description(support_passive_element, "span")
        #print(support_passive_name)
        #print(support_passive_conditions)
        #print(support_passive_description)
        self.container.add_influence_behavior(identity_id, "support passive", support_passive_name, support_passive_conditions, support_passive_description.get("description"))
        keywords = support_passive_description.get("keywords")
        if keywords:
            self.keywords_set.update(keywords)
    async def panic_effects(self, panic_element, identity_id):
        panic_name = await self.behavior_name(panic_element)
        panic_conditions = await self.behavior_condition(panic_element)
        panic_description = await self.behavior_description(panic_element, "div")
        #print(panic_type)
        #print(panic_conditions)
        #print(panic_description)
        self.container.add_influence_behavior(identity_id, "panic type", panic_name, panic_conditions, panic_description.get("description"))
        keywords = panic_description.get("keywords")
        if keywords:
            self.keywords_set.update(keywords)

    async def navigate_element_by_steps(self, element, parent_steps=0, prev_sibling_steps=0, next_sibling_steps=0):
        """
        주어진 `element`에서 부모, 이전 형제, 또는 다음 형제로 지정된 횟수만큼 이동하여 ElementHandle 반환.
        
        Args:
            element (ElementHandle): 탐색 기준이 되는 요소.
            parent_steps (int): 부모로 이동할 횟수 (0이면 이동하지 않음).
            prev_sibling_steps (int): 이전 형제로 이동할 횟수 (0이면 이동하지 않음).
            next_sibling_steps (int): 다음 형제로 이동할 횟수 (0이면 이동하지 않음).
        
        Returns:
            ElementHandle or None: 탐색된 요소를 반환하거나 없으면 None.
        """
        try:
            current_element = element
    
            # 부모로 이동
            for _ in range(parent_steps):
                current_element = await current_element.evaluate_handle('el => el.parentElement')
                if not current_element:
                    print("Parent element not found during navigation.")
                    return None
    
            # 이전 형제로 이동
            for _ in range(prev_sibling_steps):
                current_element = await current_element.evaluate_handle('el => el.previousElementSibling')
                if not current_element:
                    print("Previous sibling element not found during navigation.")
                    return None
    
            # 다음 형제로 이동
            for _ in range(next_sibling_steps):
                current_element = await current_element.evaluate_handle('el => el.nextElementSibling')
                if not current_element:
                    print("Next sibling element not found during navigation.")
                    return None
    
            return current_element
    
        except Exception as e:
            print(f"Error navigating elements: {e}")
            return None

    async def get_last_sibling(self, element):
        try:
            # 같은 부 아 마지막 형제를 가져오는 JavaScript 실행
            last_sibling = await element.evaluate_handle('el => el.parentElement.lastElementChild')
            return last_sibling
        except Exception as e:
            print(f"Error getting last sibling: {e}")
            return None

    async def behavior_name(self, element):
        parent_prev_sibling = await self.navigate_element_by_steps(element, parent_steps=1, prev_sibling_steps=1, next_sibling_steps=0)
        passive_name_element = await parent_prev_sibling.query_selector('span.block')
        if passive_name_element:
            passive_name = await passive_name_element.inner_text()
            return passive_name
        else:
            #print("Passive name element not found")
            return None

    async def behavior_condition(self, element):
        try:
            conditions = []
            parent_prev_sibling = await self.navigate_element_by_steps(element, parent_steps=1, prev_sibling_steps=1, next_sibling_steps=0)
            img_elements = await parent_prev_sibling.query_selector_all('img')
            if not img_elements:
                #print("No <img> elements found.")
                return conditions
    
            for img_element in img_elements:
                # 각 이미지의 src 속성 가져오기
                img_src = await img_element.get_attribute('src')
                #print(img_src)
    
                sin_name = None
                for color_key in resource_mapping.keys():
                    if f"{self.img_url}/ui/attr_{color_key}.png" in img_src:
                        sin_name = resource_mapping[color_key]
                        break
    
                if not sin_name:
                    #print(f"No matching Sin Color found for {img_src}.")
                    continue  # No Sin Color found, skip this img_element
    
                # 해당 이미지와 관련된 조건 텍스트 추출
                # 두 번째 부모 요소에서 시작하여 다음 형제를 찾아 조건 텍스트를 추출
                condition_element = await self.navigate_element_by_steps(img_element, parent_steps=3, prev_sibling_steps=0, next_sibling_steps=0)
                if condition_element:
                    # condition_element가 None이 아닌 경우에만 진행
                    condition_text_element = await condition_element.query_selector('.q-pl-xs')
                    if condition_text_element:
                        condition_text = await condition_text_element.inner_text()
                        condition_text = condition_text.strip()
                    else:
                        print(f"No condition text found for {img_src}.")
                        condition_text = ""  # condition_text_element가 None이면 빈 문자열로 처리
                else:
                    print(f"No condition element found for {img_src}.")
                    condition_text = ""  # condition_element가 None이면 빈 문자열로 처리
    
                # 결과를 배열에 추가
                conditions.append(f"{sin_name} {condition_text}")
            return conditions
        except Exception as e:
            print(f"Error extracting conditions: {e}")
            return 
            
    async def behavior_description(self, element, tag_type):
        parent_prev_sibling = await self.navigate_element_by_steps(
            element, parent_steps=1, prev_sibling_steps=1, next_sibling_steps=0
        )
        return await parent_prev_sibling.evaluate(f"""
            (node) => {{
                const behavior_title_node = node.querySelector('.q-pb-sm');
                const behavior_title_parent = behavior_title_node.parentElement;
                const behavior_description = behavior_title_parent.lastElementChild;
                if (!behavior_description) return null;
    
                const results = {{
                    keywords: [], // 키워드 저장
                    description: [] // 설명 저장
                }};
                
                // 매핑을 저장할 객체
                const keywordElements = behavior_description.querySelectorAll('a[href^="/en/keyword/"]');
                if (!keywordElements) return null;
                
                // 키워드 매핑 객체 초기화
                if (!results.keywordMapping) {{
                    results.keywordMapping = {{}}; // 텍스트: 키 형태로 저장
                }}
                
                keywordElements.forEach(a => {{
                    const span = a.querySelector('span.q-btn__content.text-center.col.items-center.q-anchor--skip.justify-center.row>span');
                    if (span) {{
                        const keywordText = span.textContent.trim(); // UI에 표시된 텍스트 (예: "Ammo")
                        const href = a.getAttribute('href'); // href 속성 값 (예: "/en/keyword/Bullet")
                        const keywordKey = href.split('/').pop(); // href에서 마지막 부분 추출 (예: "Bullet")
                
                        if (keywordKey && !Object.values(results.keywordMapping).includes(keywordKey)) {{
                            results.keywordMapping[keywordText] = keywordKey; // 텍스트: 키로 매핑
                        }}
                    }}
                }});

                // 2. 설명 추출
                const descriptionElements = behavior_description.querySelectorAll("{tag_type}");
                let currentEffect = []; // 초기화
                let keywordsList = new Set(Object.values(results.keywordMapping)); // keywordMapping 값으로 Set 생성
            
                descriptionElements.forEach(span => {{
                    let text = span.textContent?.trim();
                    if (!text) return;
            
                    // 공백과 기호 처리
                    text = text.replace(/\\s*([.,:;!?()\\[\\]])\\s*/g, '$1');
            
                    // 텍스트가 매핑된 키워드인지 확인
                    if (results.keywordMapping[text]) {{
                        const keywordEffect = `keyword:${{results.keywordMapping[text]}}`; // 매핑된 키 사용
                        keywordsList.add(results.keywordMapping[text]); // keywordsList에 추가
                        if (!currentEffect.includes(keywordEffect)) {{
                            currentEffect.push(keywordEffect);
                        }}
                    }} else {{
                        currentEffect.push(text);
                    }}
                }});
            
                // keywords 배열로 변환
                results.keywords = Array.from(keywordsList); // Set에서 배열로 변환하여 저장
    
                // 마지막 효과 저장
                if (currentEffect.length > 0) {{
                    const finalDescription = currentEffect.join(' ');
                    //console.log("After process:", finalDescription);
                    results.description.push(finalDescription);
                }}
                delete results.keywordMapping;
                let processedText = results.description
                    .map(word => word.trim())
                    .join(" ")
                    .replace(/\\s+([,.:])/g, "$1")
                    .replace(/\\s+'/g, "'")
                    .replace(/'\\s+/g, "'")
                    .replace(/\\[\\s+/g, "[")
                    .replace(/\\s+\\]/g, "]")
                    .replace(/\\(\\s+/g, "(")
                    .replace(/\\s+\\)/g, ")")
                    .replace(/\\s+%/g, "%")
                    .replace(/\\s+\\//g, "/")
                    .replace(/\\/\\s+/g, "/")
                    .replace(/\\s+\\/\\s+/g, "/");
                results.description = processedText;
                return results;
            }}
        """)

    async def crawl_keyword_info(self, page, keyword):
        # 키워드 설명 추출
        keyword_explain_elements = await page.query_selector_all('div.text-caption.text-grey > div')
        keyword_explains = []
        keyword_explain_text = np.nan
        for keyword_explain_element in keyword_explain_elements:
            keyword_explain_text = await keyword_explain_element.inner_text()
            keyword_explain_text = keyword_explain_text.strip()
            keyword_explains.append(keyword_explain_text)
    
        # Dispellable 값과 그 이후 텍스트 추출
        grand_parent = await self.navigate_element_by_steps(
            keyword_explain_elements[0],
            parent_steps=2, 
            prev_sibling_steps=0, 
            next_sibling_steps=0
        )
        
        # Dispellable 요소 찾기
        dispellable_element = await grand_parent.query_selector('div.q-pt-sm > div')
        dispellable_value = None
        is_dispellable = np.nan  # 초기값을 np.nan으로 설정

        if dispellable_element:
            dispellable_value = await dispellable_element.inner_text()
            dispellable_value = dispellable_value.strip()
            
            # Dispellable 값 처리
            if ':' in dispellable_value:
                dispellable_state = dispellable_value.split(':', 1)[1].strip().lower()
                if dispellable_state == 'true':
                    is_dispellable = True
                elif dispellable_state == 'false':
                    is_dispellable = False
                else:
                    is_dispellable = np.nan  # 'true'나 'false'가 아닌 경우 np.nan으로 설정
        
            # 결과 출력
            #print(f"Dispellable: {is_dispellable}, After Colon Text: {after_colon_text}")
        
        # 컨테이너에 저장
        self.container.add_keyword(keyword, keyword_explains, is_dispellable)

class Identity:
    """인격 정보를 저장하는 클래스"""
    def __init__(self, identity_id, name=None, rarity=None,\
                 skill_1_type=None, skill_1_sin_affinity=None,\
                 skill_2_type=None, skill_2_sin_affinity=None,\
                 skill_3_type=None, skill_3_sin_affinity=None,\
                 defense_type=None, defense_sin_affinity=None):
        self.identity_id = identity_id
        self.name = name
        self.rarity = rarity
        # 선언만 하는 부분 
        self.skill_1_type = skill_1_type 
        self.skill_1_sin_affinity = skill_1_sin_affinity 
        self.skill_2_type = skill_2_type 
        self.skill_2_sin_affinity = skill_2_sin_affinity 
        self.skill_3_type = skill_3_type 
        self.skill_3_sin_affinity = skill_3_sin_affinity 
        self.defense_type = defense_type 
        self.defense_sin_affinity = defense_sin_affinity
        self.level_max = 50
        self.uptie_max = 4
        self.level_stats = []  # LevelStat 객체 리스트
        self.uptie_stats = [] # UptieStat 객체 리스트
        self.resist_info = None
        self.stagger_threshold = None
        self.skillset = SkillSet()  # Skill정보를 담고 있는 skillset 객체 
        self.passive_infos = []  # 패시브 객체를 담고 있는 배열
        self.support_passive_infos = [] # 서포트 패시 객체를 담고 있는 배열
        self.panic_type = [] # 혼란상태 발생시 나타나는 효과를 담고 있는 객체 배열1

    def update_name(self, name):
        self.name = name

    def update_rarity(self, rarity):
        self.rarity = rarity

    def add_level_stat(self, level, hp, defense):
        self.level_stats.append(LevelStat(level, hp, defense))

    def add_uptie_stat(self, uptie, speed):
        self.uptie_stats.append(UptieStat(uptie, speed))

    def add_resist_info(self, resist_hit, resist_penetrate, resist_slash):
        self.resist_info = ResistInfo(resist_hit, resist_penetrate, resist_slash)

    def add_stagger_threshold(self, first_stagger, second_stagger, third_stagger):
        self.stagger_threshold = StaggerThreshold(first_stagger, second_stagger, third_stagger)

    def add_skill(self, skill_code, skill_name, coin_count, init_level, init_remain, atk_weight):
        self.skillset.add_skill(skill_code, Skill(skill_code, skill_name, coin_count, init_level, init_remain, atk_weight))
        
    def add_skill_uptie_info(self, skill_name, uptie_level, skill_power, coin_power, effect_by_condition):
        self.skillset.add_skill_uptie_info(skill_name, uptie_level, skill_power, coin_power, effect_by_condition)

    def add_influence_behavior(self, behavior_type, behavior_name, condition, description):
        if behavior_type == "passive":
            self.passive_infos.append(Passive(behavior_type, behavior_name, condition, description))
        elif behavior_type == "support passive":
            self.support_passive_infos.append(SupportPassive(behavior_type, behavior_name, condition, description))
        elif behavior_type == "panic type":
            self.panic_type.append(PanicType(behavior_type, behavior_name, condition, description))
        
    def update_details(self, page):
        # 세부 정보 업데이트 로직 (가정)
        pass
    #def init_uptie(self, page):
    #    init_uptie = 

# LevelStat 객체 정의
class LevelStat:
    def __init__(self, level, hp=None, defense=None):
        self.level = level
        self.hp = hp
        self.defense = defense
    def update_level(self, level):
        self.level = level
    def update_hp(self, hp):
        self.hp = hp
    def update_defense(self, defense):
        self.defense = defense

# UptieStat 객체 정의
class UptieStat:
    def __init__(self, uptie, speed=None):
        self.uptie = uptie
        self.speed = speed
    def update_speed(self, speed):
        self.speed = speed

# ResistInfo 객체 정의
class ResistInfo:
    def __init__(self, resist_hit=None, resist_penetrate=None, resist_slash=None):
        self.resist_hit = resist_hit
        self.resist_penetrate = resist_penetrate
        self.resist_slash = resist_slash

# ResistInfo 객체 정의
class StaggerThreshold:
    def __init__(self, first_stagger=None, second_stagger=None, third_stagger=None):
        self.first_stagger = first_stagger
        self.second_stagger = second_stagger
        self.third_stagger = third_stagger

class SkillSet:
    def __init__(self):

        self.skill_1_type = ""
        self.skill_2_type = ""
        self.skill_3_type = ""
        self.defense_skill_type = ""

        self.skill_1_sin_affinity = ""
        self.skill_2_sin_affinity = ""
        self.skill_3_sin_affinity = ""
        self.defense_skill_sin_affinity = ""

        self.skill_1_details = []
        self.skill_2_details = []
        self.skill_3_details = []
        self.defense_skill_details = []

    def set_skill_type(self, skill_code, skill_type, sin_affinity):
        if skill_code == "skill 1":
            self.skill_1_type = skill_type
            self.skill_1_sin_affinity = sin_affinity
        elif skill_code == "skill 2":
            self.skill_2_type = skill_type
            self.skill_2_sin_affinity = sin_affinity
        elif skill_code == "skill 3":
            self.skill_3_type = skill_type
            self.skill_3_sin_affinity = sin_affinity
        elif skill_code == "defense":
            self.defense_skill_type = skill_type
            self.defense_skill_sin_affinity = sin_affinity
    
    def add_skill(self, skill_code, skill):
        if skill_code == "skill 1":
            self.skill_1_details.append(skill)
            skill.skill_type = self.skill_1_type
            skill.sin_affinity = self.skill_1_sin_affinity
        elif skill_code == "skill 2":
            self.skill_2_details.append(skill)
            skill.skill_type = self.skill_2_type
            skill.sin_affinity = self.skill_2_sin_affinity
        elif skill_code == "skill 3":
            self.skill_3_details.append(skill)
            skill.skill_type = self.skill_3_type
            skill.sin_affinity = self.skill_3_sin_affinity
        elif skill_code == "defense":
            self.defense_skill_details.append(skill)
            skill.skill_type = self.defense_skill_type 
            skill.sin_affinity = self.defense_skill_sin_affinity

    def get_skill_by_name(self, skill_name):
        for skill_list in [self.skill_1_details, self.skill_2_details, self.skill_3_details, self.defense_skill_details]:
            for skill in skill_list:
                result = skill.is_skill_by_name(skill_name)
                if result:
                    return result
        return None

    def add_skill_uptie_info(self, skill_name, uptie_level, skill_power, coin_power, effect_by_condition):
        skill = self.get_skill_by_name(skill_name)
        skill.add_uptie_info(uptie_level, skill_power, coin_power, effect_by_condition)
        

class Skill:
    def __init__(self, skill_code, skill_name, coin_count, init_level, init_remain, atk_weight):
        self.skill_code = skill_code # "Skill 1"/"Skill 2"/"Skill 3"/"Defense" '''o'''
        self.skill_type = None
        self.sin_affinity = None
        self.skill_name = skill_name # 스킬 이름 '''o'''
        self.coin_count = coin_count # 동전 개수 '''o'''
        self.init_level = init_level # 스킬이 초기 배정받은 레벨, 인격 레벨에 - 1 한 값을 더해주면 스킬의 현 level을 구할 수 있다. '''o'''
        self.init_remain = init_remain #'''o'''
        self.atk_weight = atk_weight #공격 가중치 값이다. 방어스킬에는 없기 때문에 None으로 설정했다. '''o'''
        self.remain_count = init_remain # 실제 스킬이 작동하며 횟수가 차감되 나중에 모두 차감되면 다음 턴에 다시 초기횟수 리필된다.
        self.uptie_info = []  

    def is_skill_by_name(self, skill_name):
        if self.skill_name == skill_name:
            return self
        return None
        
    def add_uptie_info(self, uptie_level, skill_power, coin_power, effects_by_condition):
        self.uptie_info.append(UptieInfo(uptie_level, skill_power, coin_power, effects_by_condition))

class UptieInfo:
    def __init__(self, uptie_level, skill_power, coin_power, effects_by_condition):
        self.uptie_level = uptie_level   #'''o'''
        self.skill_power = skill_power   #'''o'''
        self.coin_power = coin_power     #'''o'''
        self.skill_effect = effects_by_condition

# 추상화된 EffectType 클래스
class EffectType:
    def __init__(self, behavior_type, name, description):
        self.behavior_type = behavior_type
        self.name = name  # Effect 이름
        self.description = description  # Effect 설명

# Passive 클래스
class Passive(EffectType):
    def __init__(self, behavior_type, name, condition, description):
        super().__init__(behavior_type, name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)  # condition이 있는지 여부 확인

# SupportPassive 클래스
class SupportPassive(EffectType):
    def __init__(self,behavior_type, name, condition, description):
        super().__init__(behavior_type, name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)

# PanicType 클래스
class PanicType(EffectType):
    def __init__(self, behavior_type, name, condition, description):
        super().__init__(behavior_type, name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)

# Keyword 클래스
class Keyword:
    def __init__(self, keyword, explain=None, dispellable=None):
        self.keyword = keyword
        self.explain = explain
        self.dispellable = dispellable

      
class DataContainer:
    """크롤링 데이터를 관리하는 클래스"""
    def __init__(self):
        self.identities = []
        self.keywords = []

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.level_max

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.uptie_max

    def update_identity_name(self, identity_id, identity_name):
        """특정 ID의 이름을 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_name(identity_name)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_identity_rarity(self, identity_id, rarity):
        """특정 ID의 희귀도를 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_rarity(rarity)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_skill(self, identity_id, skill_number, new_type):
        """
        특정 identity의 skill_type을 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_type: 새 skill_type
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_type = new_type
                elif skill_number == 2:
                    identity.skill_2_type = new_type
                elif skill_number == 3:
                    identity.skill_3_type = new_type
                elif skill_number == "defense":
                    identity.defense_type = new_type
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def update_sin_affinity(self, identity_id, skill_number, new_affinity):
        """
        특정 identity의 sin_affinity를 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_affinity: 새 sin_affinity
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_sin_affinity = new_affinity
                elif skill_number == 2:
                    identity.skill_2_sin_affinity = new_affinity
                elif skill_number == 3:
                    identity.skill_3_sin_affinity = new_affinity
                elif skill_number == "defense":
                    identity.defense_sin_affinity = new_affinity
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def add_identity(self, identity):
        """새로운 인격 데이터를 추가."""
        self.identities.append(identity)

    def add_level_stat(self, identity_id, level, hp, defense):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_level_stat(level, hp, defense)

    def add_uptie_stat(self, identity_id, uptie, speed):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_uptie_stat(uptie, speed)

    def add_resist_info(self, identity_id, resist_hit, resist_penetrate, resist_slash):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_resist_info(resist_hit, resist_penetrate, resist_slash)

    def add_stagger_threshold(self, identity_id, first_stagger, second_stagger, third_stagger):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_stagger_threshold(first_stagger, second_stagger, third_stagger)

    
    def add_skill(self, identity_id, skill_code, skill_name, coin_count, init_level, init_remain, atk_weight):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_code == "skill 1":
                    identity.skillset.set_skill_type(skill_code, identity.skill_1_type, identity.skill_1_sin_affinity)
                elif skill_code == "skill 2":
                    identity.skillset.set_skill_type(skill_code, identity.skill_2_type, identity.skill_2_sin_affinity)
                elif skill_code == "skill 3":
                    identity.skillset.set_skill_type(skill_code, identity.skill_3_type, identity.skill_3_sin_affinity)
                elif skill_code == "defense":
                    identity.skillset.set_skill_type(skill_code, identity.defense_type, identity.defense_sin_affinity)
                identity.add_skill(skill_code, skill_name, coin_count, init_level, init_remain, atk_weight)

    def add_skill_uptie_info(self, identity_id, skill_name, uptie_level, skill_power, coin_power, effects_by_condition):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_skill_uptie_info(skill_name, uptie_level, skill_power, coin_power, effects_by_condition)

    def add_influence_behavior(self, identity_id, behavior_type, behavior_name, condition, description):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_influence_behavior(behavior_type, behavior_name, condition, description)

    def add_keyword(self, keyword, explain, dispellable):
        self.keywords.append(Keyword(keyword, explain, dispellable))
        
    def display_identities(self):
        """Identity, ID, Name, Rarity 정보를 출력"""
        if not self.identities:
            print("No identities found.")
        else:
            # 데이터를 리스트로 변환
            table = []
            for idx, identity in enumerate(self.identities, 1):
                table.append([
                    idx, identity.identity_id, identity.name, identity.rarity,
                    identity.skill_1_type, identity.skill_1_sin_affinity,
                    identity.skill_2_type, identity.skill_2_sin_affinity,
                    identity.skill_3_type, identity.skill_3_sin_affinity,
                    identity.defense_type, identity.defense_sin_affinity
                ])
            
            # 테이블 헤더 정의
            headers = [
                "Index", "ID", "Name", "Rarity",
                "Skill_1 Type", "Skill_1 Sin", "Skill_2 Type", "Skill_2 Sin",
                "Skill_3 Type", "Skill_3 Sin", "Defense Type", "Defense Sin"
            ]
            
            # 테이블 출력
            print(tabulate(table, headers=headers, tablefmt="fancy_grid"))

    def export_to_csv(self, filename, fieldnames, data):
        """
        공통적으로 사용 가능한 CSV 내보내기 메서드
        :param filename: 내보낼 파일 이름 (예: 'identity.csv', 'skill.csv')
        :param fieldnames: CSV의 헤더 리스트
        :param data: CSV로 내보낼 데이터 (리스트 형태)
        """
        try:
            # 현재작업 디렉토 기준으 'csv'폴더 없으면 생성
            if not os.path.exists('./csv'):
                os.makedirs('./csv')
            # 파일 경로 설정 (상대경로 'csv' 폴더 안에 저장)
            file_path = os.path.join('./csv', filename)
            with open(file_path, mode='w', newline='', encoding='utf-8') as file:
                writer = csv.DictWriter(file, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(data)
            print(f"Data successfully exported to {filename}")
        except Exception as e:
            print(f"Error exporting to CSV: {e}")

    def get_identity_data(self):
        """
        Identity 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "ID": identity.identity_id,
                "Name": identity.name,
                "Rarity": identity.rarity,
                "Skill_1_Type": identity.skill_1_type,
                "Skill_1_Sin_Affinity": identity.skill_1_sin_affinity,
                "Skill_2_Type": identity.skill_2_type,
                "Skill_2_Sin_Affinity": identity.skill_2_sin_affinity,
                "Skill_3_Type": identity.skill_3_type,
                "Skill_3_Sin_Affinity": identity.skill_3_sin_affinity,
                "Defense_Type": identity.defense_type,
                "Defense_Sin_Affinity": identity.defense_sin_affinity,
            }
            for identity in self.identities
        ]
        
    def export_identities_data(self, filename="identities.csv"):
        """Identity 데이터를 CSV로 내보내는 메서드"""
        identity_fieldnames = [
            "ID", "Name", "Rarity",
            "Skill_1_Type", "Skill_1_Sin_Affinity",
            "Skill_2_Type", "Skill_2_Sin_Affinity",
            "Skill_3_Type", "Skill_3_Sin_Affinity",
            "Defense_Type", "Defense_Sin_Affinity"
        ]
        identity_data = self.get_identity_data()  # Identity 데이터 가져오기
        self.export_to_csv(filename, identity_fieldnames, identity_data)  # CSV로 내보내기

    def load_identities_by_id(self, filename, identity_ids):
        """지정된 identity_ids 목록에 해당하는 Identity 데이터를 CSV에서 읽어옴."""
        identity_ids_set = set(identity_ids) # 중복 제거를 위 Set 사용
        with open(filename, mode='r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                if row["ID"] in identity_ids_set:
                    identity = Identity(
                        identity_id=row["ID"],
                        name=row["Name"],
                        rarity=row["Rarity"],
                        skill_1_type=row["Skill_1_Type"],
                        skill_1_sin_affinity=row["Skill_1_Sin_Affinity"],
                        skill_2_type=row["Skill_2_Type"],
                        skill_2_sin_affinity=row["Skill_2_Sin_Affinity"],
                        skill_3_type=row["Skill_3_Type"],
                        skill_3_sin_affinity=row["Skill_3_Sin_Affinity"],
                        defense_type=row["Defense_Type"],
                        defense_sin_affinity=row["Defense_Sin_Affinity"]
                    )
                    self.identities.append(identity)

    def get_level_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Level": level_stat.level,
                "HP": level_stat.hp,
                "Defense": level_stat.defense,
            }
            for identity in self.identities
            for level_stat in identity.level_stats
        ]

    def export_level_stats_data(self, filename="levelstats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        level_stats_fieldnames = ["Identity_ID", "Level", "HP", "Defense"]
        level_stats_data = self.get_level_stats_data()  # level_stats 데이터 가져오기
        self.export_to_csv(filename, level_stats_fieldnames, level_stats_data)  # CSV로 내보내기

    def get_uptie_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Uptie": uptie_stat.uptie,
                "Speed": uptie_stat.speed,
            }
            for identity in self.identities
            for uptie_stat in identity.uptie_stats
        ]

    def export_uptie_stats_data(self, filename="Uptiestats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        uptie_stats_fieldnames = ["Identity_ID", "Uptie", "Speed"]
        uptie_stats_data = self.get_uptie_stats_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, uptie_stats_fieldnames, uptie_stats_data)  # CSV로 내보내기

    def get_resist_info_data(self):
        """
        모든 Identity의 resist_info 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Resist_Hit": identity.resist_info.resist_hit,
                "Resist_Penetrate": identity.resist_info.resist_penetrate,
                "Resist_Slash": identity.resist_info.resist_slash,
            }
            for identity in self.identities
        ]

    def export_resist_info_data(self, filename="ResistInfos.csv"):
        """ResistInfo 데이터를 CSV로 내보내는 메서드"""
        resist_infos_fieldnames = ["Identity_ID", "Resist_Hit", "Resist_Penetrate", "Resist_Slash"]
        resist_infos_data = self.get_resist_info_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, resist_infos_fieldnames, resist_infos_data)  # CSV로 내보내기

    def export_uptie_stats_data(self, filename="Uptiestats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        uptie_stats_fieldnames = ["Identity_ID", "Uptie", "Speed"]
        uptie_stats_data = self.get_uptie_stats_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, uptie_stats_fieldnames, uptie_stats_data)  # CSV로 내보내기

    def get_stagger_threshold_data(self):
        """
        모든 Identity의 stagger_threshold 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "First_Stagger": identity.stagger_threshold.first_stagger,
                "Second_Stagger": identity.stagger_threshold.second_stagger,
                "Third_Stagger": identity.stagger_threshold.third_stagger,
            }
            for identity in self.identities
        ]

    def export_stagger_threshold_data(self, filename="StaggerThresholds.csv"):
        """StaggerThresholds 데이터를 CSV로 내보내는 메서드"""
        stagger_thresholds_fieldnames = ["Identity_ID", "First_Stagger", "Second_Stagger", "Third_Stagger"]
        stagger_thresholds_data = self.get_stagger_threshold_data()  # stagger_thresholds 데이터 가져오기
        self.export_to_csv(filename, stagger_thresholds_fieldnames, stagger_thresholds_data)  # CSV로 내보내기

    def get_skillset_data(self):
        """모든 Identity의 skillset 데이터를 export_to_csv을 포맷으로 변환"""
        return [
            {
                "Identity_ID": identity.identity_id,
                "Skill_Code": skill.skill_code,
                "Skill_Type": skill.skill_type,
                "Sin_Affinity": skill.sin_affinity,
                "Skill_Name": skill.skill_name,
                "Coin_Count": skill.coin_count,
                "Init_Level": skill.init_level,
                "Init_Remain": skill.init_remain,
                "Atk_Weight": skill.atk_weight,
                "Uptie_Level": uptie_info.uptie_level,
                "Skill_Power": uptie_info.skill_power,
                "Coin_Power": uptie_info.coin_power,
                "Effect_Data": json.dumps(uptie_info.skill_effect)
            }
            for identity in self.identities
            for skill_list in [identity.skillset.skill_1_details, identity.skillset.skill_2_details, identity.skillset.skill_3_details, identity.skillset.defense_skill_details]
            for skill in skill_list
            for uptie_info in skill.uptie_info
        ]

    def export_identity_skillset_data(self, filename="IdentitiesSkills.csv"):
        """IdentitiesSkills 데이터를 CSV로 내보내는 메서드"""
        identities_skills_fieldnames = ["Identity_ID", "Skill_Code", "Skill_Type", "Sin_Affinity", "Skill_Name", "Coin_Count", "Init_Level", "Init_Remain", "Atk_Weight", "Uptie_Level", "Skill_Power", "Coin_Power", "Effect_Data"]
        identities_skills_data = self.get_skillset_data()
        self.export_to_csv(filename, identities_skills_fieldnames, identities_skills_data)

    def get_identities_behaviors_data(self):
        """모든 Identity의 influence behavior 데이터를 export_to_csv 포맷으로 변환"""
        return [
            {
                "Identity_ID": identity.identity_id,
                "Effect_Type": effect_type,
                "Behavior_Name": effect.name,
                "Behavior_Condition": effect.condition if effect.condition else np.nan,
                "Behavior_Description" : effect.description
            }
            for identity in self.identities
            for effect_list, effect_type in [(identity.passive_infos, 'Passive'), (identity.support_passive_infos, 'Support Passive'), (identity.panic_type, 'Panic Type')]
            for effect in effect_list
        ]

    def export_identities_behaviors_data(self, filename="IdentitiesBehaviors.csv"):
        """IdentitiesBehaviors 데이터를 CSV로 내보내는 메서드"""
        identities_behaviors_fieldnames = ["Identity_ID","Effect_Type","Behavior_Name","Behavior_Condition","Behavior_Description"]
        identities_behaviors_data = self.get_identities_behaviors_data()
        self.export_to_csv(filename, identities_behaviors_fieldnames, identities_behaviors_data)

    def get_keywords_data(self):
        """DataContainer의 keywords 데이터를 export_to_csv 포맷으로 변환"""
        return [
            {
                "Keyword": keyword.keyword,
                "Explain": keyword.explain,
                "Dispellable": keyword.dispellable,
            }
            for keyword in self.keywords
        ]

    def export_keywords_data(self, filename="Keywords.csv"):
        """DataContainer의 keywords 데이터를 CSV로 내보내는 메서드"""
        keywords_fieldnames = ["Keyword","Explain","Dispellable"]
        keywords_data = self.get_keywords_data()
        self.export_to_csv(filename, keywords_fieldnames, keywords_data)

            
async def main():
    data_container = DataContainer()
    scraper = Scraper(container=data_container, base_url="https://limbus.kusoge.xyz", img_url="https://img.kusoge.xyz/limbus/img")
    await scraper.scrape()

await main() 

Data successfully exported to Keywords.csv


In [20]:
import os
import csv
import re
import json
import numpy as np
import pandas as pd
from playwright.async_api import async_playwright
from tabulate import tabulate

# Sin Color 이름 변환을 위한 매핑
resource_mapping = {
    "AZURE": "Gloom",
    "VIOLET": "Envy",
    "AMBER": "Sloth",
    "SHAMROCK": "Gluttony",
    "CRIMSON": "Wrath",
    "SCARLET": "Lust",
    "INDIGO": "Pride"
}

def convert_resource(resource_name):
    """Sin Color 이름을 변환하는 함수. 매핑되지 않은 경우 원래 이름을 반환."""
    return resource_mapping.get(resource_name, resource_name)

def find_sin_for_color(url):
    # URL에 color_name을 추출
    color_name = url.split('/')[-1].split('.')[0].replace("attr_", "")
    # 해당 색상이 resource_mapping에 있는 확인
    for color, sin in resource_mapping.items():
        if color_name in color:
            return sin
    return None


class Scraper:
    """웹 페이지 크롤링을 위한 클래스"""
    def __init__(self, container, base_url, img_url):
        self.container = container
        self.base_url = base_url
        self.img_url = img_url
        self.uptie_level = 1
        self.keywords_set = set()

    async def handle_request(self, route, request):
        """이미지와 폰트 리소스 로드를 차단."""
        if request.resource_type =="image":
            #print(f"{request.url}")
            img_url = request.url
            if  re.match(f"{self.img_url}/ui/attr.*\\.png", img_url):
                await route.continue_()
            else:
                await route.fulfill(
                    status=200,
                    content_type="image/png",
                    body=b""
                )
        elif request.resource_type == "font":
            await route.abort()
        else:
            await route.continue_()
            
    async def scrape(self):
        """크롤링 메인 함수"""
        async with async_playwright() as p:
            browser = await p.chromium.launch(headless=False)
            page = await browser.new_page()
            #await page.evaluate('window.scrollTo(0, document.body.scrollHeight)')
            await page.route("**/*", self.handle_request)
            await page.wait_for_load_state("domcontentloaded")
            # Step 1: /identity 페이지에서 기본 정보 크롤링
            #await self.crawl_identity_page(page)

            # Step 2: /identity/{identity_id} 상세 페이지 크롤링
            #identity_ids_to_load = ["10104", "10310", "10710","10808", "10911", "11005"]
            #identity_ids_to_load = ["10104", "10911"]#,"10310", "10710","10808", "11005", "11210"]
            #identity_ids_to_load = ["10911","10310"]#, "10710","10808", "11005", "11210"]
            #identity_ids_to_load = ["10911", "10611", "10711"]#, "10710","10808", "11005", "11210"]
            #self.container.load_identities_by_id('./csv/identities.csv', identity_ids_to_load)
            self.container.load_all_identities('./csv/identities.csv')
            await self.crawl_identity_details_for_all(page)
            #self.container.display_identities()
            #self.container.export_identities_data("Identities.csv")
            self.container.export_level_stats_data("Levelstats.csv")
            self.container.export_uptie_stats_data("Uptiestats.csv")
            self.container.export_resist_info_data("ResistInfos.csv")
            self.container.export_stagger_threshold_data("StaggerThresholds.csv")
            self.container.export_identity_skillset_data("IdentitiesSkills.csv")
            self.container.export_identities_behaviors_data("IdentitiesBehaviors.csv")
            await self.crawl_keywords_details_for_collect(page)
            self.container.export_keywords_data("Keywords.csv")
            page.set_default_timeout(10000)
            #await page.wait_for_timeout(100000)
            await browser.close()

    async def get_identity_id_from_row(self, row):
        """테이블 행(row)에서 인격 ID를 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_url = await identity_cell.get_attribute('href')
            identity_id = identity_url.split('/identity/')[-1]
            return identity_id
        return "Unknown"

    async def get_identity_name_from_row(self, row):
        """테이블 행(row)에서 인격 이름을 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_name = (await identity_cell.inner_text()).replace("\n", " ").strip()
            return identity_name
        return "Unknown"
        
    async def crawl_identity_page(self, page):
        """/identity 페이지로 이동하여 인격 기본 정보를 추출."""
        await page.goto(self.base_url + "/identity")
        await self.extract_identity_page(page)

    async def extract_identity_page(self, page):
        """/identity 페이지의 모든 테이블 행에서 인격 정보를 추출."""
        identity_rows = await page.query_selector_all('tr')
        if len(identity_rows) > 1:
            for idx, identity_row in enumerate(identity_rows):
                if idx == 0:
                    continue # 0번째요소이름을 보여주는 열은 건너뜀
                await self.scrape_identity_name_and_identity_id(identity_row)
                await self.extract_rarity(identity_row)
                for i in range(1, 4):  # 스킬1~3 정보 추출
                    await self.extract_atk_and_resource(identity_row, i)
                await self.extract_defense_and_resource(identity_row)

    async def scrape_identity_name_and_identity_id(self, row):
        """인격 번호와 이름을 추출하여 DataContainer에 저장."""
        try:
            identity_id = await self.get_identity_id_from_row(row)
            identity_name = await self.get_identity_name_from_row(row)
            identity = Identity(identity_id)
            self.container.add_identity(identity)
            self.container.update_identity_name(identity_id, identity_name)
        except Exception as e:
            print(f"Error extracting identity name and identity id: {e}")

    async def get_element_attribute(self, element, selector, attribute):
        """요소에서 지정된 속성(attribute)을 추출."""
        try:
            target_element = await element.query_selector(selector)
            if target_element:
                return await target_element.get_attribute(attribute)
        except Exception as e:
            print(f"Error getting element attribute: {e}")
        return None
    
    async def extract_rarity(self, row):
        """테이블 행(row)에서 희귀도를 추출하여 업데이트."""
        try:
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
                sibling = await parent.evaluate_handle('node => node.nextElementSibling')
                if sibling:
                    rarity_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/rarity_"]', 'src')
                    if rarity_img_src:
                        rarity_level = rarity_img_src.split('_')[-1].split('.')[0]  # rarity_1.png -> 1
                        identity_id = await self.get_identity_id_from_row(row)
                        self.container.update_identity_rarity(identity_id, rarity_level)
                    else:
                        print("Rarity image not found in the next sibling TD.")
        except Exception as e: 
            print(f"Error extracting rarity: {e}")

        return "Unknown"

    async def extract_atk_and_resource(self, row, skill_number):
        """ 공격 타입(ATK Type)과 자원(Attribute) 추출 함수 - 스킬 번호에 따라 처리 """
        try:
            #print(f"Attempting to locate skill{skill_number} type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 해당 스킬 번호에 맞는 형제로 이동
                sibling = parent
                for _ in range(skill_number + 1):  # 스킬 번호에 맞게 이동
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                atk_type, resource = "Unknown", "Unknown"
        
                if sibling:
                    # 공격 타입 이미지 (atk_type) 찾기
                    atk_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/atk_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if atk_type_img_src:
                        atk_type = atk_type_img_src.split('_')[-1].split('.')[0]  # atk_type_SLASH.png -> SLASH
                        #print(f"Extracted attack type: {atk_type}")
                        self.container.update_skill(identity_id, skill_number, atk_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted resource: {resource}")
                        
                        # 리소스를 변환
                        resource = convert_resource(resource)
                        #print(f"Converted resource: {resource}")
                        self.container.update_sin_affinity(identity_id, skill_number, resource)
                        
        except Exception as e:
            print(f"Error extracting skill{skill_number} type and resource: {e}")

    async def extract_defense_and_resource(self, row):
        """ 방어 타입(Def Type)과 자원(Attribute) 추출 함수 """
        try:
            #print("Attempting to locate defense type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 다섯 번째 형제 td로 이동 (방어 관련 정보가 있는 <td>)
                sibling = parent
                for _ in range(5):  # 방어 정보는 5번째 형제에 있음
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                def_type, def_resource = "Unknown", "Unknown"
        
                if sibling:
                    # 방어 타입 이미지 (def_type) 찾기
                    def_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/def_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if def_type_img_src:
                        def_type = def_type_img_src.split('_')[-1].split('.')[0]  # def_type_GUARD.png -> GUARD
                        #print(f"Extracted defense type: {def_type}")
                        self.container.update_skill(identity_id, 'defense', def_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        def_resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted defense resource: {def_resource}")
                        
                        # 방어 리소스를 변환
                        def_resource = convert_resource(def_resource)
                        #print(f"Converted defense resource: {def_resource}")
                        self.container.update_sin_affinity(identity_id, 'defense', def_resource)
 
        except Exception as e:
            print(f"Error extracting defense type and resource: {e}")

    async def crawl_identity_details_for_all(self, page):
        """모든 인격의 세부 정보를 크롤링."""
        for identity in self.container.identities:
            await page.wait_for_load_state("load")
            await page.goto(self.base_url + "/identity/"+identity.identity_id)
            await self.initialize_uptie_and_level(page)
            #""" 레벨과 동기화와 관련 없는 참관타 저항은 밑에서 변경하는 것과 연관성이 적기 때문에 위에서 처리한다.
            print(f"Identity_id: {identity.identity_id}")
            results = []
            resist_type_imgs = [f"{self.img_url}/ui/atk_type_HIT.png",
                                f"{self.img_url}/ui/atk_type_PENETRATE.png",
                                f"{self.img_url}/ui/atk_type_SLASH.png"]
            for resist_type_img in resist_type_imgs:
                result = await self.extract_resist_type(page, resist_type_img, 1)
                results.append(result)
            self.container.add_resist_info(identity.identity_id, results[0], results[1], results[2])
            #"""
            #"""
            # Stagger Threshold 값 추출
            stagger_threshold = await self.extract_stagger_threshold_values(page)
            self.container.add_stagger_threshold(identity.identity_id, stagger_threshold[0], stagger_threshold[1], stagger_threshold[2])
            #"""
            #await self.initialize_uptie_and_level(page)
            #""" 레벨에 따른 level, hp, defense 다르게 되어 있는 것을 출력 확인 객체로 들어가서 csv로 옮기는 과정까지 진행
            for i in range(1, identity.level_max+1):
                await self.move_slider_to_value(page,i)
                await self.extract_level_status(page, identity.identity_id, i)
            #"""
            #"""
            #    인격에 대한 skill 정보를 가져오는 메소드 위에서 출력하는 csv파일로 인격이 가지고 있는 스킬 정보를 내보낸다.
            await self.crawl_speed_and_skills(page, identity.identity_id)
            #"""
            await self.crawl_influence_behavior(page, identity.identity_id)

    async def crawl_keywords_details_for_collect(self, page):
        """모아진 키워드의 세부 정보를 크롤링."""
        for keyword in self.keywords_set:
            await page.goto(self.base_url + "/keyword/"+keyword)
            await self.crawl_keyword_info(page, keyword)
            
    async def initialize_uptie_and_level(self, page):
        await self.uptie_click(page, 1)
        await self.move_slider_to_value(page, 1)
        self.uptie_level = 1
        #return 1
    
    async def uptie_click(self, page, phase):
        await page.wait_for_load_state("domcontentloaded")
        fab_button_selector = ".q-btn.q-btn--fab"
        await page.wait_for_selector(fab_button_selector)
        await page.click(fab_button_selector)
        #print(f"[{setup_uptie_button}] 상위 동기화 버튼 클릭 완료")
        setup_uptie_button = f'a:has(img[src="{self.img_url}/ui/uptie_{phase}.png"])'
        await page.wait_for_selector(setup_uptie_button)
        await page.click(setup_uptie_button)
        #print(f"{phase} 동기화 버튼 클릭 완료")

    async def move_slider_to_value(self, page, target_value):
        """
        슬라이더를 주어진 값으로 이동시킵니다.
        target_value는 슬라이더의 최소값부터 최대값 사이여야 합니다.
        """
        slider_selector = ".q-slider"
        # 슬라이더의 범위 확인
        min_value = int(await page.get_attribute(slider_selector, "aria-valuemin"))
        max_value = int(await page.get_attribute(slider_selector, "aria-valuemax"))
    
        if target_value < min_value or target_value > max_value:
            raise ValueError(f"Target value {target_value} is out of bounds ({min_value}, {max_value})")
    
        # 슬라이더 thumb 요소와 위치 가져오기
        slider_thumb = await page.query_selector(f"{slider_selector} .q-slider__thumb")
        thumb_box = await slider_thumb.bounding_box()
        
        # 슬라이더의 현재 위치 및 목표 위치 계산
        track = await page.query_selector(f"{slider_selector} .q-slider__track")
        track_box = await track.bounding_box()
        
        # 목표 x 위치 계산 (슬라이더 트랙 내에서 비율 기반)
        target_percentage = (target_value - min_value) / (max_value - min_value)
        target_x = track_box["x"] + target_percentage * track_box["width"]
        current_x = thumb_box["x"] + thumb_box["width"] / 2
        y = thumb_box["y"] + thumb_box["height"] / 2
    
        # 슬라이더를 드래그하여 목표 위치로 이동
        await page.mouse.move(current_x, y)
        await page.mouse.down()
        await page.mouse.move(target_x, y, steps=10)  # 부드럽게 이동
        await page.mouse.up()
    
        #print(f"슬라이더를 {target_value}로 이동 완료.")
    
        # 슬라이더 값 확인 및 강제 이벤트 트리거
        await page.evaluate(
            f"""
            (slider) => {{
                slider.dispatchEvent(new Event('input'));  // oninput 이벤트 트리거
                slider.dispatchEvent(new Event('change'));  // onchange 이벤트 트리거
            }}
            """,
            await page.query_selector(slider_selector)
        )
    
        # 이동 후 값 확인
        current_value = await page.get_attribute(slider_selector, "aria-valuenow")
        if current_value != str(target_value):
            print(f"경고: 슬라이더 값이 예상과 다릅니다. 현재 값: {current_value}, 예상 값: {target_value}")

    async def get_second_status_element(self, page):
        """
        두 번째 'Status' 요소에 접근하여 반환하는 메소드
        """
        status_elements = await page.query_selector_all("text=Status")
        if len(status_elements) > 1:
            second_status_element = status_elements[1]
            #print("두 번째 'Status' 요소에 접근합니다.")
            return second_status_element
        else:
            #print("두 번째 'Status' 요소가 존재하지 않습니다.")
            return None

    async def extract_level_status(self, page, identity_id, level):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 레벨에 따라 HP와 Defense가 변한다.
        hp_value = await self.extract_hp(second_status_element)
        defense_value = await self.extract_defense(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Level={level}, HP={hp_value}, Defense={defense_value}")
        self.container.add_level_stat(identity_id,level, hp_value, defense_value)
    
    async def extract_hp(self, status_element):
        # HP 추출
        hp_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].children[0].nextElementSibling.textContent.trim()"
        )
        return hp_value
    
    async def extract_defense(self, status_element):
        # Defense 추출
        defense_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return defense_value

    async def extract_uptie_status(self, page, identity_id, uptie):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 uptie에 따라 speed는 변한다.
        speed_value = await self.extract_speed(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Uptie={uptie}, Speed={speed_value}")
        self.container.add_uptie_stat(identity_id, uptie, speed_value)

    async def extract_speed(self, status_element):
        # Speed 추출
        speed_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return speed_value

    async def extract_resist_type(self, page, image_src, depth):
        # 이미지 요소 찾기
        #print(image_src)
        image = await page.query_selector(f'img[src="{image_src}"]')
        #print(image)
        if not image:
            return "Unknown"
    
        # 이미지의 부모 요소를 찾아서 올라감
        parent = image
        for _ in range(depth):
            parent = await parent.evaluate_handle('node => node.parentElement')
            if not parent:
                return "Unknown"
    
        # 부모의 형제 요소 찾기
        sibling = await parent.evaluate_handle('node => node.nextElementSibling')
        
        if sibling:
            value = await sibling.evaluate('node => node.textContent')
            if value:
                return value.strip()
        
        return "Unknown"

    async def extract_stagger_threshold_values(self, page):
        # 'div.row.center.flex-center.q-pt-xs' 요소들 찾기
        elements = await page.query_selector_all('div.row.center.flex-center.q-pt-xs')
        
        # 각 요소의 텍스트 내용 출력하여 'Stagger Threshold' 요소를 찾기
        for element in elements:
            text = await element.text_content()
            
            if text and "Stagger Threshold" in text:
                # 'Stagger Threshold'가 있는 요소를 찾았다면, 그 이후 형제 요소들에서 값을 추출
                parent = await element.evaluate_handle('node => node.parentElement')
                sibling_elements = await parent.query_selector_all('div.row.justify-evenly div')  # Threshold 값이 들어있는 div들
                
                stagger_values = []
                for sibling in sibling_elements:
                    text_content = await sibling.text_content()
                    match = re.search(r'(\d+%)', text_content)
                    if match:
                        stagger_values.append(match.group(1))  # 퍼센트 값만 리스트에 추가
                while len(stagger_values) < 3:
                    stagger_values.append(np.nan)
                #print(stagger_values)
                return stagger_values  # 추출된 값(또는 np.nan 포함)을 반환
        
        return [np.nan, np.nan, np.nan]  # 만약 'Stagger Threshold'를 찾지 못한 경우 기본값 반환

    async def crawl_speed_and_skills(self, page, identity_id):
        for uptie_value in range(1, 5):  # Uptie levels 1 to 4
            # 페이지 내 uptie 변경 버튼 클릭
            await self.uptie_click(page, uptie_value)
            await self.crawl_skill(page, identity_id, "skill 1", uptie_value)
            await self.crawl_skill(page, identity_id, "skill 2", uptie_value)
            await self.crawl_skill(page, identity_id, "skill 3", uptie_value)
            await self.crawl_skill(page, identity_id, "defense", uptie_value)
            await self.extract_uptie_status(page, identity_id, uptie_value)

    async def crawl_skill(self, page, identity_id, skill_type, uptie_value):
        skills_elements = await page.query_selector_all('div.skilltag')
        for skill_element in skills_elements:
            skill_text = await skill_element.text_content()
            if skill_text.strip().lower() == skill_type:
                await self.process_skill(skill_element, page, identity_id, skill_type, uptie_value)
        
    async def process_skill(self, skill_element, page, identity_id, skill_type, uptie_value):
        """
        개별 스킬 요소를 처리하고 관련 데이터를 저장.
        """
        # "skill 3"은 Uptie 3 이상일 때만 처리
        if skill_type == "skill 3" and uptie_value < 3:
            return

        # 스킬의 세부 정보를 추출
        skill_details = await self.extract_skill_details(skill_element, skill_type)
        skill_name = skill_details["skill_name"]

        # 스킬 정보를 저장
        self.container.add_skill(
            identity_id, skill_type,
            skill_name, skill_details["coin_count"],
            skill_details["init_level"], skill_details["init_remain"],
            skill_details["atk_weight"]
        )
        print(f"Added skill: {skill_type} - {skill_name} at Uptie {uptie_value}")

        # Uptie 정보 갱신
        await self.update_uptie_info(
            page, skill_element, identity_id, skill_name, skill_type, uptie_value
        )
    
    async def extract_skill_details(self, skill_element, skill_type):
        # 스킬 세부 정보를 추출하는 함수
        skill_name = await self.skill_name(skill_element)
        coin_count = await self.coin_count(skill_element, ".col-auto > .column > .row")
        init_level = await self.init_level(skill_element)
        init_remain = await self.init_remain(skill_element, skill_type, "div.row.justify-start > div.attr.flex.flex-center > div.q-px-sm")
        atk_weight = await self.atk_weight(skill_element, skill_type)
        return {
            "skill_name": skill_name,
            "coin_count": coin_count,
            "init_level": init_level,
            "init_remain": init_remain,
            "atk_weight": atk_weight
        }
    
    async def update_uptie_info(self, page, skill_element, identity_id, skill_name, skill_type, uptie_value):
        # Uptie 정보를 추출하고 업데이트하는 함수
        uptie_info = await self.extract_uptie_info(skill_element)
        effect_data = await self.skill_effects(skill_element)
        self.container.add_skill_uptie_info(
            identity_id, skill_name, 
            uptie_value, uptie_info["skill_power"], uptie_info["coin_power"], 
            effect_data.get("effectsByCondition")
        )
        keywords = effect_data.get("keywords")
        if keywords:
            self.keywords_set.update(keywords)
        print(
            f"Updated Uptie for {skill_name} (Uptie Level: {uptie_value}): "
            f"Skill Power: {uptie_info['skill_power']}, Coin Power: {uptie_info['coin_power']} "
            f"Skill Effect: {effect_data.get("effectsByCondition")}"
        )

    async def extract_uptie_info(self, skill_element):
        skill_data = await self.skill_and_coin_power(skill_element)
        return {
            "skill_power": skill_data.get("skillPower"),
            "coin_power": skill_data.get("coinPower")
        }
        
    async def crawl_influence_behavior(self, page, identity_id):
        influence_behaviors = await page.query_selector_all('div[align="right"]')
        # 텍스트를 기준으 추출하고 메소드로 파라미터 들어가 정보를 받는다.
        for influence_behavior in influence_behaviors:
            active_type = await influence_behavior.inner_text()
            active_type = active_type.strip()

            if active_type == "PASSIVE":
                """
                # PASSIVE에 대한 처리
                print(f"Processing PASSIVE logic for: {active_type}")
                # CALL method for PASSIVE
                """
                await self.passive_effects(influence_behavior, identity_id)
            elif active_type == "SUPPORT PASSIVE":
                """
                # SUPPORT PASSIVE에 대한 처리
                print(f"Processing SUPPORT PASSIVE logic for: {active_type}")
                # CALL method for SUPPORT PASSIVE
                """
                await self.support_passive_effects(influence_behavior, identity_id)
            elif active_type == "PANIC TYPE":
                """
                # PANIC TYPE에 대한 처리
                print(f"Processing PANIC TYPE logic for:{active_type}")
                # CALL method for PANIC TYPE
                """
                await self.panic_effects(influence_behavior, identity_id)

    async def skill_and_coin_power(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                // 조부모로 이동
                const grandparent = node.parentElement.parentElement;
                // 이전 형제 탐색
                const previousSibling = grandparent.previousElementSibling;
                if (!previousSibling) return {{ skillPower: null, coinPower: null }}; // 이전 형제가 없으면 기본값 반환
                // 이전 형제 내부의 .row.justify-around > div > div 탐색
                const targetDivs = previousSibling.querySelectorAll(".row.justify-around > div");
                if (targetDivs.length < 3) return {{ skillPower: null, coinPower: null }}; // 배열 길이 확인
                // 1st div: 스킬 위력
                const skillPower = targetDivs[0]?.querySelector('div')?.textContent?.trim() || null; 
                // 3rd div: 코인 파워 값 (3번째 div에서 텍스트 추출)
                const coinPower = targetDivs[2]?.textContent?.trim() || null;
                return {{ skillPower, coinPower }};
            }}
        """)


    async def coin_count(self, skill_element, hierarchy_selector):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector('{hierarchy_selector}');
                if (!targetElement) return 0;
                return targetElement.childElementCount;
            }}
        """)

    async def init_remain(self, skill_element, skill_type, hierarchy_selector):
        if skill_type == "defense":
            return -1
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector('{hierarchy_selector}');
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def skill_name(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector(".block");
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def init_level(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector("div.text-bold.text-h6.h6.q-pl-xs.q-pr-md");
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def atk_weight(self, skill_element, skill_type):
        if skill_type == "defense":
            return 0
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector("div.text-bold.text-h6.h6.q-pl-xs.q-pr-md");
                const doubleNextSibling = targetElement.nextElementSibling.nextElementSibling;
                if (!doubleNextSibling) return 0;
                return doubleNextSibling.textContent?.trim();
            }}
        """)

    async def skill_effects(self, skill_element):
        return await skill_element.evaluate("""
            node => {
                const parent = node.parentElement;
                const doubleNextSibling = parent.nextElementSibling?.nextElementSibling;
                if (!doubleNextSibling) return null;
    
                // doubleNextSibling에서 첫 번째 div.col 요소 찾기
                const textNode = doubleNextSibling.querySelector("div.col");
                if (!textNode) return null;
    
                const results = {
                    keywords: [], // 키워드 저장
                    conditions: [], // 조건 저장
                    effectsByCondition: {} // 조건별 효과
                };
    
                // 1. 조건 추출 (대괄호 텍스트)
                const conditionElements = textNode.querySelectorAll("span");
                conditionElements.forEach(span => {
                    let text = span.textContent?.trim();
                    if (text?.startsWith("[") && text?.endsWith("]")) {
                        // 부모의 전형제 노드 탐색
                        const sibling = span.parentElement.previousElementSibling;
                        if (sibling) {
                            const emblemImage = sibling.querySelector("img.emblem");
                            console.log("Sibling element found:", sibling); // 디버깅용
                            if (emblemImage) {
                                const src = emblemImage.getAttribute("src");
                                console.log("Image source:", src); // 디버깅용
                                const match = src.match(/ui_num_(\\d+)\\.png$/); // 정규식 수정
                                if (match) {
                                    const number = match[1]; // 숫자 추출
                                    text = `[Coin ${number} ${text.replace("[", "").replace("]", "")}]`;
                                    console.log("Updated condition:", text); // 디버깅용
                                }
                            }
                        }
                        results.conditions.push(text);
                        if (!results.effectsByCondition[text]) {
                            results.effectsByCondition[text] = []; // 조건별 효과를 초기화
                        }
                    }
                });
    
                // 2. 키워드 추출
                const keywordElements = textNode.querySelectorAll('a[href^="/en/keyword/"]');
                if (!keywordElements) return null;

                // 키워드 매핑 객체 초기화
                if (!results.keywordMapping) {
                    results.keywordMapping = {}; //텍스트: 키 형태로 저장
                }

                keywordElements.forEach(a => {
                    const span = a.querySelector('span.q-btn__content.text-center.col.items-center.q-anchor--skip.justify-center.row>span');
                    if (span) {
                        const keywordText = span.textContent.trim();
                        const href = a.getAttribute('href');
                        const keywordKey = href.split('/').pop();
                        if (keywordKey && !Object.values(results.keywordMapping).includes(keywordKey)) {
                            results.keywordMapping[keywordText] = keywordKey;
                        }
                    }
                });
    
                // 3. 효과 추출
                const effectElements = textNode.querySelectorAll("span");
                let currentCondition = null;
                let currentEffect = []; // 초기화
                let keywordList = new Set(Object.values(results.keywordMapping));
                effectElements.forEach(span => {
                    let text = span.textContent?.trim();
                    if (!text) return;
    
                    // 조건 발견 시 currentCondition 변경
                    if (text.startsWith("[") && text.endsWith("]")) {
                        // 이전 조건의 효과 저장
                        if (currentCondition && currentEffect.length > 0) {
                            if (!results.effectsByCondition[currentCondition]) {
                                results.effectsByCondition[currentCondition] = [];
                            }
                            // 부모의 전형제 노드 탐색
                            const sibling = span.parentElement.previousElementSibling;
                            if (sibling) {
                                const emblemImage = sibling.querySelector("img.emblem");
                                if (emblemImage) {
                                    const src = emblemImage.getAttribute("src");
                                    const match = src.match(/ui_num_(\\d+)\\.png$/); // 정규식 수정
                                    if (match) {
                                        const number = match[1]; // 숫자 추출
                                        text = `[Coin ${number} ${text.replace("[", "").replace("]", "")}]`;
                                    }
                                }
                            }
                            results.effectsByCondition[currentCondition].push(currentEffect.join(' '));
                            currentEffect = []; // 초기화
                        }
                        currentCondition = text;
                        if (!results.effectsByCondition[currentCondition]) {
                            results.effectsByCondition[currentCondition] = [];
                        }
                        return;
                    }
    
                    // 현재 조건의 효과 처리
                    if (currentCondition) {
                        if (results.keywordMapping[text]) {
                            const keywordEffect = `keyword:${results.keywordMapping[text]}`;
                            keywordList.add(results.keywordMapping[text]);
                            if (!currentEffect.includes(keywordEffect)) {
                                currentEffect.push(keywordEffect);
                            }
                        } else {
                            currentEffect.push(text);
                        }
                    }
                });

                results.keywords = Array.from(keywordList);
                delete results.keywordMapping;
                // 마지막 효과 저장
                if (currentCondition && currentEffect.length > 0) {
                    let processedText = currentEffect
                        .map(word => word.trim())
                        .join(" ")
                        .replace(/\\s+([,.:])/g, "$1")
                        .replace(/\\s+'/g, "'")
                        .replace(/'\\s+/g, "'")
                        .replace(/\\[\\s+/g, "[")
                        .replace(/\\s+\\]/g, "]")
                        .replace(/\\(\\s+/g, "(")
                        .replace(/\\s+\\)/g, ")")
                        .replace(/\\s+%/g, "%")
                        .replace(/\\s+\\//g, "/")
                        .replace(/\\/\\s+/g, "/")
                        .replace(/\\s+\\/\\s+/g, "/");
                    if (!results.effectsByCondition[currentCondition]) {
                        results.effectsByCondition[currentCondition] = [];
                    }
                    results.effectsByCondition[currentCondition].push(processedText);
                }
    
                return results;
            }
        """)

    async def passive_effects(self, passive_element, identity_id):
        passive_name = await self.behavior_name(passive_element)
        passive_conditions = await self.behavior_condition(passive_element)
        passive_description = await self.behavior_description(passive_element, "span")
        #print(passive_name)
        #print(passive_conditions)
        #print(passive_description)
        self.container.add_influence_behavior(identity_id, "passive", passive_name, passive_conditions, passive_description.get("description"))
        keywords = passive_description.get("keywords")
        if keywords:
            self.keywords_set.update(keywords)
    async def support_passive_effects(self, support_passive_element, identity_id):
        support_passive_name = await self.behavior_name(support_passive_element)
        support_passive_conditions = await self.behavior_condition(support_passive_element)
        support_passive_description = await self.behavior_description(support_passive_element, "span")
        #print(support_passive_name)
        #print(support_passive_conditions)
        #print(support_passive_description)
        self.container.add_influence_behavior(identity_id, "support passive", support_passive_name, support_passive_conditions, support_passive_description.get("description"))
        keywords = support_passive_description.get("keywords")
        if keywords:
            self.keywords_set.update(keywords)
    async def panic_effects(self, panic_element, identity_id):
        panic_name = await self.behavior_name(panic_element)
        panic_conditions = await self.behavior_condition(panic_element)
        panic_description = await self.behavior_description(panic_element, "div")
        #print(panic_type)
        #print(panic_conditions)
        #print(panic_description)
        self.container.add_influence_behavior(identity_id, "panic type", panic_name, panic_conditions, panic_description.get("description"))
        keywords = panic_description.get("keywords")
        if keywords:
            self.keywords_set.update(keywords)

    async def navigate_element_by_steps(self, element, parent_steps=0, prev_sibling_steps=0, next_sibling_steps=0):
        """
        주어진 `element`에서 부모, 이전 형제, 또는 다음 형제로 지정된 횟수만큼 이동하여 ElementHandle 반환.
        
        Args:
            element (ElementHandle): 탐색 기준이 되는 요소.
            parent_steps (int): 부모로 이동할 횟수 (0이면 이동하지 않음).
            prev_sibling_steps (int): 이전 형제로 이동할 횟수 (0이면 이동하지 않음).
            next_sibling_steps (int): 다음 형제로 이동할 횟수 (0이면 이동하지 않음).
        
        Returns:
            ElementHandle or None: 탐색된 요소를 반환하거나 없으면 None.
        """
        try:
            current_element = element
    
            # 부모로 이동
            for _ in range(parent_steps):
                current_element = await current_element.evaluate_handle('el => el.parentElement')
                if not current_element:
                    print("Parent element not found during navigation.")
                    return None
    
            # 이전 형제로 이동
            for _ in range(prev_sibling_steps):
                current_element = await current_element.evaluate_handle('el => el.previousElementSibling')
                if not current_element:
                    print("Previous sibling element not found during navigation.")
                    return None
    
            # 다음 형제로 이동
            for _ in range(next_sibling_steps):
                current_element = await current_element.evaluate_handle('el => el.nextElementSibling')
                if not current_element:
                    print("Next sibling element not found during navigation.")
                    return None
    
            return current_element
    
        except Exception as e:
            print(f"Error navigating elements: {e}")
            return None

    async def get_last_sibling(self, element):
        try:
            # 같은 부 아 마지막 형제를 가져오는 JavaScript 실행
            last_sibling = await element.evaluate_handle('el => el.parentElement.lastElementChild')
            return last_sibling
        except Exception as e:
            print(f"Error getting last sibling: {e}")
            return None

    async def behavior_name(self, element):
        parent_prev_sibling = await self.navigate_element_by_steps(element, parent_steps=1, prev_sibling_steps=1, next_sibling_steps=0)
        passive_name_element = await parent_prev_sibling.query_selector('span.block')
        if passive_name_element:
            passive_name = await passive_name_element.inner_text()
            return passive_name
        else:
            #print("Passive name element not found")
            return None

    async def behavior_condition(self, element):
        try:
            conditions = []
            parent_prev_sibling = await self.navigate_element_by_steps(element, parent_steps=1, prev_sibling_steps=1, next_sibling_steps=0)
            img_elements = await parent_prev_sibling.query_selector_all('img')
            if not img_elements:
                #print("No <img> elements found.")
                return conditions
    
            for img_element in img_elements:
                # 각 이미지의 src 속성 가져오기
                img_src = await img_element.get_attribute('src')
                #print(img_src)
    
                sin_name = None
                for color_key in resource_mapping.keys():
                    if f"{self.img_url}/ui/attr_{color_key}.png" in img_src:
                        sin_name = resource_mapping[color_key]
                        break
    
                if not sin_name:
                    #print(f"No matching Sin Color found for {img_src}.")
                    continue  # No Sin Color found, skip this img_element
    
                # 해당 이미지와 관련된 조건 텍스트 추출
                # 두 번째 부모 요소에서 시작하여 다음 형제를 찾아 조건 텍스트를 추출
                condition_element = await self.navigate_element_by_steps(img_element, parent_steps=3, prev_sibling_steps=0, next_sibling_steps=0)
                if condition_element:
                    # condition_element가 None이 아닌 경우에만 진행
                    condition_text_element = await condition_element.query_selector('.q-pl-xs')
                    if condition_text_element:
                        condition_text = await condition_text_element.inner_text()
                        condition_text = condition_text.strip()
                    else:
                        print(f"No condition text found for {img_src}.")
                        condition_text = ""  # condition_text_element가 None이면 빈 문자열로 처리
                else:
                    print(f"No condition element found for {img_src}.")
                    condition_text = ""  # condition_element가 None이면 빈 문자열로 처리
    
                # 결과를 배열에 추가
                conditions.append(f"{sin_name} {condition_text}")
            return conditions
        except Exception as e:
            print(f"Error extracting conditions: {e}")
            return 
            
    async def behavior_description(self, element, tag_type):
        parent_prev_sibling = await self.navigate_element_by_steps(
            element, parent_steps=1, prev_sibling_steps=1, next_sibling_steps=0
        )
        return await parent_prev_sibling.evaluate(f"""
            (node) => {{
                const behavior_title_node = node.querySelector('.q-pb-sm');
                const behavior_title_parent = behavior_title_node.parentElement;
                const behavior_description = behavior_title_parent.lastElementChild;
                if (!behavior_description) return null;
    
                const results = {{
                    keywords: [], // 키워드 저장
                    description: [] // 설명 저장
                }};
                
                // 매핑을 저장할 객체
                const keywordElements = behavior_description.querySelectorAll('a[href^="/en/keyword/"]');
                if (!keywordElements) return null;
                
                // 키워드 매핑 객체 초기화
                if (!results.keywordMapping) {{
                    results.keywordMapping = {{}}; // 텍스트: 키 형태로 저장
                }}
                
                keywordElements.forEach(a => {{
                    const span = a.querySelector('span.q-btn__content.text-center.col.items-center.q-anchor--skip.justify-center.row>span');
                    if (span) {{
                        const keywordText = span.textContent.trim(); // UI에 표시된 텍스트 (예: "Ammo")
                        const href = a.getAttribute('href'); // href 속성 값 (예: "/en/keyword/Bullet")
                        const keywordKey = href.split('/').pop(); // href에서 마지막 부분 추출 (예: "Bullet")
                
                        if (keywordKey && !Object.values(results.keywordMapping).includes(keywordKey)) {{
                            results.keywordMapping[keywordText] = keywordKey; // 텍스트: 키로 매핑
                        }}
                    }}
                }});

                // 2. 설명 추출
                const descriptionElements = behavior_description.querySelectorAll("{tag_type}");
                let currentEffect = []; // 초기화
                let keywordsList = new Set(Object.values(results.keywordMapping)); // keywordMapping 값으로 Set 생성
            
                descriptionElements.forEach(span => {{
                    let text = span.textContent?.trim();
                    if (!text) return;
            
                    // 공백과 기호 처리
                    text = text.replace(/\\s*([.,:;!?()\\[\\]])\\s*/g, '$1');
            
                    // 텍스트가 매핑된 키워드인지 확인
                    if (results.keywordMapping[text]) {{
                        const keywordEffect = `keyword:${{results.keywordMapping[text]}}`; // 매핑된 키 사용
                        keywordsList.add(results.keywordMapping[text]); // keywordsList에 추가
                        if (!currentEffect.includes(keywordEffect)) {{
                            currentEffect.push(keywordEffect);
                        }}
                    }} else {{
                        currentEffect.push(text);
                    }}
                }});
            
                // keywords 배열로 변환
                results.keywords = Array.from(keywordsList); // Set에서 배열로 변환하여 저장
    
                // 마지막 효과 저장
                if (currentEffect.length > 0) {{
                    const finalDescription = currentEffect.join(' ');
                    //console.log("After process:", finalDescription);
                    results.description.push(finalDescription);
                }}
                delete results.keywordMapping;
                let processedText = results.description
                    .map(word => word.trim())
                    .join(" ")
                    .replace(/\\s+([,.:])/g, "$1")
                    .replace(/\\s+'/g, "'")
                    .replace(/'\\s+/g, "'")
                    .replace(/\\[\\s+/g, "[")
                    .replace(/\\s+\\]/g, "]")
                    .replace(/\\(\\s+/g, "(")
                    .replace(/\\s+\\)/g, ")")
                    .replace(/\\s+%/g, "%")
                    .replace(/\\s+\\//g, "/")
                    .replace(/\\/\\s+/g, "/")
                    .replace(/\\s+\\/\\s+/g, "/");
                results.description = processedText;
                return results;
            }}
        """)

    async def crawl_keyword_info(self, page, keyword):
        # 키워드 설명 추출
        keyword_explain_elements = await page.query_selector_all('div.text-caption.text-grey > div')
        keyword_explains = []
        keyword_explain_text = np.nan
        for keyword_explain_element in keyword_explain_elements:
            keyword_explain_text = await keyword_explain_element.inner_text()
            keyword_explain_text = keyword_explain_text.strip()
            keyword_explains.append(keyword_explain_text)
    
        # Dispellable 값과 그 이후 텍스트 추출
        grand_parent = await self.navigate_element_by_steps(
            keyword_explain_elements[0],
            parent_steps=2, 
            prev_sibling_steps=0, 
            next_sibling_steps=0
        )
        
        # Dispellable 요소 찾기
        dispellable_element = await grand_parent.query_selector('div.q-pt-sm > div')
        dispellable_value = None
        is_dispellable = np.nan  # 초기값을 np.nan으로 설정

        if dispellable_element:
            dispellable_value = await dispellable_element.inner_text()
            dispellable_value = dispellable_value.strip()
            
            # Dispellable 값 처리
            if ':' in dispellable_value:
                dispellable_state = dispellable_value.split(':', 1)[1].strip().lower()
                if dispellable_state == 'true':
                    is_dispellable = True
                elif dispellable_state == 'false':
                    is_dispellable = False
                else:
                    is_dispellable = np.nan  # 'true'나 'false'가 아닌 경우 np.nan으로 설정
        
            # 결과 출력
            #print(f"Dispellable: {is_dispellable}, After Colon Text: {after_colon_text}")
        
        # 컨테이너에 저장
        self.container.add_keyword(keyword, keyword_explains, is_dispellable)

class Identity:
    """인격 정보를 저장하는 클래스"""
    def __init__(self, identity_id, name=None, rarity=None,\
                 skill_1_type=None, skill_1_sin_affinity=None,\
                 skill_2_type=None, skill_2_sin_affinity=None,\
                 skill_3_type=None, skill_3_sin_affinity=None,\
                 defense_type=None, defense_sin_affinity=None):
        self.identity_id = identity_id
        self.name = name
        self.rarity = rarity
        # 선언만 하는 부분 
        self.skill_1_type = skill_1_type 
        self.skill_1_sin_affinity = skill_1_sin_affinity 
        self.skill_2_type = skill_2_type 
        self.skill_2_sin_affinity = skill_2_sin_affinity 
        self.skill_3_type = skill_3_type 
        self.skill_3_sin_affinity = skill_3_sin_affinity 
        self.defense_type = defense_type 
        self.defense_sin_affinity = defense_sin_affinity
        self.level_max = 50
        self.uptie_max = 4
        self.level_stats = []  # LevelStat 객체 리스트
        self.uptie_stats = [] # UptieStat 객체 리스트
        self.resist_info = None
        self.stagger_threshold = None
        self.skillset = SkillSet()  # Skill정보를 담고 있는 skillset 객체 
        self.passive_infos = []  # 패시브 객체를 담고 있는 배열
        self.support_passive_infos = [] # 서포트 패시 객체를 담고 있는 배열
        self.panic_type = [] # 혼란상태 발생시 나타나는 효과를 담고 있는 객체 배열1

    def update_name(self, name):
        self.name = name

    def update_rarity(self, rarity):
        self.rarity = rarity

    def add_level_stat(self, level, hp, defense):
        self.level_stats.append(LevelStat(level, hp, defense))

    def add_uptie_stat(self, uptie, speed):
        self.uptie_stats.append(UptieStat(uptie, speed))

    def add_resist_info(self, resist_hit, resist_penetrate, resist_slash):
        self.resist_info = ResistInfo(resist_hit, resist_penetrate, resist_slash)

    def add_stagger_threshold(self, first_stagger, second_stagger, third_stagger):
        self.stagger_threshold = StaggerThreshold(first_stagger, second_stagger, third_stagger)

    def add_skill(self, skill_code, skill_name, coin_count, init_level, init_remain, atk_weight):
        self.skillset.add_skill(skill_code, Skill(skill_code, skill_name, coin_count, init_level, init_remain, atk_weight))
        
    def add_skill_uptie_info(self, skill_name, uptie_level, skill_power, coin_power, effect_by_condition):
        self.skillset.add_skill_uptie_info(skill_name, uptie_level, skill_power, coin_power, effect_by_condition)

    def add_influence_behavior(self, behavior_type, behavior_name, condition, description):
        if behavior_type == "passive":
            self.passive_infos.append(Passive(behavior_type, behavior_name, condition, description))
        elif behavior_type == "support passive":
            self.support_passive_infos.append(SupportPassive(behavior_type, behavior_name, condition, description))
        elif behavior_type == "panic type":
            self.panic_type.append(PanicType(behavior_type, behavior_name, condition, description))
        
    def update_details(self, page):
        # 세부 정보 업데이트 로직 (가정)
        pass
    #def init_uptie(self, page):
    #    init_uptie = 

# LevelStat 객체 정의
class LevelStat:
    def __init__(self, level, hp=None, defense=None):
        self.level = level
        self.hp = hp
        self.defense = defense
    def update_level(self, level):
        self.level = level
    def update_hp(self, hp):
        self.hp = hp
    def update_defense(self, defense):
        self.defense = defense

# UptieStat 객체 정의
class UptieStat:
    def __init__(self, uptie, speed=None):
        self.uptie = uptie
        self.speed = speed
    def update_speed(self, speed):
        self.speed = speed

# ResistInfo 객체 정의
class ResistInfo:
    def __init__(self, resist_hit=None, resist_penetrate=None, resist_slash=None):
        self.resist_hit = resist_hit
        self.resist_penetrate = resist_penetrate
        self.resist_slash = resist_slash

# ResistInfo 객체 정의
class StaggerThreshold:
    def __init__(self, first_stagger=None, second_stagger=None, third_stagger=None):
        self.first_stagger = first_stagger
        self.second_stagger = second_stagger
        self.third_stagger = third_stagger

class SkillSet:
    def __init__(self):

        self.skill_1_type = ""
        self.skill_2_type = ""
        self.skill_3_type = ""
        self.defense_skill_type = ""

        self.skill_1_sin_affinity = ""
        self.skill_2_sin_affinity = ""
        self.skill_3_sin_affinity = ""
        self.defense_skill_sin_affinity = ""

        self.skill_1_details = []
        self.skill_2_details = []
        self.skill_3_details = []
        self.defense_skill_details = []

    def set_skill_type(self, skill_code, skill_type, sin_affinity):
        if skill_code == "skill 1":
            self.skill_1_type = skill_type
            self.skill_1_sin_affinity = sin_affinity
        elif skill_code == "skill 2":
            self.skill_2_type = skill_type
            self.skill_2_sin_affinity = sin_affinity
        elif skill_code == "skill 3":
            self.skill_3_type = skill_type
            self.skill_3_sin_affinity = sin_affinity
        elif skill_code == "defense":
            self.defense_skill_type = skill_type
            self.defense_skill_sin_affinity = sin_affinity
    
    def add_skill(self, skill_code, skill):
        if skill_code == "skill 1":
            self.skill_1_details.append(skill)
            skill.skill_type = self.skill_1_type
            skill.sin_affinity = self.skill_1_sin_affinity
        elif skill_code == "skill 2":
            self.skill_2_details.append(skill)
            skill.skill_type = self.skill_2_type
            skill.sin_affinity = self.skill_2_sin_affinity
        elif skill_code == "skill 3":
            self.skill_3_details.append(skill)
            skill.skill_type = self.skill_3_type
            skill.sin_affinity = self.skill_3_sin_affinity
        elif skill_code == "defense":
            self.defense_skill_details.append(skill)
            skill.skill_type = self.defense_skill_type 
            skill.sin_affinity = self.defense_skill_sin_affinity

    def get_skill_by_name(self, skill_name):
        for skill_list in [self.skill_1_details, self.skill_2_details, self.skill_3_details, self.defense_skill_details]:
            for skill in skill_list:
                result = skill.is_skill_by_name(skill_name)
                if result:
                    return result
        return None

    def add_skill_uptie_info(self, skill_name, uptie_level, skill_power, coin_power, effect_by_condition):
        skill = self.get_skill_by_name(skill_name)
        skill.add_uptie_info(uptie_level, skill_power, coin_power, effect_by_condition)
        

class Skill:
    def __init__(self, skill_code, skill_name, coin_count, init_level, init_remain, atk_weight):
        self.skill_code = skill_code # "Skill 1"/"Skill 2"/"Skill 3"/"Defense" '''o'''
        self.skill_type = None
        self.sin_affinity = None
        self.skill_name = skill_name # 스킬 이름 '''o'''
        self.coin_count = coin_count # 동전 개수 '''o'''
        self.init_level = init_level # 스킬이 초기 배정받은 레벨, 인격 레벨에 - 1 한 값을 더해주면 스킬의 현 level을 구할 수 있다. '''o'''
        self.init_remain = init_remain #'''o'''
        self.atk_weight = atk_weight #공격 가중치 값이다. 방어스킬에는 없기 때문에 None으로 설정했다. '''o'''
        self.remain_count = init_remain # 실제 스킬이 작동하며 횟수가 차감되 나중에 모두 차감되면 다음 턴에 다시 초기횟수 리필된다.
        self.uptie_info = []  

    def is_skill_by_name(self, skill_name):
        if self.skill_name == skill_name:
            return self
        return None
        
    def add_uptie_info(self, uptie_level, skill_power, coin_power, effects_by_condition):
        self.uptie_info.append(UptieInfo(uptie_level, skill_power, coin_power, effects_by_condition))

class UptieInfo:
    def __init__(self, uptie_level, skill_power, coin_power, effects_by_condition):
        self.uptie_level = uptie_level   #'''o'''
        self.skill_power = skill_power   #'''o'''
        self.coin_power = coin_power     #'''o'''
        self.skill_effect = effects_by_condition

# 추상화된 EffectType 클래스
class EffectType:
    def __init__(self, behavior_type, name, description):
        self.behavior_type = behavior_type
        self.name = name  # Effect 이름
        self.description = description  # Effect 설명

# Passive 클래스
class Passive(EffectType):
    def __init__(self, behavior_type, name, condition, description):
        super().__init__(behavior_type, name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)  # condition이 있는지 여부 확인

# SupportPassive 클래스
class SupportPassive(EffectType):
    def __init__(self,behavior_type, name, condition, description):
        super().__init__(behavior_type, name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)

# PanicType 클래스
class PanicType(EffectType):
    def __init__(self, behavior_type, name, condition, description):
        super().__init__(behavior_type, name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)

# Keyword 클래스
class Keyword:
    def __init__(self, keyword, explain=None, dispellable=None):
        self.keyword = keyword
        self.explain = explain
        self.dispellable = dispellable

      
class DataContainer:
    """크롤링 데이터를 관리하는 클래스"""
    def __init__(self):
        self.identities = []
        self.keywords = []

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.level_max

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.uptie_max

    def update_identity_name(self, identity_id, identity_name):
        """특정 ID의 이름을 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_name(identity_name)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_identity_rarity(self, identity_id, rarity):
        """특정 ID의 희귀도를 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_rarity(rarity)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_skill(self, identity_id, skill_number, new_type):
        """
        특정 identity의 skill_type을 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_type: 새 skill_type
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_type = new_type
                elif skill_number == 2:
                    identity.skill_2_type = new_type
                elif skill_number == 3:
                    identity.skill_3_type = new_type
                elif skill_number == "defense":
                    identity.defense_type = new_type
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def update_sin_affinity(self, identity_id, skill_number, new_affinity):
        """
        특정 identity의 sin_affinity를 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_affinity: 새 sin_affinity
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_sin_affinity = new_affinity
                elif skill_number == 2:
                    identity.skill_2_sin_affinity = new_affinity
                elif skill_number == 3:
                    identity.skill_3_sin_affinity = new_affinity
                elif skill_number == "defense":
                    identity.defense_sin_affinity = new_affinity
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def add_identity(self, identity):
        """새로운 인격 데이터를 추가."""
        self.identities.append(identity)

    def add_level_stat(self, identity_id, level, hp, defense):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_level_stat(level, hp, defense)

    def add_uptie_stat(self, identity_id, uptie, speed):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_uptie_stat(uptie, speed)

    def add_resist_info(self, identity_id, resist_hit, resist_penetrate, resist_slash):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_resist_info(resist_hit, resist_penetrate, resist_slash)

    def add_stagger_threshold(self, identity_id, first_stagger, second_stagger, third_stagger):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_stagger_threshold(first_stagger, second_stagger, third_stagger)

    
    def add_skill(self, identity_id, skill_code, skill_name, coin_count, init_level, init_remain, atk_weight):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_code == "skill 1":
                    identity.skillset.set_skill_type(skill_code, identity.skill_1_type, identity.skill_1_sin_affinity)
                elif skill_code == "skill 2":
                    identity.skillset.set_skill_type(skill_code, identity.skill_2_type, identity.skill_2_sin_affinity)
                elif skill_code == "skill 3":
                    identity.skillset.set_skill_type(skill_code, identity.skill_3_type, identity.skill_3_sin_affinity)
                elif skill_code == "defense":
                    identity.skillset.set_skill_type(skill_code, identity.defense_type, identity.defense_sin_affinity)
                identity.add_skill(skill_code, skill_name, coin_count, init_level, init_remain, atk_weight)

    def add_skill_uptie_info(self, identity_id, skill_name, uptie_level, skill_power, coin_power, effects_by_condition):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_skill_uptie_info(skill_name, uptie_level, skill_power, coin_power, effects_by_condition)

    def add_influence_behavior(self, identity_id, behavior_type, behavior_name, condition, description):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_influence_behavior(behavior_type, behavior_name, condition, description)

    def add_keyword(self, keyword, explain, dispellable):
        self.keywords.append(Keyword(keyword, explain, dispellable))
        
    def display_identities(self):
        """Identity, ID, Name, Rarity 정보를 출력"""
        if not self.identities:
            print("No identities found.")
        else:
            # 데이터를 리스트로 변환
            table = []
            for idx, identity in enumerate(self.identities, 1):
                table.append([
                    idx, identity.identity_id, identity.name, identity.rarity,
                    identity.skill_1_type, identity.skill_1_sin_affinity,
                    identity.skill_2_type, identity.skill_2_sin_affinity,
                    identity.skill_3_type, identity.skill_3_sin_affinity,
                    identity.defense_type, identity.defense_sin_affinity
                ])
            
            # 테이블 헤더 정의
            headers = [
                "Index", "ID", "Name", "Rarity",
                "Skill_1 Type", "Skill_1 Sin", "Skill_2 Type", "Skill_2 Sin",
                "Skill_3 Type", "Skill_3 Sin", "Defense Type", "Defense Sin"
            ]
            
            # 테이블 출력
            print(tabulate(table, headers=headers, tablefmt="fancy_grid"))

    def export_to_csv(self, filename, fieldnames, new_data):
        """
        기존 데이터를 유지하고, 새로운 데이터를 추가하여 CSV 파일에 저장하는 메서드.
        :param filename: 내보낼 파일 이름 (예: 'identity.csv', 'skill.csv')
        :param fieldnames: CSV의 헤더 리스트
        :param new_data: CSV로 추가할 새로운 데이터 (리스트 형태)
        """
        try:
            # 'csv' 폴더가 없으면 생성
            if not os.path.exists('./csv'):
                os.makedirs('./csv')
            
            # 파일 경로 설정
            file_path = os.path.join('./csv', filename)
    
            # 기존 데이터를 읽기
            existing_data = []
            if os.path.exists(file_path):
                with open(file_path, mode='r', newline='', encoding='utf-8') as file:
                    reader = csv.DictReader(file)
                    existing_data = [row for row in reader]
    
            # 중복 제거를 위해 기존 데이터와 새로운 데이터를 병합
            existing_keywords = {row[fieldnames[0]] for row in existing_data}  # 첫 번째 필드를 기준으로 중복 체크
            combined_data = existing_data + [data for data in new_data if data[fieldnames[0]] not in existing_keywords]
    
            # 병합된 데이터를 다시 CSV 파일로 저장
            with open(file_path, mode='w', newline='', encoding='utf-8') as file:
                writer = csv.DictWriter(file, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(combined_data)
    
            print(f"Data successfully exported to {filename}")
        except Exception as e:
            print(f"Error exporting to CSV: {e}")

    def get_identity_data(self):
        """
        Identity 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "ID": identity.identity_id,
                "Name": identity.name,
                "Rarity": identity.rarity,
                "Skill_1_Type": identity.skill_1_type,
                "Skill_1_Sin_Affinity": identity.skill_1_sin_affinity,
                "Skill_2_Type": identity.skill_2_type,
                "Skill_2_Sin_Affinity": identity.skill_2_sin_affinity,
                "Skill_3_Type": identity.skill_3_type,
                "Skill_3_Sin_Affinity": identity.skill_3_sin_affinity,
                "Defense_Type": identity.defense_type,
                "Defense_Sin_Affinity": identity.defense_sin_affinity,
            }
            for identity in self.identities
        ]
        
    def export_identities_data(self, filename="Identities.csv"):
        """Identity 데이터를 CSV로 내보내는 메서드"""
        identity_fieldnames = [
            "ID", "Name", "Rarity",
            "Skill_1_Type", "Skill_1_Sin_Affinity",
            "Skill_2_Type", "Skill_2_Sin_Affinity",
            "Skill_3_Type", "Skill_3_Sin_Affinity",
            "Defense_Type", "Defense_Sin_Affinity"
        ]
        identity_data = self.get_identity_data()  # Identity 데이터 가져오기
        self.export_to_csv(filename, identity_fieldnames, identity_data)  # CSV로 내보내기

    def load_identities_by_id(self, filename, identity_ids):
        """지정된 identity_ids 목록에 해당하는 Identity 데이터를 CSV에서 읽어옴."""
        identity_ids_set = set(identity_ids) # 중복 제거를 위 Set 사용
        with open(filename, mode='r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                if row["ID"] in identity_ids_set:
                    identity = Identity(
                        identity_id=row["ID"],
                        name=row["Name"],
                        rarity=row["Rarity"],
                        skill_1_type=row["Skill_1_Type"],
                        skill_1_sin_affinity=row["Skill_1_Sin_Affinity"],
                        skill_2_type=row["Skill_2_Type"],
                        skill_2_sin_affinity=row["Skill_2_Sin_Affinity"],
                        skill_3_type=row["Skill_3_Type"],
                        skill_3_sin_affinity=row["Skill_3_Sin_Affinity"],
                        defense_type=row["Defense_Type"],
                        defense_sin_affinity=row["Defense_Sin_Affinity"]
                    )
                    self.identities.append(identity)

    def load_all_identities(self, file_path):
        """
        주어진 CSV 파일에서 모든 Identity 정보를 읽어옴.
        :param file_path: CSV 파일 경로
        """
        try:
            with open(file_path, mode='r', encoding='utf-8') as file:
                reader = csv.DictReader(file)
                for row in reader:
                    try:
                        identity = Identity(
                            identity_id=row["ID"].strip(),  # ID는 필수
                            name=row["Name"].strip(),
                            rarity=row["Rarity"].strip(),
                            skill_1_type=row.get("Skill_1_Type", "").strip(),
                            skill_1_sin_affinity=row.get("Skill_1_Sin_Affinity", "").strip(),
                            skill_2_type=row.get("Skill_2_Type", "").strip(),
                            skill_2_sin_affinity=row.get("Skill_2_Sin_Affinity", "").strip(),
                            skill_3_type=row.get("Skill_3_Type", "").strip(),
                            skill_3_sin_affinity=row.get("Skill_3_Sin_Affinity", "").strip(),
                            defense_type=row.get("Defense_Type", "").strip(),
                            defense_sin_affinity=row.get("Defense_Sin_Affinity", "").strip(),
                        )
                        self.identities.append(identity)
                    except KeyError as e:
                        print(f"Missing column in row: {e}. Skipping this row.")
                    except Exception as e:
                        print(f"Error processing row {row}: {e}. Skipping this row.")
            print(f"Successfully loaded {len(self.identities)} identities.")
        except FileNotFoundError:
            print(f"File not found: {file_path}")
        except Exception as e:
            print(f"Error loading identities: {e}")


    def get_level_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Level": level_stat.level,
                "HP": level_stat.hp,
                "Defense": level_stat.defense,
            }
            for identity in self.identities
            for level_stat in identity.level_stats
        ]

    def export_level_stats_data(self, filename="Levelstats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        level_stats_fieldnames = ["Identity_ID", "Level", "HP", "Defense"]
        level_stats_data = self.get_level_stats_data()  # level_stats 데이터 가져오기
        self.export_to_csv(filename, level_stats_fieldnames, level_stats_data)  # CSV로 내보내기

    def get_uptie_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Uptie": uptie_stat.uptie,
                "Speed": uptie_stat.speed,
            }
            for identity in self.identities
            for uptie_stat in identity.uptie_stats
        ]

    def export_uptie_stats_data(self, filename="Uptiestats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        uptie_stats_fieldnames = ["Identity_ID", "Uptie", "Speed"]
        uptie_stats_data = self.get_uptie_stats_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, uptie_stats_fieldnames, uptie_stats_data)  # CSV로 내보내기

    def get_resist_info_data(self):
        """
        모든 Identity의 resist_info 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Resist_Hit": identity.resist_info.resist_hit,
                "Resist_Penetrate": identity.resist_info.resist_penetrate,
                "Resist_Slash": identity.resist_info.resist_slash,
            }
            for identity in self.identities
        ]

    def export_resist_info_data(self, filename="ResistInfos.csv"):
        """ResistInfo 데이터를 CSV로 내보내는 메서드"""
        resist_infos_fieldnames = ["Identity_ID", "Resist_Hit", "Resist_Penetrate", "Resist_Slash"]
        resist_infos_data = self.get_resist_info_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, resist_infos_fieldnames, resist_infos_data)  # CSV로 내보내기

    def export_uptie_stats_data(self, filename="Uptiestats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        uptie_stats_fieldnames = ["Identity_ID", "Uptie", "Speed"]
        uptie_stats_data = self.get_uptie_stats_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, uptie_stats_fieldnames, uptie_stats_data)  # CSV로 내보내기

    def get_stagger_threshold_data(self):
        """
        모든 Identity의 stagger_threshold 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "First_Stagger": identity.stagger_threshold.first_stagger,
                "Second_Stagger": identity.stagger_threshold.second_stagger,
                "Third_Stagger": identity.stagger_threshold.third_stagger,
            }
            for identity in self.identities
        ]

    def export_stagger_threshold_data(self, filename="StaggerThresholds.csv"):
        """StaggerThresholds 데이터를 CSV로 내보내는 메서드"""
        stagger_thresholds_fieldnames = ["Identity_ID", "First_Stagger", "Second_Stagger", "Third_Stagger"]
        stagger_thresholds_data = self.get_stagger_threshold_data()  # stagger_thresholds 데이터 가져오기
        self.export_to_csv(filename, stagger_thresholds_fieldnames, stagger_thresholds_data)  # CSV로 내보내기

    def get_skillset_data(self):
        """모든 Identity의 skillset 데이터를 export_to_csv을 포맷으로 변환"""
        return [
            {
                "Identity_ID": identity.identity_id,
                "Skill_Code": skill.skill_code,
                "Skill_Type": skill.skill_type,
                "Sin_Affinity": skill.sin_affinity,
                "Skill_Name": skill.skill_name,
                "Coin_Count": skill.coin_count,
                "Init_Level": skill.init_level,
                "Init_Remain": skill.init_remain,
                "Atk_Weight": skill.atk_weight,
                "Uptie_Level": uptie_info.uptie_level,
                "Skill_Power": uptie_info.skill_power,
                "Coin_Power": uptie_info.coin_power,
                "Effect_Data": json.dumps(uptie_info.skill_effect)
            }
            for identity in self.identities
            for skill_list in [identity.skillset.skill_1_details, identity.skillset.skill_2_details, identity.skillset.skill_3_details, identity.skillset.defense_skill_details]
            for skill in skill_list
            for uptie_info in skill.uptie_info
        ]

    def export_identity_skillset_data(self, filename="IdentitiesSkills.csv"):
        """IdentitiesSkills 데이터를 CSV로 내보내는 메서드"""
        identities_skills_fieldnames = ["Identity_ID", "Skill_Code", "Skill_Type", "Sin_Affinity", "Skill_Name", "Coin_Count", "Init_Level", "Init_Remain", "Atk_Weight", "Uptie_Level", "Skill_Power", "Coin_Power", "Effect_Data"]
        identities_skills_data = self.get_skillset_data()
        self.export_to_csv(filename, identities_skills_fieldnames, identities_skills_data)

    def get_identities_behaviors_data(self):
        """모든 Identity의 influence behavior 데이터를 export_to_csv 포맷으로 변환"""
        return [
            {
                "Identity_ID": identity.identity_id,
                "Effect_Type": effect_type,
                "Behavior_Name": effect.name,
                "Behavior_Condition": effect.condition if effect.condition else np.nan,
                "Behavior_Description" : effect.description
            }
            for identity in self.identities
            for effect_list, effect_type in [(identity.passive_infos, 'Passive'), (identity.support_passive_infos, 'Support Passive'), (identity.panic_type, 'Panic Type')]
            for effect in effect_list
        ]

    def export_identities_behaviors_data(self, filename="IdentitiesBehaviors.csv"):
        """IdentitiesBehaviors 데이터를 CSV로 내보내는 메서드"""
        identities_behaviors_fieldnames = ["Identity_ID","Effect_Type","Behavior_Name","Behavior_Condition","Behavior_Description"]
        identities_behaviors_data = self.get_identities_behaviors_data()
        self.export_to_csv(filename, identities_behaviors_fieldnames, identities_behaviors_data)

    def get_keywords_data(self):
        """DataContainer의 keywords 데이터를 export_to_csv 포맷으로 변환"""
        return [
            {
                "Keyword": keyword.keyword,
                "Explain": keyword.explain,
                "Dispellable": keyword.dispellable,
            }
            for keyword in self.keywords
        ]

    def export_keywords_data(self, filename="Keywords.csv"):
        """DataContainer의 keywords 데이터를 CSV로 내보내는 메서드"""
        keywords_fieldnames = ["Keyword","Explain","Dispellable"]
        keywords_data = self.get_keywords_data()
        self.export_to_csv(filename, keywords_fieldnames, keywords_data)

            
async def main():
    data_container = DataContainer()
    scraper = Scraper(container=data_container, base_url="https://limbus.kusoge.xyz", img_url="https://img.kusoge.xyz/limbus/img")
    await scraper.scrape()

await main() 

Successfully loaded 126 identities.
Identity_id: 10101
Level=1, HP=74, Defense=1
Level=2, HP=77, Defense=1
Level=3, HP=79, Defense=1
Level=4, HP=82, Defense=2
Level=5, HP=84, Defense=3
Level=6, HP=87, Defense=4
Level=7, HP=89, Defense=5
Level=8, HP=92, Defense=6
Level=9, HP=94, Defense=7
Level=10, HP=97, Defense=8
Level=11, HP=99, Defense=9
Level=12, HP=102, Defense=10
Level=13, HP=104, Defense=11
Level=14, HP=107, Defense=12
Level=15, HP=109, Defense=13
Level=16, HP=112, Defense=14
Level=17, HP=114, Defense=15
Level=18, HP=117, Defense=16
Level=19, HP=119, Defense=17
Level=20, HP=122, Defense=18
Level=21, HP=124, Defense=19
Level=22, HP=127, Defense=20
Level=23, HP=129, Defense=21
Level=24, HP=132, Defense=22
Level=25, HP=134, Defense=23
Level=26, HP=136, Defense=24
Level=27, HP=139, Defense=25
Level=28, HP=141, Defense=26
Level=29, HP=144, Defense=27
Level=30, HP=146, Defense=28
Level=31, HP=149, Defense=29
Level=32, HP=151, Defense=30
Level=33, HP=154, Defense=31
Level=34, HP=156, D

In [ ]:
import os
import csv
import re
import json
import numpy as np
import pandas as pd
from playwright.async_api import async_playwright
from tabulate import tabulate

# Sin Color 이름 변환을 위한 매핑
resource_mapping = {
    "AZURE": "Gloom",
    "VIOLET": "Envy",
    "AMBER": "Sloth",
    "SHAMROCK": "Gluttony",
    "CRIMSON": "Wrath",
    "SCARLET": "Lust",
    "INDIGO": "Pride"
}

def convert_resource(resource_name):
    """Sin Color 이름을 변환하는 함수. 매핑되지 않은 경우 원래 이름을 반환."""
    return resource_mapping.get(resource_name, resource_name)

def find_sin_for_color(url):
    # URL에 color_name을 추출
    color_name = url.split('/')[-1].split('.')[0].replace("attr_", "")
    # 해당 색상이 resource_mapping에 있는 확인
    for color, sin in resource_mapping.items():
        if color_name in color:
            return sin
    return None


class Scraper:
    """웹 페이지 크롤링을 위한 클래스"""
    def __init__(self, container, base_url, img_url):
        self.container = container
        self.base_url = base_url
        self.img_url = img_url
        self.uptie_level = 1
        self.keywords_set = set()

    async def handle_request(self, route, request):
        """이미지와 폰트 리소스 로드를 차단."""
        if request.resource_type =="image":
            #print(f"{request.url}")
            img_url = request.url
            if  re.match(f"{self.img_url}/ui/attr.*\\.png", img_url):
                await route.continue_()
            else:
                await route.fulfill(
                    status=200,
                    content_type="image/png",
                    body=b""
                )
        elif request.resource_type == "font":
            await route.abort()
        else:
            await route.continue_()
            
    async def scrape(self):
        """크롤링 메인 함수"""
        async with async_playwright() as p:
            browser = await p.chromium.launch(headless=False)
            page = await browser.new_page()
            #await page.evaluate('window.scrollTo(0, document.body.scrollHeight)')
            await page.route("**/*", self.handle_request)
            await page.wait_for_load_state("domcontentloaded")
            # Step 1: /identity 페이지에서 기본 정보 크롤링
            #await self.crawl_identity_page(page)

            # Step 2: /identity/{identity_id} 상세 페이지 크롤링
            #identity_ids_to_load = ["10104", "10310", "10710","10808", "10911", "11005"]
            #identity_ids_to_load = ["10104", "10911"]#,"10310", "10710","10808", "11005", "11210"]
            #identity_ids_to_load = ["10911","10310"]#, "10710","10808", "11005", "11210"]
            #identity_ids_to_load = ["10911", "10611", "10711"]#, "10710","10808", "11005", "11210"]
            #self.container.load_identities_by_id('./csv/identities.csv', identity_ids_to_load)
            self.container.load_all_identities('./csv/identities.csv')
            await self.crawl_identity_details_for_all(page)
            #self.container.display_identities()
            #self.container.export_identities_data("Identities.csv")
            self.container.export_level_stats_data("Levelstats.csv")
            self.container.export_uptie_stats_data("Uptiestats.csv")
            self.container.export_resist_info_data("ResistInfos.csv")
            self.container.export_stagger_threshold_data("StaggerThresholds.csv")
            self.container.export_identity_skillset_data("IdentitiesSkills.csv")
            self.container.export_identities_behaviors_data("IdentitiesBehaviors.csv")
            await self.crawl_keywords_details_for_collect(page)
            self.container.export_keywords_data("Keywords.csv")
            page.set_default_timeout(10000)
            #await page.wait_for_timeout(100000)
            await browser.close()

    async def get_identity_id_from_row(self, row):
        """테이블 행(row)에서 인격 ID를 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_url = await identity_cell.get_attribute('href')
            identity_id = identity_url.split('/identity/')[-1]
            return identity_id
        return "Unknown"

    async def get_identity_name_from_row(self, row):
        """테이블 행(row)에서 인격 이름을 추출."""
        identity_cell = await row.query_selector('a[href^="/identity/"]')
        if identity_cell:
            identity_name = (await identity_cell.inner_text()).replace("\n", " ").strip()
            return identity_name
        return "Unknown"
        
    async def crawl_identity_page(self, page):
        """/identity 페이지로 이동하여 인격 기본 정보를 추출."""
        await page.goto(self.base_url + "/identity")
        await self.extract_identity_page(page)

    async def extract_identity_page(self, page):
        """/identity 페이지의 모든 테이블 행에서 인격 정보를 추출."""
        identity_rows = await page.query_selector_all('tr')
        if len(identity_rows) > 1:
            for idx, identity_row in enumerate(identity_rows):
                if idx == 0:
                    continue # 0번째요소이름을 보여주는 열은 건너뜀
                await self.scrape_identity_name_and_identity_id(identity_row)
                await self.extract_rarity(identity_row)
                for i in range(1, 4):  # 스킬1~3 정보 추출
                    await self.extract_atk_and_resource(identity_row, i)
                await self.extract_defense_and_resource(identity_row)

    async def scrape_identity_name_and_identity_id(self, row):
        """인격 번호와 이름을 추출하여 DataContainer에 저장."""
        try:
            identity_id = await self.get_identity_id_from_row(row)
            identity_name = await self.get_identity_name_from_row(row)
            identity = Identity(identity_id)
            self.container.add_identity(identity)
            self.container.update_identity_name(identity_id, identity_name)
        except Exception as e:
            print(f"Error extracting identity name and identity id: {e}")

    async def get_element_attribute(self, element, selector, attribute):
        """요소에서 지정된 속성(attribute)을 추출."""
        try:
            target_element = await element.query_selector(selector)
            if target_element:
                return await target_element.get_attribute(attribute)
        except Exception as e:
            print(f"Error getting element attribute: {e}")
        return None
    
    async def extract_rarity(self, row):
        """테이블 행(row)에서 희귀도를 추출하여 업데이트."""
        try:
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
                sibling = await parent.evaluate_handle('node => node.nextElementSibling')
                if sibling:
                    rarity_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/rarity_"]', 'src')
                    if rarity_img_src:
                        rarity_level = rarity_img_src.split('_')[-1].split('.')[0]  # rarity_1.png -> 1
                        identity_id = await self.get_identity_id_from_row(row)
                        self.container.update_identity_rarity(identity_id, rarity_level)
                    else:
                        print("Rarity image not found in the next sibling TD.")
        except Exception as e: 
            print(f"Error extracting rarity: {e}")

        return "Unknown"

    async def extract_atk_and_resource(self, row, skill_number):
        """ 공격 타입(ATK Type)과 자원(Attribute) 추출 함수 - 스킬 번호에 따라 처리 """
        try:
            #print(f"Attempting to locate skill{skill_number} type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 해당 스킬 번호에 맞는 형제로 이동
                sibling = parent
                for _ in range(skill_number + 1):  # 스킬 번호에 맞게 이동
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                atk_type, resource = "Unknown", "Unknown"
        
                if sibling:
                    # 공격 타입 이미지 (atk_type) 찾기
                    atk_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/atk_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if atk_type_img_src:
                        atk_type = atk_type_img_src.split('_')[-1].split('.')[0]  # atk_type_SLASH.png -> SLASH
                        #print(f"Extracted attack type: {atk_type}")
                        self.container.update_skill(identity_id, skill_number, atk_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted resource: {resource}")
                        
                        # 리소스를 변환
                        resource = convert_resource(resource)
                        #print(f"Converted resource: {resource}")
                        self.container.update_sin_affinity(identity_id, skill_number, resource)
                        
        except Exception as e:
            print(f"Error extracting skill{skill_number} type and resource: {e}")

    async def extract_defense_and_resource(self, row):
        """ 방어 타입(Def Type)과 자원(Attribute) 추출 함수 """
        try:
            #print("Attempting to locate defense type and resource...")
        
            # <a> 태그의 부모 <td>로 이동
            parent_td = await row.query_selector('a[href^="/identity/"]')
            if parent_td:
                # 부모 td에서 형제로 이동
                parent = await parent_td.evaluate_handle('(node) => node.parentElement')
        
                # 다섯 번째 형제 td로 이동 (방어 관련 정보가 있는 <td>)
                sibling = parent
                for _ in range(5):  # 방어 정보는 5번째 형제에 있음
                    sibling = await sibling.evaluate_handle('node => node.nextElementSibling')
        
                def_type, def_resource = "Unknown", "Unknown"
        
                if sibling:
                    # 방어 타입 이미지 (def_type) 찾기
                    def_type_img_src = await self.get_element_attribute(sibling, f'img[src^="{self.img_url}/ui/def_type_"]', 'src')
                    identity_id = await self.get_identity_id_from_row(row)
                    if def_type_img_src:
                        def_type = def_type_img_src.split('_')[-1].split('.')[0]  # def_type_GUARD.png -> GUARD
                        #print(f"Extracted defense type: {def_type}")
                        self.container.update_skill(identity_id, 'defense', def_type)
        
                    # 자원 이미지 (resource) 찾기
                    resource_img_src = await self.get_element_attribute(sibling, f'div.q-py-xs img[src^="{self.img_url}/ui/attr_"]', 'src')
                    if resource_img_src:
                        def_resource = resource_img_src.split('_')[-1].split('.')[0]  # attr_AZURE.png -> AZURE
                        #print(f"Extracted defense resource: {def_resource}")
                        
                        # 방어 리소스를 변환
                        def_resource = convert_resource(def_resource)
                        #print(f"Converted defense resource: {def_resource}")
                        self.container.update_sin_affinity(identity_id, 'defense', def_resource)
 
        except Exception as e:
            print(f"Error extracting defense type and resource: {e}")

    async def crawl_identity_details_for_all(self, page):
        """모든 인격의 세부 정보를 크롤링."""
        for identity in self.container.identities:
            await page.wait_for_load_state("load")
            await page.goto(self.base_url + "/identity/"+identity.identity_id)
            await self.initialize_uptie_and_level(page)
            #""" 레벨과 동기화와 관련 없는 참관타 저항은 밑에서 변경하는 것과 연관성이 적기 때문에 위에서 처리한다.
            print(f"Identity_id: {identity.identity_id}")
            results = []
            resist_type_imgs = [f"{self.img_url}/ui/atk_type_HIT.png",
                                f"{self.img_url}/ui/atk_type_PENETRATE.png",
                                f"{self.img_url}/ui/atk_type_SLASH.png"]
            for resist_type_img in resist_type_imgs:
                result = await self.extract_resist_type(page, resist_type_img, 1)
                results.append(result)
            self.container.add_resist_info(identity.identity_id, results[0], results[1], results[2])
            #"""
            #"""
            # Stagger Threshold 값 추출
            stagger_threshold = await self.extract_stagger_threshold_values(page)
            self.container.add_stagger_threshold(identity.identity_id, stagger_threshold[0], stagger_threshold[1], stagger_threshold[2])
            #"""
            #await self.initialize_uptie_and_level(page)
            #""" 레벨에 따른 level, hp, defense 다르게 되어 있는 것을 출력 확인 객체로 들어가서 csv로 옮기는 과정까지 진행
            for i in range(1, identity.level_max+1):
                await self.move_slider_to_value(page,i)
                await self.extract_level_status(page, identity.identity_id, i)
            #"""
            #"""
            #    인격에 대한 skill 정보를 가져오는 메소드 위에서 출력하는 csv파일로 인격이 가지고 있는 스킬 정보를 내보낸다.
            await self.crawl_speed_and_skills(page, identity.identity_id)
            #"""
            await self.crawl_influence_behavior(page, identity.identity_id)

    async def crawl_keywords_details_for_collect(self, page):
        """모아진 키워드의 세부 정보를 크롤링."""
        for keyword in self.keywords_set:
            await page.goto(self.base_url + "/keyword/"+keyword)
            await self.crawl_keyword_info(page, keyword)
            
    async def initialize_uptie_and_level(self, page):
        await self.uptie_click(page, 1)
        await self.move_slider_to_value(page, 1)
        self.uptie_level = 1
        #return 1
    
    async def uptie_click(self, page, phase):
        await page.wait_for_load_state("domcontentloaded")
        fab_button_selector = ".q-btn.q-btn--fab"
        await page.wait_for_selector(fab_button_selector)
        await page.click(fab_button_selector)
        #print(f"[{setup_uptie_button}] 상위 동기화 버튼 클릭 완료")
        setup_uptie_button = f'a:has(img[src="{self.img_url}/ui/uptie_{phase}.png"])'
        await page.wait_for_selector(setup_uptie_button)
        await page.click(setup_uptie_button)
        #print(f"{phase} 동기화 버튼 클릭 완료")

    async def move_slider_to_value(self, page, target_value):
        """
        슬라이더를 주어진 값으로 이동시킵니다.
        target_value는 슬라이더의 최소값부터 최대값 사이여야 합니다.
        """
        slider_selector = ".q-slider"
        # 슬라이더의 범위 확인
        min_value = int(await page.get_attribute(slider_selector, "aria-valuemin"))
        max_value = int(await page.get_attribute(slider_selector, "aria-valuemax"))
    
        if target_value < min_value or target_value > max_value:
            raise ValueError(f"Target value {target_value} is out of bounds ({min_value}, {max_value})")
    
        # 슬라이더 thumb 요소와 위치 가져오기
        slider_thumb = await page.query_selector(f"{slider_selector} .q-slider__thumb")
        thumb_box = await slider_thumb.bounding_box()
        
        # 슬라이더의 현재 위치 및 목표 위치 계산
        track = await page.query_selector(f"{slider_selector} .q-slider__track")
        track_box = await track.bounding_box()
        
        # 목표 x 위치 계산 (슬라이더 트랙 내에서 비율 기반)
        target_percentage = (target_value - min_value) / (max_value - min_value)
        target_x = track_box["x"] + target_percentage * track_box["width"]
        current_x = thumb_box["x"] + thumb_box["width"] / 2
        y = thumb_box["y"] + thumb_box["height"] / 2
    
        # 슬라이더를 드래그하여 목표 위치로 이동
        await page.mouse.move(current_x, y)
        await page.mouse.down()
        await page.mouse.move(target_x, y, steps=10)  # 부드럽게 이동
        await page.mouse.up()
    
        #print(f"슬라이더를 {target_value}로 이동 완료.")
    
        # 슬라이더 값 확인 및 강제 이벤트 트리거
        await page.evaluate(
            f"""
            (slider) => {{
                slider.dispatchEvent(new Event('input'));  // oninput 이벤트 트리거
                slider.dispatchEvent(new Event('change'));  // onchange 이벤트 트리거
            }}
            """,
            await page.query_selector(slider_selector)
        )
    
        # 이동 후 값 확인
        current_value = await page.get_attribute(slider_selector, "aria-valuenow")
        if current_value != str(target_value):
            print(f"경고: 슬라이더 값이 예상과 다릅니다. 현재 값: {current_value}, 예상 값: {target_value}")

    async def get_second_status_element(self, page):
        """
        두 번째 'Status' 요소에 접근하여 반환하는 메소드
        """
        status_elements = await page.query_selector_all("text=Status")
        if len(status_elements) > 1:
            second_status_element = status_elements[1]
            #print("두 번째 'Status' 요소에 접근합니다.")
            return second_status_element
        else:
            #print("두 번째 'Status' 요소가 존재하지 않습니다.")
            return None

    async def extract_level_status(self, page, identity_id, level):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 레벨에 따라 HP와 Defense가 변한다.
        hp_value = await self.extract_hp(second_status_element)
        defense_value = await self.extract_defense(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Level={level}, HP={hp_value}, Defense={defense_value}")
        self.container.add_level_stat(identity_id,level, hp_value, defense_value)
    
    async def extract_hp(self, status_element):
        # HP 추출
        hp_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].children[0].nextElementSibling.textContent.trim()"
        )
        return hp_value
    
    async def extract_defense(self, status_element):
        # Defense 추출
        defense_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return defense_value

    async def extract_uptie_status(self, page, identity_id, uptie):
        second_status_element = await self.get_second_status_element(page)

        # 각각의 값을 추출하는 함수 호출 uptie에 따라 speed는 변한다.
        speed_value = await self.extract_speed(second_status_element)

        # 값이 제대로 추출되었는지 확인
        print(f"Uptie={uptie}, Speed={speed_value}")
        self.container.add_uptie_stat(identity_id, uptie, speed_value)

    async def extract_speed(self, status_element):
        # Speed 추출
        speed_value = await status_element.evaluate(
            "node => node.nextElementSibling.children[0].nextElementSibling.children[0].nextElementSibling.textContent.trim()"
        )
        return speed_value

    async def extract_resist_type(self, page, image_src, depth):
        # 이미지 요소 찾기
        #print(image_src)
        image = await page.query_selector(f'img[src="{image_src}"]')
        #print(image)
        if not image:
            return "Unknown"
    
        # 이미지의 부모 요소를 찾아서 올라감
        parent = image
        for _ in range(depth):
            parent = await parent.evaluate_handle('node => node.parentElement')
            if not parent:
                return "Unknown"
    
        # 부모의 형제 요소 찾기
        sibling = await parent.evaluate_handle('node => node.nextElementSibling')
        
        if sibling:
            value = await sibling.evaluate('node => node.textContent')
            if value:
                return value.strip()
        
        return "Unknown"

    async def extract_stagger_threshold_values(self, page):
        # 'div.row.center.flex-center.q-pt-xs' 요소들 찾기
        elements = await page.query_selector_all('div.row.center.flex-center.q-pt-xs')
        
        # 각 요소의 텍스트 내용 출력하여 'Stagger Threshold' 요소를 찾기
        for element in elements:
            text = await element.text_content()
            
            if text and "Stagger Threshold" in text:
                # 'Stagger Threshold'가 있는 요소를 찾았다면, 그 이후 형제 요소들에서 값을 추출
                parent = await element.evaluate_handle('node => node.parentElement')
                sibling_elements = await parent.query_selector_all('div.row.justify-evenly div')  # Threshold 값이 들어있는 div들
                
                stagger_values = []
                for sibling in sibling_elements:
                    text_content = await sibling.text_content()
                    match = re.search(r'(\d+%)', text_content)
                    if match:
                        stagger_values.append(match.group(1))  # 퍼센트 값만 리스트에 추가
                while len(stagger_values) < 3:
                    stagger_values.append(np.nan)
                #print(stagger_values)
                return stagger_values  # 추출된 값(또는 np.nan 포함)을 반환
        
        return [np.nan, np.nan, np.nan]  # 만약 'Stagger Threshold'를 찾지 못한 경우 기본값 반환

    async def crawl_speed_and_skills(self, page, identity_id):
        for uptie_value in range(1, 5):  # Uptie levels 1 to 4
            # 페이지 내 uptie 변경 버튼 클릭
            await self.uptie_click(page, uptie_value)
            await self.crawl_skill(page, identity_id, "skill 1", uptie_value)
            await self.crawl_skill(page, identity_id, "skill 2", uptie_value)
            await self.crawl_skill(page, identity_id, "skill 3", uptie_value)
            await self.crawl_skill(page, identity_id, "defense", uptie_value)
            await self.extract_uptie_status(page, identity_id, uptie_value)

    async def crawl_skill(self, page, identity_id, skill_type, uptie_value):
        skills_elements = await page.query_selector_all('div.skilltag')
        for skill_element in skills_elements:
            skill_text = await skill_element.text_content()
            if skill_text.strip().lower() == skill_type:
                await self.process_skill(skill_element, page, identity_id, skill_type, uptie_value)
        
    async def process_skill(self, skill_element, page, identity_id, skill_type, uptie_value):
        """
        개별 스킬 요소를 처리하고 관련 데이터를 저장.
        """
        # "skill 3"은 Uptie 3 이상일 때만 처리
        if skill_type == "skill 3" and uptie_value < 3:
            return

        # 스킬의 세부 정보를 추출
        skill_details = await self.extract_skill_details(skill_element, skill_type)
        skill_name = skill_details["skill_name"]

        # 스킬 정보를 저장
        self.container.add_skill(
            identity_id, skill_type,
            skill_name, skill_details["coin_count"],
            skill_details["init_level"], skill_details["init_remain"],
            skill_details["atk_weight"]
        )
        print(f"Added skill: {skill_type} - {skill_name} at Uptie {uptie_value}")

        # Uptie 정보 갱신
        await self.update_uptie_info(
            page, skill_element, identity_id, skill_name, skill_type, uptie_value
        )
    
    async def extract_skill_details(self, skill_element, skill_type):
        # 스킬 세부 정보를 추출하는 함수
        skill_name = await self.skill_name(skill_element)
        coin_count = await self.coin_count(skill_element, ".col-auto > .column > .row")
        init_level = await self.init_level(skill_element)
        init_remain = await self.init_remain(skill_element, skill_type, "div.row.justify-start > div.attr.flex.flex-center > div.q-px-sm")
        atk_weight = await self.atk_weight(skill_element, skill_type)
        return {
            "skill_name": skill_name,
            "coin_count": coin_count,
            "init_level": init_level,
            "init_remain": init_remain,
            "atk_weight": atk_weight
        }
    
    async def update_uptie_info(self, page, skill_element, identity_id, skill_name, skill_type, uptie_value):
        # Uptie 정보를 추출하고 업데이트하는 함수
        uptie_info = await self.extract_uptie_info(skill_element)
        effect_data = await self.skill_effects(skill_element)
        self.container.add_skill_uptie_info(
            identity_id, skill_name, 
            uptie_value, uptie_info["skill_power"], uptie_info["coin_power"], 
            effect_data.get("effectsByCondition")
        )
        keywords = effect_data.get("keywords")
        if keywords:
            self.keywords_set.update(keywords)
        print(
            f"Updated Uptie for {skill_name} (Uptie Level: {uptie_value}): "
            f"Skill Power: {uptie_info['skill_power']}, Coin Power: {uptie_info['coin_power']} "
            f"Skill Effect: {effect_data.get("effectsByCondition")}"
        )

    async def extract_uptie_info(self, skill_element):
        skill_data = await self.skill_and_coin_power(skill_element)
        return {
            "skill_power": skill_data.get("skillPower"),
            "coin_power": skill_data.get("coinPower")
        }
        
    async def crawl_influence_behavior(self, page, identity_id):
        influence_behaviors = await page.query_selector_all('div[align="right"]')
        # 텍스트를 기준으 추출하고 메소드로 파라미터 들어가 정보를 받는다.
        for influence_behavior in influence_behaviors:
            active_type = await influence_behavior.inner_text()
            active_type = active_type.strip()

            if active_type == "PASSIVE":
                """
                # PASSIVE에 대한 처리
                print(f"Processing PASSIVE logic for: {active_type}")
                # CALL method for PASSIVE
                """
                await self.passive_effects(influence_behavior, identity_id)
            elif active_type == "SUPPORT PASSIVE":
                """
                # SUPPORT PASSIVE에 대한 처리
                print(f"Processing SUPPORT PASSIVE logic for: {active_type}")
                # CALL method for SUPPORT PASSIVE
                """
                await self.support_passive_effects(influence_behavior, identity_id)
            elif active_type == "PANIC TYPE":
                """
                # PANIC TYPE에 대한 처리
                print(f"Processing PANIC TYPE logic for:{active_type}")
                # CALL method for PANIC TYPE
                """
                await self.panic_effects(influence_behavior, identity_id)

    async def skill_and_coin_power(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                // 조부모로 이동
                const grandparent = node.parentElement.parentElement;
                // 이전 형제 탐색
                const previousSibling = grandparent.previousElementSibling;
                if (!previousSibling) return {{ skillPower: null, coinPower: null }}; // 이전 형제가 없으면 기본값 반환
                // 이전 형제 내부의 .row.justify-around > div > div 탐색
                const targetDivs = previousSibling.querySelectorAll(".row.justify-around > div");
                if (targetDivs.length < 3) return {{ skillPower: null, coinPower: null }}; // 배열 길이 확인
                // 1st div: 스킬 위력
                const skillPower = targetDivs[0]?.querySelector('div')?.textContent?.trim() || null; 
                // 3rd div: 코인 파워 값 (3번째 div에서 텍스트 추출)
                const coinPower = targetDivs[2]?.textContent?.trim() || null;
                return {{ skillPower, coinPower }};
            }}
        """)


    async def coin_count(self, skill_element, hierarchy_selector):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector('{hierarchy_selector}');
                if (!targetElement) return 0;
                return targetElement.childElementCount;
            }}
        """)

    async def init_remain(self, skill_element, skill_type, hierarchy_selector):
        if skill_type == "defense":
            return -1
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector('{hierarchy_selector}');
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def skill_name(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector(".block");
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def init_level(self, skill_element):
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector("div.text-bold.text-h6.h6.q-pl-xs.q-pr-md");
                if (!targetElement) return 0;
                return targetElement.textContent?.trim();
            }}
        """)

    async def atk_weight(self, skill_element, skill_type):
        if skill_type == "defense":
            return 0
        return await skill_element.evaluate(f"""
            node => {{
                const parent = node.parentElement;
                const nextSibling = parent.nextElementSibling;
                if (!nextSibling) return null;
                const targetElement = nextSibling.querySelector("div.text-bold.text-h6.h6.q-pl-xs.q-pr-md");
                const doubleNextSibling = targetElement.nextElementSibling.nextElementSibling;
                if (!doubleNextSibling) return 0;
                return doubleNextSibling.textContent?.trim();
            }}
        """)

    async def skill_effects(self, skill_element):
        return await skill_element.evaluate("""
            node => {
                const parent = node.parentElement;
                const doubleNextSibling = parent.nextElementSibling?.nextElementSibling;
                if (!doubleNextSibling) return null;
    
                // doubleNextSibling에서 첫 번째 div.col 요소 찾기
                const textNode = doubleNextSibling.querySelector("div.col");
                if (!textNode) return null;
    
                const results = {
                    keywords: [], // 키워드 저장
                    conditions: [], // 조건 저장
                    effectsByCondition: {} // 조건별 효과
                };
    
                // 1. 조건 추출 (대괄호 텍스트)
                const conditionElements = textNode.querySelectorAll("span");
                conditionElements.forEach(span => {
                    let text = span.textContent?.trim();
                    if (text?.startsWith("[") && text?.endsWith("]")) {
                        // 부모의 전형제 노드 탐색
                        const sibling = span.parentElement.previousElementSibling;
                        if (sibling) {
                            const emblemImage = sibling.querySelector("img.emblem");
                            console.log("Sibling element found:", sibling); // 디버깅용
                            if (emblemImage) {
                                const src = emblemImage.getAttribute("src");
                                console.log("Image source:", src); // 디버깅용
                                const match = src.match(/ui_num_(\\d+)\\.png$/); // 정규식 수정
                                if (match) {
                                    const number = match[1]; // 숫자 추출
                                    text = `[Coin ${number} ${text.replace("[", "").replace("]", "")}]`;
                                    console.log("Updated condition:", text); // 디버깅용
                                }
                            }
                        }
                        results.conditions.push(text);
                        if (!results.effectsByCondition[text]) {
                            results.effectsByCondition[text] = []; // 조건별 효과를 초기화
                        }
                    }
                });
    
                // 2. 키워드 추출
                const keywordElements = textNode.querySelectorAll('a[href^="/en/keyword/"]');
                if (!keywordElements) return null;

                // 키워드 매핑 객체 초기화
                if (!results.keywordMapping) {
                    results.keywordMapping = {}; //텍스트: 키 형태로 저장
                }

                keywordElements.forEach(a => {
                    const span = a.querySelector('span.q-btn__content.text-center.col.items-center.q-anchor--skip.justify-center.row>span');
                    if (span) {
                        const keywordText = span.textContent.trim();
                        const href = a.getAttribute('href');
                        const keywordKey = href.split('/').pop();
                        if (keywordKey && !Object.values(results.keywordMapping).includes(keywordKey)) {
                            results.keywordMapping[keywordText] = keywordKey;
                        }
                    }
                });
    
                // 3. 효과 추출
                const effectElements = textNode.querySelectorAll("span");
                let currentCondition = null;
                let currentEffect = []; // 초기화
                let keywordList = new Set(Object.values(results.keywordMapping));
                effectElements.forEach(span => {
                    let text = span.textContent?.trim();
                    if (!text) return;
    
                    // 조건 발견 시 currentCondition 변경
                    if (text.startsWith("[") && text.endsWith("]")) {
                        // 이전 조건의 효과 저장
                        if (currentCondition && currentEffect.length > 0) {
                            if (!results.effectsByCondition[currentCondition]) {
                                results.effectsByCondition[currentCondition] = [];
                            }
                            // 부모의 전형제 노드 탐색
                            const sibling = span.parentElement.previousElementSibling;
                            if (sibling) {
                                const emblemImage = sibling.querySelector("img.emblem");
                                if (emblemImage) {
                                    const src = emblemImage.getAttribute("src");
                                    const match = src.match(/ui_num_(\\d+)\\.png$/); // 정규식 수정
                                    if (match) {
                                        const number = match[1]; // 숫자 추출
                                        text = `[Coin ${number} ${text.replace("[", "").replace("]", "")}]`;
                                    }
                                }
                            }
                            results.effectsByCondition[currentCondition].push(currentEffect.join(' '));
                            currentEffect = []; // 초기화
                        }
                        currentCondition = text;
                        if (!results.effectsByCondition[currentCondition]) {
                            results.effectsByCondition[currentCondition] = [];
                        }
                        return;
                    }
    
                    // 현재 조건의 효과 처리
                    if (currentCondition) {
                        if (results.keywordMapping[text]) {
                            const keywordEffect = `keyword:${results.keywordMapping[text]}`;
                            keywordList.add(results.keywordMapping[text]);
                            if (!currentEffect.includes(keywordEffect)) {
                                currentEffect.push(keywordEffect);
                            }
                        } else {
                            currentEffect.push(text);
                        }
                    }
                });

                results.keywords = Array.from(keywordList);
                delete results.keywordMapping;
                // 마지막 효과 저장
                if (currentCondition && currentEffect.length > 0) {
                    let processedText = currentEffect
                        .map(word => word.trim())
                        .join(" ")
                        .replace(/\\s+([,.:])/g, "$1")
                        .replace(/\\s+'/g, "'")
                        .replace(/'\\s+/g, "'")
                        .replace(/\\[\\s+/g, "[")
                        .replace(/\\s+\\]/g, "]")
                        .replace(/\\(\\s+/g, "(")
                        .replace(/\\s+\\)/g, ")")
                        .replace(/\\s+%/g, "%")
                        .replace(/\\s+\\//g, "/")
                        .replace(/\\/\\s+/g, "/")
                        .replace(/\\s+\\/\\s+/g, "/");
                    if (!results.effectsByCondition[currentCondition]) {
                        results.effectsByCondition[currentCondition] = [];
                    }
                    results.effectsByCondition[currentCondition].push(processedText);
                }
    
                return results;
            }
        """)

    async def passive_effects(self, passive_element, identity_id):
        passive_name = await self.behavior_name(passive_element)
        passive_conditions = await self.behavior_condition(passive_element)
        passive_description = await self.behavior_description(passive_element, "span")
        #print(passive_name)
        #print(passive_conditions)
        #print(passive_description)
        self.container.add_influence_behavior(identity_id, "passive", passive_name, passive_conditions, passive_description.get("description"))
        keywords = passive_description.get("keywords")
        if keywords:
            self.keywords_set.update(keywords)
    async def support_passive_effects(self, support_passive_element, identity_id):
        support_passive_name = await self.behavior_name(support_passive_element)
        support_passive_conditions = await self.behavior_condition(support_passive_element)
        support_passive_description = await self.behavior_description(support_passive_element, "span")
        #print(support_passive_name)
        #print(support_passive_conditions)
        #print(support_passive_description)
        self.container.add_influence_behavior(identity_id, "support passive", support_passive_name, support_passive_conditions, support_passive_description.get("description"))
        keywords = support_passive_description.get("keywords")
        if keywords:
            self.keywords_set.update(keywords)
    async def panic_effects(self, panic_element, identity_id):
        panic_name = await self.behavior_name(panic_element)
        panic_conditions = await self.behavior_condition(panic_element)
        panic_description = await self.behavior_description(panic_element, "div")
        #print(panic_type)
        #print(panic_conditions)
        #print(panic_description)
        self.container.add_influence_behavior(identity_id, "panic type", panic_name, panic_conditions, panic_description.get("description"))
        keywords = panic_description.get("keywords")
        if keywords:
            self.keywords_set.update(keywords)

    async def navigate_element_by_steps(self, element, parent_steps=0, prev_sibling_steps=0, next_sibling_steps=0):
        """
        주어진 `element`에서 부모, 이전 형제, 또는 다음 형제로 지정된 횟수만큼 이동하여 ElementHandle 반환.
        
        Args:
            element (ElementHandle): 탐색 기준이 되는 요소.
            parent_steps (int): 부모로 이동할 횟수 (0이면 이동하지 않음).
            prev_sibling_steps (int): 이전 형제로 이동할 횟수 (0이면 이동하지 않음).
            next_sibling_steps (int): 다음 형제로 이동할 횟수 (0이면 이동하지 않음).
        
        Returns:
            ElementHandle or None: 탐색된 요소를 반환하거나 없으면 None.
        """
        try:
            current_element = element
    
            # 부모로 이동
            for _ in range(parent_steps):
                current_element = await current_element.evaluate_handle('el => el.parentElement')
                if not current_element:
                    print("Parent element not found during navigation.")
                    return None
    
            # 이전 형제로 이동
            for _ in range(prev_sibling_steps):
                current_element = await current_element.evaluate_handle('el => el.previousElementSibling')
                if not current_element:
                    print("Previous sibling element not found during navigation.")
                    return None
    
            # 다음 형제로 이동
            for _ in range(next_sibling_steps):
                current_element = await current_element.evaluate_handle('el => el.nextElementSibling')
                if not current_element:
                    print("Next sibling element not found during navigation.")
                    return None
    
            return current_element
    
        except Exception as e:
            print(f"Error navigating elements: {e}")
            return None

    async def get_last_sibling(self, element):
        try:
            # 같은 부 아 마지막 형제를 가져오는 JavaScript 실행
            last_sibling = await element.evaluate_handle('el => el.parentElement.lastElementChild')
            return last_sibling
        except Exception as e:
            print(f"Error getting last sibling: {e}")
            return None

    async def behavior_name(self, element):
        parent_prev_sibling = await self.navigate_element_by_steps(element, parent_steps=1, prev_sibling_steps=1, next_sibling_steps=0)
        passive_name_element = await parent_prev_sibling.query_selector('span.block')
        if passive_name_element:
            passive_name = await passive_name_element.inner_text()
            return passive_name
        else:
            #print("Passive name element not found")
            return None

    async def behavior_condition(self, element):
        try:
            conditions = []
            parent_prev_sibling = await self.navigate_element_by_steps(element, parent_steps=1, prev_sibling_steps=1, next_sibling_steps=0)
            img_elements = await parent_prev_sibling.query_selector_all('img')
            if not img_elements:
                #print("No <img> elements found.")
                return conditions
    
            for img_element in img_elements:
                # 각 이미지의 src 속성 가져오기
                img_src = await img_element.get_attribute('src')
                #print(img_src)
    
                sin_name = None
                for color_key in resource_mapping.keys():
                    if f"{self.img_url}/ui/attr_{color_key}.png" in img_src:
                        sin_name = resource_mapping[color_key]
                        break
    
                if not sin_name:
                    #print(f"No matching Sin Color found for {img_src}.")
                    continue  # No Sin Color found, skip this img_element
    
                # 해당 이미지와 관련된 조건 텍스트 추출
                # 두 번째 부모 요소에서 시작하여 다음 형제를 찾아 조건 텍스트를 추출
                condition_element = await self.navigate_element_by_steps(img_element, parent_steps=3, prev_sibling_steps=0, next_sibling_steps=0)
                if condition_element:
                    # condition_element가 None이 아닌 경우에만 진행
                    condition_text_element = await condition_element.query_selector('.q-pl-xs')
                    if condition_text_element:
                        condition_text = await condition_text_element.inner_text()
                        condition_text = condition_text.strip()
                    else:
                        print(f"No condition text found for {img_src}.")
                        condition_text = ""  # condition_text_element가 None이면 빈 문자열로 처리
                else:
                    print(f"No condition element found for {img_src}.")
                    condition_text = ""  # condition_element가 None이면 빈 문자열로 처리
    
                # 결과를 배열에 추가
                conditions.append(f"{sin_name} {condition_text}")
            return conditions
        except Exception as e:
            print(f"Error extracting conditions: {e}")
            return 
            
    async def behavior_description(self, element, tag_type):
        parent_prev_sibling = await self.navigate_element_by_steps(
            element, parent_steps=1, prev_sibling_steps=1, next_sibling_steps=0
        )
        return await parent_prev_sibling.evaluate(f"""
            (node) => {{
                const behavior_title_node = node.querySelector('.q-pb-sm');
                const behavior_title_parent = behavior_title_node.parentElement;
                const behavior_description = behavior_title_parent.lastElementChild;
                if (!behavior_description) return null;
    
                const results = {{
                    keywords: [], // 키워드 저장
                    description: [] // 설명 저장
                }};
                
                // 매핑을 저장할 객체
                const keywordElements = behavior_description.querySelectorAll('a[href^="/en/keyword/"]');
                if (!keywordElements) return null;
                
                // 키워드 매핑 객체 초기화
                if (!results.keywordMapping) {{
                    results.keywordMapping = {{}}; // 텍스트: 키 형태로 저장
                }}
                
                keywordElements.forEach(a => {{
                    const span = a.querySelector('span.q-btn__content.text-center.col.items-center.q-anchor--skip.justify-center.row>span');
                    if (span) {{
                        const keywordText = span.textContent.trim(); // UI에 표시된 텍스트 (예: "Ammo")
                        const href = a.getAttribute('href'); // href 속성 값 (예: "/en/keyword/Bullet")
                        const keywordKey = href.split('/').pop(); // href에서 마지막 부분 추출 (예: "Bullet")
                
                        if (keywordKey && !Object.values(results.keywordMapping).includes(keywordKey)) {{
                            results.keywordMapping[keywordText] = keywordKey; // 텍스트: 키로 매핑
                        }}
                    }}
                }});

                // 2. 설명 추출
                const descriptionElements = behavior_description.querySelectorAll("{tag_type}");
                let currentEffect = []; // 초기화
                let keywordsList = new Set(Object.values(results.keywordMapping)); // keywordMapping 값으로 Set 생성
            
                descriptionElements.forEach(span => {{
                    let text = span.textContent?.trim();
                    if (!text) return;
            
                    // 공백과 기호 처리
                    text = text.replace(/\\s*([.,:;!?()\\[\\]])\\s*/g, '$1');
            
                    // 텍스트가 매핑된 키워드인지 확인
                    if (results.keywordMapping[text]) {{
                        const keywordEffect = `keyword:${{results.keywordMapping[text]}}`; // 매핑된 키 사용
                        keywordsList.add(results.keywordMapping[text]); // keywordsList에 추가
                        if (!currentEffect.includes(keywordEffect)) {{
                            currentEffect.push(keywordEffect);
                        }}
                    }} else {{
                        currentEffect.push(text);
                    }}
                }});
            
                // keywords 배열로 변환
                results.keywords = Array.from(keywordsList); // Set에서 배열로 변환하여 저장
    
                // 마지막 효과 저장
                if (currentEffect.length > 0) {{
                    const finalDescription = currentEffect.join(' ');
                    //console.log("After process:", finalDescription);
                    results.description.push(finalDescription);
                }}
                delete results.keywordMapping;
                let processedText = results.description
                    .map(word => word.trim())
                    .join(" ")
                    .replace(/\\s+([,.:])/g, "$1")
                    .replace(/\\s+'/g, "'")
                    .replace(/'\\s+/g, "'")
                    .replace(/\\[\\s+/g, "[")
                    .replace(/\\s+\\]/g, "]")
                    .replace(/\\(\\s+/g, "(")
                    .replace(/\\s+\\)/g, ")")
                    .replace(/\\s+%/g, "%")
                    .replace(/\\s+\\//g, "/")
                    .replace(/\\/\\s+/g, "/")
                    .replace(/\\s+\\/\\s+/g, "/");
                results.description = processedText;
                return results;
            }}
        """)

    async def crawl_keyword_info(self, page, keyword):
        # 키워드 설명 추출
        keyword_explain_elements = await page.query_selector_all('div.text-caption.text-grey > div')
        keyword_explains = []
        keyword_explain_text = np.nan
        for keyword_explain_element in keyword_explain_elements:
            keyword_explain_text = await keyword_explain_element.inner_text()
            keyword_explain_text = keyword_explain_text.strip()
            keyword_explains.append(keyword_explain_text)
    
        # Dispellable 값과 그 이후 텍스트 추출
        grand_parent = await self.navigate_element_by_steps(
            keyword_explain_elements[0],
            parent_steps=2, 
            prev_sibling_steps=0, 
            next_sibling_steps=0
        )
        
        # Dispellable 요소 찾기
        dispellable_element = await grand_parent.query_selector('div.q-pt-sm > div')
        dispellable_value = None
        is_dispellable = np.nan  # 초기값을 np.nan으로 설정

        if dispellable_element:
            dispellable_value = await dispellable_element.inner_text()
            dispellable_value = dispellable_value.strip()
            
            # Dispellable 값 처리
            if ':' in dispellable_value:
                dispellable_state = dispellable_value.split(':', 1)[1].strip().lower()
                if dispellable_state == 'true':
                    is_dispellable = True
                elif dispellable_state == 'false':
                    is_dispellable = False
                else:
                    is_dispellable = np.nan  # 'true'나 'false'가 아닌 경우 np.nan으로 설정
        
            # 결과 출력
            #print(f"Dispellable: {is_dispellable}, After Colon Text: {after_colon_text}")
        
        # 컨테이너에 저장
        self.container.add_keyword(keyword, keyword_explains, is_dispellable)

class Identity:
    """인격 정보를 저장하는 클래스"""
    def __init__(self, identity_id, name=None, rarity=None,\
                 skill_1_type=None, skill_1_sin_affinity=None,\
                 skill_2_type=None, skill_2_sin_affinity=None,\
                 skill_3_type=None, skill_3_sin_affinity=None,\
                 defense_type=None, defense_sin_affinity=None):
        self.identity_id = identity_id
        self.name = name
        self.rarity = rarity
        # 선언만 하는 부분 
        self.skill_1_type = skill_1_type 
        self.skill_1_sin_affinity = skill_1_sin_affinity 
        self.skill_2_type = skill_2_type 
        self.skill_2_sin_affinity = skill_2_sin_affinity 
        self.skill_3_type = skill_3_type 
        self.skill_3_sin_affinity = skill_3_sin_affinity 
        self.defense_type = defense_type 
        self.defense_sin_affinity = defense_sin_affinity
        self.level_max = 50
        self.uptie_max = 4
        self.level_stats = []  # LevelStat 객체 리스트
        self.uptie_stats = [] # UptieStat 객체 리스트
        self.resist_info = None
        self.stagger_threshold = None
        self.skillset = SkillSet()  # Skill정보를 담고 있는 skillset 객체 
        self.passive_infos = []  # 패시브 객체를 담고 있는 배열
        self.support_passive_infos = [] # 서포트 패시 객체를 담고 있는 배열
        self.panic_type = [] # 혼란상태 발생시 나타나는 효과를 담고 있는 객체 배열1

    def update_name(self, name):
        self.name = name

    def update_rarity(self, rarity):
        self.rarity = rarity

    def add_level_stat(self, level, hp, defense):
        self.level_stats.append(LevelStat(level, hp, defense))

    def add_uptie_stat(self, uptie, speed):
        self.uptie_stats.append(UptieStat(uptie, speed))

    def add_resist_info(self, resist_hit, resist_penetrate, resist_slash):
        self.resist_info = ResistInfo(resist_hit, resist_penetrate, resist_slash)

    def add_stagger_threshold(self, first_stagger, second_stagger, third_stagger):
        self.stagger_threshold = StaggerThreshold(first_stagger, second_stagger, third_stagger)

    def add_skill(self, skill_code, skill_name, coin_count, init_level, init_remain, atk_weight):
        self.skillset.add_skill(skill_code, Skill(skill_code, skill_name, coin_count, init_level, init_remain, atk_weight))
        
    def add_skill_uptie_info(self, skill_name, uptie_level, skill_power, coin_power, effect_by_condition):
        self.skillset.add_skill_uptie_info(skill_name, uptie_level, skill_power, coin_power, effect_by_condition)

    def add_influence_behavior(self, behavior_type, behavior_name, condition, description):
        if behavior_type == "passive":
            self.passive_infos.append(Passive(behavior_type, behavior_name, condition, description))
        elif behavior_type == "support passive":
            self.support_passive_infos.append(SupportPassive(behavior_type, behavior_name, condition, description))
        elif behavior_type == "panic type":
            self.panic_type.append(PanicType(behavior_type, behavior_name, condition, description))
        
    def update_details(self, page):
        # 세부 정보 업데이트 로직 (가정)
        pass
    #def init_uptie(self, page):
    #    init_uptie = 

# LevelStat 객체 정의
class LevelStat:
    def __init__(self, level, hp=None, defense=None):
        self.level = level
        self.hp = hp
        self.defense = defense
    def update_level(self, level):
        self.level = level
    def update_hp(self, hp):
        self.hp = hp
    def update_defense(self, defense):
        self.defense = defense

# UptieStat 객체 정의
class UptieStat:
    def __init__(self, uptie, speed=None):
        self.uptie = uptie
        self.speed = speed
    def update_speed(self, speed):
        self.speed = speed

# ResistInfo 객체 정의
class ResistInfo:
    def __init__(self, resist_hit=None, resist_penetrate=None, resist_slash=None):
        self.resist_hit = resist_hit
        self.resist_penetrate = resist_penetrate
        self.resist_slash = resist_slash

# ResistInfo 객체 정의
class StaggerThreshold:
    def __init__(self, first_stagger=None, second_stagger=None, third_stagger=None):
        self.first_stagger = first_stagger
        self.second_stagger = second_stagger
        self.third_stagger = third_stagger

class SkillSet:
    def __init__(self):

        self.skill_1_type = ""
        self.skill_2_type = ""
        self.skill_3_type = ""
        self.defense_skill_type = ""

        self.skill_1_sin_affinity = ""
        self.skill_2_sin_affinity = ""
        self.skill_3_sin_affinity = ""
        self.defense_skill_sin_affinity = ""

        self.skill_1_details = []
        self.skill_2_details = []
        self.skill_3_details = []
        self.defense_skill_details = []

    def set_skill_type(self, skill_code, skill_type, sin_affinity):
        if skill_code == "skill 1":
            self.skill_1_type = skill_type
            self.skill_1_sin_affinity = sin_affinity
        elif skill_code == "skill 2":
            self.skill_2_type = skill_type
            self.skill_2_sin_affinity = sin_affinity
        elif skill_code == "skill 3":
            self.skill_3_type = skill_type
            self.skill_3_sin_affinity = sin_affinity
        elif skill_code == "defense":
            self.defense_skill_type = skill_type
            self.defense_skill_sin_affinity = sin_affinity
    
    def add_skill(self, skill_code, skill):
        if skill_code == "skill 1":
            self.skill_1_details.append(skill)
            skill.skill_type = self.skill_1_type
            skill.sin_affinity = self.skill_1_sin_affinity
        elif skill_code == "skill 2":
            self.skill_2_details.append(skill)
            skill.skill_type = self.skill_2_type
            skill.sin_affinity = self.skill_2_sin_affinity
        elif skill_code == "skill 3":
            self.skill_3_details.append(skill)
            skill.skill_type = self.skill_3_type
            skill.sin_affinity = self.skill_3_sin_affinity
        elif skill_code == "defense":
            self.defense_skill_details.append(skill)
            skill.skill_type = self.defense_skill_type 
            skill.sin_affinity = self.defense_skill_sin_affinity

    def get_skill_by_name(self, skill_name):
        for skill_list in [self.skill_1_details, self.skill_2_details, self.skill_3_details, self.defense_skill_details]:
            for skill in skill_list:
                result = skill.is_skill_by_name(skill_name)
                if result:
                    return result
        return None

    def add_skill_uptie_info(self, skill_name, uptie_level, skill_power, coin_power, effect_by_condition):
        skill = self.get_skill_by_name(skill_name)
        skill.add_uptie_info(uptie_level, skill_power, coin_power, effect_by_condition)
        

class Skill:
    def __init__(self, skill_code, skill_name, coin_count, init_level, init_remain, atk_weight):
        self.skill_code = skill_code # "Skill 1"/"Skill 2"/"Skill 3"/"Defense" '''o'''
        self.skill_type = None
        self.sin_affinity = None
        self.skill_name = skill_name # 스킬 이름 '''o'''
        self.coin_count = coin_count # 동전 개수 '''o'''
        self.init_level = init_level # 스킬이 초기 배정받은 레벨, 인격 레벨에 - 1 한 값을 더해주면 스킬의 현 level을 구할 수 있다. '''o'''
        self.init_remain = init_remain #'''o'''
        self.atk_weight = atk_weight #공격 가중치 값이다. 방어스킬에는 없기 때문에 None으로 설정했다. '''o'''
        self.remain_count = init_remain # 실제 스킬이 작동하며 횟수가 차감되 나중에 모두 차감되면 다음 턴에 다시 초기횟수 리필된다.
        self.uptie_info = []  

    def is_skill_by_name(self, skill_name):
        if self.skill_name == skill_name:
            return self
        return None
        
    def add_uptie_info(self, uptie_level, skill_power, coin_power, effects_by_condition):
        self.uptie_info.append(UptieInfo(uptie_level, skill_power, coin_power, effects_by_condition))

class UptieInfo:
    def __init__(self, uptie_level, skill_power, coin_power, effects_by_condition):
        self.uptie_level = uptie_level   #'''o'''
        self.skill_power = skill_power   #'''o'''
        self.coin_power = coin_power     #'''o'''
        self.skill_effect = effects_by_condition

# 추상화된 EffectType 클래스
class EffectType:
    def __init__(self, behavior_type, name, description):
        self.behavior_type = behavior_type
        self.name = name  # Effect 이름
        self.description = description  # Effect 설명

# Passive 클래스
class Passive(EffectType):
    def __init__(self, behavior_type, name, condition, description):
        super().__init__(behavior_type, name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)  # condition이 있는지 여부 확인

# SupportPassive 클래스
class SupportPassive(EffectType):
    def __init__(self,behavior_type, name, condition, description):
        super().__init__(behavior_type, name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)

# PanicType 클래스
class PanicType(EffectType):
    def __init__(self, behavior_type, name, condition, description):
        super().__init__(behavior_type, name, description)
        self.condition = condition  # 조건이 있을 경우에만 초기화
        self.has_condition = bool(condition)

# Keyword 클래스
class Keyword:
    def __init__(self, keyword, explain=None, dispellable=None):
        self.keyword = keyword
        self.explain = explain
        self.dispellable = dispellable

      
class DataContainer:
    """크롤링 데이터를 관리하는 클래스"""
    def __init__(self):
        self.identities = []
        self.keywords = []

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.level_max

    def get_max_level(self, identity_id):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                return identity.uptie_max

    def update_identity_name(self, identity_id, identity_name):
        """특정 ID의 이름을 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_name(identity_name)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_identity_rarity(self, identity_id, rarity):
        """특정 ID의 희귀도를 업데이트."""
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.update_rarity(rarity)
                return
        print(f"Identity {identity_id}를 찾을 수 없습니다.")

    def update_skill(self, identity_id, skill_number, new_type):
        """
        특정 identity의 skill_type을 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_type: 새 skill_type
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_type = new_type
                elif skill_number == 2:
                    identity.skill_2_type = new_type
                elif skill_number == 3:
                    identity.skill_3_type = new_type
                elif skill_number == "defense":
                    identity.defense_type = new_type
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def update_sin_affinity(self, identity_id, skill_number, new_affinity):
        """
        특정 identity의 sin_affinity를 변경하는 함수.
        :param identity_id: 변경할 Identity의 ID
        :param skill_number: 변경할 스킬 번호 (1, 2, 3 또는 'defense')
        :param new_affinity: 새 sin_affinity
        """
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_number == 1:
                    identity.skill_1_sin_affinity = new_affinity
                elif skill_number == 2:
                    identity.skill_2_sin_affinity = new_affinity
                elif skill_number == 3:
                    identity.skill_3_sin_affinity = new_affinity
                elif skill_number == "defense":
                    identity.defense_sin_affinity = new_affinity
                else:
                    print("Invalid skill number. Use 1, 2, 3, or 'defense'.")
                return
        print("Identity with the given ID not found.")

    def add_identity(self, identity):
        """새로운 인격 데이터를 추가."""
        self.identities.append(identity)

    def add_level_stat(self, identity_id, level, hp, defense):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_level_stat(level, hp, defense)

    def add_uptie_stat(self, identity_id, uptie, speed):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_uptie_stat(uptie, speed)

    def add_resist_info(self, identity_id, resist_hit, resist_penetrate, resist_slash):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_resist_info(resist_hit, resist_penetrate, resist_slash)

    def add_stagger_threshold(self, identity_id, first_stagger, second_stagger, third_stagger):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_stagger_threshold(first_stagger, second_stagger, third_stagger)

    
    def add_skill(self, identity_id, skill_code, skill_name, coin_count, init_level, init_remain, atk_weight):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                if skill_code == "skill 1":
                    identity.skillset.set_skill_type(skill_code, identity.skill_1_type, identity.skill_1_sin_affinity)
                elif skill_code == "skill 2":
                    identity.skillset.set_skill_type(skill_code, identity.skill_2_type, identity.skill_2_sin_affinity)
                elif skill_code == "skill 3":
                    identity.skillset.set_skill_type(skill_code, identity.skill_3_type, identity.skill_3_sin_affinity)
                elif skill_code == "defense":
                    identity.skillset.set_skill_type(skill_code, identity.defense_type, identity.defense_sin_affinity)
                identity.add_skill(skill_code, skill_name, coin_count, init_level, init_remain, atk_weight)

    def add_skill_uptie_info(self, identity_id, skill_name, uptie_level, skill_power, coin_power, effects_by_condition):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_skill_uptie_info(skill_name, uptie_level, skill_power, coin_power, effects_by_condition)

    def add_influence_behavior(self, identity_id, behavior_type, behavior_name, condition, description):
        for identity in self.identities:
            if identity.identity_id == identity_id:
                identity.add_influence_behavior(behavior_type, behavior_name, condition, description)

    def add_keyword(self, keyword, explain, dispellable):
        self.keywords.append(Keyword(keyword, explain, dispellable))
        
    def display_identities(self):
        """Identity, ID, Name, Rarity 정보를 출력"""
        if not self.identities:
            print("No identities found.")
        else:
            # 데이터를 리스트로 변환
            table = []
            for idx, identity in enumerate(self.identities, 1):
                table.append([
                    idx, identity.identity_id, identity.name, identity.rarity,
                    identity.skill_1_type, identity.skill_1_sin_affinity,
                    identity.skill_2_type, identity.skill_2_sin_affinity,
                    identity.skill_3_type, identity.skill_3_sin_affinity,
                    identity.defense_type, identity.defense_sin_affinity
                ])
            
            # 테이블 헤더 정의
            headers = [
                "Index", "ID", "Name", "Rarity",
                "Skill_1 Type", "Skill_1 Sin", "Skill_2 Type", "Skill_2 Sin",
                "Skill_3 Type", "Skill_3 Sin", "Defense Type", "Defense Sin"
            ]
            
            # 테이블 출력
            print(tabulate(table, headers=headers, tablefmt="fancy_grid"))

    def export_to_csv(self, filename, fieldnames, new_data):
        """
        기존 데이터를 유지하고, 새로운 데이터를 추가하여 CSV 파일에 저장하는 메서드.
        :param filename: 내보낼 파일 이름 (예: 'identity.csv', 'skill.csv')
        :param fieldnames: CSV의 헤더 리스트
        :param new_data: CSV로 추가할 새로운 데이터 (리스트 형태)
        """
        try:
            # 'csv' 폴더가 없으면 생성
            if not os.path.exists('./csv'):
                os.makedirs('./csv')
            
            # 파일 경로 설정
            file_path = os.path.join('./csv', filename)
    
            # 기존 데이터를 읽기
            existing_data = []
            if os.path.exists(file_path):
                with open(file_path, mode='r', newline='', encoding='utf-8') as file:
                    reader = csv.DictReader(file)
                    existing_data = [row for row in reader]
    
            # 중복 제거를 위해 기존 데이터와 새로운 데이터를 병합
            existing_keywords = {row[fieldnames[0]] for row in existing_data}  # 첫 번째 필드를 기준으로 중복 체크
            combined_data = existing_data + [data for data in new_data if data[fieldnames[0]] not in existing_keywords]
    
            # 병합된 데이터를 다시 CSV 파일로 저장
            with open(file_path, mode='w', newline='', encoding='utf-8') as file:
                writer = csv.DictWriter(file, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(combined_data)
    
            print(f"Data successfully exported to {filename}")
        except Exception as e:
            print(f"Error exporting to CSV: {e}")

    def get_identity_data(self):
        """
        Identity 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "ID": identity.identity_id,
                "Name": identity.name,
                "Rarity": identity.rarity,
                "Skill_1_Type": identity.skill_1_type,
                "Skill_1_Sin_Affinity": identity.skill_1_sin_affinity,
                "Skill_2_Type": identity.skill_2_type,
                "Skill_2_Sin_Affinity": identity.skill_2_sin_affinity,
                "Skill_3_Type": identity.skill_3_type,
                "Skill_3_Sin_Affinity": identity.skill_3_sin_affinity,
                "Defense_Type": identity.defense_type,
                "Defense_Sin_Affinity": identity.defense_sin_affinity,
            }
            for identity in self.identities
        ]
        
    def export_identities_data(self, filename="Identities.csv"):
        """Identity 데이터를 CSV로 내보내는 메서드"""
        identity_fieldnames = [
            "ID", "Name", "Rarity",
            "Skill_1_Type", "Skill_1_Sin_Affinity",
            "Skill_2_Type", "Skill_2_Sin_Affinity",
            "Skill_3_Type", "Skill_3_Sin_Affinity",
            "Defense_Type", "Defense_Sin_Affinity"
        ]
        identity_data = self.get_identity_data()  # Identity 데이터 가져오기
        self.export_to_csv(filename, identity_fieldnames, identity_data)  # CSV로 내보내기

    def load_identities_by_id(self, filename, identity_ids):
        """지정된 identity_ids 목록에 해당하는 Identity 데이터를 CSV에서 읽어옴."""
        identity_ids_set = set(identity_ids) # 중복 제거를 위 Set 사용
        with open(filename, mode='r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                if row["ID"] in identity_ids_set:
                    identity = Identity(
                        identity_id=row["ID"],
                        name=row["Name"],
                        rarity=row["Rarity"],
                        skill_1_type=row["Skill_1_Type"],
                        skill_1_sin_affinity=row["Skill_1_Sin_Affinity"],
                        skill_2_type=row["Skill_2_Type"],
                        skill_2_sin_affinity=row["Skill_2_Sin_Affinity"],
                        skill_3_type=row["Skill_3_Type"],
                        skill_3_sin_affinity=row["Skill_3_Sin_Affinity"],
                        defense_type=row["Defense_Type"],
                        defense_sin_affinity=row["Defense_Sin_Affinity"]
                    )
                    self.identities.append(identity)

    def load_all_identities(self, file_path):
        """
        주어진 CSV 파일에서 모든 Identity 정보를 읽어옴.
        :param file_path: CSV 파일 경로
        """
        try:
            with open(file_path, mode='r', encoding='utf-8') as file:
                reader = csv.DictReader(file)
                for row in reader:
                    try:
                        identity = Identity(
                            identity_id=row["ID"].strip(),  # ID는 필수
                            name=row["Name"].strip(),
                            rarity=row["Rarity"].strip(),
                            skill_1_type=row.get("Skill_1_Type", "").strip(),
                            skill_1_sin_affinity=row.get("Skill_1_Sin_Affinity", "").strip(),
                            skill_2_type=row.get("Skill_2_Type", "").strip(),
                            skill_2_sin_affinity=row.get("Skill_2_Sin_Affinity", "").strip(),
                            skill_3_type=row.get("Skill_3_Type", "").strip(),
                            skill_3_sin_affinity=row.get("Skill_3_Sin_Affinity", "").strip(),
                            defense_type=row.get("Defense_Type", "").strip(),
                            defense_sin_affinity=row.get("Defense_Sin_Affinity", "").strip(),
                        )
                        self.identities.append(identity)
                    except KeyError as e:
                        print(f"Missing column in row: {e}. Skipping this row.")
                    except Exception as e:
                        print(f"Error processing row {row}: {e}. Skipping this row.")
            print(f"Successfully loaded {len(self.identities)} identities.")
        except FileNotFoundError:
            print(f"File not found: {file_path}")
        except Exception as e:
            print(f"Error loading identities: {e}")


    def get_level_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Level": level_stat.level,
                "HP": level_stat.hp,
                "Defense": level_stat.defense,
            }
            for identity in self.identities
            for level_stat in identity.level_stats
        ]

    def export_level_stats_data(self, filename="Levelstats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        level_stats_fieldnames = ["Identity_ID", "Level", "HP", "Defense"]
        level_stats_data = self.get_level_stats_data()  # level_stats 데이터 가져오기
        self.export_to_csv(filename, level_stats_fieldnames, level_stats_data)  # CSV로 내보내기

    def get_uptie_stats_data(self):
        """
        모든 Identity의 level_stats 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Uptie": uptie_stat.uptie,
                "Speed": uptie_stat.speed,
            }
            for identity in self.identities
            for uptie_stat in identity.uptie_stats
        ]

    def export_uptie_stats_data(self, filename="Uptiestats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        uptie_stats_fieldnames = ["Identity_ID", "Uptie", "Speed"]
        uptie_stats_data = self.get_uptie_stats_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, uptie_stats_fieldnames, uptie_stats_data)  # CSV로 내보내기

    def get_resist_info_data(self):
        """
        모든 Identity의 resist_info 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "Resist_Hit": identity.resist_info.resist_hit,
                "Resist_Penetrate": identity.resist_info.resist_penetrate,
                "Resist_Slash": identity.resist_info.resist_slash,
            }
            for identity in self.identities
        ]

    def export_resist_info_data(self, filename="ResistInfos.csv"):
        """ResistInfo 데이터를 CSV로 내보내는 메서드"""
        resist_infos_fieldnames = ["Identity_ID", "Resist_Hit", "Resist_Penetrate", "Resist_Slash"]
        resist_infos_data = self.get_resist_info_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, resist_infos_fieldnames, resist_infos_data)  # CSV로 내보내기

    def export_uptie_stats_data(self, filename="Uptiestats.csv"):
        """LevelStat 데이터를 CSV로 내보내는 메서드"""
        uptie_stats_fieldnames = ["Identity_ID", "Uptie", "Speed"]
        uptie_stats_data = self.get_uptie_stats_data()  # uptie_stats 데이터 가져오기
        self.export_to_csv(filename, uptie_stats_fieldnames, uptie_stats_data)  # CSV로 내보내기

    def get_stagger_threshold_data(self):
        """
        모든 Identity의 stagger_threshold 데이터를 export_to_csv용 포맷으로 변환
        """
        return [
            {
                "Identity_ID": identity.identity_id,
                "First_Stagger": identity.stagger_threshold.first_stagger,
                "Second_Stagger": identity.stagger_threshold.second_stagger,
                "Third_Stagger": identity.stagger_threshold.third_stagger,
            }
            for identity in self.identities
        ]

    def export_stagger_threshold_data(self, filename="StaggerThresholds.csv"):
        """StaggerThresholds 데이터를 CSV로 내보내는 메서드"""
        stagger_thresholds_fieldnames = ["Identity_ID", "First_Stagger", "Second_Stagger", "Third_Stagger"]
        stagger_thresholds_data = self.get_stagger_threshold_data()  # stagger_thresholds 데이터 가져오기
        self.export_to_csv(filename, stagger_thresholds_fieldnames, stagger_thresholds_data)  # CSV로 내보내기

    def get_skillset_data(self):
        """모든 Identity의 skillset 데이터를 export_to_csv을 포맷으로 변환"""
        return [
            {
                "Identity_ID": identity.identity_id,
                "Skill_Code": skill.skill_code,
                "Skill_Type": skill.skill_type,
                "Sin_Affinity": skill.sin_affinity,
                "Skill_Name": skill.skill_name,
                "Coin_Count": skill.coin_count,
                "Init_Level": skill.init_level,
                "Init_Remain": skill.init_remain,
                "Atk_Weight": skill.atk_weight,
                "Uptie_Level": uptie_info.uptie_level,
                "Skill_Power": uptie_info.skill_power,
                "Coin_Power": uptie_info.coin_power,
                "Effect_Data": json.dumps(uptie_info.skill_effect)
            }
            for identity in self.identities
            for skill_list in [identity.skillset.skill_1_details, identity.skillset.skill_2_details, identity.skillset.skill_3_details, identity.skillset.defense_skill_details]
            for skill in skill_list
            for uptie_info in skill.uptie_info
        ]

    def export_identity_skillset_data(self, filename="IdentitiesSkills.csv"):
        """IdentitiesSkills 데이터를 CSV로 내보내는 메서드"""
        identities_skills_fieldnames = ["Identity_ID", "Skill_Code", "Skill_Type", "Sin_Affinity", "Skill_Name", "Coin_Count", "Init_Level", "Init_Remain", "Atk_Weight", "Uptie_Level", "Skill_Power", "Coin_Power", "Effect_Data"]
        identities_skills_data = self.get_skillset_data()
        self.export_to_csv(filename, identities_skills_fieldnames, identities_skills_data)

    def get_identities_behaviors_data(self):
        """모든 Identity의 influence behavior 데이터를 export_to_csv 포맷으로 변환"""
        return [
            {
                "Identity_ID": identity.identity_id,
                "Effect_Type": effect_type,
                "Behavior_Name": effect.name,
                "Behavior_Condition": effect.condition if effect.condition else np.nan,
                "Behavior_Description" : effect.description
            }
            for identity in self.identities
            for effect_list, effect_type in [(identity.passive_infos, 'Passive'), (identity.support_passive_infos, 'Support Passive'), (identity.panic_type, 'Panic Type')]
            for effect in effect_list
        ]

    def export_identities_behaviors_data(self, filename="IdentitiesBehaviors.csv"):
        """IdentitiesBehaviors 데이터를 CSV로 내보내는 메서드"""
        identities_behaviors_fieldnames = ["Identity_ID","Effect_Type","Behavior_Name","Behavior_Condition","Behavior_Description"]
        identities_behaviors_data = self.get_identities_behaviors_data()
        self.export_to_csv(filename, identities_behaviors_fieldnames, identities_behaviors_data)

    def get_keywords_data(self):
        """DataContainer의 keywords 데이터를 export_to_csv 포맷으로 변환"""
        return [
            {
                "Keyword": keyword.keyword,
                "Explain": keyword.explain,
                "Dispellable": keyword.dispellable,
            }
            for keyword in self.keywords
        ]

    def export_keywords_data(self, filename="Keywords.csv"):
        """DataContainer의 keywords 데이터를 CSV로 내보내는 메서드"""
        keywords_fieldnames = ["Keyword","Explain","Dispellable"]
        keywords_data = self.get_keywords_data()
        self.export_to_csv(filename, keywords_fieldnames, keywords_data)

            
async def main():
    data_container = DataContainer()
    scraper = Scraper(container=data_container, base_url="https://limbus.kusoge.xyz", img_url="https://img.kusoge.xyz/limbus/img")
    await scraper.scrape()

await main() 